# Speech Analysis of French Candidates for the Presidential Campaign 2017

## Description of the project

In this project, speeches of the French candidates are analyzed with the intention of discovering if their opinions are similar. To do so, speeches are broken down to sentences, they are represented using word2vec and then a Convolutional Neural Network is trained. In the last layer of the neural network, each neuron is associated to one candidate. When one sentence is passed as input, each neuron of the last layer will have a value that will represent how much that sentence is related to the candidate. The values of the last layer neurons add up to 1, so it can be interpreted as a percentage.

This project is based on the following paper: https://arxiv.org/pdf/1408.5882.pdf

In [130]:
from gensim.models import word2vec
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import nltk.data
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import numpy as np
import pprint
from keras.datasets import reuters
from keras.models import Sequential, Model
from keras import optimizers
from keras.layers import Dense, Dropout, Flatten, Reshape, Input, MaxPooling1D, Convolution1D, Embedding
from keras.utils import np_utils
from keras.layers.merge import Concatenate
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import nltk.data
from w2v import train_word2vec #w2v.py

The whole corpus is saved into a list of String, and also different lists are created with the speeches of each candidate divided in training and testing.

In [8]:
corpus = [
    "Mesdames messieurs , mes amis , La primaire de la droite et du centre est close et les électeurs doivent être salués pour s' être emparés massivement et librement de ce rendez-vous démocratique . La victoire me revient ; c' est une victoire de fond bâtie sur des convictions . Depuis 3 ans , je trace ma route , à l' écoute des Français , avec mon projet , avec mes valeurs , et progressivement , j' ai senti cette vague qui a brisé tous les scénarios écrits d' avance . Ma démarche a été comprise : la France ne supporte pas son décrochage . Elle veut la vérité et elle veut des actes . Ce quinquennat qui s' achève a été pathétique . Il va falloir y mettre un terme , et repartir de l' avant comme jamais nous ne l' avons fait depuis 30 ans . Pour cela , j' aurai besoin de tout le monde . Ce soir , j' ai une pensée particulière pour Nicolas Sarkozy J' adresse à Alain Juppé un message d' amitié , d' estime , de respect . Alain est un homme d' Etat et il le reste . Aucun candidat n' a démérité . Cette primaire fut digne . Pas un électeur ne doit se sentir humilié ou mis à l' écart . Ce qui nous unit est plus important que ce qui nous distingue . Voilà pourquoi , je tends la main à tous ceux qui veulent servir notre pays . Mon succès , je le mets en partage . Je le dois à mes équipes , à mes soutiens , à tous les bénévoles qui furent admirables de dévouement . Ce succès , je le dois à ces milliers de Français que j' ai rencontrés et qui m' ont nourri de leur révolte , de leurs idées , de leurs espoirs . Je me suis senti porté par leur désir de liberté et de réussite . Il y a dans notre pays un immense besoin de respect et de fierté . Il y a aussi un appel à l' autorité de l' Etat et à l' exemplarité de ceux qui le dirigent . Les électeurs de la droite et du centre ont trouvé dans ma démarche les valeurs françaises auxquelles ils sont attachés . Ces valeurs , je les défendrai et nous les partagerons avec tous ceux qui , dans leurs différences , aiment la France . Personne ne devra se sentir exclu d' une société que je veux plus juste et plus solidaire . Notre renaissance sera l' œuvre de tous . Elle viendra de notre volonté de nous battre ensemble pour le bien commun et pour que nos enfants soient heureux d' être Français . Je sais les défis , mais je suis porté par la confiance que fait naître en moi cette incroyable aventure d' un pays qui par son travail et sa bravoure a réussi à se placer parmi les cinq plus grandes puissances du monde . L' avenir nous attend , et nous avons en mains tous les atouts pour être une Nation souveraine et moderne , en tête de l' Europe . Dès demain , l' essentiel commence ! J' ai maintenant le devoir de convaincre tout un pays que notre projet est le seul qui puisse nous hisser vers le haut , pour l' emploi , pour la croissance , pour battre ces fanatiques qui nous ont déclaré la guerre . J' ai le devoir de redonner confiance aux Françaises et aux Français . J' en suis certain , ils sont des millions prêts à s' élancer . Je ne vois dans leurs yeux , ni lâcheté , ni résignation , mais une force qui attend son heure , et cette force sera extraordinaire si nous la guidons vers des changements inédits . J' ai le devoir de vaincre l' immobilisme et la démagogie . La gauche , c' est l' échec ; l' extrême droite , c' est la faillite . Je parle de vaincre ces partis politiques , pas de vaincre ces Français déçus , ces électeurs qui ne sont pas aujourd'hui des nôtres mais qu' il me revient aussi d' entrainer vers l' avenir . François Hollande a abaissé la fonction présidentielle : il faudra la restaurer . Si le peuple français me fait confiance , j' aurai le devoir de respecter le contrat noué avec lui et de tenir ma charge avec dignité . Je mesure la gravité de la situation et les attentes des mes compatriotes . Je vais aller à leur rencontre tout au long des prochains mois . Je relèverai avec eux un défi original en France : celui de la vérité et celui d' un changement de logiciel complet . Je donne rendez-vous à tous ceux qui savent que le bonheur est une conquête . Je donne rendez-vous à tous ceux qui ont dans le cœur la fierté d' être français . Mon ambition est de hisser la nation vers le meilleur d' elle-même , et je vous le certifie , si en 2017 nous prenons fermement les choses en main , notre pays ira loin car rien n' arrête un peuple qui se dresse pour son avenir . Avec ces primaires , une espérance est née . Ma mission est de la faire grandir pour la France et pour la République . ",
    "Je suis engagé dans le débat présidentiel avec un projet puissant , fondé sur la vérité et un effort responsable pour redresser notre pays . Depuis deux semaines , mes propositions font l' objet d' attaques de la gauche et de l' extrême droite . Elles ne m' impressionnent pas . Ce tir de barrage était prévisible puisque j' engage le fer contre le cortège des conservateurs et des mystificateurs . Mes adversaires s' érigent en défenseurs de notre &laquo; &raquo; mais ils en sont , en réalité , ses fossoyeurs . Ils refusent de voir que ce modèle craque de toutes parts , que le chômage , la pauvreté et les injustices se propagent , que les déficits menacent la solidarité nationale . Les critiques se sont récemment concentrées sur la santé . Quel est le diagnostic ? Notre système de soins est menacé et la situation s' est dégradée ces dernières années . Les Français sont inquiets , à juste titre : des heures d' attente éprouvantes aux urgences , plusieurs mois pour obtenir un rendez-vous chez un spécialiste , des déserts médicaux qui s' étendent . Les professionnels de santé sont exaspérés par la bureaucratisation de leur métier . Le gouvernement a opposé l' exercice libéral et le système public au lieu de les faire fonctionner de façon complémentaire . A l' hôpital , les 35 heures ont fait des dégâts . Le trou de la sécu n' est pas comblé contrairement à ce que prétend la ministre . La dette se monte à 160 milliards d' euros … Bref , au lieu de regarder les faits , mes détracteurs me soupçonnent de vouloir &laquo; privatiser &raquo; l' assurance maladie et diminuer les remboursements . C' est évidemment faux ! Au passage , la gauche préfère oublier que le poids des mutuelles et des assurances personnelles n' a cessé d' augmenter depuis des décennies , y compris dans le cadre des réformes qu' elle a elle-même votées . Les modalités de la &laquo; des complémentaires santé aux seuls salariés ont conduit , dans de nombreux cas , à diminuer les garanties offertes ( frais d' optique , soins dentaires , consultations de spécialistes et opérations chirurgicales moins bien remboursées ) et à une augmentation des cotisations . Les assurés ne savent plus ce à quoi ils ont droit et s' interrogent sur les garanties effectives dont ils disposent . Opacité et manque de transparence sont devenus la règle . Mais de tout cela , la gauche , main sur le cœur , ne veut pas entendre parler . Moi , je dis les choses telles qu' elles sont ! Oui , la situation de notre système de santé est inquiétante et ceux qui prétendent le contraire sont dans le déni et la démagogie . Mon objectif est de sauver notre système de santé qui fut l' un des meilleurs au monde et qui doit le redevenir . J' entends réaffirmer le principe d' universalité dans l' esprit des ordonnances de 1945 . L' Assurance maladie obligatoire et universelle , pilier de la solidarité , doit rester le pivot dans le parcours de soin dont le médecin généraliste est l' acteur clé . Elle continuera à couvrir les soins comme aujourd'hui et même , mieux rembourser des soins qui sont largement à la charge des assurés , comme les soins optiques et dentaires . Il n' est donc pas question de toucher à l' Assurance maladie et encore moins de la privatiser . Je souhaite assurer le meilleur remboursement pour les patients en plaçant l' Assurance maladie obligatoire et les organismes complémentaires sous le pilotage d' une agence de régulation et de contrôle dans laquelle seront représentés tous les acteurs du système de santé ainsi que les usagers . Il s' agit d' améliorer la gestion de notre système de protection sociale . J' entends faire en sorte que tous les Français puissent bénéficier d' une protection complémentaire appropriée sur la base de contrats homogènes . Il convient , par ailleurs , de responsabiliser chacun en simplifiant les nombreux dispositifs de franchise actuels et le ticket modérateur qui sont un maquis incompréhensible . Cette responsabilisation passe notamment par l' abandon de la généralisation du tiers payant . Assez de bureaucratie et assez de mensonge : la santé a un coût et au bout du compte , d' une manière ou d' une autre , c' est toujours l' assuré qui paye . Je veux réaffirmer l' équilibre de notre système de santé : médecine publique et médecine libérale doivent agir de concert , dans un climat de confiance . Le patient doit pouvoir choisir librement son établissement de soins , son professionnel de santé , ce dernier devant rester maitre de ses prescriptions . J' entends garantir un égal accès aux soins pour tous en enrayant la désertification médicale , grâce à la multiplication des maisons médicales , grâce à la revalorisation de la médecine libérale , actuellement découragée par la paperasserie et les charges , grâce aussi à la possibilité donnée aux établissements d' hospitalisation privée de participer aux missions de service public , grâce à une refonte des études médicales , grâce enfin à la télémédecine . Il faut assurer l' équilibre financier . N' en déplaise aux charlatans du &laquo; plus à n' importe quel &raquo; , on ne peut pas continuer de financer notre assurance maladie à crédit et d' en faire porter le poids aux générations futures , d'autant que les dépenses de santé vont continuer de croître . Alors , soyons courageux : sauvons notre système de santé pour garantir à tous les Français l' accès à l' innovation thérapeutique et aux nouveaux médicaments . Ces réformes sont nécessaires car sans elles notre système de protection sociale s' affaissera et les premières victimes seront nos concitoyens les plus démunis . Ces réformes se feront dans la concertation avec tous les acteurs . Je l' ai dit et je le répète , ma priorité , mon urgence , c' est la bataille pour l' emploi et la croissance . Si mes compatriotes m' accordent leur confiance , c' est sur cette bataille que je concentrerai le début de mon quinquennat . Près de 6 millions de Français au chômage ou n' ayant qu' une activité réduite , des centaines de milliers de jeunes hors de tout emploi ou formation … Une bombe sociale est sous nos pieds , et malgré cela la gauche me reproche de vouloir faire ce qui a été fait dans les pays voisins qui connaissent le plein emploi ! &laquo; , hurle -t-elle . Vieille et risible ficelle pour disqualifier tous ceux qui n' ont pas ses œillères . Je suis simplement pragmatique et je crois que la liberté est le meilleur tremplin des sociétés vivantes . J' ai la conviction que nous pouvons ramener la France au plein emploi . Pour cela , nous devons diminuer le coût du travail , simplifier et alléger notre droit du travail , mettre le paquet sur l' alternance , sortir des 35 heures et laisser aux acteurs de terrain le pouvoir de négocier leur organisation , lever la peur de l' embauche qui dissuade les PME de recruter … Ceux qui m' accusent à tort de vouloir casser le droit du travail ont cassé le travail tout court ! Je veux le restaurer , le valoriser , l' ouvrir à tous . Pour mes adversaires , l' emploi c' est encore et toujours l' affaire de l' Etat . Nous sommes l' un des pays au monde les plus réglementés , l' un des plus imposés de l' OCDE et avec 5,5 millions d' agents publics nous sommes l' état d' Europe le plus administré . Mais , pour la gauche et l' extrême droite , cette réalité importe peu . Ils préfèrent crier haro sur ceux qui , avec moi , veulent réduire ce chiffre de 8 % sur 5 ans pour réduire nos déficits et alléger les contraintes qui pèsent sur les initiatives . Oui , deux camps s' opposent : le camp de ceux qui pensent que pour redresser la France il faut encore et toujours verrouiller le système ; et celui , dont je suis avec des millions de Français , qui affirme qu' il faut le déverrouiller . Mes détracteurs croient que notre peuple ne peut pas supporter la vérité . Je crois le contraire ! Ils pensent qu' on peut relancer notre pays sans changements radicaux . Ils mentent ! Les attaques ne me feront pas dévier de ma route . Elles sont la rançon d' un projet exigeant , guidé par l' intérêt national . Avec les Français , j' aurai le courage de la vérité et de l' action . ",
    "Mes chers compatriotes , Au seuil de cette nouvelle année , je souhaitais m' adresser directement à vous . Vous le savez , l' année 2017 s' inscrit dans un moment particulier pour notre pays : malgré ses atouts , malgré ses forces qui sont celles d' une nation courageuse , d' un peuple fier , la France n' est pas épargnée par les difficultés . Ces difficultés vous les connaissez . C' est le drame du chômage et de la précarité . C' est la pauvreté qui augmente . C' est le pouvoir d' achat qui stagne . C' est aussi le terrorisme qui , à de multiples reprises , a frappé et endeuillé notre pays . Et au moment de refermer l' année 2016 , je pense aux victimes de l' attentat de Nice , je pense au Père Hamel , je pense au couple de policiers de Magnanville , je pense à celles et ceux qui ont été lâchement assassinés à Berlin il y a quelques jours . Nous sommes en guerre . Et dans cette guerre , nous n' avons pas le droit d' être faibles , nous devons être implacables face au totalitarisme islamique . La France a connu bien des épreuves dans son histoire . A chaque fois , elle les a surmontées parce qu' elle a su rester elle-même , en opposant à l' horreur de la barbarie les valeurs de sa civilisation , en étant fidèle à son engagement pour la liberté , la dignité de chacun . Avec vous , avec toutes celles et ceux qui aiment la France , je veux que nous soyons à la hauteur de cet idéal de résistance et d' espérance . Pour cette nouvelle année , je vous adresse tous mes vœux de bonheur , de succès , de santé . Que pour vous et ceux que vous aimez la vie soit au rendez-vous de ses plus belles promesses . Que celles et ceux qui ont toutes les chances en mains les fassent grandir et les mettent en partage . Que celles et ceux qui , l' année dernière , avaient le cœur serré retrouvent de l' espoir . Que ceux qui ont un genou à terre trouvent la force de se relever car en chacun il y a une flamme qui peut se rallumer . Et puis , je formule des vœux pour nous tous , peuple de France et pour notre cher pays que nous aimons tant . Je souhaite que 2017 soit l' année du redressement . Au printemps , vous élirez un nouveau Président de la République . Je suis candidat pour construire avec vous une nation libre et puissante . Le mur des critiques ne nous arrêtera pas . La vérité ne vous fait pas peur , vous êtes courageux , vous avez une énergie exceptionnelle : je veux vous rendre le pouvoir d' entreprendre , de créer et de réaliser vos projets car c' est ainsi que la France retrouvera la croissance et le plein emploi pour que la vie de chacun soit meilleure . Je veux redonner à chaque enfant de France l' égalité des chances pour réussir sa vie . Je veux que les Français vivent dans la sécurité et que chacun d' entre vous puisse être fier d' appartenir à un pays à nul autre pareil . Alors à vous tous , mes chers compatriotes , très bonne année 2017 ! Qu' elle apporte à la France et aux Français un nouvel élan : c' est là mon souhait le plus cher . Vive la République , vive la France . ",
    "Mes amis , mes chers compatriotes , C' est à Nice et c' est avec vous que je repars en campagne . A cette ville que j' aime et à vous tous , j' adresse mes vœux de bonheur et de succès pour cette nouvelle année qui commence . Je suis parmi vous , avec le cœur encore serré par la tragédie du 14 juillet . De ce souvenir monte en moi une volonté inflexible de défendre nos valeurs . Si les Français me font confiance , je prendrai tous les moyens et je solliciterai tous les Etats volontaires , de Washington à Moscou , d' Ankara à Téhéran , pour détruire l' Etat islamique et ses ramifications terroristes . Dans ce combat , j' aurai besoin de nos compatriotes musulmans pour faire le ménage contre ceux qui instrumentalisent leur foi . Nous sommes la France , nous sommes Nice , et jamais nous ne baisserons la tête devant ceux qui veulent abattre notre liberté . C' est nous qui les battrons parce que nos valeurs sont plus belles et plus fortes ! A vos côtés , je puis dire cette phrase simple et belle : &laquo; je suis Français et j' en suis fier &raquo; . Comme vous , je me sens l' héritier d' une Histoire , d' une culture , d' un patrimoine , d' un drapeau , et ce drapeau est suffisamment large pour fédérer tous les Français qui veulent , à nos côtés , servir leur pays . Cela s' appelle le patriotisme , et n' en déplaisent à certains , nous le revendiquons , nous l' assumons et nous irons au bout de notre conviction : la France doit être aimée et impérativement redressée . Durant trois mois nous avons vécu au rythme des primaires et vous avez donné une formidable leçon de démocratie à tous ceux qui doutaient de nous . Après ma victoire , je m' étais fixé trois objectifs . D'abord rassembler toutes les forces et les sensibilités de ma famille politique . C' est fait : après la compétition des primaires , les Républicains font bloc . Je salue fraternellement tous les militants et les élus qui m' ont rejoint sans état d' âme . J' aurai besoin d' eux , j' aurai besoin de leur engagement et j' aurai besoin des Alpes Maritimes ! Mon second objectif était d' unir la droite et le centre autour de ma candidature . C' est fait et je remercie les élus et sympathisants de l' UDI qui , avec un grand sens des responsabilités , ont décidé de se battre à mes côtés . Nous avons quelques différences entre nous , mais ce qui nous uni est tellement plus fort que ce qui nous distingue , et surtout , ce que nous avons à faire pour la France est bien plus important que toutes nos affaires partisanes . Enfin , je devais mettre en place une équipe de campagne ouverte à tous . C' est fait ! Maintenant , mes amis , nous sommes en ordre de bataille et nous allons foncer ensemble . Je laisse les sondages aux sondeurs , les scénarios écrits d' avance aux scénaristes , je ne crois qu' une chose : la victoire récompensera le camp le plus décidé , le plus convaincu . Ma détermination est de fer et je ne doute pas de la vôtre . Vous êtes des militants de la France et pour la France je vous demande une mobilisation et une résolution sans faille . Ne cédez à aucune intimidation , ne marchez jamais à l' ombre , affirmez ce que vous voulez et ce que vous êtes . On vous accuse de vouloir redresser la France de façon radicale . Assumez et ripostez car c' est vous qui avez raison de penser que la situation réclame du courage . On vous accuse de défendre les valeurs de la liberté , de l' autorité , de la famille , du travail . Assumez et ripostez car vous n' avez pas à vous excuser d' avoir du bon sens . Je compte sur vous et vous pouvez compter sur moi pour ne pas changer ce que je crois et ce que je veux . Il y a deux mois , je n' étais pas le candidat de l' establishment et je n' ai pas l' intention de le devenir : je vais repartir sur le terrain , replonger dans le quotidien des Français pour les écouter , les convaincre et les rassembler . Je veux leur faire partager ma volonté d' action , pas la solder en affadissant mes convictions . La démocratie , ça n' est pas des convictions pour les primaires , d' autres convictions pour les présidentielles et pourquoi pas encore des convictions pour les législatives ! Je me bats pour redresser notre pays , pas pour ruser avec lui , et , une fois au pouvoir , pour passer entre les gouttes . Je l' avais dit et je le répète , je relève un pari difficile pour un peuple si longtemps abreuvé d' illusions politiques et de douces promesses : le pari de la vérité et du changement de fond . Alors certes , inéluctablement , presque mécaniquement , la radicalité et la franchise de mon projet m' exposent aux blâmes et aux caricatures de ceux qui sont mous et flous . Les balles sifflent de partout . Visiblement , je gène certains . &laquo; Tous contre Fillon ! &raquo; , &laquo; tous contre la droite &raquo; , tous contre ces &laquo; pauvres &raquo; électeurs de la primaire qui , nous répète -t-on , ne représentent que 4 millions de Français . Mais il ne suffit pas d' être contre encore faut -il avoir quelque chose à dire . Qu' ont -ils à dire et à proposer ceux qui nous mitraillent ? Rien de nouveau mais surtout rien de fort pour répondre à l' urgence d' un pays qui décroche et qui souffre . Devant mes propositions , le système se rebiffe et use des arguments les plus fallacieux . Rien d' anormal , si ce n' est que le débat est bas . Contre le redressement national , mes adversaires ont rédigé leurs tracts : &laquo; sang et larmes &raquo; &laquo; purge &raquo; , &laquo; casse sociale &raquo; , et pourquoi pas l' apocalypse ! La casse sociale , elle est là , sous nos yeux , mais on préfère étouffer la réalité plutôt que de la regarder en face . 6 millions de Français qui pointent à Pole emploi ; 2 millions de jeunes qui ne sont ni à l' Ecole , ni en formation , ni en emploi ; la pauvreté qui augmente faute de croissance ; des quartiers déglingués par la délinquance ; des campagnes abandonnées ; des classes moyennes et des familles qui n' arrivent pas à boucler leurs fins de mois ; un système de santé débordé ; les plus beaux actifs de la France rachetés par l' étranger ; 2000 milliards de dettes pour nos enfants … Tout ça c' est la réalité , tout ça c' est maintenant , tout ça c' est le bilan de François Hollande , bilan qu' il a lui-même jugé tellement catastrophique qu' il a renoncé à se représenter . Oui , tout cela crève les yeux , mais rien n' y fait : 40 ans que la gauche nous rejoue le film de la &laquo; casse sociale &raquo; ; 40 ans qu' elle instrumentalise la peur contre tout changement ; 40 ans qu' elle croit pouvoir traiter les fonctionnaires et les ouvriers comme des clientèles électorales , avec maintenant pour acolyte l' extrême droite qui joue la même partition que l' extrême gauche . Derrière ces postures , la faillite nous guette et la colère monte mais ces gens-là vous disent qu' il ne faut rien changer . Avec les primaires de la gauche , dénommées primaires de la &laquo; Belle alliance populaire &raquo; – prière de ne pas rire ! – on est en plein déni et en plein dans le vieux logiciel socialiste . Comme disait Edgar Faure , &laquo; l' immobilisme est en marche , rien ne l' arrêtera ! &raquo; . C' est la course de lenteur entre celui qui dépensera le plus , celui qui réformera le moins et celui qui promettra le beurre et l' argent du beurre . Evacués Hollande et les résultats calamiteux de son quinquennat , oubliés les déficits , finis les engagements européens , ravalées les déclarations d' amour aux entreprises , aux orties le 49.3 et la Vème République ! Quel que soit le vainqueur de ces primaires , il y aura bien , entre nous , une différence de fond : moi , je veux redresser la France ; à gauche , on veut l' assister dans son déclin , Mr Macron compris . Je dénonce cette capitulation et rien ni personne ne m' empêchera de lui opposer un chemin plus grand et plus droit . Je suis gaulliste . Que voulez -vous , on ne se réinvente pas et , au surplus , je n' ai qu' une patrie et qu' une vie pour la servir ! Je veux incarner la fierté d' une nation qui ne se laisse pas abattre . C' est mon combat , et je viens vers vous , porté par la confiance que fait naître en moi cette incroyable aventure d' un pays qui par sa bravoure , sa culture , son goût du progrès , a réussi à se placer parmi les cinq plus grandes puissances du monde . Nous sommes les héritiers d' une Nation bâtie par des femmes et des hommes qui ont travaillé sans relâche , et tout nous commande de faire aussi bien qu' eux . Cette nation millénaire est montée sur les barricades pour la justice , s' est sacrifiée pour son indépendance afin que le peuple français soit toujours libre de dire &laquo; je suis là et j' exige le respect &raquo; . Exigez ce respect ! Exigez -le de toutes vos forces car le réveil national est d'abord entre vos mains . Depuis trois ans , je sillonne notre pays . &laquo; Laissez -nous travailler , laissez -nous réussir , qu' on nous fiche la paix &raquo; , voilà ce que m' ont dit les milliers de Français que j' ai rencontrés . Notre peuple n' est pas fatigué , c' est le système qui est complètement usé , avec cette incroyable accumulation de règles , de normes , de soupçons qui vampirisent les volontés . La bureaucratie a envahi nos vies . Tout est encadré , nivelé , passé à la moulinette d' une logique pernicieuse : mieux vaut , nous dit -on , la paperasse que l' audace , mieux vaut parait -il le chômage et les 3000 pages du code du travail que l' assouplissement de l' emploi , mieux vaut le diktat syndical que le référendum d' entreprise , mieux vaut un champ en jachère que quelques arpents non déclarés , mieux vaut les écrous administratifs que la responsabilité , mieux vaut -on vous le dit , on vous le répète- une circulaire qu' une initiative … Eh bien non ! Mieux vaut pour moi le risque d' oser que l' uniformité dans la médiocrité . Voilà ce que je crois et voilà ce que m' ont dit tous les salariés et agents publics . C' est ce système que je veux changer en redonnant au pays sa liberté ! Oui , avec vous mes amis , j' assume la première valeur de notre devise républicaine . Et d'ailleurs , si cette valeur de la liberté n' est pas portée par la droite , alors qui la portera ? Surement pas les socialistes qui de tout temps ont préféré l' égalitarisme mou . Et surement pas l' extrême droite , dont le projet économique est un copier-coller de celui de l' extrême gauche . Pour la liberté et pour le plein emploi , je veux concentrer le code du travail sur ses principes fondamentaux et renvoyer le reste au dialogue social . Je donnerai aux entreprises et à leurs salariés le pouvoir de négocier pleinement entre eux les heu- res et les conditions du travail . Moi je fais le pari de la vraie négociation sociale , dans les entreprises , là où entrepreneurs et salariés savent de quoi ils parlent . Et parce que l' Etat doit donner l' exemple d' une nation qui se retrousse les manches , nous rétablirons dans les administrations publiques les 39 heures . Là aussi il y aura des négociations car chacun devra bénéficier des fruits de cet effort , mais cela se fera car je veux une fonction publique performante , au service d' un Etat fort . Par la liberté , je donnerai à ceux qui veulent entreprendre le pouvoir de le faire en renforçant le statut des autoentrepreneurs et en créant celui de travailleur indépendant . Avec le numérique , c' est le début d' une nouvelle économie qui va dynamiser les générations à venir , avec un potentiel d' un million d' emplois nouveaux . Par la liberté , je donnerai aux investisseurs de France le goût d' investir ici , chez nous , pour nos industries , pour nos emplois . Je baisserai massivement les charges et j' allégerai la fiscalité qui pèse sur l' investissement . Je suis un patriote : je préfère supprimer l' ISF que de voir l' économie française sans capitaux , passer sous contrôle étranger . Plutôt que de nous rouler par terre pour attirer les fortunes du Qatar et les fonds de pensions américains , je veux des fortunes françaises qui investissent pour la France . N' en déplaise à ceux qui pensent que le bonheur est de marcher à genoux , c' est la liberté qui est le meilleur carburant des sociétés vivantes ; c' est la liberté qui créera des initiatives et des emplois ; c' est la liberté qui relancera l' ascenseur social aujourd'hui bloqué dans le sous-sol de ce système qui vous explique depuis trop longtemps qu' il vaut mieux l' ombre pour tous que la lumière pour ceux qui veulent en sortir . Les jeunes n' ont pas peur de la liberté . Ils n' ont que rage pour cet immobilisme poussiéreux qui les englue . Les chômeurs n' ont pas peur de la liberté . Ils ont peur de rester coincés au chômage . L' autoentrepreneur , l' artisan , le jeune agriculteur , eux qui s' endettent et se lancent , ils n' ont pas peur de la liberté . Ils la réclament à cor et à cris . Je parle de liberté , oui , et j' en parle pour les sans grade , les sans carnets d' adresse , les oubliés , les déclassés , les humiliés , les courageux , les intrépides , les audacieux , bref , ceux pour qui la liberté est une opportunité de prendre leur destin à pleine main . Et celui qui la prend s' engage pour réussir sa vie et pour prouver qu' il existe en lui une flamme ; et de fil en aiguille , de flamme en flamme , c' est tout un peuple qui se redresse et se met à gagner . En cinq ans , nous pouvons élever notre compétitivité et égaler l' Allemagne , en cinq ans nous pouvons réduire notre chômage en dessous de 7 % , remettre nos finances publiques à l' équilibre et d' ici 10 ans nous pouvons être la 1ère puissance européenne . Alors oui , mes amis , liberté à la Liberté . Mais la liberté sans l' autorité , c' est l' anarchie . Je veux rétablir l' autorité de l' Etat , l' autorité de la loi , la nécessité de l' ordre . Qu' un père de famille puisse gagner dix fois moins que le dealer qui trafique au pied de son immeuble , c' est une honte . Que nos policiers puissent être attaqués aux cocktails Molotov , ça n' est pas un fait divers , c' est un scandale . Qu' un de vos concitoyens , tout près d‘ici , se croit autorisé à décider de son propre chef , quels étrangers peuvent entrer et résider en France , c' est le contraire de la démocratie et de la République . Nul n' est au-dessus de la loi ! La loi et la sécurité doivent être partout et pour tous , avec des forces de l' ordre modernisées et une Justice ferme et rapide . Nos forces sont sous-équipées , et elles le sont notamment sur tout ce qui relève des nouvelles technologies pour cerner et chasser la délinquance . Nos forces s' éreintent dans des tâches administratives . Elles doivent passer plus de temps sur le terrain . Pour cela , la tranquillité publique commande de renforcer le rôle des maires et des polices municipales qui sont au plus près des populations . Et puis , il faut une justice rapide et ferme . Chacun est responsable de ses actes ! Un voyou est un voyou avant d' être une prétendue victime de la société . Le sentiment d' impunité des délinquants est le résultat d' un manque de fermeté et de civisme qui devraient s' exercer à tous les niveaux : dans la famille qui doit assumer ses devoirs ; à l' Ecole qui ne doit rien céder devant l' indiscipline et l' inculture des caïds en herbe ; dans les prétoires où la Justice doit passer vite et nette . L' impunité zéro doit être la règle . Je veux rétablir les peines planchers pour les récidivistes . Un délit : une sanction ! Voilà la règle qui doit être parfaitement claire pour les délinquants . Cette fermeté retrouvée réclame de reprendre le programme de construction de places de prison interrompu par Hollande et Taubira . Vous m' aurez compris : j' en appelle à un nouvel état d' esprit face à la violence et à une mobilisation nationale pour qu' à chaque coin de rue nos policiers et gendarmes se sentent respectés , soutenus et le terroriste comme le délinquant réprouvés , coincés , pourchassés . La peur doit changer de camp et nul ne doit pouvoir se dire que le mal rapporte plus que le bien . Mes amis , Il y a des lieux en France où le sentiment d' appartenir à la même nation s' est disloqué . Les replis identitaires avancent et l' intégration recule . L' unité nationale commande une autre politique de l' immigration . Il ne faut pas céder aux injonctions de la bien-pensance , qui nous adresse toujours le même message , qui se veut angélique et qui est en réalité inconscient : liberté pour les migrants et honte aux frontières . Non , la France est généreuse mais elle n' est pas une mosaïque et un territoire sans limite ; c' est une Nation , en droit de choisir qui peut la rejoindre , en droit aussi d' exiger des étrangers qu' ils se plient à ses règles et à ses coutumes . Il existe chez nous , depuis la chrétienté , une tradition de l' hospitalité pour les victimes de persécutions politiques . Il existe aussi une tradition d' ouverture aux talents venus d'ailleurs . Je pense à Romain Gary , venu de l' Est de l' Europe s' installer à Nice avec sa mère quand il était enfant , et qui toute sa vie voudra être plus français que tous les Français , s' acharnant à réussir par les littérature et par les armes . Mais ces traditions d' ouverture doivent être raisonnables et raisonnées . Nous comptons 6 millions de chômeurs et près de 9 millions de pauvres . Dans ce contexte , l' immigration doit être fermement contrôlée et réduite au minimum . Il faut tout d'abord ramener notre politique d' asile à l' aune de sa seule ambition et mettre fin à son détournement . L' asile sert à protéger les victimes politiques ; il sert aux personnes qui ont fui les persécutions ou les guerres dans leurs pays . L' accueil des véritables réfugiés est un devoir , mais ne nous laissons pas déborder par ceux qui n' ont pas ce statut . Les demandes d' asile ont explosé depuis 2012 . Les crises du Proche-Orient n' expliquent pas cette augmentation massive , car 70 % des demandes sont rejetées comme infondées . Ce qui est en cause , c' est la multiplication des détournements de procédures nourris par les filières et les passeurs . La méditerranée est devenue le tombeau de dizaine de milliers d' êtres humains , femmes et enfants . Ces passeurs de faux rêves sont des passeurs de morts qu' il faut traquer et neutraliser . Ce qui est en cause aussi , c' est la longueur des procédures qui permettent aux illégaux de s' installer et de viser la régularisation de leur situation . Seuls 4 % des déboutés de l' asile sont effectivement éloignés ; les autres se maintiennent en France contre la loi . Je veux reconstruire le système de l' asile pour mettre fin à ces abus . Il faudra veiller à ce que les demandeurs d' asile puissent présenter leur demande à l' étranger , dans nos consulats , dans les pays qui leur servent de premier refuge . Il faut , en second lieu , réduire la durée d' instruction des demandes d' asile . Obligation sera faite aux demandeurs de former leur demande dans un délai de 15 jours après l' entrée sur notre territoire , sous peine de la voir traiter par une procédure accélérée . La durée maximale d' examen d' une demande d' asile devra être limitée à 120 jours au total , voies d' appel comprises . Pour être contrôlé , l' hébergement des demandeurs d' asile sera assuré dans des centres dédiés , gérés par l' Etat ou pour son compte par des associations . Enfin , ceux des demandeurs d' asile dont la demande aura été définitivement rejetée devront impérativement être éloignés du territoire . La décision de rejet devra s' accompagner immédiatement de la notification à l' intéressé d' une obligation de quitter le territoire français . Pour faciliter leur éloignement , ceux des demandeurs d' asile dont la demande apparaîtra manifestement infondée ou qui sont originaires de pays réputés sûrs , seront placés en rétention administrative pendant l' examen de leur demande , de manière que leur éloignement soit garanti en cas de rejet de leur demande . Il faudra généraliser le passage dans les centres de rétention administrative . Si nécessaire , mon gouvernement renégociera la directive &laquo; éloignement &raquo; pour tirer les conséquences de ce qui était prévisible : l' assignation à résidence privilégiée par Monsieur Valls est un échec qui ajoute encore aux difficultés de maîtrise des procédures d' asile . Penser qu' un étranger en situation irrégulière quittera de lui-même le territoire national est naïf . La durée maximale de la rétention administrative devra être portée de 45 jours à 180 jours . A ceux qui pousseront des cris d' orfraie en entendant cette proposition , je rappelle que cette durée est de 18 mois en Allemagne et qu' elle est illimitée au Royaume-Uni et aux Pays-Bas . Ce délai de six mois permettra d' organiser , en collaboration avec les autorités des pays d' origine , le retour effectif de leurs ressortissants . On ignore souvent que le retour dans leur pays des étrangers en situation irrégulière est soumis à la délivrance d' autorisations par les pays dont ils sont les ressortissants . Or ces pays freinent parfois la délivrance de ces documents , par mauvaise volonté ou par lenteur administrative . C' est pourquoi , je veux que la coopération des pays d' origine à notre politique d' immigration soit une condition de l' aide au développement que nous leur versons et du nombre de visas que nous délivrons à leurs ressortissants . Il s' agit d' assurer une forme de réciprocité pour faire respecter notre souveraineté . En revanche , je ne crois pas moralement possible de limiter le droit à la scolarisation des enfants d' immigrés clandestins . Les enfants ne sont pas responsables . C' est l' honneur de la République que d' assurer cette scolarisation . Une politique sérieuse de l' immigration repose aussi sur le règlement des situations de crise comme celle de Calais . Le gouvernement de François Hollande a tardé à démanteler les campements , en leur laissant prendre une ampleur inconnue jusque-là . Il n' a pas négocié avec le Royaume-Uni un rééquilibrage des accords du Touquet de 2003 pour traiter les causes du problème . C' est une exigence pour nous : les Britanniques doivent prendre toute leur part dans l' accueil de ces migrants , y compris les mineurs isolés , qui veulent traverser la Manche . En 30 ans , la population de l' Afrique va plus que doubler . Avec solennité , je dis que l' immigration en France n' est pas l' avenir de l' Afrique . L' avenir des Africains est dans le développement de leurs pays , dans la création d' emplois en Afrique ! L' aide au développement peut y participer dans le cadre d' un nouveau partenariat entre la France et le continent africain . La France y apportera sa maîtrise des technologies les plus modernes en particulier dans l' accès à l' énergie avec l' initiative remarquable de Jean-Louis Borloo , ses capacités de formation de la main d' œuvre africaine et , si nécessaire , des financements ciblés . Mais c' en est fini de la période où la question de l' immigration était passée sous silence dans les discussions bilatérales entre la France et chaque pays africain , au motif qu' il s' agissait des questions trop complexes ou trop sensibles . La question de l' immigration ne doit plus être un tabou dans nos relations avec les pays d' origine . A l' évidence , mes amis , l' ampleur des enjeux migratoires commande une coordination européenne . Je le répète depuis 2012 , il faut réformer les accords de Schengen . Je veux que les frontières extérieures de la zone Schengen soient de véritables frontières , où les contrôles des voyageurs , notamment non européens , seront systématiques . Si les étrangers doivent attendre 2 heures pour entrer , ils attendront 2 heures , comme dans n‘importe quel aéroport aux Etats Unis ! Quant aux pays membres de Schengen qui ne satisfont pas aux exigences d' un contrôle rigoureux aux frontières extérieures , ils doivent sortir de l' espace Schengen . Dans le contexte de guerre contre le totalitarisme islamique , tant que les frontières de l' Europe ne seront pas protégées par nos partenaires , la France rétablira durablement de vrais contrôles à ses frontières nationales . Ici à Nice , cet engagement prend tout son sens . Dans le contexte terroriste que connaît l' Europe , il y a quelque chose d' aberrant à savoir que l' auteur de l' attentat de Berlin a pu gagner la Hollande , puis la Belgique , la France , et enfin l' Italie . Je ne condamne personne mais le moins qu' on puisse dire , est que nous avons impérativement besoin de développer nos systèmes et nos échanges d' information , et de tirer les conséquences du développement de la vidéo-surveillance . Il ne suffit pas d' équiper nos gares ou nos aéroports en caméras de vidéo-surveillance , encore faut- il exploiter de manière rigoureuse les images issues de la vidéo-surveillance . La France doit non seulement lutter contre l' immigration illégale mais elle doit aussi réduire son immigration légale au strict minimum . Les délivrances de visas et de titres de séjour n' ont cessé de dériver ces dernières années : le nombre de visas accordés en 2015 a augmenté de 40 % , par rapport à 2012 . Pour reprendre les choses en main , il faut que le Parlement puisse adopter chaque année des quotas limitant le nombre d' étrangers qui pourront être admis en France , en fonction de nos besoins économiques et de nos capacités d' intégration . Concrètement , cela signifie qu' un visa de long séjour ou une carte de séjour pourront être refusés lorsque le contingent annuel sera dépassé ; la demande devra alors être examinée l' année suivante . C' est en fixant ces plafonds que nous pourrons limiter strictement l' immigration familiale , dont le regroupement familial . Si , pour assurer cette politique , il faut réviser la Constitution mais aussi faire bouger les lignes de l' interprétation que la Cour européenne des droits de l' Homme donne de la Convention européenne des droits de l' Homme , eh bien je prendrais mes responsabilités devant le peuple français . Ces quotas d' immigrants tiendront compte non seulement des besoins de la France mais aussi des capacités d' intégration à la société française . Ces capacités seront vérifiées dans le pays d' origine , tant pour les visas de long séjour que pour les cartes de séjour , en prenant en compte la maîtrise de la langue française et l' adhésion aux valeurs de la République . Il faudra enfin , et c' est le bon sens même mais il a été complètement perdu de vue au cours des années , que ceux des étrangers qui , une fois arrivés en France , ne respectent pas leur parcours d' intégration , puissent être sanctionnés , par le non renouvellement de leur titre de séjour . Et puis , je veux prendre en compte les conséquences des migrations à l' intérieur de l' Union européenne , et notamment les effets d' une concurrence mal maîtrisée . La directive sur les travailleurs détachés a progressivement installé sur notre territoire un véritable dumping social au cœur de l' Europe . C' est le cas dans le transport par camion , dans la construction où des milliers d' emplois français sont concurrencés de façons inéquitables . Nous renégocierons cette directive et je ne céderai pas sur un principe : en Europe , pour un travail égal , le salaire doit être égal et les cotisations sociales aussi . Trop souvent , l' immigration est encouragée par l' attractivité de notre système social . Il n' est pas acceptable qu' un étranger vienne en France dans le seul but de bénéficier de la solidarité nationale . Je veux que le bénéfice de toutes les prestations sociales soit conditionné à une durée de résidence préalable pour les étrangers en situation régulière . Ceux -ci ne pourront donc postuler aux allocations logement et familiales qu' au bout de deux ans . Ce délai permettra de vérifier la volonté d' intégration du nouvel arrivant et d' apprécier s' il est réellement autonome financièrement . Pour les étrangers en situation irrégulière , il faut mettre fin au dérapage financier de l' aide médicale d' Etat , dont le coût dépasse le milliard d' euros , et dont le nombre de bénéficiaires a augmenté de 50 % depuis 2012 . Cette aide médicale d' Etat sera supprimée et remplacée par la seule prise en charge des urgences médicales et des maladies infectieuses . Lorsque la France a fait confiance à des étrangers et que ceux -ci la trompent et ne respectent pas ses lois , l' Etat doit être de la plus grande fermeté . Il en va de son autorité et de la crédibilité de notre politique . C' est pourquoi je propose qu' un étranger condamné pour des crimes et délits puisse être expulsé du territoire français vers son pays d' origine . Enfin , pour les étrangers qui , après s' être installés durablement en France et espèrent en obtenir la nationalité , il faut définir notre exigence : c' est l' assimilation , c'est-à-dire la volonté d' être pleinement membre de notre famille nationale . L' acquisition de la nationalité française est un honneur . Elle se mérite . Elle se mérite , pour l' essentiel , par un patient effort pour adopter nos valeurs , nos coutumes , nos traditions . Qui dit assimilation dit une maîtrise de la langue française parce que c' est bien le minimum quand on prétend devenir Français . Qui dit assimilation dit aussi respecter les principes fondamentaux de la République . Respecter l' égalité hommes-femmes , partout , sans condition . Respecter la liberté de conscience comme la liberté d' expression . Respecter l' autorité de l' Etat et des lois , s' approprier une histoire , un drapeau . Etre assimilé , c' est tout simplement aimer la France . Nous devons revoir nos procédures d' accès à la nationalité . Nous sommes l' un des pays qui permet le plus facilement à des ressortissants étrangers d' acquérir sa nationalité . En quatre ans , le nombre de naturalisation par décret a augmenté d' un tiers , pour concerner 113 000 nouveaux Français en 2015 . Les instructions de Monsieur Valls pour augmenter le nombre de naturalisations seront donc abrogées . La durée minimale de résidence en France exigée d' un candidat à la naturalisation sera portée à 8 ans , comme c' est le cas en Allemagne . La République pourra s' opposer à toute entrée dans la communauté nationale dès lors que l' assimilation du postulant à la nationalité ne sera pas établie . Les étrangers qui ont fait l' objet d' une condamnation pénale verront leur demande écartée . L' acquisition de la nationalité sur simple déclaration à la suite d' un mariage ne sera possible qu' après 5 ans de vie commune et l' assimilation du conjoint étranger fera l' objet d' une vérification . Les jeunes nés en France de parents étrangers pourront , comme c' est le cas aujourd'hui , accéder à la nationalité française mais cet accès devra être le fruit d' un choix , mûri par chacun de ces jeunes et formalisé dans une déclaration de volonté de devenir Français . Voilà la politique d' immigration que je veux pour notre pays . Je la crois nécessaire à notre unité et nécessaire à la fraternité des liens qui doivent unir les Français et ceux des étrangers qui nous rejoignent avec sincérité et espoir en la France . Mes amis , maintenant je vais relever avec vous le défi du redressement national . Ce défi , je le gagnerai si toutes celles et tous ceux qui en ont assez des mensonges , assez du dé- clin , se lèvent . Je m' attaque au consensus , je m' attaque au confort du conservatisme , je m' attaque à une citadelle , et je ne pourrai la percer qu' avec l' appui de tous ceux qui veulent , avec moi , enclencher la révolu- tion du bon sens . Alors , je donne rendez-vous à toutes celles et tous ceux qui ont la tête sur les épaules . J' appelle tous ceux qui ont des valeurs solides . J' appelle tous ceux qui savent que le bonheur est une conquête . J' appelle tous ceux qui ont dans le cœur la fierté d' être français . Sur le chemin de la victoire nous traverserons les vents contraires ; nous passerons à travers les bourrasques ; nous franchirons tous les ponts . Vous êtes partant et je le suis , alors j' irai , avec vous , chercher les cœurs vaillants . Nous irons au-devant de toutes les forces françaises , et nous les réveillerons , nous les soulèverons , nous les entrainerons , et vous verrez qu' elles nous porteront au-delà de nos propres espérances . Nous serons alors , tous ensemble , la France en action ! Vive la France et vive la République . ",
    "Il y a des lois et il convient de les respecter . Sinon , c' est l' anarchie . C' est l' honneur de la France de protéger ceux qui fuient la guerre et sont persécutés . Mais il y a des règles qui encadrent l' accueil des réfugiés et des étrangers qui ne sont pas tous éligibles au droit d' asile . Ceux qui ne les respectent pas s' exposent à des sanctions . M. Herrou est peut-être généreux mais il n' est , à mon sens , pas responsable car ce n' est pas à lui de juger qui peut entrer et demeurer en France . Il faut une immigration minimale et très contrôlée , répondant à nos besoins économiques et à nos capacités d' accueil et d' intégration . Je veux mettre fin à cette situation douloureuse et sans issue qui voit des hommes , des femmes et des enfants dormant dans les rues , à même le sol , été comme hiver et dans des conditions d' hygiène déplorables , qui sont la proie de tous les trafics . Les laisser entrer en France où ils n' ont comme horizon que cette vie de misère n' est pas conforme à nos valeurs . Nous devons donner un signal fort en pratiquant une politique ferme d' expulsion des étrangers en situation irrégulière vers leur pays d' origine . Et notamment tous les déboutés du droit d' asile qui , aujourd'hui , restent sur notre territoire faute d' une réelle politique de reconduite aux frontières . Nous devrons aussi renégocier les accords de Schengen qui ne permettent pas de lutter contre les arrivées massives que nous connaissons aujourd'hui . Nous avons besoin d' un corps de garde-frontières et de garde-côtes pour surveiller nos frontières terrestres et nos façades maritimes , en particulier la Méditerranée . Les Etats qui ne parviennent pas à assurer la surveillance de leurs frontières devront sortir du système Schengen qui ne peut être efficace que s' il est vraiment appliqué dans tous les pays qui y adhèrent . Sinon , la charge est reportée sur les Etats jugés les plus attractifs par les candidats à l' immigration . Enfin , nous devons pouvoir rétablir des contrôles à nos frontières lorsque la situation l' exige comme c' est le cas actuellement en raison des impératifs de l' état d' urgence . A force de mettre de &laquo; l' eau dans notre vin &raquo; comme vous dites , on ne résout pas les problèmes de la France . Depuis trois ans , je parcours notre pays de long en large et j' entends les Français dire qu' ils en ont assez des demi-mesures . Ils ont raison . On est en déclin . Pour redresser notre pays , combattre le chômage de masse et relancer la croissance , il faut des mesures franches et percutantes . J' assume la radicalité de mon projet parce que le pays est radicalement bloqué . Je fais le pari que les Français veulent la vérité et qu' ils sont prêts à des changements puissants . C' est pourquoi je n' ai pas l' intention de renier mes convictions . Avec 6 millions de personnes à Pôle Emploi et plus de 2.000 milliards de dettes , on ne peut pas dire que ça va mieux et qu' on peut continuer comme cela longtemps . La très légère embellie sur le marché de l' emploi est largement due à la multiplication des stages et des emplois aidés . En réalité , notre croissance est poussive , notre modèle social est fissuré et nos déficits menacent notre avenir . Face à cette situation , mon projet n' est nullement &laquo; brutal &raquo; . Ceux qui disent cela , ce sont les démagogues de la gauche et de l' extrême droite qui veulent endormir les Français . La vérité , c' est que mon projet est tout simplement efficace , raisonnable et responsable . Si la France se réforme et se retrousse les manches , elle peut être d' ici 10 ans , la première puissance européenne . C' est mon objectif ! La Sécurité sociale fait partie de notre unité nationale . Je n' ai absolument pas l' intention de revenir sur le principe de la solidarité . La société du chacun pour soi n' est pas la mienne . Mais sur l' assurance maladie et la santé , sur les allocations chômage , sur les retraites , il y a des réformes à faire pour être plus juste et plus efficace . Il n' y a pas de honte à affirmer notre héritage chrétien et je n' ai pas de leçons à recevoir en matière d' unité nationale . Depuis longtemps , je m' oppose au communautarisme et au repli identitaire . C' est moi qui ai fait appliquer l' interdiction des signes religieux à l' école et mis hors la loi le port du voile intégral . Je ne transige pas sur la laïcité , mais la laïcité n' est pas l' adversaire de la foi . Je me suis engagé à respecter cette loi sur le non cumul des mandats . Je pense que les Français ne comprendraient pas que nous revenions là-dessus . La primaire de la droite et du centre a été digne . J' ai tendu la main à tout le monde . Le rassemblement autour de ma candidature s' est fait rapidement et sans problème majeur . Nous sommes maintenant en ordre de marche pour mener campagne et aller chercher la victoire . Dans la primaire , je n' étais pas le candidat de l' establishment et des sondages et je n' étais pas le &laquo; &raquo; espéré de la gauche et du FN . Je suis attaqué par mes adversaires parce qu' ils sentent ma volonté et celle de l' électorat de la droite qui s' est reconnu dans mes valeurs . Ces attaques ne m' impressionnent pas . Je vais continuer de tracer ma route .",
    "Mes chers amis , C' est un plaisir de me retrouver parmi vous et puisque l' heure est encore aux vœux , je souhaite à chacune et chacun de vous une bonne année 2017 . Soyez heureux , prenez soin de vous et de ceux que vous aimez , soyez passionnés , ne renoncez pas à vos rêves , et puisque vous avez la France au cœur , engagez -vous pour elle , mobilisez -vous pour elle : elle aura bien besoin de votre audace et de votre courage . Il y a quelques semaines de cela , j' entendais François Hollande nous dire , avec le ton martial qu' on lui connaît , &laquo; qu' il n' y avait pas de France sans Etat ! &raquo; … J' ai beaucoup de respect pour l' Etat , mais il faut surtout dire qu' il n' y a pas de France sans les Français ! Ce sont les Français qui font vivre notre pays . Ce sont eux , travailleurs , entrepreneurs , agriculteurs , ingénieurs , chercheurs et tant d' autres métiers , qui font que notre pays est encore la Vème puissance du monde ; ce sont nos militaires , nos policiers , nos gendarmes qui le sécurisent et le défendent , parfois au prix de leur vie . Avant de parler de l' Etat de façon abstraite , il faut parler des Français , de leur existence , avec ses coups durs et ses espoirs . Il faut entendre leur colère à l' égard d' un système qui les ligote . Il faut entendre leur volonté de protéger leurs traditions , leur langue , leur art de vivre , leur identité . Il faut entendre cette jeunesse qui ne sait pas si elle est condamnée au chômage ou à tenter sa chance à Londres , à Barcelone ou à New York . On vous oublie . Exigez le respect et exigez -le de toutes vos forces puisque vous êtes la France ! Alors que s' achève un quinquennat calamiteux , 2017 ne peut être autre chose qu' une grande année , une année de sursaut et d' espérance . L' année où la France aura fait le choix de se redresser . On nous a habitués , ces derniers temps , à rétrécir nos ambitions , à racornir nos vues . Moi , je veux vous inviter à revoir l' avenir en grand . Sans nostalgie , mais forts de notre passé . Sans démagogie , mais certains de nos compétences . Sans mensonge , mais confiants dans notre capacité à retrouver , ensemble , le rang qui aurait dû demeurer le nôtre . Il n' y a pas de fatalité pour un peuple qui se lève pour sa réussite . Et ce ne sont pas les caricatures , les regards narquois et les surenchères populistes qui nous ferons dévier . Nous le ferons tout simplement parce que nous avons pris la mesure de ces décennies d' immobilisme et de moralisme condescendant . Nous le ferons parce que , contrairement à l' ensemble des autres candidats , nous avons compris que la réussite doit cesser d' être présumée coupable . Le piège , c' est le compromis , la réforme molle , le refus d' aller au bout . Ce que je veux , avec vous et pour vous , c' est libérer la France de ce qui la fait tomber . Nous vivons dans un monde difficile , un monde où il va falloir se défendre . Se défendre contre le totalitarisme islamique qui rêve d' abattre nos valeurs . Nous ne céderons rien : c' est nous qui battrons ces fanatiques ! Se défendre aussi face à la pression des géants économiques que sont la Chine ou les Etats-Unis . Personne ne nous fera cadeau .Pour ceux qui ne l' auraient pas compris , Mr Trump ne fait pas dans la langue de bois et l' angélisme . Inutile de s' en indigner ou de trembler comme des feuilles . Nous jugerons le nouveau président américain sur ses actes . Mais le message est clair : la France doit être forte pour être respectée ! Et pour être forte , la France doit d'abord être fière car un pays qui ne croit ni en son drapeau , ni en son avenir , est un pays à genoux . Pour être forte , la France doit être plus productive , elle doit être la patrie du travail , du mérite et de l' audace , elle doit être en capacité d' entraîner ses alliés européens , elle doit être indépendante sur la scène internationale . C' est toute l' ambition de mon projet . Je veux redresser la France et lui donner les moyens de la puissance . Depuis ma victoire aux primaires , c' est &laquo; le tous contre Fillon &raquo; .Ne vous laissez pas intimider par les tirs de barrage . Ne vous excusez pas d' avoir des valeurs . Ne vous excusez pas de croire en la France . Exigez qu' on vous écoute , qu' on vous respecte . Assumez votre idéal et tenez vos positions ! Mon projet perturbe les castes bien établies . Aucune mauvaise foi ne les arrête pour conserver leur petit pré-carré ; aucun mensonge , aucune sornette ne les freine . Le chœur des pleureuses annonce , en se tordant les mains , la fin du monde si la droite l' emportait . L' économie française , vous dit -on , s' arrêtera brutalement . On verra , par millions , des Français désœuvrés , errer , à la recherche d' un service public en état de marche . A les entendre , sans le socialisme il n' y aura plus de social et sans la firme Le Pen il n' y aura plus de nation française … Mais voilà , je suis Français et bien décidé à donner à mon pays sa sécurité et son progrès . Mais qui sont -ils , pour nous accuser ? Qui sont ces donneurs de leçons ? L' extrême-droite dont le projet économique est un copier-coller de celui de l' extrême gauche ? Retraite à 60 ans et retour du Franc : le camp de la faillite accélérée n' a aucun titre pour défendre le peuple français ! La gauche ? Depuis cinq ans , elle n' a rien fait . Par leur incompétence , leur hésitation , leur désinvolture , les socialistes ont précipité le déclin de la France . Leur bilan est sans appel . Il est tel que le Président , plutôt que de devoir l' assumer , a jugé plus prudent de s' éclipser discrètement . François Hollande sort de l' Histoire par la porte de derrière . Pas de vivats , pas de rappel , juste le silence d' une France indifférente mais à cran . Mes adversaires m' accusent de menacer le &laquo; modèle social français &raquo; . Mais son affaiblissement , c' est maintenant que nous le voyons , avec un chômage massif , avec une pauvreté et une précarité grandissantes , avec des déficits partout , des déserts médicaux , des soins de santé non remboursés , des urgences débordées par une épidémie de grippe , des structures d' hébergement saturées par le retour du froid ! Ce qui est brutal , ce n' est pas mon projet , c' est l' échec de la gauche . Ce qui est brutal , c' est le désespoir des chômeurs , des exclus , des classes moyennes . Ce qui est brutal , c' est la délinquance et la violence qui pourrissent la vie des gens honnêtes . Il y a une chose qui ne sera jamais pardonnée à François Hollande , c' est d' avoir trahi la jeunesse . Il a menti en lui promettant n' importe quoi . Il a menti en lui cachant la réalité . Il a menti sur les chiffres du chômage .La réalité , la voici . Près d' un jeune actif sur quatre est au chômage . Plus de 100 000 jeunes quittent , chaque année , le système éducatif sans diplôme ni qualification . Deux millions d' entre eux sont à la dérive , ni au collège , ni en formation , ni au travail . Et , au-delà de la jeunesse , six millions de Français pointent à Pôle emploi . La pauvreté augmente , faute de croissance . Des quartiers sont déglingués par la délinquance , des campagnes dépérissent . De plus en plus de familles n' arrivent pas à boucler leurs fins de mois . Et nous laissons 2200 milliards de dettes à nos enfants . Comment en sommes -nous arrivés là ? Mon diagnostic est clair . Et tant pis s' il ne plaît pas aux démagogues qui aiment les histoires à dormir debout . Il tient en deux phrases . D' un côté , l' excès de normes , de contraintes et d' impôts freine l' activité , l' initiative , l' investissement et les embauches . De l' autre , le chômage et la précarité nécessitent toujours plus de réglementations qui gênent les entreprises , et de dépenses publiques qui alourdissent notre endettement et poussent à augmenter les impôts , ce qui ne fait que boucler le cercle vicieux conduisant à moins de croissance et plus de chômage . En réalité , ce constat est bien connu . Ce qui manque , c' est le courage de le regarder en face et d' agir . Moi , je veux sortir de cette logique infernale !Je veux débloquer l' économie , rallumer le moteur de la croissance , revenir au plein emploi . Je veux redonner espoir à la jeunesse en lui donnant le pouvoir d' entrer dans le monde du travail . C' est le travail qui émancipe , et c' est le chômage qui aliène . La meilleure politique sociale , c' est celle qui donne du travail à chacun . Et la seule politique qui donne du travail à chacun , c' est la liberté ! Les milliers de Français que j' ai rencontrés , ces trois dernières années , me l' ont tous répété : la bureaucratie a envahi nos vies , &laquo; qu' on arrête de nous emmerder ; laissez -nous travailler , oser et réussir &raquo; . Tout est encadré , régulé , contrôlé . La paperasse a remplacé l' audace . La circulaire a remplacé l' initiative . Les normes ont remplacé le bon sens . Mes amis , c' est à nous d' assumer la première valeur de notre devise républicaine , la liberté . Ne la laissons pas se dissoudre dans les lubies égalitaristes de la gauche , les fantasmes autoritaires de l' extrême droite , et l' immobilisme économique des deux réunies . Pour libérer l' économie , je veux concentrer le code du travail sur ses principes fondamentaux , et renvoyer le reste au dialogue social . Je donnerai aux entreprises et à leurs salariés le pouvoir de négocier pleinement entre eux les heures et les conditions du travail . Les 35 heures , je ne veux pas les moduler , les atténuer , les esquiver : je veux les supprimer totalement ! Je fais le pari de la vraie négociation sociale , dans les entreprises , là où entrepreneurs et salariés savent de quoi ils parlent , parce qu' ils parlent de leurs vies , de leur pouvoir d' achat , du carnet de commandes dont tout dépend . Je renforcerai le statut de l' autoentrepreneur et créerai celui de travailleur indépendant , pour redonner à ceux qui veulent entreprendre le pouvoir de le faire . La révolution numérique va transformer en profondeur tous nos processus , tous nos métiers , toutes nos industries . Elle nous offre un potentiel d' un million d' emplois nouveaux si nous savons anticiper ces changements , former nos jeunes , adapter nos structures . Par la liberté , je donnerai aux investisseurs de France le goût d' investir ici , chez nous , pour nos industries , pour nos emplois . Je baisserai massivement les charges et j' allégerai la fiscalité qui pèse sur l' investissement . François Hollande prétendait ne pas &laquo; aimer les riches &raquo; . Très bien , on a taxé et surtaxé la réussite et les Français sont tous devenus un peu plus pauvres . Je suis un patriote : je préfère supprimer l' ISF que de voir l' économie française perdre des capitaux et tomber sous la coupe du Qatar et des fonds de pensions américains ! Débloquer l' économie , c' est aussi faire revenir la sphère publique à une taille raisonnable . Je veux mettre de l' ordre dans les finances publiques , mettre les administrations aux 39 heures et revenir à un nombre d' emplois publics équivalent à celui des principaux Etats développés . Avec 5,6 millions d' agents publics , franchement , il n' y a rien d' extravagant à réduire de 8 % en 5 ans ce nombre . Je préfère un Etat fort , avec des agents publics mieux organisés , mieux rémunérés , fiers de leurs missions , avec des plans de carrière plus motivants , qu' un Etat suradministré et croulant sous les déficits . La puissance publique doit être un atout pour la prospérité , pas un frein . Elle ne doit pas réprimer la liberté mais la protéger . La liberté est le meilleur carburant des sociétés vivantes . C' est elle qui encouragera les créations d' emplois et relancera l' ascenseur social . Ceux qui aspirent au progrès et au dépassement de soi , ils ne craignent pas liberté : ils la demandent , pour prendre les commandes de leur vie . La libération de l' économie débloquera les salaires par plus de travail . Elle permettra d' équilibrer nos comptes publics en 2022 . Elle stimulera l' investissement par une fiscalité plus favorable . Et elle bénéficiera aux Français puisqu'elle nous permettra de redistribuer 10 milliards de pouvoir d' achat supplémentaire aux salariés , à travers la hausse du salaire net et une politique familiale renforcée . Cette libération de l' économie et du marché de l' emploi , je veux la mettre au service de notre jeunesse . Dans le monde qui se dessine , la principale matière première : c' est le savoir . Ce sont les qualifications , les compétences , la formation . Je veux refonder notre école qui doit transmettre les savoirs fondamentaux à tous nos enfants avant de multiplier les activités périphériques . D' autre part , et ce fut l' objet de mes visites d' aujourd'hui , je souhaite développer massivement l' alternance . Plus du quart des jeunes Français actifs sont au chômage , contre 8 % des jeunes Allemands . Or nous avons quatre fois moins d' apprentis en France qu' en Allemagne . Là-bas , 55 % des élèves d' une classe d' âge vont en apprentissage , et beaucoup de patrons sont d' anciens apprentis . En Allemagne , au Royaume-Uni , en Suisse , où le chômage des jeunes est moins élevé , l' apprentissage est résolument tourné vers les entreprises , qui sont très impliquées dans la définition des formations . Chez nous , l' alternance stagne , alors que c' est elle qui garantit l' employabilité , du secondaire jusqu' à l' enseignement supérieur , quelle que soit la filière suivie . Nous allons réagir . Je veux que l' apprentissage devienne la voie de droit commun pour l' obtention d' un bac professionnel . Ce serait un changement majeur . Concrètement , cela impliquera que les lycées professionnels ouvrent des sections d' apprentissage en leur sein , à l' image des centres de formation des apprentis . Si notre système fonctionne si mal , c' est notamment parce que tout le monde essaye de le gérer : l' Etat , l' Education nationale , les collectivités locales , les branches professionnelles . Il est temps de mettre de l' ordre dans ce labyrinthe . Et il est temps de donner plus de liberté aux acteurs de terrain . Ce sont eux qui sont les mieux à même d' adapter l' offre de formation aux besoins locaux . Je veux placer les lycées professionnels sous l' autorité des régions et des branches professionnelles . Quant à l' Etat , il doit conserver un rôle d' évaluation . Je veux aussi inciter les entreprises à recourir à l' apprentissage , en leur offrant un cadre plus attractif . Financièrement d'abord , en exonérant totalement de charges sociales l' embauche d' un jeune en alternance . Mais aussi juridiquement , avec un cadre administratif plus souple : est -il normal qu' un apprenti ne puisse pas monter sur un escabeau pour des raisons de normes , alors que le salarié le pourrait ? Il faut aligner et rapprocher les régimes . Et il faut revoir le rythme de l' alternance , pour le rendre compatible avec celui de l' entreprise , et pour que l' apprenti soit davantage présent dans l' entreprise que sur les bancs de l' école . Bref , je veux que l' apprentissage devienne une véritable voie d' excellence ! Quant aux contrats aidés , trop d' entre eux n' assurent pas aux jeunes leur insertion professionnelle . Seule l' entreprise crée vraiment des emplois durables . Je veux donc réorienter le budget de ces contrats vers l' alternance . Soyons innovants ! Je propose , par exemple , que ce redéploiement permette de développer des dispositifs de préapprentissage dès la classe de quatrième au collège , et d' alternance dans les écoles de la deuxième chance . Je souhaite que nous fassions tout pour rattraper chaque jeune en situation d' échec ou en marge du système , afin de les rediriger vers le monde de l' entreprise . Je veux aussi développer la formation professionnelle des personnes au chômage , notamment pour sortir de ces situations ubuesques où l' on constate des pénuries de main d' œuvre qualifiée alors que le chômage n' a jamais été aussi haut . Mes amis , Pour redresser la France , il faut enrayer le fléau du chômage , mais aussi refonder les piliers de la République . La famille est un des piliers de notre société . Elle a été dédaignée . Je la protégerai , je lui redonnerai ses droits et des moyens supplémentaires ! J' ai parlé de l' école et de la formation . L' Ecole de la République est aussi l' Ecole du civisme et de la citoyenneté . Elle a dérivé loin de cet idéal . Je refonderai l' école de la République en commençant par un symbole fort : le port d' un uniforme scolaire donnant à tous les élèves , quelles que soient leurs origines , le goût d' être ensemble , tous égaux , tous conscients d' être dans un lieu singulier qui exige discipline et fraternité : le lieu du savoir . La liberté sans l' autorité , c' est l' anarchie ! Je veux réaffirmer l' autorité de l' Etat , le respect du droit , la nécessité de l' ordre . La sécurité n' est pas une option : c' est un droit fondamental . Je veux des forces de l' ordre modernisées , respectées ; je veux une Justice ferme et rapide . Je rétablirai les peines planchers pour les récidivistes . L' impunité zéro doit être la règle . Que nos pompiers , nos policiers puissent être caillassés , injuriés , brûlés à coups de cocktails Molotov , que nos enseignants aient la gorge serrée avant d' entrer dans leurs classes , c' est la preuve qu' une part de notre jeunesse part en vrille . Où est le respect , où est la morale , où sont les familles ? Partout en France , et notamment chez nos concitoyens les plus modestes , ceux qui sont au cœur des cités difficiles , on demande de l' ordre , du travail , de l' éducation . On veut la loi de la République , pas la loi de la rue ! Aujourd'hui , les gens honnêtes ont peur et les délinquants n' ont pas froid aux yeux … C' est un comble . La peur doit changer de camp car il n' y a pas de démocratie sans sécurité . Je souhaite une république unie . La France n' est pas une addition de communautés , repliées sur elles-mêmes . La France a une identité , des coutumes , une langue , une culture , un drapeau ! Nous avons des différences mais nous n' avons collectivement qu' un destin . Un pays qui se disloque de l' intérieur , qui se désagrège en clans , en ghettos , un pays qui ne croit plus en son unité faute de savoir quelle est son identité , ce pays est condamné à s' effondrer . Dans un monde de 7 milliards d' habitants , les 66 millions de Français doivent faire bloc . Au fond de leur cœur , ils doivent avoir ce sentiment d' être les membres d' une même famille qui s' engage ensemble pour la France : la France , ce vieux et magnifique pays qui est un peu notre chair à tous car nous ne sommes pas des pions anonymes , nous sommes les successeurs d' une histoire étonnante qui a fait de nous l' une des toutes premières puissances du monde . Il a fallu des siècles pour faire la France . A aucun moment notre nation ne s' est définie de manière spontanée . Nous avons fédéré nos provinces , nous nous sommes imposé une langue commune , nous avons défini et protégé des frontières , nous avons su marier nos influences latines , germaniques , atlantiques , méditerranéennes , d' outremer … Tout ceci pour dire , mes amis , que la France n' est pas le fruit du hasard , mais le fruit d' une volonté de rassembler . Rassembler les Français , cela va de soi , par-delà leurs origines sociales et leurs opinions . Mais aussi rassembler tous ceux qui , au cours des siècles , ont rejoint notre nation pour y travailler , pour y vivre , mais aussi parfois pour la défendre au péril de leur vie . Vous connaissez la formule de la légion étrangère – &laquo; , non par le sang reçu mais par le sang &raquo; . La France a accueilli et assimilé des générations d' étrangers qui lui ont apporté leur énergie . Notre pays avait besoin d' eux et il pouvait compter sur son économie pour subvenir à leurs besoins . Aujourd'hui , ça ne marche plus . Sur cette question de l' immigration , j' ai des convictions que je veux vous faire partager . D'abord , je regarde froidement les faits et je dénonce l' angélisme de ceux qui estiment qu' il n' y a pas de problème .A l' évidence , les capacités d' accueil de notre pays sont saturées . Nous sommes en crise économique et priorité doit être donnée à ceux qui , chez nous , cherchent du travail ; notre système d' intégration ne fonctionne plus ; dans certaines de nos écoles , la mixité sociale a disparu ; dans nombre de nos quartiers le communautarisme s' étend , sans parler de ceux qui instrumentalisent la religion pour défier les lois de la République . Ne pas voir cela , c' est être dans le déni . Et le déni , c' est la porte ouverte au scénario de l' explosion sociale , et ce serait un malheur pour tous .Je stopperai l' immigration sans intégration , sans assimilation . Les étrangers ont des devoirs avant d' avoir des droits . Je veux réduire l' immigration à un strict minimum en la limitant par une politique de quotas . Ceux que nous accueillons sont les bienvenus s' ils respectent leur hôte . Sinon ils n' ont rien à faire chez nous . La France accepte les vrais réfugiés politiques mais sera inflexible avec les clandestins . Je veux que l' aide au développement soit conditionnée à la coopération des Etats d' origine en matière de maîtrise des flux migratoires et de retour de leurs ressortissants . Je veux protéger notre République contre le totalitarisme islamique . Ceux qui refusent d' employer le mot &laquo; totalitarisme &raquo; et ne parlent que de &laquo; terrorisme &raquo; confondent l' arme et l' idée . Ils se voilent la face et trompent les Français ! Le totalitarisme est une idéologie qui veut la fin de notre liberté . Le terrorisme n' est que le moyen détestable utilisé dans cet objectif . Nos concitoyens musulmans doivent être protégés de cette idéologie , à laquelle ils sont loin de tous adhérer , mais qui les infiltre et les prend en otages . Je demande un strict contrôle administratif du culte musulman , tant que son ancrage dans la République ne sera pas pleinement achevé . J' exige la dissolution immédiate de tous les mouvements qui se réclament du &laquo; salafisme &raquo; ou des frères musulmans . Je veux qu' on retire leur nationalité aux Français qui sont partis nous combattre en Syrie et en Irak . Mes amis , Ce qui est en jeu avec le redressement économique de notre pays et sa nécessaire union autour des principes de la République , c' est notre souveraineté . La France doit être souveraine , dans une Europe souveraine . Notre pays doit pouvoir s' appuyer sur ses forces militaires . Leurs moyens seront augmentés . Il doit poursuivre une politique étrangère fondée sur l' intérêt des Français . La France doit redevenir la première puissance européenne et aider l' Europe à devenir la principale puissance mondiale . Je ne parle pas de l' Europe communautaire et de ses normes byzantines pour fixer la taille des choux fleurs . Je parle de l' Europe des nations , unies pour défendre la civilisation européenne . Avec Donald Trump notre Continent ne sera sans doute plus la priorité des Américains . Et avec la tragédie de Berlin , une certaine idée du pacifisme a sans doute vécu en Allemagne . A nous , Français , de remobiliser l' Union européenne autour de priorités stratégiques . D' une part , en matière de défense et de sécurité collective : nous devons mettre nos forces en commun pour protéger l' Europe face aux nouvelles menaces . D' autre part , en matière d' innovation et de numérique : nous devons viser une véritable souveraineté numérique européenne . Les investissements européens dans les infrastructures et métiers de demain doivent être massivement concentrés sur le numérique et les réseaux . L' Union européenne doit se doter de normes et de standards communs , sans attendre que ceux des grands groupes américains s' imposent à nos entreprises . Et il est indispensable de garantir l' autonomie stratégique de l' Europe dans certains domaines clés comme les microprocesseurs , la nanoélectronique , l' électronique quantique , l' identité numérique , les télécoms . J' évoquerai tous ces sujets à la fin du mois avec la chancelière allemande . C' est par un accord franco-allemand que l' on pourra enclencher une dynamique européenne . Mes chers amis , Il nous reste trois mois ! Trois mois pour redonner espoir à la jeunesse . Trois mois pour partager notre idéal . Trois mois pour balayer les frilosités , les conservatismes , le politiquement correct . Trois mois avant de tourner enfin la page du déclin et du fatalisme . Trois mois pour aller à la rencontre de chaque Français et lui redonner le goût du progrès . Trois mois pour être à l' avant-garde de la révolution du bon sens . Ces trois mois pèseront lourd dans l' avenir de notre pays . Vivez -les intensément . Vivez -les avec détermination , vivez -les avec la passion des femmes et des hommes libres . Venez avec moi ouvrir une nouvelle page de notre histoire ! Que votre volonté soit l' étincelle des autres volontés . Entrez dans cette aventure éternelle d' une nation qui toujours se redresse et s' élève : l' aventure de la France , notre France . Vive la République , vive la France ! ",
    "Mes chers compatriotes , mes amis , Des quartiers de Paris , de toutes nos régions , de tous les âges , de toutes professions , vous êtes venus en nombre . Merci aux orateurs qui viennent de s' exprimer . Merci Valérie , merci à François Baroin , merci aussi à Jean-Christophe Lagarde qui représente l' UDI . Merci tout particulièrement à Alain Juppé qui est au rendez-vous de l' unité , et j' ai une pensée pour Nicolas Sarkozy à qui j' ai parlé hier qui aimait tant être parmi vous . Nous sommes rassemblés . Vous êtes unis dans les couleurs du bleu , du blanc et du rouge ! Quelque chose de profond nous relie , quelque chose qui passe au milieu de nous comme un fluide : c' est l' instinct de la France ! Votre mobilisation est extraordinaire . Elle me touche , elle me confère aussi des devoirs . A vous , je fais le serment de donner tout ce qu' un patriote peut donner à son pays . Je vous donnerai tout : ma détermination , ma vaillance , ma volonté de vaincre . En retour , je vous demanderai d' avancer tête haute . Ne cédez à aucune intimidation , ne lâchez aucune de vos valeurs . Soyez plus grands que ceux qui nous mitraillent . C' est plus que ma personne qui est dans leur viseur : c' est une haute idée de la France qu' on veut abattre en vol . Mais on n' abat pas une idée qui est au fond de l' âme française . Cette idée a plus de 1000 ans ; cette idée a toujours 20 ans ; cette idée a pour noms , liberté , honneur , grandeur , fierté . Dans cette salle , vous êtes plus de 13.000 , 13.000 fiertés françaises que rien ne pourra arrêter ! Il fallait cette démonstration de force pour faire entendre la voix des militants de la France . Cette voix puissante – votre voix – se propagera partout dans le pays . Elle amplifiera la campagne que j' ai engagée , elle touchera ces millions de Français qui attendent le sursaut . Il fallait , mes amis , ce rassemblement populaire pour rappeler à ceux qui nous jugent et nous attaquent , de quel bois nous sommes faits . Je veux que nos adversaires vous entendent , qu' ils vous respectent . Je veux que votre clameur franchisse les murs de cette salle . Haussez le ton , haussez la voix , qu' ils comprennent que rien ne stoppe un peuple qui se dresse pour sa fierté ! Je viens de loin et on ne m' intimidera pas . Je repense à ma première campagne électorale . J' avais vingt-six ans . J' étais inconnu . C' était une élection cantonale . En 1981 . Au mois de février . Au petit matin , je prenais ma voiture qui toussait au démarrage pour aller coller mes affiches et sonner aux portes des maisons . Parfois , on me fermait la porte au nez , souvent on m' offrait du café dans la cuisine . La politique , c' est un long chemin . Un chemin rude . Depuis le début , Pénélope est à mes côtés , avec discrétion , avec dévouement . J' ai construit mon parcours avec elle . Nous n' avons rien à cacher , notre seul compte en banque est au Crédit Agricole de Sablé . A trois mois de l' élection présidentielle , comme par hasard , on construit un scandale . A travers Pénélope , on cherche à me casser . Moi , je n' ai peur de rien , j' ai le cuir solide . Si on veut m' attaquer qu' on m' attaque droit dans les yeux , mais qu' on laisse ma femme tranquille . Devant vous mes amis je veux dire à Penelope que je l' aime et que je ne pardonnerai jamais à ceux qui ont voulu nous jeter aux loups . Pour résister à cette entreprise de démolition , je vous dis , du fond du cœur , merci d' être là , avec moi , avec nous ! Mes amis , J' aime la France et les Français . J' en ai croisé , des visages … Je n' ai rien oublié . Je n' ai pas oublié les mots courageux de cet agriculteur qui travaille 70 heures par semaine pour 800 euros par mois . J' ai en mémoire le témoignage de cet officier français , en mission au Sahel , qui racontait comment sa compagnie avait reconstruit l' école détruite du village . Ça tirait encore dans tous les coins , mais pour cet officier français , pas de village sans école ! J' ai rencontré plusieurs fois ces dernières semaines des membres d' Emmaüs . Mes parents en étaient . La générosité et l' humilité de ces bénévoles est une leçon . J' ai vu tant d' entrepreneurs créatifs , tant de salariés et de fonctionnaires dévoués , tant de chercheurs étonnants , qu' il m' est insupportable d' entendre dire que la France est fichue . Le Vendée Globe , cette course du monde à la voile , en solitaire , s' est conclue par la victoire d' un marin français , Armel le Cléach . Son exploit nous donne le goût de l' aventure et du dépassement de soi . Paris est candidate à l' organisation des JO de 2024 : je vote pour l' enthousiasme , et je dis à nos handballeurs qui vont bientôt débuter leur finale : cap vers la victoire ! Les Français sont formidables et le jour où la bureaucratie arrêtera de les &laquo; emmerder &raquo; , ils le seront encore plus . Et puis , il y a eu sur mon chemin des instants qui ne s' oublient jamais . Jamais ! Le camp d' Auschwitz , au côté de Simone Veil . Nous étions tous les deux et Simone me murmurait ce qu' elle avait vécu à 16 ans , dans cet enfer . Cette visite aussi du cimetière de Verdun parmi ces milliers de croix blanches . Chaque croix porte un nom , chaque nom parle d' une vie donnée à la France . Les Français sont -ils prisonniers de leur mémoire ? Non , ils ne le sont pas et je veux bien au contraire , qu' ils en soient les gardiens . Ils n' ont pas à rougir de leur Histoire , il n' y a pas à s' excuser d' avoir eu des héros Je suis candidat à l' élection présidentielle pour la France . Je ne suis pas un produit marketing , ma parole ne m' est pas soufflée par le microcosme , elle me vient de tous les Français qui m' ont fait partager leurs difficultés et leurs espoirs . J' ai ressenti leur peur du déclassement . Je suis porté par leur énergie , leur désir de justice . Je veux incarner la fierté d' une nation qui ne se laisse pas abattre . Je m' adresse à tous les Français . Il n' y a pas un peuple de droite contre un peuple de gauche . Il n' y a qu' un peuple , le peuple français capable de toutes les grandeurs dès lors qu' on lui dit la vérité et qu' on lui fixe des buts clairs et justes . Je veux convaincre la jeunesse que l' audace du changement est en nous . Elle n' est pas chez les défenseurs du système ; elle n' est pas chez ces utopistes qui rêvent l' impossible pour ne pas construire le possible . Je veux encourager les entrepreneurs , redonner confiance aux agriculteurs et au monde rural qui ne sont pas les derniers vestiges d' une France disparue . Je veux entendre la voix des Outre-Mer qui réclament respect et progrès . Ils sont une part infiniment précieuse de la France dans son étonnante diversité . Je veux parler aux français de l' étranger qui sont souvent les meilleurs défenseurs de notre culture et de notre savoir-faire . Je veux convaincre les ouvriers que la compétitivité est la clé de leurs emplois et de leur pouvoir d' achat . Je veux réveiller la vocation des fonctionnaires , des policiers , des enseignants , des infirmières qui servent le bien public . Je veux être aux côtés des familles qui sont le creuset de la France qui veut de l' éducation , de l' autorité , du travail pour ses enfants . Je veux parler pour les chômeurs , les sans grade , les sans carnets d' adresse , les courageux , tous ceux pour qui le changement est une opportunité de casser le plafond de verre qui fige leur destin . Je m' adresse aux abstentionnistes qui s' abstiennent parce que rien ne change . Je m' adresse aux électeurs du Front national qui pensent que tout part à vau l' eau . Je m' adresse aux électeurs socialistes qui se mordent les doigts . Je veux convaincre et rassembler . On me décrit comme le représentant d' une France traditionnelle . Celui qui n' a pas de racines marche dans le vide . Je n' oublie pas ce que je suis et d' où je viens . Je ne renie rien de ce qu' on m' a transmis , rien de ce qui m' a fait , pas plus ma foi personnelle que mes engagements politiques . Mais je sais aussi que nous aurons besoin de tout le monde , qu' il me faudra être au service de tous , et croyez -moi , j' y suis prêt . Je récuse ces divisions inutiles dont s' abreuvent plusieurs de mes concurrents . N' opposons pas les riches aux pauvres , les entrepreneurs aux ouvriers , les croyants aux agnostiques , ceux qui ont choisi la France et ceux qui l' ont reçue en héritage . Je veux que chacun sache qu' une seule chose compte pour moi , sans considération d' origine , d' opinion , de religion : le désir de servir et de rendre à la France ce que nous avons reçu d' elle . Nous sommes français . Tous français . Parmi nos parents , certains n' ont jamais quitté leur village , d' autres viennent du bout du monde , mais nous partageons le même espoir : l' espoir d' une vie libre et digne . Je ne cesserai pas de vous parler de la France , de cette France que tous peuvent aimer . Une France entière et fraternelle , universelle et particulière , mais toujours à reprendre , à inventer , à sauver . C' est la même France , du baptême de Clovis à la déclaration des droits de l' Homme . C' est la France des cathédrales et des barricades de Paris . Celle des rois et des sans culottes . C' est la France de Chateaubriand et celle de Kessel , celle de de Gaulle et celle de Césaire . La France ne nous demande rien . Nous lui devons tout . Pour la servir , nous aurons besoin de courage , de volonté et d' amitié et rien de cela ne nous manque . Les défis sont immenses . Que voyons -nous autour de nous ? D'abord , un pays défait , profondément défait , dans son image et , dans l' estime qu' il a de lui-même . Un pays plongé dans une crise morale sans précédent . Un pays sans chef . Nous voyons un pays rongé par le chômage , la précarité , accablé par une dette insoutenable , un pays où l' on n' investit plus , où l' on imagine à reculons . Nous voyons une classe politique effrayée à l' idée d' affronter ce monde nouveau . C' est la révolution numérique , qui va remodeler en profondeur nos sociétés , nos manières de vivre . Plutôt que de trembler devant cette révolution , il faut lui laisser sa chance , il faut la maitriser au profit de tous . C' est la mutation écologique et l' incontestable réchauffement climatique qui met notre planète en péril . Avec responsabilité , avec des technologies nouvelles , on peut réagir et aller vers une économie décarbonée . Les Etats qui chercheraient à rompre les accords de Paris sur le climat devront en rendre compte devant l' humanité . Autour nous , c' est aussi la pression d' un islam radical expansionniste qui défie notre civilisation . Ces fanatiques sont cruels , leur projet macabre nous rappelle ce que nous avions oublié : nous avons des valeurs et ces valeurs se défendent . Et puis , il y a l' Amérique de M. Trump , qui semble -t-il , a décidé de faire céder quiconque ne se plierait pas à ses intérêts . Les Américains sont nos alliés et nos amis , mais la France n' est vassale de personne et il est temps que l' Europe se réveille . Ces défis sont gigantesques . Pour les traiter , la plupart des politiques feignent de croire qu' il suffira d' appliquer les recettes anciennes . Ne rien faire , mais avec de bons chargés de communication . Non , pas de communication , pas d' échappatoire : nous sommes 66 millions de Français dans un monde de sept milliards d' habitants . Personne ne nous fera de cadeaux . Notre seule marge de manœuvre réside dans notre capacité à nous rassembler , à travailler plus , à travailler tous , à nous saisir de toutes les potentialités scientifiques et technologiques dont l' esprit français est capable . Notre marge de manœuvre est dans notre inflexible volonté d' être souverain , écouté et respecté de par le monde . Etre debout ou à genoux : voilà la question . Ma réponse est claire : debout toujours , à genoux , jamais ! Je veux , avec vous , faire de la France une puissance gagnante . La première puissance européenne d' ici 10 ans . Nous allons faire ce que nous n' osons pas faire depuis trop longtemps : faire confiance à l' énergie française . Nous allons nous détacher de cette chape de plomb idéologique qui a piétiné le goût du travail , piétiné la récompense du mérite , piétiné ce qu' il y a de plus lumineux en nous : la liberté . La liberté , c' est le carburant des sociétés vivantes . Le temps est passé où l' Etat pouvait diriger un pays comme on le fait d' un régiment . Le devoir de l' Etat , c' est aujourd'hui de rendre les choses possibles , ce qui est très différent . Le monde vertical d' autrefois disparaît sous nos yeux . Un monde nouveau se dessine , fait de mille initiatives , un monde où d' autres solidarités apparaissent , où la créativité est décisive . Si nombre de jeunes , étudiants , entrepreneurs , artisans baissent les bras ou s' expatrient , c' est parce qu' ils sentent que la France des rentes et des structures ne les aime pas , ne les comprend pas . C' est à cela que nous mettrons fin ensemble . Nous allons faire de notre pays celui de l' audace plutôt que celui de la bureaucratie , le pays qui investit dans l' avenir plutôt que d' étouffer sous sa dette , le pays du dialogue social plutôt que celui des oukases syndicaux , le pays des initiatives plutôt que celui des circulaires . D' une société où tout est nivelé , figé au nom d' un misérabilisme mou , nous passerons à une société d' engagés et d' engagements . Avec tous les Français qui veulent réussir , je fonce pour une France du plein emploi , une France productive . C‘est ma priorité absolue ! Il faut travailler tous pour produire plus et gagner plus . Et il faut en parallèle remettre nos finances publiques en ordre afin de mettre un terme au cercle vicieux : toujours plus de dépenses , donc toujours plus de déficits et donc toujours plus d' impôts et de charges . Je baisserai massivement les charges qui pèsent sur le coût du travail et sur la feuille de paye des Français : le gain sera d' au moins 250 euros par an et de 500 euros pour les couples qui travaillent . Je généraliserai la participation avec une priorité pour les salariés des petites entreprises . Je sortirai des 35 h , sans démagogie , sans diktat , puisque je donnerai aux entreprises la liberté de négocier avec leurs salariés . Je généraliserai l' apprentissage , refonderai la formation professionnelle , créerai le statut de travailleur indépendant , développerai le statut d' autoentrepreneur , je miserai à fond sur les technologies , je ferai de la France un champion mondial de l' intelligence artificielle , je réformerai l' assurance chômage . Je vais libérer l' activité de tous les secteurs qui ont été bloqués par le matraquage réglementaire et fiscal : l' agriculture , l' industrie , les services , la construction . Nous effectuerons une marche de fond pour améliorer tous les facteurs de production , du travail à l' innovation en passant par le financement . Je veux plus de confiance pour ceux qui font tourner les moteurs de l' économie française . Je préfère valoriser les investisseurs français plutôt que de ramper devant les fonds de pensions étrangers et ceux de pays soupçonnés d' aider les totalitaires islamiques . La meilleure politique sociale c' est celle qui donne du boulot ! Je veux agir pour les jeunes qui se cassent les dents sur la rigidité du code du travail . Agir pour les ouvriers qui préfèrent travailler un peu plus que de ne plus travailler du tout . Et cette restauration de notre production permettra de distribuer les richesses créées au lieu d' égaliser la misère comme aujourd'hui . Mais tout cela ne marche que si l' Etat , en parallèle , fait des efforts d' organisation et de gestion . Salariés , indépendants , fonctionnaires , on est tous sur le même navire . Pour le remettre à flot , il faut s' y mettre ensemble . Je veux moderniser les fonctions publiques pour les rendre plus efficaces et pour améliorer leurs conditions de travail . Passer progressivement nos fonctions publiques à 39 heures pour qu' elles soient plus efficaces au service des Français , c' est raisonnable , c' est du bon sens ; c' est au surplus redonner à nos fonctionnaires des perspectives de carrière . Je préfère un Etat fort , avec des agents moins nombreux , plus motivés et mieux rémunérés qu' un Etat suradministré , croulant sous les déficits . Nous avons près de 6 millions d' agents publics , c' est plus que l' Allemagne qui compte pourtant plus d' habitants que nous . Réduire notre nombre d' agents publics de 8 % sur un quinquennat pour réduire nos déficits , c' est raisonnable , c' est du bon sens . Je vois les gens débattre à l' infini sur ce chiffre et je suis surpris . Mon but , ce n' est pas de réduire des postes pour le plaisir . Mon but , c' est de rendre efficaces nos structures publiques souvent vieillies , empilées les unes sur les autres … Je suis l' ennemi de la bureaucratie , mais je ne suis pas celui des fonctionnaires ! Il y a peu de tâches aussi nobles que celles du professeur , de l' infirmière , du magistrat , du soldat , du policier . Je respecte ceux qui ont voulu , lorsqu' ils étaient jeunes , que l' intérêt public leur serve de boussole . C' est à eux aussi que je pense . Le discrédit de l' Etat bureaucratique rejaillit sur eux et c' est profondément injuste . Je leur rendrai leur fierté . Toute cette stratégie productive me distingue de mes concurrents . A gauche , on rêve des 32 heures et d' un revenu universel : c' est l' assistanat universel . Ces gens-là ont enterré nos traditions ouvrières . Ils n' ont jamais mis un pied dans une entreprise agricole ou une start-up . Sont -ils allés voir ce qu' il en était à la Silicon Valley ? J' en doute ! Face au camp de l' anesthésie , je suis le candidat du travail et du pouvoir d' achat , car un pays qui travaille plus , gagne plus . La fin des 35 heures dans le privé , le passage aux 39 h dans le public , l' augmentation du salaire direct grâce à la baisse des charges , le relèvement du quotient familial pour les familles : tout ceci va améliorer le pouvoir d' achat des Français , notamment celui de nos concitoyens modestes . En réalité , à gauche , on ne croit plus à la croissance . Sans croissance , il ne reste donc plus qu' à partager les miettes . Cette capitulation n' est pas seulement démoralisante pour tous ceux qui travaillent dur pour améliorer leur vie , elle est aussi destructrice pour notre pacte social . J' ai quelques scrupules à rappeler au parti socialiste et à son prochain candidat , que c' est la croissance par le travail qui finance notre protection sociale . Moi je veux le meilleur pour la solidarité , et pour ce faire je veux nous en donner les moyens en agissant , pas en rêvant . Mon ambition sociale , c' est une retraite digne assurée à tous nos anciens . Pour cela il n' y a pas d' autre choix que d' aller progressivement vers la retraite à 65 ans . Nous vivrons tous plus longtemps – en moyenne au-delà de 80 ans – il est donc raisonnable de cotiser un peu plus longtemps pour assurer nos vieux jours . Mais il n' est pas possible qu' en France nos anciens les plus modestes en soient à renoncer à leurs repas pour payer leurs factures . Mon gouvernement avait beaucoup fait pour les petites retraites . Je poursuivrai l' effort avec une revalorisation de + de 300 euros pour les pensions de moins de 1000 euros et d' au moins 600 euros pour les petites pensions de réversion . On m' a fait le procès de vouloir mettre à mal notre assurance maladie et notre système de soins . C' est faux ! Je veux une santé de qualité accessible à tous . Ce que je dénonce , c' est la langue de bois de ceux qui se fichent des déserts médicaux , qui se fichent des pathologies non remboursées , qui se fichent des urgences débordées , qui se fichent de savoir que les personnels médicaux sont à bout , qui se fichent de savoir que la médecine libérale en a marre d' être maltraitée , et puis , qui se fichent des déficits . Les fossoyeurs de la sécurité sociale sont ceux qui s' opposent à toute réforme . Tout comme d'ailleurs les fossoyeurs de l' emploi sont ceux qui ne veulent pas toucher à la durée du travail ou à son coût . Je veux une santé pour tous , une santé capable de répondre au vieillissement de la population et au prix croissant des nouveaux médicaments . Et pour cela , j' agirai , en concertation avec les professionnels de la santé . Et puis , cet effort productif auquel je convie la nation doit permettre de répondre aux causes les plus humaines , les plus justes . Celle de l' autisme , celle du handicap . 8 millions de Français sont frappés par le handicap . Ce fut une des batailles du Président Chirac . Ce sera mon combat . Je pense à ces familles qui doivent être soutenues à domicile , à ces enfants qui doivent pouvoir être scolarisés comme les autres , qui doivent pouvoir être pleinement respectés . J' aime la phrase de Saint Exupéry : &laquo; Si tu diffères de moi , mon frère , loin de me léser , tu m' enrichis &raquo; . Nous venons de célébrer l' anniversaire de la mort de l' Abbé Pierre . Sommes -nous au rendez-vous de son appel au secours de l' hiver 1954 ? La pauvreté continue de s' étendre , le chômage fait ses ravages . J' ai rencontré beaucoup d' associations . Leur message fut souvent le même : nous avons besoin de clarté car le système est trop bureaucratisé ; nous avons besoin de continuité car les aides de l' Etat sont en accordéon ; nous avons besoin de liberté pour coller aux réalités du terrain . J' engagerai un plan sans précédent d' action pluriannuel de lutte contre la pauvreté et l' exclusion pour répondre à ce message . Mes amis , Il faut renforcer l' unité française . La France unie attachée à une certaine idée de la famille , a été dédaignée . Je la protégerai et lui redonnerai ses droits , car la famille est l' un des piliers de notre société . Elle est le premier cercle de la tendresse , elle n' est pas le laboratoire d' un Etat qui prétend prendre la place du papa ou de la maman . Je relèverai le quotient familial et favoriserai les donations aux enfants . La France unie se joue à l' Ecole de la République . Elle doit transmettre les savoirs fondamentaux autour desquels nous devons concentrer les efforts plutôt que de multiplier les activités périphériques . L' Ecole , préparant à un métier avec une valorisation massive de l' alternance . L' Ecole de la République , symbolisée par le port d' un uniforme scolaire donnant à tous les élèves , quelles que soient leurs origines , le goût d' être ensemble , tous égaux . Notre école est mal en point . Je suis en colère quand je vois son niveau baisser d' année en année dans les classements internationaux . Je suis en colère quand j' entends la gauche et les pédagogues qui persévèrent dans l' erreur et l' autosatisfaction . Pour l' apprentissage des mathématiques nous étions les premiers il y a vingt ans . Aujourd'hui c' est la Corée du Sud , Taiwan , Hong Kong , Singapour qui font la course en tête . Nous sommes en queue de peloton de l' Europe . Où seront nos ingénieurs , et quel sera leur niveau ? Et quel sera le rang de la France ? La gauche a fait passer l' idéologie avant le savoir . Elle a fait passer l' endoctrinement avant l' intelligence . Elle s' est attaquée à tout ce qui était synonyme d' excellence . Je comprends les enseignants qui ne se reconnaissent plus dans ce système opaque qui les traite comme des rouages d' une idéologie fumeuse . François Hollande nous avait fait miroiter le rêve français . Il nous a laissés le chaos français . Le chaos français , c' est , en plein état d' urgence , des forces de l' ordre mobilisées pour encadrer le camping sauvage de Nuit debout . C' est la construction d' un aéroport à Nantes toujours bloquée par des hors la loi . Ce sont des voitures de police incendiées au cocktail Molotov , des pompiers interdits d' accès dans certains quartiers . Ce sont des enseignants qui ont la boule au ventre en entrant dans leurs classes . C' est un pays où tant de peines de prison ne sont plus effectuées . C' est un pays où l' on invite les élèves à négocier leurs fautes d' orthographe . C' est un pays où tout se relâche , où les sanctions ne veulent plus rien dire , où il n' y a plus d' autorité . Je veux remettre de l' ordre dans le chaos français . La France unie exige l' autorité de l' Etat , l' autorité de la loi , la nécessité de l' ordre . Il faut mettre les mots sur les actes . Et je rappelle ce que j' ai déjà dit puisqu'il faut bien que ceux qui se jouent des lois l' entendent cinq sur cinq . Un délinquant n' est pas un mineur qui se cherche . C' est un délinquant ! Un criminel n' est pas une victime . C' est un criminel ! La sécurité doit être partout et pour tous , avec des forces de l' ordre modernisées et une Justice ferme et rapide . Je veux rétablir les peines planchers pour les récidivistes . Un délit : une sanction ! Voilà la règle qui doit être parfaitement claire pour les délinquants . La France unie n' est pas une addition de communautés et de clans . Elle a une identité , des coutumes , une culture , un drapeau . Le repli communautariste est un danger . Je me félicite que pascal Bruckner ait gagné son procès contre ceux qui veulent limiter notre liberté d' expression . Je serai le défenseur di droit des français à exprimer leur francité ! L' immigration sans intégration , sans assimilation , doit être stoppée . Dans la République française , les étrangers ont des devoirs avant de réclamer tous les droits . C' est une question d' unité nationale . Mais c' est aussi une question de respect car quand on entre dans la maison d' autrui , on n' y impose pas ses lois ! Je veux réduire l' immigration à un strict minimum , en l' organisant par quotas , en fonction de nos besoins économiques et de nos capacités d' intégration . A ceux que nous avons choisi d' accueillir , je veux que nous disions , avec précision et clarté , ce que nous exigeons d' eux : exercer un travail , s' approprier notre langue , respecter nos règles communes . Aux vrais réfugiés politiques , l' honneur commande que la France tende sa main . Mais avec les clandestins , elle doit être intransigeante car le droit de l' asile est sacré . Je veux que nous mettions les Etats d' origine en face de leurs responsabilités , pour qu' ils prennent leur part de la maîtrise des flux migratoires et qu' ils s' impliquent dans le retour de leurs ressortissants . Je conditionnerai l' aide au développement à leur coopération . L' Islam radical mine nos concitoyens musulmans . Il les infiltre , il les prend en otage . Les fanatiques détestent ce que nous sommes . Je les combattrai sans relâche et sans pitié . Je veux un strict contrôle administratif du culte musulman , tant que son ancrage dans la République ne sera pas pleinement achevé . Je veux la dissolution immédiate de tous les mouvements qui se réclament du &laquo; salafisme &raquo; ou des Frères musulmans . Je veux la clarification de nos relations avec l' Arabie saoudite et le Qatar qui abritent les doctrinaires de l' Islam radical et les marchands de &laquo; burkini &raquo; . Je nomme les choses . Je nomme le problème que nous avons avec l' Islam radical . Le catholicisme , le protestantisme , le judaïsme ne dénoncent pas les valeurs de la République . Après bien des affrontements , ces religions vivent en paix avec la République , avec la laïcité qui est notre protection commune . J' entends dire qu' il faudrait durcir les règles de notre laïcité . Mais pourquoi faudrait -il imposer à tous de nouvelles restrictions à la liberté religieuse , alors que seule la poussée intégriste qui enflamme le monde musulman est une menace pour notre société ? Ce n' est pas ma vision des choses . Ma vision des choses est très claire : la religion musulmane doit accepter sans condition tout ce que les autres religions ont accepté par le passé ! Je prendrais mes responsabilités mais il faut aussi que nos concitoyens musulmans se soulèvent contre ceux qui instrumentalisent leur foi . C' est une bataille idéologique et spirituelle qu' il faut livrer tous ensemble . Je dis &laquo; spirituelle &raquo; , parce que ça n' est pas en évacuant toute transcendance qu' on peut combattre la croyance ravageuse des fanatiques . Et ça n' est pas en laissant tomber lâchement les Chrétiens d' Orient qu' on créera les conditions de la paix en Orient . Dans les camps de réfugiés que j' ai visités en Irak , la prière des chrétiens et celle des musulmans se confondaient . Il n' y a aucune honte à se dire chrétien , aucune honte à se dire musulman , aucune honte à se dire athée ou agnostique . Ce qui importe c' est le respect , le respect de la loi et le respect de la France . Autour de nous , le monde est traversé de menaces . J' appelle de mes vœux une France souveraine , appuyée par des forces militaires dont les moyens seront augmentés . Ma vision des relations internationales est commandée par les intérêts de notre pays et par eux seuls . Quel que soit celui qui dirige le Kremlin à Moscou , et quel que soit celui qui dirige la Maison Blanche à Washington , l' amitié n' a rien à faire quand on défend la place de la France dans le monde et la sécurité des Français . Dans ce monde , les dirigeants passent , les régimes évoluent , les nations demeurent . La France n' y tiendra son rang que par une politique d' indépendance nationale . Elle doit être libre . Libre de combattre le totalitarisme islamique avec tous les Etats volontaires sans aucune exclusive . Libre de dire non aux Etats-Unis quand ils imposent leurs lois au mépris du droit international à nos entreprises . Libre de parler avec la Russie avec franchise . Certes le président Russe a commis des fautes , mais il ne finance pas les djihadistes qui tuent nos enfants et ne nourris pas le projet d' imposer la charia au reste du monde dans un califat de cauchemar . Evidemment , pour cela , il faut un vrai président de la République . La souveraineté française ne peut être incarnée par un homme qui navigue à vue dans son propre pays . Avec François Hollande , la France a payé le prix fort pour apprendre qu' on n' existe pas si on ne sait pas ce qu' on veut , ni où l' on va . L' Europe doit être pour la France un amplificateur de souveraineté . Je ne parle pas de l' Europe technocratique avec ses normes byzantines . Je parle de l' Europe des nations , unies pour défendre la civilisation européenne . Je me bats pour une Europe politique concentrée sur ses priorités stratégiques . Une Europe appuyée sur l' euro , qui doit devenir un atout entre les mains d' un gouvernement économique européen . Je milite pour une Europe qui tient ses frontières , pas d' une Europe passoire . Une Europe de l' innovation et du numérique qui ferait des européens la 1ère puissance technologique du monde . Je veux une Europe qui défend ses industries , ses emplois , face à la Chine , face aux Etats Unis . Je défendrai la préférence européenne pour nos industries , nos services et nos emplois . Dans les échanges commerciaux , je suis pour la réciprocité , pas pour la naïveté . Y compris entre nous , européens . La directive sur les travailleurs détachés a installé un véritable dumping social dans nos nations . Pour un travail égal , le salaire doit être égal et les cotisations sociales aussi . Si je suis élu , je renégocierai de fond en comble la directive sur les travailleurs détachés pour faire respecter ce principe . Si la négociation n' aboutit pas avant la fin de l' année , je suspendrai son application sur le territoire national . Donnons force à une Europe de la culture . J' ai lancé l' idée d' un grand musée à Strasbourg , exposant les chefs d' œuvre européens . La culture irrigue la vie , l' Europe a besoin de vie ! Je veux une Europe de la défense , dans laquelle chaque Etat , y compris l' Allemagne , prendra sa part du fardeau de l' effort militaire . Je veux &laquo; l' Europe mais avec la France debout &raquo; disait Philippe Séguin . Cher Philippe , aujourd'hui l' une et l' autre sont à terre et j' enrage de voir la civilisation européenne douter de son sort au milieu des orages . La France doit être redressée ; l' Europe , il faut la reconstruire et y parler d' une voix claire ! Voilà , mes amis , notre ambition française . La bataille électorale sera difficile . La violence des attaques qu' on nous lance est à la mesure des enjeux . Notre adversaire n' a pas un visage … il en a quatre ! Il y a plus de cinquante ans André Malraux disait : il y a quatre gauches , dont l' extrême-droite . Il avait raison . Nous y sommes . La gauche n°1 , la gauche pure et dure , rouge de chez rouge , on la connaît . C' est Mélenchon , le Fidel Castro de YouTube . La gauche n°2 , on la connaît aussi , c' est la gauche socialiste , c' est tout l' équipage des naufragés du Titanic hollandais qui viennent de s' entredévorer sur le radeau de la Méduse des primaires . Vous les avez entendus , peut-être … Ils nous expliquent qu' il faut aller toujours plus loin à gauche . Plus la réalité les met en déroute , plus ils ont foi dans leurs dogmes . Ils promettent des milliards de dépense . Ils proposent de travailler moins et de rêver plus Ils n' ont rien compris . Ils n' ont rien appris . La gauche n°3 , on la connaît moins . C' est Macron . Il dit avoir un projet ; je l' attends ! Il dit être réformateur ; à priori , il l' est moins que moi ! Il fait croire qu' il est seul et qu' il vient de nulle part ; en réalité , il a fait le programme de Monsieur Hollande mais aussi une grande partie de sa politique … Il est parti pour une échappée en solitaire , mais ses équipiers ne sont pas loin . Qui sont -ils ? Eh bien c' est toute l' équipe gouvernementale de monsieur Hollande … Bonjour la nouveauté ! Macron c' est le sortant , Macron c' est le bilan de Hollande . Macron c' est surtout le prototype des élites qui ne connaissent rien à la réalité profonde de notre pays ! Et puis il y a la gauche n°4 … C' est le Front National . Sortie de l' euro , abaissement de l' âge de la retraite à 60 ans , augmentation du salaire minimum , recrutement illimité de fonctionnaires : le programme du FN , c' est de la dépense publique à n' en plus finir , c' est le programme du parti communiste des années 70 ou celui du Front de gauche ! Pendant des années le FN a dit qu' il fallait préférer l' original à la copie . Eh bien , aujourd'hui , l' original c' est Jean-Luc Mélenchon et la copie c' est Marine Le Pen ! C' est la vieille politique de l' endettement irresponsable , des chèques en bois . C' est une politique de la ruine , ruine pour la France et pour les Français , pour les petits épargnants qui verront leurs économies partir en poussière , pour les petits patrons qui verront leurs charges sociales monter en flèche , pour toute notre économie qui sera attaquée de toutes parts par les nations du monde entier qui n' attendent que notre abaissement pour prendre notre place . Être une grande nation , ce n' est pas être isolé du monde , c' est avoir une position éminente dans la hiérarchie des puissances planétaires . C' est être capable de soutenir la comparaison . Le FN refuse toute comparaison et toute compétition mais l' histoire du monde est un combat et faire cavalier seul c' est la garantie de descendre en deuxième division . Je ne veux pas d' une France en deuxième division . Je veux une France qui fasse la course en tête . La vérité , vous la connaissez . Le parti du mouvement , c' est nous . Les conservateurs , c' est eux ! Ne nous laissons pas arrêter par la grande conjuration des impuissants . La France n' est pas à leur image . Elle n' a pas besoin de béquilles . Pas besoin de ceintures et de bretelles . Elle a l' énergie qu' il faut pour réussir seule . Nous avons besoin à la fois d' ordre et de mouvement . L' ordre , c' est l' attachement à nos valeurs , à nos principes , à ce qui nous constitue comme nation comparable à nulle autre . Le mouvement , c' est l' opportunité de créer , de jouir des fruits de son travail , d' innover , d' inventer des façons d' agir et de s' organiser . Je veux une société de nouvelles libertés où chacun dans son entreprise , son administration , son association se sent davantage maître de sa vie et plus utile aux autres . Vous voyez que nous sommes très loin de la purge ou des programmes technocratiques . Nous devons toujours garder à l' esprit le but ultime de nos réformes . C' est une France sauvée de l' éclatement , de la crise morale et du désastre économique . Une France qui respire l' initiative , qui retrouve sa voix et son image . Une France qui , s' étant réconciliée avec elle-même réconciliera tous les Français . Je m' attendais à voir monter en ligne les spécialistes de l' enfumage . Mais les Français connaissent la réalité . Ils savent qu' au fil des années , des abandons , des compromis , cette réalité est devenue inacceptable . La France qui nous a été léguée , celle que nous avons le devoir de transmettre , ça n' est ni un musée de l' histoire , ni une agence des droits . C' est une tradition du mouvement , de la conquête . C' est la langue même de l' esprit et de la liberté . Nous sommes tous invités à la parler , cette langue . On disait autrefois : &laquo; franc de nom , franc de nature &raquo; . Nous sommes le pays qui rend libre dans un monde où le plus souvent cette liberté n' a plus cours . C' est notre honneur . D' où qu' il vienne , chaque Français détient une part de cet honneur-là . Mes amis , maintenant je vais relever avec vous le défi du redressement national . Ce défi , je le gagnerai si toutes celles et tous ceux qui en ont assez de la torpeur et des mensonges se lèvent à mes côtés . Je donne rendez-vous à toutes celles et tous ceux qui ont la tête sur les épaules . J' appelle tous ceux qui ont des valeurs solides . J' appelle tous ceux qui ont dans le cœur la fierté d' être français . Sur le chemin de la victoire il y aura des vents contraires ; nous passerons à travers les bourrasques ; nous franchirons tous les ponts . Vous êtes partant et je le suis aussi , alors j' irai , avec vous , chercher les cœurs vaillants . Nous irons ensemble à la rencontre de toutes les forces de la France . Vous serez en première ligne de mon combat pour le redressement national . Cette histoire sera la vôtre . Vous allez vous engager à 100 % ! Votre énergie deviendra celle de votre ami , celle de votre ami deviendra celle de son compagnon , et de compagnon en compagnon , de cœur en cœur , de volonté en volonté , nous irons droit devant , droit vers la victoire , droit vers la grandeur , droit vers les Français , et nous les entrainerons , nous les emmènerons , et nous serons alors , tous ensemble , la France en action ! Vive la France et vive la République . ",

    "Mes chères concitoyennes , mes chers concitoyens , À se laisser guider par l' air du temps , on pourrait croire que le cœur de la France s' est arrêté . Ce &laquo; qui marche politiquement &raquo; exalte les peurs instinctives , le rejet de l' autre ou le repli sur soi . Ce n' est pas le regard que je porte sur la France . Je lui vois un cœur robuste . Le problème de la France , ce n' est pas que son cœur se soit arrêté , c' est qu' il ne gouverne plus . Qu' il gouverne enfin à nouveau : c' est l' alternative de gauche , résolue et optimiste , que je porte . Voici le fruit du travail collectif que nous avons mené ces derniers mois . Cette réflexion se poursuivra pour aboutir à d' autres propositions . N' hésitez pas à y contribuer à votre tour . À portée de main , à portée de vote , existe un futur réaliste , où le travail n' est pas facteur de souffrance mais d' émancipation , où notre modèle de production intègre réellement l' exigence écologique et où la démocratie retrouverait le souffle qu' elle a perdu . Ensemble , je vous propose de faire battre à nouveau le cœur de la France . ",
    
    "Les écologistes viennent de me désigner comme leur candidat à l' élection présidentielle . Je veux les en remercier chaleureusement . Je veux également remercier Michèle Rivasi , Cécile Duflot et Karima Delli pour la qualité de cette primaire . Ces dernières semaines , nous avons su démontrer notre respect mutuel et la force du projet que nous portons . Ensemble , avec les militantes et les militants d' Europe Ecologie les Verts , et avec toutes les personnes qui croient en l' écologie politique , nous allons construire la campagne présidentielle comme une aventure collective . Pendant la primaire , j' ai répété ma volonté de rassembler toute les familles de l' écologie : je veux poursuivre cette dynamique . C' est unie , ouverte , en regardant vers le grand large et les pieds sur terre que l' écologie sera conquérante . Je veux m' adresser ce soir , et tout au long de la campagne , à celles et ceux qui agissent chaque jour pour la planète et pour une société plus juste , à celles et ceux qui aimeraient agir mais qui n' en ont pas les moyens ou ne savent pas comment faire , à celles et ceux aussi qui ont pu être déçus par Europe Ecologie les Verts . Je suis déterminé à ce que cette campagne soit aussi la vôtre , que vous puissiez pleinement y participer , qu' elle ouvre une nouvelle page de l' écologie dans notre pays . Je mesure la responsabilité qui m' est confiée . Je la reçois avec gravité et détermination , mais aussi beaucoup d' enthousiasme . Avec gravité , car la France est au bord de la rupture . L' accroissement des inégalités , l' explosion des discriminations , l' affaissement de la démocratie , le dérèglement climatique , l' extinction des espèces , l' épuisement des ressources , les maladies liées aux pollutions : tout cela affaiblit notre pays . Nous souffrons de ne pas avoir de projet partagé et , faute d' horizon commun , la société explose dans un sauve-qui-peut généralisé . Jamais notre pays n' a connu une telle défiance , jamais la tentation du repli n' a été aussi forte . J' accueille aussi cette désignation avec détermination . Aujourd'hui s' ouvre à Marrakech la COP22 , une nouvelle conférence mondiale sur le climat . Si la diplomatie française peut être fière de l' Accord de Paris signé lors de la COP21 , chaque jour le gouvernement tourne le dos aux devoirs que cet accord impose . Obsession nucléaire , maintien des centrales charbon , choix du tout camion , diesel , soutien à l' agriculture la plus industrielle : ces choix maintenus depuis des décennies ne sont pas seulement néfastes pour le climat et pour la santé , ils nous privent de centaines de milliers d' emplois et d' entreprises dans tout le pays , maltraitent les animaux et font disparaître les services publics . Je reviens de Nantes où sont jugés les recours sur l' eau et les espèces protégées , déposés par les écologistes , contre les autorisations de travaux à Notre Dame des Landes . Que de temps perdu et d' entêtement autour de ce projet aussi inutile que destructeur -et toujours illégal au regard du droit européen . François Hollande confiait à des journalistes ne pas croire en ce projet : saisira -t-il enfin l' occasion d' y mettre fin , comme nous le demandons depuis des années ? Avec détermination donc , car , oui , les solutions sont là . Je crois en notre intelligence collective . La France n' est condamnée ni au désespoir , ni à la peur du déclassement . Il y a urgence à réformer les institutions pour déverrouiller la démocratie , à faire sauter les rentes pour libérer l' économie , à surmonter les conservatismes et les corporatismes pour débloquer la société . Ca tombe bien , notre pays regorge d' incroyables forces vitales , dans les quartiers , dans les villes et les campagnes , dans les entreprises et les associations . Nous devons absolument soutenir ces bâtisseurs d' avenir et leur donner les moyens d' essaimer . Dans cette campagne , nous déclinerons nos propositions pour faire face aux urgences écologiques , démocratiques et sociales . Pour prendre soin de la planète , nous devons réparer la société , la soigner . Protéger la nature , c' est nous protéger nous-même , c' est nous offrir une perspective d' avenir . Une France 100 % énergies renouvelables , une agriculture paysanne et biologique , une alimentation saine et conviviale , une santé protégée , des transports collectifs accessibles , une fiscalité juste qui favorise l' innovation , des choix industriels qui créent des emplois plutôt que d' en détruire … Car justement , l' écologie , c' est plus d' emplois , moins d' inégalités et de précarités . L' écologie c' est des territoires vivants et une économie relocalisée . Alors que des millions de personnes souffrent de ne pas avoir d' emploi , trop de salariés souffrent dans leur travail . Intensification des tâches , dégradation des conditions , précarisation des contrats : nous proposerons des mesures pour améliorer le travail , l' adapter aux capacités et aux talents des femmes et des hommes , le partager , et en garantir les droits . Nous voulons remettre le travail à sa juste place dans nos vies . Nous mettrons en débat notre projet pour une école égalitaire et émancipatrice . Nous voulons donner le pouvoir aux citoyennes et aux citoyens dans une République refondée . Nous affirmons qu' une société de confiance ne peut émerger sans le respect plein et entier du pacte républicain , des droits qu' il devrait garantir et des promesses qu' il porte pour chacune et chacun , arrivé hier dans notre pays ou présent depuis Astérix , habitant d' un quartier populaire ou d' une ferme du Larzac . Liberté , égalité , fraternité . La démocratie , c' est le contrôle de l' Etat par les citoyens , pas l' inverse . Le fichier global d' informations sur la population est une dérive de plus qui porte atteinte aux libertés publiques , donc à la démocratie . Nous lutterons contre toutes les discriminations qui minent la société , qu' elles soient sociales , ethniques , liées à l' orientation sexuelle , aux handicaps ou à l' état de santé , aux opinions politiques , religieuses ou syndicales . Et il est plus que temps de mettre fin aux inégalités entre les femmes et les hommes . L' écologie , c' est une société apaisée , fraternelle , ouverte sur sa diversité et sur le monde . Osons dire que nous voulons une société qui ne soit pas une compétition perpétuelle , pour avoir le droit de vivre dignement et de nous épanouir . Osons réclamer le droit au bonheur . Evidemment , la France doit se redonner un horizon européen . Cette petite mondialisation libérale qu' est devenue l' Union Européenne , technocratique et austéritaire , qui dresse des murs et des barbelés contre les réfugiés , porte en elle le risque de l' effondrement de notre civilisation alors qu' elle devrait en être le ferment . Le Tafta et le Ceta ne sont pas des solutions , mais font partie du problème . Relançons massivement l' investissement utile pour tisser de nouvelles solidarités sur notre continent , entre l' Est et l' Ouest , le Nord et le Sud , au sein de nos pays et avec le monde qui nous entoure ! La France , quant à elle , doit enfin nouer des relations saines avec les pays du Sud . Ni nostalgie régressive de la colonisation , ni Françafrique qui perdure comme au Gabon . Solidarité , soutien à la démocratie , intérêts mutuels pour un développement soutenable ! Enfin , je mènerai cette campagne avec enthousiasme . Je veux rassembler les Françaises et les Français autour d' un projet positif , un projet de sens et de cohérence ! Je veux qu' ensemble nous passions un nouveau pacte avec les Françaises et les Français , une nouvelle alliance : celle d' une écologie à la hauteur des défis de la planète et au cœur de la vie quotidienne , une écologie crédible , fière de ses combats et de ses convictions , aimable , populaire et humaniste , ni culpabilisante , ni punitive , mais aimable et bienveillante . Nous voulons redonner à chacune et à chacun sa dignité , la maîtrise de sa vie , l' envie et le pouvoir d' agir . Nous donnerons à voir de nouveaux possibles à celles et ceux qui n' attendent qu' un signe pour retrouver l' espoir , qu' une occasion pour réinvestir l' action collective et politique . Toutes et tous ne se disent pas &laquo; écologistes &raquo; , mais peu importe : ils et elles sont prêts à agir si on leur en donne les moyens , s' ils reprennent possession des leviers du changement . Le récit de l' écologie que je vous propose de porter ensemble pour l' élection présidentielle est profondément subversif , exaltant et pragmatique : mobiliser les énergies vitales de la société pour nous réconcilier avec la nature et avec l' avenir , pour nous réconcilier entre nous , pour nous réconcilier avec nous-mêmes ! La campagne des écologistes commence ce soir . Face aux artisans du renoncement , aux adeptes du &laquo; c' était mieux avant &raquo; et aux prêcheurs de haine , nous allons semer l' espoir d' une société de confiance . Je suis candidat pour que demain soit mieux qu' aujourd'hui , avec vous , et grâce à vous . ",
    
    "Chers amis , Merci à tous d' être venus si nombreux à cette nouvelle convention . Merci également à tous les intervenants pour leurs réflexions très intéressantes et leur contribution au débat sur ce thème majeur qu' est la santé . Thème majeur , en effet , puisque la santé est une des rares problématiques qui nous concerne tous , sans exception . Cela dit , si nous sommes tous concernés , nous ne sommes malheureusement pas tous égaux face à la maladie et sa prise en charge . Les inégalités de santé s' accroissent année après année : entre les hommes et les femmes , entre les jeunes et les moins jeunes , entre les Français les plus favorisés et nos concitoyens en situation de précarité , entre les citadins et les ruraux . L' hôpital public est au bord de la faillite , la dette des hôpitaux a triplé en dix ans , souvent par le biais d' emprunts toxiques . Les soignants sont soumis à des conditions de travail déplorables , et plus un mois ne se passe sans qu' un journal local rapporte le suicide d' un soignant hospitalier . L' hôpital qui voit peser de surcroît sur lui le poids d' une immigration de plus en plus lourde à assumer . Ne parlons pas des urgences , encombrées de patients mal orientées , qui ne nécessitent pas une prise en charge hospitalière , et où dans plus de la moitié des cas , l' attente est supérieure à deux heures . N' oublions pas qu' il y a un peu plus d' un mois , Marisol Touraine annonçait un plan d' économie de 1,5 milliard d' euros sur les hôpitaux … autant dire que la Ministre achève le malade . La médecine de ville , quant à elle , attire de moins en moins les jeunes médecins , en raison de charges financières et administratives croissantes et d' une augmentation du coût de la vie plus rapide que celle des tarifs conventionnés . Les décisions arbitraires du ministère , imposées sans concertation avec les acteurs de terrain , finissent par exaspérer des professionnels , qui ont l' impression qu' on essaie de les court-circuiter . Côté patients , l' accès aux soins s' est détérioré depuis la crise de 2008 , et les inégalités de santé ne cessent de se creuser . Les Français les plus défavorisés subissent une double peine sanitaire : non seulement ils vivent moins longtemps que nos compatriotes les plus favorisés , mais ils vieillissent également en plus mauvaise santé ; alors qu' un cadre peut espérer passer 9 ans de sa retraite en bonne santé , un ouvrier n' atteindra en moyenne pas la retraite sans trouble de santé . A tous les niveaux , les conséquences néfastes de décennies de politiques de santé court-termistes et sans vision stratégique se font sentir , au détriment des patients et des professionnels de santé . UMP , PS , même bilan , au final ; mais qui en doutait ? Et ce que certains envisagent pour l' avenir me laisse coite … Autant vous dire qu' à lire le projet de M. Fillon , j' en ai la nausée … et j' espère que celle -ci passera vite , parce qu' il semble que les troubles digestifs fassent partie du &laquo; petit risque &raquo; que l' Assurance Maladie ne devrait pas avoir à prendre en charge , selon Dominique Stoppa-Lyonnet , conseillère santé de l' ex-premier ministre . Oui , vous avez bien entendu . Le point principal du projet de François Fillon pour la santé est de cantonner l' Assurance Maladie aux pathologies graves ou de longue durée , le reste devant être délégué aux assurances complémentaires . Le docteur Fillon nous prescrit donc un modèle ultralibéral à l' américaine , où les riches auront les moyens de se soigner correctement , où les pauvres bénéficieront de soins de piètre qualité en dernier recours , et où les familles et les classes moyennes seront une fois de plus les grandes sacrifiées . Toujours plus loin dans l' indécence : l' équipe Fillon préconise également de sortir la grippe , l' optique , le dentaire et les prothèses auditives du panier de soins pris en charge par la Sécurité Sociale . Or , ce sont justement ces domaines qui représentent l' essentiel des renoncements aux soins , qui , je le rappelle , touche plus d' un Français sur quatre . Cette &laquo; désétatisation &raquo; , comme il le dit lui-même , n' est rien d' autre qu' une privatisation , qui entrainera pour une famille avec deux enfants un surcoût de plus de 100€ par mois , selon la Mutualité française . Pour un homme qui parle de la famille dans ses discours de campagne , il reste de très sérieux efforts à faire … notamment à l' heure où le taux de pauvreté ne cesse de croître : on dénombre aujourd'hui 9 millions de Français vivant sous le seuil de pauvreté . Alors , bien entendu pour rassurer , on promet une couverture accrue – sans en préciser les contours – aux plus défavorisés , qui au final devront tout de même souscrire un contrat d' assurance complémentaire , dont les prix auront explosé au vu du transfert des remboursements depuis l' Assurance Maladie . En somme détruisons la Sécurité Sociale et subventionnons indirectement les assurances complémentaires , déjà grassement aidées … Non seulement les organismes proposants des contrats dits &laquo; responsables &raquo; bénéficient d' avantages fiscaux , mais par la volonté de Marisol Touraine , les remboursements dans le cadre de ces fameux contrats sont plafonnés depuis le 1er janvier ! Merci Madame la Ministre ! Vous avez voulu de manière dogmatique frapper les professionnels de santé libéraux , manque de chance c' est sur les patients qui vous avez tapé : +100 % de reste à charge en moyenne à l' hôpital , et jusqu' à +130 % pour les consultations chez un spécialiste ! Ma première proposition est d'abord un engagement de fond : il s' agit de garantir à chaque Français un accès aux soins optimal , grâce à un système de Sécurité sociale qui respecte le principe fondamental de solidarité nationale . Ainsi , je m' engage à ne pas réduire la prise en charge des soins par l' Assurance maladie et je m' oppose fermement à la privatisation des dépenses de soins . Je m' engage pour une santé de qualité , qui ne laisse pas sur le bord de la route les classes populaires comme les classes moyennes , les familles comme les personnes âgées . Une Santé qui écoute des professionnels , ses médecins , des infirmiers , ses pharmaciens . Et ceux qui voudront détruire , privatiser notre Sécurité Sociale me trouveront sur leur chemin . L' un des problèmes majeurs rencontrés par les Français en matière de santé est la difficulté de l' accès aux soins , qui s' est déjà fortement dégradé et qui ne devrait pas s' améliorer au cours des mois à venir . Pis encore , les nombreux départs à la retraite attendus au cours des prochaines années laissent entrevoir une pénurie de médecins , notamment généralistes , alors que les besoins augmentent du fait du vieillissement de la population . C' est pourquoi il m' apparait capital de recruter plus de médecins : ainsi je relèverai dès notre arrivée au pouvoir le numerus clausus de manière à tenir compte de la réalité du terrain , tout en donnant aux facultés les moyens de former ces étudiants supplémentaires . Pour sensibiliser les jeunes médecins à la question des déserts médicaux , je propose d' instaurer un stage d' internat dans une zone désertifiée . Il faut en effet tout entreprendre pour mettre fin aux déserts médicaux , ces zones de non-soins qui ne sont pas admissibles . De plus , il est crucial de redonner le goût de l' exercice libéral , qui n' attire presque plus personne en raison des contraintes qui se surajoutent année après année . Aujourd'hui , M. Fillon juge qu' il est nécessaire de relever le numerus clausus pour éviter le recours à des médecins étrangers moins bien formés , or il ne l' avait relevé que de manière tout à fait dérisoire lorsqu' il était aux manettes ; force est de constater que son gouvernement n' avait pas pour priorité de garantir la pérennité de notre système de santé . Pourquoi s' en soucierait -il demain ? Niveau économies , je ne suis pas certaine non plus que les 19 millions de vaccins contre le virus H5N1 surnuméraires détruits en 2011 soient un exemple de bonne gestion … Quid également de l' aide médicale d' Etat , qu' il promet aujourd'hui de réduire , alors qu' il a eu déjà eu le loisir de le faire 5 ans durant ? Rappelons que le budget prévisionnel de l' AME pour 2017 s' élève à 815 millions d' euros , et que l' on peut raisonnablement estimer son coût réel à 1,3 milliards pour l' année à venir … dont moins d' un dixième concerne des soins urgents . Pour éviter le dévoiement de ce système , et pour enrayer le &laquo; tourisme médical &raquo; qui est une réalité indéniable , je propose depuis plusieurs années de supprimer l' AME et de la remplacer par une aide restreinte , exclusivement réservée aux soins d' urgence vitale . Il est également inacceptable de constater que des ressortissants de pays étrangers laissent aux hôpitaux français des notes très salées … en 2015 , plus de 120 millions d' euros de frais de santé restaient dus par des patients étrangers , uniquement sur les hôpitaux parisiens ! Ces créances doivent être recouvrées au plus vite et je m' y emploierai . Je ferai également de la lutte contre la fraude une priorité . Pour que notre système de protection sociale soit juste , il faut que chaque Français qui en a besoin puisse bénéficier de la solidarité nationale , et que les abus soient sévèrement punis , or le laxisme sévit dans tous les domaines . A mesure que l' on professionnalise la lutte contre la fraude , les sommes détournées détectées augmentent : elles ont dépassé le milliard d' euros l' an passé . Les arrêts maladie de complaisance doivent être farouchement combattus , pour que les deniers publics servent à ceux qui en ont réellement besoin . Les données collectées par l' Assurance Maladie doivent être analysées de manière plus précise , par des équipes anti-fraude , et de nouveaux dispositifs pourraient voir le jour pour rendre les abus plus difficiles . Une nouvelle Carte Vitale biométrique , combinée à la carte nationale d' identité , serait un bon début . La vente des médicaments à l' unité permettra également de lutter contre le trafic , au niveau national mais également international , certains patients étrangers venant se faire prescrire des traitements coûteux en France pour les revendre à moindre coût à l' étranger . Pour optimiser la gestion du système de santé , et améliorer la prise en charge des patients , les ingénieurs et les entreprises innovantes françaises doivent être mises à contribution . Des dispositifs de santé connectée peuvent aider au suivi des patients chroniques , éviter des complications très coûteuses pour la collectivité et donc permettre des économies : par exemple , le non-respect des prescriptions médicales engendre un surcoût de 9 milliards d' euros par an , juste en considérant les six principales pathologies chroniques . Les instruments de télé-assistance et de télé-surveillance permettent de veiller sur nos aînés vivant à seuls à leur domicile afin de leur porter secours au moment adéquat . Le prix des médicaments innovants doit également être jugulé , et le généricage des molécules doit être accéléré et facilité , afin de permettre des économies . Un fonds d' investissement pour l' innovation en santé sera créé en partenariat avec la BPI , et les subventions aux startups innovantes seront là , conditionnées à un engagement pour l' entreprise de rester sur le sol français et de ne pas revendre les technologies à une entreprise étrangère pendant 10 ans . Les financements de la recherche scientifique , en particulier biomédicale , doivent être réévalués à la hausse , afin de garantir un avenir à nos jeunes chercheurs , et faire rayonner notre savoir et nos technologies sur la scène internationale . La recherche permet également de faire évoluer les modes de prise en charge , et pour désengorger des hôpitaux au bord de l' asphyxie , la prise en charge ambulatoire peut être privilégiée lorsque l' état des connaissances de la médecine le permet . Nous avons un vivier de jeunes prêts à assurer ces missions , et qui ne demandent qu' à travailler . Ce ne sont pas les moyens qui manquent pour remettre notre système de protection sociale en marche , c' est la volonté politique . Un changement de paradigme est nécessaire , et je souhaite l' incarner . La santé et la protection sociale doivent être des priorités politiques , régaliennes , nationales , qui ne doivent en aucun cas être déléguées à des structures privées , à but lucratif . Vive la Sécurité Sociale , vive la République , et vive la France ! ",    
    "Merci à tous , Merci à tous les intervenants de cette convention , Merci aux membres du Collectif Banlieues Patriotes pour la qualité de leurs propositions et pour leur travail de terrain . Je suis heureuse que nous puissions prendre le temps , avant la campagne , de parler d' un sujet aussi délicat que celui de nos banlieues , en ayant le temps de développer honnêtement nos idées et en évitant toute caricature . C' est un sujet délicat , car comme tout sujet que le pouvoir politique a sciemment laissé de côté depuis des décennies , il est devenu un point névralgique . Un point sensible qui réagit au moindre choc . La France a mal à ses banlieues . Les fantômes des émeutes de 2005 ressurgissent au moindre trouble à l' ordre public . Nous ne pouvons pas laisser la France ainsi fracturée . Si nous voulons véritablement une France apaisée , il faut donc traiter cette question à la fois avec une précision pointilleuse et une volonté bienveillante . Nous devons le faire dans l' intérêt de la nation . La France des oubliés , ce sont aussi ces quartiers relégués et à la dérive . Il s' y trouve des voyous que nous neutraliserons sans pitié , mais il s' y trouve aussi et surtout des Français abandonnés , qui n' ont pas pour eux les masses d' argent de la politique de la ville , pour une raison simple : cet argent est gaspillé dans le clientélisme . On a laissé partir à vau l' eau les banlieues de nos villes . Pire encore , certains politiques se sont nourris de cette décomposition , en vrais charognards . Certaines puissances étrangères aussi . Le clientélisme y est roi , plus que dans toute autre partie de la France . Ajoutez à cela le communautarisme , la radicalisation religieuse et le chômage de masse , et vous disposez là d' une véritable bombe . Et quand vous êtes face à une bombe , vous avez deux solutions : soit vous essayez de la désamorcer , soit vous la faites exploser . Les autres dirigeants politiques , qui ont abandonné l' idée même de nation française , ont décidé de la faire exploser . Je suis la seule qui veut la désamorcer . Je suis la seule qui veut éviter le chemin du pire , et qui veut proposer une voie d' apaisement . Tous les multiculturalistes , tous les chantres de l' identité heureuse , tous les faux durs , les vrais mous , tous les Tartuffes de l' idéologie du vivre-ensemble , tous ceux-là jouent aux apprentis sorciers . En voulant défaire la France , ils préparent l' affrontement des Français entre eux . Nous ne pouvons pas nous permettre de les laisser continuer leur œuvre mortifère . C' est pour rétablir la paix civile , qui est le bien le plus précieux d' une nation , que nous devons prendre à bras le corps le problème des banlieues françaises . Notre plan est simple : il comprend trois volets . Premièrement , rétablir l' ordre républicain dans les banlieues . Deuxièmement , déraciner le fondamentalisme islamiste pour préserver les enfants des banlieues de toute dérive , et notamment les femmes et les filles . Troisièmement , ouvrir de nouvelles perspectives de développement économique à cette jeunesse pour qu' elle participe à la prospérité de la France de demain . Commençons par le rétablissement de l' ordre , sans lequel rien n' est possible . Mon objectif consiste à retranscrire ce que je nomme le &laquo; vive-en-paix &raquo; dans nos quartiers . Certaines banlieues sont devenues des zones de non-droit où l' on applique plus la loi de la République . Les premières victimes de cet état sont les habitants de ces mêmes banlieues , souvent terrorisés par des caïds qui font régner la loi du plus fort . L' Etat a abdiqué face à cette menace . Il ne se donne plus les moyens de protéger les Français . Je rappelle ce chiffre , connu de tous , mais pourtant toujours aussi ahurissant : Nicolas Sarkozy a supprimé 12 500 postes de policiers sous son mandat . Ce même Nicolas Sarkozy qui , sur RMC il y a quelques jours , disait être fier d' avoir supprimé des postes de policiers en temps de crise , parce que cela prouvait qu' il était un homme responsable ! Il est fier de ses fautes ! Il suffit pourtant d' un gramme de bon sens pour comprendre qu' en temps de crise , la sécurité est la DERNIERE chose sur laquelle il faut faire des économies . Et ce n' est pas François Hollande qui a changé quoique ce soit à cette hémorragie , puisque sur les 9000 postes qu' il avait promis , il n' en a pourvu que 390 . Du côté de nos forces de douane , le constat est tout aussi catastrophique : 4000 postes supprimés sur les dix dernières années . Qu' on ne s' étonne plus ensuite que nos frontières , qui sont plus trouées qu' une tranche de gruyère , laissent passer des milliers de kalachnikovs qu' on retrouve dispersés dans toutes les banlieues du pays , ou la drogue qui la gangrène . Le rétablissement de nos forces de police et de nos forces de douane est donc la condition sine qua non de toute résolution du problème de la sécurité de nos banlieues . A cela s' ajoute d' autres mesures immédiates , comme la présomption de légitime défense pour les policiers , et l' application réelle des peines de prison inférieures à 5 ans . L' Etat ne doit seulement arrêter de reculer , il doit avancer ! Il doit restaurer l' ordre sur un territoire qu' il n' a pas le droit d' abandonner . Car non seulement , en abandonnant ce territoire , il met en danger les villes françaises , mais il ne respecte pas non plus son devoir de protection des habitants de ces quartiers . Après le rétablissement de l' ordre , sans lequel tout futur est inenvisageable , il faut creuser plus profondément jusqu' à la racine du mal . Il faut déraciner cette idéologie qui gangrène les banlieues et qui se nourrit de la faiblesse humaine , l' idéologie salafiste . Le salafisme ne naît pas par génération spontanée . Il a des relais , il a des porte-voix , il a des financements . Les banlieues françaises n' étaient pas destinées à devenir le terreau du radicalisme . Et elles ne sont pas destinées à le rester . Pour préserver la jeunesse de nos banlieues de cette lobotomie qu' est le salafisme , il faut la préserver de ces relais de la haine . C' est une bataille idéologique . Nous devons assécher le marais du salafisme , et pour cela il y a des moyens simples . Nous devons fermer les mosquées radicales , puisque Monsieur Cazeneuve , qui l' avait pourtant promis avec ses airs de premier flic de France , n' a pas été capable de le faire . Nous devons expulser les prêcheurs de haine , qui aujourd'hui exposent leurs thèses négationnistes , misogynes , homophobes ou christianophobes dans des réunions publiques en Seine-Saint-Denis sans que cela ne semble inquiéter personne à par nous . Nous devons dissoudre l' UOIF , qui est le cache-nez du radicalisme islamiste en France , et qui a servi jusqu' ici de cheval de Troie à des revendications politico-religieuses , Il faut contraindre les réseaux délétères des frères musulmans qui essaiment , avec la bienveillance de nombreux candidats actuels à la présidentielle , Et après ces mesures immédiates , et qui pourtant n' ont jamais été mises en place , il reste une mesure à plus long terme , tout aussi fondamentale que les autres : nous devons interdire le financement des lieux de culte par des fonds étrangers , et notamment le financement des centres cultuels ou culturels par le Qatar et l' Arabie Saoudite . Nous devons également lutter contre la corruption de certains élus locaux qui rendent possible par leurs agissements l' infiltration de ces puissances étrangères sur le territoire national . A cette fin je propose la mise en place d' un corps d' inspection contre la corruption de la vie politique locale chargé en particulier de détecter le clientélisme et les contournements à la loi de 1905 sur la Laïcité . Je le répète : les banlieues ne sont pas destinées à rester des viviers du salafisme . Ce n' est pas une fatalité . Nous pouvons faire sortir la jeunesse des banlieues des griffes de l' idéologie . Nous pouvons leur redonner un avenir . Un avenir français . Mais pour cela , il faut tarir la source du problème . Et quoi de mieux pour cet objectif que de recréer à terme un nouveau Service National pour nos jeunes filles et garçons , encadrés à partir d' une armée rétablie dans ses moyens et remise au coeur de la formation du citoyen . Enfin , si nous réussissons à rétablir l' ordre et à tarir les sources de la radicalisation , notre tâche ne sera pas terminée pour autant . Si nous ne voulons pas que les banlieues redeviennent une nouvelle fois une zone de non-droit , nous devons les faire participer à la prospérité économique d' une France souveraine . Ceci , disons -le , ne sera possible qu' en rétablissant d'abord la sécurité , tant la délinquance et l' insécurité sont des obstacles majeurs à tout développement économique . En effet , ce n' est pas l' économie parallèle des dealers et des trafiquants qui doit y régner , mais l' économie réelle des entrepreneurs , des nouvelles industries , des PME , des artisans , des commerçants et de tout ce qui fait le tissu économique et social de la France , La banlieue est , comme l' ensemble de la France , bourrée de jeunes talents qui ne cherchent qu' à exprimer leurs capacités . Alors que la politique de la ville est un puits sans fond où l' Etat perd une somme d' argent monstrueux pour des résultats inexistants , il est possible d' allouer cet argent dépensé inutilement de manière beaucoup plus efficace , en soutenant la création d' entreprise et en facilitant la vie des PME de nos banlieues . Une fiscalité des entreprises plus juste permettra aussi de voir éclore des start-up qui aujourd'hui peinent à survivre . Ainsi , ma dernière proposition ce jour est la création d' un fonds d' intervention pour les start-up , car je sais qu' il y a beaucoup de gens qui veulent entreprendre et créer dans les quartiers . Enfin , si nous voulons véritablement en finir avec cette image de la banlieue à la marge du développement économique , il faudra en finir avec l' idée même de banlieue dans nos têtes , et notamment à Paris . Parlons simplement de France ! Je suis volontariste . Je ne crois pas à la fatalité d' un destin sur lequel il serait impossible d' agir . Je ne crois pas que les banlieues françaises soient vouées à une déchéance toujours plus grande , je ne crois pas que les enfants de l' immigration soient voués à rester aux marges de la nation . Je crois qu' il est possible de refaire l' unité de la France et de tous les Français , mais cela ne peut passer que par l' affirmation forte de quelques vérités trop longtemps oubliées : la nation est le cadre intangible dans lequel nous pouvons agir et abandonner ce cadre est une voie sans issue . L' ordre est la condition de la prospérité économique et celle -ci doit être soutenue . L' amour de la France est le fondement de toute vie collective pacifique . Ces vérités sont simples , mais ce qui est simple n' est pas forcément facile . Nous n' avons pas choisi la voie de facilité . Mais nous avons choisi la voie de l' action , donc de l' espoir , et il faut nous en rendre dignes . Vive le Collectif Banlieues Patriotes , vive la République , et vive la France ! ",
    "Etats-Unis : le choix de la liberté Les Américains ont voté . Ils ont refusé le statu quo . Ils ont montré par une décision qui a surpris ceux qui croient que les situations sont immuables , que le monde bouge , que le monde change , que le mouvement fait partie de la vie des Nations . Ce qui s' est passé cette nuit n' est pas la fin du monde mais la fin d' un monde . Les Américains se sont donné le président qu' ils ont choisi et non celui qu' un système installé voulait leur faire valider comme si les élections n' étaient qu' une formalité , voire une corvée pour satisfaire les apparences ou les convenances . En cela , la décision du peuple américain doit s' interpréter comme la victoire de la liberté , la liberté d' un peuple souverain , la liberté de citoyens qui ont su s' affranchir d' une campagne où le dénigrement et la peur , où l' infantilisation et la condescendance furent autant de tentatives , finalement infructueuses , de conditionnement de l' opinion . Parce que seul compte à mes yeux l' intérêt de la France et des Français , j' ose répéter ici que l' élection de Donald Trump est une bonne nouvelle pour notre pays . Refus du TAFTA et plus généralement d' une mondialisation sauvage , pacification des relations internationales notamment avec la Russie , désengagement des expéditions belliqueuses qui sont à l' origine des grandes vagues migratoires dont nous sommes directement victimes . Ces engagements , s' ils sont tenus , sont bénéfiques pour la France . D' une manière générale , et la classe politique française devrait s' en souvenir , on gagne toujours à fonder sa vision sur le respect des autres peuples et le refus des ingérences moralisatrices . Dans une démocratie , dans tous les pays , c' est le peuple et lui seul qui décide , c' est lui et lui seul qui donne la légitimité . Ici et ailleurs , il n' est pas de gouvernement digne , pas de gouvernement authentiquement légitime qui ne gouverne pas pour le peuple et au nom du peuple . Les élites politiques et médiatiques lourdement sanctionnées ce matin ne peuvent plus l' ignorer . Le référendum français en 2005 , le référendum grec en 2015 , les récents succès électoraux des patriotes dans différents pays d' Europe , le vote massif des Britanniques en faveur du Brexit bien sûr , et maintenant l' élection de Donald Trump : ce sont autant de choix démocratiques qui enterrent l' ordre ancien et autant de pierres qui construisent le monde de demain . La défense d' intérêts catégoriels , la soumission à des puissances d' argent insensées et dévastatrices , l' abdication devant les dictatures islamistes , sont des choix qui isolent chaque jour un peu plus ceux qui , ici ou ailleurs , persistent à les défendre . Ce matin , le peuple américain vient donc confirmer ce grand mouvement du monde qui voit le retour des peuples libres dans le concert mondial , l' affirmation du principe de la primauté de l' intérêt des nations sur les intérêts particuliers et sur les féodalités , l' immense soif de liberté et d' indépendance et , je l' espère , l' émergence d' une conception multipolaire des relations internationales . Le peuple américain est un peuple historiquement ami de la France . Nos histoires sont liées . Nous l' avons accompagné dans sa guerre d' indépendance , puis nous l' avons imité en empruntant derrière lui , les chemins de la liberté . Gageons que les Français , qui chérissent tant cette liberté , y trouveront une raison supplémentaire de rompre avec un système qui , eux aussi , les entrave . Avec raison , le nouveau Président des Etats-Unis d' Amérique a réaffirmé dans sa première intervention son intention de faire valoir l' intérêt de son pays . Je ne lui en ferai pas le reproche puisque nous partageons cette conception de l' action politique pour nos pays respectifs . Dans le même temps , il a tendu la main à tous les pays qui souhaitent travailler avec lui . Dans le cadre de relations fondées sur le respect de nos souverainetés et de nos intérêts nationaux , cette main tendue est un geste dont nous devons prendre en compte la portée . Qu' il me soit permis d' adresser mes félicitations au nouveau Président des Etats-Unis et mon témoignage d' amitié au peuple libre des Etats-Unis d' Amérique .",
    "Merci à tous , Merci à nos intervenants d' aujourd'hui , que l' on sent passionnés par l' idée d' une France à nouveau belle et grande , Merci aux membres du Collectif Culture et du Collectif Mer et Francophonie . Merci aussi à nos frères de cœur venus d' autres pays , qui partagent l' amour d' une culture francophone à la fois diverse et unie . Je dois vous avouer que c' est peut-être le discours le plus exaltant que j' aurai à faire lors de ces conventions présidentielles . Tous les sujets nous intéressent , et il n' est pas question pour nous de laisser quoique ce soit de côté , lorsqu' on veut présenter un projet cohérent pour la France . Mais aujourd'hui , c' est un peu spécial . Aujourd'hui , nous parlons de grandeur . Nous parlons non seulement de ce que nous pouvons apporter à la France , mais aussi de ce que la France peut apporter au monde . La France est une vieille et grande civilisation . Elle l' a un peu oublié . On lui a fait oublier . Mais il suffirait de peu de choses pour qu' elle retrouve son influence sur la scène mondiale . Il suffirait , avant tout , de volonté politique . Il fut un temps où la France découvrait les quatre coins du globe . Un temps où Jacques Cartier ouvrait le fleuve Saint-Laurent à l' Ancien Monde . Un temps où les Français exploraient toute la vallée du Mississippi . Un temps où les missionnaires de France se rendaient dans les royaumes d' Asie . Un temps où la France était dans toutes les cours d' Europe . Un temps où Descartes allait chez Christine de Suède ; où Voltaire régnait sur la cour de Frédéric de Prusse ; où Diderot se rendait chez Catherine de Russie . Un temps où toutes les élites parlaient français , non pas parce qu' on les contraignait à parler français , mais parce que le français provoquait l' admiration . Parce que le français était la langue de la raison . Un temps où la France était le phare de la liberté . Un temps où tous les opprimés du monde voyaient en la France le symbole de la lutte contre la tyrannie . Un temps où la France abolissait l' esclavage , avant toute autre nation . Un temps où l' on déclarait en français , partout où soufflait le vent de l' indépendance : &laquo; Tous les hommes naissent et demeurent libres et égaux en droits &raquo; . La France a été la civilisation de la liberté et de la raison . Certains diront que c' est une illusion rétrospective . Que nous reconstruisons l' histoire . Mais ce sont les mêmes qui abaissent constamment la France , qui ne préfèrent voir que ce qui est bas , ce qui est mesquin , ce qui est laid . Ils veulent salir la grandeur . Ils n' acceptent pas que la France ait pu être grande et peut-être encore moins qu' elle veuille le rester . Que les Français aient pu faire de belles choses et de grandes choses . Ils ne voient que la France à terre , la France défaite dans la guerre , la France soumise à des puissants qui salissent son nom . Le roman national est , entre leurs mains , devenu un cauchemar . Les censeurs de la grandeur sont en train de créer des névroses immenses : ils font des générations de Français qui se haïssent eux-mêmes , et cela est beaucoup plus dangereux qu' on ne le croit . La haine de soi-même ne peut entraîner que la rancœur , la colère et finalement la haine des autres . Ceux qui haïssent la France n' haïssent pas que la France . Heureusement , subsistent à l' extérieur de notre pays les traces de cette grandeur toujours vive . Quand les Français , à qui on répète à longueur de journée qu' ils sont dépassés , qu' ils sont mesquins , qu' ils sont de rebus de l' histoire , quand les Français oublient leur propre valeur , il faut voir avec quelle ardeur nos cousins québécois , sénégalais , ivoiriens , belges , et bien d' autres défendent une culture et une langue que nous laissons à l' abandon . Il faut voir la vitalité du cinéma québécois , de la littérature francophone en Afrique sur les traces de Senghor , il faut voir les artistes belges , les artistes non francophones même qui choisissent le français . En se haïssant elle-même , la France devient stérile . Mais la culture française n' est pas stérile . Ce n' est pas une culture de mort . Et dès qu' elle peut renaître quelque part , elle bourgeonne immédiatement , et elle prend différentes couleurs sous d' autres latitudes . Elle s' enrichit constamment . Je ne viens pas vous livrer un message désespéré . Au contraire . Je pense que la France peut redevenir plus grande encore , plus belle , plus audacieuse , plus exploratrice et plus avant-gardiste que jamais . Qu' elle peut rayonner partout , imposer le respect partout , faire éclater son génie aux yeux du monde . Je pense qu' elle peut retrouver son influence perdue . Pour cela , il faut que nous repérions les leviers grâce auxquels nous pouvons agir . J' en développerai trois . D'abord , il ne faut &laquo; cesser de reculer &raquo; comme disent certains , il faut avancer . Si la francophonie est aujourd'hui en échec , c' est parce que son cœur , la France , a baissé les bras dans la défense de sa culture . Et la culture , c' est avant tout la langue . Nous voulons une nouvelle loi Toubon , beaucoup plus contraignante . La loi Toubon partait d' une bonne intention , mais aujourd'hui , elle fuit de partout . Dans les médias , dans les administrations , dans les écoles , dans les universités . Alors que plusieurs universités pratiquent déjà le tout-anglais dans certains cursus , la loi Fioraso de 2013 et ses amendements ont introduit des exceptions concernant l' enseignement en anglais , je cite &laquo; lorsqu' elles sont justifiées par des nécessités pédagogiques &raquo; . C' est ici un cheval de Troie . Maintenant , on ne va plus étudier à la Sorbonne ou à la fac de Nanterre , mais à la &laquo; Toulouse School of Economics &raquo; ou à &laquo; Paris Sciences et Lettres Research University &raquo; . Dans une même logique , les stations de radio privées disent dorénavant ne plus pouvoir respecter les quotas de chansons en français imposés par la loi Toubon , pour la simple et bonne raison que les Français ne chantent plus … en français . Ma première proposition est donc la suivante : je veux une loi Toubon 2 , pour que la langue et la culture françaises cessent de reculer . Les Québécois nous ont précédés avec leur loi 101 , qui est bien plus rigoureuse dans la protection et la promotion de la langue officielle . Je veux une loi qui fasse respecter partout l' exigence de promotion du français , surtout dans l' enseignement , et une loi qui encourage davantage les artistes francophones . Un autre volet de cette loi Toubon 2 , ou d' une autre loi qui la suivra , sera la protection du patrimoine culturel . Il nous faut à la fois protéger ce patrimoine et le valoriser , contre toute tentative de le brader , en le laissant filer entre les mains de collectionneurs étrangers ou d' investisseurs sous forme de placements . Contre ceux aussi qui le défigurent , en imposant à nos monuments historiques un habillage qui vire souvent au massacre . La première étape , c' est celle de la protection : on arrête de reculer . La deuxième , c' est celle de l' organisation . Ici , le levier le plus puissant , c' est l' Organisation Internationale de la Francophonie , l' OIF . C' est un outil sous-exploité . Depuis l' élection de sa nouvelle présidente , Michaëlle Jean , en 2014 , l' OIF n' a toujours pas présenté de plan d' envergure pour l' apprentissage du français dans le monde . Il faut dire que les crédits sont insuffisants : les gouvernements successifs ont délaissé ce combat . Pour l' instant , la francophonie , c' est de l' or que l' on transforme en plomb . Rappelons que le français est aujourd'hui , avec 274 millions de locuteurs quotidiens , la seule langue , avec l' anglais , qui soit présente sur les cinq continents , qu' il est la cinquième langue la plus parlée , la quatrième langue sur internet , la troisième langue d' affaires , et la seconde langue d' information dans le monde . Imaginez toutes les occasions économiques et culturelles qui sont gâchées par l' abandon de ce trésor . Cesser de reculer , c' est également défendre , entretenir et promouvoir l' immense patrimoine de la France , sur le territoire national comme à l' étranger , et que le monde entier nous envie ! Ces palais , châteaux , hôtels particuliers , demeures de prestige , qui abritent nos ambassades , nos centres culturels , nos écoles nationales , et qui sont le témoignage de notre histoire hors du commun , forment le patrimoine de tous les Français . Je mettrai donc un coup d' arrêt , et c' est l' objet de ma deuxième proposition , à la dilapidation de nos trésors nationaux , par des gouvernements indignes de l' héritage qui leur a été confié par la Nation . Sous mon mandat , aucun de nos monuments ne sera bradé à des puissances étrangères ; il en va du prestige comme de l' honneur de la France . Ma troisième proposition est l' organisation rapide d' un grand sommet de la Francophonie . Ce sommet aura un triple but : relancer la promotion du français à l' étranger , encourager la coopération économique des pays francophones , et repenser le rapport des pays francophones en termes de souverainetés nationales . Cela veut dire , en particulier , développer une vision moderne des relations entre la France et les pays d' Afrique francophones . Ni ingérence , ni abandon ! Nous voulons une relation de respect mutuel , dans la préservation des libertés et des identités des peuples , pour leur permettre de défendre leurs intérêts nationaux , notamment face à la Chine ou à l' Union européenne . C' est une Afrique libre qui sera prospère , et une Afrique prospère qui sera le meilleur allié de la France . Après la protection et après l' organisation , vient le temps de l' exploration . C' est le plus exaltant de tous . C' est le temps des grandes aventures , qui feront date si nous réussissons . Et le champ d' exploration de la France , aujourd'hui , c' est la mer . Je l' ai déjà dit de nombreuses fois , mais il est bon de le répéter : l' espace maritime français est le deuxième au monde après celui des Etats-Unis avec 11,5 millions de km² . L' équivalent de près de vingt fois la France métropolitaine ! Cet espace démultiplie nos possibilités . Il nous donne accès à des ressources inexploitées . Il nous donne un avantage stratégique sans commune mesure . Il offre des perspectives de croissance immenses . Et pourtant , cet espace de vingt fois la France n' existe pas sur les radars médiatiques et politiques . Pire , nous sommes entravés dans nos possibilités de développement . Notre pêche est entravée par l' Union Européenne , qui nous limite ici comme pour tout le reste . Les 300 000 emplois liés à l' économie de la mer sont menacés . La flotte commerciale française s' amenuise , alors qu' aujourd'hui plus de 80 % des échanges commerciaux mondiaux se font par voie maritime . Les recherches sur la mer sont sous-développées , en comparaison du potentiel . Ma quatrième proposition est donc la mise en place d' un grand Ministère de la Mer , qui aura pour but de coordonner la prospection des fonds marins , le développement des infrastructures portuaires et maritimes ainsi que les ressources économiques de la pêche , des gisements de minerais et de pétrole et le développement des énergies renouvelables aquatiques . Les Américains ont été motivés pendant deux siècles par l' idée de repousser toujours plus loin la frontière de l' exploration de leur territoire , puis de l' exploration spatiale . La frontière que la France doit pousser toujours plus loin , celle qui sera exaltante , c' est la frontière de la mer . C' est un monde nouveau , que l' on croyait connu mais qui ne fait que commencer à se dévoiler . Je ne veux pas que la France manque cette occasion de montrer encore une fois au monde qu' elle sait être à la pointe de la découverte . Nous sommes des explorateurs . Nous partons d' une civilisation riche et ancienne , et nous pouvons encore l' augmenter sans limites . Nous pouvons rivaliser d' ingéniosité , d' imagination , d' intelligence avec toutes les grandes puissances de ce monde . Mais pour cela , il faut retrouver ce qui a fait notre force , et cesser de nous complaire dans une prétendue décadence . La France sera encore grande à l' avenir , j' en ai la certitude . Et je ferai tout ce qui est en mon pouvoir pour faire advenir ce jour le plus tôt possible . Vive la francophonie , vive la culture française , vive la République , et vive la France . ",
    "Mesdames , Messieurs , fonctionnaires de la Police nationale Si j' ai décidé de m' adresser directement à vous c' est pour vous dire combien nous vous comprenons et vous soutenons dans les moments difficiles que vous traversez . Confrontés à une situation que nos dirigeants qualifient eux-mêmes de &laquo; guerre &raquo; , une guerre dans laquelle vous êtes en première ligne , vous vous sentez , à juste titre , abandonnés par le pouvoir en place . Depuis deux quinquennats , les pouvoirs qui se sont succédé ont réduit sciemment vos effectifs avec une vision comptable irresponsable comme si la sécurité publique , comme si la sécurité des Français c'est-à-dire leur liberté pouvait être une variable d' ajustement des politiques publiques . Petits technocrates sans vision et sans envergure , ils ont réduit votre mission à des obligations de résultats administratifs ou statistiques ( au nombre de pv ) oubliant que le maintien de la tranquillité publique est d'abord une affaire essentiellement humaine , d' engagement , de générosité au service du bien public . Au niveau du terrain , les politiques , ceux-là mêmes qui sont en charges de vous diriger fuient souvent leurs responsabilités et vous laissent sans ordre mais , de leur bureau , savent vous accabler quand la situation risque de troubler leur déroulement de carrière . Mais surtout , vous sachant confrontés à une menace criminelle de plus en plus violente , face à une menace terroriste c'est-à-dire une menace de nature militaire , ils vous abandonnent avec des moyens techniques et juridiques souvent dérisoires et presque toujours inadaptés . Nous savons les outrages verbaux que pouvez essuyer chaque jour de la part d' individus bien connus de vos services , toujours les mêmes . Nous avons connaissance des agressions physiques que vous subissez quotidiennement par des délinquants souvent multirécidivistes . La plupart semblent jouir d' une parfaite impunité , quand ce n' est pas d' une tolérance ou d' une indifférence du pouvoir politique . Nous avons , comme chacun d' entre vous , en mémoire l' attaque de Viry-Châtillon où deux de vos collèges ont été brûlés vifs par des criminels et non des &laquo; sauvageons &raquo; comme a pu scandaleusement le dire un ministre de l' Intérieur indigne . Cet acte nous a glacés et nous glace toujours d' effroi quand on sait qu' un jeune homme de 28 ans et une jeune mère de 3 enfants vont porter toute leur vie d' effroyables séquelles de cette agression Avec le double assassinat de Magnanville , nous savons que la guerre que l' on vous demande de mener est totale : ce n' est plus votre propre vie qui est en jeu , mais aussi potentiellement celles de vos familles . Cela aussi doit être pris en compte . Les forces de gendarmerie , interdites de parler du fait de leur statut militaire , partagent la même constatation face à cette dégradation et cette progression dans l' horreur . Force est de constater que le pouvoir actuel , discrédité et usé , est dépassé par les évènements . Certaines zones de notre pays semblent plongées dans un chaos sécuritaire inquiétant . L' insécurité s' est étendue à nos campagnes et à nos écoles avec la multiplication des agressions contre vos collègues de l' Education nationale . Derrière ces attaques contre des fonctionnaires , ne nous y trompons pas , c' est l' Etat qui est visé , c' est la République française qui est visée , c' est la nation française tout entière qui est visée . C' est avec solennité que je veux vous dire mon soutien total et celui des Français attachés à nos valeurs , à nos libertés , à nos principes de vie . Vous le savez , je crois nécessaire de remettre la France en ordre . Nous devons réaffirmer la souveraineté intérieure de l' Etat . Il est urgent d' engager un plan de réarmement de nos forces de sécurité publique : > Un réarmement moral avec le témoignage de ce lien indissoluble entre la nation et ses défenseurs intérieurs , la solidarité des citoyens envers leurs policiers , le soutien indéfectible de l' Etat envers ses serviteurs . Cette reconnaissance qui doit être sans faille , passe également par une révision des situations individuelles sans laquelle vous ne pouvez pas travailler avec un esprit libre : la revalorisation des situations salariales , le paiement effectif de vos heures supplémentaires , les justes promotions et mutations aujourd'hui bloquées par la politique de régression des effectifs . Plus pratiquement , le soutien aux fonctionnaires c' est aussi faire en sorte que les policiers puissent bénéficier de logements dans des secteurs où leurs familles ne seront pas menacées . > Un réarmement humain avec une augmentation sensible des effectifs de police non pas seulement des emplois jeunes mais des fonctionnaires , formés , motivés , managés et surtout soutenus . > Un réarmement matériel en apportant les dotations budgétaires suffisantes à la modernisation des commissariats , à l' achat de véhicules performants , à la dotation de matériels et d' armements modernes et dimensionnés , à la formation des personnels aux nouvelles menaces à commencer par un entraînement très régulier au tir . > Un réarmement juridique en établissant une présomption de légitime défense pour les fonctionnaires de police nationale attaqués , en rendant anonyme leurs plaintes contre leurs agresseurs pour ne pas les désigner aux voyous , en accompagnant systématiquement les fonctionnaires plaignants avec des avocats de l' Etat qui assureront leur défense et leur obtiendront des indemnisations à la hauteur de leur atteintes corporelles et psychologiques . Au-delà , parce que la solution à des questions aussi complexes n' est pas seulement policière , le soutien juridique des forces de sécurité publiques passe aussi par un fonctionnement revu et amélioré de toute la chaîne pénale c'est-à-dire du maillon judicaire et carcéral . Sans cela , votre travail admirable continuera de vous apparaître vain et donc décourageant . Chers compatriotes , fonctionnaires de police , de notre police nationale , votre mécontentement n' est pas seulement légitime . Il est sain . Il porte en lui une réaction salutaire devant une situation qui nous concerne tous , une situation dont on voit bien qu' elle met en jeu la liberté et même la vie de chacun d' entre nous . Les menaces de sanctions qui vous visent sont dérisoires venant de gens qui n' ont , aujourd'hui , pas d' autre choix digne que de démissionner . Ce n' est pas de menaces dont vous avez besoin mais de soutien et de décisions . Nous savons que le pays vous soutient pleinement parce qu' il sait pouvoir compter sur vous y compris en ces circonstances ô combien difficiles ! Partisans d' un Etat restauré , conscients de ce que nous vous devons tous les jours , nous sommes de cœur et d' esprit avec vous . Vous pouvez compter sur nous . Vive la police française Vive la République française Vive la France ",

    "Rien n' est jamais écrit à l' avance Le peuple américain a voté et élu Donald Trump à la présidence des Etats-Unis d' Amérique . En tant qu' alliés historiques de ce grand pays , nous devons le saluer et le féliciter . Cette élection montre que rien n' est jamais écrit à l' avance . Il faut toujours écouter ce que le peuple a à dire et non ce qu' on aimerait qu' il dise . Cette élection me parait être l' expression d' un rejet du système profond , et sous-estimé . Le même qui a conduit nos amis britanniques à décider de sortir de l' Union Européenne il y a quelques mois . Le même que j' entends depuis deux ans que je me suis engagé publiquement sur la scène politique . Ce rejet rend impossible de demeurer dans le statu quo dans lequel notre pays s' est englué depuis trente ans . Ces votes expriment une demande de protection et de respect du sentiment populaire que je crois partagée par les Français . Elle doit être entendue et comprise , sans conduire pour autant à des réponses de fermeture ou de repli . C' est la condition pour les transformer non pas en rejet du présent mais en espoir de changement . C' est l' enjeu des six prochains mois . ",
    "Bonjour à toutes et tous , Mesdames , Messieurs , merci d' être là aujourd'hui , ce matin . C' est une étape importante que nous allons franchir et que nous préparons depuis plusieurs mois . Parce que je suis en effet intimement convaincu que c' est en affrontant la réalité du monde que nous pourrons retrouver de l' espérance dans notre pays , et c' est notre défi aujourd'hui . J' entends certains qui pensent que notre pays est en déclin , que le pire est à venir , que notre civilisation s' efface . Ils proposent le repli , la guerre civile ou les recettes du siècle dernier . J' en entends d' autres qui imaginent que la France peut continuer de descendre en pente douce , que le jeu de l' alternance politique suffira à nous faire respirer et qu' en quelque sorte , après la gauche , la droite et ainsi de suite , parce qu' on l' a toujours fait de la sorte . Avec les mêmes visages , les mêmes hommes , depuis tant d' années . Je suis convaincu que les uns et les autres ont tort parce que ce sont leurs modèles , leurs recettes qui ont simplement échoué . Le pays , notre pays , lui dans son ensemble n' a pas échoué et il le sait , il le sent et c' est de là que naît ce divorce entre le peuple et ses gouvernants . Je suis convaincu que notre pays a la force , le ressort , l' envie d' avancer parce qu' il a l' histoire et le peuple pour cela . Parce que la France a toujours été un pays moteur du progrès , du progrès politique et démocratique , quand la Révolution Française , ici , a ouvert le monde nouveau et un chemin . De progrès économique quand nos ingénieurs , nos entreprises , nous ont permis de créer des richesses nécessaires pour accéder à une vie meilleure . Du progrès social , parce que c' est dans notre pays que nous avons construit les protections qui allaient avec l' ère industrielle , qui ont fait longtemps notre fierté , qui ont suscité partout l' envie . Du progrès pour tous , quand nos gouvernants , nos pays , ont eu la force après trois guerres de construire avec l' Allemagne et d' autres partenaires européens , cette Union européenne qui nous tient . Du progrès pour chacun , parce que la France , c' est par dessus tout un projet d' émancipation des individus , d' émancipation de la religion , des conditions d' origine . C' est un pays dans lequel , en partant de rien , le talent , l' effort doivent pouvoir mener à tout . Mais la France aujourd'hui est sortie du chemin du progrès . Le doute s' est installé . Depuis 40 ans , nous n' avons pas réussi à régler le problème du chômage de masse , la déprise des territoires , la langueur de l' Europe , les divisions internes allant jusqu' aux fractures se sont plus récemment établies . La France est bloquée par les corporatismes , de tous ordres et n' est plus à la hauteur de sa promesse . En même temps , nous sommes rentrés dans une ère nouvelle . La mondialisation , le numérique , le changement climatique , les inégalités croissantes de notre mondialisation , les conflits géopolitiques , le terrorisme , la crise démocratique des sociétés occidentales , le doute qui s' installe au coeur de nos sociétés . Ce sont les symptômes d' un monde en plein bouleversement . Cette grande transformation que nous sommes en train de vivre , nous ne pouvons y répondre avec les mêmes hommes et les mêmes idées parce qu' elle vient encore décupler nos doutes et nos faiblesses . Nous ne pouvons proposer simplement de réparer ou d' ajuster notre modèle . Cette grande transformation , elle a des aspects tragiques et nous devons être conscients de la gravité de l' histoire et des moments que nous vivons . Mais elle a en même temps des opportunités , des promesses que nous devons saisir . Nous ne pouvons pas demander sans cesse aux Français de faire des efforts sans fin . Nous devons regarder ensemble la vérité en face , débattre de ces grandes transformations à l' oeuvre et dire où nous voulons aller , dans quelle direction , par quel chemin , parce que le temps qui nous y conduira sera long , parce que tout cela ne se fera pas en un jour . Les Français sont conscients des nouvelles exigences de notre temps , bien souvent ils le sont plus que leurs gouvernants . Ils sont moins conformistes , moins attachés à ces idées toutes faites qui n' assurent rien d' autre que le confort intellectuel de la vie politique . Face à ces défis , notre système politique est bloqué . Lorsque je dis cela je n' accuse personne et je ne parle pas des élus . Nombreux sont au contraire ceux qui par leur dévouement s' engagent pour leur pays . Mais les appareils politiques , les logiques politiciennes , paralysent aujourd'hui notre capacité d' aller de l' avant . J' ai vu de l' intérieur la vacuité de notre système politique qui empêche les majorités d' idées , au motif qu' elles fragilisent les appareils , les partis traditionnels , les intérêts acquis , qui ne poursuit plus l' intérêt général mais son propre intérêt , qui a transformé la vie des Français en simple décor de son propre théâtre d' ombres . J' ai pu mesurer ces derniers mois ce qu' il en coûte de refuser les règles obsolètes et claniques d' un système politique qui est devenu le principal obstacle à la transformation de notre pays . Je sais ce que c' est . Je sais aussi que je continuerai à l' éprouver , cela ne fait que renforcer ma détermination . Parce que ce système , je le refuse . Face à cela je sais l' énergie du peuple français . Elle peut nous conduire à donner le meilleur de nous-mêmes . C' est cette énergie qui est aujourd'hui le quotidien des milliers d‘associations qui font le travail dans l' ombre le travail de tenir notre société ensemble . C' est cette même énergie qui donne leur force à tous ceux qui innovent , travaillent , produisent chaque jour . Si nous voulons avancer , faire réussir notre pays et construire une prospérité nouvelle , il nous faut agir . La solution , elle est en nous . Elle ne dépend pas d' une liste de propositions qui seront oubliées le jour d'après , qui changeront même parfois durant le temps de l' élection et qui au fond sont là pour ne pas être appliquées . Elle ne saurait émerger non plus de compromis bancals . Elle se fera grâce à des solutions différentes , grâce à une révolution démocratique profonde . Elle prendra du temps mais elle ne dépend que de notre unité , de notre courage , de notre volonté commune . C' est cette révolution démocratique à laquelle je crois . Celle par laquelle en France et en Europe , nous conduirons ensemble notre propre révolution plutôt que de la subir . C' est cette révolution démocratique que nous avons préparée durant ces derniers mois avec En Marche . Je crois très profondément que rien n' est jamais écrit . Et c‘est pourquoi je veux porter l' optimisme de la volonté . La France n' est pas un château de cartes . Elle a surmonté tant d' épreuves , bien plus redoutables . La seule force dont nous pouvons , dont nous devons nous réclamer , c‘est la force de la France , c' est la force des Français . C' est celle-là qui doit nous guider . Dans quelques mois , à l' occasion de l' élection présidentielle , une opportunité nous est offerte . Celle de refuser enfin le statu quo pour choisir d' avancer . Parce que ce combat que nous devons livrer pour faire réussir notre pays , il commencera en mai 2017 . Pour le mener , la responsabilité du président de la République est immense . J' en suis pleinement conscient . Un président n' est pas simplement investi d' une action . Il porte aussi , de manière moins visible , les valeurs de notre pays , la continuité de son histoire , et de manière cachée , la vigueur et la dignité d' une vie publique . Je sais cela . J' y suis prêt . C' est pourquoi je suis candidat à la présidence de la République . Parce que je crois plus que tout que nous pouvons réussir , que la France peut réussir . Bien sûr , on ne se réveille pas un matin avec cette révélation . La décision de me présenter aux plus hautes charges de la République est le fruit d' une conviction intime et profonde , d' un sens de l' Histoire et d' une conscience aiguë des temps qui sont les nôtres . J' ai fait le chemin de la province à Paris , du monde de l' entreprise à la vie publique puis à la vie politique . Les responsabilités qui ont été les miennes m' ont fait pleinement mesurer les défis de notre temps et toutes ces vies m' ont conduit à cet instant . Je veux faire entrer la France dans le XXIème , je veux que mon pays redresse la tête et pour cela retrouve le fil de notre histoire millénaire , ce projet fou d' émancipation des personnes et de la société . Ce dessein , c' est le dessein français : tout faire pour rendre l' homme capable . Je ne peux me résoudre à voir une France qui a peur , qui ne regarde que ses souvenirs , une France outrancière qui insulte et qui exclut , une France fatiguée qui stagne et qui gère . Je veux une France libre , libre et fière de ce qu' elle est , de son Histoire , de sa culture , de ses paysages , de ses femmes et de ses hommes qui ont traversé tant d' épreuves et qui n' appartiennent à personne . C' est pourquoi je place ma candidature sous le signe de l' espérance . La France peut réussir . Elle doit d'abord pour cela relancer l' Europe . J' entends tant de discours de haine sur ce sujet , de renonciation ou de repli . L' Europe , elle est notre chance dans la mondialisation , et notre responsabilité est ici immense . La France doit retrouver confiance en elle et bâtir son projet . Par le travail d'abord , qui émancipe chacun , parce que c' est par le travail que nous construisons notre place dans la société . Il n' y a pas de fatalité aujourd'hui , à ce qu' un jeune qui vient d' un quartier difficile comme celui où nous nous trouvons , ne puisse pas trouver d' emploi , n' ait parfois même pas le droit à la formation . Et le lieu où nous sommes , à ce titre n' a rien d' innocent . Et je ne peux accepter , de la même façon , que dans nos provinces tant de Français aujourd'hui attendent pour travailler , ne puissent même plus vivre de leur travail dans la dignité . Le travail , c' est ce qui nous construit . Par l' investissement ensuite , l' investissement dans notre avenir . Pour réussir cette révolution numérique , pour réussir le passage vers une autre économie , vers une autre société . Pour former , pour aussi accompagner cette transformation climatique et énergétique . Par l' école et la transmission ensuite . Notre pays se redressera par sa jeunesse . Je sais que beaucoup doutent de la jeunesse ils ont tort . Il ne s' agit pas de lui faire des promesses , on lui en a trop fait . Il s' agit de transmettre notre culture , nos valeurs et ainsi de permettre à tous les jeunes de construire librement leur avenir . C' est cela croire en sa jeunesse . Bâtir notre projet , par des nouvelles protections aussi . Car dans ce monde incertain , chacun doit être libre et nous devons pour cela protéger contre les risques de la vie , contre l' insécurité . Garantir ces protections : la santé , la retraite pour les plus âgés . Parce qu' on ne construit rien de solide en laissant des millions de Français au bord du chemin . C' est cela la vraie autorité . C' est cela la crédibilité de l' Etat . Vous l' avez compris , je veux libérer l' énergie de ceux qui peuvent , et dans le même temps je veux protéger les plus faibles . Ce projet , nous devons aussi le bâtir par nos territoires , nous avons des France aujourd'hui à réconcilier . Nos métropoles réussissent dans cette mondialisation . Il faut les y aider , les encourager , leur permettre de faire davantage . Elles sont notre chance . La France des quartiers a elle besoin de mobilité . Elle est aujourd'hui assignée à résidence alors qu' elle veut réussir , qu' elle veut que nous permettions aux jeunes et moins jeunes d' accéder à l' école , d' accéder à la culture , d' accéder à l' emploi . La France périurbaine et la France rurale ont le droit de réussir , de se développer . Et pour ce faire , nous devons aussi leur permettre d' aller de l' avant , investir et laisser faire . Cette France-là le veut , cette France le peut . Je veux une France qui croit en sa chance , qui risque , qui espère , qui n' admet jamais la rente indue ou le cynisme repu . Je veux une France entreprenante , où chacun choisit sa vie , où chacun peut vivre de son travail . Et une France qui considère les plus faibles . J' ai pour cela une confiance indéfectible dans les Français . L' enjeu n' est pas pour moi aujourd'hui de rassembler la gauche , il n' est pas pour moi aujourd'hui de rassembler la droite . L' enjeu est de rassembler les Français . Car cette transformation de notre pays n' est pas un combat contre quelqu'un , contre un camp , contre une partie de la France . C' est un combat pour nous tous , pour l' intérêt général , pour nos enfants . Ce combat , c' est celui que nous conduirons ensemble . Cette révolution démocratique , je ne pourrai la mener à bien qu' avec vous . J' en appelle aujourd'hui à toutes les femmes et les hommes de bonne volonté . J' en appelle aujourd'hui à toutes celles et ceux qui dans notre pays croient dans la réconciliation de la liberté et du progrès . J' en appelle aujourd'hui à toutes celles et ceux qui ne veulent pas simplement guetter dans la pénombre une lueur d' espérance mais qui veulent l' incarner . Car c' est notre vocation profonde et je n' en sais de plus belle . Vive la République et vive la France ! Merci à vous . ",
    "Bonjour à tous ! Bravo à vous ! Bravo à vous ! Bravo à vous ! Bonjour à toutes et tous . Merci d' être là . Merci d' être là . Vous êtes venus des quatre coins de la France . Alors je vous le demande ce soir , est -ce que nous sommes seuls ? Est -ce que vous avez peur ? Est -ce que vous êtes prêts à changer la France ? Alors les choses sont claires . Nous sommes ce soir ensemble , 15000 ! 15000 , mes amis ! Et je le sais , beaucoup nous regardent en ce moment , en direct , aussi . 15 000 venus de la France entière . De toutes les régions , et vous êtes là . Mais ce qui fait notre force , ce qui nous tient debout , ce qui nous tient ensemble , c' est que nous savons qui nous sommes . Et c' est que nous savons où nous voulons aller , ensemble . Ce qui fait notre force , c' est notre détermination , notre cohérence , c' est notre sincérité . C' est vous , depuis le début . Et si vous êtes là , tous et toutes , si vous avez décidé en ce samedi après-midi , à quinze jours de Noël , de prendre du temps à vos familles , ou de venir en famille , de prendre du temps de vos proches , c' est parce que ce qui nous rassemble , c' est que nous sommes sûrs qu' il n' y a pas de fatalité . Il n' y a aucune fatalité à ce que le pays , ni dans quelques mois , que le choix entre une droite extrême et une droite conservatrice , il n' y a pas de fatalité , il y a une espérance , et c' est la nôtre ! Nous en avons parcouru du chemin , depuis le 6 avril dernier . Vous vous en souvenez ? En quelques mois , nous sommes devenus près de 120000 , ce soir , nous serons 120000 ! Ces dernières semaines , ce sont 400 réunions par semaine qui se sont tenues partout dans le pays , c' est votre mobilisation , c' est vous qui l' avez voulue ! Nous avons fait du chemin , depuis le 6 avril dernier . Nous l' avons fait ensemble . Depuis Amiens , je viens de cette ville , je suis né à Amiens . Et il n' y a rien non plus qui devait me conduire ici , pas plus que Sabrina , Marisa , Richard , Jean , Sylvie , Gérard , vous toutes et tous . Ce qui m' a conduit , c' est le travail , le goût de l' effort , la volonté de comprendre , d' apprendre , de construire , ce fut la République aussi , par l' école , qui m' a tiré , les instituteurs , les professeurs , les concours , ce que j' ai pu réussir , ce que j' ai pu rater . Le service public , que je viens de quitter , et qui m' ont tant appris . La chance , parce que , oui , j' ai eu de la chance , beaucoup . Et il en faut . Nous en avons toutes et tous , il y a parfois des déveines , des malentendus , des échecs , des maladresses , des phrases qu' on regrette , je veux d'ailleurs ce soir lever un malentendu récurrent , je n' ai rien contre celles et ceux qui portent des T-shirts . Il m' arrive moi-même d' en porter , et j' aurais sinon trop de problèmes avec les formidables helpers qui sont là et qui arborent nos magnifiques T-shirts . La famille aussi , sans laquelle on ne peut prendre des risques , et qui reste mon socle d' attache , ce qui me tient . Et la confiance , enfin , la confiance de nos maîtres , de nos amis , de nos proches , c' est cela qui nous fait . Et puis votre confiance , mes amis , votre confiance depuis le premier jour , car c' est vous qui m' avez porté , car c' est vous qui nous avez faits , car c' est vous qui nous avez transformés , car c' est par votre confiance que nous sommes là ce soir ! Mais peu importe d' où nous venons , parce que nous savons ensemble où nous voulons aller . Le combat que nous voulons mener . Ce qui est important pour nous . Ce que nous voulons , ce n' est pas rassembler la gauche , ce n' est pas rassembler la droite , c' est rassembler les Français ! C' est cela notre projet ! Alors , c' était un rêve fou il y a six mois , il est certain . Mais ce rêve fou , vous êtes en train de le réaliser . De partout en France , des quatre coins de France , vous en êtes la preuve . Et certains n' ont pas compris , et pensent encore que vous avez peur . Ils n' ont pas compris . Vous avez peur ? Ils n' ont pas compris parce que nous avons déjà , nous , pris les risques . Le risque de faire , le risque de tenter , ce que tout le monde nous disait comme impossible . Le risque d' essayer , partout , et d' aller convaincre . Le risque de quitter le gouvernement pour les uns , et la fonction publique , le risque de quitter ses attaches pour d' autres , et de braver les interdits , les menaces . Le risque de prendre du temps , le risque de l' engagement , d' être là . Alors , oui , vous avez déjà pris les risques , nous avons décidé d' oser , nous voulons le faire parce que notre volonté , notre détermination , sont claires : il y a dans notre pays une place pour réconcilier la liberté et le progrès , cette place , c' est la nôtre , c' est celle que les Français veulent ! Alors , depuis tant de mois , vous avez conduit la Grande Marche . Des milliers d' entre vous ont été frappé aux portes de 100 000 Français . Nous avons ensuite parcouru la France de Strasbourg au Mans , en passant par Montpellier , pour expliquer , rendre compte , ah , que n' ai -je entendu . Nous étions une bulle , nous n' avions rien à dire . Bien sûr , on s' était tellement habitué à ne plus écouter les Français , à ne plus faire avec eux , à ne pas chercher à comprendre ce monde qui se transforme et ses défis , pour lui imposer des potions venues d' en haut , pour lui administrer des traitements dont l' un était de gauche et l' autre de droite , et au fond , il n' y avait que cela qui comptait . Non , nous l' avons fait ensemble , nous l' avons construit avec les Français . C' est cela notre projet . Et notre projet , c' est de faire entrer la France dans le XXIe siècle , c' est de faire gagner notre pays , c' est de le faire réussir dans un monde qui se transforme , où les menaces paraissent être partout mais où les opportunités sont là . Mais avec un engagement clair que je veux ici redire , partager solennellement avec vous l' engagement qu' il n' y aura pas de laissés pour compte , que notre devoir c' est en même temps de réussir et de faire réussir notre pays , mais de s' assurer que chacune et chacun y trouve sa place ! Ce projet , il est cohérent . Il repose sur une vision d' ensemble où tout se tient , il n' est pas une série de mesures qu' on égrène , qui peuvent se remplacer ici ou là , c' est un projet qui se tient , c' est notre révolution démocratique , celle que nous voulons porter et que nous porterons jusqu' au bout ! Et notre programme , je veux commencer à le partager ici avec vous aujourd'hui . Ce seront les premières de ces propositions de transformation que nous aurons à porter pour le pays . D' autres viendront , jusqu' à la fin du mois de février , sur l' éducation , sur l' Europe , sur l' environnement , sur la culture , sur la défense , sur la sécurité , sur les territoires et l' agriculture , ce sera le contrat que nous passerons avec la nation , le contrat pour gagner , le contrat pour présider , et le contrat pour gouverner ! Mais je veux , ce soir , vous parler des premières batailles , celles contre le chômage , celles contre , précisément , tout ce qui assigne à résidence une partie de nos concitoyens dans la société française , cette bataille que nous devons conduire , la bataille économique et sociale , c' est la première , et celle-là , nous la gagnerons . Parce que depuis plus de trente ans , notre pays ne parvient pas à sortir du chômage de masse . Parce que depuis trente ans , le chômage de longue durée s' est installé dans de trop nombreux territoires , et quelle en est la conséquence ? Des centaines de milliers de nos concitoyens n' ont parfois connu que le chômage , le leur , parfois celui de leurs parents . Ce sont des valeurs à ce moment-là qui se détricotent . C' est un espoir qui s' évanouit . C' est une désespérance qui naît . Mais notre pays aujourd'hui , c' est 3 millions de chômeurs , près de 6 millions de Françaises et de Français qui connaissent une des formes de chômage , presque toutes les familles de notre pays , ce sont 9 millions de nos concitoyens qui sont dans la pauvreté . Et donc , oui , la première des batailles , celle qui fait que notre identité peut se tenir debout , que notre pays peut se porter fier , que les projets peuvent être conduits , c' est la bataille économique et sociale , et c' est celle que nous allons conduire ! Et pour cela , il ne s' agit pas de vouloir administrer contre la France et les Français des potions intenables . Il ne s' agit pas d' aller expliquer au pays qu' on veut être efficace en étant injuste . Ce n' est pas la vision que nous porterons . Je l' entends parfois , c' est une vision conservatrice . Il ne s' agit pas non plus d' aller expliquer au pays que tout sera facile , que nous allons créer de nouveaux droits , que nous allons être justes sans en avoir les moyens et sans être efficaces . La voie qui sera la nôtre , c' est celle du progressisme , celle qui consiste à proposer à notre pays des transformations efficaces et justes . Celle qui consiste pour notre pays à porter ce projet économique et social que nous avons construit ensemble , au concret , au réel , au contact , et qui précisément va nous permettre d'abord de libérer le pays , de le libérer par le travail , de le libérer par ses territoires . Et en même temps , parce que ces deux promesses sont inséparables , de protéger les Françaises et les Français contre ces risques nouveaux , contre ce qui advient , contre ce monde fait de risques . Libérer d'abord , oui , c' est bien le premier socle de notre projet et libérer par le travail , car il sera au cœur de notre action . Mais pourquoi ? Parce que qu' est -ce qui fait le travail ? Qu' est -ce qui le rend si important dans notre société ? Qu' est -ce qui fait , cher Jacques , tu t' en souviens , que près de Château-Thierry , lorsque nous avons rencontré la famille Guyon , tout était fait dans le monde qui nous entoure pour le faire échouer . Des agriculteurs spécialisés dans le porc et la volaille , pensez -vous , mais nous avons vu une famille qui se bat sans relâche par le travail et pour le travail . Qu' est -ce qui fait que d' autres jeunes , dans le pays , trouvent d' un seul coup un avenir , parce qu' une innovation technologique , parce qu' une application sur Internet leur a soudain donné un travail , une dignité , une place , la possibilité de conduire une voiture , de partir le matin et de revenir le soir , quels que soient les donneurs de leçons . Qu' est -ce qui fait Patrick , que tu es là ce soir , que je te connais ? Et chaque jour , je vois que tu défends l' apprentissage comme aucun autre , avec enthousiasme . C' est le travail . C' est ton métier de prothésiste qui t' a fait ça , qui t' a fait là ce soir . Alors , oui , le travail , ce n' est pas une souffrance . Il faut protéger au travail , mais je ne recommande pas de vouloir protéger les Françaises et les Français du travail , ce n' est pas une bonne idée . Le travail , c' est ce qui nous donne une place dans la société , c' est ce qui nous donne notre dignité , c' est ce qui nous permet de nous émanciper , de retrouver confiance en nous , ce qui nous permet de nous construire , ce qui nous permet de réussir et de faire réussir les autres , notre famille , d' entreprendre , d' être heureux . Et je sais les indignations , je sais ces millions de Français qui veulent rentrer dans le travail , et qui de Cahors à Bobigny , en passant par Brest , sont aujourd'hui bloqués , comme entravés , je sais tous ceux qui voudraient embaucher , ils n' y arrivent pas , parce qu' ils ont peur , parce qu' ils se disent que les risques sont trop grands , parce qu' il n' y a aucune fenêtre ouverte . Alors , oui , parce que je veux être le candidat de la justice , je suis le candidat du travail ! J' étais il y a quelques jours à New York , j' y rencontrais Joseph Stiglitz , un affreux gauchiste pour les américains , Prix Nobel de son état , que m' expliquait -il ? Que précisément , le travail était la clef de l' émancipation contemporaine , mais qu' il fallait sortir de cette espèce de frilosité selon laquelle il ne sert à rien de vouloir faire grossir le gâteau , il faudrait simplement le partager , réduire toujours un peu plus , que le travail ne serait qu' une quantité finie . En finir avec ces idées un peu désespérantes . Et que la solution , c' est au contraire de comprendre que le travail se transforme , qu' on ne travaillera plus demain comme aujourd'hui , ni comme hier , et qu' il faut pour cela encourager toutes les formes d' emploi à prospérer , en donnant aux individus les protections dont ils ont besoin , il faut libérer le travail , libérer l' action au travail , la création d' entreprise , l' initiative ! Alors , oui , pour cela , nous allons réconcilier notre pays avec le goût du risque , avec la volonté d' entreprendre , avec la volonté de créer , à tout niveau . Parce que l' artisan crée , parce que l' ouvrier crée , parce que l' entrepreneur crée , parce que l' artiste crée , et c' est cela le socle de notre pays . Mais pour ce faire , la France doit retrouver ce goût du risque , elle doit savoir parfois aussi accepter l' échec . Et pour cela , oui , nous devons , à travers plusieurs transformations , réduire le coût de l' échec , faciliter l' échec quand il advient , parce que ce n' est pas grave , parce que le cœur , c' est d' avoir entrepris , c' est de tenter , parce que quelques-uns échoueront , d' autres réussiront , mais le pays avancera ! Pour cela , nous continuerons à réduire le coût du travail pour les entreprises . Parce que si les coûts , quels qu' ils soient , sont trop élevés , alors les marges sont basses , et alors la capacité à investir et à embaucher n' est plus au rendez-vous . La bataille de notre économie , ce sera la bataille pour l' investissement et pour l' emploi . Elle commence aujourd'hui . Parce que dans ce monde , qui se transforme , où le numérique bouscule tout , où la transformation environnementale et énergétique change des secteurs entiers , nos entreprises se doivent d' investir , se doivent d' embaucher , se doivent de se transformer . Et donc , oui , pour ce faire , nous devons les aider à reconstituer leurs marges , nous devons les aider à créer ces emplois de demain . L' investissement productif et l' emploi , ce seront nos batailles . Alors , pour cela , non seulement je maintiendrai les allègements de cotisations déjà décidés durant les dernières années , mais je transformerai le crédit d' impôt compétitivité emploi en allègement de charge pérenne à hauteur de 6 points , pour toutes nos entreprises , quelles qu' elles soient , quel que soit le secteur . Les entreprises du secteur marchand , comme de l' économie sociale et solidaire , les grandes comme les petites , les indépendants , les artisans , tout le monde aura ses allègements de charges . De manière claire , de manière stable , et de manière lisible . Parce que c' est à la fois une politique pour la compétitivité , et pour l' emploi . Libérer l' accès au travail , cela passe aussi par la libération du dialogue social au plus près de l' entreprise et du terrain . Parce que comment pourrait -on penser aujourd'hui que la loi pourrait décider de tout et partout ? Que la loi pourrait être la même , lorsqu' on a 25 ans , 40 ans , 55 ans , et que , par définition , nous n' avons pas le même rapport au travail , surtout si l' activité au quotidien n' est pas la même . Parce qu' on n' est pas fatigués de la même façon lorsqu' on fait de la politique , lorsqu' on est posté dans une usine automobile , lorsqu' on travaille à Arc international , chère Caroline , nous nous en souvenons , lorsqu' on travaille dans le bâtiment ou les travaux publics , ou la boulangerie , nos vies ne sont plus les mêmes . Parce que , de la même façon , nos secteurs économiques sont en train de se transformer , de se recomposer , avec des logiques profondément différentes . Certains sont en train de se créer , ont besoin de plus de souplesse et de vitalité pour faire face à de nouvelles commandes , et d' autres sont frappés par des restructurations , parce que la concurrence internationale est là , parce que produire en France , sur ces secteurs , est rendu chaque jour de plus en plus difficile , et donc oui , les réalités partout sur le territoire , entre les secteurs de notre économie ne sont plus les mêmes . C' est cette souplesse que nous devons retrouver , mes amis , c' est cette dynamique différente . Il faut l' expliquer , et il faut la construire . Voilà pourquoi je propose que la loi édicte des règles simples , lisibles , définisse un ordre public social qui doit être le même partout et pour tous , lui , qui fixe la durée légale du temps de travail , elle doit rester à 35 heures . Ce n' est pas la peine , alors que je suis en train de vous dire que la réalité n' est pas la même partout dans le pays d' aller expliquer qu' il faudrait dans la loi passer de 35 à 39h , ce n' est pas cela la réponse . Il faut que la loi fixe bien entendu le salaire minimum , les principes fondamentaux et au premier chef l' égalité entre les femmes et les hommes . Et il n' est pas question ici d' obliger par la loi les jeunes à travailler plus ou à gagner moins . J' en entends certains qui voudraient déformer nos propos , qui voudraient faire croire des balivernes , ils sont tellement habitués à ce petit jeu que je veux le repréciser . Non , une loi simple qui fixe des règles minimum pour tous , parce que ce sont les garanties . Et ensuite , nous devons renforcer la négociation au niveau des branches , des secteurs d' activité et des entreprises . C' est comme cela que nous créerons cette République contractuelle à laquelle nous croyons . Qu' est -ce que cela veut dire ? Ça veut dire que je propose que dans chaque branche , que dans chaque entreprise qui le souhaite , par un accord majoritaire , et donc par le dialogue social , et un accord entre les représentants des salariés et les représentants des dirigeants d' entreprises , on puisse définir les bons compromis , les bons accords , qui permettent au plus près du terrain de créer plus d' emplois , parfois de décider de changer les seuils si c' est plus pertinent et si les uns et les autres le veulent , parfois de modifier le nombre d' heures que l' on travaille avec des heures supplémentaires dûment payées , parfois de modifier l' organisation au plus près de l' entreprise parce qu' on l' aura voulu , parce qu' on l' aura choisi , c' est cela ce que nous voulons , c' est cela qui permettra la vitalité de notre économie ! Et dans cette République , où je veux privilégier le contrat à la loi , je vous demande à toutes et tous de jouer votre rôle , parce que c' est un pouvoir que je vous rends . Parce que c' est la décision prise ici solennellement que ce n' est pas un homme qui , devenu président , par des textes ou des décisions , pourra régenter tout ce qui se passe partout dans notre pays , ce n' est plus vrai . Cela n' est plus le XXIe siècle . Parce que de la même façon , cela ne peut pas être une loi prise par un parlement pour tout le pays , qui va régenter toutes les situations , répondre à tous les enjeux pour le pays . C' est vous , vous qui aujourd'hui avez décidé d' être les acteurs du changement politique . Vous qui avez décidé de faire , je vous demande , demain , d' être les acteurs du changement social , et dans vos entreprises , partout , qu' elles soient petites ou grandes , par le dialogue , par le débat , de construire les règles qui seront adaptées à votre quotidien , qui seront adaptées à vos défis , parce que j' ai confiance dans votre intelligence , dans votre énergie , parce que c' est ça la France ! Enfin , parce que je suis le candidat du travail , je veux que le travail paye mieux . Parce qu' aujourd'hui , lorsqu' un employeur verse au SMIC 1467 euros , le salarié ne perçoit que 1141 euros de salaire net . Pourquoi ? Parce que nos grands risques sociaux sont financés sur les salaires , par le travail . Parce que c' est ainsi que nous avons construit nos équilibres , historiquement . Parce que , hier , lorsque , en 1945 , puis dans les décennies qui ont suivies , nous avons construit ce modèle , notre modèle français , qui a été notre fierté , qui a formidablement fonctionné , qui a permis le progrès , lorsqu' on a pensé ce modèle , le chômage était anecdotique . La réalité , c' était le plein emploi . La réalité , c' était une économie de cycles longs , où les bouleversements n' étaient pas d' une brutalité comme nous les connaissons aujourd'hui . Ce n' était pas le même monde . Eh oui , aujourd'hui , le travail est concurrencé non seulement par des pays à bas coût , ou à coût inférieur , mais aussi par la robotisation , par d' autres façons de créer de la richesse , et nous les saisirons , nous continuerons à les saisir . Et donc la base de financement de notre modèle social , de nos sécurités collectives , ne peut plus être le travail . Voilà pourquoi je propose que toutes les cotisations maladie et les cotisations chômage qui sont aujourd'hui payées par les salariés soient supprimées . Ce sera une augmentation du salaire net pour chacun . Une augmentation qui n' est pas payée par l' employeur et qui ne touchera donc pas la compétitivité de nos entreprises , mais qui sera transférée sur la CSG . Qu' est -ce que cela veut dire concrètement ? Cela veut dire que pour tous les salariés , tous les fonctionnaires , tous les indépendants de notre pays , toutes celles et ceux qui travaillent , le salaire net pour tous augmentera . La CSG , elle , augmentera de manière limitée , parce que sa base est beaucoup plus large , d' 1,7 point , mais je protègerai dans cette décision celles et ceux qui sont au chômage , et 40 % des retraités les plus modestes . Ils ne seront pas touchés . Et je veux ici être très clair . Qu' est -ce que cela veut dire , pour vous donner un exemple ? Qu' un couple de travailleurs qui touche aujourd'hui le SMIC aura , après cette transformation , 500 euros nets par an de pouvoir d' achat en plus . Alors , à qui je demande un effort ? Parce que notre règle , c' est de toujours dire la vérité . On ne va pas cacher les choses . Et je vous le dis , toutes celles et ceux qui aujourd'hui font des promesses formidables n' expliquent jamais à qui on demande des efforts . Je le sais bien , on dit : cachons -le , ils le verront suffisamment tard , ils s' en rendront compte après l' élection . J' ai connu ça . Ça peut marcher le temps d' une élection , ça se passe moins bien après . Alors , je vais vous dire , je demande un effort , évidemment à tous les travailleurs , mais qui est plus faible que ce qu' ils vont gagner , donc le pouvoir d' achat de tous les travailleurs va augmenter . Et je demande un effort net à deux catégories : les revenus du capital , 1,7 point de CSG . Ce n' est pas énorme . Et cela s' inscrira dans une refonte en profondeur que je présenterai dans les prochaines semaines . Et de l' autre côté , les retraités les plus aisés , la moitié des retraités les plus aisés . Je leur demande , pourquoi ? Je leur demande pour leurs enfants et leurs petits-enfants . Je n' augmenterai pas leurs impôts par ailleurs , je vous le garantis ici . Je n' augmenterai pas le coût de la maladie , je le garantis ici , et j' y reviendrai . Je leur demande simplement en toute transparence ce petit effort de quelques dizaines d' euros par mois , parce que nous en avons besoin pour que le travail dans notre pays paie . Mais en même temps , je veux vous le dire ici avec beaucoup de solennité , je protègerai le pouvoir d' achat de tous les retraités modestes pendant toute la durée du quinquennat , parce que durant les deux quinquennats qui ont été , justement durant ces dix années qui viennent de passer , ils ont beaucoup trop été mis à contribution . Tous ces retraités modestes , nous les protégerons , il n' y aura aucune baisse de leur pouvoir d' achat dans les cinq années à venir . Et donc , vous l' avez compris , le travail paiera mieux , dès demain , pour les fonctionnaires , pour les artisans , les indépendants , pour les salariés dans notre pays . Mais je suis aussi convaincu que l' incitation à la reprise d' un emploi devra être renforcée par une augmentation de la prime d' activité , parce qu' il n' est pas normal aujourd'hui qu' un salarié à mi-temps au SMIC ne voit quasiment pas sa rémunération augmenter lorsqu' il reprend une activité à temps plein . Et donc cela aussi , nous le modifierons pour que partout dans notre pays , vous ne puissiez plus entendre qu' il est plus intéressant de ne pas travailler ou de travailler moins que de se remettre au travail ! La libération ! La libération du pays que nous voulons , mes amis , c' est aussi une libération de tous les territoires , des Françaises et des Français qui font , partout dans le pays , c' est à eux que je veux rendre le pouvoir , c' est à eux que je veux donner justement toute la reconnaissance et les responsabilités . Alors comment ? D'abord en donnant plus de responsabilités à nos élus locaux , parce que , oui , nous devons aller plus loin dans la décentralisation . Parce que , oui , partout sur le terrain , nos élus locaux sont ceux qui font , qui entreprennent , au quotidien , qui savent la responsabilité . Non pas tous les cinq ans ou tous les six ans , ils la connaissent tous les jours . Tous les jours . Alors , je souhaite leur donner plus de responsabilités parce que notre République est grande , elle est indivisible , nous allons la rendre plus forte , mais elle sera aussi plus forte en reconnaissant partout ses énergies , ses spécificités , cette Bretagne conquérante , cette Corse fière d' elle-même , cette volonté justement d' entreprendre partout sur le territoire avec une énergie différente dans un même projet , le projet de la République , pas le projet du XVIIIe siècle , mais celui du XXIe siècle ! Je veux que les métropoles , oui , Gérard , cette belle métropole de Lyon , et tant d' autres , je veux qu' elle continue à réussir encore davantage . Tu as montré l' exemple en réussissant à reprendre les compétences du département , à simplifier , pour de vrai , pas dans un texte , pour de vrai , ce millefeuille dont on nous parle tant ! Je suis pour que partout , lorsqu' il y a des métropoles qui réussissent , lorsqu' il y a ces grandes cités triomphent dans la mondialisation , qu' on puisse les aider à aller plus loin , je suis pour qu' elles puissent absorber le département , qu' on simplifie notre structure pour rendre ces territoires plus forts . Parce que ces métropoles doivent continuer à rayonner , à entraîner d' autres territoires voisins , d' autres villes , c' est leur responsabilité . Parce qu' il y a aussi les quartiers populaires dans nos villes , ou les villes les plus en difficulté . Je veux que nous prenions tous les risques , que nous allons vers des nouveaux schémas d' urbanisme , que nous prenions enfin nos responsabilités en termes de rénovation urbaine . Nous avons divisé par deux le budget de l' ANRU ces dernières années , nous devons reconquérir cela pour refaire nos villes belles . Cher Roland , je te sais ici … Pour dans ces quartiers , comme à Poissy , à la Coudraie , comme dans d' autres quartiers cela a été fait , recréer de la mixité , recréer du logement , redonner de l' énergie , savoir recréer la vraie mixité sociale et économique dont notre pays a besoin ! Et dans nos territoires ruraux , là , nous avons besoin des départements . La République doit être plurielle . Nous avons besoin des départements , de ces villes-centres , nous avons besoin de ces communes , des services publics , nous avons besoin , oui , des services postaux et des écoles , de la possibilité de reconquérir des places économiques , sociales , de l' activité partout , parce que ces territoires aussi ont de l' innovation économique , sociale , parce que ces territoires veulent entreprendre , de la Lozère en passant par la Corrèze , nous devons leur faciliter la vie , et c' est ce que nous entreprendrons ! Et pour les y aider , nous allons amorcer là aussi un grand mouvement de transformation de l' Etat . Je propose une déconcentration nouvelle , c'est-à-dire je propose que la responsabilité , que la capacité de faire , ce ne soit plus celle d' un ministre , ce ne soit plus celle d' une administration , de quelques-uns en haut , mais des femmes et des hommes qui , sur le terrain , se battent pour faire , pour transformer le pays , de tous ces fonctionnaires et ces services qui , sur le terrain , qu' ils soient dans les hôpitaux , dans les écoles , dans les universités , connaissent la dynamique en œuvre , connaissent les problématiques complexes et veulent changer les choses ! Et donc , oui , je propose plus d' autonomie , la vraie autonomie , pour les universités . Je propose l' autonomie des établissements scolaires et des communautés pédagogiques . Des hôpitaux et des agences régionales de santé , pour pouvoir faire , comme je l' ai vu à Sallanches par exemple , il y a quelques semaines , où lorsqu' on donne la possibilité d' initiative , la capacité d' inventer , nos fonctionnaires , dont on nous parle tant , dont on dit tant de mal , nos fonctionnaires qu' on ne sait plus citer dans le pays que pour savoir le nombre de têtes qu' il faut couper . Nos fonctionnaires auront alors la pleine capacité de faire . Parce qu' aimer l' Etat , aimer la fonction publique , aimer le service public , ce n' est pas dire : il faut tout supprimer , il faut tout couper d' un côté ou il ne faut rien changer de l' autre et garder en l' état . Non , c' est de dire que partout dans notre pays , nous avons une fonction publique d' une qualité incroyable , nous avons une fonction publique dont nous devons être fiers , mais nous devons la libérer , nous devons la laisser innover , nous devons la laisser inventer ! Et donc , oui , je crois à une fonction publique entreprenante , qui puisse accompagner les territoires , qui puisse trouver ses solutions , qui ait la possibilité de décider . Et en même temps , je propose une véritable simplification , non pas , et nous essayerons de le faire , en supprimant des normes , parce que pour cela , vous savez , il faut imaginer Sisyphe heureux , et il n' était pas heureux , je dois vous le dire . Ma conviction , c' est que personne ne devient fonctionnaire pour appliquer bêtement des lois , pour embêter , pour traquer , pour appliquer sans réfléchir des normes parfois idiotes , pour le perroquet d' instruction . Il y a des règles à faire respecter partout , c' est une évidence , c' est notre ordre public , et la révolution que je propose , c' est de changer le rapport des Français à leur administration et le rapport de l' administration aux Français . Ce que je propose , c' est de créer partout un droit à l' erreur pour tous ! Qu' est -ce que cela veut dire ? Que les contrôles opérés par les pouvoirs publics se feront non plus d'abord pour sanctionner , mais pour orienter ! Ce que je veux dire , c' est que le contrôler de bonne foi pourra faire valoir son droit à l' erreur . Et ne sera donc pas à la première occasion sanctionné , ce sera à l' administration qu' il reviendra de prouver que l' erreur a été commise intentionnellement , ou que la personne est de mauvaise foi . Sauf , bien sûr , je vous rassure , parce que j' entends déjà les déformations , en matière pénale , ou lorsqu' elle engage la responsabilité d' autrui , où là , nous serons intangibles , mais ce droit à l' erreur , c' est la possibilité par exemple , lorsqu' un employeur oublie de déclarer à l' URSSAF la prime de Noël , qu' il verse à ses salariés , qu' il ne soit pas condamné immédiatement à une amende , ce qui est le cas aujourd'hui . Il l' a rarement fait exprès . C' est de faire qu' un employeur qui effectue en retard une déclaration préalable d' embauche est aujourd'hui inquiété par l' administration et que demain , l' administration lui conseillera d'abord , lui expliquera . C' est passer d' une administration du contrôle et de la sanction systématique à une administration du contrôle , de la vigilance mais du conseil et de l' accompagnement . Il y aura toujours des fraudeurs , il y aura des malhonnêtes . Mais nous devons en même temps redonner dans notre pays place à la confiance , c' est cela ce que permet cette transformation , c' est cela ce que permet cette révolution , elle accompagne tous ceux et celles qui osent , qui entreprennent , et qui veulent faire ! Et en même temps , mes amis , que nous devons libérer le pays , je vous le disais , nous devons protéger . Protéger nos concitoyens contre les nouveaux risques , contre ces transformations à l' œuvre , contre un monde qui change , parce que libérer pour permettre d' entreprendre , d' oser , de créer , de réussir , ou d' échouer , c' est formidable , mais il y aura toujours des fragiles , il y aura toujours des oubliés , et notre devoir , notre projet parce qu' il est progressiste , c' est de leur aménager une place , c' est de les aider . Et donc notre projet , pour protéger , sera autour de trois boucliers , un bouclier de sécurité , un bouclier social actif , et un bouclier européen , car c' est ainsi que nous protégerons les Françaises et les Français ! Alors , oui , le premier bouclier , c' est celui de la sécurité . Parce qu' il n' y a pas de liberté qui vaille si chaque jour , on a peur de l' insécurité . Et quand on a peur de l' insécurité , il n' y en a pas de petite ou de grande . Il n' y a que la peur d' être , il n' y a que la liberté d' aller et venir dont on est alors privé . La sécurité , c' est ce premier devoir de la puissance publique , c' est ce premier devoir qui nous unit au cœur du pacte républicain , de faire que partout en France , les femmes au premier chef , et les hommes , ne doivent plus être inquiétés , qu' il ne doit plus y avoir de territoire de non droit , et là-dessus , je serai clair : l' autorité de l' Etat doit être intangible , et notre autorité sera inflexible ! Parce que l' égalité que nous défendrons , la vraie égalité , c' est l' égalité face à la sécurité . Et nous le savons . Nous savons ce que vivent nos forces de l' ordre , partout dans le territoire , le manque de moyens , pas de radio , parfois pas assez d' équipements , lorsque les équipements sont là , j' étais , l' autre jour , dans un commissariat , les coffres sont trop petits , parfois les logiciels qui ne fonctionnent pas , qui mobilisent quatre fonctionnaires pour procéder aux vérifications d' usages , donc oui être à la hauteur de ce bouclier de sécurité , c' est reconnaître l' importance et la place de nos forces de l' ordre , c' est leur donner des moyens dans la durée , c' est non pas simplement avoir des grands mots pour elles , mais une reconnaissance concrète , un sérieux au quotidien . C' est cela ce que nous ferons . Alors , pour ce faire , au-delà des 9000 embauches décidées , et qui sont encore en cours , ce seront 10 000 fonctionnaires de police et de gendarmerie supplémentaires qui seront recrutés durant les trois premières années du quinquennat . C' est une nécessité , nous le ferons ! Mais il ne suffit pas de recruter , il faut rendre du temps utile aux policiers et aux gendarmes pour être efficaces , pour débarrasser de formalités inutiles , inventées parfois pour se faire plaisir , pour la pureté du droit ou parce qu' on a cru , loin du terrain , que c' était une bonne idée . Il faut simplifier les procédures . Nos forces de police et de gendarmerie le savent , nos magistrats aussi et nous avons travaillé avec eux . Nous réduirons ainsi les contraintes sans réduire en rien les droits et les libertés de nos concitoyens . A côté de ces simplifications et de ces engagements , nous devons aussi nous réorganiser . Et de deux façons . D'abord en acceptant l' idée que nous devons , oui , reconstituer notre renseignement territorial dans les endroits les plus sensibles du pays . Il a été déconstruit au début de notre siècle , de manière maladroite , à contretemps , nous devons dans les quartiers les plus sensibles , dans toutes les zones de France , ainsi identifiée , remettre des forces de renseignement , les vraies , au contact du terrain , avec des fonctionnaires de qualité , formés pour ça , parce que c' est là que le véritable risque terroriste peut être repéré , parce que c' est là que les signes avant-coureurs sont identifiés . Et dans le même temps , nous devons mieux organiser notre renseignement pour que toutes les données collectées partout sur le territoire puissent être regroupées dans une organisation unique , simplifiée , plus cohérente et efficace , sous évidemment le contrôle démocratique , qui convient , mais comme l' ont fait les Britanniques et les Américains pour que nous soyons efficaces au regard des technologies contemporaines pour lutter contre le risque terroriste . Nous devons aussi , c' est la deuxième réorganisation que je veux proposer , recréer une nouvelle police de proximité . Parce que , oui , cette police de proximité , créée à la fin des années 90 par Lionel Jospin et Jean-Pierre Chevènement , elle était une bonne idée . Parce que , oui , cette police de proximité , il faut bien le dire , elle ne servait pas uniquement à jouer au football avec les enfants . Et parce que , même quand on joue au football , avec des enfants , quand on est présent sur le terrain , on voit d' autres choses , on crée un autre rapport à l' autorité républicaine , on perçoit les signes avant-coureurs , on comprend ce qui est en train de se jouer , on prévient certains risques , on reconstruit différemment de la République partout sur le terrain . Donc , oui , nous recréerons cette police de proximité . Voilà pour la sécurité physique . Et pour protéger les Français , nous aurons un deuxième bouclier , le bouclier social , dont ils ont besoin . Ce bouclier social , c' est celui qui permettra à chacune et chacun non pas d' avoir tous les droits , sans devoir , mais qui , au regard des transformations en cours , permettra de trouver sa place dans la société , d' être considéré comme une personne , d' être pleinement reconnu et d' être protégé contre les événements adverses qui parfois nous arrivent dans la vie . Alors , le premier de ces devoirs , c' est évidemment l' éducation . J' y reviendrai plus longuement , mais je veux ici en dire un mot . Nous avons un devoir d' éducation partout dans le pays . Il ne peut pas être le même dans tous les quartiers de la République , il ne peut pas être le même partout , nous devons reconnaître qu' il faut faire plus pour ceux qui ont moins , que des inégalités se sont creusées , construites , où nous avons assigné à résidence des femmes et des hommes et leurs enfants , parce que nous n' éduquons plus comme nous devrions . Parce que celles et ceux qui , dans les quartiers populaires , sont les enfants des chômeurs de longue durée , de Françaises ou de Français qui parfois ne maîtrisent pas parfaitement notre langue , n' ont pas les mêmes chances de départ . Si nous avons un Français sur cinq , un jeune Français sur cinq en CM2 qui ne sait pas lire , écrire , compter , c' est que dans les quartiers les plus difficiles de la République , qu' ils soient ruraux ou qu' ils soient urbains , il y en a parfois un sur deux ou deux sur trois qui sont dans cette situation , c' est cela la réalité . J' étais il y a quelques semaines en Moselle . À Hombourg-Haut , cette petite ville de Moselle qui a souffert de la désindustrialisation , la fermeture des mines , il y a maintenant plus de trente ans . Et je vais dans le centre d' action sociale qui avait plusieurs fois brûlé , avec des femmes et des hommes extraordinaires qui étaient là pour créer de l' activité , pour les mères au foyer , de l' activité pour les plus jeunes après l' école , et du soutien scolaire . Ça se passait dans une école désaffectée , parce que cette ville a tant souffert qu' elle a perdu 5000 habitants et qu' une école a dû fermer . Et je croise une petite fille qui était au milieu des mamans qui faisaient du tricot , et je demande à la petite fille : tu as quel âge ? Elle me dit : j' ai 7 ans . Je suis en CE1 . Et je lui demande : qu' est -ce que tu veux faire ? On n' a pas des questions toujours très originales quand on est en déplacement . Elle me dit : je veux être traductrice . Je lui dis : c' est formidable ! Tu veux être traductrice ? Mais tu veux voyager , tu veux emmener les gens dans quel pays , tu veux partir dans quelle langue ? Elle se tourne vers sa mère . Alors , je demande à sa maman : c' est formidable . Mais qu' est -ce qu' elle veut , elle veut voyager ? Et la maman regarde sa petite fille et elle lui parle en arabe . Et j' ai alors compris que si la petite fille voulait devenir traductrice , ce n' était pas pour voyager comme je l' avais bêtement cru . Ce n' était pas pour emmener des gens très loin comme j' avais bêtement pensé . C' était simplement pour permettre à sa maman d' être dans la République . Alors , oui , pour l' école , nous ferons beaucoup . Dans ces endroits de la République , nous dédoublerons vraiment les classes , dans ces endroits de la République , nous ferons ce que nous devons faire , parce que cette petite fille , elle n' était pas née en Afghanistan , au Maroc , en Tunisie , elle était née à Hambourg-Haut , sa maman était née à Hambourg- Haut , et c' était en France ! Et notre bouclier social , ce sera aussi celui de la santé , là aussi , j' y reviendrai , mais il faut que chaque Française et Français puisse se soigner , accéder à la santé quel que soit son âge , son niveau de revenu , son origine . Et je trouve à cet égard dangereux que certains se mettent dans l' idée de dérembourser massivement des soins , des petits soins , nous dit -on , de dérembourser massivement dans le pays , pour les Françaises et les Français , ce qu' il faut pour se soigner , parce que c' est à la fois inefficace et injuste . C' est inefficace parce que la bataille de notre système de santé , c' est la prévention , parce que si la France est performante , c' est en matière de soins , si elle est défaillante , c' est en matière de santé . Et dérembourser les petits soins , quelle en est la conséquence mes amis ? C' est que les classes moyennes iront moins se soigner , c' est qu' on ira moins se couvrir contre les petits risques , c' est qu' on ira plus facilement vers les maladies les plus graves . Et en même temps , c' est injuste , parce que dérembourser les petits soins , c' est évidemment , non pas toucher les plus fortunés , qui peuvent avoir accès à d' autres couvertures privées , qui sont aussi nécessaires , ce n' est pas toucher les plus modestes , parce qu' ils sont , et nous le préserverons de manière absolue , couverts par la solidarité nationale , la couverture maladie universelle , générale et complémentaire , c' est toucher les classes moyennes , celles et ceux qui , comme d' habitude , sont au milieu , financent mais ne peuvent pas , elles , être protégées . Et donc ce projet de la droite conservatrice , quel en sera le résultat ? Je vais vous le dire simplement , avec la même transparence que quand je vous propose des choses . 1200 euros de dépenses de santé en plus par an pour un couple avec deux enfants , c' est cela le projet de la droite conservatrice , ce sont les chiffres de la fédération nationale de la Mutualité française . Un SMIC par an ! Donc ce projet , non seulement nous le combattrons , mais je veux le dire avec la plus grande clarté , pendant les cinq ans qui viennent , pas un seul soin utile ne sera déremboursé . Nous procéderons aux réformes , et j' y reviendrai , pour la prévention , pour réformer et transformer l' hôpital public , pour évidemment décloisonner le public et le privé au plus près du territoire , en donnant tout leur rôle aux mutuelles comme à la sécurité sociale , pour construire l' offre de soins adaptée partout et lutter contre les déserts médicaux . Et le bouclier social dont je veux plus particulièrement vous parler ce soir , c' est celui qui permettra que tout le monde ait les mêmes droits , et je veux ici parler du chômage et de la formation continue . C' est cela le cœur de la bataille et c' est le cœur de la bataille que nous mènerons ! Parce que dans ce monde qui se transforme , je vous le dis en toute honnêteté , le chômage ne peut plus être un risque contre lequel chacun se couvre à titre individuel . Cela ne peut plus être une assurance que chacun finance . Le chômage doit être une forme de solidarité universelle qui permettra de couvrir les Françaises et les Français dans tous les secteurs , qu' ils soient salariés , qu' ils soient indépendants et entrepreneurs . Je veux que chacune et chacun soit protégé de la même façon . C' est pour ça que je veux faire du chômage une vraie assurance universelle , une vraie solidarité nouvelle qui reconnaisse la dignité de la personne elle-même et qui ne couvre pas chacune et chacun simplement en fonction de ses cotisations . Concrètement , cela veut dire quoi ? Cela veut dire que chacune et chacun ne paiera plus son chômage , comme vous l' avez compris . C' est ce que j' expliquais tout à l' heure . Cela veut dire concrètement que chacune et chacun , quel que soit son statut , aura ce chômage et que nous sortirons de cette injustice où l' auto-entrepreneur , l' indépendant , l' artisan , l' entrepreneur n' a pas aujourd'hui ce chômage . C' est aussi pour cela que je veux que tous les cinq ans , pour éviter tout excès , tous les salariés puissent avoir le droit d' accéder également à ce droit de chômage s' il décide de démissionner . Pourquoi ? De manière très concrète . Parce que , aujourd'hui , qu' est -ce qui se passe ? Parce que , aujourd'hui , vous avez partout en France des gens qui contournent le droit , et qui ont recours à la rupture conventionnelle , pour faire semblant de démissionner et avoir le chômage . Donc ne vivons pas dans un monde d' hypocrites ! Parce que , quand ils ne font pas ça , qu' est -ce qui se passe ? Pour certains salariés , ils disent : tu ne veux pas me licencier , parce que tu ne veux pas me payer le chômage . Je vais te faire une vie impossible . C' est ça qui alimente les Prud'hommes , chaque jour . Et parce que de l' autre côté , quand il y a des salariés malheureux , ils vont voir leur employeur , et il leur dit : tu n' es pas heureux ici ? Il y en a beaucoup qui attendent dehors . Et c' est comme ça qu' on construit la souffrance au travail , ce burn out dont on parle tant , les frustrations . Donc ce que nous avons fait … Ce que nous créons ce soir ensemble , mes amis , c' est un véritable droit à la mobilité professionnelle , c' est un droit qui reconnaît à tous les travailleurs du pays la possibilité de choisir , la possibilité d' être couvert contre tous les risques , la possibilité d' être protégé si demain il y a un accident dans son secteur d' activité , s' il y a un risque . Oui ce que je veux que nous créions ensemble , c' est ce droit universel , ce droit à la mobilité professionnelle qui reconnaît pleinement la dignité de la personne au travail . Et ce droit , il ira évidemment avec des devoirs , évidemment . Le contrôle sera permanent , ce qu' il y a en face de ce droit , c' est la possibilité d' être indemnisé , si on est licencié , s' il y a un accident de la vie , si justement tous les cinq ans , face à ces contraintes , on le décide . Mais en face , dans le même temps , c' est un devoir . Le devoir de faire un bilan de compétences , et de regarder toutes les options qui sont alors proposées en face . Et le devoir , lorsqu' une offre d' emploi existe , qu' elle est décente et qu' elle correspond aux compétences , de la prendre . Donc oui , en face de ce droit , c' est un devoir posé aux individus , un devoir de travailler lorsque le travail est proposé . Mais en même temps , soyons honnêtes , que se passe -t-il , bien souvent ? C' est que nombre de nos concitoyens qui se retrouvent au chômage , alors , certains , mais ils sont rares , j' entends les discours caricaturaux , il y en a , certains décident de rester au chômage en disant : c' est mon droit , j' ai cotisé , je reste au chômage , je ne prends pas le travail qui m' est donné . Avec notre système , ce n' est plus un droit , c' est une solidarité , donc si le travail correspond aux compétences et est décent , il doit être accepté ! Mais dans l' autre temps , il y a des millions de nos concitoyens , et il y en aura de plus en plus , qui arrivent , touchés par une restructuration , un changement , et à qui on ne propose rien , et qui restent là . Notre devoir pour répondre à ce défi , ce n' est pas de considérer que leur droit , c' est d' être au chômage . Le devoir de la puissance publique , c' est alors de les former pour pouvoir devenir quelqu'un d' autre et pour pouvoir rebondir . Et aujourd'hui , nous ne le faisons que trop peu . Notre devoir , c' est alors d' accepter que pendant un an , deux ans , trois ans , toutes ces Françaises et ces Français , fragiles , qui n' ont plus de perspective , puissent être reformés pour devenir quelqu'un d' autre . Parce que qu' est -ce qu' il va se passer , mes amis ? Dans les cinq ou dix ans qui viennent , cette révolution que j' évoquais tout à l' heure va balayer des secteurs entiers d' activités . Elle va faire que certains de nos concitoyens qui sont chargés de clientèles par exemple dans le secteur bancaire vont perdre leur activité , parce qu' ils seront remplacés par des logiciels , des algorithmes , des robots . C' est la vérité . Alors , on peut décider comme on l' a tant fait , de ne pas voir , de vouloir ou de prétendre , protéger les emplois en disant : on va vous protéger , et de les laisser se faire balayer . Mais le jour où ils seront licenciés , nous , aujourd'hui , nous ne sommes pas capables de les former , aujourd'hui , nous les laissons simplement au chômage , aujourd'hui , nous proposons peut-être de les former pour se qualifier un peu mieux dans leur secteur d' activité , mais ces Françaises et ces Français-là , il faudra les former demain pendant deux ou trois ans pour changer de secteur , pour aller vers de nouveaux domaines d' activité , pour aller vers la silver economy , vers l' écologie , vers de nouveaux secteurs en développement , c' est cela notre défi . Et donc , pour ce faire , parce que je veux reconnaître la dignité de toutes les personnes et une place à chacun , je ne reconnais pas un droit d' être indemnisé , je nous reconnais un devoir de former . Alors oui , ce que nous allons créer pour chaque Française et Français , c' est ce nouveau service public de la formation et de l' activité . C' est celui qui , en réformant Pôle emploi et notre système de formation continue permettra de donner une place à chacun . Une vraie place . La place que je veux . Mes amis , c' est cela notre défi . C' est cela . C' est permettre au quotidien , grâce à ces transformations , c' est permettre de réformer notre pays , de donner une place à tous , un destin , une perspective , enfin ! Vous l' avez compris , je veux que ce bouclier social permette à chacun de construire sa vie . Que cette transformation de notre formation permette à tous et toutes de retrouver une place , avec ses droits , mais en même temps ses devoirs , parce que , à l' issue de la formation , je veux que chacune et chacun prenne ses responsabilités et l' offre qui est faite ! Enfin , protéger les Français , les protéger face à ces nouveaux risques , à cette mondialisation , c' est vouloir l' Europe , et ce bouclier européen . Ah , je vous rassure , nous n' avons sur ce sujet pas beaucoup de concurrence . Non , je vous rassure , personne ne parle plus d' Europe ! Et quand je regarde tous les candidats à l' élection présidentielle ou aux primaires dont on nous parle tant , il n' y a que des anti-européens , que des eurosceptiques ou des eurosceptiques mollement convertis . Nous , mes amis , nous aimons l' Europe ! Nous voulons l' Europe ! Parce que face à ces nouveaux risques , face aux risques de la mondialisation , seule l' Europe peut nous protéger . Parce que , oui , le monde redevient tragique . Il y a les risques climatiques , il y a les risques économiques , dont nous avons tant parlé , il y a le terrorisme , les migrations . Et il y a à quelques milliers de kilomètres d' ici , ce drame d' Alep qui nous rappelle , si nous l' avions oublié , en ce jour du 10 décembre , en ce jour des droits de l' Homme et du citoyen , il y a Alep où se joue la plus grande atrocité de notre siècle , où nous avons laissé d' autres bombarder et tuer des civils de manière atroce , où nous guettons chaque jour ces messages de la petite Bana et de sa maman Fatemah qui , sous les bombes , racontent leur calvaire . Alors , oui , dans ce monde redevenu tragique , la voix de la France est essentielle . Sa voix indépendante . Celle qui doit porter , non seulement , la lutte contre le terrorisme , mais la lutte partout intransigeante contre celles et ceux qui bafouent les droits de l' Homme , contre celles et ceux qui nous oublient ! Et jamais nous ne devons accepter la fatalité de ceux qui prétendent conduire la Real Politik en renonçant à leurs valeurs . Je crois à la Real Politik , je regarde comme vous notre monde , mais je le veux avec nos valeurs ! Alors , oui , mes amis , dans ce monde , nous avons besoin d' Europe , de plus d' Europe . Face à Google , aux géants de l' Internet , comment protéger nos concitoyens et faire réussir nos entreprises ? Je vous le dis , il n' y aura pas de géants de l' Internet sans marché unique européen du numérique . Ce n' est pas vrai ! Je ne serai pas le président du champion du numérique français . Je veux être le président du champion du numérique européen , c' est la seule voie ! Comment promettre à nos concitoyens de protéger leurs droits au respect de la vie privée ? Par l' Europe , par ce marché unique du numérique , et par aucune autre façon . Comment protéger notre environnement ? Notre écologie , et porter notre projet ? Par l' Europe , là aussi . Et résolument ! Je reviendrai sur notre projet environnemental , mais seule l' Europe peut faire un vrai marché unique , avec un prix du carbone , avec une vraie énergie européenne , avec une vraie capacité d' innovation . Comment protéger nos travailleurs , nos entreprises , face à la concurrence déloyale des Chinois ou des Indiens ? Dans les pneus ou dans l' acier ? Par l' Europe , mes amis ! Et uniquement par une vraie politique commerciale et industrielle , par une vraie politique commerciale que d' aucuns , tous les jours , accusent de tous les mots ! Comment résister aux prises de contrôles hostiles par des investisseurs étrangers dans des secteurs stratégiques ? Par l' Europe , là aussi . En nous dotant d' une procédure de contrôle des investissements étrangers à l' image de ce que nous disposons en France et que j' avais porté avec d' autres , quelques-uns de mes prédécesseurs et que nous avons appliqué ! Pour défendre notre alimentation , notre goût à la française , notre agriculture … Vous avez raison . Notre agriculture , nos pratiques , nos circuits courts , notre cuisine et notre gastronomie qui sont notre fierté . Comment le faire ? Qu' est -ce qui va protéger tout cela face à des géants internationaux qui se constituent , à la fusion de Monsanto et de Bayer qui vont décider de toutes les normes et de toutes les règles ? Mais ce n' est pas nous , tout seul , c' est l' Europe , là aussi ! L' Europe , elle est notre meilleure protection , elle est notre meilleur bouclier dans la mondialisation . L' Europe , elle est aussi notre identité , ce qui nous fait , notre rêve commun . Et dans ce monde de fous , je vous le redis , je repense à la fois à cette phrase de François Mitterrand , prononcée il y a une trentaine d' années , quand il disait que le nationalisme , c' est la guerre . Oui , l' Europe seule peut nous empêcher de tomber dans la barbarie , l' Europe seule peut nous empêcher de tomber dans le repli ! Et donc , la sécurité n' est pas à nos frontières , elle est aux frontières de l' Europe , à Lampedusa , pas à Vintimille . Donc oui , aujourd'hui , nous devons oser à nouveau rêver à l' Europe , parce que c' est notre bouclier , nous devons aussi retrouver un peu du rêve de Schuman et de cette Europe de la défense , nous devons un peu retrouver de cette volonté d' aller de l' avant ! L' Europe , nous en reparlerons , c' est notre chance , c' est notre diversité , notre unité , c' est ce qui fait aussi que nous voulons dans l' Europe , et que nous portons ce projet , chère Sylvie , cher Jean , celui que chaque jeune dans notre pays puisse passer un semestre dans un autre pays européen , pour se former , apprendre une langue , se dépayser , revenir plus fort et plus ouvert . Donc si on veut protéger notre pays , si on veut protéger les Français , oui , nous devons être européens ! Alors , mes amis , vous l' avez compris , ce n' est que le début de notre programme , il va continuer ! Pour ceux , mes amis , qui avaient peur qu' En Marche soit une aventure solitaire , ce soir , vous les avez rassurés ! Pour ceux , mes amis , qui avaient peur que nous n' ayons rien à dire , vous les avez rassurés ! Alors , mes amis , ce projet , ce programme , c' est le vôtre . Je ferai sans doute des erreurs , il faudra me pardonner , nous aurons des désaccords , il faudra les surmonter , mais ce projet , ensemble , nous allons le porter , parce que nous croyons dans l' espérance . Mais maintenant , votre responsabilité , avec moi , je vous le dis ce soir , je vous le dis avec beaucoup de gravité , et en même temps beaucoup de joie , beaucoup de fierté , vous m' avez énormément donné depuis avril dernier , énormément . Mais maintenant , notre engagement , notre programme , nous allons le continuer , c' est un engagement efficace et juste pour le pays , c' est l' espérance à laquelle nous croyons , mais je ne veux pas le porter tout seul . Moi , je le porterai dans la durée , je le porterai jusqu' au bout , mais maintenant , votre responsabilité , c' est d' aller partout en France , pour le porter , et pour gagner ! Ce que je veux , c' est que vous , partout , vous alliez le faire gagner , parce que c' est notre projet ! Vive la République , vive la France ! Merci à vous ! Merci à vous ! ",
    "Le terrorisme fanatique frappe toutes les nations , toutes les populations , toutes les religions . Il a frappé à Istanbul le 1er janvier , dans une nuit de joie devenue une nuit d' horreur . Il avait frappé à Berlin le 19 décembre . Ces drames n' ont pas lieu n' importe où . A Istanbul , il a frappé un lieu de fête où la jeunesse se réunit . A Berlin , il a frappé des symboles : un marché de Noël , un lieu de rencontres , un lieu populaire ; devant l' église du Souvenir qui porte la croix de la cathédrale de Coventry , toutes deux détruites par la guerre , emblèmes des drames qui ont déchiré notre continent . A Berlin , c' est l' Europe qui , une fois de plus , a été mise à l' épreuve . Jamais la France et l' Allemagne , depuis la réconciliation mise en œuvre par des Européens engagés et des patriotes lucides , n' ont été aussi proches face aux menaces communes . Jamais nos deux pays n' ont eu autant besoin de se témoigner leur solidarité . Avant même de proposer des projets conjoints , nous devons affirmer notre communauté de destin . Le 11 janvier 2015 , dans le plus grand élan de solidarité que notre pays a connu depuis la Libération , la chancelière , Angela Merkel , est venue à Paris exprimer avec émotion que l' Allemagne vivait ce drame avec nous . Aujourd'hui , nous sommes tous berlinois . Le terrorisme ne connaît pas les frontières . Il n' a pas la France pour ennemie ou l' Allemagne pour adversaire . Le terrorisme veut détruire l' Europe . Il veut en finir avec notre civilisation commune . Il veut défaire nos valeurs collectives . Il porte la mort quand l' union des Européens entend défendre la paix et permettre la vie commune . Voilà pourquoi nous nous sentons européens , comme des millions d' Allemands se sont sentis européens en janvier 2015 et lors des tragiques attentats qui ont suivi . L' unité sereine avec laquelle la société allemande a réagi à cette attaque , comme aux récents événements de Munich , est admirable . Les polémiques viendront . Elles commencent à poindre , pas seulement outre-Rhin . Il n' a fallu que quelques heures pour instruire le procès : en ouvrant ses frontières aux migrants , la chancelière aurait exposé l' Europe aux pires dangers , et , aujourd'hui , sa propre capitale . Rien n' est plus faux que cette abjecte simplification . Nos sociétés fragilisées , épuisées par la crise , éprouvées par le fracas du monde – guerres , migrations , ouverture rapide aux échanges , changement climatique ou bouleversement numérique – sont tentées de chercher le mal à l' extérieur . Il suffirait alors de l' expulser de nos communautés nationales , de se barricader , chacun de son côté . Les attentats de Paris , de Nice ou de Berlin seraient‑ils liés au laxisme migratoire ? Ont -ils été le fait de seuls étrangers , de migrants récemment installés ? S' il est une erreur que les Européens ont faite , c' est de ne pas avoir cherché d' emblée une réponse commune face au défi migratoire : quand l' Italie affrontait seule l' arrivée des réfugiés à Lampedusa , au point que le pape François s' en est vivement ému , ni la France ni l' Allemagne n' ont été au rendez-vous . De même , les Grecs ont longtemps été en première ligne , démunis et dépassés , face à l' afflux de réfugiés et de migrants . Pour autant , la chancelière Merkel et la société allemande dans son ensemble ont été à la hauteur de nos valeurs communes ; elles ont sauvé notre dignité collective en accueillant des réfugiés en détresse , en les logeant , en les formant . En refusant de reconstruire des murs dans une Europe qui en a trop souffert , en évitant les amalgames face aux événements les plus cruels , comme les sinistres violences contre les femmes perpétrées l' an dernier à Cologne . Ayons le courage de le dire : la solution est dans la protection , non dans la fermeture , dans une coopération européenne plus forte et non dans un repli national inefficace . La défense de notre sécurité , la protection de nos frontières , la gestion des flux migratoires , l' intégration des réfugiés sont des défis complexes , durables , que nous devons surmonter ensemble . L' Union européenne doit franchir , sans attendre , un nouveau pas : à brève échéance , nous devons créer une Europe de la souveraineté , capable de nous protéger des menaces extérieures pour mieux assurer la sécurité intérieure . Les accords de Schengen , si bénéfiques à nos échanges , comme les 350 000 travailleurs frontaliers français le vivent chaque jour , doivent être préservés et leur suspension limitée aux situations d' urgence . Dans le même temps , nous devons renforcer le corps de garde-frontières européen , qui est une avancée majeure , en lui donnant compétence pour aider à la protection durable des frontières des Etats membres qui en ont besoin , sans limiter son action aux situations d' urgence . Nous devons , au niveau européen , développer les accords de coopération avec les grands pays d' émigration et de transit , fondés sur une aide au développement , un soutien au contrôle de leurs frontières et un accord sur le retour des migrants non autorisés à entrer ou résider dans l' Union européenne . Nous devons aussi , en surmontant les réticences nationales , créer un système de renseignement commun , qui permette une traque efficace des criminels et des terroristes , voire à terme une police commune pour le crime organisé et le terrorisme . Quand l' Europe est divisée , elle est inefficace . Quand la France et l' Allemagne s' éloignent l' une de l' autre et cessent de servir l' Union tout entière , elles font fausse route . Le 19 décembre , ne pensons pas que l' Allemagne seule était visée : Berlin a été attaqué , la France a été touchée . Cette solidarité nous oblige . Elle nous oblige à faire bloc , à agir ensemble , pour préserver ce que nous sommes . Elle nous invite à être pleinement européens . **** *discours_Macron6  *candidat_Macron  *url_inconnu  *source_Nevers  *date_2017-01-06  *type_Meeting Merci à vous . Bonsoir Mesdames , bonsoir Messieurs , et bonne année . Bonne année à toutes et tous . Un très grand merci pour votre accueil . Un grand merci à tous les deux pour vos mots d' introduction et , cher Denis , pour ton accueil chaleureux dans cette belle ville . Nous avons passé l' après-midi ensemble et je suis heureux d' être parmi vous ce soir , aussi nombreux à Nevers . Merci à madame la Sénatrice , Anne EMERY-DUMAS qui , depuis les premiers jours , marche à nos côtés . Merci à mon ami François PATRIAT , qui n' a pas compté sa peine lui aussi , depuis les premiers jours , quels que soient les aléas de la vie , pour rassembler , convaincre , entraîner , et qui fut un grand Président de cette belle région . Je veux aussi saluer l' ancien Maire de Bourges , Serge LEPELTIER . Merci beaucoup d' être là aujourd'hui avec nous , traduisant en actes ce que Denis THURIOT vient de présenter : le rassemblement des bonnes volontés , de toutes celles et ceux qui , quels que soient nos histoires , nos imaginaires collectifs , nos propres convictions , nos familles de pensée , ont fait qu' autour d' un projet et de valeurs , nous nous retrouvons . Et merci à toutes et tous d' être là avec vos parcours , vos identités , votre diversité , autour de ce même projet que nous allons ensemble porter . Je vous souhaite une très belle année . Et nous savons ce que nous devons , en ces premiers jours , souhaiter à notre pays - et ce sera le fil directeur de cette campagne et de ces prochains mois : réussir . Réussir à entrer pleinement dans ce siècle nouveau . Réussir à gagner dans le XXIème siècle , à libérer les forces vives , les énergies qui sont partout présentes sur le territoire . Et nous savons quoi souhaiter aux Français : leur permettre , à tous , de trouver leur place dans cette société . De réussir avec le pays en ayant chacune et chacun son rôle . Alors , j' ai bien conscience qu' il n' est pas anodin d' organiser un premier rassemblement sur cette belle terre de Bourgogne . Cette terre est chargée d' histoire , chargée de politique , au sens noble du terme . C' est bien sûr une terre mitterrandienne , qui pendant des décennies , a été la terre de François Mitterrand . C' est aussi , et surtout ici à Nevers , la terre de Pierre BEREGOVOY , un homme de proximité , qui aimait y revenir chaque semaine . Un homme respecté par tous pour son action , pour ce qu' il incarnait , pour la transformation apportée à la ville . Je pense à la rénovation du Palais Ducal , je pense aussi au lancement de l' hôpital , sa dernière décision , cher Denis , où nous étions tout à l' heure , qui est ton combat d' aujourd'hui , pour partie . C' est une histoire qui résonne encore dans toute la Nièvre et au-delà . Et je suis content de voir ici quelques visages de cette histoire . Je pense en particulier à Marcel CHARMANT ou René-Pierre SIGNE , ancien sénateur-maire de Château-Chinon qui nous font le plaisir d' être parmi nous ce soir . Monsieur BARDIN aussi , ancien Président du Conseil Général . Et je pense aussi à Didier BOULAUD , qui nous a accueillis tout à l' heure , et qui a inscrit son engagement dans la droite ligne de Pierre BEREGOVOY . Parce qu' on ne construit rien si on ne respecte pas l' histoire , les histoires , dans lesquelles chacune et chacun s' inscrit , dans lesquelles les territoires aussi se sont construits . Non pas pour les répéter , non pas pour s' y enfermer , pour les respecter dans la fidélité vraie que nous leur devons . Celle qui consiste à inventer un avenir commun , à rassembler des énergies , en n' oubliant jamais ce qui nous a fait et les terres où nous sommes . Pour moi ce n' est pas innocent , à ce titre , d' être ici . Et , tu l' as rappelé il y a quelques instants , parce que c' est aussi une ville qui porte une partie de mon histoire familiale . C' est la ville où en effet ma grand-mère , pour la première fois , quitta ses Hautes-Pyrénées natales , pour venir passer plusieurs années , emmenant sa mère avec elle , des années de guerre . Vous savez , on grandit toujours avec ses propres souvenirs d' enfance , mais on grandit parfois avec les souvenirs des autres . On grandit avec des images de lieux qu' on n' a jamais vus , avec les histoires qu' on vous a racontées . Je peux vous dire que j' ai grandi avec Nevers , parce que j' ai grandi avec ces scènes que me racontait ma grand-mère où , jeune enseignante , elle arrivait au rectorat ; ces scènes de guerre , parce que cette ville en a souffert ; ces scènes de liberté extraordinaire , de cette génération qui malgré tout a résisté , et qui a montré , parfois plus que d' autres , l' énergie qui existe dans le peuple français . Donc je peux le dire je connais Nevers , parce que cette ville a peuplé mon enfance , de rues que je n' avais jamais vues , de femmes et d' hommes que je n' avais jamais croisés mais qui ont fait aussi mon histoire familiale . Alors aujourd ' hui , il se passe quelque chose dans le pays . Vous en êtes le témoignage vivant ce soir . Il se passe quelque chose lorsque ce qui n' était il y a quelques mois qu' une volonté folle réunit désormais des milliers de personnes . Il se passe quelque chose lorsque des jeunes , que l' on dit si peu intéressés par la chose publique , s' inscrivent si massivement sur les listes électorales . Parce qu' ils refusent que cette élection qui arrive se fasse sans eux . Il se passe quelque chose lorsque des femmes et des hommes , qui s' étaient parfois combattus ou qui ne croyaient plus dans l' action publique se retrouvent dans une même salle , pour penser , partager , vouloir agir ensemble et transformer le pays . Il se passe quelque chose lorsque tant de nos concitoyens , déçus par la politique , fatigués par tant d' années de divisions et de diversions , expriment de nouveau un intérêt , une attente , une espérance . Parce qu' ils sentent la sincérité de notre mouvement . Oui , il se passe quelque chose lorsque des femmes et des hommes , venus d' horizons différents , s' engagent non pas pour s' encarter dans un parti , non pas pour défendre leurs intérêts ou les intérêts de celui -ci mais pour ensemble , porter l' intérêt général , pour ensemble construire un projet nouveau , celui du XXIème siècle , celui que notre pays , que nos territoires méritent . Oui , il se passe quelque chose , aujourd'hui , ici . Et ce qu' il se passe , c' est cette révolution démocratique qui permettra de transformer la France . C' est cette révolution qu' ensemble , nous allons conduire . Non pas sur des clivages anciens , mais bien par une refondation profonde , une refondation idéologique , sincère , progressiste , en dépassant ce qui nous avait bloqué pour construire ensemble . Cette révolution démocratique , elle se construira en libérant toutes les énergies , en libérant les énergies qui sont dans notre société , chez chacune et chacun d ' entre nous . Et d'abord , c' est ma conviction première , je l' ai répété il y a encore plusieurs semaines , en refondant très profondément une chose : le travail . Le travail c' est le cœur de notre société , le cœur de notre économie , le cœur de nos projets à chacune et chacun d' entre nous , parce que le travail c' est ce qui permet à chacun de trouver sa place dans la vie , c' est ce qui permet de trouver un rôle de s' émanciper , de sortir de sa condition , de construire son destin . Donc oui , je l' ai dit et je le redis ce soir , je veux être le candidat du travail parce que c' est par le travail , pour le travail que nous redresserons le pays . Parce que c' est par le travail que celui qui n' a hérité de rien peut construire sa vie , parce que c' est par le travail que celui qui veut choisir son destin le trouvera , parce que c' est lorsqu' on est loin du travail que la pire injustice est subie , parce que c' est aujourd'hui dans notre société , lorsqu' on ne trouve pas un emploi , lorsqu' on est malheureux au travail , que les pires des injustices sont vécues . Alors pour être le candidat du travail , pour gagner cette bataille contre le chômage , bataille que nous avons perdue dans notre pays depuis trente-cinq ans , il faut prendre des mesures fortes . Nous sommes le seul pays d' Europe , le seul grand pays d' Europe , qui n' a pas vaincu le chômage de masse . Pour mener ce combat , il faudra relancer l' investissement , public et privé , il faudra plus d' Europe en relançant une dynamique forte . Mais cette dynamique n' existera que si nous savons nous- mêmes conduire nos propres réformes , retrouver notre crédibilité , convaincre nos partenaires que nous sommes dans une aventure commune où ils peuvent nous faire confiance . Je veux d'abord que la travail paye plus . Et il paiera plus . Pour cela , tous les travailleurs , qu' ils soient salariés , indépendants ou fonctionnaires , verront leur pouvoir d' achat augmenter : par exemple , une aide-soignante qui gagne 1800 euros bruts par mois , ce sera environ 300 euros de plus par an . Toutes et tous , quel que soit le niveau de salaire , parce que j' ai voulu cette baisse de cotisation sociale qui ne sera compensée que pour partie par une hausse de la CSG . Tous les travailleurs quels que soient leur statut , leur condition , leur situation , gagneront du pouvoir d' achat , parce que je veux que le travail soit récompensé , et plus on travaillera , mieux on gagnera sa vie . C' est la clé de cet esprit de responsabilité . Pour être le candidat du travail , je veux aussi que l' accès au travail soit plus facile et donc que les employeurs soient encouragés à embaucher . Et donc pour cela , il faut que le coût du travail diminue , il faut que le coût réel payé par l' employeur diminue . Et donc à ce titre , je supprimerai le Crédit Impôt Compétitivité Emploi pour le remplacer par un allègement de charges dans la durée pérenne , pour toutes et tous , c'est-à-dire non seulement pour les entreprises qui payent l' impôt sur les sociétés , mais pour les indépendants , pour les entreprises de l' économie sociale et solidaire . Ce sera 6 points de charges patronales en moins pour réduire précisément ce coût du travail , pour encourager l' embauche , pour développer partout en France de manière durable , l' emploi . Le travail sera aussi encouragé parce que nous aurons une économie plus agile , et c' est indispensable . Nous avons besoin d' avoir des entrepreneurs qui puissent embaucher plus facilement , parce qu' ils pourront s' adapter aux cycles économiques , aux contrecoups , à la situation de leur entreprise , de manière plus aisée . Qu' est -ce que cela veut dire ? Une réforme en profondeur de notre marché du travail , non pas par les réformes , de -ci et de-là qui diviseront la population , qui conduiront à réduire les droits des uns , de manière homogène partout dans le territoire , mais par une transformation profonde qui va permettre , partout , de réduire la place de la loi , pour faire davantage confiance aux contrats , c'est-à-dire à l' accord de branche et l' accord d' entreprise . Ce que je veux , pour notre marché du travail , c' est que la loi ne définisse plus qu' un ordre public social , mais que partout par des accords de branche et par des accords d' entreprises majoritaires , on puisse déroger à la loi pour trouver les bons équilibres , les bonnes souplesses sur le terrain . Non pas en réduisant les droits de tous les salariés , comme certains le caricaturent , mais en permettant de s' adapter à la taille de l' entreprise , aux aléas du carnet de commandes , aux contraintes du secteur , à tout ce qui fait qu' une entreprise du BTP de trois salariés à Nevers n' a rien à voir avec la situation d' un grand groupe international qui opère depuis Paris , sur tous les continents . C' est une évidence mais nous allons le faire , nous allons le faire jusqu' au bout . Et c' est ainsi que nous pourrons massivement , non seulement développer l' embauche mais permettre d' éviter nombre de licenciements . Cette réforme je la conduirai dès le début du quinquennat , c' est la réforme principale du marché du travail , ce sera l' un de mes grands objectifs . Faciliter l' embauche , c' est aussi donner de la visibilité à tous , aux employeurs comme aux salariés . Et pour ce faire , j' irai jusqu' au bout de la réforme des prud'hommes qu' en tant que ministre , j' avais conduite . En définissant des plafonds et des planchers pour les dommages et intérêts , non pas en réduisant les droits là encore , mais pour donner à toutes et tous , de la visibilité sur le terrain , pour éviter d' avoir des entrepreneurs qui puissent encore vous dire demain : “ Moi je n' embauche personne , parce que je sais très bien que je vais finir aux prud'hommes si ça se passe mal , ça va durer des années et je ne sais pas pour combien je vais en avoir ” . On le sait bien , les victimes aujourd'hui de notre système , ce sont les salariés les plus modestes , qui vont passer des années dans des procédures trop longues , qui vont parfois espérer gagner des sommes inconsidérées parce qu' on leur aura menti et ce sont les plus petits entrepreneurs qui du coup n' embaucheront plus . Donc je conduirai jusqu' au bout cette réforme des prud'hommes , parce qu' elle est importante pour l' emploi et parce que c' est une réforme juste . Enfin , je veux faciliter le travail pour toutes celles et ceux qui entreprennent , qui sont les indépendants , les professions libérales qui sont à leur compte , qui créent de l' activité pour eux-mêmes et aussi parfois pour d' autres , et qui aujourd'hui souffrent des complexités du régime social des indépendants . Je rapprocherai ainsi l' organisation du régime des indépendants de celle du régime général , avec un guichet indépendant , mais en supprimant le RSI . Cela permettra de réduire les délais insupportables , tout en préservant les exonérations et les réductions de cotisations de 6 millions d' indépendants , qui travaillent en France . Le travail , c' est la clef de notre refondation . Mais pour faire réussir la France , permettre à chacun de trouver sa place , il faut aussi libérer nos territoires , les aider à réussir , partout sur le pays . Je sais que vous avez ici , dans la Nièvre , tous les atouts pour réussir . Je sais que vous avez aussi beaucoup souffert , ces dernières décennies , d' entreprises perdues , de fermetures de services publics , de casernes . Vous avez perdu de la population . Mais vous avez des entreprises qui innovent , une population qui veut aller de l' avant , des forêts et des terrains agricoles . Une énergie . Et donc , un formidable sentiment de gâchis . Ce gâchis , c' est ce sentiment que le découpage territorial n' est pas à votre avantage , et que vous pouvez parfois vous sentir loin des métropoles . Le gâchis , c' est le sentiment que vous pouvez avoir , en considérant que la puissance publique investit moins ici que dans d' autres territoires . C' est le sentiment que peuvent avoir beaucoup de jeunes lorsqu' ils sont contraints d' aller faire leurs études ailleurs . Je veux mettre fin à ce sentiment de gâchis . Alors pour cela , ce ne sont pas des promesses inconsidérées qu' il faut faire . C' est un engagement fort . D' agir avec vous , avec le territoire , de faire davantage confiance à la population , aux élus locaux , aux services partout sur le territoire , et donc d' engager une phase nouvelle de décentralisation et de déconcentration . C' est cela à quoi je m' engage pour libérer l' énergie des territoires . Alors oui , nous devrons investir pour ce faire . Nous devrons investir davantage dans les services publics , pour assurer leur continuité . Investir pour qu' il y ait justement , partout , une couverture numérique digne de ce nom . Et ce sera l' une de mes principales priorités en matière d' investissement public . Parce qu' on peut développer , et nous allons développer , le Nivernais et Nevers . Vous êtes en train de le faire et de le réussir . On le fera avec le numérique , par le numérique . Mais pour cela , il est indispensable que la fibre soit là , que la couverture 3G et 4G soit sur le territoire . Et donc nous accélèrerons les investissements publics et l' organisation collective pour y parvenir : c' est indispensable . C' est l' infrastructure du XXIème siècle qui nous permettra d' y arriver . Enfin , pour que chacune et chacun trouve sa place , comme je le disais , pour réussir , il faut protéger , aussi . On doit libérer les énergies , les territoires , mais nous devons protéger les plus faibles . Nous devons assurer les sécurités contemporaines qui vont permettre à chacune et chacun de prendre les risques de la vie . D' être assuré qu' il ne sera pas abandonné . De protéger face aux aléas . De protéger parce que tout le monde ne peut pas réussir . C' est le cœur d' une solidarité nécessaire qui se conjugue avec la liberté . C' est le cœur d'ailleurs du projet français . Parce qu' il n' y a pas d' émancipation possible si cette émancipation n' est pas juste , si elle n' est pas faite pour tous et toutes . Alors dans ce moment de l' année qu' est celui des vœux , beaucoup ont passé du temps à parler aux Françaises et Français pour leur dire tous leurs projets , tout ce qu' ils allaient faire ou qu' ils n' avaient pas fait . Vous me direz , c' est le temps des résolutions ; je préfère celui des engagements . Mais , dans ces temps de début d' année , on se souhaite , les uns les autres , avant toute chose , la santé . Alors je veux aussi , dans les sécurités , indispensables , vous parler de santé . Parce que oui , il y a évidemment les sécurités individuelles qui vont face au chômage , pour la formation professionnelle , tout au long de la vie . J' en ai déjà parlé et j' en reparlerai . Mais je veux vous parler ce soir de santé . De santé , car ce sujet nous concerne toutes et tous sans exception . Parce que , lorsque nous avons conduit il y a quelques mois - et beaucoup d' entre vous nous ont aidé à le faire ici - cette Grande Marche , les Françaises et les Français ont exprimé une préoccupation première , parmi quelques autres : c' est la santé . C' est ce souhait d' être bien soigné . C' est cette volonté , pour eux-mêmes , pour leurs proches , d' être protégés contre ces aléas de la vie , et d' être en bonne santé . Parce qu' ils craignent pour leurs soins , pour leur prise en charge , ils craignent aussi de voir notre système solidaire s' étioler . Les Français ont un attachement singulier à leur modèle de santé . Un attachement , parce que c' est le fruit de notre histoire . Depuis plus d' un siècle , depuis plusieurs siècles , nous sommes une terre de santé , nous sommes un pays d' excellence médicale . Notre modèle , il est envié partout dans le monde . En première ligne , c' est celui qui a permis d' inventer en France le vaccin contre la rage , de procéder à des inventions aussi formidables que le cœur artificiel , la découverte du VIH . Nous sommes un pays de première mondiale , qui ne cesse de se perpétuer . Depuis 1945 , nous sommes parvenus à construire l' organisation de santé parmi les plus efficaces au monde . La réforme de 1958 , les lois DEBRE ont permis de créer l' hôpital , qui est non seulement une fierté , mais qui a été partout envié . Parce que c' est l' excellence de la formation , du soin , de la clinique et de la recherche . La santé , c' est aussi l' histoire intime des Français . Qui a partagé votre vie , à part vos proches ? Qui est le témoin de vos angoisses ? Qui vous suit ? Le médecin . Le médecin de famille , qui a jalonné nos campagnes et qui se fait de plus en plus rare - j' y reviendrai . Celui à qui , dans le secret , dans cette relation unique , vous confiez les choses . L' infirmière , l' aide-soignante , qui va faire votre quotidien lorsque vous devenez dépendant ou lorsque vous êtes affaibli . Lorsqu' on parle de santé , on parle avant tout d' humanité . On ne parle pas de chiffres , on ne parle pas d' un coût d'abord , on parle de ce qui est le cœur de notre humanité , et on doit pour cela d'abord et avant tout , en parler avec l' humilité qui convient . Lorsqu' on la réduit à un coût , à une politique publique , on se trompe . La santé c' est dans l' intimité , dans son épaisseur , ce qui fait notre vie . Alors , cette santé qui est au cœur de l' histoire française , de notre quotidien , de notre attachement , elle est face à de nombreux défis . D'abord , le défi du système de santé même que nous connaissons . Cette crise , que nous vivons depuis des décennies . Les difficultés à l' hôpital , mais qui ne sont pas les mêmes à l' hôpital de Nevers et dans un hôpital parisien . Ce défi de notre organisation collective . Le deuxième défi , c' est celui du vieillissement . C' est une chance . Une formidable opportunité . Mais c' est un défi d' organisation collective , il faut le regarder en face . En 2050 , un Français sur trois sera âgé de plus de 60 ans . En 2005 , c' était un Français sur cinq . Alors lorsqu' on est plus âgé , on est évidemment plus souvent malade , parfois moins autonome . Et mécaniquement , les patients , les personnes fragiles sont de plus en plus nombreux , et les dépenses , mécaniquement , augmentent . Et donc on doit inventer la manière de soigner , d' accompagner ce vieillissement différemment . C' est une chance , mais en même temps , alors qu' il est normal que ces dépenses continuent à augmenter , on ne peut pas avoir des dépenses de santé qui augmentent trois fois plus vite que la création des richesses dans notre pays . C' est pourquoi l' effort poursuivi ces dernières années devra être conduit . Et il ne s' agit pas de considérer qu' on va faire face à ce défi en relâchant l' effort . Mais il faut s' organiser différemment . Organiser les soins , notre système collectif , de manière plus adaptée à ce défi . Le troisième défi , c' est le développement des maladies chroniques . Là aussi , c' est une chance formidable . Il y a quelques années ou quelques décennies , on mourait systématiquement de plusieurs formes de cancer , on mourait du SIDA . On mourait de nombreuses pathologies , qui sont devenues ce qu' on appelle aujourd'hui des maladies chroniques . Ça veut dire d'abord - et c' est une formidable nouvelle qu' il faut saluer : on n' en meurt plus . Mais on vit donc longtemps avec . Avec des traitements coûteux , avec des passages à l' hôpital , avec des soins . Et donc oui , là aussi , c' est une autre forme d' organisation , et un coût , qu' il faut penser et accompagner . Le quatrième défi c' est que nous devons passer de l' excellence en soin à l' excellence en santé . La France est un pays formidable lorsqu' on est malade , on est mieux couvert , bien souvent et - je vais vous le dire ici - ça continuera à être le cas et nous nous battrons pour cela . Mais en même temps , nous prévenons mal les pathologies . Qu' est -ce que ça veut dire ? Ça veut dire qu' on prévient moins bien les maladies qui viennent du tabac ou de l' alcool ou des accidents de la route . 73 000 de nos concitoyens meurent chaque année à cause de la cigarette , 50 000 à cause de l' alcool , on se suicide beaucoup plus que dans nombre de pays voisins . La pollution tue , la pollution de l' air , beaucoup plus que dans nombre de pays voisins . J' étais il y a quelques semaines dans la vallée de l' Arve , elle sort d' une alerte pollution de 33 jours . Imaginez -vous . Tout cela fait qu' en France , notre système de prévention est moins bon . Tout ça parait pour beaucoup des idées sympathiques , nous disent “ oui c' est bien gentil mais enfin ça n' est pas le cœur du problème , le cœur du problème c' est le soin ” . Mais tout ce dont je viens de vous parler , vous allez le retrouver dans des pathologies chroniques parce qu' on aura moins bien prévenu , vous allez le retrouver dans des maladies respiratoires , dans des maladies chroniques , dans des dépendances . Et donc oui , mieux prévenir c' est aussi comme cela qu' on soignera mieux , qu' on réduira les coûts , qu' on réduira le coût pour notre société . Et donc il est indispensable d' avoir une meilleure politique en la matière . Et puis le dernier défi c' est celui des inégalités . En matière d' inégalités , on donne souvent des chiffres . On dit que l' espérance de vie d' un ouvrier est de 7 ans inférieure à celle d' un cadre : c' est vrai . On dit que les enfants d' agriculteurs ont 50 % de caries en plus que les enfants de cadres : c' est vrai . On dit que dans un tiers des départements français , on ne trouve plus de médecins après 20h : c' est vrai . Mais le plus terrible c' est que ces chiffres finissent par ne plus choquer personne et donc quand on parle de santé , on parle d' inégalités profondes , sociales , territoriales qui sont aussi aujourd'hui un défi extraordinaire que nous devons relever . Et donc face à tout cela , vous voyez bien que la réponse , elle ne peut pas être de ne rien faire . On ne protégera pas notre santé en disant “ on va faire comme si de rien n' était , on ne va toucher à rien , on va protéger les uns et les autres , les structures et on fera comme avant ” . Non , ça n' est pas possible ! La réponse , elle ne peut pas être non plus “ on va dérembourser , on va couper des têtes , on va supprimer les fonctionnaires , dérembourser des soins ” , non ! Parce que ce sont des mauvaises réponses partielles . La réponse exigeante , c' est de continuer à faire des économies indispensables en relevant ces défis . C'est-à-dire de conduire une vraie révolution culturelle de la santé ; c' est cela que nous devons faire . Non pas faire un énième coup de rabot , non pas considérer qu' il faut dérembourser ici , couper par là , déshumaniser tout , mais bien plutôt , conduire une révolution culturelle par laquelle nous nous appuierons davantage , sur les personnels de santé et les patients plutôt que sur les systèmes de soin et les établissements . Une révolution culturelle pour favoriser les initiatives de terrain , la liberté des acteurs et des choix plutôt que les décisions uniformes dictées d' en haut pour aborder la santé comme une source de richesse et d' opportunités plutôt que simplement comme un poste de coût . Pour avancer en donnant du sens , des perspectives de long-terme , plutôt que des mesures de court-terme dans lesquelles on finit par prendre des mauvaises décisions que nous payons aujourd'hui . Et donc , ce que nous devons conduire , c' est une révolution culturelle profonde autour de plusieurs grands chantiers , quatre grands chantiers sur lesquels je veux devant vous ce soir m' engager . I. Le premier chantier , c' est celui de la prévention . La révolution de la prévention c' est celle que nous devons conduire , en particulier pour passer d' un bon système de soin à un bon système de santé . Soigner les Français c' est par construction , non pas attendre simplement qu' ils soient malades , mais leur permettre de vivre le maximum de temps en bonne santé . C' est cela le cœur de la prévention , et donc c' est les inciter à vivre plus sainement , c'est-à-dire à mieux se nourrir , à bouger tous les jours à pied ou à vélo . A promouvoir , parmi les jeunes , celui qui ne fume pas , qui conduit le soir sans avoir bu . C' est renforcer l' éducation à la santé et le dépistage précoce . C' est renforcer la couverture vaccinale , qui est aujourd'hui trop faible pour être pleinement efficace . Ce sont là les véritables économies dans la durée , elles peuvent arriver très vite et cela doit être le cœur de notre ambition . Faire de la prévention , à long terme , c' est le meilleur moyen de faire des économies et de rendre notre système plus performant : parce qu' en investissant un peu dans le sport , par exemple , on évitera des hospitalisations coûteuses pour de nombreuses maladies . Alors pour cela , comment faire concrètement ? D'abord , il faut que les Français soient en contact régulier avec des professionnels . Pas une fois ponctuellement lorsqu' ils se rendent chez le médecin quand ils sont malades … non , le plus souvent possible . Et pour là aussi éviter des inégalités . Parce que vous savez , quand on est un enfant , venant d' un milieu modeste , dans bien des cas on ne va pas chez le médecin , on ne va pas chez le dentiste qui préviendra certains risques , on ne va pas chez le généraliste aussi souvent que d' autres . Et donc oui , pour que le plus souvent possible , les Françaises et les Français tout au long de la vie puissent avoir à l' école , sur le lieu de travail , accès , et encore plus que d' autres dans les zones défavorisées ou dans certains déserts médicaux , accès à la santé et aux soignants , nous devons conduire une réforme profonde . Ce que je veux faire pour cela c' est créer un service sanitaire de trois mois pour les étudiants en santé . Qu' est -ce que cela veut dire très concrètement ? Ça veut dire que tous les étudiants en santé , en médecine , en pharmacie , en dentaire , les étudiants infirmiers et infirmières : tous ces étudiants auront au moins 3 mois d' actions de prévention . Ils seront formés pour cela , ils ne feront pas d' actes de soin . Mais ils iront dans les écoles sensibiliser sur le tabac , ils iront dans les écoles , dans certains quartiers défavorisés , dans des EHPAD , chez des dépendants donc , dans des maisons de retraite , prévenir justement la mauvaise audition , certains risques liés à des maladies dentaires , certains comportements qui aujourd'hui conduisent à des maladies ou à des défaillances . C' est bon pour les professionnels de santé , c' est bon pour notre profession de santé justement , et c' est bon pour le pays . C' est un décloisonnement , c' est une révolution culturelle : nous devons la conduire . Ce service sanitaire sera intégré dans les maquettes pédagogiques . Et à terme , il pourra concerner ainsi plus de 40 000 étudiants chaque année . C' est une force de frappe formidable . Ça ne coûte pas un euro de plus . Cela responsabilisera tous les étudiants en santé , cela les ouvrira hors du champ médical . Et cela permettra à l' ensemble des Françaises et Français , les plus jeunes , les plus âgés , les plus défavorisés , d' être mieux prévenus contre certains risques . D' être ainsi mieux protégés . C' est cela , aussi , une vraie réforme de la santé . Pour développer la prévention , il faut aussi fixer des objectifs nationaux et enclencher une mobilisation générale . Je veux fixer ainsi comme objectif que la génération française qui naît aujourd ' hui sera la première génération sans tabac . C' est un objectif que nous sommes capables de relever . Là aussi , il ne s' agit pas de tomber dans des vieilles batailles . De dire , il y a d' un côté certains professionnels , qui , par exemple , sont les buralistes - et ils ont le droit de vivre , et il faudra les accompagner dans cette transition , les protéger , les respecter , en aucun cas les stigmatiser . Et il y aurait de l' autre côté les belles âmes . Non , c' est un défi collectif que nous allons conduire en responsabilité . En formant , non pas en punissant , mais en incitant . Qu' a fait l' Australie ? Ils sont en train de gagner cette bataille . Ils avaient comme nous , entre 20 et 30 % de leurs jeunes qui fumaient ; ils en ont aujourd'hui 3 % . Comment ont -il fait ? Ils n' ont pas augmenté le prix du paquet de cigarettes avant tout , ils n' ont pas eu des débats et des guerres comme nous en avons . Ils ont créé une vraie politique d' incitation pour faire que les lycées qui avaient le moins de fumeurs , qui valorisaient le mieux le sans-tabac , et bien étaient récompensés , mieux dotés , valorisés . Et donc c' est là aussi par une révolution des mentalités , par une mise en valeur des meilleures pratiques , par une récompense offerte à toutes celles et ceux qui , justement , réussissaient sur le terrain au contact du quotidien , cette transformation , qu' ils ont réussi ce combat . Non pas en stigmatisant ou en pénalisant , mais en valorisant les bons comportements , en donnant de la fierté à ceux qui conduisaient cette transformation , et en mettant là aussi de la santé hors des établissements de soin , à l' école et au travail . C' est cela que nous valoriserons . Et pour développer également la prévention , il faut la rémunérer à sa juste valeur . Et pour cela , je veux que nous changions le mécanisme de rémunération des professionnels de santé . D'abord je veux pour eux de la considération , je veux pour eux de la revalorisation de leur rémunération . Ce sera l' un des axes de notre réforme de la santé . Mais non pas simplement par une meilleure cotation des actes . Je veux que les professionnels de santé partout en France soient mieux considérés , mieux valorisés et mieux payés . Mais pour cela je veux qu' une partie de leur rémunération prenne en compte des objectifs de santé publique . Je veux que partout sur le territoire , ces professionnels de santé soient aussi mieux payés , non pas parce qu' ils feront le maximum de consultations ou le maximum d' actes , mais parce qu' ils feront le maximum de prévention . Et donc nous transformerons une partie de leur rémunération pour qu' il puisse y avoir une contractualisation sur la base d' objectifs de santé publique . Pour qu' ils soient ainsi incités à mieux accompagner certaines personnes handicapées , certaines personnes âgées , certaines personnes défavorisées . Non pas pour qu' ils les soignent davantage , mais pour qu' ils les aident à être moins malades , pour qu' ils les accompagnent en terme de soins complets et de santé . Cette révolution est fondamentale , et nous la conduirons . Parce qu' au fond , vous l' avez compris , je crois que pour développer la prévention , nous avons besoin de faire confiance avant tout aux professionnels . Je leur donnerai ainsi la liberté de s' organiser , de prendre des initiatives partout sur le territoire , contre l' alcoolisme des jeunes et les drames du samedi soir , contre les infections sexuellement transmissibles et les nouvelles infections . C' est pourquoi je permettrai aux agences régionales de santé de lancer des appels à projets en fonction des besoins locaux , des organisations spécifiques , des demandes des professionnels pour des actions adaptées aux populations . La révolution de la prévention , elle ne partira pas de l' Etat , elle se construira sur les territoires par des initiatives multiples , avec les élus et les professionnels de santé . Enfin , pour développer la prévention , il faut que les Français puissent prendre soin , sans se ruiner , de leurs yeux , de leurs dents et de leur audition . S' agissant des lunettes , des prothèses dentaires et de l' audition , je fixe l' objectif de 100 % de prise en charge d ' ici 2022 . Alors comment nous ferons ? Il ne s' agit pas de dépenser de manière inconsidérée . Il s' agit juste de dire que c' est une réforme essentielle parce que c' est une injustice et c' est une inefficacité . Quand un enfant n' est pas protégé contre sa myopie parce que ses parents ne peuvent pas lui payer des lunettes , il n' arrive pas à lire au tableau et donc il construit son propre échec scolaire . C' est intolérable . Lorsqu' une personne démunie , au chômage , ne peut pas se faire soigner les dents parce qu' elle est mal remboursée , elle ira à l' entretien d' embauche avec des dents gâtées - c' est ce qui se passe dans notre pays aujourd'hui - et donc elle n' aura pas son emploi , à coup sûr . Parce que lorsqu' une personne âgée ne pourra pas s' équiper d' une prothèse auditive , parce qu' elle est mal remboursée , et bien elle rentrera plus vite dans l' isolement , et donc dans la démence , la sénilité ou la dépendance . Et donc toutes ces petites économies qu' on fait aujourd'hui sur le dos des Français , elles conduisent à des coûts économiques , sociaux ou sanitaires beaucoup plus importants quelques années après . Elles conduisent à des injustices aujourd'hui intolérables . C' est pour ça que cette bataille est à la fois juste et efficace . Alors , comment nous allons le faire ? Nous allons pour cela davantage responsabiliser , ensemble , la Sécurité sociale et les mutuelles . La Sécurité sociale rembourse très mal ces soins aujourd'hui : 4 à 5 % , 10 % pour certains soins . Et donc ce sont largement les mutuelles , quand on peut se les payer , qui remboursent ces soins . Je veux que la Sécurité sociale travaille avec les mutuelles pour construire ensemble cette couverture à 100 % pour toute la population , mais en faisant baisser les prix . C' est comme ça que nous ferons les vraies économies . Parce qu' aujourd'hui en France , une prothèse auditive , elle est trois fois plus chère qu' en Allemagne . Qu' est ce qui le justifie ? Rien ! Parce que nous avons laissé cela se faire , parce que nous avons laissé certains s' organiser sur le dos de la population , parce que nous avons laissé trop peu de concurrence s' installer . Parce que - vous le voyez - vous pouvez , en remettant de la juste concurrence , en mettant une pression sur certains producteurs de prothèses , dentaires ou auditives , faire baisser les prix , retrouver un prix plus juste en protégeant mieux . Et donc vous voyez que vous réconciliez ce faisant la liberté et la justice , la liberté et la solidarité . Et donc tout cela passera par une meilleure transparence des prix , par plus de concurrence , par un travail partenarial de la Sécurité sociale et des mutuelles pour arriver à cet objectif . Parce que la réalité que nous vivons est intolérable . II . Le deuxième chantier que je conduirai , c' est l' accès de tous à des soins de qualité . Parce qu' il y a aujourd ' hui des inégalités sociales et des inégalités territoriales . Face aux inégalités sociales , je vais vous le dire clairement : je suis défavorable au déremboursement des petits soins . Ce n' est pas un projet radical ; c' est un projet partial , injuste et inefficace . Et donc oui , le projet que je porte est un projet qui permet , qui même encourage l' accès de tous aux soins , et donc je m' engage à ce qu' il n' y ait aucun déremboursement de soins utiles durant mon quinquennat . C' est un engagement solennel et fondamental . Pourquoi le projet conservateur , que je respecte dans ses composantes et dans son identité , mais pourquoi le projet conservateur de François Fillon , je m' y oppose ? Pourquoi je considère qu' il est à la fois inefficace et injuste ? Il est inefficace pour les raisons que je viens d' expliquer , d'abord parce que je n' ai trouvé personne pour m' expliquer ce qu' était un petit rhume et un grand rhume . J' ai compris qu' il y avait des débats , parfois téléologiques , qui s' étaient noués au sein même de sa propre équipe sur ce sujet . Mais , parce que surtout , c' est inefficace : qui va renoncer à se traiter sur le petit soin ira plus vite vers des maladies plus graves . Parce qu' en déremboursant ce qui paraît innocent alors que c' est utile , et bien on incitera beaucoup de Français à ne pas se traiter , et donc à aller plus rapidement vers des maladies plus graves , qui coûteront au final plus cher . Donc non , ça n' est pas une mesure efficace d' économies . Et en même temps c' est une mesure injuste . Pourquoi ? Parce que vous allez dérembourser . Les plus riches iront quand même se soigner . Les plus modestes - et il faut nous en féliciter - seront soignés parce qu' ils ont une couverture maladie universelle . Il y a une solidarité nationale qui les protège . Mais quelles seront les victimes de cette décision ? Toutes les classes moyennes . Toutes les classes moyennes , et en particulier les plus âgés , ceux qui ont le plus recours aux soins , c'est-à-dire ceux qui n' ont pas la CMU , mais qui n' ont pas assez d' argent pour , justement , aller se traiter sur ce petit soin . Et donc eux seront les victimes de cette réforme inique , de cette mauvaise réforme du déremboursement , à laquelle je m' oppose . De la même façon , supprimer l' Aide médicale d' Etat n' est pas une bonne idée . C' est aussi injuste et inefficace . Pourquoi ? Parce que si vous décidez de ne plus traiter des étrangers modestes sur votre territoire , ils vont développer des pathologies , ils vont recréer des épidémies partout sur le territoire , et donc ils vont contaminer votre population , ils vont fragiliser notre pays . C' est ça , ce qui est intolérable . Et donc le projet porté à la fois par le Front National et la droite conservatrice est un mauvais projet à ce titre . Parce qu' il va conduire à recréer des épidémies . J' étais il y a quelques semaines en Guyane , on voit réapparaître de la tuberculose , des maladies tropicales qui avaient disparu , parce qu' il y a de l' immigration clandestine . Expliquez demain que vous supprimez l' aide médicale d' Etat : ah c' est une idée sympathique , mais vous allez redévelopper des pathologies qui toucheront des gens qui ne sont pas du tout les étrangers , qui sont des Françaises et des Français , mais qui vivent au milieu d' eux . Par contre oui , nous devons mettre plus de responsabilité dans l' Aide Médicale d' Etat , et donc nous travaillerons à un mécanisme de responsabilité , parce que nos concitoyens ne comprennent pas non plus que ce soit un système ouvert à l' encan où il n' y a aucune responsabilité à la clé . Mais ça doit se faire dans un esprit constructif et juste . Vous l' avez compris , ce que je veux c' est donc qu' il y ait de la justice dans l' accès aux soins et que chacune et chacun soit justement traité pour accéder aux traitements auxquels il a le droit . Et en particulier , je veux ici aller au-delà sur un point très spécifique , mais qui compte . Je souhaite que l' Assurance Maladie prenne à nouveau en charge à 100 % certains patients qui ont été laissés pour compte . Vous le savez , nous prenons à 100 % en charge les affections longue durée ( ALD ) . En 2011 , une mauvaise décision a été prise : celle qui a consisté à sortir des ALD l' hypertension artérielle . Il y a 12 millions de nos concitoyens qui souffrent d' hypertension artérielle . Je ne suis pas en train de vous dire qu' on va mettre tous les hypertendus à 100 % , parce que là , je serai irresponsable . Ça pourrait faire plaisir à quelques hypertendus que j' aperçois dans la salle , à leur réaction , mais là , je ne pourrai pas vous dire honnêtement que je sais le financer . Donc je ne vais pas vous le promettre . Par contre , il y a 1 million d' hypertendus sévères , il y en a 50 000 chaque année . Et là c' est une bêtise de ne pas bien les couvrir . Pourquoi ? Parce qu' ils font , ces hypertendus , des accidents vasculaires cérébraux plus facilement , des pathologies , des lésions , des maladies beaucoup plus graves qui vont nous coûter beaucoup plus cher dans 6 mois , dans un an . Et donc je veux que les hypertendus sévères soient désormais pris en charge à 100 % par l' Assurance Maladie . C' est une décision juste , cela coûte une dizaine de millions d' euros par an . Ça n' est pas beaucoup par rapport au défi qui est le nôtre . Mais c' est une bonne mesure parce que c' est une mesure d' efficacité , de justice , qui permettra d' éviter ensuite des pathologies plus graves qui nous coûtent aujourd'hui beaucoup plus cher . S' agissant du tiers payant enfin , je veux prendre le temps d' évaluer cette mesure . Quand on parle de tiers payant ... Vous savez , quand on est dans des périodes électorales , il faut toujours regarder à qui on parle . J' ai un principe , depuis le début : je dis toujours la même chose à tout le monde . Ça m' a parfois valu beaucoup d' ennuis , mais à la fin c' est plus simple : ça évite les reniements , ça évite les corrections , ça évite les changements brutaux . Et au final , la cohérence et la constance , je pense que ça paye . Donc quand vous êtes dans une campagne et que vous parlez du tiers payant , si vous parlez aux patients , ils aiment bien , c' est très populaire ; si vous parlez aux médecins , ils détestent . Ils vous disent tout de suite que c' est abominable , qu' on leur a demandé de faire de l' administration , qu' il n' y a plus de responsabilité et qu' il faut donc en finir . En la matière je pense qu' il faut éviter les dogmes , comme sur beaucoup de sujets . Je pense que le tiers payant généralisé va dans le bon sens , parce qu' il évite que le montant qu' on doit mettre sur la table nous fasse renoncer à des soins . Il simplifie en même temps les éléments de remboursement , parce qu' on attendait que la mutuelle rembourse : là tout est pris dès le début . Mais en même temps je suis sensible à certains arguments : on a alourdi en effet , la tâche de certains professionnels de santé , on demande à des médecins de faire la trésorerie , ils vont devoir attendre plusieurs semaines ou plusieurs mois d' être remboursés par la mutuelle . Parfois ils ne le seront pas , me dit -on . Et puis il y a cet élément culturel qui n' est pas neutre . J' entends certains qui me disent : “ quand quelque chose ne fait plus l' objet d' un paiement , même minime , ça ne vaut plus rien . ” Et donc les gens ne se présentent plus à la consultation , parce qu' ils savent que ça vaut rien , ils ne la paient pas . Donc vous voyez , en la matière , je veux qu' on ne soit pas dogmatique . Donc je ne ferai ni plaisir aux patients qui voudraient que le tiers payant généralisé soit sanctuarisé , ni plaisir aux médecins qui voudraient qu' on abandonne cette idée funeste et qu' on revienne au bon système d' antan , qui était quand même un peu complexe pour tout le monde . Et en la matière , je prends un engagement de responsabilité : je demanderai une évaluation des effets du tiers payant généralisé , pour voir si en effet cela conduit à des pratiques d' irresponsabilité et des excès de la part de patients . Je regarderai les coûts induits pour les professionnels de santé , qui alors doivent être compensés . Je regarderai les dérives qu' on m' a décrites et qui sont conduites par certains autres acteurs . Et donc nous évaluerons , nous regarderons , et nous déciderons , comme on le fait trop peu souvent en France . Et puis si les Français renoncent à se soigner , c' est aussi parce qu' il y a des inégalités territoriales . Vous le vivez ici , nous le vivons , nous l' avons vu cet après-midi : il y a des territoires qui ont à faire face à des pénuries de médecins . C' est pour ça , quand j' entends que l' idée pour le système de soins , ce serait de réduire la fonction publique hospitalière , ce serait de supprimer des postes de praticiens hospitaliers , d' infirmières ou d' aides- soignantes , je dois vous dire qu' après avoir visité l' hôpital de Nevers , l' hôpital Pierre BEREGOVOY cet après-midi , j' ai du mal à comprendre . Je n' ai vu que des services où des aides-soignantes , des infirmières , des cadres , des chefs de service , des praticiens hospitaliers , me disaient : “ nous on s' organise , on arrive à recréer le service , j' ai tel médecin qui vient faire des vacations ici , nous nous organisons ... ” Il y a une inventivité extraordinaire , ici sur votre territoire . Mais partout j' ai entendu “ il manque x postes , il y a trois postes qui ne sont pas pourvus … ” Ah on fait des économies formidables , en ayant supprimé des postes de fonction publique . Mais derrière on paie beaucoup plus cher des prestataires occasionnels , on paie beaucoup plus cher des gens qui viennent faire des vacations et donc , au final cette idée , qui paraîtrait de bon sens qui dit “ on va couper des postes de fonctionnaires ” , et bien elle coûte plus cher , parce qu' on n' a pas supprimé bizarrement les patients qu' il y avait derrière . Parce qu' il continue toujours à y avoir des gens malades derrière . Alors , il n' y a pas de recette miracle , il y a un engagement à prendre . C' est en effet d' avoir davantage de médecins . Depuis 30 ans , on a , droite et gauche confondues , géré le problème du coût de la santé en se disant “ on va réduire l' offre , on va réduire le nombre de médecins qu' on met devant les Françaises et les Français , ça coûtera moins cher ” . Funeste idée parce que la population vieillit , il y a des maladies chroniques et les gens sont toujours malades , donc vous vous retrouvez avec des déserts médicaux . Vous avez en plus , de plus en plus de femmes - il faut s' en féliciter - qui sont médecins mais qui de plus en plus souvent arrêtent la médecine . Imaginez , que 4 médecins sur 10 qu' on forme aujourd'hui vont arrêter de pratiquer durant leur carrière . Donc il faut , oui , former plus de praticiens et il faut rouvrir le numerus clausus , c' est indispensable . Mais ça n' est pas suffisant . Parce que si je vous dis : “ Je vais répondre au problème de l' hôpital de Nevers aujourd'hui en ouvrant le numerus clausus et je vais répondre aux défis de l' hôpital de Nevers dans 10 à 15 ans , mais pas aujourd'hui ” . Donc il faut prendre des mesures de court terme . La première c' est de ramener des professionnels de santé partout sur le terrain , pas d'abord à l' hôpital , partout sur le terrain pour assurer la continuité des soins . Les déserts médicaux , ils sont dans ce tiers des départements français que j' évoquais où vous n' avez plus la continuité des soins . Ils sont dans ces 30 % de départements français où après 20h vous n' avez plus la possibilité d' aller devant un médecin généraliste ou tel ou tel spécialiste , et où vous n' avez plus le choix qu' aller aux services d' urgences le plus proche , qui parfois va être assez loin en voiture . Et ça coûte beaucoup plus cher là aussi , entendons -nous bien , penser à faire des économies avec ça , c' est pas une bonne idée du tout . Le jour où vous arrivez aux services d' urgence , ça va vous coûter beaucoup plus cher que si vous alliez chez un généraliste ou que s' il était venu chez vous . Donc , la continuité des soins elle se fera en rouvrant justement et en accélérant l' ouverture de ces maisons pluridisciplinaires de santé . Chaque fois qu' on ouvre une maison pluridisciplinaire de santé , on comble un désert médical . Je veux ainsi parler des maisons où travaillent ensemble des médecins , des kinésithérapeutes , des orthophonistes , des infirmières , des assistants médicaux . Et je fixe pour objectif de doubler leur nombre d' ici 2022 . Pourquoi c' est important ? C' est pas de l' argent public uniquement , c' est de l' organisation sur le territoire , c' est de l' argent qui sera parfois mis par les collectivités territoriales , elles le font déjà , mais c' est très important parce que c' est aider les territoires à réavoir de l' attractivité médicale . C' est comme ça que vous allez réattirer des professionnels de santé dans les territoires les plus ruraux , c' est fondamental , parce qu' aujourd'hui qu' est -ce qui fait peur à une jeune femme ou un jeune homme qui cherche à s' installer ? C' est qu' il ne veut plus avoir la vie du médecin de campagne d' il y a 20 ans , c' est qu' il ne veut plus pratiquer seul , c' est qu' il ne veut plus travailler de 7h à 22h , être appelé au milieu de la nuit . Il veut travailler de manière plus collégiale , avec d' autres professionnels de santé , il veut pouvoir avoir des gardes partagées , pouvoir travailler avec l' hôpital , être dans un autre cadre qui correspond aux besoins , à sa génération , à la façon de vivre des professionnels de santé , c' est cela que permettent justement ces maisons pluridisciplinaires de santé , c' est pour cela que je veux les doubler d' ici 2022 et c' est aussi ainsi qu' on coordonnera davantage le travail de ces professionnels sur le terrain avec l' hôpital et que ce faisant , nous diminuerons les coûts parce que nous éviterons à des centaines de milliers de nos concitoyens d' aller tout de suite aux urgences . Le deuxième élément extrêmement important c' est le développement de la télé- médecine , c' est fondamental . Nous devons là aussi , permettre de soigner à distance , de suivre un patient cardiaque par un objet connecté , d' envoyer des images , de soigner à distance tout en assurant de la qualité , d' accélérer une consultation chez un patient à risque ou au contraire d' annuler un déplacement long chez un patient pour lequel tout va bien . Avec la télé-médecine , on réduit par définition les distances , mais là aussi on recrée du confort et on diminue les coûts . En ayant de la télé-médecine , vous permettez à des professionnels qui sont ici , ou dans le libéral , ou à l' hôpital , ou dans un établissement de soin pour personnes dépendantes ou personnes âgées de mieux soigner , en lien avec d' autres professionnels et des spécialistes , et ainsi de prévenir mais aussi de guérir , au plus proche de la population . C' est comme ça que vous réglez ce problème d' inégalités territoriales , et que vous facilitez l' accès aux soins et c' est là aussi plus efficace et moins coûteux . Certains ne l' ont toujours pas compris . Quand vous êtes dans un EHPAD , que vous avez une personne âgée qui a besoin de faire un examen , aujourd'hui elle va attendre des semaines et des semaines pour aller dans l' hôpital , le jour où elle arrivera , il y aura eu de l' attente , ça sera peut-être plus grave , il faudra payer le transport à l' hôpital , qui coûte très cher , c' est un poste très important de l' Assurance Maladie , elle se retrouvera devant le professeur mais il n' y aura pas celui ou celle qui la soigne tous les jours , qui est à la maison de retraite ou dans l' EHPAD , qui l' accompagne . Et la personne âgée , un peu dépendante , ne pourra pas répondre à toutes ses questions . Par la télé-médecine , et je l' ai vu faire à Pessac , non loin de Bordeaux , dans un EHPAD , et bien vous avez le professeur , à l' autre bout dans son service , qui parle non seulement au patient , parfois à sa famille qui veut être là , mais également au médecin qui le suit au quotidien , au personnel soignant dans son ensemble qui est derrière l' écran et qui peut échanger beaucoup plus efficacement . Vous n' avez pas dépensé le transport et vous avez un soin beaucoup plus complet , une réponse beaucoup plus complète et beaucoup plus pertinente . C' est cela aussi une réforme en profondeur de notre système de santé . Enfin , la bataille contre les déserts médicaux c' est aussi une bataille de l' information et il n' est pas normal que d' un côté , une minorité éduquée , avisée , possède toutes les clés du système , sache à quelle porte frapper , connaisse le médecin spécialiste et obtienne un rendez-vous sans attendre et que de l' autre côté , la grande majorité des gens soit totalement dépourvue , que lorsque vous êtes dans un désert ou quasi désert médical , vous êtes perdu , vous ne savez pas où il faut aller au fond , c' est ça l' un des principaux problèmes que nous avons parfois . Vous ne savez pas où est en fait le spécialiste ou la maison de soins et donc vous allez aux urgences . Et bien , c' est pourquoi je mettrai en place sur internet ce que j' appellerai un &laquo; Doctissimo &raquo; public . C' est quand même fou , qu' à l' heure d' Internet , alors qu' on a tant d' informations sur tout qu' on peut comparer les prix sur chaque élément de la vie quotidienne , on ne puisse pas savoir la réalité des soins qu' on a autour de soi . Ce portail permettra de disposer d' informations actualisées et fiables sur les maladies , les médicaments , les services publics à proximité , les modalités de prise de rendez-vous et les délais , parce que l' égalité d' accès à l' information , ce n' est rien d' autre , aussi , que l' égalité d' accès à la santé . III . Le troisième chantier , c' est la transformation de l' hôpital . C' est un chantier fondamental . Il est vital . Il est vital en matière de soins , en matière de formation , en matière de recherche . Parce que l' hôpital c' est le pilier de notre système de santé , c' est notre fierté , c' est ce qui forme les personnels de santé partout en France . C' est le pivot parce que c' est aussi l' hôpital autour duquel tout s' organise et je veux ici non seulement saluer les personnels de santé que nous avons vus ensemble , cher Denis cet après-midi , qui font un travail extraordinaire . J' ai vu un enthousiasme , une inventivité , une volonté de faire formidables . Mais ces personnels de santé , c' est aussi ceux qui se mobilisent , qui se sont mobilisés après les attentats que nous avons vécus ces derniers mois en France , partout , qui se mobilisent dès qu' il y a une catastrophe sur le terrain , sans compter leur temps , en revenant . On a vu des médecins qui étaient partis à la retraite , des aides-soignants , des infirmiers qui étaient partis parfois de l' hôpital , revenir pour aider leurs collègues . C' est le cœur vibrant de ce qui tient ce pays ; c'est-à-dire , l' engagement profond qui nous tient ensemble . Et donc l' hôpital , il mérite d' être réformé . Il le mérite profondément , pas seulement pour la santé mais pour toutes celles et ceux qui le font , pour toutes celles et ceux qui , à l' hôpital , portent la santé . Alors le réformer , ça ne veut pas dire mettre plus d' argent . Il faudra investir , investir pour moderniser , et j' y reviendrai - ce sera une constante de mon programme et de mon engagement . Mais en même temps on continue à faire des efforts sur le fonctionnement pour les réduire . Mais il ne faut pas faire sur l' hôpital une réforme monolithique . Les défis de l' hôpital de Nevers n' ont rien à voir avec les défis de l' hôpital Lariboisière à Paris . Rien . Ici , vous manquez de professionnels de santé , vous avez des services qui ont du mal à fonctionner . Je voyais le service de neurologie , on me dit “ nous on peut pas réussir à traiter tout de suite les AVC ” et donc ils ont réussi , avec beaucoup d' inventivité , à mettre en place justement la connexion avec leurs collègues , qui peuvent traiter les AVC à distance . On conditionne le patient de la meilleure des façons , on le transporte à Dijon … C' est formidable , ce qui se passe là . Mais ça n' a rien à voir avec le problème par exemple d' un hôpital parisien , qui est submergé par ses urgences , qui ne manque pas de professionnels aujourd'hui , même s' il commence à avoir un vrai problème d' attractivité , mais qui a simplement tout le reste de la société qui vient à lui . Et donc , la réforme de l' hôpital , elle consiste à regarder en face , et à respecter d'abord tous les métiers et les engagements qui le font : les infirmières , les sages-femmes , les aides-soignantes , les cadres , les médecins , les cadres , les praticiens hospitaliers , les chefs de service , les administratifs , et ensemble avec eux , de concevoir cette réforme . Parce que l' hôpital aujourd'hui , il est en crise de quoi ? Il est en crise des 35h qu' il n' a pas pu digérer , qui l' ont profondément désorganisé - il faut le dire et le regarder en face . Parce qu' on ne parvient pas à y faire entrer pleinement le numérique , à générer les gains de productivité sur les multiples innovations , justement , auxquels on devrait pouvoir parvenir . Par insuffisance des investissements , et parce qu' on laisse l' hôpital face à lui-même , et qu' en quelque sorte il régule toute la crise du système de santé dont je parle depuis tout à l' heure . Quand il y a un désert médical , on va dans l' hôpital voisin même s' il est un peu loin , on ne va que là . Quand il n' y a pas assez de continuité des soins , quand il y a une médecine de ville qui ne fonctionne pas de la même façon , de l' angoisse et de la misère sociale , on va à l' hôpital . Quand on a peur , on va à l' hôpital . Et donc c' est tout le système de soins , ensemble , qu' il faut réorganiser autour de l' hôpital . Parce qu' on a fermé en amont trop de structures , et on n' organise pas bien aujourd'hui les soins . Et donc l' hôpital , dans de très nombreux , dans de trop nombreux territoires , c' est l' ultime recours , voire le seul recours . Alors , pour transformer l' hôpital il faut le transformer avec le reste du système . Cette réforme elle est indispensable mais il faut d'abord déconcentrer le pouvoir au niveau des agences régionales de santé et des acteurs locaux , c'est-à-dire justement de l' hôpital , des élus , pour qu' en fonction des réalités et des besoins , sur le terrain , les acteurs puissent davantage innover , s' organiser , adapter l' offre . Ensuite , il faut décloisonner entre le médical et le médico-social , entre le public et le privé , pour faire émerger de nouveaux partenariats : c' est indispensable . La réforme de l' hôpital elle passera aussi par la capacité d' ouvrir l' hôpital à des professionnels du secteur privé , à permettre - je l' ai vu très bien , là aussi , à Sallanches , il y a quelques semaines , on en a parlé tout à l' heure - et bien , permettre à des professionnels du privé qui font de la médecine de ville , de venir s' installer dans des structures de l' hôpital , pour mieux assurer la continuité des soins , parce qu' ils sont plus rassurés d' opérer à proximité d' infrastructures , parce qu' ils vont pouvoir mieux s' organiser entre eux , et parce qu' on pourra aussi plus rapidement organiser le suivi et la sortie de l' hôpital . Il faut accélérer la dynamique des groupements hospitaliers de territoire . Vous le faites formidablement ici , dans le Nivernais . Qu' est -ce que ça veut dire ? Ça veut dire qu' il faut continuer à regrouper les structures , les centres hospitaliers , l' ensemble des structures qui existent sur le territoire . Mais pour développer les synergies entre elles , pour les rapprocher , pour permettre de faire des bonnes économies en mutualisant les coûts , et en même temps pour permettre d' avoir une bonne répartition sur le territoire , un bon suivi plus intelligent , et améliorer la coordination . Il faut élargir l' expérimentation des maisons de répit et les établissements de soins de suite pour permettre ainsi d' accueillir plus de patients en dehors des hôpitaux . Qu' est- ce que ça veut dire ? Ça veut dire qu' on a besoin d' éviter que tout le monde arrive à l' hôpital . Les maisons de répit c' est ce qui évitera que certaines personnes , qui sont dépendantes , aillent pour soulager leurs proches , à l' hôpital pendant des semaines . On les mettra dans des maisons de répit , médicalisées mais adaptées , qui coûtent beaucoup moins cher . Adapter ce qu' on appelle des centres de soins de suite . Qu' est -ce que ça veut dire , ça veut dire que aujourd'hui , qu' est -ce qui coûte à l' hôpital ? Lorsqu' on vous a opéré , lorsqu' on vous a soigné , vous n' avez plus besoin de rester dans une chambre d' hôpital , sur votre lit . Mais c' est parfois un peu trop lourd pour rentrer à la maison parce que vous êtes seul , parce que vos proches ne peuvent pas apporter des soins minimaux . Parce qu' il n' y a peut-être pas d'ailleurs dans la commune , l' infirmière ou l' infirmier qui pourra passer , de manière régulière . Et donc , la seule façon pour réduire les coûts de manière intelligente , c' est d' avoir ces centres de suite et de suivi qui permettront , hors de l' hôpital , de manière beaucoup moins coûteuse , que vous sortiez de l' hôpital une fois l' opération faite dans un encadrement médical allégé , beaucoup moins coûteux mais rassurant et qui vous permettra de vivre en faisant des économies . Vous voyez bien quand je parle de révolution culturelle , il y a des tas d' économies qu' on peut faire en prévenant mieux , en s' organisant différemment ; qui ne veulent pas dire qu' on va dérembourser , qui ne veulent pas dire qu' on va couper des têtes , qui ne veulent pas dire qu' en quelque sorte la France serait face à une espèce de choix cornélien et abyssal , qui est : si on veut continuer à être juste et ambitieux , on ne peut plus le financer , et si de l' autre côté on veut être responsable il faut forcément faire des choses absolument injustes et régressives . Non ! Il y a des choses de bon sens à faire , qui sont à portée de main . Elles supposent simplement de décloisonner , de mieux s' organiser et de le faire dans la durée . C' est cela , ce que je veux pour l' hôpital . Il faut aussi pour cela faciliter le travail libéral des médecins hospitaliers pour continuer à attirer les talents vers l' hôpital public , pour briser là aussi les tabous : réorganiser les petits hôpitaux qui devront accueillir des patients après une intervention mais qui doivent continuer à être attractifs pour des praticiens hospitaliers . Et donc permettre , là aussi , une vraie fluidité . Je veux que nous avancions en changeant de méthode , en agissant davantage avec le terrain , avec les acteurs eux-mêmes . J' anticipe les débats : on va me dire “ très bien , vous fermez combien d' hôpitaux , quels sont les noms de ville ? La liste des maternités à fermer ? Les services d' urgence à supprimer ? ” Ça n' aurait aucun sens . Aucun . Il y a plein d' hôpitaux qui en fait ne sont peut -être pas à fermer parce qu' on va partiellement les transformer en centres de suivi , parce qu' on va permettre à des néphrologues du privé de faire des centres de soins qui articuleront mieux l' hospitalisation à domicile et ce centre-là . Je veux que partout il y ait des débats régionaux autour du devenir des hôpitaux de proximité . Les élus , les acteurs de terrain , affronteront avec moi la réalité . L' Etat , avec les Agences Régionales de Santé , conduiront ces débats mais la responsabilité sera prise localement , de manière inventive , en responsabilité collective . Enfin , pour transformer l' hôpital , il faut réformer son mode de tarification . Disons -le très clairement , ce qu' on appelle la tarification à l' activité a permis aux hôpitaux de gagner en efficacité . Mais les gains de productivité à l' hôpital , ça ne fait pas l' efficacité du système de santé dans son ensemble : il faut aujourd'hui imaginer et mettre en œuvre des financements qui prennent en compte le parcours de soins du patient et ses différents épisodes . Il faut donc qu' on réforme le mode de tarification à l' hôpital . Sinon on valorise là aussi la répétition des actes , l' activité . Ça n' est pas bon , ça n' est pas intelligent . Et donc il faut qu' on valorise la bonne prise en compte du parcours de soin , c'est-à-dire l' hôpital qui va travailler avec ses partenaires de santé , avec de la médecine de ville , avec d' autres acteurs de terrain . Et ainsi avoir une tarification plus adaptée à ce que nous avons vu tout à l' heure , exactement à ce que vous faites sur le terrain ici . IV . Enfin le dernier chantier , et j' en conclurai là , c' est d ' avoir une vraie politique industrielle et d ' innovation en santé ambitieuse . La santé on en parle toujours comme d' un coût ou d' un malheur ; c' est aussi un fleuron de l' industrie et de l' économie française . Nous avons en France 70 milliards de chiffre d' affaire faits dans la santé , 200 000 employés directs ou indirects , des territoires entiers ; la région Centre , voisine , est une des premières régions qui vit de l' industrie de santé . Nous sommes , aujourd'hui , parmi les premiers producteurs du monde de biotechnologies . J' étais , l' année dernière et il y a deux ans , à ce grand salon des nouvelles technologies à Las Vegas dont on a parlé ces derniers jours . Je n' y suis pas cette année , je suis avec vous . Mais j' y ai vu des formidables innovations technologiques . J' avais , avec moi , la plus grande délégation non-américaine : les Français , la fierté ! On était devant les Anglais , les Israéliens , toutes les autres nations , justement , de l' innovation dont on nous rabat les oreilles . Nous étions là , avec des start-up , deux tiers d' entre elles venaient hors de Paris , et un grand nombre était dans le numérique et la santé . C' est ici qu' elles vont se développer , cher Denis . Sur ce territoire autour de l' hôpital . Il y a un formidable levier de développement pour les nouvelles industries de santé : autour du dispositif médical , autour des nouveaux objets connectés qui développeront des nouvelles formes de diagnostic , qui vont faire qu' on va soigner et prévenir différemment la maladie . Et qui font que partout sur le terrain , on va réinventer grâce , justement , à ceux qu' il y a , grâce à la formation des infirmières qui existent ici et qui est formidable . Grâce à cet hôpital , on va redévelopper de l' activité . Et donc nous avons besoin , en France , d' encourager cette industrie de santé . Et donc je veux d'abord qu' on arrête cette politique aux petits pieds qui consiste tous les six mois ou tous les ans à changer notre politique de remboursement ou de déremboursement avec une variable d' ajustement qui sont les laboratoires pharmaceutiques . Nous sommes en train de sacrifier notre industrie pharmaceutique pour essayer de faire des économies sur son dos . Il faut continuer à valoriser le générique lorsqu' il existe , c' est une bonne pratique , mais il faut payer les médicaments au juste prix quand il y a derrière des innovations , quand il y a des investissements , pour avoir une grande industrie pharmaceutique . C' est indispensable ! Et en même temps , il faut être responsable . Et donc on va faire des économies , mais des vraies , des bonnes , des économies innovantes . Quand vous faites du mal à vos laboratoires pharmaceutiques , comme on l' a fait , ils détruisent de l' emploi , ils ferment des sites . Et puis , on voit bien , vous l' entendez comme moi ces dernières semaines , ces derniers mois , ils créent de la rareté . On entend parler de produits qu' on n' arrive plus à trouver et de médicaments qui ne sont plus à la pharmacie . Parce que qu' est -ce qu' ils disent ? “ Bah vous m' avez tellement baissé les prix en France , moi je les vends en Belgique , en Allemagne , parce qu' ils paient le juste prix , mais je les vends plus chez vous . ” Vous imaginez dans quelle situation on est en train de tomber ? Donc on va arrêter ça . On va leur donner une visibilité sur plusieurs années , on va leur demander de faire des économies justes , mais de faire leur travail , de récompenser l' innovation . Par contre , en médicaments on va faire des vraies économies . Je suis sûr que , vous qui toussez - vous n' êtes pas la seule , je vous rassure ... moi aussi , je sors d' une vilaine toux - bon , vous avez été chez le pharmacien . Vous avez acheté des pastilles au mieux ; si c' est plus grave , le médecin vous a prescrit , j' espère pas des antibiotiques encore , peut-être si ça dure , mais des médicaments . Vous êtes allé les acheter . Et puis ça va passer dans deux , trois jours - c' est tout le mal que je vous souhaite . Vous allez avoir consommé la moitié de la boîte . Puis elle va finir dans votre tiroir . Alors parfois vous allez , dans deux mois si ça revient , la réutiliser . C' est pas forcément d'ailleurs la meilleure des choses parce que vous n' avez peut-être pas la même pathologie . Mais la plupart du temps vous allez le garder , et puis quand vous ferez le rangement dans un an , deux ans , vous allez découvrir qu' elle est périmée . C' est ça ce qui se passe : on l' a tous vécu . Sous le contrôle familial , je crois être un coproducteur de ce gâchis . Ce que je ferai , ce à quoi je m' engage , c' est la généralisation de la vente à l' unité du médicament . C'est-à-dire , quand vous avez une pathologie , quand le médecin vous prescrit un médicament , et bien on vous vendra ce qui correspond à la prescription , ce qui correspond à ce à quoi vous avez besoin , ce qui correspond sinon à votre renouvellement d' ordonnance , quand vous avez une pathologie de longue durée . Parce que c' est comme ça que nous procéderons à des vraies économies . C' est comme ça , en passant à la vente à l' unité du médicament que nous ferons des économies massives , mais sans faire des mauvaises , des fausses économies . Et en parallèle , j' aurai une politique volontariste pour développer la santé en France , les laboratoires pharmaceutiques , l' innovation en santé , et donc l' innovation dans les dispositifs médicaux , qui est la révolution en cours dans laquelle nous devons gagner . L' innovation en matière de santé pour qu' il y ait plus d' objets connectés et d' innovation dans le numérique . Ça passera par de l' investissement privé , mais aussi par une simplification de nos procédures d' autorisation de mise sur le marché , par une simplification réglementaire qui fait qu' en France aujourd'hui , on doit beaucoup plus simplement innover dans ces matières , développer ces produits , développer ces innovations qui sont bonnes . Pourquoi ? Parce qu' aujourd'hui vous avez des laboratoires formidables , vous avez des innovations qui sont faites dans les hôpitaux , dans les centres de recherche , parfois on les finance au début avec du crédit-impôt recherche : c' est de l' argent public . Et puis parce que la France est un pays extraordinairement compliqué pour donner ces autorisations de mise sur le marché , et bien ces Françaises et ces Français qui ont fait ces innovations , ils vont développer le produit ou le dispositif médical en Allemagne ou aux Etats-Unis ou ailleurs . Je l' ai tellement vu ... C' est insupportable . C' est inefficace : on a financé leur formation , le début de la recherche et tout le reste . Donc on doit simplifier nos procédures , les accompagner , parce que je veux qu' on soit une grande nation en matière de santé , et donc une grande nation en matière d' industrie de santé aussi . Voilà ce que je veux , mesdames et messieurs , chers amis , et je conclurai là-dessus . Voilà ce que je veux . Je veux que nous soyons fiers de notre santé . Je veux que nous la regardions en face , comme elle doit l' être au XXIème siècle . Comme non pas simplement un coût , non pas comme une charge , comme un devoir qui conjugue la solidarité et la liberté , comme une ambition légitime que notre pays doit porter pour celles et ceux qui souffrent et sont malades , pour leurs familles , pour celles et ceux qui sont engagés dans ce secteur de la santé , pour les acteurs de terrain , élus , fonctionnaires , qui chaque jour l' accompagnent . Et pour notre pays , parce qu' il le mérite . C' est comme cela que je veux conjuguer notre énergie . Vous savez , je me suis laissé dire - et il y a dans la salle je crois des témoins directs de cette scène - que le soir de son élection , François MITTERRAND a réuni les élus du Conseil général de la Nièvre et a dit : &laquo; Ce soir mes amis , je vais vous le dire , ce pouvoir que les Français m' ont donné , je suis venu vous le rendre &raquo; . Je me retrouve dans cette philosophie . Ce que nous allons faire , c' est que nous allons prendre le pouvoir pour vous le rendre . Ce que nous allons faire … Ce que nous allons faire mes amis , c' est que nous allons , jour après jour , justement , réveiller cette flamme française , cette énergie , que j' ai vue cet après-midi à Nevers , que je sens ce soir dans cette salle , cette énergie qui veut regarder les défis qui sont les nôtres , de manière adulte et responsable . Qui sait qu' il y a des défis , des coups . Qui sait que nous aurons des efforts à faire , pendant non pas simplement les cinq ans , mais les dix ans à venir . Mais qui sait aussi que notre peuple a l' énergie pour cela . Ce que je veux pour nous toutes et tous , c' est cette flamme française , cette flamme je veux la réveiller avec vous , c' est ce qui fait que nous réconcilierons la liberté et la justice , cette volonté de faire réussir le pays , d' être fiers de ce que nous sommes en étant dans le siècle qui advient , non pas en défendant des modèles passés , non pas en galvaudant des mots dans lesquels certains se réfugient pour ne rien faire , non pas en promettant du sang , des larmes , ce qui a été fait il y a trente ans ou quarante ans dans des pays voisins . En regardant en face nos défis , mais en les relevant ensemble , en considérant que l' Etat et ceux qui prétendent dans l' Etat aux plus hautes fonctions , ont une responsabilité : donner un cap , fixer un sens , faire émerger des solutions concrètes , réelles , dans le siècle , mais que tout cela nous le ferons ensemble . Que les vraies réformes nous ne les ferons pas contre les Français , en réduisant leurs droits , mais bien en faisant émerger cette énergie , en donnant une place à cette énergie , en la faisant réussir , partout sur le territoire , en la considérant . Et donc , tout cela , tout cela mes amis , dépend de nous . Aujourd'hui , demain . Et donc dans les prochaines semaines , dans les prochains mois , c' est ensemble ce projet que nous allons porter . Je compte sur chacune et chacun d' entre vous , ici jusqu' à l' été prochain , pour avec moi porter ce projet , cette vision , pour convaincre avec votre énergie , pour persuader avec votre engagement , pour mobiliser avec notre optimisme ! Vive Nevers ! Vive la République ! et vive la France ! ",
    "Merci à vous . Bonsoir à toutes et tous . C' est formidable de voir cette énergie , là , dans cette salle . Vous êtes plus de 2500 ce soir . Nous sommes plus de 2500 ici , et je veux commencer par présenter mes excuses aux quelque 500 personnes que nous n' avons pas pu faire rentrer , qui comme vous ont attendu dans le froid et malheureusement n' ont pas pu nous rejoindre . Mais vous êtes là , nous sommes là , c' est le plus important et je vous en remercie . Merci à Guy et Valérie d' avoir , non seulement lancé notre rassemblement ce soir , mais depuis le début de vous être engagés à nos côtés pour porter le mouvement , ici , à Clermont , dans tout le département et au-delà d'ailleurs , dans la région . Et je sais que ce soir , ça n' est pas simplement Clermont , Cournon , mais bien toute la région et au-delà , qui est avec nous , et qui ce soir donne toute son énergie . Je me souviens avec beaucoup d' émotion de mes précédents passages à Clermont . Il y a près de 2 ans maintenant , lors de l' enterrement de François MICHELIN . Et je me souviens , cette gravité vibrante qu' il y avait , cette pierre noire de la cathédrale , cette gravité qui avait envahi toute la ville et qui montrait combien ici , on ressent profondément , dans le silence et la douleur . Je me souviens aussi , quelques mois plus tard , lorsque nous sommes allés à Issoire , à Champeix ... mais ce n' est pas possible que vous soyez autant sauf à ce que Roger JEAN , que je salue , ait emmené tout le monde ! Et il est avec nous depuis le début , lui aussi , avec courage . Je veux remercier toutes celles et ceux qui ce soir sont là à mes côtés , sont venus du département , de la région . Jacques MEZARD qui est là depuis le début , esprit libre et courageux , qui m' a emmené dans son Cantal - et j' en reparlerai tout à l' heure - avec bonheur et fierté , et qui continue à m' accompagner pour que chaque jour cette ruralité - sur laquelle je reviendrai - on ne puisse non pas en parler avec des mots , mais aussi avec de l' émotion . Parce que je n' oublie pas tes larmes lorsqu' un petit matin tu m' as emmené dans une exploitation laitière , et que tu m' as expliqué comment ces territoires parfois avaient été abandonnés ou délaissés . Je remercie aussi Christian , Christian TEYSSEDRE , qui vient de Rodez , et qui lui aussi depuis le début est à mes côtés , et nous avons cheminé l' été dernier , et vous êtes plusieurs à venir de la région . Remercier la Lozère et Francis , remercier la Corrèze , voilà , vous remercier toutes et tous , et René , René SOUCHON qui a dirigé la région et qui est là ce soir aussi . C' est cette réconciliation de toutes celles et ceux qui parfois d'ailleurs ont mené sur le terrain des batailles contraires , qui n' étaient pas toujours d'accord , mais qui ont toujours , avec la même conviction , oeuvré , oeuvré ici , oeuvré pour l' Auvergne et ses belles terres . C' est aussi le combat que mène votre président du conseil départemental , Jean-Yves GOUTTEBEL , il n' est pas avec nous ce soir , il n' a pas pu , mais je m' honore d' avoir son soutien et son parrainage . Et je ne peux ni ne veux passer sur ces terres , sans rendre un hommage légitime à celui qui pendant de nombreuses années a oeuvré pour l' Auvergne , le Président GISCARD D' ESTAING , qui après avoir accompagné la France dans un moment fort de modernisation , de libération aussi de sa société , a porté la cause de l' Auvergne , loin et fort , et a commencé certains combats que Jean-Yves continue aujourd'hui . Vous êtes nombreux ce soir ici , vous êtes nombreux parce que quelque chose se passe dans le pays . Parce que oui , une vague monte . Cette vague , ça n' est pas un homme seul , comme certains veulent encore le croire , ou alors vous les avez ce soir formidablement démentis . Cette vague , ça n' est pas non plus un mouvement parisien , ou alors ce soir j' ai un problème de géographie . Et cette vague , ça n' est pas une bulle , c' est une vague qui monte depuis le premier jour , parce qu' elle vient du cœur du pays , parce que ce que depuis le premier jour nous avons fait ensemble , c' est d' écouter , de construire , d' aller chercher les idées , les convictions , partout dans le pays . Nous avons fixé un cap , des valeurs , mais l' énergie est là partout et c' est celle -ci qui accompagne En Marche , c' est celle -ci qui monte , et ce soir avec vous j' en suis fier , j' en suis heureux ! Alors je suis venu ce soir , à Clermont-Ferrand pour vous présenter mes vœux . Je vous ai fait grâce de ces vœux vidéo dans lesquels tout le monde aujourd'hui a décidé de donner son talent . Parce que quand on a certaines fonctions , ils sont légitimes , on peut les faire à ses amis , mais il me semble que ce qui a le plus de sens c' est de les faire au cœur , au contact du pays , et donc je les fais ce soir devant vous . Evidemment , il y a les vœux de santé , il y a les vœux pour vous , vos proches , vos familles , il y a des vœux de mobilisation , indispensables parce que notre combat est collectif pour redresser notre pays . Mais les vœux ce soir que je veux formuler pour vous , ils tiennent en deux mots : &laquo; penser printemps &raquo; . Ces deux mots , ce ne sont pas les miens , je les emprunte sur ces belles terres radicales à une grande figure du radicalisme français , à un grand intellectuel , un homme politique , un professeur : Alain . Il fait partie de ceux qui ont rythmés mon enfance , de ces lectures qu' on n' oublie pas . Alain a écrit beaucoup de grands textes , mais il a écrit aussi ce qu' on appelle ses “ Propos ” , qui étaient une espèce de carnet quotidien . Et il y a à peu près un siècle , un peu plus , il a écrit en un 31 décembre et dans les premiers jours de janvier , un très beau petit texte qu' il a intitulé &laquo; Penser printemps &raquo; . Et il disait , et je le cite de mémoire , donc imparfaitement , qu' au moment où l' année tourne sur ses gonds , quand - disait -il - les arbres font dentelle ( et ce soir je crois qu' on peut le voir , ce sont ces moments où le givre saisit la nature ) , lorsque tout semble mort , lorsque tout semble triste , lorsque tout parfois semble perdu , morose , il faut penser printemps . Non pas penser printemps pour s' émerveiller ; non pas penser printemps par bêtise , ou par une forme de béatitude médiocre ; mais penser printemps parce que , au moment même où cette désolation s' installe dans la nature , quelque chose se prépare qui est déjà là . Alors certains ne le voient pas ou ne veulent pas le voir , certains peuvent désespérer , mais le printemps revient , et il est déjà gros dans l' hiver , il est déjà là . Alors ce soir , pour vous et vos familles , pour nous tous , je veux vous dire de penser printemps . Et penser printemps , d'abord , cela veut dire ne pas nous affaisser , ne jamais accepter le quotidien , les peurs , les doutes , parfois la panique qui nous étreint , ne jamais accepter d' être victime et mener le combat . Et je commence par cela parce qu' aujourd'hui , il y a deux ans , notre pays était frappé par les attentats . Je n' oublie pas les attentats de Toulouse , dès 2012 . Je n' oublie pas tous les attentats qui ont suivi : l' Hyper Cacher deux jours plus tard , évidemment ceux de novembre 2015 , de Magnanville , de Saint-Etienne , du Rouvraix , de Nice . Mais , il y a deux ans nous avons encore toutes et tous à l' esprit ces 3 jours de panique qui ont commencé , ces 12 morts . Cela nous devons ce soir y penser . Je veux d'ailleurs ce faisant rendre un hommage tout particulier à l' un des nôtres , l' un des vôtres , Michel RENAUD , journaliste de Clermont qui était passé là pour échanger quelques dessins avec Cabu , et qui lui aussi fit partie des victimes . Alors ce jour-là , notre pays a été frappé au cœur , et ce sont les beaux mots de Prévert qui me viennent à l' esprit : &laquo; j' ai compris le bonheur au bruit qu' il faisait en partant &raquo; . C' est cela ce qui s' est passé . Ce fut la fin de l' innocence . Je vous dis cela ce soir , non pas pour agiter les peurs , non pas pour vous dire que nous devons tout oublier de notre quotidien , de nos défis , de notre avenir , non , pas du tout . On ne construit simplement rien de solide si on ne pense pas au passé , si on n' a pas en mémoire ce qui est advenu . Mais je vous le dis aussi ce soir , parce que penser printemps dans ces moments-là , ça veut dire avoir l' esprit de responsabilité , mais ne rien céder à la peur , ne rien céder non plus aux voix qui voudraient nous faire changer , basculer , nous faire oublier ce que nous sommes au titre de la peur . Dans notre pays , certains ont voulu nier ce qui s' est passé . C' est une erreur profonde . Mais dans notre pays , bien souvent la réaction a aussi été une forme de panique , d' agitation continue , une forme d' autoritarisme excessif , comme si cela permettait au fond de répondre à l' émotion . Rien ne répond à l' émotion . Elle est là et nous devons la vivre , l' embrasser , l' accompagner mais nous devons nous tenir debout . Ne jamais accepter d' être les victimes , non pas d' une guerre - une guerre supposerait d' avoir un ennemi et une armée - mais d' une lutte , une lutte sans merci qui a alors commencé . Une lutte contre le crime terroriste absolu , contre un islamisme terroriste et radical qui abîme notre pays , qui l' abîme parce qu' il conduit certains à confondre tous les sujets , et à vouloir par ce truchement attaquer une partie des Françaises et des Français en raison de leur religion . Ils divisent le pays en son cœur , parce que certaines et certains ont commis l' erreur de vouloir sombrer dans la panique . Nous devons dans cette période qui va durer , nous tenir droits , forts , parce que c' est cela l' Histoire , le sens , le cœur du peuple français . D' être parfois attaqué , mais de ne rien céder à la panique , d' être tenu par un commandement : celui de l' autorité vraie , qui donne un cap , qui assure une sécurité , mais qui à chaque instant , refuse de voir son agenda dicté par quelques-uns qui sont des barbares , refuse de se laisser diviser par quelques-uns qui n' attendent que cela . C' est cela notre devoir , et c' est cela ce que je veux conduire . Alors cela signifie continuer à nous battre , sur tous les théâtres d' opérations où la France et ses valeurs doivent être défendues , et où les risques peuvent advenir , où les terroristes se préparent : la Syrie et l' Irak aujourd'hui , la Libye sans doute demain , une partie du Sahel . Et la France continuera à prendre ses responsabilités sur ces théâtres d' opérations , elle le continuera avec un engagement sans faille , et je veux ici , avec vous , ce soir en ce début d' année rendre un hommage vibrant à nos forces de l' ordre , à nos armées , à toutes celles et ceux , à nos enfants qui se battent sur ces théâtres d' opérations , pour défendre la France , pour la protéger contre ce qui est et ce qui peut advenir . Cela , nous continuerons à le porter . Mais nous l' inscrirons dans une stratégie qui doit être plus européenne , avec nos partenaires allemands , avec nos partenaires européens . Parce que ces défis sont des défis européens , aussi . Parce que la France ne peut porter seule , éternellement , ce fardeau . Parce que l' Europe , c' est un rêve de paix , c' est un rêve vibrant , qu' on doit cesser d' accuser de tous les maux , mais à l' égard duquel nous devons être exigeants . Et donc oui , j' irai la semaine prochaine à Berlin pour parler précisément de cela , des nouvelles menaces qui existent partout dans le monde , d' un monde qui se bouleverse , où la puissance américaine n' est plus là pour nous protéger , et où la France doit porter son rôle , son rang , mais avec ses partenaires , avec l' Allemagne , avec l' Europe . Être à la hauteur de ces défis , penser printemps aujourd'hui en se remémorant ce qui s' est passé il y a deux ans , c' est aussi prendre toutes nos responsabilités en matière de sécurité . J' entends les doutes , les colères de nos forces de l' ordre , qui se sentent aujourd'hui parfois abandonnées , qui ont besoin de moyens - et j' y reviendrai dans les prochaines semaines - qui attendent de la visibilité , une considération légitime . Elles étaient là , tout au long des mois qui ont passé . Et nous devons non seulement maintenir les engagements pris , mais aller plus loin , confirmer les 10 000 engagements d' embauche dans la police et dans la gendarmerie pour les 5 prochaines années , qui sont indispensables pour maintenir la sécurité de nos territoires , au plus proche du terrain . Par une police de proximité , par une police au contact de la population qui , elle aussi , permettra de mieux prévenir . Cette sécurité , elle est la réponse dans la durée , sans panique , sans agitation , dans la constance , la réponse à ce que nous sommes en train de vivre . Car cette lutte va durer . La réponse c' est aussi d' avoir , chevillée au corps , une vraie politique de laïcité . Ah , ce mot ! Certains lui auraient presque fait perdre son sel , en le répétant à ce point . La laïcité , ça n' est pas un mot de haine . Ça n' est pas un ennemi . Ça n' est pas non plus un mot devenu ringard , comme disent d' autres . C' est le cœur vibrant de notre République . C' est l' un de ses principes . C' est au cœur de son socle . Parce que la laïcité c' est une liberté : la liberté de croire , et de ne pas croire . La laïcité , c' est ce principe qui protège tout un chacun , qui dans un quartier , dans une ville , ne veut pas céder à la tyrannie de la majorité et veut garder son quant-à-soi , sa liberté de penser . La laïcité , c' est cette liberté qui permet à d ' autres d ' exercer leur religion en respectant les autres religions , mais de le faire partout aussi dans la République . Ne cédons rien sur ce principe . Ni à celles et ceux qui veulent défendre des communautarismes en attaquant la laïcité , qui veulent porter un projet politique contre la République en s' attaquant à la laïcité , mais ne laissons rien non plus à celles et ceux qui veulent attaquer la laïcité pour porter une forme de projet de haine ou d' exclusion . La laïcité c' est la République française tout entière . C' est un bloc , et ce bloc , c' est une liberté ! À vos côtés , ce sont ces engagements , ces principes , que par fidélité à ce que nous sommes et ce que nous avons vécu , je porterai durant les mois qui viennent et les années qui viennent . Penser printemps , c' est construire l' avenir , vous l' avez compris . C' est cela , le rêve d' Alain . Mais penser printemps , ça ne veut pas dire non plus promettre à tout va , penser que le printemps est déjà là , que sans avoir semé on pourrait déjà promettre la moisson à tout un chacun . Parfois ces périodes de campagne électorale sont une incitation à la moisson de ce qu' on n' a pas encore semé . Mais pour penser printemps , il ne faut pas non plus désespérer , il faut accepter d' expliquer qu' on peut semer sur toutes les terres , qu' on peut produire sur toutes les terres , à condition qu' on s' en donne la peine et les moyens . Le grand homme de cette ville , de cette région , Pascal , avait une très belle formule : il faut qu' il y ait dans l' éloquence , disait -il , de l' agréable et du réel . Mais il faut que cet agréable soit réel . C' est une de ses “ Pensées ” ; elle est d' actualité . Parce que ce que je vois autour de nous , c' est beaucoup de projets qui parfois fleurissent , des promesses qui aujourd'hui , dans les campagnes de certains , sont formidables : on vous promet de distribuer l' argent qu' on n' a pas , des revenus universels , de nouveaux droits , un crédit . On promet au fond l' argent de nos enfants ou de nos petits-enfants . C' est de l' agréable sans le réel . Puis de l' autre côté , j' entends des projets qui vous proposent la purge , le tournevis . L' Angleterre , mais pas celle d' aujourd'hui , celles des années 80 . La société française , mais pas celle d' aujourd'hui , celle d' avant les années 60 . Ils promettent une forme de réel , celui des arbres qui font dentelle dont nous parlait Alain . Mais enfin , c' est pas très agréable . Et donc pour penser printemps il faut conjuguer les deux . Il faut regarder la réalité telle qu' elle est , avec tout ce qu' elle emporte de contraintes , de défis . Et donc il ne faut pas semer des illusions . Mais en même temps , il faut refuser d' enfermer les Français dans une espèce d' alternative mortifère où pour progresser , il faudrait réduire les droits de certains ; où , pour aller de l' avant , il faudrait désespérer ; où , pour essayer d' entrer dans le XXIème siècle , il faudrait adopter les mesures et les techniques d' il y a 20 ou 30 ans . Penser printemps mes amis , c' est réconcilier l' ambition et le réel , c' est réconcilier la volonté de faire et les contraintes d' aujourd'hui . Et donc , le projet que je veux porter avec vous , c' est un projet de bon sens . Mais où nous allons toutes et tous nous dire la vérité : il faut d'abord produire pour pouvoir distribuer . Sinon ça n' existe pas . J' avais tout à l' heure des gens très généreux qui , devant les Halles de Clermont , m' expliquaient : “ vous … ” - malheureusement , chacune et chacun vit dans sa condition : je passe pour être un odieux banquier ; j' essaye de me remettre , de me traiter , depuis plusieurs années , mais ça ne passe pas … Et donc , on me disait “ vous , vous êtes un odieux banquier , vous ne faites rien pour les pauvres ” . Alors j' essayai de me défendre , je lui dis “ non , les gens comme vous m' avez attaqué déjà il y a deux ans , quand on a fait l' ouverture des cars ” . C' est pas mes chiffres , ça vient d' être montré par des analyses indépendantes : 900 000 personnes modestes , pauvres - parce qu' il faut appeler un chat un chat - ont pu se déplacer . Des gens qui ne pouvaient pas se payer le train , ou qui n' ont pas de voiture . Et donc , lui disais -je : “ vous vous faites une fausse idée . Vous appelez ça de la libéralisation : c' est un gros mot . ” On a ouvert un secteur , les autocars , qui était fermé , tout était interdit . Vous vouliez ouvrir une ligne entre Clermont et Lyon , ou Clermont-Marseille , il fallait demander des autorisations invraisemblables , il fallait faire du cabotage international . Si vous faisiez Clermont-Milan en passant par Lyon , vous aviez le droit . On a simplifié . Donc oui , on a libéralisé . Mais la liberté , on l' a réconciliée avec la justice . Parce que qui en bénéficie ? Des gens qui choisissent de se déplacer ainsi , et qui ne pouvaient pas quand c' était fermé avant . Et donc vous voyez bien cette fausse idée . Parce que lui il me disait : “ si vous étiez pour les pauvres , vous taxeriez les riches ” . Je lui ai dit : il faut que vous regardiez comment les autres font à côté . Il faut , vous avez raison , pour redistribuer , taxer les plus fortunés . Mais enfin , on les taxe pas mal en France déjà . Honnêtement . Je vais être honnête , je ne vais pas pour vous faire plaisir dire qu' on va taxer davantage les riches . Parfois ça a marché dans des campagnes . On a pu dire : on va les taxer encore plus . Dans une économie ouverte au monde - et la France elle est dans le monde , elle produit dans le monde - on peut décider de taxer les plus riches . Mais enfin , ils vont s' enrichir ailleurs . Donc on perd des opportunités . Il faut mieux les taxer pour réinvestir dans les services publics , dans l' économie . Mais il faut surtout les encourager à produire davantage dans le pays . Et ensuite il faut voir comment donner une place à chacune et chacun , comment ouvrir des secteurs . Ce qui est intolérable , c' est pas que des gens s' enrichissent , quand c' est pour de bonnes raisons : quand ils ont inventé quelque chose , quand ils travaillent dur , quand ils s' engagent . Ce qui est intolérable c' est que d' autres , parce qu' ils viennent de milieux modestes , ne puissent pas s' enrichir , ne puissent pas , alors qu' ils ont des talents , du travail , de la volonté de faire , aller de l' avant . C' est cela , le vrai combat qu' on doit mener ! Et donc ce que je veux , c' est qu' on réconcilie la liberté et la justice . La liberté et l' égalité . Ça n' est pas un hasard , si sur le frontispice de notre République , ces deux mots sont accolés . Ça n' est pas pour les opposer , c' est parce qu' ils se conjuguent ensemble , en action . C' est cette réconciliation à laquelle nous devons arriver . C' est cela le projet progressiste que nous allons porter ! Alors pour ce faire , il faut libérer . Libérer l' énergie , l' énergie des individus , l' énergie du peuple , et l' énergie des territoires . Ce sera ma première priorité . Le même Alain avait une phrase superbe . Très radicale , cher Jacques , du début du XXème . Il disait : “ l' individu qui pense contre la société qui dort , voilà l' histoire éternelle et le printemps aura toujours le même hiver à vaincre . ” On retrouve notre ‘ penser printemps ' . Cette bataille de l' individu , c' est celle pour la liberté , la liberté de faire , d' entreprendre , de créer , de travailler . Alors oui , je crois à la liberté des individus , à la liberté pour faire . Et je l' ai dit , le premier combat que je veux mener , c' est un combat pour le travail , je veux que chacune et chacun puisse travailler dans notre pays plus facilement , que les entrepreneurs embauchent plus facilement , que les entrepreneurs investissent plus facilement , mais que chacune et chacun puisse aussi travailler plus facilement et soit mieux récompensé de son travail . Alors pour cela , qu' est -ce que nous allons faire ? D'abord conduire une réforme du Travail ambitieuse : notre droit du travail doit être simplifié . Non pas en vous promettant une loi qui va dire “ on doit passer de 35h à 39h , on doit modifier tel ou tel dispositif ” , non . Parce que d'abord je rentrerai dans une rizière terrible , qu' on connaît par cœur - on en a vécu un énième épisode au printemps dernier . Nous devons rentrer , dans la matière , dans le cadre d' une République contractuelle . Qu' est -ce que ça veut dire ? La loi doit définir un cadre , un ordre public social ; le reste il faut le renvoyer au contrat , à la discussion sur le terrain : les accords majoritaires de branches , d' entreprises , ce qui sur le terrain , permet de construire les bonnes solutions pour , en matière de seuils , en matière de temps de travail , en matière d' organisation , permettront de s' organiser plus librement , plus concrètement . Je refuse de réduire la discussion sur le droit du travail à une espèce d' opposition entre les généreux irréalistes et les réalistes qui réduisent les droits . Ça , c' est une bataille qui mènera nulle part . Comment nos voisins ont fait ? Certains ont accepté de réduire les droits , massivement . Eux ils ont fait de l' austérité - on entend souvent parler de cela . Ce n' est pas ce que je propose . D' autres , les Allemands , certains autres voisins , ont su créer , au plus près du terrain , des vrais accords , une logique productive industrielle pertinente . C' est cela que je veux pour notre économie . Parce que , une exploitation agricole , l' exploitation que nous avons visitée ensemble dans le Cantal , elle n' a rien à voir en termes de droit du travail , en termes de contraintes , avec un grand groupe industriel qui est à Paris , qui a deux usines en France , ou même avec la Bosch , comme on dit , à côté de Rodez . Ça n' a aucun sens ! Vous avez d' un côté quelqu'un qui a deux employés , qui doit se lever tous les matins à 5h quand ça n' est pas plus tôt , qui se bat au quotidien , avec des règles qui ne sont pas faites pour lui ; et de l' autre un grand groupe , qui a des milliers d' employés , et qui est installé , avec une solidité financière , une capacité à s' organiser , à gérer la complexité . Si nous voulons avoir une réponse adaptée , et bien il faut permettre , selon les secteurs , la taille l' entreprise , d' adapter nos règles . De permettre que dans cette exploitation agricole , qui veut grandir , ou dans un restaurant , qui veut se développer - il y en a tellement qui viennent de la région et que je connais si bien à Paris … Vous pouvez les applaudir parce que ce sont des courageux ! Il faut qu' ils puissent négocier peut-être de ne pas avoir , de ne pas créer un comité d' entreprise ou un CHSCT à 50 salariés . Parce que les représentants des salariés sont prêts , parce que ça n' a pas de sens pour eux , mais en revanche ils veulent des heures supplémentaires mieux payées . C' est ça le vrai dialogue social . Le dialogue pragmatique . C' est cela que nous allons mettre en place , c' est ce sur quoi on va avancer . Ensuite , il y a un point qui me tient à cœur : c' est ce qu' on appelle le coût du travail . Là aussi , que n' entendons -nous : “ quand on baisse le coût du travail , on fait des cadeaux aux entrepreneurs ” . Non . On ne fait pas des cadeaux . De la même façon que je n' ai jamais croisé un entrepreneur qui embauchait les gens pour les licencier . Il y a des idées ancrées dans la tête de certains qui n' ont pas beaucoup de sens . Mais nous sommes au cœur d' une révolution numérique , d' une transition environnementale profonde , qui change nos habitudes de consommation , qui change notre organisation collective . Qui fait que nous sommes aujourd'hui dans ce monde ouvert , soumis à la concurrence de pays à bas coûts , de pays qui innovent , et que nous-mêmes dans beaucoup de secteurs , nous devons innover , investir , transformer le modèle productif . Et pour cela , et bien il faut prendre en compte deux choses : il faut que les employeurs puissent investir pour se moderniser , pour produire moins de CO2 , pour justement accompagner , non pas accompagner , faire cette transition énergétique et environnementale . Ce sont les entreprises qui les font sur le terrain , celles qui vont investir pour moins polluer , pour développer des panneaux solaires , pour changer leurs processus de production , pour avoir justement une meilleure gestion des flux . Et , elles doivent investir aussi - tu le sais bien cher Bruno - pour la robotique . Pour là aussi , se développer , être plus compétitives . Pour le numérique . Quand vous n' avez pas de marge , vous ne pouvez pas investir . Et donc , en 2011 , la France a atteint un niveau record de charges mais surtout elle avait le niveau record des marges les plus basses . Quand les entreprises en sont là , elles ne mènent aucun combat . Et l' investissement qu' on ne fait pas aujourd'hui , ce sont les emplois qu' on fera pas demain . Donc oui , je suis pour conduire une politique d' amélioration des marges de l' entreprise , pour qu' elles puissent investir et embaucher . Et de l' autre côté , si vous ne baissez pas le coût du travail , le travail sera de plus en plus concurrencé par le travail à l' autre bout du monde , celui qui ne coûte rien et qui ne paye aucune charge en France , celui qui est fait en Chine , en Inde , ou ailleurs ; ou la robotisation . Et c' est à ce moment-là qu' on fait des choix qui ne sont pas les bons . La robotisation , elle accompagne le travail . Mais si on ne fait pas les bonnes réformes et bien on supprimera tout , à terme . C' est pourquoi je supprimerai le CICE pour le remplacer par une baisse de charges , claire , durable , pour tous les types d ' entreprises . Pas pour faire des cadeaux . Pour défendre l' emploi . Pour défendre l' investissement dans mon pays . Et donc ce sera , pour tous les types d' entreprises , les petites , les grandes , les individuelles , les collectives , celles de l' économie sociale et solidaire comme celles qui ne paient pas l' impôt sur les sociétés : 6 points de charges en moins . Et ce , pour tous les salariés jusqu' à 2,5 SMIC . Mais en plus de ça , il faut aller plus loin . Les charges , c' est aussi et surtout un problème pour celles et ceux qui sont les plus jeunes , les moins qualifiés , qui peuvent parfois avoir une opportunité d' emploi , mais pour lesquels le SMIC , avec les charges , est trop important . Il y a aujourd'hui , sur notre territoire , beaucoup d' emplois détruits ou d' emplois qui ne sont pas créés à cause du niveau de charges qu' il y a encore au niveau du SMIC . Et donc , en plus de ces six points de charges , je supprimerai toutes les charges générales ( … ) . C' est une mesure que je n' avais pas jusqu' alors révélée qui fait que , pour tous les emplois au SMIC , il y aura 10 points de charges patronales en moins . C' est un investissement . C' est un investissement pour toutes les entreprises qui en ont besoin . Ces 4 points additionnels , et bien je les financerai en faisant successivement monter en charge cette fiscalité environnementale qui est prévue , et en particulier les convergences nécessaires . Parce que c' est comme ça que nous arriverons à faire progressivement muter notre économie . Alors pour libérer le travail il y a bien d' autres mesures , bien d' autres décisions et engagements que j' ai annoncés , et que je vais vous confirmer : nous augmenterons le pouvoir d' achat . Salariés , fonctionnaires , indépendants , professions libérales verront leur pouvoir d' achat augmenter compte tenu de la mesure de compensation que je prendrai sur la CSG . Tous ! Parce que je veux qu' en France , le travail paye mieux . Nous irons aussi plus loin sur la réforme des prud'hommes , indispensable , où je confirmerai le plafond censuré par le conseil constitutionnel , qui est absolument nécessaire pour donner de la visibilité . Je supprimerai le RSI . Je le supprimerai en faisant deux choses : d'abord en permettant un droit d' option pour tous les indépendants pour opter pour le régime général d' assurance maladie . Je vois que , on s' en est parlé ce midi ... je vous le rassure , je n' ai pas découvert cela ce midi , mais je savais que j' allais vous l' annoncer ce soir . Pourquoi ? Parce qu' aujourd'hui beaucoup d' indépendants qui payent moins de cotisations qu' au régime général découvrent qu' ils ont des risques . Ils tombent malades . Ils ont certains problèmes qui ne sont pas bien couverts par le RSI . Et à ce moment-là , ils se tournent vers d' autres garanties , dites Madelin , qui sont des régimes plus spécifiques . Et on leur demande à ce moment-là beaucoup plus que pour le régime général . Et ils découvrent qu' il faut faire des tests de santé , qu' on leur demande des charges nouvelles . Et ce qu' ils pensaient être un cadeau devient un cauchemar . Donc il faut qu' il y ait un droit d' option , parce que ça correspond à la réalité de la vie . Je supprimerai cette caisse qui gère trop peu , en fait , d' indépendants , et qui donc a des délais trop importants , a des ratés , des erreurs , et qui donc conduit à des tracasseries , parfois des traumatismes , pour beaucoup d' indépendants , parce que tous les indépendants qui travaillent en France doivent avoir les mêmes droits que l' ensemble des salariés du secteur privé ou que l' ensemble des fonctionnaires en terme de qualité des prestations , en terme de rapidité des remboursements . Et donc cette réforme , c' est cela . Alors pour penser printemps il faut libérer les énergies , le travail . Mais il faut aussi , dans le même mouvement , tenir nos équilibres , et faire plus pour ceux qui ont moins . C' est cela le deuxième pilier de ce penser printemps que je voulais partager avec vous ce soir . Faire plus pour ceux qui ont moins , ça veut dire : parce qu' on a produit davantage , parce qu' on a trouvé ces flexibilités , ces libertés , parce qu' on a ouvert des secteurs , parce qu' on a accepté d' avoir plus de flexibilité dans le travail , on doit aussi accepter de regarder les transformations en cours et les injustices qui sont là , qui se créent . On ne fait pas bien si on libère sans être juste . Le projet aujourd'hui porté par la droite conservatrice est un projet injuste parce qu' il libère … J' entends certains dire “ c' est un projet libéral ” . Non . Non . Le libéralisme politique comme économique , il libère , mais il libère les individus en regardant les inégalités de départ . Il ne libère pas pour que ceux qui ont déjà réussi réussissent mieux . Ça c' est du conservatisme . Par contre , quand on libère , quand on ouvre , quand on permet aux énergies de se déployer , il est indispensable , dans le même temps , de considérer que certaines et certains n' ont pas les mêmes chances , viennent d' une autre famille , d' un autre endroit , d' autres territoires , qu' ils n' ont pas les mêmes capacités à faire , parce qu' il y a des inégalités de départ . Et c' est pour cela que ce deuxième pilier dont je veux vous parler ce soir , ce “ faire plus pour ceux qui ont moins ” , ça n' est pas de la commisération . Ça n' est pas l' aumône - c' est insupportable . C' est le cœur de la promesse républicaine , celle de fraternité qu' on oublie trop souvent , mais c' est celle de l' égalité vraie . C' est celle qui consiste à regarder , non pas une égalité d' arrivée , qui est une promesse intenable . Je ne vais pas vous promettre ce soir , mes amis , que tout le monde à la fin , que tous et toutes , nous serons égaux , ça n' est pas vrai . Parce qu' il y en a qui sont plus grands , d' autres qui sont plus petits ; il y en a qui travailleront plus , d' autres moins ; il y en a qui sont talentueux , d' autres non ; il y en a qui auront de la chance , et d' autres qui en auront moins . Mais ce qui est insupportable c' est qu' avec les mêmes mérites , la même volonté , le même engagement , parce qu' ils seront nés dans une famille donnée , dans un territoire donné , ils n' auront pas pu réussir comme d' autres , ailleurs , c' est cela qui est intolérable aujourd'hui . Alors faire plus pour ceux qui ont moins , c' est penser ensemble nos transitions . C' est penser ensemble pour le chômage et la formation continue , des droits plus justes . Mais ce que je veux , c' est un chômage qui n' est plus seulement une assurance donnée à quelques uns ; des droits sociaux qui ne sont plus seulement une assurance donnée à quelques uns . Mais un système social où le chômage comme la maladie est un droit ouvert donné à ceux qui travaillent ou entreprennent , aux exploitants agricoles et qui n' ont pas le droit au chômage , ce qui est une injustice profonde . Aux commerçants , cher Francis , aux artisans , à toutes celles et ceux qui eux aussi prennent des risques et qui , s' ils ratent , peuvent tout perdre . C' est un élément de justice et d' efficacité . Mais en même temps , ce que nous allons faire , j' entends certains qui disent “ ce n' est pas finançable , c' est horrible ” - les gens dont je parle là , ce ne sont pas des . Il faut arrêter de penser ça . On ne va pas voir demain des artisans , des indépendants qui vont se mettre au chômage . Ils se mettent déjà pas en congé maladie . Non . Par contre , nous allons être justes en cas de faillite . Nous allons éviter - il faut bien le dire et le regarder en face - des suicides , parce que qu' est -ce qui se passe ? Sur nos territoires , ce sont des suicides , partout , dans ces catégories , parce qu' on laisse des Françaises et Français sans solution . Et nous allons le financer par une vraie politique de droits et de devoirs . Le chômage ce ne sera plus un droit acquis . J' en changerai le financement . Il ne sera plus payé par les charges salariales , mais par l' impôt . Donc ce sont des droits , mais en même temps des devoirs . Plus personne ne vous dira “ Moi j' ai droit au chômage donc je vais y rester encore . ” ; “ Moi j' ai droit au chômage donc l' offre d' emploi que vous me faites là , elle est bien ... mais elle serait mieux dans 3 mois . ” Je ne vous dis pas que c' est la généralité . Parce qu' être au chômage c' est une stigmatisation , c' est une douleur au quotidien . Mais aujourd'hui , parce que notre système est ainsi , et bien il y en a qui l' optimisent . Je déteste le discours porté par certains , parfois dans cette région , ce discours d' assistanat . Non . Il n' y a pas d' assistanat . Il y a des gens qui ne font pas leur devoir - bon , c' est une chose . Mais parler d' assistanat , c' est mettre tout le monde dans le même sac , c' est dire : “ le chômeur est un fainéant ” . Mais enfin ! Celui qui a vécu un choc industriel , une transformation profonde , dont l' entreprise s' est effondrée . Celui qui n' arrivait plus à rendre son lait , c' est un assisté ? Allons . Mais en même temps , ça donne le sentiment que tout se vaut . Par contre , l' idée que portent certains - les mêmes - qu' il faudrait réduire la durée des indemnités de chômage , elle ira plus vite vers l' assistanat . Parce que c' est l' idée que certains qui sont au chômage , si on leur réduisait les droits , iraient plus vite retrouver un emploi . La belle affaire ! Ils en trouvent sous le sabot d' un cheval ? Non , ils iraient plus vite vers les minimas sociaux . La clé , c' est un système de droits et de devoirs . Je veux que tous les travailleurs , quelle que soit leur situation , puissent avoir accès au chômage . Je veux que tous les travailleurs aient accès à la formation quand elle est nécessaire . Parce que demain , s' il y a un accident industriel dans un endroit , on ne peut pas laisser les Françaises et les Français dans le même secteur avec les mêmes qualifications . Il faudra les reformer . C' est une transformation , une révolution en profondeur de notre formation continue que je propose . Parce qu' il faudra , pour un an , deux ans , les former pour aller vers un autre secteur , parfois une autre région , pour pouvoir justement gagner leur vie . C' est cela les nouveaux droits que nous devons créer . On dépense plus de 35 milliards par an pour la formation continue . Et on forme toujours les mêmes . Ceux qui sont déjà bien formés , dans l' entreprise , qui ont des contrats longs . Ceux qui sont protégés . On ne forme pas les chômeurs . On ne forme pas ceux qui sont en intérim . On forme mal ceux qui vont avoir des transitions à venir . Donc ce sont les droits que je veux créer en redéployant notre organisation collective , en réorganisant en profondeur . Mais à côté de ça , il y aura des devoirs . Quand vous avez eu une formation , ou quand vous êtes dans une région où il y a une offre d' emploi , chez l' exploitant agricole dans le café-restaurant dont je parlais tout à l' heure , et bien vous ne pouvez pas refuser une offre d' emploi décente . C' est cela , avoir une vraie politique républicaine . Non pas dire “ tous ceux qui sont au chômage sont des assistés ” , mais d' expliquer qu' il y a des droits et des devoirs . Et donc derrière ces devoirs il y aura une obligation , et donc des contrôles . Certains seront amenés à accepter des offres d' emploi qui ne leur plaisent pas , qui n' étaient pas leur premier choix , mais qui correspond à ce qu' ils savent faire , à des besoins , parce qu' ils auront eu en même temps de vrais droits . C' est cela la République , nous l' avons oublié . Et donc ni facilité , ni laxisme , ni discours de stigmatisation , ni injustice ; le progressisme ! Faire plus pour ceux qui ont moins c' est aussi avoir une vraie politique pour l' école , j' en reparlerai plus en détail , mais on n' est pas dans la même situation quand on rentre en CP Auvergne ou dans les beaux quartiers de Clermont . C' est la réalité . Or aujourd'hui on a la même politique éducative . On a des petites différences : on enlève deux , trois élèves , on paye quelques dizaines d' euros , quelques centaines d' euros . Un enfant qui arrive dans un quartier en difficulté en CP , et bien , il a parfois 200 mots de vocabulaire . Il a parfois des parents qui savent mal le français . Il a parfois jamais été devant un pédiatre , et donc on ne sait pas s' il sait lire au tableau . On ne sait pas s' il entend très bien . Il a des parents qui ne s' occupent pas bien de lui parfois . Il y a une détresse sociale . Celui qui arrive dans les beaux quartiers , c' est formidable , c' est notre fierté , c' est la grandeur de la France . Mais , il a eu des milliers d' heures avec ses parents . Il sait déjà bien souvent lire , écrire et compter . Il a été plusieurs Lui , à l' école ça va bien se passer . Je peux déjà vous dire aujourd'hui , à 90 % près , où sont les jeunes qui iront en prison , qui auront le chômage de longue durée , qui seront des décrocheurs et qu' on retrouvera en CM2 , qui ne sauront pas lire , écrire , compter . 20 % des élèves de CM2 en France ne savent pas bien lire , écrire , compter . Mais ils sont à 80 % dans ces quartiers . Et donc oui , là il faut faire plus pour ceux qui ont moins . Mais beaucoup plus . Investir massivement . Accepter d' avoir des enseignants plus expérimentés beaucoup mieux payés et d' avoir vraiment deux fois moins d' élèves par classe . C' est cela que je détaillerai dans les prochaines semaines . La vraie injustice , c' est celle qui existe aussi en santé . Parce qu' il y a l' inégalité sociale , l' inégalité territoriale qui mine notre territoire et une partie de notre population . La santé ça n' est pas qu' un coût . On doit continuer à baisser le coût de la santé en France , c' est évident . Mais le faire de manière intelligente : avec le territoire et les professionnels , le faire de manière concrète , sur le terrain . La santé ce n' est pas la même , ici à Clermont , et à Paris . Ça n' est pas la même dans le Cantal , à Rodez . Hier à Nevers il n' y a pas un service où on ne m' a pas dit “ Nous il manque trois postes , quatre postes . En télé-médecine , cela va nous aider à faire les premiers soins . &quot; Je ne suis pas sûr qu' il faille supprimer des postes d' infirmières , d' aides-soignantes ... Par contre , ils ne sont pas bien répartis sur le territoire . Et donc , nous avons en santé des défis . Le premier : celui de la prévention . Parce que si on veut baisser les coûts , les gens doivent être moins malades . Ce que je dis est du bon sens , on l' a oublié . La France soigne bien , elle prévient mal . Et donc en la matière j' ai pris plusieurs engagements hier , je les rappellerai . Deux , ce soir , que je veux rappeler : Le premier : je veux que tous les étudiants dans les professions de santé aient ce que j' appelle un service sanitaire de 3 mois . Qu' est -ce que ça veut dire ? Que tous les étudiants infirmiers , dentistes , médecins , kinés , aient trois mois de leurs études prévues pour faire de la prévention . Ils seront formés , une semaine à deux semaines , et ils iront à l' école , dans les lieux de travail , dans les maisons de retraite , pour faire de la prévention contre le tabac , contre l' alcoolisme . Ils iront regarder à l' école si les enfants , justement , voient bien au tableau , s' il n' y a pas de la myopie qui n' est pas détectée , s' il n' y a pas des problèmes de surdité , qui ne sont pas détectés chez les plus âgés . C' est fondamental , la prévention . Ça n' a aucun coût , ce sera dans la maquette pédagogique , et c' est une armée , chaque année , de 40 000 jeunes professionnels que nous déploierons partout sur le territoire . La deuxième mesure en matière de prévention , c' est que je veux que dans les 5 années , on couvre à 100 % les lunettes , les dents et les prothèses auditives . Je vais vous dire pourquoi et comment on le finance . Pourquoi ? Parce qu' un enfant dont les parents ne peuvent pas lui payer ses lunettes , ce qui existe aujourd'hui , c' est un enfant qui ne peut pas poursuivre ses études . Parce qu' un chômeur qui ne peut pas soigner ses dents , qui a les dents gâtées - pardon de parler de choses triviales , mais enfin c' est ça notre quotidien , c' est pas des belles et grandes idées , toujours , c' est le mariage des deux - s' il a des dents gâtées et qu' il ne se les soigne pas , il ne décrochera pas l' embauche , on le sait bien . L' employeur le verra , il dira “ il a une drôle de tête ” . C' est comme ça , c' est d' une cruauté infinie . La personne âgée qui ne peut pas se payer l' appareil auditif qui coûte très cher , c' est quelqu'un qui continuera à entendre mal , qui s' accommodera de cela progressivement . Mais elle ira beaucoup plus vite vers la démence , l' isolement , la grande dépendance qui nous coûte beaucoup plus cher . Et donc vous voyez que si on traite mal ces sujets , mais comme tous les sujets en matière de prévention , et bien ça coûte in fine beaucoup plus cher à la société , beaucoup plus cher à notre système de santé . C' est fondamental . Alors , comment je vais le financer ? Et bien , par un travail entre la sécurité sociale et les complémentaires . Un travail pour justement étendre cette couverture pour tous , mais pour aussi baisser les prix . Parce qu' aujourd'hui , c' est parfois 3 fois plus cher en France que chez les voisins d' avoir des prothèses dentaires . C' est 3 fois plus cher que chez les voisins d' avoir des prothèses auditives . C' est ça la réalité ! Et donc , quand je prends cet engagement , c' est un engagement aussi d' une plus grande concurrence , d' une liberté accrue , d' une transparence tarifaire , d' une pression mise sur certains pour qu' on puisse couvrir tout le monde , au bon prix . Et puis , le défi de la santé , c' est aussi celui de la ruralité . Je veux vous dire avant de conclure quelques mots sur ce beau sujet , sur ce grand sujet , qui est essentiel lorsqu' on veut faire plus pour ceux qui ont moins . Parler de ruralité , ça n' est pas dire : “ On va faire plaisir à quelques-uns , ils sont perdus , parce que c' est des territoires reculés de la République et qu' il faut en quelque sorte , de la même façon parfois que certains ont abordé les territoires ultra-marins , en disant : On va faire notre devoir , on va s' acquitter de ce qu' on leur doit puisqu'ils se plaignent tant ” . Ça n' est pas ça parler de la ruralité . C' est tenir la promesse que j' évoquais tout à l' heure . C' est de dire : la ruralité , oui , ce sont des paysages formidables , ça fait plaisir aux métropolitains quand ils vont en weekend , ce sont des territoires qu' on aime retrouver pour les vacances , pour quelques jours . Mais enfin ! Ce sont surtout des territoires , qui veulent pouvoir vivre , qui veulent pouvoir réussir , qui veulent pouvoir avancer , et qui en ont le droit ! Et donc , faire plus pour la ruralité , ça n' est pas faire l' aumône . Faire plus pour la ruralité , c' est de donner les moyens d' être libres , d' être pleinement autonome . C' est faire le devoir républicain , parce qu' il n' y a pas la même égalité d' accès , pas la même égalité de départ , lorsqu' on dans la ruralité que lorsqu' on est en métropole , c' est cela la réalité . Ça veut dire qu' en matière de santé , il faut continuer à développer les maisons de santé pluriprofessionnelles , aider à attirer les professionnels sur le terrain en investissant , aux côtés des collectivités , en rendant la ruralité attractive . Pas en faisant des plans d' en haut , non . J' étais tout à l' heure dans l' Allier , à Saint-Pourçain , une formidable maison de santé pluridisciplinaire . Ils l' ont fait ensemble , ce sont les professionnels de santé qui l' ont créée : des kinésithérapeutes , avec des généralistes , des sages-femmes , avec des infirmières . Et ils l' ont fait en étant aidés par les collectivités locales . L' Etat aide un peu , mais il doit aider davantage . Mais , il ne faut pas faire un grand plan pour aider les maisons de santé . Il faut faire un grand plan de libération des territoires , pour que sur le terrain , les collectivités territoriales , accompagnées par les professionnels de santé et l' Etat , puissent développer cette offre , redevenir attractives , avoir des bourses d' installation et faire revenir les professionnels ! Ça veut dire aussi : permettre , dans la ruralité , à toutes celles et ceux qui entreprennent , d' entreprendre plus facilement . Ça veut dire : avoir une vraie politique en matière de foncier , pour les jeunes exploitants agricoles et les aider à s' installer . Parce qu' il y a une relève qui est là , qui veut aller de l' avant , parce qu' il y a une jeune agriculture , parce qu' il y a une jeunesse agricole . Elle est fière , et je veux l' accompagner . J' en ai assez que l' on parle de l' agriculture comme une chose qui est en train de tomber . Non ! C' est une part vibrante de nos territoires et de notre économie , qui veut réussir et aller de l' avant , et nous devons l' accompagner . Ça veut dire , des aides à l' installation et au foncier . Ça veut dire , une vraie politique de prix . Ce que demandent les agriculteurs , ce n' est pas d' être aidés ! C' est de vivre de leur travail ! Et donc en la matière , oui là , il faut remettre de la régulation . Je l' assume . Il faut remettre des contrats de filières , des vrais contrats de filières . Parce que , quel est le problème de l' agriculture ? C' est que vous avez quelques dizaines de milliers de producteurs pour lesquels c' est de plus en plus difficile , quelques rares centrales d' achat et distributeurs , et 60 millions de consommateurs . Qui fait le pouvoir ? Qui prend la décision ? Qui décide des prix ? Je ne vous fais pas de dessin . Et donc , et certains d'ailleurs dans la distribution sont en train de le faire , et je veux les saluer , par un risque de responsabilité : les fonds d' achat local , ils valorisent des contrats de filière . Et il ne faut pas tomber là dans des discours trop rapides , ou qui attaquent . Mais on doit valoriser , développer , ces relations de filières . Valoriser ces distributeurs , qui font justement de la vente de proximité , qui acceptent de fixer les bons prix , qui ne sont pas forcément les plus bas , en éduquant le consommateur , en expliquant que pour avoir des bons produits , il faut payer le travail qu' il y a derrière . Il faut payer la nourriture des bêtes , il faut payer l' entretien des terres . Et donc il faut accepter de rentrer dans ce contrat pour que les prix soient justes . Et donc nous généraliserons ces contrats de filières , ces bonnes pratiques . Nous généraliserons aussi , et nous encouragerons , ce que j' ai vu tout à l' heure dans vos Halles de Clermont : le circuit court . Je reviendrai dans les prochaines semaines sur ce sujet . Ça n' est pas la panacée sur tout , quand on doit exporter , quand on a des filières d' excellence . Mais enfin , c' est parfois la réponse . Et puis nous devons aider nos exploitants agricoles , à se diversifier , à avoir d' autres sources de revenus : dans l' énergie , par la mécanisation , ou par d' autres innovations , en développant d' autres activités , de valorisation du patrimoine - ils le font aussi . Et le patrimoine rural ne serait plus là s' il n' y avait pas les agriculteurs , des paysans , au fond - parce qu' on emploie toujours aujourd'hui des circonlocutions . Des exploitants agricoles , ce sont des paysans . C' est pas une insulte , “ paysan ” . C' est un beau mot . C' est être dans un pays , mais c' est aussi le faire , le transformer , le modeler , le façonner . Il n' y aurait plus de pays , s' il n' y avait pas de paysans . Et donc pour les aider à réussir , nous devons aussi les aider à investir . Ce sont les mesures de baisse du coût du travail que j' évoquais , et nous devons continuer , secteur par secteur , à aider les paysans à investir pour valoriser leur produit . La solution , c' est d' aller vers l' aval , comme on dit . C'est-à-dire , de ne plus vendre simplement son lait , mais de le transformer . On l' a vu ensemble , dans le Cantal , cher Jacques , il y a quelques semaines . Un formidable jeune exploitant ; il ne gagnerait pas sa vie , ce serait déjà fini , si il vendait son lait à la coopérative , il ne le rentabiliserait pas . Mais il a pu investir , et il fait du fromage . Et celui-là il le vend mieux . Et donc nous devons , partout , aider les éleveurs , qui aujourd'hui font du broutard et n' arrivent pas à bien le valoriser , à aller plus dans ces initiatives formidables , qui sont prises ici d'ailleurs - qui consistent , non plus à envoyer en Italie les broutards se faire engraisser pour ensuite payer plus cher le lait ou la mozzarella , mais en faire une ici , avec la même bête ! Le valoriser sur le terrain . Ce sont de formidables initiatives . Il faut développer l' aval , valoriser le produit , et pour cela aider nos agriculteurs , nos paysans à investir . Faire plus pour ceux qui ont moins dans la ruralité , c' est aider les commerçants . Ce que tu fais , cher Francis , est formidable en la matière , en Lozère . Les aider à se regrouper , à mieux s' organiser , pour faire face aux défis parfois de la désertification . C' est avoir une couverture numérique à la hauteur . Tout ce que je vous dis depuis tout à l' heure n' existe pas , si on n' a pas la fibre , ou la 3G au moins . Les maisons de santé avec la télé-médecine , formidable innovation , bon courage si vous ne captez pas . Et c' est le cas dans beaucoup de territoires . En tant que ministre , j' ai fait le maximum , je vous assure , on s' est battus à quelques-uns : pour développer la fibre , pour mettre encore plus de contraintes sur les opérateurs . On a fait des choses très compliquées , développer dans les centres-bourgs , ils seront tous couverts , en 2G -ce qui n' est pas suffisant , demain en 3G , d' ici l' été . Pour développer les antennes relais dans 1700 centres de priorité , au-delà des centres bourgs . Mais il faut aller plus loin . C' est l' investissement en infrastructures du XXIe siècle , et donc je vous le dit , je présenterai dans quelques semaines , un plan ambitieux de couverture du numérique , qui ira plus loin que celui en cours , en conduisant l' Etat à prendre ses responsabilités , et donc à investir lui-même à marche forcée partout où les opérateurs ne le feront pas . Parce que ce sont les infrastructures essentielles : ce sont aussi les routes , les accès . Combien se sont battus pour le TGV ou l' aéroport de proximité . Vous savez , je viens d' une ville qui ne s' est toujours pas remise de ne pas avoir eu le TGV . Avec mon épouse , nous venons d' Amiens . Je peux vous dire , j' y étais encore il y a quelques jours , on vous parle encore - et je le sais , c' est pour ça que j' en parle , on a vécu le même traumatisme - , il y a encore certains qui parlent de leurs bagarres d' il y a vingt ans . Parce que le TGV , à l' époque , il y avait des réalités politiques locales , il a été fait directement de Paris à Lille . Et bien Amiens a beaucoup perdu : de l' accès , des entreprises , des services . Et donc oui , les infrastructures de transport c' est aussi important . Alors je ne vais pas vous promettre un grand plan à 300 milliards . Parce que vous savez , les grands plans à 300 milliards d' infrastructures , certains grands Présidents de la République et Premiers ministres les ont promis . C' était en 2010-2011 , de mémoire . Les mêmes vous disaient que la France était en faillite , vous me direz . Mais enfin ces grands plans , c' était du vent , ils n' ont jamais été faits . Je ne ferai pas de promesses avec de l' argent que je n' ai pas . Par contre , ce que nous allons développer , là aussi avec les territoires dans un dialogue responsable , c' est que chaque territoire , bassin d' activité , chaque ville-centre , soit raccrochée par des infrastructures dignes du XXIème siècle , à la métropole ou au centre d' activités important pour elle . Ce ne sont pas des grands plans napoléoniens , mais c' est ça un plan efficace du XXIème siècle . Ça veut dire , que si le cœur de votre activité est dans une métropole qui est à 200 km , vous ne pouvez pas avoir 2 ou 3 heures de route , vous devez avoir un accès rapide , parce que la vie de vos entreprises , de vos concitoyens en dépend , parce que c' est le développement économique , parce que c' est votre santé , parce que c' est ça l' efficacité . C' est ce plan d' infrastructures concret avec les territoires que je veux faire et que nous porterons . Faire plus pour ceux qui ont moins , c' est aussi faire plus pour nos finances locales . Je sais qu' il y a beaucoup de maires , beaucoup d' élus locaux ce soir . Je vous le dis , en responsabilité , comme je l' ai dit pour la santé . Nous continuerons à faire des économies , c' est indispensable , je les présenterai dans les prochaines semaines . Je ferai des économies sur la sphère publique , comme on dit joliment , c' est à dire sur l' Etat , sur la santé , et sur les collectivités locales . Et ces économies , on devra les faire d'abord en faisant quelques réformes en profondeur , en faisant des réorganisations qui font de vraies économies , pas en passant le coup de rabot sur tout le monde . Parce que là il y a une victime : l' investissement local et la modernisation de nos territoires . Mais pas non plus en demandant le même effort à tout le monde . Quand vous êtes une métropole , un territoire d' attractivité , vous avez des revenus fiscaux , vous avez des entreprises qui vous paient de la taxe professionnelle , vous avez des gens qui ont de l' argent et qui peuvent payer plus facilement les impôts locaux . Lorsque vous êtes dans la ruralité , vous cumulez aujourd'hui toutes les difficultés . Et donc bien souvent , les efforts qu' on a demandés aux communes se sont traduits par quoi ? Des augmentations d' impôts locaux , fatalement . Ce qui fait que dans nombre de nos villes , dans nombre de nos villages et de nos villes des territoires , les charges que vous payez au titre de la taxe d' habitation ou de la taxe foncière sont plus importantes que celles que vous payez dans les grandes villes . Et comment derrière aller expliquer : “ il faut attirer des médecins ! Il faut attirer des entreprises ! ” Ils comparent aussi . Et donc pour cela , nous ne devrons pas demander le même effort à tous les types de collectivités . Ça n' est pas un cadeau , c' est un droit réparé . Ça n' est pas une obole , c' est notre devoir fait pour donner à chacune et chacun les mêmes armes et la possibilité de devenir à nouveau attractifs et de réussir . Parce qu' il y a partout en France , une volonté de faire , une innovation , une volonté d ' aller de l' avant . Et donc faire plus pour ceux qui ont moins , dans la ruralité comme dans les quartiers , partout en France , et bien c' est donner la possibilité aux énergies qui sont là , bloquées , assignées à résidence , parfois d'ailleurs pointées du doigt , et bien d' avoir une égalité d' accès , comme le reste du territoire , de pouvoir elles aussi réussir enfin , avoir accès à cette liberté dont je parlais tout à l' heure . C' est cela ce que je veux faire avec vous . Alors mes amis , pour conclure , vous l' avez compris , cet engagement , ces propositions que nous avons commencé à décliner , et que je déclinerai dans les jours et les semaines à venir sur le terrain , elles refusent le diktat des partis politiques actuels qui voudraient nous livrer à une forme d' espace fermé . Les uns nous disent : “ vous allez rentrer dans le XXIème siècle . Ce sera dur . Je vais vous demander des efforts . Surtout à ceux qui en ont déjà fait . Il faudra réussir . Surtout pour ceux qui ont déjà réussi . ” Et qui portent un projet conservateur . D' autres , ils vous disent : “ ça va être formidable . Avec nous , il y aura des cadeaux . Il y aura de la répartition , de la justice . Nous allons défendre , la main sur le cœur , un modèle social . Quant à savoir comment on produit , nous reviendrons demain . ” Ça ne marche pas . C' est pas du patriotisme , vous savez , de vouloir en France produire quand on ne propose que de redistribuer . Ça n' est pas de l' esprit de responsabilité ou du modèle républicain , quand on propose en France , en effet , de nouveaux droits et créances sans expliquer comment on les finance . Ça n' est pas de la justice , quand on n' a pas un vrai discours sur la liberté et la capacité à faire . Et la troisième voie qui nous est proposée , c' est celle de la haine , du repli . C' est celle portée par les extrêmes . C' est celle qui propose aujourd'hui aux Français de sortir de l' Europe , de sortir du monde , de se fracturer dans la haine . Mais celle -ci progresse , aussi . Elle progresse , de l' irresponsabilité des deux premières . Elle progresse du fait qu' on ne veut plus essayer de faire réussir le pays en donnant une place à chacun . Elle progresse du fait qu' on voudrait faire croire à tout le monde qu' on peut penser le progrès comme il y a 30 ans . Notre responsabilité , mes amis , est immense . Immense . Car nous sommes le seul rempart à l' esprit de haine et aux extrêmes . Nous sommes le seul rempart parce que nous sommes l' ambition responsable . Parce que nous regardons en face la dignité des personnes , que nous voulons faire réussir et protéger . Parce que nous regardons en face la force des territoires . Parce que nous avons conçu ce projet pour le pays ensemble . S' il y a cette vague qui monte , c' est parce que notre projet c' est vous qui l' avez porté . C' est parce que du premier jour , je suis allé vers vous , nous sommes allés vers les Françaises et les Français pour écouter , pour construire . Parce que chaque jour nos militants , nos adhérents , vous , vous construisez un projet . En vous réunissant , en réfléchissant . Parce que nous écoutons pour faire . Parce que le projet que nous sommes en train de construire , c' est un projet pour prendre le pouvoir , mais le prendre pour le rendre . Pour le rendre aux territoires , pour le rendre aux citoyens , pour leur permettre de faire . Alors je sais que le quotidien est parfois difficile . Je sais que la colère est parfois là . Je sais que , pour certains , parfois le désespoir peut s' installer . La responsabilité qui est la nôtre , celle des progressistes , c' est de porter pour eux cette espérance . A quelques kilomètres d' ici , en 1941 , après un jugement , un jeune Français a réussi à s' évader . Pas s' évader pour aller se protéger ou se cacher . Il s' est évadé pour rejoindre le Général de Gaulle , pour se battre pour son pays . Il s' est évadé , au risque de sa vie , pour que nous puissions ce soir être là , et encore proposer . Porter avec fierté ce qu' est la France . Il s' est évadé , et il a eu ces mots , quelque temps après , en parlant de son jugement , de son emprisonnement , et en disant : “ Ce n' est pas la justice de la France ; c' était celle de Hitler . Ne désespérez pas de la France . ” C' est ce qu' il a écrit . Ce jeune homme courageux , c' était Pierre Mendès France . Alors mes amis … ce soir … à Clermont , nous n' allons pas désespérer de la France . Parce que nous sommes là . Parce que vous êtes cette force debout et qui marche . Alors en vous renouvelant mes vœux , pour cette année qui commence et cette année de combat ensemble , oui , je vous dis et je vous redis , comme l' écrivait Alain , de penser printemps , de vouloir printemps , parce que le printemps sera à nous mes amis . Parce que le printemps nous allons le conquérir , pour nos idées , pour notre pays ! Parce que ce printemps sera progressiste ! Parce que ce printemps sera celui de notre conquête ! Vive Clermont ! Vive la République ! Et vive la France ! ",
    "Bonjour Lille ! Bonjour le Nord , bonjour les Hauts-de-France ! Vous m' avez l' air formidablement enthousiastes . Je vois tellement de visages amis que je voudrais embrasser . Vous êtes ce soir , ici , au Zénith de Lille , nous sommes ce soir cinq mille . Cinq mille avec nos histoires , nos parcours , nos différences et j' y tiens aussi parce que je les respecte , nos cheminements , nos imaginaires . Mais nous sommes cinq mille à vouloir porter un projet pour le pays malgré tout cela , pour la France , c' est cela qui nous rassemble ce soir ! Il y a un peu plus de neuf mois , nous avons fait un rêve , ensemble , nous n' étions peut-être pas cinq mille ce jour là , celui de lancer “ En Marche ! ” et de ne pas nous arrêter aux clivages , à tout ce que jusqu' ici bloquait le pays , aux vieilles habitudes , aux corporatismes . Et ce rêve , vous étiez des millions à le faire . Et mois après mois , cette vague a grandi , ce mouvement s' est renforcé , et vous êtes là ce soir parce que ce rêve n' était pas une folie , il était une ambition extrême pour le pays , pour notre pays , celle justement de travailler ensemble pour un projet qui nous unit plutôt que de rester divisé sur nos petites différences . Il y a quelques semaines , un reportage passait sur le mouvement dans le Nord , En Marche ! Et le journaliste demandait à quelques-uns , toutes celles et ceux qui étaient je crois autour de Christophe , s' ils avaient voté jusqu' ici à gauche , et la moitié de la salle a levé la main . Puis , le même journaliste a demandé &laquo; qui ici avait voté la dernière fois à droite ? &raquo; , et la moitié de la salle a levé la main . Ça ne veut pas dire que nous pensons tous la même chose sur tout mais ça veut dire que nous sommes tous et toutes las d' entendre les chicayas , d' être pris dans cette comédie humaine des petits jeux et que tous et toutes , ce qui nous rassemble c' est cette ambition de faire , de faire ensemble , de faire entrer en effet la France dans ce siècle nouveau et de donner à chacune et chacun sa place , c' est cela “ En Marche ! ” dans ses différences , dans sa différence , c' est notre projet . Alors je suis heureux aujourd'hui d' être dans nos Hauts-de-France , parce que moi aussi , vous savez , je suis un enfant des Hauts-de-France . Ce qu' on appelait naguère encore la Picardie . Cette terre , votre terre , notre terre , c' est une Terre-Monde . C' est un endroit où le monde s' est fait . C' est un endroit où on a fait les guerres , où on a fait le commerce au coeur de l' Europe , où on a produit . C' est ici que la révolution industrielle pour partie s' est jouée , que les mines se sont ouvertes , et on fait travailler tant et tant , d' hommes et d' enfants pendant longtemps , de femmes . C' est ici que l' acier s' est développé , que le textile a grandi . Ces terres , elles ont été au coeur du monde nouveau au XIXème siècle , et bien avant . Ces terres ont toujours été des terres d' ambition , des terres d' avenir . Ces terres des Hauts-de-France ce sont aussi mes amis des terres de combats , du combat social , celui mené dès le début et qui s' est noué ici , après Courrières où 1099 ouvriers sont morts . Des terres de batailles , où pied à pied , mois après mois , la justice sociale s' est créée dans notre pays et s' est propagée en Europe parce que des femmes et des hommes se sont battus pour construire des droits nouveaux pour que le monde soit plus juste . Des terres de combats politiques aussi , et je veux ici avoir une pensée pour Roger SALENGRO . Vous pouvez l' applaudir . Grand maire de Lille qui s' est toujours battu contre les extrêmes , l' extrême droite , les ligues . Il en est mort . Ces terres , elles ont toujours été dans le monde et elles ont toujours été dans le combat . Elles sont au coeur du défi français parce que oui notre pays aujourd'hui , il est au monde , il est dans le monde et il est face à des combats nouveaux que nous allons inventer ensemble . Des combats qui nous feront entrer dans le XXIème siècle nouveau . … Alors oui mes amis , ce que je suis venu vous dire ce soir , c' est que nous devons être à la hauteur de cette Histoire collective . Jamais , nous ne devons accepter celles et ceux qui bafouant cette Histoire voudraient nous faire croire que notre avenir se construit dans l' enfermement , dans le repli , dans l' oubli du monde . Jamais ! Jamais nous ne devons accepter le discours de celles et ceux qui voudraient nous expliquer qu' il faut accepter le monde tel qu' il vient , s' adapter aux injustices , aux incohérences , à une mondialisation parfois devenue folle . Jamais ! Nous sommes depuis deux jours parmi vous au coeur de ces Hauts-de-France et nous y avons vu les fractures profondes qui touchent notre pays parce qu' ici se joue comme un résumé du défi français . Il y a Lille , la triomphante , la conquérante , la métropole capitale de la culture . Du numérique , du commerce , de l' industrie du XXIème siècle . Lille n' a pas peur de la mondialisation , elle y vit . Elle est au coeur de l' Europe , elle voyage . Elle a un jeunesse qui étreint des continents et ce monde nouveau . Et je veux ici rendre hommage à Martine AUBRY qui n' est pas avec nous ce soir mais à laquelle je souhaite le meilleur rétablissement possible . Et je veux aussi rendre hommage à Pierre Mauroy qui a accompagné cette transformation . Ce sont , à côté de cette métropole de Lille , des terres qui se transforment , qui ont souffert de la désindustrialisation , qui ont à un moment été frappées , et qui se redressent , se réinventent avec courage . C' est Valenciennes , cher Laurent Degallaix , par le travail que tu conduis aujourd'hui et qui a été mené par Jean-Louis BORLOO il y a maintenant près de vingt ans . Ce sont des terres industrielles qui résistent comme à Arques , chère Caroline Saudemont , et nous l' avons vécu . Des terres qui se battent contre les injustices , qui innovent , qui inventent . Nous étions ce matin , cher Frédéric Marchand , à Hellemmes , et nous l' avons vu ensemble . Et puis , ce sont en même temps des terres qui ont été sacrifiées par la mondialisation . J' étais hier dans le bassin minier - vous pouvez l' applaudir - , à Lens , puis Noeux-les- Mines , puis Hénin-Beaumont . Des terres où ce que ressentent nos concitoyens , c' est d' avoir été , en quelque sorte , la chair à canon de la mondialisation . D' avoir , pendant tant et tant de décennies , travaillé dur pour produire , puis d' avoir été balayés lorsqu' on leur a soudain expliqué qu' on allait fermer la mine , qu' on allait arrêter la sidérurgie un peu plus loin . Alors , là , le chômage s' est installé , et n' est plus jamais parti . Alors , là , le désespoir s' est installé , la grande misère . Dans ces terres , que ça plaise ou non à certains , on vit beaucoup plus dur , et on vit moins longtemps , et c' est une injustice qu' il faut regarder en face , oui ! Parce que si nous en sommes arrivés là , c' est que , pendant trop longtemps , nous n' avons pas voulu regarder ce monde en train de changer . J' étais hier , avec beaucoup de commerçants , d' artisans , à Lens , et il y avait Gervais Martel , que vous connaissez bien . Je vois qu' il y a des supporters de Lens , n' ayez pas d' ennuis avec les supporters du LOSC . Il disait une chose très vraie - son père était mineur- , il disait : &laquo; mon père qui est mort avant que tout cela ne ferme me disait dès le début des années soixante-dix : la mine ça ne pourra pas durer &raquo; . Il le savait très bien . Mais nous n' avons pas voulu voir . Les dirigeants n' ont pas voulu voir . Ils n' ont pas voulu raconter le monde , tel qu' il était en train de se faire , en train de bouger , parce que , oui , le changement fait peur , parce que le changement menace des emplois , des habitudes , mais à ne pas vouloir voir , expliquer , préparer , eh bien tout s' est passé soudainement et a été d' un seul coup balayé . Alors ce que je ne veux pas aujourd'hui , pour mon pays , c' est refuser de voir , voir les risques parfois terrifiants , voir les changements qui font peur et qui tétanisent . Je veux qu' ensemble nous les nommions . Je veux qu' ensemble , lorsqu' on se rend dans ce bassin minier , on nomme les injustices , on nomme tout ce que nous n' avons pas suffisamment fait , on nomme en même temps les formidables batailles qui sont conduites sur le terrain par certains élus , par les associations , qu' on nomme la générosité des femmes et des hommes qui y vivent , qui aiment le travail , qui veulent du travail , qui ne demandent pas à être assistés , qui veulent aller de l' avant ! Et donc ce que je suis venu vous dire ce soir , c' est simple : nous sommes , toutes et tous ici , des enracinés , nous vivons en France , nous vivons sur ces terres , et nous partageons le même destin , et nous ne pouvons pas accepter qu' il y ait des terres qui réussissent formidablement bien et des terres qui soient condamnées à la stagnation ou au recul Parce que nous sommes des enracinés , et , en étant enracinés , nous partageons ce destin commun . Et l' enracinement , comme le disait formidablement Simone WEIL , ce sont des devoirs : le devoir de regarder , de nommer , de s' engager . Et ne vous trompez pas , que toutes celles et ceux qui réussissent , et que je veux aider , n' oublient jamais d' où ils viennent , où ils sont nés , où ils vivent , où iront grandir leurs enfants et leurs petits-enfants . La responsabilité de vous qui êtes dans la salle et qui réussissez , c' est d' être là , enracinés , et de vous engager pour les autres . La responsabilité de celles et ceux qui doutent , c' est de faire confiance , c' est de ne pas croire de trop nombreux discours qui voudraient vous faire croire que haïr celui qui réussit est la meilleure façon de se sortir du dénuement : ça n' est pas vrai ; qu' avoir peur de l' autre c' est au final se protéger : ça n' est pas vrai non plus . Nous avons tous et toutes une responsabilité commune : celle des enracinés . Alors , ce que je suis venu vous dire ce soir , c' est que le projet que nous allons porter , pour ne pas commettre les erreurs d' hier , c' est un projet lucide , c' est celui de l' émancipation . L' émancipation , c' est la liberté , encouragée , pour celles et ceux qui réussissent , qui veulent entreprendre , faire , oser , tenter , elle est essentielle . Et l' émancipation c' est aussi les opportunités redonnées à celles et ceux qui sont parfois bloqués , assignés à résidence , à qui la société ne propose plus rien , si ce n' est des aides , et à laquelle , en quelque sorte , on s' est habitué . L' émancipation c' est un projet commun , c' est notre responsabilité commune . L' émancipation c' est ce que nous allons ensemble porter pour toutes et tous dans notre pays ! Alors l' émancipation , mes amis , elle se construit d'abord en regardant le monde en face . Parce que nous sommes ce monde , comme je vous le disais . Nous n' existons pas comme une enclave . Notre destin est universel . Alors oui , ce monde est plein d' insécurités . Vous êtes ici aux portes de quartiers qui ont parfois fait trop parler d' eux , qui ont porté cette insécurité , qui ont nourri l' angoisse , en Belgique comme en France . Elle est là . Le mal est là , dans notre société , et il faut le regarder en face . Cette insécurité , elle est internationale , parce que c' est l' hydre de ce terrorisme nouveau , de ce projet de destruction de l' autre , de ce djihadisme totalitaire . Je ne parle pas de guerre pour ma part . La guerre suppose d' avoir un ennemi constitué et un ennemi face à soi . C' est une lutte , que nous conduisons , une lutte radicale , profonde , une bataille militaire sur certains théâtres d' opérations , mais une lutte politique , culturelle , en notre société même , et nous devons la regarder en face . Mais nous devons la regarder sans pour autant nous diviser . Le piège que nous tendent nos ennemis , c' est celui de la haine , de la fracture , du repli sur soi . Qu' attendent celles et ceux qui nous attaquent ? Que nous ayons peur , que nous nous repliions , que les quelques-uns qui tuent nos frères nous fassent nous diviser entre nous et conduisent à un malheur encore plus profond . La réponse , elle doit être claire , totale , extrême . D'abord la lutte , partout où nos intérêts se jouent , partout où la sécurité de nos concitoyens est à l' œuvre , nous devons là intervenir , mais intervenir avec intelligence , être conscients d' un ordre du monde qui est en train de profondément changer , où des puissances nouvelles émergent , avec lesquelles nous devons avoir un dialogue exigeant , intervenir partout , sur le plan diplomatique et éventuellement militaire , en construisant une diplomatie et un projet de paix . Notre sécurité , mes amis , elle ne sera pas assurée par une France qui attaquera partout , tout le temps , pas plus que notre sécurité n' a été assurée lorsque les Etats Unis d' Amérique ont décidé de réagir aux attentats de 2001 par une attaque immédiate qui a conduit à quoi ? À la déstabilisation profonde de toute une région , à la naissance de formes nouvelles d' organisations terroristes . Cela , nous ne devons pas le reproduire . La politique internationale que je veux conduire pour notre émancipation vraie et donc pour notre sécurité , c' est celle qui respectera l' équilibre , qui préservera l' indépendance française , qui assurera la stabilité des Etats et qui , partout , défendra nos valeurs et nos principes . Pour construire la paix , nous avons donc besoin de retrouver le fil de cette diplomatie d' indépendance française , celui qui s‘est construit dans l' histoire même de notre pays , qui fut porté par le Général de Gaulle , un grand lillois , puis par François Mitterrand . Cette politique , nous devons aussi la construire à l' aune de ce monde nouveau . Oui , ces risques sont absolus , le monde se recompose . À nos frontières , le terrorisme se prépare . À nos frontières , de nouvelles migrations arriveront . De Syrie aujourd'hui , d' Irak demain , de la Corne de l' Afrique . Des migrations politiques , climatiques , immanquablement , et cette diplomatie nouvelle nous devons la construire avec l' Europe , elle est essentielle . J' étais il y a quelques jours à Berlin , j' ai parlé aux Allemands , aux dirigeants , mais aussi à la jeunesse allemande , à l' université Humboldt , et je leur ai tenu le même discours – je vous dis ça parce que , bien souvent , l' habitude française , ou la chorégraphie classique , c' est de vous dire ici que nos problèmes sont européens , que nous allons renverser la table avec l' Allemagne , que nous allons être très très dur , que nous allons aller voir la chancelière et lui expliquer ce qu' elle devrait faire , puis ensuite c' est d' aller en Allemagne et de dire que la France sera le premier partenaire comme il l' a toujours été . Donc je ne vous propose pas de renverser la table , parce que nous y sommes , c' est la nôtre : l' Europe c' est nous , Bruxelles c' est nous , c' est nous qui l' avons voulu . Et nous avons besoin de l' Europe , parce que l' Europe nous rend plus grands , parce que l' Europe nous fait plus forts . Nous continuerons à avoir une diplomatie française , mais , d' évidence , nous serons plus forts au niveau européen . Regardez le monde : un pays-continent , la Chine , le bloc américain , des puissances régionales , qui ne partagent pas nos valeurs , mais avec lesquelles nous devons discuter , la Russie , l' Iran , la Turquie , l' Arabie Saoudite . Et regardez ce qui est train de se passer : l' Europe , qui a vécu dans sa chair , les attentats , fomentés pour partie en Syrie et en lien avec ce qui s' y passe , les migrations et les réfugiés qui ont bousculé l' Allemagne , venant du Proche et Moyen Orient . L' Europe vit au cœur de ces risques et l' Europe n' a même pas été à la table des discussions . Donc oui , nous devons avoir une diplomatie indépendante et exigeante , mais oui , nous devons aussi avoir une vraie Europe de la sécurité , de la défense . Et oui , si l' Europe n' avance pas assez vite , nous devons accepter de le faire à quelques-uns , c' est cela ce que j' ai dit à nos amis Allemands . Commençons par le faire à deux , déjà acceptons à deux d' avancer , de faire plus , de peser . Vous savez , quand j' entends certains dire : &laquo; la France aura vis-à-vis des Etats-Unis une politique très exigeante &raquo; , la main sur le cœur , pensant que nous sommes encore à l' époque du Général de Gaulle . Je me réfère parfois au Général de Gaulle qui est une grande figure de notre histoire , le père de nos institutions , et un homme qui a redonné à notre pays sa fierté et sa force . Mais le Général de Gaulle ne faisait pas souvent référence à Mac Mahon pour construire sa politique . Eh bien c' est à peu près aussi intempestif aujourd'hui de se réclamer d' une politique purement gaullienne . Le monde a changé , il n' est plus le même . J' en entends certains dire : &laquo; Pensez -vous , il a 39 ans . Qu' est -ce qui va se passer quand il sera face à Poutine et Trump ? &raquo; . Non , je prône la bienveillance . Je crois dans les vertus de la diplomatie et je pense qu' aussi longtemps que la diplomatie permet d' empêcher la guerre , elle est préférable . J' entends celles et ceux qui font mine de douter et j' ai des remarques de bon sens à leur dire . La première , c' est que beaucoup de celles et ceux qui disent ça ne seraient pas aujourd'hui invités à la table face à Poutine et Trump , qu' ils se rassurent parce que leur politique conduirait à cela et a parfois conduit à ce résultat . Qu' ils n' imaginent pas une seconde que si nous ne prenons pas nos responsabilités , si nous n' avons pas une politique exigeante , d' indépendance , de lucidité et que si nous n' avons pas une véritable exigence européenne , ils ne seront plus invités à la table face à monsieur . Poutine et face à monsieur Trump . Donc ce que je veux pour mon pays , c' est cette politique exigeante , d' indépendance , française , franco-allemande , européenne qui nous permettra de porter notre vision , notre sécurité , nos valeurs et qui nous permettra d' avancer dans le monde . Donc oui , tendre une main aux États-Unis et leur proposer de poursuivre cette Histoire que nous continuons ensemble depuis tant de siècles et avoir un dialogue exigeant avec la Russie parce que nous devons parler avec elle mais parler avec elle ça ne veut pas dire être fasciné par elle , ça ne veut dire oublier que nous ne partageons pas les mêmes valeurs . Eh oui , c' est cela protéger notre pays , regarder le monde tel qu' il change avec lucidité , défendre nos intérêts et en même temps le faire avec une exigence chevillée au corps . C' est pour cela que lorsque je suis allé en Allemagne , j' ai proposé à nos amis allemands d' avoir une vraie politique de sécurité collective aux frontières , d' avoir un Schengen renforcé . Ceux qui proposent de balayer Schengen sont des naïfs . Ils oublient d'ailleurs qui a été à l' origine du projet Schengen ? Pas ce que je qualifierai de progressiste mais M. Pasqua , avec d' autres , qui fut un ministre de l' intérieur respectable de la France mais qui n' était pas réputé pour son laxisme . Il l' a fait pour quoi ? Parce qu' il a vu avec ses homologues que notre sécurité était mieux assurée aux frontières d' un espace commun et partout à l' intérieur de cet espace , c' est cela Schengen . Et donc quand on nous promet , quand le front national , le parti front national , nous promet de rétablir des sécurités aux frontières , il vous ment . Il vous promet de refermer le pays , de changer la vie de toutes celles et ceux qui ici à Lille passent leur vie d' un côté et de l' autre de la frontière alors que ce qu' il nous faut , c' est un contrôle renforcé aux frontières communes , c' est une politique commune d' asile , gérer depuis des états tiers , depuis l' origine en évitant à des familles de prendre des risques absolus de traverser la Méditerranée pour ensuite attendre des mois en se demandant ce qu' elles deviendront , c' est d' avoir une politique de renseignement commune et renforcée entre nous , d' avoir une gestion des titres communs , d' avoir des services de renseignements qui échangent davantage d' informations , d' avoir aussi une politique , ensemble , vis à vis des états d' origine et des états tiers . Qu' est -ce que cela signifie ? Ça veut dire que lorsque nous refusons de donner des titres à certaines personnes qui demandent pour le statut de réfugié ou qui demande d' avoir des papiers lorsqu' ils dont des migrants économiques , eh bien nous devons pouvoir les reconduire dans leur pays d' origine , ce n' est pas une pratique injuste , c' est une pratique efficace , sinon plus rien n' a de sens . Il n' est pas acceptable qu' aujourd'hui nous laissions s' installer une forme de laxisme coupable où des femmes et des hommes qui n' ont pas obtenu ces titres , eh bien restent dans le pays et restent en s' installant dans la clandestinité , en s' installant eux-même dans la précarité et en conduisant une partie de notre population à considérer que plus rien n' a de sens , que le laxisme est là , que donc tout est possible et à confondre toutes les peurs et tous les sujets . Ce que j' ai proposé à nos partenaires européens , c' est donc d' avoir ensemble chaque année une conférence avec les états d' origine et les états de transite pour signer des accords effectifs de reconduite à la frontière , pour signer des accords qui conditionneront notre aide publique au développement à leur effectivité . C' est cela une politique d' exigence , de générosité et de responsabilité . Et quand j' entends qu' on peut nous proposer , pour la énième fois , des quotas mais c' est à supposer que lorsque les quotas ne sont pas respectés on ait la main pour les faire respecter , ce n' est pas le cas aujourd'hui ! Donc je ne propose pas une politique idéologique en matière d' immigration , je propose une politique efficace , claire , menée avec nos partenaires européens , c' est cela notre projet . Mais en même temps , nous nous battons pour notre sécurité , pour avoir cette politique européenne claire , conquérante parce que nous avons des valeurs différentes de certains d' autres . C' est aussi pour cela que lorsque je me suis rendu en Allemagne , j' ai reconnu le courage de ce pays , sa dignité , lorsqu' il a accueilli des réfugiés venant de Syrie et d' autres pays . Il les a accueilli parce que ce sont nos valeurs , parce que tout ce que je viens de vous dire n' aurait pas grand sens si au fond nous ne savions pas défendre nos valeurs . Parce qu' un homme , une femme , une famille qui a quitté son pays , qui a pris tous les risques , qui a traversé la Méditerranée ou la route des Balkans pour arriver en Europe , vivre libre , ce n' est pas quelqu'un qui a choisi d' arriver en France , ce n' est pas quelqu'un qui vient ici pour profiter de la France . Ce sont des femmes des hommes d'ailleurs qui veulent travailler , construire leur vie . Non , ils ont défendu des principes , la liberté . Ce sont des émancipés . Et parce que nous nous battons , avec lucidité , avec les principes que je viens de rappeler qui seront les nôtres . Parce que nous allons nous battre partout , avec clarté , avec détermination et vigilance . Eh bien oui , nous devons le faire en défendant nos principes et nos valeurs . CHURCHILL pendant la seconde guerre mondiale , lorsqu' il était premier ministre , a eu une très belle réplique . Son ministre du budget - qui n' est jamais un rôle facile - vient le voir et lui dit : &laquo; Monsieur le premier ministre , c' est de plus en plus coûteux , la guerre est absolument terrible pour le pays , il va falloir qu' on coupe des budgets . &raquo; . C' est souvent le cas d'ailleurs , on fait comme ça , le budget de la culture est un bon candidat . Et CHURCHILL lui répond &laquo; Mais si nous baissons le budget de la culture , alors pourquoi faisons nous la guerre ? &raquo; . Alors je vous le dit , nous menons une lutte contre le terrorisme , nous mènerons des batailles , nous avons des ennemis . Mais si nous menons cette bataille partout , à quoi bon , si nous même nous oublions nos valeurs ? A quoi bon se battre contre les terroristes , si nous divisons le pays ? Si nous prenons la déchéance de nationalité ? Ou si nous divisons le pays sur ses valeurs mêmes ? A quoi bon nous battre pour notre sécurité ? A quoi bon nous battre contre des ennemis qui ne respectent pas la liberté , nos valeurs , si en même temps nous mêmes nous ne sommes pas à la hauteur de ces dernières ? Si nous mêmes nous ne savons pas accueillir celles et ceux qui ont pris tous les risques , celles et ceux qui se sont montrés français en faisant cela . Nous nous battons pour notre sécurité , mais à la fin , nous nous battons parce que nous sommes ces émancipés , parce que nous voulons ces valeurs . Cette émancipation que nous voulons , que nous portons pour le pays , c' est aussi celle qui se construit par le travail . Oui , le travail . Ah nous avons beaucoup de débats aujourd'hui pour savoir si il faut bousculer le pays pour le faire entrer dans la mondialisation en aidant en quelque sorte celles et ceux qui sont en train de réussir à aller plus vite , ou si il faut acter de la défaite . Dire &laquo; Oui , à Lille on réussit très bien , à côté on réussira jamais &raquo; . Et donc distribuer ce que l' on ne sait plus produire , qui des revenus universels , qui des dotations améliorées , ou que sais -je … ? Le premier projet est un projet injuste , et je ne le partage pas . Parce que nous sommes les enracinés que l' évoquais . Parce que nous ne pouvons pas accepter que certaines et certains aient le privilège du travail et réussissent seuls . Le deuxième projet , il est inefficace . Parce qu' on ne peut pas promettre de redistribuer si on ne produit pas , si on ne réussit pas , si le pays n' est pas tiré . Alors oui ! Ce que je veux c' est un projet de justice efficace , de justice sociale au siècle contemporain . Le projet que nous portons , et que nous allons construire ensemble , il est basé sur le travail . Mais le travail tel que ce siècle nouveau nous permet de le construire , de le penser . C' est cela , l' émancipation contemporaine . Parce que je veux être le candidat du travail , je veux qu' on entreprenne , qu' on réussisse , qu' on prenne des risques plus facilement . Je l' ai dit , je veux simplifier les normes , pour toutes celles et ceux qui créent , qui entreprennent et qui font . Avoir un droit du travail plus simple , non pas en promettant de bouger toutes les lois , non . En actant ensemble une chose toute simple , c' est que la loi ne peut ni ne doit être la même quand on a une multinationale à Lille , à Paris , ou ailleurs , lorsqu' on est une start-up , lorsqu' on est une TPE . Ca ne peut pas être la même chose lorsqu' on est dans le BTP , lorsqu' on est dans la sidérurgie , lorsqu' on est dans la boulangerie . C' est évident , mais c' est ainsi . Et donc , ce que je veux , c' est beaucoup plus simple . C' est que la loi définisse des principes , et qu' ensuite au niveau de la branche et de l' entreprise on négocie la réalité sur le terrain , ce qui fait la vie de l' entreprise . C' est cela notre projet . Et donc pour ce faire , nous aurons des principes généraux et tout sera renvoyé à la négociations dans l' entreprise , dans la branche , pour que la réalité des entrepreneurs soit plus simple , plus concrète , au réel du quotidien . Parce que cette République contractuelle à laquelle je crois , c' est aussi celle qui est plus juste , qui permettra de mieux protéger , qui permettra de faire réussir . Parce que je crois dans l' initiative économique , je veux supprimer le RSI . Les allègements de charge seront maintenus , un droit d' option vers le régime général sera permis , mais la gestion du RSI sera adossée au régime général . Parce que la vie des entrepreneurs , des indépendants , des professions libérales , doit être facilitée . Parce que je suis le candidat du travail , je transformerai le crédit d' impôt compétitivité-emploi en allègement de charges stable , visible , pérenne . Ce seront donc six points de charges patronales en moins , pour toutes les entreprises , quelle que soit leur taille , quelle que soit leur statut . Qu' elles soient indépendantes , que ce soient des individuelles , que ce soient des grandes entreprises , que ce soient , cher Christophe , des entreprises de l' économie sociale et solidaire , c' est de la justice . Parce que ce coût du travail dont on a beaucoup parlé est aujourd'hui inefficace , et bien nous devons encore l' alléger pour les emplois les moins qualifiés et pour les plus jeunes . C' est pour cela que j' ai décidé qu' au niveau du SMIC , ce seront 10 points de charges qui seront supprimés . Parce qu' aujourd'hui nous ne finançons nos grands risques , la couverture sociale comme on l' appelle , que sur le travail . Et bien la transformation que nous vivons va nous conduire à financer davantage par l' impôt parce que c' est plus juste , parce que c' est plus cohérent , et donc à diminuer le coût du travail pour tout le monde et se faisant à faciliter les embauches . De la même façon , parce que je veux que le travail partout paye mieux , je souhaite alléger les cotisations salariales , celles qui sont payées par chacune et chacun et supprimer un peu plus de 3 points pour les remplacer par une augmentation plus limitée de la CSG . Qu' est -ce que cela veut dire ? Que toutes celles et ceux qui travaillent , quel que soit leur revenu , quel que soit leur statut , et bien demain verront leur pouvoir d' achat augmenter , c' est cela ce que je veux . L' émancipation par le travail , elle doit aussi construire les rêves et les équilibres de ce monde nouveau dans lequel nous entrons . Et donc oui je souhaite que chacune et chacun soit protégé face aux risques du chômage différemment . Qu' est -ce que cela veut dire ? Aujourd'hui nous sommes face à des injustices . Lorsque vous êtes salarié vous êtes protégé , vous avez des allocations chômage . Lorsque vous êtes un indépendant , un commerçant , un entrepreneur , et que demain vous échouez parce que chacune et chacun échoue , et il y aura d' autres échec dans la société , et l' échec fait partie d' une société du risque et bien vous n' êtes pas aujourd'hui protégé . Il y a aujourd'hui en France cette injustice fondamentale . Ce que je veux donc c' est que le chômage devienne une protection universelle pour toutes et tous . Et que face à cette protection , il y ait des droits et des devoirs , et donc il n' y aura plus un droit au chômage parce qu' on est indemnisé , il y aura un droit pour tous avec des devoirs , celui de se former ou d' accepter l' offre d' emploi décente qui est proposée , parce que je ne veux plus entendre des femmes et des hommes qui expliquent : &laquo; J' ai encore le droit de rester un peu au chômage . &raquo; , ou bien qui disent : &laquo; On ne me propose rien . &raquo; . Parce que face à cela , je veux que nous ayons une vraie politique de formation continue et de qualification , un vrai service public de la qualification et de la formation tout au long de la vie , c' est cela l' émancipation véritable . Concrètement qu' est ce que cela veut dire ? Aujourd'hui notre formation continue , plus de 35 milliards d' Euros , elle forme bien celles et ceux qui ont déjà un emploi , elle forme bien dans le même secteur mais elle ne forme pas les intérimaires , elle ne forme pas les chômeurs de longue durée , elle ne prépare pas au changement . Et donc oui , nous avons un système généreux , alors oui , on nous dit tous les jours qu' il ne faut rien changer à ce système , c' est notre modèle social . Mais il a été pensé , construit par un formidable consensus national après 1945 . Mais notre monde se transforme , regardons -le . Qu' est ce qui est en train de se passer ? Qu' est ce qui va se passer ? Un bouleversement aussi radical même bien davantage que que ce que nous avons vécu dans ce bassin minier où j' étais hier . Qu' est ce qui est en train de se passer ? des secteurs entiers d' activité vont être balayés , bousculés par le numérique , par la transition énergétique et environnementale et donc oui , des milliers , des millions d' emplois seront détruits . Mais des milliers , des millions d' autres seront créés dans d' autres secteurs qui sont autant d' opportunités , dans le numérique , dans ce qu' on appelle la silver économie parce qu' il faudra créer des secteurs d' activité pour aider les plus âgés parce que le vieillissement est là parce que la dépendance est là , et nous devons nous préoccuper des plus anciens et donc des secteurs d' activité nouveaux vont se développer . Des opportunités dans l' énergie , dans l' environnement vont se produire . Aujourd'hui , celles et ceux qui sont en train de perdre leur emplois , qui demain les perdront à cause de cette transition , rien n' est fait pour eux . Nous devons donc avoir un vrai service public moderne , contemporain , nouveau , qui va aider à cette transition , qui fera que dans 30 ans nous n' aurons pas l' équivalent du bassin minier aujourd'hui c' est à dire des femmes et des hommes à qui on a proposé le chômage ou les minima sociaux . Alors que j' y étais hier , qu' est -ce qu' ils demandent ? Du travail , du travail et que toutes celles et ceux qui parlent d' assistanat aillent entendrent ceux qui depuis des décennies vivent là-bas , ils veulent du travail , rien d' autre . Et donc ce service public de la formation , de la qualification , ce ne seront pas des nouveaux postes de fonctionnaires , c' est simplement la réorientation de notre système actuel . C' est à dire que nous devons , à chaque chômeur qui n' a plus de perspectives , proposer , offrir une formation de 6 mois , d' un an , deux ans pour pouvoir se requalifier vers ces nouveaux secteurs d' activité , pour pouvoir trouver sa place dans ce siècle qui commence , pour pouvoir trouver sa part dans ce monde qui se recompose . Cher Jean- Paul Delevoye , c' est ce que tu expliques conférence après conférence , ce que tu expliquais déjà au CESE . Ce monde nouveau qui advient , notre responsabilité c' est de le regarder en face avec ses menaces et ses opportunités et donc de préparer à tous les âges de la vie nos concitoyens . Et donc ce service public de la qualification et de la formation , il passera par : plus de de transparence , de l' évaluation des formations qui sont offertes ; celles qui ne sont pas bonnes et qui n' ont pas de résultats seront supprimées , nous ferons donc des économies . Mais celles qui sont adaptées nous les développerons , les universités prendront leur part mais nous proposerons à chacun de pouvoir se reconvertir . Et après cette formation , chacune et chacun devra prendre l' offre d' emploi qui lui est offerte si elle correspond à ses opportunités . C' est cela mes amis , l' émancipation par le travail , l' émancipation qui nous permettra de rentrer dans ce siècle . L' émancipation , enfin , c' est l' école : rien n' existe , rien ne se fait si l' école ne fonctionne plus . Comme je vous le disais , je viens d' Amiens , rien ne me prédestinait , étant né à Amiens , dans la famille qui était la mienne , d' être là ce soir devant vous , rien . Je suis là parce que j' ai eu la chance d' avoir une famille qui m' a poussé à travailler , à étudier , d' avoir une grand-mère institutrice puis directrice d' école qui avait grandi , s' était sortie de sa condition par l' école , qui n' avait vécu que pour l' école et à chaque instant j' ai pensé à cela : au fait que , à deux générations près , rien ne me prédestinait à être devant vous . Alors il y a beaucoup de chance , il y a du travail , il y a des rencontres , il y a des gens qui vous aiment . Mais le socle de l' histoire de ma famille , le socle de ce qui m' a conduit ce soir devant vous , le socle de toute émancipation de notre société : c' est l' école . Or notre école aujourd'hui , notre école elle est bloquée depuis dix ans . Ses performance sont de moins en moins bonnes , les tests PISA comme on les appelle , se dégradent . Les derniers test anglo-saxons montrent que nous sommes pour les mathématiques , en CM1 , les derniers . C' est non seulement un échec sur le plan pédagogique , académique , mais c' est une injustice parce qu' une école qui ne marche plus , ce sont des injustices qu' on reproduit , c' est une société qu' on ne transforme plus : c' est l' injustice au carré . Dans un monde qui se transforme avec des opportunités formidables , qu' allez -vous expliquer à des enfants qu' on n' a pas su former ? Et oui , il faut le regarder en face , dans les endroits ou nous avons concentrés les difficultés économiques , sociales , les problèmes d' intégration : c' est là que l' école est en crise . Nous avons aujourd'hui vingt pour cent de nos enfants qui en CM2 ne savent pas bien lire , écrire , compter . Ces enfants là , ce sont les décrocheurs de demain , ce sont pour partie certains des délinquants ou des chômeurs de longue durée d' après-demain , ces décrocheurs de la petite enfance , je peux quasiment vous en donner les noms , selon leur famille , selon leur quartier , selon leur origine . Si nous croyons dans l' émancipation , si nous croyons dans la promesse républicaine , alors oui nous devons nous indigner face à cela , c' est cela notre combat . Ce combat commence à la maternelle et j' expliquerai dans les prochaines semaines mon projet , pour la maternelle qui est fondamentale en particulier dans les endroits les plus difficiles . Ce combat , c' est ensuite pour l' école , le collège comme le lycée , de refuser le défaitisme dans lequel nous nous sommes installés et de remettre des heures d' enseignants devant les élèves . Baisser le nombre d' heures d' enseignement , ça n' est pas une réponse , c' est une défaite . C' est une injustice , en particulier dans les quartiers les plus difficiles , parce que le coeur de ces difficultés que j' évoquais se concentre dès l' école primaire , dans les quartiers les plus difficiles , les plus pauvres , les plus défavorisés . Ce que je ferai , c' est que dans tous ces quartiers , dans ces six mille classes de CP et de CE1 , je diviserai par deux le nombre d' élèves par classe . Parce que dans ces classes où la difficulté s' est concentrée , c' est bien souvent non pas 20 % mais parfois 80 , 90 % de non-apprentissage parce qu' il faut parfois de l' enseignement plus individuel , il faut d' autres méthodes . C' est un investissement véritable mais pour l' avenir , c' est un investissement pour éviter d' autres coûts : celui de la délinquance , celui de l' échec et celui du chômage . Ces douze milles postes ils seront pour partie redéployés sur les créations en cours et pour partie financés sur les économies que je ferai dans le reste du système scolaire et en particulier sur l' épreuve du baccalauréat . Ce que je veux , c' est aussi mieux rémunérer ces enseignants , ces écoles . Dans ces écoles qui sont les vraies terres de bataille de la République et de son émancipation , que devons -nous faire ? Nous devons livrer bataille et donc nous devons donner de l' autonomie pédagogique , de l' autonomie de recrutement , de la possibilité de choisir les enseignants qui y travaillent , de les recruter sur l' adhésion à un projet , de les rémunérer bien davantage parce qu' ils portent un projet et de leur permettre d' avoir beaucoup plus de liberté . Ce que nous avons vu ensemble , cher Frédéric Marchand , ce matin , à Hellemmes , dans cette école maternelle , c' est un investissement , en particulier de la mairie , de l' école , de l' ensemble des enseignants pour justement aller contre le destin . Mais dans un vrai projet pédagogique , une vraie autonomie c' est celle la que je veux partout dans l' école de la République , plus de liberté , plus de moyens mais en même temps , plus de responsabilité . Avec la liberté il y a l' évaluation , la responsabilité , avec l' autonomie supplémentaire , il doit y avoir une évolution partout pour tout le monde , une vraie transparence . On doit voir les endroits où on n' arrive à avoir des résultats et ceux où on arrive à contrecarrer le destin . C' est celle-là , l' école que je veux , c' est ce projet : plus d' autonomie , plus de reconnaissance pour les enseignants et en même temps plus de responsabilité et d' évaluation . Je reviendrai dans les prochains jours et prochaines semaines sur le projet que je veux que nous portions plus en détails sur le collège , sur le lycée . Mais deux mots ici encore : le premier pour vous dire que l' un des défis qui est le nôtre , c' est lorsque le lycée s' achève et que les enfants doivent prendre la mer , l' entrée dans ce monde nouveau , dans l' enseignement supérieur , nous avons une transformation profonde à conduire : celle de l' orientation . Celle qui consiste à accompagner tous les enfants et en particulier les boursiers , celles et ceux qui viennent des quartiers les plus défavorisés à accéder à l' information , à accéder aux bonnes formations . J' étais tout à l' heure avec des enseignants , des associations formidables , cher Pierre MATHIOT ce que tu conduis depuis tant d' années ici avec ce formidable projet des programmes d' études intégrées qui a été chercher des jeunes boursiers à Lille , puis dans la région puis plus loin pour contrecarrer la fatalité . Pour permettre à des jeunes qui sont boursiers , dont les parents n' avaient pas fait d' études d' accéder au supérieur . Parce que qu' est -ce qu' il se passe aujourd'hui ? C' est que ce sont bien souvent les mêmes qui accèdent au supérieur parce que pourquoi vous avez , dans certaines filières , plus des trois-quart des élèves qui arrêtent à l' issue de la première année d' université ? C' est parce que personne ne leur a expliqué , personne ne les a accompagné , ne les a informé , ne leur a dit ce qui était bon pour eux , ce qui correspondait à leurs goûts . Et la plus belle phrase c' est celle qu' a prononcé tout à l' heure Omar , lorsqu' il a dit : “ Avant de rentrer dans le programme , je pensais que tout ça c' était pas fait pour moi ” . Alors , oui , l' émancipation , c' est quand justement par cette transformation on fait que les plus défavorisés arrêtent de penser que tout cela n' est pas fait pour eux . Alors j' y reviendrai . Il y a aussi le travail formidable fait dans les lycées professionnels , le travail essentiel fait dans l' apprentissage pour nos jeunes , cher Patrick , j' y reviendrai dans les prochains jours et les prochaines semaines . Je ne peux pas tout vous dire ce soir vous le savez bien . On me reproche parfois même d' être trop long , mais je suis trop long parce que j' explique , parce que je décris ce monde , parce que j' essaie de porter ce projet commun , ce sens commun qui est le nôtre , de réinscrire … Alors oui , je reviendrai vous parler de l' apprentissage , de l' environnement , de la culture qui soit aussi les armes de notre émancipation collective . Mais je finirai par vous dire une chose simple : cette émancipation qui est notre projet , qui est d' aider celles et ceux qui réussissent à réussir encore mieux plus vite et plus fort , d' aider chacune et chacun à trouver sa place parce que nous vivons ensemble . Ce projet qui réconcilie l' efficacité , la liberté et la justice sociale . Il n' est possible que par votre engagement , que par l' engagement de toutes et tous . Il n' est possible que par le travail des associations , des élus locaux , du terrain , de ce qui est fait . Cette région elle se bat . J' ai parlé de beaucoup d' élus , toi Dominique , j' étais il y a quelques mois encore à Wattrelos à tes côtés , de ce qui est fait par le président de la région Xavier Bertrand à qui je veux ici aussi rendre hommage , qui se bat et qui s' est battu face au Front National . Que Jean-Paul Huchon , qui est toujours à mes côtés a conduit lorsqu' il menait les destinées de cette belle région d' Île-de-France , et il est là parmi vous , que tu as mené Jacky à la tête de CCI . Ce combat , c' est celui des élus , celui des associations , c' est votre engagement , c' est comme cela que nous le tiendrons . Alors vous savez , j' entends déjà toutes celles et ceux qui vous diront que c' est un rêve , que nous sommes des rêveurs : “ à force de croire obstinément à ses rêves , on finit par imposer sa réalité . ” Alors oui , mes amis cette phrase que vous venez d' applaudir , elle n' est pas de moi , elle est de Pierre Mauroy . Alors oui , je vais vous le dire ce soir : que toutes celles et ceux qui pensent que nous sommes des rêveurs soient rassurés , nous le sommes ! Mais , que toutes celles et ceux qui pensent que nous sommes des rêveurs soient terriblement inquiets parce que nous sommes des obstinés ! Donc ce que je vous demande ce soir , avec moi , c' est pour les cent jours qui nous restent de porter ce projet d' émancipation partout , de rassembler toutes celles et ceux qui sont aujourd'hui à droite et qui vous disent vouloir y rester mais ne pas s' y retrouver ; toutes celles et ceux qui à gauche doutent et ne s' y retrouvent plus ; toutes celles et ceux qui ont peur , ne cédez rien aujourd'hui aux discours de haine , ne cédez rien aux discours de repli , venez retrouver le mouvement de l' optimisme volontaire , celui de l' espérance , le nôtre , le vôtre ! J' ai besoin de votre force , de votre fougue ! Vive la République , vive la France ! Merci à vous ! Merci à vous ! Je compte sur vous ! ",
    "Merci à vous ! Merci à Yann et Maud ! Merci beaucoup ! Bonjour Quimper ! Bonjour le Finistère ! L' Ille-et-Vilaine ! Les Côtes d' Armor ! Le Morbihan ! La Loire-Atlantique ! Bonjour la Bretagne . Nous sommes plus de 2200 ce soir , et , malheureusement , plusieurs centaines de personnes sont restées dehors . J' ai passé un petit moment avec elles , d' où mon retard . Alors , je vous rassure , notre ambition n' est pas , ce soir , de concurrencer le festival de Lorient , ni les Vieilles Charrues , ni celui de Cornouaille ou du bout du monde . Mais ce soir , je suis venu malgré tout vous parler d' ambition . Notre ambition . Notre ambition commune . Celle qui nous porte depuis le début . Alors je suis heureux , ce soir , de me retrouver à Quimper avec nombre de mes amis bretons . Je me souviens de la visite faite il y a un peu plus d' un an , j' étais alors ministre , à Ergué-Gabéric , à Plévin , cher Richard , chez toi , le calvaire , mais aussi de nos rencontres économiques . Et un an plus tard , aujourd'hui , je suis avec vous , porteur d' autres responsabilités , que je mesure chaque jour un peu plus . Celle de porter ce mouvement collectif . Cette volonté d' aller de l' avant . De réveiller les uns et les autres . Et oui , je suis heureux ce soir d' être ici , dans ces terres d' ambition et d' aventuriers . D' ambition d'abord pour les Bretons , eux-mêmes , bien sûr . D'abord une ambition pour inventer l' avenir . C' est un territoire d' aventuriers , de découvreurs , d' explorateurs , de héros sportifs contemporains . Et un territoire qui ne cesse d' innover . Je l' ai vu à plusieurs reprises . À Rennes , là aussi comme ministre puis , plus récemment , à Lannion , il y a plusieurs mois . Et nous l' avons vu toute la journée ensemble , quels qu' en soient les secteurs : de l' agriculture aux cars , en passant par bien d' autres . L' innovation est au coeur de cette terre . Cette volonté d' inventer l' avenir , de regarder devant . Votre ambition est aussi vouée à enrichir chaque jour ce que vous êtes . A préserver votre singularité , vos atouts , votre identité . Parce qu' un Breton , au fond , c' est un enraciné . Je pense aux langues de Bretagne , qui sont une formidable richesse . On n' ose plus dire les langues chez certains . Ce sont des langues , du gallo au breton - je viens d' une terre qui porte avec fierté le picard , et le ch'ti - mais cette langue , ces langues qui sont les vôtres , c' est votre richesse et elles justifient pleinement que leur enseignement , leur transmission , soient accompagnés et encouragés . Et nous devons toutes et tous nous y engager . Et votre force surtout , c' est que l' ambition que vous portez , elle dépasse largement Quimper , elle dépasse largement le Finistère , elle dépasse largement la Bretagne . C' est une ambition pour le pays entier que vous portez . Et ici , le long de l' extrême frontière de notre belle France , continue de battre avec puissance le coeur vibrant de notre pays . Cette ambition bretonne , c' est celle qui vise d'abord à protéger le pays tout entier . Car la présence de nos forces de dissuasion nucléaire en presqu'île de Crozon , mais aussi la présence , à travers la région , de presque tous nos corps d' armée , c' est le témoignage de cette vocation du territoire breton , de cette ambition bretonne . Je veux ici saluer un ami , naguère collègue , grand ministre de la Défense , grand Président de la Région Bretagne , Jean-Yves Le Drian , qui porte aujourd'hui cette tâche . Et il a encore récemment , à travers un discours fort sur la cyberdéfense , dessiné la vocation nouvelle de ce territoire et de nos ambitions communes . Et en disant cela aussi , comment ne pas penser à l' île de Sein ? Comment ne pas penser à ce tréfond de notre histoire , au fait que cette petite île , quand on la regarde sur la carte , paraisse si innocente ? Pour certains qui ne connaissent pas notre histoire , si insignifiante . Certains qui ont oublié , qu' au moment où le pays doutait de lui-même , se pensait défait , cent vingt-huit pêcheurs ont décidé de ne rien abandonner . Cent vingt-huit pêcheurs ont décidé de résister . Comment ne pas se souvenir , quand on parle de cette vocation bretonne , de porter la fierté française , de cette ambition française d' ici , comment ne pas se souvenir de cette scène , où le Général De Gaulle , passant en revue les premiers résistants , cette poignée de femmes et d' hommes &laquo; un peu &laquo; , disaient alors quelques-uns qui étaient auprès de lui ? Voyant les uns et les autres se succéder , et tant de Seinans décliner leur identité et le lieu d' origine , s' exclamer &laquo; Mais l' Île de Sein , c' est donc le quart de la France ! &laquo; Comment ne pas se souvenir qu' en 1940 , une majorité de députés venant de cette terre , a voté contre les pleins pouvoirs à Pétain ? C' est aussi cela . Alors cette ambition qui vous dépasse , c' est la clé de votre identité . Une identité ouverte que je voudrais pour la France entière , car vous pouvez tout à la fois être profondément bretons , puissamment français et sincèrement européens . C' est cela votre identité ! Sans que jamais l' un de ces termes soit exclusif l' un de l' autre . Sans que jamais ces sentiments ne s' opposent . Sans que jamais ils ne viennent s' annuler ou s' opposer les uns aux autres . Voilà pourquoi je suis venu partager votre ambition , notre ambition . Parce que je crois que l' ambition finistérienne , l' ambition bretonne , elle est d'abord et avant tout une ambition française . Une ambition pour faire réussir notre pays , pour défendre ses valeurs , pour préserver ce qu' il est avant tout aujourd'hui , au coeur de notre défi . Alors , au fond , pourquoi vous parler d' ambition aujourd'hui ? Simplement parce que notre pays en a terriblement besoin . L' ambition n' est pas un vilain mot , quelque chose dont on devrait avoir honte . L' ambition , c' est ce qui permet de tirer le meilleur de chacune et chacun , c' est ce qui permet de retrouver sa fierté , c' est ce qui permet à celui qui est armé pour la vie , prêt à prendre tous les risques , de réussir encore davantage , et c' est ce qui permet aussi à celui qui n' a rien de créer sa propre vie , de construire son destin , de trouver sa place . L' ambition , c' est aussi ce qui a permis à notre mouvement d' éclore . Notre ambition , ce soir , mes amis , l' ambition d' En Marche ! , l' ambition de celles et ceux qui veulent faire avancer le pays . Il y a une dizaine de mois nous partions de rien . Nous sommes parvenus à réunir plus de 140 000 personnes sous la même bannière progressiste , et elle est là , ce soir , à Quimper ! Et le succès de notre mouvement , en Bretagne , en est d'ailleurs la preuve . Je veux saluer en particulier les maires et les élus bretons qui m' ont d' ores et déjà apporté leur parrainage et qui sont ici présents . Mais aussi Olivier Alain , Vice-Président de la région Bretagne à l' agriculture , qui m' a accompagné encore une fois aujourd'hui . Pierre Karleskind que vous venez de voir et qui a introduit ce meeting , chargé de la mer et de la pêche à la région . Mes deux amis : Corine Erhel , Jean-Pierre Le Roch , au sein d' En Marche ! qui portent aussi les couleurs de la Bretagne . Et bien sûr Richard Ferrand . Je dois dire que , quand je me suis demandé à qui confier la responsabilité du Secrétariat Général d' En Marche ! , l' idée de m' en remettre à un Breton , fut -il d' adoption , n' était pas le moindre des critères . C' est un déterminé , comme mes trois amis que je viens d' évoquer , un déterminé qui sait prendre les risques , la mer , et aller au large : c' était donc à lui que revenait cette mission . Le jour où j' ai quitté mon ministère , j' ai cité non pas un Breton , mais un méridional , Pagnol , et qui avait une jolie formule pour cette terre qui aime la mer , les risques , comme vous . Il disait : &laquo; Si on veut prendre la mer sans prendre , avec , les risques , mieux vaut acheter une île qu' un bateau &raquo; . C' est une terre de mer , mais aussi une terre de marins . Nous avons pris la mer , le bateau est chaque jour plus grand , et nous irons au bout de cette Odyssée ! Et l' ambition qui est la nôtre , elle correspond parfaitement à ce qui est l' histoire même de Quimper . &laquo; Kemper &raquo; , ça veut dire &laquo; confluent &raquo; , en breton . Eh oui , nous sommes une terre de confluences , eh bien nous , nous sommes un mouvement de confluences . Alors j' en entends beaucoup en ce moment qui se posent des questions . Ils disent : &laquo; Il y a des ralliements chaque jour &raquo; , &laquo; C' est une drôle d' affaire &raquo; , &laquo; Est -ce que ça devient le Parti Socialiste ? &raquo; . Je vous rassure , hier , les mêmes socialistes demandaient : &laquo; Est -ce que ce n' est pas en train de devenir la droite ? &raquo; . Quand le bébé commence à prendre forme , il trouve beaucoup de pères putatifs . C' est une confluence : il y a des socialistes , des radicaux , des écologistes , des Modem , des UDI , des républicains , des femmes et des hommes de la société civile , vibrante , qui s' étaient détournés de la vie politique . C' est tout cela , En Marche ! . Cette confluence qui a été portée , je ne l' oublierai jamais , par des engagés et des courageux dès la première heure . Et donc oui : notre quille , c' est cette confluence . Elle ne gîtera ni d' un côté , ni de l' autre . Elle aura toujours une boussole : l' intérêt général , un projet pour le pays , et pas pour une partie ou une clientèle , c' est comme ça que nous irons au bout . Parce que ce mouvement , parce que le projet que je porte , c' est une ambition , c' est notre ambition pour le progrès en France . Alors oui , nous aurons dans les semaines à venir d' autres ralliements , je l' espère , de plus en plus . Mais les ralliements que nous devons chercher mes amis , ce sont les ralliements de chacune et chacun des Français . Ce que je guette chaque matin , ce n' est pas le ralliement d' un tel ou d' un tel qui fascine le microcosme parisien et journalistique , parce que celui-là à la fin , il ne comptera pas davantage que le travail de centimier que nous faisons depuis le premier jour , que cette lutte sur le terrain que chacune et chacun d' entre vous a conduite , parce qu' à la fin du jour , à la fin de cette bataille , c' est une majorité de Françaises et de Français que nous aurons ralliés pour la victoire ! Alors l' ambition d' En Marche ! , c' est de répondre aux vrais problèmes et de parler des vrais sujets . Notre ambition , c' est , dans un monde de risques , de transformations profondes , d' apporter les sécurités individuelles , les nouvelles protections sans lesquelles les uns et les autres hésitent à prendre le large , restent parfois à l' abri , doutent , ou ont peur du changement . Le projet que nous portons , c' est un projet progressiste , parce qu' il vise à inventer ces sécurités nouvelles , à reconstruire non pas un projet d' égalité pour tous et toutes , parce que c' est un projet onirique , ce serait une promesse mensongère . Mais un projet pour que l' égalité des possibles se fasse en actes . Nous sommes toutes et tous différents . Il y a des gens plus doués que d' autres , il y en a qui sont plus travailleurs que d' autres , il y en a qui auront plus de chance que d' autres . Mais ce qui est insupportable , c' est que , à mérite équivalent , parce qu' on est né dans une famille , dans un quartier , sur un territoire plus retranché que les autres , on ne puisse pas exprimer ses talents de la même façon . C' est cela , le projet qui est le nôtre , l' ambition qui est la nôtre , c' est cette ambition de justice sociale , cette ambition première que je veux pour le pays . Alors elle se traduit comment , concrètement ? Par une ambition pour la santé . Je l' ai déjà dit et redit , et je le répéterai chaque jour de cette campagne . Nous serons demain dans un centre important de la région , qui rééduque , répare des vies . J' ai passé beaucoup de temps avec nos concitoyens qui soignent et qui s' occupent des uns et des autres . Il n' y a pas de risque qui peut être pris , il n' y a pas de société qui peut se transformer si on ne protège et ne prévient face aux aléas de la vie . Pour la santé , je veux une meilleure prévention . Pour ce faire , je me suis engagé à ce que dans toutes les études , de médecine , d' infirmiers et d' infirmières , de kinésithérapie , de dentiste , il y ait au moins trois mois dédiés à la prévention . Les jeunes étudiants iront dans les écoles , dans les maisons de retraite , dans les endroits les plus défavorisés pour faire de la prévention : c' est l' économie la plus efficace , c' est ce qui est essentiel pour notre société . Parce que ce projet pour la santé est un projet d' ambition , je veux aussi que nous puissions couvrir 100 % des dépenses liées aux lunettes , aux prothèses dentaires et aux prothèses auditives . Pourquoi ? Parce que c' est prévenir des risques plus graves , parce que c' est , là aussi , une vraie justice économique et sociale . Parce qu' un jeune qui ne peut pas lire au tableau , sous prétexte qu' il n' est pas corrigé , c' est un jeune qui n' apprend pas . Parce qu' un chômeur qui ne peut pas soigner ses dents , sous prétexte que c' est mal remboursé , c' est un demandeur d' emploi qui n' est pas embauché . Parce qu' une personne âgée qui n' entend pas et qui ne peut pas s' équiper , c' est une personne qui s' enferme dans la dépendance plus rapidement , dans la sénilité et l' isolement . Ce projet pour la santé , c' est celui par lequel aussi , je veux porter une réforme profonde de l' accès aux soins , avec une généralisation des maisons de santé pluridisciplinaires , des maisons de suite pour les soins , afin de sortir plus vite de l' hôpital et une réforme juste , en profondeur , de l' hôpital par une tarification plus adaptée , par une réorganisation responsable , qui permette de combler les déserts médicaux , et en même temps , de poursuivre les nécessaires économies , sans réduire en rien l' accès aux soins . Cette ambition pour la santé , j' y suis revenu il y a quelques semaines chez nos amis à Nevers , beaucoup plus en détail . C' est une ambition de justice , raisonnable puisque nous ferons des économies , mais ambitieuse parce qu' on ne construira pas l' entrée dans le XXIe siècle en déremboursant des soins utiles , en supprimant des fonctionnaires de la fonction publique hospitalière . Parce que le projet , respectable , qui est porté par les conservateurs de notre pays , c' est un projet qui est à la fois : - inefficace . ( Les déremboursements de petits soins , comme on nous l' a expliqué - je dois vous confesser ici que je n' ai toujours pas compris ce qu' était un petit soin ; on a parfois croisé des petits rhumes qui pouvaient devenir des rhumes graves - en tout cas , ces déremboursements , ils conduiront à des risques plus graves , ils conduiront à ne pas traiter certaines petites pathologies , précipitant vers des maladies plus graves , et , ce faisant , plus coûteuses . Le coût , à la fin , je vous le parie , sera plus important ) . Mais c' est aussi : - un projet injuste . Si les petits soins sont déremboursés , les plus fortunés d' entre nous se soigneront toujours autant , les plus modestes auront la couverture maladie universelle , mais les classes moyennes , elles , n' auront rien . Ce seront les victimes de ce projet . Mon ambition , c' est aussi d' agir pour l' école . Lorsqu' on porte un projet progressiste , on porte un projet pour que , partout , comme je le disais , quelque soit l' endroit d' où l' on vient , on puisse réussir . C' est cela , la mission de l' école de la République . Et , on le sait , c' est cela qui , il y a un siècle , dans nos campagnes , a permis à des fils et à des filles de paysans d' accéder à l' école , ce qui semblait vertigineux à leurs parents . Moi , je me souviens encore de ma grand-mère , elle n' était pas de la région mais des Hautes- Pyrénées , me raconter - son père était chef de gare et sa mère concierge - que pour ses parents , c' était invraisemblable qu' une fille , qui plus est , poursuive au-delà de 14 ans ses études . C' est l' école de la République qui a fait qu' un instituteur lui a fait confiance , lui a dit &laquo; Il faut continuer un peu plus loin &raquo; et puis un autre , un professeur de philosophie , qui fait partie de ma propre mémoire , vous imaginez ce que c' est que la République , c' est ça la République , ce n' est pas un mot qu' on envoie à la figure des gens comme un sel qui ne sale plus . La République , ce sont des visages , des gens engagés qui transforment des vies . Moi , ma vie , mes souvenirs , ils sont faits des souvenirs d' enfance de ma grand-mère , et de ce professeur de philosophie que je n' ai jamais vu , mais dont j' ai l' impression de connaître le visage . Parce qu' il lui a fait lire Paul Valéry un jour et qu' il a dit à ma grand- mère , au début de ce siècle : &laquo; Il faut que vous alliez suivre des études pour devenir institutrice à Toulouse &raquo; . Puis , après , la vie fait son oeuvre , ma famille a poursuivi son chemin et je suis devant vous aujourd'hui . Mais il y a une chose : ça n' aurait jamais été possible sans l' école de la République . Alors cette école , aujourd'hui , elle trahit ses promesses , elle reproduit , bien davantage qu' il y a 20 ans , les inégalités . Moi , j' ai fait une école à laquelle on reproche beaucoup de choses : l' ENA . Il y a des gens qui choisissent , face à ce fléau , de faire oublier qu' ils ont fait l' ENA et de dire qu' il faut la supprimer . Moi je pense que c' est toujours compliqué d' aller devant les gens , de dire &laquo; J' ai un projet pour le pays &raquo; en expliquant qu' on se renie soi-même : ce n' est pas un bon signal . J' ai fait l' ENA , c' est un concours de la République , je l' ai fait en travaillant , j' en ai raté d' autres , je vous rassure . Ce qui ne va pas , c' est ce qu' on en fait ensuite . Mais l' ENA , aujourd'hui , elle forme moins de fils d' ouvriers et de paysans qu' il y a trente ans . C' est cela notre échec . Notre école , aujourd'hui , elle fait qu' il y a 20 % de nos jeunes en CM2 qui savent imparfaitement lire , écrire , compter . Ces jeunes-là , ce sont ceux qui ne finiront pas leurs études supérieures , qui auront du mal à avoir le baccalauréat , qui décrocheront bien souvent avant , qui auront du mal à attraper un emploi et pour qui la vie sera plus injuste que pour les autres . Alors , mon projet pour l' école , je ne vais pas vous le détailler ce soir . Il y a une mesure qui me tient à coeur , c' est que dans tous les endroits de la République identifiés comme difficiles , plus pauvres que les autres , plus reculés , ce qu' on appelle les zones d' éducation prioritaire , je veux qu' on divise par deux le nombre d' élèves par classe . En CP et en CE1 , 6000 CP , 6000 CE1 . C' est très pratique ce que je vous dis là parce que si on ne fait pas ça , on ne peut pas former ces enfants , on les laisse face aux injustices et donc il faudra là réallouer une partie des maîtres et des postes qui ont été créés ces dernières années et dispersés un peu partout ; et il faudra investir un peu de ce que nous économiserons en simplifiant l' épreuve du baccalauréat et en réorganisant une partie du reste de notre école . Je veux aussi qu' il y ait plus de temps d' enseignement devant les élèves . Si nous voulons réussir ce combat de la République et de l' école , nous devons arrêter ce mouvement , commencé il y a maintenant dix ans , qui consiste à réduire le temps d' enseignement : c' est de la démagogie , c' est de la défaite . Les enfants ont besoin d' être éduqués , ils ont besoin d' enseignements . Et donc , nous stopperons ce mouvement décennal de baisse pour repartir de l' avant en valorisant aussi les carrières d' enseignants , en valorisant en particulier celles et ceux qui vont sur ces terres de batailles et ces terres de difficultés , en valorisant aussi l' autonomie pédagogique , la capacité à innover sur le plan pédagogique et sur le terrain . L' ambition que nous portons , vous l' avez compris , c' est donc d' apporter ces véritables sécurités nouvelles et contemporaines sans lesquelles on ne peut pas prendre de risques , c' est aussi cela que je porte , en réformant en profondeur notre système de chômage , notre système de formation continue , pour qu' on puisse demain former partout en France toutes celles et ceux qui n' ont pas de qualifications , qui sont des chômeurs de longue durée et qui ont besoin d' être accompagnés pour se réorienter dans leur carrière , c' est indispensable . L' ambition que je porte ensuite , que nous portons , c' est une ambition pour le travail , le travail c' est une valeur fondamentale , le travail c' est ce autour de quoi nous allons rebâtir notre projet de société : la capacité offerte à chacune et chacun de retrouver une place . Alors , parce que je suis le candidat du travail , je veux qu' on puisse entreprendre plus facilement dans notre pays , je veux qu' on puisse créer plus facilement une activité d' indépendant , de start-up , qu' on puisse innover beaucoup plus simplement . Très concrètement , en faisant quoi ? D'abord , en faisant que les indépendants aient une situation plus simple , plus transparente qu' ils ne l' ont aujourd'hui , c' est pour cela que je supprimerai le RSI ( le Régime Social des Indépendants - s' il y avait une petite dizaine de personnes qui ont l' air de ne pas connaître le RSI dans cette salle ) . Le Régime Social des Indépendants a en effet , par les difficultés de gestion , et là je n' incrimine en rien celles et ceux qui font fonctionner ce régime et à qui je veux rendre hommage parce qu' ils ne sont pas les responsables de cette situation , fait qu' aujourd'hui les délais sont trop longs , les tracasseries sont incessantes et on a vu , malheureusement , chez les indépendants , beaucoup trop de suicides ces dernières années arriver à cause de cela . Je maintiendrai tous les allégements de charges qui sont liées aux indépendants . Tous les indépendants et professions libérales seront éligibles à tous les allégements de charges que j' ajouterai et en particulier les six points de charges en moins pour tous les salariés , tous les entrepreneurs que j' ai annoncés dans mon programme , il touchera aussi les indépendants . Mais les indépendants demain auront donc la possibilité d' aller , s' ils le souhaitent , d' aller vers le régime général ou s' ils restent dans ce régime privilégié qui est le leur , eh bien d' être adossés au régime général et d' avoir les mêmes modalités , la même simplicité que tous les salariés de France , que tous les autres Françaises et Français en termes de simplicité , de rapidité , de délais . Je veux ensuite continuer à alléger le coût du travail , cela veut dire transformer le crédit d' impôt compétitivité emploi en six points de charge en moins jusqu' à 2,5 SMIC pour toutes les catégories d' entreprises qu' elles soient entreprises du secteur privé , de l' économie sociale et solidaire ou du secteur coopératif . Et parce que je suis le candidat du travail , je veux enfin que le travail paye mieux et donc , pour ce faire , que les cotisations salariales payées aujourd'hui par tous les salariés soient allégées et qu' on puisse ainsi transférer une partie du financement social qui pèse aujourd'hui sur le travail , sur les impôts et donc que le pouvoir d' achat de tous les travailleurs , fonctionnaires , secteur privé , indépendants , professions libérales augmente et il augmentera demain grâce à mon projet . Et quand je parle du travail et de notre ambition pour le travail , je parle évidemment aussi de cette révolution que nous devons faire pour tous les secteurs de notre économie , de l' innovation qu' on voit ici fleurir à travers la French Tech dans toute la région , que j' ai modestement accompagnée mais je l' ai souvent dit , la French Tech , c' est la meilleure politique publique . En tant que ministre , c' est ce dont j' ai été le plus fier parce que c' est quasiment aucun crédit public , il n' y a aucune loi , aucun décret et je n' ai à peu près rien fait . La French Tech , c' est un label mis sur l' énergie qui existe dans tous les territoires et l' innovation formidable qui est là ce soir et qu' on a mis simplement sous un même pavillon et un même étendard , c' est donc la France qui réussit . C' est cela , la French Tech et ça n' est à personne . Donc vous voyez , je vous rassure , pas plus que je crois qu' aucun politique n' a inventé l' internet , l' accès aux géants de la technologie , je ne suis pas l' inventeur de la French Tech . La French Tech , ce sont vos entrepreneurs , c' est ce coeur vibrant de l' innovation . Et donc oui , cette économie , elle se transforme , il y a des secteurs nouveaux qui apparaissent , d' autres qui vont être profondément bousculés par la transformation énergétique et environnementale qui est la nôtre , une transformation qui vient offrir des nouvelles opportunités au pays tout entier . Mais je veux ici vous parler tout particulièrement d' un secteur , un secteur qui tire la Bretagne toute entière , qui est au coeur de son économie , qui couvre 60 % de son territoire , qui concerne de près ou de loin à peu près toutes les familles bretonnes , je veux vous parler de l' agriculture . Je vais vous en parler parce qu' on l' oublie trop souvent , parce que , quand on parle de ces transformations de notre économie , on oublie le rôle qu' a l' agriculture , le rôle qu' ont l' agriculture et l' agro-alimentaire dans nos exports . On nous parle , je ne vais pas ici faire de polémique sur le sujet , je vous ai dit mon amitié pour Jean-Yves Le Drian , mais on nous parle toujours des rafales , des grandes réussites et on lui doit beaucoup là-dessus , de notre industrie de défense . On nous parle volontiers des Airbus mais on oublie que notre agriculture et notre agro-alimentaire , c' est notre fierté à l' export , c' est une vraie réussite . Mais je vais vous dire , on ne parle plus d' agriculture aujourd'hui , dans nos débats publics ou médiatiques qu' en parlant de la terrible misère qui est là , qu' il faut regarder en face ou en parlant de coût . On en parle comme si c' était un secteur qui devait s' effondrer , disparaître , qu' il faut accompagner dans , finalement , cette déliquescence . On en parle chez certains autres comme d' un coût , d' un budget qu' il faudrait supporter . On en parle comme d' une espèce de cause rétrograde quand certains , même pour faire plaisir , en pensant faire plaisir aux agriculteurs , disent &laquo; Mais oui , faut plus du tout respecter l' environnement , le climat est une affaire secondaire , on va vous supprimer toutes ces normes qu' on vous a mises &raquo; . Non , tout cela n' est pas vrai . L' agriculture c' est notre histoire , l' agriculture c' est notre pays , l' agriculture c' est un secteur économique qui exporte , l' agriculture ce sont les femmes et les hommes qui nous nourrissent chaque jour . Au cœur de mon projet , au cœur de notre ambition , je veux porter cette ambition pour l' alimentation française . On a chaque jour , quand on parle de notre culture , de ce qui nous fait , de ce qui nous distingue des Chinois ou d' un Américain , parmi tant d' autres choses , ce qui nous nourrit , il y a ces plats qu' on défend , cette gastronomie française , c' est cela l' agriculture aussi , c' est notre alimentation . Il y a l' objectif de souveraineté alimentaire que nous devons porter . Les agriculteurs , ce sont ceux qui vous nourrissent , qui vous nourrissent et qui doivent demain nourrir nos enfants . L' agriculture , et on le voit ici , on en parlait tout à l' heure , c' est aussi ce qui fait nos paysages , je le dis souvent . Dans ces mots qu' on ne doit plus prononcer , vous savez , je suis un grand maladroit parfois , je dis des mots qu' on ne doit pas dire . Alors , parfois on fait des erreurs , j' en ai fait , en particulier en Bretagne , et j' ai beaucoup d' émotion encore quand je repense à cette visite que j' ai faite , il y a quelques mois , chez Gad parce que j' avais blessé des gens de manière impropre et injustifiée , c' était une faute . Mais parfois , dans le débat public français , on ne peut même plus nommer le réel , on voudrait tomber dans une espèce de langue qui se vide , dans une circonlocution qui évite de dire les choses au fond . Il y a un mot qu' on ne veut plus dire c' est le mot de &laquo; paysan &raquo; . C' est un beau mot , &laquo; paysan &raquo; . Parce que oui , nos paysans , font nos paysages , ils font nos pays , ils transforment la France et quand on arrive , se posant comme je l' ai fait sur Brest tout à l' heure , même quand on se pose à Lannion , comme j' ai fait il y a quelques mois , on voit ce travail de dentelle , on voit cet entretien des paysages . Qui ferait les bocages ? Qui ferait ces paysages variés et multiples ? Qui débroussaillerait ? Qui fait que notre pays n' est plus une jungle impropre ou une forêt inféconde ? Nos paysans . Quand on parle d' agriculture c' est aussi de cela que nous devons parler . Et puis … Vous pouvez le dire , c' est pour eux . Et puis parler d' agriculture , c' est parler d' un savoir- faire , de traditions , c' est parler d' innovation , d' une transformation profonde , c' est parler de qualité , de produits . Alors oui , ce soir , je vous parle d' agriculture quand je vous parle de l' ambition française parce que l' agriculture en Bretagne , qui est la première région agricole française , comme dans tout le reste du pays , c' est le cœur de notre ambition pour demain ! Alors , je le sais , les années dernières ont été difficiles , 2015 et 2016 ont été marquées par des crises profondes , la crise du marché , la crise du lait , des crises sanitaires , comme la grippe aviaire dans le secteur des canards à foie gras , des crises climatiques aussi , le gel qui a touché le vignoble bourguignon , les mauvaises récoltes qui ont touché nombre de céréaliers . L' impact de ces crises est particulièrement visible en Bretagne parce que , comme je le disais , c' est la première région agricole . Alors , face aux crises , il y a eu des réponses , on a versé de l' argent , on a apporté des aides , elles étaient indispensables . Mais est -ce que cela suffit ? Non . Le cœur de notre réponse , de notre ambition , c' est de refonder un pacte profond , un pacte social entre les agriculteurs et la société française . Ce pacte social , c' est un sens , une vision à long terme pour l' agriculture française . Des agriculteurs fiers de leur travail , des citoyens fiers de leur agriculture parce que , comme je viens de le rappeler , l' agriculture , elle est au cœur de l' histoire mais aussi du projet français . Alors pour cela , nous n' y arriverons pas simplement en augmentant les aides . J' entends parfois des discours simplistes là-dessus . Non . D'ailleurs ce n' est pas ce qui nous est en premier demandé . J' étais tout à l' heure avec quelques-uns d' entre vous sur ces belles terres du Finistère à Ploéven près de Châteaulin dans la famille Nedellec - ils ont plus de 50 vaches . Je suis arrivé , on m' a dit les difficultés du quotidien . Il y a un chef d' exploitation formidable , son fils , courageux , qui va reprendre le flambeau , on ne nous a pas parlé d' aides , on m' a dit une chose , une chose , &laquo; on veut pouvoir vivre dignement de notre travail &raquo; . Alors oui , le premier pilier de ce nouveau contrat que je veux refonder , entre les paysans et les Français , c' est de pouvoir vivre dignement de son travail et pour cela c' est d' avoir des prix qui rémunèrent le travail des agriculteurs en France . Non pas des aides , non , mais d' avoir le juste prix pour les produits . Alors , pour ce faire , il faut d'abord acter d' une chose , là aussi Monsieur Nedellec me l' a dit tout à l' heure : &laquo; Moi , je travaille avec à peu près les mêmes prix qu' il y a 30 ans , j' ai dû faire des investissements , me mettre aux normes &raquo; . Olivier , tu sais bien , tu m' as dit à peu près la même chose , le marché est plus difficile , les contraintes pour produire ne sont pas les mêmes , on a évidemment un droit social qui n' est pas le même , et c' est bien normal , mais les prix n' ont pas bougé . Parfois , même , ils ont diminué . Ce que je veux , c' est renforcer le pouvoir de négociation des agriculteurs , dans le respect de la législation européenne , pour qu' ils fassent le poids face aux industriels et aux distributeurs . Pour cela , je veux des associations d' organisations de producteurs aux pouvoirs renforcés , elles pourront négocier avec un véritable mandat des agriculteurs , des contrats cadres sur les volumes et le prix du lait . Leur création doit être simplifiée et encouragée , et lorsqu' une association d' organisations de producteurs existera , l' industriel ou la centrale d' achat sera dans l' obligation de négocier directement avec elle . Au fond , mon message , aujourd'hui , c' est que l' agriculture n' est pas simplement l' affaire des agriculteurs , et c' est aussi un appel à la responsabilité collective . On doit réorganiser les filières par ces contrats et ces associations , mais il faut aussi que chacune et chacun , nous soyons responsables . Et c' est donc un appel à la responsabilité des transformateurs , pour que , dans leurs marges , ils pensent aux producteurs et distribuent de manière juste la création de richesse . C' est un appel à la responsabilité de la grande distribution , en tant que ministre , je suis celui qui a augmenté partout les contrôles sur les marges et le respect de la loi sur les grands distributeurs . Mais je sais aussi dire la responsabilité avec laquelle plusieurs enseignes ont su valoriser des produits du terroir , ont su respecter certains producteurs pour maintenir les prix , valoriser . Et je sais , là-dessus aussi , le travail que nous avons mené avec plusieurs d' entre vous , il y a quelques mois , sur l' étiquetage et la valorisation des produits bretons . Mais c' est aussi un appel à la responsabilité de tous les Français , qui doivent être un peu plus que des consommateurs , mais des acteurs engagés pour leur pays . Chacune et chacun , lorsque nous consommons , nous devons être vigilants , valoriser là aussi les produits et donner une place aux circuits courts . Les circuits courts ne sont pas la panacée , tout le monde ne peut pas y arriver . Mais il faut , partout où cela a du sens , le favoriser , l' aider , le valoriser . Et donc , vous le voyez , pour répondre à ce problème de prix , c' est cet ensemble de mesures que nous devons prendre . Mais , au-delà , la deuxième mesure que je veux prendre pour permettre de vivre dignement de son travail , c' est également de continuer et d' encourager la montée en gamme ; c' est indispensable . Qu' est -ce que cela veut dire , concrètement ? Ça veut dire que les standards qui existent dans nos régions , dans nombre de nos produits aujourd'hui , sont des bons standards . Il ne faut pas laisser dire des choses là-dessus indues ou impropres - j' entends des discours qui sont souvent désobligeants pour nos agriculteurs . Mais il y a plusieurs modèles , et je ne défends pas un modèle unique . Il faut un modèle parfois plus intensif , et , dans ces cas-là , mieux orienté vers l' export . On s' en souvient chez Richard et c' est ce qu' on a fait avec Doux et toutes celles et ceux qui travaillent autour de cette belle entreprise , et qui , en Bretagne , produisent des poulets de manière intensive et qui les exportent vers d' autres marchés . Il y aussi des modèles moins intensifs , qu' il faut également privilégier et protéger à travers des circuits qu' il faut , comme je le disais , encourager . Mais il y a une chose qui ne marche pas : c' est de ne pas répondre aux besoins des consommateurs et de son marché . Et donc , nous devons valoriser nos produits , continuer à aider à l' export , mais en même temps , permettre partout sur le territoire à nos paysans de monter en gamme , d' améliorer la qualité , d' aider à investir pour faire de meilleurs produits . Produire mieux , produire avec des labels , produire en polluant moins , produire en inventant , là aussi , de nouveaux types de production . Je veux ici être très clair avec certaines et certains qui mènent des combats rétrogrades : on a beaucoup trop opposé l' agriculture et l' environnement . Beaucoup trop ! Dans la montée en gamme , il y a l' indispensable montée en gamme pour répondre aux contraintes environnementales , et là-dessus les paysans bretons n' ont aucune leçon à recevoir , aucune ! Parce que la qualité de l' eau bretonne est largement due au travail , décennie après décennie , des paysans bretons . Mais pour permettre la montée en gamme , l' innovation , cette réconciliation , on doit aller plus loin . Et je veux , moi aussi , mener cette transformation environnementale de notre modèle productif . C' est pourquoi je lancerai un programme d' investissement d' avenir agricole sur la période 2017-2022 ! C' est une première , cela n' a jamais été fait . On parle toujours de programmes d' investissement d' avenir pour les startups , pour les secteurs qui paraissent construire la France de demain . Mais la France de demain c' est aussi cette transformation de la France agricole que j' évoquais . Et donc cela permettra l' accès à des financements nouveaux - non pas de la nouvelle dépense publique , mais du financement d' investissements pour des projets de modernisation des exploitations ayant un impact positif sur l' environnement , pour des projets de développement d' activités et de transformation et de distribution qui privilégient notamment les circuits courts en privilégiant les projets coopératifs . Parce que l' investissement est clé pour réussir cette montée en gamme partout . Pour accélérer la montée en gamme des produits agricoles , il faut aussi permettre à nos agriculteurs de relever , comme je le disais , ce &laquo; défi- environnement &raquo; , d' accélérer cette transition et cette transformation . Et , donc , c' est l' objet de ce programme que je lancerai dès le début du quinquennat . Mais pour aller plus loin , on doit également valoriser le travail pour l' environnement qui est fait par les paysans et le valoriser comme une activité en propre , elle-même . Le rôle de l' Etat , c' est de les accompagner parce que dans les missions de l' agriculture du XXIe siècle , au-delà de tout ce que j' évoquais tout à l' heure , il y aura aussi celle d' accompagner cette transformation environnementale et énergétique , celle de produire de l' énergie par la mécanisation , par la biomasse , parfois par l' éolien , et par des services environnementaux indispensables à nos territoires et à nos paysages . Pour ce faire , je veux rémunérer les services environnementaux rendus par les agriculteurs à hauteur de deux cent millions d' euros par an , à travers la création d' un service de paiement pour services environnementaux . Et là aussi , ça n' est pas de la dépense publique en plus , ce système modeste mais important , c' est de la réattribution de certains mécanismes qui ne prennent pas suffisamment en compte les nécessaires investissements pour l' environnement . Le deuxième pilier , ce sont les normes . Et nous en avons bien besoin , là-dessus je crois que je n' ai pas à vous convaincre . Il y a trop de normes , il y a des normes qui sont compliquées à appliquer . Et donc , le projet que je veux porter en la matière est simple : premièrement , revoir toutes les normes inutiles dans les cinq années à venir , porter un projet de révision de tout ce qui a , jusqu' alors , été sur-transposé pour le supprimer , porter un projet de simplification normative dans le champ de l' agriculture qui est absolument indispensable . Deuxièmement , permettre un droit à l' erreur pour tous les particuliers , pour toutes les entreprises et , en particulier , pour les exploitants agricoles . Ce droit à l' erreur , qu' est -ce que c' est ? Eh bien c' est la possibilité , quand on est contrôlé pour la première fois , et qu' on a commis une infraction qui n' est pas une infraction pénale - où on ne met pas en cause la vie d' autrui – , eh bien , de ne pas avoir une administration qui vous sanctionne , qui vous met tout de suite des pénalités , mais une administration qui vous conseille , qui vous corrige . C' est une révolution culturelle , conceptuelle , profonde . J' ai eu l' insigne honneur d' être ministre de la République . J' ai vu la dignité de nos fonctionnaires et je n' ai jamais voulu tomber dans cette espèce de discours de facilité où on oppose un secteur à l' autre . Ah , combien de ministres sont venus devant vous , dire &laquo; Ah , l' administration c' est un calvaire . &raquo; C' est facile , on veut être populaire . Quand on veut être démagogue , on va devant des paysans et on leur dit &laquo; L' administration , c' est votre problème . &raquo; . On leur dit aussi &laquo; L' Europe , c' est votre problème . &raquo; C' est faux . D'abord , il n' y a pas de mauvaise administration . Il y a des mauvais ministres , ça c' est vrai . Quand vous dirigez votre exploitation agricole , votre entreprise , votre exploitation , votre classe , votre association , que sais -je , votre mairie , il ne vous viendrait pas à l' idée de dire &laquo; Ils sont mauvais . &raquo; Mais quand on dirige l' Etat , il est de bon ton de le dire . Non . Mais , on s' est enfermé dans une espèce d' opposition culturelle où la dignité de l' administration est bien souvent de contrôler ou de sanctionner et du coup , le reste de la vie économique et sociale se sent comme oppressé . Eh bien ce droit à l' erreur , c' est le rôle reconnu à l' administration , évidemment de contrôler , d' aller sur le terrain , s' assurer de l' égalité républicaine en matière de normes , mais , partout , d' accompagner les uns et les autres , de les conseiller , jamais de ne les sanctionner la première fois . C' est fondamental en matière de normes . Enfin , le troisième pilier de cette ambition pour l' agriculture , ce pacte refondé , c' est l' Europe et la PAC . Ah oui , il y en a beaucoup qui viennent vous expliquer que l' Europe , c' est la source de tous les maux . Mais , on était tout à l' heure avec vos représentants , cher Olivier , on était avec la FDSE , on était chez vous , Monsieur Nedellec . J' étais à Rennes , il y a quelques mois . On ne m' a pas dit &laquo; On ne veut plus de l' Europe &raquo; . On ne m' a pas dit &laquo; L' Europe , c' est notre problème &raquo; . On m' a dit &laquo; Les réformes européennes qui ont été mal préparées , mal pensées , les dérégulations européennes , les incohérences européennes , c' est mon problème , mais pas l' Europe en elle-même . &raquo; C' est très différent . Alors notre projet pour l' Europe , c' est d' avoir moins d' interventionnisme hors-sol , d' avoir moins de directives , dénuées de sens , et donc d' avoir une Europe plus concrète . L' Europe ne peut fonctionner et être acceptée que si elle est un moteur de progrès , que si elle est synonyme de soutien , que si elle se bat pour les peuple et non contre eux . Et ma conviction , c' est qu' il est urgent de renégocier les règles de la concurrence pour que cette concurrence soit non-faussée . Et il n' est pas normal que des travailleurs détachés des pays de l' Est peuplent les abattoirs allemands , alors qu' ils sont payés quatre euros de l' heure avant la mise en place d' un salaire minimum ; alors qu' ils ne sont pas soumis aux mêmes règles . Et donc oui , je mènerai à son terme une réforme de la directive sur le travail détaché , elle est indispensable . Voilà pourquoi il faut défendre une harmonisation vers le haut des normes sociales et environnementales . Qu' est -ce que cela veut dire ? Non pas détricoter les règles françaises , comme certaines ou certains vous le promettent , non pas dire &laquo; On fera moins d' environnement &raquo; , mais dire &laquo; Ce qui est imposé en France doit l' être partout en Europe de la même façon , avoir les mêmes contrôles &raquo; . Et dans le même temps , il faut défendre , défendre nos agriculteurs à Bruxelles . Et je dois dire ici , parce que je l' ai vu faire moi-même au gouvernement , que , contrairement à ce que beaucoup ont parfois dit , Stéphane Le Foll en tant que ministre , s' est battu à Bruxelles et il a fait le maximum dans le cadre de la PAC qui avait été renégociée pour défendre les intérêts de l' agriculture française et je veux ici le défendre contre les caricatures parfois faites . Par contre , nous avons parfois mal négocié collectivement , de droite et de gauche , les transitions , on avait mal préparé la dérégulation , on avait mal préparé la sortie des quotas , on avait mal modernisé nos structures , et donc oui , la dérégulation européenne nous en avons été victimes et c' est cela que nous devons corriger . Voilà pourquoi je veux , dès mai 2017 , dans le cadre des négociations pour la PAC 2021 , que la France propose la création , dans le cadre du premier pilier et d' une PAC au budget préservé , d' un système de subventions contra-cyclique de garantie sur les prix ou le chiffre d' affaires . C' est indispensable . Qu' est -ce que cela veut dire , mes amis ? Ça veut dire protéger tous les agriculteurs contre la volatilité des prix agricoles , ça veut dire éviter , parce que le prix d' un seul coup bascule ou s' effondre , parce qu' il y a un éternuement en Australie ou ailleurs , parce que le marché chinois s' effondre , parce que certains sur-produisent , éviter que les prix ne s' effondrent et que , de ce fait , certaines exploitations , des milliers , parfois des centaines de milliers à travers l' Europe , ne doivent mettre la clé sous la porte . C' est éviter des suicides , c' est éviter des situations profondément injustes , liées non pas à une non-compétitivité , liées non pas au fait que ces exploitations agricoles ne peuvent pas relever les défis nouveaux mais qu' il y a une volatilité infernale , des prix devenus irrationnels , c' est le devoir de l' Europe de protéger son agriculture face à cela avec des vrais mécanismes de régulation des volumes en cas de crise . Et puis , bien sûr , je veux avoir un mot pour ces travailleurs de la mer , cher Pierre . L' autre pilier de la Bretagne , c' est la pêche . La pêche , elle est pourvoyeuse d' emplois , de dynamisme . Imaginerait -on le Guilvinec ou Lorient sans leurs bateaux de pêche , sans leurs pêcheurs ? Mais elle en dit long surtout sur les valeurs qui sont les nôtres : le courage , la solidarité , l' humilité face à la nature , face aux éléments parfois . Nous sommes , ce soir , le lendemain des treize ans du Bugaled Breizh et de ses victimes et je veux avoir un mot pour eux ce soir et une pensée . Nos pêcheurs , ce sont des courageux , ce sont des vaillants , ce sont des femmes et des hommes qui , dans la nuit , quittent leurs familles parfois pour longtemps . Ah , je les connais , moi je suis des Hauts-de-France comme on dit maintenant , c' est une autre pêche , ils sont fiers , ils aiment leur métier et le risque qui va avec et , eux aussi , ils ne demandent pas qu' on leur enlève du risque . Ce sont des aventuriers et des ambitieux au vrai sens du terme . Ces dernières années , ils ont eu des bons résultats et je m' en félicite parce qu' il y a eu un pétrole moins cher , parce que les efforts ont payé . Ah , je les connais ceux qui se sont battus pour que la langouste rouge puisse revenir dans les eaux de la mer d' Iroise , il y en a tellement qui se sont battus sur de telles conquêtes mais ces résultats , ils ne doivent pas masquer une situation qui reste préoccupante . Pour permettre à nos pêcheurs de vivre de leur travail , nous devons impérativement améliorer la compétitivité de leurs entreprises , les aider dans le renouvellement de leurs outils de travail et en particulier des bateaux de pêche pour faire justement des bateaux de pêche plus propres , mieux adaptés , plus économes , c' est cela qu' ils demandent et il faut donc adapter nos dispositifs nationaux et européens pour ce faire . Cela implique aussi de rendre la politique européenne - et ça fait partie du combat que je mènerai , pour la politique européenne en la matière - plus souple , ça veut dire avoir des stocks , des quotas qui sont adaptés à la réalité du terrain . Ne pas avoir les mêmes quotas partout et pour tout le monde parce que quand les stocks existent , on peut ré-autoriser certaines pêches . Mais aussi des perspectives car nous devons désormais nous organiser selon des quotas pluriannuels qui permettront aux pêcheurs de voir où ils vont et de cesser , sans mauvais jeu de mots , de naviguer à vue . Voilà l' ambition , mes amis , que je veux porter pour notre agriculture . Alors , je pourrais vous parler de bien d' autres de nos ambitions mais un dernier mot pour notre ambition européenne . Parce que oui , l' ambition que nous devons avoir , cette ambition de progrès , celle qui réconcilie la liberté et la protection , elle est au coeur de notre ambition européenne . Ils sont trop nombreux ceux qui l' ont oubliée , on a laissé le doute s' installer . Moi je sais que je suis ici dans des terres toutes particulières , c' est pour ça que je les aime , dans des terres où l' engagement associatif , le militantisme laïque , la démocratie chrétienne ont su se tresser , se réconcilier . Et dans ces terres , depuis le début , on est fier de la Bretagne , on aime la République parfois plus que partout et là-dessus , beaucoup se sont trompés . &laquo; Ah les Bretons peuvent se cabrer contre l' Etat &raquo; , je le sais , mais jamais ils ne le font contre la République . Mais ici , on aime l' Europe plus que tout . Ici , parce que sans la Bretagne , sans les grandes terres de l' Ouest , l' Europe ne serait pas là . Sans les grandes terres de l' Ouest , la France aurait dit non à l' euro ! Je ne l' oublierai jamais ! Alors cette Europe , je la défendrai avec vous , j' ai porté notre projet européen la semaine dernière à Berlin . C' est un projet pour relancer l' Europe . C' est un projet où la France doit prendre ses responsabilités , mener ses réformes , mener avec courage ce qu' elle doit faire pour elle . Mais en même temps , nous allons demander à l' Europe plus d' investissements , plus d' intégration à quelques uns et relancer en Europe une souveraineté nouvelle , une souveraineté pour protéger nos concitoyens face aux risques du monde : le terrorisme , l' immigration , la transformation numérique , les attaques contre notre alimentation . L' Europe , c' est un projet d' ambition et de souveraineté . L' Europe , ne l' oubliez jamais , ce sont nos valeurs portées dans le monde et , en même temps , ce qui nous protège . C' est cela l' Europe , c' est notre projet et nous le porterons ! Mes amis , je le dis souvent , nous sommes les seuls à porter le projet européen et donc nous le ferons avec fierté et je veux dire ce soir à M. Trump qui , depuis l' autre côté de notre océan , devrait avoir un peu plus d' humilité . M. Trump l' Américain , n' oubliez jamais que si vous êtes une nation libre , c' est parce que des ambitieux sont partis de ces terres avec l' amour de la liberté , avec le même rêve , le rêve français , le rêve européen ! M. Trump , n' oubliez jamais ce que vous nous devez , la liberté , votre existence . M. Trump , regardez votre histoire , c' est celle de Lafayette , c' est la nôtre ! Parce que de la même façon , M. Trump , je regarde notre histoire . Je sais les jeunes Américains qui n' avaient jamais connu l' Europe , qui n' avaient jamais connu la France et sont venus mourir chez nous , sur nos plages , sur notre terre . Je sais ces jeunes Américains , je sais . Ces Américains qui ont cru dans l' Europe avant même les Européens . Ces Américains qui nous ont aimés au moment où nous doutions , les Marshall et tant d' autres . Alors oui , je vous le dis ce soir , le destin , le destin de la France , c' est un destin européen , le destin de la France et de l' Europe , c' est aussi le destin américain ; cela personne ne peut le trahir . Voilà mes amis , voilà mes amis , ce que je voulais vous dire ce soir et je répondrai à toutes les questions après , Monsieur , cher ami , vous viendrez après , on viendra vous amener après mon meeting et je répondrai à toutes les questions sinon quel est le sens d' une démocratie où je ne répondrais qu' à une question parmi plus de deux mille ? Voilà mes amis , j' aurais pu vous parler des heures de nos ambitions , de notre ambition pour la France . J' aurais pu vous parler de l' ambition , cher Monsieur SALAÜN , de votre grand-père , vous m' en parliez tout à l' heure , quand il a quitté les terres bretonnes pour aller conquérir l' Amérique et vous permettre aujourd'hui d' aller à la conquête , avec la prochaine génération , de nouveaux marchés . J' aurais pu vous parler d' ambition de tant et tant de Bretons fiers d' eux-mêmes , de ce qu' ils sont , de ce que cette terre porte et que nous lui devons parce que la Bretagne est une terre qui ose , parce que la Bretagne est une terre ambitieuse , notre ambition , c' est le projet pour le pays que nous portons . L' ambition , elle est avant tout celle de votre engagement au quotidien et , oui , cher monsieur vous le disiez , l' engagement sur lequel je compte , c' est aussi celui des territoires , tout ne se fera plus d' en haut , l' engagement que je porte et l' ambition que je porte est aussi une ambition citoyenne des territoires ; ce n' est pas innocent . Je viens ici , sur ces terres bretonnes , à Quimper , là où le Général De Gaulle fit son dernier grand discours , celui sur la décentralisation parce que cette ambition que je porte c' est aussi une ambition pour redonner le pouvoir aux territoires , redonner le pouvoir aux régions ! Cette ambition que je porte parce que c' est la vôtre , c' est une ambition de redonner le pouvoir à chacune et chacun , aux citoyens , aux territoires , aux élus . Je vous dis , cette Révolution &laquo; En Marche ! &raquo; que nous portons depuis tant de mois et que nous allons porter jusqu' au bout , c' est un projet d' ambition mais c' est un projet qui est le nôtre mes amis ! Ce que je propose n' est en rien facile , n' est en rien innocent , ça n' est pas de traiter des clientèles , ça n' est pas de faire plaisir à quelques-uns . Ce projet , c' est le vôtre parce qu' à chaque instant , je vous demanderai , avec moi , de le porter , de le réinventer , mais une fois que nous aurons pris le pouvoir , ce sera , pour moi , pour vous le rendre . Parce que oui , mes amis , j' ai besoin , dans les cent jours qui viennent , de votre énergie pour porter ce projet d' ambition partout , parce que j' ai besoin partout de votre fougue , de votre engagement , parce que j' aurai besoin demain de l' énergie de tous les territoires , de la Bretagne , de l' énergie des élus , des associations , des citoyens parce que cette ambition , c' est la nôtre ! Alors ensemble , je vous en fais le serment , nous irons jusqu' au bout ! Vive la Bretagne , vive la République , vive la France ! Bravo ! Merci à vous ! ",
    "Merci d être ici aujourd'hui , je tenais à pouvoir vous présenter les prochaines étapes de notre campagne présidentielle et les principes qui régiront , pour En Marche ! , les élections législatives . En Marche ! est aujourd'hui un mouvement politique qui compte plus de 145 000 adhérents . Ce mouvement , inédit dans la Cinquième République , appartient à celles et ceux qui depuis le début ont eu le courage , la volonté , la pugnacité de refonder notre vie politique afin de faire réussir notre pays . Ce mouvement traduit une volonté de s' engager , de s' approprier la vie politique et civique française , et une attente profonde de transformation . Je suis très fier de cette dynamique collective et pleinement conscient , dans le contexte , de mes responsabilités . C' est parce que je suis le garant de ces aspirations , qui m' obligent , que je veux ici , pour vous et pour nos adhérents , clarifier les prochaines étapes de notre cheminement et notre volonté , précisément , de nous affranchir des jeux d' appareils qui ont conduit à bloquer notre pays . Nous avons , à En Marche ! , présenté clairement au pays , dès le mois d' avril dernier , nos valeurs et nos objectifs . Ils fondent le progressisme que nous portons : pour la transition environnementale et numérique , pour la flexi-sécurité , pour l' égalité des chances , pour le renouveau démocratique et culturel , pour une société forte et un État laïc , garant de la cohérence nationale , et enfin pour le projet européen . Nous avons ensuite établi le diagnostic du pays au contact des Français grâce à la Grande Marche qui , durant plusieurs mois , a permis d' aller entendre plus de 100 000 de nos concitoyens . Ce diagnostic a été présenté à l' automne . Notre vision a ensuite été exposée dans le livre Révolution . Cette vision était le préalable indispensable à une liste de mesures que nombre de candidats égrènent pour ensuite les modifier : la société du travail , la sortie d' une société de statuts , la possibilité de choisir sa vie et tout ce que j' ai pu détailler durant cet automne était le cœur de cette vision . Depuis le mois de décembre , je présente notre projet thème par thème : le travail , la santé , l' école , l' agriculture , l' Europe en cohérence avec cette vision initialement posée . Je poursuivrai cette présentation durant les prochaines semaines sur l' international , la culture , l' environnement , la sécurité , la défense , la justice , et bien d' autres sujets . J' aurai jusqu' à la fin de la campagne l' occasion de m' exprimer sur tous les sujets comme le requiert une élection présidentielle . Jusqu' à la fin de cette campagne , grâce à la mobilisation de tous nos comités locaux , nous améliorons et améliorerons la plateforme programmatique que je porte . Mais parce que je crois qu' une élection présidentielle exige hauteur de vue et cohérence , je présenterai d' ici à début mars le cœur de notre projet , celui d' un contrat avec la nation . Il s' agit du plan de transformation dont j' ai parlé à l' automne dernier , c'est-à-dire la méthode de gouvernement , les grands objectifs et les réformes-clés avec leur calendrier – je veux parler des réformes en profondeur , celles qui transforment et refondent , et non celles qui ajustent . J' ai commencé à en présenter quelques-unes sur la formation professionnelle , le travail , l' apprentissage , ou l' école . Ce plan de transformation , qui sera donc synthétisé d' ici début mars , sera accompagné d' un cadrage budgétaire et financier précis qui permettra d' améliorer notre capacité à investir pour préparer l' avenir et en même temps réduire nos dépenses courantes . Ce contrat avec la nation sera le cœur de mon engagement présidentiel . Ce seront les engagements qu' en tant que Président de la République je superviserai directement . En ce qui concerne les élections législatives , En Marche ! présentera 577 candidats , ce dans le but de construire une majorité présidentielle . Jusqu' aux élections présidentielles , En Marche ! restera un mouvement politique permettant l' adhésion multiple . À compter d' aujourd'hui , nous ouvrons un appel à candidatures pour nos élections législatives . Tous les membres du mouvement ou ceux qui souhaiteraient le rejoindre peuvent à compter de ce jour s' inscrire en ligne . Dans les minutes qui suivront cette conférence de presse , le formulaire sera disponible en ligne pour tout un chacun . Nous sommes la seule force politique à avoir établi des formalités aussi claires et accessibles . Mon souhait est que toutes celles et ceux qui veulent s' engager puissent le faire . Nous procéderons de la sorte pour les élections locales , nationales et européennes à venir . C' est ce qui est conforme à notre engagement initial et à ce que nos adhérents depuis le début souhaitent . Les désignations aux élections se feront sur cinq critères qu' il m' appartient , en tant que Président d' En Marche ! , d' exposer : - Le renouvellement : ces élections devront permettre de faire émerger des personnalités engagées et talentueuses , élus de terrain , engagés associatifs , retraités , étudiants , fonctionnaires , salariés , entrepreneurs , professions libérales , agriculteurs et indépendants . La diversité sera prise en compte . Ce renouvellement est l' une des traductions concrètes de l' engagement dans notre mouvement . Ce renouvellement inédit que seul notre mouvement peut porter améliorera la représentativité des Français et des territoires . Plus de la moitié de nos candidats doivent incarner ce renouvellement . - La parité réelle : les femmes représentent aujourd'hui 53 % du corps électoral , il n' est pas acceptable qu' elles représentent moins de 30 % des élus de l' Assemblée Nationale . Nous respecterons donc un principe de parité strict y compris dans les circonscriptions favorables , contrairement aux autres partis . - La probité : nos candidats doivent démontrer une intégrité absolue dans leurs valeurs et leurs actions . C' est une condition indispensable pour réconcilier les Français avec la vie politique . Aucun de nos candidats ne devra donc avoir de casier judiciaire ni avoir subi des peines d' inéligibilité . Il s' agit là aussi d' un principe qu' aucune autre force politique n' a retenu . - La pluralité politique : En Marche ! est une force politique centrale , un rassemblement large de progressistes . Il dépasse les clivages et les jeux d' appareils . À ce titre , je veux être , ce matin avec vous , clair : Il n' y aura aucun accord d' appareil avec quelque parti que ce soit ; ce sont les idées et la volonté de transformer le pays qui doivent nous unir . Je ne négocierai aucune circonscription contre un ralliement et ne sacrifierai rien de nos engagements initiaux qui nous ont menés jusqu' ici . Je ne suis le candidat d' aucun parti ou aucun appareil existant . En Marche ! est là pour accueillir et porter les candidatures de citoyens engagés . Ils peuvent être par ailleurs socialistes , radicaux , écologistes , centristes , et républicains , tant qu' ils se retrouvent dans notre projet . Tous les parlementaires dans ces formations républicaines qui se reconnaissent dans nos valeurs et notre action sont donc les bienvenus pour porter les couleurs d' En Marche ! dans le respect de l' ensemble des principes ici rappelés . Nous veillerons dans les investitures à ce que ce pluralisme soit respecté et ces équilibres tenus . - L' efficacité et la clarté : tous les candidats investis signeront le même contrat avec la nation que moi . Aussi , aucun candidat investi ne pourra exprimer de désaccord avec le cœur de notre projet et il le portera clairement devant ses électeurs . En revanche , notre mouvement est riche de diversité qui est en même temps sa force de rassemblement . Il est évident que sur de nombreux sujets nous avons des différences , elles s' expriment aujourd'hui dans les débats au sein du mouvement comme demain elles s' exprimeront au Parlement . Mais afin de gouverner efficacement , tous les candidats s' engageront sur ce contrat avec la nation . Parce que c' est le contrat sur lequel je m' engage moi-même vis-à-vis des Français . Une commission nationale en charge des investitures sera constituée dans les prochains jours . Elle sera l' instance en charge des désignations . Elle sera composée de neuf membres adhérents du mouvement s' engageant eux-mêmes à ne pas être candidats . Je n' en serai pas membre . Les investitures seront décidées par vagues successives . Avec un objectif : construire une majorité de projet pour gouverner et traduire en actes notre engagement collectif et citoyen . Les travaux de cette commission seront supervisés par une commission d' éthique composée de trois personnalités indépendantes du mouvement , et qui pourra être saisie par tout membre du mouvement ayant déposé sa candidature à l' investiture . Je vous remercie pour votre attention , et je vais maintenant répondre à cinq questions .",

    "Oui , volontiers . D'abord je souhaiterais que l' on dépersonnalise la discussion . Parce-que je trouve que depuis sept mois , on a excessivement personnalisé la critique de ma campagne , en la concentrant sur moi comme individu . C' est très mauvais , ça crée une ambiance trop tendue et au fond qui n' a pas de solution , car comme individu je ne changerai pas . Le fond de l' affaire est politique depuis le début : Nous avons eu un entretien , Pierre Laurent et moi , au mois de décembre de l' année 2015 , et je lui ai dit : &laquo; Pierre , je pense que nous avons le besoin d' une campagne de longue durée pour ancrer , car la situation va être très volatile &raquo; . Je le pressentais . Je savais qu' il y aurait une extraordinaire agitation , à la limite de la dislocation au sein du Parti socialiste . Nous savions tous qu' il y aurait une primaire à droite . Donc on pouvait assez facilement comprendre que la scène politique serait tellement agitée qu' elle pourrait être très rapidement illisible . Pas illisible par nous , militants , nous y voyons toujours clair . Mais illisible pour la masse des Français , c' est à eux qu' il faut penser , qui sont déjà dans un état de désorientation extrême . C'est-à-dire que des gens de gauche ne reconnaissent plus la gauche dans l' étiquette du gouvernement , ça ne veut plus rien dire pour eux . Si François Hollande c' est la gauche , si la loi El Khomri c' est la gauche , bon alors , qu' est -ce que la droite ? Et des gens de droite qui disent : &laquo; Mais nous , ce n' est pas ça notre vision de la Patrie , de la République de se brouiller en permanence avec l' autre partie de la population , celle qui est musulmane , ou que sais -je encore , les pauvres qui sont des profiteurs , d'après les gens de droite &raquo; . Beaucoup de gens de droite ne se reconnaissent pas là-dedans . Donc il y a en profondeur une désorientation du peuple français . C' est la raison pour laquelle il faut être clair dès le début , en disant d'abord &laquo; on va parler à tout le monde et pas seulement à ceux de gauche &raquo; . Bref , je vais être aussi synthétique que possible , Pierre m' a dit : &laquo; ça ne peut pas fonctionner s' il n' y a pas un rassemblement de la gauche &raquo; . Je ne crois pas du tout à cette thèse . Je le lui ai dit . Je ne crois pas que ce soit en rassemblant des étiquettes , des partis et des états-majors , qu' on va créer un enthousiasme populaire suffisant pour passer la barre . Je crois même le contraire . Si on donne l' impression qu' on est retournés dans les magouilles et les ententes en demi-teinte , on va encore aggraver la désorientation . Donc nous avons besoin de partir de loin . Nous nous sommes quittés sur un constat de désaccord de stratégie politique . Pierre a établi son calendrier pour le mouvement communiste , et j' ai établi le mien pour ce que j' allais entreprendre . Et c' est comme ça que j' ai décidé de démarrer le 11 février . Pourquoi cette date-là , je peux le dire maintenant , parce que nous étions dans un moment où apparaissait en même temps la candidature de Marine le Pen et où Sarkozy , s' exprimant quelques jours auparavant , avait clairement fait comprendre qu' il était candidat . Donc la scène était entièrement occupée par la droite . Et nous , notre famille politique , on serait là en train de se disputer les uns avec les autres , parce que une primaire c' est fondamentalement un exercice mortel de surenchères les uns sur les autres . J' ajoute donc à cette occasion , que le système des primaires n' est qu' apparemment démocratique . Il ne l' est pas en réalité . C' est un tamis social . Seuls les centre-ville urbains et aisés participent dans la plus grande partie des cas à des élections primaires . Et une infime partie de l' électorat . Par exemple les primaires américaines … Tout le monde a dit &laquo; , c' est formidable , c' est une culture … &raquo; . 9 % des électeurs ont voté dans toutes les primaires , c'est-à-dire démocrates et républicaines . C' est rien du tout . Et chez nous où il n' y a pas cette tradition , c' est encore moins . Par conséquent , je ne prends que cet aspect des primaires , je ne crois pas à cette méthode . Et d'ailleurs , ceux qui ont lancé l' idée d' une primaire de toute la gauche , Cohn-Bendit et le journal Libération , ne sont pas des références pour nous , en tout cas . Et d' entrée de jeu , Cohn-Bendit a dit : &laquo; Une primaire , c' est parfait parce que c' est le seul mode de désignation où Mélenchon et Pierre Laurent n' ont aucune chance &raquo; . Comme ça c' est clair . Et à peine n' avait -on même pas commencé , que déjà Julien Bayou disait &laquo; Ah oui , mais attention , si le candidat qui l' emporte est contre l' Europe , alors ça posera un problème &raquo; , parce que pour lui , être critique à l' égard de l' Union européenne , c' est être contre l' Europe . Ce n' est pas sérieux . Donc j' ai démarré le 11 février . Mais je demande aux communistes de faire appel à leur mémoire : c' est exactement la même méthode qu' en 2011 , exactement la même , avec un mois de retard . Le 21 janvier 2011 , j' ai proposé ma candidature . Tout seul . En tout cas en apparence . Comme cette fois -ci , je la propose tout seul et hors-cadre de partis . Pas &laquo; les partis , &laquo; de partis . Moi-même je suis membre d' un parti politique . Je l' ai été toute ma vie . J' ai été membre du Parti socialiste puis membre du Parti de gauche . Je n' ai pas cessé d' être membre de mon parti , je cotise . Donc je ne suis pas contre les partis , je n' ai pas cette sottise … Je suis &laquo; . Pourquoi hors-cadre ? Parce qu' il faut que tout le monde puisse investir la campagne . Je ne peux pas me présenter en disant que je suis l' homme de quatre , cinq , six , sept partis . Non . Voilà pourquoi , à la fois je propose ma candidature , et je crée le mouvement d' appui à ma candidature , la France insoumise . Vient qui veut : membres de partis , pas membres de partis , des gens qui ont voté autrement , des gens qui n' ont pas voté dans les précédentes élections … Il faut faire cette ouverture grand-angle . Et nous , qui sommes des militants , engagés politiquement toute notre vie , nous devons apprendre à traiter avec l' informel , avec l' état du pays tel qu' il est . Ce n' est pas à nous de n' accepter que le peuple qui nous convient , celui qui a l' habitude de se référer aux partis politiques . Non , il y a plein de gens qui ne sont adhérents à rien . Et alors on leur dit &laquo; non , vous ne comptez pas &raquo; . Bien sûr que non . Donc c' est exactement le même calendrier , sauf que j' ai un mois de retard . J' ai proposé ma candidature le 21 janvier 2011 , tout seul , je ne l' ai pas fait au nom du Parti de gauche , je ne l' ai pas fait au nom du Parti communiste . Le Parti communiste a engagé une procédure interne , mais bien plus tôt que cette fois -ci . Et les communistes ont tranché et ils ont voté à la date du 18 juin . Le 18 juin de l' année 2011 , les communistes avaient pris leur décision , et le 18 juin , le soir , j' étais sur TF1 pour annoncer que dorénavant j' étais candidat investi par , à l' époque , le Parti de gauche , le Parti communiste et la Gauche unitaire . C' est ça le cadre de l' union . Ce n' est pas la même chose aujourd'hui : nous savons que les cartels de partis sont condamnés . Parce que nous avons fait fonctionner le Front de gauche comme un cartel de partis . C' est l' expérience concrète qui nous montre que ce n' est pas la bonne méthode . Ça ne veut pas dire qu' on renie ce qui a été fait par le Front de gauche . Mais nous devons porter plus loin l' expérience acquise . L' expérience acquise , c' est que non seulement il faut intégrer tous ceux qui ne veulent pas prendre la carte d' un parti mais qui veulent participer aux campagne , mais aussi tous ceux qui n' avaient jamais pensé faire de la politique . Voici le cran supérieur . Ensuite , les communistes ont leur calendrier . Je m' incline . Je l' accepte . M' avez -vous entendu une seule fois dire &laquo; ce n' est pas bien , je ne suis pas d'accord &raquo; . J' ai pu exprimer des regrets , dire &laquo; moi j' aimerais avoir l' aide des communistes aujourd'hui , des militants communistes &raquo; , mais je vais vous dire les choses comme elles sont : il y a déjà des milliers de communistes engagés dans la France insoumise , qui créent des groupes d' appui , qui collent des affiches et qui discutent surtout , qui discutent avec leurs voisins , leurs collègues de travail , parce que c' est comme ça qu' on va gagner . Et je le dis parce que je viens de dire &laquo; coller les affiches &raquo; , mais ce n' est pas le plus important . Le plus important , c' est le contact personnel . Chaque personne doit se faire une liste de dix autres personnes à convaincre et à accompagner jusqu' au jour du vote . Donc nous sommes dans le même calendrier , avec la même forme , avec les mêmes mots , croyez -moi , je les ai pesés et c' était les mêmes que la fois précédente : &laquo; Je propose ma candidature &raquo; . Et puis au moment où on est arrivés à plus de cent mille appuis , il ne faut pas non plus prendre les gens pour des imbéciles , ils ont bien compris que j' étais candidat . Et bien , je l' ai dit . Quand ? Le 5 juin . Où ? Place Stalingrad où avait commencé la campagne la fois d' avant . On a appelé le rassemblement après la fin du congrès communiste . Le congrès communiste s' achève vers 13h , 13h30 , le rassemblement commence à 15h . Il n' y a jamais eu de chevauchement . Je n' ai pas compris pourquoi on me faisait ce mauvais procès . J' observe d'ailleurs qu' on ne fait pas le même aux frondeurs qui se réunissent à la Rochelle pendant que nous sommes tous ici à la Fête de l' Huma qui est l' endroit où toute la gauche a l' habitude de se retrouver . Donc , pourquoi l' ai -je mis à cette date-là ? Parce qu' il y avait le congrès communistes et que je pouvais penser que dans le congrès communiste , comme la fois d' avant , une décision serait prise qui permettrait de converger . Elle ne l' a pas été . Mais qui peut penser que je vais mener un calendrier de campagne en expliquant aux gens &laquo; écoutez , rien n' est sûr , attendez que , je ne sais pas quand , le 5 novembre , les communistes commencent leur procédure interne … &raquo; . Car que je connais les statuts du Parti communiste . Le 5 novembre , ce n' est pas la prise de décision . Le 5 novembre , ça va être la décision qui va déclencher la procédure interne . Nous allons donc attendre le mois de décembre pour entrer en campagne , au milieu de la confusion des primaires socialistes ? Il n' en est pas question ! Par conséquent mon calendrier suit son cours . Et en cours de route bienvenu à tout le monde . Regardez la démarche du programme . Le programme , je ne pars pas de rien . Je suis stupéfait de voir que l' on puisse agir comme s' il n' y avait pas eu de programme la dernière fois . Il y a un programme : l' Humain d'abord . Il a recueilli quatre millions de voix . Il faut l' actualiser depuis 2012 . C' est un mot qui dans le passé nous a amenés à quelques déboires . Qui va actualiser ? Et bien d'abord nos électeurs puisque ce sont eux qui l' ont voté la dernière fois , ceux qui veulent s' engager dans la campagne , tous ceux qui veulent s' engager . Il y a eu plus de trois milles contributions . Je ne dis pas que ce soit un chiffre extraordinaire , mais trois milles contributions . On a fait deux synthèses en cours de route , pour montrer aux gens où on en était , et rebondir . Nous avons fait des auditions de spécialistes . Et là nous sommes entrés dans un dialogue avec les groupements politiques qui décident de participer à la France insoumise . Le groupe des socialistes dissidents , de la Nouvelle gauche socialiste , a déposé un document de 75 pages de propositions . Le Parti de gauche a amené ses 1000 fiches . Des communistes qui participent à la campagne amènent leurs contributions . On discute . Mais ce n' est pas une discussion de parti à parti , d' organisation à organisation . On essaie de voir comment tout ça , par rapport à la formule actualisée du programme l' Humain d'abord , ça colle ou pas . On ne peut pas coller bout à bout ce que tout le monde veut , ce n' est pas possible . Il faut qu' il y ait une cohérence . Moi je suis très attaché à la cohérence économique , à la cohérence écologique du programme et à la cohérence républicaine . Donc tout ça compte beaucoup . Cette méthode va s' achever à l' occasion de la convention du mois d' octobre , qui se tient les 15 et 16 octobre . Pourquoi se tient -elle à ce moment-là ? Mais parce que j' ai des délais pour imprimer le livre pour qu' il soit prêt pour les cadeaux de Noël . Nous à Noël , on va offrir le programme de la campagne . Alors je vous dis un scoop , j' ai quand même quelques petits privilèges comme candidat , au moins celui de choisir quelques-uns des slogans … C' est moi qui ai choisi la France insoumise . Je cherchais une expression qui soit dans l' esprit de celle de 2012 . Vous vous rappelez , on avait fait &laquo; la France , la belle , la rebelle &raquo; , c' était pris dans la chanson de Jean Ferrat . Et on cherchait un mot qui corresponde au moment dans lequel on est . C' est l' insoumission qui brise l' ordre établi , le consentement à l' ordre , la résignation . Et la dernière période a été remplie d' insoumis , lanceurs d' alerte , ouvriers et ouvrières qui résistent dans leur entreprise . Bref , j' ai choisi &laquo; France &raquo; que je trouvais que ça parlait à tout le monde à la fois : aux syndicalistes , mais aussi à l' homme ou à la femme qui se lève chaque matin pour maintenir la cohésion de la famille , pour aller au boulot ou pour amener les gosses à l' école alors qu' il n' y a plus de boulot , parce qu' il faut tenir bon . Voilà , j' aimais ça . Le programme , on ne va pas le rappeler l' Humain d'abord , quoique c' était magnifique , mais , par respect pour ceux qui auraient voté l' Humain d'abord ou les organisations qui auraient adopté l' Humain d'abord et qui ne se retrouveraient pas dans ma campagne , je le comprends , je n' ai pas envie d' avoir des polémiques avec les uns et les autres , on va donc changer le titre . Je vais proposer que ça s' appelle &laquo; en &raquo; . &laquo; en &raquo; parce que c' est quelque chose que je partage avec les communistes et notamment les réflexions de leur dernier congrès . Nous autres , le grand courant collectiviste et progressiste , nous devons de nouveau et mieux motivé que jamais , faire partager l' idée que l' Humanité doit d'abord se préoccuper de ce qu' elle a en commun , à commencer par l' écosystème , le seul qui permette sa survie . Car l' écosystème est mis en danger par le productivisme . Et le productivisme est , lui , animé par la politique de l' offre du capitalisme financier . La planète va mourir , la civilisation humaine va mourir sous les coups du capitalisme financier et du productivisme . C' est pourquoi il s' agit d' appeler chaque personne à réfléchir , quoiqu'elle ait voté dans le passé , l' appeler à réfléchir en tant qu' être humain . Réfléchissez : notre avenir , nous l' avons en commun . Que vous soyez riche ou pauvre . Et vous autres , les riches , vous feriez bien de vous poser des questions . Vous ne le resterez pas longtemps , riches , à ce tarif là , avec des événements climatiques extrêmes . Et à quoi bon être riches , à quoi bon ? Pourquoi avoir pour rêve d' être milliardaire , comme propose Macron ? En quoi est -ce un rêve honnête ? Vous savez bien que celui qui a un milliard , il n' a qu' un rêve : en avoir un deuxième . A quoi ça sert ? Et celui qui est millionaire ne peut avoir qu' un rêve : avoir un deuxième million . Pour quoi faire ? Pour quoi faire ? Quel est le sens de cette accumulation ? Est -ce que vous êtes plus heureux quand vous passez d' un million à deux millions ? Est -ce que vous aimez d' avantage ? Est -ce que vous êtes aimé d' avantage ? Et puis pour quoi faire ? Pour acheter quoi ? Pour vivre comment ? Je pense que la richesse doit être interpelée aussi dans son immoralité fondamentale . Eux , ils pensent que la cupidité est un aiguillon de l' activité humaine . Reconnaissons que l' intérêt est un aiguillon , mais pas la cupidité , pas l' accumulation . Nous devons interpeler le sens moral de la richesse . Voilà pourquoi je pense qu' il faut appeler notre programme &laquo; en &raquo; : dans chaque personne , ne voyons d'abord que l' être humain . Au fond , c' est aller au bout de la logique de l' Humain d'abord . L' Humain d'abord dit &laquo; bon , on commence par ça &raquo; . Justement commençons par ça . Nous avons l' avenir en commun . Je vais essayer d' expliquer ça , je le fais à la Fête de l' Huma . C' est à la Fête de l' Huma que je l' annonce . Alors on me dit : &laquo; c' est toujours toi qui décides &raquo; . Il en faut bien un . Comment vous croyez que ça se passe d' habitude ? Il y a des gens qui parlent . Et à la fin les autres qu' est -ce qu' ils prennent ? Un communiquant pour décider . Pas moi . Je suis un militant politique , c' est moi qui fixe tous les mots d' ordre . En 2012 , on discute , et puis à la fin on tranche . Je ne dis pas que c' est génial . Je ne vous dis pas que tout le monde est d'accord et dit &laquo; ah c' est formidable &raquo; , la France insoumise , il n' y a rien au-dessus . On aurait pu appeler ça autrement . Mais maintenant c' est parti . Il faut arrêter l' ergotage permanent , couper les cheveux en quatre tout le temps . Les gens ont besoin de comprendre . Ils ne comprennent rien si nous mêmes on est les premiers à dire &laquo; ah ce qu' on a trouvé ça ne vaut rien . Finalement il y avait mieux que ça &raquo; . Alors on me propose des titres à rallonge . Je vous donne cet exemple parce que je sais que je m' adresse à des militants . L' enjeu du militantisme , c' est l' engagement des autres , leur conscience , l' éducation collective . Si les partis ont une fonction , c' est bien celle-là , d' être la mémoire des luttes , d' être les porteurs de l' expérience et d' être des éclaireurs , pas une avant-garde . A mon avis . Il y a des camarades qui sont avant-gardistes , je le comprends . Eclairer , ça veut dire : vous mettez la lumière qui permet de se repérer à chacun individuellement . Pas le guide suprême qui dit &laquo; allez on va tous par là &raquo; . Non regardez bien , les gens , observez ce que nous vous disons , et ensuite tirez vos conclusions . Mais écoutez ce qu' on vous dit . Ne vous laissez pas bourrer le crâne par la télé et par les grands médias . Ecoutez . Vous choisirez à la fin . Mais entendez ce que nous avons à dire . Et vous voyez bien comment l' ennemi concentre ses coups : son premier objectif c' est d' empêcher qu' on entende , c' est de vous mettre une mauvaise étiquette . Ils décident qu' ils me mettent à l' extrême gauche , mais je ne suis par d' extrême gauche . Ce n' est pas que je méprise l' extrême gauche . C' est que je ne suis pas d' extrême gauche . Il y a une extrême gauche . Ce n' est pas juste de faire croire que l' extrême gauche c' est moi . Non , l' extrême gauche c' est Nathalie Arthaud , c' est Philippe Poutou , c' est aussi le Parti communiste par certains aspects . Mais moi je ne suis pas membre du NPA , de Lutte ouvrière , et mon programme ce n' est pas ça . Donc ce n' est pas juste déjà de leur enlever le droit à l' existence , on les invisibilise si on fait comme ça . Et puis après on m' invisibilise en me classant dans une catégorie qui n' est pas la mienne , je n' ai rien à faire là-dedans . Vendredi 9 septembre , je suis allé à un truc qui s' appelle Produire en France . Vous savez ce que retient le Figaro ? Que , soi-disant , on aurait dit qu' on était tous les meilleurs pour produire en France . Mais pas du tout , c' était pas ça la compétition . Et que moi j' aurais proposé surtout des nationalisations . Ils m' ont mal écouté . Ils m' ont très mal écouté . On va remettre les choses au point . Moi j' ai dit aux patrons qui étaient là : je ne viens pas pour vous dire ce qui va vous faire plaisir , je perdrais mon temps . Je viens vous dire ce qu' il va falloir que vous fassiez pour vous rendre utiles à la Patrie . Il n' y en a pas un qui a sifflé . Personne n' a dit &laquo; mais qu' est -ce que c' est que cette manière de nous parler ? &raquo; . Je dis &laquo; pour tenir les comptes de vos entreprises , vous êtes assez grands , sinon , ce n' est pas la peine de faire patron . Mais moi je vais vous dire ce dont le pays a besoin . Est -ce que vous êtes capables de prendre votre place là-dedans ? Nous avons besoin de 400 000 personnes pour faire de l' agriculture paysanne relocalisée , 300 000 pour l' économie de la mer , 900 000 pour sortir du nucléaire . Alors ? Où je les trouve ? Ils n' y sont pas aujourd'hui , il faut les former . Et vous , vous êtes prêts à être les entreprises qui vont suivre ? Moi je vous donne de la visibilité , je vous donne de la stabilité et de la sécurité . Et vous vous n' allez pas faire que pleurer , pleurnicher que vous avez trop de charges , trop de ceci , trop de cela . C' est bon , quoi . En tout cas si vous voulez faire ça , allez en voir un autre . Mais si vous voulez entendre ce qu' on vous dit , vous verrez que votre avenir y a son intérêt , parce que nous allons faire une société d' économie mixte , on ne va pas faire le socialisme en cinq ans &raquo; . Qui est -ce qui propose de le faire en cinq ans ? Qu' il vienne , qu' on m' explique quelle est la stratégie pour faire ça ! Moi j' en ai une : l' Assemblée constituante , pour faire la transition . Bon voilà , alors merci si ce message arrive aux communistes , je serais enchanté , je sais qu' il n' est pas loin de leurs propres convictions , et pour le reste , ils vont décider . Et leur décision s' imposera à qui ? Aux communistes , pas à moi . Non . En effet , c' est une bataille . Je la mène . Je trouve qu' elle est intéressante en tant que combat , et j' appelle les militants à réfléchir à ça : sans cesse de nos faiblesses faisons une force . Je préfèrerais avoir mille signatures tranquille . Je ne les ai pas . Qu' est -ce que je fais alors ? Je me prends la tête dans les mains ? Je me mets à pleurer ? Et je gémis et me lamente ? Non , j' affronte ! C' est extrêmement pédagogique . Ça permet aux gens de comprendre que dans ce pays pour pouvoir être candidat , il faut d'abord avoir l' autorisation de notables qui n' ont rien demandé . Donc ça aide à la construction d' une conscience politique plus affinée . Maintenant , bon , on a dépassé les deux cents . Dans ces plus de deux cents , il y a un peu plus de 33 , 34 élus communistes . D'ailleurs cette nuit il y en eu d' autres . Et aujourd'hui dans la journée certains vont venir me voir … Mais oui , il y a eu de la rétention de signatures . Et puis il y a beaucoup d' élus communistes qui disent : &laquo; Bon ben , moi j' attends la décision du Parti &raquo; . Je le comprends . Ca me coûte , parce que si je les avais , on serait plus tranquilles . Et surtout , la banque prêterait des sous . Parce que la campagne sera remboursée : je pense évidemment faire beaucoup plus de 5 % . Donc à partir de là , j' ai besoin . Je ne peux pas faire une campagne qu' avec des bénévoles et des retraités , ce qui est le cas aujourd'hui . Attention , les bénévoles ne sont pas des amateurs . Mais il y a quand même quelques permanents qui doivent être dans la campagne , il y a du matériel à gérer … Donc , qu' est -ce qu' on fait ? On lance la campagne de parrainages et on lance la campagne pour les dons . Et on va gagner comme ça . Il n' y a pas de martingale , il n' y a pas de solution raccourcie , miraculeuse . Les communistes , ils viendront et il n' y aura pas qu' eux . Je pense que d' ici au 5 novembre on sera très avancés vers les cinq cents signatures . Et alors après encore plus , à mesure qu' on va avancer dans la campagne . Puis , je vais aller voir le Président de l' Union des maires , monsieur Baroin , et je vais lui dire : &laquo; mais monsieur , vous êtes de droite , vous , sauf erreur , vous êtes le Président de l' Union des maires . Alors qu' est -ce que passez votre temps à parler de démocratie et de tout ça , et vous faites de la rétention . Je vous demande en tant que Président de l' Union des maires d' appeler les maires à une réflexion , parce qu' ils ne sont pas là pour empêcher les gens d' être candidats , mais pour empêcher qu' à l' élection des candidatures folkloriques apparaissent . C' est ça leur mission &raquo; . J' ai fait quatre millions de voix la dernière fois , je ne suis pas une candidature folklorique . Je représente un courant d' opinion dans le pays . Vous pouvez signer pour moi , que vous soyez sans étiquette , que vous soyez communistes , socialistes . Les socialistes , ils vissent à double tour , ils menacent leurs élus , leur font peur . Vous verrez , à la fin c' est nous qui aurons le dernier mot , parce que contre des militants et des gens engagés , il est rare qu' il y ait une force suffisante pour nous empêcher de passer . Ça se fera . Je préfèrerais que ça se fasse avec les élus communistes , c' est évidemment mon souhait . Vous imaginez bien que je préfère être parrainé par une majorité de communistes que par une majorité de droite , mais je prendrai les signatures de là où elles viennent . Et ça m' ennuierait que les communistes arrivent les derniers , ça ne le ferait pas . ",
    "Nous n' avons pas la même vision des priorités ni de la cohésion nationale . Son programme s' appelle &laquo; Tout pour la France &raquo; , en vérité , c' est tout pour la finance . Il utilise &laquo; l' identité &raquo; comme un bélier pour abattre la résistance morale des gens . Son but est de diviser le peuple pour gagner la guerre sociale qu' il veut mener . Car son programme compile les mesures désastreuses pour les acquis sociaux : retraite à 64 ans , suppression la durée hebdomadaire légale du travail … C' est un retour au 18 ème siècle . C' est un écran de fumée . Comme , il ne peut se présenter devant les Français avec pour seul programme la régression sociale aux ordres de la Commission européenne , Nicolas Sarkozy l' agrémente de la haine pour tous . C' est d'abord le résultat d' un harcèlement inouï . Les responsables publics , et le système médiatique ont perdu tout sang-froid . L' origine des attentats terroristes est dans la guerre qui se mène au Moyen Orient . Là , il n' est pas question de religion mais d' oléoducs et de gazoducs . Mais dorénavant aussi tous les fous du monde peuvent commettre des crimes en se réclamant de l' islam même s' ils ne le pratiquent pas . Les services de sécurité font tout ce qu' ils peuvent . Mais la question des moyens humains à déployer reste posée . Je préfère des agents infiltrés présents sur le terrain à l' espionnage électronique généralisé par lequel on a cru pouvoir régler ce problème . Et bien sûr il faut arrêter la guerre . Le jour de l' entrée en dette écologique de la planète avance au 8 août mais on discute des tenues de bain ! On détourne l' attention de l' essentiel : l' argent roi . C' est lui qui pourrit nos vies , la planète et la paix . Là , la France bas des records : ce pays distribue le plus de dividendes en Europe . Il n' y a pas d' argent pour les payes et les services publics mais les actionnaires se gavent . Non . Il est candidat à la primaire du PS . Moi je serai candidat devant le pays jusqu' au bout . Sa sévère condamnation du bilan de François Hollande me sert . Elle est d'autant plus convaincante qu' il était aux premières loges . Ajouté à Benoît Hamon et Cecile Duflot , on voit ainsi pour la première fois trois anciens ministres essayer d' empêcher le président qui les a nommés d' être réélu tant il est nul . Non . A la fin , je serai là . François Hollande ne propose pas une candidature de gauche puisqu'il a continué tout le programme de Nicolas Sarkozy . Son ennemi devait être la finance et il a commencé par lui faire cadeau de 40 milliards d' euros . Si quelqu'un est capable de retrouver la trace de ces 40 milliards dans l' activité réelle , qu' il se signale . Qu' est ce qui est de gauche dans le bilan de François Hollande ? Il a fait pire que Nicolas Sarkozy . Mais le plus navrant , c' est que de Hollande à Sarkozy en passant par Juppé , Valls et Macron , tous ont comme projet de continuer à appliquer de manière encore beaucoup plus violentes des recettes qui ont tout détruits ces 20 dernières années . Ce sont les candidats de la Commission européenne . Non . La situation de l' emploi ne s' améliore pas . Nous n' observons que de petites fluctuations autour du point zéro . Ca montre que l' organisme n' est pas mort . Mais tant qu' on ne conduira pas une politique de relance , l' emploi ne repartira pas . Cette relance doit être écologiste . Je propose un choc de 100 milliards d' euros à injecter dans la planification écologique . Et c' est parfaitement finançable . On se guérira le pays par la démocratie . La présidentielle , ce n' est pas le PMU . Si on en reste à faire des paris sur le résultat sans se préoccuper des enjeux , il n' y a plus de démocratie . Le FN c' est le diable de service . Il est utilisé pour faire peur et rameuter . Mais pour quoi faire ? Continuer comme avant ? Mais c' est insupportable . Il n' y a pas une dispersion de la gauche , mais une opposition de programme . Pour certains , le problème c' est le peuple , pour moi c' est la solution . ",
    "Non , ce n' est pas un procès d' intention . François Fillon est le candidat des assureurs contre la Sécurité sociale . C' est ce qui ressort de la lecture de son programme . Tout le monde pourra aller vérifier ce que je vais relever . Le système Fillon repose sur le recul des remboursements par la Sécurité sociale et l' encouragement aux assureurs privés . Un autre indice nous a été donné la presse . France Inter affirme ainsi que François Fillon aurait déjà choisi son futur ministre de l' Économie et des Finances en cas de victoire à la présidentielle . Il nommerait à Bercy Henri de Castries , l' ancien PDG de l' assureur privé Axa pendant 16 ans ! Ce ne peut être un hasard . Henri de Castries n' a quitté la présidence de ce groupe qu' il y a quelques mois , pour devenir administrateur de la banque HSBC . Précisons , pour situer les perceptions de la vie que peut avoir le personnage : lors de sa dernière année chez Axa , en 2015 , il gagnait 5,4 millions d' euros de salaire ! Que peut -il comprendre aux raisons des 30 % de gens qui n' osent plus se soigner parce qu' une ordonnance , ça coûte trop cher … Le conflit d' intérêt serait immense si ce personnage devenait demain ministre des Finances chargé de faire voter le budget de la Sécurité sociale et la fiscalité des clients des assurances privées . Reste une évidence , Castries ou pas : le programme de François Fillon , c' est ouvertement et officiellement le début de la fin de la Sécurité sociale . À commencer par la fin de l' assurance maladie pour tous . Fillon l' a écrit noir sur blanc dans son programme pour la santé . Il veut &laquo; focaliser l' assurance publique universelle ( c'est-à-dire la Sécu ) sur des affections graves ou de longue durée , et l' assurance privée sur le reste &raquo; . Vous voilà prévenus . Le système collectif et solidaire actuel où chacun cotise selon ses moyens et reçoit selon ses besoins serait donc abandonné . Ou plus exactement limité aux seules &laquo; affections graves &raquo; . Et le reste ? Ce sera nécessairement à charge d' une assurance complémentaire . Bien sûr , pour faire bonne figure , Fillon dit que &laquo; les moins favorisés ne pouvant accéder à l' assurance privée bénéficieront d' un régime spécial de couverture accrue &raquo; mais sans dire qui cela concernera . Et quand on l' entend critiquer l' assistanat , on le voit mal élargir l' accès à la Couverture maladie universelle , la CMU . Il faudrait donc payer plus cher pour être moins remboursés quand on va chez le médecin ou chez le pharmacien et les inégalités se multiplieraient . C' est aussi un programme absurde du point de vue sanitaire . Car les &laquo; affections graves &raquo; ne le sont pas toujours depuis le début . Et les autres affections ne sont pas bénignes de la même manière pour tous , ni suivant les âges . Mais dans tous les cas , en se soignant de bonne heure , on évite souvent bien des complications et des aggravations . Il est donc totalement stupide de moins rembourser les soins courants . Le renoncement aux soins des plus pauvres fera exploser les dépenses de prises en charge aux urgences ou une fois les maladies devenues plus graves , c'est-à-dire plus douloureuses et dangereuses pour les patients et plus chères à prendre en charge . La santé n' est pas le seul domaine où François Fillon veut donner la main aux assureurs privés . Il tient le même discours à propos de la prise en charge du vieillissement de la population . En matière de retraite , il propose de repousser l' âge de départ à 65 ans . Mais aussi de &laquo; mettre en place un étage de retraite par capitalisation &raquo; , c'est-à-dire où chacun paye dans son coin une cotisation privée avec un cadeau fiscal à la clef . Naturellement cela n' a jamais garanti un montant de retraite où que ce soit dans le monde où ce régime est appliqué . C' est si vrai que partout où le retour dans le régime général a été proposé aux assurés et partout où le cadeau fiscal a été supprimé , le régime de capitalisation s' est volatilisé . À quoi bon recommencer une expérience qui ne marche pas ? Face à la dépendance aussi , François Fillon récuse le mécanisme de la solidarité et prône le chacun pour soi . Son programme prévoit d' &laquo; encourager la généralisation d' une couverture assurantielle dépendance privée sur une base volontaire par des contrats labellisés avec des incitations fiscales qui permettront aux particuliers de déduire de leurs impôts une part de leur cotisation à une assurance dépendance &raquo; . Ceux qui peuvent payer auront droit à une assurance contre la dépendance ? Et les autres ? Continueront -ils à souffrir en silence comme aujourd'hui face au manque de places accessibles dans des établissements pour personnes âgées dépendantes ? Devront -ils faire appel à la charité ? Les familles continueront -elles à se saigner en argent , temps et énergie pour aider les parents ? Ce programme a un but . Transférer au privé les milliards d' euros de cotisations maladie et retraite aujourd'hui gérés par la Sécurité sociale et offrir aux assureurs les milliards en vue pour financer la dépendance des personnes âgées . Car c' est un marché où certains voient bien quels juteux profits sont possibles . En cela , la rumeur selon laquelle Henri de Castries deviendrait ministre de l' Économie et des Finances de François Fillon est éclairante . Henri de Castries , est un symbole de l' oligarchie . Il coche toutes les cases de la caste : issu de la noblesse , diplômé de l' ENA dans la même promotion &laquo; Voltaire &raquo; que François Hollande dont il est l' ami , hier soutien de Nicolas Sarkozy , aujourd'hui ami de François Fillon . Il a aussi été &laquo; Young leader &raquo; de la French American foundation en 1994 , et préside le comité de direction du groupe Bildelberg , qui réunit le gratin de l' oligarchie mondiale . Nous défendons un tout autre modèle : la sécurité sociale . Le programme l' Avenir en commun prévoit tout le contraire de M. Fillon . Nous proposons d' améliorer la prise en charge des soins par la Sécurité sociale jusqu' à leur remboursement à 100 % . Nous voulons rétablir la retraite à 60 ans , réduire la durée de cotisation exigée pour avoir une retraite complète et renforcer le régime de retraite solidaire par répartition . Et face à la dépendance , nous défendons notamment la construction de 50 000 places en établissements pour prendre en charge les personnes dans le besoin dans un réseau public et abordable , et la baisse de ce qui reste à la charge des familles pour les personnes qui continuent de vivre chez elle . Ce modèle est plus juste et moins coûteux . Plus juste car dans le système de la sécurité sociale , le montant des cotisations ne dépend pas de l' âge ni de l' état de santé et que la protection dont on bénéficie ne dépend pas de ses moyens financiers mais de ses besoins : chacun est remboursé de la même façon par exemple . Chacun a un aperçu de ce qu' est un système d' assurance privée individuelle , à travers sa complémentaire santé s' il a les moyens de s' en payer une . Surtout , la Sécu c' est moins cher ! Il suffit de comparer les dépenses de santé entre la France et les États-Unis . En France , ces dépenses de santé représentent 11 % de la richesse annuelle du pays . Elles sont essentiellement supportées par l' assurance maladie . Aux États-Unis , les dépenses de santé sont essentiellement privées . Et elles coûtent plus cher puisqu'elles atteignent 16,5 % de la richesse du pays . Tout cela pour des résultats sanitaires mauvais là où la France reste une référence malgré les mauvais coups portés à notre système solidaire et à l' hôpital public . ",
    "Ma candidature est d'abord celle d' un programme , dont la cohérence doit être gardée , protégée , nourrie avec soin . Un programme ce n' est pas un copier-coller de toutes les revendications que l' on a entendues . Il faut qu' il y ait un fil conducteur , qui s' applique dans tous les compartiments de la proposition . Ce fil conducteur , c' est l' intérêt général humain qui est mis en cause par la menace qui pèse sur la civilisation humaine , la destruction du seul écosystème compatible avec celle -ci . Le fil conducteur qui organise tout le reste , c' est cette vision des biens communs , de ce que nous avons en commun , de notre humanité partagée . De l' Humain d'abord à l' Avenir en commun Et bien je peux dire que le candidat a quelques privilèges , - on va appeler ça comme ça – il en a un , c' est de trancher parmi la foule des propositions qui lui sont faites , en matière de mots d' ordre ou de slogans . Et bien , j' ai tranché , et j' ai choisi que l' on appelle notre nouveau programme , qui s' appelait L' Humain d'abord , j' ai choisi qu' on aille au bout de la logique de &laquo; l' humain d'abord &raquo; . Si c' est l' humain d'abord , alors l' humain est dans tout homme , toute femme , quelle que soit sa condition sociale , quelle que soit son éducation , quelle que soit sa religion – puisqu'on n' arrête pas d' en parler , et même s' il n' en a pas , ce qui est la masse du peuple français . Et bien ce qu' ils ont en commun c' est cela que nous allons valoriser , c' est de cette manière que nous allons interpeller chacun , sans nous occuper de savoir comment il a voté avant , ce qu' il faisait avant , ce qu' il disait avant . À cette heure , voici que s' avance la plus grande des menaces , nous pouvons y faire face et relever ce défi ce qui serait extraordinaire , la dette écologique est dorénavant parvenue au 8 août ce qui signifie qu' à partir du 9 août au matin , la terre se voit prélever plus que ce qu' elle ne peut reconstituer . Dès lors l' intérêt général est appelé [ … ] Cela signifie que nous devons aller au bout de l' humain d'abord , le bout de l' humain d'abord , c' est d' appeler en toute personne , cette communauté de la condition humaine . Le programme s' appellera : L' Avenir en commun . Je souhaite que vous compreniez le mot jusqu' au bout , car il n' y a pas d' avenir en commun pour les êtres humains sans tous les éléments qui constituent la biosphère . Il n' y a pas d' avenir en commun pour les êtres humains sans toutes les espèces animales et végétales qui ont permis l' émergence de la conscience . L' Avenir en commun est un projet global qui concerne tous les aspects de la biosphère et de sa conservation . Cet avenir en commun , cet intérêt général humain qui est appelé , c' est celui qui me permet de m' adresser à tous , en toutes circonstances . Le programme de la France insoumise [ … ] Nous proposons un programme , nous proposons une démarche , nous proposons un mouvement , la France insoumise , dans lequel tout le monde est appelé . Je suis un candidat hors parti , mais je ne suis pas un candidat contre les partis , je suis moi-même membre d' un parti . Mais il faut que tout le monde puisse participer à cette campagne , apporter son intelligence , sa capacité individuelle sans être obligé , jamais , de se soumettre à telle ou telle règle ou bien d' adhérer à telle ou telle organisation partisane , c' est cela ce que nous cherchons à faire , et vous voyez notre force c' est là qu' elle est . C' est 125 000 personnes , qui ne demandent rien à personne et qui font campagne dès aujourd'hui à nos côtés . Je ne suis pas capable de vous dire exactement tout ce qu' ils disent , mais je suis certain qu' ils le disent plus efficacement et avec plus de sensibilité que ne le ferait n' importe quelle organisation qui aurait décidé de passer un discours dans un moule unique , même si parfois ce discours est nécessaire pour penser . La solidarité pour sortir de la peur Il faut compter sur le peuple , sur son intelligence , son humour , sa dérision , sa finesse pour convaincre et entraîner . Car il faut convaincre et entraîner . Il faut sortir de la sinistrose , il faut sortir du déclinisme , il faut sortir de la peur qui est le seul nouveau lien social que la droite ait proposé au peuple pour se rassembler . J' y reviens un instant : permettez -moi que je m' y attarde car cela concerne la morale , cela concerne la manière de vivre ensemble . Autrefois , le grand lien social étaient la solidarité , la fraternité et personne ne trouvait à y redire quand bien même était -il de droite . La cupidité était considérée comme une tare ; aujourd'hui elle est considérée comme le premier moteur de l' activité . Chacun est amené à développer son égoïsme personnel comme une affirmation de soi . Et ce qui était autrefois le lien commun , qui fait notre vie quotidienne , c'est-à-dire l' amour , c'est-à-dire l' intérêt pour les autres sans lequel nous n' avons pas d' existence personnelle , ce lien était le lien de la société . Ils l' ont remplacé par un autre lien : la peur . [ … ] La peur tout le temps . Nous , nous remettons au centre de commande , d' autres valeurs . Et ce sont elles qui , en fonctionnant , vont nous donner l' énergie de fonctionner autrement . ",
    "Depuis le Brexit , le poste de commandement de l' Union européenne est une salle de bal vide et froide . Les lampions sont éteints et ça sent le tabac froid . Les guirlandes sont par terre et les nappes froissées . C' est clair : on ne dansera plus . Que faire d' autre ? Que proposer comme nouvel arguent de promotion puisqu'on ne peut plus dire à personne &laquo; venez chez nous , c' est une maison sympathique ! On est tous des gentils qui s' entraident et font des projets magnifiques pour un futur bienveillant &raquo; ? La preuve du pudding c' est qu' on le mange . À Londres , personne n' a eu à manger le gâteau européen pendant la campagne du Brexit . Il n' y aura jamais eu un mot de positif pour appeler les Anglais à rester dans l' Union européenne . Juste des menaces et des grimaces : la peur de l' hiver nucléaire et celle des catastrophes immobilières , financières , économique que sais -je , le fer rouge de la xénophobie , la marque noire du racisme , si le pays votait mal ! En vain ! Et désastre supplémentaire : l' immobilier est resté stable , la bourse est repartie , la croissance se maintient . Pffff , les eurocrates ne savent que faire . Le sommet de l' Union à Bratislava a eu pour grand dessein de ne pas sanctionner l' Espagne et le Portugal ! Depuis , ça se traine . On voit bien qu' il y a une activité pour essayer de trouver quelque chose à se mettre sous la dent . On a donc sur le tapis Madame Merkel en leader d' un improbable &laquo; monde libre &raquo; . Le rêve européen : la maman méritante pour tous . Plat de résistance : l' Europe de la Défense . Une foutaise en barre . Se défendre ? Contre qui ? Déjà qu' il n' y a pas de politique étrangère commune de l' Union ! Évident : elle est impossible à bâtir . Les va-t -en guerre contre la Russie de l' Europe de l' est et les paisibles méditerranéens qui ont d' autres soucis avec leur voisins n' ont pas du tout les mêmes priorités . Et celles -ci ne peuvent non plus s' additionner . Sans parler des permanences de l' histoire . Les Russes sont des partenaires naturels des Français . Et quand les Français l' oublient , la catastrophe n' est jamais loin . Quoiqu'il en soit , voyez l' incroyable évolution : on faisait l' Europe pour la paix , on nous propose de la faire pour se préparer à la guerre . Car nul n' a jamais vu qu' on s' arme et s' intègre militairement autrement que pour faire usage des moyens que l' on rassemble . La peur de l' autre et la guerre comme liant du projet européen ! Un minable retour aux sources . Car la première Europe politique proposée sur le vieux continent après la deuxième guerre mondiale , ce fut la communauté européenne de Défense ( CED ) en 1954 . Il s' agissait alors de s' armer et de coopérer contre … les Russes ! Déjà ! Ceux-là se présentaient à l' époque sous la forme de l' Union des Républiques Socialistes Soviétiques ( URSS ) . Le Parlement français vota non . Il est vrai qu' il était étrange de décider de s' allier avec l' envahisseur de la veille contre l' allié qui nous en avait libérés . Les &laquo; pères fondateurs &raquo; , un quarteron de bigots à la retraite après une guerre où ils furent très moyens , remirent l' ouvrage sur le métier . Le traité de Rome arriva donc trois ans après cet échec . Toute la gauche française vota contre , de Mendès France aux communistes . On veut l' oublier aujourd'hui . On gagnerait à relire le discours de Mendès France à l' époque … On votait donc cette semaine au Parlement européen plusieurs rapports sur le sujet . Un plan d' organisation d' une défense commune . Chacun s' est donc positionné . Le PS et la droite ont évidemment voté pour ; les Verts et nous contre . Bien sûr , nos motivations ne sont pas les mêmes . Dans ma façon de voir les choses , le système militaire de la France a une cohérence qui est et doit rester strictement défensive . Si chacun agit de même , nul ne sera tenté de sortir de ce rôle , du fait du manque des moyens et de la culture agressive qui va avec la constitution d' une puissance du type de celle que voudrait construire les eurolâtres . En toute hypothèse il n' existe aucune menace militaire contre l' Europe . Il est stupide de faire croire que la Russie serait en position belliqueuse vis-à-vis de la Pologne ou des petits pays baltes . Personne ne saurait dire quel intérêt il y aurait pour la Russie à agresser qui que ce soit de ce côté-là . Par contre il est tout à fait évident que l' Union européenne menace la Russie . Elle le fait en tant que poste avancé de l' OTAN , avec ses batteries de missiles en Pologne et ses troupes américaines positionnées sur le territoire européen . Et elle le fait notamment en Ukraine où son soutien au gouvernement d' extrême droite est extrêmement dangereux . Sans oublier la Géorgie , la Biélorussie et combien d' autres zones . Dans chacune d' elle l' Union a lancé le processus d' annexion traditionnel : discussion d' alliance militaire , inclusion dans la zone économique européenne , organisation du renversement des circuits financiers et marchands vers l' Union en général et vers l' Allemagne en particulier . Sur toutes les frontières de l' est , nous sommes à la merci d' une provocation des bellicistes . Car l' article 5 de l' OTAN et l' article 49 du traité de Lisbonne nous obligent à une solidarité militaire avec les gouvernements grossièrement anti-russes de la région . C' est évidemment un contexte favorable pour la répartition des rôles à laquelle s' emploient les stratèges nord-américains : ils veulent voir l' Europe s' occuper des Russes pendant qu' eux s' occupent de l' Asie . Au Parlement européen règne donc une ambiance très chaude de guerre … froide . On a même voté un rapport pour demander l' organisation d' une contre propagande face aux Russes … Et comme si cela ne suffisait pas , après avoir estimé le coût de &laquo; l' inexistence d' une Union européenne de la défense &raquo; à 26 milliards d' euros du fait des doublons d' emploi et des sous-investissements , la chère &laquo; Europe qui nous protège &raquo; envisage des sanctions financières contre les récalcitrants à l' intégration militaire . Je me demande ce que vont me reprocher maintenant ceux qui se donnaient des airs indignés quand je parlais de construire un &laquo; nouvel indépendantisme &raquo; . Le choix se resserre : soit ce que je propose , soit l' Europe militaire intégrée sous commandement du leader du monde libre . Puisque c' est le nouveau surnom de madame Merkel dans la presse anglo-saxonne . ",
    "Alors , je suppose que c' est à moi . Tout le monde m' entend bien ? Ça va ? Bon , il y a un petit larsen là . D'abord , je ... j' envoie un petit signal amical pour cet accueil admirable à vous tous , à ceux qui se trouvent dans le hall , et qui ont dû suivre les films sans ... sans voir les images . A ceux qui sont dans les salles qui jouxtent celle -ci et , m' a -t-on dit aussi , d' autres qui se trouvent dehors . Je les embrasse de tout cœur pour les réchauffer . Qu' ils sachent que j' en suis marri . Mais c' est comme ça . Je vois à cette affluence un signal que je comprends . Qui ne tient pas à ma personne , mais à ce que , à cet instant , je peux représenter pour vous . Je sais ... que ... qu' en cette année si extraordinaire d' élection présidentielle , je figure dans les 4 premiers , sans qu' on puisse dire à quelle place exactement . Et comme vous le savez comme moi et comme vous le sentez , vous éprouvez le besoin de vous retrouver , de vous fortifier en vous voyant les uns les autres , et pour d' aucuns , de venir entendre , apprendre , savoir de quoi il retourne avec ce personnage que l' on habille tantôt tout en rouge , tantôt en super bleu-blanc-rouge et d' une manière générale , dans des portraits si divers que parfois , moi-même , j' ai du mal à m' y reconnaître . Je remercie les Insoumis des deux Savoies qui ont organisé cette soirée . Naturellement , également les membres de mon cher Parti de Gauche qui donnent le coup de main indispensable . Les élus qui me parrainent en Savoie , on me dit qu' il y en avait 3 . Je leur en suis très reconnaissant . Je voudrais que vous sachiez que me parrainer ne signi ... ne signifie pas que l' on est d'accord avec moi . Avec vous , pour une part d' entre vous , car j' imagine que il doit y avoir ici aussi des gens qui n' ont pas encore pris leur décision pour savoir ce qu' ils feront comme vote dans quelques mois . Me parrainer , c' est seulement dire que l' on autorise ces 4 millions de Français qui , naguère , m' ont donné leur suffrage en 2012 ; on les autorise à trouver de nouveau à leur disposition celui pour lequel ils ont déjà voté une fois . On a , à la faveur de petits tripatouillages , aggravé les conditions dans lesquelles on peut être candidat dans notre pays . L' invention des 500 parrainages avait été faite , au départ , pour éviter les candidatures dites folkloriques . Le général de Gaulle avait , parait -il , mal supporté la présence de M. Marcel Barbu en 1965 , qui était un candidat très émouvant et qui était d'ailleurs lui-même très ému , puisqu'il pleurait la plupart du temps à l' écran . Et donc on a inventé ça . On n' a jamais inventé à l' époque , me semble -t-il dans une Constitution qui , déjà , instaurait mille barrages pour accéder à la fonction de monarque présidentiel - on n' avait pas imaginé que ce serait un moyen pur et simple de verrouillage pour empêcher des gens d' êtres candidats . Alors ... c' est une difficulté , vous savez , pour nous . Bien sûr . Mais de quelle difficulté ne viendrions -nous pas à bout , du seul fait de notre volonté , de notre opiniâtreté et de notre engagement ? On m' avait dit que puisque personne ne m' aidait , je n' arriverais à rien . Fort des 100.000 premiers qui ont accouru pour appuyer ma candidature , et à partir de ce soir , puisque nous avons atteint ce chiffre aujourd'hui , des 150.000 , je me trouve déjà au niveau où la réforme que suggérait Lionel Jospin : pour être candidat , il aurait fallu , ou bien 500 parrainages , ou bien 150.000 parrainages de citoyens . Et bien , quant aux citoyens , me voici déjà parrainé . Ensuite ... Quant aux élus , mesdames et messieurs , mes chers amis pour ceux qui sont mes amis ici , et mes chers camarades pour ceux qui le sont , rien , personne , jamais , à aucun moment , n' est en état de nous empêcher d' avancer . Nous creuserons les montagnes avec nos ongles et nous viderons la mer avec nos mains s' il le faut , mais nous atteindrons notre objectif , et nous sommes déjà 325 parrains et marraines autour de ma candidature . Nous arriverons donc à 500 le moment venu , c'est-à-dire lorsque , enfin , ici et là , certains qui se réservent viendront à la rescousse comme c' est leur devoir . A la rescousse , ai -je dit , parce que tout ça nous coûte de l' effort , de l' argent , bref . Cette campagne , que vous appuyez à cet instant , est d'abord une campagne populaire . Au fond , c' est une bonne chose qu' ils aient essayé , tous , de faire en sorte que je n' y sois pas . Vous vous êtes même cotisés , alors que nous ne sommes pas un parti , la France Insoumise , nous sommes un mouvement , un label commun , nous tâtonnons , nous cherchons des formes nouvelles pour nous entendre et militer . Et bien ... Vous vous êtes cotisés . Vous ne le savez peut-être pas , mais vous avez déjà donné un demi-million d' euros . Ça va . C' est bien . Evidemment , on ne peut pas avoir les grandes salles qu' on aimerait avoir , parce que la démocratie , c' est de plus en plus cher dans ce pays . Déjà , c' est de plus en plus compliqué : vous partez à l' ancienne avec vos tracts et vous dites : &quot; Tiens , je vais aller faire un porte à porte . &quot; Couic . C' est pas possible . Il y a des digicodes . Alors après , vous dites : &quot; Ben , je vais aller là où sont les gens . &quot; Où ils sont -y , les gens ? Pas à la sortie de la messe , il y en a moins qu' avant . Donc - je ne dis pas ça pour ironiser , hein - donc on va où ? Là où ils sont ! Dans le centre commercial , misère , jusqu' au dimanche . Et dans le centre commercial , on ne peut pas , c' est une surface privée . Vous voulez une salle ? C' est des dizaines de milliers d' euros , mesdames , messieurs . Vous devez le savoir . Des dizaines de milliers d' euros . Et maintenant , si vous êtes dans la rue , en plus , on vous rajoute le tri de la population qui rentre . Pour faire le meeting du 5 juin , place Stalingrad , nous avons dû débourser 30.000 euros pour organiser le contrôle des gens qui venaient là . Parce que ça n' est qu' une société agréée qui peut le faire . Je veux que vous le sachiez . C' est de plus en plus difficile de mener un combat démocratique . C' est de plus en plus difficile de vouloir participer à la démocratie de son pays . Bon , je n' ai pas fini de remercier . Je remercie pas seulement les élus qui me parrainent , mais aussi les élus qui me soutiennent . Et il y en a 3 ici qui me sont spécialement chers : Andréa Kotarac , Emilie Marche , Corinne Morel-Darleux qui sont vos élus au Conseil Régional . Et puis ... Je remercie ceux qui me donnent une démonstration des propos qui , parfois , vous paraîtront abstraits ce soir , les opposants à la ligne Lyon-Turin et les militants ... les salariés , les ouvriers et ouvrières d' Ecopla que j' ai rencontrés tout à l' heure et dont je vais vous parler . Chaque fois que je vais quelque part , on me demande pourquoi je viens là . J' étais à Boulogne : &quot; Pourquoi vous venez à Boulogne-sur-Mer ? &quot; Bon , ben j' avais une bonne raison . C' est une des villes les plus pauvres de France . La pauvreté , c' est cette lèpre qui gagne notre pays et qui fait basculer 9 millions de personnes . Et vous savez , quand on devient pauvre ? C' est quelqu'un qui , d'après les statistiques internationales , gagne moins de 1.000 euros par mois . Et ben , vous savez quoi ? 1.000€ , c' est une fortune pour bien des gens . Le minimum vieillesse , c' est même pas 500 euros . 9 millions de personnes . Ce pays est en train de basculer dans une situation : ou bien vous vous y trouvez , et votre seul souci , c' est de savoir comment vous allez faire face à demain , voire à après-demain , et puis , pour les autres , de se dire : &quot; Bon sang , pourvu que ça ne soit pas mon tour . &quot; Et ça gagne tout le monde , hein . Il ne faut pas croire . Je suis sûr que des gens dans cette salle le savent . Il ne faut pas croire que c' est seulement ceux qui étaient déjà en grande difficulté avant et qui sont dans des difficultés supérieures aujourd'hui . La pauvreté frappe à toutes les portes , à tous les étages . Tout le monde . Quelque soit sa classe sociale , enfin , à l' intérieur de la classe des salariés . Dans combien de copropriétés , les copropriétaires ne peuvent plus payer les réparations ou les charges ? Et les copropriétés accumulent les dettes et les impayés . Et les gens vivent dans la peur de ne plus avoir leur logement demain si on les en expulse , alors qu' ils croyaient en être propriétaires et avoir garanti leurs vieux jours de cette manière . Voilà pourquoi je suis allé à Boulogne . On me dit : &quot; Pourquoi vous venez à Chambéry ? &quot; Pourquoi je viendrais , après tout , hein ? On se demande . Ben , déjà , parce que je n' y suis jamais venu . J' ai cherché dans ma tête , dans ma longue ... ma longue vie de ... de militant et de lanceur d' alerte quand est -ce que j' étais venu à Chambéry . J' en ai fait des villes , vous savez . Pour moi , c' est toutes les mêmes , hein . C' est la nuit , dans des salles glauques , pourtant celle-là est belle , hein . Et après , ça m' est revenu : &quot; Ah oui , je suis venu en stop quand j' avais 20 ans . &quot; Je ne me rappelle plus pourquoi , je devais passer par là . En vérité , je suis venu parce qu' il y avait un croisement de symboles dont j' ai besoin pour faire mon discours . D'abord ... c' est la Savoie . Et la Savoie , ha ha , pour un homme comme moi , il y a un nom qui s' attache à la Savoie : c' est Ambroise Croizat . Le communiste qui a fondé la Sécurité Sociale . La Sécurité Sociale , ce n' est pas seulement un bon service , c' est une autre société , une autre manière d' organiser la société et de la penser , où les individus sont solidaires entre eux , où chacun donne selon ses moyens , mais reçoit selon ses besoins . Et surtout , la Sécurité Sociale - je ne sais pas si vous avez vu le film &quot; La Sociale &quot; , sûrement que vous avez dû le voir , pour un certain nombre d' entre vous - la Sécurité Sociale , c' était la victoire sur un fléau . Ce fléau , c' était la peur du lendemain . La peur de la maladie . La peur de l' âge qui s' avance . Et ça , c' était radicalement nouveau , on avait trouvé le moyen . Des ouvrières , des ouvriers ont fait vivre les premières caisses de Sécurité Sociale et c' est eux qui ont porté ça sur leur dos . Et bien voici que cette peur du lendemain que l' on avait vaincue , que l' on avait vaincue au lendemain d' une guerre épouvantable , alors que le pays était ruiné . Cette peur du lendemain , elle est revenue dans combien de foyers ? Depuis que l' humanité est humanité , elle lutte contre la précarité . Et oui . Les premiers groupes humains allaient d' un endroit à l' autre , et tâchaient de s' installer dans les endroits où c' était le plus profitable pour y avoir les enfants et tâcher de les faire vivre le plus longtemps possible . Et c' est comme ça que l' élevage remplace la chasse , parce que c' était plus sûr . A condition , bien sûr , de bosser dur . Et c' est comme ça que l' agriculture remplace la cueillette . Evidemment , il fallait travailler dur , mais c' était moins aléatoire . Et ainsi de suite . L' humanité a toujours voulu vaincre la peur du lendemain . S' installer dans la sécurité pour pouvoir devenir l' humanité complètement . Avoir du temps pour faire autre chose que survivre . Avoir du temps pour soi , pour les autres , pour l' art , pour la musique , bref . Toutes les choses qui comptent et qui nous font spécifiquement humains . Voilà pour Croizat . Alors , après , moi , chaque fois que je vais quelque part , je tourne des pages : alors il parait que vous étiez des Allobroges . Bon . Ca , c' est pour &quot; nos ancêtres , les Gaulois &quot; , d'accord ? Il n' y a pas un Gaulois dans la salle , mais ce n' est pas grave . Mais , ces Allobroges ... moi , vous savez que je ne crois pas que l' identité du pays soit dans ces peuplades qui ont traversé l' Europe dans tous les sens dans le passé . Moi , je crois que l' identité du pays , c' est son identité républicaine , ce qui fait que nous pouvons être tous Françaises , Français , vivre ensemble , c' est que nous sommes unis entre nous ni par la couleur de peau , ni par la religion , ni même par la langue , puisque nous la partageons avec 29 pays dans le monde , mais par un pacte politique : Liberté , Egalité , Fraternité . Et si nous sommes de cette façon , alors il faut que je regarde la Savoie de 1789 . Et revoilà les Allobroges . Il parait que vous avez tenu une Assemblée des Allobroges . Je dis &quot; vous &quot; , tu parles , là-dedans , il doit y en avoir je ne sais pas combien qui ont un grand-père ... déjà 1 sur 4 , vous avez un ... un de vos familiers qui est un étranger . Alors , vous , ça doit être les Italiens , ça circulait pas mal dans le coin , hein . Moi aussi , j' en ai des Italiens . Ça ne se voit pas , hein ? Des Siciliens , ça , ça doit se voir un peu . Un peu ombrageux , vous voyez ? Bon . Et donc , en 1792 , vous avez formé l' Assemblée des Allobroges . Et la Savoie a décidé de se joindre à la République Française , chassé son monarque et tourné la page de l' Ancien Régime , avant qu' elle n' y soit précipitée de nouveau après nos défaites . Et la Savoie est revenue de son propre gré , par son vote , à la France . L' histoire républicaine de la France a donc une part de ses racines dans la Savoie , par la volonté de ... de ceux qui composaient ce territoire à cette époque là , on ne sait pas qui c' est , peut-être quelques uns de vos ancêtres , quand même , hein , s' y trouvaient . Sa volonté républicaine et sa volonté française est parfaitement claire . C' est la même chose , je pense , dans l' esprit de tous ceux qui y ont participé . Donc , voilà pourquoi je suis là . Je viens toucher cette terre tellurique , mais pas que . Bon , alors , sur mon papier , il est écrit qu' il faut parler de l' actualité . L' actualité , c' est l' élection américaine . Evidemment que ça nous concerne , les amis . Pas parce que nous serions une espèce de grand Disneyland mondial , et qu' on regarderait toujours dans le pays d' où est arrivé Mickey ou Donald ... Bon , c' est Donald . Mais ... Ouais , ben ... je parie que vous voyez à quoi je veux en venir , ça me brûle la bouche . Ben oui , Donald , c' est Picsou , forcément . Ce n' est pas le plus malin , dans l' histoire . Voilà . Après , il y a les 3 autres qui s' agitent , là , je vous laisse attribuer les rôles . Riri , Fifi , Loulou . Mais , plaisanterie mise à part , le Président de la nation dominant le monde , ça nous implique ; en tout cas , ça nous interpelle et ça nous oblige à savoir quoi faire . Mais d'abord - ça ne sera pas le sujet de ce soir , je ne vais pas vous parler des Etats-Unis d' Amérique , pas plus que ça - mais , vous avez vu cette immense surprise de ceux qui sont tout le temps surpris ? Vous avez remarqué qu' ils sont tout le temps surpris ? C' est comme s' ils préparaient quelque chose , que ça n' a pas lieu et ils se disent surpris , alors qu' en réalité , ils sont en colère , parce qu' ils n' ont pas réussi à obtenir des gens ce qu' ils voulaient leur faire faire . Voilà . Etonnés , surpris , toujours . Que se passe -t-il là-bas ? Il se passe là-bas ce qui se passe dans le monde entier . Les sociétés telles que nous les avons connues depuis la fin de la 2ème guerre mondiale , qui avait engendré l' émergence d' une classe moyenne , partout , assez puissante , ce qui veut dire que le partage de la richesse entre le travail et le capital s' était fait d' une manière un peu plus juste , tant et si bien qu' est apparu ce qu' on appelait cette classe moyenne , c'est-à-dire des gens sachant , des gens travaillant et vivant avec une paie et un environnement qui leur permettaient de mener une vie digne . En tout cas , ils essayaient de l' améliorer , ils avaient la certitude que leurs enfants vivraient mieux qu' eux-mêmes , comme eux-mêmes avaient mieux vécu que leurs parents . Et bien , le système néo-libéral , le libre-échangisme , c'est-à-dire les marchandises qui circulent d' un bout à l' autre du monde avec un unique critère de sélection : le prix . Car si au XVIIIème siècle , on pouvait dire , aussi sottement que l' a dit le Président Hollande , que l' offre crée la demande , c'est-à-dire qu' il suffit qu' on vous amène une marchandise , si elle est assez peu chère , vous l' achèterez . Si vous y réfléchissez , c' est absurde dans tous les cas . Allez essayer de vendre des frigos à des Esquimaux . Allez vendre des radiateurs à des Sahraouis . Bien sûr que non , il ne suffit pas qu' une marchandise soit là et qu' elle ne soit pas chère pour que vous l' achetiez , mais ils le croient . Et par conséquent , ils font tout pour qu' elle ne soit pas chère . &quot; Pas cher &quot; est devenu une sorte de morale collective , d' objectif . Chacun doit se procurer à pas cher . &quot; Pas cher &quot; égal souffrance , car au bout de &quot; pas cher &quot; , il y a quelqu'un qui souffre . Qui travaille et qui est payé dans des conditions lamentables , qui travaille dans des conditions lamentables , qui est surexploité à l' autre bout du monde . Ce sont des enfants . Ce sont des adultes mal payés , toujours plus mal payés , car il y a toujours plus &quot; pas cher &quot; à un endroit qu' à un autre . Voilà que les Ukrainiens coûtent le tiers d' un Chinois . Pauvre Chinois . Pauvre Ukrainien . Et ainsi de suite . Et bien , sur cette base , nos sociétés sont en train d' être détruites . La vague s' est levée d'abord dans l' Amérique latine . De sociétés qui ne sont pas composées d' indigènes folkloriques qui jouent de la guitare , fument des cigares , dansent la salsa et boivent du rhum , comme le croient bien des nantis ici qui s' en moquent assez facilement , je trouve . Ce sont des peuples avec des sociétés complexes , organisées , bien plus de différences sociales qu' on ne se l' imagine , et bien , leurs sociétés ont été rasées par le néo-libéralisme . J' ai vu , de mes yeux vu , à Montevideo dans la gare principale , comme vous diriez la gare de Lyon à Paris ou la vôtre ici , les arbres pousser . Parce qu' il n' y a plus de train . J' ai vu en Argentine démonter les rails pour vendre l' acier , parce qu' il n' y avait plus de trains , et qu' il fallait tout faire passer sur la route . Tout ce que je vous raconte ici aujourd'hui , tout ça a déjà été fait . J' ai vu des gens se retrouver dans le noir parce que le réseau électrique s' est effondré parce que personne ne s' en occupait assez sérieusement , etc . , etc . Et alors , cette vague a atteint aussi les Etats-Unis d' Amérique , comme elle atteint en ce moment l' Europe . C' est une énorme vague . C' est le peuple . Le peuple qui n' en peut plus d' avoir deux ou trois boulots par jour pour arriver à survivre , et des fois , pour ne pas arriver à survivre . Il y en a assez de se ruiner pour payer les études des enfants . Des jeunes qui sont à l' université , endettés jusqu' au cou pour 25 ou 30 ans , comme c' est le cas aux Etats-Unis d' Amérique . Alors cette vague , elle a d'abord fait sortir M. Obama , parce que M. Obama , c' est un afro-américain , comme on dit . Donc les gens disaient : &quot; Celui-là , il a réussi , ça prouve qu' il va nous comprendre , il vient de notre milieu . &quot; Et puis c' est un afro-américain , comme Chavez , la peau mate . Comme Correa , en Equateur . Comme tous les autres . C' était la revanche de ce peuple moyen , de ce peuple populaire , qui , à peine arrivé au pouvoir en Amérique latine commence d'abord par extraire des millions de gens de l' extrême pauvreté et subissent les jets permanents de sarcasmes , de cailloux , d' insultes , mais ils le font . Des millions de gens sortent de l' extrême pauvreté . Ils font ce que personne ne croyait capable de ... possible de faire . Ils rendent la vue à ceux qui n' y voyaient plus , parce qu' ils avaient la cataracte et que personne ne les soignait . Ça s' appelait la mission Milagro . Voilà ce qui s' est passé . Ça a atteint les Etats-Unis d' Amérique , et là , qu' est -ce qui se passe ? Pas grand-chose . Mais en tout cas , assez pour avoir exaspéré les puissants et les importants . Et à l' élection suivante , mes amis , c' est celle qu' on vient de vivre , la vague , elle est toujours là . De quel côté va -t-elle aller ? Va -t-elle aller du côté de M. Trump ou du côté du camp progressiste ? Va -t-elle aller du côté de ceux qui affichent des convictions , qui , à nous , nous paraissent insupportables , ou est -ce qu' elle va aller de l' autre côté ? Et bien , M. Trump a d'abord commencé par faire quelque chose : il a exprimé les volontés du peuple , là-bas , de droite . Ça existe , vous savez , la droite populaire . Ça existe . Et les gens , ils croient de bonne foi que c' est de ce côté là que va se faire leur avenir , que c' est bien comme ça . Et puis , vous avez ceux qui préfèrent le camp progressiste . Qui pensent qu' il faut faire ensemble . Bref , je ne vais pas vous raconter la droite et la gauche , vous la connaissez aussi bien que moi , à supposer que là-dedans , on arrive à s' y retrouver , de savoir qu' est -ce qui est de droite et qu' est -ce qui est de gauche . Et bien , on l' a bouchée , la voie . D' un côté , vous aviez cet homme qui s' impose , il gagne dans sa primaire en chassant tous ces bigots qu' il y avait , du Parti Républicain , qui venaient parler de toutes sortes de choses , du bon Dieu , des choses importantes , hein ? Je ne vous dis pas le contraire , m' enfin les gens , ils ne vont pas voter pour ça , quoi . Ils ont beau être de droite , leurs problèmes c' est aussi la paie , le boulot , l' éducation , la santé , ça les intéresse . Et ils entendent un type qui leur dit : &quot; Nous , on va arrêter de fermer les usines . &quot; Et ben , vous êtes de ce que vous voulez politiquement , si c' est vous qui êtes dans l' usine , vous êtes pour le type qui vous dit qu' on ne va pas la fermer . C' est clair , non ? Et ben oui , ça s' appelle le peuple . Et ce n' est pas parce qu' il y a un peuple qui pense de deux manières différentes que ça cesse d' être le peuple . Le peuple a des intérêts , des aspirations , des revendications qui sont communes à tout le peuple . Mais qui s' expriment différemment , avec des solutions de droite , des solutions de gauche , des solutions de ce que vous voulez . Et de l' autre côté , que s' est -il passé ? On a eu un phénomène qui s' appelle Bernie Sanders . Alors , je vous garantis que moi , à côté , c' est presque la pop music , parce que Sanders , des cravates à ne pas sortir , des tenues pas racontables , même pas coiffé , il ne faisait aucun effort sur la forme . Mais sur le fond , mes amis , il n' a parlé que de revendications sociales . Et il battait Mme Clinton dans les Etats populaires dominés par les Républicains , ceux qui , ensuite , ont fait la différence . Vous comprenez ? Il a fallu tricher pour en venir à bout . Il a fallu l' invisibiliser médiatiquement , il a fallu le traiter de tous les noms . Tous les noms , je vais vous dire : un , il est trop énervé , deux , il est antisémite , c' est la croix qu' on porte tous . Dès qu' on est pas de leur bord , c' est qu' on est antisémite . Et lui , pas de pot , si j' ose dire , pour ceux qui l' accusaient , sa famille est morte en déportation , et ben , ça ne les gêne pas , les autres . Et ainsi de suite . Les mêmes injures , les mêmes caricatures d' un bord à l' autre de l' univers . Et malgré ça , il a fallu qu' ils trichent et qu' ils trichent et qu' ils trichent encore , pour l' empêcher de gagner . Mais il y avait des foules immenses qui s' assemblaient , comme les Etats-Unis n' en avaient pas vu depuis des décennies . Figurez -vous qu' à New York , à deux occasions , des foules gigantesques se sont réunies dans les parcs . Voilà ce qu' il s' est passé . Le peuple est sorti de son lit et il a rencontré un bouchon qui s' appelle Mme Clinton . C'est-à-dire cette gôche millionnaire , corrompue , liée au système . On dit : &quot; Mais l' autre aussi . &quot; &quot; Ben , peut-être bien , mais , au moins , il le dit . &quot; Il ne fait pas semblant d' avoir de la compassion pour les autres , lui . C' est ça que les gens ne supportent pas . C' est le mensonge , la duplicité . L' évidence que la personne qui parle ne fera pas ce qu' elle dit . Et caetera . Ça nous importe , bon , il a gagné . Il y a de l' eau , là-dedans ? Il a gagné . Alors , je ne vais pas commenter . Je vais vous dire ce que je lui reproche le plus . Aujourd'hui ... enfin là , depuis deux trois jours , a commencé la COP 22 . D'accord ? Maintenant , les gens ... il faut faire de la politique sérieusement . Nous tous , quelque soit notre étiquette , quelque soient nos habitudes de penser , nous sommes tous impliqués par le changement climatique . On ne doit pas oser présenter un programme qui ne parte pas de ce constat initial . Et nous , les Français , spécialement . Pourquoi nous ? Parce que nous sommes une grande nation , riche et puissante . Parce que nous avons , dans nos rangs , les milliers d' hommes , de femmes de tous âges , qui savent faire . J' ai connu des amis et des camarades qui dirigeaient des gouvernements et qui voulaient faire des grands changements . Mais quoi qu' ils décident , ils avaient un problème énorme : il n' y avait personne pour le faire . C'est-à-dire qu' il n' y avait pas la qualification , les compétences professionnelles , le savoir qui permet de tourner , et nous , nous avons tout ça . Donc , nous devons marcher devant pour ce qui est du changement de nos manières de produire , d' échanger et de consommer . Pour nous mettre à la hauteur de la tâche . Le changement climatique est commencé . L' humanité toute entière est mise au pied du mur de savoir ce qu' elle va faire face à ce qui s' annonce , à ce qui est déjà commencé , à ce qui est en cours . Ben , ça commence mal , figurez -vous , quand le maître du monde , peu me chaut qu' il soit de droite , de gauche ou de ce que vous voulez , mais qu' il dise : &quot; Je ne crois pas au changement climatique . &quot; Je le cite , là , hein . &quot; Je ne crois pas au changement climatique . On a appelé ça le réchauffement de la planète . &quot; Je ne sais pas qui lui a dit ça . Le changement climatique , maintenant ils appellent ça une météo extrême , c' est nouveau . Et voici ce qu' il dit : &quot; C' est juste la météo . Ça a toujours été comme ça , le temps change . Il y a des tempêtes , de la pluie et des belles journées . &quot; &quot; Mais &quot; , dit Trump , &quot; je ne pense pas qu' on doive mettre les gens de notre pays en danger . &quot; Voilà . Donc cet homme dirige la première puissance du monde , et il pense que le changement climatique , c' est une invention . Voilà qui est très dangereux . Et comme il le pense , ben il prend les mesures qui vont avec . Et notamment celle -ci : &quot; La réglementation qui oblige des centaines de centrales à charbon à fermer , et qui bloque la construction de nouvelles , est stupide . &quot; &quot; On va tout enlever . &quot; Tout enlever , mes chers amis . Les Etats-Unis d' Amérique et la Chine sont les deux premiers pollueurs du monde . La Chine est en train de ... d' appliquer un plan - puisque chez eux , ils planifient - et cette année , ils ont un meilleur résultat que l' année précédente . Même si ce n' est pas extraordinaire , c' est un début . J' ai lu que même M. Jadot le disait . Alors ... je peux donc parler des Chinois sans être aussitôt suspecté d' être un membre actif du bureau politique du Parti Communiste de ce pays comme on me l' a fait dans le passé . Donc ... si les Etats-Unis , qui déjà , ne faisaient rien , décident de faire moins que rien , c'est-à-dire de ré-ouvrir les centrales à charbon , alors ils vont aggraver le situation . La situation est la suivante : nous avons signé un accord à Paris , la COP 21 . Moi , je fais comme tout le monde , je dis : &quot; Bravo , c' est très bien , tout le monde a signé . &quot; Mais comme un certain nombre d' entre vous , il se trouve que j' ai lu ce qu' il y avait dedans . Et dans ce qu' il y a dedans , on se donne pour objectif que l' on augmente pas plus de deux degrés la température . On dit : un et demi , ça serait mieux . Forcément , parce que si on ne mettait pas un et demi , tous ceux qui habitent les îles n' auraient pas signé , vu qu' ils vont être noyés . Mais , ça c' est l' engagement que tout le monde a signé , à peu près . Pas les Etats-Unis d' Amérique . Ils ont signé , mais ça n' est pas ratifié . Ils ont d'ailleurs signé dans des conditions telles qu' il n' y ait pas besoin de ratifier . Donc là où il y avait des verbes à l' impératif , ils les ont mis au conditionnel . Mais dans cette COP 21 , les engagements de chaque Etat , ce que chaque Etat a promis de faire , quand vous mettez bout à bout , c' est 3 degrés . 3 degrés et demi . 3,5 degrés , c' est la catastrophe assurée . Je ne suis pas venu pour vous faire peur , mais juste pour qu' on regarde en face au moins un soir le problème qui se pose à nous et pourquoi il faut changer tous nos programmes politiques . Pour partir de là , et croyez -moi que si on part de là , alors on va au bon endroit . Pas seulement parce qu' on est utiles à l' intérêt général humain , parce qu' on est utiles à notre pays , pour relever les principaux défis qu' il doit relever , mais parce qu' on l' est au genre humain , dont nous sommes membres . Cette COP 21 prévoit que pour tenir l' objectif , il faut qu' il y ait une quantité de carbone , x , que l' on a le droit d' émettre . D'accord ? Là , on vient de faire le point . 80 % de ce à quoi on avait droit d' ici à 2030 , on l' a déjà émis . OK ? Maintenant , on est dans la situation du changement climatique et si on se laisse aller à des &quot; trumperies &quot; ( sic ) , on va au désastre . Alors , on va dire : &quot; Ah oui , tu t' en prends à l' Américain , et ben , je vous garantis que les nôtres , ils ne valent pas plus chers . Il y a des gens qui en doutent , ici , à propos ? On est bien en Savoie , j' ai un papier pour ça . 2016 devrait être l' année la plus chaude jamais enregistrée depuis les premiers relevés . Bon . Record historique du CO2 dans l' atmosphère . 2015 : le taux de CO2 , principal gaz à effet de serre a définitivement franchi le seuil de 400 ppm dans l' atmosphère . La Savoie : Alpes et crise écologique . 58 jours de pollution aux particules fines en 2013 . L' air de notre vallée est mortel : Dr Frédéric Champly . Vallée de l' Arve . Depuis 30 ans , les taux d' enneigement dans les Alpes ont diminué de 30 % en moyenne . Cette diminution , causée par le réchauffement climatique , s' exprime en durée , mais aussi en épaisseur : la hauteur de neige moyenne est notamment passée de 1m20 à 50cm . Dans 100 stations de sports d' hiver des Alpes sans neige d' ici 20 ans , les domaines skiables établis sous les 1800m d' altitude n' existeront plus dans 10 à 20 ans . Ça , c' est la réalité . Je vous ai donné la réponse de Trump . Et maintenant , voilà les autres . C'est-à-dire les simili-Trump ... français . Sarkozy : &quot; L' écologie , ça commence à bien faire . &quot; 7 mars 2010 . Et ça , ça se prétend être puissant . &quot; Il faut être arrogant comme l' homme pour penser que c' est nous qui avons changé le climat . &quot; Mais si , M. Sarkozy . Le climat est un système métastable , c'est-à-dire à la limite de l' équilibre , qu' un rien peut déséquilibrer . Par conséquent , même s' il est vrai qu' il y a eu des variations climatiques extrêmement importantes au fil de l' Histoire , il n' empêche que ce que la part de l' homme rajoute par les émissions de gaz carbonique et l' effet de réchauffement est déterminant . Donc ça n' est pas du tout être immodeste ou arrogant que de le constater , c' est juste être sachant . M. Juppé : &quot; En ce qui concerne l' exploitation des gaz de schiste &quot; , alors là , il n' y a pas au-dessus , hein . Il n' y a pas au-dessus . Si , il y a le charbon allemand , la lignite ... le lignite , hein . Alors , eux , ils rasent des villages entiers et ils extraient du lignite à ciel ouvert . Mais ne me regardez pas comme ça , hein , je ne suis pas germanophobe , je vous dis juste ce qui est . Je vous dis qu' il y a 400 personnes par an qui sont repérées comme étant mortes à cause de ces particules de charbon dans ce pays -ci . Ça , c' est aussi un fait . Et alors , puisque c' est le sujet , je viens du barrage , je vais vous en parler dans un instant . Vous le savez que j' étais au barrage tout à l' heure . Lui , il est pour privatiser EDF . &quot; L' Etat n' a pas vocation à diriger des entreprises . &quot; Il y a des ballots ici qui comptaient aller voter Juppé ? Il parait que c' est un modéré . Moi , je n' ai pas encore compris en quoi il était modéré , mais là , je vais vous dire , sur l' écologie , il n' est pas du tout modéré , il est carrément de l' autre côté . Dans le cas spécifique d' EDF , il détient 85 % du capital . Et ben il pense qu' il faut réduire ça , parce que entre 85 et 51 , il y a une marge . Oui , oui , entre 51 et 0 , il y en a une autre . On connait . Vous commencez par un bout et vous finissez . Pendant que je suis à la droite , Macron : &quot; Le diesel ... &quot; &quot; Le diesel reste au cœur de la politique industrielle française . &quot; Bon appétit , les gens ! &quot; Et du projet industriel français de la mobilité environnementale . &quot; Qu' est -ce que ça peut bien vouloir dire la &quot; mobilité environnementale &quot; , mais on entend &quot; environnement &quot; . Et comme on entend &quot; environnement &quot; , peut-être qu' on ne se rend pas compte qu' au passage , on a absorbé les particules fines . Dont on meurt , beaucoup dans notre pays , beaucoup . J' ai voulu vous lire ça après avoir commencé d' une manière un peu dramatique , pour vous faire mesurer le véritable écart qui existe entre ceux qui dirigent , ceux qui veulent diriger et les plus grands problèmes qui nous sautent à la figure . J' ai commencé par le plus grand de tous . Celui qui nous rattache à tous les autres êtres humains . Et donc la nécessité d' avoir une pensée politique qui part de l' humanisme et qui se pose la question de savoir comment il peut régler ce grand problème posé à tous les êtres humains . Et je sais pourquoi ils ne veulent pas en parler . Pourquoi ils ne veulent pas l' affronter . Parce que le capitalisme de notre époque est incapable d' y faire face . Il y est profondément hostile . Je vous l' explique en quelques mots . Dès lors qu' il organise le grand déménagement permanent du monde , il le fait par la multiplicité des acteurs qui interviennent sans aucune règle commune , chacun va à moins cher . C' est la première raison . La seconde , c' est que pour changer la manière de produire , la manière d' échanger , la manière de consommer , il faudrait , d' une part , consacrer des sommes considérables qui ne seraient plus dans les profits mais seraient dans l' investissement . Et ça , le capitalisme financier de notre époque l' interdit . Pour pouvoir servir des rendements supérieurs à 10 % , à 11 % , à 15 % ! Alors qu' autrefois , on faisait du 3 ou du 4 % de profits dans l' industrie . Pour ça , il ne peut pas consacrer les sommes à changer les machines , changer les process de production , changer les matériaux , payer de la recherche fondamentale qui , pendant des années , ne produit pas directement les résultats que l' on obtient de la recherche appliquée , etc . , etc . C' est une incompatibilité qui ne doit rien au complot ni à la méchanceté de ceux qui sont dedans , c' est une affaire de système . C' est le système lui-même qui produit ça et qui fait que chacun de ses acteurs ne peut pas faire autrement que comme ça , on vous le dit assez , non ? &quot; Puisque c' est moins cher là-bas , je ne peux pas faire autrement . &quot; Et ainsi de suite . Voilà la grande contradiction . Voilà pourquoi on ne peut pas en parler . Parce que ça remettrait en cause le fait qu' il y a deux manières de répondre aux situations . Soit maintenir la compétition de chacun contre tous en pensant que c' est ça qui fait avancer la société , soit partir de l' inverse , c'est-à-dire de la coopération entre tous . Mais nous , qui sommes pour la coopération entre tous , nous n' avons jamais dit que nous ne voulions pas de compétition . Nous l' acceptons , à la condition qu' elle ne mette pas en cause la solidarité et la coopération . Et de la même manière , j' estime qu' il doit y avoir aussi des gens de droite qui réfléchissent comme nous . Ils votent à droite . Bon , hé , ce n' est pas la guerre civile dans ce pays , quand même . Il y a des opinions contraires . Mais comme nous , leurs enfants respirent les particules fines . Comme nous , quand ça ne va pas fort , et bien , on mange moins bien . Et quand on mange moins bien , on est malade . Il y a une épidémie d' obésité dans notre pays , il y a une épidémie de diabète dont ces pauvres gens ne sont pas responsables , quoiqu'on les montre du doigt , souvent , ou qu' on les moque . Par conséquent , je pense que ce moment et cette prise de conscience pour quelqu'un qui vient comme moi de cette tradition là , de l' émancipation , de l' égalité , c' est un moment particulièrement favorable . Parce que nous sommes obligés de réfléchir . Nous sommes obligés de chercher des solutions et de ne pas nous en tenir aux routines . On a toujours fait comme ça , on fera toujours comme ça . Et si on continue à faire comme ça , on sera tous morts . Après avoir dévasté la planète . Voilà pourquoi c' est important après de regarder ce que chacun fait et met sur la table . Alors aujourd'hui , moi ... Ah oui , j' ai oublié de vous dire ça . Je vous ai lu les citations de droite , et là , il est marqué : &quot; Il faut leur dire que Le Pen n' a pas voté la COP 21 . &quot; Si vous êtes d' extrême droite , vous avez intérêt à avoir des gros poumons . Bon , est -ce qu' on y peut quelque chose ? Ben , bien sûr qu' on y peut . On peut plein de choses . Notamment en matière de production d' énergie . Parce que c' est par là que se font une énorme quantité de pollutions : les énergies carbonées , tout ce qu' on tire des ressources fossiles . Ma thèse est qu' on peut s' en passer . Ce n' est pas que la mienne , hein . Vous connaissez peut-être ma position , parce que c' est celle qui s' appliquera si c' est moi qui suis élu . Je suis partisan de la sortie du nucléaire . Pourtant , on me dit : &quot; Mais c' est une énergie propre . &quot; Je n' ai pas l' intention ... Je n' ai pas l' intention de montrer du doigt les gens qui ont mis en place l' énergie nucléaire , parce qu' ils pensaient bien faire et qu' ils ont agi dans des conditions que je juge assez remarquables , puisqu'en 50 ans , on n' a pas eu de catastrophes . Mais vous le savez comme moi . Maintenant , nous savons que c' est dangereux . Que c' est très dangereux . Il y a 58 réacteurs dans notre pays . Je ne vais pas recommencer à vous faire peur à tous , hein . Mais comme je sais où ils sont , moi , j' ai peur . Et je n' attends pas qu' il y ait un incident pour prendre des décisions . Parce que pour sortir du nucléaire , mes amis , il y en a pour 20 ans . Plus vous retardez la décision , plus vous augmentez la durée des risques . Par conséquent , il faut décider maintenant , et se mettre en mouvement , s' organiser . Il ne faut pas croire que sortir du nucléaire , c' est : on rentre dans une pièce , on appuie sur un bouton , ça y est c' est fini . Non . C' est long . Et il faut remplacer l' énergie que le nucléaire donnait par des énergies renouvelables . Nous pouvons planifier , m' entendez -vous . Planifier . Planifier , ça veut dire organiser . Je ne sais pas pourquoi , les gens , maintenant , ils se font peur avec un mot . Je le sais pourquoi , c' est justement pour ça que j' ai utilisé le mot . Planifier pour obliger à réfléchir ... ... qu' ils gesticulent d' une réunion à l' autre en lisant des papiers où , visiblement , ils ne savent pas ce qu' il y a dedans . C' est ça la vérité . C' est exaspérant . Lancer l' économie de la mer , ça donne 300.000 emplois . Ce n' est pas moi qui l' ai inventé dans un bureau avec ma bande de copains , c' est ce que disent les professionnels de la mer . &quot; Voilà ce qu' on arrive à faire , si on développe les différents secteurs : construction , déconstruction navales ... &quot; &quot; Mais , nous on habite dans la montagne , ça nous regarde pas . &quot; S' il vous plait , pardon . Bien sûr que ça vous regarde , c' est toute l' économie qui sera remise en mouvement . Voilà la clé de mon plan économique . C' est de relancer la machine d' une manière écologiquement responsable . Par l' économie de la mer , et ses 300.000 emplois , et l' argent qu' on va investir et les autres boulots qui viennent derrière . Tiens , par exemple , l' économie de la mer , c' est une affaire pour les coiffeurs . Ça n' a pas de rapport ? Ben si , ça en a un . Si vous avez des gens qui vont au travail , ils ont une paie . S' ils ont une paie , et ben ils peuvent aller chez le coiffeur . La première chose qu' ils suppriment quand il n' y a plus de paie ou quand il y a une mauvaise paie , c' est le coiffeur . Je prends cet exemple , je pourrais vous en dire 10 comme ça , pour vous montrer comment les choses se tiennent l' une à l' autre . Il faut tout changer . Pour installer une éolienne en mer , une pale , ça fait 70m de long . Il faut donc équiper autrement les ports , du travail pour mettre les ports en route . ensuite , il faut mettre les navires de service pour emmener les pales en mer , les installer , et une fois qu' elles sont installées , aller les surveiller . Et ainsi de suite . C' est donc une masse considérable de travail , et de travail hautement qualifié . Il faut donc que les gens , les jeunes , on les trouve , on leur propose ces métiers là . Donc qu' on arrête de tout désorganiser dans l' enseignement professionnel . Comment admettre que continuent à fermer des lycées d' enseignement professionnel du service public ? Avec des gens qui font des gargarismes du matin au soir avec l' apprentissage . Combien , parmi ceux qui recommandent ça , ont un gosse en apprentissage ? Est -ce qu' ils ne savent pas pourquoi on y allait ? Bien sûr qu' il y a des métiers où il en faut de l' apprentissage , mais ce n' est pas avec l' apprentissage qu' on va régler les problèmes de formation qui nous sont posés . Ils veulent en mettre 500.000 , en apprentissage . Nigauds ! Depuis qu' ils ont décidé d' en mettre 500.000 , depuis qu' ils donnent des prix pour qu' on aille en apprentissage , il y a moins de gens en apprentissage . Il se passe quelque chose , vous ne croyez pas ? Ben , les gens ne sont pas fous . Ils voient bien que ... 25 % des contrats d' apprentissage sont rompus dans les 3 premiers mois . Pas vu , pas pris . Le môme , il est en apprentissage , il est parti , il n' est plus nulle part . Donc il n' est plus dans aucune statistique , et tout le monde est content . Moi , voilà comment je vois les choses , comment on peut organiser , et donc pour ça , planifier . De ceux qui apprennent à faire , ensuite faire , et ensuite , partir de l' exigence écologique . Si vous vous occupez de la mer , vous êtes obligés de vous occuper de la terre . Parce que si vous vous occupez de la mer , vous vous apercevez que les coquillages , que les algues , que toutes les choses que vous y installez ne peuvent pas tenir le coup si vous continuez à faire l' agriculture de brutes que vous êtes en train de faire et qui est en train de tout détruire : la terre , l' air et ceux qui travaillent la terre . Ça , c' est la vérité dans le pays , record d' Europe de consommation des pesticides . Et ce que je raconte , je ne l' invente pas . Il y a déjà un territoire grand comme la France entière dans le Golfe du Mexique où il n' y a plus trace d' organisme vivant . Vous m' entendez ? Il n' y a plus rien . Rien de rien . Du sable au fond de la mer , et encore , il n' y en aura pas si longtemps que ça parce qu' à force de prendre du sable pour faire du béton , il n' y aura plus de plages d' ici à 50 ans . Alors , ça vous a ... ce soir , vraiment , il n' y a plus ni plages , ni air , ni ... Mélenchon , c' est terrifiant comme meeting , hein . Mais bon , c' est un défi , c' est ça qui est intéressant . C' est un défi donc on va le régler par de la technique , par de l' invention , par de la solidarité . Vous êtes obligés de vous occuper de l' agriculture , je viens de vous le dire , donc vous voulez une agriculture bio , hein ? Et ben oui , c' est l' agriculture paysanne , comme on dit . Ben , il faut les payer . Vous êtes d'accord ? Et ben oui , il faut les payer . Ça prend plus de temps . Donc si vous voulez les payer correctement , il vous faut une meilleure paie . Par conséquent , si le SMIC augmente , c' est écologique . Mais bien sûr ! Ou alors ... ou alors , le bon manger , c' est seulement pour ceux qui ont des sous ? Et vos gosses , qui c' est qui est d'accord pour lui faire boire du lait plein d' antibiotiques ? Qui sort de la ferme des mille vaches ? Et ben oui . Tout est comme ça . Si vous voulez que ça aille mieux , il faut que tout le monde aille mieux . Et pour que tout le monde aille mieux , il faut partager . Il faut répartir la richesse autrement . Je veux bien qu' on me traite de bolchévik pour ça , de rouge , d' exagéré , et tout ce qu' on veut , mais est -ce que quelqu'un est capable de me montrer comment on peut faire autrement , pour atteindre le même résultat ? Non . Non . Et ceux qui , pendant 10 ans , 20 ans , 30 ans nous ont bourré la tête avec leur flexibilité , leur compétitivité , leur mondialisation qui semblait tomber du ciel , à quel résultat sont -ils arrivés ? Où ça va mieux pour nous ? Dans quel domaine ? Ce n' est pas vrai . Ah ben , je vous ai parlé de l' agriculture , je termine juste là-dessus : il me faut 400.000 personnes pour faire l' agriculture paysanne dont je vous parle . Mes amis qui ont préparé le programme agricole , ils étaient tout contents , ils ont tout prévu , je vous garantis qu' il ne manque pas un bouton de guêtre . Evidemment , la FNSEA va être très fâchée . Et ben , qu' est -ce que vous voulez , c' est comme ça , hein . Si c' est eux qui gagnent , ils continueront à tout pourrir . Si c' est nous , ben ... on changera la manière de faire . Sauf que je ne fais pas le fier , parce que 400.000 personnes pour faire de l' agriculture paysanne , mes amis , il faut les trouver . Il y en a déjà 200.000 qui bossent , il va falloir qu' ils changent leur manière de bosser . Mais quand même qu' il y ait 100.000 jeunes qui attendent pour pouvoir être paysans eux-mêmes , il faut encore que j' en trouve 100.000 , ou 150.000 . Et ceux-là , je ne vais pas aller leur dire : &quot; Ah , j' ai une idée formidable pour toi ! Toute ta vie au cul des vaches . Pas de vacances , pas de loisirs , une paie de misère . Ça te va ? &quot; C' est ça , paysan , en France . Il s' en suicide un tous les deux jours . Dans une indifférence totale , on en parle jamais . Une misère , une détresse , une solitude totale . Des gens qui , au départ , se disaient : &quot; Je vais faire des animaux , je vais faire des céréales &quot; , ou que sais -je encore , ils se retrouvent à faire des paperasses pour avoir droit à ceci , droit à cela . Bon , il faut qu' on en trouve 100.000 ou 200.000 . Pareil , on va donc changer la condition paysanne . Il va falloir surtout changer l' éducation en amont , c'est-à-dire que ceux qui vont au lyc ... Il y a plein de jeunes , hein , qui veulent y aller à la terre . Et dans ... dans les fermes bio que j' ai visitées , ceux qui me racontent que c' est l' âge de la pierre , ils feraient bien d' y aller , hein . Parce que c' est tout bac+1 , bac+2 , bac+5 . Je suis allé dans une , dont je n' arrive jamais à me rappeler le nom , ils avaient là 70 vaches , ils vivaient à 10 dessus . Avec des paies , bon , moyennes , mais ... tu te payes bien , quoi , pour vivre , hein . Ils vendaient en circuit court . Vendre en circuit court . Relocaliser . Je ne dis pas qu' on peut le faire tout le temps , mais on ne doit pas avoir comme objectif qu' on ne le fera jamais , et que la seule manière de produire qui vaut la peine , c' est celle qui permet de faire de l' exportation . Non . Le circuit court est rentable . Ceux qui arrivent à installer des circuits courts paysans en vivent . Et il n' y a pas que dans la paysannerie que c' est comme ça . Il faut arrêter le grand déménagement du monde , dont je vous ai parlé tout à l' heure , qui est la cause numéro un du changement climatique et du reste des autres catastrophes écologiques . Ça , ça va m' amener à Ecopla . Parce qu' après tout , c' est un exemple , non ? C' est un exemple de deux choses : d'abord , du fait qu' il y a un problème avec les tribunaux de commerce . Je le laisse pour ce soir , j' ai assez d' ennemis pour aujourd'hui . Mais , regardez , c' est toujours la même logique : une entreprise est là . Vous avez des gens dedans et qui est -ce qui fait la fortune de l' entreprise ? Apparemment , ce n' est pas le capital , c' est bien les gens qui sont là-dedans qui savent le faire . Bref , vous connaissez l' histoire mieux que moi , puisque vous êtes du coin . Primo , six fonds de pension , un derrière l' autre , achètent cette entreprise . D' où elle sort , cette entreprise ? Ceux qui ont mon âge , là , vous vous en rappelez . Pechiney ! Pechiney , c' était le grand groupe aluminium français . Privatisé , cœur de métier , blabla , découpé en tranches , en petits morceaux , et à la fin , il ne reste plus rien . Il reste Ecopla . Ecopla , c' est une lointaine subdivision de la grande entreprise intégrée qu' était Pechiney . Et maintenant , voilà que un derrière l' autre , alors forcément , les types , ils viennent là placer de l' argent , pourquoi ? Ben , parce que cette petite entreprise produit 70 % ou 60 % des emballages aluminium qui sont utilisés dans notre pays . Les clients sont contents . Le fonds de pension fout le camp . Deux leçons à apprendre . Premièrement , on se rappelle que c' est les gens qui font la fortune de l' entreprise , deuxièmement , que le capital ne peut pas avoir tous les droits , et là , il a un droit incroyable . Il rentre , il sort , il fait ce qu' il veut . Non , moi , je ne suis pas d'accord . Je ne suis pas d'accord . Donc , comme je ne vous propose pas de nationaliser toutes les entreprises , comme on me l' accuse souvent . Heu , pff ... si c' était possible , moi , je n' y verrais pas d' inconvénient , mais comme ça a l' air compliqué à faire d' un seul coup , je ne m' y prends pas par ce bout là , et en plus , vous trouveriez toujours des raisons d' avoir peur et moi , je m' épuiserais à essayer de vous convaincre . Bon . Ben dans quoi je suis parti , là ? C' est parce que dans ma tête , il y a , vous savez que la seule solution qui reste maintenant pour rattraper tout ça , c' est de nationaliser ce machin microscopique . Parce que comme la justice est passée là-dessus , on ne peut plus rien faire . La ... la leçon numéro deux , je disais , c' est les droits du capital . Je suis pour qu' il y ait un droit différent de vote , pour le capital , suivant la durée de l' investissement . C'est-à-dire , si vous refusez de signer le papier qui dit : &quot; Je m' engage à maintenir mon investissement pour 4 ans , 5 ans ou 6 ans &quot; , et ben vous avez zéro droit de vote . Parce qu' il n' y a aucune raison que vous ayez le pouvoir de décider dans une entreprise que vous vous apprêtez à quitter . Et ça , tout le monde peut le comprendre . Ne me dites pas que c' est du communisme . Parce que j' ai connu pire que ça . Au Brésil , ils mettaient un dépôt de garantie , c'est-à-dire ils investissaient , mettons 100 , et ils mettaient un dépôt de garantie , disons 10 . Et si ils ne tenaient pas la durée de l' investissement , pfuit , 10 , ils les perdaient . Ça restait pour l' Etat . Donc vous voyez qu' il y a des mesures très dures qui ont été appliquées dans le monde pour contrôler la circulation des capitaux . Et notamment des capitaux investis dans l' activité productive , c' est celle-là qui nous intéresse . Deuxième conclusion : puisque c' est les gens - et vous aviez l' air d' être d'accord - qui font la richesse d' une entreprise , quand ils veulent , lorsque le patron s' en va , quand ils ont le bonheur de savoir qui c' est , et bien , s' ils veulent , eux , reprendre l' entreprise , ils doivent avoir un droit de préemption . C'est-à-dire que le premier qu' on écoute , dont on regarde le projet et à qui on donne l' entreprise , ce sont les salariés constitués en coopérative . Et ça , on peut le faire dans des tas d' activités . Là , c' est ce qu' ils se proposaient de faire , comme vous le savez . Mais vous avez vu , au cours des 2 ou 3 dernières années , comment , dans le mouvement ouvrier , je dis bien le mouvement ouvrier , parmi les ouvriers , les salariés , cette idée de l' utiliser , la coopérative , comme un moyen de maintenir un instrument de production , parce que seuls les salariés , les ouvriers savent le faire tourner , et ils savent qu' ils en tirent un rendement , qui , certes , est insuffisant pour un fonds de placement ou pour un capitaliste qui veut faire du 10 ou du 15 , mais eux , ils se contentent du 2 ou du 3 et d' avoir leur boulot . Et bien , il faut que ceux-là aient la priorité . C' était ce qui était prévu . Pourtant , je l' ai marqué sur mon papier : &quot; On ne parle pas du PS . &quot; Je suis obligé , c' est eux qui gouvernent . Alors , il était prévu qu' il y aurait ce droit de préemption inscrit dans la loi . M. Hamon se vante de l' y avoir mis . Hé , pas de bol , non . Juste : &quot; Les travailleurs ont le droit de présenter leur projet . &quot; Ben , ils sont contents avec un droit pareil . Et quand un entrepreneur vend l' entreprise , et bien , il y avait obligation - c' était ça qu' on avait dit - si ils vendent l' entreprise , il y a obligation qu' ils cherchent que ... qu' ils le donnent à un repreneur . Non . Maintenant , la loi a été arrangée à la sauce - comme toujours , hein , on fait les choses à moitié , et quand on les fait à moitié , ben , on ne les fait pas du tout - ils ont fait les choses à moitié . On me dit : &quot; Ah , non , obliger à retrouver un repreneur , non . A rechercher . &quot; Voilà . Ça veut dire rien , quoi . Le droit de préemption , ils ont le droit de présenter leur plan . Ça veut dire rien . Voilà . Donc , ça , c' est deux sujets sur lesquels il faudra qu' on intervienne très rapidement , parce qu' avec ça , on tient en respect un tas de monde . Parce que des entreprises très profitables en France qui ferment , il y en a. Parce qu' elle ne produisent pas assez pour le repreneur . J' ai connu Kem One , peut-être que ça vous dit quelque chose ici , ils fabriquent ... c' étaient les seuls qui fabriquaient du PVC dans le pays . Alors , c' était encore une subdivision de l' ancienne grande chimie en France . Et puis , de fil en aiguille , la boite tournait , hein , forcément , ils étaient seuls , mais elle ne faisait que ; que du 10 % , et donc le fonds de placement , lui ce qu' il voulait , c' était 15 % . Parce que le fonds de placement , ce n' est pas qu' il est cruel , c' est qu' il a promis à des gens de leur payer des retraites avec ce qu' il va ramasser comme argent . Donc , il faut bien qu' il fasse son boulot , c'est-à-dire vous presser au sang pour ... pour verser aux autres . C' est comme ça ... c' est pour ça que l' Allemagne marche comme ça . Parce qu' ils ont des retraites par capitalisation . Donc ils ont besoin qu' il y ait beaucoup de dividendes , donc des salaires bas , et deuxièmement , un Euro très fort pour qu' ils achètent à bas prix tout ce qui vient du reste du monde . Il ne faut pas chercher plus loin , hein . C' est pour ça qu' ils sont comme ça et c' est ça qu' ils imposent à tout le reste de l' Europe . Bref , donc ce Kem One , et ben , la boite a été fermée , liquidée parce qu' elle ne faisait pas 18 , et à chaque fois , qu' est -ce qui se passe ? Vous avez un vautour qui arrive . Il achète les murs , il regarde la trésorerie , comme à Ecopla , il regarde ce qu' il y a dedans , tchouf , il le prend pour rembourser l' emprunt qu' il a fait pour acheter l' entreprise en question , il fout tout le monde dehors , ça , c' est vous qui payez et puis lui , il vend le terrain , les murs , les machines en pièces détachées . Aux uns , aux autres , d' un côté , de l' autre . Et voilà comment , comme un véritable vautour , une entreprise est tuée , détruite , les gens foutus dehors , les risques écologiques , c' est pour vous . Et voilà comment tout ça fonctionne dans un même système , dans un même registre qui aboutit à ces résultats là . Donc il faut que les travailleurs puissent reprendre leurs entreprises . Je ne sais pas si je vous ai convaincus . J' espère bien . Mais moi , je pense que si on fait comme ça , on va donner un poids supplémentaire à l' économie sociale et solidaire dans notre pays . Et ça va nous servir à autre chose , c' est quand on va voter , les amis , les patrons , pas vous . Parce que dans ce pays , le MEDEF a une trop grosse bouche , et son organe principal , la glande lacrymale , pour pleurer et mendier de l' argent public en diminutions d' impôts , en contributions de cela , en subventions de ceci , ces gens là nous coûtent beaucoup trop cher . D' une façon générale , les patrons du CAC40 coûtent beaucoup trop cher . Et puisqu'ils arrivent à trouver des ouvriers à moins cher , nous , on est capable de trouver des patrons à moins cher . C' est une blague du syndicat des métallos allemands . Parce que vous avez vu que cette année , ils sont record d' Europe dans l' art de s' être goinfrés . Nous avons les patrons qui sont les mieux payés , et cette année , ils se sont augmentés à peine de 20 % , quand même . Eh , attendez , compte tenu des efforts . Et c' est comme ça sans arrêt . Alors , record d' Europe des dividendes versées . Et vous apprenez ça l' année où vous apprenez qu' on leur a filé 20 milliards de crédits d' impôts . Où ils sont passés les 20 milliards ? C' est où , les emplois ? C' est où , les machines nouvelles ? Où elles sont ? Où c' est ? Montrez -les nous , qu' on y aille . On ira même avec Hollande sous le bras pour inaugurer . Montrez -nous ! Pillards , bons à rien , parasites ! Tout cet argent a été perdu . Perdu . Pas pour tout le monde , mais perdu . Et pendant que eux s' enrichissaient comme ça : &quot; Ah , vous êtes trop simpliste , M. Mélenchon . &quot; Ah non , c' est vous qui êtes simplistes . Je connais même quelqu'un qui m' a dit une fois : &quot; Mais moi , ça ne me dérange pas qu' il y ait des riches . Ce qui m' embête , c' est qu' il y ait des pauvres . &quot; Vous ne voyez pas le rapport entre les deux ? Vous ne voyez pas le rapport ? Vous n' allez quand même pas me dire que c' est les riches qui prennent l' argent des pauvres ? Ben , si . Hé hé , à qui voulez -vous qu' ils le prennent ? C' est parce que vous avez tout déréglementé que vous avez amené ces gens à travailler pour des ... pour des paies aussi misérables qui ne permettent pas de faire vivre la famille , c' est parce que vous avez tout déréglementé que tout se casse la figure , et que des gens qui vivaient chichement vivent dans la rue , et que ceux qui vivaient moyennement vivent chichement . C' est vous qui êtes responsables de ça . Personne d' autre . Arrêtez de culpabiliser les gens . De leur dire qu' ils ne travaillent pas assez , que sais -je , je ne sais pas encore , tout ce qu' ils ont bien pu dire , d'ailleurs , vous les écoutez , j' espère . Ils ont fait des primaires . Alors ça , c' est intéressant . Pour une fois , je suis resté devant la télé , d' habitude , je ne la regarde pas . D'abord , il y a une chose dont ils ne parlent pas . On est tranquilles . C' est d' écologie . Ça ne les concernent pas , ils ne sont pas des êtres humains comme les autres , eux ils respirent très bien le fioul ... le charbon , ça leur va . Heu , le manger dégoûtant aussi . D'ailleurs , ils n' en mangent pas , ils mangent autre chose . Et caetera . Moi , je vous ai dit tout à l' heure , je suis pour relocaliser . Pour relocaliser autant qu' on peut , il faut utiliser un mot dont il ne faut pas avoir peur : c' est le protectionnisme . Et je parle de protectionnisme solidaire . Alors , il y a M. Lenglet qui m' a dit : &quot; Pfou , vous mettez deux mots côte à [ côte , ça ] ne veut rien dire . Ben si , ça veut dire . Ça veut dire qu' on protège et on négocie . Comprenez bien ça : la différence entre protéger et négocier d' un côté , ou bien dire : &quot; Il y a une règle , c' est le libre-échange . &quot; Le libre-échange , c' est ce que je vous ai raconté tout à l' heure . L' Union Européenne actuelle , elle prévoit le libre-échange . Non seulement pour elle-même , mais avec le reste du monde . Et non seulement elle et le reste du monde , mais dans le reste du monde , elle veut qu' il y ait le libre-échange . Et c' est ce libre-échange qui permet aux uns d' inonder les autres de productions , et ces autres productions détruisent les productions locales . C' est le cas notamment dans toute l' Afrique . Avec les accords de partenariats économiques où on dit : &quot; Vous avez le droit d' amener chez nous tout ce que vous voulez , et nous , on a le droit d' amener chez vous tout ce qu' on veut . &quot; C' est ce qui a été traité de &quot; sinistre plaisanterie &quot; par le Président de l' Afrique du Sud , et même le Président libéral du Sénégal a dit : &quot; J' espère que vous n' y croyez pas . &quot; Ben , si , ils y croient et ils le font . Donc tous ces Etats perdent leurs ressources de droits de douane , et surtout , ça démantèle tout ce qu' il y a derrière . Vous , vous ne le voyez pas . On me dit : &quot; Mais , M. Mélenchon , mais pourquoi est -ce que vous n' êtes pas content ? Regardez tel pays , il produit maintenant du riz qui se vend en dollars dans le monde . &quot; Ouais , ouais , d'accord , il produit du riz qui se vend en dollars dans le monde . D'ailleurs , ça tombe bien , ils ne mangent pas le riz , donc ils peuvent le vendre aux autres , hein . Ils mangent autre chose . Mais ce que vous ne voyez pas , c' est que tout ça , ça apparaît pour la première fois , parce que c' est à l' export et c' est en dollars et c' est pour le reste du monde , mais là où il y a du riz aujourd'hui - je dis du riz , je pourrais dire autre chose , hein - il y avait avant des agricultures vivrières qui , elles , n' apparaissaient pas en dollars , mais dans la monnaie locale , et des fois dans des trocs . C' étaient les productions pour vivre . Pour le petit marché du coin , et vous aviez des familles de paysans qui vivaient comme ça , et qui , du jour au lendemain , ne peuvent plus en vivre , donc ils vont en ville , et en ville , il n' y a pas de travail , donc de la ville , on part , et on se déplace . Et on émigre . Et on a le bonheur de faire de la navigation en Méditerranée . Après avoir consulté le Code du travail de chaque pays pour savoir dans quel endroit c' est le plus favorable . Enfin , je vous dis ça , si c' est que je les écoute eux . Nous , on sait ce que c' est . Nous , on sait ce que c' est : c' est des milliers de pauvres gens . Obligés de partir . Obligés , parce que si c' était vous , vous le feriez . Vous ne resteriez pas là , à vous regarder mourir et mourir vos enfants . Vous le feriez . Je ne vous parle même pas des guerres . Je vous parle de ça , de la vie de tous les jours , du mécanisme du libre-échange qui broie , détruit tout autour de lui . Et puis là-dessus arrive la sécheresse . Le changement climatique . L' ONU a prévu 250 millions de réfugiés . Tant que vous ne les aurez pas aux portes , vous ne comprendrez pas qu' il faut arrêter ? Et quand ils arrivent , que vous ne pouvez pas simplement dire : &quot; Je vais les refoutre à la mer . &quot; Parce que ce n' est pas ... ce n' est pas jouable , ce n' est pas possible , donc il faut bien intervenir sur la cause de départ . Bien sûr qu' il y a un problème , quand des foules pareilles se déplacent . Et si je le dis moi , ça ne fait pas de moi un xénophobe . J' ai vu qu' on me reprochait ça . Mais c' est absurde . Depuis l' origine du mouvement auquel j' appartiens , du camp progressiste , enfin , je ne veux pas choquer ceux qui ne seraient pas de ma famille politique , mais depuis le début , on dit qu' il faut que les gens puissent vivre et travailler au pays . Ça vaut pour nous , ça vaut pour les autres . Il faut qu' ils puissent développer leurs pays . Que c' est une honte qu' on leur prenne les meilleurs éléments , les plus courageux . Et même moi , il m' arrive de dire des bêtises , une fois , j' ai dit - on me dit : &quot; Alors , et l' immigration , vous croyez que c' est une bonne chose ? &quot; J' ai dit : &quot; Ça dépend . &quot; J' aurais mieux fait de me taire . Alors , comme je m' étais rendu , j' ai dit : &quot; Ben oui , s' il y a des médecins qui viennent . &quot; Mais non , Mélenchon , tu as tort . Tu as tort , on ne va pas leur prendre leurs médecins , en plus . C' est eux qui ont payé la formation . La bonne ligne , c' est arrêter les guerres , arrêter les accords de partenariats économiques , arrêter le libre-échange , tant qu' on peut , graduellement , en s' organisant . Avec des méthodes de travail . Je vous ai dit tout à l' heure : la relocalisation . Relocalisez l' agriculture , vous verrez que vous ferez vivre plus de monde de la terre , donc vous aurez de l' emploi , produisant des choses de meilleure qualité , et à la fin , tout le monde sera content . Enfin , pas mal seront contents . En tout cas , on ne sera plus dans la situation dans laquelle on est aujourd'hui . Et si vous ne voulez pas faire ce que je suis en train de vous dire , dites -moi voir ce que vous allez faire au juste . Le marché ! Voilà . Ça , ça marche pour l' agriculture . La dernière fois , c' était le lait . On fait du lait , ici ? Oui , hein . Dans les années 1970 , on a mis les quotas laitiers . Déjà , une série de très intelligents ont hurlé contre . &quot; Ah , vous voulez nous empêcher de travailler , vous mettez une limite aux quotas laitiers . &quot; &quot; Avec du bon quota laitier , tout ça ... &quot; Bon , bon . Très bien , on fait les quotas laitiers . Arrivent tous les libéraux : &quot; Assez , à bas les quotas laitiers , la politique agricole commune . &quot; Tout le monde oublie que c' est grâce à ça que le continent est devenu autonome sur le plan alimentaire . D'accord ? En moins de 20 ans . Que les paysans ont donné un bon paquet . Tous les gens qui vivent en ville peuvent leur dire merci , parce que , plutôt que d' augmenter les salaires des ouvriers , pfuit , on serrait la vis aux paysans , donc le prix de la nourriture baissait , donc le gars , avec le même SMIC , il avait l' impression qu' il avait plus d' argent qu' avant . C' est comme ça que ça s' est passé pendant les fameuses 30 glorieuses . C' est les paysans qui ont tiré la langue jusqu' à par terre pour que le reste se mette en route . Donc , les quotas laitiers , ils ont fait un marché . Voilà . Depuis que l' humanité est humanité , on fait des stocks , on essaye de ne pas être trop dans le cyclique , hein , parce qu' une année est bonne , l' autre est moins bonne donc on essaye un peu de faire des réserves , etc . , et ben , le marché il a inventé : il n' y a plus de saison . Il s' en fout . Parce que tantôt c' est l' hiver ici , là-bas c' est l' été , blabla , blabla , blabla . Tout le monde va faire du lait . Et bien , il y a une année , c' est bon , donc tout le monde fait du lait , il y a trop de lait , les prix s' effondrent , les paysans sont ruinés , l' année d'après , il n' y a pas assez de lait , le prix monte pour vous tous . OK ? C' est ça qu' ils ont fait . Et ils continuent comme ça , compartiment par compartiment . Le libre-échange , la concurrence libre et soi-disant non faussée , tout ça , c' est ça qui est en train de tuer tout le système productif . Je vous ai parlé de protectionnisme solidaire . Je l' assume . Vous voulez un exemple ? Non , c' est trop tard . Sur mon papier , il y en a un . Ah oui , j' ai lu ça , tiens . M. Juppé , qui n' en rate pas une : &quot; Le protectionnisme est un grand danger pour l' économie mondiale et pour la paix du monde . &quot; &quot; Je n' aime pas le mot de protectionnisme . Un mot du passé et rétrograde . &quot; Bon . Allez , un partout : &quot; Je n' aime pas le mot concurrence libre et non faussée , et libre-échangisme , un mot du passé et rétrograde . &quot; Ça , c' est Mélenchon . Alors ... un des instruments du protectionnisme solidaire , c' est les taxes à l' entrée des frontières , ou les visas , appelez ça comme vous voulez . Visas , c' est mieux . Parce qu' il y a l' idée de taxer les produits , et en même temps d' en autoriser . Visa social , visa écologique . Parce qu' aller produire à l' autre bout du monde , en éreintant des gens au travail , en les sous-payant , et surtout en ne s' occupant de rien pour l' environnement , pour que vous puissiez avoir moins cher , le tee-shirt que vous avez déjà en 3 exemplaires . Donc ces taxes , elles permettraient de filtrer . Ah , voilà encore une invention bolchevique . Et il y a un nouveau bolchévik dans la famille , c' est M. Sarkozy . Parce qu' il y a quelques jours , il a dit que si M. Trump ne respectait pas l' accord de Paris ... On fait une pause , hein . Si M. Trump ne respecte pas l' accord de Paris , point . Hé , pourquoi il le respecterait , ils ne l' ont pas signé . Mais il n' a pas l' air d' être au courant , M. Sarkozy . Il n' y a pas d' engagement des Etats-Unis à appliquer l' accord de Paris . C' est une déclaration d' intention du Président Obama et ça n' a pas valeur de loi , bref . Il dit : &quot; Si il n' applique pas l' accord de Paris , la COP 21 , alors nous mettrons une taxe carbone . &quot; Une taxe carbone ? C' est exactement ce qu' on raconte depuis 10 ans . Qu' on dit : &quot; Il faut mettre - les marchandises qui viennent de loin doivent avoir une taxe kilomètre . &quot; Parce que ... leur cheminement a coûté à l' équilibre de la planète . Donc , il faut mettre une taxe . Et puis surtout , comme eux , ils ne respectent pas les normes environnementales qu' on respecte nous , c' est exactement ce que vient de dire Sarkozy , et bien , alors on leur impose de les respecter . Parce que s' ils ne les respectent pas , leurs marchandises ne rentrent pas chez nous . C' est facile à comprendre , ça . Ça passait pour une lubie il y a encore un an . Et bien maintenant , vous le retrouvez , y compris dans des programmes de gens qui ne sont pas de mon bord , non ? Alors ... ça prouve que je réfléchis plus vite qu' eux . Et pas que moi . Bon . Donc ils font une primaire . Ils ne parlent de rien . Si , ils parlent de quelque chose . Franchement , j' espère que vous avez des copains qui votent à droite , ou des copines . Non mais , parce qu' attendez , ils vont être en grande détresse . Ils assistent à 3 débats . On ne leur parle pas une seule fois d' écologie . En fait , on leur parle de rien . Ou de choses étranges . Alors , d'abord , je commence par le modéré . M. Juppé , lui , il veut mettre la retraite à 65 ans . Et vous savez que ça veut dire ... Vous savez , il y a toujours un décalage pour la toucher à taux plein . Enfin ... à taux plein ... parce qu' il y a une décote qui est lié au fait que , ben , tout le monde , maintenant , a eu des périodes d' inactivité , etc . Donc , ça veut dire que pour la toucher à taux plein , il faut attendre 70 ans . Vous le croyez , un truc pareil ? Le pays est plus riche qu' il ne l' a jamais été , on est plus développés que jamais , et on va mettre des gens au travail , on va les maintenir jusqu' à 70 ans . Alors , bien sûr qu' il y a des métiers où c' est possible . Hein . Mais il y a des métiers où ce n' est pas possible . Parce que déjà à 50 , on n' en peut plus . On est déjà cassés . Physiquement , je ne parle pas mentalement , hein . Mais on ne peut plus continuer . Regardez le nombre de métiers où c' est comme ça . Ils sont fous , ces gens là . Ils ne vivent pas dans la vie réelle . Qui c' est qui monte dans un avion piloté par un type qui a 70 ans ? Pas moi . Ah , ben moi , avant de monter en avion , je demande qui c' est qui le pilote . Quel âge il a , d' où il vient , tout ça . Bon . Je blague , mais c' est pour vous faire comprendre - alors peut-être qu' un pilote de 70 ans , il peut être tout à fait capable de conduire un avion - mais c' est pour dire que ce n' est pas raisonnable de travailler comme ça . Vous imaginez , des instituteurs , des institutrices de 70 ans ? Allez les mettre en maternelle , pour voir . Ma mère y était , en maternelle . Moi , j' y suis allé une fois pour faire un reportage , j' étais journaliste . Je n' ai pas tenu une demi-journée . Vous le savez que ce n' est pas possible . Alors pourquoi ils le disent ? Ils savent très bien que personne ne va embaucher quelqu'un de 60 , 65 , déjà qu' ils ne les prennent pas à 55 . Alors pourquoi ils disent ça ? Pourquoi ? Pour que vous payiez des assurances avant . Pour que vous ayez des assurances-vie , pour que vous soyez obligés d' avoir des fonds de capitalisation . Voilà pourquoi . Pour que vous prépariez une retraite que vous ne pourriez pas toucher autrement , à partir d' un certain âge , blabla , blabla . Voilà pourquoi ils le font . Et parce que votre vie ne les intéresse pas , mais moi , j' en connais des gens de 60 , des gens de droite qui travaillent . Il ne faut pas croire , on ne va pas faire les malins , il n' y a pas que nous , hein . Qui travaillent , des fois dans des métiers durs , mais ils votent de droite . C' est ballot . Mais c' est comme ça . Ah ben tiens , du temps du programme commun , je ne veux pas rappeler de mauvais souvenirs à ceux qui seraient de droite dans la salle , mais 70 % des travailleurs votaient avec nous . Moi , j' ai fait programme commun . Mais il y en avait 30 % , ils ne voulaient rien savoir , hein . La retraite à 60 ans , ils ne voulaient pas en entendre parler , l' augmentation du SMIC , ça ne les concernait pas , bref , ils croyaient tout ce que leur disait le patron . Et bon , voilà , le patron , il leur avait dit : &quot; Ils vont ruiner l' économie . &quot; Ils l' ont cru . Bon , c' est pour dire , il y a toujours eu une partie du monde du travail qui a voté avec les gens de droite , et je le redis , ce n' est pas la guerre civile dans ce pays , on a le droit d' être d' un autre avis , n' est -ce pas ? Ce n' est pas la condition sociale qui fait votre conscience . Elle y est , ça c' est clair . Mais on a tous une tête , hein , quelque soit notre place dans la société . Bref . Tout ça pour dire que ces gens là , ils doivent bien être comme nous . Ils doivent se dire : &quot; Ben moi , je n' en peux plus à 50 ans . &quot; Ou bien l' autre va dire : &quot; Je n' en peux plus à 60 . &quot; Alors , ils écoutent le débat . Et qu' est -ce qu' ils entendent ? Qu' ils vont gratter jusqu' à 70 . Ils doivent être désespérés . Je ne sais pas pourquoi ils disent qu' ils attendent beaucoup de monde pour venir voter entre des gens qui leur proposent des horreurs pareilles . Alors , 64 ans la retraite pour Sarkozy , 65 pour Juppé . Les autres , je ne les ai pas dans ma feuille . Voilà . Fin des 35h et retour aux 39h . C' est M. Juppé . Le type , il n' est pas au courant que les 35h , il y a déjà un moment qu' il n' y en a plus . Parce que si vous baissez ... si vous baissez le tarif des heures supplémentaires , c' est comme s' il n' y en avait plus . Hein ? Voilà . Merci Mme El Khomri . Fin de la durée légale dans le privé . Vous comprenez ? Non , vous n' avez pas compris , je parie . Ben , ça veut dire qu' il n' y a plus de durée légale . La durée légale , aujourd'hui , c' est 35h , et après , c' est les heures supplémentaires . Ben , il n' y en aurait plus . Voilà . Ou sinon , tiens , je vais vous prendre un autre type de droite , Macron . Ben lui , il propose que la durée soit négociée entreprise par entreprise , vous savez , &quot; au plus près du terrain . &quot; Là où vous êtes avec votre pote , le patron . &quot; Salut Marcel . &quot; &quot; Salut Paul . &quot; &quot; On va négocier ma paie : tiens , ce mois -ci ... &quot; &quot; Ce mois -ci , j' ai besoin de 4 heures sup ' , là , parce que il y a les impôts locaux qui sont arrivés , j' ai quand même besoin un peu ... On est d'accord ? &quot; &quot; Ouais , ouais , bien sûr . Vas -y . Reste au boulot , je te paye . &quot; Voilà . Dans quel monde vivent tous ces gens ? Dans quel monde vivent -ils , hein ? Ben , dans le leur . Alors , moi , je ne comprends pas les gens de droite qui vont voter pour des types pareils , des allumés pareils . Fin du CDI , pour M. Juppé . Dégressivité des allocations chômage . Vous savez pourquoi : &quot; Parce que vous êtes des fainéants . Vous aimez les allocs , et vous ne cherchez pas du travail . &quot; Quand quelqu'un vous dit ça , mais il vous insulte . Pourquoi il propose la dégressivité ? Vous avez moins besoin quand vous êtes depuis plus longtemps au chômage ? Ce n' est pas vrai , c' est le contraire , vous avez plus besoin . Pas moins . Pourquoi alors ils font baisser l' allocation ? Pourquoi ? Parce qu' ils partent de l' idée que vous êtes en train de truander . Dans leur esprit , tout le monde passe sa vie à truander , parce que eux truandent . Le fisc , sans arrêt . Alors ils pensent que tout le monde est pareil . 85 milliards par an de vol fiscal . 1.000 milliards par an en Europe . Alors , évidemment , ils se figurent que les gens normaux sont comme eux , et passent leur temps à essayer de voler les autres . Quoi , encore ? Ah , les cadeaux . Déjà , on leur a donné 40 milliards , n' est -ce pas ? Au MEDEF et au CAC40 , ça c' est à M. Hollande . Ben comme , évidemment , l' autre , il a lâché la bride , alors , du coup , maintenant , tout le monde y va . 14 milliards de plus veut leur donner M. Sarkozy . 35 milliards , M. Juppé . En avant , c' est la fête au MEDEF . 35 milliards , et qui va les payer ? Alors , les gens écoutent ça : &quot; Oui , c' est vrai , il faut baisser le prix du travail . &quot; Non , ils disent le coût , eux . C' est moi qui dis le prix . Baisser le coût du travail . Ah ouais . D'accord . Enfin , il n' y a que le travail qui produit de la richesse , hein , j' espère que vous l' avez compris , ça . Il n' y a pas une machine qui marche toute seule , hein . Ça n' existe pas . Il y a toujours des êtres humains . Donc c' est les êtres humains qui font ... c' est leur travail qui produit la richesse . Alors qui c' est qui va les payer , les amis ? Puisque vous êtes tous d'accord , vous êtes tous devenus tellement intelligents , que vous avez compris qu' il faut baisser le coût du travail pour améliorer l' économie , la compétitivité , la-globalisation-dans -le monde-mondialisé . Hein ? En un seul mot , s' il vous plait , avec des traits d' union . Qui c' est qui va payer ça ? Parce que vous croyez , vous , qu' on supprime les cotisations sociales et que vous continuez à avoir la Sécu ? Ah , ben , non , vous ne pouvez pas . Il faut que quelqu'un paye . Donc si vous ne le mettez pas là , il y a une loi . Et la loi , elle dit que chaque euro dont l' Etat décide qu' il est pris aux caisses de Sécurité Sociale , parce qu' on ne la leur fait pas , il doit le rembourser . Donc c' est vous , les vrais intelligents . C' est vous qui devrez payer , par vos impôts , ce que ne sera plus payé dans l' entreprise en cotisations . Parce que c' est la loi . Et si vous enlevez 35 milliards d' euros , et bien , vous les retrouverez à l' autre bout . Et c' est pour ça que le même M. Juppé , il vous propose d' augmenter la TVA à 23,5 % , c' est intelligent , ça . Augmenter la TVA . Non seulement c' est un impôt injuste , mais vous allez payer pour le savoir , c' est que plus ça coûte cher , moins vous achetez . Vous êtes comme ça , vous êtes radins , vous . Sous prétexte que c' est trop cher , vous n' achetez pas . Alors que normalement , vous devriez acheter . Bon , ben , si vous n' achetez pas , qu' est -ce qui va se passer ? Ben l' activité va réduire . Et si l' activité réduit , ben , il y aura moins de TVA qui rentrera . Moins d' impôts , moins de travail . C' est le cycle que ces gens là amorcent , et ils vous racontent ça pendant des heures . Et tout le monde est assis en face et trouve ça très intelligent . Oui , oui , comme ça , nous allons pouvoir vendre davantage de je ne sais pas quoi à je ne sais quel Lapon perdu dans un coin , à qui on va venir proposer notre marchandise qui coûte moins chère parce qu' on a extorqué les gens dans le pays d' origine . Tout ça est absurde . Ça ne marche pas . Ça ne marche pas . 100 milliards d' euros de coupes dans les services publics . Ah ouais , il l' a même répété , Sarkozy . 100.000 ... Je ne sais pas , c' est une espèce de compétition , on ne sait plus les chiffres , il y en a un qui veut en enlever 300.000 , l' autre 100.000 , bon . Tout ça est absurde . Ca veut dire que des tas de choses vont se désorganiser . Alors évidemment , il y a quand même des gens , tiens , des fois , il y a des journalistes qui disent : &quot; Mais vous croyez que c' est raisonnable ? Dans quoi vous les enlevez ? &quot; Alors , ils ne vont pas les enlever ... pas dans l' armée , pas dans la police , dans l' éducation ? Ah , non , non , pas dans l' éduc ... si , si , on peut quand même . Mais bon , pas trop . Alors , pas dans ci , pas dans là , alors où ? Vous vous moquez de nous . Dans l' éducation , comme d' habitude . Dans les soins aux personnes , comme d' habitude . C'est-à-dire tout ce que les pauvres gens et les gens normaux se payent par le service public . Le service public , c' est notre mutuelle . C' est ... nous finançons tous quelque chose qui nous sert à tous . Voilà ce que c' est , le service public . Donc , vous n' aurez plus votre service public , vous serez responsabilisés . Voilà , autrement dit , vous êtes des irresponsables . J' ai déjà vu ça pour les soins . Vous ne l' avez pas vu , celle-là ? Il faut responsabiliser les patients . Et bien , moi , je suis d'accord pour avoir une petite grippe en août . Pas de bol , ça tombe en même temps que tous les autres en novembre . C' est absurde . Tout ces mots sont fous . Ils ne parlent pas de la réalité humaine . Tout ça n' existe pas . Ils nous font un monde qui va être un enfer , c'est-à-dire ce qu' il était avant qu' il y ait , par exemple , la Sécurité Sociale , Ambroise Croizat et le reste . Alors , la seule chose qui profite de ça , écoutez , il y a une justice , il doit y avoir une justice , il faut absolument qu' il y ait une justice , sur quoi ils sont tous d'accord . Sur le chiffre des fonctionnaires , ils ne sont pas tous d'accord , l' un veut enlever 100.000 , l' autre 300.000 , sur les cadeaux à faire aux patrons , ils ne sont pas tout à fait d'accord , l' un veut leur donner 35 milliards , l' autre , seulement 17 , enfin bon . Sur la retraite non plus , l' un veut vous faire travailler jusqu' à 64 , l' autre jusqu' à 70 . Bon , vous comprenez , il y a débat . Et il faut enrichir le débat . Voilà . Mais il y a un point sur lequel ils sont tous d'accord . Il faut supprimer l' ISF . L' impôt sur la fortune . Parce que si vous comprenez qu' il y a une chose qui plombe l' économie française , mine le moral des producteurs , abat notre joie de vivre , c' est l' ISF . Ils sont tous d'accord . Vous applaudissez , vous ne comprenez rien à l' économie . Parce que je vous explique : si l' assiette du riche est bien pleine , tellement pleine qu' elle est trop pleine , alors il en coule . Et ça s' appelle le ruissellement . Je vous jure , c' est dans les manuels d' économie . Ça va couler jusqu' à la vôtre , d' assiette . Donc si vous permettez aux très gros d' être encore plus riches , ça va ruisseler un jour sur l' autre sur vous . Pourquoi vous ne le croyez pas ? C' est des gens intelligents qui vous le disent . Mince , ils ont fait des études , et pas vous . Allons , allons . Allons , ressaisissez -vous , je vous prie . L' ISF , vous savez quoi , ils mériteraient ... d'abord , moi , je me demande combien il y a des gens de droite , parce que , franchement , hein , ce n' est pas parce que quelqu'un est de droite qu' il paie l' ISF , forcément . Hein . Il y a des gens qui paient l' ISF qui sont de gauche , en plus , alors bon , c' est vous dire . Non , mais c' est possible , la conscience , ça dépend de plein de choses . Combien ça concerne de monde , ce sujet qui les passionne ? Qu' ils sont tous là à vouloir à tout prix le régler . A qui ils parlent , quand ils parlent comme ça ? Combien ils ont eu d' électeurs la dernière fois ? Un peu plus de 20 millions , non ? A qui ils parlent , à combien de gens ? 342.000 . Tout ce baratin , toute cette histoire , toutes ces postures , c' est pour 1,7 % des contribuables . 1,7 % . Et bien , voilà , ils mériteraient d' avoir comme score 1,7 % . Ceux qui ont l' ISF votent pour eux . Bon . Allez , il faut aller à la fin , là , hein . Vous savez , j' ai eu un courrier cette semaine , terrible . Mon adresse , c' est : agenda@jeanlucmelenchon . fr . Qu' est -ce que je reçois à mon adresse ? Une lettre : Mes propositions pour les entrepreneurs . Alain Juppé , Pour la France . Pardon ? &quot; Cher ami , lundi 17 octobre , j' étais au salon Planète PME . &quot; Ben , tant mieux . &quot; Le 29 septembre , je visitais , avec Xavier Niel , le chantier de la station F , et déjeunais avec des entrepreneurs . &quot; Tant mieux pour vous . J' espère que c' était bon , le manger . &quot; Un mois auparavant , je m' exprimais aux universités d' été du MEDEF . &quot; Là , je suis en train de vous lire du Juppé , ce n' est pas tous les jours que ça arrive , dans un meeting comme ça , hein . Donc , d'abord , il me raconte sa vie . OK . Avec qui il mange , où il était , qu' est -ce qu' il fait . Bon , très bien , mais je ... Pourquoi je recevrais une carte postale de M. Juppé ? Alors , après , il me dit que je peux lire son livre , etc . Après , il me dit : &quot; Pour revenir au plein emploi , il faut avant tout relancer l' investissement en supprimant l' ISF . &quot; Ce n' est pas beau , ça ? &quot; En baissant l' impôt sur les sociétés à 24 % . &quot; 24 % . Je l' ai entendu dire à la télé , c' est confiscatoire . J' étais jeune parlementaire , l' impôt sur les sociétés , c' était 45 % . D'accord ? Et on réduisait l' impôt sur les sociétés . Maintenant , c' est à 24 % , on est pas loin du taux irlandais , 12 % . &quot; Pour les PME et 30 % pour les grandes entreprises . &quot; Alors , là , les grandes entreprises , elles vont hurler , hein . Parce que c' est déjà à plus de 30 % , mais eux , ils ne payent que 8 % , tellement ils trafiquent . Bon , je vous passe le reste , parce que ça devient saoulant , à force . Bon , vous avez compris le truc , l' ISF , baisser les charges , blabla , blabla , tout ça . &quot; Il est aussi nécessaire de supprimer les freins à l' embauche . &quot; Le frein à l' embauche . &quot; Je veux revenir une fois pour toutes sur les 35h , et baisser les charges qui pèsent sur les entreprises . Je propose aussi de supprimer toutes les charges patronales pour les salaires entre 1 et 1,8 SMIC , et d' alléger de 2 milliards d' euros les charges pour les indépendants . &quot; Allez , 5 minutes , je fais l' indépendant . Où il parle du RSI ? Le RSI , c' est la Sécu des indépendants . Comme arnaque , il n' y a pas au-dessus . Non , mais , sans blague . Il n' y a pas au-dessus . Donc , moi , j' ai quelque chose à dire aux indépendants , que je vous propose , qui est dans le programme : si c' est moi le Président , tous ceux qui relèvent du RSI pourront revenir dans le régime général de la Sécurité Sociale . Voilà . Bon . Allez . Maintenant , il faut passer aux choses sérieuses , tout ça , c' était la mise en bouche . &quot; Tout cela , nous n' avons pas osé le faire entre 2007 et 2012 . &quot; Non , ce n' est pas qu' ils n' ont pas osé , c' est qu' on ne les a pas laissés le faire . Ce n' est pas pareil , parce que M. Juppé , il est quand même le vainqueur du Juppéthon , vous vous en rappelez ? Vous n' avez pas oublié , j' espère . Tous les jours , on comptait combien il y avait de monde dans la rue . Donc ce n' est pas qu' ils n' ont pas osé , c' est qu' ils ne l' ont pas fait , et c' est lui qui s' est dégonflé , et oui , parce qu' on était plus nombreux . &quot; Il est temps de passer à l' action : dans moins d' un mois , les 20 et 27 novembre , se tiendra la primaire de la droite et du centre , ouverte à tous . &quot; Mais je m' en fous . &quot; Vous pouvez dès à présent localiser l' emplacement de votre bureau de vote grâce au lien suivant : http : //www . alainjuppe2017/ouvoter/trouvervotrebureau . J' ai besoin de vous pour cette échéance majeure pour le redressement du pays . &quot; Mais c' est une blague ? C' est à moi qu' il envoie ça ? Vous me voyez ? Non , mais je vous dis ça parce que j' ai entendu dire qu' il y avait des ballots qui se faisaient des nœuds au cerveau , en se demandant s' ils n' allaient pas , quand même , aller voter à la primaire de ... de la droite , pour que ça soit Juppé plutôt que machin , que Pierre , que Paul , que truc , parce que de toute façon , comme ils pensent d' avance qu' ils ont perdu , et que de se battre , ça ne sert à rien et qu' essayer de convaincre les gens , ça ne sert à rien , ce que , moi , je pense le contraire et que je vais y arriver , bon . Donc ils vont aller faire ça . Vous vous rendez compte ? Vous vous imaginez , si vous avez une conscience , oh , il doit y en avoir des gens de gauche , quand même , dans cette salle . Supposez que vous soyez de gauche . Et que ça fait un moment , en général . Alors , vous vous voyez ? Vous rentrez , vous avez la honte . D'ailleurs , je l' ai lu dans un article du Monde , que , il y a des gens qui : &quot; Ouais , mais moi , je ne peux pas , parce que dans mon village , tout le monde me connait . &quot; Ben , oui , quoi . La honte . Vous rentrez en rasant les murs . Déjà , vous donnez deux euros aux Républicains . Et vous en êtes là de la honte , et là , il vous dit : &quot; Attendez , il faut signer là . &quot; Voilà ce que vous êtes censés signer , les malins , là : Je partage les valeurs républicaines de la droite et du centre et je m' engage pour l' alternance , afin de réussir le redressement de la France . Et bien , quoi qu' il arrive , je peux vous dire que , je pense que ce sera le cas de tout le monde dans la salle , parce que le premier qui était tenté par autre chose , maintenant , il a tellement honte qu' il n' y pense même plus , il dit : &quot; Moi , je n' y ai pas pensé . &quot; Le minimum , mes amis , on a tellement dénoncé le cynisme , à juste titre , des dirigeants politiques ; ça serait bien que les électeurs ne deviennent pas aussi cyniques que leurs dirigeants . Ça serait bien qu' on se respecte . On se respecte soi-même assez pour ne pas aller signer un papier qui dit le contraire de ce à quoi on croit . Et que l' on respecte assez ses compatriotes , des gens qui ne sont pas de votre avis , qui doivent être respectés . Parce qu' il n' y a pas de démocratie sans points de vue contraires , et on a besoin des points de vue contraires pour faire avancer la discussion . Donc il faut respecter ceux qui ne sont pas de notre avis . Et nous ne devons pas aller truquer leurs élections , en désignant , nous , à leur place qui va les représenter . Et peu nous chaut qui ils vont décider . Parce que des gens comme nous ne voteront jamais pour eux . En aucune circonstance . Voilà la vérité . Je vous appelle à ne pas truquer l' élection des Républicains qui ont le droit de voter comme ils l' entendent et de choisir les leaders qu' ils veulent . Parce que c' est à cette condition qu' on vit en démocratie . Et après , moi , j' ai une autre question à vous poser . On me dit : &quot; C' est un listing . &quot; Ah ouais , c' est un listing , vous croyez que ça me rassure ? Comment ça se fait que mon adresse est dans le listing de M. Juppé ? Non , mais sérieusement . Il y en a peut-être d' autres , ici , qui ont reçu une lettre de ce type . Comment se fait -il qu' il aie mon adresse dans son listing ? Est -ce que quelqu'un peut me répondre ? M. Juppé , vous pouvez me répondre ? Vous êtes tellement aux abois que vous avez besoin d' un gars comme moi pour faire la différence ? Non , non . Pour battre M. Sarkozy , je compte sur les urnes , celles auxquelles va tout le monde . Et puis après , je vais vous en dire encore une autre . Non seulement je ne sais pas pourquoi je suis sur son listing , deuxièmement , je n' ai pas envie de l' aider , troisièmement , je n' ai pas envie de signer ce papelard , mais j' aimerais bien savoir pourquoi je ferais confiance à quelqu'un qui envoie des lettres à des gens , et il ne sait même pas à qui . Ce n' est pas un grave problème ? Et après , le même vient vous dire qu' il vous regarde au fond des yeux et qu' il sait de quoi vous avez besoin . Supprimer l' ISF . Bon , voilà . Je vous ai dit tout ce que j' ai à vous dire . Je n' irai pas non plus dans la primaire du Parti Socialiste . Vous savez pourquoi . Par loyauté . Par loyauté . Je ne participe qu' aux élections pour lesquelles je suis prêt à me soumettre au résultat . Je participe au vote de ma patrie , et je me soumets au résultat du vote . Quand c' est un gouvernement de droite qui fait voter des lois de droite , je me soumets aux lois qui sont votées parce que c' est les lois de mon pays . Même si je continue à protester contre elles , même si j' agis avec mes camarades dans la rue pour ne pas qu' elles s' appliquent , mais je les respecte . Je n' irai pas dans une élection dont je ne suis pas prêt à respecter le résultat , parce qu' aucune des personnes que je vois ne correspond au degré de sincérité , d' opiniâtreté , d' application auquel je crois que nous avons le droit , maintenant . Je ne veux plus faire de compromis . Je pense qu' à chaque fois , on s' abîme , bien sûr que des compromis , on en fait dans la vie sans arrêt . Et tous ceux qui sont au travail , comme je le dis souvent aux camarades , je leur dis : &quot; Chaque jour que vous vous levez , que vous allez à l' entreprise , vous faites un compromis avec le capital . &quot; Donc , ce n' est pas la question . La question , c' est de renoncer à soi . Je vois des gens , il y a quelques jours , on m' a dit : &quot; Ah , mais tu pourrais t' accorder avec Montebourg . &quot; Mais pourquoi ? Pourquoi je m' accorderais avec lui ? &quot; Parce qu' il pense presque comme vous . &quot; Ben , dans ce cas là , je préfère moi . ( Rires et applaudissements ) Parce que c' est quoi le &quot; presque &quot; ? C' est quoi le presque ? Le presque , c' est qu' il a dit qu' il voulait vendre des logements sociaux , les vendre aux gens qui sont dedans . Je ne vais pas vous faire à cette heure -ci la démonstration de l' idiotie que c' est . Qui est -ce qui a inventé ça ? Thatcher . Pourquoi ? Parce que , ben , qu' est -ce qu' on vend ? Les bons appartements . Puis les autres , ils restent sur les bras de l' office . C' est lui qui doit les entretenir , puis les gens qui sont assez ballots pour avoir acheté l' appartement , après , ils découvrent qu' il faut les entretenir , ils n' ont pas les sous pour ça , et puis si vous venez en disant ... Vous ne croyez pas qu' on vend un appartement oui , un appartement non . Hein ? C' est : on vend tout . Et vous avez celui qui est là ou celle qui est là qui a passé les 60 ans , qui a eu sa vie et qui était là , dans un logement social , on lui dit : &quot; Il faut le racheter . &quot; &quot; Mais , je n' ai pas les sous . &quot; &quot; Ben , vous n' avez qu' à aller voir la banque . &quot; La banque , bien sûr , dès qu' elle voit quelqu'un qui a 65 ans ou 66 ans , elle dit : &quot; Ah , oui , oui . combien vous voulez ? Pour combien d' années ? Allez , tiens , pouf , on vous la met pour 25 ans . &quot; Tout ça n' existe pas . La vie des gens n' est pas faite comme ça . Ben , c' est une trouvaille stupide . Et de la même , on lui dit : &quot; Vous voulez ... il faut augmenter les salaires ? &quot; Oui , moi je dis : &quot; Il faut augmenter les salaires . &quot; Je reprends les slogans d' un certain nombre d' organisations syndicales . Dont l' augmentation du SMIC , et donc de tout le reste qui va avec . Mais lui , il dit : &quot; J' augmenterai les salaires après avoir proposé une négociation en Europe . &quot; Ben , fous -toi de nous . Pourquoi continuer à parler comme ça ? Dis : &quot; Je ne peux pas , je ne veux pas , ça n' est pas possible . &quot; Dis -le franchement . Dis -le ouvertement aux gens . Et les gens sont capables de comprendre si tu as de bons arguments pour le dire . Pourquoi tu leur mens ? Pourquoi tu dis oui , pour , dans le reste de la phrase , dire non ? C' est de tout ça dont on meurt . Assumez ce que vous voulez faire . Dites comment vous allez le faire . Réfléchissez -y . De même , vous voulez changer les institutions , vous voulez en finir avec la monarchie présidentielle , et avec tout ce qui va avec , à qui voulez -vous confier ça ? Une nouvelle fois , trois personnes qui écrivent ça dans un bureau et qui vous le proposent ensuite par référendum ? Ou bien à une assemblée ? Est -ce que ce n' est pas le peuple lui-même qui peut changer les conditions dans lesquelles on doit faire la politique dans le pays ? Voilà . Allez . Voilà pourquoi il ne faut pas avoir peur , il ne faut pas ... il faut y aller . Il faut convaincre . Chacun de vous , qui partagez mon opinion . Alors quand vous applaudissez , je suppose que vous êtes d'accord . Ce n' est pas tout de m' applaudir , il faut faire . Il faut faire . Il faut prendre sa petite liste , marquer ses 10 noms et puis faire le travail . Ce n' est pas compliqué , vous savez . On a fait 4 millions de voix , la dernière fois . Un pour un . On en fait 8 , on a gagné . Ben oui . On est au deuxième tour et le piège se referme sur eux . Ce n' est pas si compliqué que ça . Il faut donc convaincre . On est en démocratie , on a cette chance . Et ben , on doit convaincre . Pas hypnotiser , subjuguer , abrutir , tromper , tricher , à quoi bon ? A quoi bon ? Convaincre . Un après l' autre . Une après l' autre . Voilà . Et puis ça fait une majorité . Ou ça fait en tout cas le nombre suffisant pour ensuite rassembler une majorité . Moi , je pense qu' on peut y arriver . Et puis , je ne vois pas ce qu' on pourrait faire d' autre . Ah , il y a des gens , bon , ils veulent se rassurer , ben , ils se rassurent , hein . Mais ils ne vont pas se rassurer longtemps . Parce que si c' est pour aller voter dans les primaires de droite parce que Juppé , je ne sais pas quoi , au deuxième tour , machin , etc . : retraite à 70 ans , bonhomme . C' est pour ça que tu as fait tout ça ? Toutes ces combines , ces arrangements avec toi-même , avec ta conscience , aller signer des papiers auxquels tu ne crois pas , et donner de l' argent à des gens que tu n' aimes pas . Tout ça pour ça ? Pour vous renier ? Pour tout abandonner alors qu' on peut faire tant de choses magnifiques , belles , immenses , j' espère que tout à l' heure , je vous en ai convaincu . De toutes ces choses qu' on va pouvoir faire dans ce pays , qui ne demandent que de la volonté . Ça ne manque ni d' argent , ni de savoir-faire . Il faut qu' il y ait des gens qui arrêtent d' avoir peur . De tout , de rien . S' ils veulent être les bons élèves de la mondialisation . Du courage . De la volonté . Oh , il n' y en a pas besoin de tant que ça , hein . Bon , maintenant , c' est fini . Alors , je lis une petite citation , toujours . D' habitude , je vous lis des poèmes , mais pas là . Comme c' était sur un moment un peu tendu , là , sur les primaires de la droite . Je vous lis juste une chose que j' ai lue , qui m' a bien plu , ça parle aussi de moi , alors c' est pour ça , peut-être . Tandis que trouver quelque chose qui parle de moi dans Victor Hugo , je peux toujours chercher , mais bon . C' est Laurent Binet qui est un écrivain . De notre pays . Il dit , après le vote des Etats-Unis , hein . Il dit ça : &quot; L' histoire des luttes politiques en résumé , deux-points : on espère avec Bernie Sanders , on se résigne avec Clinton &quot; , suivez mon regard , &quot; et à la fin , on a Trump . Ça marche bien aussi avec Mélenchon , Hollande et Le Pen . &quot; C' est un grand auteur . Bon . Et comme je suis chez vous , et pour finir pour de bon , juste 4 lignes d' Ambroise Croizat - moi , je ne suis pas communiste , hein , mais je pense qu' il y a des communistes dans la salle , ça va leur faire plaisir , et puis comme Ambroise Croizat est un grand Français , et un grand militant , et bien ça nous fera plaisir à tous . Voilà . Parce que c' est homme là , il appartient à tout le monde , comme toutes les belles pages de l' histoire sociale de la patrie . Alors , il dit ça : &quot; La lutte continue pour la renaissance française . &quot; C' est un communiste , hein , qui parle . &quot; La lutte continue pour la renaissance française . L' indépendance nationale et l' instauration d' un bien-être pour tous - l' indépendance nationale et l' instauration d' un bien-être pour tous sont indissociables . &quot; Vive la Sociale , mes amis . ",
    "Nous n' avons pas de sympathie pour Poutine , et ni Jean-Luc Mélenchon ni la France insoumise n' en ont la moindre . Nous partons d' une analyse géopolitique globale et nous constatons que les Etats-Unis et leurs alliés veulent affaiblir , sinon éliminer , toutes les puissances économiques ou militaires qui menacent leur hégémonie . Ce travail s' effectue aussi par la courroie de transmission que sont les médias atlantistes , qui servent à préparer l' opinion et justifier des guerres , avec des éléments de désinformation souvent flagrants . Ce qui est reproché à Jean-Luc Mélenchon est d'abord de déconstruire le discours de ces médias partisans . Et sa ligne directrice est d' être vigilant pour dire que , en dépit d' éventuels massacres ou faits de guerre , cela ne doit pas justifier des interventions étrangères , c'est-à-dire que nous défendons un principe de non-ingérence . On l' a vu en Libye où , bien que présentés comme une intervention humanitaire , les bombardements de notre gouvernement associés à d' autres ont entraîné un chaos total . Or , en tant que citoyens français , nous avons une responsabilité devant le monde entier lorsque la France intervient militairement [ ou soutient des agressions meurtrières comme actuellement contre les peuples palestinien ou le peuple yéménite . Comme l' explique Noam Chomsky , nous avons un devoir prioritaire à dénoncer les ingérences commises par nos gouvernements , à faire pression pour que cessent les massacres dont nous sommes responsables . ] Dans ce cas précis , il y a deux lignes d' analyse diamétralement opposées . L' une est de dire qu' il y a une intervention de la Russie sans en préciser le contexte ; l' autre est de souligner qu' il y a des attentats terroristes , une intervention djihadiste étrangère , en Syrie et que la population a peur . Ce n' était pas le cas au début du soulèvement en 2011 , mais aujourd'hui , une grande partie de la population syrienne a peur du Front Al-Nosra , du Front islamique et de Daech . [ L' intervention russe et le maintien du régime d' Al-Assad se donnent comme justification de faire rempart à ces bandes sanguinaires , qui sont indirectement soutenues par l' Occident , puisque celui -ci arme le Qatar et l' Arabie saoudite , pays qui font ensuite provenir ces armes aux groupes salafistes . ] Nous n' avons aucune sympathie pour le régime syrien , mais la priorité devrait être d'abord d' éliminer ces groupes qui terrorisent la population syrienne et donnent des arguments à Assad . Et la vraie question à poser est celle de la responsabilité de la France et de nos alliés dans cette ingérence . Jean-Luc Mélenchon dénonce la logique de guerre contre Moscou qui s' exprime à travers les grands médias . Ceux -ci instrumentalisent tous les problèmes et tout ce qui se passe en Russie afin de déstabiliser ce pays . Tout en s' appuyant sur des éléments exacts , la propagande de l' OTAN les détourne pour ses propres intérêts impérialistes contre la Russie . Il faut traiter d' égal à égal avec Poutine , qui n' est pas un interlocuteur moindre que Merkel ou Donald Trump ! Il faut quand même se poser la question de comment se voit la Russie elle-même face au reste du monde . C' est un grand pays , mais qui est entouré de bases de l' OTAN , partout en Europe centrale et orientale , et qui fait en sorte de résister aux ingérences et aux provocations étrangères à ses portes . Quant à la Syrie , ce que nous savons , c' est que la Russie a proposé dès 2012 , dans le cadre des accords de Genève , une solution négociée dont il est généralement admis aujourd'hui qu' elle était sérieuse et réaliste . Il s' agissait d' un processus de transition qui avait même été accepté par une partie de l' opposition . Et c' était pour la plupart des observateurs une bonne base de départ , pour bien sûr poursuivre les discussions . Qu' en est -il trois ans plus tard , en 2015 ? Les accords ont été refusés et la situation sur le terrain a vu les djihadistes aux portes du pouvoir . C' est à ce moment-là que la Russie est intervenue ! Et c' est ce qui l' a empêché de s' emparer du pouvoir ou du moins de les repousser d' une grande partie du territoire syrien . Il est difficile de savoir avec précision ce qu' il en est , surtout avec des djihadistes comme le Front Al Nosra qui dominent ceux que la presse occidentale appelle &laquo; les rebelles &raquo; . Mais une partie de la population syrienne , s' est rangée du côté du régime de Bachar Al-Assad car il est devenu un rempart pour elle , contre l' avancée et les exactions de ces groupes djihadistes . C' est malheureusement cela la situation aujourd'hui . Et c' est bien à partir de celle -ci qu' il faut essayer de lancer un processus de négociations comme dans tout conflit armé . La position de Jean-Luc Mélenchon depuis 2012 n' a pas varié sur ce point : cessez-le-feu , arrêt du bain de sang et négociations entre le régime et l' opposition . Quant à la nature non-démocratique du régime russe , nous insistons sur le fait que si il s' agissait d' une dictature comme les autres mais alignée dans le camp de l' OTAN , nos dirigeants seraient beaucoup plus complaisants à son égard . C' est donc en fait sa politique de non-alignement qui entraîne les nombreuses attaques faites à son encontre . ",
    "Salut les p'tits ! Ça marche ? Est -ce que ça marche , là ? Est -ce qu' on m' entend ? Bon ... Là , faut arrêter la musique ... Merci . Bon ... A vous tous , euh ... Merci d' être venus ... d' être si nombreux ... Et je vais vous dire d'abord pour commencer la même chose ... que ce que je viens de dire à tous ceux qui , hélas ... n' ont pas pu entrer ... Et qui sont dehors ; je sais pas combien y en a ... peut-être autant que dedans ... Bon , ils resteront moins longtemps , vu le froid ... J' ai été les saluer ... je leur ai dit ... que je leur présentais des excuses ... après les remerciements que je dois à tous ceux qui se sont donnés le mal ... d' organiser cette soirée ... des militants , des bénévoles ... Et attention ... C' est pas parce que ce sont ... des bénévoles ... que ce sont des amateurs ... car , voyez -vous ... Nous comptons dans nos files ... les meilleurs professionnels ... de leurs branches ... Forcément , c' est nous le peuple ... Et y a pas besoin de payer , mais quand même ... Mais quand même ... Si nous ne pouvons pas accueillir comme nous le voudrions. . Tout le monde ce soir ... je vous dis les choses , crûment et franchement ... C' est qu' il n' y a pas assez d' argent ... parce que pour avoir une salle pour faire rentrer 3000 personnes ... Ça coûte 50.000 euros ... Une soirée comme celle -ci au total ... C' est 12.000 euros ... Je veux que vous le sachiez ... Rien n' arrive comme ça ... Tout est fruit d' efforts ... Et de mobilisation ... Plus de 20.000 personnes ... Déjà , ont donné chacun leur petit écot ... Et c' est notre force qui , pour terminer , repose tout entière sur le nombre ... Et le dévouement , que chacun d' entre nous avec son intelligence ; parfois , avec ses très faibles moyens mobilise au service de la cause ... Nous ne sommes pas en train de célébrer une personne , un candidat ... nous sommes en train de célébrer des idées ... et nous sommes en train de faire une démonstration de force … de nos idées … Et ceux qui sont là dehors à se geler ... de toute façon ils ont marqué un point ... comme vous-mêmes ... car nous n' étions pas là pour autre chose que pour montrer ... la continuité , la force , la profondeur de ce que nous représentons dans le temps et dans l' histoire ... quand ils pensaient en avoir fini avec nous ... pendant toutes ces années ... où l' argent régnait sans contestation ... et où on proposait à chacun ... de n' avoir aucun autre horizon dans la vie ... que toujours d' accumuler davantage ... fusse au détriment des autres ... jusqu' à finir dans ce rêve piétiné ... nos paroles salies et souillées ... quand un de ceux qui se réclame de nous ose dire à la jeunesse ... je vous propose comme rêve d' être milliardaire ... Être milliardaire est immoral ... Accumuler est immoral ... La richesse est toujours accumulée au détriment ... des autres ... A quoi bon ... ... avoir un milliard , si c' est pour souhaiter en avoir un deuxième ? A quoi bon cette course permanente à l' ostentation ? Bon ... Un mot pour les ballots qu' ont été donner deux euros ... Deux fois de suite ... Non . Ne les huez pas ! On va leur proposer de se racheter ; c' est un langage que tout le monde comprend ... Vous avez donné deux fois de suite deux euros ... Il y en a pour un 1,6 millions que vous avez été donner à nos adversaires ... Alors , il faut en donner un peu ici ... Appel aux dons ... Vous avez donné deux fois deux euros ; vous m' en devez quatre ... Pour que quand viendra votre tour devant votre conscience ... Vous puissiez vous dire : bon ... Pourquoi est -ce que je suis venu à Bordeaux ? Eh ben , parce qu' il y a cinq ans , j' étais à Talence . Et qu' aujourd'hui ... Nous ne serions ni si nombreux ... Ni si déterminés ... Ni si nouveaux , pour certains d' entre vous , compte tenu de votre âge ... Si auparavant ... Nous n' avions déjà accumulé , rassuré cette force ... En lui montrant qu' elle existait ... Comme nous avons commencé à le faire ... En 2012 , en nous rassemblant à quatre millions ... Avec le même bulletin de vote ... Mais , je pourrais aussi être venu pour ... S' il vous plaît ... Allez ! Voilà ! Honneur au vaincu ... J' aurais pu venir en me disant : &quot; Eh bien , je viens dans la ville ... de celui qui est le président ou va être le président de la République ... &quot; Enfin , déjà vous autres , les ballots ; vous avez donné des sous pour acheter tout ça ... A -t-il gagné déjà ? Eh bien , vous avez payé pour lire ça ... Moi , je plains ... Je le dis ... sans moquerie ... Ceux qui se sont laissés affoler ... Et qui continuent maintenant ... A croire aux même fadaises ... qui ont été un peu renouvelées ... Je les plains ... Comment vous avez pu vous fourrer dans le tête ... Qu' étant ... de gauche ... Ou en tout cas , des progressistes ... Vous aviez partie liée ... Avec le vote ... De ceux de droite , qui ont bien le droit d' organiser des votes ... pour désigner leur leader ... Leur candidat ; c' est leur droit ... nous devons le respecter ... Nous n' avons pas à aller tricher dans les élections des autres ... Nous n' avions pas à y être ... Eh ben , ils y sont quand même allés . Et pourquoi ? L' intention était bonne ; j' en suis sûr ... Pas très réfléchie , mais bonne ... Alors ... Ils allaient voter pour lui pour empêcher que l' autre ... qui ensuite , verrait le premier ... Qui ... passerait ... Derrière l' autre et ... l' affaire serait dite ... Qu' est -ce qu' ils sont intelligents ! Ça , c' est la base du vote utile ... La culture du vote utile ... Le vote utile , c' est toujours un vote futile ... Parce que le seul vote qui est important pour une citoyenne ou un citoyen ... c' est celui qui fait le bien de tous ... Dont vous concluez en conscience ... que c' est le bien de tous ... C'est-à-dire , le vote nécessaire ... Et si je me présente devant vous ... Ce n' est pas en vous disant , je suis le vote utile , parce qu' aujourd'hui les sondages ... me mettent en tête de ma famille politique ... Je viens devant vous comme vous le savez ... et comme beaucoup d' entre vous en sont déjà d'accord ... parce que j' estime que c' est le vote nécessaire ... celui de l' avenir en commun . Nous autres ... Nous sommes comme nous sommes ... pour plein de raisons ... Et pas seulement ... A cause de ce que nous dit notre feuille de paie ... La vôtre ... Moi , la mienne , ça va ... Vous le savez bien ... Mais justement ... Je m' adresse aussi à ceux qui sont dans ma situation matérielle confortable ... Ceux qui ont une bonne situation ... doivent courir devant pour aller aider les autres ... parce que c' est eux qui ont les moyens ... et le temps qui les met à la disposition des autres ... Nous sommes comme nous sommes pour bien des raisons ... Et d'abord , souvent , pour des raisons morales ... Ce qu' on appelle des raisons morales ... Nous ne supportons pas la souffrance des autres ... Nous ne supportons pas l' abandon ... Ni celle des être humains , ni celle des animaux ... Ni le massacre de la nature ... Toutes ces choses heurtent en nous quelque chose de très profond ... Alors , il faut être comme la pointe du diamant ... Ne jamais en rabattre ... ni de ses rêves ... ni de ses objectifs ... Ceux qui s' emmêlent les pieds ... Dans les tristes et pauvres aventures du vote utile ... Comme le moindre mal ... Disent -ils , pensent -ils ... souvent honnêtement ... bravement ... Le moindre mal ... Ne l' oubliez jamais ... C' est quand même toujours le mal ... Hannah Arendt ... Je vous ai amené cette citation ... Dans son livre qui s' appelle &quot; Responsabilité et jugement &quot; , dit ... &quot; Politiquement ... la faiblesse de l' argument du moindre mal ... a toujours été que ceux qui choisissent le moindre mal ... oublient , très vite ... qu' ils ont choisi le mal ... &quot; Alors ... Bon ... Si je ne viens pas pour Juppé , je pourrais venir pour Montesquieu ... Il est de votre coin ... Je m' intéresse toujours à qui est -ce qui est du coin ... Tout à l' heure , on aura mieux que Montesquieu , qui était quand même un gros réac , mais enfin ... En 1772 ... La lumière de la Grande Révolution n' était pas encore passée ... Lui a théorisé la séparation des pouvoirs exécutifs et législatifs ... Alors voilà ce qu' il dit ... On va le proposer comme une pensée contemporaine ... &quot; Lorsque dans la République , le Peuple encore a la souveraine puissance ... c' est une démocratie ... Lorsque la souveraine puissance est entre les mains d' une partie du Peuple ... Cela s' appelle , une aristocratie ... &quot; Eh ben voilà ; c' est exactement ce dans quoi nous sommes ... Nous sommes sous le pouvoir ... d' une aristocratie ... d' Ancien Régime , qu' aujourd'hui nous appelons l' oligarchie ... Oligarchie ; c' est le gouvernement de quelques-uns ... Ils sont quelques-uns qui possèdent tout ... et qui tirent les ficelles ... de politiciens , de politiciennes et de médiacrates de toutes sortes ... J' ai dit du mal de Montesquieu ... Bon . Au moins , il proposait ça , et c' était un esprit avancé ... qui éclairait la réflexion de son époque ... Il croyait qu' on pouvait abolir l' absolutisme ... sans abattre la monarchie ... C' est comme ceux qui croient ... qu' on peut libérer les humains et la nature de la prédation ... sans abolir la tyrannie ... de la finance ... Non . Ce n' est pas possible ... On ne pouvait pas abolir l' absolutisme sans abattre la monarchie ... On ne pourra pas libérer les êtres humains et la nature sans abattre l' absolutisme de la finance ... C' est un raisonnement ... et un manifeste ... Je me présente devant vous , cent un ans après la loi de 1905 de séparation des églises et de l' Etat ... pour vous proposer la séparation du MEDEF et du gouvernement ... La chose extraordinaire ... C' est quand même ... Tout à l' heure , on a montré M. Juppé ... C' est qu' ils ont fini par trancher ... C' est bien ... Ça c' est pas extraordinaire ... Et ils ont désigné un candidat ... Qui est M. Fillon , à qui ( coupure vidéo ) Mais , c' est complétement immoral ... Donc , on va dire une chose . Voilà un candidat ... qui est surtout et d'abord ... entouré d' une bande de menteurs ... Ce n' est pas rassurant ... Maintenant ... J' en viens au fond ... Parce que c' est de cela dont il va être question ce soir ... et des arguments ... qu' il faut que nous portions en nous , qui doivent essayer d' éclairer ... notre regard ... sur la situation que nous vivons ... C' est une bonne chose ... que la droite ait choisi M. Fillon . Parce que c' est un cas ... politiquement clair ... Il propose une synthèse qui était contenue ... Peu visible ... Peu manifestée , de ce qu' est vraiment la droite dans notre pays . C'est-à-dire , deux choses ... Le libéralisme économique le plus absolu ... et le conservatisme ... intellectuel et moral , le plus total ... Et il en est une bonne synthèse ... C' est pourquoi , pour ma part ... Je me réjouis ... du fait que nous allons pouvoir ... confronter nos idées ... car nous sommes l' exact inverse de cela ... Mais pour pouvoir avoir ces débats , il faut des gens qui assument ... leurs positions . Et celui-là ... m' a l' air d' être prêt à les assumer ... sans faux semblants ... sans fausse casquette , sans faux nez ... sans masque , sans déguisement ... Et c' est pourquoi ... je serais heureux , s' il acceptait ... le débat que je lui propose. . Devant tout le monde ... pour qu' on entende , au moins un soir ... le choc ... naturellement ... courtois , c' est comme ça qu' on le ferait ... des deux visions du monde ... Parce que ce qui est en cause ... ce sont deux visions du monde ... Au fond ... Ce sont les deux cohérences que nous incarnons ... Lui et moi , en votre nom ... Quelle cohérence ? D' un côté ... suivant la vieille maxime ... le nôtre ... l' idée qu' il y a un avenir en commun ... Et que c' est ce commun ... qui permet cet avenir ... pour l' homme ; l' être humain , on va dire ... et son rapport à la planète ... En gros , s' il fallait prendre une petite formule , on dirait ... C' est , euh ... Un pour tous , tous pour un ... Ce n' est pas que nous soyons contre la compétition ... Bien sûr que non ... Il y a plein de compétitions dans la vie ... des compétitions amoureuses , des compétitions de toutes sortes ... Mais ... nous sommes pour la compétition , à condition qu' elle n' empêche pas la solidarité ... Et eux , de leur côté ... c' est l' idée inverse ... Ils veulent bien de la solidarité ; ce ne sont pas tous des sauvages ... Il y en a un nombre de sauvages ... Mais ... Ils sont comme nous ... Ils veulent bien ... la solidarité mais à condition que ça n' empêche pas la compétition ... Au fond , leur maxime à eux et en particulier celle de M. Fillon , c' est ... Chacun pour soi et ... Dieu pour tous ... Ce qui va nous ... Ce qui va nous mettre en mouvement nous , ce qui va organiser la cohérence du programme &quot; L' avenir en commun &quot; que je vous demande de lire , c' est une vision d' ensemble . Là , je viens de vous en livrer le principe moral mais après , il y a le mode d' exécution Mes amis , nous , nous sommes conscients du fait qu' on règle les problèmes à partir des problèmes , pas à partir de seulement l' idée qu' on s' en fait . Le problème urgent et immédiat ... et celui qui va retentir comme un défi qui va nous permettre d' avancer , ... c' est que la maison brûle , comme disait Chirac . Franchement , quand 60 % ... 50 % des espèces vertébrées ont disparu de la planète , quand la chaleur est montée de 20 points au-dessus du seuil auquel ... il est normal qu' elle se trouve dans l' Antarctique , Est -ce que vous croyez vraiment , qu' à ce moment-là , la question qui se pose , c' est celle de l' ISF ? Ne croyez pas ... que je caricature ! Donc , nous , nous sommes toujours partis de la demande , la politique ... du clan progressiste , ça a toujours été de partir ... de la demande , c'est-à-dire , qu' est -ce qui est nécessaire ? Que faut -il faire ? Et bien , là , nous avons un problème , nous devons nous mobiliser tous ... pour répondre au défi qu' a créé la prédation excessive sur la nature , mais au moment de le faire , quand on dit à tout le monde : &quot; Allez , on va faire l' effort ! &quot; Qu' est -ce qu' on découvre ? Que la société est pourrie d' inégalités , qu' il y a des masses de pauvres , des milliers de gens qui sont hors-circuit , qui n' ont qu' une chose à faire chaque jour , c' est de se demander comment ils vont tenir jusqu' au lendemain , et le jour d'après , pour eux et leurs enfants . L ' inégalité ronge l' unité de l' espèce humaine , et sa capacité à être ensemble pour faire face à cet énorme défi qui avance . Les deux choses se tiennent , le programme politique que nous présentons répond aux deux questions , pour faire face au défi du changement climatique , il faut changer complètement le mode de production , le process de production , et notre manière de consommer ainsi que la répartition des richesses , et pour le faire , il faut mobiliser des cohortes immenses de gens . Parce que c' est un travail gigantesque de faire tout ça , donc nous avons besoin de milliers d' emplois , et nous avons besoin de ces emplois pour faire cette bifurcation et pour récupérer tous ceux des notres qui ont été abandonnés en route , ceux qu' on a pas soignés , ceux qu' on a pas éduqués et qu' il nous reste 2 millions et demi de personnes illettrées dans le pays . Ceux qui sont malheureux , cassés , détruits par la violence d' un système social qui les a abattus , parce qu' on leur a dit : &quot; chacun pour soi , et si vous êtes pauvres , c' est de votre faute ! &quot; Voilà cette morale pourrie , qui a empêché qu' on fasse , la mobilisation humaine dont on a besoin . Quand quelqu'un vous parle de la sixième extinction des espèces , dites -vous que la réponse , elle est sociale . Elle commence par dire que pour être tous à la manœuvre , il faut que chacun puisse vivre dignement de son travail , et que son travail ait une utilité qui permette à tout le monde de faire face aux défis . Alors , évidemment , de l' autre côté , il y a une autre vision du monde . On la connaît : &quot; Remplissez l' assiette des riches ! Et alors , il y en aura tellement qu' il en dégoulinera quelque chose . &quot; Ça s' appelle la théorie du ruissellement . Ils vont donc ruisseler ... mais nous n' avons pas vu ... que cela ruisselât beaucoup ... jusqu' à présent . Et ça s' appelle la politique de l' offre . Je suis obligé de résumer parce que tout le monde n' a pas idée bien claire de ce que c' est . La politique de la demande , je viens de vous l' expliquer : de quoi a -t-on besoin ? Alors , de quoi a -t-on besoin ? On répond à la question et on s' en donne les moyens . La politique de l' offre , c' est : &quot; Produisez n' importe quoi , n' importe comment , dans n' importe quelles conditions , pourvu que ça ne soit pas cher , et vous trouverez des gens pour vous l' acheter . Et si vous l' achetez , alors vous devrez en produire davantage et ça fera le bonheur de tout le monde , gnagnagna , flexibilité , compétitivité , blablablbl et blablbl ... et l' emploi . &quot; Voilà , vous avez compris ? Résumé : quelqu'un frappe à la porte , toc toc toc , nous sommes au pôle Nord , vous êtes un Esquimau , vous frissonnez - enfin , plus tellement en ce moment , je peux vous faire la même chose dans le Sahara , hein - et la porte s' ouvre et il y a un type qui dit : &quot; Bonjour , je vends des frigos . &quot; L' autre dit : &quot; Ah ben , non , moi , c' est un chauffage . - Pas du tout , c' est pas la question , mon frigo n' est pas cher , achetez -le . &quot; Voilà , c' est ça , la politique de l' offre , c' est absurde . C' est absurde ! Ne vous laissez pas impressionner ! Ne vous laissez pas impressionner par ceux qui vous disent que tout ça a un sens : ça n' en a pas ! L' activité économique humaine était destinée , et elle l' est depuis que l' humanité est humanité , à satisfaire des besoins réellement exprimés . Et ceux-là satisfont des besoins qui n' existent pas , en retirent les moyens aux gens et créent les nouveaux besoins . Et c' est cette société dans laquelle nous vivons , du gaspillage . Où l' on détruit , où 40 % de la production agricole est mise à la poubelle , parce qu' elle n' est pas conforme . Et c' est ce monde là qu' ils veulent faire durer . Avec une absence totale du sens des responsabilités Ecoutez -moi , les amis , le plus grand scandale de ces débats qu' ont eu ... nos ... nos concitoyens de droite , c' est qu' ils n' aient pas parlé une seule fois des problèmes de la planète , pas une seule - un mot ! - pour parler d' écologie et de la manière d' y répondre . Et ils ont désigné quelqu'un , qui est un productiviste aveuglé . Ca commence vraiment très mal : il propose d' abroger le principe de précaution , qui a été inscrit dans la Constitution . Notez que c' est Chirac qui l' y a mis . Et ben , on le remercie . Et la première fois qu' on a mis ça dans la loi , c' était M. Barnier qui l' avait proposé , juste après le premier sommet de la Terre , à Rio en 1992 . Je suis tellement vieux que j' y étais . Donc je m' en souviens . Ce principe dit qu' on ne fait pas quelque chose ... si il y a un risque que cela dégrade , ou que cela présente un ... un inconvénient ensuite , qui serait irréversible ou très dommageable . Est -ce qu' il y a aujourd'hui une abondance de recours , qui fait qu' à cause des principes de précaution , tout est bloqué ? Jamais de la vie , il n' y en a pas eu un seul ! Alors , pourquoi ils le retirent ? Pare pure idéologie ! Parce qu' ils pensent que le problème ne se pose pas , parce qu' il est aveuglé et qu' il ne veut qu' il n' y ait aucun obstacle à la libre circulation du capital , à ses libres investissements , quoi qu' il produise , quoi qu' il fasse , alors même que nous savons aujourd'hui qu' à cause des glyphosates , et combien d' autres pesticides , la vie humaine est lourdement perturbée . La fertilité , la reproduction des êtres humains sont lourdement mis en cause Pour ne pas parler de tout ce que ... cela entraîne aussi sur la santé des nouvelles générations . Le même homme propose de prolonger les centrales nucléaires de 40 à 60 ans . Ecoutez , franchement , après Fukushima ... Non , quoi ... Non ! Il faut y réfléchir avec sérieux . Moi , vous savez que je suis pour la sortie du nucléaire , je n' ai jamais cru une seule seconde ... que on irait , on appuierait sur un bouton , et il n' y a plus de nucléaire . Il y en a pour 20 ans ou 25 ans à en sortir . Mais c' est dangereux dès maintenant . Et le dire , ça n' est pas mettre en cause ceux qui y travaillent ! Ni ceux qui ont fait tourner tout ça jusqu' à aujourd'hui , dans des conditions de sécurité qui font que les incidents , n' ont jamais dégénéré en accidents . Mais quand ça devient dangereux , ça l' est pour de bon . Je ne vais pas vous faire peur , mais c' est quand même à côté de chez vous qu' il y a la centrale du Blayais . Oui , ben , il faut y penser , hein ! Moi , je vous le dis , je la fermerai . Mais je voudrais que vous compreniez une chose . C' est que pendant que nous sommes en train de parler , le changement climatique fait monter le niveau de la mer . La dernière fois , grâce à un ingénieur particulièrement intelligent , nous avons été saufs , parce qu' il a compris qu' avec un vent de 140 km/h et des creux de 7 mètres , les prises d' eau allaient être bloquées , et que donc , la centrale aurait un accident . Cet homme a compris qu' il fallait arrêter la centrale , et il a bien fait parce que son pronostic s' est vérifié , en effet , les prises d' eau ont été bloquées . Vous êtes à 50 km de cette histoire . La flotte monte . De toute façon . Que vous soyez contents ou que vous le vouliez ou pas . Que M. Fillon accepte l' idée qu' il y ait un changement climatique ou qu' il ne l' accepte pas . De toute façon , il a lieu . Et donc , le niveau de la mer monte . Et donc , dans 20 ans , dans 30 ans , quand on arrivera au bout du cycle qu' ils décrivent , la probabilité que la centrale soit noyée , sans avoir rien besoin d' autre que de cette montée des eaux cette probabilité est extrêmement élevée . Le responsable politique doit prendre en compte des problèmes de cette nature , en ajoutant que nous avons une difficulté : nous ne savons pas réellement démonter les centrales , celle sur laquelle nous avons commencé à le faire , 10 ans après , on y est toujours . C' est une grande difficulté technique , c' est pourquoi , contrairement à ce que disent certains , démanteler le nucléaire , qui ne garantit nullement notre indépendance , parce qu' il faut aller chercher le combustible dans des pays où on est pas toujours bien disposé à nous le donner . Entre autres choses . Et bien , si nous voulons sortir du nucléaire , c' est un défi technique et scientifique et de recherche immense . D'abord parce qu' il faut mettre en route tout le reste pour que ça marche . Je sais très bien que si vous m' élisez , je ne peux pas me contenter de dire à tous les Français : &quot; Ah ben , écoutez , on va en sortir &quot; , sans expliquer ce qu' on va faire d' autre . Ce qu' on va faire d' autre , c' est développer toutes les autres énergies . Il nous faut du 100 % renouvelable . J' ai passé - vous savez , le programme , ça fait un moment qu' on travaille dessus , hein , croyez pas que j' arrive en sortant ça de mes poches , et les discussions avec les amis , et avec les camarades , et avec les spécialistes , avec des gens qui ne sont pas forcément nos amis politiques , mais à qui le simple fait qu' on propose un horizon de développement , de leur propre intelligence , de leur capacité de savoir , soulève d' enthousiasme . Nous avons besoin que , dans les deux premières années du mandat , on installe , par exemple en mer , une puissance de production électrique , ce qui est faisable , dans les deux premières années , équivalant à au moins une centrale ou deux , pour que les Français comprennent que c' est possible , qu' on sait le faire . Nous avons par milliers des jeunes gens , et des moins jeunes , femmes et hommes , ingénieurs , techniciens , ouvriers hautement spécialisés , qui savent faire tout ça . Parce que c' est de très haut niveau technique , contrairement à ce quoi croient les imbéciles , qui nous disent sans cesse qu' on va retourner dans les cavernes et s' éclairer à la bougie . Comme ils nous le disent . Ou qu' ils croient qu' ils ont répondu à tout en disant : &quot; -Ah , ben on va faire des voitures électriques ! &quot; &quot; Ah oui , et , benêt , tu la trouves où l' électricité ? &quot; Pour un million de voitures électriques , il vous faut une centrale nucléaire de plus , vous appelez une solution ? Non , ça n' en est pas une ! Aussi longtemps qu' on ne sait pas produire de l' électricité - et quand je dis &quot; on ne sait pas &quot; - , on ne peut pas produire de l' électricité dans de nouvelles conditions . Alors , maintenant , voilà ce qu' il a trouvé , l' autre . &quot; Donner la main au marché pour les énergies renouvelables &quot; . Non , ça , c' est donner le portefeuille , je connais . C' est la privatisation des profits . Par exemple , on a donné la main au marché pour les ports dans notre pays . Les &quot; ports &quot; , les bateaux , hein . Pas les cochons , les cochons , il y a longtemps que ... Il y a longtemps que c' est le marché qui décide , mais ne rigolez pas , c' est triste . Parce que comme c' est le marché , quand ça a marché , et bien , ils ont fait d' immenses élevages , où les animaux sont traités d' une manière ignoble , et l' année d'après , comme il y a trop de jambon , il y a trop de porcs , et bien , plouf , tout s' écroule , et les pauvres gens qui ont fait cet élevage et les pauvres bêtes qui ont enduré le martyr que vous savez , tout ça pour rien . Pour que la main du marché fonctionne . Voilà . Alors , il propose de donner la main au marché . Bref . Les ports , on les a privatisés . OK ? Et bien , maintenant nous voilà rendus au point où les fameux privatisés viennent demander à l' Etat quoi ? De désembourber les ports . Ah bon , on croyait que le marché allait régler le problème ! Que vous étiez là pour faire des investissements . Non , vous n' êtes là que pour prendre . Nous avons donc des raisons légitimes de nous méfier du marché pour régler ce genre de problèmes . Et vous allez voir , le plus beau va vous arriver bientôt , peut-être que vous ne le savez pas , mais l' Union Européenne , l' Europe qui nous protège , a décidé de vendre tous les barrages hydroélectriques au privé . Pour que ça soit plus compétitif , et vous pensez , qu' immédiatement , des investisseurs pleins de milliards se sont dit : &quot; Mais quelle joie ! Courrons tout de suite mettre le béton qu' il faut pour entretenir ces barrages , dont 25 % ont plus de 70 ans . &quot; Tous étaient tellement contents de se dire : &quot; Vite , courrons payer les employés qui ont fait tourner ça pendant tant d' années , augmentons leur nombre ! &quot; parce que vous savez , il en manque . L ' autre jour , je suis allé dans une centrale , oh , ben moi , je suis ... j' aime les machines , hein , c' est parce que j' ai fait philo , je pense qu' il y a un truc qui n' a jamais été réglé en moi . J' aime les machines , j' aime qu' on m' explique comment ça marche et tout , alors me voilà là-dedans , un truc , vous savez , des années 60 , une merveille . Ah , mais quand je dis une merveille , une merveille ! C'est-à-dire que non seulement c' est beau , c' est dans la montagne , ils ont fait le sol en marbre , une immense fresque à la gloire de l' électricité , et vous avez les gars d' EDF qui sont là , les filles font les choses , tout , tout , tout , très bien . Et en passant , j' en vois trois qui étaient en train de réparer un truc . Je vais vers eux , je dis : &quot; Bon , qu' est -ce que vous faites là , les gars ? &quot; , bon c' était trois hommes , et bon , &quot; qu' est -ce que vous faites ? &quot; &quot; Ben , nous , on est en train d' installer un stator &quot; , si vous le savez pas , il y a un stator et un rotor , le stator , c' est ce qui ne bouge pas , le rotor , c' est ce qui tourne . &quot; Ah bon , &quot; , et on parle comme ça , gentiment , Et le hasard , il y en a un qui dit : &quot; Mais , vous savez , M. Mélenchon , je vais vous dire une chose , j' ai été à la même école primaire que vous . &quot; C' est le hasard , non , de la vie , c' est incroyable , ce que le militantisme politique va ... l' école primaire où je suis allé moi , vous êtes pas prêts d' y aller , hein , c' est l' école Berchet à Tanger . Ah non , mais , quand même , je lui dis : &quot; Mais , dites donc , vous , si vous êtes allé à la même école primaire que moi , vous avez quel âge ? Vous avez plus de 60 ans , qu' est -ce que vous foutez là ? &quot; &quot; Vous n' êtes pas à la retraite ? &quot; Ben , il me dit : &quot; Ben , non , je ne peux pas . &quot; &quot; Pourquoi ? &quot; &quot; Ben , parce que , voilà , ils ont pas renouvelé les équipes , alors , notre entreprise qui fait de l' entretien , on a changé je ne sais pas combien de fois de patron , mais , ils n' ont pas recruté , alors maintenant , je suis obligé de revenir pour expliquer aux plus jeunes comment on fait . &quot; Parce que faire un stator , c' est un travail , bien sûr qui se lit , qui s' apprend à l' école , hein . Que celui qui croit qu' on travaille qu' avec sa main , c' est celui qui sait pas qu' il y a un cerveau au bout de la main . Vous en entendez , des imbéciles , le dire sans arrêt , ça . Hein ? L ' intelligence de la main , ben , l' intelligence de la main , ça commence par l' intelligence tout court . C'est-à-dire par le cerveau , et si vous le remplissez pas , vous avez pas des mains habiles . Je vous dis ça ... Je vous dis ça parce que les messieurs-dames qui ont voté pour M. Fillon , et M. Fillon , il dit qu' il faut ... &quot; Construire , développer l' apprentissage . &quot; Forcément , il y en a pas un qui a son gosse en apprentissage , donc ils trouvent ça très bien pour les autres . Ils n' y connaissent rien ! Bien sûr qu' il y a des métiers où il faut de l' apprentissage . Mais s' il y a ici des gens qui ne savent pas pourquoi on va en apprentissage , c' est souvent parce que bon , tu peux t' embêter à l' école , mais tu pourrais aller au lycée professionnel , pourquoi tu vas en apprentissage ? Ben , parce qu' aussi , ça met du beurre dans les épinards . Faut pas tourner autour du pot , faut dire la vérité comme elle est . Et c' est comme ça que ça se passe . Alors , bien sûr qu' il y a des métiers où c' est un bon passage , mais je veux vous dire , si c' est ça leur idée , de mettre tous les mômes en apprentissage , alors , ça , c' est une vraie bêtise . Ils voulaient déjà en mettre 500.000 , ils ont donné 1.000€ ... de primes à qui embauchait un apprenti , et à quoi ils sont arrivés ? Il y en a moins qu' avant . Voilà . Pourquoi ? Pourquoi il y en a moins qu' avant ? Ben , parce que les parents et les jeunes , ils s' occupent de l' avenir , et ils voient bien , que pour eux , c' est pas par là que ça passe . Moi , c' est l' inverse . Pour accomplir le programme que nous avons , notamment le chapitre écologie , nous avons besoin ... de ... millions , pas de milliers , de millions de gens formés à haut niveau technique . Et si vous ne le savez pas , parce que vous avez des préjugés de classe , ou de caste , mais un bac pro , ce n' est pas pour rien que ça prenait 4 ans . C' est parce que c' est difficile . Faire de la maintenance de systèmes automatisés , faire de la métallerie , faire de l' électricité , tout ça à monter , il faut des savoirs abstraits pour savoir faire tout ça . Quand vous entendez parler d' un chaudronnier , allez surtout pas croire qu' il est en train de faire des chaudrons . Un chaudronnier , il fait des carlingues . De bateaux , d' avions . Ça se règle au millimètre , au dixième de millimètre . Vous ne confiez pas des machines qui valent des centaines de millions à quelqu'un à qui vous allez filer une banane chaque fois qu' il a appuyé sur le bon bouton . C' est pas comme ça que ça se passe , l' éducation . Voilà ce qu' ils ont trouvé pour développer le pays , libérer l' énergie du marché , blabla . Et les gens qui vont faire tourner tout ça , quand est -ce qu' on en parle ? Jamais . Ils ne sont pas au courant , ils ne savent pas comment ça se passe , ils ne savent pas ce qu' est la production . Comme ils n' ont jamais fait philo , ils ne s' intéressent pas aux machines . Et comme ils n' ont pas vu une machine de leur vie de près , ils ne se demandent pas qui est capable de la faire tourner , ni de la construire , c' est pas leur sujet . Leur sujet ... c' est la &quot; bonne allocation de la ressource &quot; . Ça veut dire mettre son blé là où il va en revenir davantage . Alors . Là , je suis sur le thème de l' écologie , pour vous montrer deux manières de penser , j' espère que mon propos est assez illustré pour qu' on comprenne la méthode politique . C' est pas tout . Agriculture programme Fillon . Je vais pas vous passer tout le programme parce que j' en ai au moins pour 10 meetings , alors je garde des morceaux différents et les gens qui suivent , ils regarderont les enregistrements pour reconstituer . Agriculture . Abroger - par ordonnances , hein , attention , hein , c' est que ça urge , hein . C' est pressé , c' est pressé , c' est pressé , tellement que c' est par ordonnances . Abroger par ordonnances ... toutes les normes ajoutées aux textes européens . Ah ha ! Personne ne comprend rien . C' est fait pour . Alors , il y a donc en agriculture des normes qui ont été ajoutées aux textes européens , pendant ce temps là , vous ne parlez pas des textes européens , et un jour ou l' autre , je vous en parlerai , mais c' est pas ce soir . C' est quoi les normes en question qui ont été ajoutées ? Ben , je vais vous donner un exemple . L ' Europe permet l' utilisation des OGM , des organismes génétiquement modifiés . La France a fait jouer la clause de sauvegarde . On n' a pas le droit de planter ça chez nous . D'accord ? Et bien , si vous abrogez par ordonnances , les normes qui ont été ajoutées à celles de l' Europe , ça veut dire que vous rendez possible l' utilisation des OGM en France . Bon appétit , les gens ! Bon appétit ! Et pareil pour les pesticides . Parce qu' un certain nombre de pesticides qui sont autorisés par l' Europe sont interdits en France . Et bien , lui , il propose de l' abolir au motif que , pour certains pesticides , il n' y a pas de produit naturel équivalent , donc , comme il y a absolument de pesticides , et qu' il faut en foutre partout , il trouve qu' il y en a pas assez , on est le premier pays de l' Europe à consommer des pesticides , vous avez des milliers de gens qui se sont rendus malades à la terre , dans le travail de la terre à cause des pesticides , vous avez des kilomètres cubes , je sais pas si ça existe , qui ont déferlé dans la mer , pourrie jusqu' à la moelle , de pesticides de toutes sortes , ont tout dégueulassé , tout saccagé en cours de route , mais non , ça suffit pas , il en faut encore plus . C'est-à-dire , on renonce à toute pensée moderne , créatrice , qui essaye d' utiliser les techniques que la nature a donné aux êtres humains pour atteindre leurs objectifs dans la culture . Alors ça s' appelle , l' agriculture bio . Ne leur parlez pas de bio , ça les rend malades . C' est eux qui ont interdit , deux des personnes qui soutiennent M. Fillon , directement , M. Retailleau , qui est le président du conseil départemental ou non , il est peut-être autre chose maintenant , enfin , je sais plus où on en est . Retailleau . Retailleau et Larcher , président du Sénat . Voilà les deux personnes qui ont fait interdire l' idée , qui était pourtant pas la révolution écologiste de 20 % de bio dans la restauration collective . Voilà , nous , notre objectif , dans le programme , c' est bien , je vais vous les mettre un derrière l' autre : 100 % d' énergies renouvelables donc , on va savoir comment le faire . Les éoliennes , en mer , sur terre , moi , je préfère en mer . Il faut donc ré-équiper les ports , dont je vous parlais tout à l' heure , c' est pas pour rien que je me suis intéressé aux ports , parce que pour les installer , il faut des navires de service . Pour les installer , et ensuite pour les entretenir . Il faut donc changer les quais . Il faut changer tous les appareillages quil y a autour , ensuite , Une fois qu' on est prêts , il faut fabriquer sur place , c' est pourquoi la propriété , quoi qu' elle soit importante pour notre souveraineté , des entreprises qui font ça , n' est pas déterminante , parce que pour mettre une éolienne , vous mettez une pale qui fait 70 mètres . Celle-là , vous l' amenez pas de Hong-Kong en bateau . Vous êtes obligé de la produire sur place , donc vous voyez , j' essaye de vous faire miroiter tout ce qu' il y a à faire , et dans la salle , il doit y avoir des gens qui sont des techniciens et qui comprennent et qui disent : &quot; Mélenchon , il a pas tout à fait compris le coût des pales , mais il sait de quoi on parle . &quot; Oui , il sait de quoi on parle , il s' y est intéressé . Heu , non . Je viens de vous dire ce qu' il compte faire , j' ai oublié , bon , c' est à la fin de ma liste , hein . &quot; Engager la préparation de la mise en concurrence de la SNCF tel que prévu par les directives européennes . &quot; Il y a des cheminots dans la salle ? Enthousiasme , s' il vous plait . Les trains privés partout . Vous savez très bien ce qu' il se passe , comme vous le lisez comme ça , si vous êtes benêts , vous vous dites : &quot; Ben , c' est bien , c' est le capital privé qui va faire rouler des trains privés , qui se paieront en privé , avec des privés qui monteront dedans et qui auront donc des bénéfices privés , pour ceux qui les ont fait monter là . &quot; Et vous croyez que tout va aller ? Mais pas du tout ! Vous savez bien comment ça se passe . Si vous privatisez , il n' y a que les lignes vraiment rentables ou dans lesquelles monte tout le monde , qui sont privatisées . Et les autres ? Qu' est -ce qui passe avec les autres ? Et bien , les autres , il n' y en a plus . Ils ont déjà commencé , et à la place , vous mettez des voitures , ou des bus , des bus Macron . Bah , oui . Tout ça , c' est tout du pareil au même . C' est deux logiques : ou c' est la logique du service public et du bien commun , ou bien c' est la logique de la magie de la main privée du capital privé qui sait à quel endroit il faut aller , tout , ah , ben , c' est clair , le capital , il sait où aller , il va où c' est rentable . Et une ligne qui n' est pas rentable de leur point de vue , pour nous , elle est rentable . Parce que quand il y a un car de moins , quand il y a une voiture de moins sur la route , nous on compte aussi tout ce qu' il y a en plus dans la nature . Tout ce qu' il y a en plus dans le fait que vous prenez moins de risques . Dans le fait que ça peut être plus confortable . Donc , notre comptabilité à nous , elle est globale , elle englobe tous les aspects de la vie humaine , pas seulement un moment , le moment où vous venez , vous achetez votre billet . S' il est plus cher ou s' il est moins cher . Résumé : Je vous ai dit , que pour cet homme , le principe de précaution , le nucléaire , les énergies renouvelables , l' agriculture bio , non ! Il n' en veut pas . Vous êtes prévenus . Quant à nous , que ferons -nous ? Non seulement on n' enlèvera pas le principe de précaution dans la Constitution , celle qu' il y aura à écrire , mais on proposera que dans celle qu' il y a à écrire , on y mette la règle verte . La règle verte , ça sera une chose extraordinaire , qui va nous situer aux avant-postes de l' humanité , comme un peuple , qui est conscient de sa force , de son niveau d' instruction , et donc de sa responsabilité devant l' humanité universelle . Car plus on est , de gens , instruits , prêts à travailler , hautement qualifiés , plus on a de capital accumulé , et il y en a dans ce pays , plus que jamais , qui est la 5ème puissance économique du monde , plus alors on est responsables , devant le monde entier , devant tous les autres êtres humains . Et c' est à nous , de mettre au point les techniques , les moyens qui permettent aux autres d' en faire autant que nous . Quoique , bien souvent , il faut aller apprendre auprès des autres comment on fait . Mais c' est à nous de mettre à la disposition de ceux qui nous disent : &quot; Mais on ne vous demande pas votre argent , on ne vous demande pas vos grosses larmes . &quot; Comme je l' ai entendu dire par mes amis et camarades d' Afrique . &quot; On en a assez de votre compassion ! &quot; nous disent -ils . Car figurez -vous , qu' il y a aussi en Afrique , pour le cas où vous ne le sauriez pas , une très importante catégorie de gens qui ont été formés , éduqués , qui sont de très haut niveau que souvent , on leur pique , pour les faire travailler ici , alors qu' ils ont été instruits là-bas , avec l' argent des gens qui se trouvent là-bas , mais qu' on fait fuir , parce qu' on ruine les économies africaines . Et bien , qu' est -ce qu' ils nous disent ? Qu' est -ce qu' ils nous demandent ? C' est dans les rapports de l' ONU . Là , c' est une Française qui préside ça , qui s' occupe , de ces questions de développement . Qu' est -ce qu' on nous demande ? Pas 50.000 trucs . Premièrement , et surtout : &quot; Aidez -nous à avoir de l' énergie . &quot; Nous , Français , nous savons le faire . Nous pouvons proposer les machines et les brevets , et eux , ils savent faire avec les machines et les brevets , parce qu' ils sont , comme nous , équipés d' un cerveau . Les machines et les brevets pour se produire de l' énergie à partir de la luminescence , pas du Soleil , de la luminescence . On sait le faire . Il y a une boîte qui faisait ça , elle s' appelle Nexis . Il y avait même une chaîne de montage . Si on avait utilisé cette méthode , pour vous donner une idée , je sais pas si vous voyez comment est Marseille , il y a une grande tour , on la voit , on voit que ça . Bon , allez , prenez la tour Montparnasse à Paris , c' est la même chose . On l' aurait couverte de ces verres , la tour , elle devenait positive , c'est-à-dire qu' elle ne consommait pas plus d' énergie que celle qu' elle produisait , qu' elle produirait , non seulement la tour , mais tout l' arrondissement . On sait le faire . Ce qu' il faut libérer , ce n' est pas le capital , c' est l' intelligence , les moyens de faire , et c' est le capital qu' il faut emprisonner . Cette boîte a fermé . A mesure , je vous raconterai des histoires , comme ça , d' entreprises , parce que quand même , je me balade beaucoup , et j' ai pas les yeux dans la poche . Tous ces gens sont prêts à faire , ... ( coupure vidéo ) ... avoir comme réponse : &quot; Vous pouvez bien mourir en mer , si vous savez pas nager . &quot; Parce que c' est ça qu' on fait . Et ça , c' est ignoble . Bref . On nous demande de l' aide intellectuelle , plus exactement , de mettre à la disposition de tous , des moyens techniques . Pas autre chose . Evidemment , une autre manière de faire du commerce , une autre manière d' organiser nous-mêmes nos circuits de production . Si nous n' avions pas ces excédents invraisemblables , on éprouverait pas le besoin de le déverser sur les autres . Il faut donc changer notre modèle agricole , à nous , pas le leur . Comme on est en train de faire , car on leur dit à eux , qu' il faut changer : &quot; Ah , qu' est -ce que vous produisiez ? Ah , 4 carottes , 3 poireaux . &quot; &quot; Mais ce n' est pas ça , l' agriculture moderne , l' agriculture moderne , c' est de faire ceci , faire là &quot; , enfin bref . Les gens produisent plus qu' ils ont à produire , mais nous , on doit changer . On doit rétablir - c' est plutôt du sorgho que des carottes et des poireaux . Nous , on doit établir ces circuits courts de l' agriculture . Il faut , diable ! Il existe plein de fermes en France qui fonctionnent de cette manière . Avec des circuits courts . Qui vendent la production sur place , et vous avez plein d' endroits , où , comme ça n' existe pas , vous êtes quelque part et vous décidez que vous allez mettre 100 % bio dans votre cantine , et vous découvrez qu' il n' y a pas une personne qui produit des carottes à moins de 200 km autour de chez vous . Parce que ça , on ne sait plus le faire . Parce qu' il n' y a personne pour le faire . Parce que ça permet pas d' être payé dans des conditions telles que vous arriviez à vivre de votre travail . Ecoutez bien ça , c' est pour ça que tout se tient . Si vous voulez le bon manger pour vos enfants , et pour vous-mêmes , si vous voulez lutter contre l' épidémie d' obésité qui est en train de dévaster la masse de notre peuple . Si vous voulez , simplement , manger une pomme comme j' en ai mangé , que plus personne ne peut en manger ici , et pas vous , les petits . Moi , quand je mangeais une pomme , il y avait des vitamines . Pour que vous ayez la quantité de vitamines que je mangeais quand je mangeais une pomme , au siècle dernier , combien il faut que vous en mangiez ? Vous connaissez la réponse ? Cent ! Et bah alors , hein ! Si c' est pour les vitamines , il va falloir ronger , hein ! Voilà . Donc il faut changer notre modèle agricole , et si nous changeons notre modèle agricole , d'abord , nous vivrons mieux , mais nous devrons payer les paysans , parce que si vous ne le savez pas , c' est le travail des paysans , et le sous-paiement des paysans , qui a fait que , pendant des années , on a pu mégoter avec les travailleurs leurs augmentations de salaire et les limiter au maximum . Parce que pendant ce temps là , on baissait les prix agricoles , si bien que les gens avaient l' impression qu' ils gagnaient plus puisqu'ils pouvaient acheter davantage de choses . Ça a duré comme ça pendant vingt ans . Et puis , un jour , on a tellement pressé qu' il ne restait plus ni paysans , ni production agricole , qui contentent les besoins que nous exprimons . Il reste 900.000 paysans dans 200.000 ... fermes . Et , je vous dis , quand je dis &quot; fermes &quot; , c' est pour être sympa . Parce que certaines , ce n' est pas des fermes pour moi , c' est des usines . Et c' est des usines qui ne font pas vivre . Ni les bêtes qui s' y trouvent , ni les gens qui s' occupent d' elles . Et c' est pourquoi , dans notre pays , il se suicide un paysan tous les deux jours . Dans un silence immense , parce que vous savez quel est le vrai problème de ce pays ? L ' ISF ! C' est ça la grande question . La chose qui paralyse tout , qui fait que , nous en sommes où nous en sommes . Bon . Voilà . Je vous ai dit ce qu' on allait faire . La règle verte , qu' est -ce que ça veut dire ? Celle qu' on va mettre dans la Constitution : On ne prendra plus dorénavant à la Terre davantage que ce qu' elle peut renouveler . Et ceci est un défi technique . Vous autres , jeunes gens , et les autres , vous aurez à régler cette question , parce que dorénavant , c' est comme ça qu' on produira dans ce pays . Et on vous demande d' aller chercher dans votre instruction , dans ce que vous avez appris à l' école , et pour les autres , ce que vous savez faire , le moyen d' atteindre cet objectif . Et vous y arriverez , parce que la génération précédente , en moins de 10 ans , elle a mis en place la filière nucléaire , alors qu' on savait même pas faire une seule centrale à l' époque , on l' a fait en dix ans . Et maintenant , on doit défaire . Mais c' est pour vous dire que des choses qui paraissaient incroyables , nous les avons faits . Nous avons fait les fusées , alors qu' à l' époque , les Nord-Américains disaient : &quot; Il n' y aura jamais aucune autre fusée que les nôtres . &quot; Evidemment , les fusées , c' est des missiles , hein , vous avez compris . Nous avons fait le train à grande vitesse , nous avons équipé tout le pays de barrages . Quand on le veut , quand on le décide , tout le monde y met son énergie et son enthousiasme . Le peuple français peut faire tout ça . Donc il y aura de la planification écologique . On ne peut pas faire une agriculture bio , sans que deux conditions soient réunies . Premièrement , que vous soyez mieux payés pour pouvoir vous payer de la bonne agriculture bio . Voilà pourquoi ... augmenter les salaires ... c' est écolo ... et ceux qui vous disent et opposent sans cesse la classe ouvrière aux revendications écologiques ne connaissent rien à ce qu' est la mentalité des travailleurs aujourd'hui , parce qu' ils savent qu' ils sont les premières victimes des process de production qui ne sont pas respectueux de la nature et des hommes . Et deuxièmement , parce qu' ils connaissent toutes ces réalités que je viens de vous décrire . Deuxième condition : il faut préparer les gens , si vous voulez faire une agriculture bio , il nous faut 400.000 paysans de plus . Je ne vais pas les trouver en comptant dans vos rangs , 1 , 2 , 3 , 4 , 5 , t' es paysan , 1 , 2 , 3 , 4 , 5 , t' es paysan , ça , c' est Pol Pot qui fait ça , pas nous . Nous , on sort pas les gens des villes pour aller les coller à la campagne . Il faut donc proposer . Il faut éduquer , il y a plein de jeunes gens qui ont envie de faire ces métiers là . A condition qu' on ne vienne pas leur dire : &quot; Et bien , maintenant , voilà ce qu' on te propose pour le restant de tes jours : une paie de misère , tu restes tout seul à la ferme et tu restes au cul des vaches tout le temps . Pas de vacances . Pas de soirées . &quot; Non ! C' est donc la condition sociale des paysans et la condition culturelle des paysans qu' il faut les aider à transcender de fond en comble , et dans la paysannerie française , la tradition existe qui permet ce changement . Si vous ne le savez pas , c' est dans la paysannerie que sont , souvent , les formules socialistes les plus avancées . De mise en commun , de coopératives , d' utilisation en commun des outils de production et combien d' autres choses . Donc , le milieu est prêt , mais nous , on doit planifier . Comme je vous l' ai dit tout à l' heure , si vous voulez installer des éoliennes , et ben , vous êtes obligés de réfléchir à tout le reste . Sinon , vous ne savez pas de quoi vous parlez . C' est pas sérieux . Alors , on me dit : &quot; Ah , la planification ! &quot; Heureusement , maintenant , il y a M. Hulot ce matin , dit comme moi , il dit : &quot; Il faut planifier . &quot; Bah oui , il est sérieux . Vous ne pouvez pas dire aux gens que vous allez faire ceci et cela et rien n' organiser , c' est ça que ça veut dire , planifier . Ça ne veut pas dire que vous êtes dans un bureau à Paris et que vous décidez combien de fraises on va produire à Marseille . Ça , c' est absurde . Ce n' est pas ce que nous proposons . Et la planification , elle doit reposer sur un mécanisme souple et large , très largement décentralisé . Les réseaux doivent être nationaux , mais la production , elle doit être locale , reposer sur l' intelligence , de ceux qui savent qu' à tel endroit , on peut faire ceci , à tel endroit , on peut faire ça . Par exemple , je vous ai parlé d' éoliennes , je pourrais vous parler d' hydroliennes . Les hydroliennes , on met ça dans la mer . Ah , ben , on veut pas en mettre partout . Il faut - il y a des courants extraordinaires , vous verriez ça , c' est une machine extraordinaire . Là , les pales , elles font 9 mètres . Celles qu' on a mis , qu' on vient de mettre là - de couler . C' est un endroit où il y a un courant d' enfer , donc ça se prend des parpaings comme ça . Qui arrivent dans les pales , ping ! Le truc doit être calculé pour résister à tout ça et transformer en électricité l' énergie qui vient de la mer , et l' énergie qui est dans la mer , elle est gigantesque , il y a 75 fois ce dont on a besoin sur Terre , dans les mouvements de la mer , et tant qu' il y aura la Lune , il y aura des mouvements de la mer . Après , ben ... Les suivants se débrouilleront , hein . D'accord ? Mais pour nous , c' est ça . Mais c' est un bien grand mot , on dit hydroliennes , alors tout le monde prend un air très intellectuel , parce que : &quot; Ça va ? hydroliennes ? &quot; Mais des hydroliennes , mes amis , dans ce pays , il y en a eu des dizaines de milliers , hein . Pendant des générations et des générations . Mais on appelait pas ça &quot; hydroliennes &quot; , on appelait ça &quot; moulins &quot; . &quot; Moulins à eau &quot; . Il y en avait partout ! Des moulins à eau , vous pouvez en mettre partout . D'ailleurs , vous en avez un ici , d'après ce qu' on m' a dit , à Bordeaux , sous un pont , qu' on essaye , évidemment , on ne va pas mettre des trucs de 9 mètres de débattement , mais on peut en mettre partout , donc on peut assurer cette conversion vers le 100 % renouvelables si on planifie correctement . Voilà . Ça , c' est dans - Vous avez donc bien deux façons d' aborder ce problème . Je suis parti de l' urgence la plus importante , celle que nous avons comme responsabilité d' être humain , vous êtes obligés de vous poser la question : qu' est -ce qu' on fait en face du changement climatique ? Qu' est -ce qu' on fait concrètement ? Pas simplement regarder ... ou se lamenter ... Faire . On peut le faire . Et on peut être une nation de pointe . Il y a plein de pays qui ont déjà mis au point toutes sortes de techniques . C' est donc pas si compliqué que ça . Et pendant ce temps ... leur priorité , ce n' est pas celle-là . Je vous ai amené un petit film ... qui va vous présenter quelle est leur priorité , et avec quelles méthodes . C' est pas long . Donc quand on voit ça , on comprend , il y a deux sujets , mais avant les deux sujets , il y en a au moins un , vous ne pouvez pas dire que vous n' êtes pas prévenus . Vous , ici , vous êtes prévenus , je le sais bien , sinon , vous ne seriez pas venus . Je sais bien qui est là . Mais , maintenant , il s' agit d' aller expliquer aux gens ce qui se passe là , ce qui se prépare . Déjà , c' est la méthode . Vous avez entendu le mot : un &quot; blitzkrieg &quot; . La &quot; guerre éclair &quot; , c' est ça que ça veut dire , le climat qu' il veut changer , c' est pas celui qui est en train de se détraquer , c' est celui des affaires , avec un blitzkrieg . C' est pas une accusation de ma part , infondée , ou une exagération , &quot; blitzkrieg &quot; , ça veut dire la guerre éclair . Contre qui compte -t-il faire une guerre éclair ? Puisqu'il fait la guerre . C' est donc une guerre sociale qu' il prépare , et il l' a dit . Ne me dites pas que j' exagère , il vient de le dire . Ce n' est pas parce qu' il le dit en allemand qu' on ne comprend pas . Beaucoup d' entre nous ont été à l' école et y ont appris l' allemand , et sinon , on se renseigne . Je récapitule : la suppression de la durée légale du travail . Attendez . Attendez , j' ai besoin que vous compreniez bien . Suppression de la durée légale du travail , en Europe , il y a une limite qui s' appelle 48h par semaine . Si vous supprimez la durée légale du travail , c'est-à-dire que vous dites qu' il n' y a plus de durée légale , ça veut dire deux choses : 1 ) Qu' on peut vous faire bosser 48h par semaine sans que vous ayez le droit de protester , et 2 ) ... et 2 ) ça veut dire qu' il n' y a plus d' heures supplémentaires . Ça veut dire que toutes les heures que vous ferez au-delà de 35h ou 37h , suivant les réformes que ces messieurs-dames veulent introduire , elles sont faites au tarif ordinaire . Terminé , le plus que vous faites quand vous faites des heures sup , et maintenant , j' en viens à ces heures sup . Quand et où quelqu'un a -t-il vu que c' est l' employé ou le salarié qui décide des heures sup ? Jamais . Alors quand l' heure sup est proposée , des fois , ça tombe bien , parce que , il y a besoin de payer les impôts locaux , parce que ceci , parce que cela , parce que il faut changer les cartables et que c' est la rentrée et tout ça . Mais pas toujours . Et vous n' avez pas la possibilité de refuser ces heures là , puisqu'elles ne seront plus supplémentaires , elles seront réglementaires , vous avez compris ? Ça veut dire que par un bulletin de vote , vous reculez direct jusque dans les années 1900 et quelque chose , c'est-à-dire avant 1910 et les premières définitions de la durée légale du travail par jour . Voilà ce qu' il vient de vous proposer . Quand il parle du nouveau code du travail , vous connaissez la singerie ? Ils viennent avec le gros code du travail . &quot; C' est trop gros ! C' est trop ... &quot; Ho ho ! Une petite seconde . J' ai été parlementaire national pendant des années . J' ai fait des paquets de lois sur le travail . Et je vous garantis que ce n' est pas facile . Hein , quand vous n' êtes pas un professionnel du code du travail , pour retenir à chaque fois de quoi on parle , et quelles sont les références , c' est dur , des fois , vous avez des menteurs éhontés , comme celui-là , comment vous l' appelez , qui est président de la région Nord , là . Les Hauts de France , ça s' appelle maintenant . Voilà . Xavier Bertrand . Incroyable ce type . Il mentait , dans l' hémicycle , mais vous savez , à vous couper le souffle . &quot; Ah oui , oui , euh ... euh &quot; , bon , je vais pas vous raconter cette histoire , mais vrai qu' il ment . Et alors , nous , il fallait courir , heureusement , il y avait la pause du midi , je courrais dans le code du travail , pour vérifier , bon , pourquoi il est si gros le code du travail ? Ben , je vais vous le dire , parce qu' il est plein d' exceptions . Et qui c' est qui fout les exceptions dans le texte ? C' est pas nous ! Nous , on est pour des choses très simples , une durée du travail , dans la journée , dans la semaine , dans le mois , dans l' année et dans la vie . C' est super simple . La paie , par heure , par mois , etc . , etc . C' est super simple . Mais eux , ils n' arrêtent pas de mettre des choses , et après ils disent : &quot; Ça fait beaucoup . &quot; Et ben , très bien , si tous les livres qui sont un peu trop gros doivent être déchirés en deux , ben prenez l' annuaire . L' annuaire , c' est plein de noms . C' est plein de numéros de téléphone , c' est gros comme ça . Allez , arrachez la moitié . Et vous allez voir si vous pouvez téléphoner après . Enlevez le code du travail , et vous verrez bien , ensuite , si vous arrivez à organiser le travail . Parce que ceux qui proposent ça , c' est pas le travail qu' ils veulent organiser , c' est les rendements , c' est les profits Parce que pour que le travail fonctionne , il suffit d' aller demander à n' importe quel chef d' équipe , si vous vous méfiez du gars ou de la fille qui est du rang , et demandez -leur si on peut faire travailler les gens 48h à un niveau d' exigence de rentabilité comme celle que vous réclamez , ce n' est pas possible , mesdames , messieurs , vous tuerez les gens au travail , ils s' abîmeront , ils auront des morts . 500 ! 565 personnes meurent sur leur poste de travail chaque année . Et vous n' en voyez jamais une seule à la télévision . Vous ne le voyez pas , l' ouvrier qui est tombé de l' échafaudage , vous ne le voyez , l' homme ou la femme qui s' est fait couper les bras sur la machine . Jamais ! Ces morts là , elles ne sont pas peut-être assez exemplaires pour édifier le peuple dans la seule chose qui vaille , la peur de l' autre . 565 personnes par an meurent sur leur poste de travail . 230.000 personnes , tous les ans , malades à cause du travail . La souffrance répandue partout . Et vous les voyez qui disent que les gens ne travaillent pas assez . Ils comparent . Des comparaisons qui sont seulement faites pour abasourdir et pas pour faire réfléchir . Ah bon , c' est ceux qui travaillent le plus qui sont les plus efficaces ? Alors , dans ce cas , j' ai la réponse , c' est les Roumains . Qui est -ce qui a envie d' avoir une économie française qui ressemble à celle des Roumains ? Personne ! Pourquoi ? Nous travaillons moins parce que les machines permettent d' alléger la souffrance . Et la ... et le nombre d' heures passées au travail . La baisse du nombre d' heures de travail nécessaires pour produire la quantité de richesses est permanente depuis un siècle . Et par conséquent , tout ce temps de travail qui est économisé , ou bien vous le répartissez également , en faisant de la réduction du temps de travail , qui permet l' organisation d' autres activités , ou bien vous le répartissez à la sauvage , les uns travaillent sans fin et sans trêve , et les autres , plus du tout , ils sont au chômage . C' est pourquoi la réduction du temps du travail qui fait horreur à ces gens est une mesure de bon sens , d' équilibre au travail , d' équilibre dans la société , qui va rapporter beaucoup d' emplois qui ont créés , comme ça a été le cas quand nous avons fait les 35h parce que ce sont les années où il y a eu dans tout le siècle , la plus grande création de postes , d' emplois . Et puis ... vous aurez des gens qui vivront mieux . Peut-être avec un peu plus de bonheur , et le bonheur , ça finit toujours , puisque vous voulez tout mesurer , par du mieux-être pour toute la société . Mais vous ne le comptez jamais , vous autres , les puissants , les importants . Vous ne comptez pas le prix du malheur , vous ne vous demandez pas si les 565 personnes qui sont mortes dans l' année ne nous manquent pas terriblement - pas seulement affectivement - ces choses-là ne comptent pas pour vous , mais pour la société , pour tout ce que savait faire cette personne , pour tout ce qu' elle a apporté , pour tout le bien-être qu' elle donnait aux autres . Le malheur a un prix , vous ne le comptez pas . La destruction de la nature a un prix , vous ne la comptez jamais . L' usure au travail a un prix , vous ne la comptez pas . Le blitzkrieg ... qui comporte la fin de la durée du travail , le nouveau code du travail , la réforme de la fiscalité du capital ... mmh mmh ... C' est pas la peine que je vous raconte ce que c' est , vous savez d' avance . Ça veut dire : &quot; Ils payent moins &quot; . La réforme de l' assurance chômage . La réforme de l' assurance chômage , leur grande trouvaille , c' est : &quot; Plus vous êtes au chômage , moins vous êtes indemnisé . &quot; Pourquoi ? Pourquoi ? On a moins besoin , à mesure qu' on est au chômage ? Pourquoi ? Ils pensent que vous êtes des gens qui ne voulez pas travailler . C' est ça , leur problème . Et voilà pourquoi ils veulent cette dégressivité . Je ne vais pas plus loin . La réforme de l' apprentissage , je ne sais pas ce qu' ils comptent faire , hein . Mais je sais déjà que c' est une bêtise , vu ce qu' ils ont fait auparavant . Et après . Ça , c' est les premières mesures d' économie , vous avez entendu . Bon , bon . Là , j' ai été sur le contenu , mais je vais venir sur la méthode . Il y a quand même un point dont je veux encore parler , parmi leurs trouvailles . Il va augmenter - bon la retraite , vous avez compris , à 65 ans , vous êtes prévenus . Fin du CDI . Un contrat de travail unique à droits progressifs . Alors , les ballots , ils entendent &quot; droits progressifs &quot; . Ils ne voient pas juste que ça commence par des droits dégressifs , parce qu' avant quand vous avez un CDI , c' est un CDI . Déjà qu' ils nous ont collé deux périodes d' essai de 6 mois chacune , qu' on peut renouveler , et ben - et des CDD , qui représentent 80 % des embauches - et bien , tout ça , ils vous le mettent dans un contrat où vous êtes tous en période d' essai plus longtemps , tous en CDD plus longtemps , et à la fin , au moment d' être en CDI , vous êtes ? Virés ! Fin de la sécurité sociale . J' espère que vous allez voir ... le film &quot; La Sociale &quot; . Je ne sais pas s' il est déjà passé ? ( Oui ) Faut retourner voir ça . Ou faut voir ça , hein . Vous voyez comment des ouvrières , des ouvriers ont mis en place un système , les pauvres , ils avaient tout sur le dos . C' est les communistes qui ont fait ça . Avec Ambroise Croizat et puis la CGT . C' étaient eux qui faisaient les caisses . Il y en avait , des résistants , je ne vous raconte pas le film , vous n' avez qu' à aller le voir . Mais , la Sécurité Sociale , c' est une invention , il a raison , M. Fillon , c' est une invention , très ... heu , on va dire communiste . Mais , le mot pourrait être enfermé dans l' étiquette de parti , et ce n' est pas ça , l' idée . C' est une invention extraordinaire . Chacun donne , selon ses moyens , mais reçoit selon ses besoins . Si vous êtes très malades , vous êtes soignés , quoi que vous ayez payé . Et nous ... nous , on voudrait que ce soit comme ça pour tout . Et lui , qu' est ce qu' il a trouvé ? Focaliser ... - alors franchement , pour le français , ils sont pas là , hein . - Focaliser l' assurance publique universelle , c'est-à-dire la Sécu , sur des affections graves ou de longue durée . Il faut être très super-malade pour pouvoir aller à l' assurance sociale de M. Fillon , et c' est lui qui décide quand est -ce que vous êtes très super-malades . Et l' assurance privée sur le reste . Autrement dit , tout le reste , si vous avez une assurance vous êtes soignés , sinon , débrouillez -vous . Personne n' a été - a expliqué à ce type que quand on est un peu malade , on peut se soigner , et ensuite , on est pas très malade . Mais si on ne se soigne pas , alors on devient très malade , de plus en plus malade . Et en plus ... on rend les autres malades . Ils se croient tous très malins . Pourquoi croyez -vous que nous soyons dans cette pièce , tous là autant qu' on est , et même s' il y avait des gens qui n' étaient pas de notre avis ? Parce que depuis que l' humanité est humanité et que 99 % de l' histoire de l' humanité , c' est chasseurs-cueilleurs , tout le reste , c' est 1 % de l' histoire totale de l' humanité . Pendant 99,9 % du temps , les êtres humains qui ont survécu , c' est ceux qui s' entraidaient . C' est pour ça qu' ils ont survécu . C' est parce qu' ils étaient solidaires . Et ceux qui n' ont pas été solidaires , ils sont morts avant les autres , parce qu' à la fin , ils étaient tous seuls , et comme ils étaient tous seuls , et qu' ils mourraient avant les autres , ils n' avaient pas d' enfants , et nous ne sommes pas leurs descendants . Nous sommes les descendants des généreux . Nous sommes les descendants des bons . Nous sommes les descendants des gentils . Nous sommes les descendants des partageux . Allez lui expliquer tout ça . Un peu d' anthropologie , là , ça ne lui ferait pas de mal . Alors , après , vous travaillerez plus et vous gagnerez moins , ça , vous avez l' habitude . Et il va augmenter la TVA . Ça m' intéresse . Il va l' augmenter de deux points . Il va passer de 20 à 22 % . 20 , c' est ce qu' on s' est pris du gouvernement actuel . Mais il avait dit à l' époque : &quot; D'accord , c' est 20 , mais c' est moins que 22 , donc c' est bien &quot; C' est la théorie du moindre mal comme je vous l' ai expliqué tout à l' heure , c'est-à-dire que c' est quand même le mal . Là , il vont passer à 22 . Ça veut dire qu' ils vont prendre aux gens qui sont là , 15 milliards . C' est ça , deux points de TVA , c' est ça que ça vous donne , 15 milliards . 15 milliards ! 15 milliards ... c' est tellement que vous ne vous rendez même plus compte de ce que c' est . Ça veut dire 300€ par tête de pique . Alors , si vous gagnez 3.000€ , ça ne vous fait pas le même effet que si vous gagnez 600€ . Et ça ne vous fait pas le même effet que si vous gagnez 30.000€ . C' est pourquoi c' est un impôt injuste . Parce qu' il n' est pas progressif , et que tout le monde le paie . Ça vous coûte 300€ . Dans la poche d' un smicard , 300€ , c' est plus de la moitié du RSA mensuel . Pas vrai Pascal ? Je le vois au premier rang . Plus d' un tiers du minimum vieillesse mensuel . Je vous rappelle que c' est un monsieur qui est appuyé sur des réseaux traditionnels de croyants . Donc il va piquer la moitié du RSA d' un pauvre , le tiers du minimum vieillesse mensuel de quelqu'un qui gagne pas des mille et des cents . Et qui en toute hypothèse , se situe juste à la limite du seuil de pauvreté tel que le décrivent les institutions internationales . Un quart de SMIC mensuel , et la moitié du coût annuel de la cantine scolaire pour un enfant . Et comme leurs amis suppriment les subventions pour les cantines scolaires pour les enfants , ça veut dire que les enfants n' iront plus à la cantine . Parce qu' il n' y aura pas les moyens . Et donc , qu' est -ce qui va se passer ? Les gosses vont avoir faim . Je ne raconte pas des histoires , là . Je vous parle de choses que je connais , que j' ai vues déjà . Qui existent déjà . Pour les gens qui ne le savent pas , il y a déjà des gosses qui mangent une seule fois dans la journée . A la cantine . Ah , ça dérange , hein ? Ça trouble , ça vous rend mal . Et ce n' est pas nouveau . Ça fait un moment que ça dure . Que le problème se pose dans ce pays de gens qui sont en &quot; insécurité alimentaire &quot; . Ils n' ont pas ! &quot; Insécurité alimentaire &quot; , ça s' appelle . Des dizaines de milliers de gens sont en &quot; insécurité alimentaire &quot; . Et qui sont déjà en &quot; insécurité &quot; énergétique , et en &quot; insécurité &quot; pour la flotte , en insécurité pour tout ! Pour tout dans la vie , du matin au soir . Voilà , si vous en voulez , votez Fillon . Si vous n' en voulez pas ... Nous allons créer un nouveau climat des affaires , nous aussi . Nous allons abroger la loi El Khomri . Nous allons ... rétablir les 35h en rendant les heures supplémentaires plus chères . Nous allons rétablir la retraite à 60 ans . &quot; Ha ha ha ... comment vous payez ça , M. Mélenchon ? La retraite à 60 ans , quand même , à la fin . &quot; &quot; Qu' est -ce que vous êtes en train de faire avec ces gens , là ? Vous leur promettez des choses , vous savez pas comment vous le payez . &quot; Mais si ! On a trouvé une idée . Respecter les femmes . Mais quel rapport ? Si vous payez les femmes comme les hommes , et bien les cotisations sociales supplémentaires payent la retraite . Nous allons rembourser les soins à 100 % . Et si jamais ça coûte trop cher , nous allons avoir une discussion ... sérieuse avec l' industrie pharmaceutique . Evidemment , on va augmenter le SMIC . Ah , ça , je sais qu' il y a plein de gens qui crient , à chaque fois . Non , non non non , quand c' est nous , on améliore la vie des gens . Et pour améliorer la vie des gens , déjà , on améliore leurs salaires . Et puis après , ils en font ce qu' ils veulent . Et moi , je vais vous dire ce qu' ils en font : ils le dépensent . Hein . Pour leurs gosses , pour leur vie , pour les trucs qu' il y a à faire . Et comme ils le dépensent , et ben , hé , ils achètent des choses et comme ils les achètent , ben , faut les remplacer , donc il faut produire . L' augmentation de salaire , c' est le feu de la croissance , et je ne suis pas un partisan de la croissance aveuglée , vous l' avez entendu tout à l' heure . Je vais résumer ce que je viens de dire et la stratégie économique : nous devons relever deux défis en même temps : améliorer la vie des gens , donc , augmenter leurs moyens , et , en même temps , nous devons le faire dans des conditions qui soient respectueuses de l' objectif de durabilité , de soutenabilité écologique . Il faut faire les deux . Pas un après l' autre . Pas je commence là et puis demain , on verra . Les deux en même temps , et c' est parce que on va le faire en même temps que ça va marcher . C' est parce qu' on va faire de la planification écologique , parce qu' on va développer les emplois paysans , parce qu' on va développer l' économie de la mer avec les 300.000 postes dont on a besoin . C' est parce qu' on va le faire avec l' objectif d' un nouveau modèle écologique de la production , de l' échange et de la consommation que nous allons créer de l' emploi et des ressources . Tout se tient . C' est parce que nous sommes écolos que nous sommes socialistes . Alors , le monsieur , là , le premier qui revient me dire : &quot; Heu , Jean-Luc , tu comprends , la constituante , tout ça , on comprend rien , ça n' a rien à voir avec le social , c' est ce qui intéresse les gens . &quot; D'abord , ceux qui disent savoir ce qui intéresse les gens , souvent , se trompent . Ils prennent les gens pour des imbéciles . Les gens ont parfaitement compris que si ce type peut vous annoncer qu' il va faire une blitzkrieg , c' est parce qu' il a une Constitution qui est un coup d' Etat permanent . Et le coup d' Etat permanent , M. Fillon va l' utiliser pour faire un coup d' Etat social . Sa politique , c' est un coup d' Etat social . C' est la destruction de la Sécurité Sociale , c' est la destruction du code du travail , c' est un coup &quot; social ! Quand vous défaites toutes les lois , et que vous le faites passer en force , et que vous vous en vantez , devant une catégorie de Français bien particulière , qui se réunissaient avec lui ce jour-là , vous annoncez à ceux-là : &quot; Je vais faire un coup , je vais passer en force , il n' y aura même pas de pause pendant l' été &quot; , c' est un coup d' Etat social qu' il prépare . Voilà pourquoi la question de la constituante est si importante , les gens , comprenez -moi bien . Le cœur de la stratégie de la révolution citoyenne que je prône , la révolution citoyenne , c' est la reprise du pouvoir par les citoyens sur leur vie , c' est ça que ça veut dire , et ça c' est radicalement nouveau . Ne me dites pas : &quot; C' est facile à faire &quot; . Non , il va falloir que le peuple tout entier se mobilise . Si vous m' élisez , ne comptez pas que c' est moi qui vais tout faire depuis là-haut . C' est impossible ! Il faut que tout le monde s' y mette , pas seulement à produire , mais à changer la vie , la décision . Voilà ce que vous aurez à faire . Donc ... à peine élu , article 11 de l' actuelle Constitution , nous convoquons une Assemblée Constituante . Une assemblée , ça veut dire qu' on va élire des gens pour écrire une nouvelle Constitution . Et on en élira une partie , et moi je propose qu' on en tire au sort une partie aussi . On a besoin des deux . Des deux , pas d' un , des deux ! Des gens élus parce qu' il y a une dispute sur la Constitution , vous le savez bien . Est -ce qu' on veut un régime parlementaire ? Est -ce qu' on veut un régime présidentiel ? Est -ce qu' on veut un régime parlementaire tempéré de certains domaines dans lesquels l' exécutif est fort , ou l' inverse ? Il y a une discussion normale , légitime , souvent , on est soi-même partagé . Il faut que cette Assemblée ait lieu . Et elle ne fera que ça . Et ceux qui seront élus là ne devront jamais avoir été élu dans une assemblée précédente . Pourquoi ? Pas parce qu' on veut montrer du doigt ceux qui étaient dans les assemblées précédentes . Ce n' est pas le sujet . C' est parce qu' on veut un regard neuf , débarrassé des routines du passé . Même des miennes . J' ai été parlementaire longtemps . Je ne souhaite pas être parlementaire dans cette ... Faut dire , je serai déjà Président , je ne vais pas tout faire à la fois , bon . Mais ... c' est important qu' on ait des gens qui regardent avec un œil neuf . Et ce n' est pas parce que leur œil est neuf qu' ils ne sont pas hautement conscients , qu' ils ne sont pas hautement responsables . Quand vous tirez au sort un jury d' assises . Hein ? Vous n' allez pas regarder , vous faites confiance , parce que vous pensez que la personne qui est là est honorable . Si vous ne l' avez pas rejetée , parce qu' elle ne l' est pas . Les avocats ont droit de les rejeter , mais vous pensez une personne honorable , qui va mettre tout son cœur à bien juger , à donner la bonne opinion . Donc on doit avoir confiance dans l' action populaire , pas s' en méfier . Moi , je proposerai quelque chose à cette Assemblée Constituante , quand même , je peux . J' ai parlé , tout à l' heure , de la règle verte . Mais ce que je voudrais vous faire comprendre , c' est que notre méthode , notre stratégie révolutionnaire , c' est l' implication la plus large possible . Plus il y a de monde qui décide , plus on est contents . Alors , déjà , on commence par rectifier quelques problèmes . Il y a 3 millions de personnes qui ne sont pas inscrites sur les listes électorales dans ce pays . Il y en a 6 millions qui sont mal inscrites . Neuf millions de personnes , qui à la prochaine élection présidentielle , ne pourront pas voter si vous ne vous y mettiez pas . Nous , on a fait les caravanes , les militants insoumis , partout où on pouvait , pour inscrire les gens . Occupez -vous autour de vous de savoir si tout le monde est inscrit . Et occupez -vous de savoir s' ils sont bien inscrits , parce que des fois , ils &quot; croivent &quot; ( sic ) , ils ont déménagé . J' ai regardé ça à Strasbourg . Les Alsaciens , c' est des gens sérieux . Ils ont regardé . 176.000 inscrits , 17.000 personnes qu' ils ont dû radier . Et ben , ils se sont mis à chercher où étaient ces 17.000 personnes , où elles habitaient vraiment pour leur dire : &quot; Ben voilà , vous pouvez vous inscrire là , vous pouvez vous inscrire là , mais inscrivez -vous . &quot; Donc , dans notre manière de faire , on va d'abord rectifier ça . Deuxièmement , je proposerai qu' il y ait le vote obligatoire . Parce que , voter , ce n' est pas une faveur qu' on fait . On est membres d' une société , et tout le monde a bénéficié de l' éducation par l' école gratuite , donc , tout le monde doit mettre sa cervelle et son intelligence au service , alors évidemment ... si on veut faire ça , il faut accepter que le vote blanc soit compté , parce qu' il y a des citoyens qui ont le droit de dire qu' ils ne veulent pas . Bon . Encore un point pour élargir la base de la démocratie . Evidemment , dans l' entreprise ... Je n' y viens pas ce soir . Pardonnez -le moi , j' y viendrai un autre soir sur un discours spécifiquement réservé à cette question . Élargir la base de la démocratie , par exemple , c' est une idée - vous allez avoir du mal , hein , quelques uns d' entre vous , et peut-être même tous , moi , j' ai eu du mal avec , et je vais vous dire comment j' ai été convaincu - c' est le droit de vote à 16 ans . Quand on a passé le droit de vote de 21 ans à 18 ans , tout le monde disait : &quot; Ah , quand même , 18 ans , bon , faut pas exagérer , ce sont des gamins , quoi , hein . &quot; Bon , ben le gamin , il était content . Et il a pris sa part . Quand on m' a dit ça , je dis : &quot; mais pourquoi on fait ça ? &quot; &quot; Parce que c' est une vieille revendication &quot; , je m' en fiche . Où on le fait déjà ? Ah , on le fait dans , je crois , 5 , 6 , 7 Lands en Allemagne pour les élections locales . On le fait au Brésil . On le fait en Argentine . Ah bon , et ça fonctionne ? Oui , ça fonctionne . Moi , j' ai dit : &quot; Mais , c' est quoi , le droit d' un môme qui a 16 ans , aujourd'hui ? Et ben , un jeune qui a 16 ans aujourd'hui , il peut déjà exercer l' autorité parentale . Alors , si vous le trouvez bon pour exercer l' autorité parentale , sur un tout-petit , vous devez quand même penser qu' il est capable de se dire tout seul et de trouver ce qui est bon et ce qui est mauvais pour tout le monde . Deuxièmement , à 16 ans , tu peux être émancipé . Troisièmement , à 16 ans , tu peux bosser aujourd'hui . Parce que la scolarité obligatoire s' arrête à 16 ans . Et si tu vas bosser , tu votes aux élections professionnelles . Donc , tu as le droit de t' occuper d' un gosse , tu as le droit de payer des impôts , tu as le droit d' être émancipé , mais tu n' aurais pas le droit d' aller voter ? Non , ce n' est pas raisonnable . Faisons confiance . Et vous verrez que les jeunes gens , que vous acclamez quand ils font des manifestations pour défendre ceci ou cela , comme ils l' ont fait pour le code du travail , et bien , ces jeunes gens , votre devoir de père , de mère , d' éducateur sera augmenté bien sûr auparavant , pour qu' à 16 ans , il soit capable de se prononcer . Et je vous dis que la jeunesse de notre pays est capable de ça , et moi j' ai été convaincu , et donc je le proposerai , et j' espère qu' on le fasse . Ha ha ! Tout le monde n' a pas applaudi , hein . Non , mais il faut réfléchir , ben , on se rencontre aussi pour ça , hein . Mais vous comprenez l' idée ? Plus il y a de monde qui s' en mêle , plus on est forts . Plus il y a de gens qui sont - se sentent responsables de la société , plus notre point de vue est fort . Plus les gens comprennent de quoi on parle , moins ils voudront de ces gens que je viens de vous montrer , parce qu' ils comprendront exactement de quoi on parle , et ils ne diront pas : &quot; Ah , ben M. Mélenchon , il est un peu nerveux , tandis que M. Fillon , il est bien calme , il a une jolie cravate . &quot; Il y en a ! Il y en a ! Mais moi , vous avez compris , hein . Mon message , ce n' est pas ma personne , et vous n' allez pas m' arranger . Je ne vais pas changer . Mon message , c' est le programme qu' on porte ensemble . Le temps qu' on a pris à le mettre au point ! Je finis avec ce Fillon , parce qu' il faut que vous ayez tout . Celui-là , dans les questions internationales , Vous savez qu' il a voté contre Maastricht en 2012 , en , en 199 ... chose ... 1992 . Moi , j' ai voté pour , à l' époque . Je le dis . J' avais idée , que comme on allait mieux intégrer économiquement l' Union Européenne , il allait y avoir , mécaniquement , une sorte de citoyenneté plus grande . Un progrès politique . Je me suis trompé . Avaient raison , à l' époque , la CGT , le Parti Communiste , M. Chevènement , et un certain nombre de gens de droite qui disaient le contraire . C' est eux qui avaient raison . Et moi , je me suis trompé . Je ne suis pas le seul , hein . C' est une majorité qui a voté ça . Non , mais c' est pour dire , que il faut faire un bilan de ses actes . Il n' y a rien de pire qu' un fanatique qui part d' une idée , n' en change jamais , ne réfléchit à rien et répète tout le temps la même chose , parce que c' est lui qui l' a dit . Après , quand j' ai compris ... Je ne vous ai pas manqué , hein . Mais j' ai voté contre le statut de la Banque Centrale Européenne , j' ai voté contre le passage à l' euro , parce que je savais que c' était une folie de le faire avec une Banque Centrale Européenne pareille , et avec un statut pareil , et enfin , quand est arrivé 2005 - ah oui , j' ai oublié de vous dire que pour les deux votes dont je viens de vous parler , j' ai la joie et le bonheur d' avoir une lettre de blâme du Parti Socialiste , signée François Hollande , qui déjà était responsable de ces questions là . Bon , et après , en 2005 , j' ai voté , avec beaucoup d' entre vous , je pense , dans cette salle , on a voté Non contre le projet de Constitution Européenne , et je pense qu' on a eu raison . La preuve , c' est que ils ont pris le projet , l' ont découpé en morceaux , ils l' ont lacéré , ils l' ont jeté en l' air , il est retombé sur de la colle , ils ont dit : &quot; Voilà , c' est le Traité de Lisbonne . &quot; Oui , le peuple français a voté Non , et ils ont transformé ça en Oui . Vous êtes d'accord ? ( Public : Oui ! ) Qui a fait ça ? ( Public : Sarkozy ! ) Qui était 1er ministre ? ( Public : Fillon ! ) Voilà . Qui donc , qui donc a fait cette forfaiture de tromper la parole du peuple français ? Sinon ceux-là . Ces deux là . Et tous ceux qui ont voté avec eux , au Parlement , au Congrès de Versailles . Et qui ont donc trahi la parole . Ensuite , quand nous avons eu le débat , si vous vous en souvenez , non , vous ne vous en souvenez pas , du traité budgétaire , que M. Hollande avait dit qu' il allait le renégocier . Oui , ben il l' a pas renégocié . Rien ! Quelle erreur ils font ? Ce n' est pas seulement de vous tromper , mesdames et messieurs , ils font une autre erreur , qui est géopolitique . C' est que tous les autres nous regardent . Les dirigeants des autres pays . Et ils se disent : &quot; Ah bon ? Les Français peuvent voter ce qu' ils veulent , de toute façon , leurs dirigeants sont d'accord avec nous . Et donc , on fait ce qu' on veut avec les Français . &quot; Donc l' immense force que représente notre nombre , notre pensée , notre extraordinaire réputation dans toute l' Europe , tout ça est réduit à néant par des gens qui nous trompent , nous spolient , et donnent la mauvaise habitude aux autres de penser qu' on peut nous passer sur le corps . Dites -moi pourquoi Mme Merkel aurait un autre comportement ? Elle n' est pas chargée de s' occuper des volontés du peuple français , elle s' occupe des volontés du peuple allemand , c' est pour ça qu' elle a été élue , et en plus de la droite allemande . Vous me direz que comme elle gère avec des socialistes , il ne reste plus grand monde dehors . Il reste nos copains . Mais vous me comprenez ? Pourquoi cette femme , qui rencontre à deux reprises un Président français qui vient avec un mandat : &quot; Je vais renégocier . Je vais refuser &quot; , il arrive , et qu' est -ce qu' il lui dit ? &quot; Bonjour Madame , où est -ce qu' on signe ? &quot; C' est ça qui s' est passé . Et cette erreur , vous la payez aujourd'hui . Quand vous avez une Commission Européenne qui se sent autorisée à élever la voix devant la France , et à nous dire : &quot; Quoi ? Vous ne voulez pas privatiser vos barrages ? Et bien maintenant , je vous mets en demeure de le faire ! &quot; Où trouvent -ils l' audace de nous parler comme ça ? Où trouvent -ils l' audace de nous mettre en demeure ? De vendre ce qui a été accumulé par les générations précédentes , par leurs efforts , par leurs impôts , par leur intelligence collective . Où trouvent -ils cette audace , sinon dans les démissions constantes , les capitulations de ceux qui sont censés nous représenter et nous diriger , et M. Fillon était là-dedans , par conséquent , nous n' avons aucune raison d' avoir confiance en lui , d'autant qu' il est bizarre , moins ça marche , plus il est pour ! Il était contre le Traité de Maastricht , il est pour le Traité de Lisbonne , comment vous comprenez une chose pareille ? Sinon que ses convictions sont à géométrie variable , peut-être , ou bien qu' il n' a jamais cessé d' être autre chose que quelqu'un qui croyait que la meilleure manière d' aliéner la puissance de la France était de faire d' une façon , puis ensuite de faire d' une autre . De la même manière , vous allez avoir l' année prochaine , 2017 , 2018 , premièrement , un nouveau traité européen , parce que les cinq Présidents ont décidé qu' ils allaient proposer un nouveau traité européen . Quand j' ai dit ça à Hollande , après le Brexit , il nous a fait venir , là , alors je lui dis : &quot; Hé , et où vous en êtes avec le nouveau traité ? &quot; Il me regarde , il dit : &quot; Personne ne nous a parlé de ça . &quot; Sérieusement . Ou bien ils ne lui en ont vraiment pas parlé , ou bien il ne lit pas les journaux , et dans les deux cas , c' est quand même un problème , au lieu de parler avec les journalistes , il devrait au moins lire les journaux . Pour savoir ce qu' il se passe . Il y aura un nouveau traité . Réfléchissez bien à ce que vous allez faire . Si vous élisez quelqu'un , qui avant d' y être , est déjà d'accord , ou si vous élisez une tête dure , qui va vouloir qu' on change la règle du jeu d' un bout à l' autre , ou bien sinon , nous serons obligés de faire ce que nous n' avons pas envie de faire , mais on ne nous contraindra pas à faire le contraire , c'est-à-dire liquider notre propre pays , liquider notre propre démocratie . Parce que derrière ce nouveau traité , après vous avez TAFTA , ça va se décider 2017 , 2018 . Et ne venez pas me dire : &quot; Ah , mais Trump n' en veut plus . &quot; Vous n' en savez rien , une chose est sûre , c' est que nous , on n' en veut pas , et si on n' en veut pas , ça ne se fera pas , si c' est moi qui préside . Alors , vous avez bien compris ? Je ne suis pas juste la mauvaise tête de service . L' empêcheur de danser en rond . Toutes ces questions vont venir en 2017 , 2018 , ce n' est pas moi qui les amène , c' est la vie . Feu , Michel Rocard disait que si on signait TAFTA , il n' y a plus d' Europe , en effet , vous ne pourrez plus rien faire , puisque vous serez ficelés pour toutes les normes environnementales et sociales par le contenu du traité , et parce que vous donnerez le pouvoir aux multinationales de faire des procès aux Etats , dans des tribunaux privés , c'est-à-dire en dehors de la loi que vous avez votée . Ces gens là auront une autre loi pour eux , avec leurs propres tribunaux , pour porter des plaintes contre l' Etat , alors que jusqu' à présent , ces tribunaux étaient faits pour régler des différends entre eux . Si bien que non seulement ils porteront des plaintes contre ce que vous aurez voté , mais , souvent , au moment de préparer les lois , quelqu'un vient et vous tire la manche et dit : &quot; Ne fais pas ça . &quot; &quot; Si jamais tu écris ça , si tu proposes ça , ils vont te faire un procès derrière . &quot; Et donc vous direz : &quot; Et ben écoutez , moi , je ne le fais pas . &quot; Comment voulez -vous résister ? Quand vous êtes une grande puissance , peut-être peut -on le faire . Imaginez -vous des dirigeants du Suriname , petit pays , il y a 450.000 habitants , qui ont une immense forêt . Qu' est -ce qu' ils font ceux-là , quand une multinationale va leur dire : &quot; Ah , mais vous , si vous bougez une oreille , vous avez droit à un procès , on vous ruine pour deux générations . &quot; Comme ils l' ont fait à combien d' Etats ? Vous le savez , j' espère , que les multinationales sont en train de faire un procès à l' Australie , parce qu' elle a voté une loi contre le tabagisme . Et les cigarettiers veulent que l' Australie leur paye les cigarettes qu' ils n' ont pas fumées . Et ceux qui font un procès au Canada , parce qu' il a interdit la prospection des gaz de schistes , ou le procès au Canada pour avoir interdit l' usage d' un produit chimique qu' on met dans l' essence , pour que ça carbure davantage , qui encrasse les moteurs et les poumons , et à la fin , ils ont obtenu gain de cause . La Canada non seulement a payé , mais a dû rétablir l' autorisation d' empoisonner les gens . Voilà ce dont il est question . Ce ne sont pas des constructions idéologiques abstraites , encore qu' on a le droit d' avoir des constructions idéologiques abstraites , qui nous font prépa - préférer , comme Montesquieu , la démocratie du peuple tout entier , plutôt que le pouvoir de quelques personnes . Et voilà ce qu' ils ont prévu de faire . M. Fillon a -t-il dit qu' il était contre TAFTA ? Non , il est pour . Est -ce qu' il est contre CETA ? Non , il est pour . Est -ce qu' il est contre TISA ? C' est sur les services publics . Non , il est pour . Vous voyez que c' est deux visions du monde qui vont s' affronter , ce ne sont pas simplement des personnes , des costumes , des mots ! Si notre programme s' appelle &quot; L ' Avenir en Commun &quot; , il y a le mot &quot; commun &quot; dedans , et il y a le mot &quot; avenir &quot; qui veut dire qu' on a le goût du futur . Le sien , il n' a même pas de nom . Ca s' appelle &quot; Un Projet pour la France &quot; . Il ne peut pas le nommer . On va voir ce qu' il va dire . Je le propose , en tout cas s' il débat avec moi , vous allez voir si je ne vais pas lui faire dire . Et maintenant ... rappelez -vous bien une chose ... rappelez -vous qui c' est . C' est moi qui vous ai rafraîchi la mémoire , sur l' affaire des traités européens . Et bien , je vais continuer . Je ne veux pas de la guerre , et vous non plus . Peut-être que vous êtes moins conscients que je le suis , et d' autres , du risque de guerre généralisée qui s' étend sur la planète . Mais nous y sommes ! Et notamment , dans l' Europe de l' Est , avec cette histoire d' Ukraine , avec les provocations sans fin auxquelles se livrent les Etats-Unis d' Amérique dans le cadre de l' OTAN , en installant des missiles anti-missiles en Pologne , qui non seulement annule notre propre système de défense , mais est une menace directe sur tous les voisins , devinez lesquels . Ils sont revenus à la mentalité de la Guerre Froide , avec toutes les catastrophes que ça peut entraîner . Vous savez que l' Ukraine , il y a 7 centrales nucléaires , dont Tchernobyl ? Vous croyez que c' est un endroit où on va se battre ? Enfin ! Il faut donc tout faire pour empêcher que les choses roulent à la guerre . Et je n' ai pas peur de dire , qu' à mon avis , les Russes sont des partenaires , pas des ennemis . Je ne le crois pas . Je crois qu' on est partenaires . Réfléchissez -y ! Je sais que vous êtes peut-être sous l' emprise de bien des arguments qui roulent contre eux , mais pas la guerre ! Pas la guerre ! Ni un peu , ni moyen , ni beaucoup . Mais même un peu , pas de guerre ! Pas de guerre ! Le contraire ! C' est la raison pour laquelle j' affirme que je suis partisan de la sortie de la France de l' OTAN , qui est une organisation militaire vouée à la guerre . Je suis partisan d' une autre alliance altermondialiste , avec d' autres peuples . Et je suis partisan du fait que la France joue un rôle pionnier , aille de l' avant pour dire , si imparfaite qu' elle soit , la seule institution internationale que nous pouvons et devons respecter , c' est l' ONU . Parce que nos prédécesseurs se sont tellement battus pour qu' il y ait une Société Des Nations et qu' on cesse de s' affronter . Non , je ne suis pas d'accord pour les G8 , les G20 , et que sais -je encore , qui font la loi dans la planète . L' ONU , avec tous ses défauts , l' ONU où la France siège , et a un siège au Conseil de Sécurité , siège que pour ma part , je ne céderai jamais à personne , dans aucune circonstance . Telle est la France , et elle peut jouer son rôle . Et bien , c' est le contraire que cet homme ... Qui a ramené la France dans le commandement militaire intégré de l' OTAN ? ( Public : Sarkozy . ) Qui était 1er ministre ? ( Public : Fillon ! ) Très bien . Et pour ce qui est de faire la guerre , je ne vous parle pas de l' actuel qui en a déclenché trois , et ben oui , il faudrait quand même s' en rappeler . Mais qui a déclenché la destruction de la Libye , je vous prie ? ( Public : Sarkozy ! ) Qui était 1er ministre ? ( Public : Fillon ! ) Ce sont des questions d' une extrême importance . Si vous voulez de nouveau confier la France , avec toute sa puissance militaire , qui est considérable , aux mains de gens qui ont été capables d' une telle stupidité que celle qui a consisté à détruire la Libye , puis à engager la France dans un combat plus qu' improbable en Irak , à la suite de l' OTAN , avec des alliés aussi discutables que les Qataris et les Saoudiens , la main dans la main avec des Turcs qui nous tirent dans le dos , si vous voulez confier la France à des gens pareils , rendez-vous compte des dangers que vous lui faites courir . Car rien n' est plus facile que de vider le tube de dentifrice , mais après , on ne fait pas re-rentrer la pâte . Quand la guerre commence , c' est à la portée de tout le monde de déclencher des guerres , c' est très facile . Surtout quand on a les outils de la puissance . Mais c' est très difficile de revenir à la paix . Parce qu' entre temps , des millions de gens se sont sauté à la gorge , se haïssent , des fois pour plusieurs générations , et ils ont des bonnes raisons . Voilà pourquoi la paix est un bien précieux . Et vous autres ... si vous voulez m' aider ... c' est la chose essentielle qu' il faut que vous fassiez avancer dans l' esprit des gens qui nous entourent . Assez de ces soi-disant solutions efficaces , parce qu' elles sont violentes . Ce n' est pas vrai . La violence appelle la violence , et encore la violence et toujours plus de violence . Bon . Il faut que ça s' arrête quand même . Je viens à Bordeaux , et je vous ai parlé de son maire actuel , que je salue , en bonne tradition républicaine . Mais enfin , ce n' est pas mon préféré . Un peu de philo , pour terminer . Le meilleur maire de Bordeaux ... 1581-1585 : Montaigne ! Montaigne , évidemment . Et qui d' autre ? Evidemment . Son camarade . La Boétie . Et le beau livre , un peu compliqué , mais pas trop . En tout cas , vous autres les jeunes , vous avez intérêt à le lire , hein . Sinon , ce n' est pas la peine d' habiter ici , hein . Et pas que parce que vous habitez ici . &quot; De notre servitude volontaire &quot; Tiens , Laurent , je ne sais pas où il est ... Tout à l' heure , on cherchait des citations . On aurait pu prendre ... Du La Boétie , hein , qui dit que &quot; les tyrans ne sont forts que d' une chose : la soumission de ceux qu' ils ont dominé . &quot; &quot; S' ils l' acceptent . &quot; Il y a une autre phrase , je ne vais pas vous saouler avec ça , c' est du Machiavel . Mais Machiavel , ce n' est pas ce qu' on dit , hein , c' était un républicain , il sortait Dieu de la décision politique , alors ça l' amenait dans - sur la terre des êtres humains , hein . Ce n' est pas toujours très glorieux , mais il disait : &quot; La forteresse des tyrans - la forteresse des tyrans - c' est l' inertie des sujets . &quot; Si vous êtes inertes ... vos tyrans font ce qu' ils veulent de vous . Alors , d'abord une leçon de vie . J' adorais Montaigne pour un truc - je n' adorais pas Montaigne quand j' étais jeune , je trouvais ça mièvre , je trouvais ça ... mou , quoi - et le temps a passé , bon , je vois les choses autrement . Par exemple , Montaigne raconte , il dit , il veut montrer pourquoi on n' est jamais meilleur connaisseur de soi , que soi-même . Sa leçon de philosophie , c' est : &quot; N' allez pas demander aux autres ce qui est bon pour vous , essayez d' y réfléchir vous-mêmes &quot; . Alors , il raconte une anecdote , il dit : &quot; Quand ça vous gratte , il n' y a que vous qui savez juste l' endroit où il faut mettre le doigt pour gratter . &quot; C' est une image formidable , non ? Bon . Et alors , C' est très matérialiste , au fond , au sens philosophique du terme , hein , pas les biens matériels . Justement , il dit quelque chose , hein , 1581 . On a trouvé ça tout à l' heure , on a été trop contents . Puisqu'il y avait les deux cohérences , est -ce qu' on allait trouver dans Montaigne les deux cohérences ? Et ben , ouais , on a trouvé . Voilà ce que dit Montaigne : &quot; Les biens de la fortune ... &quot; - ça existe , hein , les biens de la fortune - &quot; encore faut -il avoir du sentiment pour les savourer . &quot; &quot; C' est d' en jouir , et non de les posséder , qui nous rend heureux . &quot; Autrement dit , la vie ne vaut que par ce qu' on en fait . Et de toutes les satisfactions qu' on peut tirer de l' existence , la première est d'abord de savoir le faire . Et de vouloir le faire . Et comment ? Entrez en vous-mêmes . Quand même , ça fait des drôles de meetings , hein , qui se terminent par Montaigne ... Voilà ce que dit Montaigne : &quot; Qui se connait , connait aussi les autres , car chaque être porte la forme entière de l' humaine condition . &quot; Ça , il dit ça au XVIème siècle . Et nous autres , les mauvaises têtes , 4 siècles à se battre avec tout le monde pour défendre cette idée . Que les êtres humains sont semblables . Et pendant combien de générations : &quot; Hé , mais qu' est -ce que vous racontez ? Ça n' existe pas ce que vous dites . &quot; &quot; Il y a des grands , il y a des petits , il y a des gros , il y a des maigres , il y a des hommes , il y a des femmes , des intelligents , des très bêtes , il n' y en a pas deux pareils . &quot; Les qualités , ça n' existe pas ! &quot; Vous êtes fou , M. Mélenchon ? &quot; Pascal ? &quot; Qu' est -ce que vous racontez ? Si vous faites un régime politique fondé sur l' idée de l' égalité , vous irez contre la nature , et par conséquent , tous vos régimes politiques égalitaires seront des violences . &quot; Voilà ce qu' ils nous ont répété pendant 3 siècles . Et à la fin , il aura fallu que tout le monde comprenne que comme l' unique écosystème compatible avec la vie humaine est en danger , alors vous avez la démonstration du fait que nous sommes tous semblables , et égaux par nos besoins et notre humaine condition , comme dit Montaigne ! Il dit aussi : &quot; La vraie liberté est de pouvoir toutes choses sur soi . &quot; Disposer de soi , c' est être émancipé . C'est-à-dire qu' il n' y a plus la main du mancipium sur votre tête , qui décide à votre place , qui vous dit , à vous les femmes , que votre corps ne vous appartient pas , mais appartient à vos hommes ou à vos enfants , ou à la société . La main qui vous dit : &quot; Vous ne disposez pas du droit de choisir pour vous-mêmes le moment d' éteindre la lumière &quot; , car ce droit appartient à la société , ou au médecin , ou à je ne sais qui , ou au prêtre . Le mancipium , c' est tout ce qui va vous empêcher d' être maître de vous en toutes circonstances , c' est la main qui vous dit : &quot; Tu ne pourras pas faire ceci , tu ne pourras pas faire cela . &quot; &quot; Tu ne pourras pas boire . Tu ne pourras pas manger . Tu ne pourras pas t' éclairer . Tu ne pourras pas te déplacer , parce que tu n' en auras pas les moyens , parce que je te les ai retirés , pour que quelques uns en aient plein , et encore davantage , jusqu' au jour où ça finira par ruisseler sur toi . &quot; Nous voici revenus à notre point de départ . Le fondamental de ce que nous défendons , dans le programme &quot; L ' Avenir en Commun &quot; , c' est la liberté . La liberté de soi sur soi . La pleine disposition de soi-même , raison pour laquelle nous mettrons dans la Constitution , si nous avons la possibilité d' avoir notre Assemblée Constituante , parce que nous serons majoritaires aux prochaines élections , nous y mettrons le droit à l' avortement , nous y mettrons le droit d' éteindre la lumière soi-même . Pour qu' on cesse ... à chaque élection de discuter ces droits fondamentaux aux personnes . Car vous savez que , de nouveau , ces questions sont posées . Guérissez -vous de tout ça avec une bonne Constitution . Et une bonne nouvelle Constitution , c' est un peuple mobilisé pour elle . Bon . La dernière citation , et après , c' est fini . Comment on va faire tout ça ? &quot; Pourquoi est -ce que vous n' allez pas à la primaire , M. Mélenchon ? &quot; Ah , parce que vous savez , il y a une martingale : &quot; Si vous allez à la primaire , ça y est , vous avez gagné . &quot; Et d'ailleurs , vous l' avez gagné aussi , la primaire . Oui , mais si je ne la gagne pas ? Parce que qui va venir ? Ce n' est pas une solution , cette histoire . La solution , la seule qu' on aie , c' est de convaincre . Je ne suis pas un batteur d' estrade . Je ne vais pas commencer , à mon âge , à faire une carrière . C' est fait . Vous avez compris ? J' ai tout eu . Ça va pour moi . J' ai beaucoup été aimé , j' aime beaucoup , tout va bien . Ce n' est pas le sujet . Le sujet , c' est celui qui vous occupe , qui fait que vous venez dans une salle comme ça ce soir écouter un type qui parle pendant deux heures . C' est la grande ambition qu' on a pour tous , pour tout le monde . Et la pensée qu' on a pour ceux qui ne sont plus en état d' être parmi nous pour faire les choses , parce qu' ils ont été privé de tout . Voilà ce qui nous met en mouvement . Donc ... quoi qu' il en coûte , quelle que soit la difficulté , de toute façon , nous irons . Il y aura des moments étranges . Où , tout d' un coup , les choses auront - donneront l' impression de s' organiser totalement en dehors de nous . Et puis , on en reviendra aux idées . Ne croyez pas que vous allez convaincre des millions de Français avec une combine . Ne croyez pas que c' est en allant participer à une vaine agitation qui va durer trois semaines , où les gens vont jeter des noms d' oiseaux et regarder dans les sondages qui est -ce qui a la plus grande chance d' être le moindre mal , dont je vous ai parlé au début , que c' est avec ça que vous allez convaincre des millions de gens qui n' en peuvent plus . Quelqu'un a dit : &quot; Il faut revenir à la normale . &quot; Non , c' est la normale qui est insupportable . Il faut en venir à autre chose . Il faut passer à autre chose . Et cet autre chose est grand , est beau . Plein de gens sont prêts à le faire . Nous avons besoin de toutes les intelligences pour le faire . Alors arrêtez de pleurnicher . Regardez -les , tous ceux qui sont là : &quot; Ah oui , mais si vous continuez comme ça , on va perdre . &quot; Non ! C' est si on continue comme ça qu' on va gagner . Si vous êtes convaincus , si je vous ai convaincus , et peut-être que vous ne serez convaincus que demain , en y réfléchissant , parce que là , ça fait beaucoup de paroles , alors , il y a les petites choses à faire . C' est une grande tâche , chacun prend sa part de l' effort . Moi , je fais ce travail là . Et d' autres . Pourquoi ? Quelle est ma force ? J' ai bossé sérieusement , comme vous tous . Ma force , c' est que c' est la dernière fois il y a déjà eu 4 millions de personnes qui se sont rassemblées . Cette force , je la tire de ce rassemblement . C' est ce qui me donne de l' autorité , de la puissance , de l' obligation de me respecter , pour nos adversaires . Mais ma force , c' est celle qu' on a tous ensemble . Je ne dis pas : &quot; Venez voter pour moi . &quot; Venez voter avec moi . Mon message , ce n' est pas ma personne . Mon objectif , ce n' est pas ma personne . C' est ça . Et le maire de Bordeaux conclut - le maire de Bordeaux , Montaigne , hein . &quot; Les choses ne sont pas si douloureuses , ni si difficiles d' elles-mêmes , mais notre faiblesse et lâcheté les font telles . &quot; Il n' y a ici ni faibles , ni lâches . Bon courage . Non . Ne commencez pas avec les mauvaises manières , hein . Ah , non non non , on fait une campagne , vous ne crierez jamais mon nom . Criez des slogans , mais pas mon nom , on mérite mieux que ça , même moi . Hein , allez . ",
    "Est -ce que ça marche , là ? On entend ma voix ? Vous entendez ma voix ou pas , là ? Arrêtez cette musique , là . Il faut bien qu' on entende ma voix , un peu . Je vais rentrer ensuite dans la salle , mais ... Vous voyez . Ça va , là , le son ? Ça va se régler , ne vous inquiétez pas . Mais avant , je voulais quand même vous dire un petit bonjour , à vous qui êtes dehors et qui êtes si aimables d' avoir la patience de rester là . Asseyez -vous tous , hein , vous savez , personne ne vous en voudra , ça fait un peu ... Regardez , ils l' ont fait , là , devant . Enfin , faîtes comme vous voulez . Merci d' être là , parce que , à votre nombre , on va voir qu' il se passe quelque chose parmi vous , de quoi je suis le point d' appui , davantage que l' aboutissement . Si vous êtes là , ce n' est pas pour moi . C' est pour les idées que nous portons en commun , et je ne l' oublie pas un seul instant . Vous voyez , venir en Martinique , ce n' est pas &quot; découvrir &quot; pour moi . Il aura fallu hier que l' on révèle , ou qu' on rappelle , que mon pauvre père , paix à ses cendres , a été receveur de la Poste par ici . Et puis , je suis déjà venu quelques fois . Il n' y a rien que je ne sache déjà des plaies qui vous blessent . Mais , malgré tout et par-dessus tout , chaque fois , je pense à l' extrême sensibilité des populations qui sont ici , que parfois , on ne comprend pas depuis la métropole , où quelques élites ont parfois des paroles dont ils ne mesurent pas à quel point elles sont désinvoltes et mal reçues , même s' ils s' en excusent ensuite . Pour moi , le littéraire que je suis s' avance ici toujours les mains tremblantes . Je suis sur la terre d' Aimé Césaire , d' Edouard Glissant , de Patrick Chamoiseau , mais quel département de la patrie contient autant de trésors de notre littérature ? Je voulais vous le dire , chacun d' entre nous réagit à sa manière aux circonstances de la vie . Et bien , moi , c' est ma manière de le faire . Je remercie ceux qui ont préparé ma venue cette fois -ci . Et pour les remercier tous , ces 300 personnes qui m' ont accordé leur appui lorsque j' ai proposé ma candidature , je vais saluer d'abord Thierry Renard et Marie-Nathalie Ravin qui m' ont accompagné pendant ces deux jours , et , en les saluant , eux , je saluerai tous ceux qui me soutiennent dans ce voyage . Les élus , qui m' ont fait l' honneur de me recevoir le meilleur de la tradition républicaine que je ne voudrais pas compromettre par mes saluts ; le président de la collectivité territoriale de la Martinique , Alfred Marie-Jeanne , mon ami , parce que , quelque soit son parcours politique et le mien , c' est un ami , Claude Lise , président de l' Assemblée , Monsieur le maire de Fort-de-France , Didier Laguerre , qui m' a reçu . Pour saluer la population de cette commune , évidemment , je salue son élu . Puis après tout , et d' une façon générale , les commerçants qui ont voulu , hier , m' accueillir d' une manière assez chaleureuse . Un grand merci à vous , M. le directeur - ou je ne sais plus comment ... quel est son titre - M. Collair ( ? ) . Et puis , ce matin , ce long moment magnifique passé avec le groupement régional des agriculteurs bio de qui je vous invite à vous rapprocher dès que vous le pourrez , et à aider de toutes vos forces , car c' est eux qui dessinent l' avenir , en tout cas , tel que moi , je l' imagine . Et bien sûr , à son secrétaire général , Claude Ducalcon qui a passé un grand moment avec ses autres amis ce matin . Je suis venu ici , parce que , comme vous le savez , je vous propose ma candidature à la présidence de la République Française . C' est un exercice un peu compliqué que d' aller au-devant d' une population qui s' auto-administre assez largement , pour faire des propositions qui concernent l' ensemble du territoire . Je vais vous le faire , et je souhaite que vous l' entendiez sans que cela porte jugement de ceci ou de cela qui se fait ou ne se fait pas , ici . C' est à vous d' apprécier la distance qui sépare mes propos des réalités dans lesquelles vous vivez . Mais je dois formuler devant vous l' idée d' un projet . Je ne suis pas venu promettre de l' aide . Je suis venu vous appeler à la rescousse . Je suis venu vous enrôler pour une cause plus grande que la vie de chacun d' entre nous , qui concerne le destin de la patrie commune et le destin des Françaises et des Français dans cet environnement particulier des Caraïbes , où ils ont une responsabilité particulière , compte tenu de leur niveau de formation , de développement , confrontés , comparés à tant de misère et à tant d' attentes autour de nous . Même si nous-mêmes , nous en connaissons , je le sais . Mais avant d' entre dans le sujet qui occupe directement la campagne et ce que j' ai à dire pour ici , sachez que je ne suis venu que pour commencer un processus , en effet , après que l' on ait établi le programme &quot; L' Avenir en Commun &quot; , nous avons décidé que chacun des compartiments et des chapitres aurait ses sous-chapitres sous la forme d' un livret , qui détaillerait plus précisément tel ou tel sujet . J' attache la plus grande importance à ces livrets , car quoi qu' il arrive , quel que soit le résultat , cette pensée ramassée , concentrée dans des textes sera notre patrimoine intellectuel commun . Et la génération suivante aura à vérifier ce qui aura été réalisé et ce qui restera à faire . Ce qui aura été démenti par les faits et ce qui aura été confirmé par ce que nous aurons fait . C' est donc d' une très grande importance que la précision de ces textes . Bien . J' ai donc ouvert sur le site jlm2017 une page où l' on peut discuter d' un document car je juge qu' il est bien démagogique de se présenter en disant : &quot; Et bien , allez , que chacun dise ce qu' il a sur le cœur . &quot; Mais si c' est pour ça , vous n' avez pas besoin de moi . Il fallait bien qu' on ait préparé , il fallait bien qu' on ait encadré la réflexion , et donc le document de travail est en ligne . Mais qu' auriez -vous dit de moi , si , depuis la métropole , je vous avais harangué en vous disant : &quot; Ah , ben , écoutez , j' ai ouvert une page et regardez et voyez , donnez votre avis . &quot; ? Il fallait que je vienne , et donc me voici . Mais je ne jouerai pas la comédie de dire qu' en 48 heures ou en 3 jours ou en 4 jours , on rencontre une population , ce n' est pas vrai . Je viens donc commencer un travail . Qui se continuera partout , où qu' on soit , comme je viens de vous le dire . Maintenant , j' entre dans l' actualité . Car je ne suis pas venu ici seulement pour vous parler de choses dont vous connaissez déjà une partie . Je veux en parler de l' actualité , parce que voici tantôt , 48 heures ou trois jours , que l' on me cherche pouilles à propos de la Syrie , et qu' on va , répétant de tous côtés que ma position est ambiguë . Mais elle ne l' est d' aucune façon . Je suis hostile au parti de la guerre . Et le parti de la guerre s' est constitué en Europe et en France à travers un soutien aveuglé à ce que j' appelle le &quot; campisme &quot; . Il y a une situation , alors chacun choisit un camp . Et bien ce n' est pas mon avis . Mon avis est que je suis du camp de la France , et la France , son seul intérêt , c' est la paix . Nous ne participons d' aucune coalition pour dépouiller la Syrie , ici d' un plateau , là-bas d' un rebord ou , là encore , d' une plaine . Ma position est parfaitement claire . Je m' oppose au parti de la guerre qui est en train de monter en puissance , dans toute l' Europe , à l' instigation des Etats-Unis d' Amérique , qui ont , en commençant par l' Ukraine et en continuant ailleurs , ourdi toutes sortes d' agitations pour subjuguer les esprits ; et croyez que je n' exagère pas en vous parlant comme je suis en train de le faire . Si vous voyiez les résolutions qui se discutent au Parlement Européen , vous seriez effrayés du ton qui est pris . De cette espèce de manière de revivre une guerre froide qui a cessé depuis bien longtemps et dont on voit bien que , si elle devait recommencer , ce serait une guerre chaude . Mes chers compatriotes , la paix est un bien précieux et il faut lutter pour elle . J' ai assez visité ce territoire pour savoir quelle a été sa contribution dans le passé aux grandes guerres , qui sont toujours mondiales quand elles s' allument en Europe , qui en porte seule la responsabilité à chacune des étapes de l' Histoire . J' ai vu , dans les villages et les villes , les monuments aux morts en Martinique , de tous ces malheureux que l' on avait emmenés là-bas combattre et y mourir et que leurs pères et mères n' ont jamais revus . La paix est une question sérieuse . Ce n' est pas un état de nature . Nous devons résister lorsque la guerre s' avance avec lucidité , avec sang-froid et refuser les pressions qui s' exercent sur nous . On voudrait tous les jours me faire condamner tel bombardement plutôt que tel autre . Je ne le ferai pas . Je continuerai à tenir ma position qui est de dire que l' on ne peut venir à bout de l' ennemi qu' on affronte et à qui nous pouvons imputer les morts du Bataclan , de l' assassinat de Charlie Hebdo , de l' épicerie casher et ainsi de suite ; que nous avons à régler ce compte là , mais que nous ne le ferons pas au prix d' une guerre qui embrase toute une région . Je ne choisis pas mes bombardements . Je déteste les bombardements , que ce soient ceux d' Alep , ceux du Yemen ou ceux de Mossoul , car toujours ce sont les civils qui payent et qui sont en état total d' impuissance , ayant à subir toutes sortes d' horreurs ; celles que leur infligent les occupants qui arrivent puis quand ils s' en vont , et les nouveaux qui arrivent . Nous savons tous que c' est cela , la guerre . Il n' y a pas de guerre propre . Je le dis avec toute cette force car vous aurez , vous autres , un rôle à jouer dans l' idée que je me fais de la réorganisation de la politique internationale de la France . Je ne veux plus que les Français soient le dernier wagon du train des Américains dans l' Organisation du Traité de l' Atlantique Nord que l' on appelle l' OTAN . Nous en sortirons , et nous en sortirons d'autant plus vivement que je vois bien comment , à travers l' installation de batteries anti-missiles en Pologne , à travers toutes ces faveurs que l' on déverse sur des pays qui ont des comptes inextinguibles à régler avec la Russie , on est en train de préparer sur le Vieux Continent un nouvel embrasement . Je n' ai aucune complaisance pour des gouvernements comme ceux de l' Ukraine , peuplés de racistes , peuplés de néo-nazis qui ont le front de porter encore les étendards que l' on a vus sur nos occupants . Voilà ce que j' ai à dire avec toute ma force . Nous sortirons de cette alliance militaire . Nous reconquerrons instantanément notre indépendance totale dans tous les domaines qui concernent la sécurité de la patrie et la manière d' aborder les voisinages qui sont les nôtres , dans les Caraïbes et , plus &quot; prochement &quot; ( sic ) , l' Amérique latine . On sortira le matériel américain du matériel français . Microsoft n' a rien à faire au siège de l' état-major de l' armée française . Ni aucun de ces logiciels dans aucun des armements français . En reconquérant notre indépendance , nous chercherons à nous lier à d' autres peuples , et c' est là que vous avez un rôle à jouer . Pas celui de je ne sais quelle improbable vitrine , mais d' un rôle actif . Nous devons entrer dans les coalitions et les regroupements de pays des Caraïbes et de l' Amérique latine . Car jamais on ne pourra oublier , et jamais il ne faudra cesser de répéter que la plus longue frontière de la France est avec le Brésil et pas avec l' Allemagne , et encore moins avec la Lituanie . Si bien que nous avons à voir avec l' Alba et avec Mercosur puisque nous en sommes voisins directement . Voilà comment je me représente ça et vous aurez à jouer ce rôle . Car mon sentiment , et je vous demande de m' écouter avec soin , est qu' une telle re-disposition des forces de la France , une telle vision du futur qui nous extrait de ce petit enfermement européen , qui fait de nous une petite nation occidentale que nous ne sommes pas , doit cesser . Nous ne sommes pas une nation occidentale . Nous sommes une nation universaliste . Nous sommes présents sur les 5 continents , nous avons toutes les couleurs , toutes les religions , et nous partageons notre langue , qui ne nous définit pas , avec 29 autres pays . La seule chose que nous ayons en commun , et qui est le plus fort , qui est la dynamique qui anime notre peuple , c' est de sa devise dont , parfois , et même souvent , et en particulier ici , nous n' avons pas toujours été à la hauteur . Que la France claque comme un message et un drapeau : Liberté , Egalité , Fraternité , mais jusqu' au bout et pour de vrai . Non , vous n' êtes pas ... ce que l' Europe appelle les régions &quot; ultra-périphériques &quot; . Ce qui est ultra-périphérique , c' est plutôt Vilnius que Fort-de-France pour des Français . Et si je le dis , c' est parce que c' est tout le regard qu' il faut changer . Les îles , voyez -vous , les îles françaises , je les pratique déjà depuis quelque temps . Combien , parmi vos élus et vos dirigeants , peuvent dire qu' ils sont allés en Nouvelle-Calédonie française ? En Kanaky ? Avec ces terribles 11h de décalage qui fait que vous arrivez pantelant , et rentrez chez vous de même . Les îles ne sont pas des territoires exotiques . Les îles sont des loupes sur nous-mêmes . Les îles font voir en gros tous nos défauts et toutes nos qualités . Et parce que ce sont des îles , quand elles cherchent à régler leurs problèmes , alors elles ouvrent des voies pour le pays tout entier . Ah , si la Corrèze pouvait imaginer qu' elle est entourée d' eau comme la Martinique , peut-être qu' elle réfléchirait autrement à son développement . Peut-être qu' elle se poserait les questions que vous avez à vous poser et à régler sur lesquelles , maintenant , je vais venir . La Martinique , la Guadeloupe , les autres territoires ont à être les territoires pilotes du nouveau modèle économique que je propose . Je viens donc vous demander de m' aider à marcher en pointe sur les questions de la planification écologique , comme je viens de vous le faire , sous vos applaudissements , pour la question de l' indépendance de la France , et pour le changement institutionnel qu' il est nécessaire d' apporter . Avec votre permission , mesdames et messieurs , qui m' écoutez ici dehors , je vais rentrer pour que nos amis ne se mettent pas un torticolis terrible . Ils sont du côté de la porte et me regardent , tandis que les autres me regardent sur l' écran . Je reviendrai tout à l' heure pour vous saluer si vous y êtes encore . Au fond ... je ne répète pas ce que je viens de dire , vous y étiez , hein . Bonjour à Bernard Teper que j' aperçois là , au milieu , bonjour vous ... ouh la la , mais que d' amis ici . Les vacances sont commencées , j' ai l' impression . Vous avez bien de la chance . Non , vous , vous n' êtes pas en vacances . Vous bossez là . Au fond , vous allez entendre beaucoup de choses pendant cette campagne , c' est bien normal . Ayez la patience d' écouter . Où est Mme Fanon ? Ah , bonjour , madame , je voulais vous saluer . Et bien , ayez la patience d' écouter tout ce qui vous est dit . Parce que c' est une élection qui a une très grande importance . Peut-être que le territoire de la métropole peut se dire qu' il a encore la force nécessaire et suffisante pour résister à un choc libéral de plus . Car c' est ce qui se prépare . Vous avez vu que M. Fillon est une synthèse - honnêtement avouée , mais au moins on peut lui reconnaître ce mérite - de ce qu' il y a dorénavant comme portrait de la droite dans notre pays . C' est un libéral absolu en matière économique , et même d' avant-poste , et un conservateur total , réactionnaire sur le plan des mœurs et des libertés publiques . Vous avez vu comment il court devant . J' espère que vous l' avez bien compris , lorsqu' il se propose de supprimer la durée légale du travail , ouvrant ainsi la possibilité qu' elle s' inscrive dans les durées prévues par l' Europe , comme un maximum de 48 heures , mais ce n' est pas la seule chose qui change avec lui . C' est que , de ce fait , les heures supplémentaires ne seront plus des heures supplémentaires . Il faut une certaine hypocrisie à d' aucuns pour aller protester contre ça . Je vois les nouveaux effarouchés qui s' en plaignent . Mais enfin , qui a voté dans la loi El Khomri que , dorénavant , les heures supplémentaires pourraient ne plus se payer que 10 % si on en convient dans l' entreprise , où , comme chacun d' entre nous le sait , règne une ambiance amicale où l' on discute avec son patron , très tranquillement , de sa paie . Et bien , les malotrus comme moi , qui ne croient pas à cette fable , trouvent qu' il est tout à fait condamnable d' avoir désarmé de cette façon les salariés , et il y a une grande inquiétude , ici aussi . Je ne vois pas pourquoi ici . Non ? Hier , j' étais à un restaurant et je discute avec les cuistots . En général , j' aime bien aller voir la cuisine , parce que le métier m' intéresse et comment font les gens , et bien de quoi m' ont -ils parlé ? On n' a pas parlé de cuisine , j' aime mieux vous le dire . Ils m' ont parlé de ça . Des heures de travail , de la Sécu . Alors là , c' est le concours en ce moment , les voilà tous repeints en médecins . M. Fillon commence à dire que les petites maladies ne seraient plus indemnisées , mais quelles petites maladies ? Qui est -ce qui va le dire ? Lui ? Les autres têtes d' œuf de son cabinet ? Il ne sait pas , cet homme , qu' une petite maladie peut devenir grande ? Il n' a jamais vu autour de lui d' aucuns pâtir ? Savez -vous , qu' à mon âge , on peut mourir d' une grippe ? Mais pas au vôtre , madame . Mais oui , ça n' existe pas , les petites maladies , et ce n' est pas la logique de la politique de santé que développe une nation développée . Tout le monde doit être soigné . Et chacun contribue au financement de ces soins selon ses moyens et reçoit selon ses besoins . C' est une invention magique , extraordinaire , ça s' appelle la Sécurité Sociale et c' est un modèle qu' on pourrait d'ailleurs étendre à bien d' autres domaines , au point que , pour ma part , je suis partisan d' en systématiser l' application . Et par conséquent , il va de soi qu' il faut être en accord avec l' idée que porte la Sécurité Sociale . Vous recevez parce que vous avez cotisé et vous cotisez parce que vous pensez un jour avoir besoin , et vous acceptez que , dans l' intervalle , ce que vous avez donné serve à l' autre . Vous n' êtes pas fous . Vous savez bien que si vous allez au travail et que celui d' à côté de vous est mort de maladie , il y a une forte probabilité pour qu' après , vous le soyez aussi . Donc , vous , vous avez compris ce que Mme Le Pen n' a toujours pas compris . Les microbes ne savent rien de qui est avec vous . Ni de sa nationalité , ni de sa religion , ni de rien du tout . C' est la raison pour laquelle il faut maintenir l' aide médicale d' Etat , de manière à ce que tous ceux qui sont malades soient soignés , non seulement parce que nous sommes des êtres humains et que le devoir d' humanité nous oblige à nous occuper des autres , et c' est d'ailleurs pourquoi nous sommes dans cette salle , tous là , autant qu' on est , si différents qu' on est , c' est parce que nous sommes les héritiers de ceux qui se sont soucié des leurs . Et les autres ont disparu . Et évidemment , le soin des autres est le cœur de la pensée progressiste qui anime l' idée de la Sécurité Sociale et qui continue de nous animer aujourd'hui . Donc vous cotisez parce que vous attendez que , bon , très bien . Alors , il parait que M. Macron veut reporter une partie de ces cotisations sur la CSG . C' est un peu compliqué , excusez -moi , mais il faut bien que j' aborde des sujets de fond . Hé , mais ça ne va pas du tout , ça . Il ne faut pas faire ça . Pourquoi ? Pour deux raisons : la première , c' est que vous allez demander à un retraité de cotiser pour quelque chose dont il ne connait pas le risque . Si vous cotisez par la CSG , quand vous êtes retraité , au chômage , vous cotisez pour quelque chose dont vous ne courrez pas le risque , puisque vous êtes à la retraite . C' est absurde . Et surtout , si vous sortez toutes les cotisations pour les mettre dans la CSG , il ne reste plus que les patrons , et donc on aura réussi le tour de force , avec quelqu'un qui prétend appartenir à la famille progressiste , de fiche dehors de la gestion de la Sécurité Sociale ceux qui l' ont créée , à savoir les salariés . Car je rappelle que si les patrons y sont , c' est par effraction , parce qu' ils n' y étaient pas au départ , à la Libération , c' est le général De Gaulle qui les a fait entrer dans la gestion de la Sécurité Sociale . Je sais que ces questions sont un peu complexes à aborder , mais il faut les regarder , parce que sinon , vous vous faites dépouiller . On vous dit : &quot; Regardez , c' est beau , c' est nouveau , c' est jeune , c' est formidable . &quot; Bon , c' est bien . C' est vrai tout ça . Sauf que là , c' est un piège mortel qui vous est tendu , par l' un comme par l' autre . Au fond , comme dit ... Qu' est -ce qu' ils ont en commun , comme dit l' adage ici , hein : &quot; Même bête , même poil . &quot; Ce sont tous des libéraux . Ils ne croient pas qu' on puisse organiser le monde autrement . Alors pour les libéraux , qu' est -ce que vous êtes , la Martinique ? Vous êtes au bout d' un gros tuyau , et dans ce tuyau , on met les marchandises qui arrivent d' Europe et on les déverse , et vous autres , et ben vous avez comme métier promis rien du tout , ça , c' est déjà le cas pour une bonne partie de la population qui est au chômage , sinon , comme on dit : les &quot; activités de service . &quot; Alors , bon , c' est très joli à entendre , sauf que c' est tout simplement insupportable dans la durée . Mais je vais vous montrer ça plus en ... un peu plus en détail . En attendant , voilà leur idée . On produit n' importe quoi , n' importe comment , le moins cher possible , on le fait passer par ici , on vote des accords avec les pays frontaliers qui sont en face de nous qui fait baisser les droits de douane , on abaisse les droits de douane ici , et en avant ! C' est parti . Arrivent les bananes dans un sens , arrivent de votre côté les voitures et je ne sais pas quoi . C' est ça leur modèle économique . Et vous ... vous ferez bien ce que vous voulez . Vous voterez bien comme vous voulez , ça va de soi . Mais dites -vous bien que c' est ça qu' ils ont préparé pour vous . C' est ça qu' ils veulent , et c' est ça qu' ils mettent en oeuvre . Ça s' appelle le libre-échange et ça part de l' idée que si l' on laisse les marchandises circuler , tout va bien . Ça , c' est une théorie du XVIIIème siècle . Alors , à l' époque , on pouvait en effet penser que telle marchandise n' arriverait valablement que venant de tel endroit , mais qu' est -ce que ça peut bien vouloir dire aujourd'hui ? Quelle est la différence entre un composant électronique produit en Chine et un autre produit ici ? Car vous pourriez en produire , vous aussi , des composants électroniques . Comment ça se fait qu' il n' y a pas d' usine de composants électroniques ici ? Il y a une malédiction ? Ça ne va pas être le climat , c' est quoi le problème ? Pourquoi les industries de pointe ne sont -elles pas ici ? On peut se poser la question . Pourtant , il y en a qui se gavent ; je vais bientôt leur régler leur sort , mais je reste sur cette histoire . Leur idée ... quelle est leur différence , je vais vous la dire : c' est la différence de salaire et la différence du fait que dans tous ces pays , on ne respecte aucune norme environnementale . Voilà la différence qui fait donc que c' est moins cher . Rappelez -vous bien de ça : moins cher , c' est toujours cher pour quelqu'un au bout de la chaîne . Ou la nature ou quelqu'un qui a travaillé et qui a été traité comme un misérable et d' une manière indigne . Ça , c' est leur vision . Alors voilà l' avenir pour vous . Les marchandises arrivent , vous en achetez , et surtout elles passent pour aller ailleurs , et puis pour le reste , on saupoudre amplement pour que le bétail puisse faire de l' engraissement fiscal . Alors on donne des avantages fiscaux à qui investit dans l' outre-mer . Fort bien . Comme toujours , ils pensent que la cupidité est le principal moteur de l' activité humaine . Et par conséquent , les gens cupides , prenant leur argent parce qu' ils en ont , se précipitent en outre-mer pour le placer . Enfin , vous les avez vus , non ? Vous ne les avez pas vus ? Ça m' étonne . Cette affaire vous coûte un milliard tous les ans . Où il est le milliard ? Je suis bien étonné en venant ici de voir qu' il y a autant de gens au chômage . Alors que normalement , les investissements devraient être extraordinaires puisqu'ils sont dégrevés . Ils ont investi dans quoi , au juste ? Voilà . Voilà la question à vous poser . Donc vous pensez que si vous faites une faveur fiscale , les gens vont prendre les bonnes décisions pour l' intérêt général ? Et ben non , c' est tout le contraire qui se produit . Et ce n' est pas moi qui le dit , parce que vous pourriez penser que je suis mauvaise langue et de parti-pris . Je le suis , de parti-pris , en effet . Et j' essaye que ma langue pique assez pour qu' on s' en souvienne . Mais là , c' est la Cour des Comptes . Elle dit : &quot; Le coût de ces défiscalisations apparaît ainsi disproportionné par rapport à celui d' autres modes d' intervention de l' Etat . Car au financement porté à l' outre-mer , s' ajoute la part importante conservée par les contribuables bénéficiaires de l' avantage fiscal qui ne profite pas à l' outre-mer . &quot; Je reconnais que la phrase est un peu longue , alors je vous la recommence . Quelqu'un qui investit ici voit retiré - par exemple pour s' acheter un bateau , qu' il pourrait louer , ah , c' est de l' activité , ça - et bien , un avantage fiscal correspondant à une part de la valeur du bateau qu' il a acheté . Et cette part fiscale , il n' est absolument pas obligé de la réinvestir ici , il se le garde pour là-bas . C' est pourquoi ça coûte si cher et c' est si peu profitable . &quot; Aucune des diverses tentatives d' évaluation &quot; , dit la Cour des Comptes , &quot; n' a pu conclure à l' efficacité de ces aides . &quot; Hein ? C' est nous qui sommes les gâcheurs , les dépensiers , blabla blablabla , compétitivité , flexibilité , vous connaissez . &quot; Aucune des diverses tentatives d' évaluation n' a pu conclure à l' efficacité de ces aides . En revanche , plusieurs ont montré des effets contraires aux résultats recherchés . &quot; Vous entendez , les gens ? Tout ce gâchis , la Cour des Comptes dit : &quot; On est arrivés au résultat exactement inverse . &quot; &quot; Ou bien le manque d' efficacité . Ces aides ne peuvent être allouées de façon optimale qu' aux investissements les plus utiles au développement de l' outre-mer car elles vont d'abord aux investissements les moins risqués et les plus rentables . Ces aides profitent également à des entreprises qui n' en ont pas besoin pour investir . &quot; 2 milliards . Je voulais vous le dire avant de passer au reste pour que ... Essayez de contrebattre en vous , par des antidotes , le réflexe que vous aurait suggéré une fréquentation trop longue des médias traditionnels . Car , dorénavant , quoi qu' on dise , et avant même qu' on ait fini la phrase , la réplique est : &quot; Mais comment vous payez ça ? &quot; Je pourrais répondre un peu grossièrement : &quot; Je paye en leur prenant ça . &quot; Mais ça fait beaucoup d' argent . Mais ce n' est pas ça que je veux faire . Je voudrais vous appeler à réfléchir sur la place de l' Etat , les collectivités humaines . L' Etat , au sens large , les collectivités , au sens inclus des collectivités locales , qui souvent , étant très proches du terrain , sont les mieux avisées pour percevoir les besoins . L' Etat , le collectif , la société , les besoins communs , toutes ces paroles que l' on a reléguées au profit d' une croyance absurde , à une sorte de génie spontané de la main invisible du marché qui réglerait tous les problèmes . Nous avons donc vocation , et nous pouvons démontrer que l' intervention publique et la règle commune ont plus d' efficacité économique , sociale et écologique que cette distribution irresponsable d' argent sur des gens qui sont motivés par leur seule cupidité . C' est pourquoi j' ai dit , il y a un instant , je voudrais que les collectivités , les territoires de l' outre-mer français , des outre-mer , car ils sont si divers - il est difficile de ranger dans la même catégorie la Polynésie , la Kanaky Calédonie française , la Martinique , la Réunion - ce sont tant d' histoires différentes , tant de contextes différents , mais enfin , pour ce qui nous occupe , il y a deux domaines dans lesquels vous pourriez jouer le rôle de pilote parce que nulle part autant que dans une île , on ne comprend le message . Ce sont les messages de notre époque . Mes chers concitoyens , on ne peut pas continuer avec le mode de production dans lequel nous sommes car il va détruire la planète . On ne peut pas continuer . Il faut donc changer le mode de production , les process de production et avant cela même , il faut changer la manière de s' approvisionner en énergies , sans lesquelles on ne change rien . L' énerige et l' accès à l' énergie est premier . Ici , je ne vais pas évoquer la question du nucléaire puisqu'elle ne touche pas directement ce territoire , vous avez bien de la chance . Mais il n' est pas possible que l' on continue à recourir aux énergies fossiles comme on le fait dans les outre-mer , et en particulier à la Martinique . Que 93 % de la ressource énergétique ici soit du pétrole est une aberration . Une aberration . Pourquoi ? Parce que vous avez devant vos yeux un potentiel qui contient 75 fois le potentiel d' énergie dont vous avez besoin sur Terre : c' est la mer . Mes amis , vous ne pouvez pas ... Vous ne pourrez pas contourner cette ambition . Ecoutez celui qui vous parle et qui porte à cet instant ce que vous avez en vous-mêmes . Car je sais très bien que je suis d'abord votre porte-parole . Je ne vous apprends peut-être rien , mais que notre réunion porte un message pour tous ceux qui ne s' y trouvent pas . Il faudra faire ce tournant . L' économie de la mer nous place en position de puissance . Je vous ai décrit quel est le modèle auquel les autres croient : faire de vous le hub commercial de l' Europe et le territoire d' engraissement fiscal . Moi , je vous appelle à autre chose . Ils vous demandent vos plages et les ports d' attache pour leurs bateaux ; je vous demande des centaines d' ingénieurs , techniciens , techniciennes , ouvrières et ouvriers de haut niveau , qualifiés , dont la patrie a besoin pour faire le tournant vers l' économie de la mer . Vous devrez être le premier territoire dans lequel on forme tout ce monde dont on a besoin partout sur le territoire de la métropole et dans toute la Caraïbe . Car si ... nous étions capables ... et nous pouvons le faire en peu de temps , un bac professionnel , c' est 3 ans ou 4 ans . Si je suis élu , j' aurais peut-être le temps de voir ça . Les diplômes , licences professionnelles , bon , c' est 3 ans après le bac . On peut faire beaucoup , vite et bien , si on s' organise , c' est pourquoi je parle de planification . On peut . Je sais que ça heurte beaucoup de conservatisme intellectuel , ce sont les pires . Je connais l' élite intellectuelle martiniquaise , j' ai été ministre de l' enseignement professionnel . Je me souviens de ce lycée où je suis allé voir comment on avait reconstitué la possibilité de faire des gommiers . C' est moi qui ai fait ça . Et oui . Je me rappelle quand je suis venu ici la dernière fois comme ministre , j' avais dans les mains un BTS que j' ai mis dans un lycée professionnel et qui ... qui a fêté récemment son 15ème anniversaire . Et pourtant , on m' avait dit qu' on ne pouvait pas faire de BTS , ici . On l' a fait . Ils sont mauvais , ceux qui sortent de là ? Non , ils sont bons . Donc , vous pouvez . Et on en a besoin . Et on en a besoin , non seulement pour nous-mêmes , mais pour toute la région . Car nous avons des devoirs . Comme nous sommes les plus instruits , comme nous sommes , sans doute , parmi les plus organisés , à l' exception , peut-être , de Cuba , nous avons un devoir . Un devoir à l' égard d' Haïti . Compte tenu de ce qu' a été l' Histoire , Haïti ne redécollera jamais sans énergie . Haïti à qui on a infligé cette horreur qui a été de faire payer le dédommagement de la fin de l' esclavage jusqu' à 1982 ! Nous avons des ... un dette . Nous avons une dette . Je dis , nous tous . Evidemment , on n' y était pas , on n' est pas responsables de tout ça , personne ici , mais quand même , la France s' honorerait à montrer qu' elle est capable de venir à la rescousse quand on a besoin d' elle . Vous vous souvenez de toutes ces critiques abominables qu' on a entendues contre Cuba , mais si j' étais un Haïtien , je prie pour être à Cuba , pour que mes enfants n' aient pas besoin de manger de la terre pour avoir ... ne plus avoir mal au ventre de la famine . Voilà la vérité . Nous devons nous développer et être le département pilote pour pouvoir amener à Haïti les machines et les process de production qui leur permettent d' avoir l' accès à une énergie facile et bon marché , notamment celle qui vient de la mer . Ce qui veut dire qu' il faut implanter ici au moins un , et peut-être deux lycées maritimes , alors qu' il y en a zéro ! Ce qui est invraisemblable dans un endroit où la mer est autour de nous . Il faut des licences professionnelles à l' université de la Martinique . Il faut des licences professionnelles qui préparent à installer les hydroliennes , et peut-être mieux , les machines qui font la différence entre l' eau froide qui se trouve au fond et l' eau chaude qui se trouve au-dessus . Mes amis , tout ça , je vais vous dire où je l' ai appris , je ne l' ai pas appris dans le département des Bouches-du-Rhône où il y a également zéro lycée professionnel maritime . Je l' ai appris à la Réunion et je le tiens d' un grand îlien : Paul Vergès , c' est lui , le premier qui a inventé un plan d' autonomie énergétique et qu' il avait préparé pour l' île . Tout ça a été dispersé par une succession hasardeuse de gens qui sont des irresponsables et qui ont tout démantelé alors que Paul avait si bien préparé . Paul joua un grand rôle dans la pensée sur le développement durable , et je veux dire que je tiens en piètre estime ceux qui ne l' ont pas invité à la COP 21 , alors qu' il aurait dû être devant , marchant en tête des Français qui se trouvaient là , parce que c' est lui qui a été le plus loin dans la réflexion , dans la planification de ce qu' est la transition énergétique . Voilà la vérité . Donc on peut le faire . Si on était capable de le faire à la Réunion , on est capable de le faire à la Martinique . Et voilà comment avec cette économie là , la France créera 300.000 emplois . La France en général . Je dis 300.000 emplois parce que je ne veux pas vous paraître trop exagéré . Ce chiffre , c' est celui du cluster maritime . Le cluster maritime , c' est l' ensemble des entreprises qui travaillent dans ce secteur , et où règne une ambiance tout à fait excellente , puisque j' y suis très bien reçu , alors même qu' il y a une série de gens qui ont des raisons particulières de se méfier de moi , je les comprends , compte tenu de ce qu' est mon point de vue politique et le leur . Mais , à un moment , vous le savez comme moi , il y a des circonstances où la passion commune peut créer des espaces de travail communs . Et on y arrive , dans ce domaine . C' est pourquoi j' ai bon espoir . Et vous savez , quand on installe une éolienne en mer - encore faut -il pouvoir le faire , parce qu' on ne peut pas installer une éolienne en mer , à part flottante , à la Réunion , mais ici peut-être sur le plateau continental - ça me fait penser que ça manque d' ingénieurs géographes , ici . Il faudrait qu' on en ait davantage , et en particulier que les navires scientifiques français puissent patrouiller et mieux calculer le prolongement sur la plateforme continentale , parce que , comme vous le savez peut-être , la France a vu son territoire augmenter d' un seul coup de 10 % sans tirer un seul coup de fusil , simplement parce que nous avions les navires qui permettaient l' inspection des plateaux continentaux , nous avons pu présenter à l' ONU notre dossier , montrant que nous avions des droits sur ces territoires . Des droits , ça vaut ce que ça vaut , hein , bon , ben , c' est comme ça . C' est la loi internationale . Et notre territoire s' est accru de 10 % et nous sommes le deuxième territoire maritime du monde , mais on ne va pas raconter d' histoires , ce n' est pas seulement grâce aux 1.000 km de côtes de la France métropolitaine . C' est surtout à cause des outre-mer . Autrement dit , le ressort principal de la puissance pour le futur , il est ici . Je vous demande de ... de le méditer . Parce que ça vous explique pourquoi je tourne mon regard , et je dis : &quot; On ne doit plus considérer les îles comme des régions ultra-périphériques &quot; , pas pour faire plaisir aux îliens , mais pour faire plaisir à la France , et à ce dont elle a besoin collectivement . Voilà pourquoi on peut le faire . Et vous savez , quand on installe des machines pour la mer , et ben , on ne peut pas les emmener en bateau depuis Hong-Kong . On est obligés de les fabriquer sur place , et de les assembler sur place . Et ainsi de suite . Si bien qu' il y a , là , un ressort pour vous tous et pour vos enfants qui ne nécessite pas qu' on vous arrose individuellement d' argent et d' avantages fiscaux dont vous ne sauriez que faire , vu que vous n' avez pas d' argent . Mais , en tout cas , les sommes sont là . Qu' on ne vienne pas me dire qu' un pays qui est capable de distribuer un milliard de dispenses fiscales à des gens qui n' en font rien n' est pas capable de mettre le même milliard pour établir des lycées professionnels , établir des ... des enseignants sur place , et ainsi de suite . Ce n' est pas vrai . On peut le faire , et il n' y a pas besoin de s' évanouir en demandant combien ça va coûter . Parce que , de toute façon , cet argent là , il n' est pas dur à trouver . Le banquier central européen donne chaque mois , chaque mois , 80 milliards d' euros aux banques qui sont en Europe pour qu' elles regarnissent leurs comptes et vous prêtent de l' argent . Elles regarnissent leurs comptes et elles ne vous prêtent rien . Et quand elles vous prêtent , vous avez intérêt à faire attention . L' autre jour , je suis allé au marché , comme vous le savez , parce que vous avez vu le petit film . Et quand je suis arrivé au bout du marché , il y a une dame dont le commerce a brûlé . Et elle a dû tout recommencer , cette femme qui a toute cette charge sur le dos , de sa famille , du reste , enfin , tout ce que vous connaissez de la vie . Et bien , elle a donc emprunté pour pouvoir se ré-équiper . Si elle voulait acheter une voiture , elle emprunte à 2,5 mais pour refaire marcher un restaurant , avec plusieurs employés , et bien elle emprunte à 7,5 . 7,5 ! Voilà . C' est de l' exploitation . La banque qui fait ça , je ne sais pas quel est le type ou la fille qui signe le papier , mais devrait avoir honte . Il devrait avoir honte de prendre une part pareille sur le travail de braves gens qui se dévouent , alors qu' à eux , le banquier central prête à presque 0 . Presque 0 pour eux , 7,5 pour cette femme qui fait vivre toute une famille et qui apporte un service commun . Voilà pourquoi quand je dis que vous n' avez rien à attendre des banques sans leur serrer la gorge , je parle peut-être fort , je parle peut-être pointu , mais je décris cette vérité humaine là . Et les gens , poudrés et parfumés , qui sont si contents d' eux-mêmes et de cette économie flexible , compétitive et le reste , évidemment , regardent tout ça de haut , et me voient moi comme un sauvage . Mais les vrais sauvages , c' est eux . C' est les gens qui sont capables d' asservir les autres de cette façon là . Je vous ai parlé de tout ça , vous voyez la différence . Alors , j' y reviens un instant . Je vous ai parlé d' investissement , de production , de gens à former , à cultiver , Alors on me dit : &quot; Oui , mais bon , l' enseignement professionnel ... &quot; Quoi , l' enseignement professionnel ? Qu' est -ce que vous croyez ? Comme vos enfants n' y sont pas , vous ne savez même pas ce qui s' y pratique , et vous êtes là , piaillant et sautillant , mesdames , messieurs les importants , à vouloir envoyer tout le monde en apprentissage , mais votre gosse n' y va pas , donc vous ne savez pas ce que c' est . Mais les enfants vont souvent en apprentissage parce que la façon avec laquelle on apprend à l' école ne leur convient pas , des fois , mais ils y vont souvent parce qu' il y a une petite paie et que ça permet de faire bouillir la marmite . Voilà la vérité que vous ne savez peut-être pas , ben , je vous la dis , alors . Et si vous ne le savez pas , la moitié de la jeunesse de France est dans l' enseignement professionnel , dans l' enseignement technologique et dans l' apprentissage . Et qu' est -ce que vous croyez qu' on fait , comme ils disent : &quot; Il faut apprendre sur le tas . &quot; Ah oui , vous croyez ça , vous ? Vous croyez qu' on apprend sur le tas . Une machine qui vaut des millions , une machine dans laquelle il faut mettre un programme , vous croyez qu' on apprend sur le tas , et quoi ? Vous allez donner une banane à chaque fois qu' il appuie sur le bon bouton ? Non . &quot; L' enseignement manuel &quot; , disent -ils , qu' est -ce que vous croyez ? Il n' y a pas de mains qui marchent sans un cerveau . Et si vous ne cultivez pas , si vous n' éduquez pas , si vous n' élevez pas , alors ça ne donnera rien , même pour la production la plus triviale . Je vais vous dire , un jour , j' ai vu des gamins , ils ne venaient plus à l' école , à un BEP de métallerie , les professeurs se cassaient la tête , ils ont fini par trouver la solution , parce que les professeurs , c' est toujours des professeurs . Et bien , ils ont organisé un cours qui s' appelait &quot; sculpture métallique géante &quot; , ils ont donc emmené ces jeunes gens voir des expositions d' art abstrait , et ensuite , eux-mêmes se sont mis à dessiner . Un , le dessin , ça fait partie des matières professionnelles , deux , ils ont dû calculer comment ils feraient la sculpture métallique géante , ça , c' est de la physique , et ensuite , il a fallu organiser la soudure des pièces . C' est de la soudure , c' est une technique professionnelle et c' est de la chimie . Et après , il a fallu le peindre , etc . , etc . Je vous ai donné cet exemple pour vous dire que les gamins , qui n' allaient plus en cours , qui y sont retournés à 90 % d' entre eux , et qu' à la fin , ils ont eu le diplôme . Mais regardez comment ils l' ont eu : en s' élevant spirituellement par la culture . Toutes ces choses que les brutes croient inutiles , un surplus . Quand ils ont fini de compter leurs sous , alors ils lisent de la bonne littérature , et ils se figurent que la culture , c' est quelque chose qui est après le reste . Non ! La culture , c' est d'abord . C' est parce que vous avez lu , c' est parce que votre cœur a été éveillé par la musique , par la poésie que vous changez en tant qu' être humain et que vous aspirez à savoir toujours plus . Et ben , les autres , ils s' en fichent . Mme Le Pen dit que les allègements fiscaux , c' est très bien pour les DOM-TOM . Voilà . Tous dans la cage , fermez-là . Et M. Fillon , mmmh ? Et bien , lui , il est surtout préoccupé par l' immigration illégale , qui est quand même le problème sur lequel vous tombez tous les jours . Par exemple , il y a un problème pour mettre la table , hein ? Ben , hé , vous avez compris , quoi , hein . Ce n' est pas la banque qui est le coupable , hein . Voilà . Ils espèrent vous faire croire à ces fables . Mais la chose qui m' a plu , c' est que ... il propose quoi ? Et ben , de faire encore mieux pour les allègements fiscaux . Il fait reposer l' essentiel de sa politique économique sur les promesses de nouveaux allègements fiscaux . Ah , et aussi des choses très nouvelles : &quot; Réduire le coût du travail . &quot; C' est nouveau , non ? Attendez , c' est génial . &quot; Réduire le coût du travail . &quot; Oui , oui , et le ... et le travailleur avec , non ? S' il avait un moins gros estomac , moins de besoins , hein . Voilà . Réduire le coût du travail . Vous gagnez trop , les gens . Mais oui , je vous dis ça , je vous le dis sur un mode humoristique parce que j' ai repéré que le rire permettait de fixer aussi bien les idées que de vous peindre tout le tableau catastrophique . Mais quand vous en entendrez un vous dire qu' il veut réduire le coût du travail , il faudra vous rappeler que c' est vous dont il est question . C' est de votre paie dont il est question , et pas trouver ça sympathique et moderne . Parce que ça n' est ni sympathique , ni moderne . Il veut aussi créer des zones franches . Des zones franches . On ne paye plus d' impôts du tout dans les zones franches . C'est-à-dire , on ne paye plus les impôts sur la société , on ne paye plus ceci , on ne paye plus cela , il faut bien que quelqu'un le paye , donc ce sera les autres . Chaque fois qu' il y en a un seul qui vient vous dire qu' il va réduire les cotisations sociales , vous les avez entendus , tous , vous le dire . Rappelez -vous de ce que je suis en train de vous dire à cet instant . J' étais sénateur quand on a voté le fait que chaque euro de dispense de cotisation sociale doit être immédiatement compensé - ce que l' Etat ne fait pas toujours - par le budget de l' Etat . Autrement dit , les ballots , chaque fois qu' on vous dit qu' on va enlever les cotisations sociales , et qu' on va vous donner les sous à vous , dans votre paie , direct , après vous être bien frottés les mains à l' idée que ça va augmenter votre salaire , rappelez -vous que vous devrez vous payer , avec ce salaire , en impôts , vous-mêmes , et c' est exactement ce que veut dire celui qui propose de vous payer en imp ... en salaire direct ce qu' il ferait ensuite passer par la CSG , comme je vous l' ai dit tout à l' heure . Alors , comme il arrive dimanche , vous lui poserez la question de savoir , à la fin , pourquoi ils ne veulent pas aller prendre l' argent là où il se trouve . Là où il y en a des masses gigantesques , immenses , considérables , accumulées , qui ne servent strictement à rien , qui passe son temps à tourner dans la sphère financière , pour y réaliser des taux de profits absolument invraisemblables qu' aucune activité humaine ne peut dégager . Parce que , autrefois dans l' industrie , et encore maintenant , le 3 , le 4 % , ce sont des taux de profits qui paraissent considérables . Mais quand vous tournez dans la sphère financière , les taux de profits , c' est du 10 , du 12 . J' ai connu une entreprise qu' on a fermée alors qu' elle était profitable , elle faisait du 8 et le fonds de pension qui avait placé son argent là-dedans attendait du 15 . Et bien , ils ont préféré fermer la boite que de continuer à gagner moins que ce qui était prévu pour ne pas effaroucher les gens qui investissaient sur ce fonds de pension . J' espère , je ne sais pas si tout le monde a bien suivi ce que je viens de raconter , mais vous vous rendez compte de la prédation que le capital financier exerce sur l' activité humaine réelle . Peut-être qu' on leur en voudrait moins si ils faisaient des sous en faisant travailler tout le monde et en produisant quelque chose qui soit utile à la société . Alors ... ça , c' est le premier défi . L' autonomie énergétique , autrement dit , le plan Vergès partout . Si on le fait à la Martinique , on pourra dire à la Corrèze : &quot; Vous pouvez en faire autant . &quot; Ah , mais ils diront : &quot; Nous , on a pas d' eau . &quot; Ah , pas de bol , hein , cette fois -ci , la situation a changé . Mais c' est vrai . Chaque département de la métropole pourrait se poser la question , et c' est là qu' on verrait comment la planification écologique oblige chaque département à réfléchir , car naturellement , la planification écologique , c' est prévoir , comme je vous l' ai montré tout à l' heure , la formation puis la mise en oeuvre , etc . Mais la planification écologique , c' est la réflexion au plus près du terrain . Mais on va pas parler pour la Corrèze , j' irai bientôt si ils veulent , mais vous savez , les hydroliennes , on peut les mettre en mer , bien sûr , on en met en mer , on en a quelques unes au large des côtes françaises , entre l' Angleterre et la France , parce qu' il y a là un courant du feu de Dieu qui passe à une vitesse incroyable , il fait tourner des moulins qui ont des pales de 9 mètres , vous vous rendez compte . Et c' est tellement violent que ces pales se ramassent des cailloux qui sont arrachés du sol , c' est une réalisation technique extraordinaire , enfin , ça s' appelle une hydrolienne . Mais bon , moi j' utilise aussi le mot parce que je trouve que ça fait bien . Enfin , des hydroliennes , dans notre pays , il y en a eu des milliers , hein , pendant des années , ça s' appelait des moulins , avant , hein . Bon , ben , on se rappelle des moulins . Donc , des moulins , on peut en mettre partout , dans toutes les rivières de France . Et dans celles de Corrèze aussi , encore faut -il se poser la question . Autrement dit , la production doit être pensée localement , la diffusion , elle est forcément nationale , et le réseau doit rester national . On y a intérêt , parce qu' un réseau électrique comporte des moments de délestage , des moments de surcharge donc , il faut travailler tout ça . Vous vous demandez pourquoi je vous parle de tout ça , hein . Ben parce que c' est vos affaires , mes amis , qu' est -ce que vous voulez que je vous dise ? Personne ne vous en parle , mais c' est la vérité . Si vous n' avez compris que tous les barrages de France vont être privatisés , vous vous imaginez une idiotie plus grande que celle-là ? Vous voyez , l' investisseur , il se lève le matin , il dit : &quot; Ah oui , aujourd'hui , je vais produire de l' électricité . J' achète un barrage . &quot; Non . Il se lève , il dit : &quot; Aujourd'hui , je vais produire du fric . Où est -ce qu' il y en a à prendre ? Dans le barrage . &quot; Et si quelqu'un lui dit : &quot; Ah oui , mais vous comprenez , bon , hein , 25 % des barrages ont déjà plus de 70 ans . &quot; &quot; Quelle horreur , j' enlève mon argent . &quot; Parce qu' il n' a pas l' intention de couler un seul mètre cube de béton là-dedans . Il n' est pas venu pour ça . Rappelez -vous en quand on vous fera des belles promesses . Deuxième défi où vous pouvez être pilote et obliger donc tous les autres , parce que si vous y arrivez , ils y arrivent . Deuxième défi , c' est celui de la souveraineté alimentaire . Alors là , c' est une grande question . Parce que quand je vous dis : on va changer le mode de production , le mode d' échange , en général , vous voulez bien m' écouter . Mais quand je vous parle de souveraineté alimentaire , vous êtes encore d'accord . Mais après , on dit : &quot; Oui , mais qu' est -ce qu' on va manger ? &quot; C' est là que ça ne va plus . Mais , mes amis , on ne peut pas faire changer notre mode de production pour être conforme à l' intérêt général humain , on ne peut pas changer notre mode d' échange si on ne change pas notre mode de consommation . Si nous continuons à être l' esclave de désirs qui nous sont inculqués , de modes et de pratiques que nous n' avons jamais choisies , nous ne pourrons pas changer le système . Le système , il est dans notre peau . Il est entré ... dans nos goûts ... notre langue , nos papilles . Tenez , le vin - je ne sais pas si ça vous intéresse comme sujet , moi , ça me passionne - et ben , le vin , autrefois , si vous mettiez du sucre dedans et que vous mettiez des copeaux de bois et que vous touilliez un bon coup , on vous coupait les mains - ah , c' est au Moyen-Age , hein - pour avoir fait ça . Aujourd'hui , c' est encouragé parce qu' il y a un goût universel pour pouvoir vendre des quantités extravagantes de vin . Ben alors , vous direz : &quot; Oui , M. Mélenchon , vous êtes tranquille parce que du vin , on ne peut pas en faire tant que du Bordeaux , hein , forcément , il y a une limite . Et puis du champagne non plus . &quot; Ben si , les ballots . C' est réglé . C' est réglé , ça s' appelle grand marché transatlantique et on enlève les normes et les appellations , et voilà , vous aurez du champagne du Mississippi , du Bordeaux du Tennessee avec des copeaux , du sucre , tout ce que vous voulez , bien touillé . Et après , vous irez boire ça , et après , vous direz : &quot; Tiens , c' est curieux , il y a une épidémie d' obésité chez nous , d' où ça vient ? &quot; Et bien c' est ce qui arrive . La malbouffe a une conséquence terrible sur nos populations . Jamais on avait vu ça sur le territoire de la République Française . Autant de gens malades de cette épidémie d' obésité , qui crée toutes sortes de problèmes . Toutes sortes de problèmes , d'abord aux malheureux qui en souffrent . Surtout à eux . Alors , on laisse faire , on continue ? Et j' ai dit &quot; les désirs qui nous sont inculqués &quot; , cette uniformisation de l' humanité , quelle tristesse ! Je débarque à l' aéroport Fort-de-France . Je me promène , et qu' est -ce que je vois ? La même chose que ce que je viens de quitter . A 10.000 km de là . Le même McDonald's ... le même bâtiment dégueulasse . Alors ... c' est &quot; ah la la , c' est le moment de parler de culture &quot; , hein . C' est tout ce que vous savez faire ? Les mêmes horreurs qu' il y a ailleurs . Et on parlait avec une camarade , elle me dit : &quot; Mais ça , je connais , j' ai vu la même chose en Equateur , à Miami . &quot; C' est partout pareil . Ils ont tout nivelé . Tout homogénéisé . Tous pareils , sauf la paie . Les mêmes consommations , les mêmes objets , c' est de ça de quoi il faut s' arracher . Donc on doit changer nos modes de consommation . Et quand il s' agit d' alimentation , c' est pour ça que j' ai été voir le groupement régional des agriculteurs bio , c' est pour ça que je suis allé les voir , parce que je voulais vérifier que ce que je raconte est jouable ou pas . Je leur ai dit : &quot; Ecoutez , est -ce que c' est possible , la souveraineté alimentaire , l' autonomie alimentaire ? Est -ce qu' on est capable de produire de quoi nourrir tout le monde ici , parce que vous êtes déjà 450.000 . Est -ce qu' on est capable ou pas ? &quot; Est -ce qu' on a la technique ? Réponse : oui . C' est déjà pas mal . Est -ce qu' on a la terre ? Ça commence à être moins sûr . Pour deux raisons . Une partie a été pourrie . Le chlordécone , tout le monde connait . Ah , mais attendez , en métropole , personne n' est au courant , ou peu de monde . Et c' est quand même monstrueux , cette histoire , hein . Et ça continue . On en a encore pour un bon moment , il va falloir trouver le moyen de décontaminer tout ça . Allez , les ingénieurs . Venez ici , réglez -moi ça . Mais oui , parce qu' on peut le régler . Ce n' est pas une calamité contre laquelle on ne peut rien , ce n' est pas vrai . On est capable de décontaminer , on a les techniques , on a les méthodes . Et puis , pendant qu' on y est , on va commencer à discuter de savoir si c' est une si bonne idée que ça d' avoir comme modèle d' organisation de l' agriculture , comme unique objectif , exporter . Non . Le premier objectif , c' est nourrir . Donc relocaliser . Donc ça veut dire qu' il faut réorganiser , peut-être , l' usage de la terre . Est -ce qu' on peut ? Voilà la question . Est -ce qu' on peut ? Je discute avec ceux qui font de la banane , puisque le problème , c' était la banane , et que c' est toujours la banane , je discute avec M. Blondel . Il était là ce matin au marché . Et je lui dis : &quot; Mais vous arrivez à faire des bananes sans pesticides ? &quot; Il me dit : &quot; J' y arrive . &quot; &quot; Ah , comment vous faites ? Vous avez des bouillies ? &quot; , j' en connais un rayon , sur ces trucs là , à cause de la vigne . C' est ça , l' avantage d' avoir une activité politique , vous pouvez vous intéresser à des tas de choses , moi j' ai toujours été fantasque , je me suis toujours intéressé à plein de choses qui n' étaient pas dans mon domaine , hein . C' est comme ça que j' ai découvert la littérature , parce que si jamais j' avais fait ce qu' on m' avait dit , j' aurais connu que les auteurs classiques . Heureusement qu' il y a eu la littérature de gare pour que j' achète un bouquin de Glissant . Sinon , je ne savais pas que ça existait , hein . Comme Steinbeck et le reste . Je reviens à mon histoire . Est -ce qu' on peut ? Réponse : oui , on peut produire les bouillies qui ont la fonction de pesticide à partir des plantes qu' on a ici . Donc on peut produire en masse pour nourrir sans pesticides et avec d' autres méthodes d' agriculture . Donc il nous faut des lycées agricoles qui prennent en compte la nécessité de travailler autrement . Parce que ce n' est pas tout de dire que vous allez faire une révolution agricole si vous n' avez pas d' agriculteurs pour la faire . Hé , les gens , si je vous dis qu' on va avoir 400.000 postes de travail , ou 300.000 , ça dépend des évaluations , pour le nouveau modèle agricole , mes camarades étaient tout contents d' avoir trouvé ça , ils avaient fait le total et tout , ils avaient demandé aux fédérations , à la confédération paysanne et tout . Ah oui , oui , très bien . Et où vous les trouvez ? Où ils sont , les 400.000 en question ? Je ne vais pas prendre comme ça , 1 , 2 , 3 , 4 , toi , tu es paysan , 1 , 2 , 3 , 4 , toi , tu es paysan . Non . Il faut donc que le goût soit là , donc on transforme la condition paysanne . Vous ne pouvez pas dire à des gens qu' ils vont aller travailler à la terre comme des animaux . Personne n' a envie d' une vie comme ça . Et ben vous , vous n' en voulez pas , et ben , eux non plus . Donc il va falloir organiser tout ça différemment . Bon , est -ce qu' on peut le faire ? Réponse : oui , mais il faut changer vos habitudes . Franchement , il y a assez ici de légumes qui ont été adaptés avec le temps . Je ne dis pas que ça s' est fait d' une fois , hein . Ah , ben oui d'accord , non mais , si vous voulez à tout prix manger du riz et des pâtes , c' est votre affaire , on ne vous empêchera pas de le faire , hein , mais ce n' est pas normal qu' on soit dans cette situation , où on importe 96 % des poulets que vous mangez . Vous ne pouvez pas faire un poulet ici ? Qu' est -ce qui se passe ? Où vous en êtes ? Qu' est -ce que c' est que cette histoire qu' un poulet qui court dans la rue coûte plus cher qu' un qui arrive par avion ? Qu' est -ce que c' est que ce monde de fous ? Qu' est -ce que c' est , ce monde de fous ? C' est à ça que je demande de ... vous riez , mais vous avez raison de rire , mais il faut le regarder avec ce recul en disant : &quot; Mais comment on est arrivés là ? Comment on en est arrivés là ? Comment on en est arrivés à aller au magasin où on sait qu' on va se faire assommer pour manger des choses qui ne sont pas bonnes ? &quot; Et quel est le mystère de la formation des prix qui fait que vous vous faites dévaliser de la sorte ? Ah ouais ... Comme j' étais dans la rue , je me fais arrêter par un groupe de gens , là , très aimables et qui me payent un café , d'ailleurs , et puis on discute et ils commencent à me parler de l' octroi de mer , ils disent : &quot; On paye très cher , etc . &quot; Peut-être bien , mais l' octroi de mer , ça a servi de barrière , un temps , pour empêcher que les marchandises qui arrivent de l' extérieur exterminent les marchandises qu' on produisait là . Ben oui , le problème , c' est qu' on ne produit plus rien . Par conséquent , il se contente d' exterminer . Je vois bien qu' il y a quelque chose qui ne va pas , mais d' un autre côté , est -ce que vous allez dire aux collectivités locales dont c' est la principale ressource qu' il n' y a plus d' octroi de mer ? On peut le décider . Et on peut surtout dire que c' est complètement aberrant , parce que la collectivité locale a des sous pour aider des gens à condition qu' ils aillent consommer des choses dans le magasin qui leur coûtent cher . Tout ça est absurde , c' est comme ce que je disais tout à l' heure , c' est un cercle vicieux . Donc il faut trouver un autre moyen , premièrement , de financer les collectivités locales sans leur raconter d' histoires , parce que , elles , elles prennent des décisions , elles mettent en route des autobus , elles font ceci , cela , on s' occupe de la cantine , du reste , soit dit par parenthèses , la cantine , ça devrait être 100 % bio , le bio , ça ne doit pas être réservé aux riches , tous les gosses ont les mêmes droits , on est d'accord ? Donc , tout bio . et , mais si la collectivité locale , elle lance tout ça , et puis , tout d' un coup , couic , il n' y a plus d' argent , l' Etat n' en donne plus , ou en donne moins , évidemment , qu' est -ce qu' elle fait , la collectivité ? Soit elle vous enlève le service , et vous n' êtes pas contents , soit elle augmente vos impôts , et vous n' êtes pas contents non plus . Donc il faut que les collectivités locales aient une ressource qui soit assurée et garantie . Alors , bon , je ne vais pas vous lancer dans cette histoire là , c' est un petit peu compliqué , mais ça compte . Et puis , la deuxième chose , c' est que oui , il faut des barrières . Moi , je ne suis pas un libre-échangiste , je ne suis pas d'accord , pas d'accord pour que toutes les marchandises rentrent et sortent comme elles veulent , je ne suis pas d'accord . Je pense par exemple qu' une banane dont on est pas garanti qu' elle ne contienne aucun pesticide et qu' elle ait été cultivée par des gens qui sont esclavagisés , comme c' est le cas sur le continent , je ne suis pas d'accord pour qu' elles arrivent dans mon assiette et que j' ai pour motif de me dire : &quot; Ah , ben , je suis content , c' est moins cher . &quot; Je ne suis pas d'accord . Je ne suis pas d'accord pour que toute une série de produits de cette nature soit mis à la disposition de tout le monde au seul motif que ce n' est pas cher . Non . Donc , il faut que nous mettions des barrières , mais des barrières raisonnables , c' est ce que j' appelle le protectionnisme solidaire . Protectionnisme , oui , on protège nos productions . Voilà . Si vous voulez faire des poulets , si on veut qu' il y ait des poulets qui soient produits localement et que vous les mangiez , ben , il faut les protéger des poulets qui arrivent de l' extérieur avec ... Pas les poulets étrangers , hein , je n' ai pas dit ça , hein . Les autres poulets , quoi . Ces poulets , c' est toujours un bon exemple dans les meetings . Une fois , il y a un ami qui m' a dit ça , il m' a dit ... il faisait le registre électoral , oui , ben , ne vous marrez pas parce que ça vous concerne . Alors , c' était dans un pays qui est très exotique , hein . La Bolivie , ce n' est pas chez nous que ça arriverait . Ils avaient plusieurs millions de personnes qui n' avaient aucune inscription sur les listes électorales , ce n' est pas ici que ça se produirait , hein , on est tranquilles , ouais . Alors , donc , ils ont commencé à les organiser . Et alors , évidemment , les réactionnaires ont commencé tout de suite à hurler : &quot; Quoi , c' est les Cubains qui font le registre électoral ? Ils ont emporté les listes à Cuba . &quot; Vous savez , ils lisent les listes à Cuba , ça leur fait du bien . Et ça faisait peur aux gens . Et alors , le gars ... le gars , façon de parler , pardon , je suis irrespectueux , c' était le premier ministre , c' est un ami . Il s' appelle Ramon Quintana , et donc on discute et il me dit : &quot; Ecoutez , vous vous rendez compte , c' est quand même incroyable , ici , on peut manger une aile de poulet - oui , alors , les poulets qui reviennent - et le poulet , on sait d' où il vient , de quelle ferme , qui c' est qui lui a donné à manger , qu' est -ce qu' il a mangé , combien de fois . Parce qu' il y a dix codes-barres qui indiquent tout ça . Et ici , il y a des gens , on ne sait même pas qui c' est , ni d' où ils sortent . Et tout le monde s' en fout . &quot; Et bien , on devrait se poser la question de savoir , si nous , ici , on n' est pas un peu pareils , hein . Pensez -y avant la prochaine élection . Vérifiez un peu autour de vous de savoir si les gens sont bien maîtres de leur destin . Faites attention qu' on leur ait pas trop bourré le crâne à leur dire : &quot; C' est tous les mêmes , ça ne vaut pas la peine , on reste à la maison , on va bouder . &quot; Et c' est les mêmes qui gardent le pouvoir , les mêmes qui continuent à pratiquer les mêmes politiques . Il faut vous inscrire et il faut voter pour les faire dégager , il n' y a pas d' autre moyen , c' est comme ça dans la démocratie . Qu' est -ce que je ne vous ai pas encore dit , qui est sur mon papier ? Non , parce qu' après , vous allez vous fâcher si je continue comme ça . Avec tout ce qui ne me convient pas . Pas la façon avec laquelle nous traitons la situation . Alors voilà . Ou bien ça continue comme avant . Toutes les habitudes sont prises , il n' y a rien qui change . Vous continuerez à râler au même endroit sur les mêmes sujets . Moi , je vous dis juste une chose . Des territoires comme les nôtres , ici , sont fragiles . Vous ne pouvez pas vous figurer que vous allez continuer à diminuer la proportion de terres arables comme vous êtes en train de le faire . Diviser par 4 la surface de terres utilisables . Parce que quand on construit , figurez -vous que ceux qui investissent pour construire , ils ne vont pas à l' endroit où il y a des cailloux , des pentes et le reste . Ils vont à l' endroit où c' est plat . L' endroit où c' est plat , c' est justement l' endroit où on peut planter . Je ne vous donne que ça , comme exemple . Mais vous voyez que vous êtes dans une impasse . Je ne connais pas , moi , la réponse à chaque cas , et pourquoi je la connaîtrais , et pourquoi ce serait à moi de le décider ? C' est bien à vous , quand même , c' est vous qui avez le nez dessus . Mais il faut que vous puissiez intervenir , et pour ça , il faut changer la logique . Si la logique , c' est que le Président de la République vous dit : &quot; Vous allez être le pays pilote pour l' autonomie énergétique , vous allez être le pays pilote pour l' autosuffisance alimentaire . &quot; Et ben , vous allez vous y mettre . Et vous allez tous le faire . Et les collectivités vont s' y mettre , parce qu' elles sauront qu' on va les aider et pas les lâcher en cours de route , une fois qu' elles auront commencé . Vous voyez , c' est à ça que moi , je voudrais appeler . Je ne viens pas vous raconter tout ce qui ne va pas , vous le savez mieux que moi . Comment on s' est moqué de vous , vous le savez mieux que moi . Je viens plutôt vous appeler à ça . A cet élan , on va faire les choses . Nos gamins ont un avenir . Nous-mêmes , on en a , le monsieur qui m' a arrêté dans la rue , qui m' a dit : &quot; M. Mélenchon , qu' est -ce que vous allez dire sur le chômage , parce que moi , je voudrais travailler . &quot; Tout le monde veut vivre dignement en contribuant à l' effort collectif par son travail . Ce n' est pas vrai que le pays soit rempli &quot; d' assistés &quot; , comme ils disent , avec ce vocabulaire qui n' est pas le nôtre . Nous , on parle de solidarité . Ce n' est pas vrai . Il y en a assez qu' on montre du doigt les pauvres en les rendant responsables de leur pauvreté . Les chômeurs en les rendant responsables de leur chômage . Les gens malades en les rendant responsables de maladies qui sont arrivées par l' environnement . Et qu' on entende dire : &quot; Les gens , il faut vous responsabiliser . &quot; Voilà , responsabilisez -vous . Attrapez la grippe en été , il y en a moins qui l' ont . Qu' est -ce que ça veut dire , ces histoires ? Cette manière , toujours , d' inculquer aux braves gens une sorte de dégoût d' eux-mêmes , une méfiance à l' égard d' eux . Leur enlever leur fierté , le goût de ce qu' ils savent faire , de leur envie de contribuer au bien commun , en les humiliant sans cesse , en les montrant du doigt . Vous avez vu cette émission infâme où on montrait des pauvres gens qui étaient culpabilisés parce qu' ils avaient des aides sociales ? Et le peu qu' ils avaient , on le leur reprochait encore . Ça suffit . Ouvrez vos cœurs . Ne supportez plus qu' on vous parle comme ça . Ne laissez plus dire ces choses . Bon , je l' ai fait à votre place , pour le coup , là . Mais enfin , quand même , vous ne pouvez pas être la terre de Césaire et des autres et ne pas entendre ce que je viens de dire , je sais que vous l' entendez . Maintenant , il faut convaincre les autres . Ça ne peut plus durer comme ça . Ça ne peut plus durer , d'abord , ça ne peut plus durer pour la planète . Ça ne peut plus durer pour nos gosses . Ça ne peut plus durer pour la vie qu' ils nous font , l' image qu' on a de nous-mêmes quand on se regarde . Voilà toutes ces choses qu' il faut reprendre . C' est ça , l' enjeu d' une élection . Ce n' est pas juste de savoir si vous allez élire celui -ci ou celui-là , encore que c' est celui -ci qu' il faut élire . Bon . Là , il va falloir finir , alors je vais aller un petit coup dehors , quand même , les pauvres , ils ont été patients . Il reste quelqu'un dehors ou pas ? Qu' est -ce que vous en dites ? Wouaouh . Ça va , là ? Mais vous êtes restés là , tout ce temps ? De qui je pourrais dire du mal , maintenant ? Non , je viens maintenant vous dire ... je viens vous dire au revoir après tout ce que j' ai déjà dit . Alors , les gens , je vais vous parler pays . D'abord , des excuses pour les littéraires qui m' ont entendu , et vous avez entendu tout à l' heure , là , sur la vidéo , citer Aimé Césaire . Mais je l' ai mal fait . Il y a un complot , là . Comment je dois me mettre ? Ça compte marcher un jour , là ? Merci , monsieur . Vous m' entendez encore ? Bon , ça va aller , alors . Je l' ai mal cité . Mais j' espère que je me suis montré à la hauteur , aujourd'hui , de ce que j' ai voulu citer hier . Alors , je vous la relis , telle qu' elle est dans le texte d' Aimé Césaire . Et à vous tous , chacun à sa façon , d' en faire autant . &quot; Ma bouche &quot; , dit Césaire , &quot; ma bouche sera la bouche des malheurs qui n' ont point de bouche . Ma voix , la liberté de celles qui s' affaissent au cachot du désespoir . &quot; La leçon qu' enseigne Césaire , à cet instant , est qu' il n' est pire maladie de l' âme et du cœur que la résignation . Renoncer à changer le monde , c' est renoncer à être soi-même dans un monde meilleur . Ce à quoi je vous invite , quoi qu' il arrive , quelle que soit l' issue de la volonté de notre seul souverain , le peuple , c' est à garder au cœur cette idée , que génération après génération , vous autres , là , moi , qui n' ai plus votre âge , nous tenons le même fil rouge dans le temps , portant la même lumière , la même promesse , pour toujours , de rebond , de renouveau , de recommencement , d' espérance , de nouveauté , de bonheurs possibles . Césaire , encore , discourant contre le colonialisme . Le colonialisme , mes amis , qui est entré comme un pied dans sa chaussure dans le nouvel âge du capitalisme , le colonialisme qui faisait de cette terre , une terre de monoculture pour l' exportation au prix du travail qui coûtait le moins cher possible , celui des esclaves . Le colonialisme a remis son pied dans le capital financier transnationalisé , lui aussi , il vous appelle à continuer à produire pour l' exportation au moins cher possible . C'est-à-dire sur le dos de la souffrance et de l' humiliation du grand nombre , les uns , condamnés au travail toujours plus intense , les autres , à la désespérance du chômage . Le colonialisme , qui pratique le même aveuglement inhumain qu' il pratiquait autrefois , à sa façon aujourd'hui , quand il refuse de voir que 40 % des espèces vertébrées ont déjà disparu de la surface de la planète , que c' est dans les DOM et les TOM que disparaissent le plus grand nombre des êtres vivants , dont la diversité était la magie de vivre , celle des imaginaires , celle des poésies , celle des comparaisons , ce ne sont pas seulement des bêtes qui disparaissent , ce ne sont pas seulement des mondes qui s' engloutissent , c' est une manière d' être un être humain qui disparaît . Voilà le vrai défi auquel vous êtes appelés , et à vous spécialement , jeunes gens , jeunes hommes , jeunes femmes , à vous , qui portez nos espérances , votre devoir est d' infléchir le cours de l' histoire , de ne pas permettre qu' ait lieu cette immense catastrophe qui se dessine sur la civilisation humaine . Portez en vous , pour toujours , la leçon d' Aimé Césaire . &quot; Une civilisation qui s' avère incapable de résoudre les problèmes que suscite son fonctionnement est une civilisation décadente &quot; , dit Césaire . Cette décadence s' interrompra si , à mon appel , puisque c' est moi qui porte cette parole , aujourd'hui , pour vous tous , à mon appel , à notre appel à tous , nous invitons les peuples qui sont ici en Caraïbe , qu' ils soient français , ou qu' ils ne le soient pas , tout notre environnement , géographique et humain , culturel commun , que nous appelons avec cette formule qui est contenue dans le titre de notre programme , à un avenir commun . Nous sommes tous , d'abord , des êtres humains semblables par nos besoins , nos rêves , nos espérances , nos amours . Faites prévaloir ... l' intérêt général humain . N' ayez jamais honte d' être bons , d' être tendres , d' être partageux . N' ayez jamais honte d' être des êtres humains et de mépriser les obstacles que l' on vous présente parce que vous vous sentez capables de les dépasser . Allez ! ",
    "Vous m' entendez , vous m' entendez bien ? Parfait . Bah il faut écouter , elle dit non , elle là-bas ! Bah écoute ! Vous entendez , ce coup-là ? Bon . Je commence avec vous là dehors , parce qu' on a plus assez de place pour que vous soyez tous dedans , et puis ensuite j' irai dedans , et si vous y êtes encore , je viendrai vous saluer avant qu' on se quitte . On m' avait dit : &quot; Il y a tellement de brume ! &quot; Mais moi je vous connais , il n' y a pas de brume assez épaisse pour que vous ne trouviez pas votre chemin pour faire ce que l' on est en train de faire à cet instant . Bien sûr , on est venus écouter quelqu'un qui a des idées à présenter à propos de la condition des salariés . Mais on est d'abord venus pour marquer un moment de protestation contre , d' indignation humaine , contre le sort qui est réservé a tant de milliers d' ouvrières et d' ouvriers , de salariés , de petits rangs qui ne peuvent se défendre parce que toutes leurs vies dépendent de ce pauvre travail . De ces heures qu' on leur accorde , de ces temps de travail qui ne sont jamais complets , et où l' on vit dans l' attente d' heures complémentaires pour pouvoir améliorer la paye à la fin du mois , que les puissants sont prêts à tout moment à leur disputer au centime près ... Quand eux regorgent et se gavent - quelques centimes ici , que l' on supprime aux heures supplémentaires ; quelques heures de travail que l' on va payer moins ; une amplitude horaire que l' on va exagérer , laissant cette femme qui est partie tôt de chez elle le matin , laissant ses enfants , et qui ensuite va rester deux heures , trois heures sans emploi avant de reprendre le travail . C' est ça que nous sommes venus d'abord dire , que vous êtes venus d'abord dire avant que de m' écouter . C' est un acte , notre rassemblement - ce déboulé , comme je l' ai appelé - un acte de solidarité avec cette jeune femme qui a perdu son enfant uniquement du fait du mépris que ceux qui en avaient la responsabilité ont eu à son égard , qui n' ont pas écouté ce qu' elle avait à dire . Mais vous savez , ce n' est pas la première fois , et c' est bien parce que ce n' est pas la première fois que je suis venu . Souvenez -vous : c' était pas si loin d' ici . Souvenez -vous , si vous croyez quelque part qu' il y a une limite à la brutalité de ce monde de la flexibilité , de la compétitivité , bla bla bla ... Souvenez -vous de cette jeune postière - qui est dans la salle sans doute -Émeline , et si vous ne vous souvenez pas , regardez ce film . Il passe là , en principe . Est -ce qu' il passe ? Je vais vous le raconter , parce que visiblement ça ne marche pas . C' est une jeune femme , et elle a un CDD , qu' on renouvelle , comme vous connaissez ça - c' est absolument illégal , hein , mais ça fait rien , ils renouvellent . Et de CDD en CDD , comme elle est postière on la met jamais sur le même parcours . Si bien qu' à chaque fois , tout l' effort particulier du postier ... Car il y a ceux qui n' ont jamais fait aucun travail de cette sorte , qui croient que c' est facile , qu' on passe d' une porte à l' autre avec le courrier . Non ! Il faut connaître sa tournée , et si vous la connaissez bien vous la faites plus vite , donc vous avez une chance de terminer à l' heure qui est prévue - parce que si vous n' avez pas terminé à l' heure qui est prévue , ce qui est prévu c' est que vous ayez fini la tournée . Donc vous allez faire du temps de travail de plus , un temps de travail qui ne vous est pas payé . Eh bien , cette jeune femme , comme elle a la tête un peu dure , les chefs l' ont repérée . J' en profite pour appeler chaque chef à se souvenir de sa responsabilité humaine dans la conduite des équipes . Eh bien , ils la font changer de place , et cette femme a une faiblesse à un moment donné et elle prévient , elle dit : &quot; Mais moi je ne peux pas continuer , je suis mal &quot; &quot; Roh , tu t' écoutes trop , continue ! &quot; Et ainsi de suite , vous connaissez la suite de cette histoire . A la fin elle fait un AVC , et c' est tout juste si on l' a pas traitée de simulatrice . Et comme on n' est pas intervenu assez vite , elle reste avec un handicap . Mesdames et messieurs , mes chèr(e)s ami(e)s , je n' ai pas l' intention de montrer du doigt celui -ci ou celle-là , personnellement responsable de cette situation . Vous voyez , je fais comme vous ! A ma manière j' ai bon cœur , et je suis prêt à oublier la brutalité si celui qui l' a commise s' en repent et prend les mesures pour ne pas la recommencer . Mais que dois -je dire devant vous à cette heure , quand je sais qu' avoir après proposé un obscur arrangement à cette jeune femme , pour terminer , la Poste se déshonore en lui refusant le CDI qu' elle devrait lui donner maintenant après ces souffrances et cette marque qu' elle porte à vie ! De leur indifférence au sort des autres , les puissants veulent toujours avoir le dernier mot ; et c' est avec ça qu' ils impressionnent , qu' ils font peur et que les gens ont peur . Et c' est de cette peur dont nous voulons délivrer le monde du travail , peur du lendemain , peur de perdre son emploi , peur de ne pas avoir sa paye ! Et après ça , quoi ? Peut-être de ne plus avoir son appartement et de se retrouver comme tant de gens , bringuebaler d' un endroit à l' autre ! On m' a dit : &quot; Ce n' est pas la peine de faire du Zola &quot; . Je ne fais pas du Zola ! C' est la vie qui fait du Zola ! C' est vous qui faites du Zola ! C' est vous qui méprisez tous ces gens , qui faites du Zola ! C' est vous qui exagérez ! Mais il y a un principe d' humanité qui est aussi un principe de morale individuelle . A Auchan , c' est la deuxième fois . La première fois , une malheureuse s' était trompée de 85 centimes . Elle a été immédiatement expulsée ; fort heureusement , le syndicat l' a rétablie dans ses droits - pas tous ses droits , mais il a fallu l' action syndicale . Pendant ce temps , évidemment après un crime aussi affreux , on espère qu' elle se repent ! De son côté , madame Lagarde avouait une négligence de 400 millions , pris dans vos poches ! Et ils ne lui en ont tenu d' aucune manière . On lui a fait quelques remontrances , ça ne sera pas dans son casier judiciaire et elle ne fera pas un jour de prison . Et elle ne perdra pas , non plus , un jour de sa paye car le jour où elle était au tribunal elle n' était pas au travail . Eh bien ici , j' ai à dire - mais d'abord je veux que vous entendiez , si cela se peut , de sa bouche l' histoire de Fadila . Et cette fois -ci peut-être que ça va marcher . * Le 22 novembre , Fadila , employée de Auchan City , fait une fausse couche sur son lieu de travail . L' entreprise au courant de sa grossesse et malaises répétés n' a pourtant rien fait pour l' aider . Ici vous avez l' ensemble , ou presque l' ensemble , des bases ( c' est à dire les entreprises où nous sommes présents où nous avons des délégués , ou des militants ) . Déjà pour information , vous voyez le gros paquet qui est là , c' est Auchan City . Tous les gens qui sont derrière ces dossiers-là , c' est des gens qui sont anonymes , qui font pas la Une des médias , et tous ces gens qui sont derrière ces dossiers ils morflent . Parce que quand vous vous êtes présenté aujourd'hui dans une entreprise avec un mandat CGT , avec des revendications , vous avez affaire à une répression qui est absolument terrible et qui reste secrète , que personne ne connaît . Fadila : Aujourd'hui , pourquoi on porte plainte contre Auchan ? Bah c' est justement pour avoir des réponses , c' est la seule manière d' avoir une réponse de la part d' Auchan , parce qu' ils disent n' importe quoi sur l' article qu' ils ont sortis à la presse , c' est tout des mensonges . Encore une fois , ils me font passer , moi , pour la menteuse - oui , la fautive et la menteuse . &quot; Non , on n' a jamais donné de négatif pour aller aux toilettes à Fadila &quot; alors je suis resté dans mon siège , je m' amusais de saigner à ce moment-là &quot; Bah si , tombe mon enfant , meurs ! &quot; Je sais pas , c' est ça qu' ils veulent me dire ? &quot; Bah en fait , elle s' amusait à rester dans son siège et à saigner &quot; Mais c' est n' importe quoi ça ! Et aussi &quot; Non , on est intervenus très vite &quot; . Vous êtes intervenus parce que j' ai fermé ma caisse , si j' aurais pas fermé ma caisse , vous seriez jamais venus me voir . C' est moi qui est intervenue pour moi-même , c' est moi qui a pris la décision de fermer ma caisse et c' est pas vous qui avez appelé les pompiers , c' est moi qui a appelé les pompiers . C' est moi qui a parlé avec les pompiers , il y a un rapport de pompier derrière . Et ici , on reçoit donc énormément de personnes , qui sont dans des situations analogues , et notamment les jeunes précarisés , contrats de professionnalisation , apprentis , CDD , intérim , voilà : le tout-venant . Là on est pas dans la bureaucratie , on est pas dans l' état-major , on construit pas un rapport de force d' entreprise . On le fait aussi , hein , mais au quotidien on est littéralement débordés . On est dans une ville qui connaît , qui a 1/4 de sa population qui vit sous le seuil de pauvreté , avec un taux de chômage considérable et une précarité énorme , et Mulliez comme employeur tout-puissant , avec les méthodes qu' on connait . Voilà . Il y a un nom qui a été cité , eh bien , je vais lui dire ce que j' ai décidé . Quand on est très puissant on a de très grandes responsabilités . Je sais bien , il y a des gens qui sont milliardaires mais l' argent ne donne pas tous les droits . La valeur d' une personne n' est pas décrite par son compte en banque . J' ajoute qu' être milliardaire est immoral . A quoi bon d' avoir 1 million si c' est pour en vouloir un deuxième ? A quoi bon d' avoir 1 milliard si c' est pour en vouloir un deuxième ? Je ne crois pas qu' il y ait de vertu à accumuler de cette manière car cette accumulation , d' une façon ou d' une autre , c' est de la valeur soustraite à la société . C' est de la richesse captée à un endroit qui pourrait être répartie différemment sur tous les autres . Mais qu' on n' aille pas donner de leçons à Fadila , qu' on n' aille pas la traiter de &quot; menteuse &quot; , qu' on n' aille pas répandre des rumeurs sur son compte , qu' on n' aille pas dire qu' elle n' aurait pas respectée son contrat ! Tiens , à propos : son contrat , c' est un contrat de professionnalisation . Ils ont une inventivité extraordinaire pour trouver des mots pour habiller des situations inacceptables . &quot; De professionnalisation &quot; , voici qu' elle apprend le métier de caissière , ce qui permet , parce qu' elle est en contrat de professionnalisation , de ne pas la payer au SMIC . Et ensuite , pour celui qui l' emploie , de bénéficier d' allègements de cotisations sociales , c'est-à-dire qu' il ne paie pas . Qui les paye , ballot ? C' est écrit dans la loi : chaque fois qu' un euro n' est pas versé en cotisations sociales , c' est le budget de l' État qui le prend en charge . Qui c' est l' État ? C' est vous ! C' est avec vos impôts qu' est financée cette situation inacceptable . Cette subvention , cet assistanat qui est fait à qui ? A une chaîne milliardaire ! Pas à des gens qui auraient besoin de solidarité ! Alors , qui sont -ils ? Quels sont leurs droits particuliers , personnels à se comporter de cette façon ? Qui est cet homme ? Où il habite ? En Belgique . Il a le droit , pourquoi pas , mais ses impôts habitent où ? En Belgique aussi ! Evidemment , tout cela doit avoir une fin , et il y aura une fin à cette situation , si vous le décidez . Si je suis élu , l' impôt sera perçu sur la base de la nationalité du cotisant quand il n' est plus en France , c'est-à-dire qu' il sera universel . Et Monsieur Mulliez pourra bien aller en enfer , que je m' y retrouverai pour prendre sa part d' impôt . Les citoyens français devront déclarer partout dans le monde ce qu' ils payent comme impôt à la nation qui les accueille , et il est bien normal qu' ils en payent là où ils se trouvent , car nous pratiquons de même chez nous . Mais ils devront déclarer ce montant au fisc , et celui -ci regardera si - après avoir payé cette somme , en France - ils ne devraient pas encore d' argent . L' impôt sera donc universel . Qu' on ne me dise pas que c' est impossible , puisque ça se pratique aux États-Unis d' Amérique et j' aime mieux vous dire qu' ils le font avec une certaine férocité . Comme vous le savez , des gens comme moi , finissent par tout savoir , et vous savez pourquoi ? Parce que je suis lié à ceux qui font , et ceux qui font viennent me raconter ce qu' ils font . Et je connais les employés de banque qui m' ont expliqué comment on leur a demandé de rechercher les suspicions &quot; d' américanitude &quot; de certains de leurs clients . Vous voyez ? On le fera . J' en prend l' engagement solennel devant vous , et Monsieur Mulliez , comme les autres , paiera ses impôts en France . Nonobstant les autres enquêtes qui courent sur son compte . Mesdames et Messieurs , je ne suis pas venu , seulement , mais ce serait déjà bien , pour parler de ce cas , et faire qu' ensemble nous mettions tout notre poids dans la balance pour que les dirigeants d' Auchan sachent que s' ils ne règlent pas la situation particulière de ce magasin qui s' est spécialisé dans les brutalités , nous allons nous occuper , nous , de faire connaitre la marque pour d' autres raisons que les produits qu' ils vendent . Et que tous autant que nous sommes là , nous saurons quoi faire et que dire autour de nous à propos de cette firme . Mais , je me suis dit &quot; c' est le moment &quot; : puisque c' est une campagne présidentielle , tout le monde écoute . Les gens , c' est bien normal , s' intéressent à des questions que le reste du temps ils laissent de côté , c' est normal , mais maintenant ils s' y intéressent , ils écoutent . Il faut donc faire une campagne électorale qui soit une campagne instructive , et pas seulement une campagne où je viendrais vous raconter mes mérites qui n' ont aucune espèce d' importance dans la situation que nous connaissons . Une campagne où tout d' un coup on repense à tous les problèmes de notre patrie et comment les régler . J' ai commencé une campagne , déjà , en 2012 , dans un endroit , c' était Fralib , peut-être que vous vous souvenez . Les ouvrières , les ouvriers qui étaient là , on fait de leur entreprise qui allait fermer une multinationale déjà , une coopérative ouvrière . Ils ont bataillé , galéré , miséré , mais à la fin ils ont eu le dernier mot . Et à cette occasion , j' étais venu pour commencer dans leur usine et j' avais dit : &quot; Voilà , je vais rendre visible ceux qui ne le sont pas , par ma campagne . &quot; Le mot a eu du succès , je l' ai retrouvé dans plein d' autre bouche . Les invisibles devenant visibles . Mais c' est le moment - dans l' ambiance particulière que je vois déjà se dessiner - où une nouvelle fois revoici les professeurs Cyclotron , les inventeurs permanents d' une même -idée ... Pressez le citron , il en sortira du jus ! Et si on n' a pas assez pressé , pressez encore ! Je viens témoigner , parler pour et au nom de 6 millions d' ouvriers : la classe ouvrière existe ! De 7 millions d' employés qui ont un statut souvent très proche de ce qu' on appelle un ouvrier . 13 millions de personnes ! Il n' y a pas de classe plus puissante en France , ni de classe plus utile car c' est elle qui fait tout ! Tout ! Sans le travail humain il n' y a aucune richesse produite , et personne ne peut ni répartir ni accaparer ! Mais comme on a mal agi , Comme on a culpabilisé ces gens du moment où ils n' étaient plus présents dans d' immenses entreprises où on les voyait , où on les craignait , on s' est dit : &quot; Les voici dispersés ! &quot; Et on les a convaincus eux-mêmes qu' ils n' existaient pas . Les ouvriers , les employés sont les absents de toute représentation . 3 % des gens qui sont interviewés à la TV sont des ouvriers . 61 % sont des cadres - j' ai rien contre les cadres , il y en a plein ici , mais ça c' est pas la vraie vie ! La vraie vie c' est tout le monde , c' est pas seulement certains et les autres pas du tout . Si bien que chacun rentre la tête dans les épaules , et combien ont perdu la fierté que leur enseignaient leur père et leur mère , qui ont sur cette Terre , fait naître tous les droits à partir desquels nous avons fondé la République , et cette forme particulière qu' est la nôtre ? A qui la devons -nous ? Pas à la bienveillance des puissants qui n' ont jamais rien cédés sans qu' on le leur arrache Nous le devons à la détermination , la constance , l' implication au travail et à la lutte des ouvrières , des ouvriers et des salariés : telle est l' histoire de France , et c' est à elle que je rend hommage à cet instant . Les ouvrières , les ouvriers , les salariés , les petits grades n' apparaissent à l' écran que lorsqu' on veut montrer des violences après les manifestations . Les ouvrières , les ouvriers , les petites gens ne sont citées que pour dire qu' ils sont responsables de la montée du Front National , ce qui est un mensonge ! Répété , et à force d' être répété avec l' espoir qu' il soit cru et que ceux-là à qui le message est adressé leur rentre dans la tête . La vérité c' est que 60 % des ouvriers et des petits grades du monde du travail , de l' emploi , ne votent pas : ils s' abstiennent . Je viens leur dire &quot; Sortez de votre trou , mêlez -vous de vos affaires , emparez -vous de cette situation ! &quot; Ne vous auto-éliminez pas de la décision ! N' abandonnez pas vos bulletins de vote ! C' est après eux que sont accrochés votre liberté et votre dignité , car la nouvelle vague des beaux parleurs qui est revenue , est revenue pour vous annoncer de nouveaux sacrifices , de nouvelles brutalités , de nouveaux viols de vos droits , de votre sécurité sociale que l' un veut vous arracher , que l' autre veut vous trafiquer et ainsi de suite . Il le faut ! Il faut que dans cette campagne enfin on accepte de prendre conscience de ce qu' est la situation de cette immense classe . Est -ce que nous sommes pas capables de nous mobiliser d' une manière extraordinaire quand il s' agit de lutter contre le terrorisme ? Et nous avons bien raison ! Mais alors , pourquoi est -ce que nous ne nous mobilisons pas d' une manière extraordinaire quand les conditions dans lesquelles les gens travaillent aujourd'hui fait qu' il y a chaque année 565 morts sur le poste de travail ! Comme ça a failli être le sort de cette malheureuse postière et de combien d' autres de ceux que vous connaissez ! Ensuite , ce sont les accidents de trajet . Non , ce n' est pas les plus importants , c' est sur le poste de travail que ça se passe . 3 morts par jour , 3 morts chaque jour . Chaque jour nous est enlevé quelqu'un dont le savoir-faire , l' éducation , le talent était une contribution à la vie de la société . Chaque jour est enlevé à l' amour de ses proches quelqu'un qui allait gagner leur vie . Voilà la brutalité de ce système . Que fait -on ? Que se passe -t-il ? Rien . Et c' est pas tout . Combien de gens dorénavant soumis à une forme et à une cadence de travail que les puissants ensuite nient se suicident ? Notre présence aujourd'hui accompagne comme une clameur le suicide des infirmières de cet été , de ces 50 postiers paraît -il , qui ont mis un terme à leur vie parce que leurs conditions de travail étaient devenues insupportables . De cette paysanne ou de ce paysan qui tous les 2 jours met fin à sa vie , parce que c' est trop , ce qui leur est demandé au travail . C' est contre tout ça qu' il est temps de se mobiliser , de se rassembler , mais tout commence par le sentiment d' humanité , le sentiment de partager quelque chose avec les autres , qui s' appelle la vie ! Que nous avons en commun , avec nos amours , avec tout ce qui fait le bonheur de vivre . Cessez de regarder les gens comme des pions , comme des machines qu' on peut traiter indifféremment de leur sentiment , de leur état de santé . Regardez comme ils passent leur temps continuellement à instiller le doute . Il n' y a pas une émission où il n' y a pas un Monsieur très savant avec des graphiques à la main qui vous montre quelle bande de tire-aux-flancs vous êtes . Les français travaillent moins que les autres et vous , vous rentrez la tête dans les épaules ! Non ! Ce n' est pas vrai ! Ils travaillent autant que les autres , mieux que les autres ! Parce qu' ils ont une productivité 20 % supérieure à celle de nos principaux concurrents , puisqu'ils vivent tout en terme de concurrence ! On n' en parle jamais de ça . Alors ils vous sortent un graphique et vous , vous êtes au milieu , vous vous dites &quot; Oh là là , on est pas loin du bas &quot; . Vous n' avez rien compris : depuis quand travailler beaucoup est un gage de succès pour une économie ? Vous voulez savoir qui est -ce qui travaille le plus en Europe ? C' est les bulgares ou les roumains . Vous voulez faire bulgares ou roumains ? Ça vous intéresse ? Déjà que j' ai eu un mal de chien à vous convaincre de ne pas vouloir faire les Allemands , jusqu' à ce que vous vous rendiez compte qu' il y a plus de pauvres là-bas qu' il y en a ici ! Enfin , tout ça est absurde ! Tout ça est absurde ! Je viens de vous parler de ces accidents , de ces suicides , de ces maladies et vous avez tous l' air d' être d'accord avec moi que c' est pas une situation acceptable ? Mais vous , vous êtes pas très intelligents , vous n' avez pas fait les grandes écoles et vous êtes même pas chefs ! Et vous êtes pas ministres socialistes . Ni de droite non plus . Alors de quoi vous vous mêlez , hein ? Qu' est -ce qu' ils font pour répondre aux problèmes ? Tout à l' heure j' ai dit rien , non ! Maintenant ça a progressé , ils font moins que rien . Parce que là nous venons de parler de questions de santé . J' ai entendu il y a pas longtemps , Monsieur Macron , tout d' un coup dire que ... Depuis qu' ils n' ont plus de responsabilités , tous ceux-là ils ont tous des idées . Et souvent le contraire de ce qu' ils faisaient et ils nous croient assez ballots pour les croire . Ils se disent que ... Comme on ... Hein ... Alors on va les croire . &quot; Alors Monsieur Macron , il parait que vous dites qu' il faut faire du &quot; préventif &quot; plutôt que de soigner ? &quot; Non mais commençons par dire qu' on va soigner les gens , hein . Non mais avec vous , ça mérite qu' on vérifie ! Mais si vous voulez faire du préventif , comment ça se fait que vous avez fait passer la loi El Khomri qui supprime la visite médicale obligatoire quand on reprend un poste de travail ? Est -ce que le préventif ça commence pas par ça ? Quand ces 2 jeunes femmes ont -elles eut droit à un examen par le médecin du travail ? Notamment Fadila ? Jamais ! Et pourtant la loi s' appliquait jusqu' au 1er janvier . Le 1er janvier maintenant , c' est la loi El Khomri . Et maintenant , il n' y a plus de visite médicale obligatoire . Un infirmier peut vous examiner . Mais un infirmier c' est un boulot spécial , particulier , c' est un métier , vous autres le savez pas , les puissants ? Infirmier , c' est un métier , médecin c' est un autre métier . Brancardier c' est encore un autre métier et ainsi de suite . Vous ne connaissez rien au travail ! Et pourtant vous gesticulez . Le travail , celui -ci est le candidat du travail , l' autre aime le travail et vous faites des discours sur la valeur travail car vous êtes censés être tous des tire-aux-flanc . Vous ne savez pas la valeur du travail , pourtant vous en vivez . Pourtant , vous , vous avez personne à la maison qui vient faire votre travail à votre place . La première valeur du travail ce sont les travailleurs , ce sont les salariés et c' est en leur direction qu' il faut tout organiser . Si vous voulez faire de la prévention , alors rétablissez les droits de la médecine du travail , redonnez à un médecin du travail ce qui était autrefois son droit qui était que lorsqu' un salarié se présentait devant lui , et que d'après son analyse la nature du poste de travail qu' il occupait pouvait nuire à sa santé compte tenu des conditions particulières de telle ou telle personne , qui ne peuvent se mettre dans aucune loi ni aucun texte général , alors il pouvait recommander OU un changement du poste de travail OU des conditions de ce travail OU de l' endroit où pouvait aller la personne en question . Tout ça , c' est fini . Vous arrivez , contrat de professionnalisation parce que vous êtes une femme , que vous êtes jeune et que vous avez pas le choix : &quot; Allez , on va te professionnaliser , tu as le droit à des heures de formation ... DES c' est trop , UNE . On va t' apprendre comment on fait marcher la caisse parce que les autres ils ne savent pas que faire marcher une caisse c' est pas simple et c' est épuisant . Et ils pensent qu' en une heure on apprend , et que ça suffit largement , et le reste du temps &quot; Allez , travaille , tais -toi ! &quot; &quot; Qu' est -ce qu' il y a ? un délégué syndical ? Ça va pas ? Tu veux perdre ton boulot ? &quot; Voilà ! Ce que sont les conditions quotidiennes , la vraie vie , les vraies conditions dans lesquelles ça se passe pour 13 millions de personnes . Je ne dis pas que ça soit comme ça partout , heureusement . Mais il faut bien qu' on parle pour ceux qui en pâtissent le plus , sinon qui le fera ? Et il faut bien qu' en pensant aux plus désarmés , aux plus faibles , on organise la société parce que c' est comme ça qu' on la rend meilleure , depuis toujours . C' est en s' occupant de ceux qui ont le moins de moyens de le faire par eux-mêmes , voilà comment on fait avancer et progresser la société . Donc il faut rétablir cette médecine du travail . Ça revient à dire une chose : il faut abolir la loi El Khomri , et je le ferai . Mais je m' y arrête encore un instant , pour bien me faire comprendre , et je voudrais que vous repartiez avec l' esprit critique dont vous allez avoir besoin pour ne pas vous laisser emballer dans le papier journal dans lequel il y a tous ces beaux articles pour ces belles personnes . Un peu d' esprit critique . Cet homme parle de &quot; préventif &quot; après avoir supprimé la médecine du travail . Quand on fait du préventif , comment le fait -on ? On va pas dire &quot; Dans chaque entreprise , discutez ! &quot; Ah bah non , on sait comment ça va se finir ! Parce que celui qui s' occupe de l' entreprise , son métier c' est pas médecin , c' est entrepreneur . Mais bien sûr , chacun fait d'après ce qu' il comprend et ce qu' il croit urgent . La seule chose qui protège tout le monde c' est la loi . La loi , le règlement , qui prévoit des durées de travail , je vais en parler dans un instant , etc . etc . Par conséquent , si vous voulez que la santé des gens soit protégée et qu' elle ne soit pas soumise aux aléas qu' on peut comprendre , de la nécessité d' une production qui fait qu' on se tourne vers un travailleur et on lui dit &quot; Bon écoute , c' est pas parfait , bon d'accord il manque un clapet pour la sécurité &quot; ou bien &quot; Bon c' est vrai , il y a beaucoup de poussières , enfin , bon , tu vois bien , il faut qu' on rende le produit si on veut que la boîte tourne . &quot; Et le gars ou la fille le fait parce qu' il y a une chose que tous ces gens oublient , c' est que les gens aiment leur métier . Quel qu' il soit , et ils aiment faire bien leur travail quel qu' il soit . Ca c' est une constante de tout le monde , les gens font pour le mieux . Alors , si vous vous soumettez , les décisions concernant la santé , les discussions dans les entreprises , ça va être vite réglé ! Surtout dans les entreprises où c' est les gros durs . Parce qu' en plus ça se fait pas de dire qu' on n' est pas bien , que c' est trop lourd , ça se fait pas . Alors on dit rien et la vie va comme ça . Donc c' est la loi qui vous protège . Où est la loi ? Avant , il y avait la loi et puis la convention collective de branche et puis l' accord dans l' entreprise . C' était impossible d' avoir un accord dans l' entreprise qui soit moins bon que l' accord de branche et l' accord de branche pouvait pas être moins bon que la loi . Ça s' appelle le principe de faveur . On a lutté , tous les syndicats ont lutté pendant des décennies et des générations pour obtenir cet ordre là . Extraordinaire mes ami(e)s , et quand dans une branche on avait une bonne négociation , la loi étendait le bénéfice à tous les travailleurs . Raison pour laquelle , dans notre pays , on est pas obligé d' être syndiqué pour bénéficier d' un accord syndical . Tandis que quand vous allez en Allemagne vous ne bénéficiez pas d' un accord syndical si vous n' êtes pas membre du syndicat . Et si votre patron n' est pas membre du syndicat patronal qui a signé avec le syndicat ouvrier , vous n' avez pas l' avantage . Chez nous , si , tout le monde . C' est donc ça notre démocratie sociale , elle est fragile , elle est délicate . C' est un mouvement entre le bas et le haut et à la fin c' est la loi . Parce que dans toute situation , il y a au moins 3 partenaires : le patron les ouvriers les salariés , les employés , de tous niveaux ; ce que je raconte là c' est pas simplement à la mine , c' est aussi au bureau . Et c' est aussi dans les professions les plus avancés intellectuellement aujourd'hui qui nécessitent un stress psychologique terrible . Bon , ça se passait dans cet ordre là . Eh bien , maintenant ils ont décidé que tout pouvait se négocier dans l' entreprise . Et l' entreprise peut avoir un accord moins bon que la loi , moins bon que l' accord de branche et les mêmes reviennent vous dire qu' il faut faire de la prévention alors qu' ils ont organisé toutes les conditions pour que tout ça se règle par le bas . Et d' un abominable dumping , je vous demande d' y réfléchir , vous savez ce qu' est le dumping , c' est qu' il y en a un qui vend quelque chose moins cher que l' autre et comment il s' est obtenu ce moins cher ? En payant moins cher le salarié . On comprend tous ça , c' est pour ça qu' on est tous contre le statut de &quot; travailleur détaché &quot; . C' est pas la nationalité du travailleur détaché qui compte , ce qui compte c' est qu' on l' ai détaché et qu' on l' ait amené pour faire concurrence sur le marché de l' emploi , et pour obliger à régler par le bas . Qu' est ce qui se passe ? Eh ben si l' un produit à moins cher parce qu' il a abaissé les droits sociaux , qu' il vous a fait travailler plus longtemps , qu' il ne vous paye pas les heures supplémentaires comme avant , alors celui qui est à côté , qui produit la même chose , dans le cadre de la concurrence , qu' est -ce que vous croyez qu' il va faire ? La même chose . Et il viendra vous dire , un par un à votre poste de travail , &quot; Je ne peux pas faire autrement &quot; et vous , vous allez pas dire &quot; Bah on le connait , c' est le patron , c' est une petite boîte ! &quot; et voilà comment tout va et tout se nivelle par le bas . Voilà pourquoi , quand ils ont détruit la hiérarchie des normes - c' est comme ça que ça s' appelle - la loi , puis l' accord de branche , puis l' accord d' entreprise , ils ont détruit tout un ordre social ; et vous allez voir , parce que depuis le 1er janvier seulement s' applique la loi , mais c' est dans toutes vos entreprises , par un accord d' entreprise qu' on va discuter du montant de vos heures supplémentaires - qui de toute façon vont descendre jusqu' à 10 % . Parce que dans l' entreprise où on aura laissé à 15 % , eh bien , celle d' à côté sera à 10 % et alors ils baisseront à 10 % . Et la loi permet cela . Et encore , tenez -vous pour content . Mordez -vous les doigts mais pensez que le suivant vous fera pire . La Madame Le Pen qui soi-disant représente les ouvriers , le Monsieur Fillon qui vous a déjà collé 1 million de chômeurs sur les bras du temps qu' il était premier ministre , ces deux-là disent qu' il faut en finir avec les 35 heures ; mais ils sont pas bien renseignés : il y a longtemps que les 35 heures elles vont mal . Pourquoi ? Parce que 35 heures ça vaut à partir de 35 heures et 1h si on vous paye en heures supplémentaires ; si on vous le paye plus en heures supplémentaires , la durée légale elle sert à quoi ? C' est pourquoi ils ont été au bout , ils ont dit &quot; Ca sert à rien &quot; , donc il y en aura plus ! Et s' il y a plus de durée légale hebdomadaire du travail , ce que Monsieur Fillon vous a annoncé d' avance et quelques autres et la Madame Le Pen qui dit qu' on aura droit de discuter ça dans l' entreprise , vous avez compris que vous allez encore vous faire blouser . Parce que si ça se discute dans l' entreprise , la discussion va avoir vite fait , elle va être vite réglée . Vous le savez comme moi . Eh bien ça veut dire qu' il y aura plus de durée légale hebdomadaire du travail , sinon la limite qui a été proposée par l' Europe : 48h . Et de 35 à 48 , pour ce qui est de la paie &quot; heures supplémentaires &quot; , rien du tout . Voilà ce qu' ils ont prévu , voilà ce qu' il y a dans les cartons . Voilà pourquoi je demande à chacun d' entre nous d' aller au-devant des autres , de se présenter devant l' ami , le camarade , celui ou celle dont on sait que les élections bla bla , c' est tous les mêmes , bla bla ! Il y a alerte mes ami(e)s ! N' écoutez plus les bonimenteurs , regardez de près ce qui va vous arriver si vous ne faites pas le choix qu' on tourne la page de cette période épouvantable où on a cru qu' en compressant les salaires et en poussant sans arrêt les gens dans le dos , en leur tordant les bras , on ferait décoller , magiquement , l' économie et la production . C' est tout le contraire qui s' est produit , ça fait 20 ans que ça dure ! Est -ce que c' est pas le moment d' arrêter ça ? Est -ce que c' est pas le moment de dire que le problème c' est celui de ceux qui se gavent ! Ce n' est pas le problème de l' immigré ! Ce n' est pas le problème de celui qui n' a pas votre religion ! Le problème c' est le banquier , le problème c' est Mulliez ! Voilà le problème : il a un nom , il a une adresse . On peut changer tant et tant de choses , et en particulier ces salariés et ouvriers , ces employés à tous les niveaux au bureau et à l' atelier . Nous allons avoir un besoin immense de mobilisation de tout le monde , parce que nous allons faire changer de haut en bas toute notre manière de produire pour faire la grande transition écologique dont l' humanité a besoin ! Pas seulement vous , sur vos postes de travail pour cesser d' absorber des produits nocifs , pas seulement les Français parce que nous nous occupons de notre pays , mais l' humanité toute entière a besoin que les pays les plus avancés , ceux où le niveau de culture et d' éducation est le plus élevé , ceux-là montrent l' exemple , marchent les premiers , mettent au point les processus de production , les machines , les process d' une façon générale , qu' on pourra ensuite proposer aux autres par transfert de technologies , pour que les pays où l' on est riche et ceux où on est pauvre , TOUS aient les moyens de manger correctement des produits de bonne qualité , d' avoir de l' énergie gratuite et non dangereuse ! Si j' en appelle à la classe ouvrière aujourd'hui - puisque je suis venu ici pour ça - c' est pour lui dire non seulement qu' elle a tort de se laisser maltraiter de cette façon et qu' elle a l' occasion de se défendre avec ses bulletins de vote . Mais aussi pour lui dire : &quot; Nous avons besoin de vous tous ! &quot; Il va falloir se mettre au travail dans des conditions complètement différentes , et ce changement viendra à la fois d' en haut par la loi , par les ingénieurs , les techniciennes et les techniciens , mais il viendra aussi de tous ceux qui sont sur leur poste de travail et qui devront faire , eux , leurs propositions pour les modifier , ces postes de travail . Pour signaler où sont les dangers , parce que vous savez où ils sont les dangers . Ceux qui respiraient de la poussière de charbon savaient qu' ils allaient mourir de la silicose . Combien vous en avez eu ici ? Combien dans cette région ? Et ensuite celui qui était au bout du tapis roulant dont les autres pensaient qu' il avait la bonne place , parce qu' il était pas sur le front de taille - quoique la position était fort dure à l' époque - celui-là respirait toutes les semaines de sa vie , en plus de la poussière de charbon , a respiré de l' amiante parce que les courroies qui faisaient tourner les tapis étaient en amiante , et qu' elles s' usaient à la cadence d' une par semaine . Ce temps peut paraître lointain , mais faites le tour autour de vous et vous savez bien que certaines des productions auxquelles vous participez ne sont pas tenables dans la durée pour votre santé ou pour les gens qui utilisent les produits qui sortent dans ces conditions . Le meilleur agent d' alerte et de transformation écologique , c' est la classe ouvrière ! Il n' y a pas de contradiction entre les deux , quoi que croient certains . Bon , maintenant je rentre . Et je vous retrouve juste après . *musique* Nous sommes voués à un idéal et ils savent bien qu' à cet instant c' est mon tour . J' incarne - pour vous - , je porte AVEC vous . Clairement , vous ne vous attendiez pas à venir à une réunion pour entendre chanter des &quot; Alléluia &quot; à la flexibilité , à la compétitivité et tous les autres mantras des néo-libéraux . On parle de la condition des salariés . J' ai évoqué les plus démunis , les plus désarmés , je vais reprendre un cas plus général . Si on pouvait à peine baisser le son , ce serait parfait pour moi parce que c' est très assourdissant . J' ai traité de questions générales , mais commençons par le commencement pour nous tous . Quand on examine ce qui fait la vie au travail , la vie du travail des salariés ... La première chose c' est d' avoir du travail . Et du travail , il y en a beaucoup qui est nécessaire . Mais tout le travail nécessaire n' est pas aujourd'hui rentable . C' est dommage , il faut donc changer la manière de regarder . Au lieu de partir d' abstractions - produire n' importe quoi , n' importe comment , dans n' importe quelles conditions , du moment que c' est pas cher , et qu' on finira par trouver quelqu'un pour l' acheter - prenons les problèmes par l' autre bout . De quoi a -t-on besoin ? Que faut -il faire ? Quelles sont les tâches nécessaires à ce moment ? Non seulement de l' histoire de notre pays , de l' histoire du monde ? C' est par là que j' ai terminé dehors . Le moment c' est celui de la transition écologique du modèle économique . Ce n' est pas un slogan , c' est une tâche collective , que le pays tout entier doit porter , exactement comme il l' a fait , après avoir été envahi et détruit il a tout reconstruit , et pour tout reconstruire , nos anciens avaient eu le pressentiment qu' on ne pourrait pas reconstruire , qu' on n' aurait pas cet enthousiasme collectif si on ne crée pas les conditions qui permettent à chaque personne de maîtriser sa propre vie et d' être protégé , ou en tout cas accompagné dans les grands défis de la condition humaine - le vieillissement , la maladie - et les bonnes nouvelles : les enfants ... Et c' est par cette solidarité accrue qu' on a redéployé la machine à produire et qu' en quelques années tout à été rattrapé , reconstruit et la société complètement changée . J' avance dans cette campagne avec cet état d' esprit : d'abord , tout ce que nous aurons à faire , je viens de le résumer ; mais je ne vais pas passer à côté d' une question extrêmement importante : c' est celle de la durée du travail . C' est un fait : depuis 1870 , la quantité de travail nécessaire pour produire la richesse que nous produisons a été divisée par 2 . Et la richesse qui a été produite par cette quantité de travail totale divisée par 2 a été multipliée par 30 . Et c' est une très bonne nouvelle ! Les machines , les perfectionnements des processus de production permettent qu' avec une quantité de travail humain moins grande , donc moins de peine , moins de souffrance on produise bien plus que ce dont on a besoin , car notre société est avant toutes choses , une société du gaspillage . La diminution du temps de travail , soit on la laisse se dérouler d' une manière sauvage - c' est à dire &quot; on verra bien &quot; - et elle a besoin de moins , donc on prend moins de monde et les autres on s' en fout . &quot; C' est pas mon sujet &quot; dit l' un , &quot; C' est pas mon sujet &quot; dit l' autre , et vous avez d' un côté des gens qui travaillent beaucoup , qui s' usent au travail et de l' autre d' autres qui n' ont pas de travail et qui souffrent , qui souffrent ... Je veux rester dans mon thème d' aujourd'hui : qui souffrent ! Selon l' INSERM - qui n' est pas une organisation &quot; bolchévique &quot; , mais un institut de recherche sur la santé et les médicaments - 14.000 personnes par an meurent du fait du chômage . Vous m' entendez ? 14.000 ! Par la dépression , par le stress , par l' abandon de soi quand on ne sait plus quoi faire . Le chômage de masse est une souffrance de masse . Je le dis pour rappeler à tous les petits intelligents qui passent leur temps à expliquer qu' il faut stimuler l' envie de retour au travail qu' ils font insulte à des millions de personnes en parlant de cette manière-là . Et que quand un Fillon et quelques autres s' imaginent qu ' en mettant des taux dégressifs d' indemnisation , on va - en vous prenant à la gorge - vous pousser à trouver du travail comme si cela dépendait de vous . Comme si un seul jour vous vous étiez dit &quot; Je n' y vais pas parce que ça me fait plaisir &quot; , ce n' est pas vrai . Chaque fois que quelqu'un vous dit qu' il va faire des taux dégressifs , avec sa belle bouche , son air profond , qui réfléchit aux problèmes auxquels vous n' êtes censés ne rien comprendre , rappelez -vous que celui-là est en train de vous traiter de fainéant , rappelez -vous que celui-là est en train d' insulter celui qui dans votre famille se désespère de ne pas avoir de travail , et allez pas voter pour des marioles pareils ! Ou alors , vous vous faites injure à vous-même . Ensuite , souvenez -vous que la dégressivité , ils ont déjà essayé de la faire une fois , et ça a été fort mal accueilli . Parce que nous autres on sait aussi autre chose , c' est que la moitié des gens qui sont au chômage n' ont aucune indemnité . Aucune ! Alors leur sujet cela n' a pas été de se dire &quot; Comment on pourrait faire pour que les gens , du moins , ne soient pas dans une situation si épouvantable ? &quot; Non , ça a été de se dire : &quot; A ceux qui ont quelque chose , on va voir comment on va pouvoir leur enlever . &quot; Voilà ce qu' est leur manière de voir . Et quand on parle de diminution de temps de travail , pour qu' il y ait une répartition sur l' ensemble des travailleurs , l' ensemble du corps productif , ils disent : &quot; Oh là là , ça va coûter cher , c' est ruineux &quot; ... Puisque j' ai la parole autant que vous sachiez . On a créé 350.000 emplois avec les 35h . Chacun de ces emplois a créé en aide 13.000 euros . C' est un rapport parlementaire , il y a donc la majorité et l' opposition . Le crédit d' impôt compétitivité , on pense que ça a créé ou que ça a aidé à sauvegarder de 50.000 à 100.000 emplois . Nous , 355.000 , eux , 50.000 à 100.000 emplois . Et combien ça nous a coûté ça ? 28 milliards , c' est à dire 280.000 euros d' aide publique par &quot; emploi &quot; créé par le CICE , 13.000 par les 35h . Autrement dit , l' argent qu' on a versé dans ce puits sans fond que sont la marre aux réclamations et aux pleurnicheries du CAC40 , la plus grande organisation d' assistés par ce pays a été ruineux pour rien ! Si Monsieur Gattaz avait tenu parole , s' il avait créé son million d' emplois , et bien même dans ces conditions-là - quoi qu' aucun d' entre nous n' ait cru un seul jour aux clowneries de cet homme , avec son badge 1 million d' emplois - eh bien même dans ces conditions là , compte tenu de ce que ça coûte , chacun de ces emplois en question aurait coûté 3 fois ce que coûte un emploi aidé pour qu' on passe à 35h . Vous n' avez donc aucun complexe à avoir . Mon discours d' aujourd'hui , c' est un discours pour la fierté , pour le fait qu' on relève la tête , qu' on arrête de se soumettre à leur récitation qui vise à nous culpabiliser , à nous humilier , pour nous conduire à faire toujours davantage comme ils veulent . Et davantage moins , toujours moins par rapport à ce que nous voulons faire de nos propres vies . La diminution du temps de travail est une nécessité . Je conviens qu' il faut en discuter , il y a la diminution du temps de travail , dans la vie ( c' est l' âge de la retraite ) , dans l' année ( puisque certains ont un contrat de travail avec une annualisation de leur temps de travail ) , il y en a ici ? Non ? Eh bah je vais vous la raconter , non ? Je vais pas vous la raconter , vous le savez . Mais l' annualisation du temps de travail a permis de généraliser le forfait qui autrefois s' appelait le &quot; forfait jour &quot; , il y avait que les cadres qui avaient ça , parce que comme ils avaient de l' autonomie on a dit : &quot; Bon , calculons gros , déjà tu es bien assez payé ? Tu n' es pas content ? Tu peux t' en aller &quot; &quot; Ah non , je suis content ! &quot; Et on a étendu ça progressivement à tout le monde . Je le sais , parce que quand on a voté la loi sur les 35h , j' étais parlementaire , donc je les ai vus - et j' étais au Sénat hein , c' est le cul-de-basse-fosse de la réaction , donc je peux vous dire que dès qu' il s' agit de lâcher une minute alors là c' était pas possible . Et donc , moi , j' ai déposé un amendement contre le forfait-jour , c' était pour les cadres . J' ai dit : &quot; Je sais comment ça va terminer : non seulement vous allez sur-exploiter les cadres , mais à la fin vous sur-exploiterez tout le monde . &quot; Et c' est bien ce qui s' est produit . Donc la durée du travail dans la semaine , la durée de travail dans l' année , la durée de travail dans la journée , tout ça , ça était une longue bataille mes ami(e)s , ça a pas été une discussion tranquille entre gens bien élevés . Ça a été une bataille , avec un certain moment des morts parce que la troupe tirait sur la foule qui venait réclamer la limitation de la durée du travail . Vous voyez le triangle rouge que je porte là ? C' est celui des déportés socialistes , communistes , syndicalistes par les Nazis ; mais pas que ça , c' est le triangle rouge que portaient les ouvrières et les ouvriers du premier 1er mai en France . Ils avaient mis un petit triangle de cuir rouge , 8h de loisirs , 8h de sommeil , et 8h de travail . La question du temps de travail , elle est essentielle , pourquoi ? Parce qu' elle a avoir avec la façon dont on répartit la richesse . Vous voulez voir le lien ? Je vous l' explique . Le système capitaliste fonctionne sur la gratuité . La gratuité de votre travail . Si vous faites 10h , on va faire rond , ils vous donnent pas la valeur que vous avez produit en 10h . Ils vous donnent 2h ou 3h et tout le reste c' est pour quelqu'un d' autre . Pour payer le matériel , les locaux , les matières premières et les dividendes et l' investissement . Suivant comment vous répartissez cette richesse , ça peut être du temps de libre pour les uns , de l' amélioration des salaires , etc . etc . Nous sommes donc collectivement riches - en oubliant quels sont les rapports de propriété - de 2.000 milliards d' euros par an que produit ce pays . La question qui est posée , c' est exclusivement la question de la répartition et du partage . Comment on partage ça ? Alors la règle actuelle , c' est tout pour les uns , rien pour les autres . Ou le minimum de ce qu' on est obligé de leur lâcher . En 10 ans ... Non , en 30 ans , dans l' ancienne répartition entre le travail et la capital , 10 points sont passés des poches du travail à celles du capital . Du coup , deux choses : nous sommes un des pays d' Europe où il y a le plus de millionnaires . On sait pas pourquoi , il y aussi de plus en plus de pauvres . Plus il y a de millionnaires , plus il y a de pauvres . Il doit y avoir un rapport peut-être ? Un jour quelqu'un m' a dit : &quot; Oh vous faites du misérabilisme , moi ce qui me pose un problème c' est pas qu' il y ait des riches , c' est qu' il y ait des pauvres &quot; . A cette altitude de pensée , je ne peux plus suivre . Mais si vous arrivez pas à comprendre que certains ont beaucoup parce que les autres ont rien , il faut retourner à l' école . Et savoir comment on fait les additions , les multiplications et les divisions . Donc , dans la répartition , une quantité croissante va aux uns , alors on vous dit : &quot; Et alors ? Ils sont plus intelligents , ils ont des idées , ils ont beaucoup d' argent et avec ça ils savent où mettre l' argent , tandis que vous ne savez rien &quot; . &quot; Eux ils savent comment il faut faire et tout &quot; , mais c' est pas vrai . La preuve , c' est que d' année en année , sur cette part qu' ils ne donnent pas au travail , sur cette part qui ne va pas pour les bâtiments , sur cette part qui ne va pas pour la matière première , sur tout ce morceau-là , une fois qu' il est là et qu' il faut le partager entre le futur et le présent ... Le présent : leur jouissance immédiate ; le futur : les machines , les process de production ; eh bien d' année en année , à mesure qu' il y a de plus en plus de gens très riches , la proportion des dividendes s' accroît et ils se payent toujours mieux . Nous sommes le record d' Europe des dividendes , bande de paresseux ! Alors , tout le temps en train de râler ! Eh bien vous voyez , il y a des gens qui réussissent . Et c' est pas tout . Non seulement ils prennent de plus en plus , mais tout ce qu' ils prennent , ils le mettent pas dans l' investissement . Si bien que maintenant , il y a bien plus d' argent qui part en dividendes ( c' est ce qu' on donne a quelqu'un qui a des actions dans une société ) qu' il y a de l' argent qui part dans l' investissement . Autrement dit , petit à petit les mêmes que vous entendez pleurnicher continuellement sur le fait que leur taux de marge n' est pas suffisant et tout un tas de baratin qui sont autant de menteries , les mêmes sont ceux qui n' investissent pas , ne prévoient rien pour l' avenir . Et les grands entreprises du CAC40 -parce que je lis leur paperasse quand même pour savoir - que disent -elles ? &quot; Ah bah nous , nous attendons une indication de tendance du marché &quot; . Mais le marché c' est vous ! Qu' est -ce que vous attendez ? Que ça soit l' épicier du coin ? Le plâtrier du village ? Le maçon du bourg qui crée la tendance ? Non c' est vous qui avez les grandes entreprises et qui pouvez vous lancer dans des grands plans . Eh bien puisque vous ne le faites pas , nous allons le faire nous . C' est ça le sens de la planification écologique . D'abord parce qu' il faut organiser , d'abord prendre conscience du danger . Prendre conscience du danger , vous autres , si vous faites campagne avec moi , passez du temps a faire comprendre que cette fois -ci c' est pas un choix parmi d' autres . Nous devons relever un défi pour l' humanité toute entière ; une fois qu' on a compris ça , eh bien il faut organiser , articuler les uns avec les autres la production , la formation et l' argent qu' on investit . Et si nous le faisons , alors nous allons remettre en marche toute la mécanique , parce qu' eux ne le font pas . Eux prennent pour les dividendes . Eux prennent pour placer dans la sphère financière , parce que dans la sphère financière vous pouvez spéculer et gagner du 10 , du 12 , du 15 % , aucune production jusqu' à présent ne le permettait , sauf à tuer la machine et à tuer les ouvriers . Cet argent qui s' en va dans la sphère financière , naturellement nous allons couper les cordons par lesquels ils passent , parce que c' est toujours de l' argent qui s' accumule d' une manière particulière , c'est-à-dire de l' argent qui pour une part échappe à l' impôt , qui pour une autre part est spéculatif . Et enfin c' est beaucoup d' argent qui circule dans le monde , mes ami(e)s . J' ai pas le temps , c' est pas le sujet aujourd'hui . Beaucoup , beaucoup , beaucoup d' argent . Et qui circule sans aucun rapport avec la production réelle . Sans aucun rapport avec la production réelle ! A l' heure à laquelle je vous parle , le système bancaire lui-même s' en alarme . Comment expliquez -vous que le banquier central européen - qui a décidé de mettre 2.000 milliards , qui en verse 80 tous les mois ... Tous les mois le banquier central donne 80 à 85 milliards - le banquier central c' est celui de la BCE - aux banques pour qu' elles consolident leur bilan et qu' elles placent dans l' activité . Où vous avez vu la queue d' un euro de ces 85 milliards ? Vous savez ce que ça représente ? Chaque mois ? 6 millions de SMIC ! Ça devrait se sentir quand même quelque part ... Même si c' est amorti en cours de route par des gens qui prennent . Rien ! Ça signifie que l' essentiel de cette masse énorme d' argent est directement repartie dans la sphère financière . Vous comprenez pourquoi , quand dit que nous allons lutter et lutter d' une manière implacable contre la financiarisation de l' économie , ce n' est pas un slogan idéologique , seulement ... Bien sûr , c' est un slogan idéologique ; quand je vous dis &quot; être milliardaire c' est immoral &quot; , je reconnais : c' est un choix moral , un choix philosophique , c' est un choix éthique , ce que vous voulez . Mais c' est surtout un choix concret , pratique . On ne peut pas continuer comme ça , et surtout pas nous les français qui sommes un peuple jeune , avec de plus en plus de jeunes . De ce chapitre de mon discours , je veux que vous reteniez une chose : 1 ) il y a besoin de travail , 2 ) il y a l' argent , 3 ) on sait le faire , 4 ) c' est urgent qu' on le fasse . Voilà pourquoi , moi , quand je parle du discours sur le travail pour le condamner , je parle des salariés . Je reviens à une idée toute simple : c' est le progrès social qui entraîne le progrès économique . C' est le progrès écologique qui entraîne le progrès économique . Ce n' est pas le contraire ! Alors ... J' ai parlé du temps de travail , voici ma position et celle du programme &quot; L' avenir en commun &quot; . Très bien , merci ( rire ) . Premièrement , retour aux 35 heures , abolition de la loi El Khomri . La loi El Khomri est inacceptable , inefficace et brutale . C' est bien que vous applaudissiez : allez demander des comptes aux autres , parce que pour ce qui est de bavarder tout le monde est là ... Mais pour faire , c' est une autre paire de manches . La loi El Khomri , c' est comme la 6ème République , la planification écologique : c' est pas à moitié , c' est ou pour ou contre . Voilà , nous on est contre , on l' abrogera . Et les autres doivent dire un par un ce qu' ils comptent faire . Et sûr : après avoir abrogé la loi El Khomri , il y a un retour aux 35h . Je commence par les 35h , les vraies 35h . C' est pas tout d' écrire dans un papier &quot; durée légale , 35h &quot; . Ah oui ? Et la 36ème heure ça vaut combien ? Eh bah ça vaut rien . C' est ce qu' a fait la loi El Khomri . Si vous payer la 36ème heure 10 % de plus , ça ne coûte rien , ça crée pas 1 emploi , bon ça vous rapporte 10 % de plus , mais jusqu' à présent c' était 25 % , non ? Eh bah maintenant c' est 10 % . C' est bon d' avoir un gouvernement de gôche ! D'accord ? Donc il faut renchérir le paiement des heures supplémentaires pour que ça crée de l' emploi et que aussi les gens puissent vivre avec un salaire digne , qui leur permet avec 35h de vivre dignement . Car la vie c' est pas que le travail salarié il y a aussi tout le reste de l' existence qui souvent comporte beaucoup de travail aussi , à la maison , avec les enfants et le reste tout ce qui vous passe par la tête parce que vous n' êtes pas les rouages d' une machine . Vous êtes des êtres humains ! Vous êtes des êtres humains ! Dites bien aux gens , dites -leur bien , vous v'là prévenus . Venez pas après pleurer . Si vous recommencez avec des Macron , des Fillon , des &quot; je sais pas quoi &quot; , de tous ces gens qui ont la même doctrine . Simplement qui vous emballent la marchandise d' une manière différente exactement pour faire avec vous comme fait le marchand d' Auchan quand il vous met des yaourts différents dans les rayons , que c' est pourtant toujours la même chose . C' est ça que font ces gens . Si vous avez Fillon , il n' y a plus de durée légale du travail , vous avez bien compris ? Expliquez -le aux gens . Plus de durée légale du travail , ça veut dire qu' il y a plus d' heures supplémentaires et que quand vous serez annualisé et au forfait-jour vous travaillerez aussi longtemps que celui qui a le carnet de commandes en main aura décidé de vous faire travailler , et vous n' aurez aucun de moyen de vous défendre - sinon d' invoquer le texte européen qui dit qu' on ne peut pas aller au delà de 48h , ce qui nous les français , nous ramènerait au siècle précédent et au début du siècle précédent . Le temps de travail est une question extrêmement sérieuse , ce n' est pas seulement sa durée et son intensité qui est en cause , c' est la façon avec laquelle il roule . Vous savez comme moi que quand vous avez un horaire discontinu , que vous avez d'abord fait 3 ou 4 heures et qu' ensuite votre poste de travail reprend , après une pause de 3 ou 4 heures , la pause en question ne vous est pas payée , et vous tournez en rond . Par conséquent , quand on arrive à faire une journée , il faut que tout le monde puisse la faire dans des conditions où il maîtrise son emploi du temps . La loi El Khomri prévoit maintenant qu' on peut vous prévenir 3 jours avant que votre emploi du temps a changé . Qui peut changer l' organisation de son existence 3 jours avant ? Comment on fait 3 jours avant pour trouver la nounou si il faut ? Changer l' heure de qui va aller chercher le môme à l' école ou à la crèche ? Quelle vie ils ont , quelle vision ils ont de la vie des gens ? Qu' est -ce qui compte d'abord ? Nos gamins ou leurs foutues marchandises ? Bien sûr que ce sont nos gosses et nos vies . Il faut il faut en revenir à la semaine de 7 jours avec 2 jours de congés consécutifs . Et pour tout le monde au même moment . Une fois j' avais une camarade , qui est super sympa , une socialiste , et elle me dit &quot; Rah , voilà comment vous êtes vous , vous voulez tout planifier , organiser , contrôler ! Après tout si les gens veulent faire autrement ? &quot; Mais les gens ils peuvent vouloir ce qu' ils veulent , c' est pas ce qu' on veut nous . Ce qui compte c' est quel genre de vie on veut organiser . Si vous commencez à multiplier le travail le dimanche , bien sûr qu' au début vous avez une personne ou l' autre que ça arrange dans sa vie , je le sais bien . Mais il y a surtout une masse qui n' a pas les moyens de défendre , à qui on va dire &quot; Bah , tu viens dimanche . &quot; &quot; Ah bah c' est pas ce que j' avais prévu ! &quot; &quot; Eh bah si ! Sinon tu peux t' en aller &quot; Et celui-là obligé d' y aller , et celle-là est obligée d' y aller , et plus vous êtes pauvres , plus votre situation est difficile , plus vos horaires de travail sont discontinus , plus vous avez besoin d' heures complémentaires . Les heures complémentaires c' est le rêve de celle ou celui qui n' a pas un temps de travail complet , il passe son temps à espérer qu' il va pouvoir en faire un peu plus . Et en général c' était un peu mieux payé , mais grâce à la loi El Khomri c' est fini . Et les autres se proposent de faire pire avec leur suppression de la durée légale . Vous voyez ? Voilà comment tout ça s' organise . C' est cette brutalité là qu' ils répandent , les horaires de travail et la semaine , elle doit être la même pour tous le monde . Sinon vous transformez votre maison en un hôtel-restaurant . Les uns vont au boulot , les autres sont là , et puis ceux qui sont là s' en vont à l' heure à laquelle les autres arrivent . Et vous savez bien comment ça se passe , quelqu'un devient le domestique de tout le monde dans cette histoire . C' est pas la peine que je dise qui , on connait . C'est-à-dire que tout ça , ça ne fait qu' une chose , d' un bout à l' autre : aggraver la situation des femmes au travail , et tout ça est fait avec le propos et l' intention délibérés de faire que les femmes retournent à la maison , pour que leur poste de travail soit occupé par des hommes et qu' on obtienne de cette manière la baisse du chômage - parce qu' il savent que ça va finir par sauter . Tout ça est fait d' une manière très méthodique , et va toujours dans le même sens . En ayant raccourci les délais pour prévenir d' un changement d' horaire , c' est d'abord les femmes qu' on élimine du travail . Avec l' exemple que j' ai donné tout à l' heure , avec l' imposition du travail du dimanche , c' est d'abord les femmes qu' on élimine du travail . Parce qu' elles savent ce qu' il faudra sacrifier à ce moment-là . Voilà comment , au bout du bout , comme toujours , la violence d' une situation se décharge sur le plus faible , sur celui qui a le moins de droits . C' est pourquoi je m' oppose - et le programme s' oppose - à la continuation du travail du dimanche . Je vais vous parler maintenant des droits des salariés . J' ai parlé du temps de travail , j' ai parlé de la répartition de la richesse , sur les grands objectifs qu' on avait : la transition écologique , mais évidemment ça veut dire aussi la répartition dans les salaires . Clairement , on peut pas continuer comme ça . Vous avez un SMIC qui est à 100 euros de plus que le seuil de pauvreté ! Non mais ils sont toujours en train de venir pleurer sur le fait que certain qui ne travaillent pas , obtiendraient autant que ceux qui travaillent . Ça leur coûte rien de raconter des salades , ce n' est jamais vrai . Mais ça permet qu' on se batte entre nous , ça fait qu' on regarde par-dessus son épaule dans l' assiette de l' autre , au lieu de regarder ce qu' il y a dans celle de celui qui vous donne ce genre de conseils . Parce que comment ça se fait qu' un pays aussi riche , où il y a autant de millionnaires , et où la richesse a autant cru , que les gens qui sont payés au minimum sont payés 100 euros de plus que le seuil de pauvreté ? C' est quand même ça le problème , non ? C' est pas l' autre le problème , c' est ça le vrai problème . Le seuil de pauvreté , ça se calcule de la manière suivante : on prend 60 % du salaire médian , c' est une norme internationale , c' est pas nous qui avons inventé ça , donc tout le monde fait pareil . L' autre jour j' écoutais là , des fois faut écouter , on est pas obligés , j' ai écouté . On me dit &quot; Alors qu' est -ce que vous en pensez , le salaire minimum européen ? &quot; De quoi vous me parlez ? &quot; C' est Monsieur Valls qui a dit ça &quot; Je le connais , c' est un réac , il n' a pas pu dire ça . Donc , je vais chercher le papier , mais c' est comme pour tout , il faut jamais croire ce qu' on vous dit qu' il faut croire . Regardez d'abord , vérifiez . &quot; Ah non , ah non , attendez c' est pas un Smic européen c' était un Smic dans chaque pays ? &quot; Ah , bah attendez , les élections présidentielles c' est en France , pas en Bulgarie . Donc on parle de ce qu' on fait ici , si vous allez proposer un SMIC dans chaque pays européen pourquoi pas après tout , on pourrait faire un critère de convergence . Mais c' est pas ça qu' il dit . &quot; On va proposer un SMIC dans chaque pays européen qui soit à 60 % du revenu médian . &quot; Hop , on fait les calculs et bah vous vous êtes fait avoir , une nouvelle fois , dans le papier Europe , qu' est -ce qu' il y a ? Une baisse du SMIC . Parce que 60 % du revenu médian pour nous ça fait moins que ce que ça fait aujourd'hui , voilà . Moi je ferai le contraire , j' ai pas peur , nous allons , nous , augmenter le SMIC . Parce que notre objectif c' est d' être à 1.700 euros brut pour commencer , net pour terminer . Et on commence tout de suite avec 150 euros de plus . Des ami(e)s m' ont dit &quot; Attention , si tu fais ça on va avoir un problème dans la petite entreprise &quot; , je le sais . C' est la raison pour laquelle de tout ça on discutera bien tranquillement - sachant quelle est ma décision , qui ne changera pas - de la façon de la mettre en oeuvre . Plusieurs ami(e)s économistes ont eu une idée que j' ai trouvée brillante et que pour finir ils se sont accordés entre eux , pour la mettre dans le programme . C' est de faire une caisse commune des cotisations sociales qui permette de faire de cette manière là , de prendre sur l' ensemble du patronat la somme nécessaire pour financer l' ensemble des cotisations . De cette manière , on pourrait le faire reposer différemment sur les uns et sur les autres . Et on serait pas obligés d' écraser , en effet , par une cotisation qui serait trop dure à porter par la petite entreprise , ce que la grande entreprise porte sans difficulté d'autant que les grandes entreprises ont eu cette habileté - soi-disant pour s' occuper de leur cœur de métier - de tout découper en petits morceaux et de faire des petites entreprises qui sont souvent des entreprises qui n' ont qu' un seul client : c' est le cas de 80 % d' entre elles , si bien qu' on pourrait les considérer comme des succursales . Si bien qu' au bout du bout , non seulement la responsabilité du donneur d' ordres ultime ça doit être celui qui donne les ordres , mais c' est aussi lui qui doit payer . Je sais bien qu' il y a pleins d' ami(e)s qui disent : &quot; Ah mais il faut pas y aller trop fort ! &quot; Si vous ne voulez pas y aller trop fort , allez en trouver un autre . Parce que il faut bien regarder cette situation , je vous ai déjà dit 3-4 fois : la richesse qui s' accumule , etc . Je vous donne juste un chiffre , et après je verrai , vers la fin , ce que j' ai à dire . Quelque chose qui va bien , bien vous aider à réfléchir et à vous décomplexer . Au lieu d' avoir tout le temps peur que tout va se casser la figure , qu' on n' a rien le droit de demander , que c' est déjà bien comme ça , etc . Le 3 janvier dernier , ça vous dit quelque chose ? &quot; Non . C' était 2 jours après le 1er janvier . &quot; Peut-être il y en avait qui avait encore mal au crâne , j' en sais rien . Le 3 janvier dernier , à 18h , non plus ? Eh ben vous avez raté quelque chose . Parce que ce jour là , le 3 janvier dernier à 18h précises , les 120 plus grands patrons français ont déjà gagné plus que tous les français gagneront en 2017 . ( sifflets ) Ils leur suffit de 2 jours et 18h pour gagner autant que tous les autres , voilà . En 2016 , ils ont gagné 132 fois plus que le salaire moyen . C' est normal , parce que vous , vous êtes au salaire moyen , vous avez des besoins . Et eux ils ont des besoins 132 fois plus importants que vous . C' est facile à comprendre . Bon , je crois que je vous ai un peu préparé pour entendre le dernier chiffre sans que vous vous effondriez tout de suite en larmes . Le salarié moyen il a a gagné l' année dernière 72 euros par jour . Et le Monsieur ? Et la Madame ? 9 589 en moyenne . Voilà . Vous comprenez ? Alors si vous avez des complexes après pour dire &quot; on va changer la clé de répartition &quot; c' est que , vraiment , c' est mal parti . On peut pas faire fonctionner durablement une société dans un tel déséquilibre . On peut le faire qu' à une condition : c' est que vous soyez morts de peur . Et c' est pour ça qu' on fait cette réunion , qu' on fait ce rassemblement pour dire : &quot; On va faire reculer la peur ! &quot; Mais on va la faire reculer en regardant les possibilités qu' on a de changer notre vie complètement . De revenir à l' idée qu' on peut avoir une bonne vie , que ça vaut la peine de se lever le matin pour aller au travail parce que ça sera pas seulement pour y souffrir mais pour y faire quelque chose . D' important . Et ensuite gagner sa vie et vivre sa vie . En pensant à autre chose que de savoir comment on va boucler le budget pour la fin du mois , en pensant à autre chose que de savoir si on va payer ou le loyer ou l' électricité ou le chauffage . Cette abjecte comptabilité qui pourrit la vie des gens inutilement sans raison , sans raison parce que la richesse accumulée a été immense et qu' elle est là , et qu' on a tant de belles choses à faire . Alors ... Pour ça , il faut aussi , que les droits des salariés changent , qu' on cesse d' avoir une masse confuse avec peu de droits et aucune capacité d' intervention . Le grand Jaurès disait : &quot; La révolution de 1789 a rendu les français rois dans la cité , comme citoyens , mais elle les a laissés serfs dans l' entreprise &quot; . Ces droits ce n' est pas comme le croient les importants une addition de récriminations , ce n' est pas vrai , ils ne comprennent rien à la mentalité d' un salarié . Je l' ai dit tout à l' heure , la mentalité du salarié c' est qu' il aime son travail , le plus souvent . Il connait les moyens de l' améliorer et il le fait des fois à ses dépens . Les salariés doivent avoir des droits plus grands , notamment dans une situation : c' est celle où celui qui possède une entreprise décide de la vendre , ou bien que les tribunaux de commerce - que nous réformerons profondément , pour qu' on puisse parler à ce sujet de tribunaux , avec l' idée d' une justice , et pas seulement d' arrangements de plus en plus obscurs et inexplicables . Comme celui qui a conduit par exemple à préférer qu' on vende toutes les machines de l' entreprise Ecopla , dont vous avez peut-être entendu parler , qui est la dernière boite de ce pays qui fabrique des emballages aluminium . Il n' y en a qu' une ! Elle a été évidemment prise dans le cycle du démantèlement de la grande industrie de l' aluminium qui existait en France , et qui à l' époque s' appelait Pechiney . On découpait tout cela en petits bouts , et de petit bout en petit bout , racheté par l' un , racheté par l' autre , celui-là prenait les machiness l' autre prenait ci , enfin bref . Tout ça a été démantelé , et à la fin il en reste dans des coins des petites entreprises , qui autrefois participaient de cette grande entreprise ; et il y a celle-là : Ecopla . Les clients de cette entreprise disent : &quot; Ah bah nous , nous sommes d'accord pour continuer à vous acheter à vous &quot; - le carnet de commandes est plein . Entre-temps , ils ont été achetés par un fond de pension qui lui-même l' a revendu à un autre fond de pension , et maintenant voilà que le dernier propriétaire du fond de pension - qui a d'ailleurs disparu , il n' est même pas venu - a décidé de fermer la boutique . Les fonds de pensions ils font comme ça , ce sont des parasites . Ils viennent , ils achètent et puis au bout d' un an , une fois qu' ils ont bien regardé , ils achètent en empruntant de l' argent pour acheter , ils vident la trésorerie pour rembourser l' emprunt , ils foutent tout le monde dehors , et une fois qu' ils ont foutu tout le monde dehors ils vendent le terrain et le bâtiment , et ils en tirent au total ... C' est une bonne opération pour eux puisqu'ils ont siphonné la trésorerie , pris pendant un temps de la richesse , des fois ils vendent les brevets que la boite avait accumulés , etc . Donc une entreprise - mot avec lequel tous ces gens font tant de belles phrases - une entreprise c' est d'abord un collectif humain , un savoir-faire et une utilité sociale . Là , dans le cas de Ecopla , il y a tout , et les gens qui sont dans cette entreprise disent : &quot; Nous sommes d'accord , nous voulons nous-mêmes continuer cette entreprise , parce que nous , on sait comment la faire vivre , on sait travailler , on sait faire ça , les clients sont d'accord etc . &quot; &quot; Et on vous demande rien : un différé pour la dette sociale . &quot; Mais dans la dette sociale de l' entreprise , il y a les frais de licenciement des gens , donc si on les licencie pas , ça change déjà un peu la donne . Vous êtes d'accord ? Dans ce cas-là que s' est -il passé ? Le tribunal de commerce a dit : &quot; Pour payer la dette sociale , on vend les machines et on ferme la boîte &quot; . Donc on a payé , ils ont décidé de donner l' entreprise à un Italien qui , lui a annoncé d' avance qu' il resterait pas là et qu' il s' en allait , il prenait le matériel , le carnet de commande , il prenait les machines . Combien valent les machines ? A peu près la dette sociale . 150 millions . Je vous raconte cette histoire , je pourrais vous en raconter 10 comme ça , parce que les gens comprennent plus , c' est des braves gens , c' est pas des enragés , ils comprennent pas , ils disent : &quot; Mais ça c' est pas logique , c' est pas cohérent ! &quot; Et surtout ils sont prêts à faire tourner la boite , ils sont quand même 40 là-dedans . C' est pas rien , 40 familles , vous savez comme moi qu' il n' y a pas autant d' emploi autour de nous . Donc , pour moi , il y a un droit fondamental . Quand une entreprise est mise en vente , il y a un droit de préemption pour les travailleurs si ils veulent en prendre la propriété sous la forme d' une coopérative . Deuxième chose : les travailleurs d' une entreprise ont des idées sur leur entreprise . C' est la raison pour laquelle il faut leur donner 2 possibilités de contribuer à l' intelligence collective , en plus de celle que je viens vous donner . Premièrement , ils doivent avoir un droit de remontrance , c'est-à-dire de voter une motion de défiance à l' égard de la direction d' une entreprise dont le salariés se diraient : &quot; Ce qu' ils sont en train de faire là , ça va mal tourner , ce qu' ils sont en train de faire là c' est suicidaire , donc nous lançons collectivement une alerte &quot; et cette alerte pourrait être un document de départ obligeant tout le monde à discuter et à re-discuter des décisions qui ont été prises , troisièmement . Quatrièmement il est important que dans les entreprises où il y en a , les comités d' entreprise aient un droit de véto sur un plan de licenciement économique , jusqu' à ce que toutes les autres possibilités aient été examinées et vérifiées dans leurs viabilité . Voilà des droits fondamentaux qui introduiraient de la citoyenneté dans l' entreprise , ; voyez -vous , quelle est l' idée ? L' idée c' est que dans un collectif humain , on est plus intelligents tous ensemble que chacun séparément . Et qu' une entreprise c' est d'abord un collectif humain , et c' est d'abord à lui qu' on doit pouvoir faire confiance . Naturellement , c' est le contraire de toutes les brutalités que ces gens-là ont imaginées . J' ai donné quelques pistes . Le 11 prochain me voilà parti à Nantes pour parler de sécurité sociale intégrale . On a beaucoup parlé du travail , depuis tout à l' heure - enfin c' est moi qui parle . Les salaires , la durée , la façon de travailler dans une entreprise . Juste un mot pour terminer . Comme vous le savez , j' ai été ministre de l' enseignement professionnel . Je ne dis pas que je sais tout , loin de là . Mais je crois que je sais un certain nombre de choses sur le travail qualifié . Je n' entends jamais parler de ça . On ne parle jamais de qualification , comme si le travail était un don . D'ailleurs dans certaines catégories sociales , comme on se transmet de père en fils ou de père en fille le même attachement à un métier , on finit par croire que c' est un don familial , quasi-génétique . Celui -ci est médecin , sa fille ou son fils est médecin , ce sont de bons métiers , ou bien architecte ou bien je-ne-sais-quoi . On oublie que tout est métier , et que le niveau des métiers va s' élevant en exigence de qualification . Quand vous entendez quelqu'un dire : &quot; Il faut l' intelligence de la main &quot; , dites -vous que celui qui parle est un con . La main n' a pas d' intelligence ! L' intelligence c' est là , pas là . Et là , il y a tout le reste avec . Les niveaux de qualification pour une production moderne sont de plus en plus élevés . Et vous ne faites pas un grand pays développé uniquement avec des cadres supérieurs - il en faut , vous ne faites pas un pays développé uniquement avec des ingénieurs , il en faut ! Croyez -moi , il en faut . Vous le faites avec toute sorte de cadres intermédiaires et d' ouvriers et d' ouvrières hautement spécialisé(e)s . Allez voir un chaudronnier au travail , un chaudronnier ne fait pas de chaudrons , il fait des carlingues de bateau , d' avion , de train , c' est réglé au millimètre et des fois en dessous du millimètre , c' est un BAC PRO . On ne fait pas un grand pays sans beaucoup de BAC PRO et la question n' est pas sottement de &quot; revaloriser &quot; l' enseignement professionnel , pour la raison qu' il a toujours été valorisé . La preuve , c' est que ceux qui en sortent trouvent du boulot . La question est de savoir quelle place on donne aux qualifications . Si ils ne veulent pas en parler , c' est parce qu' ils ne veulent pas les payer . Je trouve que c' est une honte ce qu' ils ont fait ces dernières années . Entre le gouvernement de droite de Monsieur Fillon qui avait déjà réussi l' exploit de nous donner 1 million de chômeurs , comme je vous l' ai rappelé tout à l' heure , et le gouvernement de gôche ... Ces deux-là ont fermé 226 lycées professionnels . Il y a plus de 60.000 élèves de moins dans cette filière , alors que pendant ce temps , la population scolaire a augmenté . Ils ont donc mis durablement et pour longtemps - si nous n' inversons pas le sens des événements - le pays dans une impasse . On ne peut pas faire la transition écologique dont je vous ai parlé tout à l' heure , on ne peut pas faire l' économie de la mer on ne peut rien faire de tout ça , si on n' a pas les milliers , les dizaines de milliers , les centaines de milliers d' ouvriers et d' ouvrières hautement qualifié(e)s dans tous ces domaines . C' est la raison pour laquelle c' est une tâche d' urgence de reconstituer dans toute son étendue un système de formation professionnelle à la hauteur de l' objectif que l' on se donne . Avec avec une fluidité avec une fluidité qui permet à celui qui a commencé ou à celle qui a commencé un CAP , une fois qu' il sera allé jusqu' au BAC professionnel qui , lui , sera rétabli en 4 ans , parce que c' est très difficile de passer un BAC PRO ... Eh bien il faut ensuite qu' il puisse aller à un Brevet de Technicien Supérieur ( BTS ) parce que nous avons besoin de ces techniciens supérieurs , formés en 2 ans ; à une licence professionnelle - c' est moi qui les ai créées . Il faut que cette fluidité soit garantie , il faut qu' on s' occupe de ces milliers de gens qui sont d'accord pour faire ce travail-là , qui ont compris quel est leur intérêt , celui du pays et celui des choses qu' il y a à faire . Voilà tout ce qu' il faut reconstruire , méfiez vous de ceux qui viennent vous dire qu' il faut de l' apprentissage partout . L' apprentissage ça va pour certains métiers , mais on voit bien que pas un de leurs gosses n' y est . Ils ignorent ce que c' est , ils ignorent pourquoi on va en apprentissage , souvent c' est parce qu' il faut mettre un peu , comme on dit , de beurre dans les épinards . Parce que l' apprenti il a un petit quelque chose , comme vous le savez ; et puis parce qu' on pense souvent avec espoir et confiance que notre jeune qui aura été en apprentissage , ensuite il sera recruté par le patron ... Mais tout ça c' est des histoires . Ça n' existe pas , 25 % des contrats d' apprentissage sont rompus dans les 3 premiers mois . Ce n' est pas une bonne idée , en tout cas ce n' est pas généralisable . Et le gouvernement qui est en place leur a donné 1.000 euros par apprenti qu' ils prennent , eh bien vous savez ce qui se passe ? Il y a de moins en moins d' apprentis . Pourquoi ? Parce que les gens ont compris que ça ne fonctionne pas . Ils veulent pas que leur gosse y aille parce qu' ils se disent : &quot; C' est une impasse &quot; . On peut pas parler de la condition des salariés , on ne peut pas parler du travail sans parler de son contenu , c' est à dire des qualifications qu' il faut . Et une qualification c' est pas comme je vous l' ai dit tout à l' heure , la main : c' est la tête qui compte . Parce que le travail c' est des maths , c' est de la langue , c' est des langues , c' est de la physique , c' est de la chimie , les lois de la nature sont les mêmes pour tous le monde quelle que soit la voie par laquelle on étudie à l' école , que ce soit l' enseignement général ou l' enseignement professionnel , ce n' est pas vrai qu' il y a une école de pauvres et pour les pauvres , ce n' est pas vrai . Plus le contenu intellectuel d' un jeune est développé par le goût qu' on lui insuffle des sciences , de la poésie , de la musique , plus sa compréhension globale du monde va mieux , et plus son aptitude à se qualifier dans les secteurs intellectuels les plus pointus est grande - l' être humain est un tout . Il n' est pas un appendice de machine , ni quand il travaille , ni quand il se prépare à travailler . Bon . Vous m' écouterez mardi hein parce que sinon on passe la nuit , là . C' est de bon coeur Mes ami(e)s J' ai fait pour le mieux , je sais pas ce qui va être retenu de ce que je vous aurai dit . On a un bon travail à faire avec tout le programme que je vous ai présenté . Vous avez compris , hein , ça peut pas marcher que d' en haut , qu' à partir d' en haut . Il faut qu' il y ait une implication populaire , j' essaye de faire ce type de campagne avec vous tous . C' est pas juste une campagne &quot; On va voter Mélenchon &quot; ... Bien sûr qu' il faudra le faire , mais il faut qu' on fasse avancer les idées dans notre pays . Il faut qu' on - même d' un millimètre , même d' un mètre , ce sera ça de pris . Ouvrir les têtes , redonner du courage , redonner de l' envie de vivre , de lutter , se dire que c' est possible de changer profondément l' existence , la nôtre mais celle du monde . Moi je crois à tout ça . Là , en ce moment , on est rediffusés , il y a des milliers d' ami(e)s qui doivent être en train de nous regarder , et puis on va prendre des petits bouts . Nous construisons un mouvement . Un mouvement c' est pas un parti . C' est un mouvement . Là , on le fait pour une tâche : une élection présidentielle . Si nous gagnons - et cette fois -ci , une possibilité se présente qu' on y arrive , à condition d' être opiniâtres , patients , méthodiques , et ainsi de suite . Et si nous y arrivons , le changement qu' il faudra déclencher c' est pas au bout de 2 mois de parlote . C' est séance tenante , dans toutes les directions , tout à la fois . Ça fonctionne si il y a une masse immense de gens conscients . Et le peuple français , vous autres , nous tous , le niveau de conscience s' est beaucoup élevé , nous avons appris beaucoup de choses , et notamment depuis 2005 , ça se voit pas comme ça . Et depuis 2005 , le vote contre le traité constitutionnel européen , des milliers de gens sont entrés dans la conscience politique . Et maintenant , la jeune génération vient aussi aux questions générales et politiques . Elle entre par une porte , elle a compris mieux que les plus anciens , à quel point la défense de l' écosystème est devenue une priorité , parce qu' à leur tour ils pensent à leurs gosses , à leur futur , à ce qu' ils vont faire , et ils voient bien qu' on est en train de transformer les mers en cloaques , que l' air devient irrespirable et ainsi de suite . Donc des dizaines , des centaines de milliers de gens de tous âges , des millions sont disponibles . Pour changer , tout ce qui est là , de fond en comble , une vraie révolution citoyenne - c'est-à-dire où les gens s' emparent chacun du domaine où il est . Voilà ce qu' on va faire , mais on va le construire étape par étape . Une campagne , on alerte , on fait réfléchir , on oblige un peu des fois -faut se bousculer , on est obligés de lire le texte , des fois on comprend rien ... Faites pas les malins , des fois on comprend rien . Moi je comprends rien aussi , alors je me fais expliquer , et voilà comment on se forme . On élève son esprit . Bon , nous faisons un mouvement . J' ai besoin que d' ici à la mi-janvier ... D'abord il me faut des sous , bah voilà quoi , il faut des sous parce que pour les matériels , tout ça là , on a un problème de local donc on lance un appel au don , il y en a pour 70.000 euros à trouver , cette fois -ci . Ensuite , on a besoin que le mouvement sans cesse s' élargisse . Alors il y en a beaucoup , ils sont déjà dans la campagne . Ils font des choses , chacun d' entre vous peut être un comité de campagne , c' est pas que moi qui fait le boulot ni l' équipe qu' il y autour . Je voudrais qu' on arrive à 200.000 signatures d' ici à ... Bon , quelques jours quoi . Ca va , on est à 2 mètres d' y arriver . Si vous en pouvez encore de m' entendre , ce soir on met en ligne une émission pilote . Alors vous verrez ! Vous avez qu' à regarder sur la chaîne YouTube et puis voilà , vous en saurez davantage . Ensuite , chacun d' entre nous peut avoir sa liste , je sais pas si vous avez vu le petit carnet , je ne l' ai pas amené avec moi , mais on travaille sérieusement . Il est où ? Qui est -ce qui en a un ? C' est discipliné là-derrière ... Bon on a inventé ça , mais vous regardez , l' idée c' est que chacun peut ... Ca va il y a 4 mois ! Donc 4 mois pour vérifier que tonton , tata , mon copain , ma copine , le copain de la copine , euh ... Si il est correctement inscrit , après on va pas tordre les bras des gens , mais est -ce qu' au moins ils se sont intéressés à ce qu' on fait , ce qu' on raconte , on a le programme à la main et tout ça . Le petit carnet sert à ça . Je raconte tout ça parce que je pense que la transmission est terminée là . Et ça , c' est le logo de la campagne . Pour que vous le fassiez un peu connaître les ami(e)s . Parce que celui-là c' est moi qui décide qui c' est qui l' a , je vous fais pas un dessin d'accord ? Et puis il faut que les gens puissent nous repérer parce que il y a des gens qui ont du mal quand ils arrivent dans le bureau de vote il y a pleins de piles et puis des fois il y a des gens qui ont du mal à lire il y a 3 millions de personnes dans notre pays qui sont illettrées Attention , ça veut pas dire que c' est des bons à rien , juste il y a une difficulté pour eux . Donc faut penser à tout le monde et nous , on est le camp des petits , des déshérités , des abandonnés - pas que ça hein , on est aussi le camp de très intelligents , très cultivés , des meilleurs joueurs de musique , les meilleurs ingénieurs les meilleurs techniciens , on se débrouille pas trop mal , non , quand même jusqu' à présent ! Donc , il faut qu' on fasse connaitre le logo . Je pense que vous savez tous ce qu' il veut dire . Je vous raconte quand même que moi j' avais trouvé une idée géniale mais les gens qui étaient avec moi ne la trouvaient pas si géniale que ça , donc on a laissé tomber , et on dit : &quot; Qu' est -ce qu' on va faire ? &quot; &quot; C' est pas possible , on peut pas rester comme ça ... &quot; Et je dis &quot; bon , on va prendre les initiales F-I , ça fait FI . Et Alexis Corbière , qui est mon porte parole ... C' est un prof d' histoire , donc comment n' y ai -je pas pensé ? Me dit : &quot; Bah on n' a qu' à prendre phi , c' est la lettre grecque ( φ ) , parce que FI ça fait PHI ( φ ) , et PHI ( φ ) ça fait FI , d'accord ? Voilà la lettre grecque , elle est dessinée pour que vous puissiez le faire avec 1 doigt , comme ça . Pourquoi je vous dis ça , parce qu' il y en a bien qui doivent être facétieux , comme je l' ai été quand j' avais 20 ans . On allait sur les parkings quand il y avait de la neige , eh hop , on faisait des symboles , je ne vous dis pas lesquels ... Sur toutes les bagnoles . On est pas obligés mais enfin , c' est pour dire . Hein ? Fi ( Phi φ ) Fi ( Phi φ ) De la lettre grecque , les créateurs de la philosophie ! Alors , bon , rigolez pas , j' ai une licence de philosophie , moi . Il faut s' en rappeler de temps à autres , et le responsable général du service d' ordre c' est un docteur en philosophie . Et celui -ci qui est tout le temps à côté de moi - regardez , il est mal rasé- aussi , c' est un prof de philo , donc je suis entouré de profs de philo , la philosophie , le mot &quot; philo &quot; , ça veut dire &quot; amour de la sagesse &quot; , c' est un bon symbole pour nous parce que ce que nous proposons , c' est une sagesse du rapport aux autres et enfin , c' est la proportion harmonieuse , c' est comme ça . La proportion harmonieuse dans la nature , 1,61 . Alors les personnes qui mesurent 1m61 , peuvent savoir qu' elles incarnent une forme d' harmonie hein ... Donc c' est comme ça C' est le chiffre de l' harmonie , 1,61 . C' est un bon logo , vous pouvez le porter avec élégance , vous pouvez enlever mon nom si ça pose un problème , on va essayer de faire quelque chose qui ressemble à ça . Notre campagne , alors après le reste je vous le passe , la musique , le reste ... C' est une campagne qui doit être ... C' est pas une campagne de communication , les gens . Comprenez , bon , voilà , de la communication il y en a dans tout , ça aussi c' en est , d' une certaine façon . Mais c' est avant tout une campagne d' implication qui touche la tête et le cœur . On parle pas seulement par des raisonnements , on parle aussi par des affects . Vous êtes dans une société qui se déshumanise , qui se brutalise , c' est pour ça que je suis là aujourd'hui avec vous tous . Donc notre manière de réagir à tout ça , c' est bien sûr des slogans politique , bien sûr de l' explication politique , mais c' est avant tout une contribution par un raisonnement , par de l' affection , par de l' intérêt pour les autres , par la manière de mettre en avant ce qui concerne le caractère le plus humain de notre existence quotidienne . Rien qu' en faisant ça , on commence à changer la société dans laquelle on est . Chacun d' entre vous , quand vous sortez d' ici , demandez -vous quelle genre de révolution citoyenne vous ferez dans votre entreprise . Comment vous allez modifier la manière de travailler - oh vous ne le dites pas aux autres , ce n' est pas la peine , ils vont rigoler . Réfléchissez -y et vous verrez tout d' un coup , qu' en vous sentant responsable du monde , en oubliant les maîtres qui le dominent et leurs raisons absurdes alors vous regardez la vie et tout autour de vous différemment . Voilà tout ce que je vous souhaite , bon succès à nous tous . ",
    "C' est juste ... Je crois qu' une fois de plus c' était pas prévu . Il y en a partout . Alors on imaginait que peut-être il y aurait du monde dans le Hall mais pas dans 2 autres salles , enfin j' ai rien compris à ce qu' on m' a expliqué on m' a dit : &quot; Il y a du monde partout &quot; . Je pense qu' en vous déplaçant vous saviez ce que vous faisiez et que vous faites une démonstration de force et pour tout ce que nous avons à faire ensemble , je vous exprime ma profonde reconnaissance pour la batterie que vous rechargez , là , en ce moment . Je veux vous dire , je parle doucement parce que c' est une cathédrale , si je parle plus fort on n' entend plus rien . C' est difficile pour moi de commencer le discours ici , faut pas m' en vouloir , compte tenu des conditions acoustiques . C' est pas prévu pour ça hein , c' est la faute à personne . Donc vous suivrez , je pense , vous avez déjà entendu le son , il devait être bon pour vous ou pas ? Désolé de vous infliger ça , peut-être que tout à l' heure je repasse faire un p'tit coucou . Merci à vous tous d' être là . Je pense que c' est très impressionnant , d'après ce qu' on m' a dit ça fait 20 ans qu' on a pas rempli cette salle mais c' est sûr qu' on l' a jamais remplie jusqu' à dehors . Ça c' est une première . C' est très encourageant parce que ça a un sens politique , évidemment votre démarche ... Je vois un visage ami devant moi là , Mélenchon : Qu' est -ce que tu fais là , tu es pas d' ici ? Si , tu es de là maintenant ? C' est les Essonniens de la région parisienne , vous savez on est tous des émigrés en région parisienne alors on fini tous par rentrer chez soi . Ça signifie quelque chose que vous soyez si nombreux . J' ai vu des salles qu' on a calculé un peu trop petit parce qu' on s' est trompé puis parce qu' on a pas d' argent , mais là ça va au-delà . Ça signifie un niveau de mobilisation , de sensibilité à cette élection qui est extraordinaire , et c' est la clé du succès . Je vais vous dire pourquoi . Après je ferais le discours là-haut . Eux ils sont bien assis , ils pourront écouter pendant 2 heures . Mais , parce que notre tactique c' est d' agglomérer sans cesse , autour d' un noyau solide . C' est pourquoi nous ne faisons pas de la com ' , nous faisons de l' implication . Nous ne faisons pas des harangues , nous faisons une campagne instructive . Normalement , quand on s' est quitté , quand on se sera quitté , vous aurez mieux compris certaine(s ) chose(s ) , j' espère voilà c' est que vraiment je serai nul , mais vous aurez mieux compris certaine(s ) chose(s ) et vous pourrez vous même les transmettre et c' est comme ça qu' on va avancer . Au mois de février , le tableau se sera décanté et vous allez avoir 10 millions de personnes qui tout d' un coup vont s' intéresser à la campagne , et puis ensuite quand on va arriver vers le mois de mars et d' avril , encore plusieurs millions . A chaque fois ils arrivent et ils découvrent un tableau . Et naturellement , plus tard ils s' y impliquent ça signifie que leur intérêt politique est moyen , après tout ils ont le droit hein , mais ils se raccrochent à ce qui est là , à ceux qui ont déjà pris parti , ils demandent l' avis autour d' eux , ils regardent les réunions et ça qu' on est en train de construire patiemment . J' imagine que si vous êtes là et que si vous supportez ça debout c' est que déjà vous vous sentez assez proche de ce qu' on est en train de faire tous . Soyez patient et discipliner . C' est à dire , discipliner pas par rapport à moi vous ferez bien ce que vous vous voudrez , mais dans la manière de vous organiser pour convaincre . Que chacun d' entre nous se fasse la liste de 10 personnes , s' assurer que les gens sont bien inscrits , qu' ils ont bien compris , que vous même vous ayez bien comprit le pacte qui existe en nous tous , pas seulement avec moi . C' est que on sera jamais d'accord à 100 % , hein , encore heureux , on sera jamais d'accord à 100 % mais le programme &quot; L' avenir en commun &quot; c' est un cadre qui permet de s' y retrouver , je sais pas dans quelle proportion vous vous y retrouvez . C' est ça qui faut proposer aux autres , on leur demande pas de marcher aux pas , de s' embrigader , de prendre une carte , rien de tout ça . Nous sommes un mouvement autour d' un programme . Et c' est très important que cette construction patiente , pédagogique , il doit bien y avoir des profs ici j' espère , c' est très important parce que non seulement ça sert pour gagner mais surtout de toute l' expérience que j' ai de ces situations où nous finissons par gagner alors que tout le monde pense qu' on est pas dans le coup . C' est qu' ensuite il faut gouverner et il faut gouverner avec un mouvement qui part de la population elle-même . On ne peut pas changer une société du dessus , on peut faire beaucoup de choses , il faut avoir la clé d' entrée au dessus , mais on ne peut pas le faire si des millions de gens ne s' y mettent pas car ceux à quoi j' appelle c' est une révolution citoyenne , et je ne veux pas galvauder les mots . C' est pas un euphémisme , une manière de pas oser dire je ne sais quoi , non , ça signifie le retour du pouvoir des individus sur leur propre vie . Et ça c' est vrai en général , parce qu' on fera une constituante , mais c' est vrai en particulier , dans chacune de vos administrations , dans chacune de vos entreprises il faudra révolutionner de manière citoyenne . C' est à dire qu' il faudra changer les process de production , la manière de produire , les matériaux qu' on utilise , la façon de faire . Tout doit être transformer de A jusqu' à Z pour relever le défi qu' il y a devant nous . Et c' est pas seulement un défi social , on va en parler ce soir , mais c' est un défi pour toute l' humanité . Nous sommes confrontés à un événement majeur qui est la transformation de notre écosystème et quand ce soir on va parler de solidarité , de rapports entre les êtres humains , c' est de ça dont on parle . Est -ce que la civilisation humaine est capable de prendre en charge un choc pareil ? Voilà c' est ça le sujet de ce soir . Pardon il faut que j' y aille parce que sinon je fais un discours ici , un autre dedans , ils vont plus pouvoir tenir le coup . A tout à l' heure . *Public : Président ! C' est très impressionnant ! C' est très très impressionnant qu' il y ait tant de monde , on m' avait dit &quot; Bon , on va y arriver , on ouvre et si c' est ... &quot; mais là , vraiment , ouf ! Et on est juste au début du mois de janvier . Vous avez entendu ce que j' ai dit en bas , je vous le redis pas c' est pas la peine . Donc je commence à mon propos . D'abord j' ai une nouvelle à vous donner , que peut-être vous n' avez pas eu , mais qui à mes yeux a une certaine importance et je suis sûr que vous partagerez ce sentiment avec moi . Cet après-midi les sénateurs du groupe communiste ont fait quelque chose de très important et de très utile . Moi je ne suis pas communiste mais je tiens à rendre hommage à ce groupe qui fonctionne sous la présidence de Éliane Assassi . Ils ont déposés une proposition de loi pour abroger la loi El Khomri . Et vous autres , les gens , je sais à vos applaudissements et tout le reste , qu' on pense pareil de ça . Donc , écoutez -moi avec soin parce qu' il va falloir aller l' expliquer à d' autres , il y a beaucoup de naïf , il y a beaucoup d' illusion . Vous savez pourquoi ? Parce que les gens sont braves . Les gens ils veulent croire que tout ça est un malentendu que , ils préféreraient croire que ils ont pas été trahis , et puis ça leur coûte tellement une telle humiliation . Donc ils sont toujours prêt à pardonner . Même à ceux qui les maltraitent . Et donc comme la bataille pour la loi El Khomri c' était il y a quelque mois c' était avant les vacances , donc ils seraient prêts aujourd'hui à dire &quot; Allez , zou , c' est fini , on passe à autre chose &quot; . Ils savent bienque non on va pas passer à autre chose , parce que depuis le 1er janvier cette loi s' applique . Et c' est pas pour rien qu' on leur a dit &quot; Loi El Khomri , vie pourrie &quot; parce que c' est la vie pourrie qui commence . Avec la remise en cause des durées de travail hebdomadaire , du contenu du contrat de travail , l' impossibilité de protester , parce qu' autre fois vous pouviez dire &quot; Bah écoute , tu fais ce que tu veux mais moi j' ai signé un contrat de travail &quot; &quot; Je veux pas de tes nouveaux horaires &quot; , c' est fini tout ça . Et le délai de prévenance qui passe de 7 à 3 jours , vous savez tous ce que ça veut dire . Pour le grand bourgeois , lui , c' est pas un problème , il est souple , dynamique . Comme il fout rien , bon il fait ce qu' il a à faire , mais sinon il téléphone à la nounou &quot; Finalement vous viendrez 1 jour de plus ou vous irez à l ' école &quot; mais vous autres , vous faites pas comme ça , c' est impossible . Vous pouvez pas changer l' organisation de votre vie en 3 jours . Tous les rendez-vous que vous avez pris , supposez , on va parler ce soir de la sécurité sociale , supposez que vous ayez un rendez-vous avec un médecin , un spécialiste . Tiens , va lui dire : &quot; Bah finalement , non ça sera pas jeudi , ça sera vendredi &quot; . Rien du tout , c' est 3 mois après . Bref , c' est ça le contenu pratique , quotidien de la vie des gens qui est dans cette loi , Donc , il y a une loi qui arrive pour abroger la loi El Khomri , c' était cet après-midi . Il y avait un rassemblement avant . Et vous , vous savez pas le résultat mais je vais quand même vous le donner et dans le détail . Au moment du vote , la Droite n' a pas pris part au vote . Pas pris part au vote . Ni pour , ni contre . C' est à dire ni pour abroger la loi , ni pour la maintenir , rien . Exit la droite , elle est majoritaire , sauf un , Serge Dassault et lui il est resté pour soutenir la loi El Khomri , bah il vote avec sa classe , lui . Bah c' est normal , non ? C' est le seul là-dedans qui fait un vote raisonnable si on y réfléchit bien . IL y a eu 25 sénateurs pour abroger la loi El Khomri . Ceux-là aussi , ils votent avec les leur . 20 communistes , Pierre-Yves Collombat , qui est le 1er sénateur qui m' a donné son parrainage , qui était un sénateur socialiste Et 4 sénateurs socialistes , Madame Lienemann , Monsieur Jérôme Durain , Henri Cabanel et Gisèle Jourda . Vous avez bien entendu ? Il y a donc eu , à la fin , 25 sénateurs pour abroger la loi El Khomri , la droite ne s' occupant pas du vote . Et il y a eu 115 sénateurs pour le maintien de la loi El Khomri . 115 , dont 108 du PS . 108 ! Ils sont donc exclusivement responsable de la loi El Khomri , même la droite n' y a pas mis les doigts . Si cette après-midi , ils avaient dit &quot; Tout compte fait , vu le bazar que ça était , on abroge la loi El Khomri &quot; il y avait plus de loi El Khomri aujourd'hui . Je voulais que vous le sachiez . Et à côté de ça vous avez eu 10 abstentions , les 10 écolos , 5 PS et Robert Hue . Voilà , je voulais que vous le sachiez . La loi El Khomri ce n' est pas un petit incident de parcours . C' est une transformation réactionnaire de l' ordre public social . C' est un coup direct porté à ce que nous , nous avons appelé la &quot; République Sociale &quot; , c' est à dire ce mécanisme particulier qui faisait que à mesure que les victoires électorales le permettaient , que la transformation de la société par la loi , le permettait on allait vers une République de plus en plus sociale . Et la colonne vertébrale de cette affaire , excusez -moi si c' est un peu théorique , si c' est de la théorie politique , la colonne vertébrale de cette affaire c' était que la démocratie sociale était en quelque sorte emboîté avec la démocratie politique , celle où on fait les lois . Comment ? Quand vous aviez un accord qui se négocié dans une branche . Eh bien cet accord négocié dans la branche est étendu à tous les travailleurs de la branche , par la loi . Et ensuite , vous aviez un ordre qui se créer . Si l' accord de branche , comme il est , était bon , il s' appliquait à tous le monde , donc on ne pouvait pas voter dans l' entreprise ou décidé dans l' entreprise quelque chose qui soit contre l' accord de branche . Mais l' accord de branche lui-même , il pouvait ou être amélioré ou surplombé par une loi sociale . Vous suivez le mécanisme ? En haut la loi , ensuite des accords avec un principe qu' on appelle &quot; Deux faveurs &quot; d' un étage à l' autre . La loi s' applique , aucun accord de branche , aucun accord d' entreprise n' est supérieur à la loi , il peut pas le détricoter . Vous avez un accord de branche , il est voté pour tous les travailleurs et il s' applique à l' entreprise . Vous voyez comment ça va ? Ça c' était une chose particulière aux français . Quand on vous entend , on vous parler beaucoup , vous savez de la sociale-démocratie , elle a ses mérites , dans le reste de l' Europe . Mais comment ça se passe dans un pays social-démocrate ? Vous avez un accord dans votre entreprise ou dans votre branche mais vous ne bénéficiez de l' accord que si vous êtes membre du syndicat qu' a signé l' accord . Et vous n' avez d'accord qui s' applique dans l' usine à côté qui fait exactement la même chose , que si le patron a signé l' accord de branche . Il y a pas la loi pour tout le monde . Ce sont des accords entre le syndicat et le patronat qui s' applique entre signataires . Résultat : le jour où vous avez un fond de pension qu' arrive là-dedans , il dit : &quot; Mais moi je m' en fous de vos histoires , je ne discute rien du tout , je suis venu placé de l' argent là , je prend tout ce que je peux et je m' en vais &quot; &quot; Donc je négocie rien du tout , je signe rien du tout et c' est pas la peine d' essayer &quot; . Résultat : Au Danemark , qui était un pays phare de la social-démocratie , vous avez 20 % des travailleurs , maintenant qui n' ont plus aucune protection de cette nature parce que le patron il part tout simplement . Donc quand on a fait la loi El Khomri , quand ils ont fait la loi El Khomri , ils ont décidés que tout pouvait se discuter au niveau de l' entreprise Et vous arrêtez pas de les entendre dire , au plus près du terrain , au plus près des réalités , tout ça , c' est des discours fumeux de gens qui enfument les autres parce qu' au plus près du terrain , il y a qu' une chose c' est un rapport de force . C' est comme ça , il faut le comprendre , c' est pas méchanceté , chacun est dans sa logique . Le patron qui essaye , lui , de faire que les coûts de production soient les plus bas , il est dans sa logique . C' est pas sadique . Il est dans la logique qu' il vendra mieux si c' est plus bas et il vendra mieux si c' est plus bas , s' il paye etc . Mais si le patron de cette entreprise qui produit , je dis n' importe quoi , des chaussettes obtient un accord dans son entreprise où les gens travaillent plus , où il y a plus d' heure supplémentaire les gens font le travail mais ne sont pas payés en heure supplémentaire . Bah il a un avantage , le patron de la boite d' à côté , il peut pas tenir le coup , il est obligé d' en faire autant et il va voir ses employé et il leur dit &quot; Ecoute , moi j' y peux rien , mais tu vois bien , ils arrivent à faire moins cher que nous , donc il faut bien qu' on s' aligne &quot; Et comme les gens sont braves , en général , et qu' ils ont tous la peur qu' on comprend , de perdre leur boulot , eh beh ils acceptent . C' est donc une logique de la peur , toute cette société est gouvernée par la peur . Et ils vous présentent ça comme de la modernité , la souplesse , la flexibilité , et blablabla , tout ce baratin que vous entendez sans arrêt , c' est mal traité les gens toujours plus . Et ... moi en général je m' arrête là de ma démonstration mais je leur dis : &quot; Vous êtes tellement bête , vous êtes tellement cupide , vous êtes tellement des technocrates butés , bornés , qui ne comprenaient rien à la société humaine &quot; &quot; Vous ne comprenez pas que des gens qu' on mal-traite travaillent moins bien parce qu' ils en ont ras le bol et puis parce qu' ils ont pas envie quand ils ont déjà mis tout ce qu' ils pouvaient d' énergie d' en mettre encore &quot; &quot; En sachant qu' il y a rien qui revient &quot; Quand ils ont décidé de supprimer les heures supplémentaires , le paiement des heures supplémentaires , qu' ils l' ont passé de 25 % que ça devait être à 10 % , évidemment que ça dégringole dans toutes les boites . 10 % c' est des petites sommes mais qui comptent beaucoup pour les gens , sur un SMIC , on pourrait dire : &quot; Bah qu' est -ce que c' est 2 euros &quot; mais c' est très important quand on vit avec un SMIC 2 euros pour une heure supplémentaire d'ailleurs on fait pas une heure supplémentaire par plaisir , on la fait pour avoir les 2 euros en question . Comment ne le comprennent -ils pas ? Comment comptent -ils que les gens vont travailler toujours davantage sans avoir jamais un retour , une satisfaction , un merci ? Non , ce n' est pas possible et dans cette loi El Khomri , c' est tout le long cet état d' esprit de persécution dans le détail pensant que les gens vont faire plus parce qu' on les brutalise et qu' ils ont peur . C' est pourquoi ... cette loi , il a été impossible pour eux de la passer d' une manière démocratique . C' est une des premières fois que l' on voit dans une grande démocratie , comme est la France , une loi d' une telle importance réorganisant l' ensemble des rapports sociaux et de la négociation entre les travailleurs et leurs employeurs . Une loi d' une telle ampleur qui changeait tant de chose et jusqu' au règle de fonctionnement , ils l' ont fait passée par 49:3 . Autrement dit , personne n' a jamais voté cette loi , à part aujourd'hui où la possibilité était donnée de l' abrogée et où ils ont pu lever la main pour donner leur avis , et quand on leur a demandé de donner leur avis , ils ont votés contre les salariés , voilà ce qu' ils ont fait aujourd'hui ! Alors je vous demande que vous vous en souveniez ! Souvenez -vous en ! Pourquoi ? Parce qu' il y en a des ballots , j' en suis sûr qu' il y en a aussi dans cette salle si vous me le permettez . Il y a des primaires qui vont arriver là , la situation est en train de se décanter . Déjà la dernière fois il a fallu que je me fâche , ça a pas servi à grand chose soit dit par parenthèse , mais au moins je leur ai dis ce que je pensais . Les gens disaient &quot; Ah tu comprends nous on est de Gauche mais on va aller à la primaire de la Droite pour voter pour celui -ci , pour empêcher l' autre de j' sais pas quoi &quot; , qu' est -ce que c' est que cette histoire . Il faut avoir du respect pour soi-même d'abord . Vous dénoncez , vous dénoncez les responsables politiques , vous dites &quot; Ce sont des cyniques , ils n' ont pas de paroles &quot; mais vous avez vu ce que vous êtes en train de faire ? Vous allez à l' élection ou chez quelqu'un qu' est pas de votre bord pour truquer l' élection et faire élire quelqu'un pour qui vous voterez pas , c' est pas honnête . Il faut donc se respecter soi-même , donc vous ferez bien ce que vous voulez les gens , mais venez pas me dire que vous allez voter à la primaire du PS pour faire j' sais pas quoi , parce que moi je vous ris . La chose suivante , voter à cette primaire c' est les amnistier , c' est rien d' autre . Eh ben moi je ne les amnistie pas ! Et d'ailleurs je vais vous dire une chose , après j' en parlerais plus parce que ça me fatigue . D'abord je veux pas perdre mon temps . Réfléchissez -y , vous voulez quoi ? Les aider à avoir beaucoup de voix pour qu' ils disent : &quot; Vous voyez , il y a beaucoup de gens qui nous suivent &quot; , pourquoi ? Pour me rendre le travail plus facile après ? Parce qu' il y a une chose dont je veux que vous soyez bien conscient tous les ami(e)s , tous bien conscient , parce que j' ai lu par ci , par là , que machin , truc et bidule , s' il était élu , le lendemain va me téléphoner , tiens ça va me faire du neuf , moi je répond à tous le monde . Tiens , ça les prend tout d' un coup au bout de 5 ans , de m' appeler au téléphone , et tu pouvais m' appeler avant , par exemple pour parler de l' amnistie sociale . Quand j' ai demandé l' amnistie sociale pour les camarades , les syndicalistes qui ont été condamnés pour avoir défendu et il y avait personne qui me téléphonait , personne . Faites ce que vous voulez , c' est votre droit , je respecte vos procédures mais s' il vous plait , ne nous prenez pas pour des poissons rouges . Ne croyez pas qu' on a tout tout oublié et ne nous racontez pas d' histoire , ce n' est pas la primaire de la Gauche , c' est la primaire du PS . Le PS a le droit d' organiser sa primaire mais alors il faut lui donner le nom qu' elle porte , c' est la primaire du PS , du parti de Gauche , là-dedans il y a le PS et le PRG , c' est tout ! Tout les autres sont dehors . Y compris , y compris ceux qui étaient pour qu' il y ait une primaire comme Monsieur Jadot qui entre temps à changé d' avis , on ne sait pas pourquoi , mais en tout cas il est allé faire une autre primaire . Un peu de respect pour nous tous . Ce que je veux que vous sachiez c' est que quoiqu'il arrive , et d'autant plus maintenant , quoiqu'il arrive , ma candidature et le programme &quot; L' avenir en commun &quot; iront jusqu' au bout de l' élection . Je veux bien qu' on me téléphone tant qu' on veut , je suis prêt à accueillir s' il le faut qui voudrait rejoindre notre combat , nous ne sommes pas des sectaires , nous ne réglons pas des comptes . Nous ne travaillerons pas à la rancune , mais changer la ligne pour laquelle on a eu tant de mal à exister , se battre , mettre au point ce programme , se rassembler comme on est en train de le faire , jamais . Ça , ça sera présent jusqu' au bout ! Alors , après , faut que je finisse avec ça parce que j' ai des sujets plus important , mais bon , il faut ce qu' il faut . Là il y a marqué : &quot; Le tableau se précise &quot; bon voilà . Je viens de vous dire quoi ? Dites franchement , supposer qu' on me téléphone , alors qu' est -ce que je fais ? Mais oui bien sûr alors on va commencer pendant 8 jours , non c' est toi , oui c' est moi , etc . Qui c' est le candidat , on sait pas , bah si je le sais , c' est moi . Déjà il y a un point qui est réglé . Deuxième point , le programme , bien sûr qu' on doit en discuter , et déjà eux , comment ils vont faire ? Comme Hollande est partis , toute la cohésion qui avait là-dedans , elle a disparu . Donc ils ont chacun leur programme , ils ont le droit , je veux pas m' en moquer , mais ils ont chacun le leur . Et après Mesdames , Messieurs , et pour beaucoup d' entre vous , mes chers camarades , qui en connaissaient donc un rayon sur ce genre de chose , il leur reste 92 jours pour se mettre d'accord sur un programme commun entre eux , avant de venir parler avec les autres avant le 1er tour de l' élection présidentielle . Est -ce que vous trouvez que c' est sérieux ? franchement , ils avaient toute la vie pour se préparer , ils ont gouvernés pendant 5 ans , et là , il y a rien de prêt . Non , ce n' est pas sérieux . Hein . Alors avant ils disaient &quot; De toute façon , sérieux ou pas , on s' en fout , c' st nous qui sommes devant , on est le vote utile &quot; . J' ai jamais aimé le vote utile et c' est pas parce que maintenant c' est moi qui suis devant que je vais m' en servir . Mais je sais maintenant au moins qu' il y a un vote inutile , ça me permet de dire : &quot; A quoi bon un candidat du PS ? &quot; Bon , alors après , bon , j' en parle plus parce que pour parler du contenu du programme , il faut d'abord qu' ils en aient arrêter un et qu' on sache lequel . On va pas perdre notre temps à faire le tour , hein . Je viens aux autres alors là , ouf , ça s' aggrave , mais vous mais vous savez il y a un petit air de panique là ... Chez beaucoup . Hein , le casse noix se met bien place , pour qu' il y ait un casse noix il faut que ça sert des 2 bords . Macron ou Mélenchon . Macron ... Mélenchon ... Macron , Mélenchon , et au milieu ça fait de l' huile . Mais ça c' est la bataille à la loyale , vous savez bien , il y a 2 programmes , la solidarité ou la compétition , moi cet homme je le respecte , il défend ses idées , il va au bout , il raconte ses trucs , c' est normal , la démocratie c' est quand même ça , on ne demande pas que tout le monde dise la même chose ou tout le monde dise la même que nous , c' est pas le sujet . Chacun va exposer son programme , les français vont trier , c' est une Constitution absurde qui fait qu' au 1er tour on choisit , au 2ème on élimine , le résultat est qu' on est jamais content . Mais c' est la démocratie , moi j' en accepte toute la règle et à la fin je ferais comme vous tous , je m' inclinerais . Justement , je voudrais bien savoir , vous savez il y a des inclinaisons qui coûtent plus cher que d' autres , et justement , la dame qui avait déjà gagnée l' élection , que c' était même pas la peine de l' organiser n' est -ce pas ? Ou si elle avait pas gagnée il y avait juste à s' occuper du second , notez au passage que le piège pourrait se refermer sur eux , hein . Le second pourrait être quelqu'un qu' ils ont pas prévu , eh oui . Alors , elle , elle commence à paniquer , je pense que c' est un bon signe . Dans cette salle déjà , pour que vous soyez venu à une réunion tout ça , c' est que vous vous intéressez au contenu des échanges , des discussions , de ce que chacun dit . Déjà , vous avez dû vous rendre compte qu' il y a quelque chose qui tournait pas rond . Cette femme là elle nous singe . D' habitude on dit &quot; Singe savant &quot; , là j' ai pas dit &quot; Savant &quot; , j' ai dit &quot; Elle nous singe &quot; . Les discours , on reprend des phrases de nous , entières , des formules , après vous avez toute la ribambelle à micro qui rapplique en disant : &quot; Vous dites comme elle ! &quot; Ah non non , pardon ... Enfin vous avez compris . Ça embrouille tout , personne comprend plus rien et moi , à chaque fois , et ramer et encore ramer . Mais là , apparemment ça va très loin . Vous savez peut-être que j' ai fait une chaîne YouTube , il y en a peut-être d' entre vous qui regardent ça . Alors je suis là , assis sur mon p'tit truc , je raconte dans la semaine comment j' ai vu les choses et il y a pas longtemps on m' a dit &quot; Tiens , si au passage tu pouvais faire un peu de pub , on un mug il y a marqué dessus &quot; L' insoumis(e ) &quot; &quot; . Vous voyez , c' est un p'tit mug . Alors me voilà avec mon p'tit mug , je dis &quot; C' est très bon &quot; tout ça . On vend le truc quoi ! Et là on me dit : &quot; Monsieur Philippot aussi fait la chaîne YouTube &quot; , ouh ! Alors je me dis &quot; Bah il me singe &quot; , quand même c' est moi qui ai commencé , mais le gars alors , eh oh , un peu d' imagination . Il a aussi un peu mug à la main ! Et un mug rouge ! Le mien il est blanc ! Bon ... Je fais la plaisanterie mais je suis comme beaucoup d' entre vous . J' ai de l' expérience , je suis pas un perdreau de l' année , je sors pas de l' œuf . Je repère l' odeur de la panique chez l' ennemi longtemps avant qu' ils aient fini de l' émettre . Déjà avoir trouvé comme idée pour Madame Le Pen comme symbole de prendre une rose , par les temps qui courent , fallait vraiment le trouver ... Au bout d' une dalle mortuaire . Alors il parait que c' est un fleuret , je sais pas qui c' est qu' elle compte menée comme ça au fleuret , mais pas nous . Ce qui est sûr c' est que ça le fait pas quoi . Hein . Ils paniquent , ils descendent , on peut les battre . Et vous savez pourquoi je sais qu' ils paniquent ? Parce que chaque fois qu' ils paniquent ils tournent et retournent à leur fondamentaux , c' est à dire que chaque fois qu' ils ont l' impression qu' ils perdent pied , ils reviennent sur des gros trucs racistes et ainsi de suite et là , elle s' est dépassée parce que comme ils avaient déjà injurié la moitié de la Terre , la quasi-totalité des gens qui n' ont pas sa religion . Alors que faire ? Eh bah elle a trouvé une nouvelle cible , les gosses . Faut le faire hein . Moi je pensais que même les racistes ils respectaient les gosses , ou les xénophobes , j' dis bon , on est tous les mêmes hein . Vous savez , c' est un invariant d' échelle , tout le monde partout dans le monde , on a le même réflexe , c' est à dire si un gosse tombe , qui que vous soyez , quelque soit votre âge , où que vous soyez , vous vous avancez pour l' empêcher de tomber . C' est un geste , un réflexe . C' est pas pareil comme c' est un adulte , si quelqu'un tombe , vous commencez par faire &quot; Ehlala ! &quot; Si c' est un gosse ... Donc c' est pour dire que c' est ce qu' il y a de plus profondément ancré dans le cœur d' un être humain . C' est protéger les plus petits . C' est de ça dont on parle ce soir , protéger celui qui a le plus besoin des autres . Eh bah elle trouve le moyen de dire &quot; Je considère que la solidarité nationale doit s' exprimer à l' égard des Français . &quot; &quot; Je n' ai rien contre les étrangers &quot; , heureusement parce qu' est -ce qu' elle ferait ? &quot; Mais je leur dis : &quot; Si vous venez dans notre pays ne vous attendez pas à ce que vous soyez pris en charge , à être soigné , que vos enfants soient éduqués gratuitement &quot; , maintenant c' est terminé . &quot; Je relis : &quot; Ne vous attendez pas à ce que vous soyez soigné &quot; . Eh bah ça , c' est pas une bonne idée du tout parce que celui qui est malade , il est malade et rend malade les autres si on le soigne pas . Donc , si elle veut que la solidarité nationale doit s' exprimer à l' égard des Français , il faut soigner les étrangers . Il y a pas le choix . Deuxio &quot; Que vos enfants soient éduqués gratuitement &quot; , ah mais ça c' est pas de la solidarité , c' est une organisation du pays . Nous avons fait le choix , nous les Français qui n' avons aucune matière première , d' élever sans cesse le niveau d' éducation de notre peuple pour être parmi ceux qui manient le mieux les techniques , les machines , les sciences et on réussit très bien . &quot; Que vos enfants soient éduqué gratuitement , maintenant c' est terminé &quot; . Qu' est -ce que ça veut dire ? Vous allez laisser les petits pitchounes , on va rien leur apprendre ? Ils vont pas savoir lire , ils vont pas savoir écrire , ils vont pas savoir compter . Leurs pauvres parents , qui regarde ça , alors ils vont parler quelle langue ces petits ? Juste la langue qu' on parle à la maison ? Reste plus qu' à espérer que c' est le Français . Mais les parents étaient trop contents que les gosses s' en sortent , qu' ils aillent à l' école , qu' ils s' améliorent et ils reviennent et c' est comme ça que ça se passe dans une famille d' étranger . C' est comme ça que ça se passe . Et c' est ça que vous voulez arrêter ? Vous allez faire quoi ? Dire &quot; Non , toi on ne veut pas de toi &quot; , alors , le gosse il a son copain dans la cité , ou sa copine , peut-être qu' ils jouent ensemble etc . Mais à l' heure d' aller à l' école , &quot; Toi non ! &quot; &quot; Toi oui ! Rentre mon petit &quot; , &quot; Toi non ! Dehors ! Va-t -en ! &quot; Qu' est -ce que c' est que ce monde ... Comment ils pensent les choses ? Et maintenant , si j' oublie l' aspect humain , parce que dans notre société , quand on parle d' aspect humain , on passe pour un rêveur ou un poète et le poète est méprisé . Alors que moi je considère que c' est la principale vertu de l' esprit , mais bon , si on met ça de côté ... Qu' est -ce que c' est que cette bêtise de croire qu' une société va être meilleur avec des ignorants . Mais il y a que des ignorants pour croire cela ! La société est meilleur plus nous sommes éduqués et être éduqué c' est pas seulement apprendre comme certains croient des choses qui servent à quelque chose comme ils disent , c' est tout ce qui développe la personne humaine . A l' école , on apprend aussi de la musique , on apprend de la littérature , on lit des poèmes , on fait des dessins , et toutes ces choses vous transforment en un être humain meilleur et quand vous êtes meilleur vous , alors toute la société est meilleure . C' est le pari sur lequel repose l' humanisme progressiste que nous incarnons . Le Républicanisme . Bon j' arrête avec elle parce qu' elle va nous rendre malade . Alors maintenant , évidemment , on m' a dit : &quot; Vous vous allez dans la Sarthe pour affronter Monsieur Fillon &quot; , mais renseignez -vous , Monsieur Fillon n' est pas de la Sarthe , il s' est enfui . Les gens de la Sarthe l' ont foutu dehors . Ah oui mais écoutez , faut pas nous en vouloir si vous êtes de la Sarthe , mais on a l' intention de vous le rendre . Il a son château que est par là , bah il a le droit . Tout le mal qui s' est donné pour avoir un château . Bon , mais c' est pas le sujet . Le sujet c' est que cet homme est réputé être notre prochain président de la République , vous savez tous pourquoi , parce que bien sûr il va arriver au deuxième tour , on se demande pourquoi on va voter et justement au deuxième tour il y aura Madame Le Pen , on se demande pourquoi on va voter et quand il sera là , eh ben , vous n' aurez qu' un seul choix , c' est de voter pour lui pour pas voter pour l' autre , on se demande pourquoi on va voter . Ils feraient mieux de le nommer tout de suite . Comme ça on s' épargnerait tout ça . Bien sûr , rien de tout cela ne va se passer comme prévu parce que ça fait déjà une paire d' élection que rien ne se passe comme prévu , mais en attendant , où vous êtes interrogé et invité à réfléchir tout le temps sur cette base stupide . Le vote utile , le vote ceci , le vote cela . Vous n' avez à vous soucier que d' une chose : Quel est le vote nécessaire ? Qu' est -ce qui est bon pour tous ? Qu' est -ce qui est bon pour la patrie ? Qu' est -ce qui est bon pour les autres personnes qui vous entourent ? Et le reste vous ne vous en occupez pas parce que ça s' appelle la théorie du jeu , aller au PMU plutôt que dans un isoloir pour voter , parce que si vous y allez avec une mentalité de PMU , si vous abandonnez vos convictions à la porte du bureau de vote , faut pas vous étonnez pas si vous les retrouver pas en sortant . D'accord ? Donc , Monsieur Fillon étant comme vous le savez par avance , notre prochain président de la République , nous a déjà annoncé son programme et dans son programme il y a le fait qu' il a décidé de supprimer la Sécu . Non non , faut pas dire : &quot; J' ai pas dit ça &quot; , il l' a dit . Alors avec le langage ... J' espère que je vais tomber pile-poil sur ma page ... Oui , j' ai peut-être été un peu vite mais c' est pas grave . Projet santé , avant décembre : &quot; Il faut &quot; ... Je vous le traduirais , parce que c' est de la langue de techno ... &quot; Focaliser l' assurance publique universelle &quot; ça c' est la Sécu . &quot; Focaliser &quot; ça veut dire &quot; elle doit juste s' occuper d' eux &quot; &quot; Sur des affections graves ou de longues durées &quot; Lui , docteur Fillon , et les gens qui l' entourent vont vous dire ce qu' est une affection grave , que les toubibs eux-mêmes ne sont pas capables de vous dire parce que ça dépend l' âge que vous avez , ça dépend la maladie que vous avez , on meurt plus facilement d' une grippe à 60 ans qu ' 25 ans , mais ça ils ne le savent pas eux . &quot; Affection grave ou de longue durée et l' assurance privée sur le reste &quot; Bon je vous le relis pas , tout le monde a compris . Les grandes douleurs pour la Sécurité Sociale , les petits bobos pour l' assurance privée , vous allez vous assurer . Naturellement , tous on a fait , on s' est dit : &quot; ' Mais ils sont fous ! &quot; , de quoi ils parlent ? Qu' est -ce que c' est une petite maladie ? Qu' est -ce que c' est une grande ? C' est pas comme ça qu' il faut prendre le problème . On doit tout rembourser , parce qu' une petite maladie peut devenir une grande et parce que quand une petite maladie est pas soignée , elle devient une grande maladie et à ce moment-là , ça coûte encore plus cher Messieurs les comptables que ça n' aurait coûter si on l' avait régler au début . Ça c' est ce qu' il a dit , comme tout le monde à hurler ... Il revient le 12 décembre et il vous dit ... Le contraire . Mais bien emballé , que vous savez pas ce que ça veut dire . &quot; L' assurance maladie obligatoire universelle , pilier de la solidarité continuera à couvrir les soins comme aujourd'hui et même mieux ... &quot; &quot; Rembourser des soins qui sont largement à la charge des assuré(e)s comme les soins optiques et dentaires &quot; , vous avez vu ? Suffit de gueuler un coup et il baisse le ton . &quot; Il n' est donc pas question de toucher à l' Assurance Maladie et encore moins de la privatiser &quot; Comme nous sommes de vieux guerriers , on le laisse parler et on écoute la phrase suivante . &quot; Il convient par ailleurs ... &quot; C' est là que ça se gâte ... &quot; De responsabiliser chacun &quot; . Ah ... Vous avez compris ? &quot; En simplifiant les nombreux dispositifs de franchise actuel &quot; déjà la phrase elle est un peu longue . La franchise vous savez ce que c' est , c' est un truc tordu et mensonger qui vous fait croire que vous êtes remboursé mais vous l' êtes pas , ça s' appelle une &quot; Franchise &quot; . C' est à dire vous payez . Si vous allez à l' hôpital , 18 euros par jour . Alors il y a plein de franchise . &quot; Quel fouillis , il y a plein de franchise , on comprend rien &quot; bah oui , c' est lui qui les a inventé . Donc il sait qu' on comprend rien , et c' était fait pour , d'ailleurs vous vous souciez pas de comprendre les franchises . Votre souci c' est de savoir si vous êtes à l' hôpital , si vous pouvez payer vos soins , c' est tout . Après , savoir qu' il y a une franchise différente pour autre chose , c' est pas votre sujet . Pour lui , si . Bon , il en était là . Le 12 décembre , il avance bien ce petit . Et nous voici rendu le 11 janvier , là comme on l' a ... bien bombardé , on l' a pas lâché . Voilà ce qu' il dit : &quot; Sur la Sécurité Sociale , on n' a pas été bons &quot; . Ça c' est clair . &quot; Ce que nous avions écrit à été mal compris &quot; , non justement c' est parce qu' on avait compris que vous avez pris peur . &quot; Je l' ai retiré &quot; . Ah bah voilà qu' il commence bien , il est pas encore arrivé qu' il a déjà retiré son projet . &quot; J' ai lancé un débat avec l' ensemble des responsables de la santé blablabla , qui est en train de se dérouler en ce moment , je referais des propositions avant la fin du mois de janvier &quot; eh bah nous , on va l' attendre . Hein ? Mais vous avez compris les gens ce qu' il faut faire ? Il faut y aller . Et dire à tous ceux qui vous entoure : &quot; Vous savez ? Il veut plus payer les petites maladies &quot; , alors les gens vont dire : &quot; C' est quoi la petite maladie parce que moi ça me fait très mal &quot; justement on veut plus vous rembourser , faites comme ça . Et vous allez voir si ça va pas remonter jusqu' à lui et à la fin il oubliera jusqu' à la première lettre . Il n' y a pas de petite victoire pour nous mes ami(e)s . Vous savez s' ils ont le sentiment que vous êtes prêt à vous laissez faire , ils vont jusqu' au bout . Quand j' étais jeune homme , qu' on nous faisait des cours pour nous expliquer comment c' était la lutte sociale , on nous posait la question &quot; Quelle est la limite à l' exploitation ? &quot; Tu connais la réponse ? La résistance à l' exploitation ! C' est tout , il n' y en a pas d' autre . Eh bien là c' est pareil . Dans le débat qui commence vous devez être vigilant , nous devons tous être vigilant à la première heure . Ces gens là sont décidé à passer à l' offensive parce que la masse d' argent qui recouvre toute la protection sociale est un trésor immense qu' ils veulent faire passer dans les mains privées , dans les assurances , dans tout ce qui est privé parce que c' est une masse immense que nous avons précisément réussi à accumuler pour le soustraire aux mains privées . Et c' est parce que nous l' avons soustrait aux mains privées , au fil de nos luttes , que nous avons remporté ce résultat qui est nous tous ici grâce à nos anciens . Nous tous ici nous avons été soignés , nous tous ici nous avons été pris en charge , nous tous ici nous avons mis nos enfants au monde dans des conditions telles que la mortalité de nos compagnes pour les hommes a diminué extraordinairement et la mortalité de nos enfants a diminuée d' une manière radicale . Gloire à la Sécurité Sociale parce que c' est elle qui a permit tout ça . Dans votre ville , vous allez voir le film &quot; La Sociale &quot; , je crois que c' est le 12 que ça passe , hein ou je me trompe de date , voilà jeudi 12 . C' est un bon film . Il faut que vous alliez le voir . C' est Gilles Perret qui a fait ça . Et puis après , vous allez voir , ça va vous redonner le goût , parce que c' est ça qui faut qu' on fasse dans cette campagne , tout le monde a tellement peur . Les gens en ont tellement prit la tête que ils n' arrivent même plus à croire qu' un progrès est possible . Ils pensent que ce qu' on a , si déjà on peut le garder c' est bien , et ils ont bien raison , il faut le défendre , mais bon , ça on a jamais agit comme ça dans le passé , on voyait toujours la masse extraordinaire de richesse et on disait : &quot; Bah il faut partager autrement &quot; c' était ça le progrès , c' est à cette idée là qu' il faut qu' on revienne . Le progrès est possible . On peut améliorer notre vie quotidienne . On peut vivre mieux , il y a rien qui nous condamne à ça parce que le pays détruit ( ? ) à la Sécurité Sociale Et maintenant qu' il est plus riche qu' il ne l' a jamais été de son histoire , on ne pourrait pas faire vivre ça ? Qu' est -ce que c' est que cette histoire . Il y a un mensonge en route . Depuis 1870 , on a multiplié par 3 l' ampleur de la richesse produite par ce pays . Et on a divisé par 2 le temps de travail qu' il fallait pour la produire . Donc ce pays est plus riche que jamais et vous verrez dans ce film &quot; La Sociale &quot; , il y a un vieil homme , il a 93 ans je crois , le malheureux , c' est un bon camarade , il est mort ... après que le film soit sorti , il avait dit : &quot; Bah le film , ça sera mon héritage &quot; . Je sais pas si vous avez vu ce film , mais ces gars à l' époque , ils ont crée la Sécu en rassemblant dans une seule caisse tout ce qui était éparpillé de tous les côtés , c' était des gars de la CGT . Peut-être que vous aimez pas la CGT certains d' entre vous mais c' était la CGT et alors , ils étaient là , vous comprenez , ils étaient la nuit , le jour en train d' organiser leur truc pour que ça marche , pour qu' on puisse rembourser et ça paraissait tellement incroyable aux gens d' être libéré de tout ce qui auparavant avait écrasé leur existence , la peur de la maladie , la peur de l' âge qui vient . C' est tellement vrai que le ministre qui s' occupait de ça , qui est un communiste , vous allez croire que je fais de la pub pour le parti Communiste toute la soirée mais c' est la vérité , il faut rendre à ceux qui l' ont fait ce qu' ils ont fait . C' était Ambroise Croizat . Le pauvre Croizat , il en est mort d' épuisement . Je vous raconte ça , pas pour vous parler d' un parti ou pour vous parler d' une institution , juste pour que , si vous voyez le film , vous voyez , dans les yeux de ce vieil homme , brillé cette flamme extraordinaire qui est la nôtre , celle qu' on a dans le cœur . Ce monde peut nous appartenir , et nous n' avons pas des revendications extraordinaires , nous demandons des choses simples , éduquer nos enfants , que nos anciens puissent se retirer du poste de travail quand le moment est venu , que nous soyons soigné quand il le faut , et pour le reste foutez nous la paix . On saura organisé notre propre bonheur personnellement , on a pas besoin que vous veniez , nous casser la tête , avec vos obligations de ceci , vos obligations de cela , votre injonction à être comme ceci ou comme cela , porter tel habit pour ressembler à ci ou à ça , manger ceci ou cela , toute cette pourriture que vous faites entrer dans le cœur , dans le corps des gens , par la publicité , par la répétition sans cesse d' un seul modèle d' existence humaine . La Sécu ça vient dans la catégorie suivante , d'abord il y a eu l' invention du feu , de l' agriculture , de l' élevage , et la Sécu ! La Sécu elle installe pour des centaines de milliers de gens une règle , les aléas de la condition humaine , tout ce qui nous atteindra à tous à un moment ou un autre , tous , autant que nous sommes ! Nous sommes et nous serons malade . Tous autant que nous sommes . Nous avançons dans la vie et arrive un moment où nous n' avons plus la force d' accomplir ce que nous accomplissions avant , plus facilement . Et puis , compte tenu de la vie que nous avons eu , nous nous présentons à son extrémité d' une manière différente , suivant le parcours que l' on aura eu . Et cela ne tiens pas spécialement à nos gènes , cela tiens au métier que l' on aura effectué . L' espérance de vie en bonne santé est de 59 ans pour un ouvrier et de 63 ans ou de 62 ans pour quelqu'un qui ne l' est pas , je parle d' un homme . 63 ans pour nos compagnes . 62 ans pour les hommes . 59 ans quand on est ouvrier . Eh ben c' est pas parce que les ouvriers ont des gènes moins résistants que ceux des autres . C' est les conditions sociales qui font ça , donc la société toute entière a une responsabilité devant chaque individu et elle l' assume en répondant &quot; Quoiqu'il arrive , qui que tu sois , homme femme , riche ou pauvre , de chacun selon ses moyens , chacun , tous ses besoins , voilà pourquoi , que que tu sois , quelque soit la maladie que tu as , alors nous viendrons tous à la rescousse et nous mettrons les moyens qui le faut pour te guérir sans te demander plus selon qui tu es &quot; . Autrefois , les anciens , avaient repérés , ce qu' on appelait des &quot; risques &quot; , bon le risque &quot; famille &quot; je pense pas que ça soit un risque mais tout le monde comprend ce que ça voulait dire , quand tu as une famille à charge , tu te comportes pas de la même manière que si tu dois juste t' occuper de toi tout seul , tu as la responsabilité du petit groupe que tu as fondé , ta famille . La santé , le temps de travail , etc . il y a 4 caisses . Mais dites les gens , maintenant il y a une nouveauté c' est les risques liés à l' environnement . Donc , ceux qui vous disent : &quot; ' Ah bah on vit mieux , plus longtemps &quot; , il faudrait qu' ils réfléchissent à ce qui est en train de se passer parce que quand vous interrogez les ami(e)s qui travaillent dans la santé , ils vous disent que 80 % des maladies qu' ils ont à traité ont une origine environnementale . C' est une nouveauté , la pollution 48 000 morts par an . Les pesticides , il y en a partout , on est le premier pays d' Europe à en consommer . Pour ma part je me suis opposé à l' usage des glyphosates , l' Europe a encore permis une rallonge . Le premier qui meurt de ces saletés , c' est celui qui les répand par terre , le paysan qui n' a pas le choix parce qu' il est pris comme ça , faut qu' il produise , beaucoup , et comme lui a dit la banque qui lui a prêtée l' argent &quot; Tu dois faire ceci , tu dois faire cela , dans telles et telles conditions et de telles et telles manières et pas autrement . &quot; Les perturbateurs endocriniens . Vous le savez les gens , qu' il y a de plus en plus de mômes qui naissent bizarrement . On voit toute sorte de malformation qu' on voyait pas encore tant que ça , je veux pas gâcher la soirée hein , mais je pense que beaucoup d' entre vous comprennent ce que je suis en train de dire . Et puis vous avez des maladies qui reviennent , qui y étaient plus , mais qui reviennent du fait d' un côté de l' appauvrissement des gens , bah oui ! comme ils sont plus pauvres , ils mangent moins bien , ils habitent dans des maisons où on a froid , où il y a du plomb , où il y a toutes sortes de saletés qui pourrissent la vie des gens . Ça c' est l' appauvrissement . L' appauvrissement de la population il a un coût sanitaire et pas seulement un coût social , qui est déjà énorme . Et puis , à ce coût , s' ajoute , qui y était pas , les maladies qu' existaient pas et qui viennent à cause du réchauffement climatique et vous allez en voir de plus en plus , parce qu' on en voit partout , alors moi ce que j' ai pu voir c' est pas des maladies , mais j' ai vu mes copains vignerons du Jura qui vous racontent que non seulement on vendange un peu plus tôt , mais voilà que maintenant arrive la mouche drosophile chez eux . Et des mouches drosophiles dans le Jura il fallait courir pour en trouver une . Comme il fait plus chaud , elles arrivent là . Donc nous allons être confronté à un nouveau type de maladie , par conséquent , on peut pas dire : &quot; Maintenant , le problème de la santé des êtres humains il est réglé &quot; , parce qu' on a tout réglé , on est au courant de tout , on connait toutes les maladies , on le soigne toutes . Ce n' est pas vrai . Regardez d'ailleurs la limite du système . Voilà la grippe , vous êtes au courant , 1 million de personnes l' ont déjà chopée . Comme nous vivons tous dans des structures de plus en plus denses , on va pas s' en plaindre , mais , c' est le métro , c' est le bus , c' est le train , c' est bien tout ça . Ça nous permet de circuler plus facilement et sans empoisonner l' atmosphère des autres . Mais ça veut dire aussi que ce répand plus facilement les épidémies . Alors là , alors là , 2014-2015 il y a marqué sur mon papier , 18 000 morts de la grippe . Allez expliquer ça à Fillon hein . Au début les gens ils toussent , c' est tout , &quot; c' est une petite maladie on la soigne pas &quot; , pouf , tout le paquet de maison le prend . La preuve , les malheureuses 10 personnes qui sont mortes dans la même résidence de personne âgée ( ? ) pour comprendre comment ça se passe tout ça . 1 million de personnes malades , alors que dit , de quoi sommes -nous malade ? Madame la Ministre de la Santé a appelée ce mercredi , vous allez voir pourquoi c' est pas vous les chefs , c' est des gens intelligents quoi ... Ils pensent mieux que nous ... Madame la Ministre de la Santé a appelée ce mercredi les hôpitaux à reporter les opérations non urgentes . C' est reparti pour un tour , après les maladies pas grave , voici les opérations non urgentes . Sans doute , ils ont du décidé dans son bureau lesquelles . Pour soulager quoi ? Les urgences , qui elles , sont submergées . Et c' est la vérité . Et de quoi sont malades tous ces gens ? De la grippe , oui mais pas seulement . Ils sont malades des plans de santé et de restrictions de moyens partout . Voilà pourquoi on en peut plus aux urgences , voilà pourquoi ça marche plus nulle part . Moi je vais dire , je profite de ce meeting pour dire à Madame Marisol Touraine . &quot; Dans le cas où vous ne sauriez pas , vous avez prévu de fermer encore 16 000 lits d' hôpitaux , supposez que ça soit fait aujourd'hui , qu' est -ce que vous diriez ? Vous diriez quoi aux gens ? Il y a 16 000 lits de moins et en plus , ceux qui s' y trouvent dehors . Parce qu' il y en a d' autres qui arrivent pour les urgences . Après je vais vous en parler un peu plus , mais quand même . Ils veulent supprimer 22 000 emplois et ils sont tous d'accord avec ça . Si vous êtes allé dans des urgences vous le savez . Vous savez la difficulté que vous avez , déjà pour trouver un spécialiste , hein . Vous êtes sans doute déjà allé , l' un ou l' autre ou vous en avez entendu parler de ce que sont les urgences . Vous savez bien que tout ça ne tiens que par le courage et le dévouement absolu des personnels de santé , s' ils étaient pas là le truc se serait effondré , effondré ! Voilà , elle a trouvé la solution , qu' est -ce qu' on fait ? On reporte opérations qui sont pas urgentes . Alors , ça m' amène ça , à entre un peu dans le détail de cette affaire de maladie . Je vous ai montré tout à l' heure comment Monsieur Fillon avait l' intention de procéder . Il a reconnu lui-même qu' il était pas bon , je le répète une deuxième fois . Il a prévu de pas vous rembourser si vous êtes malade , je le répète une deuxième fois . Et il a prévu de vous responsabiliser parce que vous êtes des irresponsables , vous attrapez la grippe tous en même temps au lieu d' étaler , hein . Franchement , et en plus tous à la même période en plein hiver quand il fait froid , pourquoi vous ne l' attrapez pas un peu la grippe en août ? Ou en juillet , pendant vos périodes de congés comme ça , ça compterait pas comme du temps de travail où vous allez vous reposer . Bref , il va vous responsabiliser et , et , il va vous mettre un dispositif de franchise , cette menterie , et un ticket modérateur sur tout . Alors comme il y a plusieurs niveaux vous pouvez faire le pari , il va vous donner le ticket modérateur le plus bas ou le plus haut , le plus haut , c' est connu . Voilà pour lui , mais , du coup ... C' est à moi à dire ce qu' on va faire nous . Moi je vous dis , nous pouvons faire une chose , d'abord commencer par nous dire que toute maladie doit être soignée et qu' il faut y mettre les moyens pour qu' elle soit guérie . Que tout les affaiblissements du corps des êtres humains , comme nous connaissons des moyens d' y pourvoir depuis un bon moment , les lunettes , les prothèses etc . on doit se donner un objectif . L' objectif est le suivant : tout le monde est soigné totalement et tout est 100 % remboursable . Voilà l' objectif du programme . Ah oui mais ... Dit quelqu'un qu' est pas très malin ... &quot; Oui mais Monsieur Macron il dit pareil &quot; Très bien . Tant mieux , non ? Bon , sauf que l' on est rendu méfiant par toute sorte de mésaventure qui nous est arrivé auparavant , on a été voir comment il compte s' y prendre . Moi je vais vous dire comment moi je compte m' y prendre , parce que je vais vous dire pourquoi il y avait un soupçon . J' ai un soupçon ... Je pense qu' il est en train de nous enfumer . Je sais pas pourquoi , j' ai un soupçon . Enfin si , je sais pourquoi , parce que ... J' ai lu dans Médiapart , le 2 novembre 2016 , écoutez vous autres , vous allez reconnaître la langue ... C' est Monsieur macron qui dit : &quot; Je &quot; . C' est déjà ça , il impose pas , il propose ... &quot; Que chacun puisse se responsabiliser &quot; &quot; Et payer en fonction de ses propres risques &quot; Vous avez compris ? L' horreur . Ecoutez -moi bien pour que vous compreniez jusqu' où se machin va , je pense que ce jeune homme sait pas ce qu' il dit , Non mais oui , ils lui ont passé des fiches , il connait rien à tout ça . Il dit oui oui oui , parce que comme il connait que ça la banque , le machin et tout . &quot; Faut se responsabiliser et puis je m' assure contre mes risques , par exemple j' habite au sommet de la Tour Eiffel , je m' assure contre le risque de tomber &quot; Mais là on parle pas de ça mon bonhomme , on parle de la vie à vie , quoi . Qu' est -ce que ça peut bien vouloir dire ? Je vous refais pas le numéro deux fois . Se responsabiliser ... Ecoutez -moi bien les gens , quand vous en entendez un vous dire à la télé ou ailleurs &quot; Il faut se responsabiliser &quot; , il faut que vous compreniez que l' arnaque commence . Rappelez -vous bien de ce mot , &quot; se responsabiliser &quot; , à propos de santé c' est l' arnaque . Personne n' a envie de tomber malade . Personne n' a envie de se gaver de médicament , il peut y arriver qu' on le fasse par erreur , mais c' est pas vrai , c' est pas une envie . On se lève pas le matin en se disant : &quot; Ah , si je pouvais avoir une appendicite du tonnerre de Dieu &quot; Ou bien : &quot; Waw un cancer pour moi &quot; Qu' est -ce que c' est que cette histoire de responsabiliser . Bien . Mais c' est pas tout . Payer en fonction de ses propres risques . C' est 2 horreurs dites en même temps ... C' est quoi votre risque à vous ? Et à vous ? Et à vous ? Je vais vous le dire ... Si vous le savez pas , je vais le savoir à votre place . Vous fumez ? C' est pas bien ... Alors vous fumez , vous vous rendez malade . Et vous venez me demander à moi de vous rembourser . Je veux pas , vous n' avez qu' à pas fumer . Ah , vous faites l' apéro C' est pas sérieux , d'ailleurs vous avez une drôle de tête Quoi ? Vous avez une cirrhose ? Ah zut , mais non non Monsieur , j' ai une cirrhose mais ça n' a rien à voir . M' enfin vous dites ce que vous voulez . Bla bla bla . Mais je l' ai entendu celle-là , je ne l' invente pas ce que je suis en train de vous dire . Ça c' est passé en Angleterre sous Monsieur Tony Blair , qu' ils ont commencé à dire qu' il faut responsabiliser les gens , donc si vous avez un comportement irresponsable , eh bah venez pas vous plaindre et venez pas demander qu' on vous rembourse . Ça c' est pour de l' extérieur à l' intérieur . Puis , après , il y a chacun pour soi . Alors moi je vais dire , &quot; écoutez , j' ai une santé de fer &quot; , ce qui est vrai , j' ai une santé de fer , tout va bien , bon patrimoine génétique , je suis né la bonne année . Eh bah oui Monsieur figurez -vous , eheh 1951 , quand vous achetez une pomme , quand vous achetez une pomme en 1950 , vous mangez une pomme et pour avoir la même quantité de vitamine qu' il y avait dans la pomme que je mangeais quand j' étais petit , vous il faut en manger 10 . Bon appétit . C' est ça la vérité . C' est l' alimentation , c' est comment j' ai été traité , j' ai été soigné , dès tout petit , pour les dents , pour ceci , pour cela , etc . Et voilà comment vous arrivez après , puis j' ai eu la chance de ne pas être dans un métier qui m' a usé le corps comme c' est le cas pour plein de gens . Voilà . Donc si moi je dis : &quot; Bah écoutez , moi , je crois que toute façon , maintenant ça va quoi , j' ai vu ce qui avait à voir , j' espère en voir encore pleins d' autres , mais je m' en plains pas , j' ai pas envie de payer pour vous , là , vous êtes trop jeunes &quot; &quot; Vous allez attraper toutes les maladies , tout ça , pourquoi je paierais ? &quot; C' est ça dire &quot; je vais m' assurer pour mes propres risques &quot; , dire &quot; je m' occupe plus des autres &quot; &quot; Je m' occupe que de moi &quot; . Ça c' est le système de l' assurance . Evidemment l' autre qui lui a des gros risques , &quot; j' ai pas de chance &quot; , d'abord il a fait un travail de chien , il a eu pleins de sortes d' embêtements dans la vie et alors lui , il arrive et son risque ça se voit , rien qu' à le voir Et il va s' assurer , on va dire : &quot; Chacun s' est responsabilisé et paye en fonction de ses risques &quot; , donc si vous avez une tête à risque , vous allez bien payer et vous pourrez pas payer Messieurs , Dames , les gros malins les comptables , vous pourrez pas payer , c' est pas parce que vous inventez ça quand vous avez la quarantaine et que vous crachez le feu , que vous courrez partout , et que vous avez une vie qu' est pas trop épuisante , compte tenu de vos moyens , faut pas que vous croyez que ça va être tout le temps comme ça , parce que tout le monde est malade et tout le monde meurt ! Et 60 % des dépenses de santé , vous les faites dans les derniers temps de votre vie . Alors ça c' est un propos de martien , c' est pas un propos de terrien . Ça ça existe pas ce qu' ils disent . Sauf ... dans une société de violence . La Sécurité Sociale c' est que chacun est responsable de tous les autres comme nous le sommes réellement dans la vie parce que dans la vie nous sommes réellement responsables de tous les autres . Où il est mon papier ... Voilà ... Alors après , vous dites : &quot; Bon d'accord on a compris Monsieur Mélenchon , c' est pas bien et vous , vous payez ça comment ? &quot; Vous voyez comment vous êtes complètement déformé ? Et moi je suis obligé d' apprendre une fiche entière comme ça pour vous répondre ... Mais oui parce que maintenant on connait plus que ça . Vous avez même pas le temps de terminer la phrase qu' on vous enfonce le micro dans la bouche . &quot; Et vous payez ça comment ? &quot; &quot; Comment ? &quot; &quot; Il y en a pour combien ? &quot; Eh ben il en a pour combien , ben je suis content parce que le papé il est vivant , je suis content parce que le gamin on l' a soigné , et combien ça coûte ? Je m' en fous ! Voilà ! Evidemment que je m' en fous pas ... Je suis responsable . Mais qui il leur donne ce droit de nous regarder de haut , en écoutant pour dire après &quot; Oui , comment vous payez ça ? &quot; Parce qu' on y a pas pensé , on est tellement bête . Nous on est le peuple hein , on a pas d' idée , tandis que eux ils sont intelligents , là preuve , il y a qu' à voir comment cette société marche bien depuis qu' ils s' en occupent , c' est à dire depuis toujours , ou presque . Ah ça marche , il y a pas de chômage , tout le monde est content , et ainsi de suite . Alors comment on paye . 100 % , moi je répond Si vous croisez Monsieur macron , demandez -lui , qui est -ce qui paye ? La Sécu ou les complémentaires ? Tout le monde sait ce qu' est une complémentaire ici ? Vous avez la Sécu et pour le reste vous prenez une complémentaire . Ouch ! Ils vous en remboursent encore un petit bout , ah il y en manque encore un petit bout , sur-complémentaire là-dessus Hollande est passé , vous avez vu le geste que j' ai fait , complémentaire , sur-complémentaire , Hollande il est passé : complémentaire , sur-complémentaire , C' est à dire que dans beaucoup de cas , vous êtes moins protégé que vous ne l' étiez avant par la complémentaire que que vous preniez tout seul parce que maintenant vous êtes dans une entreprise , c' est l' entreprise qui négocie le contrat et le contrat qui vous négocie , comme il est mauvais vous avez moins qu' avant . C' est vrai ce que je raconte , vous le savez ? Bon , très bien , mais c' est important qu' on se comprenne parce que ça , c' est des choses tous les jours , et maintenant ... vous avez pris la retraite Monsieur . Qu' est -ce qu' ils vous disent à la complémentaire ? &quot; Ah bah vous êtes en retraite alors vous gagner un peu moins , donc vous payez un peu moins . &quot; C' est ça qu' ils vous disent ? Public : Non . &quot; Vous êtes à la retraite ? Vous payez encore plus ! &quot; On est d'accord , voilà . Alors avant , nous on disait : &quot; C' est nos mutuelles , donc c' est à nous ! &quot; Ça c' était avant . Maintenant grâce à l' Europe qui nous protège , les mutuelles sont devenues des compagnies d' assurance , comme les autres . Ou à peu près , non à peu près parce que des fois c' est moins performant . Non mais c' est la vérité . Moi je connais pas tout le monde ici , je connais même personne , mais je suis sûr j' en prend 3 ou 4 qui étaient à la mutuelle , de telle ou telle branche que je vais pas nommer , et parce qu' avec le syndicat on était d'accord , hein . Et maintenant la mutuelle en question , elle ne fait pas du tout vos affaires , et vous , vous restez fidèle parce que vous pensez que c' est celle du syndicat mais la vérité c' est que tout le système est une arnaque . Donc moi je pense et je vous le dis bien franchement , cet homme Monsieur Macron doit répondre , 100 % remboursable , qui rembourse ? La Sécurité Sociale ou la mutuelle , la complémentaire . Eh ben le résultat sera pas du tout le même parce que s' il vient vous dire : &quot; Je vous rembourse à 100 % et c' est la complémentaire qui va s' en occuper &quot; , qui c' est qui va payer ? C' est vous ! Donc , oui oui vous êtes remboursé à 100 % mais vous crachez plein pot et comme c' est trop cher pour vous , vous vous le payez pas , et comme vous vous le payez pas , parce que la liberté etc . etc . vous êtes pas obligé . Donc c' est un pur mensonge . IL y a qu' une façon de faire du 100 % renouvelable , qui soit sérieux , c' est 100 % par la Sécurité Sociale parce que ça on sait comment ça marche et comment ça rembourse et que nous tous on en a marre de faire des papiers parce que les patrons arrêtent pas de se plaindre qu' on leur fait faire des papiers , et ceci et cela , nous on en a marre de faire des papiers , un coup la Sécu , un coup la complémentaire , la sur-complémentaire , c' est là qu' est déjà , parce que avant je me suis déjà engagé et ça n' a pas changer truc là . Voilà . On connait tous ça ! Donc la Sécu , comment on fait ? Bon là , on rappelle qu' il y a un tiers des français qui renoncent à se soigner , vous avez compris pourquoi . Alors , il y a 2 choses à faire . Premièrement , faut que ça coûte moins cher , à la Sécu . Ça veut dire que , on a une intervention à faire sur le prix du médicament et sur la façon avec lesquels les médicaments sont commercialisés , non seulement comment physiquement ils sont commercialisés que si tu me donnes une boîte pour 1 semaine , c' est pas la peine de me mettre des cachets dedans pour 3 mois . Premier élément mais surtout , deuxième élément : il faut un pôle public du médicament parce que mes ami(e)s , le médicament aujourd'hui c' est la vache à lait de ces gens là , c' est la vache à lait , il y a aucun effort à faire . Tu as trouvé une molécule , un petit coup de ceci , un petit coup de cela , c' est normal que la recherche soit récompensé , c' est normal que ... mais c' est pas normal que ça dure pendant des années et des années et qu' ils continuent à engranger des profits , des bénéfices sur des médicaments dont on peut pas se passer . Et vous savez ceux qui les trouvent les médicaments hein , c' est pas les gros bonnets , pardon de parler vulgairement , un jour je fais une manif avec Sanofi , vous voyez lesquelles c' est , Sanofi c' est une boite formidable , ils ont nommé un PDG américain , flanqué d' un autre qui sont venus tou les deux ils se sont installés un magnifique Q. G à Paris , avenue Foch , avec une belle salle à manger , un chef 4 étoiles , des choses qui importe dans la pharmacie . Et puis , après ils ont commencé à virer les gens , et puis ils ont commencé à étudier ce que les gens faisaient , alors mettons par exemple , tiens j' ai 1 000 euros , je prend un chiffre comme ça , j' ai 1 000 euros , qu' est -ce qu' il fait lui ? Il cherche une molécule pour empêcher vos dents de tomber , il y en a pour combien de temps ? Je sais pas , je cherche . Oui bon ... Et une fois que tu auras trouvé , ça prend combien de temps pour aller sur le marché ? Ah bah les dents ... Ça prend 4 ans . Et toi , qu' est -ce que tu cherches ? &quot; Comment faire que les cheveux blancs devinent pas jaune ? &quot; Combien de temps ça prend ? &quot; Beh écoute , on s' est mis d'accord avec Pepsi-Cola d' un côté , et la savon de Marseille de l' autre , chacun a mit un petit peu , on pense qu' on peut arriver en 2 ans , puis après il nous faut 1 an pour commercialiser ça &quot; Tiens , les cheveux jaunes c' est insupportable , le truc des dents on verra on verra. . D'ailleurs il y a plus d' argent , aller ça va , fais autre chose ... Voilà comment ça se passe . Ce que je vous raconte là c' est les gens de Sanofi qui me l' ont raconté , c' est pas un truc que j' invente . J' étais dans la manif , je marche à côté d' un gars qui ronchonnait et je lui dis : &quot; Alors vous êtes Sanofi , vous ? &quot; Il a un peu les cheveux blancs , je vois bien qu' il est de mon âge , Alors il me dit : &quot; Oui oui , bon maintenant je suis à la retraite &quot; bon , alors &quot; Et qu' est -ce que vous faisiez à Sanofi ? &quot; &quot; Moi j' ai découvert la molécule qui empêche les artères de se boucher &quot; Ce type était là , je me dis : &quot; Mais c' est un Saint cet homme &quot; , faut lui faire une statue , je lui dis &quot; Il y a du monde qui a du être tiré grâce à votre ... &quot; &quot; Oui oui , j' ai &quot; &quot; , il me dit &quot; Je sais pas , peut être 500 000 , 600 000 , peut-être 1 million , j' en sais rien &quot; Ce type avait travaillé et trouvé quelque chose qui a sauvé 1 million de personnes , je lui dis : &quot; Ils vous ont donné quelque chose pour ça ? &quot; , il me regarde et puis , je vois qu' il y a un blanc qui passe comme ça &quot; Je veux dire , vous avez eu ... &quot; , il me dit &quot; Bah oui , j' avais une bonne paye &quot; le gars il a eu une bonne paye , et c' est tout . Et les autres ont fait des millions et des millions et des millions sur sa trouvaille , tant mieux hein , il a sauvé pleins de gens , mais tout ces millions , c' est quoi la justification pour que ça aille dans la poche de Messieurs les actionnaires et pas dans celle des assurés de la Sécurité Sociale dont ce Monsieur qui travaillait et ne pouvait travailler tranquillement que parce qu' on le soignait et quand il était malade il pouvait compter sur la Sécurité Sociale pour être guéri pour retourner chercher , et trouver la molécule qui sauvait les autres . Vous voyez comment tout se tient ? Par conséquent il y a aucune raison pour qu' on admette l' idée que pour trouver et pour chercher il faille qu' il y en a un qui arrive un grand fouet pour vous dire &quot; Allez , il faut être compétitif , flexible , trouver quelque chose qui intéresse les gens qui arrive vite sur le marché et que ça me rapporte des millions &quot; ces millions devraient être à la disposition de tous , donc ... premièrement élément : peser sur le prix du médicament , donc le pôle public du médicament , ça va être long mais qu' est -ce que vous voulez moi j' explique . Qu' est -ce qui vous reste à payer , vous , une fois que vous avez fait la complémentaire , la sur-complémentaire et tout le reste ? Vous en avez pour 16 milliards , il faut donc que je trouve 16 milliards , sans compter la baisse du prix du médicament , pour amener à 100 % payé par la Sécurité Sociale . Comment je m' y prend ? Premièrement , nous supprimons les dépassements d' honoraire chez les médecins . 8 milliards d' euros ! Deuxièmement , faut pas trop qu' ils me cherchent ceux-là , on va dire les choses comme elles sont , ils ont inventé une horreur qui s' appelle le &quot; Numerus Clausus &quot; , résultat il y a pas assez d' étudiant en médecine , que aujourd'hui quand vous voulez faire médecine , vous êtes obligé de faire une école prépa , que c' est vos parents qui vous la payent , alors qu' avant on rentrait directement en 1ère année , maintenant si vous avez pas fait la prépa ... Justement dans laquelle enseigne les types qui ensuite sont dedans qui font les examens , vous ne pouvez pas être médecin . Et cette invention aboutit à ce résultat qui il y a peu de personne capable d' intervenir et notamment dans les spécialités et ils vont tous au même endroit . Voilà pourquoi il y a des déserts dans le pays , et c' est la raison pour laquelle , c' est un petit incident , nous allons faire une médecine publique . Voilà , avec des centres de santé public avec des médecins fonctionnaires . Vous entendez cette horreur ? Fonctionnaire . Des médecins qui sont d'accord et vous avez pleins de jeunes gens qui sont d'accord , vous avez pleins de jeunes gens qui sont d'accord pour avoir cette vie , la preuve , c' est que souvent les jeunes médecins vous en avez pleins qui font des cabinets collectifs , donc ça doit possible , non ? Mais ceux qui veulent pas sont pas obligés , mais veux qui veulent ils pourront . Et à ce moment , on aura à nouveau un maillage du territoire qui fait qu' on pourra prendre les maladies à temps , soigner les gens à temps , avec des toubibs qui font pas une consultations par quart d' heure et où tout le monde passe là dedans ... Il y a beaucoup de toubibs qui font des efforts inouïs , hein , mais c' est absurde tout ça , comment vous êtes mis dans une situation pareille ... Donc les dépassements d' honoraires , non ! Il y en a pour 8 milliards , ensuite , je vous ai parlé des complémentaires tout à l' heure , vous savez que ces gens passent leur temps à nous expliquer que le public ça coûte cher et le privé c' est merveilleux . Résultat : Ça coûte 6 , 5 milliards d' euros de frais de gestion pour les complémentaires . 6,5 milliards , c' est à dire que quand c' est la Sécu ça coûte 8 % en frais de gestion et quand c' est tous les autres c' est à partir de 18 et jusqu' à 25 % de frais de gestion . Autrement dit , ça coûte plus cher que la Sécu les complémentaires et tout ce qu' il y autour . Avec ça ... en ramenant dans la Sécu tout le personnel , mais en ramenant tout le personnel dans la Sécurité Sociale ce qui permettrait de travailler plus tranquillement , ça vous donne 6,5 milliards à rajouter aux 8 que je vous ai dit tout à l' heure nous voici déjà rendu à 14 . Et après , en faisant les comptes , mes camarades découvrent qu' il y a 3,5 milliards d' euros d' exonération de cotisations sociales en faveur des complémentaires santé . Je les reprend . Egal : un peu plus que ce qu' il y a besoin , 17 milliards . Il ne faut pas dire ce n' est pas possible et si quelqu'un conteste mon addition là , je lui propose d' aller visiter un pays lointain et exotique où voir une Sécurité Sociale qui rembourse à 90 % , bien sûr il faut prendre un ou deux avions , 4 ou 5 bateaux , pour finir on y arrive ... La capitale de ce pays étrange , s' appelle Strasbourg , ça se passe en Alsace-Moselle . Localement la Sécurité Sociale du coin rembourse à 90 % et les gens ont 1 point , 1 % de cotisations supplémentaire par rapport à ce que vous avez vous . Eh ben ce % supplémentaire je suis sûr que vous êtes prêt à le payer , si je fais toutes les économies que je viens de vous expliquer . Donc si on peut le faire , je dis à mes ami(e)s d' Alsace-Moselle ... Parce qu' à chaque fois que je parle d' eux je vais abroger le Concordat , il y a toujours un gros malin qui arrive derrière en disant : &quot; Ah oui mais notre droit social il est dedans Mélenchon veut le se supprimer &quot; . Non non ... Mélenchon veut supprimer le Concordat , ça oui , il le supprimera mais le droit local c' est l' inverse , je propose d' étendre à toute la France votre système en l' améliorant un chouia , pour que ça fasse 100 % . Par conséquent , on peut le faire ! Va falloir faire aussi un peu de prévention . Monsieur Macron a dit qu' il était d'accord , fallait faire de la prévention . &quot; Ah non pas vous , ceux qui ont supprimé la médecine du travail , c' est vous , avec la loi El Khomri &quot; , vous le savez ... Qui a supprimé la médecine scolaire ? Mais les gens , si vous ne comprenez pas , vous comprenez ça ? Les petits , si à l' école , vous avez un toubib aujourd'hui pour 11 000 élèves , mais à l' école c' est là qu' on peut regarder comment ça tourne ou pas , on les passe , il faut le temps qu' il faut , il y a toute l' année pour faire la visite et dire attention à ci , à ça , et puis quand un petit jeune à un problème il va pas forcément aller le raconter à ses parents qu' il a un problème , il va voir l' infirmière ou l' infirmier à l' école , on sait tous ça , qu' est -ce qu' on préfère ? Savoir la bêtise ou qu' elle soit réparée ? Bah on préfère qu' elle soit réparée bon sang de bois , on est tous fait du même bois je pense , non ? Donc il faut qu' il y ait cette médecine préventive , il va falloir le faire . Et la politique qu' on propose c' est une politique préventive parce que quand vous dites que vous allez sortir des énergies carbonées grâce à la planification écologique . Eh ben vous faites de la médecine préventive parce qu' il y a moins de gens qui s' empoisonnent avec les produits qui sont liés à la pollution . Quand vous dites que vous faites de l' agriculture bio , vous faites de la médecine préventive parce que vous arrêtez la malbouffe . La malbouffe qui fait qu' aujourd'hui , il y a une telle épidémie d' obésité dans ce pays . C' est pas parce que les gens s' empiffrent , c' est parce qu' ils mangent des choses qui ne sont pas bonnes . L' écologie c' est de la médecine préventive . Et enfin , il y a des défis qu' il faut qu' on relève , vous le savez pas parce que peu de gens vous en parlent mais l' ONU , Qu' est la seule instance internationale que nous pouvons respecter et qu' il faut essayer de suivre , eh bien l' ONU a lancée une alerte , demander à tous les pays de mettre au point une politique de santé de lutte contre la résistance microbienne parce qu' à force de foutre des antibiotiques dans le manger des vaches , des cochons , des poissons , et bien la résistance de l' organisme reste la même tandis que celle des agresseurs viraux , elle augmente . Et donc il va falloir un plan de cette nature , et naturellement si c' est nous qui l' emportons , on le fera . Bon ... Je suis sur la maladie , la prévention , un mot du chômage dont le seul remède est le suivant : le travail . Si vous commencez à partir de l' idée qu' un jour il y aura pas de travail , vous mettez le doigt dans un mauvais engrenage parce que c' est pas vrai . Il y aura toujours une quantité de travail nécessaire . Mon meeting ce soir c' est pas sur le temps de travail , mais il y aura toujours une quantité de travail nécessaire . Si vous voulez que le robot fasse le travail à la place des gens il faut construire le robot et il se construira pas tout seul , ah si , par d' autres robots et à la fin vous croyez réellement que la valeur continuerait à être créer sans travail humain , ça n' existe pas . Mais je met cette question de côté . Moi j' en suis pas à vous dire que je vais faire tout ça et qu' on peut le faire tous ensemble en 5 ans , ce n' est pas vrai . Ce qu' on peut faire c' est relancer l' activité , j' ai déjà dit comment , c' est pas mon sujet ce soir , mais il va nous falloir 3 millions et demi de personnes de plus au travail . Mais je parle du chômage sous un autre angle , sous lequel on en parle pas . Le chômage ça rend malade , et si jamais on le dit ça , le chômage ça rend malade de stress , d' angoisse ... de maladie qui viennent parce que quand vous êtes là à miséré , tourner en rond , vous commencez à plus très bien vous occupez de vous , à plus faire attention à certaine chose , à d' autre . On connait l' addition à la fin , l' addition c' est 18 000 morts à cause du chômage . C' est pas moi qui l' ai inventé , c' est pas mon équipe , c' est l' INSERM qui ont établi ce bilan , 18 000 morts à cause de ça . Le chômage c' est quelque chose qui tue la personne de l' intérieur . Je le dis parce que souvent on oublie ça quand on parle à nos copains qui sont plus au travail . Alors , que vous propose dans cette situation , Monsieur Fillon ? Il a trouvé le remède . Eh bien oui , je vous explique . Vous êtes globalement une bande de tire-au-flanc . C' est bien connu . Par conséquent , Monsieur Fillon propose qu' il y ai une dégressivité de l' indemnisation , c' est à dire plus il y a longtemps que vous êtes au chômage , que je viens de vous dire plus il y a longtemps plus vous êtes malades , plus vous êtes pas bien , justement à ce moment là , on vous enlève des ressources , et pourquoi ? Pour vous obliger à aller au travail où vous vous voulez pas aller . Quelqu'un peut me montrer une personne qui veut pas aller au travail ? C' est ce que je vous ai dit tout à l' heure , ça n' existe pas . C' est des calomnies . C' est pur vilenie de parler comme ça . Peut-être qu' ici il y a des gens ou au travail ou en retraite ou en train d' étudier , mais pensez à celui ou à celle qui n' est pas au travail . Quel effet ça lui fait quand il s' entend dire : &quot; On va faire dégressif parce que vous abusez &quot; , vous abusez ... Eux ils abusent tout le temps , toute la vie , en permanence . Leur seul statut social est un abus . Mais qui abuse ? Ces gens parce qu' ils auraient des indemnités pendant 2 ans , ils veulent les rendre dégressives . Je pense exactement le contraire , non seulement faut pas les rendre dégressives , mais au bout d' un certain temps , tout le monde ait d' une manière ou d' une autre un droit opposable au travail . Alors on me dit : &quot; Comment vous faites une chose pareille , et comment vous le financez ? &quot; A l' heure à laquelle je vous parle tout n' est pas au point dans cette idée , mais ce que je sais , c' est que l' expérimentation a commencé . On dit : &quot; Pas une personne doit être sans au moins comme salaire un SMIC &quot; . Il y a une expérience qui est commencée , grâce à une association qui a fait un travail magnifique , c' est &quot; ATD Quart Monde &quot; peut-être que vous connaissez ce sigle ... Eh bien ils ont montés une activité qui est expérimentale , moi je l' ai vu et j' ai parlé avec des responsable à Châteldon , dans le Puy de Dome , il se trouve que le maire est un de mes ami . Et il y a cette association qui travaille de la manière suivante : elle encaisse ce qu' aurait reçu le chômeur en tant qu' indemnité diverse , et avec ça , ça fait la base qui lui permet d' être , ils appellent ça une association à but d' emploi , c' est elle qui embauche la personne et qui ensuite la place , en recevant cette somme et elle la place en faisant des offres de service , naturellement , il faut discuter avec les petites entreprises du coin pour pas tout fracasser et amener de la main d'oeuvre d' un côté , subventionné , qui tue le travail ordinaire mais on y arrive et la preuve c' est que eux y arrivent . C' est la raison pour laquelle je crois qu' on peut se donner cet objectif , en toute hypothèse . Moi je fais le pari et vous verrez bientôt quand je vous montrerais mes chiffres qui n' est pas encore prêt . Vous verrez que le décollage de l' activité doit nous permettre de pouvoir encaisser ce choc . Mais vous savez que la société a été détruite . Nous devons reconstruire parce que quelqu'un qui est resté longtemps en dehors des circuits , il a du mal a y venir . Un de mes ami m' a raconté , qui est éducateur de rue , il me dit qu' une personne , je prend un cas extrême , qui est resté dans la rue pendant 1 an , il y a beaucoup de monde vous savez dans la rue ... Là déjà depuis le début janvier , il y en a 5 qui sont morts attribués au froid , ça sera + ... A la fin de l' année ça fait 2 000 personnes et il y a des gosses là-dedans , il y a des gosses hein . Tous les ans , qui meurent de froid , d' abandon , dans ce grand pays riche . On le voit moins on m' a dit en province ... Comme on disait la province ... Mais je vous garantie qu' à Paris , il n' y a pas de moyens de ne pas le voir . Les gens sont couchés dans la rue , en bas de chez vous . C' est terrifiant C' est terrible ... Alors , excusez -moi parce que quand je pars là-dedans ça va moins bien après , je passe à la suite allez . Je parlais du chômage qui rend malade et je vous expliquais comment on pouvait faire pour que tout le monde ait son revenu et sa place dans la société . Alors je vous ai dit on va faire un plan de développement de l' économie , ça a tourner , il y a des gens qui vont être embauché , tout ça . Ceux qui seront , qui auront du mal à se remettre en route , on va les accompagner , on va avoir besoin d' eux . Vous savez , j' ai dit 100 fois &quot; faites le prix du malheur &quot; , vous autres les riches , les puissants , vous passez votre temps à pleurer sur les coûts , vous appelez ça des coûts non . Il y a un prix au travail , il y a un prix à la santé , mais faites le prix , faites les coûts du malheur . Combien ça nous coûte ? Cette pauvre personne que vous voyez là , vous êtes certains que vous en aviez pas besoin ? Vous pensez qu' il savait rien faire d' utile pour la société ? Qu' il apportait rien à d' autres ? Qu' il avait pas de famille ? Qu' il avait pas de gens à aimer ? Qu' il était pas aimé lui-même ? Les pauvres nous manque ! Il y en a 9 millions dans ce pays ! 9 millions de personnes qui sont retirées de ce qu' elles pouvaient apporter aux autres . La pauvreté c' est le prix du malheur . C' est des milliers de gens . 9 millions de personnes qui sont mis hors circuit d' une manière ou d' une autre et qui manque à la société . Bon , eh ben on va faire comme on peut mais on va se donner pour objectif que ça cesse ! Nous ne voulons plus vivre notre bonheur personnel au milieu d' un océan de malheur . Nous ne voulons pas ! Alors une autre solution pour qu' il y ait du travail , eh ben , c' est de redonner aux gens le droit de partir à la retraite quand le moment est venu . Ca non plus , il n' y a pas besoin d' être très intelligent pour comprendre qu' en repoussant sans cesse l' âge de la retraite , vous créez quoi ? Des chômeurs de plus , c' est tout ! C' est tout ce que vous faites ! Parce que , je dis vous , ceux qui décidaient ça , comment ils s' appellent déjà ? Celui qui a abrogé la retraite à 60 ans , vous vous en rappelez ? Fillon ! Exactement ! C' est lui qui l' a amenée à 62 ans , et l' autre le revoilà qu' il vient pour vous proposer quoi ? Vous êtes des masochistes ou quoi ? Il vous l' a amené à 62 ans , il vient vous la proposer à 65 ans ... Ce qui veut dire que pour ceux qui ont des carrières incomplètes ou plus exactement qu' ont pas le nombre d' années de cotisations , qui correspondent à cette sauvagerie , ça veut dire que vous attendrez 70 ans pour partir . Tout à l' heure je vous ai donné un âge , je vous ai dit : &quot; l' espérance de vie en bonne santé c' et 63 ans &quot; Ca veut dire que 1 ) vous allez augmenter le nombre de chômeurs , le nombre de malade et de gens que vous allez tuer au travail , parce que c' est ce qui va se passer si vous faites ça . Le travail fera mourir des gens de plus , qui sinon auraient apporté à leur famille , aux siens et à eux-mêmes la part de bonheur qu' ils étaient en droit d' attendre de l' existence et des efforts qu' ils ont accomplis auparavant . Ce sont des brutes et des ignorants qui disent que puisqu'on vit plus longtemps , il faut travailler plus longtemps , mais non on vit plus longtemps parce qu' on travaillait moins longtemps ! Voilà la vérité . Eh ben voilà , ils ont déjà réussi , l' espérance de vie des français a reculé en 2015 pour la première fois depuis 1969 , merci Monsieur Fillon . - 3 mois pour les femmes et - 3 pour les hommes . Pour une fois c' est l' égalité . Voilà , selon l' INSEE , pas l' institut bolchévique . Selon l' INSEE , l' espérance de vie en retraite sans incapacité sévère devrait être stable pour les générations nées entre 1960 et 1990 . Sourire dans le salle ... Sauf ... en cas de poursuite de l' augmentation de la durée de cotisation . Vous êtes dedans donc vous êtes pas dans la bonne cohorte . Nous pouvons faire la retraite à 60 ans et ça ça va aller vite . Nous pouvons faire la retraite à 60 ans avec 40 annuités , vous me croyez pas , je l' entend à la qualité du silence . Vous vous demandez quel lapin je vais sortir de mon chapeau parce qu' évidemment la question que vous allez posé c' est : &quot; Comment on va payer ça ? &quot; Eh bah oui , tout le monde en est là aujourd'hui . Eh bah je vais vous répondre . Vous la connaissez peut-être la réponse et si vous la connaissez pas , vous l' apprenez ce soir et vous la répéterez aux autres . Nous les progressistes , nous pensons la chose suivante : le progrès social fait du progrès économique . C' est ça notre idée fondamentale . On fait pas de la production et de l' économie en partant de produire n' importe quoi , n' importe comment , pourvu que ça soit pas cher . On part de &quot; quoi a -t-on besoin ? &quot; On regarde de quoi on a besoin et on regarde qui c' est qui peut le produire , le public , le privé , etc . etc . Et là c' est pareil . On part de ce qui est nécessaire . Voilà , on a besoin de la retraite à 60 ans . Et on a besoin au bout de 40 annuités , parce que c' est ce qu' on a gagné . me dites pas que le pays a été ruiné par la retraite à 60 ans avec 40 annuités , c' était 39 . Il y avait un problème dans les caisses , oui , il y avait un problème aussi longtemps qu' il y a du chômage . Je me rappelle le nombre de discours que j' ai pu faire au Sénat sur le thème &quot; Mettez 1 million de gens au travail et il y a plus de trous dans la caisse de la Sécu &quot; , eh ben en effet . Eh ben en effet . Ben ça , on l' a vécu , en 2002 , 2001 , 2000 , voilà , c' est comme ça , à cet époque là tous les comptes étaient au vert , comment ? On a eu 2 millions qui sont revenues au travail et parmi ces 2 millions , il y en a , stricto sensu , 355 000 grâce à la réduction du temps de travail . Mais comme ceux-là sont allés bossés , comme ceux-là ont crée de l' activité , ils ont donc crée de l' activité pour les autres . Vous suivez ? Donc au total , ça vous donne 2 millions de personnes . Et comme on a eu 2 millions de personnes au travail et 350 000 postes crée comme ça , eh ben les ami(e)s , c' était la période la plus forte de création d' emploi du siècle ! Et chaque emploi qui était crée là , coûtait en exonération diverse , payé par l' impôt etc . etc . 13 000 euros , c' est à dire 100 fois moins que coûtent ces soit disant emplois que Monsieur Gattaz avait promis et qu' il a jamais fait avec les 40 milliards qu' on lui a donné sans aucune contrepartie . Comment on fait alors ? eh ben je vous répond par une mesure de justice . Ecoutez bien parce que ça permet après à se marrer en société . Je dis ben voilà , on peut payer . Pour ça , faut + de cotisations , tout le monde est d'accord , c' est des maths élémentaires . Alors , où va -t-on trouver les cotisations supplémentaires ? Voilà l' idée , payer les femmes comme les hommes et le surplus de cotisation que ça procurera permet la retraite à 60 ans avec 40 annuités . Alors , ils ont été faire comptes parce qu' ils se sont dit : &quot; Oui Mélenchon il raconte des trucs &quot; non , il est prudent Mélenchon et puis , c' est souvent ils se trompent avec moi , parce que ils croient que comme on est les derniers arrivés , qu' on a pas d' argent , qu' on a pas de grand machins et tout ça , ils pensent que ... mais non ils ont rien compris , je suis entouré d' une foule de gens qui en réalité préparent , travaillent , organisent , et dont je suis le porte parole . Ce sont des bénévoles , mais dites donc les gens , les bénévoles c' est pas des amateurs . C' est ça qu' ils ont pas l' air de savoir , c' est que les bénévoles c' est tout les gens qui font le boulot , la plupart du temps pour les autres parce qu' ils sont payés pour ça , celui qui fait des chiffrages pour le gars de droite ou du PS eh ben des fois dans sa tête , il est avec nous . Eh ben oui , alors qu' est -ce qu' il fait ? Il vient nous le raconter . Il nous explique comment on fait , comment on procède , eh bah oui , et il y en a partout , ahah ! Partout ! Dans tous les métiers ! Les plus improbables ! Dans l' armée , dans la diplomatie , partout , ils se disaient &quot; c' est pas possible &quot; , si , il y en a un , il y en a une , etc . Et donc les chiffres qu' il y a là ils sont en béton armé donc ils ont renoncé à essayer de me prendre en défaut . Alors si , ils ont quand même essayé l' autre matin pour ceux qui suivent la radio , Monsieur Bourdin a dit : &quot; Mais alors vous vous rendez pas compte , ça va être un choc terrible pour l' économie &quot; c' est facile là , je lui dis &quot; Ah bon ? Vous êtes d'accord pour que les femmes soient payés moins que les hommes &quot; &quot; Ah non non je n' ai pas dis ça &quot; Si vous dites ça , vous dites comme moi et voilà , perdu . C' est indéfendable , le contraire . Indéfendable . Alors il y en a un qui l' a dit l' autre soir à la télévision c' est Monsieur Matthieu Croissando parce qu' il avait en face Monsieur Barbier de l' Express , un gars qui assume ses idées et il disait : &quot; Oui mais Mélenchon il parle , il promet des trucs , c' est toujours plus ou moins vrai &quot; il a pas dit en quoi c' était plus ou moins mais voilà , ça sème le doute vous voyez , au fond je suis assez sympathique comme bonhomme , puis bon , à force de répéter tout le temps que je suis méchant , maintenant ça lasse on passe à autre chose . Donc je suis intelligent , cultivé , ils l' ont dit , c' est pas moi hein , mais bon , c' est comment elle a dit Madame , il y a une journaliste s' appelle Madame Ottenheimer elle a dit j' étais Vintage . Alors , ce qu' elle sait pas la Dame , c' est qu' elle est encore plus vintage que moi , parce qu' elle sait pas qu' aujourd'hui vintage c' est vachement à la mode . Alors elle , elle croyait qu' elle allait me démolir mais elle fait l' inverse . Bon enfin bref , et le directeur de l' Obs , c' est quand même le directeur de l' Obs , il dit &quot; Oui m' enfin bon &quot; , il les regarde tous et il dit &quot; bon il y en a un ici qui a envie de dire qu' on paye pas les femmes autant que les hommes ? &quot; Tout le monde ... Terminé . Alors Monsieur Barbier il a du courage parce qu' il a relancé , non mais écoutez quand on a des gens avec qui on peut discuter sur les idées de fond et pas sur mon allure , ma manière de parler et tout ça , ce qui est injuste et blessant Mais sur le fond , alors lui il dit : &quot; C' est pas possible , pour 2 raisons &quot; écoutez ça les filles ça va vous plaire plus que les hommes . &quot; Première raison c' est un choc insupportable pour l' économie &quot; , ah ah , c' est un aveu , si on exploite pas les femmes on peut pas faire tourner l' économie capitaliste correctement ? Premier aveu . Deuxième aveu , vous allez voir , &quot; oui mais si elles sont mieux payées , quand elles vont être à la retraite , elles vont avoir des retraites plus élevées et c' est ruineux &quot; . Voilà , voilà 2 raisons pour lesquelles il est normal de continuer à exploiter les femmes pour qu' elles coûtent pas trop cher à la production et quand elles sont à la retraite , qu' elles nous coûtent pas trop cher en retraire , parce que , parce que c' est comme ça et voilà ! parce qu' on a toujours fait comme ça et on voit pas pourquoi on continuerait pas , on paye moins les femmes , un point c' est tout Monsieur Mélenchon ça suffit . Voilà , là-dessus arrive Madame Le Pen . Alors , j' ai bien réfléchi , on a tout essayé mais je crois que ce qui les embête le plus ça va vous choquer un peu . Ce qui les embête le plus c' est qu' on leur dise qu' ils nous copient . Ça ça les fait rager , parce qu' ils se battent après entre eux , ils sont tous de droite là dedans . Donc quand on leur dit &quot; Vous dites comme Mélenchon &quot; alors ça les met mal et alors à moi ils me disent &quot; Vous dites comme Madame Le Pen , la retraite à 60 ans &quot; normalement si je réagis comme il faudrait je dirais &quot; Moi je n' ai rien à voir avec cette femme , je sais même pas ce qu' elle dit &quot; . Vous voulez la retraite à 60 ans madame Le Pen ? Pourquoi pas , c' est ce qu' on réclame , moi j' ais ça c' était dans le programme commun , il y en a ici qui ont connu le programme commun ? Alors on allait toquer , &quot; toc toc toc &quot; , on allait aux gens expliquait la retraite à 60 ans , alors les plus anciens ils disaient &quot; C' est bien , vous êtes jeune , vous y croyez , ça sera pas pour nous &quot; il faut vous rendre compte que c' était la génération de la Libération et que leur parent à eux sont morts au travail , si je fais l' ordre des générations , mon arrière grand père il est mort à 75 ans , une semaine après avoir arrêté le travail parce qu' on lui a dit &quot; tu peux rester chez toi Barnabé tu n' as plus besoin d' être là &quot; voilà , il est mort après , terminé . Voilà , ça se passait comme ça avant . Donc la génération suivante , elle était bien contente de ce qu' elle avait eu à la libération et quand on leur disait &quot; la retraite à 60 ans &quot; ils disaient comme vous quand je vous raconte 100 % remboursable , vous vous dites &quot; Il est sympa , il y croit , c' est bien , bon ça sera pas pour nous mais bon ça se fera &quot; ben les gens vous me croyez pas , on l' a fait ! D'accord ? Donc c' est pour dire , moi je ne suis dérangé par personne qui dit la retraite à 60 ans avec 40 annuités , je suis d'accord ? Je suis d'accord . Si Madame Le Pen veut faire la retraite à 60 ans avec 40 annuités , elle peut prendre aussi sa carte à la CGT , et alors , ils en veulent pas ! Le problème c' est qu' ils en veulent pas ! Non , enfin , il y a des limites quoi . Ils en veulent pas ... Mais si elle veut , elle a le droit . On va lui donner une adresse , ah non il y en a pas là où elle est , il n' y a pas de CGT . Comment ils l' appellent leur château là , il y a pas de CGT dans le coin . Si ça se trouve les employés qui travaillent chez elle sont à la CGT . Donc ... Moi on me dit : &quot; Vous dites que madame Le Pen et que je me défend plus &quot; sur la retraite à 60 ans et 40 annuités , ils lui ont fait le même numéro à elle , alors là , 3 sauts périlleux arrière parce que ils veulent bien tout ce que l' on veut , mais ça , ceux qui poussent Madame Le Pen dans le dos , ils ont pas inventés tous bastringues pour qu' elle finisse par raconter les mêmes choses que nous . Alors elle a dit : &quot; Je veux être très claire &quot; , en général quand ça commence comme ça , c' est rarement vrai . &quot; la première étape c' est de récréer de l' emploi &quot; En mettant les immigrés dehors et en mettant les autres à la place . &quot; La deuxième étape c' est de stopper et récupérer les dépenses exorbitantes de l' immigration &quot; &quot; Si malgré cela &quot; une fois qu' on a fait les poches au travailleur immigré , une fois que , une fois que , &quot; Il apparaît qu' on ne peut pas maintenir notre système de retraite &quot; , eh , &quot; Maintenir le système de retraite &quot; , &quot; et accorder un départ à 60 ans &quot; &quot; qui a toujours été subordonné dans notre esprit à la condition sine qua none d' avoir 40 ans de cotisations , alors nous nous tournerons vers les français en leur disant la vérité &quot; Tout ça tourner la page , voilà la vérité une fois que rien de tout cela n' aura été fait parce que c' est pas faisable . &quot; Nous avons fait toutes les économies que nous devions faire mais nous avons un système de retraite qui est en danger et fragile , je crois alors que l' État se sera montré exemplaire et que les français &quot; dit Madame Le Pen &quot; accepteront les sacrifices qu' on leur demandera &quot; D'accord ? Vous voilà revenu à la case départ . Elle a commencé par vous dire qu' elle ferait surement la retraite à 60 ans mais qu' à la fin vous vous serrerez la ceinture d' un cran de plus . Point . D'ailleurs , la dame qui s' occupe de ça chez eux , d'abord le communiqué du FN &quot; Dans notre programme il y a pas de retour de la retraite à 60 ans &quot; ça a chauffé entre eux , &quot; et puis quoi à la fin tu vas faire comme Mélenchon , qu' est -ce que c' est que cette histoire ? &quot; &quot; Mais une retraite pleine lorsqu' on aura cotisé 40 ans &quot; dit le Front National , très bien , allez à la CGT . Mais Madame Joëlle Melin , responsable du programme du FN , elle dit &quot; Ah &quot; &quot; Nous aurons du mal &quot; dit -elle , &quot; a rester sur les 40 ans de cotisation &quot; eh ben voilà , c' est qui les ballots ? C' est tous ceux qui les auront cru . &quot; Il faut tenir &quot; vous allez voir l' argument , il est d' une nouveauté , waw , &quot; Nous aurons du mal à rester sur les 40 ans de cotisations car il faut tenir compte &quot; de quoi ? De l' allongement de la vie ! Et voilà ! Le FN et les autres c' est la même chose . Ils sont sur la même ligne politique et quand vous entendrez dire que le FN est pour la retraite à 60 ans , vous rappellerez ce que je viens de vous dire ce soir , je vous avais prévenu ça prend 10 minutes . C' est plus facile de colporter un mensonge que de rétablir une vérité . On le vérifie une fois de plus . Bon , je termine sur un point de tout ça , parce que ça commence être long hein je fatigue , c' est la dépendance , mes ami(e)s , parce que ça ... On va pas pouvoir passer et continuer comme on est là , ça va pas , les gens sont trop maltraités , et il faut que , je vais vous lire la phrase qui est dans notre programme parce que c' est le plus simple , comme ça dure pas 2 heures . Il faut faire face collectivement au vieillissement de la population à la perte de l' autonomie . Il faut construire une prise en charge solidaire de la dépense , et les camarades ont précisé , mes ami(e)s les économistes , financé par une mise à contribution des revenus immobiliers et des successions sur les gros patrimoines . Fussent d'ailleurs des gros patrimoines des personnes en question . Parce que vous savez que des fois , pour garantir des successions , on finit par refuser de payer ce qu' est quand même du à la personne qu' en a besoin . Voilà ce que je dis moi . C' est un gros dossier , Fillon lui dit : &quot; Encourager la généralisation d' une couverture assurantielle dépendance privé sur une base volontaire &quot; autrement dit , vous vous payez votre truc , je vois pas que ça change avec la situation d' aujourd'hui . Quiconque a eu à faire face à ce genre de situation sait que c' est un coût &quot; qui permettront au particulier de déduire de leurs impôts &quot; , tu parles , les malheureux , &quot; une part de leur cotisation à une assurance dépendance &quot; , moi je suis contre ça . Une nouvelle fois on revient sur celui qui peut s' assurer au bon niveau , il a le droit a une prestation , celui qui peut pas ben il reste là où il est dans la situation dans laquelle il est . On peut pas être d'accord avec ça . Mais je vais vous dire une autre chose , parce que ça vous comprenez , c' est pas la peine que j' en parler 2 heures . Je voudrais vous mettre en garde contre une autre idée , parce que j' ai vu ça et ça m' a pas plu . Il y a des gens qui disent &quot; Eh ben voilà , alors , pour aider les personnes qui en ont besoin il faut faire de la traçabilité lumineuse dans les appartement et puis il faut mettre une alarme , et puis il faut mettre une vidéo surveillance pour les aider &quot; Et au bout d' un moment j' écoutais ça , je me disais &quot; qu' est -ce qu' il raconte ? &quot; Quand on est dépendant , on a pas envie de parler avec une caméra ni voir le parterre s' illuminait entre votre lit et les toilettes Ni avoir un machin qui se met en route tout seul . C' est pas ça le sujet . Qui conçoit , on a besoin des autres . Le premier des soins c' est celui qui donne les soins , ou celle qui le donne , et donc , moi je vous met en garde mes ami(e)s d' une façon générale contre l' illusion technocratique . Non , la vie des êtres humains ça sera une vie d' être humain et pendant qu' on y est , je vais vous dire une deuxième chose , figurez -vous que les gens qui aujourd'hui s' occupent des personnes dépendantes , ça peut être pour une dépendance provisoire , c' est pas tout le temps , le personnel qui fait ça , c' est à dire les gens qui viennent à domicile , ils ont souvent des contrats de travail , mais surtout ... c' est un personnel qui est souvent féminin , eh ben figurez -vous que quand on regarde les accidents du travail , il y a une espèce de course avec des moments qui changent , tantôt c' est le bâtiment qui arrive en 1 pour les accidents du travail , tantôt c' est qui ? Ce personnel là , et 80 à 90 % des problèmes physiques que rencontrent les gens qui font ce travail , est lié à la manutention du corps des personnes qui sont dépendantes . Par conséquent , nous avons , non seulement à régler le problème de la façon avec laquelle on prend en charge , au plan financier , mais nous avons aussi à réfléchir et à trouver les moyens et à nous dire qu' on ne fera pas d' économie là-dessus . Ça existe pas faire de l' économie sur l' attention qu' on doit aux autres . Parce que l' attention qu' on aura pas eu pour la personne concernée , on l' aura pas eu pour la personne qui s' en occupe et donc ça nous fera 2 malheureux au lieu d' 1 . Par conséquent , il y a besoin que les conditions de travail de ces personnels soient pris au sérieux , ces gens ne sont pas seulement là , comme se le figure certain , pour manutentionner . Il y a toute une disponibilité qu' ils doivent avoir et pour ça changer leurs conditions de travail . Il faut que ... J' en aurais des choses à dire mais il faut que j' aille à ma conclusion . * Une personne dans le public parle à Jean-Luc JLM : Attends c' est la Sécu ce soir , la Sécu intégrale , le logement ça sera une autre fois , d'accord ... Je fais ce que je peux ... Je vous met en garde contre l' illusion technocratique , je vous l' ai dis il y a un instant . Je vous dois encore quelques mots . Vous savez ce que c' est que la DMS ? Vous savez pas ? La Durée Moyenne de Séjour , c' est de ça dont je vous parle maintenant . Comment un système devient fou , cruel et anti-humain ? La Durée Moyenne de Séjour à l' hôpital . Vous là , fémur cassé ? 20 jours . Le 18ème jour le personnel s' affole , &quot; Mais Monsieur Machin &quot; il a le col du fémur pas en état , il peut pas ... faut qu' on le garde , c' est pas possible ! Durée Moyenne de Séjour 20 jours , 20 jours arrive , dehors . Le personnel de santé qu' est là &quot; Qu' est -ce qu' on fait ? C' est inhumain , on va pas faire ça , donc il faut qu' on le mette dans un centre de repos ou de soin &quot; on téléphone , il n' y a pas de place bien évidemment , puis après on dit quelqu'un dit &quot; bon d'accord , on va faire un effort mais il est autonome ? &quot; Oui oui &quot; c' est pas vrai , et j' ai une amie vous pouvez la regarder , elle fait des petites vidéos qui sont géniales , s' appelle Sabrina Aurora et elle raconte comment le personnel de santé se raconte des histoires les uns aux autres pour que les gens se passent les gens , parce que ce personnel de santé il est de bonne volonté , il veut pas laisser quelqu'un dans le malheur , parce que vous l' emmenez et puis quoi , ça s' appelle le &quot; virage ambulatoire &quot; , ces technocrates ont toujours toutes sortes de mots , le virage ambulatoire c' est &quot; vous sortez de l' hosto , vous restez chez vous &quot; c' est sympa chez moi , sauf si c' est au 6ème étage et que j' ai une patte cassée et qu' il y a pas d' ascenseur . Là c' est plus sympa du tout parce que ça veut dire que je vais être seul et misérable . Et ainsi de suite . Voilà ce que c' est et puis la tarification à l' acte , vous en avez entendu parler ? Le piège total , vous arrivez vous êtes une chose . &quot; Vous avez quoi vous ? Telle maladie , pas telle autre je m' en occupe pas , c' est pas mon sujet &quot; poum poum poum , il y a ça qui est prévu . Si vous en faites moins , tire-au-flanc ! Si vous en faites trop il y a de l' abus , vous en faites moins plouf ! On vous supprime des lits . Il y a donc toute une organisation de la pensée qui est liée au soin , au rapport à la personne , à partir d' une vision technocratique et comptable qui défigure la société et rend malade tout le monde y compris ceux qui s' occupent des malades . Et au fond , c' est ça notre sujet , quand on fait une réunion comme celle -ci , c' est à ça qu' on doit réfléchir . C' est la société qu' on veut . Où c' est sauvé mon dernier papier ? Bon , tant pis , va falloir improviser ... Non pas tout à fait ... On va bien vous saouler dans les temps qui vont venir , vous allez voir les têtes d' œuf qui vont revenir . &quot; Les dépenses obligatoires sont trop élevées &quot; Bon , &quot; nous allons le comparer à la Finlande , l' Islande &quot; enfin ils sont 1/10ème moins nombreux que nous &quot; mais c' est pas grave c' est pour donner une idée &quot; OK . Bref . Tout ces gens vous parlent de prélèvements obligatoires et ils comparent des choses qu' ils n' ont rien à voir les unes avec les autres . Je m' en tiens seulement à la Sécu . Chaque fois que vous entendrez dire que les prélèvements obligatoires sont plus importants chez nous qu' ailleurs , il faut que vous sachiez qu' ailleurs n' a pas la même organisation que nous et que chez nous 40 % de ce qui s' appelle &quot; les prélèvements obligatoires &quot; sont liés à des dépenses sociales de santé et de ce qui va avec . Tandis que dans tous les autres pays , la moyenne c' est 26 % . Autrement dit ils ne comptent pas les dépenses obligatoires assuré par le privé parce que si vous vous cassez une patte , que vous soyez chez nous ou ailleurs , vous vous faites soigné mais si vous vous faites soigné ici , c' est compté dans les dépenses obligatoires tandis que si vous vous faites soigné là-bas , c' est compté dans votre fantaisie et ça rentre pas dans les comptes publics . Résultat , ça vous donne un niveau de prélèvement qui est supérieur ou égal aux États-Unis , à ce que nous avons nous , pour des dépenses de santé qui reste à charge de beaucoup plus considérables , le double ou le triple de ce qui se passe chez nous . Ne vous laissez pas entortiller par les chiffres , ce n' est pas vrai que nous dépensons trop , que la France vit au-dessus de ses moyens . Il y a un sujet sur lequel en effet la France vit au-dessus de ses moyens , c' est que ça classe dominante sociale coûte abominablement cher et que nous pourrions trouver des gens qui nous coûtent beaucoup moins cher pour faire la même chose ! Oui , nous vivons au-dessus de nos moyens avec une oligarchie pareil recors d' Europe de distribution des dividendes alors qu' on a besoin d' investissement massif dans le pays . J' attend qu' on vienne me démontrer le contraire , j' attend qu' au lieu de venir parler de mon style oratoire , de ma cravate , de mon âge , de mes dents ou que sais -je , on vienne me montrer que j' ai tord , que ce n' est pas vrai . Parce que c' est vrai ! Parce que c' est une oligarchie toute puissante qui est un parasite sur ce système et quand je dis qu' il ne doit pas y avoir d' écart de salaire supérieur de 1 à 20 c' est à dire que quand Monsieur Gohsn veut s' enrichir sans rien faire de 6 millions d' euros supplémentaires , eh ben il va falloir qu' il en donne autant à tous ceux qui sont en bas de la hiérarchie parce que c' est ça un écart de 1 à 20 et 1 à 20 c' est beaucoup ! C' est beaucoup beaucoup ! Cette revendication , c' est à nous de la prendre , c' est celle de la Confédération Européenne des Syndicats . La société française , comme toutes les autres , comme toute la civilisation humaine , au delà de tout ce que l' on vient de se raconter , au fond quoi , 2 visions de la vie , pour les uns ... c' est pas des brutes , on est pas non plus dans une ambiance de guerre civile , ceux qui sont pas , qu' on pas le cœur comme nous , qui voit pas les choses comme nous , c' est pas non plus des sauvages , eux aussi ont des familles . Quelle est leur idée ? Leur idée c' est que dans la vie , ce qu' est le moteur de la société c' est la compétition , voilà c' est ça leur idée fondamentale . Alors ça fait pas d' eux des brutes , si pour beaucoup oui , mais bon , pas tous , et ils disent &quot; la compétition , nous on veut bien la solidarité , c' est pas un problème pour nous . On est d'accord pour être solidaire , on est bien élevé &quot; &quot; Mais pas si ça met en cause la compétition &quot; donc la solidarité , oui , jusqu' au point où commence la compétition . Ça c' est la vision des libéraux . Toute la bande là , Macron , Fillon , tout ça . Ils voient la vie comme ça . Et puis nous , on vient de plus loin , de beaucoup plus loin , nous , on se dit &quot; boupf , on est là , on est les enfants desquels ? &quot; si vous pensez à l' histoire profonde de l' humanité . Les enfants desquels ? De celui ou de celle qui s' occupait de son voisin , de celui ou de celle qui s' occupait du plus faible , du blessé qui le soignait , de la femme qui allait mettre l' enfant au monde qu' on protégeait pour pas qu' elle fasse l' objet d' une agression à ce moment là , les petits dont on s' occupe , ceux là ils ont survécus , les autres qu' en avaient rien à foutre , qui s' occupaient de personne , que d' eux mêmes , bah ils ont disparu plus vite que les autres . Et eux , les enfants , il y en a plus . Enfin , on pouvait penser ... Nous , nous sommes les descendants des gentils , des doux , des solidaires , des préoccupés des autres , qui bercent , qui chantent , qui dansent pour mettre une ambiance , pour réconforter , pour rassurer le groupe humain . J' exagère ? Oui , un peu , pas trop . C' est ça le fond qui nous sépare , et maintenant , dans la vie de la vie , ça peut être pas grave , quand tout allait bien , pour tout le monde . Mais maintenant , ça va mal pour un nombre de gens croissant parce que le système est devenu fou , eux-mêmes ne le contrôle plus , plus personne ne le contrôle . L' énorme accumulation du capital qui se fait dans la sphère financière , plus personne ne le contrôle . La folie de la cupidité de la puissance dominante , plus personne ne la contrôle vous avez vu cette histoire avec Trump ? C' est un truc entre eux là , c' est très dangereux , ils sont en train de se battre entre eux , les services de sécurité et le reste , les uns veulent faire la guerre à la Russie , les autres à la Chine , c' est des fous ! Et ils sont en train d' essayer , j' sais pas faire quoi , à leur prochain président , c' est très dangereux tout ça . Ça c' est la guerre , mais c' est pas de ça sur quoi je veux terminer . Nous tous , les êtres humains ... Quelque soit notre religion , notre couleur de peau , notre âge , ce que nous avons prévu , nous sommes confronté au changement climatique . Mes ami(e)s , il est commencé , ne vous demandez pas si on va pouvoir l' arrêter , ce n' est pas possible . Ce n' est plus possible . Les engagements de la COP 21 , déjà qui ne seront pas tenus , prévoient 2 degrés de plus , ils ont mis un degré d' augmentation pour faire plaisir aux îles qui pouvaient pas signer parce que si on mettait pas 1 et demi , ça voulait dire qu' ils signaient qu' 'ils étaient d'accord pour être noyé . On a mit 1 et demi , mais quand on fait le total de ce que chaque État a prévu de faire , ça fait 3 degré et demi d' augmentation de la température . C' est intenable . Le climat va donc changer , les courants froids et chauds dans les mers changent . En décembre dernier , la société de ... géologie , je crois , internationale , a donné un nouveau nom à la période dans laquelle nous vivons , vous vous rappelez peut-être de l' école , le pléistocène , le tertiaire , le quaternaire etc . Bon d'accord ? Ben l' ère dans laquelle nous sommes , le moment dans lequel nous sommes , ils l' appellent &quot; L' anthropocène &quot; , c' est à dire la période où l' être humain change la nature . L' être humain , cette petite créature , sans armes , sans griffes , l' être humain charrie plus de gravats que toutes les forces de la nature . Les fleuves , les rivières , le vent et ainsi de suite . Les traces sont incontestables , on les verra pendant 10 000 ans , 20 000 ans , 50 00 ans , 1 millions d' années il y aura encore la trace de ce que nous avons fait . De choses qui n' existaient pas et qui existent depuis le plastique , des choses qui n' existaient pas et qui existent depuis le combustible nucléaire et les poussières nucléaires qui se sont répandues sur la totalité de la surface de la Terre . Maintenant vous savez que tout le destin de l' humanité est dans les mains de l' humanité , ce à quoi nous devons nous préparer , nous , c' est à affronter cette situation . Comment allons -nous faire ? Chacun pour soi ? Ou tous ensemble ? C' est la vieille leçon , la première leçon que devait tirer les premiers groupes d' êtres humains , pour survivre . Qu' est -ce qui compte ? Moi d'abord ? Ou tous les autres avec moi ? Cette grande question qui partage les tenants de la compétition tout le temps et les tenants de la solidarité revient devant la scène . Vous ne pouvez pas être réellement écologiste et vous l' êtes tous . C' est pas une carte de parti écologiste , c' est une conscience . Nous ne sommes pas , si nous ne tranchons pas cette question là , il n' y a pas de capitalisme vert . Le problème du capitalisme vert c' est la couleur . C' est que le capitalisme ne peut pas être vert . Ils ne peuvent pas faire l' accumulation s' ils refusent l' investissement pour changer les machines , pour changer la production , pour changer le sort des travailleurs qui travaillent derrière ces machines . C' est ce défi qu' on va devoir relever . Alors moi je vous dis , si nous la France nous changeons de cap , si juste nous faisons ça , une constituante et ensuite la planification écologique . Ce que nous allons faire , nous les français , parce que nous sommes une grande puissance technicienne , je suis pas en train de faire du nationalisme , c' est pas ça le sujet , nous sommes une grande puissance technique , et vous savez pourquoi ? Parce que nous avons eu une bonne école républicaine pour tous , parce que nous avons enseigné à tout le monde , y compris les gosses d' étrangers . Parce qu' on a fait ce choix là , parce qu' on a bien traité les gens , mieux qu' ailleurs , on a pu accumulé ce qui compte , la plus grande et meilleure des richesses , le savoir , l' intelligence . C' est une richesse qui , plus vous la partager , plus il y en a , au contraire de toutes les autres richesses . Le savoir , qui va nous aider à répondre et parce que nous sommes cette nation là , nous avons une responsabilité à l' égard du reste de l' humanité . En pensant à ces milliards d' êtres humains qui n' ont pas accès à l' eau potable , qui n' ont pas accès à l' énergie , nous pouvons , nous , parce que nous connaissons les process de production pour le faire , mettre en partage la science et la technique qui permettra à tout ces peuples d' avoir ce dont ils ont besoins , en rompant avec l' ancien modèle d' accumulation , parce que s' ils font tous comme nous , il y a plus de vie possible sur la Terre . Et donc pour que la vie continue , il faudrait que mécaniquement on continue à les exploiter , qu' on continue à les rabaisser , parce que s' ils se mettent au même niveau que nous pour avoir le même nombre de voiture par habitant , le même nombre de produit qui servent à rien , la planète va mourir . Voilà pourquoi il y a un lien entre l' idée de sauver la civilisation humaine , le paradigme écologique , l' idée d' avoir cet objectif et la République qui consiste à dire &quot; nous faisons ce qui est bon pour tous &quot; . Comme nous sommes cette nation , nous pouvons rompre le cercle infernal dans lequel nous sommes , vous avez vu quel président ils ont élus aux États-Unis d' Amérique ? Il croit que le réchauffement climatique est une invention des chinois . Et ainsi de suite , tout ces gens ne s' occupent de rien , les glaciers sont en train de fondre , vous savez de quoi ils s' occupent ? De savoir à qui appartient le sol qu' il y a en dessous . Voilà pourquoi les uns et les autres , les russes , les américains , les danois , tout le monde avancent les navires de guerre , pour dire &quot; ça c' est à moi &quot; Autrement dit toutes les conditions se réunissent , d' une bataille du type du XIXe siècle pour l' accès aux matières premières et aux territoires alors qu' on est au XXIe siècle et devant un péril majeur . C' est pour quoi , nous serons cette puissance de paix et parce que nous ferons ça , je vous dis , parce que nous ferons ça , ce sera un déclic . Un événement , tel qu' on peut dire le siècle comment sera en 2017 , parce qu' en 2017 , une des principales nations du monde aura rompu avec l' ancien système et se sera orienté vers les nouveaux horizons nécessaires à l' humanité toute entière . Exactement ! Voilà à quoi je voudrais passionnément vous servir , je suis votre porte parole , je le vois puisque vous applaudissez ce que je viens de dire . Vous savez comme moi que c' est le plus urgent . C' est à ça que vous voulez qu' on travaille . Vous ne voulez pas perdre votre temps à des querelles secondaires pour éliminer ou choisir entre des individus qui racontent la même chose et qui de toutes façons sont incapables d' imaginer un autre monde . Vous savez les périodes révolutionnaires , moi je plaide pour la révolution citoyenne , elles sont caractérisées par ça , les élites ne croient pas qu' un autre monde soit possible . Ils n' arrivent même pas à l' envisager . Ce ballot de Louis XVI n' arrivait même pas à s' imaginer ce que ça pouvait être un gouvernement du peuple . Ça lui venait pas à l' esprit . Il n' avait jamais connu une chose pareille . Il croyait qu' il y avait des rois , des évêques , et des machins ? tout ça pour maintenir le système en place jusqu' à que ce que tout explose . Ne comptez pas sur leur prise de conscience . Ne croyez pas qu' ils vont comprendre un jour ou l' autre , ils ne comprendront jamais ! Ils ne peuvent pas ! Leurs intérêts , la logique dans laquelle ils sont leur interdit de comprendre qu' il faut passer à autre chose . Que les biens communs sont plus important que les biens particuliers , que l' intérêt général humain est plus important que les intérêts particuliers . Ça il y a que nous qui pouvons le faire , c' est pourquoi je vous dis on commencerait le siècle de cette manière . J' allais vous dire , en pensant à d' autres événements qui se sont passés dans le passé , que quand un engrenage mortel se met en route , rappelez -vous ce qui s' est passé avec la première Guerre Mondiale , seule une intervention populaire rompant avec tout a permis que ça s' arrête parce qu' ils ont eu tellement peur de voir les soldats qui voulaient plus monter et sortir de la tranchée et tirer sur les autres parce qu' ils ont eu tellement peur quand ils ont commencé à voir qu' il fallait les fusiller pour les obliger à obéir parce qu' ils ont eu tellement peur en voyant que les russes étaient soustrait à ce système abominable , qu' ils ont été obligé d' arrêter , quelqu'un a fini par capituler . Eh bien nous , il faut qu' on leur fasse tellement peur en prenant tranquillement , par la démocratie , par les urnes , une décision politique majeure . Nous allons changer le monde et nous allons le faire en changeant notre pays , voilà par quel bout on prend le problème . Et c' est pas trop pour nous , c' est pas exagéré parce que le jour où nous le ferons , vous verrez que des milliards d' êtres humains se tourneront aussi vers nous et regarderont comment nous sommes entrain de faire parce que eux aussi ils veulent pas qu' on dégueulasse toute la planète Eux aussi ils veulent que leurs enfants aient droit à la beauté d' un matin qui se lève , d' une eau qui coule dans la rivière , d' une écrevisse qui s' y trouve , parce que toute vie n' a pas été détruite ! Voilà , bon pour finir , parce que vraiment il faut finir ... Je vous ai parlé de Sécurité Sociale , je vous ai présenté un modèle de la Sécurité Sociale intégrale , vous avez le droit de vous poser des questions , d' en poser , la plateforme est ouverte pour ça , vous serez les bienvenus . Nos ami(e)s travaillent beaucoup , parce que vous savez on exige toujours plus de nous que de n' importe qui d' autre hein , mais le fond va pas le perdre . On parle de Sécu . Et ses petits droits de la vie . Alors , j' ai parlé de solidarité et Jaurès apporte une touche de plus , je le dis avec ses mots , parce que moi j' ai assez dit des choses , des grandes choses auxquelles je croyais et je crois ? écoutez ça : &quot; Entre l' assurance et l' assistance , même littéralement organisé , il y a un abîme . l' assisté , même quand la loi lui donne ce qu' il appelle un &quot; droit &quot; est obligé de plaider pour avoir la réalisation de ce droit &quot; ça se passe en 1910 , j' ai choisi ce discours je sais pourquoi , 1910 c' est les premières retraites ouvrières à 65 ans , alors Jaurès dit &quot; les plus durs dans le mouvement de l' époque disait &quot; ah va pas signer un truc pareil 65 ans , on va cotiser pour les morts &quot; on disait à l' époque et Jaurès dit &quot; non non faut qu' on accepte ça , on l' améliorera &quot; c' était la retraite à 65 ans , vous entendez les gens ? Ce que vous propose Monsieur Fillon c' est de revenir ce qu' on avait à l' époque du Grand Jaurès , bon , 1910 &quot; Il faut que le vieillard de 70 ans démontre qu' il est indigent et son indigence est évaluée , arbitrairement par les pouvoirs et puis il faut que l' individu quémande et dans son attitude d' assisté , il sent encore pesé sur ses épaules , courbés par le travail , le poids de la servitude sociale &quot; &quot; Au contraire , l' assuré a un plein droit . Un droit absolu , un droit inconditionnel , son titre est là aussi certain que l' est pour les bourgeois le titre de rente . A l' âge fixé par la loi , il a le droit d' exiger , il exige , il reçoit un certain chiffre et il ne reçoit pas seulement un recours matériel &quot; Il reçoit un secours de dignité morale . Magnifique idée d' assurance sociale qui crée pour tous les salariés un droit tangible sans humiliation &quot; . Merci ! "

    ]

fillon = [
    "Mesdames messieurs , mes amis , La primaire de la droite et du centre est close et les électeurs doivent être salués pour s' être emparés massivement et librement de ce rendez-vous démocratique . La victoire me revient ; c' est une victoire de fond bâtie sur des convictions . Depuis 3 ans , je trace ma route , à l' écoute des Français , avec mon projet , avec mes valeurs , et progressivement , j' ai senti cette vague qui a brisé tous les scénarios écrits d' avance . Ma démarche a été comprise : la France ne supporte pas son décrochage . Elle veut la vérité et elle veut des actes . Ce quinquennat qui s' achève a été pathétique . Il va falloir y mettre un terme , et repartir de l' avant comme jamais nous ne l' avons fait depuis 30 ans . Pour cela , j' aurai besoin de tout le monde . Ce soir , j' ai une pensée particulière pour Nicolas Sarkozy J' adresse à Alain Juppé un message d' amitié , d' estime , de respect . Alain est un homme d' Etat et il le reste . Aucun candidat n' a démérité . Cette primaire fut digne . Pas un électeur ne doit se sentir humilié ou mis à l' écart . Ce qui nous unit est plus important que ce qui nous distingue . Voilà pourquoi , je tends la main à tous ceux qui veulent servir notre pays . Mon succès , je le mets en partage . Je le dois à mes équipes , à mes soutiens , à tous les bénévoles qui furent admirables de dévouement . Ce succès , je le dois à ces milliers de Français que j' ai rencontrés et qui m' ont nourri de leur révolte , de leurs idées , de leurs espoirs . Je me suis senti porté par leur désir de liberté et de réussite . Il y a dans notre pays un immense besoin de respect et de fierté . Il y a aussi un appel à l' autorité de l' Etat et à l' exemplarité de ceux qui le dirigent . Les électeurs de la droite et du centre ont trouvé dans ma démarche les valeurs françaises auxquelles ils sont attachés . Ces valeurs , je les défendrai et nous les partagerons avec tous ceux qui , dans leurs différences , aiment la France . Personne ne devra se sentir exclu d' une société que je veux plus juste et plus solidaire . Notre renaissance sera l' œuvre de tous . Elle viendra de notre volonté de nous battre ensemble pour le bien commun et pour que nos enfants soient heureux d' être Français . Je sais les défis , mais je suis porté par la confiance que fait naître en moi cette incroyable aventure d' un pays qui par son travail et sa bravoure a réussi à se placer parmi les cinq plus grandes puissances du monde . L' avenir nous attend , et nous avons en mains tous les atouts pour être une Nation souveraine et moderne , en tête de l' Europe . Dès demain , l' essentiel commence ! J' ai maintenant le devoir de convaincre tout un pays que notre projet est le seul qui puisse nous hisser vers le haut , pour l' emploi , pour la croissance , pour battre ces fanatiques qui nous ont déclaré la guerre . J' ai le devoir de redonner confiance aux Françaises et aux Français . J' en suis certain , ils sont des millions prêts à s' élancer . Je ne vois dans leurs yeux , ni lâcheté , ni résignation , mais une force qui attend son heure , et cette force sera extraordinaire si nous la guidons vers des changements inédits . J' ai le devoir de vaincre l' immobilisme et la démagogie . La gauche , c' est l' échec ; l' extrême droite , c' est la faillite . Je parle de vaincre ces partis politiques , pas de vaincre ces Français déçus , ces électeurs qui ne sont pas aujourd'hui des nôtres mais qu' il me revient aussi d' entrainer vers l' avenir . François Hollande a abaissé la fonction présidentielle : il faudra la restaurer . Si le peuple français me fait confiance , j' aurai le devoir de respecter le contrat noué avec lui et de tenir ma charge avec dignité . Je mesure la gravité de la situation et les attentes des mes compatriotes . Je vais aller à leur rencontre tout au long des prochains mois . Je relèverai avec eux un défi original en France : celui de la vérité et celui d' un changement de logiciel complet . Je donne rendez-vous à tous ceux qui savent que le bonheur est une conquête . Je donne rendez-vous à tous ceux qui ont dans le cœur la fierté d' être français . Mon ambition est de hisser la nation vers le meilleur d' elle-même , et je vous le certifie , si en 2017 nous prenons fermement les choses en main , notre pays ira loin car rien n' arrête un peuple qui se dresse pour son avenir . Avec ces primaires , une espérance est née . Ma mission est de la faire grandir pour la France et pour la République . ",
    "Je suis engagé dans le débat présidentiel avec un projet puissant , fondé sur la vérité et un effort responsable pour redresser notre pays . Depuis deux semaines , mes propositions font l' objet d' attaques de la gauche et de l' extrême droite . Elles ne m' impressionnent pas . Ce tir de barrage était prévisible puisque j' engage le fer contre le cortège des conservateurs et des mystificateurs . Mes adversaires s' érigent en défenseurs de notre &laquo; &raquo; mais ils en sont , en réalité , ses fossoyeurs . Ils refusent de voir que ce modèle craque de toutes parts , que le chômage , la pauvreté et les injustices se propagent , que les déficits menacent la solidarité nationale . Les critiques se sont récemment concentrées sur la santé . Quel est le diagnostic ? Notre système de soins est menacé et la situation s' est dégradée ces dernières années . Les Français sont inquiets , à juste titre : des heures d' attente éprouvantes aux urgences , plusieurs mois pour obtenir un rendez-vous chez un spécialiste , des déserts médicaux qui s' étendent . Les professionnels de santé sont exaspérés par la bureaucratisation de leur métier . Le gouvernement a opposé l' exercice libéral et le système public au lieu de les faire fonctionner de façon complémentaire . A l' hôpital , les 35 heures ont fait des dégâts . Le trou de la sécu n' est pas comblé contrairement à ce que prétend la ministre . La dette se monte à 160 milliards d' euros … Bref , au lieu de regarder les faits , mes détracteurs me soupçonnent de vouloir &laquo; privatiser &raquo; l' assurance maladie et diminuer les remboursements . C' est évidemment faux ! Au passage , la gauche préfère oublier que le poids des mutuelles et des assurances personnelles n' a cessé d' augmenter depuis des décennies , y compris dans le cadre des réformes qu' elle a elle-même votées . Les modalités de la &laquo; des complémentaires santé aux seuls salariés ont conduit , dans de nombreux cas , à diminuer les garanties offertes ( frais d' optique , soins dentaires , consultations de spécialistes et opérations chirurgicales moins bien remboursées ) et à une augmentation des cotisations . Les assurés ne savent plus ce à quoi ils ont droit et s' interrogent sur les garanties effectives dont ils disposent . Opacité et manque de transparence sont devenus la règle . Mais de tout cela , la gauche , main sur le cœur , ne veut pas entendre parler . Moi , je dis les choses telles qu' elles sont ! Oui , la situation de notre système de santé est inquiétante et ceux qui prétendent le contraire sont dans le déni et la démagogie . Mon objectif est de sauver notre système de santé qui fut l' un des meilleurs au monde et qui doit le redevenir . J' entends réaffirmer le principe d' universalité dans l' esprit des ordonnances de 1945 . L' Assurance maladie obligatoire et universelle , pilier de la solidarité , doit rester le pivot dans le parcours de soin dont le médecin généraliste est l' acteur clé . Elle continuera à couvrir les soins comme aujourd'hui et même , mieux rembourser des soins qui sont largement à la charge des assurés , comme les soins optiques et dentaires . Il n' est donc pas question de toucher à l' Assurance maladie et encore moins de la privatiser . Je souhaite assurer le meilleur remboursement pour les patients en plaçant l' Assurance maladie obligatoire et les organismes complémentaires sous le pilotage d' une agence de régulation et de contrôle dans laquelle seront représentés tous les acteurs du système de santé ainsi que les usagers . Il s' agit d' améliorer la gestion de notre système de protection sociale . J' entends faire en sorte que tous les Français puissent bénéficier d' une protection complémentaire appropriée sur la base de contrats homogènes . Il convient , par ailleurs , de responsabiliser chacun en simplifiant les nombreux dispositifs de franchise actuels et le ticket modérateur qui sont un maquis incompréhensible . Cette responsabilisation passe notamment par l' abandon de la généralisation du tiers payant . Assez de bureaucratie et assez de mensonge : la santé a un coût et au bout du compte , d' une manière ou d' une autre , c' est toujours l' assuré qui paye . Je veux réaffirmer l' équilibre de notre système de santé : médecine publique et médecine libérale doivent agir de concert , dans un climat de confiance . Le patient doit pouvoir choisir librement son établissement de soins , son professionnel de santé , ce dernier devant rester maitre de ses prescriptions . J' entends garantir un égal accès aux soins pour tous en enrayant la désertification médicale , grâce à la multiplication des maisons médicales , grâce à la revalorisation de la médecine libérale , actuellement découragée par la paperasserie et les charges , grâce aussi à la possibilité donnée aux établissements d' hospitalisation privée de participer aux missions de service public , grâce à une refonte des études médicales , grâce enfin à la télémédecine . Il faut assurer l' équilibre financier . N' en déplaise aux charlatans du &laquo; plus à n' importe quel &raquo; , on ne peut pas continuer de financer notre assurance maladie à crédit et d' en faire porter le poids aux générations futures , d'autant que les dépenses de santé vont continuer de croître . Alors , soyons courageux : sauvons notre système de santé pour garantir à tous les Français l' accès à l' innovation thérapeutique et aux nouveaux médicaments . Ces réformes sont nécessaires car sans elles notre système de protection sociale s' affaissera et les premières victimes seront nos concitoyens les plus démunis . Ces réformes se feront dans la concertation avec tous les acteurs . Je l' ai dit et je le répète , ma priorité , mon urgence , c' est la bataille pour l' emploi et la croissance . Si mes compatriotes m' accordent leur confiance , c' est sur cette bataille que je concentrerai le début de mon quinquennat . Près de 6 millions de Français au chômage ou n' ayant qu' une activité réduite , des centaines de milliers de jeunes hors de tout emploi ou formation … Une bombe sociale est sous nos pieds , et malgré cela la gauche me reproche de vouloir faire ce qui a été fait dans les pays voisins qui connaissent le plein emploi ! &laquo; , hurle -t-elle . Vieille et risible ficelle pour disqualifier tous ceux qui n' ont pas ses œillères . Je suis simplement pragmatique et je crois que la liberté est le meilleur tremplin des sociétés vivantes . J' ai la conviction que nous pouvons ramener la France au plein emploi . Pour cela , nous devons diminuer le coût du travail , simplifier et alléger notre droit du travail , mettre le paquet sur l' alternance , sortir des 35 heures et laisser aux acteurs de terrain le pouvoir de négocier leur organisation , lever la peur de l' embauche qui dissuade les PME de recruter … Ceux qui m' accusent à tort de vouloir casser le droit du travail ont cassé le travail tout court ! Je veux le restaurer , le valoriser , l' ouvrir à tous . Pour mes adversaires , l' emploi c' est encore et toujours l' affaire de l' Etat . Nous sommes l' un des pays au monde les plus réglementés , l' un des plus imposés de l' OCDE et avec 5,5 millions d' agents publics nous sommes l' état d' Europe le plus administré . Mais , pour la gauche et l' extrême droite , cette réalité importe peu . Ils préfèrent crier haro sur ceux qui , avec moi , veulent réduire ce chiffre de 8 % sur 5 ans pour réduire nos déficits et alléger les contraintes qui pèsent sur les initiatives . Oui , deux camps s' opposent : le camp de ceux qui pensent que pour redresser la France il faut encore et toujours verrouiller le système ; et celui , dont je suis avec des millions de Français , qui affirme qu' il faut le déverrouiller . Mes détracteurs croient que notre peuple ne peut pas supporter la vérité . Je crois le contraire ! Ils pensent qu' on peut relancer notre pays sans changements radicaux . Ils mentent ! Les attaques ne me feront pas dévier de ma route . Elles sont la rançon d' un projet exigeant , guidé par l' intérêt national . Avec les Français , j' aurai le courage de la vérité et de l' action . ",
    "Mes chers compatriotes , Au seuil de cette nouvelle année , je souhaitais m' adresser directement à vous . Vous le savez , l' année 2017 s' inscrit dans un moment particulier pour notre pays : malgré ses atouts , malgré ses forces qui sont celles d' une nation courageuse , d' un peuple fier , la France n' est pas épargnée par les difficultés . Ces difficultés vous les connaissez . C' est le drame du chômage et de la précarité . C' est la pauvreté qui augmente . C' est le pouvoir d' achat qui stagne . C' est aussi le terrorisme qui , à de multiples reprises , a frappé et endeuillé notre pays . Et au moment de refermer l' année 2016 , je pense aux victimes de l' attentat de Nice , je pense au Père Hamel , je pense au couple de policiers de Magnanville , je pense à celles et ceux qui ont été lâchement assassinés à Berlin il y a quelques jours . Nous sommes en guerre . Et dans cette guerre , nous n' avons pas le droit d' être faibles , nous devons être implacables face au totalitarisme islamique . La France a connu bien des épreuves dans son histoire . A chaque fois , elle les a surmontées parce qu' elle a su rester elle-même , en opposant à l' horreur de la barbarie les valeurs de sa civilisation , en étant fidèle à son engagement pour la liberté , la dignité de chacun . Avec vous , avec toutes celles et ceux qui aiment la France , je veux que nous soyons à la hauteur de cet idéal de résistance et d' espérance . Pour cette nouvelle année , je vous adresse tous mes vœux de bonheur , de succès , de santé . Que pour vous et ceux que vous aimez la vie soit au rendez-vous de ses plus belles promesses . Que celles et ceux qui ont toutes les chances en mains les fassent grandir et les mettent en partage . Que celles et ceux qui , l' année dernière , avaient le cœur serré retrouvent de l' espoir . Que ceux qui ont un genou à terre trouvent la force de se relever car en chacun il y a une flamme qui peut se rallumer . Et puis , je formule des vœux pour nous tous , peuple de France et pour notre cher pays que nous aimons tant . Je souhaite que 2017 soit l' année du redressement . Au printemps , vous élirez un nouveau Président de la République . Je suis candidat pour construire avec vous une nation libre et puissante . Le mur des critiques ne nous arrêtera pas . La vérité ne vous fait pas peur , vous êtes courageux , vous avez une énergie exceptionnelle : je veux vous rendre le pouvoir d' entreprendre , de créer et de réaliser vos projets car c' est ainsi que la France retrouvera la croissance et le plein emploi pour que la vie de chacun soit meilleure . Je veux redonner à chaque enfant de France l' égalité des chances pour réussir sa vie . Je veux que les Français vivent dans la sécurité et que chacun d' entre vous puisse être fier d' appartenir à un pays à nul autre pareil . Alors à vous tous , mes chers compatriotes , très bonne année 2017 ! Qu' elle apporte à la France et aux Français un nouvel élan : c' est là mon souhait le plus cher . Vive la République , vive la France . ",
    "Mes amis , mes chers compatriotes , C' est à Nice et c' est avec vous que je repars en campagne . A cette ville que j' aime et à vous tous , j' adresse mes vœux de bonheur et de succès pour cette nouvelle année qui commence . Je suis parmi vous , avec le cœur encore serré par la tragédie du 14 juillet . De ce souvenir monte en moi une volonté inflexible de défendre nos valeurs . Si les Français me font confiance , je prendrai tous les moyens et je solliciterai tous les Etats volontaires , de Washington à Moscou , d' Ankara à Téhéran , pour détruire l' Etat islamique et ses ramifications terroristes . Dans ce combat , j' aurai besoin de nos compatriotes musulmans pour faire le ménage contre ceux qui instrumentalisent leur foi . Nous sommes la France , nous sommes Nice , et jamais nous ne baisserons la tête devant ceux qui veulent abattre notre liberté . C' est nous qui les battrons parce que nos valeurs sont plus belles et plus fortes ! A vos côtés , je puis dire cette phrase simple et belle : &laquo; je suis Français et j' en suis fier &raquo; . Comme vous , je me sens l' héritier d' une Histoire , d' une culture , d' un patrimoine , d' un drapeau , et ce drapeau est suffisamment large pour fédérer tous les Français qui veulent , à nos côtés , servir leur pays . Cela s' appelle le patriotisme , et n' en déplaisent à certains , nous le revendiquons , nous l' assumons et nous irons au bout de notre conviction : la France doit être aimée et impérativement redressée . Durant trois mois nous avons vécu au rythme des primaires et vous avez donné une formidable leçon de démocratie à tous ceux qui doutaient de nous . Après ma victoire , je m' étais fixé trois objectifs . D'abord rassembler toutes les forces et les sensibilités de ma famille politique . C' est fait : après la compétition des primaires , les Républicains font bloc . Je salue fraternellement tous les militants et les élus qui m' ont rejoint sans état d' âme . J' aurai besoin d' eux , j' aurai besoin de leur engagement et j' aurai besoin des Alpes Maritimes ! Mon second objectif était d' unir la droite et le centre autour de ma candidature . C' est fait et je remercie les élus et sympathisants de l' UDI qui , avec un grand sens des responsabilités , ont décidé de se battre à mes côtés . Nous avons quelques différences entre nous , mais ce qui nous uni est tellement plus fort que ce qui nous distingue , et surtout , ce que nous avons à faire pour la France est bien plus important que toutes nos affaires partisanes . Enfin , je devais mettre en place une équipe de campagne ouverte à tous . C' est fait ! Maintenant , mes amis , nous sommes en ordre de bataille et nous allons foncer ensemble . Je laisse les sondages aux sondeurs , les scénarios écrits d' avance aux scénaristes , je ne crois qu' une chose : la victoire récompensera le camp le plus décidé , le plus convaincu . Ma détermination est de fer et je ne doute pas de la vôtre . Vous êtes des militants de la France et pour la France je vous demande une mobilisation et une résolution sans faille . Ne cédez à aucune intimidation , ne marchez jamais à l' ombre , affirmez ce que vous voulez et ce que vous êtes . On vous accuse de vouloir redresser la France de façon radicale . Assumez et ripostez car c' est vous qui avez raison de penser que la situation réclame du courage . On vous accuse de défendre les valeurs de la liberté , de l' autorité , de la famille , du travail . Assumez et ripostez car vous n' avez pas à vous excuser d' avoir du bon sens . Je compte sur vous et vous pouvez compter sur moi pour ne pas changer ce que je crois et ce que je veux . Il y a deux mois , je n' étais pas le candidat de l' establishment et je n' ai pas l' intention de le devenir : je vais repartir sur le terrain , replonger dans le quotidien des Français pour les écouter , les convaincre et les rassembler . Je veux leur faire partager ma volonté d' action , pas la solder en affadissant mes convictions . La démocratie , ça n' est pas des convictions pour les primaires , d' autres convictions pour les présidentielles et pourquoi pas encore des convictions pour les législatives ! Je me bats pour redresser notre pays , pas pour ruser avec lui , et , une fois au pouvoir , pour passer entre les gouttes . Je l' avais dit et je le répète , je relève un pari difficile pour un peuple si longtemps abreuvé d' illusions politiques et de douces promesses : le pari de la vérité et du changement de fond . Alors certes , inéluctablement , presque mécaniquement , la radicalité et la franchise de mon projet m' exposent aux blâmes et aux caricatures de ceux qui sont mous et flous . Les balles sifflent de partout . Visiblement , je gène certains . &laquo; Tous contre Fillon ! &raquo; , &laquo; tous contre la droite &raquo; , tous contre ces &laquo; pauvres &raquo; électeurs de la primaire qui , nous répète -t-on , ne représentent que 4 millions de Français . Mais il ne suffit pas d' être contre encore faut -il avoir quelque chose à dire . Qu' ont -ils à dire et à proposer ceux qui nous mitraillent ? Rien de nouveau mais surtout rien de fort pour répondre à l' urgence d' un pays qui décroche et qui souffre . Devant mes propositions , le système se rebiffe et use des arguments les plus fallacieux . Rien d' anormal , si ce n' est que le débat est bas . Contre le redressement national , mes adversaires ont rédigé leurs tracts : &laquo; sang et larmes &raquo; &laquo; purge &raquo; , &laquo; casse sociale &raquo; , et pourquoi pas l' apocalypse ! La casse sociale , elle est là , sous nos yeux , mais on préfère étouffer la réalité plutôt que de la regarder en face . 6 millions de Français qui pointent à Pole emploi ; 2 millions de jeunes qui ne sont ni à l' Ecole , ni en formation , ni en emploi ; la pauvreté qui augmente faute de croissance ; des quartiers déglingués par la délinquance ; des campagnes abandonnées ; des classes moyennes et des familles qui n' arrivent pas à boucler leurs fins de mois ; un système de santé débordé ; les plus beaux actifs de la France rachetés par l' étranger ; 2000 milliards de dettes pour nos enfants … Tout ça c' est la réalité , tout ça c' est maintenant , tout ça c' est le bilan de François Hollande , bilan qu' il a lui-même jugé tellement catastrophique qu' il a renoncé à se représenter . Oui , tout cela crève les yeux , mais rien n' y fait : 40 ans que la gauche nous rejoue le film de la &laquo; casse sociale &raquo; ; 40 ans qu' elle instrumentalise la peur contre tout changement ; 40 ans qu' elle croit pouvoir traiter les fonctionnaires et les ouvriers comme des clientèles électorales , avec maintenant pour acolyte l' extrême droite qui joue la même partition que l' extrême gauche . Derrière ces postures , la faillite nous guette et la colère monte mais ces gens-là vous disent qu' il ne faut rien changer . Avec les primaires de la gauche , dénommées primaires de la &laquo; Belle alliance populaire &raquo; – prière de ne pas rire ! – on est en plein déni et en plein dans le vieux logiciel socialiste . Comme disait Edgar Faure , &laquo; l' immobilisme est en marche , rien ne l' arrêtera ! &raquo; . C' est la course de lenteur entre celui qui dépensera le plus , celui qui réformera le moins et celui qui promettra le beurre et l' argent du beurre . Evacués Hollande et les résultats calamiteux de son quinquennat , oubliés les déficits , finis les engagements européens , ravalées les déclarations d' amour aux entreprises , aux orties le 49.3 et la Vème République ! Quel que soit le vainqueur de ces primaires , il y aura bien , entre nous , une différence de fond : moi , je veux redresser la France ; à gauche , on veut l' assister dans son déclin , Mr Macron compris . Je dénonce cette capitulation et rien ni personne ne m' empêchera de lui opposer un chemin plus grand et plus droit . Je suis gaulliste . Que voulez -vous , on ne se réinvente pas et , au surplus , je n' ai qu' une patrie et qu' une vie pour la servir ! Je veux incarner la fierté d' une nation qui ne se laisse pas abattre . C' est mon combat , et je viens vers vous , porté par la confiance que fait naître en moi cette incroyable aventure d' un pays qui par sa bravoure , sa culture , son goût du progrès , a réussi à se placer parmi les cinq plus grandes puissances du monde . Nous sommes les héritiers d' une Nation bâtie par des femmes et des hommes qui ont travaillé sans relâche , et tout nous commande de faire aussi bien qu' eux . Cette nation millénaire est montée sur les barricades pour la justice , s' est sacrifiée pour son indépendance afin que le peuple français soit toujours libre de dire &laquo; je suis là et j' exige le respect &raquo; . Exigez ce respect ! Exigez -le de toutes vos forces car le réveil national est d'abord entre vos mains . Depuis trois ans , je sillonne notre pays . &laquo; Laissez -nous travailler , laissez -nous réussir , qu' on nous fiche la paix &raquo; , voilà ce que m' ont dit les milliers de Français que j' ai rencontrés . Notre peuple n' est pas fatigué , c' est le système qui est complètement usé , avec cette incroyable accumulation de règles , de normes , de soupçons qui vampirisent les volontés . La bureaucratie a envahi nos vies . Tout est encadré , nivelé , passé à la moulinette d' une logique pernicieuse : mieux vaut , nous dit -on , la paperasse que l' audace , mieux vaut parait -il le chômage et les 3000 pages du code du travail que l' assouplissement de l' emploi , mieux vaut le diktat syndical que le référendum d' entreprise , mieux vaut un champ en jachère que quelques arpents non déclarés , mieux vaut les écrous administratifs que la responsabilité , mieux vaut -on vous le dit , on vous le répète- une circulaire qu' une initiative … Eh bien non ! Mieux vaut pour moi le risque d' oser que l' uniformité dans la médiocrité . Voilà ce que je crois et voilà ce que m' ont dit tous les salariés et agents publics . C' est ce système que je veux changer en redonnant au pays sa liberté ! Oui , avec vous mes amis , j' assume la première valeur de notre devise républicaine . Et d'ailleurs , si cette valeur de la liberté n' est pas portée par la droite , alors qui la portera ? Surement pas les socialistes qui de tout temps ont préféré l' égalitarisme mou . Et surement pas l' extrême droite , dont le projet économique est un copier-coller de celui de l' extrême gauche . Pour la liberté et pour le plein emploi , je veux concentrer le code du travail sur ses principes fondamentaux et renvoyer le reste au dialogue social . Je donnerai aux entreprises et à leurs salariés le pouvoir de négocier pleinement entre eux les heu- res et les conditions du travail . Moi je fais le pari de la vraie négociation sociale , dans les entreprises , là où entrepreneurs et salariés savent de quoi ils parlent . Et parce que l' Etat doit donner l' exemple d' une nation qui se retrousse les manches , nous rétablirons dans les administrations publiques les 39 heures . Là aussi il y aura des négociations car chacun devra bénéficier des fruits de cet effort , mais cela se fera car je veux une fonction publique performante , au service d' un Etat fort . Par la liberté , je donnerai à ceux qui veulent entreprendre le pouvoir de le faire en renforçant le statut des autoentrepreneurs et en créant celui de travailleur indépendant . Avec le numérique , c' est le début d' une nouvelle économie qui va dynamiser les générations à venir , avec un potentiel d' un million d' emplois nouveaux . Par la liberté , je donnerai aux investisseurs de France le goût d' investir ici , chez nous , pour nos industries , pour nos emplois . Je baisserai massivement les charges et j' allégerai la fiscalité qui pèse sur l' investissement . Je suis un patriote : je préfère supprimer l' ISF que de voir l' économie française sans capitaux , passer sous contrôle étranger . Plutôt que de nous rouler par terre pour attirer les fortunes du Qatar et les fonds de pensions américains , je veux des fortunes françaises qui investissent pour la France . N' en déplaise à ceux qui pensent que le bonheur est de marcher à genoux , c' est la liberté qui est le meilleur carburant des sociétés vivantes ; c' est la liberté qui créera des initiatives et des emplois ; c' est la liberté qui relancera l' ascenseur social aujourd'hui bloqué dans le sous-sol de ce système qui vous explique depuis trop longtemps qu' il vaut mieux l' ombre pour tous que la lumière pour ceux qui veulent en sortir . Les jeunes n' ont pas peur de la liberté . Ils n' ont que rage pour cet immobilisme poussiéreux qui les englue . Les chômeurs n' ont pas peur de la liberté . Ils ont peur de rester coincés au chômage . L' autoentrepreneur , l' artisan , le jeune agriculteur , eux qui s' endettent et se lancent , ils n' ont pas peur de la liberté . Ils la réclament à cor et à cris . Je parle de liberté , oui , et j' en parle pour les sans grade , les sans carnets d' adresse , les oubliés , les déclassés , les humiliés , les courageux , les intrépides , les audacieux , bref , ceux pour qui la liberté est une opportunité de prendre leur destin à pleine main . Et celui qui la prend s' engage pour réussir sa vie et pour prouver qu' il existe en lui une flamme ; et de fil en aiguille , de flamme en flamme , c' est tout un peuple qui se redresse et se met à gagner . En cinq ans , nous pouvons élever notre compétitivité et égaler l' Allemagne , en cinq ans nous pouvons réduire notre chômage en dessous de 7 % , remettre nos finances publiques à l' équilibre et d' ici 10 ans nous pouvons être la 1ère puissance européenne . Alors oui , mes amis , liberté à la Liberté . Mais la liberté sans l' autorité , c' est l' anarchie . Je veux rétablir l' autorité de l' Etat , l' autorité de la loi , la nécessité de l' ordre . Qu' un père de famille puisse gagner dix fois moins que le dealer qui trafique au pied de son immeuble , c' est une honte . Que nos policiers puissent être attaqués aux cocktails Molotov , ça n' est pas un fait divers , c' est un scandale . Qu' un de vos concitoyens , tout près d‘ici , se croit autorisé à décider de son propre chef , quels étrangers peuvent entrer et résider en France , c' est le contraire de la démocratie et de la République . Nul n' est au-dessus de la loi ! La loi et la sécurité doivent être partout et pour tous , avec des forces de l' ordre modernisées et une Justice ferme et rapide . Nos forces sont sous-équipées , et elles le sont notamment sur tout ce qui relève des nouvelles technologies pour cerner et chasser la délinquance . Nos forces s' éreintent dans des tâches administratives . Elles doivent passer plus de temps sur le terrain . Pour cela , la tranquillité publique commande de renforcer le rôle des maires et des polices municipales qui sont au plus près des populations . Et puis , il faut une justice rapide et ferme . Chacun est responsable de ses actes ! Un voyou est un voyou avant d' être une prétendue victime de la société . Le sentiment d' impunité des délinquants est le résultat d' un manque de fermeté et de civisme qui devraient s' exercer à tous les niveaux : dans la famille qui doit assumer ses devoirs ; à l' Ecole qui ne doit rien céder devant l' indiscipline et l' inculture des caïds en herbe ; dans les prétoires où la Justice doit passer vite et nette . L' impunité zéro doit être la règle . Je veux rétablir les peines planchers pour les récidivistes . Un délit : une sanction ! Voilà la règle qui doit être parfaitement claire pour les délinquants . Cette fermeté retrouvée réclame de reprendre le programme de construction de places de prison interrompu par Hollande et Taubira . Vous m' aurez compris : j' en appelle à un nouvel état d' esprit face à la violence et à une mobilisation nationale pour qu' à chaque coin de rue nos policiers et gendarmes se sentent respectés , soutenus et le terroriste comme le délinquant réprouvés , coincés , pourchassés . La peur doit changer de camp et nul ne doit pouvoir se dire que le mal rapporte plus que le bien . Mes amis , Il y a des lieux en France où le sentiment d' appartenir à la même nation s' est disloqué . Les replis identitaires avancent et l' intégration recule . L' unité nationale commande une autre politique de l' immigration . Il ne faut pas céder aux injonctions de la bien-pensance , qui nous adresse toujours le même message , qui se veut angélique et qui est en réalité inconscient : liberté pour les migrants et honte aux frontières . Non , la France est généreuse mais elle n' est pas une mosaïque et un territoire sans limite ; c' est une Nation , en droit de choisir qui peut la rejoindre , en droit aussi d' exiger des étrangers qu' ils se plient à ses règles et à ses coutumes . Il existe chez nous , depuis la chrétienté , une tradition de l' hospitalité pour les victimes de persécutions politiques . Il existe aussi une tradition d' ouverture aux talents venus d'ailleurs . Je pense à Romain Gary , venu de l' Est de l' Europe s' installer à Nice avec sa mère quand il était enfant , et qui toute sa vie voudra être plus français que tous les Français , s' acharnant à réussir par les littérature et par les armes . Mais ces traditions d' ouverture doivent être raisonnables et raisonnées . Nous comptons 6 millions de chômeurs et près de 9 millions de pauvres . Dans ce contexte , l' immigration doit être fermement contrôlée et réduite au minimum . Il faut tout d'abord ramener notre politique d' asile à l' aune de sa seule ambition et mettre fin à son détournement . L' asile sert à protéger les victimes politiques ; il sert aux personnes qui ont fui les persécutions ou les guerres dans leurs pays . L' accueil des véritables réfugiés est un devoir , mais ne nous laissons pas déborder par ceux qui n' ont pas ce statut . Les demandes d' asile ont explosé depuis 2012 . Les crises du Proche-Orient n' expliquent pas cette augmentation massive , car 70 % des demandes sont rejetées comme infondées . Ce qui est en cause , c' est la multiplication des détournements de procédures nourris par les filières et les passeurs . La méditerranée est devenue le tombeau de dizaine de milliers d' êtres humains , femmes et enfants . Ces passeurs de faux rêves sont des passeurs de morts qu' il faut traquer et neutraliser . Ce qui est en cause aussi , c' est la longueur des procédures qui permettent aux illégaux de s' installer et de viser la régularisation de leur situation . Seuls 4 % des déboutés de l' asile sont effectivement éloignés ; les autres se maintiennent en France contre la loi . Je veux reconstruire le système de l' asile pour mettre fin à ces abus . Il faudra veiller à ce que les demandeurs d' asile puissent présenter leur demande à l' étranger , dans nos consulats , dans les pays qui leur servent de premier refuge . Il faut , en second lieu , réduire la durée d' instruction des demandes d' asile . Obligation sera faite aux demandeurs de former leur demande dans un délai de 15 jours après l' entrée sur notre territoire , sous peine de la voir traiter par une procédure accélérée . La durée maximale d' examen d' une demande d' asile devra être limitée à 120 jours au total , voies d' appel comprises . Pour être contrôlé , l' hébergement des demandeurs d' asile sera assuré dans des centres dédiés , gérés par l' Etat ou pour son compte par des associations . Enfin , ceux des demandeurs d' asile dont la demande aura été définitivement rejetée devront impérativement être éloignés du territoire . La décision de rejet devra s' accompagner immédiatement de la notification à l' intéressé d' une obligation de quitter le territoire français . Pour faciliter leur éloignement , ceux des demandeurs d' asile dont la demande apparaîtra manifestement infondée ou qui sont originaires de pays réputés sûrs , seront placés en rétention administrative pendant l' examen de leur demande , de manière que leur éloignement soit garanti en cas de rejet de leur demande . Il faudra généraliser le passage dans les centres de rétention administrative . Si nécessaire , mon gouvernement renégociera la directive &laquo; éloignement &raquo; pour tirer les conséquences de ce qui était prévisible : l' assignation à résidence privilégiée par Monsieur Valls est un échec qui ajoute encore aux difficultés de maîtrise des procédures d' asile . Penser qu' un étranger en situation irrégulière quittera de lui-même le territoire national est naïf . La durée maximale de la rétention administrative devra être portée de 45 jours à 180 jours . A ceux qui pousseront des cris d' orfraie en entendant cette proposition , je rappelle que cette durée est de 18 mois en Allemagne et qu' elle est illimitée au Royaume-Uni et aux Pays-Bas . Ce délai de six mois permettra d' organiser , en collaboration avec les autorités des pays d' origine , le retour effectif de leurs ressortissants . On ignore souvent que le retour dans leur pays des étrangers en situation irrégulière est soumis à la délivrance d' autorisations par les pays dont ils sont les ressortissants . Or ces pays freinent parfois la délivrance de ces documents , par mauvaise volonté ou par lenteur administrative . C' est pourquoi , je veux que la coopération des pays d' origine à notre politique d' immigration soit une condition de l' aide au développement que nous leur versons et du nombre de visas que nous délivrons à leurs ressortissants . Il s' agit d' assurer une forme de réciprocité pour faire respecter notre souveraineté . En revanche , je ne crois pas moralement possible de limiter le droit à la scolarisation des enfants d' immigrés clandestins . Les enfants ne sont pas responsables . C' est l' honneur de la République que d' assurer cette scolarisation . Une politique sérieuse de l' immigration repose aussi sur le règlement des situations de crise comme celle de Calais . Le gouvernement de François Hollande a tardé à démanteler les campements , en leur laissant prendre une ampleur inconnue jusque-là . Il n' a pas négocié avec le Royaume-Uni un rééquilibrage des accords du Touquet de 2003 pour traiter les causes du problème . C' est une exigence pour nous : les Britanniques doivent prendre toute leur part dans l' accueil de ces migrants , y compris les mineurs isolés , qui veulent traverser la Manche . En 30 ans , la population de l' Afrique va plus que doubler . Avec solennité , je dis que l' immigration en France n' est pas l' avenir de l' Afrique . L' avenir des Africains est dans le développement de leurs pays , dans la création d' emplois en Afrique ! L' aide au développement peut y participer dans le cadre d' un nouveau partenariat entre la France et le continent africain . La France y apportera sa maîtrise des technologies les plus modernes en particulier dans l' accès à l' énergie avec l' initiative remarquable de Jean-Louis Borloo , ses capacités de formation de la main d' œuvre africaine et , si nécessaire , des financements ciblés . Mais c' en est fini de la période où la question de l' immigration était passée sous silence dans les discussions bilatérales entre la France et chaque pays africain , au motif qu' il s' agissait des questions trop complexes ou trop sensibles . La question de l' immigration ne doit plus être un tabou dans nos relations avec les pays d' origine . A l' évidence , mes amis , l' ampleur des enjeux migratoires commande une coordination européenne . Je le répète depuis 2012 , il faut réformer les accords de Schengen . Je veux que les frontières extérieures de la zone Schengen soient de véritables frontières , où les contrôles des voyageurs , notamment non européens , seront systématiques . Si les étrangers doivent attendre 2 heures pour entrer , ils attendront 2 heures , comme dans n‘importe quel aéroport aux Etats Unis ! Quant aux pays membres de Schengen qui ne satisfont pas aux exigences d' un contrôle rigoureux aux frontières extérieures , ils doivent sortir de l' espace Schengen . Dans le contexte de guerre contre le totalitarisme islamique , tant que les frontières de l' Europe ne seront pas protégées par nos partenaires , la France rétablira durablement de vrais contrôles à ses frontières nationales . Ici à Nice , cet engagement prend tout son sens . Dans le contexte terroriste que connaît l' Europe , il y a quelque chose d' aberrant à savoir que l' auteur de l' attentat de Berlin a pu gagner la Hollande , puis la Belgique , la France , et enfin l' Italie . Je ne condamne personne mais le moins qu' on puisse dire , est que nous avons impérativement besoin de développer nos systèmes et nos échanges d' information , et de tirer les conséquences du développement de la vidéo-surveillance . Il ne suffit pas d' équiper nos gares ou nos aéroports en caméras de vidéo-surveillance , encore faut- il exploiter de manière rigoureuse les images issues de la vidéo-surveillance . La France doit non seulement lutter contre l' immigration illégale mais elle doit aussi réduire son immigration légale au strict minimum . Les délivrances de visas et de titres de séjour n' ont cessé de dériver ces dernières années : le nombre de visas accordés en 2015 a augmenté de 40 % , par rapport à 2012 . Pour reprendre les choses en main , il faut que le Parlement puisse adopter chaque année des quotas limitant le nombre d' étrangers qui pourront être admis en France , en fonction de nos besoins économiques et de nos capacités d' intégration . Concrètement , cela signifie qu' un visa de long séjour ou une carte de séjour pourront être refusés lorsque le contingent annuel sera dépassé ; la demande devra alors être examinée l' année suivante . C' est en fixant ces plafonds que nous pourrons limiter strictement l' immigration familiale , dont le regroupement familial . Si , pour assurer cette politique , il faut réviser la Constitution mais aussi faire bouger les lignes de l' interprétation que la Cour européenne des droits de l' Homme donne de la Convention européenne des droits de l' Homme , eh bien je prendrais mes responsabilités devant le peuple français . Ces quotas d' immigrants tiendront compte non seulement des besoins de la France mais aussi des capacités d' intégration à la société française . Ces capacités seront vérifiées dans le pays d' origine , tant pour les visas de long séjour que pour les cartes de séjour , en prenant en compte la maîtrise de la langue française et l' adhésion aux valeurs de la République . Il faudra enfin , et c' est le bon sens même mais il a été complètement perdu de vue au cours des années , que ceux des étrangers qui , une fois arrivés en France , ne respectent pas leur parcours d' intégration , puissent être sanctionnés , par le non renouvellement de leur titre de séjour . Et puis , je veux prendre en compte les conséquences des migrations à l' intérieur de l' Union européenne , et notamment les effets d' une concurrence mal maîtrisée . La directive sur les travailleurs détachés a progressivement installé sur notre territoire un véritable dumping social au cœur de l' Europe . C' est le cas dans le transport par camion , dans la construction où des milliers d' emplois français sont concurrencés de façons inéquitables . Nous renégocierons cette directive et je ne céderai pas sur un principe : en Europe , pour un travail égal , le salaire doit être égal et les cotisations sociales aussi . Trop souvent , l' immigration est encouragée par l' attractivité de notre système social . Il n' est pas acceptable qu' un étranger vienne en France dans le seul but de bénéficier de la solidarité nationale . Je veux que le bénéfice de toutes les prestations sociales soit conditionné à une durée de résidence préalable pour les étrangers en situation régulière . Ceux -ci ne pourront donc postuler aux allocations logement et familiales qu' au bout de deux ans . Ce délai permettra de vérifier la volonté d' intégration du nouvel arrivant et d' apprécier s' il est réellement autonome financièrement . Pour les étrangers en situation irrégulière , il faut mettre fin au dérapage financier de l' aide médicale d' Etat , dont le coût dépasse le milliard d' euros , et dont le nombre de bénéficiaires a augmenté de 50 % depuis 2012 . Cette aide médicale d' Etat sera supprimée et remplacée par la seule prise en charge des urgences médicales et des maladies infectieuses . Lorsque la France a fait confiance à des étrangers et que ceux -ci la trompent et ne respectent pas ses lois , l' Etat doit être de la plus grande fermeté . Il en va de son autorité et de la crédibilité de notre politique . C' est pourquoi je propose qu' un étranger condamné pour des crimes et délits puisse être expulsé du territoire français vers son pays d' origine . Enfin , pour les étrangers qui , après s' être installés durablement en France et espèrent en obtenir la nationalité , il faut définir notre exigence : c' est l' assimilation , c'est-à-dire la volonté d' être pleinement membre de notre famille nationale . L' acquisition de la nationalité française est un honneur . Elle se mérite . Elle se mérite , pour l' essentiel , par un patient effort pour adopter nos valeurs , nos coutumes , nos traditions . Qui dit assimilation dit une maîtrise de la langue française parce que c' est bien le minimum quand on prétend devenir Français . Qui dit assimilation dit aussi respecter les principes fondamentaux de la République . Respecter l' égalité hommes-femmes , partout , sans condition . Respecter la liberté de conscience comme la liberté d' expression . Respecter l' autorité de l' Etat et des lois , s' approprier une histoire , un drapeau . Etre assimilé , c' est tout simplement aimer la France . Nous devons revoir nos procédures d' accès à la nationalité . Nous sommes l' un des pays qui permet le plus facilement à des ressortissants étrangers d' acquérir sa nationalité . En quatre ans , le nombre de naturalisation par décret a augmenté d' un tiers , pour concerner 113 000 nouveaux Français en 2015 . Les instructions de Monsieur Valls pour augmenter le nombre de naturalisations seront donc abrogées . La durée minimale de résidence en France exigée d' un candidat à la naturalisation sera portée à 8 ans , comme c' est le cas en Allemagne . La République pourra s' opposer à toute entrée dans la communauté nationale dès lors que l' assimilation du postulant à la nationalité ne sera pas établie . Les étrangers qui ont fait l' objet d' une condamnation pénale verront leur demande écartée . L' acquisition de la nationalité sur simple déclaration à la suite d' un mariage ne sera possible qu' après 5 ans de vie commune et l' assimilation du conjoint étranger fera l' objet d' une vérification . Les jeunes nés en France de parents étrangers pourront , comme c' est le cas aujourd'hui , accéder à la nationalité française mais cet accès devra être le fruit d' un choix , mûri par chacun de ces jeunes et formalisé dans une déclaration de volonté de devenir Français . Voilà la politique d' immigration que je veux pour notre pays . Je la crois nécessaire à notre unité et nécessaire à la fraternité des liens qui doivent unir les Français et ceux des étrangers qui nous rejoignent avec sincérité et espoir en la France . Mes amis , maintenant je vais relever avec vous le défi du redressement national . Ce défi , je le gagnerai si toutes celles et tous ceux qui en ont assez des mensonges , assez du dé- clin , se lèvent . Je m' attaque au consensus , je m' attaque au confort du conservatisme , je m' attaque à une citadelle , et je ne pourrai la percer qu' avec l' appui de tous ceux qui veulent , avec moi , enclencher la révolu- tion du bon sens . Alors , je donne rendez-vous à toutes celles et tous ceux qui ont la tête sur les épaules . J' appelle tous ceux qui ont des valeurs solides . J' appelle tous ceux qui savent que le bonheur est une conquête . J' appelle tous ceux qui ont dans le cœur la fierté d' être français . Sur le chemin de la victoire nous traverserons les vents contraires ; nous passerons à travers les bourrasques ; nous franchirons tous les ponts . Vous êtes partant et je le suis , alors j' irai , avec vous , chercher les cœurs vaillants . Nous irons au-devant de toutes les forces françaises , et nous les réveillerons , nous les soulèverons , nous les entrainerons , et vous verrez qu' elles nous porteront au-delà de nos propres espérances . Nous serons alors , tous ensemble , la France en action ! Vive la France et vive la République . ",
    "Il y a des lois et il convient de les respecter . Sinon , c' est l' anarchie . C' est l' honneur de la France de protéger ceux qui fuient la guerre et sont persécutés . Mais il y a des règles qui encadrent l' accueil des réfugiés et des étrangers qui ne sont pas tous éligibles au droit d' asile . Ceux qui ne les respectent pas s' exposent à des sanctions . M. Herrou est peut-être généreux mais il n' est , à mon sens , pas responsable car ce n' est pas à lui de juger qui peut entrer et demeurer en France . Il faut une immigration minimale et très contrôlée , répondant à nos besoins économiques et à nos capacités d' accueil et d' intégration . Je veux mettre fin à cette situation douloureuse et sans issue qui voit des hommes , des femmes et des enfants dormant dans les rues , à même le sol , été comme hiver et dans des conditions d' hygiène déplorables , qui sont la proie de tous les trafics . Les laisser entrer en France où ils n' ont comme horizon que cette vie de misère n' est pas conforme à nos valeurs . Nous devons donner un signal fort en pratiquant une politique ferme d' expulsion des étrangers en situation irrégulière vers leur pays d' origine . Et notamment tous les déboutés du droit d' asile qui , aujourd'hui , restent sur notre territoire faute d' une réelle politique de reconduite aux frontières . Nous devrons aussi renégocier les accords de Schengen qui ne permettent pas de lutter contre les arrivées massives que nous connaissons aujourd'hui . Nous avons besoin d' un corps de garde-frontières et de garde-côtes pour surveiller nos frontières terrestres et nos façades maritimes , en particulier la Méditerranée . Les Etats qui ne parviennent pas à assurer la surveillance de leurs frontières devront sortir du système Schengen qui ne peut être efficace que s' il est vraiment appliqué dans tous les pays qui y adhèrent . Sinon , la charge est reportée sur les Etats jugés les plus attractifs par les candidats à l' immigration . Enfin , nous devons pouvoir rétablir des contrôles à nos frontières lorsque la situation l' exige comme c' est le cas actuellement en raison des impératifs de l' état d' urgence . A force de mettre de &laquo; l' eau dans notre vin &raquo; comme vous dites , on ne résout pas les problèmes de la France . Depuis trois ans , je parcours notre pays de long en large et j' entends les Français dire qu' ils en ont assez des demi-mesures . Ils ont raison . On est en déclin . Pour redresser notre pays , combattre le chômage de masse et relancer la croissance , il faut des mesures franches et percutantes . J' assume la radicalité de mon projet parce que le pays est radicalement bloqué . Je fais le pari que les Français veulent la vérité et qu' ils sont prêts à des changements puissants . C' est pourquoi je n' ai pas l' intention de renier mes convictions . Avec 6 millions de personnes à Pôle Emploi et plus de 2.000 milliards de dettes , on ne peut pas dire que ça va mieux et qu' on peut continuer comme cela longtemps . La très légère embellie sur le marché de l' emploi est largement due à la multiplication des stages et des emplois aidés . En réalité , notre croissance est poussive , notre modèle social est fissuré et nos déficits menacent notre avenir . Face à cette situation , mon projet n' est nullement &laquo; brutal &raquo; . Ceux qui disent cela , ce sont les démagogues de la gauche et de l' extrême droite qui veulent endormir les Français . La vérité , c' est que mon projet est tout simplement efficace , raisonnable et responsable . Si la France se réforme et se retrousse les manches , elle peut être d' ici 10 ans , la première puissance européenne . C' est mon objectif ! La Sécurité sociale fait partie de notre unité nationale . Je n' ai absolument pas l' intention de revenir sur le principe de la solidarité . La société du chacun pour soi n' est pas la mienne . Mais sur l' assurance maladie et la santé , sur les allocations chômage , sur les retraites , il y a des réformes à faire pour être plus juste et plus efficace . Il n' y a pas de honte à affirmer notre héritage chrétien et je n' ai pas de leçons à recevoir en matière d' unité nationale . Depuis longtemps , je m' oppose au communautarisme et au repli identitaire . C' est moi qui ai fait appliquer l' interdiction des signes religieux à l' école et mis hors la loi le port du voile intégral . Je ne transige pas sur la laïcité , mais la laïcité n' est pas l' adversaire de la foi . Je me suis engagé à respecter cette loi sur le non cumul des mandats . Je pense que les Français ne comprendraient pas que nous revenions là-dessus . La primaire de la droite et du centre a été digne . J' ai tendu la main à tout le monde . Le rassemblement autour de ma candidature s' est fait rapidement et sans problème majeur . Nous sommes maintenant en ordre de marche pour mener campagne et aller chercher la victoire . Dans la primaire , je n' étais pas le candidat de l' establishment et des sondages et je n' étais pas le &laquo; &raquo; espéré de la gauche et du FN . Je suis attaqué par mes adversaires parce qu' ils sentent ma volonté et celle de l' électorat de la droite qui s' est reconnu dans mes valeurs . Ces attaques ne m' impressionnent pas . Je vais continuer de tracer ma route .",
    "Mes chers amis , C' est un plaisir de me retrouver parmi vous et puisque l' heure est encore aux vœux , je souhaite à chacune et chacun de vous une bonne année 2017 . Soyez heureux , prenez soin de vous et de ceux que vous aimez , soyez passionnés , ne renoncez pas à vos rêves , et puisque vous avez la France au cœur , engagez -vous pour elle , mobilisez -vous pour elle : elle aura bien besoin de votre audace et de votre courage . Il y a quelques semaines de cela , j' entendais François Hollande nous dire , avec le ton martial qu' on lui connaît , &laquo; qu' il n' y avait pas de France sans Etat ! &raquo; … J' ai beaucoup de respect pour l' Etat , mais il faut surtout dire qu' il n' y a pas de France sans les Français ! Ce sont les Français qui font vivre notre pays . Ce sont eux , travailleurs , entrepreneurs , agriculteurs , ingénieurs , chercheurs et tant d' autres métiers , qui font que notre pays est encore la Vème puissance du monde ; ce sont nos militaires , nos policiers , nos gendarmes qui le sécurisent et le défendent , parfois au prix de leur vie . Avant de parler de l' Etat de façon abstraite , il faut parler des Français , de leur existence , avec ses coups durs et ses espoirs . Il faut entendre leur colère à l' égard d' un système qui les ligote . Il faut entendre leur volonté de protéger leurs traditions , leur langue , leur art de vivre , leur identité . Il faut entendre cette jeunesse qui ne sait pas si elle est condamnée au chômage ou à tenter sa chance à Londres , à Barcelone ou à New York . On vous oublie . Exigez le respect et exigez -le de toutes vos forces puisque vous êtes la France ! Alors que s' achève un quinquennat calamiteux , 2017 ne peut être autre chose qu' une grande année , une année de sursaut et d' espérance . L' année où la France aura fait le choix de se redresser . On nous a habitués , ces derniers temps , à rétrécir nos ambitions , à racornir nos vues . Moi , je veux vous inviter à revoir l' avenir en grand . Sans nostalgie , mais forts de notre passé . Sans démagogie , mais certains de nos compétences . Sans mensonge , mais confiants dans notre capacité à retrouver , ensemble , le rang qui aurait dû demeurer le nôtre . Il n' y a pas de fatalité pour un peuple qui se lève pour sa réussite . Et ce ne sont pas les caricatures , les regards narquois et les surenchères populistes qui nous ferons dévier . Nous le ferons tout simplement parce que nous avons pris la mesure de ces décennies d' immobilisme et de moralisme condescendant . Nous le ferons parce que , contrairement à l' ensemble des autres candidats , nous avons compris que la réussite doit cesser d' être présumée coupable . Le piège , c' est le compromis , la réforme molle , le refus d' aller au bout . Ce que je veux , avec vous et pour vous , c' est libérer la France de ce qui la fait tomber . Nous vivons dans un monde difficile , un monde où il va falloir se défendre . Se défendre contre le totalitarisme islamique qui rêve d' abattre nos valeurs . Nous ne céderons rien : c' est nous qui battrons ces fanatiques ! Se défendre aussi face à la pression des géants économiques que sont la Chine ou les Etats-Unis . Personne ne nous fera cadeau .Pour ceux qui ne l' auraient pas compris , Mr Trump ne fait pas dans la langue de bois et l' angélisme . Inutile de s' en indigner ou de trembler comme des feuilles . Nous jugerons le nouveau président américain sur ses actes . Mais le message est clair : la France doit être forte pour être respectée ! Et pour être forte , la France doit d'abord être fière car un pays qui ne croit ni en son drapeau , ni en son avenir , est un pays à genoux . Pour être forte , la France doit être plus productive , elle doit être la patrie du travail , du mérite et de l' audace , elle doit être en capacité d' entraîner ses alliés européens , elle doit être indépendante sur la scène internationale . C' est toute l' ambition de mon projet . Je veux redresser la France et lui donner les moyens de la puissance . Depuis ma victoire aux primaires , c' est &laquo; le tous contre Fillon &raquo; .Ne vous laissez pas intimider par les tirs de barrage . Ne vous excusez pas d' avoir des valeurs . Ne vous excusez pas de croire en la France . Exigez qu' on vous écoute , qu' on vous respecte . Assumez votre idéal et tenez vos positions ! Mon projet perturbe les castes bien établies . Aucune mauvaise foi ne les arrête pour conserver leur petit pré-carré ; aucun mensonge , aucune sornette ne les freine . Le chœur des pleureuses annonce , en se tordant les mains , la fin du monde si la droite l' emportait . L' économie française , vous dit -on , s' arrêtera brutalement . On verra , par millions , des Français désœuvrés , errer , à la recherche d' un service public en état de marche . A les entendre , sans le socialisme il n' y aura plus de social et sans la firme Le Pen il n' y aura plus de nation française … Mais voilà , je suis Français et bien décidé à donner à mon pays sa sécurité et son progrès . Mais qui sont -ils , pour nous accuser ? Qui sont ces donneurs de leçons ? L' extrême-droite dont le projet économique est un copier-coller de celui de l' extrême gauche ? Retraite à 60 ans et retour du Franc : le camp de la faillite accélérée n' a aucun titre pour défendre le peuple français ! La gauche ? Depuis cinq ans , elle n' a rien fait . Par leur incompétence , leur hésitation , leur désinvolture , les socialistes ont précipité le déclin de la France . Leur bilan est sans appel . Il est tel que le Président , plutôt que de devoir l' assumer , a jugé plus prudent de s' éclipser discrètement . François Hollande sort de l' Histoire par la porte de derrière . Pas de vivats , pas de rappel , juste le silence d' une France indifférente mais à cran . Mes adversaires m' accusent de menacer le &laquo; modèle social français &raquo; . Mais son affaiblissement , c' est maintenant que nous le voyons , avec un chômage massif , avec une pauvreté et une précarité grandissantes , avec des déficits partout , des déserts médicaux , des soins de santé non remboursés , des urgences débordées par une épidémie de grippe , des structures d' hébergement saturées par le retour du froid ! Ce qui est brutal , ce n' est pas mon projet , c' est l' échec de la gauche . Ce qui est brutal , c' est le désespoir des chômeurs , des exclus , des classes moyennes . Ce qui est brutal , c' est la délinquance et la violence qui pourrissent la vie des gens honnêtes . Il y a une chose qui ne sera jamais pardonnée à François Hollande , c' est d' avoir trahi la jeunesse . Il a menti en lui promettant n' importe quoi . Il a menti en lui cachant la réalité . Il a menti sur les chiffres du chômage .La réalité , la voici . Près d' un jeune actif sur quatre est au chômage . Plus de 100 000 jeunes quittent , chaque année , le système éducatif sans diplôme ni qualification . Deux millions d' entre eux sont à la dérive , ni au collège , ni en formation , ni au travail . Et , au-delà de la jeunesse , six millions de Français pointent à Pôle emploi . La pauvreté augmente , faute de croissance . Des quartiers sont déglingués par la délinquance , des campagnes dépérissent . De plus en plus de familles n' arrivent pas à boucler leurs fins de mois . Et nous laissons 2200 milliards de dettes à nos enfants . Comment en sommes -nous arrivés là ? Mon diagnostic est clair . Et tant pis s' il ne plaît pas aux démagogues qui aiment les histoires à dormir debout . Il tient en deux phrases . D' un côté , l' excès de normes , de contraintes et d' impôts freine l' activité , l' initiative , l' investissement et les embauches . De l' autre , le chômage et la précarité nécessitent toujours plus de réglementations qui gênent les entreprises , et de dépenses publiques qui alourdissent notre endettement et poussent à augmenter les impôts , ce qui ne fait que boucler le cercle vicieux conduisant à moins de croissance et plus de chômage . En réalité , ce constat est bien connu . Ce qui manque , c' est le courage de le regarder en face et d' agir . Moi , je veux sortir de cette logique infernale !Je veux débloquer l' économie , rallumer le moteur de la croissance , revenir au plein emploi . Je veux redonner espoir à la jeunesse en lui donnant le pouvoir d' entrer dans le monde du travail . C' est le travail qui émancipe , et c' est le chômage qui aliène . La meilleure politique sociale , c' est celle qui donne du travail à chacun . Et la seule politique qui donne du travail à chacun , c' est la liberté ! Les milliers de Français que j' ai rencontrés , ces trois dernières années , me l' ont tous répété : la bureaucratie a envahi nos vies , &laquo; qu' on arrête de nous emmerder ; laissez -nous travailler , oser et réussir &raquo; . Tout est encadré , régulé , contrôlé . La paperasse a remplacé l' audace . La circulaire a remplacé l' initiative . Les normes ont remplacé le bon sens . Mes amis , c' est à nous d' assumer la première valeur de notre devise républicaine , la liberté . Ne la laissons pas se dissoudre dans les lubies égalitaristes de la gauche , les fantasmes autoritaires de l' extrême droite , et l' immobilisme économique des deux réunies . Pour libérer l' économie , je veux concentrer le code du travail sur ses principes fondamentaux , et renvoyer le reste au dialogue social . Je donnerai aux entreprises et à leurs salariés le pouvoir de négocier pleinement entre eux les heures et les conditions du travail . Les 35 heures , je ne veux pas les moduler , les atténuer , les esquiver : je veux les supprimer totalement ! Je fais le pari de la vraie négociation sociale , dans les entreprises , là où entrepreneurs et salariés savent de quoi ils parlent , parce qu' ils parlent de leurs vies , de leur pouvoir d' achat , du carnet de commandes dont tout dépend . Je renforcerai le statut de l' autoentrepreneur et créerai celui de travailleur indépendant , pour redonner à ceux qui veulent entreprendre le pouvoir de le faire . La révolution numérique va transformer en profondeur tous nos processus , tous nos métiers , toutes nos industries . Elle nous offre un potentiel d' un million d' emplois nouveaux si nous savons anticiper ces changements , former nos jeunes , adapter nos structures . Par la liberté , je donnerai aux investisseurs de France le goût d' investir ici , chez nous , pour nos industries , pour nos emplois . Je baisserai massivement les charges et j' allégerai la fiscalité qui pèse sur l' investissement . François Hollande prétendait ne pas &laquo; aimer les riches &raquo; . Très bien , on a taxé et surtaxé la réussite et les Français sont tous devenus un peu plus pauvres . Je suis un patriote : je préfère supprimer l' ISF que de voir l' économie française perdre des capitaux et tomber sous la coupe du Qatar et des fonds de pensions américains ! Débloquer l' économie , c' est aussi faire revenir la sphère publique à une taille raisonnable . Je veux mettre de l' ordre dans les finances publiques , mettre les administrations aux 39 heures et revenir à un nombre d' emplois publics équivalent à celui des principaux Etats développés . Avec 5,6 millions d' agents publics , franchement , il n' y a rien d' extravagant à réduire de 8 % en 5 ans ce nombre . Je préfère un Etat fort , avec des agents publics mieux organisés , mieux rémunérés , fiers de leurs missions , avec des plans de carrière plus motivants , qu' un Etat suradministré et croulant sous les déficits . La puissance publique doit être un atout pour la prospérité , pas un frein . Elle ne doit pas réprimer la liberté mais la protéger . La liberté est le meilleur carburant des sociétés vivantes . C' est elle qui encouragera les créations d' emplois et relancera l' ascenseur social . Ceux qui aspirent au progrès et au dépassement de soi , ils ne craignent pas liberté : ils la demandent , pour prendre les commandes de leur vie . La libération de l' économie débloquera les salaires par plus de travail . Elle permettra d' équilibrer nos comptes publics en 2022 . Elle stimulera l' investissement par une fiscalité plus favorable . Et elle bénéficiera aux Français puisqu'elle nous permettra de redistribuer 10 milliards de pouvoir d' achat supplémentaire aux salariés , à travers la hausse du salaire net et une politique familiale renforcée . Cette libération de l' économie et du marché de l' emploi , je veux la mettre au service de notre jeunesse . Dans le monde qui se dessine , la principale matière première : c' est le savoir . Ce sont les qualifications , les compétences , la formation . Je veux refonder notre école qui doit transmettre les savoirs fondamentaux à tous nos enfants avant de multiplier les activités périphériques . D' autre part , et ce fut l' objet de mes visites d' aujourd'hui , je souhaite développer massivement l' alternance . Plus du quart des jeunes Français actifs sont au chômage , contre 8 % des jeunes Allemands . Or nous avons quatre fois moins d' apprentis en France qu' en Allemagne . Là-bas , 55 % des élèves d' une classe d' âge vont en apprentissage , et beaucoup de patrons sont d' anciens apprentis . En Allemagne , au Royaume-Uni , en Suisse , où le chômage des jeunes est moins élevé , l' apprentissage est résolument tourné vers les entreprises , qui sont très impliquées dans la définition des formations . Chez nous , l' alternance stagne , alors que c' est elle qui garantit l' employabilité , du secondaire jusqu' à l' enseignement supérieur , quelle que soit la filière suivie . Nous allons réagir . Je veux que l' apprentissage devienne la voie de droit commun pour l' obtention d' un bac professionnel . Ce serait un changement majeur . Concrètement , cela impliquera que les lycées professionnels ouvrent des sections d' apprentissage en leur sein , à l' image des centres de formation des apprentis . Si notre système fonctionne si mal , c' est notamment parce que tout le monde essaye de le gérer : l' Etat , l' Education nationale , les collectivités locales , les branches professionnelles . Il est temps de mettre de l' ordre dans ce labyrinthe . Et il est temps de donner plus de liberté aux acteurs de terrain . Ce sont eux qui sont les mieux à même d' adapter l' offre de formation aux besoins locaux . Je veux placer les lycées professionnels sous l' autorité des régions et des branches professionnelles . Quant à l' Etat , il doit conserver un rôle d' évaluation . Je veux aussi inciter les entreprises à recourir à l' apprentissage , en leur offrant un cadre plus attractif . Financièrement d'abord , en exonérant totalement de charges sociales l' embauche d' un jeune en alternance . Mais aussi juridiquement , avec un cadre administratif plus souple : est -il normal qu' un apprenti ne puisse pas monter sur un escabeau pour des raisons de normes , alors que le salarié le pourrait ? Il faut aligner et rapprocher les régimes . Et il faut revoir le rythme de l' alternance , pour le rendre compatible avec celui de l' entreprise , et pour que l' apprenti soit davantage présent dans l' entreprise que sur les bancs de l' école . Bref , je veux que l' apprentissage devienne une véritable voie d' excellence ! Quant aux contrats aidés , trop d' entre eux n' assurent pas aux jeunes leur insertion professionnelle . Seule l' entreprise crée vraiment des emplois durables . Je veux donc réorienter le budget de ces contrats vers l' alternance . Soyons innovants ! Je propose , par exemple , que ce redéploiement permette de développer des dispositifs de préapprentissage dès la classe de quatrième au collège , et d' alternance dans les écoles de la deuxième chance . Je souhaite que nous fassions tout pour rattraper chaque jeune en situation d' échec ou en marge du système , afin de les rediriger vers le monde de l' entreprise . Je veux aussi développer la formation professionnelle des personnes au chômage , notamment pour sortir de ces situations ubuesques où l' on constate des pénuries de main d' œuvre qualifiée alors que le chômage n' a jamais été aussi haut . Mes amis , Pour redresser la France , il faut enrayer le fléau du chômage , mais aussi refonder les piliers de la République . La famille est un des piliers de notre société . Elle a été dédaignée . Je la protégerai , je lui redonnerai ses droits et des moyens supplémentaires ! J' ai parlé de l' école et de la formation . L' Ecole de la République est aussi l' Ecole du civisme et de la citoyenneté . Elle a dérivé loin de cet idéal . Je refonderai l' école de la République en commençant par un symbole fort : le port d' un uniforme scolaire donnant à tous les élèves , quelles que soient leurs origines , le goût d' être ensemble , tous égaux , tous conscients d' être dans un lieu singulier qui exige discipline et fraternité : le lieu du savoir . La liberté sans l' autorité , c' est l' anarchie ! Je veux réaffirmer l' autorité de l' Etat , le respect du droit , la nécessité de l' ordre . La sécurité n' est pas une option : c' est un droit fondamental . Je veux des forces de l' ordre modernisées , respectées ; je veux une Justice ferme et rapide . Je rétablirai les peines planchers pour les récidivistes . L' impunité zéro doit être la règle . Que nos pompiers , nos policiers puissent être caillassés , injuriés , brûlés à coups de cocktails Molotov , que nos enseignants aient la gorge serrée avant d' entrer dans leurs classes , c' est la preuve qu' une part de notre jeunesse part en vrille . Où est le respect , où est la morale , où sont les familles ? Partout en France , et notamment chez nos concitoyens les plus modestes , ceux qui sont au cœur des cités difficiles , on demande de l' ordre , du travail , de l' éducation . On veut la loi de la République , pas la loi de la rue ! Aujourd'hui , les gens honnêtes ont peur et les délinquants n' ont pas froid aux yeux … C' est un comble . La peur doit changer de camp car il n' y a pas de démocratie sans sécurité . Je souhaite une république unie . La France n' est pas une addition de communautés , repliées sur elles-mêmes . La France a une identité , des coutumes , une langue , une culture , un drapeau ! Nous avons des différences mais nous n' avons collectivement qu' un destin . Un pays qui se disloque de l' intérieur , qui se désagrège en clans , en ghettos , un pays qui ne croit plus en son unité faute de savoir quelle est son identité , ce pays est condamné à s' effondrer . Dans un monde de 7 milliards d' habitants , les 66 millions de Français doivent faire bloc . Au fond de leur cœur , ils doivent avoir ce sentiment d' être les membres d' une même famille qui s' engage ensemble pour la France : la France , ce vieux et magnifique pays qui est un peu notre chair à tous car nous ne sommes pas des pions anonymes , nous sommes les successeurs d' une histoire étonnante qui a fait de nous l' une des toutes premières puissances du monde . Il a fallu des siècles pour faire la France . A aucun moment notre nation ne s' est définie de manière spontanée . Nous avons fédéré nos provinces , nous nous sommes imposé une langue commune , nous avons défini et protégé des frontières , nous avons su marier nos influences latines , germaniques , atlantiques , méditerranéennes , d' outremer … Tout ceci pour dire , mes amis , que la France n' est pas le fruit du hasard , mais le fruit d' une volonté de rassembler . Rassembler les Français , cela va de soi , par-delà leurs origines sociales et leurs opinions . Mais aussi rassembler tous ceux qui , au cours des siècles , ont rejoint notre nation pour y travailler , pour y vivre , mais aussi parfois pour la défendre au péril de leur vie . Vous connaissez la formule de la légion étrangère – &laquo; , non par le sang reçu mais par le sang &raquo; . La France a accueilli et assimilé des générations d' étrangers qui lui ont apporté leur énergie . Notre pays avait besoin d' eux et il pouvait compter sur son économie pour subvenir à leurs besoins . Aujourd'hui , ça ne marche plus . Sur cette question de l' immigration , j' ai des convictions que je veux vous faire partager . D'abord , je regarde froidement les faits et je dénonce l' angélisme de ceux qui estiment qu' il n' y a pas de problème .A l' évidence , les capacités d' accueil de notre pays sont saturées . Nous sommes en crise économique et priorité doit être donnée à ceux qui , chez nous , cherchent du travail ; notre système d' intégration ne fonctionne plus ; dans certaines de nos écoles , la mixité sociale a disparu ; dans nombre de nos quartiers le communautarisme s' étend , sans parler de ceux qui instrumentalisent la religion pour défier les lois de la République . Ne pas voir cela , c' est être dans le déni . Et le déni , c' est la porte ouverte au scénario de l' explosion sociale , et ce serait un malheur pour tous .Je stopperai l' immigration sans intégration , sans assimilation . Les étrangers ont des devoirs avant d' avoir des droits . Je veux réduire l' immigration à un strict minimum en la limitant par une politique de quotas . Ceux que nous accueillons sont les bienvenus s' ils respectent leur hôte . Sinon ils n' ont rien à faire chez nous . La France accepte les vrais réfugiés politiques mais sera inflexible avec les clandestins . Je veux que l' aide au développement soit conditionnée à la coopération des Etats d' origine en matière de maîtrise des flux migratoires et de retour de leurs ressortissants . Je veux protéger notre République contre le totalitarisme islamique . Ceux qui refusent d' employer le mot &laquo; totalitarisme &raquo; et ne parlent que de &laquo; terrorisme &raquo; confondent l' arme et l' idée . Ils se voilent la face et trompent les Français ! Le totalitarisme est une idéologie qui veut la fin de notre liberté . Le terrorisme n' est que le moyen détestable utilisé dans cet objectif . Nos concitoyens musulmans doivent être protégés de cette idéologie , à laquelle ils sont loin de tous adhérer , mais qui les infiltre et les prend en otages . Je demande un strict contrôle administratif du culte musulman , tant que son ancrage dans la République ne sera pas pleinement achevé . J' exige la dissolution immédiate de tous les mouvements qui se réclament du &laquo; salafisme &raquo; ou des frères musulmans . Je veux qu' on retire leur nationalité aux Français qui sont partis nous combattre en Syrie et en Irak . Mes amis , Ce qui est en jeu avec le redressement économique de notre pays et sa nécessaire union autour des principes de la République , c' est notre souveraineté . La France doit être souveraine , dans une Europe souveraine . Notre pays doit pouvoir s' appuyer sur ses forces militaires . Leurs moyens seront augmentés . Il doit poursuivre une politique étrangère fondée sur l' intérêt des Français . La France doit redevenir la première puissance européenne et aider l' Europe à devenir la principale puissance mondiale . Je ne parle pas de l' Europe communautaire et de ses normes byzantines pour fixer la taille des choux fleurs . Je parle de l' Europe des nations , unies pour défendre la civilisation européenne . Avec Donald Trump notre Continent ne sera sans doute plus la priorité des Américains . Et avec la tragédie de Berlin , une certaine idée du pacifisme a sans doute vécu en Allemagne . A nous , Français , de remobiliser l' Union européenne autour de priorités stratégiques . D' une part , en matière de défense et de sécurité collective : nous devons mettre nos forces en commun pour protéger l' Europe face aux nouvelles menaces . D' autre part , en matière d' innovation et de numérique : nous devons viser une véritable souveraineté numérique européenne . Les investissements européens dans les infrastructures et métiers de demain doivent être massivement concentrés sur le numérique et les réseaux . L' Union européenne doit se doter de normes et de standards communs , sans attendre que ceux des grands groupes américains s' imposent à nos entreprises . Et il est indispensable de garantir l' autonomie stratégique de l' Europe dans certains domaines clés comme les microprocesseurs , la nanoélectronique , l' électronique quantique , l' identité numérique , les télécoms . J' évoquerai tous ces sujets à la fin du mois avec la chancelière allemande . C' est par un accord franco-allemand que l' on pourra enclencher une dynamique européenne . Mes chers amis , Il nous reste trois mois ! Trois mois pour redonner espoir à la jeunesse . Trois mois pour partager notre idéal . Trois mois pour balayer les frilosités , les conservatismes , le politiquement correct . Trois mois avant de tourner enfin la page du déclin et du fatalisme . Trois mois pour aller à la rencontre de chaque Français et lui redonner le goût du progrès . Trois mois pour être à l' avant-garde de la révolution du bon sens . Ces trois mois pèseront lourd dans l' avenir de notre pays . Vivez -les intensément . Vivez -les avec détermination , vivez -les avec la passion des femmes et des hommes libres . Venez avec moi ouvrir une nouvelle page de notre histoire ! Que votre volonté soit l' étincelle des autres volontés . Entrez dans cette aventure éternelle d' une nation qui toujours se redresse et s' élève : l' aventure de la France , notre France . Vive la République , vive la France ! "    
    ]
fillon_test = [
    "Mes chers compatriotes , mes amis , Des quartiers de Paris , de toutes nos régions , de tous les âges , de toutes professions , vous êtes venus en nombre . Merci aux orateurs qui viennent de s' exprimer . Merci Valérie , merci à François Baroin , merci aussi à Jean-Christophe Lagarde qui représente l' UDI . Merci tout particulièrement à Alain Juppé qui est au rendez-vous de l' unité , et j' ai une pensée pour Nicolas Sarkozy à qui j' ai parlé hier qui aimait tant être parmi vous . Nous sommes rassemblés . Vous êtes unis dans les couleurs du bleu , du blanc et du rouge ! Quelque chose de profond nous relie , quelque chose qui passe au milieu de nous comme un fluide : c' est l' instinct de la France ! Votre mobilisation est extraordinaire . Elle me touche , elle me confère aussi des devoirs . A vous , je fais le serment de donner tout ce qu' un patriote peut donner à son pays . Je vous donnerai tout : ma détermination , ma vaillance , ma volonté de vaincre . En retour , je vous demanderai d' avancer tête haute . Ne cédez à aucune intimidation , ne lâchez aucune de vos valeurs . Soyez plus grands que ceux qui nous mitraillent . C' est plus que ma personne qui est dans leur viseur : c' est une haute idée de la France qu' on veut abattre en vol . Mais on n' abat pas une idée qui est au fond de l' âme française . Cette idée a plus de 1000 ans ; cette idée a toujours 20 ans ; cette idée a pour noms , liberté , honneur , grandeur , fierté . Dans cette salle , vous êtes plus de 13.000 , 13.000 fiertés françaises que rien ne pourra arrêter ! Il fallait cette démonstration de force pour faire entendre la voix des militants de la France . Cette voix puissante – votre voix – se propagera partout dans le pays . Elle amplifiera la campagne que j' ai engagée , elle touchera ces millions de Français qui attendent le sursaut . Il fallait , mes amis , ce rassemblement populaire pour rappeler à ceux qui nous jugent et nous attaquent , de quel bois nous sommes faits . Je veux que nos adversaires vous entendent , qu' ils vous respectent . Je veux que votre clameur franchisse les murs de cette salle . Haussez le ton , haussez la voix , qu' ils comprennent que rien ne stoppe un peuple qui se dresse pour sa fierté ! Je viens de loin et on ne m' intimidera pas . Je repense à ma première campagne électorale . J' avais vingt-six ans . J' étais inconnu . C' était une élection cantonale . En 1981 . Au mois de février . Au petit matin , je prenais ma voiture qui toussait au démarrage pour aller coller mes affiches et sonner aux portes des maisons . Parfois , on me fermait la porte au nez , souvent on m' offrait du café dans la cuisine . La politique , c' est un long chemin . Un chemin rude . Depuis le début , Pénélope est à mes côtés , avec discrétion , avec dévouement . J' ai construit mon parcours avec elle . Nous n' avons rien à cacher , notre seul compte en banque est au Crédit Agricole de Sablé . A trois mois de l' élection présidentielle , comme par hasard , on construit un scandale . A travers Pénélope , on cherche à me casser . Moi , je n' ai peur de rien , j' ai le cuir solide . Si on veut m' attaquer qu' on m' attaque droit dans les yeux , mais qu' on laisse ma femme tranquille . Devant vous mes amis je veux dire à Penelope que je l' aime et que je ne pardonnerai jamais à ceux qui ont voulu nous jeter aux loups . Pour résister à cette entreprise de démolition , je vous dis , du fond du cœur , merci d' être là , avec moi , avec nous ! Mes amis , J' aime la France et les Français . J' en ai croisé , des visages … Je n' ai rien oublié . Je n' ai pas oublié les mots courageux de cet agriculteur qui travaille 70 heures par semaine pour 800 euros par mois . J' ai en mémoire le témoignage de cet officier français , en mission au Sahel , qui racontait comment sa compagnie avait reconstruit l' école détruite du village . Ça tirait encore dans tous les coins , mais pour cet officier français , pas de village sans école ! J' ai rencontré plusieurs fois ces dernières semaines des membres d' Emmaüs . Mes parents en étaient . La générosité et l' humilité de ces bénévoles est une leçon . J' ai vu tant d' entrepreneurs créatifs , tant de salariés et de fonctionnaires dévoués , tant de chercheurs étonnants , qu' il m' est insupportable d' entendre dire que la France est fichue . Le Vendée Globe , cette course du monde à la voile , en solitaire , s' est conclue par la victoire d' un marin français , Armel le Cléach . Son exploit nous donne le goût de l' aventure et du dépassement de soi . Paris est candidate à l' organisation des JO de 2024 : je vote pour l' enthousiasme , et je dis à nos handballeurs qui vont bientôt débuter leur finale : cap vers la victoire ! Les Français sont formidables et le jour où la bureaucratie arrêtera de les &laquo; emmerder &raquo; , ils le seront encore plus . Et puis , il y a eu sur mon chemin des instants qui ne s' oublient jamais . Jamais ! Le camp d' Auschwitz , au côté de Simone Veil . Nous étions tous les deux et Simone me murmurait ce qu' elle avait vécu à 16 ans , dans cet enfer . Cette visite aussi du cimetière de Verdun parmi ces milliers de croix blanches . Chaque croix porte un nom , chaque nom parle d' une vie donnée à la France . Les Français sont -ils prisonniers de leur mémoire ? Non , ils ne le sont pas et je veux bien au contraire , qu' ils en soient les gardiens . Ils n' ont pas à rougir de leur Histoire , il n' y a pas à s' excuser d' avoir eu des héros Je suis candidat à l' élection présidentielle pour la France . Je ne suis pas un produit marketing , ma parole ne m' est pas soufflée par le microcosme , elle me vient de tous les Français qui m' ont fait partager leurs difficultés et leurs espoirs . J' ai ressenti leur peur du déclassement . Je suis porté par leur énergie , leur désir de justice . Je veux incarner la fierté d' une nation qui ne se laisse pas abattre . Je m' adresse à tous les Français . Il n' y a pas un peuple de droite contre un peuple de gauche . Il n' y a qu' un peuple , le peuple français capable de toutes les grandeurs dès lors qu' on lui dit la vérité et qu' on lui fixe des buts clairs et justes . Je veux convaincre la jeunesse que l' audace du changement est en nous . Elle n' est pas chez les défenseurs du système ; elle n' est pas chez ces utopistes qui rêvent l' impossible pour ne pas construire le possible . Je veux encourager les entrepreneurs , redonner confiance aux agriculteurs et au monde rural qui ne sont pas les derniers vestiges d' une France disparue . Je veux entendre la voix des Outre-Mer qui réclament respect et progrès . Ils sont une part infiniment précieuse de la France dans son étonnante diversité . Je veux parler aux français de l' étranger qui sont souvent les meilleurs défenseurs de notre culture et de notre savoir-faire . Je veux convaincre les ouvriers que la compétitivité est la clé de leurs emplois et de leur pouvoir d' achat . Je veux réveiller la vocation des fonctionnaires , des policiers , des enseignants , des infirmières qui servent le bien public . Je veux être aux côtés des familles qui sont le creuset de la France qui veut de l' éducation , de l' autorité , du travail pour ses enfants . Je veux parler pour les chômeurs , les sans grade , les sans carnets d' adresse , les courageux , tous ceux pour qui le changement est une opportunité de casser le plafond de verre qui fige leur destin . Je m' adresse aux abstentionnistes qui s' abstiennent parce que rien ne change . Je m' adresse aux électeurs du Front national qui pensent que tout part à vau l' eau . Je m' adresse aux électeurs socialistes qui se mordent les doigts . Je veux convaincre et rassembler . On me décrit comme le représentant d' une France traditionnelle . Celui qui n' a pas de racines marche dans le vide . Je n' oublie pas ce que je suis et d' où je viens . Je ne renie rien de ce qu' on m' a transmis , rien de ce qui m' a fait , pas plus ma foi personnelle que mes engagements politiques . Mais je sais aussi que nous aurons besoin de tout le monde , qu' il me faudra être au service de tous , et croyez -moi , j' y suis prêt . Je récuse ces divisions inutiles dont s' abreuvent plusieurs de mes concurrents . N' opposons pas les riches aux pauvres , les entrepreneurs aux ouvriers , les croyants aux agnostiques , ceux qui ont choisi la France et ceux qui l' ont reçue en héritage . Je veux que chacun sache qu' une seule chose compte pour moi , sans considération d' origine , d' opinion , de religion : le désir de servir et de rendre à la France ce que nous avons reçu d' elle . Nous sommes français . Tous français . Parmi nos parents , certains n' ont jamais quitté leur village , d' autres viennent du bout du monde , mais nous partageons le même espoir : l' espoir d' une vie libre et digne . Je ne cesserai pas de vous parler de la France , de cette France que tous peuvent aimer . Une France entière et fraternelle , universelle et particulière , mais toujours à reprendre , à inventer , à sauver . C' est la même France , du baptême de Clovis à la déclaration des droits de l' Homme . C' est la France des cathédrales et des barricades de Paris . Celle des rois et des sans culottes . C' est la France de Chateaubriand et celle de Kessel , celle de de Gaulle et celle de Césaire . La France ne nous demande rien . Nous lui devons tout . Pour la servir , nous aurons besoin de courage , de volonté et d' amitié et rien de cela ne nous manque . Les défis sont immenses . Que voyons -nous autour de nous ? D'abord , un pays défait , profondément défait , dans son image et , dans l' estime qu' il a de lui-même . Un pays plongé dans une crise morale sans précédent . Un pays sans chef . Nous voyons un pays rongé par le chômage , la précarité , accablé par une dette insoutenable , un pays où l' on n' investit plus , où l' on imagine à reculons . Nous voyons une classe politique effrayée à l' idée d' affronter ce monde nouveau . C' est la révolution numérique , qui va remodeler en profondeur nos sociétés , nos manières de vivre . Plutôt que de trembler devant cette révolution , il faut lui laisser sa chance , il faut la maitriser au profit de tous . C' est la mutation écologique et l' incontestable réchauffement climatique qui met notre planète en péril . Avec responsabilité , avec des technologies nouvelles , on peut réagir et aller vers une économie décarbonée . Les Etats qui chercheraient à rompre les accords de Paris sur le climat devront en rendre compte devant l' humanité . Autour nous , c' est aussi la pression d' un islam radical expansionniste qui défie notre civilisation . Ces fanatiques sont cruels , leur projet macabre nous rappelle ce que nous avions oublié : nous avons des valeurs et ces valeurs se défendent . Et puis , il y a l' Amérique de M. Trump , qui semble -t-il , a décidé de faire céder quiconque ne se plierait pas à ses intérêts . Les Américains sont nos alliés et nos amis , mais la France n' est vassale de personne et il est temps que l' Europe se réveille . Ces défis sont gigantesques . Pour les traiter , la plupart des politiques feignent de croire qu' il suffira d' appliquer les recettes anciennes . Ne rien faire , mais avec de bons chargés de communication . Non , pas de communication , pas d' échappatoire : nous sommes 66 millions de Français dans un monde de sept milliards d' habitants . Personne ne nous fera de cadeaux . Notre seule marge de manœuvre réside dans notre capacité à nous rassembler , à travailler plus , à travailler tous , à nous saisir de toutes les potentialités scientifiques et technologiques dont l' esprit français est capable . Notre marge de manœuvre est dans notre inflexible volonté d' être souverain , écouté et respecté de par le monde . Etre debout ou à genoux : voilà la question . Ma réponse est claire : debout toujours , à genoux , jamais ! Je veux , avec vous , faire de la France une puissance gagnante . La première puissance européenne d' ici 10 ans . Nous allons faire ce que nous n' osons pas faire depuis trop longtemps : faire confiance à l' énergie française . Nous allons nous détacher de cette chape de plomb idéologique qui a piétiné le goût du travail , piétiné la récompense du mérite , piétiné ce qu' il y a de plus lumineux en nous : la liberté . La liberté , c' est le carburant des sociétés vivantes . Le temps est passé où l' Etat pouvait diriger un pays comme on le fait d' un régiment . Le devoir de l' Etat , c' est aujourd'hui de rendre les choses possibles , ce qui est très différent . Le monde vertical d' autrefois disparaît sous nos yeux . Un monde nouveau se dessine , fait de mille initiatives , un monde où d' autres solidarités apparaissent , où la créativité est décisive . Si nombre de jeunes , étudiants , entrepreneurs , artisans baissent les bras ou s' expatrient , c' est parce qu' ils sentent que la France des rentes et des structures ne les aime pas , ne les comprend pas . C' est à cela que nous mettrons fin ensemble . Nous allons faire de notre pays celui de l' audace plutôt que celui de la bureaucratie , le pays qui investit dans l' avenir plutôt que d' étouffer sous sa dette , le pays du dialogue social plutôt que celui des oukases syndicaux , le pays des initiatives plutôt que celui des circulaires . D' une société où tout est nivelé , figé au nom d' un misérabilisme mou , nous passerons à une société d' engagés et d' engagements . Avec tous les Français qui veulent réussir , je fonce pour une France du plein emploi , une France productive . C‘est ma priorité absolue ! Il faut travailler tous pour produire plus et gagner plus . Et il faut en parallèle remettre nos finances publiques en ordre afin de mettre un terme au cercle vicieux : toujours plus de dépenses , donc toujours plus de déficits et donc toujours plus d' impôts et de charges . Je baisserai massivement les charges qui pèsent sur le coût du travail et sur la feuille de paye des Français : le gain sera d' au moins 250 euros par an et de 500 euros pour les couples qui travaillent . Je généraliserai la participation avec une priorité pour les salariés des petites entreprises . Je sortirai des 35 h , sans démagogie , sans diktat , puisque je donnerai aux entreprises la liberté de négocier avec leurs salariés . Je généraliserai l' apprentissage , refonderai la formation professionnelle , créerai le statut de travailleur indépendant , développerai le statut d' autoentrepreneur , je miserai à fond sur les technologies , je ferai de la France un champion mondial de l' intelligence artificielle , je réformerai l' assurance chômage . Je vais libérer l' activité de tous les secteurs qui ont été bloqués par le matraquage réglementaire et fiscal : l' agriculture , l' industrie , les services , la construction . Nous effectuerons une marche de fond pour améliorer tous les facteurs de production , du travail à l' innovation en passant par le financement . Je veux plus de confiance pour ceux qui font tourner les moteurs de l' économie française . Je préfère valoriser les investisseurs français plutôt que de ramper devant les fonds de pensions étrangers et ceux de pays soupçonnés d' aider les totalitaires islamiques . La meilleure politique sociale c' est celle qui donne du boulot ! Je veux agir pour les jeunes qui se cassent les dents sur la rigidité du code du travail . Agir pour les ouvriers qui préfèrent travailler un peu plus que de ne plus travailler du tout . Et cette restauration de notre production permettra de distribuer les richesses créées au lieu d' égaliser la misère comme aujourd'hui . Mais tout cela ne marche que si l' Etat , en parallèle , fait des efforts d' organisation et de gestion . Salariés , indépendants , fonctionnaires , on est tous sur le même navire . Pour le remettre à flot , il faut s' y mettre ensemble . Je veux moderniser les fonctions publiques pour les rendre plus efficaces et pour améliorer leurs conditions de travail . Passer progressivement nos fonctions publiques à 39 heures pour qu' elles soient plus efficaces au service des Français , c' est raisonnable , c' est du bon sens ; c' est au surplus redonner à nos fonctionnaires des perspectives de carrière . Je préfère un Etat fort , avec des agents moins nombreux , plus motivés et mieux rémunérés qu' un Etat suradministré , croulant sous les déficits . Nous avons près de 6 millions d' agents publics , c' est plus que l' Allemagne qui compte pourtant plus d' habitants que nous . Réduire notre nombre d' agents publics de 8 % sur un quinquennat pour réduire nos déficits , c' est raisonnable , c' est du bon sens . Je vois les gens débattre à l' infini sur ce chiffre et je suis surpris . Mon but , ce n' est pas de réduire des postes pour le plaisir . Mon but , c' est de rendre efficaces nos structures publiques souvent vieillies , empilées les unes sur les autres … Je suis l' ennemi de la bureaucratie , mais je ne suis pas celui des fonctionnaires ! Il y a peu de tâches aussi nobles que celles du professeur , de l' infirmière , du magistrat , du soldat , du policier . Je respecte ceux qui ont voulu , lorsqu' ils étaient jeunes , que l' intérêt public leur serve de boussole . C' est à eux aussi que je pense . Le discrédit de l' Etat bureaucratique rejaillit sur eux et c' est profondément injuste . Je leur rendrai leur fierté . Toute cette stratégie productive me distingue de mes concurrents . A gauche , on rêve des 32 heures et d' un revenu universel : c' est l' assistanat universel . Ces gens-là ont enterré nos traditions ouvrières . Ils n' ont jamais mis un pied dans une entreprise agricole ou une start-up . Sont -ils allés voir ce qu' il en était à la Silicon Valley ? J' en doute ! Face au camp de l' anesthésie , je suis le candidat du travail et du pouvoir d' achat , car un pays qui travaille plus , gagne plus . La fin des 35 heures dans le privé , le passage aux 39 h dans le public , l' augmentation du salaire direct grâce à la baisse des charges , le relèvement du quotient familial pour les familles : tout ceci va améliorer le pouvoir d' achat des Français , notamment celui de nos concitoyens modestes . En réalité , à gauche , on ne croit plus à la croissance . Sans croissance , il ne reste donc plus qu' à partager les miettes . Cette capitulation n' est pas seulement démoralisante pour tous ceux qui travaillent dur pour améliorer leur vie , elle est aussi destructrice pour notre pacte social . J' ai quelques scrupules à rappeler au parti socialiste et à son prochain candidat , que c' est la croissance par le travail qui finance notre protection sociale . Moi je veux le meilleur pour la solidarité , et pour ce faire je veux nous en donner les moyens en agissant , pas en rêvant . Mon ambition sociale , c' est une retraite digne assurée à tous nos anciens . Pour cela il n' y a pas d' autre choix que d' aller progressivement vers la retraite à 65 ans . Nous vivrons tous plus longtemps – en moyenne au-delà de 80 ans – il est donc raisonnable de cotiser un peu plus longtemps pour assurer nos vieux jours . Mais il n' est pas possible qu' en France nos anciens les plus modestes en soient à renoncer à leurs repas pour payer leurs factures . Mon gouvernement avait beaucoup fait pour les petites retraites . Je poursuivrai l' effort avec une revalorisation de + de 300 euros pour les pensions de moins de 1000 euros et d' au moins 600 euros pour les petites pensions de réversion . On m' a fait le procès de vouloir mettre à mal notre assurance maladie et notre système de soins . C' est faux ! Je veux une santé de qualité accessible à tous . Ce que je dénonce , c' est la langue de bois de ceux qui se fichent des déserts médicaux , qui se fichent des pathologies non remboursées , qui se fichent des urgences débordées , qui se fichent de savoir que les personnels médicaux sont à bout , qui se fichent de savoir que la médecine libérale en a marre d' être maltraitée , et puis , qui se fichent des déficits . Les fossoyeurs de la sécurité sociale sont ceux qui s' opposent à toute réforme . Tout comme d'ailleurs les fossoyeurs de l' emploi sont ceux qui ne veulent pas toucher à la durée du travail ou à son coût . Je veux une santé pour tous , une santé capable de répondre au vieillissement de la population et au prix croissant des nouveaux médicaments . Et pour cela , j' agirai , en concertation avec les professionnels de la santé . Et puis , cet effort productif auquel je convie la nation doit permettre de répondre aux causes les plus humaines , les plus justes . Celle de l' autisme , celle du handicap . 8 millions de Français sont frappés par le handicap . Ce fut une des batailles du Président Chirac . Ce sera mon combat . Je pense à ces familles qui doivent être soutenues à domicile , à ces enfants qui doivent pouvoir être scolarisés comme les autres , qui doivent pouvoir être pleinement respectés . J' aime la phrase de Saint Exupéry : &laquo; Si tu diffères de moi , mon frère , loin de me léser , tu m' enrichis &raquo; . Nous venons de célébrer l' anniversaire de la mort de l' Abbé Pierre . Sommes -nous au rendez-vous de son appel au secours de l' hiver 1954 ? La pauvreté continue de s' étendre , le chômage fait ses ravages . J' ai rencontré beaucoup d' associations . Leur message fut souvent le même : nous avons besoin de clarté car le système est trop bureaucratisé ; nous avons besoin de continuité car les aides de l' Etat sont en accordéon ; nous avons besoin de liberté pour coller aux réalités du terrain . J' engagerai un plan sans précédent d' action pluriannuel de lutte contre la pauvreté et l' exclusion pour répondre à ce message . Mes amis , Il faut renforcer l' unité française . La France unie attachée à une certaine idée de la famille , a été dédaignée . Je la protégerai et lui redonnerai ses droits , car la famille est l' un des piliers de notre société . Elle est le premier cercle de la tendresse , elle n' est pas le laboratoire d' un Etat qui prétend prendre la place du papa ou de la maman . Je relèverai le quotient familial et favoriserai les donations aux enfants . La France unie se joue à l' Ecole de la République . Elle doit transmettre les savoirs fondamentaux autour desquels nous devons concentrer les efforts plutôt que de multiplier les activités périphériques . L' Ecole , préparant à un métier avec une valorisation massive de l' alternance . L' Ecole de la République , symbolisée par le port d' un uniforme scolaire donnant à tous les élèves , quelles que soient leurs origines , le goût d' être ensemble , tous égaux . Notre école est mal en point . Je suis en colère quand je vois son niveau baisser d' année en année dans les classements internationaux . Je suis en colère quand j' entends la gauche et les pédagogues qui persévèrent dans l' erreur et l' autosatisfaction . Pour l' apprentissage des mathématiques nous étions les premiers il y a vingt ans . Aujourd'hui c' est la Corée du Sud , Taiwan , Hong Kong , Singapour qui font la course en tête . Nous sommes en queue de peloton de l' Europe . Où seront nos ingénieurs , et quel sera leur niveau ? Et quel sera le rang de la France ? La gauche a fait passer l' idéologie avant le savoir . Elle a fait passer l' endoctrinement avant l' intelligence . Elle s' est attaquée à tout ce qui était synonyme d' excellence . Je comprends les enseignants qui ne se reconnaissent plus dans ce système opaque qui les traite comme des rouages d' une idéologie fumeuse . François Hollande nous avait fait miroiter le rêve français . Il nous a laissés le chaos français . Le chaos français , c' est , en plein état d' urgence , des forces de l' ordre mobilisées pour encadrer le camping sauvage de Nuit debout . C' est la construction d' un aéroport à Nantes toujours bloquée par des hors la loi . Ce sont des voitures de police incendiées au cocktail Molotov , des pompiers interdits d' accès dans certains quartiers . Ce sont des enseignants qui ont la boule au ventre en entrant dans leurs classes . C' est un pays où tant de peines de prison ne sont plus effectuées . C' est un pays où l' on invite les élèves à négocier leurs fautes d' orthographe . C' est un pays où tout se relâche , où les sanctions ne veulent plus rien dire , où il n' y a plus d' autorité . Je veux remettre de l' ordre dans le chaos français . La France unie exige l' autorité de l' Etat , l' autorité de la loi , la nécessité de l' ordre . Il faut mettre les mots sur les actes . Et je rappelle ce que j' ai déjà dit puisqu'il faut bien que ceux qui se jouent des lois l' entendent cinq sur cinq . Un délinquant n' est pas un mineur qui se cherche . C' est un délinquant ! Un criminel n' est pas une victime . C' est un criminel ! La sécurité doit être partout et pour tous , avec des forces de l' ordre modernisées et une Justice ferme et rapide . Je veux rétablir les peines planchers pour les récidivistes . Un délit : une sanction ! Voilà la règle qui doit être parfaitement claire pour les délinquants . La France unie n' est pas une addition de communautés et de clans . Elle a une identité , des coutumes , une culture , un drapeau . Le repli communautariste est un danger . Je me félicite que pascal Bruckner ait gagné son procès contre ceux qui veulent limiter notre liberté d' expression . Je serai le défenseur di droit des français à exprimer leur francité ! L' immigration sans intégration , sans assimilation , doit être stoppée . Dans la République française , les étrangers ont des devoirs avant de réclamer tous les droits . C' est une question d' unité nationale . Mais c' est aussi une question de respect car quand on entre dans la maison d' autrui , on n' y impose pas ses lois ! Je veux réduire l' immigration à un strict minimum , en l' organisant par quotas , en fonction de nos besoins économiques et de nos capacités d' intégration . A ceux que nous avons choisi d' accueillir , je veux que nous disions , avec précision et clarté , ce que nous exigeons d' eux : exercer un travail , s' approprier notre langue , respecter nos règles communes . Aux vrais réfugiés politiques , l' honneur commande que la France tende sa main . Mais avec les clandestins , elle doit être intransigeante car le droit de l' asile est sacré . Je veux que nous mettions les Etats d' origine en face de leurs responsabilités , pour qu' ils prennent leur part de la maîtrise des flux migratoires et qu' ils s' impliquent dans le retour de leurs ressortissants . Je conditionnerai l' aide au développement à leur coopération . L' Islam radical mine nos concitoyens musulmans . Il les infiltre , il les prend en otage . Les fanatiques détestent ce que nous sommes . Je les combattrai sans relâche et sans pitié . Je veux un strict contrôle administratif du culte musulman , tant que son ancrage dans la République ne sera pas pleinement achevé . Je veux la dissolution immédiate de tous les mouvements qui se réclament du &laquo; salafisme &raquo; ou des Frères musulmans . Je veux la clarification de nos relations avec l' Arabie saoudite et le Qatar qui abritent les doctrinaires de l' Islam radical et les marchands de &laquo; burkini &raquo; . Je nomme les choses . Je nomme le problème que nous avons avec l' Islam radical . Le catholicisme , le protestantisme , le judaïsme ne dénoncent pas les valeurs de la République . Après bien des affrontements , ces religions vivent en paix avec la République , avec la laïcité qui est notre protection commune . J' entends dire qu' il faudrait durcir les règles de notre laïcité . Mais pourquoi faudrait -il imposer à tous de nouvelles restrictions à la liberté religieuse , alors que seule la poussée intégriste qui enflamme le monde musulman est une menace pour notre société ? Ce n' est pas ma vision des choses . Ma vision des choses est très claire : la religion musulmane doit accepter sans condition tout ce que les autres religions ont accepté par le passé ! Je prendrais mes responsabilités mais il faut aussi que nos concitoyens musulmans se soulèvent contre ceux qui instrumentalisent leur foi . C' est une bataille idéologique et spirituelle qu' il faut livrer tous ensemble . Je dis &laquo; spirituelle &raquo; , parce que ça n' est pas en évacuant toute transcendance qu' on peut combattre la croyance ravageuse des fanatiques . Et ça n' est pas en laissant tomber lâchement les Chrétiens d' Orient qu' on créera les conditions de la paix en Orient . Dans les camps de réfugiés que j' ai visités en Irak , la prière des chrétiens et celle des musulmans se confondaient . Il n' y a aucune honte à se dire chrétien , aucune honte à se dire musulman , aucune honte à se dire athée ou agnostique . Ce qui importe c' est le respect , le respect de la loi et le respect de la France . Autour de nous , le monde est traversé de menaces . J' appelle de mes vœux une France souveraine , appuyée par des forces militaires dont les moyens seront augmentés . Ma vision des relations internationales est commandée par les intérêts de notre pays et par eux seuls . Quel que soit celui qui dirige le Kremlin à Moscou , et quel que soit celui qui dirige la Maison Blanche à Washington , l' amitié n' a rien à faire quand on défend la place de la France dans le monde et la sécurité des Français . Dans ce monde , les dirigeants passent , les régimes évoluent , les nations demeurent . La France n' y tiendra son rang que par une politique d' indépendance nationale . Elle doit être libre . Libre de combattre le totalitarisme islamique avec tous les Etats volontaires sans aucune exclusive . Libre de dire non aux Etats-Unis quand ils imposent leurs lois au mépris du droit international à nos entreprises . Libre de parler avec la Russie avec franchise . Certes le président Russe a commis des fautes , mais il ne finance pas les djihadistes qui tuent nos enfants et ne nourris pas le projet d' imposer la charia au reste du monde dans un califat de cauchemar . Evidemment , pour cela , il faut un vrai président de la République . La souveraineté française ne peut être incarnée par un homme qui navigue à vue dans son propre pays . Avec François Hollande , la France a payé le prix fort pour apprendre qu' on n' existe pas si on ne sait pas ce qu' on veut , ni où l' on va . L' Europe doit être pour la France un amplificateur de souveraineté . Je ne parle pas de l' Europe technocratique avec ses normes byzantines . Je parle de l' Europe des nations , unies pour défendre la civilisation européenne . Je me bats pour une Europe politique concentrée sur ses priorités stratégiques . Une Europe appuyée sur l' euro , qui doit devenir un atout entre les mains d' un gouvernement économique européen . Je milite pour une Europe qui tient ses frontières , pas d' une Europe passoire . Une Europe de l' innovation et du numérique qui ferait des européens la 1ère puissance technologique du monde . Je veux une Europe qui défend ses industries , ses emplois , face à la Chine , face aux Etats Unis . Je défendrai la préférence européenne pour nos industries , nos services et nos emplois . Dans les échanges commerciaux , je suis pour la réciprocité , pas pour la naïveté . Y compris entre nous , européens . La directive sur les travailleurs détachés a installé un véritable dumping social dans nos nations . Pour un travail égal , le salaire doit être égal et les cotisations sociales aussi . Si je suis élu , je renégocierai de fond en comble la directive sur les travailleurs détachés pour faire respecter ce principe . Si la négociation n' aboutit pas avant la fin de l' année , je suspendrai son application sur le territoire national . Donnons force à une Europe de la culture . J' ai lancé l' idée d' un grand musée à Strasbourg , exposant les chefs d' œuvre européens . La culture irrigue la vie , l' Europe a besoin de vie ! Je veux une Europe de la défense , dans laquelle chaque Etat , y compris l' Allemagne , prendra sa part du fardeau de l' effort militaire . Je veux &laquo; l' Europe mais avec la France debout &raquo; disait Philippe Séguin . Cher Philippe , aujourd'hui l' une et l' autre sont à terre et j' enrage de voir la civilisation européenne douter de son sort au milieu des orages . La France doit être redressée ; l' Europe , il faut la reconstruire et y parler d' une voix claire ! Voilà , mes amis , notre ambition française . La bataille électorale sera difficile . La violence des attaques qu' on nous lance est à la mesure des enjeux . Notre adversaire n' a pas un visage … il en a quatre ! Il y a plus de cinquante ans André Malraux disait : il y a quatre gauches , dont l' extrême-droite . Il avait raison . Nous y sommes . La gauche n°1 , la gauche pure et dure , rouge de chez rouge , on la connaît . C' est Mélenchon , le Fidel Castro de YouTube . La gauche n°2 , on la connaît aussi , c' est la gauche socialiste , c' est tout l' équipage des naufragés du Titanic hollandais qui viennent de s' entredévorer sur le radeau de la Méduse des primaires . Vous les avez entendus , peut-être … Ils nous expliquent qu' il faut aller toujours plus loin à gauche . Plus la réalité les met en déroute , plus ils ont foi dans leurs dogmes . Ils promettent des milliards de dépense . Ils proposent de travailler moins et de rêver plus Ils n' ont rien compris . Ils n' ont rien appris . La gauche n°3 , on la connaît moins . C' est Macron . Il dit avoir un projet ; je l' attends ! Il dit être réformateur ; à priori , il l' est moins que moi ! Il fait croire qu' il est seul et qu' il vient de nulle part ; en réalité , il a fait le programme de Monsieur Hollande mais aussi une grande partie de sa politique … Il est parti pour une échappée en solitaire , mais ses équipiers ne sont pas loin . Qui sont -ils ? Eh bien c' est toute l' équipe gouvernementale de monsieur Hollande … Bonjour la nouveauté ! Macron c' est le sortant , Macron c' est le bilan de Hollande . Macron c' est surtout le prototype des élites qui ne connaissent rien à la réalité profonde de notre pays ! Et puis il y a la gauche n°4 … C' est le Front National . Sortie de l' euro , abaissement de l' âge de la retraite à 60 ans , augmentation du salaire minimum , recrutement illimité de fonctionnaires : le programme du FN , c' est de la dépense publique à n' en plus finir , c' est le programme du parti communiste des années 70 ou celui du Front de gauche ! Pendant des années le FN a dit qu' il fallait préférer l' original à la copie . Eh bien , aujourd'hui , l' original c' est Jean-Luc Mélenchon et la copie c' est Marine Le Pen ! C' est la vieille politique de l' endettement irresponsable , des chèques en bois . C' est une politique de la ruine , ruine pour la France et pour les Français , pour les petits épargnants qui verront leurs économies partir en poussière , pour les petits patrons qui verront leurs charges sociales monter en flèche , pour toute notre économie qui sera attaquée de toutes parts par les nations du monde entier qui n' attendent que notre abaissement pour prendre notre place . Être une grande nation , ce n' est pas être isolé du monde , c' est avoir une position éminente dans la hiérarchie des puissances planétaires . C' est être capable de soutenir la comparaison . Le FN refuse toute comparaison et toute compétition mais l' histoire du monde est un combat et faire cavalier seul c' est la garantie de descendre en deuxième division . Je ne veux pas d' une France en deuxième division . Je veux une France qui fasse la course en tête . La vérité , vous la connaissez . Le parti du mouvement , c' est nous . Les conservateurs , c' est eux ! Ne nous laissons pas arrêter par la grande conjuration des impuissants . La France n' est pas à leur image . Elle n' a pas besoin de béquilles . Pas besoin de ceintures et de bretelles . Elle a l' énergie qu' il faut pour réussir seule . Nous avons besoin à la fois d' ordre et de mouvement . L' ordre , c' est l' attachement à nos valeurs , à nos principes , à ce qui nous constitue comme nation comparable à nulle autre . Le mouvement , c' est l' opportunité de créer , de jouir des fruits de son travail , d' innover , d' inventer des façons d' agir et de s' organiser . Je veux une société de nouvelles libertés où chacun dans son entreprise , son administration , son association se sent davantage maître de sa vie et plus utile aux autres . Vous voyez que nous sommes très loin de la purge ou des programmes technocratiques . Nous devons toujours garder à l' esprit le but ultime de nos réformes . C' est une France sauvée de l' éclatement , de la crise morale et du désastre économique . Une France qui respire l' initiative , qui retrouve sa voix et son image . Une France qui , s' étant réconciliée avec elle-même réconciliera tous les Français . Je m' attendais à voir monter en ligne les spécialistes de l' enfumage . Mais les Français connaissent la réalité . Ils savent qu' au fil des années , des abandons , des compromis , cette réalité est devenue inacceptable . La France qui nous a été léguée , celle que nous avons le devoir de transmettre , ça n' est ni un musée de l' histoire , ni une agence des droits . C' est une tradition du mouvement , de la conquête . C' est la langue même de l' esprit et de la liberté . Nous sommes tous invités à la parler , cette langue . On disait autrefois : &laquo; franc de nom , franc de nature &raquo; . Nous sommes le pays qui rend libre dans un monde où le plus souvent cette liberté n' a plus cours . C' est notre honneur . D' où qu' il vienne , chaque Français détient une part de cet honneur-là . Mes amis , maintenant je vais relever avec vous le défi du redressement national . Ce défi , je le gagnerai si toutes celles et tous ceux qui en ont assez de la torpeur et des mensonges se lèvent à mes côtés . Je donne rendez-vous à toutes celles et tous ceux qui ont la tête sur les épaules . J' appelle tous ceux qui ont des valeurs solides . J' appelle tous ceux qui ont dans le cœur la fierté d' être français . Sur le chemin de la victoire il y aura des vents contraires ; nous passerons à travers les bourrasques ; nous franchirons tous les ponts . Vous êtes partant et je le suis aussi , alors j' irai , avec vous , chercher les cœurs vaillants . Nous irons ensemble à la rencontre de toutes les forces de la France . Vous serez en première ligne de mon combat pour le redressement national . Cette histoire sera la vôtre . Vous allez vous engager à 100 % ! Votre énergie deviendra celle de votre ami , celle de votre ami deviendra celle de son compagnon , et de compagnon en compagnon , de cœur en cœur , de volonté en volonté , nous irons droit devant , droit vers la victoire , droit vers la grandeur , droit vers les Français , et nous les entrainerons , nous les emmènerons , et nous serons alors , tous ensemble , la France en action ! Vive la France et vive la République . "
]

hamon = [
    "Mes chères concitoyennes , mes chers concitoyens , À se laisser guider par l' air du temps , on pourrait croire que le cœur de la France s' est arrêté . Ce &laquo; qui marche politiquement &raquo; exalte les peurs instinctives , le rejet de l' autre ou le repli sur soi . Ce n' est pas le regard que je porte sur la France . Je lui vois un cœur robuste . Le problème de la France , ce n' est pas que son cœur se soit arrêté , c' est qu' il ne gouverne plus . Qu' il gouverne enfin à nouveau : c' est l' alternative de gauche , résolue et optimiste , que je porte . Voici le fruit du travail collectif que nous avons mené ces derniers mois . Cette réflexion se poursuivra pour aboutir à d' autres propositions . N' hésitez pas à y contribuer à votre tour . À portée de main , à portée de vote , existe un futur réaliste , où le travail n' est pas facteur de souffrance mais d' émancipation , où notre modèle de production intègre réellement l' exigence écologique et où la démocratie retrouverait le souffle qu' elle a perdu . Ensemble , je vous propose de faire battre à nouveau le cœur de la France . "
]

jadot = [
    "Les écologistes viennent de me désigner comme leur candidat à l' élection présidentielle . Je veux les en remercier chaleureusement . Je veux également remercier Michèle Rivasi , Cécile Duflot et Karima Delli pour la qualité de cette primaire . Ces dernières semaines , nous avons su démontrer notre respect mutuel et la force du projet que nous portons . Ensemble , avec les militantes et les militants d' Europe Ecologie les Verts , et avec toutes les personnes qui croient en l' écologie politique , nous allons construire la campagne présidentielle comme une aventure collective . Pendant la primaire , j' ai répété ma volonté de rassembler toute les familles de l' écologie : je veux poursuivre cette dynamique . C' est unie , ouverte , en regardant vers le grand large et les pieds sur terre que l' écologie sera conquérante . Je veux m' adresser ce soir , et tout au long de la campagne , à celles et ceux qui agissent chaque jour pour la planète et pour une société plus juste , à celles et ceux qui aimeraient agir mais qui n' en ont pas les moyens ou ne savent pas comment faire , à celles et ceux aussi qui ont pu être déçus par Europe Ecologie les Verts . Je suis déterminé à ce que cette campagne soit aussi la vôtre , que vous puissiez pleinement y participer , qu' elle ouvre une nouvelle page de l' écologie dans notre pays . Je mesure la responsabilité qui m' est confiée . Je la reçois avec gravité et détermination , mais aussi beaucoup d' enthousiasme . Avec gravité , car la France est au bord de la rupture . L' accroissement des inégalités , l' explosion des discriminations , l' affaissement de la démocratie , le dérèglement climatique , l' extinction des espèces , l' épuisement des ressources , les maladies liées aux pollutions : tout cela affaiblit notre pays . Nous souffrons de ne pas avoir de projet partagé et , faute d' horizon commun , la société explose dans un sauve-qui-peut généralisé . Jamais notre pays n' a connu une telle défiance , jamais la tentation du repli n' a été aussi forte . J' accueille aussi cette désignation avec détermination . Aujourd'hui s' ouvre à Marrakech la COP22 , une nouvelle conférence mondiale sur le climat . Si la diplomatie française peut être fière de l' Accord de Paris signé lors de la COP21 , chaque jour le gouvernement tourne le dos aux devoirs que cet accord impose . Obsession nucléaire , maintien des centrales charbon , choix du tout camion , diesel , soutien à l' agriculture la plus industrielle : ces choix maintenus depuis des décennies ne sont pas seulement néfastes pour le climat et pour la santé , ils nous privent de centaines de milliers d' emplois et d' entreprises dans tout le pays , maltraitent les animaux et font disparaître les services publics . Je reviens de Nantes où sont jugés les recours sur l' eau et les espèces protégées , déposés par les écologistes , contre les autorisations de travaux à Notre Dame des Landes . Que de temps perdu et d' entêtement autour de ce projet aussi inutile que destructeur -et toujours illégal au regard du droit européen . François Hollande confiait à des journalistes ne pas croire en ce projet : saisira -t-il enfin l' occasion d' y mettre fin , comme nous le demandons depuis des années ? Avec détermination donc , car , oui , les solutions sont là . Je crois en notre intelligence collective . La France n' est condamnée ni au désespoir , ni à la peur du déclassement . Il y a urgence à réformer les institutions pour déverrouiller la démocratie , à faire sauter les rentes pour libérer l' économie , à surmonter les conservatismes et les corporatismes pour débloquer la société . Ca tombe bien , notre pays regorge d' incroyables forces vitales , dans les quartiers , dans les villes et les campagnes , dans les entreprises et les associations . Nous devons absolument soutenir ces bâtisseurs d' avenir et leur donner les moyens d' essaimer . Dans cette campagne , nous déclinerons nos propositions pour faire face aux urgences écologiques , démocratiques et sociales . Pour prendre soin de la planète , nous devons réparer la société , la soigner . Protéger la nature , c' est nous protéger nous-même , c' est nous offrir une perspective d' avenir . Une France 100 % énergies renouvelables , une agriculture paysanne et biologique , une alimentation saine et conviviale , une santé protégée , des transports collectifs accessibles , une fiscalité juste qui favorise l' innovation , des choix industriels qui créent des emplois plutôt que d' en détruire … Car justement , l' écologie , c' est plus d' emplois , moins d' inégalités et de précarités . L' écologie c' est des territoires vivants et une économie relocalisée . Alors que des millions de personnes souffrent de ne pas avoir d' emploi , trop de salariés souffrent dans leur travail . Intensification des tâches , dégradation des conditions , précarisation des contrats : nous proposerons des mesures pour améliorer le travail , l' adapter aux capacités et aux talents des femmes et des hommes , le partager , et en garantir les droits . Nous voulons remettre le travail à sa juste place dans nos vies . Nous mettrons en débat notre projet pour une école égalitaire et émancipatrice . Nous voulons donner le pouvoir aux citoyennes et aux citoyens dans une République refondée . Nous affirmons qu' une société de confiance ne peut émerger sans le respect plein et entier du pacte républicain , des droits qu' il devrait garantir et des promesses qu' il porte pour chacune et chacun , arrivé hier dans notre pays ou présent depuis Astérix , habitant d' un quartier populaire ou d' une ferme du Larzac . Liberté , égalité , fraternité . La démocratie , c' est le contrôle de l' Etat par les citoyens , pas l' inverse . Le fichier global d' informations sur la population est une dérive de plus qui porte atteinte aux libertés publiques , donc à la démocratie . Nous lutterons contre toutes les discriminations qui minent la société , qu' elles soient sociales , ethniques , liées à l' orientation sexuelle , aux handicaps ou à l' état de santé , aux opinions politiques , religieuses ou syndicales . Et il est plus que temps de mettre fin aux inégalités entre les femmes et les hommes . L' écologie , c' est une société apaisée , fraternelle , ouverte sur sa diversité et sur le monde . Osons dire que nous voulons une société qui ne soit pas une compétition perpétuelle , pour avoir le droit de vivre dignement et de nous épanouir . Osons réclamer le droit au bonheur . Evidemment , la France doit se redonner un horizon européen . Cette petite mondialisation libérale qu' est devenue l' Union Européenne , technocratique et austéritaire , qui dresse des murs et des barbelés contre les réfugiés , porte en elle le risque de l' effondrement de notre civilisation alors qu' elle devrait en être le ferment . Le Tafta et le Ceta ne sont pas des solutions , mais font partie du problème . Relançons massivement l' investissement utile pour tisser de nouvelles solidarités sur notre continent , entre l' Est et l' Ouest , le Nord et le Sud , au sein de nos pays et avec le monde qui nous entoure ! La France , quant à elle , doit enfin nouer des relations saines avec les pays du Sud . Ni nostalgie régressive de la colonisation , ni Françafrique qui perdure comme au Gabon . Solidarité , soutien à la démocratie , intérêts mutuels pour un développement soutenable ! Enfin , je mènerai cette campagne avec enthousiasme . Je veux rassembler les Françaises et les Français autour d' un projet positif , un projet de sens et de cohérence ! Je veux qu' ensemble nous passions un nouveau pacte avec les Françaises et les Français , une nouvelle alliance : celle d' une écologie à la hauteur des défis de la planète et au cœur de la vie quotidienne , une écologie crédible , fière de ses combats et de ses convictions , aimable , populaire et humaniste , ni culpabilisante , ni punitive , mais aimable et bienveillante . Nous voulons redonner à chacune et à chacun sa dignité , la maîtrise de sa vie , l' envie et le pouvoir d' agir . Nous donnerons à voir de nouveaux possibles à celles et ceux qui n' attendent qu' un signe pour retrouver l' espoir , qu' une occasion pour réinvestir l' action collective et politique . Toutes et tous ne se disent pas &laquo; écologistes &raquo; , mais peu importe : ils et elles sont prêts à agir si on leur en donne les moyens , s' ils reprennent possession des leviers du changement . Le récit de l' écologie que je vous propose de porter ensemble pour l' élection présidentielle est profondément subversif , exaltant et pragmatique : mobiliser les énergies vitales de la société pour nous réconcilier avec la nature et avec l' avenir , pour nous réconcilier entre nous , pour nous réconcilier avec nous-mêmes ! La campagne des écologistes commence ce soir . Face aux artisans du renoncement , aux adeptes du &laquo; c' était mieux avant &raquo; et aux prêcheurs de haine , nous allons semer l' espoir d' une société de confiance . Je suis candidat pour que demain soit mieux qu' aujourd'hui , avec vous , et grâce à vous . "
]

lepen = [
    "Chers amis , Merci à tous d' être venus si nombreux à cette nouvelle convention . Merci également à tous les intervenants pour leurs réflexions très intéressantes et leur contribution au débat sur ce thème majeur qu' est la santé . Thème majeur , en effet , puisque la santé est une des rares problématiques qui nous concerne tous , sans exception . Cela dit , si nous sommes tous concernés , nous ne sommes malheureusement pas tous égaux face à la maladie et sa prise en charge . Les inégalités de santé s' accroissent année après année : entre les hommes et les femmes , entre les jeunes et les moins jeunes , entre les Français les plus favorisés et nos concitoyens en situation de précarité , entre les citadins et les ruraux . L' hôpital public est au bord de la faillite , la dette des hôpitaux a triplé en dix ans , souvent par le biais d' emprunts toxiques . Les soignants sont soumis à des conditions de travail déplorables , et plus un mois ne se passe sans qu' un journal local rapporte le suicide d' un soignant hospitalier . L' hôpital qui voit peser de surcroît sur lui le poids d' une immigration de plus en plus lourde à assumer . Ne parlons pas des urgences , encombrées de patients mal orientées , qui ne nécessitent pas une prise en charge hospitalière , et où dans plus de la moitié des cas , l' attente est supérieure à deux heures . N' oublions pas qu' il y a un peu plus d' un mois , Marisol Touraine annonçait un plan d' économie de 1,5 milliard d' euros sur les hôpitaux … autant dire que la Ministre achève le malade . La médecine de ville , quant à elle , attire de moins en moins les jeunes médecins , en raison de charges financières et administratives croissantes et d' une augmentation du coût de la vie plus rapide que celle des tarifs conventionnés . Les décisions arbitraires du ministère , imposées sans concertation avec les acteurs de terrain , finissent par exaspérer des professionnels , qui ont l' impression qu' on essaie de les court-circuiter . Côté patients , l' accès aux soins s' est détérioré depuis la crise de 2008 , et les inégalités de santé ne cessent de se creuser . Les Français les plus défavorisés subissent une double peine sanitaire : non seulement ils vivent moins longtemps que nos compatriotes les plus favorisés , mais ils vieillissent également en plus mauvaise santé ; alors qu' un cadre peut espérer passer 9 ans de sa retraite en bonne santé , un ouvrier n' atteindra en moyenne pas la retraite sans trouble de santé . A tous les niveaux , les conséquences néfastes de décennies de politiques de santé court-termistes et sans vision stratégique se font sentir , au détriment des patients et des professionnels de santé . UMP , PS , même bilan , au final ; mais qui en doutait ? Et ce que certains envisagent pour l' avenir me laisse coite … Autant vous dire qu' à lire le projet de M. Fillon , j' en ai la nausée … et j' espère que celle -ci passera vite , parce qu' il semble que les troubles digestifs fassent partie du &laquo; petit risque &raquo; que l' Assurance Maladie ne devrait pas avoir à prendre en charge , selon Dominique Stoppa-Lyonnet , conseillère santé de l' ex-premier ministre . Oui , vous avez bien entendu . Le point principal du projet de François Fillon pour la santé est de cantonner l' Assurance Maladie aux pathologies graves ou de longue durée , le reste devant être délégué aux assurances complémentaires . Le docteur Fillon nous prescrit donc un modèle ultralibéral à l' américaine , où les riches auront les moyens de se soigner correctement , où les pauvres bénéficieront de soins de piètre qualité en dernier recours , et où les familles et les classes moyennes seront une fois de plus les grandes sacrifiées . Toujours plus loin dans l' indécence : l' équipe Fillon préconise également de sortir la grippe , l' optique , le dentaire et les prothèses auditives du panier de soins pris en charge par la Sécurité Sociale . Or , ce sont justement ces domaines qui représentent l' essentiel des renoncements aux soins , qui , je le rappelle , touche plus d' un Français sur quatre . Cette &laquo; désétatisation &raquo; , comme il le dit lui-même , n' est rien d' autre qu' une privatisation , qui entrainera pour une famille avec deux enfants un surcoût de plus de 100€ par mois , selon la Mutualité française . Pour un homme qui parle de la famille dans ses discours de campagne , il reste de très sérieux efforts à faire … notamment à l' heure où le taux de pauvreté ne cesse de croître : on dénombre aujourd'hui 9 millions de Français vivant sous le seuil de pauvreté . Alors , bien entendu pour rassurer , on promet une couverture accrue – sans en préciser les contours – aux plus défavorisés , qui au final devront tout de même souscrire un contrat d' assurance complémentaire , dont les prix auront explosé au vu du transfert des remboursements depuis l' Assurance Maladie . En somme détruisons la Sécurité Sociale et subventionnons indirectement les assurances complémentaires , déjà grassement aidées … Non seulement les organismes proposants des contrats dits &laquo; responsables &raquo; bénéficient d' avantages fiscaux , mais par la volonté de Marisol Touraine , les remboursements dans le cadre de ces fameux contrats sont plafonnés depuis le 1er janvier ! Merci Madame la Ministre ! Vous avez voulu de manière dogmatique frapper les professionnels de santé libéraux , manque de chance c' est sur les patients qui vous avez tapé : +100 % de reste à charge en moyenne à l' hôpital , et jusqu' à +130 % pour les consultations chez un spécialiste ! Ma première proposition est d'abord un engagement de fond : il s' agit de garantir à chaque Français un accès aux soins optimal , grâce à un système de Sécurité sociale qui respecte le principe fondamental de solidarité nationale . Ainsi , je m' engage à ne pas réduire la prise en charge des soins par l' Assurance maladie et je m' oppose fermement à la privatisation des dépenses de soins . Je m' engage pour une santé de qualité , qui ne laisse pas sur le bord de la route les classes populaires comme les classes moyennes , les familles comme les personnes âgées . Une Santé qui écoute des professionnels , ses médecins , des infirmiers , ses pharmaciens . Et ceux qui voudront détruire , privatiser notre Sécurité Sociale me trouveront sur leur chemin . L' un des problèmes majeurs rencontrés par les Français en matière de santé est la difficulté de l' accès aux soins , qui s' est déjà fortement dégradé et qui ne devrait pas s' améliorer au cours des mois à venir . Pis encore , les nombreux départs à la retraite attendus au cours des prochaines années laissent entrevoir une pénurie de médecins , notamment généralistes , alors que les besoins augmentent du fait du vieillissement de la population . C' est pourquoi il m' apparait capital de recruter plus de médecins : ainsi je relèverai dès notre arrivée au pouvoir le numerus clausus de manière à tenir compte de la réalité du terrain , tout en donnant aux facultés les moyens de former ces étudiants supplémentaires . Pour sensibiliser les jeunes médecins à la question des déserts médicaux , je propose d' instaurer un stage d' internat dans une zone désertifiée . Il faut en effet tout entreprendre pour mettre fin aux déserts médicaux , ces zones de non-soins qui ne sont pas admissibles . De plus , il est crucial de redonner le goût de l' exercice libéral , qui n' attire presque plus personne en raison des contraintes qui se surajoutent année après année . Aujourd'hui , M. Fillon juge qu' il est nécessaire de relever le numerus clausus pour éviter le recours à des médecins étrangers moins bien formés , or il ne l' avait relevé que de manière tout à fait dérisoire lorsqu' il était aux manettes ; force est de constater que son gouvernement n' avait pas pour priorité de garantir la pérennité de notre système de santé . Pourquoi s' en soucierait -il demain ? Niveau économies , je ne suis pas certaine non plus que les 19 millions de vaccins contre le virus H5N1 surnuméraires détruits en 2011 soient un exemple de bonne gestion … Quid également de l' aide médicale d' Etat , qu' il promet aujourd'hui de réduire , alors qu' il a eu déjà eu le loisir de le faire 5 ans durant ? Rappelons que le budget prévisionnel de l' AME pour 2017 s' élève à 815 millions d' euros , et que l' on peut raisonnablement estimer son coût réel à 1,3 milliards pour l' année à venir … dont moins d' un dixième concerne des soins urgents . Pour éviter le dévoiement de ce système , et pour enrayer le &laquo; tourisme médical &raquo; qui est une réalité indéniable , je propose depuis plusieurs années de supprimer l' AME et de la remplacer par une aide restreinte , exclusivement réservée aux soins d' urgence vitale . Il est également inacceptable de constater que des ressortissants de pays étrangers laissent aux hôpitaux français des notes très salées … en 2015 , plus de 120 millions d' euros de frais de santé restaient dus par des patients étrangers , uniquement sur les hôpitaux parisiens ! Ces créances doivent être recouvrées au plus vite et je m' y emploierai . Je ferai également de la lutte contre la fraude une priorité . Pour que notre système de protection sociale soit juste , il faut que chaque Français qui en a besoin puisse bénéficier de la solidarité nationale , et que les abus soient sévèrement punis , or le laxisme sévit dans tous les domaines . A mesure que l' on professionnalise la lutte contre la fraude , les sommes détournées détectées augmentent : elles ont dépassé le milliard d' euros l' an passé . Les arrêts maladie de complaisance doivent être farouchement combattus , pour que les deniers publics servent à ceux qui en ont réellement besoin . Les données collectées par l' Assurance Maladie doivent être analysées de manière plus précise , par des équipes anti-fraude , et de nouveaux dispositifs pourraient voir le jour pour rendre les abus plus difficiles . Une nouvelle Carte Vitale biométrique , combinée à la carte nationale d' identité , serait un bon début . La vente des médicaments à l' unité permettra également de lutter contre le trafic , au niveau national mais également international , certains patients étrangers venant se faire prescrire des traitements coûteux en France pour les revendre à moindre coût à l' étranger . Pour optimiser la gestion du système de santé , et améliorer la prise en charge des patients , les ingénieurs et les entreprises innovantes françaises doivent être mises à contribution . Des dispositifs de santé connectée peuvent aider au suivi des patients chroniques , éviter des complications très coûteuses pour la collectivité et donc permettre des économies : par exemple , le non-respect des prescriptions médicales engendre un surcoût de 9 milliards d' euros par an , juste en considérant les six principales pathologies chroniques . Les instruments de télé-assistance et de télé-surveillance permettent de veiller sur nos aînés vivant à seuls à leur domicile afin de leur porter secours au moment adéquat . Le prix des médicaments innovants doit également être jugulé , et le généricage des molécules doit être accéléré et facilité , afin de permettre des économies . Un fonds d' investissement pour l' innovation en santé sera créé en partenariat avec la BPI , et les subventions aux startups innovantes seront là , conditionnées à un engagement pour l' entreprise de rester sur le sol français et de ne pas revendre les technologies à une entreprise étrangère pendant 10 ans . Les financements de la recherche scientifique , en particulier biomédicale , doivent être réévalués à la hausse , afin de garantir un avenir à nos jeunes chercheurs , et faire rayonner notre savoir et nos technologies sur la scène internationale . La recherche permet également de faire évoluer les modes de prise en charge , et pour désengorger des hôpitaux au bord de l' asphyxie , la prise en charge ambulatoire peut être privilégiée lorsque l' état des connaissances de la médecine le permet . Nous avons un vivier de jeunes prêts à assurer ces missions , et qui ne demandent qu' à travailler . Ce ne sont pas les moyens qui manquent pour remettre notre système de protection sociale en marche , c' est la volonté politique . Un changement de paradigme est nécessaire , et je souhaite l' incarner . La santé et la protection sociale doivent être des priorités politiques , régaliennes , nationales , qui ne doivent en aucun cas être déléguées à des structures privées , à but lucratif . Vive la Sécurité Sociale , vive la République , et vive la France ! ",    
    "Merci à tous , Merci à tous les intervenants de cette convention , Merci aux membres du Collectif Banlieues Patriotes pour la qualité de leurs propositions et pour leur travail de terrain . Je suis heureuse que nous puissions prendre le temps , avant la campagne , de parler d' un sujet aussi délicat que celui de nos banlieues , en ayant le temps de développer honnêtement nos idées et en évitant toute caricature . C' est un sujet délicat , car comme tout sujet que le pouvoir politique a sciemment laissé de côté depuis des décennies , il est devenu un point névralgique . Un point sensible qui réagit au moindre choc . La France a mal à ses banlieues . Les fantômes des émeutes de 2005 ressurgissent au moindre trouble à l' ordre public . Nous ne pouvons pas laisser la France ainsi fracturée . Si nous voulons véritablement une France apaisée , il faut donc traiter cette question à la fois avec une précision pointilleuse et une volonté bienveillante . Nous devons le faire dans l' intérêt de la nation . La France des oubliés , ce sont aussi ces quartiers relégués et à la dérive . Il s' y trouve des voyous que nous neutraliserons sans pitié , mais il s' y trouve aussi et surtout des Français abandonnés , qui n' ont pas pour eux les masses d' argent de la politique de la ville , pour une raison simple : cet argent est gaspillé dans le clientélisme . On a laissé partir à vau l' eau les banlieues de nos villes . Pire encore , certains politiques se sont nourris de cette décomposition , en vrais charognards . Certaines puissances étrangères aussi . Le clientélisme y est roi , plus que dans toute autre partie de la France . Ajoutez à cela le communautarisme , la radicalisation religieuse et le chômage de masse , et vous disposez là d' une véritable bombe . Et quand vous êtes face à une bombe , vous avez deux solutions : soit vous essayez de la désamorcer , soit vous la faites exploser . Les autres dirigeants politiques , qui ont abandonné l' idée même de nation française , ont décidé de la faire exploser . Je suis la seule qui veut la désamorcer . Je suis la seule qui veut éviter le chemin du pire , et qui veut proposer une voie d' apaisement . Tous les multiculturalistes , tous les chantres de l' identité heureuse , tous les faux durs , les vrais mous , tous les Tartuffes de l' idéologie du vivre-ensemble , tous ceux-là jouent aux apprentis sorciers . En voulant défaire la France , ils préparent l' affrontement des Français entre eux . Nous ne pouvons pas nous permettre de les laisser continuer leur œuvre mortifère . C' est pour rétablir la paix civile , qui est le bien le plus précieux d' une nation , que nous devons prendre à bras le corps le problème des banlieues françaises . Notre plan est simple : il comprend trois volets . Premièrement , rétablir l' ordre républicain dans les banlieues . Deuxièmement , déraciner le fondamentalisme islamiste pour préserver les enfants des banlieues de toute dérive , et notamment les femmes et les filles . Troisièmement , ouvrir de nouvelles perspectives de développement économique à cette jeunesse pour qu' elle participe à la prospérité de la France de demain . Commençons par le rétablissement de l' ordre , sans lequel rien n' est possible . Mon objectif consiste à retranscrire ce que je nomme le &laquo; vive-en-paix &raquo; dans nos quartiers . Certaines banlieues sont devenues des zones de non-droit où l' on applique plus la loi de la République . Les premières victimes de cet état sont les habitants de ces mêmes banlieues , souvent terrorisés par des caïds qui font régner la loi du plus fort . L' Etat a abdiqué face à cette menace . Il ne se donne plus les moyens de protéger les Français . Je rappelle ce chiffre , connu de tous , mais pourtant toujours aussi ahurissant : Nicolas Sarkozy a supprimé 12 500 postes de policiers sous son mandat . Ce même Nicolas Sarkozy qui , sur RMC il y a quelques jours , disait être fier d' avoir supprimé des postes de policiers en temps de crise , parce que cela prouvait qu' il était un homme responsable ! Il est fier de ses fautes ! Il suffit pourtant d' un gramme de bon sens pour comprendre qu' en temps de crise , la sécurité est la DERNIERE chose sur laquelle il faut faire des économies . Et ce n' est pas François Hollande qui a changé quoique ce soit à cette hémorragie , puisque sur les 9000 postes qu' il avait promis , il n' en a pourvu que 390 . Du côté de nos forces de douane , le constat est tout aussi catastrophique : 4000 postes supprimés sur les dix dernières années . Qu' on ne s' étonne plus ensuite que nos frontières , qui sont plus trouées qu' une tranche de gruyère , laissent passer des milliers de kalachnikovs qu' on retrouve dispersés dans toutes les banlieues du pays , ou la drogue qui la gangrène . Le rétablissement de nos forces de police et de nos forces de douane est donc la condition sine qua non de toute résolution du problème de la sécurité de nos banlieues . A cela s' ajoute d' autres mesures immédiates , comme la présomption de légitime défense pour les policiers , et l' application réelle des peines de prison inférieures à 5 ans . L' Etat ne doit seulement arrêter de reculer , il doit avancer ! Il doit restaurer l' ordre sur un territoire qu' il n' a pas le droit d' abandonner . Car non seulement , en abandonnant ce territoire , il met en danger les villes françaises , mais il ne respecte pas non plus son devoir de protection des habitants de ces quartiers . Après le rétablissement de l' ordre , sans lequel tout futur est inenvisageable , il faut creuser plus profondément jusqu' à la racine du mal . Il faut déraciner cette idéologie qui gangrène les banlieues et qui se nourrit de la faiblesse humaine , l' idéologie salafiste . Le salafisme ne naît pas par génération spontanée . Il a des relais , il a des porte-voix , il a des financements . Les banlieues françaises n' étaient pas destinées à devenir le terreau du radicalisme . Et elles ne sont pas destinées à le rester . Pour préserver la jeunesse de nos banlieues de cette lobotomie qu' est le salafisme , il faut la préserver de ces relais de la haine . C' est une bataille idéologique . Nous devons assécher le marais du salafisme , et pour cela il y a des moyens simples . Nous devons fermer les mosquées radicales , puisque Monsieur Cazeneuve , qui l' avait pourtant promis avec ses airs de premier flic de France , n' a pas été capable de le faire . Nous devons expulser les prêcheurs de haine , qui aujourd'hui exposent leurs thèses négationnistes , misogynes , homophobes ou christianophobes dans des réunions publiques en Seine-Saint-Denis sans que cela ne semble inquiéter personne à par nous . Nous devons dissoudre l' UOIF , qui est le cache-nez du radicalisme islamiste en France , et qui a servi jusqu' ici de cheval de Troie à des revendications politico-religieuses , Il faut contraindre les réseaux délétères des frères musulmans qui essaiment , avec la bienveillance de nombreux candidats actuels à la présidentielle , Et après ces mesures immédiates , et qui pourtant n' ont jamais été mises en place , il reste une mesure à plus long terme , tout aussi fondamentale que les autres : nous devons interdire le financement des lieux de culte par des fonds étrangers , et notamment le financement des centres cultuels ou culturels par le Qatar et l' Arabie Saoudite . Nous devons également lutter contre la corruption de certains élus locaux qui rendent possible par leurs agissements l' infiltration de ces puissances étrangères sur le territoire national . A cette fin je propose la mise en place d' un corps d' inspection contre la corruption de la vie politique locale chargé en particulier de détecter le clientélisme et les contournements à la loi de 1905 sur la Laïcité . Je le répète : les banlieues ne sont pas destinées à rester des viviers du salafisme . Ce n' est pas une fatalité . Nous pouvons faire sortir la jeunesse des banlieues des griffes de l' idéologie . Nous pouvons leur redonner un avenir . Un avenir français . Mais pour cela , il faut tarir la source du problème . Et quoi de mieux pour cet objectif que de recréer à terme un nouveau Service National pour nos jeunes filles et garçons , encadrés à partir d' une armée rétablie dans ses moyens et remise au coeur de la formation du citoyen . Enfin , si nous réussissons à rétablir l' ordre et à tarir les sources de la radicalisation , notre tâche ne sera pas terminée pour autant . Si nous ne voulons pas que les banlieues redeviennent une nouvelle fois une zone de non-droit , nous devons les faire participer à la prospérité économique d' une France souveraine . Ceci , disons -le , ne sera possible qu' en rétablissant d'abord la sécurité , tant la délinquance et l' insécurité sont des obstacles majeurs à tout développement économique . En effet , ce n' est pas l' économie parallèle des dealers et des trafiquants qui doit y régner , mais l' économie réelle des entrepreneurs , des nouvelles industries , des PME , des artisans , des commerçants et de tout ce qui fait le tissu économique et social de la France , La banlieue est , comme l' ensemble de la France , bourrée de jeunes talents qui ne cherchent qu' à exprimer leurs capacités . Alors que la politique de la ville est un puits sans fond où l' Etat perd une somme d' argent monstrueux pour des résultats inexistants , il est possible d' allouer cet argent dépensé inutilement de manière beaucoup plus efficace , en soutenant la création d' entreprise et en facilitant la vie des PME de nos banlieues . Une fiscalité des entreprises plus juste permettra aussi de voir éclore des start-up qui aujourd'hui peinent à survivre . Ainsi , ma dernière proposition ce jour est la création d' un fonds d' intervention pour les start-up , car je sais qu' il y a beaucoup de gens qui veulent entreprendre et créer dans les quartiers . Enfin , si nous voulons véritablement en finir avec cette image de la banlieue à la marge du développement économique , il faudra en finir avec l' idée même de banlieue dans nos têtes , et notamment à Paris . Parlons simplement de France ! Je suis volontariste . Je ne crois pas à la fatalité d' un destin sur lequel il serait impossible d' agir . Je ne crois pas que les banlieues françaises soient vouées à une déchéance toujours plus grande , je ne crois pas que les enfants de l' immigration soient voués à rester aux marges de la nation . Je crois qu' il est possible de refaire l' unité de la France et de tous les Français , mais cela ne peut passer que par l' affirmation forte de quelques vérités trop longtemps oubliées : la nation est le cadre intangible dans lequel nous pouvons agir et abandonner ce cadre est une voie sans issue . L' ordre est la condition de la prospérité économique et celle -ci doit être soutenue . L' amour de la France est le fondement de toute vie collective pacifique . Ces vérités sont simples , mais ce qui est simple n' est pas forcément facile . Nous n' avons pas choisi la voie de facilité . Mais nous avons choisi la voie de l' action , donc de l' espoir , et il faut nous en rendre dignes . Vive le Collectif Banlieues Patriotes , vive la République , et vive la France ! ",
    "Etats-Unis : le choix de la liberté Les Américains ont voté . Ils ont refusé le statu quo . Ils ont montré par une décision qui a surpris ceux qui croient que les situations sont immuables , que le monde bouge , que le monde change , que le mouvement fait partie de la vie des Nations . Ce qui s' est passé cette nuit n' est pas la fin du monde mais la fin d' un monde . Les Américains se sont donné le président qu' ils ont choisi et non celui qu' un système installé voulait leur faire valider comme si les élections n' étaient qu' une formalité , voire une corvée pour satisfaire les apparences ou les convenances . En cela , la décision du peuple américain doit s' interpréter comme la victoire de la liberté , la liberté d' un peuple souverain , la liberté de citoyens qui ont su s' affranchir d' une campagne où le dénigrement et la peur , où l' infantilisation et la condescendance furent autant de tentatives , finalement infructueuses , de conditionnement de l' opinion . Parce que seul compte à mes yeux l' intérêt de la France et des Français , j' ose répéter ici que l' élection de Donald Trump est une bonne nouvelle pour notre pays . Refus du TAFTA et plus généralement d' une mondialisation sauvage , pacification des relations internationales notamment avec la Russie , désengagement des expéditions belliqueuses qui sont à l' origine des grandes vagues migratoires dont nous sommes directement victimes . Ces engagements , s' ils sont tenus , sont bénéfiques pour la France . D' une manière générale , et la classe politique française devrait s' en souvenir , on gagne toujours à fonder sa vision sur le respect des autres peuples et le refus des ingérences moralisatrices . Dans une démocratie , dans tous les pays , c' est le peuple et lui seul qui décide , c' est lui et lui seul qui donne la légitimité . Ici et ailleurs , il n' est pas de gouvernement digne , pas de gouvernement authentiquement légitime qui ne gouverne pas pour le peuple et au nom du peuple . Les élites politiques et médiatiques lourdement sanctionnées ce matin ne peuvent plus l' ignorer . Le référendum français en 2005 , le référendum grec en 2015 , les récents succès électoraux des patriotes dans différents pays d' Europe , le vote massif des Britanniques en faveur du Brexit bien sûr , et maintenant l' élection de Donald Trump : ce sont autant de choix démocratiques qui enterrent l' ordre ancien et autant de pierres qui construisent le monde de demain . La défense d' intérêts catégoriels , la soumission à des puissances d' argent insensées et dévastatrices , l' abdication devant les dictatures islamistes , sont des choix qui isolent chaque jour un peu plus ceux qui , ici ou ailleurs , persistent à les défendre . Ce matin , le peuple américain vient donc confirmer ce grand mouvement du monde qui voit le retour des peuples libres dans le concert mondial , l' affirmation du principe de la primauté de l' intérêt des nations sur les intérêts particuliers et sur les féodalités , l' immense soif de liberté et d' indépendance et , je l' espère , l' émergence d' une conception multipolaire des relations internationales . Le peuple américain est un peuple historiquement ami de la France . Nos histoires sont liées . Nous l' avons accompagné dans sa guerre d' indépendance , puis nous l' avons imité en empruntant derrière lui , les chemins de la liberté . Gageons que les Français , qui chérissent tant cette liberté , y trouveront une raison supplémentaire de rompre avec un système qui , eux aussi , les entrave . Avec raison , le nouveau Président des Etats-Unis d' Amérique a réaffirmé dans sa première intervention son intention de faire valoir l' intérêt de son pays . Je ne lui en ferai pas le reproche puisque nous partageons cette conception de l' action politique pour nos pays respectifs . Dans le même temps , il a tendu la main à tous les pays qui souhaitent travailler avec lui . Dans le cadre de relations fondées sur le respect de nos souverainetés et de nos intérêts nationaux , cette main tendue est un geste dont nous devons prendre en compte la portée . Qu' il me soit permis d' adresser mes félicitations au nouveau Président des Etats-Unis et mon témoignage d' amitié au peuple libre des Etats-Unis d' Amérique .",
    "Merci à tous , Merci à nos intervenants d' aujourd'hui , que l' on sent passionnés par l' idée d' une France à nouveau belle et grande , Merci aux membres du Collectif Culture et du Collectif Mer et Francophonie . Merci aussi à nos frères de cœur venus d' autres pays , qui partagent l' amour d' une culture francophone à la fois diverse et unie . Je dois vous avouer que c' est peut-être le discours le plus exaltant que j' aurai à faire lors de ces conventions présidentielles . Tous les sujets nous intéressent , et il n' est pas question pour nous de laisser quoique ce soit de côté , lorsqu' on veut présenter un projet cohérent pour la France . Mais aujourd'hui , c' est un peu spécial . Aujourd'hui , nous parlons de grandeur . Nous parlons non seulement de ce que nous pouvons apporter à la France , mais aussi de ce que la France peut apporter au monde . La France est une vieille et grande civilisation . Elle l' a un peu oublié . On lui a fait oublier . Mais il suffirait de peu de choses pour qu' elle retrouve son influence sur la scène mondiale . Il suffirait , avant tout , de volonté politique . Il fut un temps où la France découvrait les quatre coins du globe . Un temps où Jacques Cartier ouvrait le fleuve Saint-Laurent à l' Ancien Monde . Un temps où les Français exploraient toute la vallée du Mississippi . Un temps où les missionnaires de France se rendaient dans les royaumes d' Asie . Un temps où la France était dans toutes les cours d' Europe . Un temps où Descartes allait chez Christine de Suède ; où Voltaire régnait sur la cour de Frédéric de Prusse ; où Diderot se rendait chez Catherine de Russie . Un temps où toutes les élites parlaient français , non pas parce qu' on les contraignait à parler français , mais parce que le français provoquait l' admiration . Parce que le français était la langue de la raison . Un temps où la France était le phare de la liberté . Un temps où tous les opprimés du monde voyaient en la France le symbole de la lutte contre la tyrannie . Un temps où la France abolissait l' esclavage , avant toute autre nation . Un temps où l' on déclarait en français , partout où soufflait le vent de l' indépendance : &laquo; Tous les hommes naissent et demeurent libres et égaux en droits &raquo; . La France a été la civilisation de la liberté et de la raison . Certains diront que c' est une illusion rétrospective . Que nous reconstruisons l' histoire . Mais ce sont les mêmes qui abaissent constamment la France , qui ne préfèrent voir que ce qui est bas , ce qui est mesquin , ce qui est laid . Ils veulent salir la grandeur . Ils n' acceptent pas que la France ait pu être grande et peut-être encore moins qu' elle veuille le rester . Que les Français aient pu faire de belles choses et de grandes choses . Ils ne voient que la France à terre , la France défaite dans la guerre , la France soumise à des puissants qui salissent son nom . Le roman national est , entre leurs mains , devenu un cauchemar . Les censeurs de la grandeur sont en train de créer des névroses immenses : ils font des générations de Français qui se haïssent eux-mêmes , et cela est beaucoup plus dangereux qu' on ne le croit . La haine de soi-même ne peut entraîner que la rancœur , la colère et finalement la haine des autres . Ceux qui haïssent la France n' haïssent pas que la France . Heureusement , subsistent à l' extérieur de notre pays les traces de cette grandeur toujours vive . Quand les Français , à qui on répète à longueur de journée qu' ils sont dépassés , qu' ils sont mesquins , qu' ils sont de rebus de l' histoire , quand les Français oublient leur propre valeur , il faut voir avec quelle ardeur nos cousins québécois , sénégalais , ivoiriens , belges , et bien d' autres défendent une culture et une langue que nous laissons à l' abandon . Il faut voir la vitalité du cinéma québécois , de la littérature francophone en Afrique sur les traces de Senghor , il faut voir les artistes belges , les artistes non francophones même qui choisissent le français . En se haïssant elle-même , la France devient stérile . Mais la culture française n' est pas stérile . Ce n' est pas une culture de mort . Et dès qu' elle peut renaître quelque part , elle bourgeonne immédiatement , et elle prend différentes couleurs sous d' autres latitudes . Elle s' enrichit constamment . Je ne viens pas vous livrer un message désespéré . Au contraire . Je pense que la France peut redevenir plus grande encore , plus belle , plus audacieuse , plus exploratrice et plus avant-gardiste que jamais . Qu' elle peut rayonner partout , imposer le respect partout , faire éclater son génie aux yeux du monde . Je pense qu' elle peut retrouver son influence perdue . Pour cela , il faut que nous repérions les leviers grâce auxquels nous pouvons agir . J' en développerai trois . D'abord , il ne faut &laquo; cesser de reculer &raquo; comme disent certains , il faut avancer . Si la francophonie est aujourd'hui en échec , c' est parce que son cœur , la France , a baissé les bras dans la défense de sa culture . Et la culture , c' est avant tout la langue . Nous voulons une nouvelle loi Toubon , beaucoup plus contraignante . La loi Toubon partait d' une bonne intention , mais aujourd'hui , elle fuit de partout . Dans les médias , dans les administrations , dans les écoles , dans les universités . Alors que plusieurs universités pratiquent déjà le tout-anglais dans certains cursus , la loi Fioraso de 2013 et ses amendements ont introduit des exceptions concernant l' enseignement en anglais , je cite &laquo; lorsqu' elles sont justifiées par des nécessités pédagogiques &raquo; . C' est ici un cheval de Troie . Maintenant , on ne va plus étudier à la Sorbonne ou à la fac de Nanterre , mais à la &laquo; Toulouse School of Economics &raquo; ou à &laquo; Paris Sciences et Lettres Research University &raquo; . Dans une même logique , les stations de radio privées disent dorénavant ne plus pouvoir respecter les quotas de chansons en français imposés par la loi Toubon , pour la simple et bonne raison que les Français ne chantent plus … en français . Ma première proposition est donc la suivante : je veux une loi Toubon 2 , pour que la langue et la culture françaises cessent de reculer . Les Québécois nous ont précédés avec leur loi 101 , qui est bien plus rigoureuse dans la protection et la promotion de la langue officielle . Je veux une loi qui fasse respecter partout l' exigence de promotion du français , surtout dans l' enseignement , et une loi qui encourage davantage les artistes francophones . Un autre volet de cette loi Toubon 2 , ou d' une autre loi qui la suivra , sera la protection du patrimoine culturel . Il nous faut à la fois protéger ce patrimoine et le valoriser , contre toute tentative de le brader , en le laissant filer entre les mains de collectionneurs étrangers ou d' investisseurs sous forme de placements . Contre ceux aussi qui le défigurent , en imposant à nos monuments historiques un habillage qui vire souvent au massacre . La première étape , c' est celle de la protection : on arrête de reculer . La deuxième , c' est celle de l' organisation . Ici , le levier le plus puissant , c' est l' Organisation Internationale de la Francophonie , l' OIF . C' est un outil sous-exploité . Depuis l' élection de sa nouvelle présidente , Michaëlle Jean , en 2014 , l' OIF n' a toujours pas présenté de plan d' envergure pour l' apprentissage du français dans le monde . Il faut dire que les crédits sont insuffisants : les gouvernements successifs ont délaissé ce combat . Pour l' instant , la francophonie , c' est de l' or que l' on transforme en plomb . Rappelons que le français est aujourd'hui , avec 274 millions de locuteurs quotidiens , la seule langue , avec l' anglais , qui soit présente sur les cinq continents , qu' il est la cinquième langue la plus parlée , la quatrième langue sur internet , la troisième langue d' affaires , et la seconde langue d' information dans le monde . Imaginez toutes les occasions économiques et culturelles qui sont gâchées par l' abandon de ce trésor . Cesser de reculer , c' est également défendre , entretenir et promouvoir l' immense patrimoine de la France , sur le territoire national comme à l' étranger , et que le monde entier nous envie ! Ces palais , châteaux , hôtels particuliers , demeures de prestige , qui abritent nos ambassades , nos centres culturels , nos écoles nationales , et qui sont le témoignage de notre histoire hors du commun , forment le patrimoine de tous les Français . Je mettrai donc un coup d' arrêt , et c' est l' objet de ma deuxième proposition , à la dilapidation de nos trésors nationaux , par des gouvernements indignes de l' héritage qui leur a été confié par la Nation . Sous mon mandat , aucun de nos monuments ne sera bradé à des puissances étrangères ; il en va du prestige comme de l' honneur de la France . Ma troisième proposition est l' organisation rapide d' un grand sommet de la Francophonie . Ce sommet aura un triple but : relancer la promotion du français à l' étranger , encourager la coopération économique des pays francophones , et repenser le rapport des pays francophones en termes de souverainetés nationales . Cela veut dire , en particulier , développer une vision moderne des relations entre la France et les pays d' Afrique francophones . Ni ingérence , ni abandon ! Nous voulons une relation de respect mutuel , dans la préservation des libertés et des identités des peuples , pour leur permettre de défendre leurs intérêts nationaux , notamment face à la Chine ou à l' Union européenne . C' est une Afrique libre qui sera prospère , et une Afrique prospère qui sera le meilleur allié de la France . Après la protection et après l' organisation , vient le temps de l' exploration . C' est le plus exaltant de tous . C' est le temps des grandes aventures , qui feront date si nous réussissons . Et le champ d' exploration de la France , aujourd'hui , c' est la mer . Je l' ai déjà dit de nombreuses fois , mais il est bon de le répéter : l' espace maritime français est le deuxième au monde après celui des Etats-Unis avec 11,5 millions de km² . L' équivalent de près de vingt fois la France métropolitaine ! Cet espace démultiplie nos possibilités . Il nous donne accès à des ressources inexploitées . Il nous donne un avantage stratégique sans commune mesure . Il offre des perspectives de croissance immenses . Et pourtant , cet espace de vingt fois la France n' existe pas sur les radars médiatiques et politiques . Pire , nous sommes entravés dans nos possibilités de développement . Notre pêche est entravée par l' Union Européenne , qui nous limite ici comme pour tout le reste . Les 300 000 emplois liés à l' économie de la mer sont menacés . La flotte commerciale française s' amenuise , alors qu' aujourd'hui plus de 80 % des échanges commerciaux mondiaux se font par voie maritime . Les recherches sur la mer sont sous-développées , en comparaison du potentiel . Ma quatrième proposition est donc la mise en place d' un grand Ministère de la Mer , qui aura pour but de coordonner la prospection des fonds marins , le développement des infrastructures portuaires et maritimes ainsi que les ressources économiques de la pêche , des gisements de minerais et de pétrole et le développement des énergies renouvelables aquatiques . Les Américains ont été motivés pendant deux siècles par l' idée de repousser toujours plus loin la frontière de l' exploration de leur territoire , puis de l' exploration spatiale . La frontière que la France doit pousser toujours plus loin , celle qui sera exaltante , c' est la frontière de la mer . C' est un monde nouveau , que l' on croyait connu mais qui ne fait que commencer à se dévoiler . Je ne veux pas que la France manque cette occasion de montrer encore une fois au monde qu' elle sait être à la pointe de la découverte . Nous sommes des explorateurs . Nous partons d' une civilisation riche et ancienne , et nous pouvons encore l' augmenter sans limites . Nous pouvons rivaliser d' ingéniosité , d' imagination , d' intelligence avec toutes les grandes puissances de ce monde . Mais pour cela , il faut retrouver ce qui a fait notre force , et cesser de nous complaire dans une prétendue décadence . La France sera encore grande à l' avenir , j' en ai la certitude . Et je ferai tout ce qui est en mon pouvoir pour faire advenir ce jour le plus tôt possible . Vive la francophonie , vive la culture française , vive la République , et vive la France . "
   ]
lepen_test = [
     "Mesdames , Messieurs , fonctionnaires de la Police nationale Si j' ai décidé de m' adresser directement à vous c' est pour vous dire combien nous vous comprenons et vous soutenons dans les moments difficiles que vous traversez . Confrontés à une situation que nos dirigeants qualifient eux-mêmes de &laquo; guerre &raquo; , une guerre dans laquelle vous êtes en première ligne , vous vous sentez , à juste titre , abandonnés par le pouvoir en place . Depuis deux quinquennats , les pouvoirs qui se sont succédé ont réduit sciemment vos effectifs avec une vision comptable irresponsable comme si la sécurité publique , comme si la sécurité des Français c'est-à-dire leur liberté pouvait être une variable d' ajustement des politiques publiques . Petits technocrates sans vision et sans envergure , ils ont réduit votre mission à des obligations de résultats administratifs ou statistiques ( au nombre de pv ) oubliant que le maintien de la tranquillité publique est d'abord une affaire essentiellement humaine , d' engagement , de générosité au service du bien public . Au niveau du terrain , les politiques , ceux-là mêmes qui sont en charges de vous diriger fuient souvent leurs responsabilités et vous laissent sans ordre mais , de leur bureau , savent vous accabler quand la situation risque de troubler leur déroulement de carrière . Mais surtout , vous sachant confrontés à une menace criminelle de plus en plus violente , face à une menace terroriste c'est-à-dire une menace de nature militaire , ils vous abandonnent avec des moyens techniques et juridiques souvent dérisoires et presque toujours inadaptés . Nous savons les outrages verbaux que pouvez essuyer chaque jour de la part d' individus bien connus de vos services , toujours les mêmes . Nous avons connaissance des agressions physiques que vous subissez quotidiennement par des délinquants souvent multirécidivistes . La plupart semblent jouir d' une parfaite impunité , quand ce n' est pas d' une tolérance ou d' une indifférence du pouvoir politique . Nous avons , comme chacun d' entre vous , en mémoire l' attaque de Viry-Châtillon où deux de vos collèges ont été brûlés vifs par des criminels et non des &laquo; sauvageons &raquo; comme a pu scandaleusement le dire un ministre de l' Intérieur indigne . Cet acte nous a glacés et nous glace toujours d' effroi quand on sait qu' un jeune homme de 28 ans et une jeune mère de 3 enfants vont porter toute leur vie d' effroyables séquelles de cette agression Avec le double assassinat de Magnanville , nous savons que la guerre que l' on vous demande de mener est totale : ce n' est plus votre propre vie qui est en jeu , mais aussi potentiellement celles de vos familles . Cela aussi doit être pris en compte . Les forces de gendarmerie , interdites de parler du fait de leur statut militaire , partagent la même constatation face à cette dégradation et cette progression dans l' horreur . Force est de constater que le pouvoir actuel , discrédité et usé , est dépassé par les évènements . Certaines zones de notre pays semblent plongées dans un chaos sécuritaire inquiétant . L' insécurité s' est étendue à nos campagnes et à nos écoles avec la multiplication des agressions contre vos collègues de l' Education nationale . Derrière ces attaques contre des fonctionnaires , ne nous y trompons pas , c' est l' Etat qui est visé , c' est la République française qui est visée , c' est la nation française tout entière qui est visée . C' est avec solennité que je veux vous dire mon soutien total et celui des Français attachés à nos valeurs , à nos libertés , à nos principes de vie . Vous le savez , je crois nécessaire de remettre la France en ordre . Nous devons réaffirmer la souveraineté intérieure de l' Etat . Il est urgent d' engager un plan de réarmement de nos forces de sécurité publique : > Un réarmement moral avec le témoignage de ce lien indissoluble entre la nation et ses défenseurs intérieurs , la solidarité des citoyens envers leurs policiers , le soutien indéfectible de l' Etat envers ses serviteurs . Cette reconnaissance qui doit être sans faille , passe également par une révision des situations individuelles sans laquelle vous ne pouvez pas travailler avec un esprit libre : la revalorisation des situations salariales , le paiement effectif de vos heures supplémentaires , les justes promotions et mutations aujourd'hui bloquées par la politique de régression des effectifs . Plus pratiquement , le soutien aux fonctionnaires c' est aussi faire en sorte que les policiers puissent bénéficier de logements dans des secteurs où leurs familles ne seront pas menacées . > Un réarmement humain avec une augmentation sensible des effectifs de police non pas seulement des emplois jeunes mais des fonctionnaires , formés , motivés , managés et surtout soutenus . > Un réarmement matériel en apportant les dotations budgétaires suffisantes à la modernisation des commissariats , à l' achat de véhicules performants , à la dotation de matériels et d' armements modernes et dimensionnés , à la formation des personnels aux nouvelles menaces à commencer par un entraînement très régulier au tir . > Un réarmement juridique en établissant une présomption de légitime défense pour les fonctionnaires de police nationale attaqués , en rendant anonyme leurs plaintes contre leurs agresseurs pour ne pas les désigner aux voyous , en accompagnant systématiquement les fonctionnaires plaignants avec des avocats de l' Etat qui assureront leur défense et leur obtiendront des indemnisations à la hauteur de leur atteintes corporelles et psychologiques . Au-delà , parce que la solution à des questions aussi complexes n' est pas seulement policière , le soutien juridique des forces de sécurité publiques passe aussi par un fonctionnement revu et amélioré de toute la chaîne pénale c'est-à-dire du maillon judicaire et carcéral . Sans cela , votre travail admirable continuera de vous apparaître vain et donc décourageant . Chers compatriotes , fonctionnaires de police , de notre police nationale , votre mécontentement n' est pas seulement légitime . Il est sain . Il porte en lui une réaction salutaire devant une situation qui nous concerne tous , une situation dont on voit bien qu' elle met en jeu la liberté et même la vie de chacun d' entre nous . Les menaces de sanctions qui vous visent sont dérisoires venant de gens qui n' ont , aujourd'hui , pas d' autre choix digne que de démissionner . Ce n' est pas de menaces dont vous avez besoin mais de soutien et de décisions . Nous savons que le pays vous soutient pleinement parce qu' il sait pouvoir compter sur vous y compris en ces circonstances ô combien difficiles ! Partisans d' un Etat restauré , conscients de ce que nous vous devons tous les jours , nous sommes de cœur et d' esprit avec vous . Vous pouvez compter sur nous . Vive la police française Vive la République française Vive la France "
]

macron = [
    "Rien n' est jamais écrit à l' avance Le peuple américain a voté et élu Donald Trump à la présidence des Etats-Unis d' Amérique . En tant qu' alliés historiques de ce grand pays , nous devons le saluer et le féliciter . Cette élection montre que rien n' est jamais écrit à l' avance . Il faut toujours écouter ce que le peuple a à dire et non ce qu' on aimerait qu' il dise . Cette élection me parait être l' expression d' un rejet du système profond , et sous-estimé . Le même qui a conduit nos amis britanniques à décider de sortir de l' Union Européenne il y a quelques mois . Le même que j' entends depuis deux ans que je me suis engagé publiquement sur la scène politique . Ce rejet rend impossible de demeurer dans le statu quo dans lequel notre pays s' est englué depuis trente ans . Ces votes expriment une demande de protection et de respect du sentiment populaire que je crois partagée par les Français . Elle doit être entendue et comprise , sans conduire pour autant à des réponses de fermeture ou de repli . C' est la condition pour les transformer non pas en rejet du présent mais en espoir de changement . C' est l' enjeu des six prochains mois . ",
    "Bonjour à toutes et tous , Mesdames , Messieurs , merci d' être là aujourd'hui , ce matin . C' est une étape importante que nous allons franchir et que nous préparons depuis plusieurs mois . Parce que je suis en effet intimement convaincu que c' est en affrontant la réalité du monde que nous pourrons retrouver de l' espérance dans notre pays , et c' est notre défi aujourd'hui . J' entends certains qui pensent que notre pays est en déclin , que le pire est à venir , que notre civilisation s' efface . Ils proposent le repli , la guerre civile ou les recettes du siècle dernier . J' en entends d' autres qui imaginent que la France peut continuer de descendre en pente douce , que le jeu de l' alternance politique suffira à nous faire respirer et qu' en quelque sorte , après la gauche , la droite et ainsi de suite , parce qu' on l' a toujours fait de la sorte . Avec les mêmes visages , les mêmes hommes , depuis tant d' années . Je suis convaincu que les uns et les autres ont tort parce que ce sont leurs modèles , leurs recettes qui ont simplement échoué . Le pays , notre pays , lui dans son ensemble n' a pas échoué et il le sait , il le sent et c' est de là que naît ce divorce entre le peuple et ses gouvernants . Je suis convaincu que notre pays a la force , le ressort , l' envie d' avancer parce qu' il a l' histoire et le peuple pour cela . Parce que la France a toujours été un pays moteur du progrès , du progrès politique et démocratique , quand la Révolution Française , ici , a ouvert le monde nouveau et un chemin . De progrès économique quand nos ingénieurs , nos entreprises , nous ont permis de créer des richesses nécessaires pour accéder à une vie meilleure . Du progrès social , parce que c' est dans notre pays que nous avons construit les protections qui allaient avec l' ère industrielle , qui ont fait longtemps notre fierté , qui ont suscité partout l' envie . Du progrès pour tous , quand nos gouvernants , nos pays , ont eu la force après trois guerres de construire avec l' Allemagne et d' autres partenaires européens , cette Union européenne qui nous tient . Du progrès pour chacun , parce que la France , c' est par dessus tout un projet d' émancipation des individus , d' émancipation de la religion , des conditions d' origine . C' est un pays dans lequel , en partant de rien , le talent , l' effort doivent pouvoir mener à tout . Mais la France aujourd'hui est sortie du chemin du progrès . Le doute s' est installé . Depuis 40 ans , nous n' avons pas réussi à régler le problème du chômage de masse , la déprise des territoires , la langueur de l' Europe , les divisions internes allant jusqu' aux fractures se sont plus récemment établies . La France est bloquée par les corporatismes , de tous ordres et n' est plus à la hauteur de sa promesse . En même temps , nous sommes rentrés dans une ère nouvelle . La mondialisation , le numérique , le changement climatique , les inégalités croissantes de notre mondialisation , les conflits géopolitiques , le terrorisme , la crise démocratique des sociétés occidentales , le doute qui s' installe au coeur de nos sociétés . Ce sont les symptômes d' un monde en plein bouleversement . Cette grande transformation que nous sommes en train de vivre , nous ne pouvons y répondre avec les mêmes hommes et les mêmes idées parce qu' elle vient encore décupler nos doutes et nos faiblesses . Nous ne pouvons proposer simplement de réparer ou d' ajuster notre modèle . Cette grande transformation , elle a des aspects tragiques et nous devons être conscients de la gravité de l' histoire et des moments que nous vivons . Mais elle a en même temps des opportunités , des promesses que nous devons saisir . Nous ne pouvons pas demander sans cesse aux Français de faire des efforts sans fin . Nous devons regarder ensemble la vérité en face , débattre de ces grandes transformations à l' oeuvre et dire où nous voulons aller , dans quelle direction , par quel chemin , parce que le temps qui nous y conduira sera long , parce que tout cela ne se fera pas en un jour . Les Français sont conscients des nouvelles exigences de notre temps , bien souvent ils le sont plus que leurs gouvernants . Ils sont moins conformistes , moins attachés à ces idées toutes faites qui n' assurent rien d' autre que le confort intellectuel de la vie politique . Face à ces défis , notre système politique est bloqué . Lorsque je dis cela je n' accuse personne et je ne parle pas des élus . Nombreux sont au contraire ceux qui par leur dévouement s' engagent pour leur pays . Mais les appareils politiques , les logiques politiciennes , paralysent aujourd'hui notre capacité d' aller de l' avant . J' ai vu de l' intérieur la vacuité de notre système politique qui empêche les majorités d' idées , au motif qu' elles fragilisent les appareils , les partis traditionnels , les intérêts acquis , qui ne poursuit plus l' intérêt général mais son propre intérêt , qui a transformé la vie des Français en simple décor de son propre théâtre d' ombres . J' ai pu mesurer ces derniers mois ce qu' il en coûte de refuser les règles obsolètes et claniques d' un système politique qui est devenu le principal obstacle à la transformation de notre pays . Je sais ce que c' est . Je sais aussi que je continuerai à l' éprouver , cela ne fait que renforcer ma détermination . Parce que ce système , je le refuse . Face à cela je sais l' énergie du peuple français . Elle peut nous conduire à donner le meilleur de nous-mêmes . C' est cette énergie qui est aujourd'hui le quotidien des milliers d‘associations qui font le travail dans l' ombre le travail de tenir notre société ensemble . C' est cette même énergie qui donne leur force à tous ceux qui innovent , travaillent , produisent chaque jour . Si nous voulons avancer , faire réussir notre pays et construire une prospérité nouvelle , il nous faut agir . La solution , elle est en nous . Elle ne dépend pas d' une liste de propositions qui seront oubliées le jour d'après , qui changeront même parfois durant le temps de l' élection et qui au fond sont là pour ne pas être appliquées . Elle ne saurait émerger non plus de compromis bancals . Elle se fera grâce à des solutions différentes , grâce à une révolution démocratique profonde . Elle prendra du temps mais elle ne dépend que de notre unité , de notre courage , de notre volonté commune . C' est cette révolution démocratique à laquelle je crois . Celle par laquelle en France et en Europe , nous conduirons ensemble notre propre révolution plutôt que de la subir . C' est cette révolution démocratique que nous avons préparée durant ces derniers mois avec En Marche . Je crois très profondément que rien n' est jamais écrit . Et c‘est pourquoi je veux porter l' optimisme de la volonté . La France n' est pas un château de cartes . Elle a surmonté tant d' épreuves , bien plus redoutables . La seule force dont nous pouvons , dont nous devons nous réclamer , c‘est la force de la France , c' est la force des Français . C' est celle-là qui doit nous guider . Dans quelques mois , à l' occasion de l' élection présidentielle , une opportunité nous est offerte . Celle de refuser enfin le statu quo pour choisir d' avancer . Parce que ce combat que nous devons livrer pour faire réussir notre pays , il commencera en mai 2017 . Pour le mener , la responsabilité du président de la République est immense . J' en suis pleinement conscient . Un président n' est pas simplement investi d' une action . Il porte aussi , de manière moins visible , les valeurs de notre pays , la continuité de son histoire , et de manière cachée , la vigueur et la dignité d' une vie publique . Je sais cela . J' y suis prêt . C' est pourquoi je suis candidat à la présidence de la République . Parce que je crois plus que tout que nous pouvons réussir , que la France peut réussir . Bien sûr , on ne se réveille pas un matin avec cette révélation . La décision de me présenter aux plus hautes charges de la République est le fruit d' une conviction intime et profonde , d' un sens de l' Histoire et d' une conscience aiguë des temps qui sont les nôtres . J' ai fait le chemin de la province à Paris , du monde de l' entreprise à la vie publique puis à la vie politique . Les responsabilités qui ont été les miennes m' ont fait pleinement mesurer les défis de notre temps et toutes ces vies m' ont conduit à cet instant . Je veux faire entrer la France dans le XXIème , je veux que mon pays redresse la tête et pour cela retrouve le fil de notre histoire millénaire , ce projet fou d' émancipation des personnes et de la société . Ce dessein , c' est le dessein français : tout faire pour rendre l' homme capable . Je ne peux me résoudre à voir une France qui a peur , qui ne regarde que ses souvenirs , une France outrancière qui insulte et qui exclut , une France fatiguée qui stagne et qui gère . Je veux une France libre , libre et fière de ce qu' elle est , de son Histoire , de sa culture , de ses paysages , de ses femmes et de ses hommes qui ont traversé tant d' épreuves et qui n' appartiennent à personne . C' est pourquoi je place ma candidature sous le signe de l' espérance . La France peut réussir . Elle doit d'abord pour cela relancer l' Europe . J' entends tant de discours de haine sur ce sujet , de renonciation ou de repli . L' Europe , elle est notre chance dans la mondialisation , et notre responsabilité est ici immense . La France doit retrouver confiance en elle et bâtir son projet . Par le travail d'abord , qui émancipe chacun , parce que c' est par le travail que nous construisons notre place dans la société . Il n' y a pas de fatalité aujourd'hui , à ce qu' un jeune qui vient d' un quartier difficile comme celui où nous nous trouvons , ne puisse pas trouver d' emploi , n' ait parfois même pas le droit à la formation . Et le lieu où nous sommes , à ce titre n' a rien d' innocent . Et je ne peux accepter , de la même façon , que dans nos provinces tant de Français aujourd'hui attendent pour travailler , ne puissent même plus vivre de leur travail dans la dignité . Le travail , c' est ce qui nous construit . Par l' investissement ensuite , l' investissement dans notre avenir . Pour réussir cette révolution numérique , pour réussir le passage vers une autre économie , vers une autre société . Pour former , pour aussi accompagner cette transformation climatique et énergétique . Par l' école et la transmission ensuite . Notre pays se redressera par sa jeunesse . Je sais que beaucoup doutent de la jeunesse ils ont tort . Il ne s' agit pas de lui faire des promesses , on lui en a trop fait . Il s' agit de transmettre notre culture , nos valeurs et ainsi de permettre à tous les jeunes de construire librement leur avenir . C' est cela croire en sa jeunesse . Bâtir notre projet , par des nouvelles protections aussi . Car dans ce monde incertain , chacun doit être libre et nous devons pour cela protéger contre les risques de la vie , contre l' insécurité . Garantir ces protections : la santé , la retraite pour les plus âgés . Parce qu' on ne construit rien de solide en laissant des millions de Français au bord du chemin . C' est cela la vraie autorité . C' est cela la crédibilité de l' Etat . Vous l' avez compris , je veux libérer l' énergie de ceux qui peuvent , et dans le même temps je veux protéger les plus faibles . Ce projet , nous devons aussi le bâtir par nos territoires , nous avons des France aujourd'hui à réconcilier . Nos métropoles réussissent dans cette mondialisation . Il faut les y aider , les encourager , leur permettre de faire davantage . Elles sont notre chance . La France des quartiers a elle besoin de mobilité . Elle est aujourd'hui assignée à résidence alors qu' elle veut réussir , qu' elle veut que nous permettions aux jeunes et moins jeunes d' accéder à l' école , d' accéder à la culture , d' accéder à l' emploi . La France périurbaine et la France rurale ont le droit de réussir , de se développer . Et pour ce faire , nous devons aussi leur permettre d' aller de l' avant , investir et laisser faire . Cette France-là le veut , cette France le peut . Je veux une France qui croit en sa chance , qui risque , qui espère , qui n' admet jamais la rente indue ou le cynisme repu . Je veux une France entreprenante , où chacun choisit sa vie , où chacun peut vivre de son travail . Et une France qui considère les plus faibles . J' ai pour cela une confiance indéfectible dans les Français . L' enjeu n' est pas pour moi aujourd'hui de rassembler la gauche , il n' est pas pour moi aujourd'hui de rassembler la droite . L' enjeu est de rassembler les Français . Car cette transformation de notre pays n' est pas un combat contre quelqu'un , contre un camp , contre une partie de la France . C' est un combat pour nous tous , pour l' intérêt général , pour nos enfants . Ce combat , c' est celui que nous conduirons ensemble . Cette révolution démocratique , je ne pourrai la mener à bien qu' avec vous . J' en appelle aujourd'hui à toutes les femmes et les hommes de bonne volonté . J' en appelle aujourd'hui à toutes celles et ceux qui dans notre pays croient dans la réconciliation de la liberté et du progrès . J' en appelle aujourd'hui à toutes celles et ceux qui ne veulent pas simplement guetter dans la pénombre une lueur d' espérance mais qui veulent l' incarner . Car c' est notre vocation profonde et je n' en sais de plus belle . Vive la République et vive la France ! Merci à vous . ",
    "Bonjour à tous ! Bravo à vous ! Bravo à vous ! Bravo à vous ! Bonjour à toutes et tous . Merci d' être là . Merci d' être là . Vous êtes venus des quatre coins de la France . Alors je vous le demande ce soir , est -ce que nous sommes seuls ? Est -ce que vous avez peur ? Est -ce que vous êtes prêts à changer la France ? Alors les choses sont claires . Nous sommes ce soir ensemble , 15000 ! 15000 , mes amis ! Et je le sais , beaucoup nous regardent en ce moment , en direct , aussi . 15 000 venus de la France entière . De toutes les régions , et vous êtes là . Mais ce qui fait notre force , ce qui nous tient debout , ce qui nous tient ensemble , c' est que nous savons qui nous sommes . Et c' est que nous savons où nous voulons aller , ensemble . Ce qui fait notre force , c' est notre détermination , notre cohérence , c' est notre sincérité . C' est vous , depuis le début . Et si vous êtes là , tous et toutes , si vous avez décidé en ce samedi après-midi , à quinze jours de Noël , de prendre du temps à vos familles , ou de venir en famille , de prendre du temps de vos proches , c' est parce que ce qui nous rassemble , c' est que nous sommes sûrs qu' il n' y a pas de fatalité . Il n' y a aucune fatalité à ce que le pays , ni dans quelques mois , que le choix entre une droite extrême et une droite conservatrice , il n' y a pas de fatalité , il y a une espérance , et c' est la nôtre ! Nous en avons parcouru du chemin , depuis le 6 avril dernier . Vous vous en souvenez ? En quelques mois , nous sommes devenus près de 120000 , ce soir , nous serons 120000 ! Ces dernières semaines , ce sont 400 réunions par semaine qui se sont tenues partout dans le pays , c' est votre mobilisation , c' est vous qui l' avez voulue ! Nous avons fait du chemin , depuis le 6 avril dernier . Nous l' avons fait ensemble . Depuis Amiens , je viens de cette ville , je suis né à Amiens . Et il n' y a rien non plus qui devait me conduire ici , pas plus que Sabrina , Marisa , Richard , Jean , Sylvie , Gérard , vous toutes et tous . Ce qui m' a conduit , c' est le travail , le goût de l' effort , la volonté de comprendre , d' apprendre , de construire , ce fut la République aussi , par l' école , qui m' a tiré , les instituteurs , les professeurs , les concours , ce que j' ai pu réussir , ce que j' ai pu rater . Le service public , que je viens de quitter , et qui m' ont tant appris . La chance , parce que , oui , j' ai eu de la chance , beaucoup . Et il en faut . Nous en avons toutes et tous , il y a parfois des déveines , des malentendus , des échecs , des maladresses , des phrases qu' on regrette , je veux d'ailleurs ce soir lever un malentendu récurrent , je n' ai rien contre celles et ceux qui portent des T-shirts . Il m' arrive moi-même d' en porter , et j' aurais sinon trop de problèmes avec les formidables helpers qui sont là et qui arborent nos magnifiques T-shirts . La famille aussi , sans laquelle on ne peut prendre des risques , et qui reste mon socle d' attache , ce qui me tient . Et la confiance , enfin , la confiance de nos maîtres , de nos amis , de nos proches , c' est cela qui nous fait . Et puis votre confiance , mes amis , votre confiance depuis le premier jour , car c' est vous qui m' avez porté , car c' est vous qui nous avez faits , car c' est vous qui nous avez transformés , car c' est par votre confiance que nous sommes là ce soir ! Mais peu importe d' où nous venons , parce que nous savons ensemble où nous voulons aller . Le combat que nous voulons mener . Ce qui est important pour nous . Ce que nous voulons , ce n' est pas rassembler la gauche , ce n' est pas rassembler la droite , c' est rassembler les Français ! C' est cela notre projet ! Alors , c' était un rêve fou il y a six mois , il est certain . Mais ce rêve fou , vous êtes en train de le réaliser . De partout en France , des quatre coins de France , vous en êtes la preuve . Et certains n' ont pas compris , et pensent encore que vous avez peur . Ils n' ont pas compris . Vous avez peur ? Ils n' ont pas compris parce que nous avons déjà , nous , pris les risques . Le risque de faire , le risque de tenter , ce que tout le monde nous disait comme impossible . Le risque d' essayer , partout , et d' aller convaincre . Le risque de quitter le gouvernement pour les uns , et la fonction publique , le risque de quitter ses attaches pour d' autres , et de braver les interdits , les menaces . Le risque de prendre du temps , le risque de l' engagement , d' être là . Alors , oui , vous avez déjà pris les risques , nous avons décidé d' oser , nous voulons le faire parce que notre volonté , notre détermination , sont claires : il y a dans notre pays une place pour réconcilier la liberté et le progrès , cette place , c' est la nôtre , c' est celle que les Français veulent ! Alors , depuis tant de mois , vous avez conduit la Grande Marche . Des milliers d' entre vous ont été frappé aux portes de 100 000 Français . Nous avons ensuite parcouru la France de Strasbourg au Mans , en passant par Montpellier , pour expliquer , rendre compte , ah , que n' ai -je entendu . Nous étions une bulle , nous n' avions rien à dire . Bien sûr , on s' était tellement habitué à ne plus écouter les Français , à ne plus faire avec eux , à ne pas chercher à comprendre ce monde qui se transforme et ses défis , pour lui imposer des potions venues d' en haut , pour lui administrer des traitements dont l' un était de gauche et l' autre de droite , et au fond , il n' y avait que cela qui comptait . Non , nous l' avons fait ensemble , nous l' avons construit avec les Français . C' est cela notre projet . Et notre projet , c' est de faire entrer la France dans le XXIe siècle , c' est de faire gagner notre pays , c' est de le faire réussir dans un monde qui se transforme , où les menaces paraissent être partout mais où les opportunités sont là . Mais avec un engagement clair que je veux ici redire , partager solennellement avec vous l' engagement qu' il n' y aura pas de laissés pour compte , que notre devoir c' est en même temps de réussir et de faire réussir notre pays , mais de s' assurer que chacune et chacun y trouve sa place ! Ce projet , il est cohérent . Il repose sur une vision d' ensemble où tout se tient , il n' est pas une série de mesures qu' on égrène , qui peuvent se remplacer ici ou là , c' est un projet qui se tient , c' est notre révolution démocratique , celle que nous voulons porter et que nous porterons jusqu' au bout ! Et notre programme , je veux commencer à le partager ici avec vous aujourd'hui . Ce seront les premières de ces propositions de transformation que nous aurons à porter pour le pays . D' autres viendront , jusqu' à la fin du mois de février , sur l' éducation , sur l' Europe , sur l' environnement , sur la culture , sur la défense , sur la sécurité , sur les territoires et l' agriculture , ce sera le contrat que nous passerons avec la nation , le contrat pour gagner , le contrat pour présider , et le contrat pour gouverner ! Mais je veux , ce soir , vous parler des premières batailles , celles contre le chômage , celles contre , précisément , tout ce qui assigne à résidence une partie de nos concitoyens dans la société française , cette bataille que nous devons conduire , la bataille économique et sociale , c' est la première , et celle-là , nous la gagnerons . Parce que depuis plus de trente ans , notre pays ne parvient pas à sortir du chômage de masse . Parce que depuis trente ans , le chômage de longue durée s' est installé dans de trop nombreux territoires , et quelle en est la conséquence ? Des centaines de milliers de nos concitoyens n' ont parfois connu que le chômage , le leur , parfois celui de leurs parents . Ce sont des valeurs à ce moment-là qui se détricotent . C' est un espoir qui s' évanouit . C' est une désespérance qui naît . Mais notre pays aujourd'hui , c' est 3 millions de chômeurs , près de 6 millions de Françaises et de Français qui connaissent une des formes de chômage , presque toutes les familles de notre pays , ce sont 9 millions de nos concitoyens qui sont dans la pauvreté . Et donc , oui , la première des batailles , celle qui fait que notre identité peut se tenir debout , que notre pays peut se porter fier , que les projets peuvent être conduits , c' est la bataille économique et sociale , et c' est celle que nous allons conduire ! Et pour cela , il ne s' agit pas de vouloir administrer contre la France et les Français des potions intenables . Il ne s' agit pas d' aller expliquer au pays qu' on veut être efficace en étant injuste . Ce n' est pas la vision que nous porterons . Je l' entends parfois , c' est une vision conservatrice . Il ne s' agit pas non plus d' aller expliquer au pays que tout sera facile , que nous allons créer de nouveaux droits , que nous allons être justes sans en avoir les moyens et sans être efficaces . La voie qui sera la nôtre , c' est celle du progressisme , celle qui consiste à proposer à notre pays des transformations efficaces et justes . Celle qui consiste pour notre pays à porter ce projet économique et social que nous avons construit ensemble , au concret , au réel , au contact , et qui précisément va nous permettre d'abord de libérer le pays , de le libérer par le travail , de le libérer par ses territoires . Et en même temps , parce que ces deux promesses sont inséparables , de protéger les Françaises et les Français contre ces risques nouveaux , contre ce qui advient , contre ce monde fait de risques . Libérer d'abord , oui , c' est bien le premier socle de notre projet et libérer par le travail , car il sera au cœur de notre action . Mais pourquoi ? Parce que qu' est -ce qui fait le travail ? Qu' est -ce qui le rend si important dans notre société ? Qu' est -ce qui fait , cher Jacques , tu t' en souviens , que près de Château-Thierry , lorsque nous avons rencontré la famille Guyon , tout était fait dans le monde qui nous entoure pour le faire échouer . Des agriculteurs spécialisés dans le porc et la volaille , pensez -vous , mais nous avons vu une famille qui se bat sans relâche par le travail et pour le travail . Qu' est -ce qui fait que d' autres jeunes , dans le pays , trouvent d' un seul coup un avenir , parce qu' une innovation technologique , parce qu' une application sur Internet leur a soudain donné un travail , une dignité , une place , la possibilité de conduire une voiture , de partir le matin et de revenir le soir , quels que soient les donneurs de leçons . Qu' est -ce qui fait Patrick , que tu es là ce soir , que je te connais ? Et chaque jour , je vois que tu défends l' apprentissage comme aucun autre , avec enthousiasme . C' est le travail . C' est ton métier de prothésiste qui t' a fait ça , qui t' a fait là ce soir . Alors , oui , le travail , ce n' est pas une souffrance . Il faut protéger au travail , mais je ne recommande pas de vouloir protéger les Françaises et les Français du travail , ce n' est pas une bonne idée . Le travail , c' est ce qui nous donne une place dans la société , c' est ce qui nous donne notre dignité , c' est ce qui nous permet de nous émanciper , de retrouver confiance en nous , ce qui nous permet de nous construire , ce qui nous permet de réussir et de faire réussir les autres , notre famille , d' entreprendre , d' être heureux . Et je sais les indignations , je sais ces millions de Français qui veulent rentrer dans le travail , et qui de Cahors à Bobigny , en passant par Brest , sont aujourd'hui bloqués , comme entravés , je sais tous ceux qui voudraient embaucher , ils n' y arrivent pas , parce qu' ils ont peur , parce qu' ils se disent que les risques sont trop grands , parce qu' il n' y a aucune fenêtre ouverte . Alors , oui , parce que je veux être le candidat de la justice , je suis le candidat du travail ! J' étais il y a quelques jours à New York , j' y rencontrais Joseph Stiglitz , un affreux gauchiste pour les américains , Prix Nobel de son état , que m' expliquait -il ? Que précisément , le travail était la clef de l' émancipation contemporaine , mais qu' il fallait sortir de cette espèce de frilosité selon laquelle il ne sert à rien de vouloir faire grossir le gâteau , il faudrait simplement le partager , réduire toujours un peu plus , que le travail ne serait qu' une quantité finie . En finir avec ces idées un peu désespérantes . Et que la solution , c' est au contraire de comprendre que le travail se transforme , qu' on ne travaillera plus demain comme aujourd'hui , ni comme hier , et qu' il faut pour cela encourager toutes les formes d' emploi à prospérer , en donnant aux individus les protections dont ils ont besoin , il faut libérer le travail , libérer l' action au travail , la création d' entreprise , l' initiative ! Alors , oui , pour cela , nous allons réconcilier notre pays avec le goût du risque , avec la volonté d' entreprendre , avec la volonté de créer , à tout niveau . Parce que l' artisan crée , parce que l' ouvrier crée , parce que l' entrepreneur crée , parce que l' artiste crée , et c' est cela le socle de notre pays . Mais pour ce faire , la France doit retrouver ce goût du risque , elle doit savoir parfois aussi accepter l' échec . Et pour cela , oui , nous devons , à travers plusieurs transformations , réduire le coût de l' échec , faciliter l' échec quand il advient , parce que ce n' est pas grave , parce que le cœur , c' est d' avoir entrepris , c' est de tenter , parce que quelques-uns échoueront , d' autres réussiront , mais le pays avancera ! Pour cela , nous continuerons à réduire le coût du travail pour les entreprises . Parce que si les coûts , quels qu' ils soient , sont trop élevés , alors les marges sont basses , et alors la capacité à investir et à embaucher n' est plus au rendez-vous . La bataille de notre économie , ce sera la bataille pour l' investissement et pour l' emploi . Elle commence aujourd'hui . Parce que dans ce monde , qui se transforme , où le numérique bouscule tout , où la transformation environnementale et énergétique change des secteurs entiers , nos entreprises se doivent d' investir , se doivent d' embaucher , se doivent de se transformer . Et donc , oui , pour ce faire , nous devons les aider à reconstituer leurs marges , nous devons les aider à créer ces emplois de demain . L' investissement productif et l' emploi , ce seront nos batailles . Alors , pour cela , non seulement je maintiendrai les allègements de cotisations déjà décidés durant les dernières années , mais je transformerai le crédit d' impôt compétitivité emploi en allègement de charge pérenne à hauteur de 6 points , pour toutes nos entreprises , quelles qu' elles soient , quel que soit le secteur . Les entreprises du secteur marchand , comme de l' économie sociale et solidaire , les grandes comme les petites , les indépendants , les artisans , tout le monde aura ses allègements de charges . De manière claire , de manière stable , et de manière lisible . Parce que c' est à la fois une politique pour la compétitivité , et pour l' emploi . Libérer l' accès au travail , cela passe aussi par la libération du dialogue social au plus près de l' entreprise et du terrain . Parce que comment pourrait -on penser aujourd'hui que la loi pourrait décider de tout et partout ? Que la loi pourrait être la même , lorsqu' on a 25 ans , 40 ans , 55 ans , et que , par définition , nous n' avons pas le même rapport au travail , surtout si l' activité au quotidien n' est pas la même . Parce qu' on n' est pas fatigués de la même façon lorsqu' on fait de la politique , lorsqu' on est posté dans une usine automobile , lorsqu' on travaille à Arc international , chère Caroline , nous nous en souvenons , lorsqu' on travaille dans le bâtiment ou les travaux publics , ou la boulangerie , nos vies ne sont plus les mêmes . Parce que , de la même façon , nos secteurs économiques sont en train de se transformer , de se recomposer , avec des logiques profondément différentes . Certains sont en train de se créer , ont besoin de plus de souplesse et de vitalité pour faire face à de nouvelles commandes , et d' autres sont frappés par des restructurations , parce que la concurrence internationale est là , parce que produire en France , sur ces secteurs , est rendu chaque jour de plus en plus difficile , et donc oui , les réalités partout sur le territoire , entre les secteurs de notre économie ne sont plus les mêmes . C' est cette souplesse que nous devons retrouver , mes amis , c' est cette dynamique différente . Il faut l' expliquer , et il faut la construire . Voilà pourquoi je propose que la loi édicte des règles simples , lisibles , définisse un ordre public social qui doit être le même partout et pour tous , lui , qui fixe la durée légale du temps de travail , elle doit rester à 35 heures . Ce n' est pas la peine , alors que je suis en train de vous dire que la réalité n' est pas la même partout dans le pays d' aller expliquer qu' il faudrait dans la loi passer de 35 à 39h , ce n' est pas cela la réponse . Il faut que la loi fixe bien entendu le salaire minimum , les principes fondamentaux et au premier chef l' égalité entre les femmes et les hommes . Et il n' est pas question ici d' obliger par la loi les jeunes à travailler plus ou à gagner moins . J' en entends certains qui voudraient déformer nos propos , qui voudraient faire croire des balivernes , ils sont tellement habitués à ce petit jeu que je veux le repréciser . Non , une loi simple qui fixe des règles minimum pour tous , parce que ce sont les garanties . Et ensuite , nous devons renforcer la négociation au niveau des branches , des secteurs d' activité et des entreprises . C' est comme cela que nous créerons cette République contractuelle à laquelle nous croyons . Qu' est -ce que cela veut dire ? Ça veut dire que je propose que dans chaque branche , que dans chaque entreprise qui le souhaite , par un accord majoritaire , et donc par le dialogue social , et un accord entre les représentants des salariés et les représentants des dirigeants d' entreprises , on puisse définir les bons compromis , les bons accords , qui permettent au plus près du terrain de créer plus d' emplois , parfois de décider de changer les seuils si c' est plus pertinent et si les uns et les autres le veulent , parfois de modifier le nombre d' heures que l' on travaille avec des heures supplémentaires dûment payées , parfois de modifier l' organisation au plus près de l' entreprise parce qu' on l' aura voulu , parce qu' on l' aura choisi , c' est cela ce que nous voulons , c' est cela qui permettra la vitalité de notre économie ! Et dans cette République , où je veux privilégier le contrat à la loi , je vous demande à toutes et tous de jouer votre rôle , parce que c' est un pouvoir que je vous rends . Parce que c' est la décision prise ici solennellement que ce n' est pas un homme qui , devenu président , par des textes ou des décisions , pourra régenter tout ce qui se passe partout dans notre pays , ce n' est plus vrai . Cela n' est plus le XXIe siècle . Parce que de la même façon , cela ne peut pas être une loi prise par un parlement pour tout le pays , qui va régenter toutes les situations , répondre à tous les enjeux pour le pays . C' est vous , vous qui aujourd'hui avez décidé d' être les acteurs du changement politique . Vous qui avez décidé de faire , je vous demande , demain , d' être les acteurs du changement social , et dans vos entreprises , partout , qu' elles soient petites ou grandes , par le dialogue , par le débat , de construire les règles qui seront adaptées à votre quotidien , qui seront adaptées à vos défis , parce que j' ai confiance dans votre intelligence , dans votre énergie , parce que c' est ça la France ! Enfin , parce que je suis le candidat du travail , je veux que le travail paye mieux . Parce qu' aujourd'hui , lorsqu' un employeur verse au SMIC 1467 euros , le salarié ne perçoit que 1141 euros de salaire net . Pourquoi ? Parce que nos grands risques sociaux sont financés sur les salaires , par le travail . Parce que c' est ainsi que nous avons construit nos équilibres , historiquement . Parce que , hier , lorsque , en 1945 , puis dans les décennies qui ont suivies , nous avons construit ce modèle , notre modèle français , qui a été notre fierté , qui a formidablement fonctionné , qui a permis le progrès , lorsqu' on a pensé ce modèle , le chômage était anecdotique . La réalité , c' était le plein emploi . La réalité , c' était une économie de cycles longs , où les bouleversements n' étaient pas d' une brutalité comme nous les connaissons aujourd'hui . Ce n' était pas le même monde . Eh oui , aujourd'hui , le travail est concurrencé non seulement par des pays à bas coût , ou à coût inférieur , mais aussi par la robotisation , par d' autres façons de créer de la richesse , et nous les saisirons , nous continuerons à les saisir . Et donc la base de financement de notre modèle social , de nos sécurités collectives , ne peut plus être le travail . Voilà pourquoi je propose que toutes les cotisations maladie et les cotisations chômage qui sont aujourd'hui payées par les salariés soient supprimées . Ce sera une augmentation du salaire net pour chacun . Une augmentation qui n' est pas payée par l' employeur et qui ne touchera donc pas la compétitivité de nos entreprises , mais qui sera transférée sur la CSG . Qu' est -ce que cela veut dire concrètement ? Cela veut dire que pour tous les salariés , tous les fonctionnaires , tous les indépendants de notre pays , toutes celles et ceux qui travaillent , le salaire net pour tous augmentera . La CSG , elle , augmentera de manière limitée , parce que sa base est beaucoup plus large , d' 1,7 point , mais je protègerai dans cette décision celles et ceux qui sont au chômage , et 40 % des retraités les plus modestes . Ils ne seront pas touchés . Et je veux ici être très clair . Qu' est -ce que cela veut dire , pour vous donner un exemple ? Qu' un couple de travailleurs qui touche aujourd'hui le SMIC aura , après cette transformation , 500 euros nets par an de pouvoir d' achat en plus . Alors , à qui je demande un effort ? Parce que notre règle , c' est de toujours dire la vérité . On ne va pas cacher les choses . Et je vous le dis , toutes celles et ceux qui aujourd'hui font des promesses formidables n' expliquent jamais à qui on demande des efforts . Je le sais bien , on dit : cachons -le , ils le verront suffisamment tard , ils s' en rendront compte après l' élection . J' ai connu ça . Ça peut marcher le temps d' une élection , ça se passe moins bien après . Alors , je vais vous dire , je demande un effort , évidemment à tous les travailleurs , mais qui est plus faible que ce qu' ils vont gagner , donc le pouvoir d' achat de tous les travailleurs va augmenter . Et je demande un effort net à deux catégories : les revenus du capital , 1,7 point de CSG . Ce n' est pas énorme . Et cela s' inscrira dans une refonte en profondeur que je présenterai dans les prochaines semaines . Et de l' autre côté , les retraités les plus aisés , la moitié des retraités les plus aisés . Je leur demande , pourquoi ? Je leur demande pour leurs enfants et leurs petits-enfants . Je n' augmenterai pas leurs impôts par ailleurs , je vous le garantis ici . Je n' augmenterai pas le coût de la maladie , je le garantis ici , et j' y reviendrai . Je leur demande simplement en toute transparence ce petit effort de quelques dizaines d' euros par mois , parce que nous en avons besoin pour que le travail dans notre pays paie . Mais en même temps , je veux vous le dire ici avec beaucoup de solennité , je protègerai le pouvoir d' achat de tous les retraités modestes pendant toute la durée du quinquennat , parce que durant les deux quinquennats qui ont été , justement durant ces dix années qui viennent de passer , ils ont beaucoup trop été mis à contribution . Tous ces retraités modestes , nous les protégerons , il n' y aura aucune baisse de leur pouvoir d' achat dans les cinq années à venir . Et donc , vous l' avez compris , le travail paiera mieux , dès demain , pour les fonctionnaires , pour les artisans , les indépendants , pour les salariés dans notre pays . Mais je suis aussi convaincu que l' incitation à la reprise d' un emploi devra être renforcée par une augmentation de la prime d' activité , parce qu' il n' est pas normal aujourd'hui qu' un salarié à mi-temps au SMIC ne voit quasiment pas sa rémunération augmenter lorsqu' il reprend une activité à temps plein . Et donc cela aussi , nous le modifierons pour que partout dans notre pays , vous ne puissiez plus entendre qu' il est plus intéressant de ne pas travailler ou de travailler moins que de se remettre au travail ! La libération ! La libération du pays que nous voulons , mes amis , c' est aussi une libération de tous les territoires , des Françaises et des Français qui font , partout dans le pays , c' est à eux que je veux rendre le pouvoir , c' est à eux que je veux donner justement toute la reconnaissance et les responsabilités . Alors comment ? D'abord en donnant plus de responsabilités à nos élus locaux , parce que , oui , nous devons aller plus loin dans la décentralisation . Parce que , oui , partout sur le terrain , nos élus locaux sont ceux qui font , qui entreprennent , au quotidien , qui savent la responsabilité . Non pas tous les cinq ans ou tous les six ans , ils la connaissent tous les jours . Tous les jours . Alors , je souhaite leur donner plus de responsabilités parce que notre République est grande , elle est indivisible , nous allons la rendre plus forte , mais elle sera aussi plus forte en reconnaissant partout ses énergies , ses spécificités , cette Bretagne conquérante , cette Corse fière d' elle-même , cette volonté justement d' entreprendre partout sur le territoire avec une énergie différente dans un même projet , le projet de la République , pas le projet du XVIIIe siècle , mais celui du XXIe siècle ! Je veux que les métropoles , oui , Gérard , cette belle métropole de Lyon , et tant d' autres , je veux qu' elle continue à réussir encore davantage . Tu as montré l' exemple en réussissant à reprendre les compétences du département , à simplifier , pour de vrai , pas dans un texte , pour de vrai , ce millefeuille dont on nous parle tant ! Je suis pour que partout , lorsqu' il y a des métropoles qui réussissent , lorsqu' il y a ces grandes cités triomphent dans la mondialisation , qu' on puisse les aider à aller plus loin , je suis pour qu' elles puissent absorber le département , qu' on simplifie notre structure pour rendre ces territoires plus forts . Parce que ces métropoles doivent continuer à rayonner , à entraîner d' autres territoires voisins , d' autres villes , c' est leur responsabilité . Parce qu' il y a aussi les quartiers populaires dans nos villes , ou les villes les plus en difficulté . Je veux que nous prenions tous les risques , que nous allons vers des nouveaux schémas d' urbanisme , que nous prenions enfin nos responsabilités en termes de rénovation urbaine . Nous avons divisé par deux le budget de l' ANRU ces dernières années , nous devons reconquérir cela pour refaire nos villes belles . Cher Roland , je te sais ici … Pour dans ces quartiers , comme à Poissy , à la Coudraie , comme dans d' autres quartiers cela a été fait , recréer de la mixité , recréer du logement , redonner de l' énergie , savoir recréer la vraie mixité sociale et économique dont notre pays a besoin ! Et dans nos territoires ruraux , là , nous avons besoin des départements . La République doit être plurielle . Nous avons besoin des départements , de ces villes-centres , nous avons besoin de ces communes , des services publics , nous avons besoin , oui , des services postaux et des écoles , de la possibilité de reconquérir des places économiques , sociales , de l' activité partout , parce que ces territoires aussi ont de l' innovation économique , sociale , parce que ces territoires veulent entreprendre , de la Lozère en passant par la Corrèze , nous devons leur faciliter la vie , et c' est ce que nous entreprendrons ! Et pour les y aider , nous allons amorcer là aussi un grand mouvement de transformation de l' Etat . Je propose une déconcentration nouvelle , c'est-à-dire je propose que la responsabilité , que la capacité de faire , ce ne soit plus celle d' un ministre , ce ne soit plus celle d' une administration , de quelques-uns en haut , mais des femmes et des hommes qui , sur le terrain , se battent pour faire , pour transformer le pays , de tous ces fonctionnaires et ces services qui , sur le terrain , qu' ils soient dans les hôpitaux , dans les écoles , dans les universités , connaissent la dynamique en œuvre , connaissent les problématiques complexes et veulent changer les choses ! Et donc , oui , je propose plus d' autonomie , la vraie autonomie , pour les universités . Je propose l' autonomie des établissements scolaires et des communautés pédagogiques . Des hôpitaux et des agences régionales de santé , pour pouvoir faire , comme je l' ai vu à Sallanches par exemple , il y a quelques semaines , où lorsqu' on donne la possibilité d' initiative , la capacité d' inventer , nos fonctionnaires , dont on nous parle tant , dont on dit tant de mal , nos fonctionnaires qu' on ne sait plus citer dans le pays que pour savoir le nombre de têtes qu' il faut couper . Nos fonctionnaires auront alors la pleine capacité de faire . Parce qu' aimer l' Etat , aimer la fonction publique , aimer le service public , ce n' est pas dire : il faut tout supprimer , il faut tout couper d' un côté ou il ne faut rien changer de l' autre et garder en l' état . Non , c' est de dire que partout dans notre pays , nous avons une fonction publique d' une qualité incroyable , nous avons une fonction publique dont nous devons être fiers , mais nous devons la libérer , nous devons la laisser innover , nous devons la laisser inventer ! Et donc , oui , je crois à une fonction publique entreprenante , qui puisse accompagner les territoires , qui puisse trouver ses solutions , qui ait la possibilité de décider . Et en même temps , je propose une véritable simplification , non pas , et nous essayerons de le faire , en supprimant des normes , parce que pour cela , vous savez , il faut imaginer Sisyphe heureux , et il n' était pas heureux , je dois vous le dire . Ma conviction , c' est que personne ne devient fonctionnaire pour appliquer bêtement des lois , pour embêter , pour traquer , pour appliquer sans réfléchir des normes parfois idiotes , pour le perroquet d' instruction . Il y a des règles à faire respecter partout , c' est une évidence , c' est notre ordre public , et la révolution que je propose , c' est de changer le rapport des Français à leur administration et le rapport de l' administration aux Français . Ce que je propose , c' est de créer partout un droit à l' erreur pour tous ! Qu' est -ce que cela veut dire ? Que les contrôles opérés par les pouvoirs publics se feront non plus d'abord pour sanctionner , mais pour orienter ! Ce que je veux dire , c' est que le contrôler de bonne foi pourra faire valoir son droit à l' erreur . Et ne sera donc pas à la première occasion sanctionné , ce sera à l' administration qu' il reviendra de prouver que l' erreur a été commise intentionnellement , ou que la personne est de mauvaise foi . Sauf , bien sûr , je vous rassure , parce que j' entends déjà les déformations , en matière pénale , ou lorsqu' elle engage la responsabilité d' autrui , où là , nous serons intangibles , mais ce droit à l' erreur , c' est la possibilité par exemple , lorsqu' un employeur oublie de déclarer à l' URSSAF la prime de Noël , qu' il verse à ses salariés , qu' il ne soit pas condamné immédiatement à une amende , ce qui est le cas aujourd'hui . Il l' a rarement fait exprès . C' est de faire qu' un employeur qui effectue en retard une déclaration préalable d' embauche est aujourd'hui inquiété par l' administration et que demain , l' administration lui conseillera d'abord , lui expliquera . C' est passer d' une administration du contrôle et de la sanction systématique à une administration du contrôle , de la vigilance mais du conseil et de l' accompagnement . Il y aura toujours des fraudeurs , il y aura des malhonnêtes . Mais nous devons en même temps redonner dans notre pays place à la confiance , c' est cela ce que permet cette transformation , c' est cela ce que permet cette révolution , elle accompagne tous ceux et celles qui osent , qui entreprennent , et qui veulent faire ! Et en même temps , mes amis , que nous devons libérer le pays , je vous le disais , nous devons protéger . Protéger nos concitoyens contre les nouveaux risques , contre ces transformations à l' œuvre , contre un monde qui change , parce que libérer pour permettre d' entreprendre , d' oser , de créer , de réussir , ou d' échouer , c' est formidable , mais il y aura toujours des fragiles , il y aura toujours des oubliés , et notre devoir , notre projet parce qu' il est progressiste , c' est de leur aménager une place , c' est de les aider . Et donc notre projet , pour protéger , sera autour de trois boucliers , un bouclier de sécurité , un bouclier social actif , et un bouclier européen , car c' est ainsi que nous protégerons les Françaises et les Français ! Alors , oui , le premier bouclier , c' est celui de la sécurité . Parce qu' il n' y a pas de liberté qui vaille si chaque jour , on a peur de l' insécurité . Et quand on a peur de l' insécurité , il n' y en a pas de petite ou de grande . Il n' y a que la peur d' être , il n' y a que la liberté d' aller et venir dont on est alors privé . La sécurité , c' est ce premier devoir de la puissance publique , c' est ce premier devoir qui nous unit au cœur du pacte républicain , de faire que partout en France , les femmes au premier chef , et les hommes , ne doivent plus être inquiétés , qu' il ne doit plus y avoir de territoire de non droit , et là-dessus , je serai clair : l' autorité de l' Etat doit être intangible , et notre autorité sera inflexible ! Parce que l' égalité que nous défendrons , la vraie égalité , c' est l' égalité face à la sécurité . Et nous le savons . Nous savons ce que vivent nos forces de l' ordre , partout dans le territoire , le manque de moyens , pas de radio , parfois pas assez d' équipements , lorsque les équipements sont là , j' étais , l' autre jour , dans un commissariat , les coffres sont trop petits , parfois les logiciels qui ne fonctionnent pas , qui mobilisent quatre fonctionnaires pour procéder aux vérifications d' usages , donc oui être à la hauteur de ce bouclier de sécurité , c' est reconnaître l' importance et la place de nos forces de l' ordre , c' est leur donner des moyens dans la durée , c' est non pas simplement avoir des grands mots pour elles , mais une reconnaissance concrète , un sérieux au quotidien . C' est cela ce que nous ferons . Alors , pour ce faire , au-delà des 9000 embauches décidées , et qui sont encore en cours , ce seront 10 000 fonctionnaires de police et de gendarmerie supplémentaires qui seront recrutés durant les trois premières années du quinquennat . C' est une nécessité , nous le ferons ! Mais il ne suffit pas de recruter , il faut rendre du temps utile aux policiers et aux gendarmes pour être efficaces , pour débarrasser de formalités inutiles , inventées parfois pour se faire plaisir , pour la pureté du droit ou parce qu' on a cru , loin du terrain , que c' était une bonne idée . Il faut simplifier les procédures . Nos forces de police et de gendarmerie le savent , nos magistrats aussi et nous avons travaillé avec eux . Nous réduirons ainsi les contraintes sans réduire en rien les droits et les libertés de nos concitoyens . A côté de ces simplifications et de ces engagements , nous devons aussi nous réorganiser . Et de deux façons . D'abord en acceptant l' idée que nous devons , oui , reconstituer notre renseignement territorial dans les endroits les plus sensibles du pays . Il a été déconstruit au début de notre siècle , de manière maladroite , à contretemps , nous devons dans les quartiers les plus sensibles , dans toutes les zones de France , ainsi identifiée , remettre des forces de renseignement , les vraies , au contact du terrain , avec des fonctionnaires de qualité , formés pour ça , parce que c' est là que le véritable risque terroriste peut être repéré , parce que c' est là que les signes avant-coureurs sont identifiés . Et dans le même temps , nous devons mieux organiser notre renseignement pour que toutes les données collectées partout sur le territoire puissent être regroupées dans une organisation unique , simplifiée , plus cohérente et efficace , sous évidemment le contrôle démocratique , qui convient , mais comme l' ont fait les Britanniques et les Américains pour que nous soyons efficaces au regard des technologies contemporaines pour lutter contre le risque terroriste . Nous devons aussi , c' est la deuxième réorganisation que je veux proposer , recréer une nouvelle police de proximité . Parce que , oui , cette police de proximité , créée à la fin des années 90 par Lionel Jospin et Jean-Pierre Chevènement , elle était une bonne idée . Parce que , oui , cette police de proximité , il faut bien le dire , elle ne servait pas uniquement à jouer au football avec les enfants . Et parce que , même quand on joue au football , avec des enfants , quand on est présent sur le terrain , on voit d' autres choses , on crée un autre rapport à l' autorité républicaine , on perçoit les signes avant-coureurs , on comprend ce qui est en train de se jouer , on prévient certains risques , on reconstruit différemment de la République partout sur le terrain . Donc , oui , nous recréerons cette police de proximité . Voilà pour la sécurité physique . Et pour protéger les Français , nous aurons un deuxième bouclier , le bouclier social , dont ils ont besoin . Ce bouclier social , c' est celui qui permettra à chacune et chacun non pas d' avoir tous les droits , sans devoir , mais qui , au regard des transformations en cours , permettra de trouver sa place dans la société , d' être considéré comme une personne , d' être pleinement reconnu et d' être protégé contre les événements adverses qui parfois nous arrivent dans la vie . Alors , le premier de ces devoirs , c' est évidemment l' éducation . J' y reviendrai plus longuement , mais je veux ici en dire un mot . Nous avons un devoir d' éducation partout dans le pays . Il ne peut pas être le même dans tous les quartiers de la République , il ne peut pas être le même partout , nous devons reconnaître qu' il faut faire plus pour ceux qui ont moins , que des inégalités se sont creusées , construites , où nous avons assigné à résidence des femmes et des hommes et leurs enfants , parce que nous n' éduquons plus comme nous devrions . Parce que celles et ceux qui , dans les quartiers populaires , sont les enfants des chômeurs de longue durée , de Françaises ou de Français qui parfois ne maîtrisent pas parfaitement notre langue , n' ont pas les mêmes chances de départ . Si nous avons un Français sur cinq , un jeune Français sur cinq en CM2 qui ne sait pas lire , écrire , compter , c' est que dans les quartiers les plus difficiles de la République , qu' ils soient ruraux ou qu' ils soient urbains , il y en a parfois un sur deux ou deux sur trois qui sont dans cette situation , c' est cela la réalité . J' étais il y a quelques semaines en Moselle . À Hombourg-Haut , cette petite ville de Moselle qui a souffert de la désindustrialisation , la fermeture des mines , il y a maintenant plus de trente ans . Et je vais dans le centre d' action sociale qui avait plusieurs fois brûlé , avec des femmes et des hommes extraordinaires qui étaient là pour créer de l' activité , pour les mères au foyer , de l' activité pour les plus jeunes après l' école , et du soutien scolaire . Ça se passait dans une école désaffectée , parce que cette ville a tant souffert qu' elle a perdu 5000 habitants et qu' une école a dû fermer . Et je croise une petite fille qui était au milieu des mamans qui faisaient du tricot , et je demande à la petite fille : tu as quel âge ? Elle me dit : j' ai 7 ans . Je suis en CE1 . Et je lui demande : qu' est -ce que tu veux faire ? On n' a pas des questions toujours très originales quand on est en déplacement . Elle me dit : je veux être traductrice . Je lui dis : c' est formidable ! Tu veux être traductrice ? Mais tu veux voyager , tu veux emmener les gens dans quel pays , tu veux partir dans quelle langue ? Elle se tourne vers sa mère . Alors , je demande à sa maman : c' est formidable . Mais qu' est -ce qu' elle veut , elle veut voyager ? Et la maman regarde sa petite fille et elle lui parle en arabe . Et j' ai alors compris que si la petite fille voulait devenir traductrice , ce n' était pas pour voyager comme je l' avais bêtement cru . Ce n' était pas pour emmener des gens très loin comme j' avais bêtement pensé . C' était simplement pour permettre à sa maman d' être dans la République . Alors , oui , pour l' école , nous ferons beaucoup . Dans ces endroits de la République , nous dédoublerons vraiment les classes , dans ces endroits de la République , nous ferons ce que nous devons faire , parce que cette petite fille , elle n' était pas née en Afghanistan , au Maroc , en Tunisie , elle était née à Hambourg-Haut , sa maman était née à Hambourg- Haut , et c' était en France ! Et notre bouclier social , ce sera aussi celui de la santé , là aussi , j' y reviendrai , mais il faut que chaque Française et Français puisse se soigner , accéder à la santé quel que soit son âge , son niveau de revenu , son origine . Et je trouve à cet égard dangereux que certains se mettent dans l' idée de dérembourser massivement des soins , des petits soins , nous dit -on , de dérembourser massivement dans le pays , pour les Françaises et les Français , ce qu' il faut pour se soigner , parce que c' est à la fois inefficace et injuste . C' est inefficace parce que la bataille de notre système de santé , c' est la prévention , parce que si la France est performante , c' est en matière de soins , si elle est défaillante , c' est en matière de santé . Et dérembourser les petits soins , quelle en est la conséquence mes amis ? C' est que les classes moyennes iront moins se soigner , c' est qu' on ira moins se couvrir contre les petits risques , c' est qu' on ira plus facilement vers les maladies les plus graves . Et en même temps , c' est injuste , parce que dérembourser les petits soins , c' est évidemment , non pas toucher les plus fortunés , qui peuvent avoir accès à d' autres couvertures privées , qui sont aussi nécessaires , ce n' est pas toucher les plus modestes , parce qu' ils sont , et nous le préserverons de manière absolue , couverts par la solidarité nationale , la couverture maladie universelle , générale et complémentaire , c' est toucher les classes moyennes , celles et ceux qui , comme d' habitude , sont au milieu , financent mais ne peuvent pas , elles , être protégées . Et donc ce projet de la droite conservatrice , quel en sera le résultat ? Je vais vous le dire simplement , avec la même transparence que quand je vous propose des choses . 1200 euros de dépenses de santé en plus par an pour un couple avec deux enfants , c' est cela le projet de la droite conservatrice , ce sont les chiffres de la fédération nationale de la Mutualité française . Un SMIC par an ! Donc ce projet , non seulement nous le combattrons , mais je veux le dire avec la plus grande clarté , pendant les cinq ans qui viennent , pas un seul soin utile ne sera déremboursé . Nous procéderons aux réformes , et j' y reviendrai , pour la prévention , pour réformer et transformer l' hôpital public , pour évidemment décloisonner le public et le privé au plus près du territoire , en donnant tout leur rôle aux mutuelles comme à la sécurité sociale , pour construire l' offre de soins adaptée partout et lutter contre les déserts médicaux . Et le bouclier social dont je veux plus particulièrement vous parler ce soir , c' est celui qui permettra que tout le monde ait les mêmes droits , et je veux ici parler du chômage et de la formation continue . C' est cela le cœur de la bataille et c' est le cœur de la bataille que nous mènerons ! Parce que dans ce monde qui se transforme , je vous le dis en toute honnêteté , le chômage ne peut plus être un risque contre lequel chacun se couvre à titre individuel . Cela ne peut plus être une assurance que chacun finance . Le chômage doit être une forme de solidarité universelle qui permettra de couvrir les Françaises et les Français dans tous les secteurs , qu' ils soient salariés , qu' ils soient indépendants et entrepreneurs . Je veux que chacune et chacun soit protégé de la même façon . C' est pour ça que je veux faire du chômage une vraie assurance universelle , une vraie solidarité nouvelle qui reconnaisse la dignité de la personne elle-même et qui ne couvre pas chacune et chacun simplement en fonction de ses cotisations . Concrètement , cela veut dire quoi ? Cela veut dire que chacune et chacun ne paiera plus son chômage , comme vous l' avez compris . C' est ce que j' expliquais tout à l' heure . Cela veut dire concrètement que chacune et chacun , quel que soit son statut , aura ce chômage et que nous sortirons de cette injustice où l' auto-entrepreneur , l' indépendant , l' artisan , l' entrepreneur n' a pas aujourd'hui ce chômage . C' est aussi pour cela que je veux que tous les cinq ans , pour éviter tout excès , tous les salariés puissent avoir le droit d' accéder également à ce droit de chômage s' il décide de démissionner . Pourquoi ? De manière très concrète . Parce que , aujourd'hui , qu' est -ce qui se passe ? Parce que , aujourd'hui , vous avez partout en France des gens qui contournent le droit , et qui ont recours à la rupture conventionnelle , pour faire semblant de démissionner et avoir le chômage . Donc ne vivons pas dans un monde d' hypocrites ! Parce que , quand ils ne font pas ça , qu' est -ce qui se passe ? Pour certains salariés , ils disent : tu ne veux pas me licencier , parce que tu ne veux pas me payer le chômage . Je vais te faire une vie impossible . C' est ça qui alimente les Prud'hommes , chaque jour . Et parce que de l' autre côté , quand il y a des salariés malheureux , ils vont voir leur employeur , et il leur dit : tu n' es pas heureux ici ? Il y en a beaucoup qui attendent dehors . Et c' est comme ça qu' on construit la souffrance au travail , ce burn out dont on parle tant , les frustrations . Donc ce que nous avons fait … Ce que nous créons ce soir ensemble , mes amis , c' est un véritable droit à la mobilité professionnelle , c' est un droit qui reconnaît à tous les travailleurs du pays la possibilité de choisir , la possibilité d' être couvert contre tous les risques , la possibilité d' être protégé si demain il y a un accident dans son secteur d' activité , s' il y a un risque . Oui ce que je veux que nous créions ensemble , c' est ce droit universel , ce droit à la mobilité professionnelle qui reconnaît pleinement la dignité de la personne au travail . Et ce droit , il ira évidemment avec des devoirs , évidemment . Le contrôle sera permanent , ce qu' il y a en face de ce droit , c' est la possibilité d' être indemnisé , si on est licencié , s' il y a un accident de la vie , si justement tous les cinq ans , face à ces contraintes , on le décide . Mais en face , dans le même temps , c' est un devoir . Le devoir de faire un bilan de compétences , et de regarder toutes les options qui sont alors proposées en face . Et le devoir , lorsqu' une offre d' emploi existe , qu' elle est décente et qu' elle correspond aux compétences , de la prendre . Donc oui , en face de ce droit , c' est un devoir posé aux individus , un devoir de travailler lorsque le travail est proposé . Mais en même temps , soyons honnêtes , que se passe -t-il , bien souvent ? C' est que nombre de nos concitoyens qui se retrouvent au chômage , alors , certains , mais ils sont rares , j' entends les discours caricaturaux , il y en a , certains décident de rester au chômage en disant : c' est mon droit , j' ai cotisé , je reste au chômage , je ne prends pas le travail qui m' est donné . Avec notre système , ce n' est plus un droit , c' est une solidarité , donc si le travail correspond aux compétences et est décent , il doit être accepté ! Mais dans l' autre temps , il y a des millions de nos concitoyens , et il y en aura de plus en plus , qui arrivent , touchés par une restructuration , un changement , et à qui on ne propose rien , et qui restent là . Notre devoir pour répondre à ce défi , ce n' est pas de considérer que leur droit , c' est d' être au chômage . Le devoir de la puissance publique , c' est alors de les former pour pouvoir devenir quelqu'un d' autre et pour pouvoir rebondir . Et aujourd'hui , nous ne le faisons que trop peu . Notre devoir , c' est alors d' accepter que pendant un an , deux ans , trois ans , toutes ces Françaises et ces Français , fragiles , qui n' ont plus de perspective , puissent être reformés pour devenir quelqu'un d' autre . Parce que qu' est -ce qu' il va se passer , mes amis ? Dans les cinq ou dix ans qui viennent , cette révolution que j' évoquais tout à l' heure va balayer des secteurs entiers d' activités . Elle va faire que certains de nos concitoyens qui sont chargés de clientèles par exemple dans le secteur bancaire vont perdre leur activité , parce qu' ils seront remplacés par des logiciels , des algorithmes , des robots . C' est la vérité . Alors , on peut décider comme on l' a tant fait , de ne pas voir , de vouloir ou de prétendre , protéger les emplois en disant : on va vous protéger , et de les laisser se faire balayer . Mais le jour où ils seront licenciés , nous , aujourd'hui , nous ne sommes pas capables de les former , aujourd'hui , nous les laissons simplement au chômage , aujourd'hui , nous proposons peut-être de les former pour se qualifier un peu mieux dans leur secteur d' activité , mais ces Françaises et ces Français-là , il faudra les former demain pendant deux ou trois ans pour changer de secteur , pour aller vers de nouveaux domaines d' activité , pour aller vers la silver economy , vers l' écologie , vers de nouveaux secteurs en développement , c' est cela notre défi . Et donc , pour ce faire , parce que je veux reconnaître la dignité de toutes les personnes et une place à chacun , je ne reconnais pas un droit d' être indemnisé , je nous reconnais un devoir de former . Alors oui , ce que nous allons créer pour chaque Française et Français , c' est ce nouveau service public de la formation et de l' activité . C' est celui qui , en réformant Pôle emploi et notre système de formation continue permettra de donner une place à chacun . Une vraie place . La place que je veux . Mes amis , c' est cela notre défi . C' est cela . C' est permettre au quotidien , grâce à ces transformations , c' est permettre de réformer notre pays , de donner une place à tous , un destin , une perspective , enfin ! Vous l' avez compris , je veux que ce bouclier social permette à chacun de construire sa vie . Que cette transformation de notre formation permette à tous et toutes de retrouver une place , avec ses droits , mais en même temps ses devoirs , parce que , à l' issue de la formation , je veux que chacune et chacun prenne ses responsabilités et l' offre qui est faite ! Enfin , protéger les Français , les protéger face à ces nouveaux risques , à cette mondialisation , c' est vouloir l' Europe , et ce bouclier européen . Ah , je vous rassure , nous n' avons sur ce sujet pas beaucoup de concurrence . Non , je vous rassure , personne ne parle plus d' Europe ! Et quand je regarde tous les candidats à l' élection présidentielle ou aux primaires dont on nous parle tant , il n' y a que des anti-européens , que des eurosceptiques ou des eurosceptiques mollement convertis . Nous , mes amis , nous aimons l' Europe ! Nous voulons l' Europe ! Parce que face à ces nouveaux risques , face aux risques de la mondialisation , seule l' Europe peut nous protéger . Parce que , oui , le monde redevient tragique . Il y a les risques climatiques , il y a les risques économiques , dont nous avons tant parlé , il y a le terrorisme , les migrations . Et il y a à quelques milliers de kilomètres d' ici , ce drame d' Alep qui nous rappelle , si nous l' avions oublié , en ce jour du 10 décembre , en ce jour des droits de l' Homme et du citoyen , il y a Alep où se joue la plus grande atrocité de notre siècle , où nous avons laissé d' autres bombarder et tuer des civils de manière atroce , où nous guettons chaque jour ces messages de la petite Bana et de sa maman Fatemah qui , sous les bombes , racontent leur calvaire . Alors , oui , dans ce monde redevenu tragique , la voix de la France est essentielle . Sa voix indépendante . Celle qui doit porter , non seulement , la lutte contre le terrorisme , mais la lutte partout intransigeante contre celles et ceux qui bafouent les droits de l' Homme , contre celles et ceux qui nous oublient ! Et jamais nous ne devons accepter la fatalité de ceux qui prétendent conduire la Real Politik en renonçant à leurs valeurs . Je crois à la Real Politik , je regarde comme vous notre monde , mais je le veux avec nos valeurs ! Alors , oui , mes amis , dans ce monde , nous avons besoin d' Europe , de plus d' Europe . Face à Google , aux géants de l' Internet , comment protéger nos concitoyens et faire réussir nos entreprises ? Je vous le dis , il n' y aura pas de géants de l' Internet sans marché unique européen du numérique . Ce n' est pas vrai ! Je ne serai pas le président du champion du numérique français . Je veux être le président du champion du numérique européen , c' est la seule voie ! Comment promettre à nos concitoyens de protéger leurs droits au respect de la vie privée ? Par l' Europe , par ce marché unique du numérique , et par aucune autre façon . Comment protéger notre environnement ? Notre écologie , et porter notre projet ? Par l' Europe , là aussi . Et résolument ! Je reviendrai sur notre projet environnemental , mais seule l' Europe peut faire un vrai marché unique , avec un prix du carbone , avec une vraie énergie européenne , avec une vraie capacité d' innovation . Comment protéger nos travailleurs , nos entreprises , face à la concurrence déloyale des Chinois ou des Indiens ? Dans les pneus ou dans l' acier ? Par l' Europe , mes amis ! Et uniquement par une vraie politique commerciale et industrielle , par une vraie politique commerciale que d' aucuns , tous les jours , accusent de tous les mots ! Comment résister aux prises de contrôles hostiles par des investisseurs étrangers dans des secteurs stratégiques ? Par l' Europe , là aussi . En nous dotant d' une procédure de contrôle des investissements étrangers à l' image de ce que nous disposons en France et que j' avais porté avec d' autres , quelques-uns de mes prédécesseurs et que nous avons appliqué ! Pour défendre notre alimentation , notre goût à la française , notre agriculture … Vous avez raison . Notre agriculture , nos pratiques , nos circuits courts , notre cuisine et notre gastronomie qui sont notre fierté . Comment le faire ? Qu' est -ce qui va protéger tout cela face à des géants internationaux qui se constituent , à la fusion de Monsanto et de Bayer qui vont décider de toutes les normes et de toutes les règles ? Mais ce n' est pas nous , tout seul , c' est l' Europe , là aussi ! L' Europe , elle est notre meilleure protection , elle est notre meilleur bouclier dans la mondialisation . L' Europe , elle est aussi notre identité , ce qui nous fait , notre rêve commun . Et dans ce monde de fous , je vous le redis , je repense à la fois à cette phrase de François Mitterrand , prononcée il y a une trentaine d' années , quand il disait que le nationalisme , c' est la guerre . Oui , l' Europe seule peut nous empêcher de tomber dans la barbarie , l' Europe seule peut nous empêcher de tomber dans le repli ! Et donc , la sécurité n' est pas à nos frontières , elle est aux frontières de l' Europe , à Lampedusa , pas à Vintimille . Donc oui , aujourd'hui , nous devons oser à nouveau rêver à l' Europe , parce que c' est notre bouclier , nous devons aussi retrouver un peu du rêve de Schuman et de cette Europe de la défense , nous devons un peu retrouver de cette volonté d' aller de l' avant ! L' Europe , nous en reparlerons , c' est notre chance , c' est notre diversité , notre unité , c' est ce qui fait aussi que nous voulons dans l' Europe , et que nous portons ce projet , chère Sylvie , cher Jean , celui que chaque jeune dans notre pays puisse passer un semestre dans un autre pays européen , pour se former , apprendre une langue , se dépayser , revenir plus fort et plus ouvert . Donc si on veut protéger notre pays , si on veut protéger les Français , oui , nous devons être européens ! Alors , mes amis , vous l' avez compris , ce n' est que le début de notre programme , il va continuer ! Pour ceux , mes amis , qui avaient peur qu' En Marche soit une aventure solitaire , ce soir , vous les avez rassurés ! Pour ceux , mes amis , qui avaient peur que nous n' ayons rien à dire , vous les avez rassurés ! Alors , mes amis , ce projet , ce programme , c' est le vôtre . Je ferai sans doute des erreurs , il faudra me pardonner , nous aurons des désaccords , il faudra les surmonter , mais ce projet , ensemble , nous allons le porter , parce que nous croyons dans l' espérance . Mais maintenant , votre responsabilité , avec moi , je vous le dis ce soir , je vous le dis avec beaucoup de gravité , et en même temps beaucoup de joie , beaucoup de fierté , vous m' avez énormément donné depuis avril dernier , énormément . Mais maintenant , notre engagement , notre programme , nous allons le continuer , c' est un engagement efficace et juste pour le pays , c' est l' espérance à laquelle nous croyons , mais je ne veux pas le porter tout seul . Moi , je le porterai dans la durée , je le porterai jusqu' au bout , mais maintenant , votre responsabilité , c' est d' aller partout en France , pour le porter , et pour gagner ! Ce que je veux , c' est que vous , partout , vous alliez le faire gagner , parce que c' est notre projet ! Vive la République , vive la France ! Merci à vous ! Merci à vous ! ",
    "Le terrorisme fanatique frappe toutes les nations , toutes les populations , toutes les religions . Il a frappé à Istanbul le 1er janvier , dans une nuit de joie devenue une nuit d' horreur . Il avait frappé à Berlin le 19 décembre . Ces drames n' ont pas lieu n' importe où . A Istanbul , il a frappé un lieu de fête où la jeunesse se réunit . A Berlin , il a frappé des symboles : un marché de Noël , un lieu de rencontres , un lieu populaire ; devant l' église du Souvenir qui porte la croix de la cathédrale de Coventry , toutes deux détruites par la guerre , emblèmes des drames qui ont déchiré notre continent . A Berlin , c' est l' Europe qui , une fois de plus , a été mise à l' épreuve . Jamais la France et l' Allemagne , depuis la réconciliation mise en œuvre par des Européens engagés et des patriotes lucides , n' ont été aussi proches face aux menaces communes . Jamais nos deux pays n' ont eu autant besoin de se témoigner leur solidarité . Avant même de proposer des projets conjoints , nous devons affirmer notre communauté de destin . Le 11 janvier 2015 , dans le plus grand élan de solidarité que notre pays a connu depuis la Libération , la chancelière , Angela Merkel , est venue à Paris exprimer avec émotion que l' Allemagne vivait ce drame avec nous . Aujourd'hui , nous sommes tous berlinois . Le terrorisme ne connaît pas les frontières . Il n' a pas la France pour ennemie ou l' Allemagne pour adversaire . Le terrorisme veut détruire l' Europe . Il veut en finir avec notre civilisation commune . Il veut défaire nos valeurs collectives . Il porte la mort quand l' union des Européens entend défendre la paix et permettre la vie commune . Voilà pourquoi nous nous sentons européens , comme des millions d' Allemands se sont sentis européens en janvier 2015 et lors des tragiques attentats qui ont suivi . L' unité sereine avec laquelle la société allemande a réagi à cette attaque , comme aux récents événements de Munich , est admirable . Les polémiques viendront . Elles commencent à poindre , pas seulement outre-Rhin . Il n' a fallu que quelques heures pour instruire le procès : en ouvrant ses frontières aux migrants , la chancelière aurait exposé l' Europe aux pires dangers , et , aujourd'hui , sa propre capitale . Rien n' est plus faux que cette abjecte simplification . Nos sociétés fragilisées , épuisées par la crise , éprouvées par le fracas du monde – guerres , migrations , ouverture rapide aux échanges , changement climatique ou bouleversement numérique – sont tentées de chercher le mal à l' extérieur . Il suffirait alors de l' expulser de nos communautés nationales , de se barricader , chacun de son côté . Les attentats de Paris , de Nice ou de Berlin seraient‑ils liés au laxisme migratoire ? Ont -ils été le fait de seuls étrangers , de migrants récemment installés ? S' il est une erreur que les Européens ont faite , c' est de ne pas avoir cherché d' emblée une réponse commune face au défi migratoire : quand l' Italie affrontait seule l' arrivée des réfugiés à Lampedusa , au point que le pape François s' en est vivement ému , ni la France ni l' Allemagne n' ont été au rendez-vous . De même , les Grecs ont longtemps été en première ligne , démunis et dépassés , face à l' afflux de réfugiés et de migrants . Pour autant , la chancelière Merkel et la société allemande dans son ensemble ont été à la hauteur de nos valeurs communes ; elles ont sauvé notre dignité collective en accueillant des réfugiés en détresse , en les logeant , en les formant . En refusant de reconstruire des murs dans une Europe qui en a trop souffert , en évitant les amalgames face aux événements les plus cruels , comme les sinistres violences contre les femmes perpétrées l' an dernier à Cologne . Ayons le courage de le dire : la solution est dans la protection , non dans la fermeture , dans une coopération européenne plus forte et non dans un repli national inefficace . La défense de notre sécurité , la protection de nos frontières , la gestion des flux migratoires , l' intégration des réfugiés sont des défis complexes , durables , que nous devons surmonter ensemble . L' Union européenne doit franchir , sans attendre , un nouveau pas : à brève échéance , nous devons créer une Europe de la souveraineté , capable de nous protéger des menaces extérieures pour mieux assurer la sécurité intérieure . Les accords de Schengen , si bénéfiques à nos échanges , comme les 350 000 travailleurs frontaliers français le vivent chaque jour , doivent être préservés et leur suspension limitée aux situations d' urgence . Dans le même temps , nous devons renforcer le corps de garde-frontières européen , qui est une avancée majeure , en lui donnant compétence pour aider à la protection durable des frontières des Etats membres qui en ont besoin , sans limiter son action aux situations d' urgence . Nous devons , au niveau européen , développer les accords de coopération avec les grands pays d' émigration et de transit , fondés sur une aide au développement , un soutien au contrôle de leurs frontières et un accord sur le retour des migrants non autorisés à entrer ou résider dans l' Union européenne . Nous devons aussi , en surmontant les réticences nationales , créer un système de renseignement commun , qui permette une traque efficace des criminels et des terroristes , voire à terme une police commune pour le crime organisé et le terrorisme . Quand l' Europe est divisée , elle est inefficace . Quand la France et l' Allemagne s' éloignent l' une de l' autre et cessent de servir l' Union tout entière , elles font fausse route . Le 19 décembre , ne pensons pas que l' Allemagne seule était visée : Berlin a été attaqué , la France a été touchée . Cette solidarité nous oblige . Elle nous oblige à faire bloc , à agir ensemble , pour préserver ce que nous sommes . Elle nous invite à être pleinement européens . **** *discours_Macron6  *candidat_Macron  *url_inconnu  *source_Nevers  *date_2017-01-06  *type_Meeting Merci à vous . Bonsoir Mesdames , bonsoir Messieurs , et bonne année . Bonne année à toutes et tous . Un très grand merci pour votre accueil . Un grand merci à tous les deux pour vos mots d' introduction et , cher Denis , pour ton accueil chaleureux dans cette belle ville . Nous avons passé l' après-midi ensemble et je suis heureux d' être parmi vous ce soir , aussi nombreux à Nevers . Merci à madame la Sénatrice , Anne EMERY-DUMAS qui , depuis les premiers jours , marche à nos côtés . Merci à mon ami François PATRIAT , qui n' a pas compté sa peine lui aussi , depuis les premiers jours , quels que soient les aléas de la vie , pour rassembler , convaincre , entraîner , et qui fut un grand Président de cette belle région . Je veux aussi saluer l' ancien Maire de Bourges , Serge LEPELTIER . Merci beaucoup d' être là aujourd'hui avec nous , traduisant en actes ce que Denis THURIOT vient de présenter : le rassemblement des bonnes volontés , de toutes celles et ceux qui , quels que soient nos histoires , nos imaginaires collectifs , nos propres convictions , nos familles de pensée , ont fait qu' autour d' un projet et de valeurs , nous nous retrouvons . Et merci à toutes et tous d' être là avec vos parcours , vos identités , votre diversité , autour de ce même projet que nous allons ensemble porter . Je vous souhaite une très belle année . Et nous savons ce que nous devons , en ces premiers jours , souhaiter à notre pays - et ce sera le fil directeur de cette campagne et de ces prochains mois : réussir . Réussir à entrer pleinement dans ce siècle nouveau . Réussir à gagner dans le XXIème siècle , à libérer les forces vives , les énergies qui sont partout présentes sur le territoire . Et nous savons quoi souhaiter aux Français : leur permettre , à tous , de trouver leur place dans cette société . De réussir avec le pays en ayant chacune et chacun son rôle . Alors , j' ai bien conscience qu' il n' est pas anodin d' organiser un premier rassemblement sur cette belle terre de Bourgogne . Cette terre est chargée d' histoire , chargée de politique , au sens noble du terme . C' est bien sûr une terre mitterrandienne , qui pendant des décennies , a été la terre de François Mitterrand . C' est aussi , et surtout ici à Nevers , la terre de Pierre BEREGOVOY , un homme de proximité , qui aimait y revenir chaque semaine . Un homme respecté par tous pour son action , pour ce qu' il incarnait , pour la transformation apportée à la ville . Je pense à la rénovation du Palais Ducal , je pense aussi au lancement de l' hôpital , sa dernière décision , cher Denis , où nous étions tout à l' heure , qui est ton combat d' aujourd'hui , pour partie . C' est une histoire qui résonne encore dans toute la Nièvre et au-delà . Et je suis content de voir ici quelques visages de cette histoire . Je pense en particulier à Marcel CHARMANT ou René-Pierre SIGNE , ancien sénateur-maire de Château-Chinon qui nous font le plaisir d' être parmi nous ce soir . Monsieur BARDIN aussi , ancien Président du Conseil Général . Et je pense aussi à Didier BOULAUD , qui nous a accueillis tout à l' heure , et qui a inscrit son engagement dans la droite ligne de Pierre BEREGOVOY . Parce qu' on ne construit rien si on ne respecte pas l' histoire , les histoires , dans lesquelles chacune et chacun s' inscrit , dans lesquelles les territoires aussi se sont construits . Non pas pour les répéter , non pas pour s' y enfermer , pour les respecter dans la fidélité vraie que nous leur devons . Celle qui consiste à inventer un avenir commun , à rassembler des énergies , en n' oubliant jamais ce qui nous a fait et les terres où nous sommes . Pour moi ce n' est pas innocent , à ce titre , d' être ici . Et , tu l' as rappelé il y a quelques instants , parce que c' est aussi une ville qui porte une partie de mon histoire familiale . C' est la ville où en effet ma grand-mère , pour la première fois , quitta ses Hautes-Pyrénées natales , pour venir passer plusieurs années , emmenant sa mère avec elle , des années de guerre . Vous savez , on grandit toujours avec ses propres souvenirs d' enfance , mais on grandit parfois avec les souvenirs des autres . On grandit avec des images de lieux qu' on n' a jamais vus , avec les histoires qu' on vous a racontées . Je peux vous dire que j' ai grandi avec Nevers , parce que j' ai grandi avec ces scènes que me racontait ma grand-mère où , jeune enseignante , elle arrivait au rectorat ; ces scènes de guerre , parce que cette ville en a souffert ; ces scènes de liberté extraordinaire , de cette génération qui malgré tout a résisté , et qui a montré , parfois plus que d' autres , l' énergie qui existe dans le peuple français . Donc je peux le dire je connais Nevers , parce que cette ville a peuplé mon enfance , de rues que je n' avais jamais vues , de femmes et d' hommes que je n' avais jamais croisés mais qui ont fait aussi mon histoire familiale . Alors aujourd ' hui , il se passe quelque chose dans le pays . Vous en êtes le témoignage vivant ce soir . Il se passe quelque chose lorsque ce qui n' était il y a quelques mois qu' une volonté folle réunit désormais des milliers de personnes . Il se passe quelque chose lorsque des jeunes , que l' on dit si peu intéressés par la chose publique , s' inscrivent si massivement sur les listes électorales . Parce qu' ils refusent que cette élection qui arrive se fasse sans eux . Il se passe quelque chose lorsque des femmes et des hommes , qui s' étaient parfois combattus ou qui ne croyaient plus dans l' action publique se retrouvent dans une même salle , pour penser , partager , vouloir agir ensemble et transformer le pays . Il se passe quelque chose lorsque tant de nos concitoyens , déçus par la politique , fatigués par tant d' années de divisions et de diversions , expriment de nouveau un intérêt , une attente , une espérance . Parce qu' ils sentent la sincérité de notre mouvement . Oui , il se passe quelque chose lorsque des femmes et des hommes , venus d' horizons différents , s' engagent non pas pour s' encarter dans un parti , non pas pour défendre leurs intérêts ou les intérêts de celui -ci mais pour ensemble , porter l' intérêt général , pour ensemble construire un projet nouveau , celui du XXIème siècle , celui que notre pays , que nos territoires méritent . Oui , il se passe quelque chose , aujourd'hui , ici . Et ce qu' il se passe , c' est cette révolution démocratique qui permettra de transformer la France . C' est cette révolution qu' ensemble , nous allons conduire . Non pas sur des clivages anciens , mais bien par une refondation profonde , une refondation idéologique , sincère , progressiste , en dépassant ce qui nous avait bloqué pour construire ensemble . Cette révolution démocratique , elle se construira en libérant toutes les énergies , en libérant les énergies qui sont dans notre société , chez chacune et chacun d ' entre nous . Et d'abord , c' est ma conviction première , je l' ai répété il y a encore plusieurs semaines , en refondant très profondément une chose : le travail . Le travail c' est le cœur de notre société , le cœur de notre économie , le cœur de nos projets à chacune et chacun d' entre nous , parce que le travail c' est ce qui permet à chacun de trouver sa place dans la vie , c' est ce qui permet de trouver un rôle de s' émanciper , de sortir de sa condition , de construire son destin . Donc oui , je l' ai dit et je le redis ce soir , je veux être le candidat du travail parce que c' est par le travail , pour le travail que nous redresserons le pays . Parce que c' est par le travail que celui qui n' a hérité de rien peut construire sa vie , parce que c' est par le travail que celui qui veut choisir son destin le trouvera , parce que c' est lorsqu' on est loin du travail que la pire injustice est subie , parce que c' est aujourd'hui dans notre société , lorsqu' on ne trouve pas un emploi , lorsqu' on est malheureux au travail , que les pires des injustices sont vécues . Alors pour être le candidat du travail , pour gagner cette bataille contre le chômage , bataille que nous avons perdue dans notre pays depuis trente-cinq ans , il faut prendre des mesures fortes . Nous sommes le seul pays d' Europe , le seul grand pays d' Europe , qui n' a pas vaincu le chômage de masse . Pour mener ce combat , il faudra relancer l' investissement , public et privé , il faudra plus d' Europe en relançant une dynamique forte . Mais cette dynamique n' existera que si nous savons nous- mêmes conduire nos propres réformes , retrouver notre crédibilité , convaincre nos partenaires que nous sommes dans une aventure commune où ils peuvent nous faire confiance . Je veux d'abord que la travail paye plus . Et il paiera plus . Pour cela , tous les travailleurs , qu' ils soient salariés , indépendants ou fonctionnaires , verront leur pouvoir d' achat augmenter : par exemple , une aide-soignante qui gagne 1800 euros bruts par mois , ce sera environ 300 euros de plus par an . Toutes et tous , quel que soit le niveau de salaire , parce que j' ai voulu cette baisse de cotisation sociale qui ne sera compensée que pour partie par une hausse de la CSG . Tous les travailleurs quels que soient leur statut , leur condition , leur situation , gagneront du pouvoir d' achat , parce que je veux que le travail soit récompensé , et plus on travaillera , mieux on gagnera sa vie . C' est la clé de cet esprit de responsabilité . Pour être le candidat du travail , je veux aussi que l' accès au travail soit plus facile et donc que les employeurs soient encouragés à embaucher . Et donc pour cela , il faut que le coût du travail diminue , il faut que le coût réel payé par l' employeur diminue . Et donc à ce titre , je supprimerai le Crédit Impôt Compétitivité Emploi pour le remplacer par un allègement de charges dans la durée pérenne , pour toutes et tous , c'est-à-dire non seulement pour les entreprises qui payent l' impôt sur les sociétés , mais pour les indépendants , pour les entreprises de l' économie sociale et solidaire . Ce sera 6 points de charges patronales en moins pour réduire précisément ce coût du travail , pour encourager l' embauche , pour développer partout en France de manière durable , l' emploi . Le travail sera aussi encouragé parce que nous aurons une économie plus agile , et c' est indispensable . Nous avons besoin d' avoir des entrepreneurs qui puissent embaucher plus facilement , parce qu' ils pourront s' adapter aux cycles économiques , aux contrecoups , à la situation de leur entreprise , de manière plus aisée . Qu' est -ce que cela veut dire ? Une réforme en profondeur de notre marché du travail , non pas par les réformes , de -ci et de-là qui diviseront la population , qui conduiront à réduire les droits des uns , de manière homogène partout dans le territoire , mais par une transformation profonde qui va permettre , partout , de réduire la place de la loi , pour faire davantage confiance aux contrats , c'est-à-dire à l' accord de branche et l' accord d' entreprise . Ce que je veux , pour notre marché du travail , c' est que la loi ne définisse plus qu' un ordre public social , mais que partout par des accords de branche et par des accords d' entreprises majoritaires , on puisse déroger à la loi pour trouver les bons équilibres , les bonnes souplesses sur le terrain . Non pas en réduisant les droits de tous les salariés , comme certains le caricaturent , mais en permettant de s' adapter à la taille de l' entreprise , aux aléas du carnet de commandes , aux contraintes du secteur , à tout ce qui fait qu' une entreprise du BTP de trois salariés à Nevers n' a rien à voir avec la situation d' un grand groupe international qui opère depuis Paris , sur tous les continents . C' est une évidence mais nous allons le faire , nous allons le faire jusqu' au bout . Et c' est ainsi que nous pourrons massivement , non seulement développer l' embauche mais permettre d' éviter nombre de licenciements . Cette réforme je la conduirai dès le début du quinquennat , c' est la réforme principale du marché du travail , ce sera l' un de mes grands objectifs . Faciliter l' embauche , c' est aussi donner de la visibilité à tous , aux employeurs comme aux salariés . Et pour ce faire , j' irai jusqu' au bout de la réforme des prud'hommes qu' en tant que ministre , j' avais conduite . En définissant des plafonds et des planchers pour les dommages et intérêts , non pas en réduisant les droits là encore , mais pour donner à toutes et tous , de la visibilité sur le terrain , pour éviter d' avoir des entrepreneurs qui puissent encore vous dire demain : “ Moi je n' embauche personne , parce que je sais très bien que je vais finir aux prud'hommes si ça se passe mal , ça va durer des années et je ne sais pas pour combien je vais en avoir ” . On le sait bien , les victimes aujourd'hui de notre système , ce sont les salariés les plus modestes , qui vont passer des années dans des procédures trop longues , qui vont parfois espérer gagner des sommes inconsidérées parce qu' on leur aura menti et ce sont les plus petits entrepreneurs qui du coup n' embaucheront plus . Donc je conduirai jusqu' au bout cette réforme des prud'hommes , parce qu' elle est importante pour l' emploi et parce que c' est une réforme juste . Enfin , je veux faciliter le travail pour toutes celles et ceux qui entreprennent , qui sont les indépendants , les professions libérales qui sont à leur compte , qui créent de l' activité pour eux-mêmes et aussi parfois pour d' autres , et qui aujourd'hui souffrent des complexités du régime social des indépendants . Je rapprocherai ainsi l' organisation du régime des indépendants de celle du régime général , avec un guichet indépendant , mais en supprimant le RSI . Cela permettra de réduire les délais insupportables , tout en préservant les exonérations et les réductions de cotisations de 6 millions d' indépendants , qui travaillent en France . Le travail , c' est la clef de notre refondation . Mais pour faire réussir la France , permettre à chacun de trouver sa place , il faut aussi libérer nos territoires , les aider à réussir , partout sur le pays . Je sais que vous avez ici , dans la Nièvre , tous les atouts pour réussir . Je sais que vous avez aussi beaucoup souffert , ces dernières décennies , d' entreprises perdues , de fermetures de services publics , de casernes . Vous avez perdu de la population . Mais vous avez des entreprises qui innovent , une population qui veut aller de l' avant , des forêts et des terrains agricoles . Une énergie . Et donc , un formidable sentiment de gâchis . Ce gâchis , c' est ce sentiment que le découpage territorial n' est pas à votre avantage , et que vous pouvez parfois vous sentir loin des métropoles . Le gâchis , c' est le sentiment que vous pouvez avoir , en considérant que la puissance publique investit moins ici que dans d' autres territoires . C' est le sentiment que peuvent avoir beaucoup de jeunes lorsqu' ils sont contraints d' aller faire leurs études ailleurs . Je veux mettre fin à ce sentiment de gâchis . Alors pour cela , ce ne sont pas des promesses inconsidérées qu' il faut faire . C' est un engagement fort . D' agir avec vous , avec le territoire , de faire davantage confiance à la population , aux élus locaux , aux services partout sur le territoire , et donc d' engager une phase nouvelle de décentralisation et de déconcentration . C' est cela à quoi je m' engage pour libérer l' énergie des territoires . Alors oui , nous devrons investir pour ce faire . Nous devrons investir davantage dans les services publics , pour assurer leur continuité . Investir pour qu' il y ait justement , partout , une couverture numérique digne de ce nom . Et ce sera l' une de mes principales priorités en matière d' investissement public . Parce qu' on peut développer , et nous allons développer , le Nivernais et Nevers . Vous êtes en train de le faire et de le réussir . On le fera avec le numérique , par le numérique . Mais pour cela , il est indispensable que la fibre soit là , que la couverture 3G et 4G soit sur le territoire . Et donc nous accélèrerons les investissements publics et l' organisation collective pour y parvenir : c' est indispensable . C' est l' infrastructure du XXIème siècle qui nous permettra d' y arriver . Enfin , pour que chacune et chacun trouve sa place , comme je le disais , pour réussir , il faut protéger , aussi . On doit libérer les énergies , les territoires , mais nous devons protéger les plus faibles . Nous devons assurer les sécurités contemporaines qui vont permettre à chacune et chacun de prendre les risques de la vie . D' être assuré qu' il ne sera pas abandonné . De protéger face aux aléas . De protéger parce que tout le monde ne peut pas réussir . C' est le cœur d' une solidarité nécessaire qui se conjugue avec la liberté . C' est le cœur d'ailleurs du projet français . Parce qu' il n' y a pas d' émancipation possible si cette émancipation n' est pas juste , si elle n' est pas faite pour tous et toutes . Alors dans ce moment de l' année qu' est celui des vœux , beaucoup ont passé du temps à parler aux Françaises et Français pour leur dire tous leurs projets , tout ce qu' ils allaient faire ou qu' ils n' avaient pas fait . Vous me direz , c' est le temps des résolutions ; je préfère celui des engagements . Mais , dans ces temps de début d' année , on se souhaite , les uns les autres , avant toute chose , la santé . Alors je veux aussi , dans les sécurités , indispensables , vous parler de santé . Parce que oui , il y a évidemment les sécurités individuelles qui vont face au chômage , pour la formation professionnelle , tout au long de la vie . J' en ai déjà parlé et j' en reparlerai . Mais je veux vous parler ce soir de santé . De santé , car ce sujet nous concerne toutes et tous sans exception . Parce que , lorsque nous avons conduit il y a quelques mois - et beaucoup d' entre vous nous ont aidé à le faire ici - cette Grande Marche , les Françaises et les Français ont exprimé une préoccupation première , parmi quelques autres : c' est la santé . C' est ce souhait d' être bien soigné . C' est cette volonté , pour eux-mêmes , pour leurs proches , d' être protégés contre ces aléas de la vie , et d' être en bonne santé . Parce qu' ils craignent pour leurs soins , pour leur prise en charge , ils craignent aussi de voir notre système solidaire s' étioler . Les Français ont un attachement singulier à leur modèle de santé . Un attachement , parce que c' est le fruit de notre histoire . Depuis plus d' un siècle , depuis plusieurs siècles , nous sommes une terre de santé , nous sommes un pays d' excellence médicale . Notre modèle , il est envié partout dans le monde . En première ligne , c' est celui qui a permis d' inventer en France le vaccin contre la rage , de procéder à des inventions aussi formidables que le cœur artificiel , la découverte du VIH . Nous sommes un pays de première mondiale , qui ne cesse de se perpétuer . Depuis 1945 , nous sommes parvenus à construire l' organisation de santé parmi les plus efficaces au monde . La réforme de 1958 , les lois DEBRE ont permis de créer l' hôpital , qui est non seulement une fierté , mais qui a été partout envié . Parce que c' est l' excellence de la formation , du soin , de la clinique et de la recherche . La santé , c' est aussi l' histoire intime des Français . Qui a partagé votre vie , à part vos proches ? Qui est le témoin de vos angoisses ? Qui vous suit ? Le médecin . Le médecin de famille , qui a jalonné nos campagnes et qui se fait de plus en plus rare - j' y reviendrai . Celui à qui , dans le secret , dans cette relation unique , vous confiez les choses . L' infirmière , l' aide-soignante , qui va faire votre quotidien lorsque vous devenez dépendant ou lorsque vous êtes affaibli . Lorsqu' on parle de santé , on parle avant tout d' humanité . On ne parle pas de chiffres , on ne parle pas d' un coût d'abord , on parle de ce qui est le cœur de notre humanité , et on doit pour cela d'abord et avant tout , en parler avec l' humilité qui convient . Lorsqu' on la réduit à un coût , à une politique publique , on se trompe . La santé c' est dans l' intimité , dans son épaisseur , ce qui fait notre vie . Alors , cette santé qui est au cœur de l' histoire française , de notre quotidien , de notre attachement , elle est face à de nombreux défis . D'abord , le défi du système de santé même que nous connaissons . Cette crise , que nous vivons depuis des décennies . Les difficultés à l' hôpital , mais qui ne sont pas les mêmes à l' hôpital de Nevers et dans un hôpital parisien . Ce défi de notre organisation collective . Le deuxième défi , c' est celui du vieillissement . C' est une chance . Une formidable opportunité . Mais c' est un défi d' organisation collective , il faut le regarder en face . En 2050 , un Français sur trois sera âgé de plus de 60 ans . En 2005 , c' était un Français sur cinq . Alors lorsqu' on est plus âgé , on est évidemment plus souvent malade , parfois moins autonome . Et mécaniquement , les patients , les personnes fragiles sont de plus en plus nombreux , et les dépenses , mécaniquement , augmentent . Et donc on doit inventer la manière de soigner , d' accompagner ce vieillissement différemment . C' est une chance , mais en même temps , alors qu' il est normal que ces dépenses continuent à augmenter , on ne peut pas avoir des dépenses de santé qui augmentent trois fois plus vite que la création des richesses dans notre pays . C' est pourquoi l' effort poursuivi ces dernières années devra être conduit . Et il ne s' agit pas de considérer qu' on va faire face à ce défi en relâchant l' effort . Mais il faut s' organiser différemment . Organiser les soins , notre système collectif , de manière plus adaptée à ce défi . Le troisième défi , c' est le développement des maladies chroniques . Là aussi , c' est une chance formidable . Il y a quelques années ou quelques décennies , on mourait systématiquement de plusieurs formes de cancer , on mourait du SIDA . On mourait de nombreuses pathologies , qui sont devenues ce qu' on appelle aujourd'hui des maladies chroniques . Ça veut dire d'abord - et c' est une formidable nouvelle qu' il faut saluer : on n' en meurt plus . Mais on vit donc longtemps avec . Avec des traitements coûteux , avec des passages à l' hôpital , avec des soins . Et donc oui , là aussi , c' est une autre forme d' organisation , et un coût , qu' il faut penser et accompagner . Le quatrième défi c' est que nous devons passer de l' excellence en soin à l' excellence en santé . La France est un pays formidable lorsqu' on est malade , on est mieux couvert , bien souvent et - je vais vous le dire ici - ça continuera à être le cas et nous nous battrons pour cela . Mais en même temps , nous prévenons mal les pathologies . Qu' est -ce que ça veut dire ? Ça veut dire qu' on prévient moins bien les maladies qui viennent du tabac ou de l' alcool ou des accidents de la route . 73 000 de nos concitoyens meurent chaque année à cause de la cigarette , 50 000 à cause de l' alcool , on se suicide beaucoup plus que dans nombre de pays voisins . La pollution tue , la pollution de l' air , beaucoup plus que dans nombre de pays voisins . J' étais il y a quelques semaines dans la vallée de l' Arve , elle sort d' une alerte pollution de 33 jours . Imaginez -vous . Tout cela fait qu' en France , notre système de prévention est moins bon . Tout ça parait pour beaucoup des idées sympathiques , nous disent “ oui c' est bien gentil mais enfin ça n' est pas le cœur du problème , le cœur du problème c' est le soin ” . Mais tout ce dont je viens de vous parler , vous allez le retrouver dans des pathologies chroniques parce qu' on aura moins bien prévenu , vous allez le retrouver dans des maladies respiratoires , dans des maladies chroniques , dans des dépendances . Et donc oui , mieux prévenir c' est aussi comme cela qu' on soignera mieux , qu' on réduira les coûts , qu' on réduira le coût pour notre société . Et donc il est indispensable d' avoir une meilleure politique en la matière . Et puis le dernier défi c' est celui des inégalités . En matière d' inégalités , on donne souvent des chiffres . On dit que l' espérance de vie d' un ouvrier est de 7 ans inférieure à celle d' un cadre : c' est vrai . On dit que les enfants d' agriculteurs ont 50 % de caries en plus que les enfants de cadres : c' est vrai . On dit que dans un tiers des départements français , on ne trouve plus de médecins après 20h : c' est vrai . Mais le plus terrible c' est que ces chiffres finissent par ne plus choquer personne et donc quand on parle de santé , on parle d' inégalités profondes , sociales , territoriales qui sont aussi aujourd'hui un défi extraordinaire que nous devons relever . Et donc face à tout cela , vous voyez bien que la réponse , elle ne peut pas être de ne rien faire . On ne protégera pas notre santé en disant “ on va faire comme si de rien n' était , on ne va toucher à rien , on va protéger les uns et les autres , les structures et on fera comme avant ” . Non , ça n' est pas possible ! La réponse , elle ne peut pas être non plus “ on va dérembourser , on va couper des têtes , on va supprimer les fonctionnaires , dérembourser des soins ” , non ! Parce que ce sont des mauvaises réponses partielles . La réponse exigeante , c' est de continuer à faire des économies indispensables en relevant ces défis . C'est-à-dire de conduire une vraie révolution culturelle de la santé ; c' est cela que nous devons faire . Non pas faire un énième coup de rabot , non pas considérer qu' il faut dérembourser ici , couper par là , déshumaniser tout , mais bien plutôt , conduire une révolution culturelle par laquelle nous nous appuierons davantage , sur les personnels de santé et les patients plutôt que sur les systèmes de soin et les établissements . Une révolution culturelle pour favoriser les initiatives de terrain , la liberté des acteurs et des choix plutôt que les décisions uniformes dictées d' en haut pour aborder la santé comme une source de richesse et d' opportunités plutôt que simplement comme un poste de coût . Pour avancer en donnant du sens , des perspectives de long-terme , plutôt que des mesures de court-terme dans lesquelles on finit par prendre des mauvaises décisions que nous payons aujourd'hui . Et donc , ce que nous devons conduire , c' est une révolution culturelle profonde autour de plusieurs grands chantiers , quatre grands chantiers sur lesquels je veux devant vous ce soir m' engager . I. Le premier chantier , c' est celui de la prévention . La révolution de la prévention c' est celle que nous devons conduire , en particulier pour passer d' un bon système de soin à un bon système de santé . Soigner les Français c' est par construction , non pas attendre simplement qu' ils soient malades , mais leur permettre de vivre le maximum de temps en bonne santé . C' est cela le cœur de la prévention , et donc c' est les inciter à vivre plus sainement , c'est-à-dire à mieux se nourrir , à bouger tous les jours à pied ou à vélo . A promouvoir , parmi les jeunes , celui qui ne fume pas , qui conduit le soir sans avoir bu . C' est renforcer l' éducation à la santé et le dépistage précoce . C' est renforcer la couverture vaccinale , qui est aujourd'hui trop faible pour être pleinement efficace . Ce sont là les véritables économies dans la durée , elles peuvent arriver très vite et cela doit être le cœur de notre ambition . Faire de la prévention , à long terme , c' est le meilleur moyen de faire des économies et de rendre notre système plus performant : parce qu' en investissant un peu dans le sport , par exemple , on évitera des hospitalisations coûteuses pour de nombreuses maladies . Alors pour cela , comment faire concrètement ? D'abord , il faut que les Français soient en contact régulier avec des professionnels . Pas une fois ponctuellement lorsqu' ils se rendent chez le médecin quand ils sont malades … non , le plus souvent possible . Et pour là aussi éviter des inégalités . Parce que vous savez , quand on est un enfant , venant d' un milieu modeste , dans bien des cas on ne va pas chez le médecin , on ne va pas chez le dentiste qui préviendra certains risques , on ne va pas chez le généraliste aussi souvent que d' autres . Et donc oui , pour que le plus souvent possible , les Françaises et les Français tout au long de la vie puissent avoir à l' école , sur le lieu de travail , accès , et encore plus que d' autres dans les zones défavorisées ou dans certains déserts médicaux , accès à la santé et aux soignants , nous devons conduire une réforme profonde . Ce que je veux faire pour cela c' est créer un service sanitaire de trois mois pour les étudiants en santé . Qu' est -ce que cela veut dire très concrètement ? Ça veut dire que tous les étudiants en santé , en médecine , en pharmacie , en dentaire , les étudiants infirmiers et infirmières : tous ces étudiants auront au moins 3 mois d' actions de prévention . Ils seront formés pour cela , ils ne feront pas d' actes de soin . Mais ils iront dans les écoles sensibiliser sur le tabac , ils iront dans les écoles , dans certains quartiers défavorisés , dans des EHPAD , chez des dépendants donc , dans des maisons de retraite , prévenir justement la mauvaise audition , certains risques liés à des maladies dentaires , certains comportements qui aujourd'hui conduisent à des maladies ou à des défaillances . C' est bon pour les professionnels de santé , c' est bon pour notre profession de santé justement , et c' est bon pour le pays . C' est un décloisonnement , c' est une révolution culturelle : nous devons la conduire . Ce service sanitaire sera intégré dans les maquettes pédagogiques . Et à terme , il pourra concerner ainsi plus de 40 000 étudiants chaque année . C' est une force de frappe formidable . Ça ne coûte pas un euro de plus . Cela responsabilisera tous les étudiants en santé , cela les ouvrira hors du champ médical . Et cela permettra à l' ensemble des Françaises et Français , les plus jeunes , les plus âgés , les plus défavorisés , d' être mieux prévenus contre certains risques . D' être ainsi mieux protégés . C' est cela , aussi , une vraie réforme de la santé . Pour développer la prévention , il faut aussi fixer des objectifs nationaux et enclencher une mobilisation générale . Je veux fixer ainsi comme objectif que la génération française qui naît aujourd ' hui sera la première génération sans tabac . C' est un objectif que nous sommes capables de relever . Là aussi , il ne s' agit pas de tomber dans des vieilles batailles . De dire , il y a d' un côté certains professionnels , qui , par exemple , sont les buralistes - et ils ont le droit de vivre , et il faudra les accompagner dans cette transition , les protéger , les respecter , en aucun cas les stigmatiser . Et il y aurait de l' autre côté les belles âmes . Non , c' est un défi collectif que nous allons conduire en responsabilité . En formant , non pas en punissant , mais en incitant . Qu' a fait l' Australie ? Ils sont en train de gagner cette bataille . Ils avaient comme nous , entre 20 et 30 % de leurs jeunes qui fumaient ; ils en ont aujourd'hui 3 % . Comment ont -il fait ? Ils n' ont pas augmenté le prix du paquet de cigarettes avant tout , ils n' ont pas eu des débats et des guerres comme nous en avons . Ils ont créé une vraie politique d' incitation pour faire que les lycées qui avaient le moins de fumeurs , qui valorisaient le mieux le sans-tabac , et bien étaient récompensés , mieux dotés , valorisés . Et donc c' est là aussi par une révolution des mentalités , par une mise en valeur des meilleures pratiques , par une récompense offerte à toutes celles et ceux qui , justement , réussissaient sur le terrain au contact du quotidien , cette transformation , qu' ils ont réussi ce combat . Non pas en stigmatisant ou en pénalisant , mais en valorisant les bons comportements , en donnant de la fierté à ceux qui conduisaient cette transformation , et en mettant là aussi de la santé hors des établissements de soin , à l' école et au travail . C' est cela que nous valoriserons . Et pour développer également la prévention , il faut la rémunérer à sa juste valeur . Et pour cela , je veux que nous changions le mécanisme de rémunération des professionnels de santé . D'abord je veux pour eux de la considération , je veux pour eux de la revalorisation de leur rémunération . Ce sera l' un des axes de notre réforme de la santé . Mais non pas simplement par une meilleure cotation des actes . Je veux que les professionnels de santé partout en France soient mieux considérés , mieux valorisés et mieux payés . Mais pour cela je veux qu' une partie de leur rémunération prenne en compte des objectifs de santé publique . Je veux que partout sur le territoire , ces professionnels de santé soient aussi mieux payés , non pas parce qu' ils feront le maximum de consultations ou le maximum d' actes , mais parce qu' ils feront le maximum de prévention . Et donc nous transformerons une partie de leur rémunération pour qu' il puisse y avoir une contractualisation sur la base d' objectifs de santé publique . Pour qu' ils soient ainsi incités à mieux accompagner certaines personnes handicapées , certaines personnes âgées , certaines personnes défavorisées . Non pas pour qu' ils les soignent davantage , mais pour qu' ils les aident à être moins malades , pour qu' ils les accompagnent en terme de soins complets et de santé . Cette révolution est fondamentale , et nous la conduirons . Parce qu' au fond , vous l' avez compris , je crois que pour développer la prévention , nous avons besoin de faire confiance avant tout aux professionnels . Je leur donnerai ainsi la liberté de s' organiser , de prendre des initiatives partout sur le territoire , contre l' alcoolisme des jeunes et les drames du samedi soir , contre les infections sexuellement transmissibles et les nouvelles infections . C' est pourquoi je permettrai aux agences régionales de santé de lancer des appels à projets en fonction des besoins locaux , des organisations spécifiques , des demandes des professionnels pour des actions adaptées aux populations . La révolution de la prévention , elle ne partira pas de l' Etat , elle se construira sur les territoires par des initiatives multiples , avec les élus et les professionnels de santé . Enfin , pour développer la prévention , il faut que les Français puissent prendre soin , sans se ruiner , de leurs yeux , de leurs dents et de leur audition . S' agissant des lunettes , des prothèses dentaires et de l' audition , je fixe l' objectif de 100 % de prise en charge d ' ici 2022 . Alors comment nous ferons ? Il ne s' agit pas de dépenser de manière inconsidérée . Il s' agit juste de dire que c' est une réforme essentielle parce que c' est une injustice et c' est une inefficacité . Quand un enfant n' est pas protégé contre sa myopie parce que ses parents ne peuvent pas lui payer des lunettes , il n' arrive pas à lire au tableau et donc il construit son propre échec scolaire . C' est intolérable . Lorsqu' une personne démunie , au chômage , ne peut pas se faire soigner les dents parce qu' elle est mal remboursée , elle ira à l' entretien d' embauche avec des dents gâtées - c' est ce qui se passe dans notre pays aujourd'hui - et donc elle n' aura pas son emploi , à coup sûr . Parce que lorsqu' une personne âgée ne pourra pas s' équiper d' une prothèse auditive , parce qu' elle est mal remboursée , et bien elle rentrera plus vite dans l' isolement , et donc dans la démence , la sénilité ou la dépendance . Et donc toutes ces petites économies qu' on fait aujourd'hui sur le dos des Français , elles conduisent à des coûts économiques , sociaux ou sanitaires beaucoup plus importants quelques années après . Elles conduisent à des injustices aujourd'hui intolérables . C' est pour ça que cette bataille est à la fois juste et efficace . Alors , comment nous allons le faire ? Nous allons pour cela davantage responsabiliser , ensemble , la Sécurité sociale et les mutuelles . La Sécurité sociale rembourse très mal ces soins aujourd'hui : 4 à 5 % , 10 % pour certains soins . Et donc ce sont largement les mutuelles , quand on peut se les payer , qui remboursent ces soins . Je veux que la Sécurité sociale travaille avec les mutuelles pour construire ensemble cette couverture à 100 % pour toute la population , mais en faisant baisser les prix . C' est comme ça que nous ferons les vraies économies . Parce qu' aujourd'hui en France , une prothèse auditive , elle est trois fois plus chère qu' en Allemagne . Qu' est ce qui le justifie ? Rien ! Parce que nous avons laissé cela se faire , parce que nous avons laissé certains s' organiser sur le dos de la population , parce que nous avons laissé trop peu de concurrence s' installer . Parce que - vous le voyez - vous pouvez , en remettant de la juste concurrence , en mettant une pression sur certains producteurs de prothèses , dentaires ou auditives , faire baisser les prix , retrouver un prix plus juste en protégeant mieux . Et donc vous voyez que vous réconciliez ce faisant la liberté et la justice , la liberté et la solidarité . Et donc tout cela passera par une meilleure transparence des prix , par plus de concurrence , par un travail partenarial de la Sécurité sociale et des mutuelles pour arriver à cet objectif . Parce que la réalité que nous vivons est intolérable . II . Le deuxième chantier que je conduirai , c' est l' accès de tous à des soins de qualité . Parce qu' il y a aujourd ' hui des inégalités sociales et des inégalités territoriales . Face aux inégalités sociales , je vais vous le dire clairement : je suis défavorable au déremboursement des petits soins . Ce n' est pas un projet radical ; c' est un projet partial , injuste et inefficace . Et donc oui , le projet que je porte est un projet qui permet , qui même encourage l' accès de tous aux soins , et donc je m' engage à ce qu' il n' y ait aucun déremboursement de soins utiles durant mon quinquennat . C' est un engagement solennel et fondamental . Pourquoi le projet conservateur , que je respecte dans ses composantes et dans son identité , mais pourquoi le projet conservateur de François Fillon , je m' y oppose ? Pourquoi je considère qu' il est à la fois inefficace et injuste ? Il est inefficace pour les raisons que je viens d' expliquer , d'abord parce que je n' ai trouvé personne pour m' expliquer ce qu' était un petit rhume et un grand rhume . J' ai compris qu' il y avait des débats , parfois téléologiques , qui s' étaient noués au sein même de sa propre équipe sur ce sujet . Mais , parce que surtout , c' est inefficace : qui va renoncer à se traiter sur le petit soin ira plus vite vers des maladies plus graves . Parce qu' en déremboursant ce qui paraît innocent alors que c' est utile , et bien on incitera beaucoup de Français à ne pas se traiter , et donc à aller plus rapidement vers des maladies plus graves , qui coûteront au final plus cher . Donc non , ça n' est pas une mesure efficace d' économies . Et en même temps c' est une mesure injuste . Pourquoi ? Parce que vous allez dérembourser . Les plus riches iront quand même se soigner . Les plus modestes - et il faut nous en féliciter - seront soignés parce qu' ils ont une couverture maladie universelle . Il y a une solidarité nationale qui les protège . Mais quelles seront les victimes de cette décision ? Toutes les classes moyennes . Toutes les classes moyennes , et en particulier les plus âgés , ceux qui ont le plus recours aux soins , c'est-à-dire ceux qui n' ont pas la CMU , mais qui n' ont pas assez d' argent pour , justement , aller se traiter sur ce petit soin . Et donc eux seront les victimes de cette réforme inique , de cette mauvaise réforme du déremboursement , à laquelle je m' oppose . De la même façon , supprimer l' Aide médicale d' Etat n' est pas une bonne idée . C' est aussi injuste et inefficace . Pourquoi ? Parce que si vous décidez de ne plus traiter des étrangers modestes sur votre territoire , ils vont développer des pathologies , ils vont recréer des épidémies partout sur le territoire , et donc ils vont contaminer votre population , ils vont fragiliser notre pays . C' est ça , ce qui est intolérable . Et donc le projet porté à la fois par le Front National et la droite conservatrice est un mauvais projet à ce titre . Parce qu' il va conduire à recréer des épidémies . J' étais il y a quelques semaines en Guyane , on voit réapparaître de la tuberculose , des maladies tropicales qui avaient disparu , parce qu' il y a de l' immigration clandestine . Expliquez demain que vous supprimez l' aide médicale d' Etat : ah c' est une idée sympathique , mais vous allez redévelopper des pathologies qui toucheront des gens qui ne sont pas du tout les étrangers , qui sont des Françaises et des Français , mais qui vivent au milieu d' eux . Par contre oui , nous devons mettre plus de responsabilité dans l' Aide Médicale d' Etat , et donc nous travaillerons à un mécanisme de responsabilité , parce que nos concitoyens ne comprennent pas non plus que ce soit un système ouvert à l' encan où il n' y a aucune responsabilité à la clé . Mais ça doit se faire dans un esprit constructif et juste . Vous l' avez compris , ce que je veux c' est donc qu' il y ait de la justice dans l' accès aux soins et que chacune et chacun soit justement traité pour accéder aux traitements auxquels il a le droit . Et en particulier , je veux ici aller au-delà sur un point très spécifique , mais qui compte . Je souhaite que l' Assurance Maladie prenne à nouveau en charge à 100 % certains patients qui ont été laissés pour compte . Vous le savez , nous prenons à 100 % en charge les affections longue durée ( ALD ) . En 2011 , une mauvaise décision a été prise : celle qui a consisté à sortir des ALD l' hypertension artérielle . Il y a 12 millions de nos concitoyens qui souffrent d' hypertension artérielle . Je ne suis pas en train de vous dire qu' on va mettre tous les hypertendus à 100 % , parce que là , je serai irresponsable . Ça pourrait faire plaisir à quelques hypertendus que j' aperçois dans la salle , à leur réaction , mais là , je ne pourrai pas vous dire honnêtement que je sais le financer . Donc je ne vais pas vous le promettre . Par contre , il y a 1 million d' hypertendus sévères , il y en a 50 000 chaque année . Et là c' est une bêtise de ne pas bien les couvrir . Pourquoi ? Parce qu' ils font , ces hypertendus , des accidents vasculaires cérébraux plus facilement , des pathologies , des lésions , des maladies beaucoup plus graves qui vont nous coûter beaucoup plus cher dans 6 mois , dans un an . Et donc je veux que les hypertendus sévères soient désormais pris en charge à 100 % par l' Assurance Maladie . C' est une décision juste , cela coûte une dizaine de millions d' euros par an . Ça n' est pas beaucoup par rapport au défi qui est le nôtre . Mais c' est une bonne mesure parce que c' est une mesure d' efficacité , de justice , qui permettra d' éviter ensuite des pathologies plus graves qui nous coûtent aujourd'hui beaucoup plus cher . S' agissant du tiers payant enfin , je veux prendre le temps d' évaluer cette mesure . Quand on parle de tiers payant ... Vous savez , quand on est dans des périodes électorales , il faut toujours regarder à qui on parle . J' ai un principe , depuis le début : je dis toujours la même chose à tout le monde . Ça m' a parfois valu beaucoup d' ennuis , mais à la fin c' est plus simple : ça évite les reniements , ça évite les corrections , ça évite les changements brutaux . Et au final , la cohérence et la constance , je pense que ça paye . Donc quand vous êtes dans une campagne et que vous parlez du tiers payant , si vous parlez aux patients , ils aiment bien , c' est très populaire ; si vous parlez aux médecins , ils détestent . Ils vous disent tout de suite que c' est abominable , qu' on leur a demandé de faire de l' administration , qu' il n' y a plus de responsabilité et qu' il faut donc en finir . En la matière je pense qu' il faut éviter les dogmes , comme sur beaucoup de sujets . Je pense que le tiers payant généralisé va dans le bon sens , parce qu' il évite que le montant qu' on doit mettre sur la table nous fasse renoncer à des soins . Il simplifie en même temps les éléments de remboursement , parce qu' on attendait que la mutuelle rembourse : là tout est pris dès le début . Mais en même temps je suis sensible à certains arguments : on a alourdi en effet , la tâche de certains professionnels de santé , on demande à des médecins de faire la trésorerie , ils vont devoir attendre plusieurs semaines ou plusieurs mois d' être remboursés par la mutuelle . Parfois ils ne le seront pas , me dit -on . Et puis il y a cet élément culturel qui n' est pas neutre . J' entends certains qui me disent : “ quand quelque chose ne fait plus l' objet d' un paiement , même minime , ça ne vaut plus rien . ” Et donc les gens ne se présentent plus à la consultation , parce qu' ils savent que ça vaut rien , ils ne la paient pas . Donc vous voyez , en la matière , je veux qu' on ne soit pas dogmatique . Donc je ne ferai ni plaisir aux patients qui voudraient que le tiers payant généralisé soit sanctuarisé , ni plaisir aux médecins qui voudraient qu' on abandonne cette idée funeste et qu' on revienne au bon système d' antan , qui était quand même un peu complexe pour tout le monde . Et en la matière , je prends un engagement de responsabilité : je demanderai une évaluation des effets du tiers payant généralisé , pour voir si en effet cela conduit à des pratiques d' irresponsabilité et des excès de la part de patients . Je regarderai les coûts induits pour les professionnels de santé , qui alors doivent être compensés . Je regarderai les dérives qu' on m' a décrites et qui sont conduites par certains autres acteurs . Et donc nous évaluerons , nous regarderons , et nous déciderons , comme on le fait trop peu souvent en France . Et puis si les Français renoncent à se soigner , c' est aussi parce qu' il y a des inégalités territoriales . Vous le vivez ici , nous le vivons , nous l' avons vu cet après-midi : il y a des territoires qui ont à faire face à des pénuries de médecins . C' est pour ça , quand j' entends que l' idée pour le système de soins , ce serait de réduire la fonction publique hospitalière , ce serait de supprimer des postes de praticiens hospitaliers , d' infirmières ou d' aides- soignantes , je dois vous dire qu' après avoir visité l' hôpital de Nevers , l' hôpital Pierre BEREGOVOY cet après-midi , j' ai du mal à comprendre . Je n' ai vu que des services où des aides-soignantes , des infirmières , des cadres , des chefs de service , des praticiens hospitaliers , me disaient : “ nous on s' organise , on arrive à recréer le service , j' ai tel médecin qui vient faire des vacations ici , nous nous organisons ... ” Il y a une inventivité extraordinaire , ici sur votre territoire . Mais partout j' ai entendu “ il manque x postes , il y a trois postes qui ne sont pas pourvus … ” Ah on fait des économies formidables , en ayant supprimé des postes de fonction publique . Mais derrière on paie beaucoup plus cher des prestataires occasionnels , on paie beaucoup plus cher des gens qui viennent faire des vacations et donc , au final cette idée , qui paraîtrait de bon sens qui dit “ on va couper des postes de fonctionnaires ” , et bien elle coûte plus cher , parce qu' on n' a pas supprimé bizarrement les patients qu' il y avait derrière . Parce qu' il continue toujours à y avoir des gens malades derrière . Alors , il n' y a pas de recette miracle , il y a un engagement à prendre . C' est en effet d' avoir davantage de médecins . Depuis 30 ans , on a , droite et gauche confondues , géré le problème du coût de la santé en se disant “ on va réduire l' offre , on va réduire le nombre de médecins qu' on met devant les Françaises et les Français , ça coûtera moins cher ” . Funeste idée parce que la population vieillit , il y a des maladies chroniques et les gens sont toujours malades , donc vous vous retrouvez avec des déserts médicaux . Vous avez en plus , de plus en plus de femmes - il faut s' en féliciter - qui sont médecins mais qui de plus en plus souvent arrêtent la médecine . Imaginez , que 4 médecins sur 10 qu' on forme aujourd'hui vont arrêter de pratiquer durant leur carrière . Donc il faut , oui , former plus de praticiens et il faut rouvrir le numerus clausus , c' est indispensable . Mais ça n' est pas suffisant . Parce que si je vous dis : “ Je vais répondre au problème de l' hôpital de Nevers aujourd'hui en ouvrant le numerus clausus et je vais répondre aux défis de l' hôpital de Nevers dans 10 à 15 ans , mais pas aujourd'hui ” . Donc il faut prendre des mesures de court terme . La première c' est de ramener des professionnels de santé partout sur le terrain , pas d'abord à l' hôpital , partout sur le terrain pour assurer la continuité des soins . Les déserts médicaux , ils sont dans ce tiers des départements français que j' évoquais où vous n' avez plus la continuité des soins . Ils sont dans ces 30 % de départements français où après 20h vous n' avez plus la possibilité d' aller devant un médecin généraliste ou tel ou tel spécialiste , et où vous n' avez plus le choix qu' aller aux services d' urgences le plus proche , qui parfois va être assez loin en voiture . Et ça coûte beaucoup plus cher là aussi , entendons -nous bien , penser à faire des économies avec ça , c' est pas une bonne idée du tout . Le jour où vous arrivez aux services d' urgence , ça va vous coûter beaucoup plus cher que si vous alliez chez un généraliste ou que s' il était venu chez vous . Donc , la continuité des soins elle se fera en rouvrant justement et en accélérant l' ouverture de ces maisons pluridisciplinaires de santé . Chaque fois qu' on ouvre une maison pluridisciplinaire de santé , on comble un désert médical . Je veux ainsi parler des maisons où travaillent ensemble des médecins , des kinésithérapeutes , des orthophonistes , des infirmières , des assistants médicaux . Et je fixe pour objectif de doubler leur nombre d' ici 2022 . Pourquoi c' est important ? C' est pas de l' argent public uniquement , c' est de l' organisation sur le territoire , c' est de l' argent qui sera parfois mis par les collectivités territoriales , elles le font déjà , mais c' est très important parce que c' est aider les territoires à réavoir de l' attractivité médicale . C' est comme ça que vous allez réattirer des professionnels de santé dans les territoires les plus ruraux , c' est fondamental , parce qu' aujourd'hui qu' est -ce qui fait peur à une jeune femme ou un jeune homme qui cherche à s' installer ? C' est qu' il ne veut plus avoir la vie du médecin de campagne d' il y a 20 ans , c' est qu' il ne veut plus pratiquer seul , c' est qu' il ne veut plus travailler de 7h à 22h , être appelé au milieu de la nuit . Il veut travailler de manière plus collégiale , avec d' autres professionnels de santé , il veut pouvoir avoir des gardes partagées , pouvoir travailler avec l' hôpital , être dans un autre cadre qui correspond aux besoins , à sa génération , à la façon de vivre des professionnels de santé , c' est cela que permettent justement ces maisons pluridisciplinaires de santé , c' est pour cela que je veux les doubler d' ici 2022 et c' est aussi ainsi qu' on coordonnera davantage le travail de ces professionnels sur le terrain avec l' hôpital et que ce faisant , nous diminuerons les coûts parce que nous éviterons à des centaines de milliers de nos concitoyens d' aller tout de suite aux urgences . Le deuxième élément extrêmement important c' est le développement de la télé- médecine , c' est fondamental . Nous devons là aussi , permettre de soigner à distance , de suivre un patient cardiaque par un objet connecté , d' envoyer des images , de soigner à distance tout en assurant de la qualité , d' accélérer une consultation chez un patient à risque ou au contraire d' annuler un déplacement long chez un patient pour lequel tout va bien . Avec la télé-médecine , on réduit par définition les distances , mais là aussi on recrée du confort et on diminue les coûts . En ayant de la télé-médecine , vous permettez à des professionnels qui sont ici , ou dans le libéral , ou à l' hôpital , ou dans un établissement de soin pour personnes dépendantes ou personnes âgées de mieux soigner , en lien avec d' autres professionnels et des spécialistes , et ainsi de prévenir mais aussi de guérir , au plus proche de la population . C' est comme ça que vous réglez ce problème d' inégalités territoriales , et que vous facilitez l' accès aux soins et c' est là aussi plus efficace et moins coûteux . Certains ne l' ont toujours pas compris . Quand vous êtes dans un EHPAD , que vous avez une personne âgée qui a besoin de faire un examen , aujourd'hui elle va attendre des semaines et des semaines pour aller dans l' hôpital , le jour où elle arrivera , il y aura eu de l' attente , ça sera peut-être plus grave , il faudra payer le transport à l' hôpital , qui coûte très cher , c' est un poste très important de l' Assurance Maladie , elle se retrouvera devant le professeur mais il n' y aura pas celui ou celle qui la soigne tous les jours , qui est à la maison de retraite ou dans l' EHPAD , qui l' accompagne . Et la personne âgée , un peu dépendante , ne pourra pas répondre à toutes ses questions . Par la télé-médecine , et je l' ai vu faire à Pessac , non loin de Bordeaux , dans un EHPAD , et bien vous avez le professeur , à l' autre bout dans son service , qui parle non seulement au patient , parfois à sa famille qui veut être là , mais également au médecin qui le suit au quotidien , au personnel soignant dans son ensemble qui est derrière l' écran et qui peut échanger beaucoup plus efficacement . Vous n' avez pas dépensé le transport et vous avez un soin beaucoup plus complet , une réponse beaucoup plus complète et beaucoup plus pertinente . C' est cela aussi une réforme en profondeur de notre système de santé . Enfin , la bataille contre les déserts médicaux c' est aussi une bataille de l' information et il n' est pas normal que d' un côté , une minorité éduquée , avisée , possède toutes les clés du système , sache à quelle porte frapper , connaisse le médecin spécialiste et obtienne un rendez-vous sans attendre et que de l' autre côté , la grande majorité des gens soit totalement dépourvue , que lorsque vous êtes dans un désert ou quasi désert médical , vous êtes perdu , vous ne savez pas où il faut aller au fond , c' est ça l' un des principaux problèmes que nous avons parfois . Vous ne savez pas où est en fait le spécialiste ou la maison de soins et donc vous allez aux urgences . Et bien , c' est pourquoi je mettrai en place sur internet ce que j' appellerai un &laquo; Doctissimo &raquo; public . C' est quand même fou , qu' à l' heure d' Internet , alors qu' on a tant d' informations sur tout qu' on peut comparer les prix sur chaque élément de la vie quotidienne , on ne puisse pas savoir la réalité des soins qu' on a autour de soi . Ce portail permettra de disposer d' informations actualisées et fiables sur les maladies , les médicaments , les services publics à proximité , les modalités de prise de rendez-vous et les délais , parce que l' égalité d' accès à l' information , ce n' est rien d' autre , aussi , que l' égalité d' accès à la santé . III . Le troisième chantier , c' est la transformation de l' hôpital . C' est un chantier fondamental . Il est vital . Il est vital en matière de soins , en matière de formation , en matière de recherche . Parce que l' hôpital c' est le pilier de notre système de santé , c' est notre fierté , c' est ce qui forme les personnels de santé partout en France . C' est le pivot parce que c' est aussi l' hôpital autour duquel tout s' organise et je veux ici non seulement saluer les personnels de santé que nous avons vus ensemble , cher Denis cet après-midi , qui font un travail extraordinaire . J' ai vu un enthousiasme , une inventivité , une volonté de faire formidables . Mais ces personnels de santé , c' est aussi ceux qui se mobilisent , qui se sont mobilisés après les attentats que nous avons vécus ces derniers mois en France , partout , qui se mobilisent dès qu' il y a une catastrophe sur le terrain , sans compter leur temps , en revenant . On a vu des médecins qui étaient partis à la retraite , des aides-soignants , des infirmiers qui étaient partis parfois de l' hôpital , revenir pour aider leurs collègues . C' est le cœur vibrant de ce qui tient ce pays ; c'est-à-dire , l' engagement profond qui nous tient ensemble . Et donc l' hôpital , il mérite d' être réformé . Il le mérite profondément , pas seulement pour la santé mais pour toutes celles et ceux qui le font , pour toutes celles et ceux qui , à l' hôpital , portent la santé . Alors le réformer , ça ne veut pas dire mettre plus d' argent . Il faudra investir , investir pour moderniser , et j' y reviendrai - ce sera une constante de mon programme et de mon engagement . Mais en même temps on continue à faire des efforts sur le fonctionnement pour les réduire . Mais il ne faut pas faire sur l' hôpital une réforme monolithique . Les défis de l' hôpital de Nevers n' ont rien à voir avec les défis de l' hôpital Lariboisière à Paris . Rien . Ici , vous manquez de professionnels de santé , vous avez des services qui ont du mal à fonctionner . Je voyais le service de neurologie , on me dit “ nous on peut pas réussir à traiter tout de suite les AVC ” et donc ils ont réussi , avec beaucoup d' inventivité , à mettre en place justement la connexion avec leurs collègues , qui peuvent traiter les AVC à distance . On conditionne le patient de la meilleure des façons , on le transporte à Dijon … C' est formidable , ce qui se passe là . Mais ça n' a rien à voir avec le problème par exemple d' un hôpital parisien , qui est submergé par ses urgences , qui ne manque pas de professionnels aujourd'hui , même s' il commence à avoir un vrai problème d' attractivité , mais qui a simplement tout le reste de la société qui vient à lui . Et donc , la réforme de l' hôpital , elle consiste à regarder en face , et à respecter d'abord tous les métiers et les engagements qui le font : les infirmières , les sages-femmes , les aides-soignantes , les cadres , les médecins , les cadres , les praticiens hospitaliers , les chefs de service , les administratifs , et ensemble avec eux , de concevoir cette réforme . Parce que l' hôpital aujourd'hui , il est en crise de quoi ? Il est en crise des 35h qu' il n' a pas pu digérer , qui l' ont profondément désorganisé - il faut le dire et le regarder en face . Parce qu' on ne parvient pas à y faire entrer pleinement le numérique , à générer les gains de productivité sur les multiples innovations , justement , auxquels on devrait pouvoir parvenir . Par insuffisance des investissements , et parce qu' on laisse l' hôpital face à lui-même , et qu' en quelque sorte il régule toute la crise du système de santé dont je parle depuis tout à l' heure . Quand il y a un désert médical , on va dans l' hôpital voisin même s' il est un peu loin , on ne va que là . Quand il n' y a pas assez de continuité des soins , quand il y a une médecine de ville qui ne fonctionne pas de la même façon , de l' angoisse et de la misère sociale , on va à l' hôpital . Quand on a peur , on va à l' hôpital . Et donc c' est tout le système de soins , ensemble , qu' il faut réorganiser autour de l' hôpital . Parce qu' on a fermé en amont trop de structures , et on n' organise pas bien aujourd'hui les soins . Et donc l' hôpital , dans de très nombreux , dans de trop nombreux territoires , c' est l' ultime recours , voire le seul recours . Alors , pour transformer l' hôpital il faut le transformer avec le reste du système . Cette réforme elle est indispensable mais il faut d'abord déconcentrer le pouvoir au niveau des agences régionales de santé et des acteurs locaux , c'est-à-dire justement de l' hôpital , des élus , pour qu' en fonction des réalités et des besoins , sur le terrain , les acteurs puissent davantage innover , s' organiser , adapter l' offre . Ensuite , il faut décloisonner entre le médical et le médico-social , entre le public et le privé , pour faire émerger de nouveaux partenariats : c' est indispensable . La réforme de l' hôpital elle passera aussi par la capacité d' ouvrir l' hôpital à des professionnels du secteur privé , à permettre - je l' ai vu très bien , là aussi , à Sallanches , il y a quelques semaines , on en a parlé tout à l' heure - et bien , permettre à des professionnels du privé qui font de la médecine de ville , de venir s' installer dans des structures de l' hôpital , pour mieux assurer la continuité des soins , parce qu' ils sont plus rassurés d' opérer à proximité d' infrastructures , parce qu' ils vont pouvoir mieux s' organiser entre eux , et parce qu' on pourra aussi plus rapidement organiser le suivi et la sortie de l' hôpital . Il faut accélérer la dynamique des groupements hospitaliers de territoire . Vous le faites formidablement ici , dans le Nivernais . Qu' est -ce que ça veut dire ? Ça veut dire qu' il faut continuer à regrouper les structures , les centres hospitaliers , l' ensemble des structures qui existent sur le territoire . Mais pour développer les synergies entre elles , pour les rapprocher , pour permettre de faire des bonnes économies en mutualisant les coûts , et en même temps pour permettre d' avoir une bonne répartition sur le territoire , un bon suivi plus intelligent , et améliorer la coordination . Il faut élargir l' expérimentation des maisons de répit et les établissements de soins de suite pour permettre ainsi d' accueillir plus de patients en dehors des hôpitaux . Qu' est- ce que ça veut dire ? Ça veut dire qu' on a besoin d' éviter que tout le monde arrive à l' hôpital . Les maisons de répit c' est ce qui évitera que certaines personnes , qui sont dépendantes , aillent pour soulager leurs proches , à l' hôpital pendant des semaines . On les mettra dans des maisons de répit , médicalisées mais adaptées , qui coûtent beaucoup moins cher . Adapter ce qu' on appelle des centres de soins de suite . Qu' est -ce que ça veut dire , ça veut dire que aujourd'hui , qu' est -ce qui coûte à l' hôpital ? Lorsqu' on vous a opéré , lorsqu' on vous a soigné , vous n' avez plus besoin de rester dans une chambre d' hôpital , sur votre lit . Mais c' est parfois un peu trop lourd pour rentrer à la maison parce que vous êtes seul , parce que vos proches ne peuvent pas apporter des soins minimaux . Parce qu' il n' y a peut-être pas d'ailleurs dans la commune , l' infirmière ou l' infirmier qui pourra passer , de manière régulière . Et donc , la seule façon pour réduire les coûts de manière intelligente , c' est d' avoir ces centres de suite et de suivi qui permettront , hors de l' hôpital , de manière beaucoup moins coûteuse , que vous sortiez de l' hôpital une fois l' opération faite dans un encadrement médical allégé , beaucoup moins coûteux mais rassurant et qui vous permettra de vivre en faisant des économies . Vous voyez bien quand je parle de révolution culturelle , il y a des tas d' économies qu' on peut faire en prévenant mieux , en s' organisant différemment ; qui ne veulent pas dire qu' on va dérembourser , qui ne veulent pas dire qu' on va couper des têtes , qui ne veulent pas dire qu' en quelque sorte la France serait face à une espèce de choix cornélien et abyssal , qui est : si on veut continuer à être juste et ambitieux , on ne peut plus le financer , et si de l' autre côté on veut être responsable il faut forcément faire des choses absolument injustes et régressives . Non ! Il y a des choses de bon sens à faire , qui sont à portée de main . Elles supposent simplement de décloisonner , de mieux s' organiser et de le faire dans la durée . C' est cela , ce que je veux pour l' hôpital . Il faut aussi pour cela faciliter le travail libéral des médecins hospitaliers pour continuer à attirer les talents vers l' hôpital public , pour briser là aussi les tabous : réorganiser les petits hôpitaux qui devront accueillir des patients après une intervention mais qui doivent continuer à être attractifs pour des praticiens hospitaliers . Et donc permettre , là aussi , une vraie fluidité . Je veux que nous avancions en changeant de méthode , en agissant davantage avec le terrain , avec les acteurs eux-mêmes . J' anticipe les débats : on va me dire “ très bien , vous fermez combien d' hôpitaux , quels sont les noms de ville ? La liste des maternités à fermer ? Les services d' urgence à supprimer ? ” Ça n' aurait aucun sens . Aucun . Il y a plein d' hôpitaux qui en fait ne sont peut -être pas à fermer parce qu' on va partiellement les transformer en centres de suivi , parce qu' on va permettre à des néphrologues du privé de faire des centres de soins qui articuleront mieux l' hospitalisation à domicile et ce centre-là . Je veux que partout il y ait des débats régionaux autour du devenir des hôpitaux de proximité . Les élus , les acteurs de terrain , affronteront avec moi la réalité . L' Etat , avec les Agences Régionales de Santé , conduiront ces débats mais la responsabilité sera prise localement , de manière inventive , en responsabilité collective . Enfin , pour transformer l' hôpital , il faut réformer son mode de tarification . Disons -le très clairement , ce qu' on appelle la tarification à l' activité a permis aux hôpitaux de gagner en efficacité . Mais les gains de productivité à l' hôpital , ça ne fait pas l' efficacité du système de santé dans son ensemble : il faut aujourd'hui imaginer et mettre en œuvre des financements qui prennent en compte le parcours de soins du patient et ses différents épisodes . Il faut donc qu' on réforme le mode de tarification à l' hôpital . Sinon on valorise là aussi la répétition des actes , l' activité . Ça n' est pas bon , ça n' est pas intelligent . Et donc il faut qu' on valorise la bonne prise en compte du parcours de soin , c'est-à-dire l' hôpital qui va travailler avec ses partenaires de santé , avec de la médecine de ville , avec d' autres acteurs de terrain . Et ainsi avoir une tarification plus adaptée à ce que nous avons vu tout à l' heure , exactement à ce que vous faites sur le terrain ici . IV . Enfin le dernier chantier , et j' en conclurai là , c' est d ' avoir une vraie politique industrielle et d ' innovation en santé ambitieuse . La santé on en parle toujours comme d' un coût ou d' un malheur ; c' est aussi un fleuron de l' industrie et de l' économie française . Nous avons en France 70 milliards de chiffre d' affaire faits dans la santé , 200 000 employés directs ou indirects , des territoires entiers ; la région Centre , voisine , est une des premières régions qui vit de l' industrie de santé . Nous sommes , aujourd'hui , parmi les premiers producteurs du monde de biotechnologies . J' étais , l' année dernière et il y a deux ans , à ce grand salon des nouvelles technologies à Las Vegas dont on a parlé ces derniers jours . Je n' y suis pas cette année , je suis avec vous . Mais j' y ai vu des formidables innovations technologiques . J' avais , avec moi , la plus grande délégation non-américaine : les Français , la fierté ! On était devant les Anglais , les Israéliens , toutes les autres nations , justement , de l' innovation dont on nous rabat les oreilles . Nous étions là , avec des start-up , deux tiers d' entre elles venaient hors de Paris , et un grand nombre était dans le numérique et la santé . C' est ici qu' elles vont se développer , cher Denis . Sur ce territoire autour de l' hôpital . Il y a un formidable levier de développement pour les nouvelles industries de santé : autour du dispositif médical , autour des nouveaux objets connectés qui développeront des nouvelles formes de diagnostic , qui vont faire qu' on va soigner et prévenir différemment la maladie . Et qui font que partout sur le terrain , on va réinventer grâce , justement , à ceux qu' il y a , grâce à la formation des infirmières qui existent ici et qui est formidable . Grâce à cet hôpital , on va redévelopper de l' activité . Et donc nous avons besoin , en France , d' encourager cette industrie de santé . Et donc je veux d'abord qu' on arrête cette politique aux petits pieds qui consiste tous les six mois ou tous les ans à changer notre politique de remboursement ou de déremboursement avec une variable d' ajustement qui sont les laboratoires pharmaceutiques . Nous sommes en train de sacrifier notre industrie pharmaceutique pour essayer de faire des économies sur son dos . Il faut continuer à valoriser le générique lorsqu' il existe , c' est une bonne pratique , mais il faut payer les médicaments au juste prix quand il y a derrière des innovations , quand il y a des investissements , pour avoir une grande industrie pharmaceutique . C' est indispensable ! Et en même temps , il faut être responsable . Et donc on va faire des économies , mais des vraies , des bonnes , des économies innovantes . Quand vous faites du mal à vos laboratoires pharmaceutiques , comme on l' a fait , ils détruisent de l' emploi , ils ferment des sites . Et puis , on voit bien , vous l' entendez comme moi ces dernières semaines , ces derniers mois , ils créent de la rareté . On entend parler de produits qu' on n' arrive plus à trouver et de médicaments qui ne sont plus à la pharmacie . Parce que qu' est -ce qu' ils disent ? “ Bah vous m' avez tellement baissé les prix en France , moi je les vends en Belgique , en Allemagne , parce qu' ils paient le juste prix , mais je les vends plus chez vous . ” Vous imaginez dans quelle situation on est en train de tomber ? Donc on va arrêter ça . On va leur donner une visibilité sur plusieurs années , on va leur demander de faire des économies justes , mais de faire leur travail , de récompenser l' innovation . Par contre , en médicaments on va faire des vraies économies . Je suis sûr que , vous qui toussez - vous n' êtes pas la seule , je vous rassure ... moi aussi , je sors d' une vilaine toux - bon , vous avez été chez le pharmacien . Vous avez acheté des pastilles au mieux ; si c' est plus grave , le médecin vous a prescrit , j' espère pas des antibiotiques encore , peut-être si ça dure , mais des médicaments . Vous êtes allé les acheter . Et puis ça va passer dans deux , trois jours - c' est tout le mal que je vous souhaite . Vous allez avoir consommé la moitié de la boîte . Puis elle va finir dans votre tiroir . Alors parfois vous allez , dans deux mois si ça revient , la réutiliser . C' est pas forcément d'ailleurs la meilleure des choses parce que vous n' avez peut-être pas la même pathologie . Mais la plupart du temps vous allez le garder , et puis quand vous ferez le rangement dans un an , deux ans , vous allez découvrir qu' elle est périmée . C' est ça ce qui se passe : on l' a tous vécu . Sous le contrôle familial , je crois être un coproducteur de ce gâchis . Ce que je ferai , ce à quoi je m' engage , c' est la généralisation de la vente à l' unité du médicament . C'est-à-dire , quand vous avez une pathologie , quand le médecin vous prescrit un médicament , et bien on vous vendra ce qui correspond à la prescription , ce qui correspond à ce à quoi vous avez besoin , ce qui correspond sinon à votre renouvellement d' ordonnance , quand vous avez une pathologie de longue durée . Parce que c' est comme ça que nous procéderons à des vraies économies . C' est comme ça , en passant à la vente à l' unité du médicament que nous ferons des économies massives , mais sans faire des mauvaises , des fausses économies . Et en parallèle , j' aurai une politique volontariste pour développer la santé en France , les laboratoires pharmaceutiques , l' innovation en santé , et donc l' innovation dans les dispositifs médicaux , qui est la révolution en cours dans laquelle nous devons gagner . L' innovation en matière de santé pour qu' il y ait plus d' objets connectés et d' innovation dans le numérique . Ça passera par de l' investissement privé , mais aussi par une simplification de nos procédures d' autorisation de mise sur le marché , par une simplification réglementaire qui fait qu' en France aujourd'hui , on doit beaucoup plus simplement innover dans ces matières , développer ces produits , développer ces innovations qui sont bonnes . Pourquoi ? Parce qu' aujourd'hui vous avez des laboratoires formidables , vous avez des innovations qui sont faites dans les hôpitaux , dans les centres de recherche , parfois on les finance au début avec du crédit-impôt recherche : c' est de l' argent public . Et puis parce que la France est un pays extraordinairement compliqué pour donner ces autorisations de mise sur le marché , et bien ces Françaises et ces Français qui ont fait ces innovations , ils vont développer le produit ou le dispositif médical en Allemagne ou aux Etats-Unis ou ailleurs . Je l' ai tellement vu ... C' est insupportable . C' est inefficace : on a financé leur formation , le début de la recherche et tout le reste . Donc on doit simplifier nos procédures , les accompagner , parce que je veux qu' on soit une grande nation en matière de santé , et donc une grande nation en matière d' industrie de santé aussi . Voilà ce que je veux , mesdames et messieurs , chers amis , et je conclurai là-dessus . Voilà ce que je veux . Je veux que nous soyons fiers de notre santé . Je veux que nous la regardions en face , comme elle doit l' être au XXIème siècle . Comme non pas simplement un coût , non pas comme une charge , comme un devoir qui conjugue la solidarité et la liberté , comme une ambition légitime que notre pays doit porter pour celles et ceux qui souffrent et sont malades , pour leurs familles , pour celles et ceux qui sont engagés dans ce secteur de la santé , pour les acteurs de terrain , élus , fonctionnaires , qui chaque jour l' accompagnent . Et pour notre pays , parce qu' il le mérite . C' est comme cela que je veux conjuguer notre énergie . Vous savez , je me suis laissé dire - et il y a dans la salle je crois des témoins directs de cette scène - que le soir de son élection , François MITTERRAND a réuni les élus du Conseil général de la Nièvre et a dit : &laquo; Ce soir mes amis , je vais vous le dire , ce pouvoir que les Français m' ont donné , je suis venu vous le rendre &raquo; . Je me retrouve dans cette philosophie . Ce que nous allons faire , c' est que nous allons prendre le pouvoir pour vous le rendre . Ce que nous allons faire … Ce que nous allons faire mes amis , c' est que nous allons , jour après jour , justement , réveiller cette flamme française , cette énergie , que j' ai vue cet après-midi à Nevers , que je sens ce soir dans cette salle , cette énergie qui veut regarder les défis qui sont les nôtres , de manière adulte et responsable . Qui sait qu' il y a des défis , des coups . Qui sait que nous aurons des efforts à faire , pendant non pas simplement les cinq ans , mais les dix ans à venir . Mais qui sait aussi que notre peuple a l' énergie pour cela . Ce que je veux pour nous toutes et tous , c' est cette flamme française , cette flamme je veux la réveiller avec vous , c' est ce qui fait que nous réconcilierons la liberté et la justice , cette volonté de faire réussir le pays , d' être fiers de ce que nous sommes en étant dans le siècle qui advient , non pas en défendant des modèles passés , non pas en galvaudant des mots dans lesquels certains se réfugient pour ne rien faire , non pas en promettant du sang , des larmes , ce qui a été fait il y a trente ans ou quarante ans dans des pays voisins . En regardant en face nos défis , mais en les relevant ensemble , en considérant que l' Etat et ceux qui prétendent dans l' Etat aux plus hautes fonctions , ont une responsabilité : donner un cap , fixer un sens , faire émerger des solutions concrètes , réelles , dans le siècle , mais que tout cela nous le ferons ensemble . Que les vraies réformes nous ne les ferons pas contre les Français , en réduisant leurs droits , mais bien en faisant émerger cette énergie , en donnant une place à cette énergie , en la faisant réussir , partout sur le territoire , en la considérant . Et donc , tout cela , tout cela mes amis , dépend de nous . Aujourd'hui , demain . Et donc dans les prochaines semaines , dans les prochains mois , c' est ensemble ce projet que nous allons porter . Je compte sur chacune et chacun d' entre vous , ici jusqu' à l' été prochain , pour avec moi porter ce projet , cette vision , pour convaincre avec votre énergie , pour persuader avec votre engagement , pour mobiliser avec notre optimisme ! Vive Nevers ! Vive la République ! et vive la France ! ",
    "Merci à vous . Bonsoir à toutes et tous . C' est formidable de voir cette énergie , là , dans cette salle . Vous êtes plus de 2500 ce soir . Nous sommes plus de 2500 ici , et je veux commencer par présenter mes excuses aux quelque 500 personnes que nous n' avons pas pu faire rentrer , qui comme vous ont attendu dans le froid et malheureusement n' ont pas pu nous rejoindre . Mais vous êtes là , nous sommes là , c' est le plus important et je vous en remercie . Merci à Guy et Valérie d' avoir , non seulement lancé notre rassemblement ce soir , mais depuis le début de vous être engagés à nos côtés pour porter le mouvement , ici , à Clermont , dans tout le département et au-delà d'ailleurs , dans la région . Et je sais que ce soir , ça n' est pas simplement Clermont , Cournon , mais bien toute la région et au-delà , qui est avec nous , et qui ce soir donne toute son énergie . Je me souviens avec beaucoup d' émotion de mes précédents passages à Clermont . Il y a près de 2 ans maintenant , lors de l' enterrement de François MICHELIN . Et je me souviens , cette gravité vibrante qu' il y avait , cette pierre noire de la cathédrale , cette gravité qui avait envahi toute la ville et qui montrait combien ici , on ressent profondément , dans le silence et la douleur . Je me souviens aussi , quelques mois plus tard , lorsque nous sommes allés à Issoire , à Champeix ... mais ce n' est pas possible que vous soyez autant sauf à ce que Roger JEAN , que je salue , ait emmené tout le monde ! Et il est avec nous depuis le début , lui aussi , avec courage . Je veux remercier toutes celles et ceux qui ce soir sont là à mes côtés , sont venus du département , de la région . Jacques MEZARD qui est là depuis le début , esprit libre et courageux , qui m' a emmené dans son Cantal - et j' en reparlerai tout à l' heure - avec bonheur et fierté , et qui continue à m' accompagner pour que chaque jour cette ruralité - sur laquelle je reviendrai - on ne puisse non pas en parler avec des mots , mais aussi avec de l' émotion . Parce que je n' oublie pas tes larmes lorsqu' un petit matin tu m' as emmené dans une exploitation laitière , et que tu m' as expliqué comment ces territoires parfois avaient été abandonnés ou délaissés . Je remercie aussi Christian , Christian TEYSSEDRE , qui vient de Rodez , et qui lui aussi depuis le début est à mes côtés , et nous avons cheminé l' été dernier , et vous êtes plusieurs à venir de la région . Remercier la Lozère et Francis , remercier la Corrèze , voilà , vous remercier toutes et tous , et René , René SOUCHON qui a dirigé la région et qui est là ce soir aussi . C' est cette réconciliation de toutes celles et ceux qui parfois d'ailleurs ont mené sur le terrain des batailles contraires , qui n' étaient pas toujours d'accord , mais qui ont toujours , avec la même conviction , oeuvré , oeuvré ici , oeuvré pour l' Auvergne et ses belles terres . C' est aussi le combat que mène votre président du conseil départemental , Jean-Yves GOUTTEBEL , il n' est pas avec nous ce soir , il n' a pas pu , mais je m' honore d' avoir son soutien et son parrainage . Et je ne peux ni ne veux passer sur ces terres , sans rendre un hommage légitime à celui qui pendant de nombreuses années a oeuvré pour l' Auvergne , le Président GISCARD D' ESTAING , qui après avoir accompagné la France dans un moment fort de modernisation , de libération aussi de sa société , a porté la cause de l' Auvergne , loin et fort , et a commencé certains combats que Jean-Yves continue aujourd'hui . Vous êtes nombreux ce soir ici , vous êtes nombreux parce que quelque chose se passe dans le pays . Parce que oui , une vague monte . Cette vague , ça n' est pas un homme seul , comme certains veulent encore le croire , ou alors vous les avez ce soir formidablement démentis . Cette vague , ça n' est pas non plus un mouvement parisien , ou alors ce soir j' ai un problème de géographie . Et cette vague , ça n' est pas une bulle , c' est une vague qui monte depuis le premier jour , parce qu' elle vient du cœur du pays , parce que ce que depuis le premier jour nous avons fait ensemble , c' est d' écouter , de construire , d' aller chercher les idées , les convictions , partout dans le pays . Nous avons fixé un cap , des valeurs , mais l' énergie est là partout et c' est celle -ci qui accompagne En Marche , c' est celle -ci qui monte , et ce soir avec vous j' en suis fier , j' en suis heureux ! Alors je suis venu ce soir , à Clermont-Ferrand pour vous présenter mes vœux . Je vous ai fait grâce de ces vœux vidéo dans lesquels tout le monde aujourd'hui a décidé de donner son talent . Parce que quand on a certaines fonctions , ils sont légitimes , on peut les faire à ses amis , mais il me semble que ce qui a le plus de sens c' est de les faire au cœur , au contact du pays , et donc je les fais ce soir devant vous . Evidemment , il y a les vœux de santé , il y a les vœux pour vous , vos proches , vos familles , il y a des vœux de mobilisation , indispensables parce que notre combat est collectif pour redresser notre pays . Mais les vœux ce soir que je veux formuler pour vous , ils tiennent en deux mots : &laquo; penser printemps &raquo; . Ces deux mots , ce ne sont pas les miens , je les emprunte sur ces belles terres radicales à une grande figure du radicalisme français , à un grand intellectuel , un homme politique , un professeur : Alain . Il fait partie de ceux qui ont rythmés mon enfance , de ces lectures qu' on n' oublie pas . Alain a écrit beaucoup de grands textes , mais il a écrit aussi ce qu' on appelle ses “ Propos ” , qui étaient une espèce de carnet quotidien . Et il y a à peu près un siècle , un peu plus , il a écrit en un 31 décembre et dans les premiers jours de janvier , un très beau petit texte qu' il a intitulé &laquo; Penser printemps &raquo; . Et il disait , et je le cite de mémoire , donc imparfaitement , qu' au moment où l' année tourne sur ses gonds , quand - disait -il - les arbres font dentelle ( et ce soir je crois qu' on peut le voir , ce sont ces moments où le givre saisit la nature ) , lorsque tout semble mort , lorsque tout semble triste , lorsque tout parfois semble perdu , morose , il faut penser printemps . Non pas penser printemps pour s' émerveiller ; non pas penser printemps par bêtise , ou par une forme de béatitude médiocre ; mais penser printemps parce que , au moment même où cette désolation s' installe dans la nature , quelque chose se prépare qui est déjà là . Alors certains ne le voient pas ou ne veulent pas le voir , certains peuvent désespérer , mais le printemps revient , et il est déjà gros dans l' hiver , il est déjà là . Alors ce soir , pour vous et vos familles , pour nous tous , je veux vous dire de penser printemps . Et penser printemps , d'abord , cela veut dire ne pas nous affaisser , ne jamais accepter le quotidien , les peurs , les doutes , parfois la panique qui nous étreint , ne jamais accepter d' être victime et mener le combat . Et je commence par cela parce qu' aujourd'hui , il y a deux ans , notre pays était frappé par les attentats . Je n' oublie pas les attentats de Toulouse , dès 2012 . Je n' oublie pas tous les attentats qui ont suivi : l' Hyper Cacher deux jours plus tard , évidemment ceux de novembre 2015 , de Magnanville , de Saint-Etienne , du Rouvraix , de Nice . Mais , il y a deux ans nous avons encore toutes et tous à l' esprit ces 3 jours de panique qui ont commencé , ces 12 morts . Cela nous devons ce soir y penser . Je veux d'ailleurs ce faisant rendre un hommage tout particulier à l' un des nôtres , l' un des vôtres , Michel RENAUD , journaliste de Clermont qui était passé là pour échanger quelques dessins avec Cabu , et qui lui aussi fit partie des victimes . Alors ce jour-là , notre pays a été frappé au cœur , et ce sont les beaux mots de Prévert qui me viennent à l' esprit : &laquo; j' ai compris le bonheur au bruit qu' il faisait en partant &raquo; . C' est cela ce qui s' est passé . Ce fut la fin de l' innocence . Je vous dis cela ce soir , non pas pour agiter les peurs , non pas pour vous dire que nous devons tout oublier de notre quotidien , de nos défis , de notre avenir , non , pas du tout . On ne construit simplement rien de solide si on ne pense pas au passé , si on n' a pas en mémoire ce qui est advenu . Mais je vous le dis aussi ce soir , parce que penser printemps dans ces moments-là , ça veut dire avoir l' esprit de responsabilité , mais ne rien céder à la peur , ne rien céder non plus aux voix qui voudraient nous faire changer , basculer , nous faire oublier ce que nous sommes au titre de la peur . Dans notre pays , certains ont voulu nier ce qui s' est passé . C' est une erreur profonde . Mais dans notre pays , bien souvent la réaction a aussi été une forme de panique , d' agitation continue , une forme d' autoritarisme excessif , comme si cela permettait au fond de répondre à l' émotion . Rien ne répond à l' émotion . Elle est là et nous devons la vivre , l' embrasser , l' accompagner mais nous devons nous tenir debout . Ne jamais accepter d' être les victimes , non pas d' une guerre - une guerre supposerait d' avoir un ennemi et une armée - mais d' une lutte , une lutte sans merci qui a alors commencé . Une lutte contre le crime terroriste absolu , contre un islamisme terroriste et radical qui abîme notre pays , qui l' abîme parce qu' il conduit certains à confondre tous les sujets , et à vouloir par ce truchement attaquer une partie des Françaises et des Français en raison de leur religion . Ils divisent le pays en son cœur , parce que certaines et certains ont commis l' erreur de vouloir sombrer dans la panique . Nous devons dans cette période qui va durer , nous tenir droits , forts , parce que c' est cela l' Histoire , le sens , le cœur du peuple français . D' être parfois attaqué , mais de ne rien céder à la panique , d' être tenu par un commandement : celui de l' autorité vraie , qui donne un cap , qui assure une sécurité , mais qui à chaque instant , refuse de voir son agenda dicté par quelques-uns qui sont des barbares , refuse de se laisser diviser par quelques-uns qui n' attendent que cela . C' est cela notre devoir , et c' est cela ce que je veux conduire . Alors cela signifie continuer à nous battre , sur tous les théâtres d' opérations où la France et ses valeurs doivent être défendues , et où les risques peuvent advenir , où les terroristes se préparent : la Syrie et l' Irak aujourd'hui , la Libye sans doute demain , une partie du Sahel . Et la France continuera à prendre ses responsabilités sur ces théâtres d' opérations , elle le continuera avec un engagement sans faille , et je veux ici , avec vous , ce soir en ce début d' année rendre un hommage vibrant à nos forces de l' ordre , à nos armées , à toutes celles et ceux , à nos enfants qui se battent sur ces théâtres d' opérations , pour défendre la France , pour la protéger contre ce qui est et ce qui peut advenir . Cela , nous continuerons à le porter . Mais nous l' inscrirons dans une stratégie qui doit être plus européenne , avec nos partenaires allemands , avec nos partenaires européens . Parce que ces défis sont des défis européens , aussi . Parce que la France ne peut porter seule , éternellement , ce fardeau . Parce que l' Europe , c' est un rêve de paix , c' est un rêve vibrant , qu' on doit cesser d' accuser de tous les maux , mais à l' égard duquel nous devons être exigeants . Et donc oui , j' irai la semaine prochaine à Berlin pour parler précisément de cela , des nouvelles menaces qui existent partout dans le monde , d' un monde qui se bouleverse , où la puissance américaine n' est plus là pour nous protéger , et où la France doit porter son rôle , son rang , mais avec ses partenaires , avec l' Allemagne , avec l' Europe . Être à la hauteur de ces défis , penser printemps aujourd'hui en se remémorant ce qui s' est passé il y a deux ans , c' est aussi prendre toutes nos responsabilités en matière de sécurité . J' entends les doutes , les colères de nos forces de l' ordre , qui se sentent aujourd'hui parfois abandonnées , qui ont besoin de moyens - et j' y reviendrai dans les prochaines semaines - qui attendent de la visibilité , une considération légitime . Elles étaient là , tout au long des mois qui ont passé . Et nous devons non seulement maintenir les engagements pris , mais aller plus loin , confirmer les 10 000 engagements d' embauche dans la police et dans la gendarmerie pour les 5 prochaines années , qui sont indispensables pour maintenir la sécurité de nos territoires , au plus proche du terrain . Par une police de proximité , par une police au contact de la population qui , elle aussi , permettra de mieux prévenir . Cette sécurité , elle est la réponse dans la durée , sans panique , sans agitation , dans la constance , la réponse à ce que nous sommes en train de vivre . Car cette lutte va durer . La réponse c' est aussi d' avoir , chevillée au corps , une vraie politique de laïcité . Ah , ce mot ! Certains lui auraient presque fait perdre son sel , en le répétant à ce point . La laïcité , ça n' est pas un mot de haine . Ça n' est pas un ennemi . Ça n' est pas non plus un mot devenu ringard , comme disent d' autres . C' est le cœur vibrant de notre République . C' est l' un de ses principes . C' est au cœur de son socle . Parce que la laïcité c' est une liberté : la liberté de croire , et de ne pas croire . La laïcité , c' est ce principe qui protège tout un chacun , qui dans un quartier , dans une ville , ne veut pas céder à la tyrannie de la majorité et veut garder son quant-à-soi , sa liberté de penser . La laïcité , c' est cette liberté qui permet à d ' autres d ' exercer leur religion en respectant les autres religions , mais de le faire partout aussi dans la République . Ne cédons rien sur ce principe . Ni à celles et ceux qui veulent défendre des communautarismes en attaquant la laïcité , qui veulent porter un projet politique contre la République en s' attaquant à la laïcité , mais ne laissons rien non plus à celles et ceux qui veulent attaquer la laïcité pour porter une forme de projet de haine ou d' exclusion . La laïcité c' est la République française tout entière . C' est un bloc , et ce bloc , c' est une liberté ! À vos côtés , ce sont ces engagements , ces principes , que par fidélité à ce que nous sommes et ce que nous avons vécu , je porterai durant les mois qui viennent et les années qui viennent . Penser printemps , c' est construire l' avenir , vous l' avez compris . C' est cela , le rêve d' Alain . Mais penser printemps , ça ne veut pas dire non plus promettre à tout va , penser que le printemps est déjà là , que sans avoir semé on pourrait déjà promettre la moisson à tout un chacun . Parfois ces périodes de campagne électorale sont une incitation à la moisson de ce qu' on n' a pas encore semé . Mais pour penser printemps , il ne faut pas non plus désespérer , il faut accepter d' expliquer qu' on peut semer sur toutes les terres , qu' on peut produire sur toutes les terres , à condition qu' on s' en donne la peine et les moyens . Le grand homme de cette ville , de cette région , Pascal , avait une très belle formule : il faut qu' il y ait dans l' éloquence , disait -il , de l' agréable et du réel . Mais il faut que cet agréable soit réel . C' est une de ses “ Pensées ” ; elle est d' actualité . Parce que ce que je vois autour de nous , c' est beaucoup de projets qui parfois fleurissent , des promesses qui aujourd'hui , dans les campagnes de certains , sont formidables : on vous promet de distribuer l' argent qu' on n' a pas , des revenus universels , de nouveaux droits , un crédit . On promet au fond l' argent de nos enfants ou de nos petits-enfants . C' est de l' agréable sans le réel . Puis de l' autre côté , j' entends des projets qui vous proposent la purge , le tournevis . L' Angleterre , mais pas celle d' aujourd'hui , celles des années 80 . La société française , mais pas celle d' aujourd'hui , celle d' avant les années 60 . Ils promettent une forme de réel , celui des arbres qui font dentelle dont nous parlait Alain . Mais enfin , c' est pas très agréable . Et donc pour penser printemps il faut conjuguer les deux . Il faut regarder la réalité telle qu' elle est , avec tout ce qu' elle emporte de contraintes , de défis . Et donc il ne faut pas semer des illusions . Mais en même temps , il faut refuser d' enfermer les Français dans une espèce d' alternative mortifère où pour progresser , il faudrait réduire les droits de certains ; où , pour aller de l' avant , il faudrait désespérer ; où , pour essayer d' entrer dans le XXIème siècle , il faudrait adopter les mesures et les techniques d' il y a 20 ou 30 ans . Penser printemps mes amis , c' est réconcilier l' ambition et le réel , c' est réconcilier la volonté de faire et les contraintes d' aujourd'hui . Et donc , le projet que je veux porter avec vous , c' est un projet de bon sens . Mais où nous allons toutes et tous nous dire la vérité : il faut d'abord produire pour pouvoir distribuer . Sinon ça n' existe pas . J' avais tout à l' heure des gens très généreux qui , devant les Halles de Clermont , m' expliquaient : “ vous … ” - malheureusement , chacune et chacun vit dans sa condition : je passe pour être un odieux banquier ; j' essaye de me remettre , de me traiter , depuis plusieurs années , mais ça ne passe pas … Et donc , on me disait “ vous , vous êtes un odieux banquier , vous ne faites rien pour les pauvres ” . Alors j' essayai de me défendre , je lui dis “ non , les gens comme vous m' avez attaqué déjà il y a deux ans , quand on a fait l' ouverture des cars ” . C' est pas mes chiffres , ça vient d' être montré par des analyses indépendantes : 900 000 personnes modestes , pauvres - parce qu' il faut appeler un chat un chat - ont pu se déplacer . Des gens qui ne pouvaient pas se payer le train , ou qui n' ont pas de voiture . Et donc , lui disais -je : “ vous vous faites une fausse idée . Vous appelez ça de la libéralisation : c' est un gros mot . ” On a ouvert un secteur , les autocars , qui était fermé , tout était interdit . Vous vouliez ouvrir une ligne entre Clermont et Lyon , ou Clermont-Marseille , il fallait demander des autorisations invraisemblables , il fallait faire du cabotage international . Si vous faisiez Clermont-Milan en passant par Lyon , vous aviez le droit . On a simplifié . Donc oui , on a libéralisé . Mais la liberté , on l' a réconciliée avec la justice . Parce que qui en bénéficie ? Des gens qui choisissent de se déplacer ainsi , et qui ne pouvaient pas quand c' était fermé avant . Et donc vous voyez bien cette fausse idée . Parce que lui il me disait : “ si vous étiez pour les pauvres , vous taxeriez les riches ” . Je lui ai dit : il faut que vous regardiez comment les autres font à côté . Il faut , vous avez raison , pour redistribuer , taxer les plus fortunés . Mais enfin , on les taxe pas mal en France déjà . Honnêtement . Je vais être honnête , je ne vais pas pour vous faire plaisir dire qu' on va taxer davantage les riches . Parfois ça a marché dans des campagnes . On a pu dire : on va les taxer encore plus . Dans une économie ouverte au monde - et la France elle est dans le monde , elle produit dans le monde - on peut décider de taxer les plus riches . Mais enfin , ils vont s' enrichir ailleurs . Donc on perd des opportunités . Il faut mieux les taxer pour réinvestir dans les services publics , dans l' économie . Mais il faut surtout les encourager à produire davantage dans le pays . Et ensuite il faut voir comment donner une place à chacune et chacun , comment ouvrir des secteurs . Ce qui est intolérable , c' est pas que des gens s' enrichissent , quand c' est pour de bonnes raisons : quand ils ont inventé quelque chose , quand ils travaillent dur , quand ils s' engagent . Ce qui est intolérable c' est que d' autres , parce qu' ils viennent de milieux modestes , ne puissent pas s' enrichir , ne puissent pas , alors qu' ils ont des talents , du travail , de la volonté de faire , aller de l' avant . C' est cela , le vrai combat qu' on doit mener ! Et donc ce que je veux , c' est qu' on réconcilie la liberté et la justice . La liberté et l' égalité . Ça n' est pas un hasard , si sur le frontispice de notre République , ces deux mots sont accolés . Ça n' est pas pour les opposer , c' est parce qu' ils se conjuguent ensemble , en action . C' est cette réconciliation à laquelle nous devons arriver . C' est cela le projet progressiste que nous allons porter ! Alors pour ce faire , il faut libérer . Libérer l' énergie , l' énergie des individus , l' énergie du peuple , et l' énergie des territoires . Ce sera ma première priorité . Le même Alain avait une phrase superbe . Très radicale , cher Jacques , du début du XXème . Il disait : “ l' individu qui pense contre la société qui dort , voilà l' histoire éternelle et le printemps aura toujours le même hiver à vaincre . ” On retrouve notre ‘ penser printemps ' . Cette bataille de l' individu , c' est celle pour la liberté , la liberté de faire , d' entreprendre , de créer , de travailler . Alors oui , je crois à la liberté des individus , à la liberté pour faire . Et je l' ai dit , le premier combat que je veux mener , c' est un combat pour le travail , je veux que chacune et chacun puisse travailler dans notre pays plus facilement , que les entrepreneurs embauchent plus facilement , que les entrepreneurs investissent plus facilement , mais que chacune et chacun puisse aussi travailler plus facilement et soit mieux récompensé de son travail . Alors pour cela , qu' est -ce que nous allons faire ? D'abord conduire une réforme du Travail ambitieuse : notre droit du travail doit être simplifié . Non pas en vous promettant une loi qui va dire “ on doit passer de 35h à 39h , on doit modifier tel ou tel dispositif ” , non . Parce que d'abord je rentrerai dans une rizière terrible , qu' on connaît par cœur - on en a vécu un énième épisode au printemps dernier . Nous devons rentrer , dans la matière , dans le cadre d' une République contractuelle . Qu' est -ce que ça veut dire ? La loi doit définir un cadre , un ordre public social ; le reste il faut le renvoyer au contrat , à la discussion sur le terrain : les accords majoritaires de branches , d' entreprises , ce qui sur le terrain , permet de construire les bonnes solutions pour , en matière de seuils , en matière de temps de travail , en matière d' organisation , permettront de s' organiser plus librement , plus concrètement . Je refuse de réduire la discussion sur le droit du travail à une espèce d' opposition entre les généreux irréalistes et les réalistes qui réduisent les droits . Ça , c' est une bataille qui mènera nulle part . Comment nos voisins ont fait ? Certains ont accepté de réduire les droits , massivement . Eux ils ont fait de l' austérité - on entend souvent parler de cela . Ce n' est pas ce que je propose . D' autres , les Allemands , certains autres voisins , ont su créer , au plus près du terrain , des vrais accords , une logique productive industrielle pertinente . C' est cela que je veux pour notre économie . Parce que , une exploitation agricole , l' exploitation que nous avons visitée ensemble dans le Cantal , elle n' a rien à voir en termes de droit du travail , en termes de contraintes , avec un grand groupe industriel qui est à Paris , qui a deux usines en France , ou même avec la Bosch , comme on dit , à côté de Rodez . Ça n' a aucun sens ! Vous avez d' un côté quelqu'un qui a deux employés , qui doit se lever tous les matins à 5h quand ça n' est pas plus tôt , qui se bat au quotidien , avec des règles qui ne sont pas faites pour lui ; et de l' autre un grand groupe , qui a des milliers d' employés , et qui est installé , avec une solidité financière , une capacité à s' organiser , à gérer la complexité . Si nous voulons avoir une réponse adaptée , et bien il faut permettre , selon les secteurs , la taille l' entreprise , d' adapter nos règles . De permettre que dans cette exploitation agricole , qui veut grandir , ou dans un restaurant , qui veut se développer - il y en a tellement qui viennent de la région et que je connais si bien à Paris … Vous pouvez les applaudir parce que ce sont des courageux ! Il faut qu' ils puissent négocier peut-être de ne pas avoir , de ne pas créer un comité d' entreprise ou un CHSCT à 50 salariés . Parce que les représentants des salariés sont prêts , parce que ça n' a pas de sens pour eux , mais en revanche ils veulent des heures supplémentaires mieux payées . C' est ça le vrai dialogue social . Le dialogue pragmatique . C' est cela que nous allons mettre en place , c' est ce sur quoi on va avancer . Ensuite , il y a un point qui me tient à cœur : c' est ce qu' on appelle le coût du travail . Là aussi , que n' entendons -nous : “ quand on baisse le coût du travail , on fait des cadeaux aux entrepreneurs ” . Non . On ne fait pas des cadeaux . De la même façon que je n' ai jamais croisé un entrepreneur qui embauchait les gens pour les licencier . Il y a des idées ancrées dans la tête de certains qui n' ont pas beaucoup de sens . Mais nous sommes au cœur d' une révolution numérique , d' une transition environnementale profonde , qui change nos habitudes de consommation , qui change notre organisation collective . Qui fait que nous sommes aujourd'hui dans ce monde ouvert , soumis à la concurrence de pays à bas coûts , de pays qui innovent , et que nous-mêmes dans beaucoup de secteurs , nous devons innover , investir , transformer le modèle productif . Et pour cela , et bien il faut prendre en compte deux choses : il faut que les employeurs puissent investir pour se moderniser , pour produire moins de CO2 , pour justement accompagner , non pas accompagner , faire cette transition énergétique et environnementale . Ce sont les entreprises qui les font sur le terrain , celles qui vont investir pour moins polluer , pour développer des panneaux solaires , pour changer leurs processus de production , pour avoir justement une meilleure gestion des flux . Et , elles doivent investir aussi - tu le sais bien cher Bruno - pour la robotique . Pour là aussi , se développer , être plus compétitives . Pour le numérique . Quand vous n' avez pas de marge , vous ne pouvez pas investir . Et donc , en 2011 , la France a atteint un niveau record de charges mais surtout elle avait le niveau record des marges les plus basses . Quand les entreprises en sont là , elles ne mènent aucun combat . Et l' investissement qu' on ne fait pas aujourd'hui , ce sont les emplois qu' on fera pas demain . Donc oui , je suis pour conduire une politique d' amélioration des marges de l' entreprise , pour qu' elles puissent investir et embaucher . Et de l' autre côté , si vous ne baissez pas le coût du travail , le travail sera de plus en plus concurrencé par le travail à l' autre bout du monde , celui qui ne coûte rien et qui ne paye aucune charge en France , celui qui est fait en Chine , en Inde , ou ailleurs ; ou la robotisation . Et c' est à ce moment-là qu' on fait des choix qui ne sont pas les bons . La robotisation , elle accompagne le travail . Mais si on ne fait pas les bonnes réformes et bien on supprimera tout , à terme . C' est pourquoi je supprimerai le CICE pour le remplacer par une baisse de charges , claire , durable , pour tous les types d ' entreprises . Pas pour faire des cadeaux . Pour défendre l' emploi . Pour défendre l' investissement dans mon pays . Et donc ce sera , pour tous les types d' entreprises , les petites , les grandes , les individuelles , les collectives , celles de l' économie sociale et solidaire comme celles qui ne paient pas l' impôt sur les sociétés : 6 points de charges en moins . Et ce , pour tous les salariés jusqu' à 2,5 SMIC . Mais en plus de ça , il faut aller plus loin . Les charges , c' est aussi et surtout un problème pour celles et ceux qui sont les plus jeunes , les moins qualifiés , qui peuvent parfois avoir une opportunité d' emploi , mais pour lesquels le SMIC , avec les charges , est trop important . Il y a aujourd'hui , sur notre territoire , beaucoup d' emplois détruits ou d' emplois qui ne sont pas créés à cause du niveau de charges qu' il y a encore au niveau du SMIC . Et donc , en plus de ces six points de charges , je supprimerai toutes les charges générales ( … ) . C' est une mesure que je n' avais pas jusqu' alors révélée qui fait que , pour tous les emplois au SMIC , il y aura 10 points de charges patronales en moins . C' est un investissement . C' est un investissement pour toutes les entreprises qui en ont besoin . Ces 4 points additionnels , et bien je les financerai en faisant successivement monter en charge cette fiscalité environnementale qui est prévue , et en particulier les convergences nécessaires . Parce que c' est comme ça que nous arriverons à faire progressivement muter notre économie . Alors pour libérer le travail il y a bien d' autres mesures , bien d' autres décisions et engagements que j' ai annoncés , et que je vais vous confirmer : nous augmenterons le pouvoir d' achat . Salariés , fonctionnaires , indépendants , professions libérales verront leur pouvoir d' achat augmenter compte tenu de la mesure de compensation que je prendrai sur la CSG . Tous ! Parce que je veux qu' en France , le travail paye mieux . Nous irons aussi plus loin sur la réforme des prud'hommes , indispensable , où je confirmerai le plafond censuré par le conseil constitutionnel , qui est absolument nécessaire pour donner de la visibilité . Je supprimerai le RSI . Je le supprimerai en faisant deux choses : d'abord en permettant un droit d' option pour tous les indépendants pour opter pour le régime général d' assurance maladie . Je vois que , on s' en est parlé ce midi ... je vous le rassure , je n' ai pas découvert cela ce midi , mais je savais que j' allais vous l' annoncer ce soir . Pourquoi ? Parce qu' aujourd'hui beaucoup d' indépendants qui payent moins de cotisations qu' au régime général découvrent qu' ils ont des risques . Ils tombent malades . Ils ont certains problèmes qui ne sont pas bien couverts par le RSI . Et à ce moment-là , ils se tournent vers d' autres garanties , dites Madelin , qui sont des régimes plus spécifiques . Et on leur demande à ce moment-là beaucoup plus que pour le régime général . Et ils découvrent qu' il faut faire des tests de santé , qu' on leur demande des charges nouvelles . Et ce qu' ils pensaient être un cadeau devient un cauchemar . Donc il faut qu' il y ait un droit d' option , parce que ça correspond à la réalité de la vie . Je supprimerai cette caisse qui gère trop peu , en fait , d' indépendants , et qui donc a des délais trop importants , a des ratés , des erreurs , et qui donc conduit à des tracasseries , parfois des traumatismes , pour beaucoup d' indépendants , parce que tous les indépendants qui travaillent en France doivent avoir les mêmes droits que l' ensemble des salariés du secteur privé ou que l' ensemble des fonctionnaires en terme de qualité des prestations , en terme de rapidité des remboursements . Et donc cette réforme , c' est cela . Alors pour penser printemps il faut libérer les énergies , le travail . Mais il faut aussi , dans le même mouvement , tenir nos équilibres , et faire plus pour ceux qui ont moins . C' est cela le deuxième pilier de ce penser printemps que je voulais partager avec vous ce soir . Faire plus pour ceux qui ont moins , ça veut dire : parce qu' on a produit davantage , parce qu' on a trouvé ces flexibilités , ces libertés , parce qu' on a ouvert des secteurs , parce qu' on a accepté d' avoir plus de flexibilité dans le travail , on doit aussi accepter de regarder les transformations en cours et les injustices qui sont là , qui se créent . On ne fait pas bien si on libère sans être juste . Le projet aujourd'hui porté par la droite conservatrice est un projet injuste parce qu' il libère … J' entends certains dire “ c' est un projet libéral ” . Non . Non . Le libéralisme politique comme économique , il libère , mais il libère les individus en regardant les inégalités de départ . Il ne libère pas pour que ceux qui ont déjà réussi réussissent mieux . Ça c' est du conservatisme . Par contre , quand on libère , quand on ouvre , quand on permet aux énergies de se déployer , il est indispensable , dans le même temps , de considérer que certaines et certains n' ont pas les mêmes chances , viennent d' une autre famille , d' un autre endroit , d' autres territoires , qu' ils n' ont pas les mêmes capacités à faire , parce qu' il y a des inégalités de départ . Et c' est pour cela que ce deuxième pilier dont je veux vous parler ce soir , ce “ faire plus pour ceux qui ont moins ” , ça n' est pas de la commisération . Ça n' est pas l' aumône - c' est insupportable . C' est le cœur de la promesse républicaine , celle de fraternité qu' on oublie trop souvent , mais c' est celle de l' égalité vraie . C' est celle qui consiste à regarder , non pas une égalité d' arrivée , qui est une promesse intenable . Je ne vais pas vous promettre ce soir , mes amis , que tout le monde à la fin , que tous et toutes , nous serons égaux , ça n' est pas vrai . Parce qu' il y en a qui sont plus grands , d' autres qui sont plus petits ; il y en a qui travailleront plus , d' autres moins ; il y en a qui sont talentueux , d' autres non ; il y en a qui auront de la chance , et d' autres qui en auront moins . Mais ce qui est insupportable c' est qu' avec les mêmes mérites , la même volonté , le même engagement , parce qu' ils seront nés dans une famille donnée , dans un territoire donné , ils n' auront pas pu réussir comme d' autres , ailleurs , c' est cela qui est intolérable aujourd'hui . Alors faire plus pour ceux qui ont moins , c' est penser ensemble nos transitions . C' est penser ensemble pour le chômage et la formation continue , des droits plus justes . Mais ce que je veux , c' est un chômage qui n' est plus seulement une assurance donnée à quelques uns ; des droits sociaux qui ne sont plus seulement une assurance donnée à quelques uns . Mais un système social où le chômage comme la maladie est un droit ouvert donné à ceux qui travaillent ou entreprennent , aux exploitants agricoles et qui n' ont pas le droit au chômage , ce qui est une injustice profonde . Aux commerçants , cher Francis , aux artisans , à toutes celles et ceux qui eux aussi prennent des risques et qui , s' ils ratent , peuvent tout perdre . C' est un élément de justice et d' efficacité . Mais en même temps , ce que nous allons faire , j' entends certains qui disent “ ce n' est pas finançable , c' est horrible ” - les gens dont je parle là , ce ne sont pas des . Il faut arrêter de penser ça . On ne va pas voir demain des artisans , des indépendants qui vont se mettre au chômage . Ils se mettent déjà pas en congé maladie . Non . Par contre , nous allons être justes en cas de faillite . Nous allons éviter - il faut bien le dire et le regarder en face - des suicides , parce que qu' est -ce qui se passe ? Sur nos territoires , ce sont des suicides , partout , dans ces catégories , parce qu' on laisse des Françaises et Français sans solution . Et nous allons le financer par une vraie politique de droits et de devoirs . Le chômage ce ne sera plus un droit acquis . J' en changerai le financement . Il ne sera plus payé par les charges salariales , mais par l' impôt . Donc ce sont des droits , mais en même temps des devoirs . Plus personne ne vous dira “ Moi j' ai droit au chômage donc je vais y rester encore . ” ; “ Moi j' ai droit au chômage donc l' offre d' emploi que vous me faites là , elle est bien ... mais elle serait mieux dans 3 mois . ” Je ne vous dis pas que c' est la généralité . Parce qu' être au chômage c' est une stigmatisation , c' est une douleur au quotidien . Mais aujourd'hui , parce que notre système est ainsi , et bien il y en a qui l' optimisent . Je déteste le discours porté par certains , parfois dans cette région , ce discours d' assistanat . Non . Il n' y a pas d' assistanat . Il y a des gens qui ne font pas leur devoir - bon , c' est une chose . Mais parler d' assistanat , c' est mettre tout le monde dans le même sac , c' est dire : “ le chômeur est un fainéant ” . Mais enfin ! Celui qui a vécu un choc industriel , une transformation profonde , dont l' entreprise s' est effondrée . Celui qui n' arrivait plus à rendre son lait , c' est un assisté ? Allons . Mais en même temps , ça donne le sentiment que tout se vaut . Par contre , l' idée que portent certains - les mêmes - qu' il faudrait réduire la durée des indemnités de chômage , elle ira plus vite vers l' assistanat . Parce que c' est l' idée que certains qui sont au chômage , si on leur réduisait les droits , iraient plus vite retrouver un emploi . La belle affaire ! Ils en trouvent sous le sabot d' un cheval ? Non , ils iraient plus vite vers les minimas sociaux . La clé , c' est un système de droits et de devoirs . Je veux que tous les travailleurs , quelle que soit leur situation , puissent avoir accès au chômage . Je veux que tous les travailleurs aient accès à la formation quand elle est nécessaire . Parce que demain , s' il y a un accident industriel dans un endroit , on ne peut pas laisser les Françaises et les Français dans le même secteur avec les mêmes qualifications . Il faudra les reformer . C' est une transformation , une révolution en profondeur de notre formation continue que je propose . Parce qu' il faudra , pour un an , deux ans , les former pour aller vers un autre secteur , parfois une autre région , pour pouvoir justement gagner leur vie . C' est cela les nouveaux droits que nous devons créer . On dépense plus de 35 milliards par an pour la formation continue . Et on forme toujours les mêmes . Ceux qui sont déjà bien formés , dans l' entreprise , qui ont des contrats longs . Ceux qui sont protégés . On ne forme pas les chômeurs . On ne forme pas ceux qui sont en intérim . On forme mal ceux qui vont avoir des transitions à venir . Donc ce sont les droits que je veux créer en redéployant notre organisation collective , en réorganisant en profondeur . Mais à côté de ça , il y aura des devoirs . Quand vous avez eu une formation , ou quand vous êtes dans une région où il y a une offre d' emploi , chez l' exploitant agricole dans le café-restaurant dont je parlais tout à l' heure , et bien vous ne pouvez pas refuser une offre d' emploi décente . C' est cela , avoir une vraie politique républicaine . Non pas dire “ tous ceux qui sont au chômage sont des assistés ” , mais d' expliquer qu' il y a des droits et des devoirs . Et donc derrière ces devoirs il y aura une obligation , et donc des contrôles . Certains seront amenés à accepter des offres d' emploi qui ne leur plaisent pas , qui n' étaient pas leur premier choix , mais qui correspond à ce qu' ils savent faire , à des besoins , parce qu' ils auront eu en même temps de vrais droits . C' est cela la République , nous l' avons oublié . Et donc ni facilité , ni laxisme , ni discours de stigmatisation , ni injustice ; le progressisme ! Faire plus pour ceux qui ont moins c' est aussi avoir une vraie politique pour l' école , j' en reparlerai plus en détail , mais on n' est pas dans la même situation quand on rentre en CP Auvergne ou dans les beaux quartiers de Clermont . C' est la réalité . Or aujourd'hui on a la même politique éducative . On a des petites différences : on enlève deux , trois élèves , on paye quelques dizaines d' euros , quelques centaines d' euros . Un enfant qui arrive dans un quartier en difficulté en CP , et bien , il a parfois 200 mots de vocabulaire . Il a parfois des parents qui savent mal le français . Il a parfois jamais été devant un pédiatre , et donc on ne sait pas s' il sait lire au tableau . On ne sait pas s' il entend très bien . Il a des parents qui ne s' occupent pas bien de lui parfois . Il y a une détresse sociale . Celui qui arrive dans les beaux quartiers , c' est formidable , c' est notre fierté , c' est la grandeur de la France . Mais , il a eu des milliers d' heures avec ses parents . Il sait déjà bien souvent lire , écrire et compter . Il a été plusieurs Lui , à l' école ça va bien se passer . Je peux déjà vous dire aujourd'hui , à 90 % près , où sont les jeunes qui iront en prison , qui auront le chômage de longue durée , qui seront des décrocheurs et qu' on retrouvera en CM2 , qui ne sauront pas lire , écrire , compter . 20 % des élèves de CM2 en France ne savent pas bien lire , écrire , compter . Mais ils sont à 80 % dans ces quartiers . Et donc oui , là il faut faire plus pour ceux qui ont moins . Mais beaucoup plus . Investir massivement . Accepter d' avoir des enseignants plus expérimentés beaucoup mieux payés et d' avoir vraiment deux fois moins d' élèves par classe . C' est cela que je détaillerai dans les prochaines semaines . La vraie injustice , c' est celle qui existe aussi en santé . Parce qu' il y a l' inégalité sociale , l' inégalité territoriale qui mine notre territoire et une partie de notre population . La santé ça n' est pas qu' un coût . On doit continuer à baisser le coût de la santé en France , c' est évident . Mais le faire de manière intelligente : avec le territoire et les professionnels , le faire de manière concrète , sur le terrain . La santé ce n' est pas la même , ici à Clermont , et à Paris . Ça n' est pas la même dans le Cantal , à Rodez . Hier à Nevers il n' y a pas un service où on ne m' a pas dit “ Nous il manque trois postes , quatre postes . En télé-médecine , cela va nous aider à faire les premiers soins . &quot; Je ne suis pas sûr qu' il faille supprimer des postes d' infirmières , d' aides-soignantes ... Par contre , ils ne sont pas bien répartis sur le territoire . Et donc , nous avons en santé des défis . Le premier : celui de la prévention . Parce que si on veut baisser les coûts , les gens doivent être moins malades . Ce que je dis est du bon sens , on l' a oublié . La France soigne bien , elle prévient mal . Et donc en la matière j' ai pris plusieurs engagements hier , je les rappellerai . Deux , ce soir , que je veux rappeler : Le premier : je veux que tous les étudiants dans les professions de santé aient ce que j' appelle un service sanitaire de 3 mois . Qu' est -ce que ça veut dire ? Que tous les étudiants infirmiers , dentistes , médecins , kinés , aient trois mois de leurs études prévues pour faire de la prévention . Ils seront formés , une semaine à deux semaines , et ils iront à l' école , dans les lieux de travail , dans les maisons de retraite , pour faire de la prévention contre le tabac , contre l' alcoolisme . Ils iront regarder à l' école si les enfants , justement , voient bien au tableau , s' il n' y a pas de la myopie qui n' est pas détectée , s' il n' y a pas des problèmes de surdité , qui ne sont pas détectés chez les plus âgés . C' est fondamental , la prévention . Ça n' a aucun coût , ce sera dans la maquette pédagogique , et c' est une armée , chaque année , de 40 000 jeunes professionnels que nous déploierons partout sur le territoire . La deuxième mesure en matière de prévention , c' est que je veux que dans les 5 années , on couvre à 100 % les lunettes , les dents et les prothèses auditives . Je vais vous dire pourquoi et comment on le finance . Pourquoi ? Parce qu' un enfant dont les parents ne peuvent pas lui payer ses lunettes , ce qui existe aujourd'hui , c' est un enfant qui ne peut pas poursuivre ses études . Parce qu' un chômeur qui ne peut pas soigner ses dents , qui a les dents gâtées - pardon de parler de choses triviales , mais enfin c' est ça notre quotidien , c' est pas des belles et grandes idées , toujours , c' est le mariage des deux - s' il a des dents gâtées et qu' il ne se les soigne pas , il ne décrochera pas l' embauche , on le sait bien . L' employeur le verra , il dira “ il a une drôle de tête ” . C' est comme ça , c' est d' une cruauté infinie . La personne âgée qui ne peut pas se payer l' appareil auditif qui coûte très cher , c' est quelqu'un qui continuera à entendre mal , qui s' accommodera de cela progressivement . Mais elle ira beaucoup plus vite vers la démence , l' isolement , la grande dépendance qui nous coûte beaucoup plus cher . Et donc vous voyez que si on traite mal ces sujets , mais comme tous les sujets en matière de prévention , et bien ça coûte in fine beaucoup plus cher à la société , beaucoup plus cher à notre système de santé . C' est fondamental . Alors , comment je vais le financer ? Et bien , par un travail entre la sécurité sociale et les complémentaires . Un travail pour justement étendre cette couverture pour tous , mais pour aussi baisser les prix . Parce qu' aujourd'hui , c' est parfois 3 fois plus cher en France que chez les voisins d' avoir des prothèses dentaires . C' est 3 fois plus cher que chez les voisins d' avoir des prothèses auditives . C' est ça la réalité ! Et donc , quand je prends cet engagement , c' est un engagement aussi d' une plus grande concurrence , d' une liberté accrue , d' une transparence tarifaire , d' une pression mise sur certains pour qu' on puisse couvrir tout le monde , au bon prix . Et puis , le défi de la santé , c' est aussi celui de la ruralité . Je veux vous dire avant de conclure quelques mots sur ce beau sujet , sur ce grand sujet , qui est essentiel lorsqu' on veut faire plus pour ceux qui ont moins . Parler de ruralité , ça n' est pas dire : “ On va faire plaisir à quelques-uns , ils sont perdus , parce que c' est des territoires reculés de la République et qu' il faut en quelque sorte , de la même façon parfois que certains ont abordé les territoires ultra-marins , en disant : On va faire notre devoir , on va s' acquitter de ce qu' on leur doit puisqu'ils se plaignent tant ” . Ça n' est pas ça parler de la ruralité . C' est tenir la promesse que j' évoquais tout à l' heure . C' est de dire : la ruralité , oui , ce sont des paysages formidables , ça fait plaisir aux métropolitains quand ils vont en weekend , ce sont des territoires qu' on aime retrouver pour les vacances , pour quelques jours . Mais enfin ! Ce sont surtout des territoires , qui veulent pouvoir vivre , qui veulent pouvoir réussir , qui veulent pouvoir avancer , et qui en ont le droit ! Et donc , faire plus pour la ruralité , ça n' est pas faire l' aumône . Faire plus pour la ruralité , c' est de donner les moyens d' être libres , d' être pleinement autonome . C' est faire le devoir républicain , parce qu' il n' y a pas la même égalité d' accès , pas la même égalité de départ , lorsqu' on dans la ruralité que lorsqu' on est en métropole , c' est cela la réalité . Ça veut dire qu' en matière de santé , il faut continuer à développer les maisons de santé pluriprofessionnelles , aider à attirer les professionnels sur le terrain en investissant , aux côtés des collectivités , en rendant la ruralité attractive . Pas en faisant des plans d' en haut , non . J' étais tout à l' heure dans l' Allier , à Saint-Pourçain , une formidable maison de santé pluridisciplinaire . Ils l' ont fait ensemble , ce sont les professionnels de santé qui l' ont créée : des kinésithérapeutes , avec des généralistes , des sages-femmes , avec des infirmières . Et ils l' ont fait en étant aidés par les collectivités locales . L' Etat aide un peu , mais il doit aider davantage . Mais , il ne faut pas faire un grand plan pour aider les maisons de santé . Il faut faire un grand plan de libération des territoires , pour que sur le terrain , les collectivités territoriales , accompagnées par les professionnels de santé et l' Etat , puissent développer cette offre , redevenir attractives , avoir des bourses d' installation et faire revenir les professionnels ! Ça veut dire aussi : permettre , dans la ruralité , à toutes celles et ceux qui entreprennent , d' entreprendre plus facilement . Ça veut dire : avoir une vraie politique en matière de foncier , pour les jeunes exploitants agricoles et les aider à s' installer . Parce qu' il y a une relève qui est là , qui veut aller de l' avant , parce qu' il y a une jeune agriculture , parce qu' il y a une jeunesse agricole . Elle est fière , et je veux l' accompagner . J' en ai assez que l' on parle de l' agriculture comme une chose qui est en train de tomber . Non ! C' est une part vibrante de nos territoires et de notre économie , qui veut réussir et aller de l' avant , et nous devons l' accompagner . Ça veut dire , des aides à l' installation et au foncier . Ça veut dire , une vraie politique de prix . Ce que demandent les agriculteurs , ce n' est pas d' être aidés ! C' est de vivre de leur travail ! Et donc en la matière , oui là , il faut remettre de la régulation . Je l' assume . Il faut remettre des contrats de filières , des vrais contrats de filières . Parce que , quel est le problème de l' agriculture ? C' est que vous avez quelques dizaines de milliers de producteurs pour lesquels c' est de plus en plus difficile , quelques rares centrales d' achat et distributeurs , et 60 millions de consommateurs . Qui fait le pouvoir ? Qui prend la décision ? Qui décide des prix ? Je ne vous fais pas de dessin . Et donc , et certains d'ailleurs dans la distribution sont en train de le faire , et je veux les saluer , par un risque de responsabilité : les fonds d' achat local , ils valorisent des contrats de filière . Et il ne faut pas tomber là dans des discours trop rapides , ou qui attaquent . Mais on doit valoriser , développer , ces relations de filières . Valoriser ces distributeurs , qui font justement de la vente de proximité , qui acceptent de fixer les bons prix , qui ne sont pas forcément les plus bas , en éduquant le consommateur , en expliquant que pour avoir des bons produits , il faut payer le travail qu' il y a derrière . Il faut payer la nourriture des bêtes , il faut payer l' entretien des terres . Et donc il faut accepter de rentrer dans ce contrat pour que les prix soient justes . Et donc nous généraliserons ces contrats de filières , ces bonnes pratiques . Nous généraliserons aussi , et nous encouragerons , ce que j' ai vu tout à l' heure dans vos Halles de Clermont : le circuit court . Je reviendrai dans les prochaines semaines sur ce sujet . Ça n' est pas la panacée sur tout , quand on doit exporter , quand on a des filières d' excellence . Mais enfin , c' est parfois la réponse . Et puis nous devons aider nos exploitants agricoles , à se diversifier , à avoir d' autres sources de revenus : dans l' énergie , par la mécanisation , ou par d' autres innovations , en développant d' autres activités , de valorisation du patrimoine - ils le font aussi . Et le patrimoine rural ne serait plus là s' il n' y avait pas les agriculteurs , des paysans , au fond - parce qu' on emploie toujours aujourd'hui des circonlocutions . Des exploitants agricoles , ce sont des paysans . C' est pas une insulte , “ paysan ” . C' est un beau mot . C' est être dans un pays , mais c' est aussi le faire , le transformer , le modeler , le façonner . Il n' y aurait plus de pays , s' il n' y avait pas de paysans . Et donc pour les aider à réussir , nous devons aussi les aider à investir . Ce sont les mesures de baisse du coût du travail que j' évoquais , et nous devons continuer , secteur par secteur , à aider les paysans à investir pour valoriser leur produit . La solution , c' est d' aller vers l' aval , comme on dit . C'est-à-dire , de ne plus vendre simplement son lait , mais de le transformer . On l' a vu ensemble , dans le Cantal , cher Jacques , il y a quelques semaines . Un formidable jeune exploitant ; il ne gagnerait pas sa vie , ce serait déjà fini , si il vendait son lait à la coopérative , il ne le rentabiliserait pas . Mais il a pu investir , et il fait du fromage . Et celui-là il le vend mieux . Et donc nous devons , partout , aider les éleveurs , qui aujourd'hui font du broutard et n' arrivent pas à bien le valoriser , à aller plus dans ces initiatives formidables , qui sont prises ici d'ailleurs - qui consistent , non plus à envoyer en Italie les broutards se faire engraisser pour ensuite payer plus cher le lait ou la mozzarella , mais en faire une ici , avec la même bête ! Le valoriser sur le terrain . Ce sont de formidables initiatives . Il faut développer l' aval , valoriser le produit , et pour cela aider nos agriculteurs , nos paysans à investir . Faire plus pour ceux qui ont moins dans la ruralité , c' est aider les commerçants . Ce que tu fais , cher Francis , est formidable en la matière , en Lozère . Les aider à se regrouper , à mieux s' organiser , pour faire face aux défis parfois de la désertification . C' est avoir une couverture numérique à la hauteur . Tout ce que je vous dis depuis tout à l' heure n' existe pas , si on n' a pas la fibre , ou la 3G au moins . Les maisons de santé avec la télé-médecine , formidable innovation , bon courage si vous ne captez pas . Et c' est le cas dans beaucoup de territoires . En tant que ministre , j' ai fait le maximum , je vous assure , on s' est battus à quelques-uns : pour développer la fibre , pour mettre encore plus de contraintes sur les opérateurs . On a fait des choses très compliquées , développer dans les centres-bourgs , ils seront tous couverts , en 2G -ce qui n' est pas suffisant , demain en 3G , d' ici l' été . Pour développer les antennes relais dans 1700 centres de priorité , au-delà des centres bourgs . Mais il faut aller plus loin . C' est l' investissement en infrastructures du XXIe siècle , et donc je vous le dit , je présenterai dans quelques semaines , un plan ambitieux de couverture du numérique , qui ira plus loin que celui en cours , en conduisant l' Etat à prendre ses responsabilités , et donc à investir lui-même à marche forcée partout où les opérateurs ne le feront pas . Parce que ce sont les infrastructures essentielles : ce sont aussi les routes , les accès . Combien se sont battus pour le TGV ou l' aéroport de proximité . Vous savez , je viens d' une ville qui ne s' est toujours pas remise de ne pas avoir eu le TGV . Avec mon épouse , nous venons d' Amiens . Je peux vous dire , j' y étais encore il y a quelques jours , on vous parle encore - et je le sais , c' est pour ça que j' en parle , on a vécu le même traumatisme - , il y a encore certains qui parlent de leurs bagarres d' il y a vingt ans . Parce que le TGV , à l' époque , il y avait des réalités politiques locales , il a été fait directement de Paris à Lille . Et bien Amiens a beaucoup perdu : de l' accès , des entreprises , des services . Et donc oui , les infrastructures de transport c' est aussi important . Alors je ne vais pas vous promettre un grand plan à 300 milliards . Parce que vous savez , les grands plans à 300 milliards d' infrastructures , certains grands Présidents de la République et Premiers ministres les ont promis . C' était en 2010-2011 , de mémoire . Les mêmes vous disaient que la France était en faillite , vous me direz . Mais enfin ces grands plans , c' était du vent , ils n' ont jamais été faits . Je ne ferai pas de promesses avec de l' argent que je n' ai pas . Par contre , ce que nous allons développer , là aussi avec les territoires dans un dialogue responsable , c' est que chaque territoire , bassin d' activité , chaque ville-centre , soit raccrochée par des infrastructures dignes du XXIème siècle , à la métropole ou au centre d' activités important pour elle . Ce ne sont pas des grands plans napoléoniens , mais c' est ça un plan efficace du XXIème siècle . Ça veut dire , que si le cœur de votre activité est dans une métropole qui est à 200 km , vous ne pouvez pas avoir 2 ou 3 heures de route , vous devez avoir un accès rapide , parce que la vie de vos entreprises , de vos concitoyens en dépend , parce que c' est le développement économique , parce que c' est votre santé , parce que c' est ça l' efficacité . C' est ce plan d' infrastructures concret avec les territoires que je veux faire et que nous porterons . Faire plus pour ceux qui ont moins , c' est aussi faire plus pour nos finances locales . Je sais qu' il y a beaucoup de maires , beaucoup d' élus locaux ce soir . Je vous le dis , en responsabilité , comme je l' ai dit pour la santé . Nous continuerons à faire des économies , c' est indispensable , je les présenterai dans les prochaines semaines . Je ferai des économies sur la sphère publique , comme on dit joliment , c' est à dire sur l' Etat , sur la santé , et sur les collectivités locales . Et ces économies , on devra les faire d'abord en faisant quelques réformes en profondeur , en faisant des réorganisations qui font de vraies économies , pas en passant le coup de rabot sur tout le monde . Parce que là il y a une victime : l' investissement local et la modernisation de nos territoires . Mais pas non plus en demandant le même effort à tout le monde . Quand vous êtes une métropole , un territoire d' attractivité , vous avez des revenus fiscaux , vous avez des entreprises qui vous paient de la taxe professionnelle , vous avez des gens qui ont de l' argent et qui peuvent payer plus facilement les impôts locaux . Lorsque vous êtes dans la ruralité , vous cumulez aujourd'hui toutes les difficultés . Et donc bien souvent , les efforts qu' on a demandés aux communes se sont traduits par quoi ? Des augmentations d' impôts locaux , fatalement . Ce qui fait que dans nombre de nos villes , dans nombre de nos villages et de nos villes des territoires , les charges que vous payez au titre de la taxe d' habitation ou de la taxe foncière sont plus importantes que celles que vous payez dans les grandes villes . Et comment derrière aller expliquer : “ il faut attirer des médecins ! Il faut attirer des entreprises ! ” Ils comparent aussi . Et donc pour cela , nous ne devrons pas demander le même effort à tous les types de collectivités . Ça n' est pas un cadeau , c' est un droit réparé . Ça n' est pas une obole , c' est notre devoir fait pour donner à chacune et chacun les mêmes armes et la possibilité de devenir à nouveau attractifs et de réussir . Parce qu' il y a partout en France , une volonté de faire , une innovation , une volonté d ' aller de l' avant . Et donc faire plus pour ceux qui ont moins , dans la ruralité comme dans les quartiers , partout en France , et bien c' est donner la possibilité aux énergies qui sont là , bloquées , assignées à résidence , parfois d'ailleurs pointées du doigt , et bien d' avoir une égalité d' accès , comme le reste du territoire , de pouvoir elles aussi réussir enfin , avoir accès à cette liberté dont je parlais tout à l' heure . C' est cela ce que je veux faire avec vous . Alors mes amis , pour conclure , vous l' avez compris , cet engagement , ces propositions que nous avons commencé à décliner , et que je déclinerai dans les jours et les semaines à venir sur le terrain , elles refusent le diktat des partis politiques actuels qui voudraient nous livrer à une forme d' espace fermé . Les uns nous disent : “ vous allez rentrer dans le XXIème siècle . Ce sera dur . Je vais vous demander des efforts . Surtout à ceux qui en ont déjà fait . Il faudra réussir . Surtout pour ceux qui ont déjà réussi . ” Et qui portent un projet conservateur . D' autres , ils vous disent : “ ça va être formidable . Avec nous , il y aura des cadeaux . Il y aura de la répartition , de la justice . Nous allons défendre , la main sur le cœur , un modèle social . Quant à savoir comment on produit , nous reviendrons demain . ” Ça ne marche pas . C' est pas du patriotisme , vous savez , de vouloir en France produire quand on ne propose que de redistribuer . Ça n' est pas de l' esprit de responsabilité ou du modèle républicain , quand on propose en France , en effet , de nouveaux droits et créances sans expliquer comment on les finance . Ça n' est pas de la justice , quand on n' a pas un vrai discours sur la liberté et la capacité à faire . Et la troisième voie qui nous est proposée , c' est celle de la haine , du repli . C' est celle portée par les extrêmes . C' est celle qui propose aujourd'hui aux Français de sortir de l' Europe , de sortir du monde , de se fracturer dans la haine . Mais celle -ci progresse , aussi . Elle progresse , de l' irresponsabilité des deux premières . Elle progresse du fait qu' on ne veut plus essayer de faire réussir le pays en donnant une place à chacun . Elle progresse du fait qu' on voudrait faire croire à tout le monde qu' on peut penser le progrès comme il y a 30 ans . Notre responsabilité , mes amis , est immense . Immense . Car nous sommes le seul rempart à l' esprit de haine et aux extrêmes . Nous sommes le seul rempart parce que nous sommes l' ambition responsable . Parce que nous regardons en face la dignité des personnes , que nous voulons faire réussir et protéger . Parce que nous regardons en face la force des territoires . Parce que nous avons conçu ce projet pour le pays ensemble . S' il y a cette vague qui monte , c' est parce que notre projet c' est vous qui l' avez porté . C' est parce que du premier jour , je suis allé vers vous , nous sommes allés vers les Françaises et les Français pour écouter , pour construire . Parce que chaque jour nos militants , nos adhérents , vous , vous construisez un projet . En vous réunissant , en réfléchissant . Parce que nous écoutons pour faire . Parce que le projet que nous sommes en train de construire , c' est un projet pour prendre le pouvoir , mais le prendre pour le rendre . Pour le rendre aux territoires , pour le rendre aux citoyens , pour leur permettre de faire . Alors je sais que le quotidien est parfois difficile . Je sais que la colère est parfois là . Je sais que , pour certains , parfois le désespoir peut s' installer . La responsabilité qui est la nôtre , celle des progressistes , c' est de porter pour eux cette espérance . A quelques kilomètres d' ici , en 1941 , après un jugement , un jeune Français a réussi à s' évader . Pas s' évader pour aller se protéger ou se cacher . Il s' est évadé pour rejoindre le Général de Gaulle , pour se battre pour son pays . Il s' est évadé , au risque de sa vie , pour que nous puissions ce soir être là , et encore proposer . Porter avec fierté ce qu' est la France . Il s' est évadé , et il a eu ces mots , quelque temps après , en parlant de son jugement , de son emprisonnement , et en disant : “ Ce n' est pas la justice de la France ; c' était celle de Hitler . Ne désespérez pas de la France . ” C' est ce qu' il a écrit . Ce jeune homme courageux , c' était Pierre Mendès France . Alors mes amis … ce soir … à Clermont , nous n' allons pas désespérer de la France . Parce que nous sommes là . Parce que vous êtes cette force debout et qui marche . Alors en vous renouvelant mes vœux , pour cette année qui commence et cette année de combat ensemble , oui , je vous dis et je vous redis , comme l' écrivait Alain , de penser printemps , de vouloir printemps , parce que le printemps sera à nous mes amis . Parce que le printemps nous allons le conquérir , pour nos idées , pour notre pays ! Parce que ce printemps sera progressiste ! Parce que ce printemps sera celui de notre conquête ! Vive Clermont ! Vive la République ! Et vive la France ! ",
    "Bonjour Lille ! Bonjour le Nord , bonjour les Hauts-de-France ! Vous m' avez l' air formidablement enthousiastes . Je vois tellement de visages amis que je voudrais embrasser . Vous êtes ce soir , ici , au Zénith de Lille , nous sommes ce soir cinq mille . Cinq mille avec nos histoires , nos parcours , nos différences et j' y tiens aussi parce que je les respecte , nos cheminements , nos imaginaires . Mais nous sommes cinq mille à vouloir porter un projet pour le pays malgré tout cela , pour la France , c' est cela qui nous rassemble ce soir ! Il y a un peu plus de neuf mois , nous avons fait un rêve , ensemble , nous n' étions peut-être pas cinq mille ce jour là , celui de lancer “ En Marche ! ” et de ne pas nous arrêter aux clivages , à tout ce que jusqu' ici bloquait le pays , aux vieilles habitudes , aux corporatismes . Et ce rêve , vous étiez des millions à le faire . Et mois après mois , cette vague a grandi , ce mouvement s' est renforcé , et vous êtes là ce soir parce que ce rêve n' était pas une folie , il était une ambition extrême pour le pays , pour notre pays , celle justement de travailler ensemble pour un projet qui nous unit plutôt que de rester divisé sur nos petites différences . Il y a quelques semaines , un reportage passait sur le mouvement dans le Nord , En Marche ! Et le journaliste demandait à quelques-uns , toutes celles et ceux qui étaient je crois autour de Christophe , s' ils avaient voté jusqu' ici à gauche , et la moitié de la salle a levé la main . Puis , le même journaliste a demandé &laquo; qui ici avait voté la dernière fois à droite ? &raquo; , et la moitié de la salle a levé la main . Ça ne veut pas dire que nous pensons tous la même chose sur tout mais ça veut dire que nous sommes tous et toutes las d' entendre les chicayas , d' être pris dans cette comédie humaine des petits jeux et que tous et toutes , ce qui nous rassemble c' est cette ambition de faire , de faire ensemble , de faire entrer en effet la France dans ce siècle nouveau et de donner à chacune et chacun sa place , c' est cela “ En Marche ! ” dans ses différences , dans sa différence , c' est notre projet . Alors je suis heureux aujourd'hui d' être dans nos Hauts-de-France , parce que moi aussi , vous savez , je suis un enfant des Hauts-de-France . Ce qu' on appelait naguère encore la Picardie . Cette terre , votre terre , notre terre , c' est une Terre-Monde . C' est un endroit où le monde s' est fait . C' est un endroit où on a fait les guerres , où on a fait le commerce au coeur de l' Europe , où on a produit . C' est ici que la révolution industrielle pour partie s' est jouée , que les mines se sont ouvertes , et on fait travailler tant et tant , d' hommes et d' enfants pendant longtemps , de femmes . C' est ici que l' acier s' est développé , que le textile a grandi . Ces terres , elles ont été au coeur du monde nouveau au XIXème siècle , et bien avant . Ces terres ont toujours été des terres d' ambition , des terres d' avenir . Ces terres des Hauts-de-France ce sont aussi mes amis des terres de combats , du combat social , celui mené dès le début et qui s' est noué ici , après Courrières où 1099 ouvriers sont morts . Des terres de batailles , où pied à pied , mois après mois , la justice sociale s' est créée dans notre pays et s' est propagée en Europe parce que des femmes et des hommes se sont battus pour construire des droits nouveaux pour que le monde soit plus juste . Des terres de combats politiques aussi , et je veux ici avoir une pensée pour Roger SALENGRO . Vous pouvez l' applaudir . Grand maire de Lille qui s' est toujours battu contre les extrêmes , l' extrême droite , les ligues . Il en est mort . Ces terres , elles ont toujours été dans le monde et elles ont toujours été dans le combat . Elles sont au coeur du défi français parce que oui notre pays aujourd'hui , il est au monde , il est dans le monde et il est face à des combats nouveaux que nous allons inventer ensemble . Des combats qui nous feront entrer dans le XXIème siècle nouveau . … Alors oui mes amis , ce que je suis venu vous dire ce soir , c' est que nous devons être à la hauteur de cette Histoire collective . Jamais , nous ne devons accepter celles et ceux qui bafouant cette Histoire voudraient nous faire croire que notre avenir se construit dans l' enfermement , dans le repli , dans l' oubli du monde . Jamais ! Jamais nous ne devons accepter le discours de celles et ceux qui voudraient nous expliquer qu' il faut accepter le monde tel qu' il vient , s' adapter aux injustices , aux incohérences , à une mondialisation parfois devenue folle . Jamais ! Nous sommes depuis deux jours parmi vous au coeur de ces Hauts-de-France et nous y avons vu les fractures profondes qui touchent notre pays parce qu' ici se joue comme un résumé du défi français . Il y a Lille , la triomphante , la conquérante , la métropole capitale de la culture . Du numérique , du commerce , de l' industrie du XXIème siècle . Lille n' a pas peur de la mondialisation , elle y vit . Elle est au coeur de l' Europe , elle voyage . Elle a un jeunesse qui étreint des continents et ce monde nouveau . Et je veux ici rendre hommage à Martine AUBRY qui n' est pas avec nous ce soir mais à laquelle je souhaite le meilleur rétablissement possible . Et je veux aussi rendre hommage à Pierre Mauroy qui a accompagné cette transformation . Ce sont , à côté de cette métropole de Lille , des terres qui se transforment , qui ont souffert de la désindustrialisation , qui ont à un moment été frappées , et qui se redressent , se réinventent avec courage . C' est Valenciennes , cher Laurent Degallaix , par le travail que tu conduis aujourd'hui et qui a été mené par Jean-Louis BORLOO il y a maintenant près de vingt ans . Ce sont des terres industrielles qui résistent comme à Arques , chère Caroline Saudemont , et nous l' avons vécu . Des terres qui se battent contre les injustices , qui innovent , qui inventent . Nous étions ce matin , cher Frédéric Marchand , à Hellemmes , et nous l' avons vu ensemble . Et puis , ce sont en même temps des terres qui ont été sacrifiées par la mondialisation . J' étais hier dans le bassin minier - vous pouvez l' applaudir - , à Lens , puis Noeux-les- Mines , puis Hénin-Beaumont . Des terres où ce que ressentent nos concitoyens , c' est d' avoir été , en quelque sorte , la chair à canon de la mondialisation . D' avoir , pendant tant et tant de décennies , travaillé dur pour produire , puis d' avoir été balayés lorsqu' on leur a soudain expliqué qu' on allait fermer la mine , qu' on allait arrêter la sidérurgie un peu plus loin . Alors , là , le chômage s' est installé , et n' est plus jamais parti . Alors , là , le désespoir s' est installé , la grande misère . Dans ces terres , que ça plaise ou non à certains , on vit beaucoup plus dur , et on vit moins longtemps , et c' est une injustice qu' il faut regarder en face , oui ! Parce que si nous en sommes arrivés là , c' est que , pendant trop longtemps , nous n' avons pas voulu regarder ce monde en train de changer . J' étais hier , avec beaucoup de commerçants , d' artisans , à Lens , et il y avait Gervais Martel , que vous connaissez bien . Je vois qu' il y a des supporters de Lens , n' ayez pas d' ennuis avec les supporters du LOSC . Il disait une chose très vraie - son père était mineur- , il disait : &laquo; mon père qui est mort avant que tout cela ne ferme me disait dès le début des années soixante-dix : la mine ça ne pourra pas durer &raquo; . Il le savait très bien . Mais nous n' avons pas voulu voir . Les dirigeants n' ont pas voulu voir . Ils n' ont pas voulu raconter le monde , tel qu' il était en train de se faire , en train de bouger , parce que , oui , le changement fait peur , parce que le changement menace des emplois , des habitudes , mais à ne pas vouloir voir , expliquer , préparer , eh bien tout s' est passé soudainement et a été d' un seul coup balayé . Alors ce que je ne veux pas aujourd'hui , pour mon pays , c' est refuser de voir , voir les risques parfois terrifiants , voir les changements qui font peur et qui tétanisent . Je veux qu' ensemble nous les nommions . Je veux qu' ensemble , lorsqu' on se rend dans ce bassin minier , on nomme les injustices , on nomme tout ce que nous n' avons pas suffisamment fait , on nomme en même temps les formidables batailles qui sont conduites sur le terrain par certains élus , par les associations , qu' on nomme la générosité des femmes et des hommes qui y vivent , qui aiment le travail , qui veulent du travail , qui ne demandent pas à être assistés , qui veulent aller de l' avant ! Et donc ce que je suis venu vous dire ce soir , c' est simple : nous sommes , toutes et tous ici , des enracinés , nous vivons en France , nous vivons sur ces terres , et nous partageons le même destin , et nous ne pouvons pas accepter qu' il y ait des terres qui réussissent formidablement bien et des terres qui soient condamnées à la stagnation ou au recul Parce que nous sommes des enracinés , et , en étant enracinés , nous partageons ce destin commun . Et l' enracinement , comme le disait formidablement Simone WEIL , ce sont des devoirs : le devoir de regarder , de nommer , de s' engager . Et ne vous trompez pas , que toutes celles et ceux qui réussissent , et que je veux aider , n' oublient jamais d' où ils viennent , où ils sont nés , où ils vivent , où iront grandir leurs enfants et leurs petits-enfants . La responsabilité de vous qui êtes dans la salle et qui réussissez , c' est d' être là , enracinés , et de vous engager pour les autres . La responsabilité de celles et ceux qui doutent , c' est de faire confiance , c' est de ne pas croire de trop nombreux discours qui voudraient vous faire croire que haïr celui qui réussit est la meilleure façon de se sortir du dénuement : ça n' est pas vrai ; qu' avoir peur de l' autre c' est au final se protéger : ça n' est pas vrai non plus . Nous avons tous et toutes une responsabilité commune : celle des enracinés . Alors , ce que je suis venu vous dire ce soir , c' est que le projet que nous allons porter , pour ne pas commettre les erreurs d' hier , c' est un projet lucide , c' est celui de l' émancipation . L' émancipation , c' est la liberté , encouragée , pour celles et ceux qui réussissent , qui veulent entreprendre , faire , oser , tenter , elle est essentielle . Et l' émancipation c' est aussi les opportunités redonnées à celles et ceux qui sont parfois bloqués , assignés à résidence , à qui la société ne propose plus rien , si ce n' est des aides , et à laquelle , en quelque sorte , on s' est habitué . L' émancipation c' est un projet commun , c' est notre responsabilité commune . L' émancipation c' est ce que nous allons ensemble porter pour toutes et tous dans notre pays ! Alors l' émancipation , mes amis , elle se construit d'abord en regardant le monde en face . Parce que nous sommes ce monde , comme je vous le disais . Nous n' existons pas comme une enclave . Notre destin est universel . Alors oui , ce monde est plein d' insécurités . Vous êtes ici aux portes de quartiers qui ont parfois fait trop parler d' eux , qui ont porté cette insécurité , qui ont nourri l' angoisse , en Belgique comme en France . Elle est là . Le mal est là , dans notre société , et il faut le regarder en face . Cette insécurité , elle est internationale , parce que c' est l' hydre de ce terrorisme nouveau , de ce projet de destruction de l' autre , de ce djihadisme totalitaire . Je ne parle pas de guerre pour ma part . La guerre suppose d' avoir un ennemi constitué et un ennemi face à soi . C' est une lutte , que nous conduisons , une lutte radicale , profonde , une bataille militaire sur certains théâtres d' opérations , mais une lutte politique , culturelle , en notre société même , et nous devons la regarder en face . Mais nous devons la regarder sans pour autant nous diviser . Le piège que nous tendent nos ennemis , c' est celui de la haine , de la fracture , du repli sur soi . Qu' attendent celles et ceux qui nous attaquent ? Que nous ayons peur , que nous nous repliions , que les quelques-uns qui tuent nos frères nous fassent nous diviser entre nous et conduisent à un malheur encore plus profond . La réponse , elle doit être claire , totale , extrême . D'abord la lutte , partout où nos intérêts se jouent , partout où la sécurité de nos concitoyens est à l' œuvre , nous devons là intervenir , mais intervenir avec intelligence , être conscients d' un ordre du monde qui est en train de profondément changer , où des puissances nouvelles émergent , avec lesquelles nous devons avoir un dialogue exigeant , intervenir partout , sur le plan diplomatique et éventuellement militaire , en construisant une diplomatie et un projet de paix . Notre sécurité , mes amis , elle ne sera pas assurée par une France qui attaquera partout , tout le temps , pas plus que notre sécurité n' a été assurée lorsque les Etats Unis d' Amérique ont décidé de réagir aux attentats de 2001 par une attaque immédiate qui a conduit à quoi ? À la déstabilisation profonde de toute une région , à la naissance de formes nouvelles d' organisations terroristes . Cela , nous ne devons pas le reproduire . La politique internationale que je veux conduire pour notre émancipation vraie et donc pour notre sécurité , c' est celle qui respectera l' équilibre , qui préservera l' indépendance française , qui assurera la stabilité des Etats et qui , partout , défendra nos valeurs et nos principes . Pour construire la paix , nous avons donc besoin de retrouver le fil de cette diplomatie d' indépendance française , celui qui s‘est construit dans l' histoire même de notre pays , qui fut porté par le Général de Gaulle , un grand lillois , puis par François Mitterrand . Cette politique , nous devons aussi la construire à l' aune de ce monde nouveau . Oui , ces risques sont absolus , le monde se recompose . À nos frontières , le terrorisme se prépare . À nos frontières , de nouvelles migrations arriveront . De Syrie aujourd'hui , d' Irak demain , de la Corne de l' Afrique . Des migrations politiques , climatiques , immanquablement , et cette diplomatie nouvelle nous devons la construire avec l' Europe , elle est essentielle . J' étais il y a quelques jours à Berlin , j' ai parlé aux Allemands , aux dirigeants , mais aussi à la jeunesse allemande , à l' université Humboldt , et je leur ai tenu le même discours – je vous dis ça parce que , bien souvent , l' habitude française , ou la chorégraphie classique , c' est de vous dire ici que nos problèmes sont européens , que nous allons renverser la table avec l' Allemagne , que nous allons être très très dur , que nous allons aller voir la chancelière et lui expliquer ce qu' elle devrait faire , puis ensuite c' est d' aller en Allemagne et de dire que la France sera le premier partenaire comme il l' a toujours été . Donc je ne vous propose pas de renverser la table , parce que nous y sommes , c' est la nôtre : l' Europe c' est nous , Bruxelles c' est nous , c' est nous qui l' avons voulu . Et nous avons besoin de l' Europe , parce que l' Europe nous rend plus grands , parce que l' Europe nous fait plus forts . Nous continuerons à avoir une diplomatie française , mais , d' évidence , nous serons plus forts au niveau européen . Regardez le monde : un pays-continent , la Chine , le bloc américain , des puissances régionales , qui ne partagent pas nos valeurs , mais avec lesquelles nous devons discuter , la Russie , l' Iran , la Turquie , l' Arabie Saoudite . Et regardez ce qui est train de se passer : l' Europe , qui a vécu dans sa chair , les attentats , fomentés pour partie en Syrie et en lien avec ce qui s' y passe , les migrations et les réfugiés qui ont bousculé l' Allemagne , venant du Proche et Moyen Orient . L' Europe vit au cœur de ces risques et l' Europe n' a même pas été à la table des discussions . Donc oui , nous devons avoir une diplomatie indépendante et exigeante , mais oui , nous devons aussi avoir une vraie Europe de la sécurité , de la défense . Et oui , si l' Europe n' avance pas assez vite , nous devons accepter de le faire à quelques-uns , c' est cela ce que j' ai dit à nos amis Allemands . Commençons par le faire à deux , déjà acceptons à deux d' avancer , de faire plus , de peser . Vous savez , quand j' entends certains dire : &laquo; la France aura vis-à-vis des Etats-Unis une politique très exigeante &raquo; , la main sur le cœur , pensant que nous sommes encore à l' époque du Général de Gaulle . Je me réfère parfois au Général de Gaulle qui est une grande figure de notre histoire , le père de nos institutions , et un homme qui a redonné à notre pays sa fierté et sa force . Mais le Général de Gaulle ne faisait pas souvent référence à Mac Mahon pour construire sa politique . Eh bien c' est à peu près aussi intempestif aujourd'hui de se réclamer d' une politique purement gaullienne . Le monde a changé , il n' est plus le même . J' en entends certains dire : &laquo; Pensez -vous , il a 39 ans . Qu' est -ce qui va se passer quand il sera face à Poutine et Trump ? &raquo; . Non , je prône la bienveillance . Je crois dans les vertus de la diplomatie et je pense qu' aussi longtemps que la diplomatie permet d' empêcher la guerre , elle est préférable . J' entends celles et ceux qui font mine de douter et j' ai des remarques de bon sens à leur dire . La première , c' est que beaucoup de celles et ceux qui disent ça ne seraient pas aujourd'hui invités à la table face à Poutine et Trump , qu' ils se rassurent parce que leur politique conduirait à cela et a parfois conduit à ce résultat . Qu' ils n' imaginent pas une seconde que si nous ne prenons pas nos responsabilités , si nous n' avons pas une politique exigeante , d' indépendance , de lucidité et que si nous n' avons pas une véritable exigence européenne , ils ne seront plus invités à la table face à monsieur . Poutine et face à monsieur Trump . Donc ce que je veux pour mon pays , c' est cette politique exigeante , d' indépendance , française , franco-allemande , européenne qui nous permettra de porter notre vision , notre sécurité , nos valeurs et qui nous permettra d' avancer dans le monde . Donc oui , tendre une main aux États-Unis et leur proposer de poursuivre cette Histoire que nous continuons ensemble depuis tant de siècles et avoir un dialogue exigeant avec la Russie parce que nous devons parler avec elle mais parler avec elle ça ne veut pas dire être fasciné par elle , ça ne veut dire oublier que nous ne partageons pas les mêmes valeurs . Eh oui , c' est cela protéger notre pays , regarder le monde tel qu' il change avec lucidité , défendre nos intérêts et en même temps le faire avec une exigence chevillée au corps . C' est pour cela que lorsque je suis allé en Allemagne , j' ai proposé à nos amis allemands d' avoir une vraie politique de sécurité collective aux frontières , d' avoir un Schengen renforcé . Ceux qui proposent de balayer Schengen sont des naïfs . Ils oublient d'ailleurs qui a été à l' origine du projet Schengen ? Pas ce que je qualifierai de progressiste mais M. Pasqua , avec d' autres , qui fut un ministre de l' intérieur respectable de la France mais qui n' était pas réputé pour son laxisme . Il l' a fait pour quoi ? Parce qu' il a vu avec ses homologues que notre sécurité était mieux assurée aux frontières d' un espace commun et partout à l' intérieur de cet espace , c' est cela Schengen . Et donc quand on nous promet , quand le front national , le parti front national , nous promet de rétablir des sécurités aux frontières , il vous ment . Il vous promet de refermer le pays , de changer la vie de toutes celles et ceux qui ici à Lille passent leur vie d' un côté et de l' autre de la frontière alors que ce qu' il nous faut , c' est un contrôle renforcé aux frontières communes , c' est une politique commune d' asile , gérer depuis des états tiers , depuis l' origine en évitant à des familles de prendre des risques absolus de traverser la Méditerranée pour ensuite attendre des mois en se demandant ce qu' elles deviendront , c' est d' avoir une politique de renseignement commune et renforcée entre nous , d' avoir une gestion des titres communs , d' avoir des services de renseignements qui échangent davantage d' informations , d' avoir aussi une politique , ensemble , vis à vis des états d' origine et des états tiers . Qu' est -ce que cela signifie ? Ça veut dire que lorsque nous refusons de donner des titres à certaines personnes qui demandent pour le statut de réfugié ou qui demande d' avoir des papiers lorsqu' ils dont des migrants économiques , eh bien nous devons pouvoir les reconduire dans leur pays d' origine , ce n' est pas une pratique injuste , c' est une pratique efficace , sinon plus rien n' a de sens . Il n' est pas acceptable qu' aujourd'hui nous laissions s' installer une forme de laxisme coupable où des femmes et des hommes qui n' ont pas obtenu ces titres , eh bien restent dans le pays et restent en s' installant dans la clandestinité , en s' installant eux-même dans la précarité et en conduisant une partie de notre population à considérer que plus rien n' a de sens , que le laxisme est là , que donc tout est possible et à confondre toutes les peurs et tous les sujets . Ce que j' ai proposé à nos partenaires européens , c' est donc d' avoir ensemble chaque année une conférence avec les états d' origine et les états de transite pour signer des accords effectifs de reconduite à la frontière , pour signer des accords qui conditionneront notre aide publique au développement à leur effectivité . C' est cela une politique d' exigence , de générosité et de responsabilité . Et quand j' entends qu' on peut nous proposer , pour la énième fois , des quotas mais c' est à supposer que lorsque les quotas ne sont pas respectés on ait la main pour les faire respecter , ce n' est pas le cas aujourd'hui ! Donc je ne propose pas une politique idéologique en matière d' immigration , je propose une politique efficace , claire , menée avec nos partenaires européens , c' est cela notre projet . Mais en même temps , nous nous battons pour notre sécurité , pour avoir cette politique européenne claire , conquérante parce que nous avons des valeurs différentes de certains d' autres . C' est aussi pour cela que lorsque je me suis rendu en Allemagne , j' ai reconnu le courage de ce pays , sa dignité , lorsqu' il a accueilli des réfugiés venant de Syrie et d' autres pays . Il les a accueilli parce que ce sont nos valeurs , parce que tout ce que je viens de vous dire n' aurait pas grand sens si au fond nous ne savions pas défendre nos valeurs . Parce qu' un homme , une femme , une famille qui a quitté son pays , qui a pris tous les risques , qui a traversé la Méditerranée ou la route des Balkans pour arriver en Europe , vivre libre , ce n' est pas quelqu'un qui a choisi d' arriver en France , ce n' est pas quelqu'un qui vient ici pour profiter de la France . Ce sont des femmes des hommes d'ailleurs qui veulent travailler , construire leur vie . Non , ils ont défendu des principes , la liberté . Ce sont des émancipés . Et parce que nous nous battons , avec lucidité , avec les principes que je viens de rappeler qui seront les nôtres . Parce que nous allons nous battre partout , avec clarté , avec détermination et vigilance . Eh bien oui , nous devons le faire en défendant nos principes et nos valeurs . CHURCHILL pendant la seconde guerre mondiale , lorsqu' il était premier ministre , a eu une très belle réplique . Son ministre du budget - qui n' est jamais un rôle facile - vient le voir et lui dit : &laquo; Monsieur le premier ministre , c' est de plus en plus coûteux , la guerre est absolument terrible pour le pays , il va falloir qu' on coupe des budgets . &raquo; . C' est souvent le cas d'ailleurs , on fait comme ça , le budget de la culture est un bon candidat . Et CHURCHILL lui répond &laquo; Mais si nous baissons le budget de la culture , alors pourquoi faisons nous la guerre ? &raquo; . Alors je vous le dit , nous menons une lutte contre le terrorisme , nous mènerons des batailles , nous avons des ennemis . Mais si nous menons cette bataille partout , à quoi bon , si nous même nous oublions nos valeurs ? A quoi bon se battre contre les terroristes , si nous divisons le pays ? Si nous prenons la déchéance de nationalité ? Ou si nous divisons le pays sur ses valeurs mêmes ? A quoi bon nous battre pour notre sécurité ? A quoi bon nous battre contre des ennemis qui ne respectent pas la liberté , nos valeurs , si en même temps nous mêmes nous ne sommes pas à la hauteur de ces dernières ? Si nous mêmes nous ne savons pas accueillir celles et ceux qui ont pris tous les risques , celles et ceux qui se sont montrés français en faisant cela . Nous nous battons pour notre sécurité , mais à la fin , nous nous battons parce que nous sommes ces émancipés , parce que nous voulons ces valeurs . Cette émancipation que nous voulons , que nous portons pour le pays , c' est aussi celle qui se construit par le travail . Oui , le travail . Ah nous avons beaucoup de débats aujourd'hui pour savoir si il faut bousculer le pays pour le faire entrer dans la mondialisation en aidant en quelque sorte celles et ceux qui sont en train de réussir à aller plus vite , ou si il faut acter de la défaite . Dire &laquo; Oui , à Lille on réussit très bien , à côté on réussira jamais &raquo; . Et donc distribuer ce que l' on ne sait plus produire , qui des revenus universels , qui des dotations améliorées , ou que sais -je … ? Le premier projet est un projet injuste , et je ne le partage pas . Parce que nous sommes les enracinés que l' évoquais . Parce que nous ne pouvons pas accepter que certaines et certains aient le privilège du travail et réussissent seuls . Le deuxième projet , il est inefficace . Parce qu' on ne peut pas promettre de redistribuer si on ne produit pas , si on ne réussit pas , si le pays n' est pas tiré . Alors oui ! Ce que je veux c' est un projet de justice efficace , de justice sociale au siècle contemporain . Le projet que nous portons , et que nous allons construire ensemble , il est basé sur le travail . Mais le travail tel que ce siècle nouveau nous permet de le construire , de le penser . C' est cela , l' émancipation contemporaine . Parce que je veux être le candidat du travail , je veux qu' on entreprenne , qu' on réussisse , qu' on prenne des risques plus facilement . Je l' ai dit , je veux simplifier les normes , pour toutes celles et ceux qui créent , qui entreprennent et qui font . Avoir un droit du travail plus simple , non pas en promettant de bouger toutes les lois , non . En actant ensemble une chose toute simple , c' est que la loi ne peut ni ne doit être la même quand on a une multinationale à Lille , à Paris , ou ailleurs , lorsqu' on est une start-up , lorsqu' on est une TPE . Ca ne peut pas être la même chose lorsqu' on est dans le BTP , lorsqu' on est dans la sidérurgie , lorsqu' on est dans la boulangerie . C' est évident , mais c' est ainsi . Et donc , ce que je veux , c' est beaucoup plus simple . C' est que la loi définisse des principes , et qu' ensuite au niveau de la branche et de l' entreprise on négocie la réalité sur le terrain , ce qui fait la vie de l' entreprise . C' est cela notre projet . Et donc pour ce faire , nous aurons des principes généraux et tout sera renvoyé à la négociations dans l' entreprise , dans la branche , pour que la réalité des entrepreneurs soit plus simple , plus concrète , au réel du quotidien . Parce que cette République contractuelle à laquelle je crois , c' est aussi celle qui est plus juste , qui permettra de mieux protéger , qui permettra de faire réussir . Parce que je crois dans l' initiative économique , je veux supprimer le RSI . Les allègements de charge seront maintenus , un droit d' option vers le régime général sera permis , mais la gestion du RSI sera adossée au régime général . Parce que la vie des entrepreneurs , des indépendants , des professions libérales , doit être facilitée . Parce que je suis le candidat du travail , je transformerai le crédit d' impôt compétitivité-emploi en allègement de charges stable , visible , pérenne . Ce seront donc six points de charges patronales en moins , pour toutes les entreprises , quelle que soit leur taille , quelle que soit leur statut . Qu' elles soient indépendantes , que ce soient des individuelles , que ce soient des grandes entreprises , que ce soient , cher Christophe , des entreprises de l' économie sociale et solidaire , c' est de la justice . Parce que ce coût du travail dont on a beaucoup parlé est aujourd'hui inefficace , et bien nous devons encore l' alléger pour les emplois les moins qualifiés et pour les plus jeunes . C' est pour cela que j' ai décidé qu' au niveau du SMIC , ce seront 10 points de charges qui seront supprimés . Parce qu' aujourd'hui nous ne finançons nos grands risques , la couverture sociale comme on l' appelle , que sur le travail . Et bien la transformation que nous vivons va nous conduire à financer davantage par l' impôt parce que c' est plus juste , parce que c' est plus cohérent , et donc à diminuer le coût du travail pour tout le monde et se faisant à faciliter les embauches . De la même façon , parce que je veux que le travail partout paye mieux , je souhaite alléger les cotisations salariales , celles qui sont payées par chacune et chacun et supprimer un peu plus de 3 points pour les remplacer par une augmentation plus limitée de la CSG . Qu' est -ce que cela veut dire ? Que toutes celles et ceux qui travaillent , quel que soit leur revenu , quel que soit leur statut , et bien demain verront leur pouvoir d' achat augmenter , c' est cela ce que je veux . L' émancipation par le travail , elle doit aussi construire les rêves et les équilibres de ce monde nouveau dans lequel nous entrons . Et donc oui je souhaite que chacune et chacun soit protégé face aux risques du chômage différemment . Qu' est -ce que cela veut dire ? Aujourd'hui nous sommes face à des injustices . Lorsque vous êtes salarié vous êtes protégé , vous avez des allocations chômage . Lorsque vous êtes un indépendant , un commerçant , un entrepreneur , et que demain vous échouez parce que chacune et chacun échoue , et il y aura d' autres échec dans la société , et l' échec fait partie d' une société du risque et bien vous n' êtes pas aujourd'hui protégé . Il y a aujourd'hui en France cette injustice fondamentale . Ce que je veux donc c' est que le chômage devienne une protection universelle pour toutes et tous . Et que face à cette protection , il y ait des droits et des devoirs , et donc il n' y aura plus un droit au chômage parce qu' on est indemnisé , il y aura un droit pour tous avec des devoirs , celui de se former ou d' accepter l' offre d' emploi décente qui est proposée , parce que je ne veux plus entendre des femmes et des hommes qui expliquent : &laquo; J' ai encore le droit de rester un peu au chômage . &raquo; , ou bien qui disent : &laquo; On ne me propose rien . &raquo; . Parce que face à cela , je veux que nous ayons une vraie politique de formation continue et de qualification , un vrai service public de la qualification et de la formation tout au long de la vie , c' est cela l' émancipation véritable . Concrètement qu' est ce que cela veut dire ? Aujourd'hui notre formation continue , plus de 35 milliards d' Euros , elle forme bien celles et ceux qui ont déjà un emploi , elle forme bien dans le même secteur mais elle ne forme pas les intérimaires , elle ne forme pas les chômeurs de longue durée , elle ne prépare pas au changement . Et donc oui , nous avons un système généreux , alors oui , on nous dit tous les jours qu' il ne faut rien changer à ce système , c' est notre modèle social . Mais il a été pensé , construit par un formidable consensus national après 1945 . Mais notre monde se transforme , regardons -le . Qu' est ce qui est en train de se passer ? Qu' est ce qui va se passer ? Un bouleversement aussi radical même bien davantage que que ce que nous avons vécu dans ce bassin minier où j' étais hier . Qu' est ce qui est en train de se passer ? des secteurs entiers d' activité vont être balayés , bousculés par le numérique , par la transition énergétique et environnementale et donc oui , des milliers , des millions d' emplois seront détruits . Mais des milliers , des millions d' autres seront créés dans d' autres secteurs qui sont autant d' opportunités , dans le numérique , dans ce qu' on appelle la silver économie parce qu' il faudra créer des secteurs d' activité pour aider les plus âgés parce que le vieillissement est là parce que la dépendance est là , et nous devons nous préoccuper des plus anciens et donc des secteurs d' activité nouveaux vont se développer . Des opportunités dans l' énergie , dans l' environnement vont se produire . Aujourd'hui , celles et ceux qui sont en train de perdre leur emplois , qui demain les perdront à cause de cette transition , rien n' est fait pour eux . Nous devons donc avoir un vrai service public moderne , contemporain , nouveau , qui va aider à cette transition , qui fera que dans 30 ans nous n' aurons pas l' équivalent du bassin minier aujourd'hui c' est à dire des femmes et des hommes à qui on a proposé le chômage ou les minima sociaux . Alors que j' y étais hier , qu' est -ce qu' ils demandent ? Du travail , du travail et que toutes celles et ceux qui parlent d' assistanat aillent entendrent ceux qui depuis des décennies vivent là-bas , ils veulent du travail , rien d' autre . Et donc ce service public de la formation , de la qualification , ce ne seront pas des nouveaux postes de fonctionnaires , c' est simplement la réorientation de notre système actuel . C' est à dire que nous devons , à chaque chômeur qui n' a plus de perspectives , proposer , offrir une formation de 6 mois , d' un an , deux ans pour pouvoir se requalifier vers ces nouveaux secteurs d' activité , pour pouvoir trouver sa place dans ce siècle qui commence , pour pouvoir trouver sa part dans ce monde qui se recompose . Cher Jean- Paul Delevoye , c' est ce que tu expliques conférence après conférence , ce que tu expliquais déjà au CESE . Ce monde nouveau qui advient , notre responsabilité c' est de le regarder en face avec ses menaces et ses opportunités et donc de préparer à tous les âges de la vie nos concitoyens . Et donc ce service public de la qualification et de la formation , il passera par : plus de de transparence , de l' évaluation des formations qui sont offertes ; celles qui ne sont pas bonnes et qui n' ont pas de résultats seront supprimées , nous ferons donc des économies . Mais celles qui sont adaptées nous les développerons , les universités prendront leur part mais nous proposerons à chacun de pouvoir se reconvertir . Et après cette formation , chacune et chacun devra prendre l' offre d' emploi qui lui est offerte si elle correspond à ses opportunités . C' est cela mes amis , l' émancipation par le travail , l' émancipation qui nous permettra de rentrer dans ce siècle . L' émancipation , enfin , c' est l' école : rien n' existe , rien ne se fait si l' école ne fonctionne plus . Comme je vous le disais , je viens d' Amiens , rien ne me prédestinait , étant né à Amiens , dans la famille qui était la mienne , d' être là ce soir devant vous , rien . Je suis là parce que j' ai eu la chance d' avoir une famille qui m' a poussé à travailler , à étudier , d' avoir une grand-mère institutrice puis directrice d' école qui avait grandi , s' était sortie de sa condition par l' école , qui n' avait vécu que pour l' école et à chaque instant j' ai pensé à cela : au fait que , à deux générations près , rien ne me prédestinait à être devant vous . Alors il y a beaucoup de chance , il y a du travail , il y a des rencontres , il y a des gens qui vous aiment . Mais le socle de l' histoire de ma famille , le socle de ce qui m' a conduit ce soir devant vous , le socle de toute émancipation de notre société : c' est l' école . Or notre école aujourd'hui , notre école elle est bloquée depuis dix ans . Ses performance sont de moins en moins bonnes , les tests PISA comme on les appelle , se dégradent . Les derniers test anglo-saxons montrent que nous sommes pour les mathématiques , en CM1 , les derniers . C' est non seulement un échec sur le plan pédagogique , académique , mais c' est une injustice parce qu' une école qui ne marche plus , ce sont des injustices qu' on reproduit , c' est une société qu' on ne transforme plus : c' est l' injustice au carré . Dans un monde qui se transforme avec des opportunités formidables , qu' allez -vous expliquer à des enfants qu' on n' a pas su former ? Et oui , il faut le regarder en face , dans les endroits ou nous avons concentrés les difficultés économiques , sociales , les problèmes d' intégration : c' est là que l' école est en crise . Nous avons aujourd'hui vingt pour cent de nos enfants qui en CM2 ne savent pas bien lire , écrire , compter . Ces enfants là , ce sont les décrocheurs de demain , ce sont pour partie certains des délinquants ou des chômeurs de longue durée d' après-demain , ces décrocheurs de la petite enfance , je peux quasiment vous en donner les noms , selon leur famille , selon leur quartier , selon leur origine . Si nous croyons dans l' émancipation , si nous croyons dans la promesse républicaine , alors oui nous devons nous indigner face à cela , c' est cela notre combat . Ce combat commence à la maternelle et j' expliquerai dans les prochaines semaines mon projet , pour la maternelle qui est fondamentale en particulier dans les endroits les plus difficiles . Ce combat , c' est ensuite pour l' école , le collège comme le lycée , de refuser le défaitisme dans lequel nous nous sommes installés et de remettre des heures d' enseignants devant les élèves . Baisser le nombre d' heures d' enseignement , ça n' est pas une réponse , c' est une défaite . C' est une injustice , en particulier dans les quartiers les plus difficiles , parce que le coeur de ces difficultés que j' évoquais se concentre dès l' école primaire , dans les quartiers les plus difficiles , les plus pauvres , les plus défavorisés . Ce que je ferai , c' est que dans tous ces quartiers , dans ces six mille classes de CP et de CE1 , je diviserai par deux le nombre d' élèves par classe . Parce que dans ces classes où la difficulté s' est concentrée , c' est bien souvent non pas 20 % mais parfois 80 , 90 % de non-apprentissage parce qu' il faut parfois de l' enseignement plus individuel , il faut d' autres méthodes . C' est un investissement véritable mais pour l' avenir , c' est un investissement pour éviter d' autres coûts : celui de la délinquance , celui de l' échec et celui du chômage . Ces douze milles postes ils seront pour partie redéployés sur les créations en cours et pour partie financés sur les économies que je ferai dans le reste du système scolaire et en particulier sur l' épreuve du baccalauréat . Ce que je veux , c' est aussi mieux rémunérer ces enseignants , ces écoles . Dans ces écoles qui sont les vraies terres de bataille de la République et de son émancipation , que devons -nous faire ? Nous devons livrer bataille et donc nous devons donner de l' autonomie pédagogique , de l' autonomie de recrutement , de la possibilité de choisir les enseignants qui y travaillent , de les recruter sur l' adhésion à un projet , de les rémunérer bien davantage parce qu' ils portent un projet et de leur permettre d' avoir beaucoup plus de liberté . Ce que nous avons vu ensemble , cher Frédéric Marchand , ce matin , à Hellemmes , dans cette école maternelle , c' est un investissement , en particulier de la mairie , de l' école , de l' ensemble des enseignants pour justement aller contre le destin . Mais dans un vrai projet pédagogique , une vraie autonomie c' est celle la que je veux partout dans l' école de la République , plus de liberté , plus de moyens mais en même temps , plus de responsabilité . Avec la liberté il y a l' évaluation , la responsabilité , avec l' autonomie supplémentaire , il doit y avoir une évolution partout pour tout le monde , une vraie transparence . On doit voir les endroits où on n' arrive à avoir des résultats et ceux où on arrive à contrecarrer le destin . C' est celle-là , l' école que je veux , c' est ce projet : plus d' autonomie , plus de reconnaissance pour les enseignants et en même temps plus de responsabilité et d' évaluation . Je reviendrai dans les prochains jours et prochaines semaines sur le projet que je veux que nous portions plus en détails sur le collège , sur le lycée . Mais deux mots ici encore : le premier pour vous dire que l' un des défis qui est le nôtre , c' est lorsque le lycée s' achève et que les enfants doivent prendre la mer , l' entrée dans ce monde nouveau , dans l' enseignement supérieur , nous avons une transformation profonde à conduire : celle de l' orientation . Celle qui consiste à accompagner tous les enfants et en particulier les boursiers , celles et ceux qui viennent des quartiers les plus défavorisés à accéder à l' information , à accéder aux bonnes formations . J' étais tout à l' heure avec des enseignants , des associations formidables , cher Pierre MATHIOT ce que tu conduis depuis tant d' années ici avec ce formidable projet des programmes d' études intégrées qui a été chercher des jeunes boursiers à Lille , puis dans la région puis plus loin pour contrecarrer la fatalité . Pour permettre à des jeunes qui sont boursiers , dont les parents n' avaient pas fait d' études d' accéder au supérieur . Parce que qu' est -ce qu' il se passe aujourd'hui ? C' est que ce sont bien souvent les mêmes qui accèdent au supérieur parce que pourquoi vous avez , dans certaines filières , plus des trois-quart des élèves qui arrêtent à l' issue de la première année d' université ? C' est parce que personne ne leur a expliqué , personne ne les a accompagné , ne les a informé , ne leur a dit ce qui était bon pour eux , ce qui correspondait à leurs goûts . Et la plus belle phrase c' est celle qu' a prononcé tout à l' heure Omar , lorsqu' il a dit : “ Avant de rentrer dans le programme , je pensais que tout ça c' était pas fait pour moi ” . Alors , oui , l' émancipation , c' est quand justement par cette transformation on fait que les plus défavorisés arrêtent de penser que tout cela n' est pas fait pour eux . Alors j' y reviendrai . Il y a aussi le travail formidable fait dans les lycées professionnels , le travail essentiel fait dans l' apprentissage pour nos jeunes , cher Patrick , j' y reviendrai dans les prochains jours et les prochaines semaines . Je ne peux pas tout vous dire ce soir vous le savez bien . On me reproche parfois même d' être trop long , mais je suis trop long parce que j' explique , parce que je décris ce monde , parce que j' essaie de porter ce projet commun , ce sens commun qui est le nôtre , de réinscrire … Alors oui , je reviendrai vous parler de l' apprentissage , de l' environnement , de la culture qui soit aussi les armes de notre émancipation collective . Mais je finirai par vous dire une chose simple : cette émancipation qui est notre projet , qui est d' aider celles et ceux qui réussissent à réussir encore mieux plus vite et plus fort , d' aider chacune et chacun à trouver sa place parce que nous vivons ensemble . Ce projet qui réconcilie l' efficacité , la liberté et la justice sociale . Il n' est possible que par votre engagement , que par l' engagement de toutes et tous . Il n' est possible que par le travail des associations , des élus locaux , du terrain , de ce qui est fait . Cette région elle se bat . J' ai parlé de beaucoup d' élus , toi Dominique , j' étais il y a quelques mois encore à Wattrelos à tes côtés , de ce qui est fait par le président de la région Xavier Bertrand à qui je veux ici aussi rendre hommage , qui se bat et qui s' est battu face au Front National . Que Jean-Paul Huchon , qui est toujours à mes côtés a conduit lorsqu' il menait les destinées de cette belle région d' Île-de-France , et il est là parmi vous , que tu as mené Jacky à la tête de CCI . Ce combat , c' est celui des élus , celui des associations , c' est votre engagement , c' est comme cela que nous le tiendrons . Alors vous savez , j' entends déjà toutes celles et ceux qui vous diront que c' est un rêve , que nous sommes des rêveurs : “ à force de croire obstinément à ses rêves , on finit par imposer sa réalité . ” Alors oui , mes amis cette phrase que vous venez d' applaudir , elle n' est pas de moi , elle est de Pierre Mauroy . Alors oui , je vais vous le dire ce soir : que toutes celles et ceux qui pensent que nous sommes des rêveurs soient rassurés , nous le sommes ! Mais , que toutes celles et ceux qui pensent que nous sommes des rêveurs soient terriblement inquiets parce que nous sommes des obstinés ! Donc ce que je vous demande ce soir , avec moi , c' est pour les cent jours qui nous restent de porter ce projet d' émancipation partout , de rassembler toutes celles et ceux qui sont aujourd'hui à droite et qui vous disent vouloir y rester mais ne pas s' y retrouver ; toutes celles et ceux qui à gauche doutent et ne s' y retrouvent plus ; toutes celles et ceux qui ont peur , ne cédez rien aujourd'hui aux discours de haine , ne cédez rien aux discours de repli , venez retrouver le mouvement de l' optimisme volontaire , celui de l' espérance , le nôtre , le vôtre ! J' ai besoin de votre force , de votre fougue ! Vive la République , vive la France ! Merci à vous ! Merci à vous ! Je compte sur vous ! "
    ]
macron_test = [
     "Merci à vous ! Merci à Yann et Maud ! Merci beaucoup ! Bonjour Quimper ! Bonjour le Finistère ! L' Ille-et-Vilaine ! Les Côtes d' Armor ! Le Morbihan ! La Loire-Atlantique ! Bonjour la Bretagne . Nous sommes plus de 2200 ce soir , et , malheureusement , plusieurs centaines de personnes sont restées dehors . J' ai passé un petit moment avec elles , d' où mon retard . Alors , je vous rassure , notre ambition n' est pas , ce soir , de concurrencer le festival de Lorient , ni les Vieilles Charrues , ni celui de Cornouaille ou du bout du monde . Mais ce soir , je suis venu malgré tout vous parler d' ambition . Notre ambition . Notre ambition commune . Celle qui nous porte depuis le début . Alors je suis heureux , ce soir , de me retrouver à Quimper avec nombre de mes amis bretons . Je me souviens de la visite faite il y a un peu plus d' un an , j' étais alors ministre , à Ergué-Gabéric , à Plévin , cher Richard , chez toi , le calvaire , mais aussi de nos rencontres économiques . Et un an plus tard , aujourd'hui , je suis avec vous , porteur d' autres responsabilités , que je mesure chaque jour un peu plus . Celle de porter ce mouvement collectif . Cette volonté d' aller de l' avant . De réveiller les uns et les autres . Et oui , je suis heureux ce soir d' être ici , dans ces terres d' ambition et d' aventuriers . D' ambition d'abord pour les Bretons , eux-mêmes , bien sûr . D'abord une ambition pour inventer l' avenir . C' est un territoire d' aventuriers , de découvreurs , d' explorateurs , de héros sportifs contemporains . Et un territoire qui ne cesse d' innover . Je l' ai vu à plusieurs reprises . À Rennes , là aussi comme ministre puis , plus récemment , à Lannion , il y a plusieurs mois . Et nous l' avons vu toute la journée ensemble , quels qu' en soient les secteurs : de l' agriculture aux cars , en passant par bien d' autres . L' innovation est au coeur de cette terre . Cette volonté d' inventer l' avenir , de regarder devant . Votre ambition est aussi vouée à enrichir chaque jour ce que vous êtes . A préserver votre singularité , vos atouts , votre identité . Parce qu' un Breton , au fond , c' est un enraciné . Je pense aux langues de Bretagne , qui sont une formidable richesse . On n' ose plus dire les langues chez certains . Ce sont des langues , du gallo au breton - je viens d' une terre qui porte avec fierté le picard , et le ch'ti - mais cette langue , ces langues qui sont les vôtres , c' est votre richesse et elles justifient pleinement que leur enseignement , leur transmission , soient accompagnés et encouragés . Et nous devons toutes et tous nous y engager . Et votre force surtout , c' est que l' ambition que vous portez , elle dépasse largement Quimper , elle dépasse largement le Finistère , elle dépasse largement la Bretagne . C' est une ambition pour le pays entier que vous portez . Et ici , le long de l' extrême frontière de notre belle France , continue de battre avec puissance le coeur vibrant de notre pays . Cette ambition bretonne , c' est celle qui vise d'abord à protéger le pays tout entier . Car la présence de nos forces de dissuasion nucléaire en presqu'île de Crozon , mais aussi la présence , à travers la région , de presque tous nos corps d' armée , c' est le témoignage de cette vocation du territoire breton , de cette ambition bretonne . Je veux ici saluer un ami , naguère collègue , grand ministre de la Défense , grand Président de la Région Bretagne , Jean-Yves Le Drian , qui porte aujourd'hui cette tâche . Et il a encore récemment , à travers un discours fort sur la cyberdéfense , dessiné la vocation nouvelle de ce territoire et de nos ambitions communes . Et en disant cela aussi , comment ne pas penser à l' île de Sein ? Comment ne pas penser à ce tréfond de notre histoire , au fait que cette petite île , quand on la regarde sur la carte , paraisse si innocente ? Pour certains qui ne connaissent pas notre histoire , si insignifiante . Certains qui ont oublié , qu' au moment où le pays doutait de lui-même , se pensait défait , cent vingt-huit pêcheurs ont décidé de ne rien abandonner . Cent vingt-huit pêcheurs ont décidé de résister . Comment ne pas se souvenir , quand on parle de cette vocation bretonne , de porter la fierté française , de cette ambition française d' ici , comment ne pas se souvenir de cette scène , où le Général De Gaulle , passant en revue les premiers résistants , cette poignée de femmes et d' hommes &laquo; un peu &laquo; , disaient alors quelques-uns qui étaient auprès de lui ? Voyant les uns et les autres se succéder , et tant de Seinans décliner leur identité et le lieu d' origine , s' exclamer &laquo; Mais l' Île de Sein , c' est donc le quart de la France ! &laquo; Comment ne pas se souvenir qu' en 1940 , une majorité de députés venant de cette terre , a voté contre les pleins pouvoirs à Pétain ? C' est aussi cela . Alors cette ambition qui vous dépasse , c' est la clé de votre identité . Une identité ouverte que je voudrais pour la France entière , car vous pouvez tout à la fois être profondément bretons , puissamment français et sincèrement européens . C' est cela votre identité ! Sans que jamais l' un de ces termes soit exclusif l' un de l' autre . Sans que jamais ces sentiments ne s' opposent . Sans que jamais ils ne viennent s' annuler ou s' opposer les uns aux autres . Voilà pourquoi je suis venu partager votre ambition , notre ambition . Parce que je crois que l' ambition finistérienne , l' ambition bretonne , elle est d'abord et avant tout une ambition française . Une ambition pour faire réussir notre pays , pour défendre ses valeurs , pour préserver ce qu' il est avant tout aujourd'hui , au coeur de notre défi . Alors , au fond , pourquoi vous parler d' ambition aujourd'hui ? Simplement parce que notre pays en a terriblement besoin . L' ambition n' est pas un vilain mot , quelque chose dont on devrait avoir honte . L' ambition , c' est ce qui permet de tirer le meilleur de chacune et chacun , c' est ce qui permet de retrouver sa fierté , c' est ce qui permet à celui qui est armé pour la vie , prêt à prendre tous les risques , de réussir encore davantage , et c' est ce qui permet aussi à celui qui n' a rien de créer sa propre vie , de construire son destin , de trouver sa place . L' ambition , c' est aussi ce qui a permis à notre mouvement d' éclore . Notre ambition , ce soir , mes amis , l' ambition d' En Marche ! , l' ambition de celles et ceux qui veulent faire avancer le pays . Il y a une dizaine de mois nous partions de rien . Nous sommes parvenus à réunir plus de 140 000 personnes sous la même bannière progressiste , et elle est là , ce soir , à Quimper ! Et le succès de notre mouvement , en Bretagne , en est d'ailleurs la preuve . Je veux saluer en particulier les maires et les élus bretons qui m' ont d' ores et déjà apporté leur parrainage et qui sont ici présents . Mais aussi Olivier Alain , Vice-Président de la région Bretagne à l' agriculture , qui m' a accompagné encore une fois aujourd'hui . Pierre Karleskind que vous venez de voir et qui a introduit ce meeting , chargé de la mer et de la pêche à la région . Mes deux amis : Corine Erhel , Jean-Pierre Le Roch , au sein d' En Marche ! qui portent aussi les couleurs de la Bretagne . Et bien sûr Richard Ferrand . Je dois dire que , quand je me suis demandé à qui confier la responsabilité du Secrétariat Général d' En Marche ! , l' idée de m' en remettre à un Breton , fut -il d' adoption , n' était pas le moindre des critères . C' est un déterminé , comme mes trois amis que je viens d' évoquer , un déterminé qui sait prendre les risques , la mer , et aller au large : c' était donc à lui que revenait cette mission . Le jour où j' ai quitté mon ministère , j' ai cité non pas un Breton , mais un méridional , Pagnol , et qui avait une jolie formule pour cette terre qui aime la mer , les risques , comme vous . Il disait : &laquo; Si on veut prendre la mer sans prendre , avec , les risques , mieux vaut acheter une île qu' un bateau &raquo; . C' est une terre de mer , mais aussi une terre de marins . Nous avons pris la mer , le bateau est chaque jour plus grand , et nous irons au bout de cette Odyssée ! Et l' ambition qui est la nôtre , elle correspond parfaitement à ce qui est l' histoire même de Quimper . &laquo; Kemper &raquo; , ça veut dire &laquo; confluent &raquo; , en breton . Eh oui , nous sommes une terre de confluences , eh bien nous , nous sommes un mouvement de confluences . Alors j' en entends beaucoup en ce moment qui se posent des questions . Ils disent : &laquo; Il y a des ralliements chaque jour &raquo; , &laquo; C' est une drôle d' affaire &raquo; , &laquo; Est -ce que ça devient le Parti Socialiste ? &raquo; . Je vous rassure , hier , les mêmes socialistes demandaient : &laquo; Est -ce que ce n' est pas en train de devenir la droite ? &raquo; . Quand le bébé commence à prendre forme , il trouve beaucoup de pères putatifs . C' est une confluence : il y a des socialistes , des radicaux , des écologistes , des Modem , des UDI , des républicains , des femmes et des hommes de la société civile , vibrante , qui s' étaient détournés de la vie politique . C' est tout cela , En Marche ! . Cette confluence qui a été portée , je ne l' oublierai jamais , par des engagés et des courageux dès la première heure . Et donc oui : notre quille , c' est cette confluence . Elle ne gîtera ni d' un côté , ni de l' autre . Elle aura toujours une boussole : l' intérêt général , un projet pour le pays , et pas pour une partie ou une clientèle , c' est comme ça que nous irons au bout . Parce que ce mouvement , parce que le projet que je porte , c' est une ambition , c' est notre ambition pour le progrès en France . Alors oui , nous aurons dans les semaines à venir d' autres ralliements , je l' espère , de plus en plus . Mais les ralliements que nous devons chercher mes amis , ce sont les ralliements de chacune et chacun des Français . Ce que je guette chaque matin , ce n' est pas le ralliement d' un tel ou d' un tel qui fascine le microcosme parisien et journalistique , parce que celui-là à la fin , il ne comptera pas davantage que le travail de centimier que nous faisons depuis le premier jour , que cette lutte sur le terrain que chacune et chacun d' entre vous a conduite , parce qu' à la fin du jour , à la fin de cette bataille , c' est une majorité de Françaises et de Français que nous aurons ralliés pour la victoire ! Alors l' ambition d' En Marche ! , c' est de répondre aux vrais problèmes et de parler des vrais sujets . Notre ambition , c' est , dans un monde de risques , de transformations profondes , d' apporter les sécurités individuelles , les nouvelles protections sans lesquelles les uns et les autres hésitent à prendre le large , restent parfois à l' abri , doutent , ou ont peur du changement . Le projet que nous portons , c' est un projet progressiste , parce qu' il vise à inventer ces sécurités nouvelles , à reconstruire non pas un projet d' égalité pour tous et toutes , parce que c' est un projet onirique , ce serait une promesse mensongère . Mais un projet pour que l' égalité des possibles se fasse en actes . Nous sommes toutes et tous différents . Il y a des gens plus doués que d' autres , il y en a qui sont plus travailleurs que d' autres , il y en a qui auront plus de chance que d' autres . Mais ce qui est insupportable , c' est que , à mérite équivalent , parce qu' on est né dans une famille , dans un quartier , sur un territoire plus retranché que les autres , on ne puisse pas exprimer ses talents de la même façon . C' est cela , le projet qui est le nôtre , l' ambition qui est la nôtre , c' est cette ambition de justice sociale , cette ambition première que je veux pour le pays . Alors elle se traduit comment , concrètement ? Par une ambition pour la santé . Je l' ai déjà dit et redit , et je le répéterai chaque jour de cette campagne . Nous serons demain dans un centre important de la région , qui rééduque , répare des vies . J' ai passé beaucoup de temps avec nos concitoyens qui soignent et qui s' occupent des uns et des autres . Il n' y a pas de risque qui peut être pris , il n' y a pas de société qui peut se transformer si on ne protège et ne prévient face aux aléas de la vie . Pour la santé , je veux une meilleure prévention . Pour ce faire , je me suis engagé à ce que dans toutes les études , de médecine , d' infirmiers et d' infirmières , de kinésithérapie , de dentiste , il y ait au moins trois mois dédiés à la prévention . Les jeunes étudiants iront dans les écoles , dans les maisons de retraite , dans les endroits les plus défavorisés pour faire de la prévention : c' est l' économie la plus efficace , c' est ce qui est essentiel pour notre société . Parce que ce projet pour la santé est un projet d' ambition , je veux aussi que nous puissions couvrir 100 % des dépenses liées aux lunettes , aux prothèses dentaires et aux prothèses auditives . Pourquoi ? Parce que c' est prévenir des risques plus graves , parce que c' est , là aussi , une vraie justice économique et sociale . Parce qu' un jeune qui ne peut pas lire au tableau , sous prétexte qu' il n' est pas corrigé , c' est un jeune qui n' apprend pas . Parce qu' un chômeur qui ne peut pas soigner ses dents , sous prétexte que c' est mal remboursé , c' est un demandeur d' emploi qui n' est pas embauché . Parce qu' une personne âgée qui n' entend pas et qui ne peut pas s' équiper , c' est une personne qui s' enferme dans la dépendance plus rapidement , dans la sénilité et l' isolement . Ce projet pour la santé , c' est celui par lequel aussi , je veux porter une réforme profonde de l' accès aux soins , avec une généralisation des maisons de santé pluridisciplinaires , des maisons de suite pour les soins , afin de sortir plus vite de l' hôpital et une réforme juste , en profondeur , de l' hôpital par une tarification plus adaptée , par une réorganisation responsable , qui permette de combler les déserts médicaux , et en même temps , de poursuivre les nécessaires économies , sans réduire en rien l' accès aux soins . Cette ambition pour la santé , j' y suis revenu il y a quelques semaines chez nos amis à Nevers , beaucoup plus en détail . C' est une ambition de justice , raisonnable puisque nous ferons des économies , mais ambitieuse parce qu' on ne construira pas l' entrée dans le XXIe siècle en déremboursant des soins utiles , en supprimant des fonctionnaires de la fonction publique hospitalière . Parce que le projet , respectable , qui est porté par les conservateurs de notre pays , c' est un projet qui est à la fois : - inefficace . ( Les déremboursements de petits soins , comme on nous l' a expliqué - je dois vous confesser ici que je n' ai toujours pas compris ce qu' était un petit soin ; on a parfois croisé des petits rhumes qui pouvaient devenir des rhumes graves - en tout cas , ces déremboursements , ils conduiront à des risques plus graves , ils conduiront à ne pas traiter certaines petites pathologies , précipitant vers des maladies plus graves , et , ce faisant , plus coûteuses . Le coût , à la fin , je vous le parie , sera plus important ) . Mais c' est aussi : - un projet injuste . Si les petits soins sont déremboursés , les plus fortunés d' entre nous se soigneront toujours autant , les plus modestes auront la couverture maladie universelle , mais les classes moyennes , elles , n' auront rien . Ce seront les victimes de ce projet . Mon ambition , c' est aussi d' agir pour l' école . Lorsqu' on porte un projet progressiste , on porte un projet pour que , partout , comme je le disais , quelque soit l' endroit d' où l' on vient , on puisse réussir . C' est cela , la mission de l' école de la République . Et , on le sait , c' est cela qui , il y a un siècle , dans nos campagnes , a permis à des fils et à des filles de paysans d' accéder à l' école , ce qui semblait vertigineux à leurs parents . Moi , je me souviens encore de ma grand-mère , elle n' était pas de la région mais des Hautes- Pyrénées , me raconter - son père était chef de gare et sa mère concierge - que pour ses parents , c' était invraisemblable qu' une fille , qui plus est , poursuive au-delà de 14 ans ses études . C' est l' école de la République qui a fait qu' un instituteur lui a fait confiance , lui a dit &laquo; Il faut continuer un peu plus loin &raquo; et puis un autre , un professeur de philosophie , qui fait partie de ma propre mémoire , vous imaginez ce que c' est que la République , c' est ça la République , ce n' est pas un mot qu' on envoie à la figure des gens comme un sel qui ne sale plus . La République , ce sont des visages , des gens engagés qui transforment des vies . Moi , ma vie , mes souvenirs , ils sont faits des souvenirs d' enfance de ma grand-mère , et de ce professeur de philosophie que je n' ai jamais vu , mais dont j' ai l' impression de connaître le visage . Parce qu' il lui a fait lire Paul Valéry un jour et qu' il a dit à ma grand- mère , au début de ce siècle : &laquo; Il faut que vous alliez suivre des études pour devenir institutrice à Toulouse &raquo; . Puis , après , la vie fait son oeuvre , ma famille a poursuivi son chemin et je suis devant vous aujourd'hui . Mais il y a une chose : ça n' aurait jamais été possible sans l' école de la République . Alors cette école , aujourd'hui , elle trahit ses promesses , elle reproduit , bien davantage qu' il y a 20 ans , les inégalités . Moi , j' ai fait une école à laquelle on reproche beaucoup de choses : l' ENA . Il y a des gens qui choisissent , face à ce fléau , de faire oublier qu' ils ont fait l' ENA et de dire qu' il faut la supprimer . Moi je pense que c' est toujours compliqué d' aller devant les gens , de dire &laquo; J' ai un projet pour le pays &raquo; en expliquant qu' on se renie soi-même : ce n' est pas un bon signal . J' ai fait l' ENA , c' est un concours de la République , je l' ai fait en travaillant , j' en ai raté d' autres , je vous rassure . Ce qui ne va pas , c' est ce qu' on en fait ensuite . Mais l' ENA , aujourd'hui , elle forme moins de fils d' ouvriers et de paysans qu' il y a trente ans . C' est cela notre échec . Notre école , aujourd'hui , elle fait qu' il y a 20 % de nos jeunes en CM2 qui savent imparfaitement lire , écrire , compter . Ces jeunes-là , ce sont ceux qui ne finiront pas leurs études supérieures , qui auront du mal à avoir le baccalauréat , qui décrocheront bien souvent avant , qui auront du mal à attraper un emploi et pour qui la vie sera plus injuste que pour les autres . Alors , mon projet pour l' école , je ne vais pas vous le détailler ce soir . Il y a une mesure qui me tient à coeur , c' est que dans tous les endroits de la République identifiés comme difficiles , plus pauvres que les autres , plus reculés , ce qu' on appelle les zones d' éducation prioritaire , je veux qu' on divise par deux le nombre d' élèves par classe . En CP et en CE1 , 6000 CP , 6000 CE1 . C' est très pratique ce que je vous dis là parce que si on ne fait pas ça , on ne peut pas former ces enfants , on les laisse face aux injustices et donc il faudra là réallouer une partie des maîtres et des postes qui ont été créés ces dernières années et dispersés un peu partout ; et il faudra investir un peu de ce que nous économiserons en simplifiant l' épreuve du baccalauréat et en réorganisant une partie du reste de notre école . Je veux aussi qu' il y ait plus de temps d' enseignement devant les élèves . Si nous voulons réussir ce combat de la République et de l' école , nous devons arrêter ce mouvement , commencé il y a maintenant dix ans , qui consiste à réduire le temps d' enseignement : c' est de la démagogie , c' est de la défaite . Les enfants ont besoin d' être éduqués , ils ont besoin d' enseignements . Et donc , nous stopperons ce mouvement décennal de baisse pour repartir de l' avant en valorisant aussi les carrières d' enseignants , en valorisant en particulier celles et ceux qui vont sur ces terres de batailles et ces terres de difficultés , en valorisant aussi l' autonomie pédagogique , la capacité à innover sur le plan pédagogique et sur le terrain . L' ambition que nous portons , vous l' avez compris , c' est donc d' apporter ces véritables sécurités nouvelles et contemporaines sans lesquelles on ne peut pas prendre de risques , c' est aussi cela que je porte , en réformant en profondeur notre système de chômage , notre système de formation continue , pour qu' on puisse demain former partout en France toutes celles et ceux qui n' ont pas de qualifications , qui sont des chômeurs de longue durée et qui ont besoin d' être accompagnés pour se réorienter dans leur carrière , c' est indispensable . L' ambition que je porte ensuite , que nous portons , c' est une ambition pour le travail , le travail c' est une valeur fondamentale , le travail c' est ce autour de quoi nous allons rebâtir notre projet de société : la capacité offerte à chacune et chacun de retrouver une place . Alors , parce que je suis le candidat du travail , je veux qu' on puisse entreprendre plus facilement dans notre pays , je veux qu' on puisse créer plus facilement une activité d' indépendant , de start-up , qu' on puisse innover beaucoup plus simplement . Très concrètement , en faisant quoi ? D'abord , en faisant que les indépendants aient une situation plus simple , plus transparente qu' ils ne l' ont aujourd'hui , c' est pour cela que je supprimerai le RSI ( le Régime Social des Indépendants - s' il y avait une petite dizaine de personnes qui ont l' air de ne pas connaître le RSI dans cette salle ) . Le Régime Social des Indépendants a en effet , par les difficultés de gestion , et là je n' incrimine en rien celles et ceux qui font fonctionner ce régime et à qui je veux rendre hommage parce qu' ils ne sont pas les responsables de cette situation , fait qu' aujourd'hui les délais sont trop longs , les tracasseries sont incessantes et on a vu , malheureusement , chez les indépendants , beaucoup trop de suicides ces dernières années arriver à cause de cela . Je maintiendrai tous les allégements de charges qui sont liées aux indépendants . Tous les indépendants et professions libérales seront éligibles à tous les allégements de charges que j' ajouterai et en particulier les six points de charges en moins pour tous les salariés , tous les entrepreneurs que j' ai annoncés dans mon programme , il touchera aussi les indépendants . Mais les indépendants demain auront donc la possibilité d' aller , s' ils le souhaitent , d' aller vers le régime général ou s' ils restent dans ce régime privilégié qui est le leur , eh bien d' être adossés au régime général et d' avoir les mêmes modalités , la même simplicité que tous les salariés de France , que tous les autres Françaises et Français en termes de simplicité , de rapidité , de délais . Je veux ensuite continuer à alléger le coût du travail , cela veut dire transformer le crédit d' impôt compétitivité emploi en six points de charge en moins jusqu' à 2,5 SMIC pour toutes les catégories d' entreprises qu' elles soient entreprises du secteur privé , de l' économie sociale et solidaire ou du secteur coopératif . Et parce que je suis le candidat du travail , je veux enfin que le travail paye mieux et donc , pour ce faire , que les cotisations salariales payées aujourd'hui par tous les salariés soient allégées et qu' on puisse ainsi transférer une partie du financement social qui pèse aujourd'hui sur le travail , sur les impôts et donc que le pouvoir d' achat de tous les travailleurs , fonctionnaires , secteur privé , indépendants , professions libérales augmente et il augmentera demain grâce à mon projet . Et quand je parle du travail et de notre ambition pour le travail , je parle évidemment aussi de cette révolution que nous devons faire pour tous les secteurs de notre économie , de l' innovation qu' on voit ici fleurir à travers la French Tech dans toute la région , que j' ai modestement accompagnée mais je l' ai souvent dit , la French Tech , c' est la meilleure politique publique . En tant que ministre , c' est ce dont j' ai été le plus fier parce que c' est quasiment aucun crédit public , il n' y a aucune loi , aucun décret et je n' ai à peu près rien fait . La French Tech , c' est un label mis sur l' énergie qui existe dans tous les territoires et l' innovation formidable qui est là ce soir et qu' on a mis simplement sous un même pavillon et un même étendard , c' est donc la France qui réussit . C' est cela , la French Tech et ça n' est à personne . Donc vous voyez , je vous rassure , pas plus que je crois qu' aucun politique n' a inventé l' internet , l' accès aux géants de la technologie , je ne suis pas l' inventeur de la French Tech . La French Tech , ce sont vos entrepreneurs , c' est ce coeur vibrant de l' innovation . Et donc oui , cette économie , elle se transforme , il y a des secteurs nouveaux qui apparaissent , d' autres qui vont être profondément bousculés par la transformation énergétique et environnementale qui est la nôtre , une transformation qui vient offrir des nouvelles opportunités au pays tout entier . Mais je veux ici vous parler tout particulièrement d' un secteur , un secteur qui tire la Bretagne toute entière , qui est au coeur de son économie , qui couvre 60 % de son territoire , qui concerne de près ou de loin à peu près toutes les familles bretonnes , je veux vous parler de l' agriculture . Je vais vous en parler parce qu' on l' oublie trop souvent , parce que , quand on parle de ces transformations de notre économie , on oublie le rôle qu' a l' agriculture , le rôle qu' ont l' agriculture et l' agro-alimentaire dans nos exports . On nous parle , je ne vais pas ici faire de polémique sur le sujet , je vous ai dit mon amitié pour Jean-Yves Le Drian , mais on nous parle toujours des rafales , des grandes réussites et on lui doit beaucoup là-dessus , de notre industrie de défense . On nous parle volontiers des Airbus mais on oublie que notre agriculture et notre agro-alimentaire , c' est notre fierté à l' export , c' est une vraie réussite . Mais je vais vous dire , on ne parle plus d' agriculture aujourd'hui , dans nos débats publics ou médiatiques qu' en parlant de la terrible misère qui est là , qu' il faut regarder en face ou en parlant de coût . On en parle comme si c' était un secteur qui devait s' effondrer , disparaître , qu' il faut accompagner dans , finalement , cette déliquescence . On en parle chez certains autres comme d' un coût , d' un budget qu' il faudrait supporter . On en parle comme d' une espèce de cause rétrograde quand certains , même pour faire plaisir , en pensant faire plaisir aux agriculteurs , disent &laquo; Mais oui , faut plus du tout respecter l' environnement , le climat est une affaire secondaire , on va vous supprimer toutes ces normes qu' on vous a mises &raquo; . Non , tout cela n' est pas vrai . L' agriculture c' est notre histoire , l' agriculture c' est notre pays , l' agriculture c' est un secteur économique qui exporte , l' agriculture ce sont les femmes et les hommes qui nous nourrissent chaque jour . Au cœur de mon projet , au cœur de notre ambition , je veux porter cette ambition pour l' alimentation française . On a chaque jour , quand on parle de notre culture , de ce qui nous fait , de ce qui nous distingue des Chinois ou d' un Américain , parmi tant d' autres choses , ce qui nous nourrit , il y a ces plats qu' on défend , cette gastronomie française , c' est cela l' agriculture aussi , c' est notre alimentation . Il y a l' objectif de souveraineté alimentaire que nous devons porter . Les agriculteurs , ce sont ceux qui vous nourrissent , qui vous nourrissent et qui doivent demain nourrir nos enfants . L' agriculture , et on le voit ici , on en parlait tout à l' heure , c' est aussi ce qui fait nos paysages , je le dis souvent . Dans ces mots qu' on ne doit plus prononcer , vous savez , je suis un grand maladroit parfois , je dis des mots qu' on ne doit pas dire . Alors , parfois on fait des erreurs , j' en ai fait , en particulier en Bretagne , et j' ai beaucoup d' émotion encore quand je repense à cette visite que j' ai faite , il y a quelques mois , chez Gad parce que j' avais blessé des gens de manière impropre et injustifiée , c' était une faute . Mais parfois , dans le débat public français , on ne peut même plus nommer le réel , on voudrait tomber dans une espèce de langue qui se vide , dans une circonlocution qui évite de dire les choses au fond . Il y a un mot qu' on ne veut plus dire c' est le mot de &laquo; paysan &raquo; . C' est un beau mot , &laquo; paysan &raquo; . Parce que oui , nos paysans , font nos paysages , ils font nos pays , ils transforment la France et quand on arrive , se posant comme je l' ai fait sur Brest tout à l' heure , même quand on se pose à Lannion , comme j' ai fait il y a quelques mois , on voit ce travail de dentelle , on voit cet entretien des paysages . Qui ferait les bocages ? Qui ferait ces paysages variés et multiples ? Qui débroussaillerait ? Qui fait que notre pays n' est plus une jungle impropre ou une forêt inféconde ? Nos paysans . Quand on parle d' agriculture c' est aussi de cela que nous devons parler . Et puis … Vous pouvez le dire , c' est pour eux . Et puis parler d' agriculture , c' est parler d' un savoir- faire , de traditions , c' est parler d' innovation , d' une transformation profonde , c' est parler de qualité , de produits . Alors oui , ce soir , je vous parle d' agriculture quand je vous parle de l' ambition française parce que l' agriculture en Bretagne , qui est la première région agricole française , comme dans tout le reste du pays , c' est le cœur de notre ambition pour demain ! Alors , je le sais , les années dernières ont été difficiles , 2015 et 2016 ont été marquées par des crises profondes , la crise du marché , la crise du lait , des crises sanitaires , comme la grippe aviaire dans le secteur des canards à foie gras , des crises climatiques aussi , le gel qui a touché le vignoble bourguignon , les mauvaises récoltes qui ont touché nombre de céréaliers . L' impact de ces crises est particulièrement visible en Bretagne parce que , comme je le disais , c' est la première région agricole . Alors , face aux crises , il y a eu des réponses , on a versé de l' argent , on a apporté des aides , elles étaient indispensables . Mais est -ce que cela suffit ? Non . Le cœur de notre réponse , de notre ambition , c' est de refonder un pacte profond , un pacte social entre les agriculteurs et la société française . Ce pacte social , c' est un sens , une vision à long terme pour l' agriculture française . Des agriculteurs fiers de leur travail , des citoyens fiers de leur agriculture parce que , comme je viens de le rappeler , l' agriculture , elle est au cœur de l' histoire mais aussi du projet français . Alors pour cela , nous n' y arriverons pas simplement en augmentant les aides . J' entends parfois des discours simplistes là-dessus . Non . D'ailleurs ce n' est pas ce qui nous est en premier demandé . J' étais tout à l' heure avec quelques-uns d' entre vous sur ces belles terres du Finistère à Ploéven près de Châteaulin dans la famille Nedellec - ils ont plus de 50 vaches . Je suis arrivé , on m' a dit les difficultés du quotidien . Il y a un chef d' exploitation formidable , son fils , courageux , qui va reprendre le flambeau , on ne nous a pas parlé d' aides , on m' a dit une chose , une chose , &laquo; on veut pouvoir vivre dignement de notre travail &raquo; . Alors oui , le premier pilier de ce nouveau contrat que je veux refonder , entre les paysans et les Français , c' est de pouvoir vivre dignement de son travail et pour cela c' est d' avoir des prix qui rémunèrent le travail des agriculteurs en France . Non pas des aides , non , mais d' avoir le juste prix pour les produits . Alors , pour ce faire , il faut d'abord acter d' une chose , là aussi Monsieur Nedellec me l' a dit tout à l' heure : &laquo; Moi , je travaille avec à peu près les mêmes prix qu' il y a 30 ans , j' ai dû faire des investissements , me mettre aux normes &raquo; . Olivier , tu sais bien , tu m' as dit à peu près la même chose , le marché est plus difficile , les contraintes pour produire ne sont pas les mêmes , on a évidemment un droit social qui n' est pas le même , et c' est bien normal , mais les prix n' ont pas bougé . Parfois , même , ils ont diminué . Ce que je veux , c' est renforcer le pouvoir de négociation des agriculteurs , dans le respect de la législation européenne , pour qu' ils fassent le poids face aux industriels et aux distributeurs . Pour cela , je veux des associations d' organisations de producteurs aux pouvoirs renforcés , elles pourront négocier avec un véritable mandat des agriculteurs , des contrats cadres sur les volumes et le prix du lait . Leur création doit être simplifiée et encouragée , et lorsqu' une association d' organisations de producteurs existera , l' industriel ou la centrale d' achat sera dans l' obligation de négocier directement avec elle . Au fond , mon message , aujourd'hui , c' est que l' agriculture n' est pas simplement l' affaire des agriculteurs , et c' est aussi un appel à la responsabilité collective . On doit réorganiser les filières par ces contrats et ces associations , mais il faut aussi que chacune et chacun , nous soyons responsables . Et c' est donc un appel à la responsabilité des transformateurs , pour que , dans leurs marges , ils pensent aux producteurs et distribuent de manière juste la création de richesse . C' est un appel à la responsabilité de la grande distribution , en tant que ministre , je suis celui qui a augmenté partout les contrôles sur les marges et le respect de la loi sur les grands distributeurs . Mais je sais aussi dire la responsabilité avec laquelle plusieurs enseignes ont su valoriser des produits du terroir , ont su respecter certains producteurs pour maintenir les prix , valoriser . Et je sais , là-dessus aussi , le travail que nous avons mené avec plusieurs d' entre vous , il y a quelques mois , sur l' étiquetage et la valorisation des produits bretons . Mais c' est aussi un appel à la responsabilité de tous les Français , qui doivent être un peu plus que des consommateurs , mais des acteurs engagés pour leur pays . Chacune et chacun , lorsque nous consommons , nous devons être vigilants , valoriser là aussi les produits et donner une place aux circuits courts . Les circuits courts ne sont pas la panacée , tout le monde ne peut pas y arriver . Mais il faut , partout où cela a du sens , le favoriser , l' aider , le valoriser . Et donc , vous le voyez , pour répondre à ce problème de prix , c' est cet ensemble de mesures que nous devons prendre . Mais , au-delà , la deuxième mesure que je veux prendre pour permettre de vivre dignement de son travail , c' est également de continuer et d' encourager la montée en gamme ; c' est indispensable . Qu' est -ce que cela veut dire , concrètement ? Ça veut dire que les standards qui existent dans nos régions , dans nombre de nos produits aujourd'hui , sont des bons standards . Il ne faut pas laisser dire des choses là-dessus indues ou impropres - j' entends des discours qui sont souvent désobligeants pour nos agriculteurs . Mais il y a plusieurs modèles , et je ne défends pas un modèle unique . Il faut un modèle parfois plus intensif , et , dans ces cas-là , mieux orienté vers l' export . On s' en souvient chez Richard et c' est ce qu' on a fait avec Doux et toutes celles et ceux qui travaillent autour de cette belle entreprise , et qui , en Bretagne , produisent des poulets de manière intensive et qui les exportent vers d' autres marchés . Il y aussi des modèles moins intensifs , qu' il faut également privilégier et protéger à travers des circuits qu' il faut , comme je le disais , encourager . Mais il y a une chose qui ne marche pas : c' est de ne pas répondre aux besoins des consommateurs et de son marché . Et donc , nous devons valoriser nos produits , continuer à aider à l' export , mais en même temps , permettre partout sur le territoire à nos paysans de monter en gamme , d' améliorer la qualité , d' aider à investir pour faire de meilleurs produits . Produire mieux , produire avec des labels , produire en polluant moins , produire en inventant , là aussi , de nouveaux types de production . Je veux ici être très clair avec certaines et certains qui mènent des combats rétrogrades : on a beaucoup trop opposé l' agriculture et l' environnement . Beaucoup trop ! Dans la montée en gamme , il y a l' indispensable montée en gamme pour répondre aux contraintes environnementales , et là-dessus les paysans bretons n' ont aucune leçon à recevoir , aucune ! Parce que la qualité de l' eau bretonne est largement due au travail , décennie après décennie , des paysans bretons . Mais pour permettre la montée en gamme , l' innovation , cette réconciliation , on doit aller plus loin . Et je veux , moi aussi , mener cette transformation environnementale de notre modèle productif . C' est pourquoi je lancerai un programme d' investissement d' avenir agricole sur la période 2017-2022 ! C' est une première , cela n' a jamais été fait . On parle toujours de programmes d' investissement d' avenir pour les startups , pour les secteurs qui paraissent construire la France de demain . Mais la France de demain c' est aussi cette transformation de la France agricole que j' évoquais . Et donc cela permettra l' accès à des financements nouveaux - non pas de la nouvelle dépense publique , mais du financement d' investissements pour des projets de modernisation des exploitations ayant un impact positif sur l' environnement , pour des projets de développement d' activités et de transformation et de distribution qui privilégient notamment les circuits courts en privilégiant les projets coopératifs . Parce que l' investissement est clé pour réussir cette montée en gamme partout . Pour accélérer la montée en gamme des produits agricoles , il faut aussi permettre à nos agriculteurs de relever , comme je le disais , ce &laquo; défi- environnement &raquo; , d' accélérer cette transition et cette transformation . Et , donc , c' est l' objet de ce programme que je lancerai dès le début du quinquennat . Mais pour aller plus loin , on doit également valoriser le travail pour l' environnement qui est fait par les paysans et le valoriser comme une activité en propre , elle-même . Le rôle de l' Etat , c' est de les accompagner parce que dans les missions de l' agriculture du XXIe siècle , au-delà de tout ce que j' évoquais tout à l' heure , il y aura aussi celle d' accompagner cette transformation environnementale et énergétique , celle de produire de l' énergie par la mécanisation , par la biomasse , parfois par l' éolien , et par des services environnementaux indispensables à nos territoires et à nos paysages . Pour ce faire , je veux rémunérer les services environnementaux rendus par les agriculteurs à hauteur de deux cent millions d' euros par an , à travers la création d' un service de paiement pour services environnementaux . Et là aussi , ça n' est pas de la dépense publique en plus , ce système modeste mais important , c' est de la réattribution de certains mécanismes qui ne prennent pas suffisamment en compte les nécessaires investissements pour l' environnement . Le deuxième pilier , ce sont les normes . Et nous en avons bien besoin , là-dessus je crois que je n' ai pas à vous convaincre . Il y a trop de normes , il y a des normes qui sont compliquées à appliquer . Et donc , le projet que je veux porter en la matière est simple : premièrement , revoir toutes les normes inutiles dans les cinq années à venir , porter un projet de révision de tout ce qui a , jusqu' alors , été sur-transposé pour le supprimer , porter un projet de simplification normative dans le champ de l' agriculture qui est absolument indispensable . Deuxièmement , permettre un droit à l' erreur pour tous les particuliers , pour toutes les entreprises et , en particulier , pour les exploitants agricoles . Ce droit à l' erreur , qu' est -ce que c' est ? Eh bien c' est la possibilité , quand on est contrôlé pour la première fois , et qu' on a commis une infraction qui n' est pas une infraction pénale - où on ne met pas en cause la vie d' autrui – , eh bien , de ne pas avoir une administration qui vous sanctionne , qui vous met tout de suite des pénalités , mais une administration qui vous conseille , qui vous corrige . C' est une révolution culturelle , conceptuelle , profonde . J' ai eu l' insigne honneur d' être ministre de la République . J' ai vu la dignité de nos fonctionnaires et je n' ai jamais voulu tomber dans cette espèce de discours de facilité où on oppose un secteur à l' autre . Ah , combien de ministres sont venus devant vous , dire &laquo; Ah , l' administration c' est un calvaire . &raquo; C' est facile , on veut être populaire . Quand on veut être démagogue , on va devant des paysans et on leur dit &laquo; L' administration , c' est votre problème . &raquo; . On leur dit aussi &laquo; L' Europe , c' est votre problème . &raquo; C' est faux . D'abord , il n' y a pas de mauvaise administration . Il y a des mauvais ministres , ça c' est vrai . Quand vous dirigez votre exploitation agricole , votre entreprise , votre exploitation , votre classe , votre association , que sais -je , votre mairie , il ne vous viendrait pas à l' idée de dire &laquo; Ils sont mauvais . &raquo; Mais quand on dirige l' Etat , il est de bon ton de le dire . Non . Mais , on s' est enfermé dans une espèce d' opposition culturelle où la dignité de l' administration est bien souvent de contrôler ou de sanctionner et du coup , le reste de la vie économique et sociale se sent comme oppressé . Eh bien ce droit à l' erreur , c' est le rôle reconnu à l' administration , évidemment de contrôler , d' aller sur le terrain , s' assurer de l' égalité républicaine en matière de normes , mais , partout , d' accompagner les uns et les autres , de les conseiller , jamais de ne les sanctionner la première fois . C' est fondamental en matière de normes . Enfin , le troisième pilier de cette ambition pour l' agriculture , ce pacte refondé , c' est l' Europe et la PAC . Ah oui , il y en a beaucoup qui viennent vous expliquer que l' Europe , c' est la source de tous les maux . Mais , on était tout à l' heure avec vos représentants , cher Olivier , on était avec la FDSE , on était chez vous , Monsieur Nedellec . J' étais à Rennes , il y a quelques mois . On ne m' a pas dit &laquo; On ne veut plus de l' Europe &raquo; . On ne m' a pas dit &laquo; L' Europe , c' est notre problème &raquo; . On m' a dit &laquo; Les réformes européennes qui ont été mal préparées , mal pensées , les dérégulations européennes , les incohérences européennes , c' est mon problème , mais pas l' Europe en elle-même . &raquo; C' est très différent . Alors notre projet pour l' Europe , c' est d' avoir moins d' interventionnisme hors-sol , d' avoir moins de directives , dénuées de sens , et donc d' avoir une Europe plus concrète . L' Europe ne peut fonctionner et être acceptée que si elle est un moteur de progrès , que si elle est synonyme de soutien , que si elle se bat pour les peuple et non contre eux . Et ma conviction , c' est qu' il est urgent de renégocier les règles de la concurrence pour que cette concurrence soit non-faussée . Et il n' est pas normal que des travailleurs détachés des pays de l' Est peuplent les abattoirs allemands , alors qu' ils sont payés quatre euros de l' heure avant la mise en place d' un salaire minimum ; alors qu' ils ne sont pas soumis aux mêmes règles . Et donc oui , je mènerai à son terme une réforme de la directive sur le travail détaché , elle est indispensable . Voilà pourquoi il faut défendre une harmonisation vers le haut des normes sociales et environnementales . Qu' est -ce que cela veut dire ? Non pas détricoter les règles françaises , comme certaines ou certains vous le promettent , non pas dire &laquo; On fera moins d' environnement &raquo; , mais dire &laquo; Ce qui est imposé en France doit l' être partout en Europe de la même façon , avoir les mêmes contrôles &raquo; . Et dans le même temps , il faut défendre , défendre nos agriculteurs à Bruxelles . Et je dois dire ici , parce que je l' ai vu faire moi-même au gouvernement , que , contrairement à ce que beaucoup ont parfois dit , Stéphane Le Foll en tant que ministre , s' est battu à Bruxelles et il a fait le maximum dans le cadre de la PAC qui avait été renégociée pour défendre les intérêts de l' agriculture française et je veux ici le défendre contre les caricatures parfois faites . Par contre , nous avons parfois mal négocié collectivement , de droite et de gauche , les transitions , on avait mal préparé la dérégulation , on avait mal préparé la sortie des quotas , on avait mal modernisé nos structures , et donc oui , la dérégulation européenne nous en avons été victimes et c' est cela que nous devons corriger . Voilà pourquoi je veux , dès mai 2017 , dans le cadre des négociations pour la PAC 2021 , que la France propose la création , dans le cadre du premier pilier et d' une PAC au budget préservé , d' un système de subventions contra-cyclique de garantie sur les prix ou le chiffre d' affaires . C' est indispensable . Qu' est -ce que cela veut dire , mes amis ? Ça veut dire protéger tous les agriculteurs contre la volatilité des prix agricoles , ça veut dire éviter , parce que le prix d' un seul coup bascule ou s' effondre , parce qu' il y a un éternuement en Australie ou ailleurs , parce que le marché chinois s' effondre , parce que certains sur-produisent , éviter que les prix ne s' effondrent et que , de ce fait , certaines exploitations , des milliers , parfois des centaines de milliers à travers l' Europe , ne doivent mettre la clé sous la porte . C' est éviter des suicides , c' est éviter des situations profondément injustes , liées non pas à une non-compétitivité , liées non pas au fait que ces exploitations agricoles ne peuvent pas relever les défis nouveaux mais qu' il y a une volatilité infernale , des prix devenus irrationnels , c' est le devoir de l' Europe de protéger son agriculture face à cela avec des vrais mécanismes de régulation des volumes en cas de crise . Et puis , bien sûr , je veux avoir un mot pour ces travailleurs de la mer , cher Pierre . L' autre pilier de la Bretagne , c' est la pêche . La pêche , elle est pourvoyeuse d' emplois , de dynamisme . Imaginerait -on le Guilvinec ou Lorient sans leurs bateaux de pêche , sans leurs pêcheurs ? Mais elle en dit long surtout sur les valeurs qui sont les nôtres : le courage , la solidarité , l' humilité face à la nature , face aux éléments parfois . Nous sommes , ce soir , le lendemain des treize ans du Bugaled Breizh et de ses victimes et je veux avoir un mot pour eux ce soir et une pensée . Nos pêcheurs , ce sont des courageux , ce sont des vaillants , ce sont des femmes et des hommes qui , dans la nuit , quittent leurs familles parfois pour longtemps . Ah , je les connais , moi je suis des Hauts-de-France comme on dit maintenant , c' est une autre pêche , ils sont fiers , ils aiment leur métier et le risque qui va avec et , eux aussi , ils ne demandent pas qu' on leur enlève du risque . Ce sont des aventuriers et des ambitieux au vrai sens du terme . Ces dernières années , ils ont eu des bons résultats et je m' en félicite parce qu' il y a eu un pétrole moins cher , parce que les efforts ont payé . Ah , je les connais ceux qui se sont battus pour que la langouste rouge puisse revenir dans les eaux de la mer d' Iroise , il y en a tellement qui se sont battus sur de telles conquêtes mais ces résultats , ils ne doivent pas masquer une situation qui reste préoccupante . Pour permettre à nos pêcheurs de vivre de leur travail , nous devons impérativement améliorer la compétitivité de leurs entreprises , les aider dans le renouvellement de leurs outils de travail et en particulier des bateaux de pêche pour faire justement des bateaux de pêche plus propres , mieux adaptés , plus économes , c' est cela qu' ils demandent et il faut donc adapter nos dispositifs nationaux et européens pour ce faire . Cela implique aussi de rendre la politique européenne - et ça fait partie du combat que je mènerai , pour la politique européenne en la matière - plus souple , ça veut dire avoir des stocks , des quotas qui sont adaptés à la réalité du terrain . Ne pas avoir les mêmes quotas partout et pour tout le monde parce que quand les stocks existent , on peut ré-autoriser certaines pêches . Mais aussi des perspectives car nous devons désormais nous organiser selon des quotas pluriannuels qui permettront aux pêcheurs de voir où ils vont et de cesser , sans mauvais jeu de mots , de naviguer à vue . Voilà l' ambition , mes amis , que je veux porter pour notre agriculture . Alors , je pourrais vous parler de bien d' autres de nos ambitions mais un dernier mot pour notre ambition européenne . Parce que oui , l' ambition que nous devons avoir , cette ambition de progrès , celle qui réconcilie la liberté et la protection , elle est au coeur de notre ambition européenne . Ils sont trop nombreux ceux qui l' ont oubliée , on a laissé le doute s' installer . Moi je sais que je suis ici dans des terres toutes particulières , c' est pour ça que je les aime , dans des terres où l' engagement associatif , le militantisme laïque , la démocratie chrétienne ont su se tresser , se réconcilier . Et dans ces terres , depuis le début , on est fier de la Bretagne , on aime la République parfois plus que partout et là-dessus , beaucoup se sont trompés . &laquo; Ah les Bretons peuvent se cabrer contre l' Etat &raquo; , je le sais , mais jamais ils ne le font contre la République . Mais ici , on aime l' Europe plus que tout . Ici , parce que sans la Bretagne , sans les grandes terres de l' Ouest , l' Europe ne serait pas là . Sans les grandes terres de l' Ouest , la France aurait dit non à l' euro ! Je ne l' oublierai jamais ! Alors cette Europe , je la défendrai avec vous , j' ai porté notre projet européen la semaine dernière à Berlin . C' est un projet pour relancer l' Europe . C' est un projet où la France doit prendre ses responsabilités , mener ses réformes , mener avec courage ce qu' elle doit faire pour elle . Mais en même temps , nous allons demander à l' Europe plus d' investissements , plus d' intégration à quelques uns et relancer en Europe une souveraineté nouvelle , une souveraineté pour protéger nos concitoyens face aux risques du monde : le terrorisme , l' immigration , la transformation numérique , les attaques contre notre alimentation . L' Europe , c' est un projet d' ambition et de souveraineté . L' Europe , ne l' oubliez jamais , ce sont nos valeurs portées dans le monde et , en même temps , ce qui nous protège . C' est cela l' Europe , c' est notre projet et nous le porterons ! Mes amis , je le dis souvent , nous sommes les seuls à porter le projet européen et donc nous le ferons avec fierté et je veux dire ce soir à M. Trump qui , depuis l' autre côté de notre océan , devrait avoir un peu plus d' humilité . M. Trump l' Américain , n' oubliez jamais que si vous êtes une nation libre , c' est parce que des ambitieux sont partis de ces terres avec l' amour de la liberté , avec le même rêve , le rêve français , le rêve européen ! M. Trump , n' oubliez jamais ce que vous nous devez , la liberté , votre existence . M. Trump , regardez votre histoire , c' est celle de Lafayette , c' est la nôtre ! Parce que de la même façon , M. Trump , je regarde notre histoire . Je sais les jeunes Américains qui n' avaient jamais connu l' Europe , qui n' avaient jamais connu la France et sont venus mourir chez nous , sur nos plages , sur notre terre . Je sais ces jeunes Américains , je sais . Ces Américains qui ont cru dans l' Europe avant même les Européens . Ces Américains qui nous ont aimés au moment où nous doutions , les Marshall et tant d' autres . Alors oui , je vous le dis ce soir , le destin , le destin de la France , c' est un destin européen , le destin de la France et de l' Europe , c' est aussi le destin américain ; cela personne ne peut le trahir . Voilà mes amis , voilà mes amis , ce que je voulais vous dire ce soir et je répondrai à toutes les questions après , Monsieur , cher ami , vous viendrez après , on viendra vous amener après mon meeting et je répondrai à toutes les questions sinon quel est le sens d' une démocratie où je ne répondrais qu' à une question parmi plus de deux mille ? Voilà mes amis , j' aurais pu vous parler des heures de nos ambitions , de notre ambition pour la France . J' aurais pu vous parler de l' ambition , cher Monsieur SALAÜN , de votre grand-père , vous m' en parliez tout à l' heure , quand il a quitté les terres bretonnes pour aller conquérir l' Amérique et vous permettre aujourd'hui d' aller à la conquête , avec la prochaine génération , de nouveaux marchés . J' aurais pu vous parler d' ambition de tant et tant de Bretons fiers d' eux-mêmes , de ce qu' ils sont , de ce que cette terre porte et que nous lui devons parce que la Bretagne est une terre qui ose , parce que la Bretagne est une terre ambitieuse , notre ambition , c' est le projet pour le pays que nous portons . L' ambition , elle est avant tout celle de votre engagement au quotidien et , oui , cher monsieur vous le disiez , l' engagement sur lequel je compte , c' est aussi celui des territoires , tout ne se fera plus d' en haut , l' engagement que je porte et l' ambition que je porte est aussi une ambition citoyenne des territoires ; ce n' est pas innocent . Je viens ici , sur ces terres bretonnes , à Quimper , là où le Général De Gaulle fit son dernier grand discours , celui sur la décentralisation parce que cette ambition que je porte c' est aussi une ambition pour redonner le pouvoir aux territoires , redonner le pouvoir aux régions ! Cette ambition que je porte parce que c' est la vôtre , c' est une ambition de redonner le pouvoir à chacune et chacun , aux citoyens , aux territoires , aux élus . Je vous dis , cette Révolution &laquo; En Marche ! &raquo; que nous portons depuis tant de mois et que nous allons porter jusqu' au bout , c' est un projet d' ambition mais c' est un projet qui est le nôtre mes amis ! Ce que je propose n' est en rien facile , n' est en rien innocent , ça n' est pas de traiter des clientèles , ça n' est pas de faire plaisir à quelques-uns . Ce projet , c' est le vôtre parce qu' à chaque instant , je vous demanderai , avec moi , de le porter , de le réinventer , mais une fois que nous aurons pris le pouvoir , ce sera , pour moi , pour vous le rendre . Parce que oui , mes amis , j' ai besoin , dans les cent jours qui viennent , de votre énergie pour porter ce projet d' ambition partout , parce que j' ai besoin partout de votre fougue , de votre engagement , parce que j' aurai besoin demain de l' énergie de tous les territoires , de la Bretagne , de l' énergie des élus , des associations , des citoyens parce que cette ambition , c' est la nôtre ! Alors ensemble , je vous en fais le serment , nous irons jusqu' au bout ! Vive la Bretagne , vive la République , vive la France ! Bravo ! Merci à vous ! ",
    "Merci d être ici aujourd'hui , je tenais à pouvoir vous présenter les prochaines étapes de notre campagne présidentielle et les principes qui régiront , pour En Marche ! , les élections législatives . En Marche ! est aujourd'hui un mouvement politique qui compte plus de 145 000 adhérents . Ce mouvement , inédit dans la Cinquième République , appartient à celles et ceux qui depuis le début ont eu le courage , la volonté , la pugnacité de refonder notre vie politique afin de faire réussir notre pays . Ce mouvement traduit une volonté de s' engager , de s' approprier la vie politique et civique française , et une attente profonde de transformation . Je suis très fier de cette dynamique collective et pleinement conscient , dans le contexte , de mes responsabilités . C' est parce que je suis le garant de ces aspirations , qui m' obligent , que je veux ici , pour vous et pour nos adhérents , clarifier les prochaines étapes de notre cheminement et notre volonté , précisément , de nous affranchir des jeux d' appareils qui ont conduit à bloquer notre pays . Nous avons , à En Marche ! , présenté clairement au pays , dès le mois d' avril dernier , nos valeurs et nos objectifs . Ils fondent le progressisme que nous portons : pour la transition environnementale et numérique , pour la flexi-sécurité , pour l' égalité des chances , pour le renouveau démocratique et culturel , pour une société forte et un État laïc , garant de la cohérence nationale , et enfin pour le projet européen . Nous avons ensuite établi le diagnostic du pays au contact des Français grâce à la Grande Marche qui , durant plusieurs mois , a permis d' aller entendre plus de 100 000 de nos concitoyens . Ce diagnostic a été présenté à l' automne . Notre vision a ensuite été exposée dans le livre Révolution . Cette vision était le préalable indispensable à une liste de mesures que nombre de candidats égrènent pour ensuite les modifier : la société du travail , la sortie d' une société de statuts , la possibilité de choisir sa vie et tout ce que j' ai pu détailler durant cet automne était le cœur de cette vision . Depuis le mois de décembre , je présente notre projet thème par thème : le travail , la santé , l' école , l' agriculture , l' Europe en cohérence avec cette vision initialement posée . Je poursuivrai cette présentation durant les prochaines semaines sur l' international , la culture , l' environnement , la sécurité , la défense , la justice , et bien d' autres sujets . J' aurai jusqu' à la fin de la campagne l' occasion de m' exprimer sur tous les sujets comme le requiert une élection présidentielle . Jusqu' à la fin de cette campagne , grâce à la mobilisation de tous nos comités locaux , nous améliorons et améliorerons la plateforme programmatique que je porte . Mais parce que je crois qu' une élection présidentielle exige hauteur de vue et cohérence , je présenterai d' ici à début mars le cœur de notre projet , celui d' un contrat avec la nation . Il s' agit du plan de transformation dont j' ai parlé à l' automne dernier , c'est-à-dire la méthode de gouvernement , les grands objectifs et les réformes-clés avec leur calendrier – je veux parler des réformes en profondeur , celles qui transforment et refondent , et non celles qui ajustent . J' ai commencé à en présenter quelques-unes sur la formation professionnelle , le travail , l' apprentissage , ou l' école . Ce plan de transformation , qui sera donc synthétisé d' ici début mars , sera accompagné d' un cadrage budgétaire et financier précis qui permettra d' améliorer notre capacité à investir pour préparer l' avenir et en même temps réduire nos dépenses courantes . Ce contrat avec la nation sera le cœur de mon engagement présidentiel . Ce seront les engagements qu' en tant que Président de la République je superviserai directement . En ce qui concerne les élections législatives , En Marche ! présentera 577 candidats , ce dans le but de construire une majorité présidentielle . Jusqu' aux élections présidentielles , En Marche ! restera un mouvement politique permettant l' adhésion multiple . À compter d' aujourd'hui , nous ouvrons un appel à candidatures pour nos élections législatives . Tous les membres du mouvement ou ceux qui souhaiteraient le rejoindre peuvent à compter de ce jour s' inscrire en ligne . Dans les minutes qui suivront cette conférence de presse , le formulaire sera disponible en ligne pour tout un chacun . Nous sommes la seule force politique à avoir établi des formalités aussi claires et accessibles . Mon souhait est que toutes celles et ceux qui veulent s' engager puissent le faire . Nous procéderons de la sorte pour les élections locales , nationales et européennes à venir . C' est ce qui est conforme à notre engagement initial et à ce que nos adhérents depuis le début souhaitent . Les désignations aux élections se feront sur cinq critères qu' il m' appartient , en tant que Président d' En Marche ! , d' exposer : - Le renouvellement : ces élections devront permettre de faire émerger des personnalités engagées et talentueuses , élus de terrain , engagés associatifs , retraités , étudiants , fonctionnaires , salariés , entrepreneurs , professions libérales , agriculteurs et indépendants . La diversité sera prise en compte . Ce renouvellement est l' une des traductions concrètes de l' engagement dans notre mouvement . Ce renouvellement inédit que seul notre mouvement peut porter améliorera la représentativité des Français et des territoires . Plus de la moitié de nos candidats doivent incarner ce renouvellement . - La parité réelle : les femmes représentent aujourd'hui 53 % du corps électoral , il n' est pas acceptable qu' elles représentent moins de 30 % des élus de l' Assemblée Nationale . Nous respecterons donc un principe de parité strict y compris dans les circonscriptions favorables , contrairement aux autres partis . - La probité : nos candidats doivent démontrer une intégrité absolue dans leurs valeurs et leurs actions . C' est une condition indispensable pour réconcilier les Français avec la vie politique . Aucun de nos candidats ne devra donc avoir de casier judiciaire ni avoir subi des peines d' inéligibilité . Il s' agit là aussi d' un principe qu' aucune autre force politique n' a retenu . - La pluralité politique : En Marche ! est une force politique centrale , un rassemblement large de progressistes . Il dépasse les clivages et les jeux d' appareils . À ce titre , je veux être , ce matin avec vous , clair : Il n' y aura aucun accord d' appareil avec quelque parti que ce soit ; ce sont les idées et la volonté de transformer le pays qui doivent nous unir . Je ne négocierai aucune circonscription contre un ralliement et ne sacrifierai rien de nos engagements initiaux qui nous ont menés jusqu' ici . Je ne suis le candidat d' aucun parti ou aucun appareil existant . En Marche ! est là pour accueillir et porter les candidatures de citoyens engagés . Ils peuvent être par ailleurs socialistes , radicaux , écologistes , centristes , et républicains , tant qu' ils se retrouvent dans notre projet . Tous les parlementaires dans ces formations républicaines qui se reconnaissent dans nos valeurs et notre action sont donc les bienvenus pour porter les couleurs d' En Marche ! dans le respect de l' ensemble des principes ici rappelés . Nous veillerons dans les investitures à ce que ce pluralisme soit respecté et ces équilibres tenus . - L' efficacité et la clarté : tous les candidats investis signeront le même contrat avec la nation que moi . Aussi , aucun candidat investi ne pourra exprimer de désaccord avec le cœur de notre projet et il le portera clairement devant ses électeurs . En revanche , notre mouvement est riche de diversité qui est en même temps sa force de rassemblement . Il est évident que sur de nombreux sujets nous avons des différences , elles s' expriment aujourd'hui dans les débats au sein du mouvement comme demain elles s' exprimeront au Parlement . Mais afin de gouverner efficacement , tous les candidats s' engageront sur ce contrat avec la nation . Parce que c' est le contrat sur lequel je m' engage moi-même vis-à-vis des Français . Une commission nationale en charge des investitures sera constituée dans les prochains jours . Elle sera l' instance en charge des désignations . Elle sera composée de neuf membres adhérents du mouvement s' engageant eux-mêmes à ne pas être candidats . Je n' en serai pas membre . Les investitures seront décidées par vagues successives . Avec un objectif : construire une majorité de projet pour gouverner et traduire en actes notre engagement collectif et citoyen . Les travaux de cette commission seront supervisés par une commission d' éthique composée de trois personnalités indépendantes du mouvement , et qui pourra être saisie par tout membre du mouvement ayant déposé sa candidature à l' investiture . Je vous remercie pour votre attention , et je vais maintenant répondre à cinq questions ."
]


melenchon = [
    "Oui , volontiers . D'abord je souhaiterais que l' on dépersonnalise la discussion . Parce-que je trouve que depuis sept mois , on a excessivement personnalisé la critique de ma campagne , en la concentrant sur moi comme individu . C' est très mauvais , ça crée une ambiance trop tendue et au fond qui n' a pas de solution , car comme individu je ne changerai pas . Le fond de l' affaire est politique depuis le début : Nous avons eu un entretien , Pierre Laurent et moi , au mois de décembre de l' année 2015 , et je lui ai dit : &laquo; Pierre , je pense que nous avons le besoin d' une campagne de longue durée pour ancrer , car la situation va être très volatile &raquo; . Je le pressentais . Je savais qu' il y aurait une extraordinaire agitation , à la limite de la dislocation au sein du Parti socialiste . Nous savions tous qu' il y aurait une primaire à droite . Donc on pouvait assez facilement comprendre que la scène politique serait tellement agitée qu' elle pourrait être très rapidement illisible . Pas illisible par nous , militants , nous y voyons toujours clair . Mais illisible pour la masse des Français , c' est à eux qu' il faut penser , qui sont déjà dans un état de désorientation extrême . C'est-à-dire que des gens de gauche ne reconnaissent plus la gauche dans l' étiquette du gouvernement , ça ne veut plus rien dire pour eux . Si François Hollande c' est la gauche , si la loi El Khomri c' est la gauche , bon alors , qu' est -ce que la droite ? Et des gens de droite qui disent : &laquo; Mais nous , ce n' est pas ça notre vision de la Patrie , de la République de se brouiller en permanence avec l' autre partie de la population , celle qui est musulmane , ou que sais -je encore , les pauvres qui sont des profiteurs , d'après les gens de droite &raquo; . Beaucoup de gens de droite ne se reconnaissent pas là-dedans . Donc il y a en profondeur une désorientation du peuple français . C' est la raison pour laquelle il faut être clair dès le début , en disant d'abord &laquo; on va parler à tout le monde et pas seulement à ceux de gauche &raquo; . Bref , je vais être aussi synthétique que possible , Pierre m' a dit : &laquo; ça ne peut pas fonctionner s' il n' y a pas un rassemblement de la gauche &raquo; . Je ne crois pas du tout à cette thèse . Je le lui ai dit . Je ne crois pas que ce soit en rassemblant des étiquettes , des partis et des états-majors , qu' on va créer un enthousiasme populaire suffisant pour passer la barre . Je crois même le contraire . Si on donne l' impression qu' on est retournés dans les magouilles et les ententes en demi-teinte , on va encore aggraver la désorientation . Donc nous avons besoin de partir de loin . Nous nous sommes quittés sur un constat de désaccord de stratégie politique . Pierre a établi son calendrier pour le mouvement communiste , et j' ai établi le mien pour ce que j' allais entreprendre . Et c' est comme ça que j' ai décidé de démarrer le 11 février . Pourquoi cette date-là , je peux le dire maintenant , parce que nous étions dans un moment où apparaissait en même temps la candidature de Marine le Pen et où Sarkozy , s' exprimant quelques jours auparavant , avait clairement fait comprendre qu' il était candidat . Donc la scène était entièrement occupée par la droite . Et nous , notre famille politique , on serait là en train de se disputer les uns avec les autres , parce que une primaire c' est fondamentalement un exercice mortel de surenchères les uns sur les autres . J' ajoute donc à cette occasion , que le système des primaires n' est qu' apparemment démocratique . Il ne l' est pas en réalité . C' est un tamis social . Seuls les centre-ville urbains et aisés participent dans la plus grande partie des cas à des élections primaires . Et une infime partie de l' électorat . Par exemple les primaires américaines … Tout le monde a dit &laquo; , c' est formidable , c' est une culture … &raquo; . 9 % des électeurs ont voté dans toutes les primaires , c'est-à-dire démocrates et républicaines . C' est rien du tout . Et chez nous où il n' y a pas cette tradition , c' est encore moins . Par conséquent , je ne prends que cet aspect des primaires , je ne crois pas à cette méthode . Et d'ailleurs , ceux qui ont lancé l' idée d' une primaire de toute la gauche , Cohn-Bendit et le journal Libération , ne sont pas des références pour nous , en tout cas . Et d' entrée de jeu , Cohn-Bendit a dit : &laquo; Une primaire , c' est parfait parce que c' est le seul mode de désignation où Mélenchon et Pierre Laurent n' ont aucune chance &raquo; . Comme ça c' est clair . Et à peine n' avait -on même pas commencé , que déjà Julien Bayou disait &laquo; Ah oui , mais attention , si le candidat qui l' emporte est contre l' Europe , alors ça posera un problème &raquo; , parce que pour lui , être critique à l' égard de l' Union européenne , c' est être contre l' Europe . Ce n' est pas sérieux . Donc j' ai démarré le 11 février . Mais je demande aux communistes de faire appel à leur mémoire : c' est exactement la même méthode qu' en 2011 , exactement la même , avec un mois de retard . Le 21 janvier 2011 , j' ai proposé ma candidature . Tout seul . En tout cas en apparence . Comme cette fois -ci , je la propose tout seul et hors-cadre de partis . Pas &laquo; les partis , &laquo; de partis . Moi-même je suis membre d' un parti politique . Je l' ai été toute ma vie . J' ai été membre du Parti socialiste puis membre du Parti de gauche . Je n' ai pas cessé d' être membre de mon parti , je cotise . Donc je ne suis pas contre les partis , je n' ai pas cette sottise … Je suis &laquo; . Pourquoi hors-cadre ? Parce qu' il faut que tout le monde puisse investir la campagne . Je ne peux pas me présenter en disant que je suis l' homme de quatre , cinq , six , sept partis . Non . Voilà pourquoi , à la fois je propose ma candidature , et je crée le mouvement d' appui à ma candidature , la France insoumise . Vient qui veut : membres de partis , pas membres de partis , des gens qui ont voté autrement , des gens qui n' ont pas voté dans les précédentes élections … Il faut faire cette ouverture grand-angle . Et nous , qui sommes des militants , engagés politiquement toute notre vie , nous devons apprendre à traiter avec l' informel , avec l' état du pays tel qu' il est . Ce n' est pas à nous de n' accepter que le peuple qui nous convient , celui qui a l' habitude de se référer aux partis politiques . Non , il y a plein de gens qui ne sont adhérents à rien . Et alors on leur dit &laquo; non , vous ne comptez pas &raquo; . Bien sûr que non . Donc c' est exactement le même calendrier , sauf que j' ai un mois de retard . J' ai proposé ma candidature le 21 janvier 2011 , tout seul , je ne l' ai pas fait au nom du Parti de gauche , je ne l' ai pas fait au nom du Parti communiste . Le Parti communiste a engagé une procédure interne , mais bien plus tôt que cette fois -ci . Et les communistes ont tranché et ils ont voté à la date du 18 juin . Le 18 juin de l' année 2011 , les communistes avaient pris leur décision , et le 18 juin , le soir , j' étais sur TF1 pour annoncer que dorénavant j' étais candidat investi par , à l' époque , le Parti de gauche , le Parti communiste et la Gauche unitaire . C' est ça le cadre de l' union . Ce n' est pas la même chose aujourd'hui : nous savons que les cartels de partis sont condamnés . Parce que nous avons fait fonctionner le Front de gauche comme un cartel de partis . C' est l' expérience concrète qui nous montre que ce n' est pas la bonne méthode . Ça ne veut pas dire qu' on renie ce qui a été fait par le Front de gauche . Mais nous devons porter plus loin l' expérience acquise . L' expérience acquise , c' est que non seulement il faut intégrer tous ceux qui ne veulent pas prendre la carte d' un parti mais qui veulent participer aux campagne , mais aussi tous ceux qui n' avaient jamais pensé faire de la politique . Voici le cran supérieur . Ensuite , les communistes ont leur calendrier . Je m' incline . Je l' accepte . M' avez -vous entendu une seule fois dire &laquo; ce n' est pas bien , je ne suis pas d'accord &raquo; . J' ai pu exprimer des regrets , dire &laquo; moi j' aimerais avoir l' aide des communistes aujourd'hui , des militants communistes &raquo; , mais je vais vous dire les choses comme elles sont : il y a déjà des milliers de communistes engagés dans la France insoumise , qui créent des groupes d' appui , qui collent des affiches et qui discutent surtout , qui discutent avec leurs voisins , leurs collègues de travail , parce que c' est comme ça qu' on va gagner . Et je le dis parce que je viens de dire &laquo; coller les affiches &raquo; , mais ce n' est pas le plus important . Le plus important , c' est le contact personnel . Chaque personne doit se faire une liste de dix autres personnes à convaincre et à accompagner jusqu' au jour du vote . Donc nous sommes dans le même calendrier , avec la même forme , avec les mêmes mots , croyez -moi , je les ai pesés et c' était les mêmes que la fois précédente : &laquo; Je propose ma candidature &raquo; . Et puis au moment où on est arrivés à plus de cent mille appuis , il ne faut pas non plus prendre les gens pour des imbéciles , ils ont bien compris que j' étais candidat . Et bien , je l' ai dit . Quand ? Le 5 juin . Où ? Place Stalingrad où avait commencé la campagne la fois d' avant . On a appelé le rassemblement après la fin du congrès communiste . Le congrès communiste s' achève vers 13h , 13h30 , le rassemblement commence à 15h . Il n' y a jamais eu de chevauchement . Je n' ai pas compris pourquoi on me faisait ce mauvais procès . J' observe d'ailleurs qu' on ne fait pas le même aux frondeurs qui se réunissent à la Rochelle pendant que nous sommes tous ici à la Fête de l' Huma qui est l' endroit où toute la gauche a l' habitude de se retrouver . Donc , pourquoi l' ai -je mis à cette date-là ? Parce qu' il y avait le congrès communistes et que je pouvais penser que dans le congrès communiste , comme la fois d' avant , une décision serait prise qui permettrait de converger . Elle ne l' a pas été . Mais qui peut penser que je vais mener un calendrier de campagne en expliquant aux gens &laquo; écoutez , rien n' est sûr , attendez que , je ne sais pas quand , le 5 novembre , les communistes commencent leur procédure interne … &raquo; . Car que je connais les statuts du Parti communiste . Le 5 novembre , ce n' est pas la prise de décision . Le 5 novembre , ça va être la décision qui va déclencher la procédure interne . Nous allons donc attendre le mois de décembre pour entrer en campagne , au milieu de la confusion des primaires socialistes ? Il n' en est pas question ! Par conséquent mon calendrier suit son cours . Et en cours de route bienvenu à tout le monde . Regardez la démarche du programme . Le programme , je ne pars pas de rien . Je suis stupéfait de voir que l' on puisse agir comme s' il n' y avait pas eu de programme la dernière fois . Il y a un programme : l' Humain d'abord . Il a recueilli quatre millions de voix . Il faut l' actualiser depuis 2012 . C' est un mot qui dans le passé nous a amenés à quelques déboires . Qui va actualiser ? Et bien d'abord nos électeurs puisque ce sont eux qui l' ont voté la dernière fois , ceux qui veulent s' engager dans la campagne , tous ceux qui veulent s' engager . Il y a eu plus de trois milles contributions . Je ne dis pas que ce soit un chiffre extraordinaire , mais trois milles contributions . On a fait deux synthèses en cours de route , pour montrer aux gens où on en était , et rebondir . Nous avons fait des auditions de spécialistes . Et là nous sommes entrés dans un dialogue avec les groupements politiques qui décident de participer à la France insoumise . Le groupe des socialistes dissidents , de la Nouvelle gauche socialiste , a déposé un document de 75 pages de propositions . Le Parti de gauche a amené ses 1000 fiches . Des communistes qui participent à la campagne amènent leurs contributions . On discute . Mais ce n' est pas une discussion de parti à parti , d' organisation à organisation . On essaie de voir comment tout ça , par rapport à la formule actualisée du programme l' Humain d'abord , ça colle ou pas . On ne peut pas coller bout à bout ce que tout le monde veut , ce n' est pas possible . Il faut qu' il y ait une cohérence . Moi je suis très attaché à la cohérence économique , à la cohérence écologique du programme et à la cohérence républicaine . Donc tout ça compte beaucoup . Cette méthode va s' achever à l' occasion de la convention du mois d' octobre , qui se tient les 15 et 16 octobre . Pourquoi se tient -elle à ce moment-là ? Mais parce que j' ai des délais pour imprimer le livre pour qu' il soit prêt pour les cadeaux de Noël . Nous à Noël , on va offrir le programme de la campagne . Alors je vous dis un scoop , j' ai quand même quelques petits privilèges comme candidat , au moins celui de choisir quelques-uns des slogans … C' est moi qui ai choisi la France insoumise . Je cherchais une expression qui soit dans l' esprit de celle de 2012 . Vous vous rappelez , on avait fait &laquo; la France , la belle , la rebelle &raquo; , c' était pris dans la chanson de Jean Ferrat . Et on cherchait un mot qui corresponde au moment dans lequel on est . C' est l' insoumission qui brise l' ordre établi , le consentement à l' ordre , la résignation . Et la dernière période a été remplie d' insoumis , lanceurs d' alerte , ouvriers et ouvrières qui résistent dans leur entreprise . Bref , j' ai choisi &laquo; France &raquo; que je trouvais que ça parlait à tout le monde à la fois : aux syndicalistes , mais aussi à l' homme ou à la femme qui se lève chaque matin pour maintenir la cohésion de la famille , pour aller au boulot ou pour amener les gosses à l' école alors qu' il n' y a plus de boulot , parce qu' il faut tenir bon . Voilà , j' aimais ça . Le programme , on ne va pas le rappeler l' Humain d'abord , quoique c' était magnifique , mais , par respect pour ceux qui auraient voté l' Humain d'abord ou les organisations qui auraient adopté l' Humain d'abord et qui ne se retrouveraient pas dans ma campagne , je le comprends , je n' ai pas envie d' avoir des polémiques avec les uns et les autres , on va donc changer le titre . Je vais proposer que ça s' appelle &laquo; en &raquo; . &laquo; en &raquo; parce que c' est quelque chose que je partage avec les communistes et notamment les réflexions de leur dernier congrès . Nous autres , le grand courant collectiviste et progressiste , nous devons de nouveau et mieux motivé que jamais , faire partager l' idée que l' Humanité doit d'abord se préoccuper de ce qu' elle a en commun , à commencer par l' écosystème , le seul qui permette sa survie . Car l' écosystème est mis en danger par le productivisme . Et le productivisme est , lui , animé par la politique de l' offre du capitalisme financier . La planète va mourir , la civilisation humaine va mourir sous les coups du capitalisme financier et du productivisme . C' est pourquoi il s' agit d' appeler chaque personne à réfléchir , quoiqu'elle ait voté dans le passé , l' appeler à réfléchir en tant qu' être humain . Réfléchissez : notre avenir , nous l' avons en commun . Que vous soyez riche ou pauvre . Et vous autres , les riches , vous feriez bien de vous poser des questions . Vous ne le resterez pas longtemps , riches , à ce tarif là , avec des événements climatiques extrêmes . Et à quoi bon être riches , à quoi bon ? Pourquoi avoir pour rêve d' être milliardaire , comme propose Macron ? En quoi est -ce un rêve honnête ? Vous savez bien que celui qui a un milliard , il n' a qu' un rêve : en avoir un deuxième . A quoi ça sert ? Et celui qui est millionaire ne peut avoir qu' un rêve : avoir un deuxième million . Pour quoi faire ? Pour quoi faire ? Quel est le sens de cette accumulation ? Est -ce que vous êtes plus heureux quand vous passez d' un million à deux millions ? Est -ce que vous aimez d' avantage ? Est -ce que vous êtes aimé d' avantage ? Et puis pour quoi faire ? Pour acheter quoi ? Pour vivre comment ? Je pense que la richesse doit être interpelée aussi dans son immoralité fondamentale . Eux , ils pensent que la cupidité est un aiguillon de l' activité humaine . Reconnaissons que l' intérêt est un aiguillon , mais pas la cupidité , pas l' accumulation . Nous devons interpeler le sens moral de la richesse . Voilà pourquoi je pense qu' il faut appeler notre programme &laquo; en &raquo; : dans chaque personne , ne voyons d'abord que l' être humain . Au fond , c' est aller au bout de la logique de l' Humain d'abord . L' Humain d'abord dit &laquo; bon , on commence par ça &raquo; . Justement commençons par ça . Nous avons l' avenir en commun . Je vais essayer d' expliquer ça , je le fais à la Fête de l' Huma . C' est à la Fête de l' Huma que je l' annonce . Alors on me dit : &laquo; c' est toujours toi qui décides &raquo; . Il en faut bien un . Comment vous croyez que ça se passe d' habitude ? Il y a des gens qui parlent . Et à la fin les autres qu' est -ce qu' ils prennent ? Un communiquant pour décider . Pas moi . Je suis un militant politique , c' est moi qui fixe tous les mots d' ordre . En 2012 , on discute , et puis à la fin on tranche . Je ne dis pas que c' est génial . Je ne vous dis pas que tout le monde est d'accord et dit &laquo; ah c' est formidable &raquo; , la France insoumise , il n' y a rien au-dessus . On aurait pu appeler ça autrement . Mais maintenant c' est parti . Il faut arrêter l' ergotage permanent , couper les cheveux en quatre tout le temps . Les gens ont besoin de comprendre . Ils ne comprennent rien si nous mêmes on est les premiers à dire &laquo; ah ce qu' on a trouvé ça ne vaut rien . Finalement il y avait mieux que ça &raquo; . Alors on me propose des titres à rallonge . Je vous donne cet exemple parce que je sais que je m' adresse à des militants . L' enjeu du militantisme , c' est l' engagement des autres , leur conscience , l' éducation collective . Si les partis ont une fonction , c' est bien celle-là , d' être la mémoire des luttes , d' être les porteurs de l' expérience et d' être des éclaireurs , pas une avant-garde . A mon avis . Il y a des camarades qui sont avant-gardistes , je le comprends . Eclairer , ça veut dire : vous mettez la lumière qui permet de se repérer à chacun individuellement . Pas le guide suprême qui dit &laquo; allez on va tous par là &raquo; . Non regardez bien , les gens , observez ce que nous vous disons , et ensuite tirez vos conclusions . Mais écoutez ce qu' on vous dit . Ne vous laissez pas bourrer le crâne par la télé et par les grands médias . Ecoutez . Vous choisirez à la fin . Mais entendez ce que nous avons à dire . Et vous voyez bien comment l' ennemi concentre ses coups : son premier objectif c' est d' empêcher qu' on entende , c' est de vous mettre une mauvaise étiquette . Ils décident qu' ils me mettent à l' extrême gauche , mais je ne suis par d' extrême gauche . Ce n' est pas que je méprise l' extrême gauche . C' est que je ne suis pas d' extrême gauche . Il y a une extrême gauche . Ce n' est pas juste de faire croire que l' extrême gauche c' est moi . Non , l' extrême gauche c' est Nathalie Arthaud , c' est Philippe Poutou , c' est aussi le Parti communiste par certains aspects . Mais moi je ne suis pas membre du NPA , de Lutte ouvrière , et mon programme ce n' est pas ça . Donc ce n' est pas juste déjà de leur enlever le droit à l' existence , on les invisibilise si on fait comme ça . Et puis après on m' invisibilise en me classant dans une catégorie qui n' est pas la mienne , je n' ai rien à faire là-dedans . Vendredi 9 septembre , je suis allé à un truc qui s' appelle Produire en France . Vous savez ce que retient le Figaro ? Que , soi-disant , on aurait dit qu' on était tous les meilleurs pour produire en France . Mais pas du tout , c' était pas ça la compétition . Et que moi j' aurais proposé surtout des nationalisations . Ils m' ont mal écouté . Ils m' ont très mal écouté . On va remettre les choses au point . Moi j' ai dit aux patrons qui étaient là : je ne viens pas pour vous dire ce qui va vous faire plaisir , je perdrais mon temps . Je viens vous dire ce qu' il va falloir que vous fassiez pour vous rendre utiles à la Patrie . Il n' y en a pas un qui a sifflé . Personne n' a dit &laquo; mais qu' est -ce que c' est que cette manière de nous parler ? &raquo; . Je dis &laquo; pour tenir les comptes de vos entreprises , vous êtes assez grands , sinon , ce n' est pas la peine de faire patron . Mais moi je vais vous dire ce dont le pays a besoin . Est -ce que vous êtes capables de prendre votre place là-dedans ? Nous avons besoin de 400 000 personnes pour faire de l' agriculture paysanne relocalisée , 300 000 pour l' économie de la mer , 900 000 pour sortir du nucléaire . Alors ? Où je les trouve ? Ils n' y sont pas aujourd'hui , il faut les former . Et vous , vous êtes prêts à être les entreprises qui vont suivre ? Moi je vous donne de la visibilité , je vous donne de la stabilité et de la sécurité . Et vous vous n' allez pas faire que pleurer , pleurnicher que vous avez trop de charges , trop de ceci , trop de cela . C' est bon , quoi . En tout cas si vous voulez faire ça , allez en voir un autre . Mais si vous voulez entendre ce qu' on vous dit , vous verrez que votre avenir y a son intérêt , parce que nous allons faire une société d' économie mixte , on ne va pas faire le socialisme en cinq ans &raquo; . Qui est -ce qui propose de le faire en cinq ans ? Qu' il vienne , qu' on m' explique quelle est la stratégie pour faire ça ! Moi j' en ai une : l' Assemblée constituante , pour faire la transition . Bon voilà , alors merci si ce message arrive aux communistes , je serais enchanté , je sais qu' il n' est pas loin de leurs propres convictions , et pour le reste , ils vont décider . Et leur décision s' imposera à qui ? Aux communistes , pas à moi . Non . En effet , c' est une bataille . Je la mène . Je trouve qu' elle est intéressante en tant que combat , et j' appelle les militants à réfléchir à ça : sans cesse de nos faiblesses faisons une force . Je préfèrerais avoir mille signatures tranquille . Je ne les ai pas . Qu' est -ce que je fais alors ? Je me prends la tête dans les mains ? Je me mets à pleurer ? Et je gémis et me lamente ? Non , j' affronte ! C' est extrêmement pédagogique . Ça permet aux gens de comprendre que dans ce pays pour pouvoir être candidat , il faut d'abord avoir l' autorisation de notables qui n' ont rien demandé . Donc ça aide à la construction d' une conscience politique plus affinée . Maintenant , bon , on a dépassé les deux cents . Dans ces plus de deux cents , il y a un peu plus de 33 , 34 élus communistes . D'ailleurs cette nuit il y en eu d' autres . Et aujourd'hui dans la journée certains vont venir me voir … Mais oui , il y a eu de la rétention de signatures . Et puis il y a beaucoup d' élus communistes qui disent : &laquo; Bon ben , moi j' attends la décision du Parti &raquo; . Je le comprends . Ca me coûte , parce que si je les avais , on serait plus tranquilles . Et surtout , la banque prêterait des sous . Parce que la campagne sera remboursée : je pense évidemment faire beaucoup plus de 5 % . Donc à partir de là , j' ai besoin . Je ne peux pas faire une campagne qu' avec des bénévoles et des retraités , ce qui est le cas aujourd'hui . Attention , les bénévoles ne sont pas des amateurs . Mais il y a quand même quelques permanents qui doivent être dans la campagne , il y a du matériel à gérer … Donc , qu' est -ce qu' on fait ? On lance la campagne de parrainages et on lance la campagne pour les dons . Et on va gagner comme ça . Il n' y a pas de martingale , il n' y a pas de solution raccourcie , miraculeuse . Les communistes , ils viendront et il n' y aura pas qu' eux . Je pense que d' ici au 5 novembre on sera très avancés vers les cinq cents signatures . Et alors après encore plus , à mesure qu' on va avancer dans la campagne . Puis , je vais aller voir le Président de l' Union des maires , monsieur Baroin , et je vais lui dire : &laquo; mais monsieur , vous êtes de droite , vous , sauf erreur , vous êtes le Président de l' Union des maires . Alors qu' est -ce que passez votre temps à parler de démocratie et de tout ça , et vous faites de la rétention . Je vous demande en tant que Président de l' Union des maires d' appeler les maires à une réflexion , parce qu' ils ne sont pas là pour empêcher les gens d' être candidats , mais pour empêcher qu' à l' élection des candidatures folkloriques apparaissent . C' est ça leur mission &raquo; . J' ai fait quatre millions de voix la dernière fois , je ne suis pas une candidature folklorique . Je représente un courant d' opinion dans le pays . Vous pouvez signer pour moi , que vous soyez sans étiquette , que vous soyez communistes , socialistes . Les socialistes , ils vissent à double tour , ils menacent leurs élus , leur font peur . Vous verrez , à la fin c' est nous qui aurons le dernier mot , parce que contre des militants et des gens engagés , il est rare qu' il y ait une force suffisante pour nous empêcher de passer . Ça se fera . Je préfèrerais que ça se fasse avec les élus communistes , c' est évidemment mon souhait . Vous imaginez bien que je préfère être parrainé par une majorité de communistes que par une majorité de droite , mais je prendrai les signatures de là où elles viennent . Et ça m' ennuierait que les communistes arrivent les derniers , ça ne le ferait pas . ",
    "Nous n' avons pas la même vision des priorités ni de la cohésion nationale . Son programme s' appelle &laquo; Tout pour la France &raquo; , en vérité , c' est tout pour la finance . Il utilise &laquo; l' identité &raquo; comme un bélier pour abattre la résistance morale des gens . Son but est de diviser le peuple pour gagner la guerre sociale qu' il veut mener . Car son programme compile les mesures désastreuses pour les acquis sociaux : retraite à 64 ans , suppression la durée hebdomadaire légale du travail … C' est un retour au 18 ème siècle . C' est un écran de fumée . Comme , il ne peut se présenter devant les Français avec pour seul programme la régression sociale aux ordres de la Commission européenne , Nicolas Sarkozy l' agrémente de la haine pour tous . C' est d'abord le résultat d' un harcèlement inouï . Les responsables publics , et le système médiatique ont perdu tout sang-froid . L' origine des attentats terroristes est dans la guerre qui se mène au Moyen Orient . Là , il n' est pas question de religion mais d' oléoducs et de gazoducs . Mais dorénavant aussi tous les fous du monde peuvent commettre des crimes en se réclamant de l' islam même s' ils ne le pratiquent pas . Les services de sécurité font tout ce qu' ils peuvent . Mais la question des moyens humains à déployer reste posée . Je préfère des agents infiltrés présents sur le terrain à l' espionnage électronique généralisé par lequel on a cru pouvoir régler ce problème . Et bien sûr il faut arrêter la guerre . Le jour de l' entrée en dette écologique de la planète avance au 8 août mais on discute des tenues de bain ! On détourne l' attention de l' essentiel : l' argent roi . C' est lui qui pourrit nos vies , la planète et la paix . Là , la France bas des records : ce pays distribue le plus de dividendes en Europe . Il n' y a pas d' argent pour les payes et les services publics mais les actionnaires se gavent . Non . Il est candidat à la primaire du PS . Moi je serai candidat devant le pays jusqu' au bout . Sa sévère condamnation du bilan de François Hollande me sert . Elle est d'autant plus convaincante qu' il était aux premières loges . Ajouté à Benoît Hamon et Cecile Duflot , on voit ainsi pour la première fois trois anciens ministres essayer d' empêcher le président qui les a nommés d' être réélu tant il est nul . Non . A la fin , je serai là . François Hollande ne propose pas une candidature de gauche puisqu'il a continué tout le programme de Nicolas Sarkozy . Son ennemi devait être la finance et il a commencé par lui faire cadeau de 40 milliards d' euros . Si quelqu'un est capable de retrouver la trace de ces 40 milliards dans l' activité réelle , qu' il se signale . Qu' est ce qui est de gauche dans le bilan de François Hollande ? Il a fait pire que Nicolas Sarkozy . Mais le plus navrant , c' est que de Hollande à Sarkozy en passant par Juppé , Valls et Macron , tous ont comme projet de continuer à appliquer de manière encore beaucoup plus violentes des recettes qui ont tout détruits ces 20 dernières années . Ce sont les candidats de la Commission européenne . Non . La situation de l' emploi ne s' améliore pas . Nous n' observons que de petites fluctuations autour du point zéro . Ca montre que l' organisme n' est pas mort . Mais tant qu' on ne conduira pas une politique de relance , l' emploi ne repartira pas . Cette relance doit être écologiste . Je propose un choc de 100 milliards d' euros à injecter dans la planification écologique . Et c' est parfaitement finançable . On se guérira le pays par la démocratie . La présidentielle , ce n' est pas le PMU . Si on en reste à faire des paris sur le résultat sans se préoccuper des enjeux , il n' y a plus de démocratie . Le FN c' est le diable de service . Il est utilisé pour faire peur et rameuter . Mais pour quoi faire ? Continuer comme avant ? Mais c' est insupportable . Il n' y a pas une dispersion de la gauche , mais une opposition de programme . Pour certains , le problème c' est le peuple , pour moi c' est la solution . ",
    "Non , ce n' est pas un procès d' intention . François Fillon est le candidat des assureurs contre la Sécurité sociale . C' est ce qui ressort de la lecture de son programme . Tout le monde pourra aller vérifier ce que je vais relever . Le système Fillon repose sur le recul des remboursements par la Sécurité sociale et l' encouragement aux assureurs privés . Un autre indice nous a été donné la presse . France Inter affirme ainsi que François Fillon aurait déjà choisi son futur ministre de l' Économie et des Finances en cas de victoire à la présidentielle . Il nommerait à Bercy Henri de Castries , l' ancien PDG de l' assureur privé Axa pendant 16 ans ! Ce ne peut être un hasard . Henri de Castries n' a quitté la présidence de ce groupe qu' il y a quelques mois , pour devenir administrateur de la banque HSBC . Précisons , pour situer les perceptions de la vie que peut avoir le personnage : lors de sa dernière année chez Axa , en 2015 , il gagnait 5,4 millions d' euros de salaire ! Que peut -il comprendre aux raisons des 30 % de gens qui n' osent plus se soigner parce qu' une ordonnance , ça coûte trop cher … Le conflit d' intérêt serait immense si ce personnage devenait demain ministre des Finances chargé de faire voter le budget de la Sécurité sociale et la fiscalité des clients des assurances privées . Reste une évidence , Castries ou pas : le programme de François Fillon , c' est ouvertement et officiellement le début de la fin de la Sécurité sociale . À commencer par la fin de l' assurance maladie pour tous . Fillon l' a écrit noir sur blanc dans son programme pour la santé . Il veut &laquo; focaliser l' assurance publique universelle ( c'est-à-dire la Sécu ) sur des affections graves ou de longue durée , et l' assurance privée sur le reste &raquo; . Vous voilà prévenus . Le système collectif et solidaire actuel où chacun cotise selon ses moyens et reçoit selon ses besoins serait donc abandonné . Ou plus exactement limité aux seules &laquo; affections graves &raquo; . Et le reste ? Ce sera nécessairement à charge d' une assurance complémentaire . Bien sûr , pour faire bonne figure , Fillon dit que &laquo; les moins favorisés ne pouvant accéder à l' assurance privée bénéficieront d' un régime spécial de couverture accrue &raquo; mais sans dire qui cela concernera . Et quand on l' entend critiquer l' assistanat , on le voit mal élargir l' accès à la Couverture maladie universelle , la CMU . Il faudrait donc payer plus cher pour être moins remboursés quand on va chez le médecin ou chez le pharmacien et les inégalités se multiplieraient . C' est aussi un programme absurde du point de vue sanitaire . Car les &laquo; affections graves &raquo; ne le sont pas toujours depuis le début . Et les autres affections ne sont pas bénignes de la même manière pour tous , ni suivant les âges . Mais dans tous les cas , en se soignant de bonne heure , on évite souvent bien des complications et des aggravations . Il est donc totalement stupide de moins rembourser les soins courants . Le renoncement aux soins des plus pauvres fera exploser les dépenses de prises en charge aux urgences ou une fois les maladies devenues plus graves , c'est-à-dire plus douloureuses et dangereuses pour les patients et plus chères à prendre en charge . La santé n' est pas le seul domaine où François Fillon veut donner la main aux assureurs privés . Il tient le même discours à propos de la prise en charge du vieillissement de la population . En matière de retraite , il propose de repousser l' âge de départ à 65 ans . Mais aussi de &laquo; mettre en place un étage de retraite par capitalisation &raquo; , c'est-à-dire où chacun paye dans son coin une cotisation privée avec un cadeau fiscal à la clef . Naturellement cela n' a jamais garanti un montant de retraite où que ce soit dans le monde où ce régime est appliqué . C' est si vrai que partout où le retour dans le régime général a été proposé aux assurés et partout où le cadeau fiscal a été supprimé , le régime de capitalisation s' est volatilisé . À quoi bon recommencer une expérience qui ne marche pas ? Face à la dépendance aussi , François Fillon récuse le mécanisme de la solidarité et prône le chacun pour soi . Son programme prévoit d' &laquo; encourager la généralisation d' une couverture assurantielle dépendance privée sur une base volontaire par des contrats labellisés avec des incitations fiscales qui permettront aux particuliers de déduire de leurs impôts une part de leur cotisation à une assurance dépendance &raquo; . Ceux qui peuvent payer auront droit à une assurance contre la dépendance ? Et les autres ? Continueront -ils à souffrir en silence comme aujourd'hui face au manque de places accessibles dans des établissements pour personnes âgées dépendantes ? Devront -ils faire appel à la charité ? Les familles continueront -elles à se saigner en argent , temps et énergie pour aider les parents ? Ce programme a un but . Transférer au privé les milliards d' euros de cotisations maladie et retraite aujourd'hui gérés par la Sécurité sociale et offrir aux assureurs les milliards en vue pour financer la dépendance des personnes âgées . Car c' est un marché où certains voient bien quels juteux profits sont possibles . En cela , la rumeur selon laquelle Henri de Castries deviendrait ministre de l' Économie et des Finances de François Fillon est éclairante . Henri de Castries , est un symbole de l' oligarchie . Il coche toutes les cases de la caste : issu de la noblesse , diplômé de l' ENA dans la même promotion &laquo; Voltaire &raquo; que François Hollande dont il est l' ami , hier soutien de Nicolas Sarkozy , aujourd'hui ami de François Fillon . Il a aussi été &laquo; Young leader &raquo; de la French American foundation en 1994 , et préside le comité de direction du groupe Bildelberg , qui réunit le gratin de l' oligarchie mondiale . Nous défendons un tout autre modèle : la sécurité sociale . Le programme l' Avenir en commun prévoit tout le contraire de M. Fillon . Nous proposons d' améliorer la prise en charge des soins par la Sécurité sociale jusqu' à leur remboursement à 100 % . Nous voulons rétablir la retraite à 60 ans , réduire la durée de cotisation exigée pour avoir une retraite complète et renforcer le régime de retraite solidaire par répartition . Et face à la dépendance , nous défendons notamment la construction de 50 000 places en établissements pour prendre en charge les personnes dans le besoin dans un réseau public et abordable , et la baisse de ce qui reste à la charge des familles pour les personnes qui continuent de vivre chez elle . Ce modèle est plus juste et moins coûteux . Plus juste car dans le système de la sécurité sociale , le montant des cotisations ne dépend pas de l' âge ni de l' état de santé et que la protection dont on bénéficie ne dépend pas de ses moyens financiers mais de ses besoins : chacun est remboursé de la même façon par exemple . Chacun a un aperçu de ce qu' est un système d' assurance privée individuelle , à travers sa complémentaire santé s' il a les moyens de s' en payer une . Surtout , la Sécu c' est moins cher ! Il suffit de comparer les dépenses de santé entre la France et les États-Unis . En France , ces dépenses de santé représentent 11 % de la richesse annuelle du pays . Elles sont essentiellement supportées par l' assurance maladie . Aux États-Unis , les dépenses de santé sont essentiellement privées . Et elles coûtent plus cher puisqu'elles atteignent 16,5 % de la richesse du pays . Tout cela pour des résultats sanitaires mauvais là où la France reste une référence malgré les mauvais coups portés à notre système solidaire et à l' hôpital public . ",
    "Ma candidature est d'abord celle d' un programme , dont la cohérence doit être gardée , protégée , nourrie avec soin . Un programme ce n' est pas un copier-coller de toutes les revendications que l' on a entendues . Il faut qu' il y ait un fil conducteur , qui s' applique dans tous les compartiments de la proposition . Ce fil conducteur , c' est l' intérêt général humain qui est mis en cause par la menace qui pèse sur la civilisation humaine , la destruction du seul écosystème compatible avec celle -ci . Le fil conducteur qui organise tout le reste , c' est cette vision des biens communs , de ce que nous avons en commun , de notre humanité partagée . De l' Humain d'abord à l' Avenir en commun Et bien je peux dire que le candidat a quelques privilèges , - on va appeler ça comme ça – il en a un , c' est de trancher parmi la foule des propositions qui lui sont faites , en matière de mots d' ordre ou de slogans . Et bien , j' ai tranché , et j' ai choisi que l' on appelle notre nouveau programme , qui s' appelait L' Humain d'abord , j' ai choisi qu' on aille au bout de la logique de &laquo; l' humain d'abord &raquo; . Si c' est l' humain d'abord , alors l' humain est dans tout homme , toute femme , quelle que soit sa condition sociale , quelle que soit son éducation , quelle que soit sa religion – puisqu'on n' arrête pas d' en parler , et même s' il n' en a pas , ce qui est la masse du peuple français . Et bien ce qu' ils ont en commun c' est cela que nous allons valoriser , c' est de cette manière que nous allons interpeller chacun , sans nous occuper de savoir comment il a voté avant , ce qu' il faisait avant , ce qu' il disait avant . À cette heure , voici que s' avance la plus grande des menaces , nous pouvons y faire face et relever ce défi ce qui serait extraordinaire , la dette écologique est dorénavant parvenue au 8 août ce qui signifie qu' à partir du 9 août au matin , la terre se voit prélever plus que ce qu' elle ne peut reconstituer . Dès lors l' intérêt général est appelé [ … ] Cela signifie que nous devons aller au bout de l' humain d'abord , le bout de l' humain d'abord , c' est d' appeler en toute personne , cette communauté de la condition humaine . Le programme s' appellera : L' Avenir en commun . Je souhaite que vous compreniez le mot jusqu' au bout , car il n' y a pas d' avenir en commun pour les êtres humains sans tous les éléments qui constituent la biosphère . Il n' y a pas d' avenir en commun pour les êtres humains sans toutes les espèces animales et végétales qui ont permis l' émergence de la conscience . L' Avenir en commun est un projet global qui concerne tous les aspects de la biosphère et de sa conservation . Cet avenir en commun , cet intérêt général humain qui est appelé , c' est celui qui me permet de m' adresser à tous , en toutes circonstances . Le programme de la France insoumise [ … ] Nous proposons un programme , nous proposons une démarche , nous proposons un mouvement , la France insoumise , dans lequel tout le monde est appelé . Je suis un candidat hors parti , mais je ne suis pas un candidat contre les partis , je suis moi-même membre d' un parti . Mais il faut que tout le monde puisse participer à cette campagne , apporter son intelligence , sa capacité individuelle sans être obligé , jamais , de se soumettre à telle ou telle règle ou bien d' adhérer à telle ou telle organisation partisane , c' est cela ce que nous cherchons à faire , et vous voyez notre force c' est là qu' elle est . C' est 125 000 personnes , qui ne demandent rien à personne et qui font campagne dès aujourd'hui à nos côtés . Je ne suis pas capable de vous dire exactement tout ce qu' ils disent , mais je suis certain qu' ils le disent plus efficacement et avec plus de sensibilité que ne le ferait n' importe quelle organisation qui aurait décidé de passer un discours dans un moule unique , même si parfois ce discours est nécessaire pour penser . La solidarité pour sortir de la peur Il faut compter sur le peuple , sur son intelligence , son humour , sa dérision , sa finesse pour convaincre et entraîner . Car il faut convaincre et entraîner . Il faut sortir de la sinistrose , il faut sortir du déclinisme , il faut sortir de la peur qui est le seul nouveau lien social que la droite ait proposé au peuple pour se rassembler . J' y reviens un instant : permettez -moi que je m' y attarde car cela concerne la morale , cela concerne la manière de vivre ensemble . Autrefois , le grand lien social étaient la solidarité , la fraternité et personne ne trouvait à y redire quand bien même était -il de droite . La cupidité était considérée comme une tare ; aujourd'hui elle est considérée comme le premier moteur de l' activité . Chacun est amené à développer son égoïsme personnel comme une affirmation de soi . Et ce qui était autrefois le lien commun , qui fait notre vie quotidienne , c'est-à-dire l' amour , c'est-à-dire l' intérêt pour les autres sans lequel nous n' avons pas d' existence personnelle , ce lien était le lien de la société . Ils l' ont remplacé par un autre lien : la peur . [ … ] La peur tout le temps . Nous , nous remettons au centre de commande , d' autres valeurs . Et ce sont elles qui , en fonctionnant , vont nous donner l' énergie de fonctionner autrement . ",
    "Depuis le Brexit , le poste de commandement de l' Union européenne est une salle de bal vide et froide . Les lampions sont éteints et ça sent le tabac froid . Les guirlandes sont par terre et les nappes froissées . C' est clair : on ne dansera plus . Que faire d' autre ? Que proposer comme nouvel arguent de promotion puisqu'on ne peut plus dire à personne &laquo; venez chez nous , c' est une maison sympathique ! On est tous des gentils qui s' entraident et font des projets magnifiques pour un futur bienveillant &raquo; ? La preuve du pudding c' est qu' on le mange . À Londres , personne n' a eu à manger le gâteau européen pendant la campagne du Brexit . Il n' y aura jamais eu un mot de positif pour appeler les Anglais à rester dans l' Union européenne . Juste des menaces et des grimaces : la peur de l' hiver nucléaire et celle des catastrophes immobilières , financières , économique que sais -je , le fer rouge de la xénophobie , la marque noire du racisme , si le pays votait mal ! En vain ! Et désastre supplémentaire : l' immobilier est resté stable , la bourse est repartie , la croissance se maintient . Pffff , les eurocrates ne savent que faire . Le sommet de l' Union à Bratislava a eu pour grand dessein de ne pas sanctionner l' Espagne et le Portugal ! Depuis , ça se traine . On voit bien qu' il y a une activité pour essayer de trouver quelque chose à se mettre sous la dent . On a donc sur le tapis Madame Merkel en leader d' un improbable &laquo; monde libre &raquo; . Le rêve européen : la maman méritante pour tous . Plat de résistance : l' Europe de la Défense . Une foutaise en barre . Se défendre ? Contre qui ? Déjà qu' il n' y a pas de politique étrangère commune de l' Union ! Évident : elle est impossible à bâtir . Les va-t -en guerre contre la Russie de l' Europe de l' est et les paisibles méditerranéens qui ont d' autres soucis avec leur voisins n' ont pas du tout les mêmes priorités . Et celles -ci ne peuvent non plus s' additionner . Sans parler des permanences de l' histoire . Les Russes sont des partenaires naturels des Français . Et quand les Français l' oublient , la catastrophe n' est jamais loin . Quoiqu'il en soit , voyez l' incroyable évolution : on faisait l' Europe pour la paix , on nous propose de la faire pour se préparer à la guerre . Car nul n' a jamais vu qu' on s' arme et s' intègre militairement autrement que pour faire usage des moyens que l' on rassemble . La peur de l' autre et la guerre comme liant du projet européen ! Un minable retour aux sources . Car la première Europe politique proposée sur le vieux continent après la deuxième guerre mondiale , ce fut la communauté européenne de Défense ( CED ) en 1954 . Il s' agissait alors de s' armer et de coopérer contre … les Russes ! Déjà ! Ceux-là se présentaient à l' époque sous la forme de l' Union des Républiques Socialistes Soviétiques ( URSS ) . Le Parlement français vota non . Il est vrai qu' il était étrange de décider de s' allier avec l' envahisseur de la veille contre l' allié qui nous en avait libérés . Les &laquo; pères fondateurs &raquo; , un quarteron de bigots à la retraite après une guerre où ils furent très moyens , remirent l' ouvrage sur le métier . Le traité de Rome arriva donc trois ans après cet échec . Toute la gauche française vota contre , de Mendès France aux communistes . On veut l' oublier aujourd'hui . On gagnerait à relire le discours de Mendès France à l' époque … On votait donc cette semaine au Parlement européen plusieurs rapports sur le sujet . Un plan d' organisation d' une défense commune . Chacun s' est donc positionné . Le PS et la droite ont évidemment voté pour ; les Verts et nous contre . Bien sûr , nos motivations ne sont pas les mêmes . Dans ma façon de voir les choses , le système militaire de la France a une cohérence qui est et doit rester strictement défensive . Si chacun agit de même , nul ne sera tenté de sortir de ce rôle , du fait du manque des moyens et de la culture agressive qui va avec la constitution d' une puissance du type de celle que voudrait construire les eurolâtres . En toute hypothèse il n' existe aucune menace militaire contre l' Europe . Il est stupide de faire croire que la Russie serait en position belliqueuse vis-à-vis de la Pologne ou des petits pays baltes . Personne ne saurait dire quel intérêt il y aurait pour la Russie à agresser qui que ce soit de ce côté-là . Par contre il est tout à fait évident que l' Union européenne menace la Russie . Elle le fait en tant que poste avancé de l' OTAN , avec ses batteries de missiles en Pologne et ses troupes américaines positionnées sur le territoire européen . Et elle le fait notamment en Ukraine où son soutien au gouvernement d' extrême droite est extrêmement dangereux . Sans oublier la Géorgie , la Biélorussie et combien d' autres zones . Dans chacune d' elle l' Union a lancé le processus d' annexion traditionnel : discussion d' alliance militaire , inclusion dans la zone économique européenne , organisation du renversement des circuits financiers et marchands vers l' Union en général et vers l' Allemagne en particulier . Sur toutes les frontières de l' est , nous sommes à la merci d' une provocation des bellicistes . Car l' article 5 de l' OTAN et l' article 49 du traité de Lisbonne nous obligent à une solidarité militaire avec les gouvernements grossièrement anti-russes de la région . C' est évidemment un contexte favorable pour la répartition des rôles à laquelle s' emploient les stratèges nord-américains : ils veulent voir l' Europe s' occuper des Russes pendant qu' eux s' occupent de l' Asie . Au Parlement européen règne donc une ambiance très chaude de guerre … froide . On a même voté un rapport pour demander l' organisation d' une contre propagande face aux Russes … Et comme si cela ne suffisait pas , après avoir estimé le coût de &laquo; l' inexistence d' une Union européenne de la défense &raquo; à 26 milliards d' euros du fait des doublons d' emploi et des sous-investissements , la chère &laquo; Europe qui nous protège &raquo; envisage des sanctions financières contre les récalcitrants à l' intégration militaire . Je me demande ce que vont me reprocher maintenant ceux qui se donnaient des airs indignés quand je parlais de construire un &laquo; nouvel indépendantisme &raquo; . Le choix se resserre : soit ce que je propose , soit l' Europe militaire intégrée sous commandement du leader du monde libre . Puisque c' est le nouveau surnom de madame Merkel dans la presse anglo-saxonne . ",
    "Alors , je suppose que c' est à moi . Tout le monde m' entend bien ? Ça va ? Bon , il y a un petit larsen là . D'abord , je ... j' envoie un petit signal amical pour cet accueil admirable à vous tous , à ceux qui se trouvent dans le hall , et qui ont dû suivre les films sans ... sans voir les images . A ceux qui sont dans les salles qui jouxtent celle -ci et , m' a -t-on dit aussi , d' autres qui se trouvent dehors . Je les embrasse de tout cœur pour les réchauffer . Qu' ils sachent que j' en suis marri . Mais c' est comme ça . Je vois à cette affluence un signal que je comprends . Qui ne tient pas à ma personne , mais à ce que , à cet instant , je peux représenter pour vous . Je sais ... que ... qu' en cette année si extraordinaire d' élection présidentielle , je figure dans les 4 premiers , sans qu' on puisse dire à quelle place exactement . Et comme vous le savez comme moi et comme vous le sentez , vous éprouvez le besoin de vous retrouver , de vous fortifier en vous voyant les uns les autres , et pour d' aucuns , de venir entendre , apprendre , savoir de quoi il retourne avec ce personnage que l' on habille tantôt tout en rouge , tantôt en super bleu-blanc-rouge et d' une manière générale , dans des portraits si divers que parfois , moi-même , j' ai du mal à m' y reconnaître . Je remercie les Insoumis des deux Savoies qui ont organisé cette soirée . Naturellement , également les membres de mon cher Parti de Gauche qui donnent le coup de main indispensable . Les élus qui me parrainent en Savoie , on me dit qu' il y en avait 3 . Je leur en suis très reconnaissant . Je voudrais que vous sachiez que me parrainer ne signi ... ne signifie pas que l' on est d'accord avec moi . Avec vous , pour une part d' entre vous , car j' imagine que il doit y avoir ici aussi des gens qui n' ont pas encore pris leur décision pour savoir ce qu' ils feront comme vote dans quelques mois . Me parrainer , c' est seulement dire que l' on autorise ces 4 millions de Français qui , naguère , m' ont donné leur suffrage en 2012 ; on les autorise à trouver de nouveau à leur disposition celui pour lequel ils ont déjà voté une fois . On a , à la faveur de petits tripatouillages , aggravé les conditions dans lesquelles on peut être candidat dans notre pays . L' invention des 500 parrainages avait été faite , au départ , pour éviter les candidatures dites folkloriques . Le général de Gaulle avait , parait -il , mal supporté la présence de M. Marcel Barbu en 1965 , qui était un candidat très émouvant et qui était d'ailleurs lui-même très ému , puisqu'il pleurait la plupart du temps à l' écran . Et donc on a inventé ça . On n' a jamais inventé à l' époque , me semble -t-il dans une Constitution qui , déjà , instaurait mille barrages pour accéder à la fonction de monarque présidentiel - on n' avait pas imaginé que ce serait un moyen pur et simple de verrouillage pour empêcher des gens d' êtres candidats . Alors ... c' est une difficulté , vous savez , pour nous . Bien sûr . Mais de quelle difficulté ne viendrions -nous pas à bout , du seul fait de notre volonté , de notre opiniâtreté et de notre engagement ? On m' avait dit que puisque personne ne m' aidait , je n' arriverais à rien . Fort des 100.000 premiers qui ont accouru pour appuyer ma candidature , et à partir de ce soir , puisque nous avons atteint ce chiffre aujourd'hui , des 150.000 , je me trouve déjà au niveau où la réforme que suggérait Lionel Jospin : pour être candidat , il aurait fallu , ou bien 500 parrainages , ou bien 150.000 parrainages de citoyens . Et bien , quant aux citoyens , me voici déjà parrainé . Ensuite ... Quant aux élus , mesdames et messieurs , mes chers amis pour ceux qui sont mes amis ici , et mes chers camarades pour ceux qui le sont , rien , personne , jamais , à aucun moment , n' est en état de nous empêcher d' avancer . Nous creuserons les montagnes avec nos ongles et nous viderons la mer avec nos mains s' il le faut , mais nous atteindrons notre objectif , et nous sommes déjà 325 parrains et marraines autour de ma candidature . Nous arriverons donc à 500 le moment venu , c'est-à-dire lorsque , enfin , ici et là , certains qui se réservent viendront à la rescousse comme c' est leur devoir . A la rescousse , ai -je dit , parce que tout ça nous coûte de l' effort , de l' argent , bref . Cette campagne , que vous appuyez à cet instant , est d'abord une campagne populaire . Au fond , c' est une bonne chose qu' ils aient essayé , tous , de faire en sorte que je n' y sois pas . Vous vous êtes même cotisés , alors que nous ne sommes pas un parti , la France Insoumise , nous sommes un mouvement , un label commun , nous tâtonnons , nous cherchons des formes nouvelles pour nous entendre et militer . Et bien ... Vous vous êtes cotisés . Vous ne le savez peut-être pas , mais vous avez déjà donné un demi-million d' euros . Ça va . C' est bien . Evidemment , on ne peut pas avoir les grandes salles qu' on aimerait avoir , parce que la démocratie , c' est de plus en plus cher dans ce pays . Déjà , c' est de plus en plus compliqué : vous partez à l' ancienne avec vos tracts et vous dites : &quot; Tiens , je vais aller faire un porte à porte . &quot; Couic . C' est pas possible . Il y a des digicodes . Alors après , vous dites : &quot; Ben , je vais aller là où sont les gens . &quot; Où ils sont -y , les gens ? Pas à la sortie de la messe , il y en a moins qu' avant . Donc - je ne dis pas ça pour ironiser , hein - donc on va où ? Là où ils sont ! Dans le centre commercial , misère , jusqu' au dimanche . Et dans le centre commercial , on ne peut pas , c' est une surface privée . Vous voulez une salle ? C' est des dizaines de milliers d' euros , mesdames , messieurs . Vous devez le savoir . Des dizaines de milliers d' euros . Et maintenant , si vous êtes dans la rue , en plus , on vous rajoute le tri de la population qui rentre . Pour faire le meeting du 5 juin , place Stalingrad , nous avons dû débourser 30.000 euros pour organiser le contrôle des gens qui venaient là . Parce que ça n' est qu' une société agréée qui peut le faire . Je veux que vous le sachiez . C' est de plus en plus difficile de mener un combat démocratique . C' est de plus en plus difficile de vouloir participer à la démocratie de son pays . Bon , je n' ai pas fini de remercier . Je remercie pas seulement les élus qui me parrainent , mais aussi les élus qui me soutiennent . Et il y en a 3 ici qui me sont spécialement chers : Andréa Kotarac , Emilie Marche , Corinne Morel-Darleux qui sont vos élus au Conseil Régional . Et puis ... Je remercie ceux qui me donnent une démonstration des propos qui , parfois , vous paraîtront abstraits ce soir , les opposants à la ligne Lyon-Turin et les militants ... les salariés , les ouvriers et ouvrières d' Ecopla que j' ai rencontrés tout à l' heure et dont je vais vous parler . Chaque fois que je vais quelque part , on me demande pourquoi je viens là . J' étais à Boulogne : &quot; Pourquoi vous venez à Boulogne-sur-Mer ? &quot; Bon , ben j' avais une bonne raison . C' est une des villes les plus pauvres de France . La pauvreté , c' est cette lèpre qui gagne notre pays et qui fait basculer 9 millions de personnes . Et vous savez , quand on devient pauvre ? C' est quelqu'un qui , d'après les statistiques internationales , gagne moins de 1.000 euros par mois . Et ben , vous savez quoi ? 1.000€ , c' est une fortune pour bien des gens . Le minimum vieillesse , c' est même pas 500 euros . 9 millions de personnes . Ce pays est en train de basculer dans une situation : ou bien vous vous y trouvez , et votre seul souci , c' est de savoir comment vous allez faire face à demain , voire à après-demain , et puis , pour les autres , de se dire : &quot; Bon sang , pourvu que ça ne soit pas mon tour . &quot; Et ça gagne tout le monde , hein . Il ne faut pas croire . Je suis sûr que des gens dans cette salle le savent . Il ne faut pas croire que c' est seulement ceux qui étaient déjà en grande difficulté avant et qui sont dans des difficultés supérieures aujourd'hui . La pauvreté frappe à toutes les portes , à tous les étages . Tout le monde . Quelque soit sa classe sociale , enfin , à l' intérieur de la classe des salariés . Dans combien de copropriétés , les copropriétaires ne peuvent plus payer les réparations ou les charges ? Et les copropriétés accumulent les dettes et les impayés . Et les gens vivent dans la peur de ne plus avoir leur logement demain si on les en expulse , alors qu' ils croyaient en être propriétaires et avoir garanti leurs vieux jours de cette manière . Voilà pourquoi je suis allé à Boulogne . On me dit : &quot; Pourquoi vous venez à Chambéry ? &quot; Pourquoi je viendrais , après tout , hein ? On se demande . Ben , déjà , parce que je n' y suis jamais venu . J' ai cherché dans ma tête , dans ma longue ... ma longue vie de ... de militant et de lanceur d' alerte quand est -ce que j' étais venu à Chambéry . J' en ai fait des villes , vous savez . Pour moi , c' est toutes les mêmes , hein . C' est la nuit , dans des salles glauques , pourtant celle-là est belle , hein . Et après , ça m' est revenu : &quot; Ah oui , je suis venu en stop quand j' avais 20 ans . &quot; Je ne me rappelle plus pourquoi , je devais passer par là . En vérité , je suis venu parce qu' il y avait un croisement de symboles dont j' ai besoin pour faire mon discours . D'abord ... c' est la Savoie . Et la Savoie , ha ha , pour un homme comme moi , il y a un nom qui s' attache à la Savoie : c' est Ambroise Croizat . Le communiste qui a fondé la Sécurité Sociale . La Sécurité Sociale , ce n' est pas seulement un bon service , c' est une autre société , une autre manière d' organiser la société et de la penser , où les individus sont solidaires entre eux , où chacun donne selon ses moyens , mais reçoit selon ses besoins . Et surtout , la Sécurité Sociale - je ne sais pas si vous avez vu le film &quot; La Sociale &quot; , sûrement que vous avez dû le voir , pour un certain nombre d' entre vous - la Sécurité Sociale , c' était la victoire sur un fléau . Ce fléau , c' était la peur du lendemain . La peur de la maladie . La peur de l' âge qui s' avance . Et ça , c' était radicalement nouveau , on avait trouvé le moyen . Des ouvrières , des ouvriers ont fait vivre les premières caisses de Sécurité Sociale et c' est eux qui ont porté ça sur leur dos . Et bien voici que cette peur du lendemain que l' on avait vaincue , que l' on avait vaincue au lendemain d' une guerre épouvantable , alors que le pays était ruiné . Cette peur du lendemain , elle est revenue dans combien de foyers ? Depuis que l' humanité est humanité , elle lutte contre la précarité . Et oui . Les premiers groupes humains allaient d' un endroit à l' autre , et tâchaient de s' installer dans les endroits où c' était le plus profitable pour y avoir les enfants et tâcher de les faire vivre le plus longtemps possible . Et c' est comme ça que l' élevage remplace la chasse , parce que c' était plus sûr . A condition , bien sûr , de bosser dur . Et c' est comme ça que l' agriculture remplace la cueillette . Evidemment , il fallait travailler dur , mais c' était moins aléatoire . Et ainsi de suite . L' humanité a toujours voulu vaincre la peur du lendemain . S' installer dans la sécurité pour pouvoir devenir l' humanité complètement . Avoir du temps pour faire autre chose que survivre . Avoir du temps pour soi , pour les autres , pour l' art , pour la musique , bref . Toutes les choses qui comptent et qui nous font spécifiquement humains . Voilà pour Croizat . Alors , après , moi , chaque fois que je vais quelque part , je tourne des pages : alors il parait que vous étiez des Allobroges . Bon . Ca , c' est pour &quot; nos ancêtres , les Gaulois &quot; , d'accord ? Il n' y a pas un Gaulois dans la salle , mais ce n' est pas grave . Mais , ces Allobroges ... moi , vous savez que je ne crois pas que l' identité du pays soit dans ces peuplades qui ont traversé l' Europe dans tous les sens dans le passé . Moi , je crois que l' identité du pays , c' est son identité républicaine , ce qui fait que nous pouvons être tous Françaises , Français , vivre ensemble , c' est que nous sommes unis entre nous ni par la couleur de peau , ni par la religion , ni même par la langue , puisque nous la partageons avec 29 pays dans le monde , mais par un pacte politique : Liberté , Egalité , Fraternité . Et si nous sommes de cette façon , alors il faut que je regarde la Savoie de 1789 . Et revoilà les Allobroges . Il parait que vous avez tenu une Assemblée des Allobroges . Je dis &quot; vous &quot; , tu parles , là-dedans , il doit y en avoir je ne sais pas combien qui ont un grand-père ... déjà 1 sur 4 , vous avez un ... un de vos familiers qui est un étranger . Alors , vous , ça doit être les Italiens , ça circulait pas mal dans le coin , hein . Moi aussi , j' en ai des Italiens . Ça ne se voit pas , hein ? Des Siciliens , ça , ça doit se voir un peu . Un peu ombrageux , vous voyez ? Bon . Et donc , en 1792 , vous avez formé l' Assemblée des Allobroges . Et la Savoie a décidé de se joindre à la République Française , chassé son monarque et tourné la page de l' Ancien Régime , avant qu' elle n' y soit précipitée de nouveau après nos défaites . Et la Savoie est revenue de son propre gré , par son vote , à la France . L' histoire républicaine de la France a donc une part de ses racines dans la Savoie , par la volonté de ... de ceux qui composaient ce territoire à cette époque là , on ne sait pas qui c' est , peut-être quelques uns de vos ancêtres , quand même , hein , s' y trouvaient . Sa volonté républicaine et sa volonté française est parfaitement claire . C' est la même chose , je pense , dans l' esprit de tous ceux qui y ont participé . Donc , voilà pourquoi je suis là . Je viens toucher cette terre tellurique , mais pas que . Bon , alors , sur mon papier , il est écrit qu' il faut parler de l' actualité . L' actualité , c' est l' élection américaine . Evidemment que ça nous concerne , les amis . Pas parce que nous serions une espèce de grand Disneyland mondial , et qu' on regarderait toujours dans le pays d' où est arrivé Mickey ou Donald ... Bon , c' est Donald . Mais ... Ouais , ben ... je parie que vous voyez à quoi je veux en venir , ça me brûle la bouche . Ben oui , Donald , c' est Picsou , forcément . Ce n' est pas le plus malin , dans l' histoire . Voilà . Après , il y a les 3 autres qui s' agitent , là , je vous laisse attribuer les rôles . Riri , Fifi , Loulou . Mais , plaisanterie mise à part , le Président de la nation dominant le monde , ça nous implique ; en tout cas , ça nous interpelle et ça nous oblige à savoir quoi faire . Mais d'abord - ça ne sera pas le sujet de ce soir , je ne vais pas vous parler des Etats-Unis d' Amérique , pas plus que ça - mais , vous avez vu cette immense surprise de ceux qui sont tout le temps surpris ? Vous avez remarqué qu' ils sont tout le temps surpris ? C' est comme s' ils préparaient quelque chose , que ça n' a pas lieu et ils se disent surpris , alors qu' en réalité , ils sont en colère , parce qu' ils n' ont pas réussi à obtenir des gens ce qu' ils voulaient leur faire faire . Voilà . Etonnés , surpris , toujours . Que se passe -t-il là-bas ? Il se passe là-bas ce qui se passe dans le monde entier . Les sociétés telles que nous les avons connues depuis la fin de la 2ème guerre mondiale , qui avait engendré l' émergence d' une classe moyenne , partout , assez puissante , ce qui veut dire que le partage de la richesse entre le travail et le capital s' était fait d' une manière un peu plus juste , tant et si bien qu' est apparu ce qu' on appelait cette classe moyenne , c'est-à-dire des gens sachant , des gens travaillant et vivant avec une paie et un environnement qui leur permettaient de mener une vie digne . En tout cas , ils essayaient de l' améliorer , ils avaient la certitude que leurs enfants vivraient mieux qu' eux-mêmes , comme eux-mêmes avaient mieux vécu que leurs parents . Et bien , le système néo-libéral , le libre-échangisme , c'est-à-dire les marchandises qui circulent d' un bout à l' autre du monde avec un unique critère de sélection : le prix . Car si au XVIIIème siècle , on pouvait dire , aussi sottement que l' a dit le Président Hollande , que l' offre crée la demande , c'est-à-dire qu' il suffit qu' on vous amène une marchandise , si elle est assez peu chère , vous l' achèterez . Si vous y réfléchissez , c' est absurde dans tous les cas . Allez essayer de vendre des frigos à des Esquimaux . Allez vendre des radiateurs à des Sahraouis . Bien sûr que non , il ne suffit pas qu' une marchandise soit là et qu' elle ne soit pas chère pour que vous l' achetiez , mais ils le croient . Et par conséquent , ils font tout pour qu' elle ne soit pas chère . &quot; Pas cher &quot; est devenu une sorte de morale collective , d' objectif . Chacun doit se procurer à pas cher . &quot; Pas cher &quot; égal souffrance , car au bout de &quot; pas cher &quot; , il y a quelqu'un qui souffre . Qui travaille et qui est payé dans des conditions lamentables , qui travaille dans des conditions lamentables , qui est surexploité à l' autre bout du monde . Ce sont des enfants . Ce sont des adultes mal payés , toujours plus mal payés , car il y a toujours plus &quot; pas cher &quot; à un endroit qu' à un autre . Voilà que les Ukrainiens coûtent le tiers d' un Chinois . Pauvre Chinois . Pauvre Ukrainien . Et ainsi de suite . Et bien , sur cette base , nos sociétés sont en train d' être détruites . La vague s' est levée d'abord dans l' Amérique latine . De sociétés qui ne sont pas composées d' indigènes folkloriques qui jouent de la guitare , fument des cigares , dansent la salsa et boivent du rhum , comme le croient bien des nantis ici qui s' en moquent assez facilement , je trouve . Ce sont des peuples avec des sociétés complexes , organisées , bien plus de différences sociales qu' on ne se l' imagine , et bien , leurs sociétés ont été rasées par le néo-libéralisme . J' ai vu , de mes yeux vu , à Montevideo dans la gare principale , comme vous diriez la gare de Lyon à Paris ou la vôtre ici , les arbres pousser . Parce qu' il n' y a plus de train . J' ai vu en Argentine démonter les rails pour vendre l' acier , parce qu' il n' y avait plus de trains , et qu' il fallait tout faire passer sur la route . Tout ce que je vous raconte ici aujourd'hui , tout ça a déjà été fait . J' ai vu des gens se retrouver dans le noir parce que le réseau électrique s' est effondré parce que personne ne s' en occupait assez sérieusement , etc . , etc . Et alors , cette vague a atteint aussi les Etats-Unis d' Amérique , comme elle atteint en ce moment l' Europe . C' est une énorme vague . C' est le peuple . Le peuple qui n' en peut plus d' avoir deux ou trois boulots par jour pour arriver à survivre , et des fois , pour ne pas arriver à survivre . Il y en a assez de se ruiner pour payer les études des enfants . Des jeunes qui sont à l' université , endettés jusqu' au cou pour 25 ou 30 ans , comme c' est le cas aux Etats-Unis d' Amérique . Alors cette vague , elle a d'abord fait sortir M. Obama , parce que M. Obama , c' est un afro-américain , comme on dit . Donc les gens disaient : &quot; Celui-là , il a réussi , ça prouve qu' il va nous comprendre , il vient de notre milieu . &quot; Et puis c' est un afro-américain , comme Chavez , la peau mate . Comme Correa , en Equateur . Comme tous les autres . C' était la revanche de ce peuple moyen , de ce peuple populaire , qui , à peine arrivé au pouvoir en Amérique latine commence d'abord par extraire des millions de gens de l' extrême pauvreté et subissent les jets permanents de sarcasmes , de cailloux , d' insultes , mais ils le font . Des millions de gens sortent de l' extrême pauvreté . Ils font ce que personne ne croyait capable de ... possible de faire . Ils rendent la vue à ceux qui n' y voyaient plus , parce qu' ils avaient la cataracte et que personne ne les soignait . Ça s' appelait la mission Milagro . Voilà ce qui s' est passé . Ça a atteint les Etats-Unis d' Amérique , et là , qu' est -ce qui se passe ? Pas grand-chose . Mais en tout cas , assez pour avoir exaspéré les puissants et les importants . Et à l' élection suivante , mes amis , c' est celle qu' on vient de vivre , la vague , elle est toujours là . De quel côté va -t-elle aller ? Va -t-elle aller du côté de M. Trump ou du côté du camp progressiste ? Va -t-elle aller du côté de ceux qui affichent des convictions , qui , à nous , nous paraissent insupportables , ou est -ce qu' elle va aller de l' autre côté ? Et bien , M. Trump a d'abord commencé par faire quelque chose : il a exprimé les volontés du peuple , là-bas , de droite . Ça existe , vous savez , la droite populaire . Ça existe . Et les gens , ils croient de bonne foi que c' est de ce côté là que va se faire leur avenir , que c' est bien comme ça . Et puis , vous avez ceux qui préfèrent le camp progressiste . Qui pensent qu' il faut faire ensemble . Bref , je ne vais pas vous raconter la droite et la gauche , vous la connaissez aussi bien que moi , à supposer que là-dedans , on arrive à s' y retrouver , de savoir qu' est -ce qui est de droite et qu' est -ce qui est de gauche . Et bien , on l' a bouchée , la voie . D' un côté , vous aviez cet homme qui s' impose , il gagne dans sa primaire en chassant tous ces bigots qu' il y avait , du Parti Républicain , qui venaient parler de toutes sortes de choses , du bon Dieu , des choses importantes , hein ? Je ne vous dis pas le contraire , m' enfin les gens , ils ne vont pas voter pour ça , quoi . Ils ont beau être de droite , leurs problèmes c' est aussi la paie , le boulot , l' éducation , la santé , ça les intéresse . Et ils entendent un type qui leur dit : &quot; Nous , on va arrêter de fermer les usines . &quot; Et ben , vous êtes de ce que vous voulez politiquement , si c' est vous qui êtes dans l' usine , vous êtes pour le type qui vous dit qu' on ne va pas la fermer . C' est clair , non ? Et ben oui , ça s' appelle le peuple . Et ce n' est pas parce qu' il y a un peuple qui pense de deux manières différentes que ça cesse d' être le peuple . Le peuple a des intérêts , des aspirations , des revendications qui sont communes à tout le peuple . Mais qui s' expriment différemment , avec des solutions de droite , des solutions de gauche , des solutions de ce que vous voulez . Et de l' autre côté , que s' est -il passé ? On a eu un phénomène qui s' appelle Bernie Sanders . Alors , je vous garantis que moi , à côté , c' est presque la pop music , parce que Sanders , des cravates à ne pas sortir , des tenues pas racontables , même pas coiffé , il ne faisait aucun effort sur la forme . Mais sur le fond , mes amis , il n' a parlé que de revendications sociales . Et il battait Mme Clinton dans les Etats populaires dominés par les Républicains , ceux qui , ensuite , ont fait la différence . Vous comprenez ? Il a fallu tricher pour en venir à bout . Il a fallu l' invisibiliser médiatiquement , il a fallu le traiter de tous les noms . Tous les noms , je vais vous dire : un , il est trop énervé , deux , il est antisémite , c' est la croix qu' on porte tous . Dès qu' on est pas de leur bord , c' est qu' on est antisémite . Et lui , pas de pot , si j' ose dire , pour ceux qui l' accusaient , sa famille est morte en déportation , et ben , ça ne les gêne pas , les autres . Et ainsi de suite . Les mêmes injures , les mêmes caricatures d' un bord à l' autre de l' univers . Et malgré ça , il a fallu qu' ils trichent et qu' ils trichent et qu' ils trichent encore , pour l' empêcher de gagner . Mais il y avait des foules immenses qui s' assemblaient , comme les Etats-Unis n' en avaient pas vu depuis des décennies . Figurez -vous qu' à New York , à deux occasions , des foules gigantesques se sont réunies dans les parcs . Voilà ce qu' il s' est passé . Le peuple est sorti de son lit et il a rencontré un bouchon qui s' appelle Mme Clinton . C'est-à-dire cette gôche millionnaire , corrompue , liée au système . On dit : &quot; Mais l' autre aussi . &quot; &quot; Ben , peut-être bien , mais , au moins , il le dit . &quot; Il ne fait pas semblant d' avoir de la compassion pour les autres , lui . C' est ça que les gens ne supportent pas . C' est le mensonge , la duplicité . L' évidence que la personne qui parle ne fera pas ce qu' elle dit . Et caetera . Ça nous importe , bon , il a gagné . Il y a de l' eau , là-dedans ? Il a gagné . Alors , je ne vais pas commenter . Je vais vous dire ce que je lui reproche le plus . Aujourd'hui ... enfin là , depuis deux trois jours , a commencé la COP 22 . D'accord ? Maintenant , les gens ... il faut faire de la politique sérieusement . Nous tous , quelque soit notre étiquette , quelque soient nos habitudes de penser , nous sommes tous impliqués par le changement climatique . On ne doit pas oser présenter un programme qui ne parte pas de ce constat initial . Et nous , les Français , spécialement . Pourquoi nous ? Parce que nous sommes une grande nation , riche et puissante . Parce que nous avons , dans nos rangs , les milliers d' hommes , de femmes de tous âges , qui savent faire . J' ai connu des amis et des camarades qui dirigeaient des gouvernements et qui voulaient faire des grands changements . Mais quoi qu' ils décident , ils avaient un problème énorme : il n' y avait personne pour le faire . C'est-à-dire qu' il n' y avait pas la qualification , les compétences professionnelles , le savoir qui permet de tourner , et nous , nous avons tout ça . Donc , nous devons marcher devant pour ce qui est du changement de nos manières de produire , d' échanger et de consommer . Pour nous mettre à la hauteur de la tâche . Le changement climatique est commencé . L' humanité toute entière est mise au pied du mur de savoir ce qu' elle va faire face à ce qui s' annonce , à ce qui est déjà commencé , à ce qui est en cours . Ben , ça commence mal , figurez -vous , quand le maître du monde , peu me chaut qu' il soit de droite , de gauche ou de ce que vous voulez , mais qu' il dise : &quot; Je ne crois pas au changement climatique . &quot; Je le cite , là , hein . &quot; Je ne crois pas au changement climatique . On a appelé ça le réchauffement de la planète . &quot; Je ne sais pas qui lui a dit ça . Le changement climatique , maintenant ils appellent ça une météo extrême , c' est nouveau . Et voici ce qu' il dit : &quot; C' est juste la météo . Ça a toujours été comme ça , le temps change . Il y a des tempêtes , de la pluie et des belles journées . &quot; &quot; Mais &quot; , dit Trump , &quot; je ne pense pas qu' on doive mettre les gens de notre pays en danger . &quot; Voilà . Donc cet homme dirige la première puissance du monde , et il pense que le changement climatique , c' est une invention . Voilà qui est très dangereux . Et comme il le pense , ben il prend les mesures qui vont avec . Et notamment celle -ci : &quot; La réglementation qui oblige des centaines de centrales à charbon à fermer , et qui bloque la construction de nouvelles , est stupide . &quot; &quot; On va tout enlever . &quot; Tout enlever , mes chers amis . Les Etats-Unis d' Amérique et la Chine sont les deux premiers pollueurs du monde . La Chine est en train de ... d' appliquer un plan - puisque chez eux , ils planifient - et cette année , ils ont un meilleur résultat que l' année précédente . Même si ce n' est pas extraordinaire , c' est un début . J' ai lu que même M. Jadot le disait . Alors ... je peux donc parler des Chinois sans être aussitôt suspecté d' être un membre actif du bureau politique du Parti Communiste de ce pays comme on me l' a fait dans le passé . Donc ... si les Etats-Unis , qui déjà , ne faisaient rien , décident de faire moins que rien , c'est-à-dire de ré-ouvrir les centrales à charbon , alors ils vont aggraver le situation . La situation est la suivante : nous avons signé un accord à Paris , la COP 21 . Moi , je fais comme tout le monde , je dis : &quot; Bravo , c' est très bien , tout le monde a signé . &quot; Mais comme un certain nombre d' entre vous , il se trouve que j' ai lu ce qu' il y avait dedans . Et dans ce qu' il y a dedans , on se donne pour objectif que l' on augmente pas plus de deux degrés la température . On dit : un et demi , ça serait mieux . Forcément , parce que si on ne mettait pas un et demi , tous ceux qui habitent les îles n' auraient pas signé , vu qu' ils vont être noyés . Mais , ça c' est l' engagement que tout le monde a signé , à peu près . Pas les Etats-Unis d' Amérique . Ils ont signé , mais ça n' est pas ratifié . Ils ont d'ailleurs signé dans des conditions telles qu' il n' y ait pas besoin de ratifier . Donc là où il y avait des verbes à l' impératif , ils les ont mis au conditionnel . Mais dans cette COP 21 , les engagements de chaque Etat , ce que chaque Etat a promis de faire , quand vous mettez bout à bout , c' est 3 degrés . 3 degrés et demi . 3,5 degrés , c' est la catastrophe assurée . Je ne suis pas venu pour vous faire peur , mais juste pour qu' on regarde en face au moins un soir le problème qui se pose à nous et pourquoi il faut changer tous nos programmes politiques . Pour partir de là , et croyez -moi que si on part de là , alors on va au bon endroit . Pas seulement parce qu' on est utiles à l' intérêt général humain , parce qu' on est utiles à notre pays , pour relever les principaux défis qu' il doit relever , mais parce qu' on l' est au genre humain , dont nous sommes membres . Cette COP 21 prévoit que pour tenir l' objectif , il faut qu' il y ait une quantité de carbone , x , que l' on a le droit d' émettre . D'accord ? Là , on vient de faire le point . 80 % de ce à quoi on avait droit d' ici à 2030 , on l' a déjà émis . OK ? Maintenant , on est dans la situation du changement climatique et si on se laisse aller à des &quot; trumperies &quot; ( sic ) , on va au désastre . Alors , on va dire : &quot; Ah oui , tu t' en prends à l' Américain , et ben , je vous garantis que les nôtres , ils ne valent pas plus chers . Il y a des gens qui en doutent , ici , à propos ? On est bien en Savoie , j' ai un papier pour ça . 2016 devrait être l' année la plus chaude jamais enregistrée depuis les premiers relevés . Bon . Record historique du CO2 dans l' atmosphère . 2015 : le taux de CO2 , principal gaz à effet de serre a définitivement franchi le seuil de 400 ppm dans l' atmosphère . La Savoie : Alpes et crise écologique . 58 jours de pollution aux particules fines en 2013 . L' air de notre vallée est mortel : Dr Frédéric Champly . Vallée de l' Arve . Depuis 30 ans , les taux d' enneigement dans les Alpes ont diminué de 30 % en moyenne . Cette diminution , causée par le réchauffement climatique , s' exprime en durée , mais aussi en épaisseur : la hauteur de neige moyenne est notamment passée de 1m20 à 50cm . Dans 100 stations de sports d' hiver des Alpes sans neige d' ici 20 ans , les domaines skiables établis sous les 1800m d' altitude n' existeront plus dans 10 à 20 ans . Ça , c' est la réalité . Je vous ai donné la réponse de Trump . Et maintenant , voilà les autres . C'est-à-dire les simili-Trump ... français . Sarkozy : &quot; L' écologie , ça commence à bien faire . &quot; 7 mars 2010 . Et ça , ça se prétend être puissant . &quot; Il faut être arrogant comme l' homme pour penser que c' est nous qui avons changé le climat . &quot; Mais si , M. Sarkozy . Le climat est un système métastable , c'est-à-dire à la limite de l' équilibre , qu' un rien peut déséquilibrer . Par conséquent , même s' il est vrai qu' il y a eu des variations climatiques extrêmement importantes au fil de l' Histoire , il n' empêche que ce que la part de l' homme rajoute par les émissions de gaz carbonique et l' effet de réchauffement est déterminant . Donc ça n' est pas du tout être immodeste ou arrogant que de le constater , c' est juste être sachant . M. Juppé : &quot; En ce qui concerne l' exploitation des gaz de schiste &quot; , alors là , il n' y a pas au-dessus , hein . Il n' y a pas au-dessus . Si , il y a le charbon allemand , la lignite ... le lignite , hein . Alors , eux , ils rasent des villages entiers et ils extraient du lignite à ciel ouvert . Mais ne me regardez pas comme ça , hein , je ne suis pas germanophobe , je vous dis juste ce qui est . Je vous dis qu' il y a 400 personnes par an qui sont repérées comme étant mortes à cause de ces particules de charbon dans ce pays -ci . Ça , c' est aussi un fait . Et alors , puisque c' est le sujet , je viens du barrage , je vais vous en parler dans un instant . Vous le savez que j' étais au barrage tout à l' heure . Lui , il est pour privatiser EDF . &quot; L' Etat n' a pas vocation à diriger des entreprises . &quot; Il y a des ballots ici qui comptaient aller voter Juppé ? Il parait que c' est un modéré . Moi , je n' ai pas encore compris en quoi il était modéré , mais là , je vais vous dire , sur l' écologie , il n' est pas du tout modéré , il est carrément de l' autre côté . Dans le cas spécifique d' EDF , il détient 85 % du capital . Et ben il pense qu' il faut réduire ça , parce que entre 85 et 51 , il y a une marge . Oui , oui , entre 51 et 0 , il y en a une autre . On connait . Vous commencez par un bout et vous finissez . Pendant que je suis à la droite , Macron : &quot; Le diesel ... &quot; &quot; Le diesel reste au cœur de la politique industrielle française . &quot; Bon appétit , les gens ! &quot; Et du projet industriel français de la mobilité environnementale . &quot; Qu' est -ce que ça peut bien vouloir dire la &quot; mobilité environnementale &quot; , mais on entend &quot; environnement &quot; . Et comme on entend &quot; environnement &quot; , peut-être qu' on ne se rend pas compte qu' au passage , on a absorbé les particules fines . Dont on meurt , beaucoup dans notre pays , beaucoup . J' ai voulu vous lire ça après avoir commencé d' une manière un peu dramatique , pour vous faire mesurer le véritable écart qui existe entre ceux qui dirigent , ceux qui veulent diriger et les plus grands problèmes qui nous sautent à la figure . J' ai commencé par le plus grand de tous . Celui qui nous rattache à tous les autres êtres humains . Et donc la nécessité d' avoir une pensée politique qui part de l' humanisme et qui se pose la question de savoir comment il peut régler ce grand problème posé à tous les êtres humains . Et je sais pourquoi ils ne veulent pas en parler . Pourquoi ils ne veulent pas l' affronter . Parce que le capitalisme de notre époque est incapable d' y faire face . Il y est profondément hostile . Je vous l' explique en quelques mots . Dès lors qu' il organise le grand déménagement permanent du monde , il le fait par la multiplicité des acteurs qui interviennent sans aucune règle commune , chacun va à moins cher . C' est la première raison . La seconde , c' est que pour changer la manière de produire , la manière d' échanger , la manière de consommer , il faudrait , d' une part , consacrer des sommes considérables qui ne seraient plus dans les profits mais seraient dans l' investissement . Et ça , le capitalisme financier de notre époque l' interdit . Pour pouvoir servir des rendements supérieurs à 10 % , à 11 % , à 15 % ! Alors qu' autrefois , on faisait du 3 ou du 4 % de profits dans l' industrie . Pour ça , il ne peut pas consacrer les sommes à changer les machines , changer les process de production , changer les matériaux , payer de la recherche fondamentale qui , pendant des années , ne produit pas directement les résultats que l' on obtient de la recherche appliquée , etc . , etc . C' est une incompatibilité qui ne doit rien au complot ni à la méchanceté de ceux qui sont dedans , c' est une affaire de système . C' est le système lui-même qui produit ça et qui fait que chacun de ses acteurs ne peut pas faire autrement que comme ça , on vous le dit assez , non ? &quot; Puisque c' est moins cher là-bas , je ne peux pas faire autrement . &quot; Et ainsi de suite . Voilà la grande contradiction . Voilà pourquoi on ne peut pas en parler . Parce que ça remettrait en cause le fait qu' il y a deux manières de répondre aux situations . Soit maintenir la compétition de chacun contre tous en pensant que c' est ça qui fait avancer la société , soit partir de l' inverse , c'est-à-dire de la coopération entre tous . Mais nous , qui sommes pour la coopération entre tous , nous n' avons jamais dit que nous ne voulions pas de compétition . Nous l' acceptons , à la condition qu' elle ne mette pas en cause la solidarité et la coopération . Et de la même manière , j' estime qu' il doit y avoir aussi des gens de droite qui réfléchissent comme nous . Ils votent à droite . Bon , hé , ce n' est pas la guerre civile dans ce pays , quand même . Il y a des opinions contraires . Mais comme nous , leurs enfants respirent les particules fines . Comme nous , quand ça ne va pas fort , et bien , on mange moins bien . Et quand on mange moins bien , on est malade . Il y a une épidémie d' obésité dans notre pays , il y a une épidémie de diabète dont ces pauvres gens ne sont pas responsables , quoiqu'on les montre du doigt , souvent , ou qu' on les moque . Par conséquent , je pense que ce moment et cette prise de conscience pour quelqu'un qui vient comme moi de cette tradition là , de l' émancipation , de l' égalité , c' est un moment particulièrement favorable . Parce que nous sommes obligés de réfléchir . Nous sommes obligés de chercher des solutions et de ne pas nous en tenir aux routines . On a toujours fait comme ça , on fera toujours comme ça . Et si on continue à faire comme ça , on sera tous morts . Après avoir dévasté la planète . Voilà pourquoi c' est important après de regarder ce que chacun fait et met sur la table . Alors aujourd'hui , moi ... Ah oui , j' ai oublié de vous dire ça . Je vous ai lu les citations de droite , et là , il est marqué : &quot; Il faut leur dire que Le Pen n' a pas voté la COP 21 . &quot; Si vous êtes d' extrême droite , vous avez intérêt à avoir des gros poumons . Bon , est -ce qu' on y peut quelque chose ? Ben , bien sûr qu' on y peut . On peut plein de choses . Notamment en matière de production d' énergie . Parce que c' est par là que se font une énorme quantité de pollutions : les énergies carbonées , tout ce qu' on tire des ressources fossiles . Ma thèse est qu' on peut s' en passer . Ce n' est pas que la mienne , hein . Vous connaissez peut-être ma position , parce que c' est celle qui s' appliquera si c' est moi qui suis élu . Je suis partisan de la sortie du nucléaire . Pourtant , on me dit : &quot; Mais c' est une énergie propre . &quot; Je n' ai pas l' intention ... Je n' ai pas l' intention de montrer du doigt les gens qui ont mis en place l' énergie nucléaire , parce qu' ils pensaient bien faire et qu' ils ont agi dans des conditions que je juge assez remarquables , puisqu'en 50 ans , on n' a pas eu de catastrophes . Mais vous le savez comme moi . Maintenant , nous savons que c' est dangereux . Que c' est très dangereux . Il y a 58 réacteurs dans notre pays . Je ne vais pas recommencer à vous faire peur à tous , hein . Mais comme je sais où ils sont , moi , j' ai peur . Et je n' attends pas qu' il y ait un incident pour prendre des décisions . Parce que pour sortir du nucléaire , mes amis , il y en a pour 20 ans . Plus vous retardez la décision , plus vous augmentez la durée des risques . Par conséquent , il faut décider maintenant , et se mettre en mouvement , s' organiser . Il ne faut pas croire que sortir du nucléaire , c' est : on rentre dans une pièce , on appuie sur un bouton , ça y est c' est fini . Non . C' est long . Et il faut remplacer l' énergie que le nucléaire donnait par des énergies renouvelables . Nous pouvons planifier , m' entendez -vous . Planifier . Planifier , ça veut dire organiser . Je ne sais pas pourquoi , les gens , maintenant , ils se font peur avec un mot . Je le sais pourquoi , c' est justement pour ça que j' ai utilisé le mot . Planifier pour obliger à réfléchir ... ... qu' ils gesticulent d' une réunion à l' autre en lisant des papiers où , visiblement , ils ne savent pas ce qu' il y a dedans . C' est ça la vérité . C' est exaspérant . Lancer l' économie de la mer , ça donne 300.000 emplois . Ce n' est pas moi qui l' ai inventé dans un bureau avec ma bande de copains , c' est ce que disent les professionnels de la mer . &quot; Voilà ce qu' on arrive à faire , si on développe les différents secteurs : construction , déconstruction navales ... &quot; &quot; Mais , nous on habite dans la montagne , ça nous regarde pas . &quot; S' il vous plait , pardon . Bien sûr que ça vous regarde , c' est toute l' économie qui sera remise en mouvement . Voilà la clé de mon plan économique . C' est de relancer la machine d' une manière écologiquement responsable . Par l' économie de la mer , et ses 300.000 emplois , et l' argent qu' on va investir et les autres boulots qui viennent derrière . Tiens , par exemple , l' économie de la mer , c' est une affaire pour les coiffeurs . Ça n' a pas de rapport ? Ben si , ça en a un . Si vous avez des gens qui vont au travail , ils ont une paie . S' ils ont une paie , et ben ils peuvent aller chez le coiffeur . La première chose qu' ils suppriment quand il n' y a plus de paie ou quand il y a une mauvaise paie , c' est le coiffeur . Je prends cet exemple , je pourrais vous en dire 10 comme ça , pour vous montrer comment les choses se tiennent l' une à l' autre . Il faut tout changer . Pour installer une éolienne en mer , une pale , ça fait 70m de long . Il faut donc équiper autrement les ports , du travail pour mettre les ports en route . ensuite , il faut mettre les navires de service pour emmener les pales en mer , les installer , et une fois qu' elles sont installées , aller les surveiller . Et ainsi de suite . C' est donc une masse considérable de travail , et de travail hautement qualifié . Il faut donc que les gens , les jeunes , on les trouve , on leur propose ces métiers là . Donc qu' on arrête de tout désorganiser dans l' enseignement professionnel . Comment admettre que continuent à fermer des lycées d' enseignement professionnel du service public ? Avec des gens qui font des gargarismes du matin au soir avec l' apprentissage . Combien , parmi ceux qui recommandent ça , ont un gosse en apprentissage ? Est -ce qu' ils ne savent pas pourquoi on y allait ? Bien sûr qu' il y a des métiers où il en faut de l' apprentissage , mais ce n' est pas avec l' apprentissage qu' on va régler les problèmes de formation qui nous sont posés . Ils veulent en mettre 500.000 , en apprentissage . Nigauds ! Depuis qu' ils ont décidé d' en mettre 500.000 , depuis qu' ils donnent des prix pour qu' on aille en apprentissage , il y a moins de gens en apprentissage . Il se passe quelque chose , vous ne croyez pas ? Ben , les gens ne sont pas fous . Ils voient bien que ... 25 % des contrats d' apprentissage sont rompus dans les 3 premiers mois . Pas vu , pas pris . Le môme , il est en apprentissage , il est parti , il n' est plus nulle part . Donc il n' est plus dans aucune statistique , et tout le monde est content . Moi , voilà comment je vois les choses , comment on peut organiser , et donc pour ça , planifier . De ceux qui apprennent à faire , ensuite faire , et ensuite , partir de l' exigence écologique . Si vous vous occupez de la mer , vous êtes obligés de vous occuper de la terre . Parce que si vous vous occupez de la mer , vous vous apercevez que les coquillages , que les algues , que toutes les choses que vous y installez ne peuvent pas tenir le coup si vous continuez à faire l' agriculture de brutes que vous êtes en train de faire et qui est en train de tout détruire : la terre , l' air et ceux qui travaillent la terre . Ça , c' est la vérité dans le pays , record d' Europe de consommation des pesticides . Et ce que je raconte , je ne l' invente pas . Il y a déjà un territoire grand comme la France entière dans le Golfe du Mexique où il n' y a plus trace d' organisme vivant . Vous m' entendez ? Il n' y a plus rien . Rien de rien . Du sable au fond de la mer , et encore , il n' y en aura pas si longtemps que ça parce qu' à force de prendre du sable pour faire du béton , il n' y aura plus de plages d' ici à 50 ans . Alors , ça vous a ... ce soir , vraiment , il n' y a plus ni plages , ni air , ni ... Mélenchon , c' est terrifiant comme meeting , hein . Mais bon , c' est un défi , c' est ça qui est intéressant . C' est un défi donc on va le régler par de la technique , par de l' invention , par de la solidarité . Vous êtes obligés de vous occuper de l' agriculture , je viens de vous le dire , donc vous voulez une agriculture bio , hein ? Et ben oui , c' est l' agriculture paysanne , comme on dit . Ben , il faut les payer . Vous êtes d'accord ? Et ben oui , il faut les payer . Ça prend plus de temps . Donc si vous voulez les payer correctement , il vous faut une meilleure paie . Par conséquent , si le SMIC augmente , c' est écologique . Mais bien sûr ! Ou alors ... ou alors , le bon manger , c' est seulement pour ceux qui ont des sous ? Et vos gosses , qui c' est qui est d'accord pour lui faire boire du lait plein d' antibiotiques ? Qui sort de la ferme des mille vaches ? Et ben oui . Tout est comme ça . Si vous voulez que ça aille mieux , il faut que tout le monde aille mieux . Et pour que tout le monde aille mieux , il faut partager . Il faut répartir la richesse autrement . Je veux bien qu' on me traite de bolchévik pour ça , de rouge , d' exagéré , et tout ce qu' on veut , mais est -ce que quelqu'un est capable de me montrer comment on peut faire autrement , pour atteindre le même résultat ? Non . Non . Et ceux qui , pendant 10 ans , 20 ans , 30 ans nous ont bourré la tête avec leur flexibilité , leur compétitivité , leur mondialisation qui semblait tomber du ciel , à quel résultat sont -ils arrivés ? Où ça va mieux pour nous ? Dans quel domaine ? Ce n' est pas vrai . Ah ben , je vous ai parlé de l' agriculture , je termine juste là-dessus : il me faut 400.000 personnes pour faire l' agriculture paysanne dont je vous parle . Mes amis qui ont préparé le programme agricole , ils étaient tout contents , ils ont tout prévu , je vous garantis qu' il ne manque pas un bouton de guêtre . Evidemment , la FNSEA va être très fâchée . Et ben , qu' est -ce que vous voulez , c' est comme ça , hein . Si c' est eux qui gagnent , ils continueront à tout pourrir . Si c' est nous , ben ... on changera la manière de faire . Sauf que je ne fais pas le fier , parce que 400.000 personnes pour faire de l' agriculture paysanne , mes amis , il faut les trouver . Il y en a déjà 200.000 qui bossent , il va falloir qu' ils changent leur manière de bosser . Mais quand même qu' il y ait 100.000 jeunes qui attendent pour pouvoir être paysans eux-mêmes , il faut encore que j' en trouve 100.000 , ou 150.000 . Et ceux-là , je ne vais pas aller leur dire : &quot; Ah , j' ai une idée formidable pour toi ! Toute ta vie au cul des vaches . Pas de vacances , pas de loisirs , une paie de misère . Ça te va ? &quot; C' est ça , paysan , en France . Il s' en suicide un tous les deux jours . Dans une indifférence totale , on en parle jamais . Une misère , une détresse , une solitude totale . Des gens qui , au départ , se disaient : &quot; Je vais faire des animaux , je vais faire des céréales &quot; , ou que sais -je encore , ils se retrouvent à faire des paperasses pour avoir droit à ceci , droit à cela . Bon , il faut qu' on en trouve 100.000 ou 200.000 . Pareil , on va donc changer la condition paysanne . Il va falloir surtout changer l' éducation en amont , c'est-à-dire que ceux qui vont au lyc ... Il y a plein de jeunes , hein , qui veulent y aller à la terre . Et dans ... dans les fermes bio que j' ai visitées , ceux qui me racontent que c' est l' âge de la pierre , ils feraient bien d' y aller , hein . Parce que c' est tout bac+1 , bac+2 , bac+5 . Je suis allé dans une , dont je n' arrive jamais à me rappeler le nom , ils avaient là 70 vaches , ils vivaient à 10 dessus . Avec des paies , bon , moyennes , mais ... tu te payes bien , quoi , pour vivre , hein . Ils vendaient en circuit court . Vendre en circuit court . Relocaliser . Je ne dis pas qu' on peut le faire tout le temps , mais on ne doit pas avoir comme objectif qu' on ne le fera jamais , et que la seule manière de produire qui vaut la peine , c' est celle qui permet de faire de l' exportation . Non . Le circuit court est rentable . Ceux qui arrivent à installer des circuits courts paysans en vivent . Et il n' y a pas que dans la paysannerie que c' est comme ça . Il faut arrêter le grand déménagement du monde , dont je vous ai parlé tout à l' heure , qui est la cause numéro un du changement climatique et du reste des autres catastrophes écologiques . Ça , ça va m' amener à Ecopla . Parce qu' après tout , c' est un exemple , non ? C' est un exemple de deux choses : d'abord , du fait qu' il y a un problème avec les tribunaux de commerce . Je le laisse pour ce soir , j' ai assez d' ennemis pour aujourd'hui . Mais , regardez , c' est toujours la même logique : une entreprise est là . Vous avez des gens dedans et qui est -ce qui fait la fortune de l' entreprise ? Apparemment , ce n' est pas le capital , c' est bien les gens qui sont là-dedans qui savent le faire . Bref , vous connaissez l' histoire mieux que moi , puisque vous êtes du coin . Primo , six fonds de pension , un derrière l' autre , achètent cette entreprise . D' où elle sort , cette entreprise ? Ceux qui ont mon âge , là , vous vous en rappelez . Pechiney ! Pechiney , c' était le grand groupe aluminium français . Privatisé , cœur de métier , blabla , découpé en tranches , en petits morceaux , et à la fin , il ne reste plus rien . Il reste Ecopla . Ecopla , c' est une lointaine subdivision de la grande entreprise intégrée qu' était Pechiney . Et maintenant , voilà que un derrière l' autre , alors forcément , les types , ils viennent là placer de l' argent , pourquoi ? Ben , parce que cette petite entreprise produit 70 % ou 60 % des emballages aluminium qui sont utilisés dans notre pays . Les clients sont contents . Le fonds de pension fout le camp . Deux leçons à apprendre . Premièrement , on se rappelle que c' est les gens qui font la fortune de l' entreprise , deuxièmement , que le capital ne peut pas avoir tous les droits , et là , il a un droit incroyable . Il rentre , il sort , il fait ce qu' il veut . Non , moi , je ne suis pas d'accord . Je ne suis pas d'accord . Donc , comme je ne vous propose pas de nationaliser toutes les entreprises , comme on me l' accuse souvent . Heu , pff ... si c' était possible , moi , je n' y verrais pas d' inconvénient , mais comme ça a l' air compliqué à faire d' un seul coup , je ne m' y prends pas par ce bout là , et en plus , vous trouveriez toujours des raisons d' avoir peur et moi , je m' épuiserais à essayer de vous convaincre . Bon . Ben dans quoi je suis parti , là ? C' est parce que dans ma tête , il y a , vous savez que la seule solution qui reste maintenant pour rattraper tout ça , c' est de nationaliser ce machin microscopique . Parce que comme la justice est passée là-dessus , on ne peut plus rien faire . La ... la leçon numéro deux , je disais , c' est les droits du capital . Je suis pour qu' il y ait un droit différent de vote , pour le capital , suivant la durée de l' investissement . C'est-à-dire , si vous refusez de signer le papier qui dit : &quot; Je m' engage à maintenir mon investissement pour 4 ans , 5 ans ou 6 ans &quot; , et ben vous avez zéro droit de vote . Parce qu' il n' y a aucune raison que vous ayez le pouvoir de décider dans une entreprise que vous vous apprêtez à quitter . Et ça , tout le monde peut le comprendre . Ne me dites pas que c' est du communisme . Parce que j' ai connu pire que ça . Au Brésil , ils mettaient un dépôt de garantie , c'est-à-dire ils investissaient , mettons 100 , et ils mettaient un dépôt de garantie , disons 10 . Et si ils ne tenaient pas la durée de l' investissement , pfuit , 10 , ils les perdaient . Ça restait pour l' Etat . Donc vous voyez qu' il y a des mesures très dures qui ont été appliquées dans le monde pour contrôler la circulation des capitaux . Et notamment des capitaux investis dans l' activité productive , c' est celle-là qui nous intéresse . Deuxième conclusion : puisque c' est les gens - et vous aviez l' air d' être d'accord - qui font la richesse d' une entreprise , quand ils veulent , lorsque le patron s' en va , quand ils ont le bonheur de savoir qui c' est , et bien , s' ils veulent , eux , reprendre l' entreprise , ils doivent avoir un droit de préemption . C'est-à-dire que le premier qu' on écoute , dont on regarde le projet et à qui on donne l' entreprise , ce sont les salariés constitués en coopérative . Et ça , on peut le faire dans des tas d' activités . Là , c' est ce qu' ils se proposaient de faire , comme vous le savez . Mais vous avez vu , au cours des 2 ou 3 dernières années , comment , dans le mouvement ouvrier , je dis bien le mouvement ouvrier , parmi les ouvriers , les salariés , cette idée de l' utiliser , la coopérative , comme un moyen de maintenir un instrument de production , parce que seuls les salariés , les ouvriers savent le faire tourner , et ils savent qu' ils en tirent un rendement , qui , certes , est insuffisant pour un fonds de placement ou pour un capitaliste qui veut faire du 10 ou du 15 , mais eux , ils se contentent du 2 ou du 3 et d' avoir leur boulot . Et bien , il faut que ceux-là aient la priorité . C' était ce qui était prévu . Pourtant , je l' ai marqué sur mon papier : &quot; On ne parle pas du PS . &quot; Je suis obligé , c' est eux qui gouvernent . Alors , il était prévu qu' il y aurait ce droit de préemption inscrit dans la loi . M. Hamon se vante de l' y avoir mis . Hé , pas de bol , non . Juste : &quot; Les travailleurs ont le droit de présenter leur projet . &quot; Ben , ils sont contents avec un droit pareil . Et quand un entrepreneur vend l' entreprise , et bien , il y avait obligation - c' était ça qu' on avait dit - si ils vendent l' entreprise , il y a obligation qu' ils cherchent que ... qu' ils le donnent à un repreneur . Non . Maintenant , la loi a été arrangée à la sauce - comme toujours , hein , on fait les choses à moitié , et quand on les fait à moitié , ben , on ne les fait pas du tout - ils ont fait les choses à moitié . On me dit : &quot; Ah , non , obliger à retrouver un repreneur , non . A rechercher . &quot; Voilà . Ça veut dire rien , quoi . Le droit de préemption , ils ont le droit de présenter leur plan . Ça veut dire rien . Voilà . Donc , ça , c' est deux sujets sur lesquels il faudra qu' on intervienne très rapidement , parce qu' avec ça , on tient en respect un tas de monde . Parce que des entreprises très profitables en France qui ferment , il y en a. Parce qu' elle ne produisent pas assez pour le repreneur . J' ai connu Kem One , peut-être que ça vous dit quelque chose ici , ils fabriquent ... c' étaient les seuls qui fabriquaient du PVC dans le pays . Alors , c' était encore une subdivision de l' ancienne grande chimie en France . Et puis , de fil en aiguille , la boite tournait , hein , forcément , ils étaient seuls , mais elle ne faisait que ; que du 10 % , et donc le fonds de placement , lui ce qu' il voulait , c' était 15 % . Parce que le fonds de placement , ce n' est pas qu' il est cruel , c' est qu' il a promis à des gens de leur payer des retraites avec ce qu' il va ramasser comme argent . Donc , il faut bien qu' il fasse son boulot , c'est-à-dire vous presser au sang pour ... pour verser aux autres . C' est comme ça ... c' est pour ça que l' Allemagne marche comme ça . Parce qu' ils ont des retraites par capitalisation . Donc ils ont besoin qu' il y ait beaucoup de dividendes , donc des salaires bas , et deuxièmement , un Euro très fort pour qu' ils achètent à bas prix tout ce qui vient du reste du monde . Il ne faut pas chercher plus loin , hein . C' est pour ça qu' ils sont comme ça et c' est ça qu' ils imposent à tout le reste de l' Europe . Bref , donc ce Kem One , et ben , la boite a été fermée , liquidée parce qu' elle ne faisait pas 18 , et à chaque fois , qu' est -ce qui se passe ? Vous avez un vautour qui arrive . Il achète les murs , il regarde la trésorerie , comme à Ecopla , il regarde ce qu' il y a dedans , tchouf , il le prend pour rembourser l' emprunt qu' il a fait pour acheter l' entreprise en question , il fout tout le monde dehors , ça , c' est vous qui payez et puis lui , il vend le terrain , les murs , les machines en pièces détachées . Aux uns , aux autres , d' un côté , de l' autre . Et voilà comment , comme un véritable vautour , une entreprise est tuée , détruite , les gens foutus dehors , les risques écologiques , c' est pour vous . Et voilà comment tout ça fonctionne dans un même système , dans un même registre qui aboutit à ces résultats là . Donc il faut que les travailleurs puissent reprendre leurs entreprises . Je ne sais pas si je vous ai convaincus . J' espère bien . Mais moi , je pense que si on fait comme ça , on va donner un poids supplémentaire à l' économie sociale et solidaire dans notre pays . Et ça va nous servir à autre chose , c' est quand on va voter , les amis , les patrons , pas vous . Parce que dans ce pays , le MEDEF a une trop grosse bouche , et son organe principal , la glande lacrymale , pour pleurer et mendier de l' argent public en diminutions d' impôts , en contributions de cela , en subventions de ceci , ces gens là nous coûtent beaucoup trop cher . D' une façon générale , les patrons du CAC40 coûtent beaucoup trop cher . Et puisqu'ils arrivent à trouver des ouvriers à moins cher , nous , on est capable de trouver des patrons à moins cher . C' est une blague du syndicat des métallos allemands . Parce que vous avez vu que cette année , ils sont record d' Europe dans l' art de s' être goinfrés . Nous avons les patrons qui sont les mieux payés , et cette année , ils se sont augmentés à peine de 20 % , quand même . Eh , attendez , compte tenu des efforts . Et c' est comme ça sans arrêt . Alors , record d' Europe des dividendes versées . Et vous apprenez ça l' année où vous apprenez qu' on leur a filé 20 milliards de crédits d' impôts . Où ils sont passés les 20 milliards ? C' est où , les emplois ? C' est où , les machines nouvelles ? Où elles sont ? Où c' est ? Montrez -les nous , qu' on y aille . On ira même avec Hollande sous le bras pour inaugurer . Montrez -nous ! Pillards , bons à rien , parasites ! Tout cet argent a été perdu . Perdu . Pas pour tout le monde , mais perdu . Et pendant que eux s' enrichissaient comme ça : &quot; Ah , vous êtes trop simpliste , M. Mélenchon . &quot; Ah non , c' est vous qui êtes simplistes . Je connais même quelqu'un qui m' a dit une fois : &quot; Mais moi , ça ne me dérange pas qu' il y ait des riches . Ce qui m' embête , c' est qu' il y ait des pauvres . &quot; Vous ne voyez pas le rapport entre les deux ? Vous ne voyez pas le rapport ? Vous n' allez quand même pas me dire que c' est les riches qui prennent l' argent des pauvres ? Ben , si . Hé hé , à qui voulez -vous qu' ils le prennent ? C' est parce que vous avez tout déréglementé que vous avez amené ces gens à travailler pour des ... pour des paies aussi misérables qui ne permettent pas de faire vivre la famille , c' est parce que vous avez tout déréglementé que tout se casse la figure , et que des gens qui vivaient chichement vivent dans la rue , et que ceux qui vivaient moyennement vivent chichement . C' est vous qui êtes responsables de ça . Personne d' autre . Arrêtez de culpabiliser les gens . De leur dire qu' ils ne travaillent pas assez , que sais -je , je ne sais pas encore , tout ce qu' ils ont bien pu dire , d'ailleurs , vous les écoutez , j' espère . Ils ont fait des primaires . Alors ça , c' est intéressant . Pour une fois , je suis resté devant la télé , d' habitude , je ne la regarde pas . D'abord , il y a une chose dont ils ne parlent pas . On est tranquilles . C' est d' écologie . Ça ne les concernent pas , ils ne sont pas des êtres humains comme les autres , eux ils respirent très bien le fioul ... le charbon , ça leur va . Heu , le manger dégoûtant aussi . D'ailleurs , ils n' en mangent pas , ils mangent autre chose . Et caetera . Moi , je vous ai dit tout à l' heure , je suis pour relocaliser . Pour relocaliser autant qu' on peut , il faut utiliser un mot dont il ne faut pas avoir peur : c' est le protectionnisme . Et je parle de protectionnisme solidaire . Alors , il y a M. Lenglet qui m' a dit : &quot; Pfou , vous mettez deux mots côte à [ côte , ça ] ne veut rien dire . Ben si , ça veut dire . Ça veut dire qu' on protège et on négocie . Comprenez bien ça : la différence entre protéger et négocier d' un côté , ou bien dire : &quot; Il y a une règle , c' est le libre-échange . &quot; Le libre-échange , c' est ce que je vous ai raconté tout à l' heure . L' Union Européenne actuelle , elle prévoit le libre-échange . Non seulement pour elle-même , mais avec le reste du monde . Et non seulement elle et le reste du monde , mais dans le reste du monde , elle veut qu' il y ait le libre-échange . Et c' est ce libre-échange qui permet aux uns d' inonder les autres de productions , et ces autres productions détruisent les productions locales . C' est le cas notamment dans toute l' Afrique . Avec les accords de partenariats économiques où on dit : &quot; Vous avez le droit d' amener chez nous tout ce que vous voulez , et nous , on a le droit d' amener chez vous tout ce qu' on veut . &quot; C' est ce qui a été traité de &quot; sinistre plaisanterie &quot; par le Président de l' Afrique du Sud , et même le Président libéral du Sénégal a dit : &quot; J' espère que vous n' y croyez pas . &quot; Ben , si , ils y croient et ils le font . Donc tous ces Etats perdent leurs ressources de droits de douane , et surtout , ça démantèle tout ce qu' il y a derrière . Vous , vous ne le voyez pas . On me dit : &quot; Mais , M. Mélenchon , mais pourquoi est -ce que vous n' êtes pas content ? Regardez tel pays , il produit maintenant du riz qui se vend en dollars dans le monde . &quot; Ouais , ouais , d'accord , il produit du riz qui se vend en dollars dans le monde . D'ailleurs , ça tombe bien , ils ne mangent pas le riz , donc ils peuvent le vendre aux autres , hein . Ils mangent autre chose . Mais ce que vous ne voyez pas , c' est que tout ça , ça apparaît pour la première fois , parce que c' est à l' export et c' est en dollars et c' est pour le reste du monde , mais là où il y a du riz aujourd'hui - je dis du riz , je pourrais dire autre chose , hein - il y avait avant des agricultures vivrières qui , elles , n' apparaissaient pas en dollars , mais dans la monnaie locale , et des fois dans des trocs . C' étaient les productions pour vivre . Pour le petit marché du coin , et vous aviez des familles de paysans qui vivaient comme ça , et qui , du jour au lendemain , ne peuvent plus en vivre , donc ils vont en ville , et en ville , il n' y a pas de travail , donc de la ville , on part , et on se déplace . Et on émigre . Et on a le bonheur de faire de la navigation en Méditerranée . Après avoir consulté le Code du travail de chaque pays pour savoir dans quel endroit c' est le plus favorable . Enfin , je vous dis ça , si c' est que je les écoute eux . Nous , on sait ce que c' est . Nous , on sait ce que c' est : c' est des milliers de pauvres gens . Obligés de partir . Obligés , parce que si c' était vous , vous le feriez . Vous ne resteriez pas là , à vous regarder mourir et mourir vos enfants . Vous le feriez . Je ne vous parle même pas des guerres . Je vous parle de ça , de la vie de tous les jours , du mécanisme du libre-échange qui broie , détruit tout autour de lui . Et puis là-dessus arrive la sécheresse . Le changement climatique . L' ONU a prévu 250 millions de réfugiés . Tant que vous ne les aurez pas aux portes , vous ne comprendrez pas qu' il faut arrêter ? Et quand ils arrivent , que vous ne pouvez pas simplement dire : &quot; Je vais les refoutre à la mer . &quot; Parce que ce n' est pas ... ce n' est pas jouable , ce n' est pas possible , donc il faut bien intervenir sur la cause de départ . Bien sûr qu' il y a un problème , quand des foules pareilles se déplacent . Et si je le dis moi , ça ne fait pas de moi un xénophobe . J' ai vu qu' on me reprochait ça . Mais c' est absurde . Depuis l' origine du mouvement auquel j' appartiens , du camp progressiste , enfin , je ne veux pas choquer ceux qui ne seraient pas de ma famille politique , mais depuis le début , on dit qu' il faut que les gens puissent vivre et travailler au pays . Ça vaut pour nous , ça vaut pour les autres . Il faut qu' ils puissent développer leurs pays . Que c' est une honte qu' on leur prenne les meilleurs éléments , les plus courageux . Et même moi , il m' arrive de dire des bêtises , une fois , j' ai dit - on me dit : &quot; Alors , et l' immigration , vous croyez que c' est une bonne chose ? &quot; J' ai dit : &quot; Ça dépend . &quot; J' aurais mieux fait de me taire . Alors , comme je m' étais rendu , j' ai dit : &quot; Ben oui , s' il y a des médecins qui viennent . &quot; Mais non , Mélenchon , tu as tort . Tu as tort , on ne va pas leur prendre leurs médecins , en plus . C' est eux qui ont payé la formation . La bonne ligne , c' est arrêter les guerres , arrêter les accords de partenariats économiques , arrêter le libre-échange , tant qu' on peut , graduellement , en s' organisant . Avec des méthodes de travail . Je vous ai dit tout à l' heure : la relocalisation . Relocalisez l' agriculture , vous verrez que vous ferez vivre plus de monde de la terre , donc vous aurez de l' emploi , produisant des choses de meilleure qualité , et à la fin , tout le monde sera content . Enfin , pas mal seront contents . En tout cas , on ne sera plus dans la situation dans laquelle on est aujourd'hui . Et si vous ne voulez pas faire ce que je suis en train de vous dire , dites -moi voir ce que vous allez faire au juste . Le marché ! Voilà . Ça , ça marche pour l' agriculture . La dernière fois , c' était le lait . On fait du lait , ici ? Oui , hein . Dans les années 1970 , on a mis les quotas laitiers . Déjà , une série de très intelligents ont hurlé contre . &quot; Ah , vous voulez nous empêcher de travailler , vous mettez une limite aux quotas laitiers . &quot; &quot; Avec du bon quota laitier , tout ça ... &quot; Bon , bon . Très bien , on fait les quotas laitiers . Arrivent tous les libéraux : &quot; Assez , à bas les quotas laitiers , la politique agricole commune . &quot; Tout le monde oublie que c' est grâce à ça que le continent est devenu autonome sur le plan alimentaire . D'accord ? En moins de 20 ans . Que les paysans ont donné un bon paquet . Tous les gens qui vivent en ville peuvent leur dire merci , parce que , plutôt que d' augmenter les salaires des ouvriers , pfuit , on serrait la vis aux paysans , donc le prix de la nourriture baissait , donc le gars , avec le même SMIC , il avait l' impression qu' il avait plus d' argent qu' avant . C' est comme ça que ça s' est passé pendant les fameuses 30 glorieuses . C' est les paysans qui ont tiré la langue jusqu' à par terre pour que le reste se mette en route . Donc , les quotas laitiers , ils ont fait un marché . Voilà . Depuis que l' humanité est humanité , on fait des stocks , on essaye de ne pas être trop dans le cyclique , hein , parce qu' une année est bonne , l' autre est moins bonne donc on essaye un peu de faire des réserves , etc . , et ben , le marché il a inventé : il n' y a plus de saison . Il s' en fout . Parce que tantôt c' est l' hiver ici , là-bas c' est l' été , blabla , blabla , blabla . Tout le monde va faire du lait . Et bien , il y a une année , c' est bon , donc tout le monde fait du lait , il y a trop de lait , les prix s' effondrent , les paysans sont ruinés , l' année d'après , il n' y a pas assez de lait , le prix monte pour vous tous . OK ? C' est ça qu' ils ont fait . Et ils continuent comme ça , compartiment par compartiment . Le libre-échange , la concurrence libre et soi-disant non faussée , tout ça , c' est ça qui est en train de tuer tout le système productif . Je vous ai parlé de protectionnisme solidaire . Je l' assume . Vous voulez un exemple ? Non , c' est trop tard . Sur mon papier , il y en a un . Ah oui , j' ai lu ça , tiens . M. Juppé , qui n' en rate pas une : &quot; Le protectionnisme est un grand danger pour l' économie mondiale et pour la paix du monde . &quot; &quot; Je n' aime pas le mot de protectionnisme . Un mot du passé et rétrograde . &quot; Bon . Allez , un partout : &quot; Je n' aime pas le mot concurrence libre et non faussée , et libre-échangisme , un mot du passé et rétrograde . &quot; Ça , c' est Mélenchon . Alors ... un des instruments du protectionnisme solidaire , c' est les taxes à l' entrée des frontières , ou les visas , appelez ça comme vous voulez . Visas , c' est mieux . Parce qu' il y a l' idée de taxer les produits , et en même temps d' en autoriser . Visa social , visa écologique . Parce qu' aller produire à l' autre bout du monde , en éreintant des gens au travail , en les sous-payant , et surtout en ne s' occupant de rien pour l' environnement , pour que vous puissiez avoir moins cher , le tee-shirt que vous avez déjà en 3 exemplaires . Donc ces taxes , elles permettraient de filtrer . Ah , voilà encore une invention bolchevique . Et il y a un nouveau bolchévik dans la famille , c' est M. Sarkozy . Parce qu' il y a quelques jours , il a dit que si M. Trump ne respectait pas l' accord de Paris ... On fait une pause , hein . Si M. Trump ne respecte pas l' accord de Paris , point . Hé , pourquoi il le respecterait , ils ne l' ont pas signé . Mais il n' a pas l' air d' être au courant , M. Sarkozy . Il n' y a pas d' engagement des Etats-Unis à appliquer l' accord de Paris . C' est une déclaration d' intention du Président Obama et ça n' a pas valeur de loi , bref . Il dit : &quot; Si il n' applique pas l' accord de Paris , la COP 21 , alors nous mettrons une taxe carbone . &quot; Une taxe carbone ? C' est exactement ce qu' on raconte depuis 10 ans . Qu' on dit : &quot; Il faut mettre - les marchandises qui viennent de loin doivent avoir une taxe kilomètre . &quot; Parce que ... leur cheminement a coûté à l' équilibre de la planète . Donc , il faut mettre une taxe . Et puis surtout , comme eux , ils ne respectent pas les normes environnementales qu' on respecte nous , c' est exactement ce que vient de dire Sarkozy , et bien , alors on leur impose de les respecter . Parce que s' ils ne les respectent pas , leurs marchandises ne rentrent pas chez nous . C' est facile à comprendre , ça . Ça passait pour une lubie il y a encore un an . Et bien maintenant , vous le retrouvez , y compris dans des programmes de gens qui ne sont pas de mon bord , non ? Alors ... ça prouve que je réfléchis plus vite qu' eux . Et pas que moi . Bon . Donc ils font une primaire . Ils ne parlent de rien . Si , ils parlent de quelque chose . Franchement , j' espère que vous avez des copains qui votent à droite , ou des copines . Non mais , parce qu' attendez , ils vont être en grande détresse . Ils assistent à 3 débats . On ne leur parle pas une seule fois d' écologie . En fait , on leur parle de rien . Ou de choses étranges . Alors , d'abord , je commence par le modéré . M. Juppé , lui , il veut mettre la retraite à 65 ans . Et vous savez que ça veut dire ... Vous savez , il y a toujours un décalage pour la toucher à taux plein . Enfin ... à taux plein ... parce qu' il y a une décote qui est lié au fait que , ben , tout le monde , maintenant , a eu des périodes d' inactivité , etc . Donc , ça veut dire que pour la toucher à taux plein , il faut attendre 70 ans . Vous le croyez , un truc pareil ? Le pays est plus riche qu' il ne l' a jamais été , on est plus développés que jamais , et on va mettre des gens au travail , on va les maintenir jusqu' à 70 ans . Alors , bien sûr qu' il y a des métiers où c' est possible . Hein . Mais il y a des métiers où ce n' est pas possible . Parce que déjà à 50 , on n' en peut plus . On est déjà cassés . Physiquement , je ne parle pas mentalement , hein . Mais on ne peut plus continuer . Regardez le nombre de métiers où c' est comme ça . Ils sont fous , ces gens là . Ils ne vivent pas dans la vie réelle . Qui c' est qui monte dans un avion piloté par un type qui a 70 ans ? Pas moi . Ah , ben moi , avant de monter en avion , je demande qui c' est qui le pilote . Quel âge il a , d' où il vient , tout ça . Bon . Je blague , mais c' est pour vous faire comprendre - alors peut-être qu' un pilote de 70 ans , il peut être tout à fait capable de conduire un avion - mais c' est pour dire que ce n' est pas raisonnable de travailler comme ça . Vous imaginez , des instituteurs , des institutrices de 70 ans ? Allez les mettre en maternelle , pour voir . Ma mère y était , en maternelle . Moi , j' y suis allé une fois pour faire un reportage , j' étais journaliste . Je n' ai pas tenu une demi-journée . Vous le savez que ce n' est pas possible . Alors pourquoi ils le disent ? Ils savent très bien que personne ne va embaucher quelqu'un de 60 , 65 , déjà qu' ils ne les prennent pas à 55 . Alors pourquoi ils disent ça ? Pourquoi ? Pour que vous payiez des assurances avant . Pour que vous ayez des assurances-vie , pour que vous soyez obligés d' avoir des fonds de capitalisation . Voilà pourquoi . Pour que vous prépariez une retraite que vous ne pourriez pas toucher autrement , à partir d' un certain âge , blabla , blabla . Voilà pourquoi ils le font . Et parce que votre vie ne les intéresse pas , mais moi , j' en connais des gens de 60 , des gens de droite qui travaillent . Il ne faut pas croire , on ne va pas faire les malins , il n' y a pas que nous , hein . Qui travaillent , des fois dans des métiers durs , mais ils votent de droite . C' est ballot . Mais c' est comme ça . Ah ben tiens , du temps du programme commun , je ne veux pas rappeler de mauvais souvenirs à ceux qui seraient de droite dans la salle , mais 70 % des travailleurs votaient avec nous . Moi , j' ai fait programme commun . Mais il y en avait 30 % , ils ne voulaient rien savoir , hein . La retraite à 60 ans , ils ne voulaient pas en entendre parler , l' augmentation du SMIC , ça ne les concernait pas , bref , ils croyaient tout ce que leur disait le patron . Et bon , voilà , le patron , il leur avait dit : &quot; Ils vont ruiner l' économie . &quot; Ils l' ont cru . Bon , c' est pour dire , il y a toujours eu une partie du monde du travail qui a voté avec les gens de droite , et je le redis , ce n' est pas la guerre civile dans ce pays , on a le droit d' être d' un autre avis , n' est -ce pas ? Ce n' est pas la condition sociale qui fait votre conscience . Elle y est , ça c' est clair . Mais on a tous une tête , hein , quelque soit notre place dans la société . Bref . Tout ça pour dire que ces gens là , ils doivent bien être comme nous . Ils doivent se dire : &quot; Ben moi , je n' en peux plus à 50 ans . &quot; Ou bien l' autre va dire : &quot; Je n' en peux plus à 60 . &quot; Alors , ils écoutent le débat . Et qu' est -ce qu' ils entendent ? Qu' ils vont gratter jusqu' à 70 . Ils doivent être désespérés . Je ne sais pas pourquoi ils disent qu' ils attendent beaucoup de monde pour venir voter entre des gens qui leur proposent des horreurs pareilles . Alors , 64 ans la retraite pour Sarkozy , 65 pour Juppé . Les autres , je ne les ai pas dans ma feuille . Voilà . Fin des 35h et retour aux 39h . C' est M. Juppé . Le type , il n' est pas au courant que les 35h , il y a déjà un moment qu' il n' y en a plus . Parce que si vous baissez ... si vous baissez le tarif des heures supplémentaires , c' est comme s' il n' y en avait plus . Hein ? Voilà . Merci Mme El Khomri . Fin de la durée légale dans le privé . Vous comprenez ? Non , vous n' avez pas compris , je parie . Ben , ça veut dire qu' il n' y a plus de durée légale . La durée légale , aujourd'hui , c' est 35h , et après , c' est les heures supplémentaires . Ben , il n' y en aurait plus . Voilà . Ou sinon , tiens , je vais vous prendre un autre type de droite , Macron . Ben lui , il propose que la durée soit négociée entreprise par entreprise , vous savez , &quot; au plus près du terrain . &quot; Là où vous êtes avec votre pote , le patron . &quot; Salut Marcel . &quot; &quot; Salut Paul . &quot; &quot; On va négocier ma paie : tiens , ce mois -ci ... &quot; &quot; Ce mois -ci , j' ai besoin de 4 heures sup ' , là , parce que il y a les impôts locaux qui sont arrivés , j' ai quand même besoin un peu ... On est d'accord ? &quot; &quot; Ouais , ouais , bien sûr . Vas -y . Reste au boulot , je te paye . &quot; Voilà . Dans quel monde vivent tous ces gens ? Dans quel monde vivent -ils , hein ? Ben , dans le leur . Alors , moi , je ne comprends pas les gens de droite qui vont voter pour des types pareils , des allumés pareils . Fin du CDI , pour M. Juppé . Dégressivité des allocations chômage . Vous savez pourquoi : &quot; Parce que vous êtes des fainéants . Vous aimez les allocs , et vous ne cherchez pas du travail . &quot; Quand quelqu'un vous dit ça , mais il vous insulte . Pourquoi il propose la dégressivité ? Vous avez moins besoin quand vous êtes depuis plus longtemps au chômage ? Ce n' est pas vrai , c' est le contraire , vous avez plus besoin . Pas moins . Pourquoi alors ils font baisser l' allocation ? Pourquoi ? Parce qu' ils partent de l' idée que vous êtes en train de truander . Dans leur esprit , tout le monde passe sa vie à truander , parce que eux truandent . Le fisc , sans arrêt . Alors ils pensent que tout le monde est pareil . 85 milliards par an de vol fiscal . 1.000 milliards par an en Europe . Alors , évidemment , ils se figurent que les gens normaux sont comme eux , et passent leur temps à essayer de voler les autres . Quoi , encore ? Ah , les cadeaux . Déjà , on leur a donné 40 milliards , n' est -ce pas ? Au MEDEF et au CAC40 , ça c' est à M. Hollande . Ben comme , évidemment , l' autre , il a lâché la bride , alors , du coup , maintenant , tout le monde y va . 14 milliards de plus veut leur donner M. Sarkozy . 35 milliards , M. Juppé . En avant , c' est la fête au MEDEF . 35 milliards , et qui va les payer ? Alors , les gens écoutent ça : &quot; Oui , c' est vrai , il faut baisser le prix du travail . &quot; Non , ils disent le coût , eux . C' est moi qui dis le prix . Baisser le coût du travail . Ah ouais . D'accord . Enfin , il n' y a que le travail qui produit de la richesse , hein , j' espère que vous l' avez compris , ça . Il n' y a pas une machine qui marche toute seule , hein . Ça n' existe pas . Il y a toujours des êtres humains . Donc c' est les êtres humains qui font ... c' est leur travail qui produit la richesse . Alors qui c' est qui va les payer , les amis ? Puisque vous êtes tous d'accord , vous êtes tous devenus tellement intelligents , que vous avez compris qu' il faut baisser le coût du travail pour améliorer l' économie , la compétitivité , la-globalisation-dans -le monde-mondialisé . Hein ? En un seul mot , s' il vous plait , avec des traits d' union . Qui c' est qui va payer ça ? Parce que vous croyez , vous , qu' on supprime les cotisations sociales et que vous continuez à avoir la Sécu ? Ah , ben , non , vous ne pouvez pas . Il faut que quelqu'un paye . Donc si vous ne le mettez pas là , il y a une loi . Et la loi , elle dit que chaque euro dont l' Etat décide qu' il est pris aux caisses de Sécurité Sociale , parce qu' on ne la leur fait pas , il doit le rembourser . Donc c' est vous , les vrais intelligents . C' est vous qui devrez payer , par vos impôts , ce que ne sera plus payé dans l' entreprise en cotisations . Parce que c' est la loi . Et si vous enlevez 35 milliards d' euros , et bien , vous les retrouverez à l' autre bout . Et c' est pour ça que le même M. Juppé , il vous propose d' augmenter la TVA à 23,5 % , c' est intelligent , ça . Augmenter la TVA . Non seulement c' est un impôt injuste , mais vous allez payer pour le savoir , c' est que plus ça coûte cher , moins vous achetez . Vous êtes comme ça , vous êtes radins , vous . Sous prétexte que c' est trop cher , vous n' achetez pas . Alors que normalement , vous devriez acheter . Bon , ben , si vous n' achetez pas , qu' est -ce qui va se passer ? Ben l' activité va réduire . Et si l' activité réduit , ben , il y aura moins de TVA qui rentrera . Moins d' impôts , moins de travail . C' est le cycle que ces gens là amorcent , et ils vous racontent ça pendant des heures . Et tout le monde est assis en face et trouve ça très intelligent . Oui , oui , comme ça , nous allons pouvoir vendre davantage de je ne sais pas quoi à je ne sais quel Lapon perdu dans un coin , à qui on va venir proposer notre marchandise qui coûte moins chère parce qu' on a extorqué les gens dans le pays d' origine . Tout ça est absurde . Ça ne marche pas . Ça ne marche pas . 100 milliards d' euros de coupes dans les services publics . Ah ouais , il l' a même répété , Sarkozy . 100.000 ... Je ne sais pas , c' est une espèce de compétition , on ne sait plus les chiffres , il y en a un qui veut en enlever 300.000 , l' autre 100.000 , bon . Tout ça est absurde . Ca veut dire que des tas de choses vont se désorganiser . Alors évidemment , il y a quand même des gens , tiens , des fois , il y a des journalistes qui disent : &quot; Mais vous croyez que c' est raisonnable ? Dans quoi vous les enlevez ? &quot; Alors , ils ne vont pas les enlever ... pas dans l' armée , pas dans la police , dans l' éducation ? Ah , non , non , pas dans l' éduc ... si , si , on peut quand même . Mais bon , pas trop . Alors , pas dans ci , pas dans là , alors où ? Vous vous moquez de nous . Dans l' éducation , comme d' habitude . Dans les soins aux personnes , comme d' habitude . C'est-à-dire tout ce que les pauvres gens et les gens normaux se payent par le service public . Le service public , c' est notre mutuelle . C' est ... nous finançons tous quelque chose qui nous sert à tous . Voilà ce que c' est , le service public . Donc , vous n' aurez plus votre service public , vous serez responsabilisés . Voilà , autrement dit , vous êtes des irresponsables . J' ai déjà vu ça pour les soins . Vous ne l' avez pas vu , celle-là ? Il faut responsabiliser les patients . Et bien , moi , je suis d'accord pour avoir une petite grippe en août . Pas de bol , ça tombe en même temps que tous les autres en novembre . C' est absurde . Tout ces mots sont fous . Ils ne parlent pas de la réalité humaine . Tout ça n' existe pas . Ils nous font un monde qui va être un enfer , c'est-à-dire ce qu' il était avant qu' il y ait , par exemple , la Sécurité Sociale , Ambroise Croizat et le reste . Alors , la seule chose qui profite de ça , écoutez , il y a une justice , il doit y avoir une justice , il faut absolument qu' il y ait une justice , sur quoi ils sont tous d'accord . Sur le chiffre des fonctionnaires , ils ne sont pas tous d'accord , l' un veut enlever 100.000 , l' autre 300.000 , sur les cadeaux à faire aux patrons , ils ne sont pas tout à fait d'accord , l' un veut leur donner 35 milliards , l' autre , seulement 17 , enfin bon . Sur la retraite non plus , l' un veut vous faire travailler jusqu' à 64 , l' autre jusqu' à 70 . Bon , vous comprenez , il y a débat . Et il faut enrichir le débat . Voilà . Mais il y a un point sur lequel ils sont tous d'accord . Il faut supprimer l' ISF . L' impôt sur la fortune . Parce que si vous comprenez qu' il y a une chose qui plombe l' économie française , mine le moral des producteurs , abat notre joie de vivre , c' est l' ISF . Ils sont tous d'accord . Vous applaudissez , vous ne comprenez rien à l' économie . Parce que je vous explique : si l' assiette du riche est bien pleine , tellement pleine qu' elle est trop pleine , alors il en coule . Et ça s' appelle le ruissellement . Je vous jure , c' est dans les manuels d' économie . Ça va couler jusqu' à la vôtre , d' assiette . Donc si vous permettez aux très gros d' être encore plus riches , ça va ruisseler un jour sur l' autre sur vous . Pourquoi vous ne le croyez pas ? C' est des gens intelligents qui vous le disent . Mince , ils ont fait des études , et pas vous . Allons , allons . Allons , ressaisissez -vous , je vous prie . L' ISF , vous savez quoi , ils mériteraient ... d'abord , moi , je me demande combien il y a des gens de droite , parce que , franchement , hein , ce n' est pas parce que quelqu'un est de droite qu' il paie l' ISF , forcément . Hein . Il y a des gens qui paient l' ISF qui sont de gauche , en plus , alors bon , c' est vous dire . Non , mais c' est possible , la conscience , ça dépend de plein de choses . Combien ça concerne de monde , ce sujet qui les passionne ? Qu' ils sont tous là à vouloir à tout prix le régler . A qui ils parlent , quand ils parlent comme ça ? Combien ils ont eu d' électeurs la dernière fois ? Un peu plus de 20 millions , non ? A qui ils parlent , à combien de gens ? 342.000 . Tout ce baratin , toute cette histoire , toutes ces postures , c' est pour 1,7 % des contribuables . 1,7 % . Et bien , voilà , ils mériteraient d' avoir comme score 1,7 % . Ceux qui ont l' ISF votent pour eux . Bon . Allez , il faut aller à la fin , là , hein . Vous savez , j' ai eu un courrier cette semaine , terrible . Mon adresse , c' est : agenda@jeanlucmelenchon . fr . Qu' est -ce que je reçois à mon adresse ? Une lettre : Mes propositions pour les entrepreneurs . Alain Juppé , Pour la France . Pardon ? &quot; Cher ami , lundi 17 octobre , j' étais au salon Planète PME . &quot; Ben , tant mieux . &quot; Le 29 septembre , je visitais , avec Xavier Niel , le chantier de la station F , et déjeunais avec des entrepreneurs . &quot; Tant mieux pour vous . J' espère que c' était bon , le manger . &quot; Un mois auparavant , je m' exprimais aux universités d' été du MEDEF . &quot; Là , je suis en train de vous lire du Juppé , ce n' est pas tous les jours que ça arrive , dans un meeting comme ça , hein . Donc , d'abord , il me raconte sa vie . OK . Avec qui il mange , où il était , qu' est -ce qu' il fait . Bon , très bien , mais je ... Pourquoi je recevrais une carte postale de M. Juppé ? Alors , après , il me dit que je peux lire son livre , etc . Après , il me dit : &quot; Pour revenir au plein emploi , il faut avant tout relancer l' investissement en supprimant l' ISF . &quot; Ce n' est pas beau , ça ? &quot; En baissant l' impôt sur les sociétés à 24 % . &quot; 24 % . Je l' ai entendu dire à la télé , c' est confiscatoire . J' étais jeune parlementaire , l' impôt sur les sociétés , c' était 45 % . D'accord ? Et on réduisait l' impôt sur les sociétés . Maintenant , c' est à 24 % , on est pas loin du taux irlandais , 12 % . &quot; Pour les PME et 30 % pour les grandes entreprises . &quot; Alors , là , les grandes entreprises , elles vont hurler , hein . Parce que c' est déjà à plus de 30 % , mais eux , ils ne payent que 8 % , tellement ils trafiquent . Bon , je vous passe le reste , parce que ça devient saoulant , à force . Bon , vous avez compris le truc , l' ISF , baisser les charges , blabla , blabla , tout ça . &quot; Il est aussi nécessaire de supprimer les freins à l' embauche . &quot; Le frein à l' embauche . &quot; Je veux revenir une fois pour toutes sur les 35h , et baisser les charges qui pèsent sur les entreprises . Je propose aussi de supprimer toutes les charges patronales pour les salaires entre 1 et 1,8 SMIC , et d' alléger de 2 milliards d' euros les charges pour les indépendants . &quot; Allez , 5 minutes , je fais l' indépendant . Où il parle du RSI ? Le RSI , c' est la Sécu des indépendants . Comme arnaque , il n' y a pas au-dessus . Non , mais , sans blague . Il n' y a pas au-dessus . Donc , moi , j' ai quelque chose à dire aux indépendants , que je vous propose , qui est dans le programme : si c' est moi le Président , tous ceux qui relèvent du RSI pourront revenir dans le régime général de la Sécurité Sociale . Voilà . Bon . Allez . Maintenant , il faut passer aux choses sérieuses , tout ça , c' était la mise en bouche . &quot; Tout cela , nous n' avons pas osé le faire entre 2007 et 2012 . &quot; Non , ce n' est pas qu' ils n' ont pas osé , c' est qu' on ne les a pas laissés le faire . Ce n' est pas pareil , parce que M. Juppé , il est quand même le vainqueur du Juppéthon , vous vous en rappelez ? Vous n' avez pas oublié , j' espère . Tous les jours , on comptait combien il y avait de monde dans la rue . Donc ce n' est pas qu' ils n' ont pas osé , c' est qu' ils ne l' ont pas fait , et c' est lui qui s' est dégonflé , et oui , parce qu' on était plus nombreux . &quot; Il est temps de passer à l' action : dans moins d' un mois , les 20 et 27 novembre , se tiendra la primaire de la droite et du centre , ouverte à tous . &quot; Mais je m' en fous . &quot; Vous pouvez dès à présent localiser l' emplacement de votre bureau de vote grâce au lien suivant : http : //www . alainjuppe2017/ouvoter/trouvervotrebureau . J' ai besoin de vous pour cette échéance majeure pour le redressement du pays . &quot; Mais c' est une blague ? C' est à moi qu' il envoie ça ? Vous me voyez ? Non , mais je vous dis ça parce que j' ai entendu dire qu' il y avait des ballots qui se faisaient des nœuds au cerveau , en se demandant s' ils n' allaient pas , quand même , aller voter à la primaire de ... de la droite , pour que ça soit Juppé plutôt que machin , que Pierre , que Paul , que truc , parce que de toute façon , comme ils pensent d' avance qu' ils ont perdu , et que de se battre , ça ne sert à rien et qu' essayer de convaincre les gens , ça ne sert à rien , ce que , moi , je pense le contraire et que je vais y arriver , bon . Donc ils vont aller faire ça . Vous vous rendez compte ? Vous vous imaginez , si vous avez une conscience , oh , il doit y en avoir des gens de gauche , quand même , dans cette salle . Supposez que vous soyez de gauche . Et que ça fait un moment , en général . Alors , vous vous voyez ? Vous rentrez , vous avez la honte . D'ailleurs , je l' ai lu dans un article du Monde , que , il y a des gens qui : &quot; Ouais , mais moi , je ne peux pas , parce que dans mon village , tout le monde me connait . &quot; Ben , oui , quoi . La honte . Vous rentrez en rasant les murs . Déjà , vous donnez deux euros aux Républicains . Et vous en êtes là de la honte , et là , il vous dit : &quot; Attendez , il faut signer là . &quot; Voilà ce que vous êtes censés signer , les malins , là : Je partage les valeurs républicaines de la droite et du centre et je m' engage pour l' alternance , afin de réussir le redressement de la France . Et bien , quoi qu' il arrive , je peux vous dire que , je pense que ce sera le cas de tout le monde dans la salle , parce que le premier qui était tenté par autre chose , maintenant , il a tellement honte qu' il n' y pense même plus , il dit : &quot; Moi , je n' y ai pas pensé . &quot; Le minimum , mes amis , on a tellement dénoncé le cynisme , à juste titre , des dirigeants politiques ; ça serait bien que les électeurs ne deviennent pas aussi cyniques que leurs dirigeants . Ça serait bien qu' on se respecte . On se respecte soi-même assez pour ne pas aller signer un papier qui dit le contraire de ce à quoi on croit . Et que l' on respecte assez ses compatriotes , des gens qui ne sont pas de votre avis , qui doivent être respectés . Parce qu' il n' y a pas de démocratie sans points de vue contraires , et on a besoin des points de vue contraires pour faire avancer la discussion . Donc il faut respecter ceux qui ne sont pas de notre avis . Et nous ne devons pas aller truquer leurs élections , en désignant , nous , à leur place qui va les représenter . Et peu nous chaut qui ils vont décider . Parce que des gens comme nous ne voteront jamais pour eux . En aucune circonstance . Voilà la vérité . Je vous appelle à ne pas truquer l' élection des Républicains qui ont le droit de voter comme ils l' entendent et de choisir les leaders qu' ils veulent . Parce que c' est à cette condition qu' on vit en démocratie . Et après , moi , j' ai une autre question à vous poser . On me dit : &quot; C' est un listing . &quot; Ah ouais , c' est un listing , vous croyez que ça me rassure ? Comment ça se fait que mon adresse est dans le listing de M. Juppé ? Non , mais sérieusement . Il y en a peut-être d' autres , ici , qui ont reçu une lettre de ce type . Comment se fait -il qu' il aie mon adresse dans son listing ? Est -ce que quelqu'un peut me répondre ? M. Juppé , vous pouvez me répondre ? Vous êtes tellement aux abois que vous avez besoin d' un gars comme moi pour faire la différence ? Non , non . Pour battre M. Sarkozy , je compte sur les urnes , celles auxquelles va tout le monde . Et puis après , je vais vous en dire encore une autre . Non seulement je ne sais pas pourquoi je suis sur son listing , deuxièmement , je n' ai pas envie de l' aider , troisièmement , je n' ai pas envie de signer ce papelard , mais j' aimerais bien savoir pourquoi je ferais confiance à quelqu'un qui envoie des lettres à des gens , et il ne sait même pas à qui . Ce n' est pas un grave problème ? Et après , le même vient vous dire qu' il vous regarde au fond des yeux et qu' il sait de quoi vous avez besoin . Supprimer l' ISF . Bon , voilà . Je vous ai dit tout ce que j' ai à vous dire . Je n' irai pas non plus dans la primaire du Parti Socialiste . Vous savez pourquoi . Par loyauté . Par loyauté . Je ne participe qu' aux élections pour lesquelles je suis prêt à me soumettre au résultat . Je participe au vote de ma patrie , et je me soumets au résultat du vote . Quand c' est un gouvernement de droite qui fait voter des lois de droite , je me soumets aux lois qui sont votées parce que c' est les lois de mon pays . Même si je continue à protester contre elles , même si j' agis avec mes camarades dans la rue pour ne pas qu' elles s' appliquent , mais je les respecte . Je n' irai pas dans une élection dont je ne suis pas prêt à respecter le résultat , parce qu' aucune des personnes que je vois ne correspond au degré de sincérité , d' opiniâtreté , d' application auquel je crois que nous avons le droit , maintenant . Je ne veux plus faire de compromis . Je pense qu' à chaque fois , on s' abîme , bien sûr que des compromis , on en fait dans la vie sans arrêt . Et tous ceux qui sont au travail , comme je le dis souvent aux camarades , je leur dis : &quot; Chaque jour que vous vous levez , que vous allez à l' entreprise , vous faites un compromis avec le capital . &quot; Donc , ce n' est pas la question . La question , c' est de renoncer à soi . Je vois des gens , il y a quelques jours , on m' a dit : &quot; Ah , mais tu pourrais t' accorder avec Montebourg . &quot; Mais pourquoi ? Pourquoi je m' accorderais avec lui ? &quot; Parce qu' il pense presque comme vous . &quot; Ben , dans ce cas là , je préfère moi . ( Rires et applaudissements ) Parce que c' est quoi le &quot; presque &quot; ? C' est quoi le presque ? Le presque , c' est qu' il a dit qu' il voulait vendre des logements sociaux , les vendre aux gens qui sont dedans . Je ne vais pas vous faire à cette heure -ci la démonstration de l' idiotie que c' est . Qui est -ce qui a inventé ça ? Thatcher . Pourquoi ? Parce que , ben , qu' est -ce qu' on vend ? Les bons appartements . Puis les autres , ils restent sur les bras de l' office . C' est lui qui doit les entretenir , puis les gens qui sont assez ballots pour avoir acheté l' appartement , après , ils découvrent qu' il faut les entretenir , ils n' ont pas les sous pour ça , et puis si vous venez en disant ... Vous ne croyez pas qu' on vend un appartement oui , un appartement non . Hein ? C' est : on vend tout . Et vous avez celui qui est là ou celle qui est là qui a passé les 60 ans , qui a eu sa vie et qui était là , dans un logement social , on lui dit : &quot; Il faut le racheter . &quot; &quot; Mais , je n' ai pas les sous . &quot; &quot; Ben , vous n' avez qu' à aller voir la banque . &quot; La banque , bien sûr , dès qu' elle voit quelqu'un qui a 65 ans ou 66 ans , elle dit : &quot; Ah , oui , oui . combien vous voulez ? Pour combien d' années ? Allez , tiens , pouf , on vous la met pour 25 ans . &quot; Tout ça n' existe pas . La vie des gens n' est pas faite comme ça . Ben , c' est une trouvaille stupide . Et de la même , on lui dit : &quot; Vous voulez ... il faut augmenter les salaires ? &quot; Oui , moi je dis : &quot; Il faut augmenter les salaires . &quot; Je reprends les slogans d' un certain nombre d' organisations syndicales . Dont l' augmentation du SMIC , et donc de tout le reste qui va avec . Mais lui , il dit : &quot; J' augmenterai les salaires après avoir proposé une négociation en Europe . &quot; Ben , fous -toi de nous . Pourquoi continuer à parler comme ça ? Dis : &quot; Je ne peux pas , je ne veux pas , ça n' est pas possible . &quot; Dis -le franchement . Dis -le ouvertement aux gens . Et les gens sont capables de comprendre si tu as de bons arguments pour le dire . Pourquoi tu leur mens ? Pourquoi tu dis oui , pour , dans le reste de la phrase , dire non ? C' est de tout ça dont on meurt . Assumez ce que vous voulez faire . Dites comment vous allez le faire . Réfléchissez -y . De même , vous voulez changer les institutions , vous voulez en finir avec la monarchie présidentielle , et avec tout ce qui va avec , à qui voulez -vous confier ça ? Une nouvelle fois , trois personnes qui écrivent ça dans un bureau et qui vous le proposent ensuite par référendum ? Ou bien à une assemblée ? Est -ce que ce n' est pas le peuple lui-même qui peut changer les conditions dans lesquelles on doit faire la politique dans le pays ? Voilà . Allez . Voilà pourquoi il ne faut pas avoir peur , il ne faut pas ... il faut y aller . Il faut convaincre . Chacun de vous , qui partagez mon opinion . Alors quand vous applaudissez , je suppose que vous êtes d'accord . Ce n' est pas tout de m' applaudir , il faut faire . Il faut faire . Il faut prendre sa petite liste , marquer ses 10 noms et puis faire le travail . Ce n' est pas compliqué , vous savez . On a fait 4 millions de voix , la dernière fois . Un pour un . On en fait 8 , on a gagné . Ben oui . On est au deuxième tour et le piège se referme sur eux . Ce n' est pas si compliqué que ça . Il faut donc convaincre . On est en démocratie , on a cette chance . Et ben , on doit convaincre . Pas hypnotiser , subjuguer , abrutir , tromper , tricher , à quoi bon ? A quoi bon ? Convaincre . Un après l' autre . Une après l' autre . Voilà . Et puis ça fait une majorité . Ou ça fait en tout cas le nombre suffisant pour ensuite rassembler une majorité . Moi , je pense qu' on peut y arriver . Et puis , je ne vois pas ce qu' on pourrait faire d' autre . Ah , il y a des gens , bon , ils veulent se rassurer , ben , ils se rassurent , hein . Mais ils ne vont pas se rassurer longtemps . Parce que si c' est pour aller voter dans les primaires de droite parce que Juppé , je ne sais pas quoi , au deuxième tour , machin , etc . : retraite à 70 ans , bonhomme . C' est pour ça que tu as fait tout ça ? Toutes ces combines , ces arrangements avec toi-même , avec ta conscience , aller signer des papiers auxquels tu ne crois pas , et donner de l' argent à des gens que tu n' aimes pas . Tout ça pour ça ? Pour vous renier ? Pour tout abandonner alors qu' on peut faire tant de choses magnifiques , belles , immenses , j' espère que tout à l' heure , je vous en ai convaincu . De toutes ces choses qu' on va pouvoir faire dans ce pays , qui ne demandent que de la volonté . Ça ne manque ni d' argent , ni de savoir-faire . Il faut qu' il y ait des gens qui arrêtent d' avoir peur . De tout , de rien . S' ils veulent être les bons élèves de la mondialisation . Du courage . De la volonté . Oh , il n' y en a pas besoin de tant que ça , hein . Bon , maintenant , c' est fini . Alors , je lis une petite citation , toujours . D' habitude , je vous lis des poèmes , mais pas là . Comme c' était sur un moment un peu tendu , là , sur les primaires de la droite . Je vous lis juste une chose que j' ai lue , qui m' a bien plu , ça parle aussi de moi , alors c' est pour ça , peut-être . Tandis que trouver quelque chose qui parle de moi dans Victor Hugo , je peux toujours chercher , mais bon . C' est Laurent Binet qui est un écrivain . De notre pays . Il dit , après le vote des Etats-Unis , hein . Il dit ça : &quot; L' histoire des luttes politiques en résumé , deux-points : on espère avec Bernie Sanders , on se résigne avec Clinton &quot; , suivez mon regard , &quot; et à la fin , on a Trump . Ça marche bien aussi avec Mélenchon , Hollande et Le Pen . &quot; C' est un grand auteur . Bon . Et comme je suis chez vous , et pour finir pour de bon , juste 4 lignes d' Ambroise Croizat - moi , je ne suis pas communiste , hein , mais je pense qu' il y a des communistes dans la salle , ça va leur faire plaisir , et puis comme Ambroise Croizat est un grand Français , et un grand militant , et bien ça nous fera plaisir à tous . Voilà . Parce que c' est homme là , il appartient à tout le monde , comme toutes les belles pages de l' histoire sociale de la patrie . Alors , il dit ça : &quot; La lutte continue pour la renaissance française . &quot; C' est un communiste , hein , qui parle . &quot; La lutte continue pour la renaissance française . L' indépendance nationale et l' instauration d' un bien-être pour tous - l' indépendance nationale et l' instauration d' un bien-être pour tous sont indissociables . &quot; Vive la Sociale , mes amis . ",
    "Nous n' avons pas de sympathie pour Poutine , et ni Jean-Luc Mélenchon ni la France insoumise n' en ont la moindre . Nous partons d' une analyse géopolitique globale et nous constatons que les Etats-Unis et leurs alliés veulent affaiblir , sinon éliminer , toutes les puissances économiques ou militaires qui menacent leur hégémonie . Ce travail s' effectue aussi par la courroie de transmission que sont les médias atlantistes , qui servent à préparer l' opinion et justifier des guerres , avec des éléments de désinformation souvent flagrants . Ce qui est reproché à Jean-Luc Mélenchon est d'abord de déconstruire le discours de ces médias partisans . Et sa ligne directrice est d' être vigilant pour dire que , en dépit d' éventuels massacres ou faits de guerre , cela ne doit pas justifier des interventions étrangères , c'est-à-dire que nous défendons un principe de non-ingérence . On l' a vu en Libye où , bien que présentés comme une intervention humanitaire , les bombardements de notre gouvernement associés à d' autres ont entraîné un chaos total . Or , en tant que citoyens français , nous avons une responsabilité devant le monde entier lorsque la France intervient militairement [ ou soutient des agressions meurtrières comme actuellement contre les peuples palestinien ou le peuple yéménite . Comme l' explique Noam Chomsky , nous avons un devoir prioritaire à dénoncer les ingérences commises par nos gouvernements , à faire pression pour que cessent les massacres dont nous sommes responsables . ] Dans ce cas précis , il y a deux lignes d' analyse diamétralement opposées . L' une est de dire qu' il y a une intervention de la Russie sans en préciser le contexte ; l' autre est de souligner qu' il y a des attentats terroristes , une intervention djihadiste étrangère , en Syrie et que la population a peur . Ce n' était pas le cas au début du soulèvement en 2011 , mais aujourd'hui , une grande partie de la population syrienne a peur du Front Al-Nosra , du Front islamique et de Daech . [ L' intervention russe et le maintien du régime d' Al-Assad se donnent comme justification de faire rempart à ces bandes sanguinaires , qui sont indirectement soutenues par l' Occident , puisque celui -ci arme le Qatar et l' Arabie saoudite , pays qui font ensuite provenir ces armes aux groupes salafistes . ] Nous n' avons aucune sympathie pour le régime syrien , mais la priorité devrait être d'abord d' éliminer ces groupes qui terrorisent la population syrienne et donnent des arguments à Assad . Et la vraie question à poser est celle de la responsabilité de la France et de nos alliés dans cette ingérence . Jean-Luc Mélenchon dénonce la logique de guerre contre Moscou qui s' exprime à travers les grands médias . Ceux -ci instrumentalisent tous les problèmes et tout ce qui se passe en Russie afin de déstabiliser ce pays . Tout en s' appuyant sur des éléments exacts , la propagande de l' OTAN les détourne pour ses propres intérêts impérialistes contre la Russie . Il faut traiter d' égal à égal avec Poutine , qui n' est pas un interlocuteur moindre que Merkel ou Donald Trump ! Il faut quand même se poser la question de comment se voit la Russie elle-même face au reste du monde . C' est un grand pays , mais qui est entouré de bases de l' OTAN , partout en Europe centrale et orientale , et qui fait en sorte de résister aux ingérences et aux provocations étrangères à ses portes . Quant à la Syrie , ce que nous savons , c' est que la Russie a proposé dès 2012 , dans le cadre des accords de Genève , une solution négociée dont il est généralement admis aujourd'hui qu' elle était sérieuse et réaliste . Il s' agissait d' un processus de transition qui avait même été accepté par une partie de l' opposition . Et c' était pour la plupart des observateurs une bonne base de départ , pour bien sûr poursuivre les discussions . Qu' en est -il trois ans plus tard , en 2015 ? Les accords ont été refusés et la situation sur le terrain a vu les djihadistes aux portes du pouvoir . C' est à ce moment-là que la Russie est intervenue ! Et c' est ce qui l' a empêché de s' emparer du pouvoir ou du moins de les repousser d' une grande partie du territoire syrien . Il est difficile de savoir avec précision ce qu' il en est , surtout avec des djihadistes comme le Front Al Nosra qui dominent ceux que la presse occidentale appelle &laquo; les rebelles &raquo; . Mais une partie de la population syrienne , s' est rangée du côté du régime de Bachar Al-Assad car il est devenu un rempart pour elle , contre l' avancée et les exactions de ces groupes djihadistes . C' est malheureusement cela la situation aujourd'hui . Et c' est bien à partir de celle -ci qu' il faut essayer de lancer un processus de négociations comme dans tout conflit armé . La position de Jean-Luc Mélenchon depuis 2012 n' a pas varié sur ce point : cessez-le-feu , arrêt du bain de sang et négociations entre le régime et l' opposition . Quant à la nature non-démocratique du régime russe , nous insistons sur le fait que si il s' agissait d' une dictature comme les autres mais alignée dans le camp de l' OTAN , nos dirigeants seraient beaucoup plus complaisants à son égard . C' est donc en fait sa politique de non-alignement qui entraîne les nombreuses attaques faites à son encontre . ",
    "Salut les p'tits ! Ça marche ? Est -ce que ça marche , là ? Est -ce qu' on m' entend ? Bon ... Là , faut arrêter la musique ... Merci . Bon ... A vous tous , euh ... Merci d' être venus ... d' être si nombreux ... Et je vais vous dire d'abord pour commencer la même chose ... que ce que je viens de dire à tous ceux qui , hélas ... n' ont pas pu entrer ... Et qui sont dehors ; je sais pas combien y en a ... peut-être autant que dedans ... Bon , ils resteront moins longtemps , vu le froid ... J' ai été les saluer ... je leur ai dit ... que je leur présentais des excuses ... après les remerciements que je dois à tous ceux qui se sont donnés le mal ... d' organiser cette soirée ... des militants , des bénévoles ... Et attention ... C' est pas parce que ce sont ... des bénévoles ... que ce sont des amateurs ... car , voyez -vous ... Nous comptons dans nos files ... les meilleurs professionnels ... de leurs branches ... Forcément , c' est nous le peuple ... Et y a pas besoin de payer , mais quand même ... Mais quand même ... Si nous ne pouvons pas accueillir comme nous le voudrions. . Tout le monde ce soir ... je vous dis les choses , crûment et franchement ... C' est qu' il n' y a pas assez d' argent ... parce que pour avoir une salle pour faire rentrer 3000 personnes ... Ça coûte 50.000 euros ... Une soirée comme celle -ci au total ... C' est 12.000 euros ... Je veux que vous le sachiez ... Rien n' arrive comme ça ... Tout est fruit d' efforts ... Et de mobilisation ... Plus de 20.000 personnes ... Déjà , ont donné chacun leur petit écot ... Et c' est notre force qui , pour terminer , repose tout entière sur le nombre ... Et le dévouement , que chacun d' entre nous avec son intelligence ; parfois , avec ses très faibles moyens mobilise au service de la cause ... Nous ne sommes pas en train de célébrer une personne , un candidat ... nous sommes en train de célébrer des idées ... et nous sommes en train de faire une démonstration de force … de nos idées … Et ceux qui sont là dehors à se geler ... de toute façon ils ont marqué un point ... comme vous-mêmes ... car nous n' étions pas là pour autre chose que pour montrer ... la continuité , la force , la profondeur de ce que nous représentons dans le temps et dans l' histoire ... quand ils pensaient en avoir fini avec nous ... pendant toutes ces années ... où l' argent régnait sans contestation ... et où on proposait à chacun ... de n' avoir aucun autre horizon dans la vie ... que toujours d' accumuler davantage ... fusse au détriment des autres ... jusqu' à finir dans ce rêve piétiné ... nos paroles salies et souillées ... quand un de ceux qui se réclame de nous ose dire à la jeunesse ... je vous propose comme rêve d' être milliardaire ... Être milliardaire est immoral ... Accumuler est immoral ... La richesse est toujours accumulée au détriment ... des autres ... A quoi bon ... ... avoir un milliard , si c' est pour souhaiter en avoir un deuxième ? A quoi bon cette course permanente à l' ostentation ? Bon ... Un mot pour les ballots qu' ont été donner deux euros ... Deux fois de suite ... Non . Ne les huez pas ! On va leur proposer de se racheter ; c' est un langage que tout le monde comprend ... Vous avez donné deux fois de suite deux euros ... Il y en a pour un 1,6 millions que vous avez été donner à nos adversaires ... Alors , il faut en donner un peu ici ... Appel aux dons ... Vous avez donné deux fois deux euros ; vous m' en devez quatre ... Pour que quand viendra votre tour devant votre conscience ... Vous puissiez vous dire : bon ... Pourquoi est -ce que je suis venu à Bordeaux ? Eh ben , parce qu' il y a cinq ans , j' étais à Talence . Et qu' aujourd'hui ... Nous ne serions ni si nombreux ... Ni si déterminés ... Ni si nouveaux , pour certains d' entre vous , compte tenu de votre âge ... Si auparavant ... Nous n' avions déjà accumulé , rassuré cette force ... En lui montrant qu' elle existait ... Comme nous avons commencé à le faire ... En 2012 , en nous rassemblant à quatre millions ... Avec le même bulletin de vote ... Mais , je pourrais aussi être venu pour ... S' il vous plaît ... Allez ! Voilà ! Honneur au vaincu ... J' aurais pu venir en me disant : &quot; Eh bien , je viens dans la ville ... de celui qui est le président ou va être le président de la République ... &quot; Enfin , déjà vous autres , les ballots ; vous avez donné des sous pour acheter tout ça ... A -t-il gagné déjà ? Eh bien , vous avez payé pour lire ça ... Moi , je plains ... Je le dis ... sans moquerie ... Ceux qui se sont laissés affoler ... Et qui continuent maintenant ... A croire aux même fadaises ... qui ont été un peu renouvelées ... Je les plains ... Comment vous avez pu vous fourrer dans le tête ... Qu' étant ... de gauche ... Ou en tout cas , des progressistes ... Vous aviez partie liée ... Avec le vote ... De ceux de droite , qui ont bien le droit d' organiser des votes ... pour désigner leur leader ... Leur candidat ; c' est leur droit ... nous devons le respecter ... Nous n' avons pas à aller tricher dans les élections des autres ... Nous n' avions pas à y être ... Eh ben , ils y sont quand même allés . Et pourquoi ? L' intention était bonne ; j' en suis sûr ... Pas très réfléchie , mais bonne ... Alors ... Ils allaient voter pour lui pour empêcher que l' autre ... qui ensuite , verrait le premier ... Qui ... passerait ... Derrière l' autre et ... l' affaire serait dite ... Qu' est -ce qu' ils sont intelligents ! Ça , c' est la base du vote utile ... La culture du vote utile ... Le vote utile , c' est toujours un vote futile ... Parce que le seul vote qui est important pour une citoyenne ou un citoyen ... c' est celui qui fait le bien de tous ... Dont vous concluez en conscience ... que c' est le bien de tous ... C'est-à-dire , le vote nécessaire ... Et si je me présente devant vous ... Ce n' est pas en vous disant , je suis le vote utile , parce qu' aujourd'hui les sondages ... me mettent en tête de ma famille politique ... Je viens devant vous comme vous le savez ... et comme beaucoup d' entre vous en sont déjà d'accord ... parce que j' estime que c' est le vote nécessaire ... celui de l' avenir en commun . Nous autres ... Nous sommes comme nous sommes ... pour plein de raisons ... Et pas seulement ... A cause de ce que nous dit notre feuille de paie ... La vôtre ... Moi , la mienne , ça va ... Vous le savez bien ... Mais justement ... Je m' adresse aussi à ceux qui sont dans ma situation matérielle confortable ... Ceux qui ont une bonne situation ... doivent courir devant pour aller aider les autres ... parce que c' est eux qui ont les moyens ... et le temps qui les met à la disposition des autres ... Nous sommes comme nous sommes pour bien des raisons ... Et d'abord , souvent , pour des raisons morales ... Ce qu' on appelle des raisons morales ... Nous ne supportons pas la souffrance des autres ... Nous ne supportons pas l' abandon ... Ni celle des être humains , ni celle des animaux ... Ni le massacre de la nature ... Toutes ces choses heurtent en nous quelque chose de très profond ... Alors , il faut être comme la pointe du diamant ... Ne jamais en rabattre ... ni de ses rêves ... ni de ses objectifs ... Ceux qui s' emmêlent les pieds ... Dans les tristes et pauvres aventures du vote utile ... Comme le moindre mal ... Disent -ils , pensent -ils ... souvent honnêtement ... bravement ... Le moindre mal ... Ne l' oubliez jamais ... C' est quand même toujours le mal ... Hannah Arendt ... Je vous ai amené cette citation ... Dans son livre qui s' appelle &quot; Responsabilité et jugement &quot; , dit ... &quot; Politiquement ... la faiblesse de l' argument du moindre mal ... a toujours été que ceux qui choisissent le moindre mal ... oublient , très vite ... qu' ils ont choisi le mal ... &quot; Alors ... Bon ... Si je ne viens pas pour Juppé , je pourrais venir pour Montesquieu ... Il est de votre coin ... Je m' intéresse toujours à qui est -ce qui est du coin ... Tout à l' heure , on aura mieux que Montesquieu , qui était quand même un gros réac , mais enfin ... En 1772 ... La lumière de la Grande Révolution n' était pas encore passée ... Lui a théorisé la séparation des pouvoirs exécutifs et législatifs ... Alors voilà ce qu' il dit ... On va le proposer comme une pensée contemporaine ... &quot; Lorsque dans la République , le Peuple encore a la souveraine puissance ... c' est une démocratie ... Lorsque la souveraine puissance est entre les mains d' une partie du Peuple ... Cela s' appelle , une aristocratie ... &quot; Eh ben voilà ; c' est exactement ce dans quoi nous sommes ... Nous sommes sous le pouvoir ... d' une aristocratie ... d' Ancien Régime , qu' aujourd'hui nous appelons l' oligarchie ... Oligarchie ; c' est le gouvernement de quelques-uns ... Ils sont quelques-uns qui possèdent tout ... et qui tirent les ficelles ... de politiciens , de politiciennes et de médiacrates de toutes sortes ... J' ai dit du mal de Montesquieu ... Bon . Au moins , il proposait ça , et c' était un esprit avancé ... qui éclairait la réflexion de son époque ... Il croyait qu' on pouvait abolir l' absolutisme ... sans abattre la monarchie ... C' est comme ceux qui croient ... qu' on peut libérer les humains et la nature de la prédation ... sans abolir la tyrannie ... de la finance ... Non . Ce n' est pas possible ... On ne pouvait pas abolir l' absolutisme sans abattre la monarchie ... On ne pourra pas libérer les êtres humains et la nature sans abattre l' absolutisme de la finance ... C' est un raisonnement ... et un manifeste ... Je me présente devant vous , cent un ans après la loi de 1905 de séparation des églises et de l' Etat ... pour vous proposer la séparation du MEDEF et du gouvernement ... La chose extraordinaire ... C' est quand même ... Tout à l' heure , on a montré M. Juppé ... C' est qu' ils ont fini par trancher ... C' est bien ... Ça c' est pas extraordinaire ... Et ils ont désigné un candidat ... Qui est M. Fillon , à qui ( coupure vidéo ) Mais , c' est complétement immoral ... Donc , on va dire une chose . Voilà un candidat ... qui est surtout et d'abord ... entouré d' une bande de menteurs ... Ce n' est pas rassurant ... Maintenant ... J' en viens au fond ... Parce que c' est de cela dont il va être question ce soir ... et des arguments ... qu' il faut que nous portions en nous , qui doivent essayer d' éclairer ... notre regard ... sur la situation que nous vivons ... C' est une bonne chose ... que la droite ait choisi M. Fillon . Parce que c' est un cas ... politiquement clair ... Il propose une synthèse qui était contenue ... Peu visible ... Peu manifestée , de ce qu' est vraiment la droite dans notre pays . C'est-à-dire , deux choses ... Le libéralisme économique le plus absolu ... et le conservatisme ... intellectuel et moral , le plus total ... Et il en est une bonne synthèse ... C' est pourquoi , pour ma part ... Je me réjouis ... du fait que nous allons pouvoir ... confronter nos idées ... car nous sommes l' exact inverse de cela ... Mais pour pouvoir avoir ces débats , il faut des gens qui assument ... leurs positions . Et celui-là ... m' a l' air d' être prêt à les assumer ... sans faux semblants ... sans fausse casquette , sans faux nez ... sans masque , sans déguisement ... Et c' est pourquoi ... je serais heureux , s' il acceptait ... le débat que je lui propose. . Devant tout le monde ... pour qu' on entende , au moins un soir ... le choc ... naturellement ... courtois , c' est comme ça qu' on le ferait ... des deux visions du monde ... Parce que ce qui est en cause ... ce sont deux visions du monde ... Au fond ... Ce sont les deux cohérences que nous incarnons ... Lui et moi , en votre nom ... Quelle cohérence ? D' un côté ... suivant la vieille maxime ... le nôtre ... l' idée qu' il y a un avenir en commun ... Et que c' est ce commun ... qui permet cet avenir ... pour l' homme ; l' être humain , on va dire ... et son rapport à la planète ... En gros , s' il fallait prendre une petite formule , on dirait ... C' est , euh ... Un pour tous , tous pour un ... Ce n' est pas que nous soyons contre la compétition ... Bien sûr que non ... Il y a plein de compétitions dans la vie ... des compétitions amoureuses , des compétitions de toutes sortes ... Mais ... nous sommes pour la compétition , à condition qu' elle n' empêche pas la solidarité ... Et eux , de leur côté ... c' est l' idée inverse ... Ils veulent bien de la solidarité ; ce ne sont pas tous des sauvages ... Il y en a un nombre de sauvages ... Mais ... Ils sont comme nous ... Ils veulent bien ... la solidarité mais à condition que ça n' empêche pas la compétition ... Au fond , leur maxime à eux et en particulier celle de M. Fillon , c' est ... Chacun pour soi et ... Dieu pour tous ... Ce qui va nous ... Ce qui va nous mettre en mouvement nous , ce qui va organiser la cohérence du programme &quot; L' avenir en commun &quot; que je vous demande de lire , c' est une vision d' ensemble . Là , je viens de vous en livrer le principe moral mais après , il y a le mode d' exécution Mes amis , nous , nous sommes conscients du fait qu' on règle les problèmes à partir des problèmes , pas à partir de seulement l' idée qu' on s' en fait . Le problème urgent et immédiat ... et celui qui va retentir comme un défi qui va nous permettre d' avancer , ... c' est que la maison brûle , comme disait Chirac . Franchement , quand 60 % ... 50 % des espèces vertébrées ont disparu de la planète , quand la chaleur est montée de 20 points au-dessus du seuil auquel ... il est normal qu' elle se trouve dans l' Antarctique , Est -ce que vous croyez vraiment , qu' à ce moment-là , la question qui se pose , c' est celle de l' ISF ? Ne croyez pas ... que je caricature ! Donc , nous , nous sommes toujours partis de la demande , la politique ... du clan progressiste , ça a toujours été de partir ... de la demande , c'est-à-dire , qu' est -ce qui est nécessaire ? Que faut -il faire ? Et bien , là , nous avons un problème , nous devons nous mobiliser tous ... pour répondre au défi qu' a créé la prédation excessive sur la nature , mais au moment de le faire , quand on dit à tout le monde : &quot; Allez , on va faire l' effort ! &quot; Qu' est -ce qu' on découvre ? Que la société est pourrie d' inégalités , qu' il y a des masses de pauvres , des milliers de gens qui sont hors-circuit , qui n' ont qu' une chose à faire chaque jour , c' est de se demander comment ils vont tenir jusqu' au lendemain , et le jour d'après , pour eux et leurs enfants . L ' inégalité ronge l' unité de l' espèce humaine , et sa capacité à être ensemble pour faire face à cet énorme défi qui avance . Les deux choses se tiennent , le programme politique que nous présentons répond aux deux questions , pour faire face au défi du changement climatique , il faut changer complètement le mode de production , le process de production , et notre manière de consommer ainsi que la répartition des richesses , et pour le faire , il faut mobiliser des cohortes immenses de gens . Parce que c' est un travail gigantesque de faire tout ça , donc nous avons besoin de milliers d' emplois , et nous avons besoin de ces emplois pour faire cette bifurcation et pour récupérer tous ceux des notres qui ont été abandonnés en route , ceux qu' on a pas soignés , ceux qu' on a pas éduqués et qu' il nous reste 2 millions et demi de personnes illettrées dans le pays . Ceux qui sont malheureux , cassés , détruits par la violence d' un système social qui les a abattus , parce qu' on leur a dit : &quot; chacun pour soi , et si vous êtes pauvres , c' est de votre faute ! &quot; Voilà cette morale pourrie , qui a empêché qu' on fasse , la mobilisation humaine dont on a besoin . Quand quelqu'un vous parle de la sixième extinction des espèces , dites -vous que la réponse , elle est sociale . Elle commence par dire que pour être tous à la manœuvre , il faut que chacun puisse vivre dignement de son travail , et que son travail ait une utilité qui permette à tout le monde de faire face aux défis . Alors , évidemment , de l' autre côté , il y a une autre vision du monde . On la connaît : &quot; Remplissez l' assiette des riches ! Et alors , il y en aura tellement qu' il en dégoulinera quelque chose . &quot; Ça s' appelle la théorie du ruissellement . Ils vont donc ruisseler ... mais nous n' avons pas vu ... que cela ruisselât beaucoup ... jusqu' à présent . Et ça s' appelle la politique de l' offre . Je suis obligé de résumer parce que tout le monde n' a pas idée bien claire de ce que c' est . La politique de la demande , je viens de vous l' expliquer : de quoi a -t-on besoin ? Alors , de quoi a -t-on besoin ? On répond à la question et on s' en donne les moyens . La politique de l' offre , c' est : &quot; Produisez n' importe quoi , n' importe comment , dans n' importe quelles conditions , pourvu que ça ne soit pas cher , et vous trouverez des gens pour vous l' acheter . Et si vous l' achetez , alors vous devrez en produire davantage et ça fera le bonheur de tout le monde , gnagnagna , flexibilité , compétitivité , blablablbl et blablbl ... et l' emploi . &quot; Voilà , vous avez compris ? Résumé : quelqu'un frappe à la porte , toc toc toc , nous sommes au pôle Nord , vous êtes un Esquimau , vous frissonnez - enfin , plus tellement en ce moment , je peux vous faire la même chose dans le Sahara , hein - et la porte s' ouvre et il y a un type qui dit : &quot; Bonjour , je vends des frigos . &quot; L' autre dit : &quot; Ah ben , non , moi , c' est un chauffage . - Pas du tout , c' est pas la question , mon frigo n' est pas cher , achetez -le . &quot; Voilà , c' est ça , la politique de l' offre , c' est absurde . C' est absurde ! Ne vous laissez pas impressionner ! Ne vous laissez pas impressionner par ceux qui vous disent que tout ça a un sens : ça n' en a pas ! L' activité économique humaine était destinée , et elle l' est depuis que l' humanité est humanité , à satisfaire des besoins réellement exprimés . Et ceux-là satisfont des besoins qui n' existent pas , en retirent les moyens aux gens et créent les nouveaux besoins . Et c' est cette société dans laquelle nous vivons , du gaspillage . Où l' on détruit , où 40 % de la production agricole est mise à la poubelle , parce qu' elle n' est pas conforme . Et c' est ce monde là qu' ils veulent faire durer . Avec une absence totale du sens des responsabilités Ecoutez -moi , les amis , le plus grand scandale de ces débats qu' ont eu ... nos ... nos concitoyens de droite , c' est qu' ils n' aient pas parlé une seule fois des problèmes de la planète , pas une seule - un mot ! - pour parler d' écologie et de la manière d' y répondre . Et ils ont désigné quelqu'un , qui est un productiviste aveuglé . Ca commence vraiment très mal : il propose d' abroger le principe de précaution , qui a été inscrit dans la Constitution . Notez que c' est Chirac qui l' y a mis . Et ben , on le remercie . Et la première fois qu' on a mis ça dans la loi , c' était M. Barnier qui l' avait proposé , juste après le premier sommet de la Terre , à Rio en 1992 . Je suis tellement vieux que j' y étais . Donc je m' en souviens . Ce principe dit qu' on ne fait pas quelque chose ... si il y a un risque que cela dégrade , ou que cela présente un ... un inconvénient ensuite , qui serait irréversible ou très dommageable . Est -ce qu' il y a aujourd'hui une abondance de recours , qui fait qu' à cause des principes de précaution , tout est bloqué ? Jamais de la vie , il n' y en a pas eu un seul ! Alors , pourquoi ils le retirent ? Pare pure idéologie ! Parce qu' ils pensent que le problème ne se pose pas , parce qu' il est aveuglé et qu' il ne veut qu' il n' y ait aucun obstacle à la libre circulation du capital , à ses libres investissements , quoi qu' il produise , quoi qu' il fasse , alors même que nous savons aujourd'hui qu' à cause des glyphosates , et combien d' autres pesticides , la vie humaine est lourdement perturbée . La fertilité , la reproduction des êtres humains sont lourdement mis en cause Pour ne pas parler de tout ce que ... cela entraîne aussi sur la santé des nouvelles générations . Le même homme propose de prolonger les centrales nucléaires de 40 à 60 ans . Ecoutez , franchement , après Fukushima ... Non , quoi ... Non ! Il faut y réfléchir avec sérieux . Moi , vous savez que je suis pour la sortie du nucléaire , je n' ai jamais cru une seule seconde ... que on irait , on appuierait sur un bouton , et il n' y a plus de nucléaire . Il y en a pour 20 ans ou 25 ans à en sortir . Mais c' est dangereux dès maintenant . Et le dire , ça n' est pas mettre en cause ceux qui y travaillent ! Ni ceux qui ont fait tourner tout ça jusqu' à aujourd'hui , dans des conditions de sécurité qui font que les incidents , n' ont jamais dégénéré en accidents . Mais quand ça devient dangereux , ça l' est pour de bon . Je ne vais pas vous faire peur , mais c' est quand même à côté de chez vous qu' il y a la centrale du Blayais . Oui , ben , il faut y penser , hein ! Moi , je vous le dis , je la fermerai . Mais je voudrais que vous compreniez une chose . C' est que pendant que nous sommes en train de parler , le changement climatique fait monter le niveau de la mer . La dernière fois , grâce à un ingénieur particulièrement intelligent , nous avons été saufs , parce qu' il a compris qu' avec un vent de 140 km/h et des creux de 7 mètres , les prises d' eau allaient être bloquées , et que donc , la centrale aurait un accident . Cet homme a compris qu' il fallait arrêter la centrale , et il a bien fait parce que son pronostic s' est vérifié , en effet , les prises d' eau ont été bloquées . Vous êtes à 50 km de cette histoire . La flotte monte . De toute façon . Que vous soyez contents ou que vous le vouliez ou pas . Que M. Fillon accepte l' idée qu' il y ait un changement climatique ou qu' il ne l' accepte pas . De toute façon , il a lieu . Et donc , le niveau de la mer monte . Et donc , dans 20 ans , dans 30 ans , quand on arrivera au bout du cycle qu' ils décrivent , la probabilité que la centrale soit noyée , sans avoir rien besoin d' autre que de cette montée des eaux cette probabilité est extrêmement élevée . Le responsable politique doit prendre en compte des problèmes de cette nature , en ajoutant que nous avons une difficulté : nous ne savons pas réellement démonter les centrales , celle sur laquelle nous avons commencé à le faire , 10 ans après , on y est toujours . C' est une grande difficulté technique , c' est pourquoi , contrairement à ce que disent certains , démanteler le nucléaire , qui ne garantit nullement notre indépendance , parce qu' il faut aller chercher le combustible dans des pays où on est pas toujours bien disposé à nous le donner . Entre autres choses . Et bien , si nous voulons sortir du nucléaire , c' est un défi technique et scientifique et de recherche immense . D'abord parce qu' il faut mettre en route tout le reste pour que ça marche . Je sais très bien que si vous m' élisez , je ne peux pas me contenter de dire à tous les Français : &quot; Ah ben , écoutez , on va en sortir &quot; , sans expliquer ce qu' on va faire d' autre . Ce qu' on va faire d' autre , c' est développer toutes les autres énergies . Il nous faut du 100 % renouvelable . J' ai passé - vous savez , le programme , ça fait un moment qu' on travaille dessus , hein , croyez pas que j' arrive en sortant ça de mes poches , et les discussions avec les amis , et avec les camarades , et avec les spécialistes , avec des gens qui ne sont pas forcément nos amis politiques , mais à qui le simple fait qu' on propose un horizon de développement , de leur propre intelligence , de leur capacité de savoir , soulève d' enthousiasme . Nous avons besoin que , dans les deux premières années du mandat , on installe , par exemple en mer , une puissance de production électrique , ce qui est faisable , dans les deux premières années , équivalant à au moins une centrale ou deux , pour que les Français comprennent que c' est possible , qu' on sait le faire . Nous avons par milliers des jeunes gens , et des moins jeunes , femmes et hommes , ingénieurs , techniciens , ouvriers hautement spécialisés , qui savent faire tout ça . Parce que c' est de très haut niveau technique , contrairement à ce quoi croient les imbéciles , qui nous disent sans cesse qu' on va retourner dans les cavernes et s' éclairer à la bougie . Comme ils nous le disent . Ou qu' ils croient qu' ils ont répondu à tout en disant : &quot; -Ah , ben on va faire des voitures électriques ! &quot; &quot; Ah oui , et , benêt , tu la trouves où l' électricité ? &quot; Pour un million de voitures électriques , il vous faut une centrale nucléaire de plus , vous appelez une solution ? Non , ça n' en est pas une ! Aussi longtemps qu' on ne sait pas produire de l' électricité - et quand je dis &quot; on ne sait pas &quot; - , on ne peut pas produire de l' électricité dans de nouvelles conditions . Alors , maintenant , voilà ce qu' il a trouvé , l' autre . &quot; Donner la main au marché pour les énergies renouvelables &quot; . Non , ça , c' est donner le portefeuille , je connais . C' est la privatisation des profits . Par exemple , on a donné la main au marché pour les ports dans notre pays . Les &quot; ports &quot; , les bateaux , hein . Pas les cochons , les cochons , il y a longtemps que ... Il y a longtemps que c' est le marché qui décide , mais ne rigolez pas , c' est triste . Parce que comme c' est le marché , quand ça a marché , et bien , ils ont fait d' immenses élevages , où les animaux sont traités d' une manière ignoble , et l' année d'après , comme il y a trop de jambon , il y a trop de porcs , et bien , plouf , tout s' écroule , et les pauvres gens qui ont fait cet élevage et les pauvres bêtes qui ont enduré le martyr que vous savez , tout ça pour rien . Pour que la main du marché fonctionne . Voilà . Alors , il propose de donner la main au marché . Bref . Les ports , on les a privatisés . OK ? Et bien , maintenant nous voilà rendus au point où les fameux privatisés viennent demander à l' Etat quoi ? De désembourber les ports . Ah bon , on croyait que le marché allait régler le problème ! Que vous étiez là pour faire des investissements . Non , vous n' êtes là que pour prendre . Nous avons donc des raisons légitimes de nous méfier du marché pour régler ce genre de problèmes . Et vous allez voir , le plus beau va vous arriver bientôt , peut-être que vous ne le savez pas , mais l' Union Européenne , l' Europe qui nous protège , a décidé de vendre tous les barrages hydroélectriques au privé . Pour que ça soit plus compétitif , et vous pensez , qu' immédiatement , des investisseurs pleins de milliards se sont dit : &quot; Mais quelle joie ! Courrons tout de suite mettre le béton qu' il faut pour entretenir ces barrages , dont 25 % ont plus de 70 ans . &quot; Tous étaient tellement contents de se dire : &quot; Vite , courrons payer les employés qui ont fait tourner ça pendant tant d' années , augmentons leur nombre ! &quot; parce que vous savez , il en manque . L ' autre jour , je suis allé dans une centrale , oh , ben moi , je suis ... j' aime les machines , hein , c' est parce que j' ai fait philo , je pense qu' il y a un truc qui n' a jamais été réglé en moi . J' aime les machines , j' aime qu' on m' explique comment ça marche et tout , alors me voilà là-dedans , un truc , vous savez , des années 60 , une merveille . Ah , mais quand je dis une merveille , une merveille ! C'est-à-dire que non seulement c' est beau , c' est dans la montagne , ils ont fait le sol en marbre , une immense fresque à la gloire de l' électricité , et vous avez les gars d' EDF qui sont là , les filles font les choses , tout , tout , tout , très bien . Et en passant , j' en vois trois qui étaient en train de réparer un truc . Je vais vers eux , je dis : &quot; Bon , qu' est -ce que vous faites là , les gars ? &quot; , bon c' était trois hommes , et bon , &quot; qu' est -ce que vous faites ? &quot; &quot; Ben , nous , on est en train d' installer un stator &quot; , si vous le savez pas , il y a un stator et un rotor , le stator , c' est ce qui ne bouge pas , le rotor , c' est ce qui tourne . &quot; Ah bon , &quot; , et on parle comme ça , gentiment , Et le hasard , il y en a un qui dit : &quot; Mais , vous savez , M. Mélenchon , je vais vous dire une chose , j' ai été à la même école primaire que vous . &quot; C' est le hasard , non , de la vie , c' est incroyable , ce que le militantisme politique va ... l' école primaire où je suis allé moi , vous êtes pas prêts d' y aller , hein , c' est l' école Berchet à Tanger . Ah non , mais , quand même , je lui dis : &quot; Mais , dites donc , vous , si vous êtes allé à la même école primaire que moi , vous avez quel âge ? Vous avez plus de 60 ans , qu' est -ce que vous foutez là ? &quot; &quot; Vous n' êtes pas à la retraite ? &quot; Ben , il me dit : &quot; Ben , non , je ne peux pas . &quot; &quot; Pourquoi ? &quot; &quot; Ben , parce que , voilà , ils ont pas renouvelé les équipes , alors , notre entreprise qui fait de l' entretien , on a changé je ne sais pas combien de fois de patron , mais , ils n' ont pas recruté , alors maintenant , je suis obligé de revenir pour expliquer aux plus jeunes comment on fait . &quot; Parce que faire un stator , c' est un travail , bien sûr qui se lit , qui s' apprend à l' école , hein . Que celui qui croit qu' on travaille qu' avec sa main , c' est celui qui sait pas qu' il y a un cerveau au bout de la main . Vous en entendez , des imbéciles , le dire sans arrêt , ça . Hein ? L ' intelligence de la main , ben , l' intelligence de la main , ça commence par l' intelligence tout court . C'est-à-dire par le cerveau , et si vous le remplissez pas , vous avez pas des mains habiles . Je vous dis ça ... Je vous dis ça parce que les messieurs-dames qui ont voté pour M. Fillon , et M. Fillon , il dit qu' il faut ... &quot; Construire , développer l' apprentissage . &quot; Forcément , il y en a pas un qui a son gosse en apprentissage , donc ils trouvent ça très bien pour les autres . Ils n' y connaissent rien ! Bien sûr qu' il y a des métiers où il faut de l' apprentissage . Mais s' il y a ici des gens qui ne savent pas pourquoi on va en apprentissage , c' est souvent parce que bon , tu peux t' embêter à l' école , mais tu pourrais aller au lycée professionnel , pourquoi tu vas en apprentissage ? Ben , parce qu' aussi , ça met du beurre dans les épinards . Faut pas tourner autour du pot , faut dire la vérité comme elle est . Et c' est comme ça que ça se passe . Alors , bien sûr qu' il y a des métiers où c' est un bon passage , mais je veux vous dire , si c' est ça leur idée , de mettre tous les mômes en apprentissage , alors , ça , c' est une vraie bêtise . Ils voulaient déjà en mettre 500.000 , ils ont donné 1.000€ ... de primes à qui embauchait un apprenti , et à quoi ils sont arrivés ? Il y en a moins qu' avant . Voilà . Pourquoi ? Pourquoi il y en a moins qu' avant ? Ben , parce que les parents et les jeunes , ils s' occupent de l' avenir , et ils voient bien , que pour eux , c' est pas par là que ça passe . Moi , c' est l' inverse . Pour accomplir le programme que nous avons , notamment le chapitre écologie , nous avons besoin ... de ... millions , pas de milliers , de millions de gens formés à haut niveau technique . Et si vous ne le savez pas , parce que vous avez des préjugés de classe , ou de caste , mais un bac pro , ce n' est pas pour rien que ça prenait 4 ans . C' est parce que c' est difficile . Faire de la maintenance de systèmes automatisés , faire de la métallerie , faire de l' électricité , tout ça à monter , il faut des savoirs abstraits pour savoir faire tout ça . Quand vous entendez parler d' un chaudronnier , allez surtout pas croire qu' il est en train de faire des chaudrons . Un chaudronnier , il fait des carlingues . De bateaux , d' avions . Ça se règle au millimètre , au dixième de millimètre . Vous ne confiez pas des machines qui valent des centaines de millions à quelqu'un à qui vous allez filer une banane chaque fois qu' il a appuyé sur le bon bouton . C' est pas comme ça que ça se passe , l' éducation . Voilà ce qu' ils ont trouvé pour développer le pays , libérer l' énergie du marché , blabla . Et les gens qui vont faire tourner tout ça , quand est -ce qu' on en parle ? Jamais . Ils ne sont pas au courant , ils ne savent pas comment ça se passe , ils ne savent pas ce qu' est la production . Comme ils n' ont jamais fait philo , ils ne s' intéressent pas aux machines . Et comme ils n' ont pas vu une machine de leur vie de près , ils ne se demandent pas qui est capable de la faire tourner , ni de la construire , c' est pas leur sujet . Leur sujet ... c' est la &quot; bonne allocation de la ressource &quot; . Ça veut dire mettre son blé là où il va en revenir davantage . Alors . Là , je suis sur le thème de l' écologie , pour vous montrer deux manières de penser , j' espère que mon propos est assez illustré pour qu' on comprenne la méthode politique . C' est pas tout . Agriculture programme Fillon . Je vais pas vous passer tout le programme parce que j' en ai au moins pour 10 meetings , alors je garde des morceaux différents et les gens qui suivent , ils regarderont les enregistrements pour reconstituer . Agriculture . Abroger - par ordonnances , hein , attention , hein , c' est que ça urge , hein . C' est pressé , c' est pressé , c' est pressé , tellement que c' est par ordonnances . Abroger par ordonnances ... toutes les normes ajoutées aux textes européens . Ah ha ! Personne ne comprend rien . C' est fait pour . Alors , il y a donc en agriculture des normes qui ont été ajoutées aux textes européens , pendant ce temps là , vous ne parlez pas des textes européens , et un jour ou l' autre , je vous en parlerai , mais c' est pas ce soir . C' est quoi les normes en question qui ont été ajoutées ? Ben , je vais vous donner un exemple . L ' Europe permet l' utilisation des OGM , des organismes génétiquement modifiés . La France a fait jouer la clause de sauvegarde . On n' a pas le droit de planter ça chez nous . D'accord ? Et bien , si vous abrogez par ordonnances , les normes qui ont été ajoutées à celles de l' Europe , ça veut dire que vous rendez possible l' utilisation des OGM en France . Bon appétit , les gens ! Bon appétit ! Et pareil pour les pesticides . Parce qu' un certain nombre de pesticides qui sont autorisés par l' Europe sont interdits en France . Et bien , lui , il propose de l' abolir au motif que , pour certains pesticides , il n' y a pas de produit naturel équivalent , donc , comme il y a absolument de pesticides , et qu' il faut en foutre partout , il trouve qu' il y en a pas assez , on est le premier pays de l' Europe à consommer des pesticides , vous avez des milliers de gens qui se sont rendus malades à la terre , dans le travail de la terre à cause des pesticides , vous avez des kilomètres cubes , je sais pas si ça existe , qui ont déferlé dans la mer , pourrie jusqu' à la moelle , de pesticides de toutes sortes , ont tout dégueulassé , tout saccagé en cours de route , mais non , ça suffit pas , il en faut encore plus . C'est-à-dire , on renonce à toute pensée moderne , créatrice , qui essaye d' utiliser les techniques que la nature a donné aux êtres humains pour atteindre leurs objectifs dans la culture . Alors ça s' appelle , l' agriculture bio . Ne leur parlez pas de bio , ça les rend malades . C' est eux qui ont interdit , deux des personnes qui soutiennent M. Fillon , directement , M. Retailleau , qui est le président du conseil départemental ou non , il est peut-être autre chose maintenant , enfin , je sais plus où on en est . Retailleau . Retailleau et Larcher , président du Sénat . Voilà les deux personnes qui ont fait interdire l' idée , qui était pourtant pas la révolution écologiste de 20 % de bio dans la restauration collective . Voilà , nous , notre objectif , dans le programme , c' est bien , je vais vous les mettre un derrière l' autre : 100 % d' énergies renouvelables donc , on va savoir comment le faire . Les éoliennes , en mer , sur terre , moi , je préfère en mer . Il faut donc ré-équiper les ports , dont je vous parlais tout à l' heure , c' est pas pour rien que je me suis intéressé aux ports , parce que pour les installer , il faut des navires de service . Pour les installer , et ensuite pour les entretenir . Il faut donc changer les quais . Il faut changer tous les appareillages quil y a autour , ensuite , Une fois qu' on est prêts , il faut fabriquer sur place , c' est pourquoi la propriété , quoi qu' elle soit importante pour notre souveraineté , des entreprises qui font ça , n' est pas déterminante , parce que pour mettre une éolienne , vous mettez une pale qui fait 70 mètres . Celle-là , vous l' amenez pas de Hong-Kong en bateau . Vous êtes obligé de la produire sur place , donc vous voyez , j' essaye de vous faire miroiter tout ce qu' il y a à faire , et dans la salle , il doit y avoir des gens qui sont des techniciens et qui comprennent et qui disent : &quot; Mélenchon , il a pas tout à fait compris le coût des pales , mais il sait de quoi on parle . &quot; Oui , il sait de quoi on parle , il s' y est intéressé . Heu , non . Je viens de vous dire ce qu' il compte faire , j' ai oublié , bon , c' est à la fin de ma liste , hein . &quot; Engager la préparation de la mise en concurrence de la SNCF tel que prévu par les directives européennes . &quot; Il y a des cheminots dans la salle ? Enthousiasme , s' il vous plait . Les trains privés partout . Vous savez très bien ce qu' il se passe , comme vous le lisez comme ça , si vous êtes benêts , vous vous dites : &quot; Ben , c' est bien , c' est le capital privé qui va faire rouler des trains privés , qui se paieront en privé , avec des privés qui monteront dedans et qui auront donc des bénéfices privés , pour ceux qui les ont fait monter là . &quot; Et vous croyez que tout va aller ? Mais pas du tout ! Vous savez bien comment ça se passe . Si vous privatisez , il n' y a que les lignes vraiment rentables ou dans lesquelles monte tout le monde , qui sont privatisées . Et les autres ? Qu' est -ce qui passe avec les autres ? Et bien , les autres , il n' y en a plus . Ils ont déjà commencé , et à la place , vous mettez des voitures , ou des bus , des bus Macron . Bah , oui . Tout ça , c' est tout du pareil au même . C' est deux logiques : ou c' est la logique du service public et du bien commun , ou bien c' est la logique de la magie de la main privée du capital privé qui sait à quel endroit il faut aller , tout , ah , ben , c' est clair , le capital , il sait où aller , il va où c' est rentable . Et une ligne qui n' est pas rentable de leur point de vue , pour nous , elle est rentable . Parce que quand il y a un car de moins , quand il y a une voiture de moins sur la route , nous on compte aussi tout ce qu' il y a en plus dans la nature . Tout ce qu' il y a en plus dans le fait que vous prenez moins de risques . Dans le fait que ça peut être plus confortable . Donc , notre comptabilité à nous , elle est globale , elle englobe tous les aspects de la vie humaine , pas seulement un moment , le moment où vous venez , vous achetez votre billet . S' il est plus cher ou s' il est moins cher . Résumé : Je vous ai dit , que pour cet homme , le principe de précaution , le nucléaire , les énergies renouvelables , l' agriculture bio , non ! Il n' en veut pas . Vous êtes prévenus . Quant à nous , que ferons -nous ? Non seulement on n' enlèvera pas le principe de précaution dans la Constitution , celle qu' il y aura à écrire , mais on proposera que dans celle qu' il y a à écrire , on y mette la règle verte . La règle verte , ça sera une chose extraordinaire , qui va nous situer aux avant-postes de l' humanité , comme un peuple , qui est conscient de sa force , de son niveau d' instruction , et donc de sa responsabilité devant l' humanité universelle . Car plus on est , de gens , instruits , prêts à travailler , hautement qualifiés , plus on a de capital accumulé , et il y en a dans ce pays , plus que jamais , qui est la 5ème puissance économique du monde , plus alors on est responsables , devant le monde entier , devant tous les autres êtres humains . Et c' est à nous , de mettre au point les techniques , les moyens qui permettent aux autres d' en faire autant que nous . Quoique , bien souvent , il faut aller apprendre auprès des autres comment on fait . Mais c' est à nous de mettre à la disposition de ceux qui nous disent : &quot; Mais on ne vous demande pas votre argent , on ne vous demande pas vos grosses larmes . &quot; Comme je l' ai entendu dire par mes amis et camarades d' Afrique . &quot; On en a assez de votre compassion ! &quot; nous disent -ils . Car figurez -vous , qu' il y a aussi en Afrique , pour le cas où vous ne le sauriez pas , une très importante catégorie de gens qui ont été formés , éduqués , qui sont de très haut niveau que souvent , on leur pique , pour les faire travailler ici , alors qu' ils ont été instruits là-bas , avec l' argent des gens qui se trouvent là-bas , mais qu' on fait fuir , parce qu' on ruine les économies africaines . Et bien , qu' est -ce qu' ils nous disent ? Qu' est -ce qu' ils nous demandent ? C' est dans les rapports de l' ONU . Là , c' est une Française qui préside ça , qui s' occupe , de ces questions de développement . Qu' est -ce qu' on nous demande ? Pas 50.000 trucs . Premièrement , et surtout : &quot; Aidez -nous à avoir de l' énergie . &quot; Nous , Français , nous savons le faire . Nous pouvons proposer les machines et les brevets , et eux , ils savent faire avec les machines et les brevets , parce qu' ils sont , comme nous , équipés d' un cerveau . Les machines et les brevets pour se produire de l' énergie à partir de la luminescence , pas du Soleil , de la luminescence . On sait le faire . Il y a une boîte qui faisait ça , elle s' appelle Nexis . Il y avait même une chaîne de montage . Si on avait utilisé cette méthode , pour vous donner une idée , je sais pas si vous voyez comment est Marseille , il y a une grande tour , on la voit , on voit que ça . Bon , allez , prenez la tour Montparnasse à Paris , c' est la même chose . On l' aurait couverte de ces verres , la tour , elle devenait positive , c'est-à-dire qu' elle ne consommait pas plus d' énergie que celle qu' elle produisait , qu' elle produirait , non seulement la tour , mais tout l' arrondissement . On sait le faire . Ce qu' il faut libérer , ce n' est pas le capital , c' est l' intelligence , les moyens de faire , et c' est le capital qu' il faut emprisonner . Cette boîte a fermé . A mesure , je vous raconterai des histoires , comme ça , d' entreprises , parce que quand même , je me balade beaucoup , et j' ai pas les yeux dans la poche . Tous ces gens sont prêts à faire , ... ( coupure vidéo ) ... avoir comme réponse : &quot; Vous pouvez bien mourir en mer , si vous savez pas nager . &quot; Parce que c' est ça qu' on fait . Et ça , c' est ignoble . Bref . On nous demande de l' aide intellectuelle , plus exactement , de mettre à la disposition de tous , des moyens techniques . Pas autre chose . Evidemment , une autre manière de faire du commerce , une autre manière d' organiser nous-mêmes nos circuits de production . Si nous n' avions pas ces excédents invraisemblables , on éprouverait pas le besoin de le déverser sur les autres . Il faut donc changer notre modèle agricole , à nous , pas le leur . Comme on est en train de faire , car on leur dit à eux , qu' il faut changer : &quot; Ah , qu' est -ce que vous produisiez ? Ah , 4 carottes , 3 poireaux . &quot; &quot; Mais ce n' est pas ça , l' agriculture moderne , l' agriculture moderne , c' est de faire ceci , faire là &quot; , enfin bref . Les gens produisent plus qu' ils ont à produire , mais nous , on doit changer . On doit rétablir - c' est plutôt du sorgho que des carottes et des poireaux . Nous , on doit établir ces circuits courts de l' agriculture . Il faut , diable ! Il existe plein de fermes en France qui fonctionnent de cette manière . Avec des circuits courts . Qui vendent la production sur place , et vous avez plein d' endroits , où , comme ça n' existe pas , vous êtes quelque part et vous décidez que vous allez mettre 100 % bio dans votre cantine , et vous découvrez qu' il n' y a pas une personne qui produit des carottes à moins de 200 km autour de chez vous . Parce que ça , on ne sait plus le faire . Parce qu' il n' y a personne pour le faire . Parce que ça permet pas d' être payé dans des conditions telles que vous arriviez à vivre de votre travail . Ecoutez bien ça , c' est pour ça que tout se tient . Si vous voulez le bon manger pour vos enfants , et pour vous-mêmes , si vous voulez lutter contre l' épidémie d' obésité qui est en train de dévaster la masse de notre peuple . Si vous voulez , simplement , manger une pomme comme j' en ai mangé , que plus personne ne peut en manger ici , et pas vous , les petits . Moi , quand je mangeais une pomme , il y avait des vitamines . Pour que vous ayez la quantité de vitamines que je mangeais quand je mangeais une pomme , au siècle dernier , combien il faut que vous en mangiez ? Vous connaissez la réponse ? Cent ! Et bah alors , hein ! Si c' est pour les vitamines , il va falloir ronger , hein ! Voilà . Donc il faut changer notre modèle agricole , et si nous changeons notre modèle agricole , d'abord , nous vivrons mieux , mais nous devrons payer les paysans , parce que si vous ne le savez pas , c' est le travail des paysans , et le sous-paiement des paysans , qui a fait que , pendant des années , on a pu mégoter avec les travailleurs leurs augmentations de salaire et les limiter au maximum . Parce que pendant ce temps là , on baissait les prix agricoles , si bien que les gens avaient l' impression qu' ils gagnaient plus puisqu'ils pouvaient acheter davantage de choses . Ça a duré comme ça pendant vingt ans . Et puis , un jour , on a tellement pressé qu' il ne restait plus ni paysans , ni production agricole , qui contentent les besoins que nous exprimons . Il reste 900.000 paysans dans 200.000 ... fermes . Et , je vous dis , quand je dis &quot; fermes &quot; , c' est pour être sympa . Parce que certaines , ce n' est pas des fermes pour moi , c' est des usines . Et c' est des usines qui ne font pas vivre . Ni les bêtes qui s' y trouvent , ni les gens qui s' occupent d' elles . Et c' est pourquoi , dans notre pays , il se suicide un paysan tous les deux jours . Dans un silence immense , parce que vous savez quel est le vrai problème de ce pays ? L ' ISF ! C' est ça la grande question . La chose qui paralyse tout , qui fait que , nous en sommes où nous en sommes . Bon . Voilà . Je vous ai dit ce qu' on allait faire . La règle verte , qu' est -ce que ça veut dire ? Celle qu' on va mettre dans la Constitution : On ne prendra plus dorénavant à la Terre davantage que ce qu' elle peut renouveler . Et ceci est un défi technique . Vous autres , jeunes gens , et les autres , vous aurez à régler cette question , parce que dorénavant , c' est comme ça qu' on produira dans ce pays . Et on vous demande d' aller chercher dans votre instruction , dans ce que vous avez appris à l' école , et pour les autres , ce que vous savez faire , le moyen d' atteindre cet objectif . Et vous y arriverez , parce que la génération précédente , en moins de 10 ans , elle a mis en place la filière nucléaire , alors qu' on savait même pas faire une seule centrale à l' époque , on l' a fait en dix ans . Et maintenant , on doit défaire . Mais c' est pour vous dire que des choses qui paraissaient incroyables , nous les avons faits . Nous avons fait les fusées , alors qu' à l' époque , les Nord-Américains disaient : &quot; Il n' y aura jamais aucune autre fusée que les nôtres . &quot; Evidemment , les fusées , c' est des missiles , hein , vous avez compris . Nous avons fait le train à grande vitesse , nous avons équipé tout le pays de barrages . Quand on le veut , quand on le décide , tout le monde y met son énergie et son enthousiasme . Le peuple français peut faire tout ça . Donc il y aura de la planification écologique . On ne peut pas faire une agriculture bio , sans que deux conditions soient réunies . Premièrement , que vous soyez mieux payés pour pouvoir vous payer de la bonne agriculture bio . Voilà pourquoi ... augmenter les salaires ... c' est écolo ... et ceux qui vous disent et opposent sans cesse la classe ouvrière aux revendications écologiques ne connaissent rien à ce qu' est la mentalité des travailleurs aujourd'hui , parce qu' ils savent qu' ils sont les premières victimes des process de production qui ne sont pas respectueux de la nature et des hommes . Et deuxièmement , parce qu' ils connaissent toutes ces réalités que je viens de vous décrire . Deuxième condition : il faut préparer les gens , si vous voulez faire une agriculture bio , il nous faut 400.000 paysans de plus . Je ne vais pas les trouver en comptant dans vos rangs , 1 , 2 , 3 , 4 , 5 , t' es paysan , 1 , 2 , 3 , 4 , 5 , t' es paysan , ça , c' est Pol Pot qui fait ça , pas nous . Nous , on sort pas les gens des villes pour aller les coller à la campagne . Il faut donc proposer . Il faut éduquer , il y a plein de jeunes gens qui ont envie de faire ces métiers là . A condition qu' on ne vienne pas leur dire : &quot; Et bien , maintenant , voilà ce qu' on te propose pour le restant de tes jours : une paie de misère , tu restes tout seul à la ferme et tu restes au cul des vaches tout le temps . Pas de vacances . Pas de soirées . &quot; Non ! C' est donc la condition sociale des paysans et la condition culturelle des paysans qu' il faut les aider à transcender de fond en comble , et dans la paysannerie française , la tradition existe qui permet ce changement . Si vous ne le savez pas , c' est dans la paysannerie que sont , souvent , les formules socialistes les plus avancées . De mise en commun , de coopératives , d' utilisation en commun des outils de production et combien d' autres choses . Donc , le milieu est prêt , mais nous , on doit planifier . Comme je vous l' ai dit tout à l' heure , si vous voulez installer des éoliennes , et ben , vous êtes obligés de réfléchir à tout le reste . Sinon , vous ne savez pas de quoi vous parlez . C' est pas sérieux . Alors , on me dit : &quot; Ah , la planification ! &quot; Heureusement , maintenant , il y a M. Hulot ce matin , dit comme moi , il dit : &quot; Il faut planifier . &quot; Bah oui , il est sérieux . Vous ne pouvez pas dire aux gens que vous allez faire ceci et cela et rien n' organiser , c' est ça que ça veut dire , planifier . Ça ne veut pas dire que vous êtes dans un bureau à Paris et que vous décidez combien de fraises on va produire à Marseille . Ça , c' est absurde . Ce n' est pas ce que nous proposons . Et la planification , elle doit reposer sur un mécanisme souple et large , très largement décentralisé . Les réseaux doivent être nationaux , mais la production , elle doit être locale , reposer sur l' intelligence , de ceux qui savent qu' à tel endroit , on peut faire ceci , à tel endroit , on peut faire ça . Par exemple , je vous ai parlé d' éoliennes , je pourrais vous parler d' hydroliennes . Les hydroliennes , on met ça dans la mer . Ah , ben , on veut pas en mettre partout . Il faut - il y a des courants extraordinaires , vous verriez ça , c' est une machine extraordinaire . Là , les pales , elles font 9 mètres . Celles qu' on a mis , qu' on vient de mettre là - de couler . C' est un endroit où il y a un courant d' enfer , donc ça se prend des parpaings comme ça . Qui arrivent dans les pales , ping ! Le truc doit être calculé pour résister à tout ça et transformer en électricité l' énergie qui vient de la mer , et l' énergie qui est dans la mer , elle est gigantesque , il y a 75 fois ce dont on a besoin sur Terre , dans les mouvements de la mer , et tant qu' il y aura la Lune , il y aura des mouvements de la mer . Après , ben ... Les suivants se débrouilleront , hein . D'accord ? Mais pour nous , c' est ça . Mais c' est un bien grand mot , on dit hydroliennes , alors tout le monde prend un air très intellectuel , parce que : &quot; Ça va ? hydroliennes ? &quot; Mais des hydroliennes , mes amis , dans ce pays , il y en a eu des dizaines de milliers , hein . Pendant des générations et des générations . Mais on appelait pas ça &quot; hydroliennes &quot; , on appelait ça &quot; moulins &quot; . &quot; Moulins à eau &quot; . Il y en avait partout ! Des moulins à eau , vous pouvez en mettre partout . D'ailleurs , vous en avez un ici , d'après ce qu' on m' a dit , à Bordeaux , sous un pont , qu' on essaye , évidemment , on ne va pas mettre des trucs de 9 mètres de débattement , mais on peut en mettre partout , donc on peut assurer cette conversion vers le 100 % renouvelables si on planifie correctement . Voilà . Ça , c' est dans - Vous avez donc bien deux façons d' aborder ce problème . Je suis parti de l' urgence la plus importante , celle que nous avons comme responsabilité d' être humain , vous êtes obligés de vous poser la question : qu' est -ce qu' on fait en face du changement climatique ? Qu' est -ce qu' on fait concrètement ? Pas simplement regarder ... ou se lamenter ... Faire . On peut le faire . Et on peut être une nation de pointe . Il y a plein de pays qui ont déjà mis au point toutes sortes de techniques . C' est donc pas si compliqué que ça . Et pendant ce temps ... leur priorité , ce n' est pas celle-là . Je vous ai amené un petit film ... qui va vous présenter quelle est leur priorité , et avec quelles méthodes . C' est pas long . Donc quand on voit ça , on comprend , il y a deux sujets , mais avant les deux sujets , il y en a au moins un , vous ne pouvez pas dire que vous n' êtes pas prévenus . Vous , ici , vous êtes prévenus , je le sais bien , sinon , vous ne seriez pas venus . Je sais bien qui est là . Mais , maintenant , il s' agit d' aller expliquer aux gens ce qui se passe là , ce qui se prépare . Déjà , c' est la méthode . Vous avez entendu le mot : un &quot; blitzkrieg &quot; . La &quot; guerre éclair &quot; , c' est ça que ça veut dire , le climat qu' il veut changer , c' est pas celui qui est en train de se détraquer , c' est celui des affaires , avec un blitzkrieg . C' est pas une accusation de ma part , infondée , ou une exagération , &quot; blitzkrieg &quot; , ça veut dire la guerre éclair . Contre qui compte -t-il faire une guerre éclair ? Puisqu'il fait la guerre . C' est donc une guerre sociale qu' il prépare , et il l' a dit . Ne me dites pas que j' exagère , il vient de le dire . Ce n' est pas parce qu' il le dit en allemand qu' on ne comprend pas . Beaucoup d' entre nous ont été à l' école et y ont appris l' allemand , et sinon , on se renseigne . Je récapitule : la suppression de la durée légale du travail . Attendez . Attendez , j' ai besoin que vous compreniez bien . Suppression de la durée légale du travail , en Europe , il y a une limite qui s' appelle 48h par semaine . Si vous supprimez la durée légale du travail , c'est-à-dire que vous dites qu' il n' y a plus de durée légale , ça veut dire deux choses : 1 ) Qu' on peut vous faire bosser 48h par semaine sans que vous ayez le droit de protester , et 2 ) ... et 2 ) ça veut dire qu' il n' y a plus d' heures supplémentaires . Ça veut dire que toutes les heures que vous ferez au-delà de 35h ou 37h , suivant les réformes que ces messieurs-dames veulent introduire , elles sont faites au tarif ordinaire . Terminé , le plus que vous faites quand vous faites des heures sup , et maintenant , j' en viens à ces heures sup . Quand et où quelqu'un a -t-il vu que c' est l' employé ou le salarié qui décide des heures sup ? Jamais . Alors quand l' heure sup est proposée , des fois , ça tombe bien , parce que , il y a besoin de payer les impôts locaux , parce que ceci , parce que cela , parce que il faut changer les cartables et que c' est la rentrée et tout ça . Mais pas toujours . Et vous n' avez pas la possibilité de refuser ces heures là , puisqu'elles ne seront plus supplémentaires , elles seront réglementaires , vous avez compris ? Ça veut dire que par un bulletin de vote , vous reculez direct jusque dans les années 1900 et quelque chose , c'est-à-dire avant 1910 et les premières définitions de la durée légale du travail par jour . Voilà ce qu' il vient de vous proposer . Quand il parle du nouveau code du travail , vous connaissez la singerie ? Ils viennent avec le gros code du travail . &quot; C' est trop gros ! C' est trop ... &quot; Ho ho ! Une petite seconde . J' ai été parlementaire national pendant des années . J' ai fait des paquets de lois sur le travail . Et je vous garantis que ce n' est pas facile . Hein , quand vous n' êtes pas un professionnel du code du travail , pour retenir à chaque fois de quoi on parle , et quelles sont les références , c' est dur , des fois , vous avez des menteurs éhontés , comme celui-là , comment vous l' appelez , qui est président de la région Nord , là . Les Hauts de France , ça s' appelle maintenant . Voilà . Xavier Bertrand . Incroyable ce type . Il mentait , dans l' hémicycle , mais vous savez , à vous couper le souffle . &quot; Ah oui , oui , euh ... euh &quot; , bon , je vais pas vous raconter cette histoire , mais vrai qu' il ment . Et alors , nous , il fallait courir , heureusement , il y avait la pause du midi , je courrais dans le code du travail , pour vérifier , bon , pourquoi il est si gros le code du travail ? Ben , je vais vous le dire , parce qu' il est plein d' exceptions . Et qui c' est qui fout les exceptions dans le texte ? C' est pas nous ! Nous , on est pour des choses très simples , une durée du travail , dans la journée , dans la semaine , dans le mois , dans l' année et dans la vie . C' est super simple . La paie , par heure , par mois , etc . , etc . C' est super simple . Mais eux , ils n' arrêtent pas de mettre des choses , et après ils disent : &quot; Ça fait beaucoup . &quot; Et ben , très bien , si tous les livres qui sont un peu trop gros doivent être déchirés en deux , ben prenez l' annuaire . L' annuaire , c' est plein de noms . C' est plein de numéros de téléphone , c' est gros comme ça . Allez , arrachez la moitié . Et vous allez voir si vous pouvez téléphoner après . Enlevez le code du travail , et vous verrez bien , ensuite , si vous arrivez à organiser le travail . Parce que ceux qui proposent ça , c' est pas le travail qu' ils veulent organiser , c' est les rendements , c' est les profits Parce que pour que le travail fonctionne , il suffit d' aller demander à n' importe quel chef d' équipe , si vous vous méfiez du gars ou de la fille qui est du rang , et demandez -leur si on peut faire travailler les gens 48h à un niveau d' exigence de rentabilité comme celle que vous réclamez , ce n' est pas possible , mesdames , messieurs , vous tuerez les gens au travail , ils s' abîmeront , ils auront des morts . 500 ! 565 personnes meurent sur leur poste de travail chaque année . Et vous n' en voyez jamais une seule à la télévision . Vous ne le voyez pas , l' ouvrier qui est tombé de l' échafaudage , vous ne le voyez , l' homme ou la femme qui s' est fait couper les bras sur la machine . Jamais ! Ces morts là , elles ne sont pas peut-être assez exemplaires pour édifier le peuple dans la seule chose qui vaille , la peur de l' autre . 565 personnes par an meurent sur leur poste de travail . 230.000 personnes , tous les ans , malades à cause du travail . La souffrance répandue partout . Et vous les voyez qui disent que les gens ne travaillent pas assez . Ils comparent . Des comparaisons qui sont seulement faites pour abasourdir et pas pour faire réfléchir . Ah bon , c' est ceux qui travaillent le plus qui sont les plus efficaces ? Alors , dans ce cas , j' ai la réponse , c' est les Roumains . Qui est -ce qui a envie d' avoir une économie française qui ressemble à celle des Roumains ? Personne ! Pourquoi ? Nous travaillons moins parce que les machines permettent d' alléger la souffrance . Et la ... et le nombre d' heures passées au travail . La baisse du nombre d' heures de travail nécessaires pour produire la quantité de richesses est permanente depuis un siècle . Et par conséquent , tout ce temps de travail qui est économisé , ou bien vous le répartissez également , en faisant de la réduction du temps de travail , qui permet l' organisation d' autres activités , ou bien vous le répartissez à la sauvage , les uns travaillent sans fin et sans trêve , et les autres , plus du tout , ils sont au chômage . C' est pourquoi la réduction du temps du travail qui fait horreur à ces gens est une mesure de bon sens , d' équilibre au travail , d' équilibre dans la société , qui va rapporter beaucoup d' emplois qui ont créés , comme ça a été le cas quand nous avons fait les 35h parce que ce sont les années où il y a eu dans tout le siècle , la plus grande création de postes , d' emplois . Et puis ... vous aurez des gens qui vivront mieux . Peut-être avec un peu plus de bonheur , et le bonheur , ça finit toujours , puisque vous voulez tout mesurer , par du mieux-être pour toute la société . Mais vous ne le comptez jamais , vous autres , les puissants , les importants . Vous ne comptez pas le prix du malheur , vous ne vous demandez pas si les 565 personnes qui sont mortes dans l' année ne nous manquent pas terriblement - pas seulement affectivement - ces choses-là ne comptent pas pour vous , mais pour la société , pour tout ce que savait faire cette personne , pour tout ce qu' elle a apporté , pour tout le bien-être qu' elle donnait aux autres . Le malheur a un prix , vous ne le comptez pas . La destruction de la nature a un prix , vous ne la comptez jamais . L' usure au travail a un prix , vous ne la comptez pas . Le blitzkrieg ... qui comporte la fin de la durée du travail , le nouveau code du travail , la réforme de la fiscalité du capital ... mmh mmh ... C' est pas la peine que je vous raconte ce que c' est , vous savez d' avance . Ça veut dire : &quot; Ils payent moins &quot; . La réforme de l' assurance chômage . La réforme de l' assurance chômage , leur grande trouvaille , c' est : &quot; Plus vous êtes au chômage , moins vous êtes indemnisé . &quot; Pourquoi ? Pourquoi ? On a moins besoin , à mesure qu' on est au chômage ? Pourquoi ? Ils pensent que vous êtes des gens qui ne voulez pas travailler . C' est ça , leur problème . Et voilà pourquoi ils veulent cette dégressivité . Je ne vais pas plus loin . La réforme de l' apprentissage , je ne sais pas ce qu' ils comptent faire , hein . Mais je sais déjà que c' est une bêtise , vu ce qu' ils ont fait auparavant . Et après . Ça , c' est les premières mesures d' économie , vous avez entendu . Bon , bon . Là , j' ai été sur le contenu , mais je vais venir sur la méthode . Il y a quand même un point dont je veux encore parler , parmi leurs trouvailles . Il va augmenter - bon la retraite , vous avez compris , à 65 ans , vous êtes prévenus . Fin du CDI . Un contrat de travail unique à droits progressifs . Alors , les ballots , ils entendent &quot; droits progressifs &quot; . Ils ne voient pas juste que ça commence par des droits dégressifs , parce qu' avant quand vous avez un CDI , c' est un CDI . Déjà qu' ils nous ont collé deux périodes d' essai de 6 mois chacune , qu' on peut renouveler , et ben - et des CDD , qui représentent 80 % des embauches - et bien , tout ça , ils vous le mettent dans un contrat où vous êtes tous en période d' essai plus longtemps , tous en CDD plus longtemps , et à la fin , au moment d' être en CDI , vous êtes ? Virés ! Fin de la sécurité sociale . J' espère que vous allez voir ... le film &quot; La Sociale &quot; . Je ne sais pas s' il est déjà passé ? ( Oui ) Faut retourner voir ça . Ou faut voir ça , hein . Vous voyez comment des ouvrières , des ouvriers ont mis en place un système , les pauvres , ils avaient tout sur le dos . C' est les communistes qui ont fait ça . Avec Ambroise Croizat et puis la CGT . C' étaient eux qui faisaient les caisses . Il y en avait , des résistants , je ne vous raconte pas le film , vous n' avez qu' à aller le voir . Mais , la Sécurité Sociale , c' est une invention , il a raison , M. Fillon , c' est une invention , très ... heu , on va dire communiste . Mais , le mot pourrait être enfermé dans l' étiquette de parti , et ce n' est pas ça , l' idée . C' est une invention extraordinaire . Chacun donne , selon ses moyens , mais reçoit selon ses besoins . Si vous êtes très malades , vous êtes soignés , quoi que vous ayez payé . Et nous ... nous , on voudrait que ce soit comme ça pour tout . Et lui , qu' est ce qu' il a trouvé ? Focaliser ... - alors franchement , pour le français , ils sont pas là , hein . - Focaliser l' assurance publique universelle , c'est-à-dire la Sécu , sur des affections graves ou de longue durée . Il faut être très super-malade pour pouvoir aller à l' assurance sociale de M. Fillon , et c' est lui qui décide quand est -ce que vous êtes très super-malades . Et l' assurance privée sur le reste . Autrement dit , tout le reste , si vous avez une assurance vous êtes soignés , sinon , débrouillez -vous . Personne n' a été - a expliqué à ce type que quand on est un peu malade , on peut se soigner , et ensuite , on est pas très malade . Mais si on ne se soigne pas , alors on devient très malade , de plus en plus malade . Et en plus ... on rend les autres malades . Ils se croient tous très malins . Pourquoi croyez -vous que nous soyons dans cette pièce , tous là autant qu' on est , et même s' il y avait des gens qui n' étaient pas de notre avis ? Parce que depuis que l' humanité est humanité et que 99 % de l' histoire de l' humanité , c' est chasseurs-cueilleurs , tout le reste , c' est 1 % de l' histoire totale de l' humanité . Pendant 99,9 % du temps , les êtres humains qui ont survécu , c' est ceux qui s' entraidaient . C' est pour ça qu' ils ont survécu . C' est parce qu' ils étaient solidaires . Et ceux qui n' ont pas été solidaires , ils sont morts avant les autres , parce qu' à la fin , ils étaient tous seuls , et comme ils étaient tous seuls , et qu' ils mourraient avant les autres , ils n' avaient pas d' enfants , et nous ne sommes pas leurs descendants . Nous sommes les descendants des généreux . Nous sommes les descendants des bons . Nous sommes les descendants des gentils . Nous sommes les descendants des partageux . Allez lui expliquer tout ça . Un peu d' anthropologie , là , ça ne lui ferait pas de mal . Alors , après , vous travaillerez plus et vous gagnerez moins , ça , vous avez l' habitude . Et il va augmenter la TVA . Ça m' intéresse . Il va l' augmenter de deux points . Il va passer de 20 à 22 % . 20 , c' est ce qu' on s' est pris du gouvernement actuel . Mais il avait dit à l' époque : &quot; D'accord , c' est 20 , mais c' est moins que 22 , donc c' est bien &quot; C' est la théorie du moindre mal comme je vous l' ai expliqué tout à l' heure , c'est-à-dire que c' est quand même le mal . Là , il vont passer à 22 . Ça veut dire qu' ils vont prendre aux gens qui sont là , 15 milliards . C' est ça , deux points de TVA , c' est ça que ça vous donne , 15 milliards . 15 milliards ! 15 milliards ... c' est tellement que vous ne vous rendez même plus compte de ce que c' est . Ça veut dire 300€ par tête de pique . Alors , si vous gagnez 3.000€ , ça ne vous fait pas le même effet que si vous gagnez 600€ . Et ça ne vous fait pas le même effet que si vous gagnez 30.000€ . C' est pourquoi c' est un impôt injuste . Parce qu' il n' est pas progressif , et que tout le monde le paie . Ça vous coûte 300€ . Dans la poche d' un smicard , 300€ , c' est plus de la moitié du RSA mensuel . Pas vrai Pascal ? Je le vois au premier rang . Plus d' un tiers du minimum vieillesse mensuel . Je vous rappelle que c' est un monsieur qui est appuyé sur des réseaux traditionnels de croyants . Donc il va piquer la moitié du RSA d' un pauvre , le tiers du minimum vieillesse mensuel de quelqu'un qui gagne pas des mille et des cents . Et qui en toute hypothèse , se situe juste à la limite du seuil de pauvreté tel que le décrivent les institutions internationales . Un quart de SMIC mensuel , et la moitié du coût annuel de la cantine scolaire pour un enfant . Et comme leurs amis suppriment les subventions pour les cantines scolaires pour les enfants , ça veut dire que les enfants n' iront plus à la cantine . Parce qu' il n' y aura pas les moyens . Et donc , qu' est -ce qui va se passer ? Les gosses vont avoir faim . Je ne raconte pas des histoires , là . Je vous parle de choses que je connais , que j' ai vues déjà . Qui existent déjà . Pour les gens qui ne le savent pas , il y a déjà des gosses qui mangent une seule fois dans la journée . A la cantine . Ah , ça dérange , hein ? Ça trouble , ça vous rend mal . Et ce n' est pas nouveau . Ça fait un moment que ça dure . Que le problème se pose dans ce pays de gens qui sont en &quot; insécurité alimentaire &quot; . Ils n' ont pas ! &quot; Insécurité alimentaire &quot; , ça s' appelle . Des dizaines de milliers de gens sont en &quot; insécurité alimentaire &quot; . Et qui sont déjà en &quot; insécurité &quot; énergétique , et en &quot; insécurité &quot; pour la flotte , en insécurité pour tout ! Pour tout dans la vie , du matin au soir . Voilà , si vous en voulez , votez Fillon . Si vous n' en voulez pas ... Nous allons créer un nouveau climat des affaires , nous aussi . Nous allons abroger la loi El Khomri . Nous allons ... rétablir les 35h en rendant les heures supplémentaires plus chères . Nous allons rétablir la retraite à 60 ans . &quot; Ha ha ha ... comment vous payez ça , M. Mélenchon ? La retraite à 60 ans , quand même , à la fin . &quot; &quot; Qu' est -ce que vous êtes en train de faire avec ces gens , là ? Vous leur promettez des choses , vous savez pas comment vous le payez . &quot; Mais si ! On a trouvé une idée . Respecter les femmes . Mais quel rapport ? Si vous payez les femmes comme les hommes , et bien les cotisations sociales supplémentaires payent la retraite . Nous allons rembourser les soins à 100 % . Et si jamais ça coûte trop cher , nous allons avoir une discussion ... sérieuse avec l' industrie pharmaceutique . Evidemment , on va augmenter le SMIC . Ah , ça , je sais qu' il y a plein de gens qui crient , à chaque fois . Non , non non non , quand c' est nous , on améliore la vie des gens . Et pour améliorer la vie des gens , déjà , on améliore leurs salaires . Et puis après , ils en font ce qu' ils veulent . Et moi , je vais vous dire ce qu' ils en font : ils le dépensent . Hein . Pour leurs gosses , pour leur vie , pour les trucs qu' il y a à faire . Et comme ils le dépensent , et ben , hé , ils achètent des choses et comme ils les achètent , ben , faut les remplacer , donc il faut produire . L' augmentation de salaire , c' est le feu de la croissance , et je ne suis pas un partisan de la croissance aveuglée , vous l' avez entendu tout à l' heure . Je vais résumer ce que je viens de dire et la stratégie économique : nous devons relever deux défis en même temps : améliorer la vie des gens , donc , augmenter leurs moyens , et , en même temps , nous devons le faire dans des conditions qui soient respectueuses de l' objectif de durabilité , de soutenabilité écologique . Il faut faire les deux . Pas un après l' autre . Pas je commence là et puis demain , on verra . Les deux en même temps , et c' est parce que on va le faire en même temps que ça va marcher . C' est parce qu' on va faire de la planification écologique , parce qu' on va développer les emplois paysans , parce qu' on va développer l' économie de la mer avec les 300.000 postes dont on a besoin . C' est parce qu' on va le faire avec l' objectif d' un nouveau modèle écologique de la production , de l' échange et de la consommation que nous allons créer de l' emploi et des ressources . Tout se tient . C' est parce que nous sommes écolos que nous sommes socialistes . Alors , le monsieur , là , le premier qui revient me dire : &quot; Heu , Jean-Luc , tu comprends , la constituante , tout ça , on comprend rien , ça n' a rien à voir avec le social , c' est ce qui intéresse les gens . &quot; D'abord , ceux qui disent savoir ce qui intéresse les gens , souvent , se trompent . Ils prennent les gens pour des imbéciles . Les gens ont parfaitement compris que si ce type peut vous annoncer qu' il va faire une blitzkrieg , c' est parce qu' il a une Constitution qui est un coup d' Etat permanent . Et le coup d' Etat permanent , M. Fillon va l' utiliser pour faire un coup d' Etat social . Sa politique , c' est un coup d' Etat social . C' est la destruction de la Sécurité Sociale , c' est la destruction du code du travail , c' est un coup &quot; social ! Quand vous défaites toutes les lois , et que vous le faites passer en force , et que vous vous en vantez , devant une catégorie de Français bien particulière , qui se réunissaient avec lui ce jour-là , vous annoncez à ceux-là : &quot; Je vais faire un coup , je vais passer en force , il n' y aura même pas de pause pendant l' été &quot; , c' est un coup d' Etat social qu' il prépare . Voilà pourquoi la question de la constituante est si importante , les gens , comprenez -moi bien . Le cœur de la stratégie de la révolution citoyenne que je prône , la révolution citoyenne , c' est la reprise du pouvoir par les citoyens sur leur vie , c' est ça que ça veut dire , et ça c' est radicalement nouveau . Ne me dites pas : &quot; C' est facile à faire &quot; . Non , il va falloir que le peuple tout entier se mobilise . Si vous m' élisez , ne comptez pas que c' est moi qui vais tout faire depuis là-haut . C' est impossible ! Il faut que tout le monde s' y mette , pas seulement à produire , mais à changer la vie , la décision . Voilà ce que vous aurez à faire . Donc ... à peine élu , article 11 de l' actuelle Constitution , nous convoquons une Assemblée Constituante . Une assemblée , ça veut dire qu' on va élire des gens pour écrire une nouvelle Constitution . Et on en élira une partie , et moi je propose qu' on en tire au sort une partie aussi . On a besoin des deux . Des deux , pas d' un , des deux ! Des gens élus parce qu' il y a une dispute sur la Constitution , vous le savez bien . Est -ce qu' on veut un régime parlementaire ? Est -ce qu' on veut un régime présidentiel ? Est -ce qu' on veut un régime parlementaire tempéré de certains domaines dans lesquels l' exécutif est fort , ou l' inverse ? Il y a une discussion normale , légitime , souvent , on est soi-même partagé . Il faut que cette Assemblée ait lieu . Et elle ne fera que ça . Et ceux qui seront élus là ne devront jamais avoir été élu dans une assemblée précédente . Pourquoi ? Pas parce qu' on veut montrer du doigt ceux qui étaient dans les assemblées précédentes . Ce n' est pas le sujet . C' est parce qu' on veut un regard neuf , débarrassé des routines du passé . Même des miennes . J' ai été parlementaire longtemps . Je ne souhaite pas être parlementaire dans cette ... Faut dire , je serai déjà Président , je ne vais pas tout faire à la fois , bon . Mais ... c' est important qu' on ait des gens qui regardent avec un œil neuf . Et ce n' est pas parce que leur œil est neuf qu' ils ne sont pas hautement conscients , qu' ils ne sont pas hautement responsables . Quand vous tirez au sort un jury d' assises . Hein ? Vous n' allez pas regarder , vous faites confiance , parce que vous pensez que la personne qui est là est honorable . Si vous ne l' avez pas rejetée , parce qu' elle ne l' est pas . Les avocats ont droit de les rejeter , mais vous pensez une personne honorable , qui va mettre tout son cœur à bien juger , à donner la bonne opinion . Donc on doit avoir confiance dans l' action populaire , pas s' en méfier . Moi , je proposerai quelque chose à cette Assemblée Constituante , quand même , je peux . J' ai parlé , tout à l' heure , de la règle verte . Mais ce que je voudrais vous faire comprendre , c' est que notre méthode , notre stratégie révolutionnaire , c' est l' implication la plus large possible . Plus il y a de monde qui décide , plus on est contents . Alors , déjà , on commence par rectifier quelques problèmes . Il y a 3 millions de personnes qui ne sont pas inscrites sur les listes électorales dans ce pays . Il y en a 6 millions qui sont mal inscrites . Neuf millions de personnes , qui à la prochaine élection présidentielle , ne pourront pas voter si vous ne vous y mettiez pas . Nous , on a fait les caravanes , les militants insoumis , partout où on pouvait , pour inscrire les gens . Occupez -vous autour de vous de savoir si tout le monde est inscrit . Et occupez -vous de savoir s' ils sont bien inscrits , parce que des fois , ils &quot; croivent &quot; ( sic ) , ils ont déménagé . J' ai regardé ça à Strasbourg . Les Alsaciens , c' est des gens sérieux . Ils ont regardé . 176.000 inscrits , 17.000 personnes qu' ils ont dû radier . Et ben , ils se sont mis à chercher où étaient ces 17.000 personnes , où elles habitaient vraiment pour leur dire : &quot; Ben voilà , vous pouvez vous inscrire là , vous pouvez vous inscrire là , mais inscrivez -vous . &quot; Donc , dans notre manière de faire , on va d'abord rectifier ça . Deuxièmement , je proposerai qu' il y ait le vote obligatoire . Parce que , voter , ce n' est pas une faveur qu' on fait . On est membres d' une société , et tout le monde a bénéficié de l' éducation par l' école gratuite , donc , tout le monde doit mettre sa cervelle et son intelligence au service , alors évidemment ... si on veut faire ça , il faut accepter que le vote blanc soit compté , parce qu' il y a des citoyens qui ont le droit de dire qu' ils ne veulent pas . Bon . Encore un point pour élargir la base de la démocratie . Evidemment , dans l' entreprise ... Je n' y viens pas ce soir . Pardonnez -le moi , j' y viendrai un autre soir sur un discours spécifiquement réservé à cette question . Élargir la base de la démocratie , par exemple , c' est une idée - vous allez avoir du mal , hein , quelques uns d' entre vous , et peut-être même tous , moi , j' ai eu du mal avec , et je vais vous dire comment j' ai été convaincu - c' est le droit de vote à 16 ans . Quand on a passé le droit de vote de 21 ans à 18 ans , tout le monde disait : &quot; Ah , quand même , 18 ans , bon , faut pas exagérer , ce sont des gamins , quoi , hein . &quot; Bon , ben le gamin , il était content . Et il a pris sa part . Quand on m' a dit ça , je dis : &quot; mais pourquoi on fait ça ? &quot; &quot; Parce que c' est une vieille revendication &quot; , je m' en fiche . Où on le fait déjà ? Ah , on le fait dans , je crois , 5 , 6 , 7 Lands en Allemagne pour les élections locales . On le fait au Brésil . On le fait en Argentine . Ah bon , et ça fonctionne ? Oui , ça fonctionne . Moi , j' ai dit : &quot; Mais , c' est quoi , le droit d' un môme qui a 16 ans , aujourd'hui ? Et ben , un jeune qui a 16 ans aujourd'hui , il peut déjà exercer l' autorité parentale . Alors , si vous le trouvez bon pour exercer l' autorité parentale , sur un tout-petit , vous devez quand même penser qu' il est capable de se dire tout seul et de trouver ce qui est bon et ce qui est mauvais pour tout le monde . Deuxièmement , à 16 ans , tu peux être émancipé . Troisièmement , à 16 ans , tu peux bosser aujourd'hui . Parce que la scolarité obligatoire s' arrête à 16 ans . Et si tu vas bosser , tu votes aux élections professionnelles . Donc , tu as le droit de t' occuper d' un gosse , tu as le droit de payer des impôts , tu as le droit d' être émancipé , mais tu n' aurais pas le droit d' aller voter ? Non , ce n' est pas raisonnable . Faisons confiance . Et vous verrez que les jeunes gens , que vous acclamez quand ils font des manifestations pour défendre ceci ou cela , comme ils l' ont fait pour le code du travail , et bien , ces jeunes gens , votre devoir de père , de mère , d' éducateur sera augmenté bien sûr auparavant , pour qu' à 16 ans , il soit capable de se prononcer . Et je vous dis que la jeunesse de notre pays est capable de ça , et moi j' ai été convaincu , et donc je le proposerai , et j' espère qu' on le fasse . Ha ha ! Tout le monde n' a pas applaudi , hein . Non , mais il faut réfléchir , ben , on se rencontre aussi pour ça , hein . Mais vous comprenez l' idée ? Plus il y a de monde qui s' en mêle , plus on est forts . Plus il y a de gens qui sont - se sentent responsables de la société , plus notre point de vue est fort . Plus les gens comprennent de quoi on parle , moins ils voudront de ces gens que je viens de vous montrer , parce qu' ils comprendront exactement de quoi on parle , et ils ne diront pas : &quot; Ah , ben M. Mélenchon , il est un peu nerveux , tandis que M. Fillon , il est bien calme , il a une jolie cravate . &quot; Il y en a ! Il y en a ! Mais moi , vous avez compris , hein . Mon message , ce n' est pas ma personne , et vous n' allez pas m' arranger . Je ne vais pas changer . Mon message , c' est le programme qu' on porte ensemble . Le temps qu' on a pris à le mettre au point ! Je finis avec ce Fillon , parce qu' il faut que vous ayez tout . Celui-là , dans les questions internationales , Vous savez qu' il a voté contre Maastricht en 2012 , en , en 199 ... chose ... 1992 . Moi , j' ai voté pour , à l' époque . Je le dis . J' avais idée , que comme on allait mieux intégrer économiquement l' Union Européenne , il allait y avoir , mécaniquement , une sorte de citoyenneté plus grande . Un progrès politique . Je me suis trompé . Avaient raison , à l' époque , la CGT , le Parti Communiste , M. Chevènement , et un certain nombre de gens de droite qui disaient le contraire . C' est eux qui avaient raison . Et moi , je me suis trompé . Je ne suis pas le seul , hein . C' est une majorité qui a voté ça . Non , mais c' est pour dire , que il faut faire un bilan de ses actes . Il n' y a rien de pire qu' un fanatique qui part d' une idée , n' en change jamais , ne réfléchit à rien et répète tout le temps la même chose , parce que c' est lui qui l' a dit . Après , quand j' ai compris ... Je ne vous ai pas manqué , hein . Mais j' ai voté contre le statut de la Banque Centrale Européenne , j' ai voté contre le passage à l' euro , parce que je savais que c' était une folie de le faire avec une Banque Centrale Européenne pareille , et avec un statut pareil , et enfin , quand est arrivé 2005 - ah oui , j' ai oublié de vous dire que pour les deux votes dont je viens de vous parler , j' ai la joie et le bonheur d' avoir une lettre de blâme du Parti Socialiste , signée François Hollande , qui déjà était responsable de ces questions là . Bon , et après , en 2005 , j' ai voté , avec beaucoup d' entre vous , je pense , dans cette salle , on a voté Non contre le projet de Constitution Européenne , et je pense qu' on a eu raison . La preuve , c' est que ils ont pris le projet , l' ont découpé en morceaux , ils l' ont lacéré , ils l' ont jeté en l' air , il est retombé sur de la colle , ils ont dit : &quot; Voilà , c' est le Traité de Lisbonne . &quot; Oui , le peuple français a voté Non , et ils ont transformé ça en Oui . Vous êtes d'accord ? ( Public : Oui ! ) Qui a fait ça ? ( Public : Sarkozy ! ) Qui était 1er ministre ? ( Public : Fillon ! ) Voilà . Qui donc , qui donc a fait cette forfaiture de tromper la parole du peuple français ? Sinon ceux-là . Ces deux là . Et tous ceux qui ont voté avec eux , au Parlement , au Congrès de Versailles . Et qui ont donc trahi la parole . Ensuite , quand nous avons eu le débat , si vous vous en souvenez , non , vous ne vous en souvenez pas , du traité budgétaire , que M. Hollande avait dit qu' il allait le renégocier . Oui , ben il l' a pas renégocié . Rien ! Quelle erreur ils font ? Ce n' est pas seulement de vous tromper , mesdames et messieurs , ils font une autre erreur , qui est géopolitique . C' est que tous les autres nous regardent . Les dirigeants des autres pays . Et ils se disent : &quot; Ah bon ? Les Français peuvent voter ce qu' ils veulent , de toute façon , leurs dirigeants sont d'accord avec nous . Et donc , on fait ce qu' on veut avec les Français . &quot; Donc l' immense force que représente notre nombre , notre pensée , notre extraordinaire réputation dans toute l' Europe , tout ça est réduit à néant par des gens qui nous trompent , nous spolient , et donnent la mauvaise habitude aux autres de penser qu' on peut nous passer sur le corps . Dites -moi pourquoi Mme Merkel aurait un autre comportement ? Elle n' est pas chargée de s' occuper des volontés du peuple français , elle s' occupe des volontés du peuple allemand , c' est pour ça qu' elle a été élue , et en plus de la droite allemande . Vous me direz que comme elle gère avec des socialistes , il ne reste plus grand monde dehors . Il reste nos copains . Mais vous me comprenez ? Pourquoi cette femme , qui rencontre à deux reprises un Président français qui vient avec un mandat : &quot; Je vais renégocier . Je vais refuser &quot; , il arrive , et qu' est -ce qu' il lui dit ? &quot; Bonjour Madame , où est -ce qu' on signe ? &quot; C' est ça qui s' est passé . Et cette erreur , vous la payez aujourd'hui . Quand vous avez une Commission Européenne qui se sent autorisée à élever la voix devant la France , et à nous dire : &quot; Quoi ? Vous ne voulez pas privatiser vos barrages ? Et bien maintenant , je vous mets en demeure de le faire ! &quot; Où trouvent -ils l' audace de nous parler comme ça ? Où trouvent -ils l' audace de nous mettre en demeure ? De vendre ce qui a été accumulé par les générations précédentes , par leurs efforts , par leurs impôts , par leur intelligence collective . Où trouvent -ils cette audace , sinon dans les démissions constantes , les capitulations de ceux qui sont censés nous représenter et nous diriger , et M. Fillon était là-dedans , par conséquent , nous n' avons aucune raison d' avoir confiance en lui , d'autant qu' il est bizarre , moins ça marche , plus il est pour ! Il était contre le Traité de Maastricht , il est pour le Traité de Lisbonne , comment vous comprenez une chose pareille ? Sinon que ses convictions sont à géométrie variable , peut-être , ou bien qu' il n' a jamais cessé d' être autre chose que quelqu'un qui croyait que la meilleure manière d' aliéner la puissance de la France était de faire d' une façon , puis ensuite de faire d' une autre . De la même manière , vous allez avoir l' année prochaine , 2017 , 2018 , premièrement , un nouveau traité européen , parce que les cinq Présidents ont décidé qu' ils allaient proposer un nouveau traité européen . Quand j' ai dit ça à Hollande , après le Brexit , il nous a fait venir , là , alors je lui dis : &quot; Hé , et où vous en êtes avec le nouveau traité ? &quot; Il me regarde , il dit : &quot; Personne ne nous a parlé de ça . &quot; Sérieusement . Ou bien ils ne lui en ont vraiment pas parlé , ou bien il ne lit pas les journaux , et dans les deux cas , c' est quand même un problème , au lieu de parler avec les journalistes , il devrait au moins lire les journaux . Pour savoir ce qu' il se passe . Il y aura un nouveau traité . Réfléchissez bien à ce que vous allez faire . Si vous élisez quelqu'un , qui avant d' y être , est déjà d'accord , ou si vous élisez une tête dure , qui va vouloir qu' on change la règle du jeu d' un bout à l' autre , ou bien sinon , nous serons obligés de faire ce que nous n' avons pas envie de faire , mais on ne nous contraindra pas à faire le contraire , c'est-à-dire liquider notre propre pays , liquider notre propre démocratie . Parce que derrière ce nouveau traité , après vous avez TAFTA , ça va se décider 2017 , 2018 . Et ne venez pas me dire : &quot; Ah , mais Trump n' en veut plus . &quot; Vous n' en savez rien , une chose est sûre , c' est que nous , on n' en veut pas , et si on n' en veut pas , ça ne se fera pas , si c' est moi qui préside . Alors , vous avez bien compris ? Je ne suis pas juste la mauvaise tête de service . L' empêcheur de danser en rond . Toutes ces questions vont venir en 2017 , 2018 , ce n' est pas moi qui les amène , c' est la vie . Feu , Michel Rocard disait que si on signait TAFTA , il n' y a plus d' Europe , en effet , vous ne pourrez plus rien faire , puisque vous serez ficelés pour toutes les normes environnementales et sociales par le contenu du traité , et parce que vous donnerez le pouvoir aux multinationales de faire des procès aux Etats , dans des tribunaux privés , c'est-à-dire en dehors de la loi que vous avez votée . Ces gens là auront une autre loi pour eux , avec leurs propres tribunaux , pour porter des plaintes contre l' Etat , alors que jusqu' à présent , ces tribunaux étaient faits pour régler des différends entre eux . Si bien que non seulement ils porteront des plaintes contre ce que vous aurez voté , mais , souvent , au moment de préparer les lois , quelqu'un vient et vous tire la manche et dit : &quot; Ne fais pas ça . &quot; &quot; Si jamais tu écris ça , si tu proposes ça , ils vont te faire un procès derrière . &quot; Et donc vous direz : &quot; Et ben écoutez , moi , je ne le fais pas . &quot; Comment voulez -vous résister ? Quand vous êtes une grande puissance , peut-être peut -on le faire . Imaginez -vous des dirigeants du Suriname , petit pays , il y a 450.000 habitants , qui ont une immense forêt . Qu' est -ce qu' ils font ceux-là , quand une multinationale va leur dire : &quot; Ah , mais vous , si vous bougez une oreille , vous avez droit à un procès , on vous ruine pour deux générations . &quot; Comme ils l' ont fait à combien d' Etats ? Vous le savez , j' espère , que les multinationales sont en train de faire un procès à l' Australie , parce qu' elle a voté une loi contre le tabagisme . Et les cigarettiers veulent que l' Australie leur paye les cigarettes qu' ils n' ont pas fumées . Et ceux qui font un procès au Canada , parce qu' il a interdit la prospection des gaz de schistes , ou le procès au Canada pour avoir interdit l' usage d' un produit chimique qu' on met dans l' essence , pour que ça carbure davantage , qui encrasse les moteurs et les poumons , et à la fin , ils ont obtenu gain de cause . La Canada non seulement a payé , mais a dû rétablir l' autorisation d' empoisonner les gens . Voilà ce dont il est question . Ce ne sont pas des constructions idéologiques abstraites , encore qu' on a le droit d' avoir des constructions idéologiques abstraites , qui nous font prépa - préférer , comme Montesquieu , la démocratie du peuple tout entier , plutôt que le pouvoir de quelques personnes . Et voilà ce qu' ils ont prévu de faire . M. Fillon a -t-il dit qu' il était contre TAFTA ? Non , il est pour . Est -ce qu' il est contre CETA ? Non , il est pour . Est -ce qu' il est contre TISA ? C' est sur les services publics . Non , il est pour . Vous voyez que c' est deux visions du monde qui vont s' affronter , ce ne sont pas simplement des personnes , des costumes , des mots ! Si notre programme s' appelle &quot; L ' Avenir en Commun &quot; , il y a le mot &quot; commun &quot; dedans , et il y a le mot &quot; avenir &quot; qui veut dire qu' on a le goût du futur . Le sien , il n' a même pas de nom . Ca s' appelle &quot; Un Projet pour la France &quot; . Il ne peut pas le nommer . On va voir ce qu' il va dire . Je le propose , en tout cas s' il débat avec moi , vous allez voir si je ne vais pas lui faire dire . Et maintenant ... rappelez -vous bien une chose ... rappelez -vous qui c' est . C' est moi qui vous ai rafraîchi la mémoire , sur l' affaire des traités européens . Et bien , je vais continuer . Je ne veux pas de la guerre , et vous non plus . Peut-être que vous êtes moins conscients que je le suis , et d' autres , du risque de guerre généralisée qui s' étend sur la planète . Mais nous y sommes ! Et notamment , dans l' Europe de l' Est , avec cette histoire d' Ukraine , avec les provocations sans fin auxquelles se livrent les Etats-Unis d' Amérique dans le cadre de l' OTAN , en installant des missiles anti-missiles en Pologne , qui non seulement annule notre propre système de défense , mais est une menace directe sur tous les voisins , devinez lesquels . Ils sont revenus à la mentalité de la Guerre Froide , avec toutes les catastrophes que ça peut entraîner . Vous savez que l' Ukraine , il y a 7 centrales nucléaires , dont Tchernobyl ? Vous croyez que c' est un endroit où on va se battre ? Enfin ! Il faut donc tout faire pour empêcher que les choses roulent à la guerre . Et je n' ai pas peur de dire , qu' à mon avis , les Russes sont des partenaires , pas des ennemis . Je ne le crois pas . Je crois qu' on est partenaires . Réfléchissez -y ! Je sais que vous êtes peut-être sous l' emprise de bien des arguments qui roulent contre eux , mais pas la guerre ! Pas la guerre ! Ni un peu , ni moyen , ni beaucoup . Mais même un peu , pas de guerre ! Pas de guerre ! Le contraire ! C' est la raison pour laquelle j' affirme que je suis partisan de la sortie de la France de l' OTAN , qui est une organisation militaire vouée à la guerre . Je suis partisan d' une autre alliance altermondialiste , avec d' autres peuples . Et je suis partisan du fait que la France joue un rôle pionnier , aille de l' avant pour dire , si imparfaite qu' elle soit , la seule institution internationale que nous pouvons et devons respecter , c' est l' ONU . Parce que nos prédécesseurs se sont tellement battus pour qu' il y ait une Société Des Nations et qu' on cesse de s' affronter . Non , je ne suis pas d'accord pour les G8 , les G20 , et que sais -je encore , qui font la loi dans la planète . L' ONU , avec tous ses défauts , l' ONU où la France siège , et a un siège au Conseil de Sécurité , siège que pour ma part , je ne céderai jamais à personne , dans aucune circonstance . Telle est la France , et elle peut jouer son rôle . Et bien , c' est le contraire que cet homme ... Qui a ramené la France dans le commandement militaire intégré de l' OTAN ? ( Public : Sarkozy . ) Qui était 1er ministre ? ( Public : Fillon ! ) Très bien . Et pour ce qui est de faire la guerre , je ne vous parle pas de l' actuel qui en a déclenché trois , et ben oui , il faudrait quand même s' en rappeler . Mais qui a déclenché la destruction de la Libye , je vous prie ? ( Public : Sarkozy ! ) Qui était 1er ministre ? ( Public : Fillon ! ) Ce sont des questions d' une extrême importance . Si vous voulez de nouveau confier la France , avec toute sa puissance militaire , qui est considérable , aux mains de gens qui ont été capables d' une telle stupidité que celle qui a consisté à détruire la Libye , puis à engager la France dans un combat plus qu' improbable en Irak , à la suite de l' OTAN , avec des alliés aussi discutables que les Qataris et les Saoudiens , la main dans la main avec des Turcs qui nous tirent dans le dos , si vous voulez confier la France à des gens pareils , rendez-vous compte des dangers que vous lui faites courir . Car rien n' est plus facile que de vider le tube de dentifrice , mais après , on ne fait pas re-rentrer la pâte . Quand la guerre commence , c' est à la portée de tout le monde de déclencher des guerres , c' est très facile . Surtout quand on a les outils de la puissance . Mais c' est très difficile de revenir à la paix . Parce qu' entre temps , des millions de gens se sont sauté à la gorge , se haïssent , des fois pour plusieurs générations , et ils ont des bonnes raisons . Voilà pourquoi la paix est un bien précieux . Et vous autres ... si vous voulez m' aider ... c' est la chose essentielle qu' il faut que vous fassiez avancer dans l' esprit des gens qui nous entourent . Assez de ces soi-disant solutions efficaces , parce qu' elles sont violentes . Ce n' est pas vrai . La violence appelle la violence , et encore la violence et toujours plus de violence . Bon . Il faut que ça s' arrête quand même . Je viens à Bordeaux , et je vous ai parlé de son maire actuel , que je salue , en bonne tradition républicaine . Mais enfin , ce n' est pas mon préféré . Un peu de philo , pour terminer . Le meilleur maire de Bordeaux ... 1581-1585 : Montaigne ! Montaigne , évidemment . Et qui d' autre ? Evidemment . Son camarade . La Boétie . Et le beau livre , un peu compliqué , mais pas trop . En tout cas , vous autres les jeunes , vous avez intérêt à le lire , hein . Sinon , ce n' est pas la peine d' habiter ici , hein . Et pas que parce que vous habitez ici . &quot; De notre servitude volontaire &quot; Tiens , Laurent , je ne sais pas où il est ... Tout à l' heure , on cherchait des citations . On aurait pu prendre ... Du La Boétie , hein , qui dit que &quot; les tyrans ne sont forts que d' une chose : la soumission de ceux qu' ils ont dominé . &quot; &quot; S' ils l' acceptent . &quot; Il y a une autre phrase , je ne vais pas vous saouler avec ça , c' est du Machiavel . Mais Machiavel , ce n' est pas ce qu' on dit , hein , c' était un républicain , il sortait Dieu de la décision politique , alors ça l' amenait dans - sur la terre des êtres humains , hein . Ce n' est pas toujours très glorieux , mais il disait : &quot; La forteresse des tyrans - la forteresse des tyrans - c' est l' inertie des sujets . &quot; Si vous êtes inertes ... vos tyrans font ce qu' ils veulent de vous . Alors , d'abord une leçon de vie . J' adorais Montaigne pour un truc - je n' adorais pas Montaigne quand j' étais jeune , je trouvais ça mièvre , je trouvais ça ... mou , quoi - et le temps a passé , bon , je vois les choses autrement . Par exemple , Montaigne raconte , il dit , il veut montrer pourquoi on n' est jamais meilleur connaisseur de soi , que soi-même . Sa leçon de philosophie , c' est : &quot; N' allez pas demander aux autres ce qui est bon pour vous , essayez d' y réfléchir vous-mêmes &quot; . Alors , il raconte une anecdote , il dit : &quot; Quand ça vous gratte , il n' y a que vous qui savez juste l' endroit où il faut mettre le doigt pour gratter . &quot; C' est une image formidable , non ? Bon . Et alors , C' est très matérialiste , au fond , au sens philosophique du terme , hein , pas les biens matériels . Justement , il dit quelque chose , hein , 1581 . On a trouvé ça tout à l' heure , on a été trop contents . Puisqu'il y avait les deux cohérences , est -ce qu' on allait trouver dans Montaigne les deux cohérences ? Et ben , ouais , on a trouvé . Voilà ce que dit Montaigne : &quot; Les biens de la fortune ... &quot; - ça existe , hein , les biens de la fortune - &quot; encore faut -il avoir du sentiment pour les savourer . &quot; &quot; C' est d' en jouir , et non de les posséder , qui nous rend heureux . &quot; Autrement dit , la vie ne vaut que par ce qu' on en fait . Et de toutes les satisfactions qu' on peut tirer de l' existence , la première est d'abord de savoir le faire . Et de vouloir le faire . Et comment ? Entrez en vous-mêmes . Quand même , ça fait des drôles de meetings , hein , qui se terminent par Montaigne ... Voilà ce que dit Montaigne : &quot; Qui se connait , connait aussi les autres , car chaque être porte la forme entière de l' humaine condition . &quot; Ça , il dit ça au XVIème siècle . Et nous autres , les mauvaises têtes , 4 siècles à se battre avec tout le monde pour défendre cette idée . Que les êtres humains sont semblables . Et pendant combien de générations : &quot; Hé , mais qu' est -ce que vous racontez ? Ça n' existe pas ce que vous dites . &quot; &quot; Il y a des grands , il y a des petits , il y a des gros , il y a des maigres , il y a des hommes , il y a des femmes , des intelligents , des très bêtes , il n' y en a pas deux pareils . &quot; Les qualités , ça n' existe pas ! &quot; Vous êtes fou , M. Mélenchon ? &quot; Pascal ? &quot; Qu' est -ce que vous racontez ? Si vous faites un régime politique fondé sur l' idée de l' égalité , vous irez contre la nature , et par conséquent , tous vos régimes politiques égalitaires seront des violences . &quot; Voilà ce qu' ils nous ont répété pendant 3 siècles . Et à la fin , il aura fallu que tout le monde comprenne que comme l' unique écosystème compatible avec la vie humaine est en danger , alors vous avez la démonstration du fait que nous sommes tous semblables , et égaux par nos besoins et notre humaine condition , comme dit Montaigne ! Il dit aussi : &quot; La vraie liberté est de pouvoir toutes choses sur soi . &quot; Disposer de soi , c' est être émancipé . C'est-à-dire qu' il n' y a plus la main du mancipium sur votre tête , qui décide à votre place , qui vous dit , à vous les femmes , que votre corps ne vous appartient pas , mais appartient à vos hommes ou à vos enfants , ou à la société . La main qui vous dit : &quot; Vous ne disposez pas du droit de choisir pour vous-mêmes le moment d' éteindre la lumière &quot; , car ce droit appartient à la société , ou au médecin , ou à je ne sais qui , ou au prêtre . Le mancipium , c' est tout ce qui va vous empêcher d' être maître de vous en toutes circonstances , c' est la main qui vous dit : &quot; Tu ne pourras pas faire ceci , tu ne pourras pas faire cela . &quot; &quot; Tu ne pourras pas boire . Tu ne pourras pas manger . Tu ne pourras pas t' éclairer . Tu ne pourras pas te déplacer , parce que tu n' en auras pas les moyens , parce que je te les ai retirés , pour que quelques uns en aient plein , et encore davantage , jusqu' au jour où ça finira par ruisseler sur toi . &quot; Nous voici revenus à notre point de départ . Le fondamental de ce que nous défendons , dans le programme &quot; L ' Avenir en Commun &quot; , c' est la liberté . La liberté de soi sur soi . La pleine disposition de soi-même , raison pour laquelle nous mettrons dans la Constitution , si nous avons la possibilité d' avoir notre Assemblée Constituante , parce que nous serons majoritaires aux prochaines élections , nous y mettrons le droit à l' avortement , nous y mettrons le droit d' éteindre la lumière soi-même . Pour qu' on cesse ... à chaque élection de discuter ces droits fondamentaux aux personnes . Car vous savez que , de nouveau , ces questions sont posées . Guérissez -vous de tout ça avec une bonne Constitution . Et une bonne nouvelle Constitution , c' est un peuple mobilisé pour elle . Bon . La dernière citation , et après , c' est fini . Comment on va faire tout ça ? &quot; Pourquoi est -ce que vous n' allez pas à la primaire , M. Mélenchon ? &quot; Ah , parce que vous savez , il y a une martingale : &quot; Si vous allez à la primaire , ça y est , vous avez gagné . &quot; Et d'ailleurs , vous l' avez gagné aussi , la primaire . Oui , mais si je ne la gagne pas ? Parce que qui va venir ? Ce n' est pas une solution , cette histoire . La solution , la seule qu' on aie , c' est de convaincre . Je ne suis pas un batteur d' estrade . Je ne vais pas commencer , à mon âge , à faire une carrière . C' est fait . Vous avez compris ? J' ai tout eu . Ça va pour moi . J' ai beaucoup été aimé , j' aime beaucoup , tout va bien . Ce n' est pas le sujet . Le sujet , c' est celui qui vous occupe , qui fait que vous venez dans une salle comme ça ce soir écouter un type qui parle pendant deux heures . C' est la grande ambition qu' on a pour tous , pour tout le monde . Et la pensée qu' on a pour ceux qui ne sont plus en état d' être parmi nous pour faire les choses , parce qu' ils ont été privé de tout . Voilà ce qui nous met en mouvement . Donc ... quoi qu' il en coûte , quelle que soit la difficulté , de toute façon , nous irons . Il y aura des moments étranges . Où , tout d' un coup , les choses auront - donneront l' impression de s' organiser totalement en dehors de nous . Et puis , on en reviendra aux idées . Ne croyez pas que vous allez convaincre des millions de Français avec une combine . Ne croyez pas que c' est en allant participer à une vaine agitation qui va durer trois semaines , où les gens vont jeter des noms d' oiseaux et regarder dans les sondages qui est -ce qui a la plus grande chance d' être le moindre mal , dont je vous ai parlé au début , que c' est avec ça que vous allez convaincre des millions de gens qui n' en peuvent plus . Quelqu'un a dit : &quot; Il faut revenir à la normale . &quot; Non , c' est la normale qui est insupportable . Il faut en venir à autre chose . Il faut passer à autre chose . Et cet autre chose est grand , est beau . Plein de gens sont prêts à le faire . Nous avons besoin de toutes les intelligences pour le faire . Alors arrêtez de pleurnicher . Regardez -les , tous ceux qui sont là : &quot; Ah oui , mais si vous continuez comme ça , on va perdre . &quot; Non ! C' est si on continue comme ça qu' on va gagner . Si vous êtes convaincus , si je vous ai convaincus , et peut-être que vous ne serez convaincus que demain , en y réfléchissant , parce que là , ça fait beaucoup de paroles , alors , il y a les petites choses à faire . C' est une grande tâche , chacun prend sa part de l' effort . Moi , je fais ce travail là . Et d' autres . Pourquoi ? Quelle est ma force ? J' ai bossé sérieusement , comme vous tous . Ma force , c' est que c' est la dernière fois il y a déjà eu 4 millions de personnes qui se sont rassemblées . Cette force , je la tire de ce rassemblement . C' est ce qui me donne de l' autorité , de la puissance , de l' obligation de me respecter , pour nos adversaires . Mais ma force , c' est celle qu' on a tous ensemble . Je ne dis pas : &quot; Venez voter pour moi . &quot; Venez voter avec moi . Mon message , ce n' est pas ma personne . Mon objectif , ce n' est pas ma personne . C' est ça . Et le maire de Bordeaux conclut - le maire de Bordeaux , Montaigne , hein . &quot; Les choses ne sont pas si douloureuses , ni si difficiles d' elles-mêmes , mais notre faiblesse et lâcheté les font telles . &quot; Il n' y a ici ni faibles , ni lâches . Bon courage . Non . Ne commencez pas avec les mauvaises manières , hein . Ah , non non non , on fait une campagne , vous ne crierez jamais mon nom . Criez des slogans , mais pas mon nom , on mérite mieux que ça , même moi . Hein , allez . ",
    "Est -ce que ça marche , là ? On entend ma voix ? Vous entendez ma voix ou pas , là ? Arrêtez cette musique , là . Il faut bien qu' on entende ma voix , un peu . Je vais rentrer ensuite dans la salle , mais ... Vous voyez . Ça va , là , le son ? Ça va se régler , ne vous inquiétez pas . Mais avant , je voulais quand même vous dire un petit bonjour , à vous qui êtes dehors et qui êtes si aimables d' avoir la patience de rester là . Asseyez -vous tous , hein , vous savez , personne ne vous en voudra , ça fait un peu ... Regardez , ils l' ont fait , là , devant . Enfin , faîtes comme vous voulez . Merci d' être là , parce que , à votre nombre , on va voir qu' il se passe quelque chose parmi vous , de quoi je suis le point d' appui , davantage que l' aboutissement . Si vous êtes là , ce n' est pas pour moi . C' est pour les idées que nous portons en commun , et je ne l' oublie pas un seul instant . Vous voyez , venir en Martinique , ce n' est pas &quot; découvrir &quot; pour moi . Il aura fallu hier que l' on révèle , ou qu' on rappelle , que mon pauvre père , paix à ses cendres , a été receveur de la Poste par ici . Et puis , je suis déjà venu quelques fois . Il n' y a rien que je ne sache déjà des plaies qui vous blessent . Mais , malgré tout et par-dessus tout , chaque fois , je pense à l' extrême sensibilité des populations qui sont ici , que parfois , on ne comprend pas depuis la métropole , où quelques élites ont parfois des paroles dont ils ne mesurent pas à quel point elles sont désinvoltes et mal reçues , même s' ils s' en excusent ensuite . Pour moi , le littéraire que je suis s' avance ici toujours les mains tremblantes . Je suis sur la terre d' Aimé Césaire , d' Edouard Glissant , de Patrick Chamoiseau , mais quel département de la patrie contient autant de trésors de notre littérature ? Je voulais vous le dire , chacun d' entre nous réagit à sa manière aux circonstances de la vie . Et bien , moi , c' est ma manière de le faire . Je remercie ceux qui ont préparé ma venue cette fois -ci . Et pour les remercier tous , ces 300 personnes qui m' ont accordé leur appui lorsque j' ai proposé ma candidature , je vais saluer d'abord Thierry Renard et Marie-Nathalie Ravin qui m' ont accompagné pendant ces deux jours , et , en les saluant , eux , je saluerai tous ceux qui me soutiennent dans ce voyage . Les élus , qui m' ont fait l' honneur de me recevoir le meilleur de la tradition républicaine que je ne voudrais pas compromettre par mes saluts ; le président de la collectivité territoriale de la Martinique , Alfred Marie-Jeanne , mon ami , parce que , quelque soit son parcours politique et le mien , c' est un ami , Claude Lise , président de l' Assemblée , Monsieur le maire de Fort-de-France , Didier Laguerre , qui m' a reçu . Pour saluer la population de cette commune , évidemment , je salue son élu . Puis après tout , et d' une façon générale , les commerçants qui ont voulu , hier , m' accueillir d' une manière assez chaleureuse . Un grand merci à vous , M. le directeur - ou je ne sais plus comment ... quel est son titre - M. Collair ( ? ) . Et puis , ce matin , ce long moment magnifique passé avec le groupement régional des agriculteurs bio de qui je vous invite à vous rapprocher dès que vous le pourrez , et à aider de toutes vos forces , car c' est eux qui dessinent l' avenir , en tout cas , tel que moi , je l' imagine . Et bien sûr , à son secrétaire général , Claude Ducalcon qui a passé un grand moment avec ses autres amis ce matin . Je suis venu ici , parce que , comme vous le savez , je vous propose ma candidature à la présidence de la République Française . C' est un exercice un peu compliqué que d' aller au-devant d' une population qui s' auto-administre assez largement , pour faire des propositions qui concernent l' ensemble du territoire . Je vais vous le faire , et je souhaite que vous l' entendiez sans que cela porte jugement de ceci ou de cela qui se fait ou ne se fait pas , ici . C' est à vous d' apprécier la distance qui sépare mes propos des réalités dans lesquelles vous vivez . Mais je dois formuler devant vous l' idée d' un projet . Je ne suis pas venu promettre de l' aide . Je suis venu vous appeler à la rescousse . Je suis venu vous enrôler pour une cause plus grande que la vie de chacun d' entre nous , qui concerne le destin de la patrie commune et le destin des Françaises et des Français dans cet environnement particulier des Caraïbes , où ils ont une responsabilité particulière , compte tenu de leur niveau de formation , de développement , confrontés , comparés à tant de misère et à tant d' attentes autour de nous . Même si nous-mêmes , nous en connaissons , je le sais . Mais avant d' entre dans le sujet qui occupe directement la campagne et ce que j' ai à dire pour ici , sachez que je ne suis venu que pour commencer un processus , en effet , après que l' on ait établi le programme &quot; L' Avenir en Commun &quot; , nous avons décidé que chacun des compartiments et des chapitres aurait ses sous-chapitres sous la forme d' un livret , qui détaillerait plus précisément tel ou tel sujet . J' attache la plus grande importance à ces livrets , car quoi qu' il arrive , quel que soit le résultat , cette pensée ramassée , concentrée dans des textes sera notre patrimoine intellectuel commun . Et la génération suivante aura à vérifier ce qui aura été réalisé et ce qui restera à faire . Ce qui aura été démenti par les faits et ce qui aura été confirmé par ce que nous aurons fait . C' est donc d' une très grande importance que la précision de ces textes . Bien . J' ai donc ouvert sur le site jlm2017 une page où l' on peut discuter d' un document car je juge qu' il est bien démagogique de se présenter en disant : &quot; Et bien , allez , que chacun dise ce qu' il a sur le cœur . &quot; Mais si c' est pour ça , vous n' avez pas besoin de moi . Il fallait bien qu' on ait préparé , il fallait bien qu' on ait encadré la réflexion , et donc le document de travail est en ligne . Mais qu' auriez -vous dit de moi , si , depuis la métropole , je vous avais harangué en vous disant : &quot; Ah , ben , écoutez , j' ai ouvert une page et regardez et voyez , donnez votre avis . &quot; ? Il fallait que je vienne , et donc me voici . Mais je ne jouerai pas la comédie de dire qu' en 48 heures ou en 3 jours ou en 4 jours , on rencontre une population , ce n' est pas vrai . Je viens donc commencer un travail . Qui se continuera partout , où qu' on soit , comme je viens de vous le dire . Maintenant , j' entre dans l' actualité . Car je ne suis pas venu ici seulement pour vous parler de choses dont vous connaissez déjà une partie . Je veux en parler de l' actualité , parce que voici tantôt , 48 heures ou trois jours , que l' on me cherche pouilles à propos de la Syrie , et qu' on va , répétant de tous côtés que ma position est ambiguë . Mais elle ne l' est d' aucune façon . Je suis hostile au parti de la guerre . Et le parti de la guerre s' est constitué en Europe et en France à travers un soutien aveuglé à ce que j' appelle le &quot; campisme &quot; . Il y a une situation , alors chacun choisit un camp . Et bien ce n' est pas mon avis . Mon avis est que je suis du camp de la France , et la France , son seul intérêt , c' est la paix . Nous ne participons d' aucune coalition pour dépouiller la Syrie , ici d' un plateau , là-bas d' un rebord ou , là encore , d' une plaine . Ma position est parfaitement claire . Je m' oppose au parti de la guerre qui est en train de monter en puissance , dans toute l' Europe , à l' instigation des Etats-Unis d' Amérique , qui ont , en commençant par l' Ukraine et en continuant ailleurs , ourdi toutes sortes d' agitations pour subjuguer les esprits ; et croyez que je n' exagère pas en vous parlant comme je suis en train de le faire . Si vous voyiez les résolutions qui se discutent au Parlement Européen , vous seriez effrayés du ton qui est pris . De cette espèce de manière de revivre une guerre froide qui a cessé depuis bien longtemps et dont on voit bien que , si elle devait recommencer , ce serait une guerre chaude . Mes chers compatriotes , la paix est un bien précieux et il faut lutter pour elle . J' ai assez visité ce territoire pour savoir quelle a été sa contribution dans le passé aux grandes guerres , qui sont toujours mondiales quand elles s' allument en Europe , qui en porte seule la responsabilité à chacune des étapes de l' Histoire . J' ai vu , dans les villages et les villes , les monuments aux morts en Martinique , de tous ces malheureux que l' on avait emmenés là-bas combattre et y mourir et que leurs pères et mères n' ont jamais revus . La paix est une question sérieuse . Ce n' est pas un état de nature . Nous devons résister lorsque la guerre s' avance avec lucidité , avec sang-froid et refuser les pressions qui s' exercent sur nous . On voudrait tous les jours me faire condamner tel bombardement plutôt que tel autre . Je ne le ferai pas . Je continuerai à tenir ma position qui est de dire que l' on ne peut venir à bout de l' ennemi qu' on affronte et à qui nous pouvons imputer les morts du Bataclan , de l' assassinat de Charlie Hebdo , de l' épicerie casher et ainsi de suite ; que nous avons à régler ce compte là , mais que nous ne le ferons pas au prix d' une guerre qui embrase toute une région . Je ne choisis pas mes bombardements . Je déteste les bombardements , que ce soient ceux d' Alep , ceux du Yemen ou ceux de Mossoul , car toujours ce sont les civils qui payent et qui sont en état total d' impuissance , ayant à subir toutes sortes d' horreurs ; celles que leur infligent les occupants qui arrivent puis quand ils s' en vont , et les nouveaux qui arrivent . Nous savons tous que c' est cela , la guerre . Il n' y a pas de guerre propre . Je le dis avec toute cette force car vous aurez , vous autres , un rôle à jouer dans l' idée que je me fais de la réorganisation de la politique internationale de la France . Je ne veux plus que les Français soient le dernier wagon du train des Américains dans l' Organisation du Traité de l' Atlantique Nord que l' on appelle l' OTAN . Nous en sortirons , et nous en sortirons d'autant plus vivement que je vois bien comment , à travers l' installation de batteries anti-missiles en Pologne , à travers toutes ces faveurs que l' on déverse sur des pays qui ont des comptes inextinguibles à régler avec la Russie , on est en train de préparer sur le Vieux Continent un nouvel embrasement . Je n' ai aucune complaisance pour des gouvernements comme ceux de l' Ukraine , peuplés de racistes , peuplés de néo-nazis qui ont le front de porter encore les étendards que l' on a vus sur nos occupants . Voilà ce que j' ai à dire avec toute ma force . Nous sortirons de cette alliance militaire . Nous reconquerrons instantanément notre indépendance totale dans tous les domaines qui concernent la sécurité de la patrie et la manière d' aborder les voisinages qui sont les nôtres , dans les Caraïbes et , plus &quot; prochement &quot; ( sic ) , l' Amérique latine . On sortira le matériel américain du matériel français . Microsoft n' a rien à faire au siège de l' état-major de l' armée française . Ni aucun de ces logiciels dans aucun des armements français . En reconquérant notre indépendance , nous chercherons à nous lier à d' autres peuples , et c' est là que vous avez un rôle à jouer . Pas celui de je ne sais quelle improbable vitrine , mais d' un rôle actif . Nous devons entrer dans les coalitions et les regroupements de pays des Caraïbes et de l' Amérique latine . Car jamais on ne pourra oublier , et jamais il ne faudra cesser de répéter que la plus longue frontière de la France est avec le Brésil et pas avec l' Allemagne , et encore moins avec la Lituanie . Si bien que nous avons à voir avec l' Alba et avec Mercosur puisque nous en sommes voisins directement . Voilà comment je me représente ça et vous aurez à jouer ce rôle . Car mon sentiment , et je vous demande de m' écouter avec soin , est qu' une telle re-disposition des forces de la France , une telle vision du futur qui nous extrait de ce petit enfermement européen , qui fait de nous une petite nation occidentale que nous ne sommes pas , doit cesser . Nous ne sommes pas une nation occidentale . Nous sommes une nation universaliste . Nous sommes présents sur les 5 continents , nous avons toutes les couleurs , toutes les religions , et nous partageons notre langue , qui ne nous définit pas , avec 29 autres pays . La seule chose que nous ayons en commun , et qui est le plus fort , qui est la dynamique qui anime notre peuple , c' est de sa devise dont , parfois , et même souvent , et en particulier ici , nous n' avons pas toujours été à la hauteur . Que la France claque comme un message et un drapeau : Liberté , Egalité , Fraternité , mais jusqu' au bout et pour de vrai . Non , vous n' êtes pas ... ce que l' Europe appelle les régions &quot; ultra-périphériques &quot; . Ce qui est ultra-périphérique , c' est plutôt Vilnius que Fort-de-France pour des Français . Et si je le dis , c' est parce que c' est tout le regard qu' il faut changer . Les îles , voyez -vous , les îles françaises , je les pratique déjà depuis quelque temps . Combien , parmi vos élus et vos dirigeants , peuvent dire qu' ils sont allés en Nouvelle-Calédonie française ? En Kanaky ? Avec ces terribles 11h de décalage qui fait que vous arrivez pantelant , et rentrez chez vous de même . Les îles ne sont pas des territoires exotiques . Les îles sont des loupes sur nous-mêmes . Les îles font voir en gros tous nos défauts et toutes nos qualités . Et parce que ce sont des îles , quand elles cherchent à régler leurs problèmes , alors elles ouvrent des voies pour le pays tout entier . Ah , si la Corrèze pouvait imaginer qu' elle est entourée d' eau comme la Martinique , peut-être qu' elle réfléchirait autrement à son développement . Peut-être qu' elle se poserait les questions que vous avez à vous poser et à régler sur lesquelles , maintenant , je vais venir . La Martinique , la Guadeloupe , les autres territoires ont à être les territoires pilotes du nouveau modèle économique que je propose . Je viens donc vous demander de m' aider à marcher en pointe sur les questions de la planification écologique , comme je viens de vous le faire , sous vos applaudissements , pour la question de l' indépendance de la France , et pour le changement institutionnel qu' il est nécessaire d' apporter . Avec votre permission , mesdames et messieurs , qui m' écoutez ici dehors , je vais rentrer pour que nos amis ne se mettent pas un torticolis terrible . Ils sont du côté de la porte et me regardent , tandis que les autres me regardent sur l' écran . Je reviendrai tout à l' heure pour vous saluer si vous y êtes encore . Au fond ... je ne répète pas ce que je viens de dire , vous y étiez , hein . Bonjour à Bernard Teper que j' aperçois là , au milieu , bonjour vous ... ouh la la , mais que d' amis ici . Les vacances sont commencées , j' ai l' impression . Vous avez bien de la chance . Non , vous , vous n' êtes pas en vacances . Vous bossez là . Au fond , vous allez entendre beaucoup de choses pendant cette campagne , c' est bien normal . Ayez la patience d' écouter . Où est Mme Fanon ? Ah , bonjour , madame , je voulais vous saluer . Et bien , ayez la patience d' écouter tout ce qui vous est dit . Parce que c' est une élection qui a une très grande importance . Peut-être que le territoire de la métropole peut se dire qu' il a encore la force nécessaire et suffisante pour résister à un choc libéral de plus . Car c' est ce qui se prépare . Vous avez vu que M. Fillon est une synthèse - honnêtement avouée , mais au moins on peut lui reconnaître ce mérite - de ce qu' il y a dorénavant comme portrait de la droite dans notre pays . C' est un libéral absolu en matière économique , et même d' avant-poste , et un conservateur total , réactionnaire sur le plan des mœurs et des libertés publiques . Vous avez vu comment il court devant . J' espère que vous l' avez bien compris , lorsqu' il se propose de supprimer la durée légale du travail , ouvrant ainsi la possibilité qu' elle s' inscrive dans les durées prévues par l' Europe , comme un maximum de 48 heures , mais ce n' est pas la seule chose qui change avec lui . C' est que , de ce fait , les heures supplémentaires ne seront plus des heures supplémentaires . Il faut une certaine hypocrisie à d' aucuns pour aller protester contre ça . Je vois les nouveaux effarouchés qui s' en plaignent . Mais enfin , qui a voté dans la loi El Khomri que , dorénavant , les heures supplémentaires pourraient ne plus se payer que 10 % si on en convient dans l' entreprise , où , comme chacun d' entre nous le sait , règne une ambiance amicale où l' on discute avec son patron , très tranquillement , de sa paie . Et bien , les malotrus comme moi , qui ne croient pas à cette fable , trouvent qu' il est tout à fait condamnable d' avoir désarmé de cette façon les salariés , et il y a une grande inquiétude , ici aussi . Je ne vois pas pourquoi ici . Non ? Hier , j' étais à un restaurant et je discute avec les cuistots . En général , j' aime bien aller voir la cuisine , parce que le métier m' intéresse et comment font les gens , et bien de quoi m' ont -ils parlé ? On n' a pas parlé de cuisine , j' aime mieux vous le dire . Ils m' ont parlé de ça . Des heures de travail , de la Sécu . Alors là , c' est le concours en ce moment , les voilà tous repeints en médecins . M. Fillon commence à dire que les petites maladies ne seraient plus indemnisées , mais quelles petites maladies ? Qui est -ce qui va le dire ? Lui ? Les autres têtes d' œuf de son cabinet ? Il ne sait pas , cet homme , qu' une petite maladie peut devenir grande ? Il n' a jamais vu autour de lui d' aucuns pâtir ? Savez -vous , qu' à mon âge , on peut mourir d' une grippe ? Mais pas au vôtre , madame . Mais oui , ça n' existe pas , les petites maladies , et ce n' est pas la logique de la politique de santé que développe une nation développée . Tout le monde doit être soigné . Et chacun contribue au financement de ces soins selon ses moyens et reçoit selon ses besoins . C' est une invention magique , extraordinaire , ça s' appelle la Sécurité Sociale et c' est un modèle qu' on pourrait d'ailleurs étendre à bien d' autres domaines , au point que , pour ma part , je suis partisan d' en systématiser l' application . Et par conséquent , il va de soi qu' il faut être en accord avec l' idée que porte la Sécurité Sociale . Vous recevez parce que vous avez cotisé et vous cotisez parce que vous pensez un jour avoir besoin , et vous acceptez que , dans l' intervalle , ce que vous avez donné serve à l' autre . Vous n' êtes pas fous . Vous savez bien que si vous allez au travail et que celui d' à côté de vous est mort de maladie , il y a une forte probabilité pour qu' après , vous le soyez aussi . Donc , vous , vous avez compris ce que Mme Le Pen n' a toujours pas compris . Les microbes ne savent rien de qui est avec vous . Ni de sa nationalité , ni de sa religion , ni de rien du tout . C' est la raison pour laquelle il faut maintenir l' aide médicale d' Etat , de manière à ce que tous ceux qui sont malades soient soignés , non seulement parce que nous sommes des êtres humains et que le devoir d' humanité nous oblige à nous occuper des autres , et c' est d'ailleurs pourquoi nous sommes dans cette salle , tous là , autant qu' on est , si différents qu' on est , c' est parce que nous sommes les héritiers de ceux qui se sont soucié des leurs . Et les autres ont disparu . Et évidemment , le soin des autres est le cœur de la pensée progressiste qui anime l' idée de la Sécurité Sociale et qui continue de nous animer aujourd'hui . Donc vous cotisez parce que vous attendez que , bon , très bien . Alors , il parait que M. Macron veut reporter une partie de ces cotisations sur la CSG . C' est un peu compliqué , excusez -moi , mais il faut bien que j' aborde des sujets de fond . Hé , mais ça ne va pas du tout , ça . Il ne faut pas faire ça . Pourquoi ? Pour deux raisons : la première , c' est que vous allez demander à un retraité de cotiser pour quelque chose dont il ne connait pas le risque . Si vous cotisez par la CSG , quand vous êtes retraité , au chômage , vous cotisez pour quelque chose dont vous ne courrez pas le risque , puisque vous êtes à la retraite . C' est absurde . Et surtout , si vous sortez toutes les cotisations pour les mettre dans la CSG , il ne reste plus que les patrons , et donc on aura réussi le tour de force , avec quelqu'un qui prétend appartenir à la famille progressiste , de fiche dehors de la gestion de la Sécurité Sociale ceux qui l' ont créée , à savoir les salariés . Car je rappelle que si les patrons y sont , c' est par effraction , parce qu' ils n' y étaient pas au départ , à la Libération , c' est le général De Gaulle qui les a fait entrer dans la gestion de la Sécurité Sociale . Je sais que ces questions sont un peu complexes à aborder , mais il faut les regarder , parce que sinon , vous vous faites dépouiller . On vous dit : &quot; Regardez , c' est beau , c' est nouveau , c' est jeune , c' est formidable . &quot; Bon , c' est bien . C' est vrai tout ça . Sauf que là , c' est un piège mortel qui vous est tendu , par l' un comme par l' autre . Au fond , comme dit ... Qu' est -ce qu' ils ont en commun , comme dit l' adage ici , hein : &quot; Même bête , même poil . &quot; Ce sont tous des libéraux . Ils ne croient pas qu' on puisse organiser le monde autrement . Alors pour les libéraux , qu' est -ce que vous êtes , la Martinique ? Vous êtes au bout d' un gros tuyau , et dans ce tuyau , on met les marchandises qui arrivent d' Europe et on les déverse , et vous autres , et ben vous avez comme métier promis rien du tout , ça , c' est déjà le cas pour une bonne partie de la population qui est au chômage , sinon , comme on dit : les &quot; activités de service . &quot; Alors , bon , c' est très joli à entendre , sauf que c' est tout simplement insupportable dans la durée . Mais je vais vous montrer ça plus en ... un peu plus en détail . En attendant , voilà leur idée . On produit n' importe quoi , n' importe comment , le moins cher possible , on le fait passer par ici , on vote des accords avec les pays frontaliers qui sont en face de nous qui fait baisser les droits de douane , on abaisse les droits de douane ici , et en avant ! C' est parti . Arrivent les bananes dans un sens , arrivent de votre côté les voitures et je ne sais pas quoi . C' est ça leur modèle économique . Et vous ... vous ferez bien ce que vous voulez . Vous voterez bien comme vous voulez , ça va de soi . Mais dites -vous bien que c' est ça qu' ils ont préparé pour vous . C' est ça qu' ils veulent , et c' est ça qu' ils mettent en oeuvre . Ça s' appelle le libre-échange et ça part de l' idée que si l' on laisse les marchandises circuler , tout va bien . Ça , c' est une théorie du XVIIIème siècle . Alors , à l' époque , on pouvait en effet penser que telle marchandise n' arriverait valablement que venant de tel endroit , mais qu' est -ce que ça peut bien vouloir dire aujourd'hui ? Quelle est la différence entre un composant électronique produit en Chine et un autre produit ici ? Car vous pourriez en produire , vous aussi , des composants électroniques . Comment ça se fait qu' il n' y a pas d' usine de composants électroniques ici ? Il y a une malédiction ? Ça ne va pas être le climat , c' est quoi le problème ? Pourquoi les industries de pointe ne sont -elles pas ici ? On peut se poser la question . Pourtant , il y en a qui se gavent ; je vais bientôt leur régler leur sort , mais je reste sur cette histoire . Leur idée ... quelle est leur différence , je vais vous la dire : c' est la différence de salaire et la différence du fait que dans tous ces pays , on ne respecte aucune norme environnementale . Voilà la différence qui fait donc que c' est moins cher . Rappelez -vous bien de ça : moins cher , c' est toujours cher pour quelqu'un au bout de la chaîne . Ou la nature ou quelqu'un qui a travaillé et qui a été traité comme un misérable et d' une manière indigne . Ça , c' est leur vision . Alors voilà l' avenir pour vous . Les marchandises arrivent , vous en achetez , et surtout elles passent pour aller ailleurs , et puis pour le reste , on saupoudre amplement pour que le bétail puisse faire de l' engraissement fiscal . Alors on donne des avantages fiscaux à qui investit dans l' outre-mer . Fort bien . Comme toujours , ils pensent que la cupidité est le principal moteur de l' activité humaine . Et par conséquent , les gens cupides , prenant leur argent parce qu' ils en ont , se précipitent en outre-mer pour le placer . Enfin , vous les avez vus , non ? Vous ne les avez pas vus ? Ça m' étonne . Cette affaire vous coûte un milliard tous les ans . Où il est le milliard ? Je suis bien étonné en venant ici de voir qu' il y a autant de gens au chômage . Alors que normalement , les investissements devraient être extraordinaires puisqu'ils sont dégrevés . Ils ont investi dans quoi , au juste ? Voilà . Voilà la question à vous poser . Donc vous pensez que si vous faites une faveur fiscale , les gens vont prendre les bonnes décisions pour l' intérêt général ? Et ben non , c' est tout le contraire qui se produit . Et ce n' est pas moi qui le dit , parce que vous pourriez penser que je suis mauvaise langue et de parti-pris . Je le suis , de parti-pris , en effet . Et j' essaye que ma langue pique assez pour qu' on s' en souvienne . Mais là , c' est la Cour des Comptes . Elle dit : &quot; Le coût de ces défiscalisations apparaît ainsi disproportionné par rapport à celui d' autres modes d' intervention de l' Etat . Car au financement porté à l' outre-mer , s' ajoute la part importante conservée par les contribuables bénéficiaires de l' avantage fiscal qui ne profite pas à l' outre-mer . &quot; Je reconnais que la phrase est un peu longue , alors je vous la recommence . Quelqu'un qui investit ici voit retiré - par exemple pour s' acheter un bateau , qu' il pourrait louer , ah , c' est de l' activité , ça - et bien , un avantage fiscal correspondant à une part de la valeur du bateau qu' il a acheté . Et cette part fiscale , il n' est absolument pas obligé de la réinvestir ici , il se le garde pour là-bas . C' est pourquoi ça coûte si cher et c' est si peu profitable . &quot; Aucune des diverses tentatives d' évaluation &quot; , dit la Cour des Comptes , &quot; n' a pu conclure à l' efficacité de ces aides . &quot; Hein ? C' est nous qui sommes les gâcheurs , les dépensiers , blabla blablabla , compétitivité , flexibilité , vous connaissez . &quot; Aucune des diverses tentatives d' évaluation n' a pu conclure à l' efficacité de ces aides . En revanche , plusieurs ont montré des effets contraires aux résultats recherchés . &quot; Vous entendez , les gens ? Tout ce gâchis , la Cour des Comptes dit : &quot; On est arrivés au résultat exactement inverse . &quot; &quot; Ou bien le manque d' efficacité . Ces aides ne peuvent être allouées de façon optimale qu' aux investissements les plus utiles au développement de l' outre-mer car elles vont d'abord aux investissements les moins risqués et les plus rentables . Ces aides profitent également à des entreprises qui n' en ont pas besoin pour investir . &quot; 2 milliards . Je voulais vous le dire avant de passer au reste pour que ... Essayez de contrebattre en vous , par des antidotes , le réflexe que vous aurait suggéré une fréquentation trop longue des médias traditionnels . Car , dorénavant , quoi qu' on dise , et avant même qu' on ait fini la phrase , la réplique est : &quot; Mais comment vous payez ça ? &quot; Je pourrais répondre un peu grossièrement : &quot; Je paye en leur prenant ça . &quot; Mais ça fait beaucoup d' argent . Mais ce n' est pas ça que je veux faire . Je voudrais vous appeler à réfléchir sur la place de l' Etat , les collectivités humaines . L' Etat , au sens large , les collectivités , au sens inclus des collectivités locales , qui souvent , étant très proches du terrain , sont les mieux avisées pour percevoir les besoins . L' Etat , le collectif , la société , les besoins communs , toutes ces paroles que l' on a reléguées au profit d' une croyance absurde , à une sorte de génie spontané de la main invisible du marché qui réglerait tous les problèmes . Nous avons donc vocation , et nous pouvons démontrer que l' intervention publique et la règle commune ont plus d' efficacité économique , sociale et écologique que cette distribution irresponsable d' argent sur des gens qui sont motivés par leur seule cupidité . C' est pourquoi j' ai dit , il y a un instant , je voudrais que les collectivités , les territoires de l' outre-mer français , des outre-mer , car ils sont si divers - il est difficile de ranger dans la même catégorie la Polynésie , la Kanaky Calédonie française , la Martinique , la Réunion - ce sont tant d' histoires différentes , tant de contextes différents , mais enfin , pour ce qui nous occupe , il y a deux domaines dans lesquels vous pourriez jouer le rôle de pilote parce que nulle part autant que dans une île , on ne comprend le message . Ce sont les messages de notre époque . Mes chers concitoyens , on ne peut pas continuer avec le mode de production dans lequel nous sommes car il va détruire la planète . On ne peut pas continuer . Il faut donc changer le mode de production , les process de production et avant cela même , il faut changer la manière de s' approvisionner en énergies , sans lesquelles on ne change rien . L' énerige et l' accès à l' énergie est premier . Ici , je ne vais pas évoquer la question du nucléaire puisqu'elle ne touche pas directement ce territoire , vous avez bien de la chance . Mais il n' est pas possible que l' on continue à recourir aux énergies fossiles comme on le fait dans les outre-mer , et en particulier à la Martinique . Que 93 % de la ressource énergétique ici soit du pétrole est une aberration . Une aberration . Pourquoi ? Parce que vous avez devant vos yeux un potentiel qui contient 75 fois le potentiel d' énergie dont vous avez besoin sur Terre : c' est la mer . Mes amis , vous ne pouvez pas ... Vous ne pourrez pas contourner cette ambition . Ecoutez celui qui vous parle et qui porte à cet instant ce que vous avez en vous-mêmes . Car je sais très bien que je suis d'abord votre porte-parole . Je ne vous apprends peut-être rien , mais que notre réunion porte un message pour tous ceux qui ne s' y trouvent pas . Il faudra faire ce tournant . L' économie de la mer nous place en position de puissance . Je vous ai décrit quel est le modèle auquel les autres croient : faire de vous le hub commercial de l' Europe et le territoire d' engraissement fiscal . Moi , je vous appelle à autre chose . Ils vous demandent vos plages et les ports d' attache pour leurs bateaux ; je vous demande des centaines d' ingénieurs , techniciens , techniciennes , ouvrières et ouvriers de haut niveau , qualifiés , dont la patrie a besoin pour faire le tournant vers l' économie de la mer . Vous devrez être le premier territoire dans lequel on forme tout ce monde dont on a besoin partout sur le territoire de la métropole et dans toute la Caraïbe . Car si ... nous étions capables ... et nous pouvons le faire en peu de temps , un bac professionnel , c' est 3 ans ou 4 ans . Si je suis élu , j' aurais peut-être le temps de voir ça . Les diplômes , licences professionnelles , bon , c' est 3 ans après le bac . On peut faire beaucoup , vite et bien , si on s' organise , c' est pourquoi je parle de planification . On peut . Je sais que ça heurte beaucoup de conservatisme intellectuel , ce sont les pires . Je connais l' élite intellectuelle martiniquaise , j' ai été ministre de l' enseignement professionnel . Je me souviens de ce lycée où je suis allé voir comment on avait reconstitué la possibilité de faire des gommiers . C' est moi qui ai fait ça . Et oui . Je me rappelle quand je suis venu ici la dernière fois comme ministre , j' avais dans les mains un BTS que j' ai mis dans un lycée professionnel et qui ... qui a fêté récemment son 15ème anniversaire . Et pourtant , on m' avait dit qu' on ne pouvait pas faire de BTS , ici . On l' a fait . Ils sont mauvais , ceux qui sortent de là ? Non , ils sont bons . Donc , vous pouvez . Et on en a besoin . Et on en a besoin , non seulement pour nous-mêmes , mais pour toute la région . Car nous avons des devoirs . Comme nous sommes les plus instruits , comme nous sommes , sans doute , parmi les plus organisés , à l' exception , peut-être , de Cuba , nous avons un devoir . Un devoir à l' égard d' Haïti . Compte tenu de ce qu' a été l' Histoire , Haïti ne redécollera jamais sans énergie . Haïti à qui on a infligé cette horreur qui a été de faire payer le dédommagement de la fin de l' esclavage jusqu' à 1982 ! Nous avons des ... un dette . Nous avons une dette . Je dis , nous tous . Evidemment , on n' y était pas , on n' est pas responsables de tout ça , personne ici , mais quand même , la France s' honorerait à montrer qu' elle est capable de venir à la rescousse quand on a besoin d' elle . Vous vous souvenez de toutes ces critiques abominables qu' on a entendues contre Cuba , mais si j' étais un Haïtien , je prie pour être à Cuba , pour que mes enfants n' aient pas besoin de manger de la terre pour avoir ... ne plus avoir mal au ventre de la famine . Voilà la vérité . Nous devons nous développer et être le département pilote pour pouvoir amener à Haïti les machines et les process de production qui leur permettent d' avoir l' accès à une énergie facile et bon marché , notamment celle qui vient de la mer . Ce qui veut dire qu' il faut implanter ici au moins un , et peut-être deux lycées maritimes , alors qu' il y en a zéro ! Ce qui est invraisemblable dans un endroit où la mer est autour de nous . Il faut des licences professionnelles à l' université de la Martinique . Il faut des licences professionnelles qui préparent à installer les hydroliennes , et peut-être mieux , les machines qui font la différence entre l' eau froide qui se trouve au fond et l' eau chaude qui se trouve au-dessus . Mes amis , tout ça , je vais vous dire où je l' ai appris , je ne l' ai pas appris dans le département des Bouches-du-Rhône où il y a également zéro lycée professionnel maritime . Je l' ai appris à la Réunion et je le tiens d' un grand îlien : Paul Vergès , c' est lui , le premier qui a inventé un plan d' autonomie énergétique et qu' il avait préparé pour l' île . Tout ça a été dispersé par une succession hasardeuse de gens qui sont des irresponsables et qui ont tout démantelé alors que Paul avait si bien préparé . Paul joua un grand rôle dans la pensée sur le développement durable , et je veux dire que je tiens en piètre estime ceux qui ne l' ont pas invité à la COP 21 , alors qu' il aurait dû être devant , marchant en tête des Français qui se trouvaient là , parce que c' est lui qui a été le plus loin dans la réflexion , dans la planification de ce qu' est la transition énergétique . Voilà la vérité . Donc on peut le faire . Si on était capable de le faire à la Réunion , on est capable de le faire à la Martinique . Et voilà comment avec cette économie là , la France créera 300.000 emplois . La France en général . Je dis 300.000 emplois parce que je ne veux pas vous paraître trop exagéré . Ce chiffre , c' est celui du cluster maritime . Le cluster maritime , c' est l' ensemble des entreprises qui travaillent dans ce secteur , et où règne une ambiance tout à fait excellente , puisque j' y suis très bien reçu , alors même qu' il y a une série de gens qui ont des raisons particulières de se méfier de moi , je les comprends , compte tenu de ce qu' est mon point de vue politique et le leur . Mais , à un moment , vous le savez comme moi , il y a des circonstances où la passion commune peut créer des espaces de travail communs . Et on y arrive , dans ce domaine . C' est pourquoi j' ai bon espoir . Et vous savez , quand on installe une éolienne en mer - encore faut -il pouvoir le faire , parce qu' on ne peut pas installer une éolienne en mer , à part flottante , à la Réunion , mais ici peut-être sur le plateau continental - ça me fait penser que ça manque d' ingénieurs géographes , ici . Il faudrait qu' on en ait davantage , et en particulier que les navires scientifiques français puissent patrouiller et mieux calculer le prolongement sur la plateforme continentale , parce que , comme vous le savez peut-être , la France a vu son territoire augmenter d' un seul coup de 10 % sans tirer un seul coup de fusil , simplement parce que nous avions les navires qui permettaient l' inspection des plateaux continentaux , nous avons pu présenter à l' ONU notre dossier , montrant que nous avions des droits sur ces territoires . Des droits , ça vaut ce que ça vaut , hein , bon , ben , c' est comme ça . C' est la loi internationale . Et notre territoire s' est accru de 10 % et nous sommes le deuxième territoire maritime du monde , mais on ne va pas raconter d' histoires , ce n' est pas seulement grâce aux 1.000 km de côtes de la France métropolitaine . C' est surtout à cause des outre-mer . Autrement dit , le ressort principal de la puissance pour le futur , il est ici . Je vous demande de ... de le méditer . Parce que ça vous explique pourquoi je tourne mon regard , et je dis : &quot; On ne doit plus considérer les îles comme des régions ultra-périphériques &quot; , pas pour faire plaisir aux îliens , mais pour faire plaisir à la France , et à ce dont elle a besoin collectivement . Voilà pourquoi on peut le faire . Et vous savez , quand on installe des machines pour la mer , et ben , on ne peut pas les emmener en bateau depuis Hong-Kong . On est obligés de les fabriquer sur place , et de les assembler sur place . Et ainsi de suite . Si bien qu' il y a , là , un ressort pour vous tous et pour vos enfants qui ne nécessite pas qu' on vous arrose individuellement d' argent et d' avantages fiscaux dont vous ne sauriez que faire , vu que vous n' avez pas d' argent . Mais , en tout cas , les sommes sont là . Qu' on ne vienne pas me dire qu' un pays qui est capable de distribuer un milliard de dispenses fiscales à des gens qui n' en font rien n' est pas capable de mettre le même milliard pour établir des lycées professionnels , établir des ... des enseignants sur place , et ainsi de suite . Ce n' est pas vrai . On peut le faire , et il n' y a pas besoin de s' évanouir en demandant combien ça va coûter . Parce que , de toute façon , cet argent là , il n' est pas dur à trouver . Le banquier central européen donne chaque mois , chaque mois , 80 milliards d' euros aux banques qui sont en Europe pour qu' elles regarnissent leurs comptes et vous prêtent de l' argent . Elles regarnissent leurs comptes et elles ne vous prêtent rien . Et quand elles vous prêtent , vous avez intérêt à faire attention . L' autre jour , je suis allé au marché , comme vous le savez , parce que vous avez vu le petit film . Et quand je suis arrivé au bout du marché , il y a une dame dont le commerce a brûlé . Et elle a dû tout recommencer , cette femme qui a toute cette charge sur le dos , de sa famille , du reste , enfin , tout ce que vous connaissez de la vie . Et bien , elle a donc emprunté pour pouvoir se ré-équiper . Si elle voulait acheter une voiture , elle emprunte à 2,5 mais pour refaire marcher un restaurant , avec plusieurs employés , et bien elle emprunte à 7,5 . 7,5 ! Voilà . C' est de l' exploitation . La banque qui fait ça , je ne sais pas quel est le type ou la fille qui signe le papier , mais devrait avoir honte . Il devrait avoir honte de prendre une part pareille sur le travail de braves gens qui se dévouent , alors qu' à eux , le banquier central prête à presque 0 . Presque 0 pour eux , 7,5 pour cette femme qui fait vivre toute une famille et qui apporte un service commun . Voilà pourquoi quand je dis que vous n' avez rien à attendre des banques sans leur serrer la gorge , je parle peut-être fort , je parle peut-être pointu , mais je décris cette vérité humaine là . Et les gens , poudrés et parfumés , qui sont si contents d' eux-mêmes et de cette économie flexible , compétitive et le reste , évidemment , regardent tout ça de haut , et me voient moi comme un sauvage . Mais les vrais sauvages , c' est eux . C' est les gens qui sont capables d' asservir les autres de cette façon là . Je vous ai parlé de tout ça , vous voyez la différence . Alors , j' y reviens un instant . Je vous ai parlé d' investissement , de production , de gens à former , à cultiver , Alors on me dit : &quot; Oui , mais bon , l' enseignement professionnel ... &quot; Quoi , l' enseignement professionnel ? Qu' est -ce que vous croyez ? Comme vos enfants n' y sont pas , vous ne savez même pas ce qui s' y pratique , et vous êtes là , piaillant et sautillant , mesdames , messieurs les importants , à vouloir envoyer tout le monde en apprentissage , mais votre gosse n' y va pas , donc vous ne savez pas ce que c' est . Mais les enfants vont souvent en apprentissage parce que la façon avec laquelle on apprend à l' école ne leur convient pas , des fois , mais ils y vont souvent parce qu' il y a une petite paie et que ça permet de faire bouillir la marmite . Voilà la vérité que vous ne savez peut-être pas , ben , je vous la dis , alors . Et si vous ne le savez pas , la moitié de la jeunesse de France est dans l' enseignement professionnel , dans l' enseignement technologique et dans l' apprentissage . Et qu' est -ce que vous croyez qu' on fait , comme ils disent : &quot; Il faut apprendre sur le tas . &quot; Ah oui , vous croyez ça , vous ? Vous croyez qu' on apprend sur le tas . Une machine qui vaut des millions , une machine dans laquelle il faut mettre un programme , vous croyez qu' on apprend sur le tas , et quoi ? Vous allez donner une banane à chaque fois qu' il appuie sur le bon bouton ? Non . &quot; L' enseignement manuel &quot; , disent -ils , qu' est -ce que vous croyez ? Il n' y a pas de mains qui marchent sans un cerveau . Et si vous ne cultivez pas , si vous n' éduquez pas , si vous n' élevez pas , alors ça ne donnera rien , même pour la production la plus triviale . Je vais vous dire , un jour , j' ai vu des gamins , ils ne venaient plus à l' école , à un BEP de métallerie , les professeurs se cassaient la tête , ils ont fini par trouver la solution , parce que les professeurs , c' est toujours des professeurs . Et bien , ils ont organisé un cours qui s' appelait &quot; sculpture métallique géante &quot; , ils ont donc emmené ces jeunes gens voir des expositions d' art abstrait , et ensuite , eux-mêmes se sont mis à dessiner . Un , le dessin , ça fait partie des matières professionnelles , deux , ils ont dû calculer comment ils feraient la sculpture métallique géante , ça , c' est de la physique , et ensuite , il a fallu organiser la soudure des pièces . C' est de la soudure , c' est une technique professionnelle et c' est de la chimie . Et après , il a fallu le peindre , etc . , etc . Je vous ai donné cet exemple pour vous dire que les gamins , qui n' allaient plus en cours , qui y sont retournés à 90 % d' entre eux , et qu' à la fin , ils ont eu le diplôme . Mais regardez comment ils l' ont eu : en s' élevant spirituellement par la culture . Toutes ces choses que les brutes croient inutiles , un surplus . Quand ils ont fini de compter leurs sous , alors ils lisent de la bonne littérature , et ils se figurent que la culture , c' est quelque chose qui est après le reste . Non ! La culture , c' est d'abord . C' est parce que vous avez lu , c' est parce que votre cœur a été éveillé par la musique , par la poésie que vous changez en tant qu' être humain et que vous aspirez à savoir toujours plus . Et ben , les autres , ils s' en fichent . Mme Le Pen dit que les allègements fiscaux , c' est très bien pour les DOM-TOM . Voilà . Tous dans la cage , fermez-là . Et M. Fillon , mmmh ? Et bien , lui , il est surtout préoccupé par l' immigration illégale , qui est quand même le problème sur lequel vous tombez tous les jours . Par exemple , il y a un problème pour mettre la table , hein ? Ben , hé , vous avez compris , quoi , hein . Ce n' est pas la banque qui est le coupable , hein . Voilà . Ils espèrent vous faire croire à ces fables . Mais la chose qui m' a plu , c' est que ... il propose quoi ? Et ben , de faire encore mieux pour les allègements fiscaux . Il fait reposer l' essentiel de sa politique économique sur les promesses de nouveaux allègements fiscaux . Ah , et aussi des choses très nouvelles : &quot; Réduire le coût du travail . &quot; C' est nouveau , non ? Attendez , c' est génial . &quot; Réduire le coût du travail . &quot; Oui , oui , et le ... et le travailleur avec , non ? S' il avait un moins gros estomac , moins de besoins , hein . Voilà . Réduire le coût du travail . Vous gagnez trop , les gens . Mais oui , je vous dis ça , je vous le dis sur un mode humoristique parce que j' ai repéré que le rire permettait de fixer aussi bien les idées que de vous peindre tout le tableau catastrophique . Mais quand vous en entendrez un vous dire qu' il veut réduire le coût du travail , il faudra vous rappeler que c' est vous dont il est question . C' est de votre paie dont il est question , et pas trouver ça sympathique et moderne . Parce que ça n' est ni sympathique , ni moderne . Il veut aussi créer des zones franches . Des zones franches . On ne paye plus d' impôts du tout dans les zones franches . C'est-à-dire , on ne paye plus les impôts sur la société , on ne paye plus ceci , on ne paye plus cela , il faut bien que quelqu'un le paye , donc ce sera les autres . Chaque fois qu' il y en a un seul qui vient vous dire qu' il va réduire les cotisations sociales , vous les avez entendus , tous , vous le dire . Rappelez -vous de ce que je suis en train de vous dire à cet instant . J' étais sénateur quand on a voté le fait que chaque euro de dispense de cotisation sociale doit être immédiatement compensé - ce que l' Etat ne fait pas toujours - par le budget de l' Etat . Autrement dit , les ballots , chaque fois qu' on vous dit qu' on va enlever les cotisations sociales , et qu' on va vous donner les sous à vous , dans votre paie , direct , après vous être bien frottés les mains à l' idée que ça va augmenter votre salaire , rappelez -vous que vous devrez vous payer , avec ce salaire , en impôts , vous-mêmes , et c' est exactement ce que veut dire celui qui propose de vous payer en imp ... en salaire direct ce qu' il ferait ensuite passer par la CSG , comme je vous l' ai dit tout à l' heure . Alors , comme il arrive dimanche , vous lui poserez la question de savoir , à la fin , pourquoi ils ne veulent pas aller prendre l' argent là où il se trouve . Là où il y en a des masses gigantesques , immenses , considérables , accumulées , qui ne servent strictement à rien , qui passe son temps à tourner dans la sphère financière , pour y réaliser des taux de profits absolument invraisemblables qu' aucune activité humaine ne peut dégager . Parce que , autrefois dans l' industrie , et encore maintenant , le 3 , le 4 % , ce sont des taux de profits qui paraissent considérables . Mais quand vous tournez dans la sphère financière , les taux de profits , c' est du 10 , du 12 . J' ai connu une entreprise qu' on a fermée alors qu' elle était profitable , elle faisait du 8 et le fonds de pension qui avait placé son argent là-dedans attendait du 15 . Et bien , ils ont préféré fermer la boite que de continuer à gagner moins que ce qui était prévu pour ne pas effaroucher les gens qui investissaient sur ce fonds de pension . J' espère , je ne sais pas si tout le monde a bien suivi ce que je viens de raconter , mais vous vous rendez compte de la prédation que le capital financier exerce sur l' activité humaine réelle . Peut-être qu' on leur en voudrait moins si ils faisaient des sous en faisant travailler tout le monde et en produisant quelque chose qui soit utile à la société . Alors ... ça , c' est le premier défi . L' autonomie énergétique , autrement dit , le plan Vergès partout . Si on le fait à la Martinique , on pourra dire à la Corrèze : &quot; Vous pouvez en faire autant . &quot; Ah , mais ils diront : &quot; Nous , on a pas d' eau . &quot; Ah , pas de bol , hein , cette fois -ci , la situation a changé . Mais c' est vrai . Chaque département de la métropole pourrait se poser la question , et c' est là qu' on verrait comment la planification écologique oblige chaque département à réfléchir , car naturellement , la planification écologique , c' est prévoir , comme je vous l' ai montré tout à l' heure , la formation puis la mise en oeuvre , etc . Mais la planification écologique , c' est la réflexion au plus près du terrain . Mais on va pas parler pour la Corrèze , j' irai bientôt si ils veulent , mais vous savez , les hydroliennes , on peut les mettre en mer , bien sûr , on en met en mer , on en a quelques unes au large des côtes françaises , entre l' Angleterre et la France , parce qu' il y a là un courant du feu de Dieu qui passe à une vitesse incroyable , il fait tourner des moulins qui ont des pales de 9 mètres , vous vous rendez compte . Et c' est tellement violent que ces pales se ramassent des cailloux qui sont arrachés du sol , c' est une réalisation technique extraordinaire , enfin , ça s' appelle une hydrolienne . Mais bon , moi j' utilise aussi le mot parce que je trouve que ça fait bien . Enfin , des hydroliennes , dans notre pays , il y en a eu des milliers , hein , pendant des années , ça s' appelait des moulins , avant , hein . Bon , ben , on se rappelle des moulins . Donc , des moulins , on peut en mettre partout , dans toutes les rivières de France . Et dans celles de Corrèze aussi , encore faut -il se poser la question . Autrement dit , la production doit être pensée localement , la diffusion , elle est forcément nationale , et le réseau doit rester national . On y a intérêt , parce qu' un réseau électrique comporte des moments de délestage , des moments de surcharge donc , il faut travailler tout ça . Vous vous demandez pourquoi je vous parle de tout ça , hein . Ben parce que c' est vos affaires , mes amis , qu' est -ce que vous voulez que je vous dise ? Personne ne vous en parle , mais c' est la vérité . Si vous n' avez compris que tous les barrages de France vont être privatisés , vous vous imaginez une idiotie plus grande que celle-là ? Vous voyez , l' investisseur , il se lève le matin , il dit : &quot; Ah oui , aujourd'hui , je vais produire de l' électricité . J' achète un barrage . &quot; Non . Il se lève , il dit : &quot; Aujourd'hui , je vais produire du fric . Où est -ce qu' il y en a à prendre ? Dans le barrage . &quot; Et si quelqu'un lui dit : &quot; Ah oui , mais vous comprenez , bon , hein , 25 % des barrages ont déjà plus de 70 ans . &quot; &quot; Quelle horreur , j' enlève mon argent . &quot; Parce qu' il n' a pas l' intention de couler un seul mètre cube de béton là-dedans . Il n' est pas venu pour ça . Rappelez -vous en quand on vous fera des belles promesses . Deuxième défi où vous pouvez être pilote et obliger donc tous les autres , parce que si vous y arrivez , ils y arrivent . Deuxième défi , c' est celui de la souveraineté alimentaire . Alors là , c' est une grande question . Parce que quand je vous dis : on va changer le mode de production , le mode d' échange , en général , vous voulez bien m' écouter . Mais quand je vous parle de souveraineté alimentaire , vous êtes encore d'accord . Mais après , on dit : &quot; Oui , mais qu' est -ce qu' on va manger ? &quot; C' est là que ça ne va plus . Mais , mes amis , on ne peut pas faire changer notre mode de production pour être conforme à l' intérêt général humain , on ne peut pas changer notre mode d' échange si on ne change pas notre mode de consommation . Si nous continuons à être l' esclave de désirs qui nous sont inculqués , de modes et de pratiques que nous n' avons jamais choisies , nous ne pourrons pas changer le système . Le système , il est dans notre peau . Il est entré ... dans nos goûts ... notre langue , nos papilles . Tenez , le vin - je ne sais pas si ça vous intéresse comme sujet , moi , ça me passionne - et ben , le vin , autrefois , si vous mettiez du sucre dedans et que vous mettiez des copeaux de bois et que vous touilliez un bon coup , on vous coupait les mains - ah , c' est au Moyen-Age , hein - pour avoir fait ça . Aujourd'hui , c' est encouragé parce qu' il y a un goût universel pour pouvoir vendre des quantités extravagantes de vin . Ben alors , vous direz : &quot; Oui , M. Mélenchon , vous êtes tranquille parce que du vin , on ne peut pas en faire tant que du Bordeaux , hein , forcément , il y a une limite . Et puis du champagne non plus . &quot; Ben si , les ballots . C' est réglé . C' est réglé , ça s' appelle grand marché transatlantique et on enlève les normes et les appellations , et voilà , vous aurez du champagne du Mississippi , du Bordeaux du Tennessee avec des copeaux , du sucre , tout ce que vous voulez , bien touillé . Et après , vous irez boire ça , et après , vous direz : &quot; Tiens , c' est curieux , il y a une épidémie d' obésité chez nous , d' où ça vient ? &quot; Et bien c' est ce qui arrive . La malbouffe a une conséquence terrible sur nos populations . Jamais on avait vu ça sur le territoire de la République Française . Autant de gens malades de cette épidémie d' obésité , qui crée toutes sortes de problèmes . Toutes sortes de problèmes , d'abord aux malheureux qui en souffrent . Surtout à eux . Alors , on laisse faire , on continue ? Et j' ai dit &quot; les désirs qui nous sont inculqués &quot; , cette uniformisation de l' humanité , quelle tristesse ! Je débarque à l' aéroport Fort-de-France . Je me promène , et qu' est -ce que je vois ? La même chose que ce que je viens de quitter . A 10.000 km de là . Le même McDonald's ... le même bâtiment dégueulasse . Alors ... c' est &quot; ah la la , c' est le moment de parler de culture &quot; , hein . C' est tout ce que vous savez faire ? Les mêmes horreurs qu' il y a ailleurs . Et on parlait avec une camarade , elle me dit : &quot; Mais ça , je connais , j' ai vu la même chose en Equateur , à Miami . &quot; C' est partout pareil . Ils ont tout nivelé . Tout homogénéisé . Tous pareils , sauf la paie . Les mêmes consommations , les mêmes objets , c' est de ça de quoi il faut s' arracher . Donc on doit changer nos modes de consommation . Et quand il s' agit d' alimentation , c' est pour ça que j' ai été voir le groupement régional des agriculteurs bio , c' est pour ça que je suis allé les voir , parce que je voulais vérifier que ce que je raconte est jouable ou pas . Je leur ai dit : &quot; Ecoutez , est -ce que c' est possible , la souveraineté alimentaire , l' autonomie alimentaire ? Est -ce qu' on est capable de produire de quoi nourrir tout le monde ici , parce que vous êtes déjà 450.000 . Est -ce qu' on est capable ou pas ? &quot; Est -ce qu' on a la technique ? Réponse : oui . C' est déjà pas mal . Est -ce qu' on a la terre ? Ça commence à être moins sûr . Pour deux raisons . Une partie a été pourrie . Le chlordécone , tout le monde connait . Ah , mais attendez , en métropole , personne n' est au courant , ou peu de monde . Et c' est quand même monstrueux , cette histoire , hein . Et ça continue . On en a encore pour un bon moment , il va falloir trouver le moyen de décontaminer tout ça . Allez , les ingénieurs . Venez ici , réglez -moi ça . Mais oui , parce qu' on peut le régler . Ce n' est pas une calamité contre laquelle on ne peut rien , ce n' est pas vrai . On est capable de décontaminer , on a les techniques , on a les méthodes . Et puis , pendant qu' on y est , on va commencer à discuter de savoir si c' est une si bonne idée que ça d' avoir comme modèle d' organisation de l' agriculture , comme unique objectif , exporter . Non . Le premier objectif , c' est nourrir . Donc relocaliser . Donc ça veut dire qu' il faut réorganiser , peut-être , l' usage de la terre . Est -ce qu' on peut ? Voilà la question . Est -ce qu' on peut ? Je discute avec ceux qui font de la banane , puisque le problème , c' était la banane , et que c' est toujours la banane , je discute avec M. Blondel . Il était là ce matin au marché . Et je lui dis : &quot; Mais vous arrivez à faire des bananes sans pesticides ? &quot; Il me dit : &quot; J' y arrive . &quot; &quot; Ah , comment vous faites ? Vous avez des bouillies ? &quot; , j' en connais un rayon , sur ces trucs là , à cause de la vigne . C' est ça , l' avantage d' avoir une activité politique , vous pouvez vous intéresser à des tas de choses , moi j' ai toujours été fantasque , je me suis toujours intéressé à plein de choses qui n' étaient pas dans mon domaine , hein . C' est comme ça que j' ai découvert la littérature , parce que si jamais j' avais fait ce qu' on m' avait dit , j' aurais connu que les auteurs classiques . Heureusement qu' il y a eu la littérature de gare pour que j' achète un bouquin de Glissant . Sinon , je ne savais pas que ça existait , hein . Comme Steinbeck et le reste . Je reviens à mon histoire . Est -ce qu' on peut ? Réponse : oui , on peut produire les bouillies qui ont la fonction de pesticide à partir des plantes qu' on a ici . Donc on peut produire en masse pour nourrir sans pesticides et avec d' autres méthodes d' agriculture . Donc il nous faut des lycées agricoles qui prennent en compte la nécessité de travailler autrement . Parce que ce n' est pas tout de dire que vous allez faire une révolution agricole si vous n' avez pas d' agriculteurs pour la faire . Hé , les gens , si je vous dis qu' on va avoir 400.000 postes de travail , ou 300.000 , ça dépend des évaluations , pour le nouveau modèle agricole , mes camarades étaient tout contents d' avoir trouvé ça , ils avaient fait le total et tout , ils avaient demandé aux fédérations , à la confédération paysanne et tout . Ah oui , oui , très bien . Et où vous les trouvez ? Où ils sont , les 400.000 en question ? Je ne vais pas prendre comme ça , 1 , 2 , 3 , 4 , toi , tu es paysan , 1 , 2 , 3 , 4 , toi , tu es paysan . Non . Il faut donc que le goût soit là , donc on transforme la condition paysanne . Vous ne pouvez pas dire à des gens qu' ils vont aller travailler à la terre comme des animaux . Personne n' a envie d' une vie comme ça . Et ben vous , vous n' en voulez pas , et ben , eux non plus . Donc il va falloir organiser tout ça différemment . Bon , est -ce qu' on peut le faire ? Réponse : oui , mais il faut changer vos habitudes . Franchement , il y a assez ici de légumes qui ont été adaptés avec le temps . Je ne dis pas que ça s' est fait d' une fois , hein . Ah , ben oui d'accord , non mais , si vous voulez à tout prix manger du riz et des pâtes , c' est votre affaire , on ne vous empêchera pas de le faire , hein , mais ce n' est pas normal qu' on soit dans cette situation , où on importe 96 % des poulets que vous mangez . Vous ne pouvez pas faire un poulet ici ? Qu' est -ce qui se passe ? Où vous en êtes ? Qu' est -ce que c' est que cette histoire qu' un poulet qui court dans la rue coûte plus cher qu' un qui arrive par avion ? Qu' est -ce que c' est que ce monde de fous ? Qu' est -ce que c' est , ce monde de fous ? C' est à ça que je demande de ... vous riez , mais vous avez raison de rire , mais il faut le regarder avec ce recul en disant : &quot; Mais comment on est arrivés là ? Comment on en est arrivés là ? Comment on en est arrivés à aller au magasin où on sait qu' on va se faire assommer pour manger des choses qui ne sont pas bonnes ? &quot; Et quel est le mystère de la formation des prix qui fait que vous vous faites dévaliser de la sorte ? Ah ouais ... Comme j' étais dans la rue , je me fais arrêter par un groupe de gens , là , très aimables et qui me payent un café , d'ailleurs , et puis on discute et ils commencent à me parler de l' octroi de mer , ils disent : &quot; On paye très cher , etc . &quot; Peut-être bien , mais l' octroi de mer , ça a servi de barrière , un temps , pour empêcher que les marchandises qui arrivent de l' extérieur exterminent les marchandises qu' on produisait là . Ben oui , le problème , c' est qu' on ne produit plus rien . Par conséquent , il se contente d' exterminer . Je vois bien qu' il y a quelque chose qui ne va pas , mais d' un autre côté , est -ce que vous allez dire aux collectivités locales dont c' est la principale ressource qu' il n' y a plus d' octroi de mer ? On peut le décider . Et on peut surtout dire que c' est complètement aberrant , parce que la collectivité locale a des sous pour aider des gens à condition qu' ils aillent consommer des choses dans le magasin qui leur coûtent cher . Tout ça est absurde , c' est comme ce que je disais tout à l' heure , c' est un cercle vicieux . Donc il faut trouver un autre moyen , premièrement , de financer les collectivités locales sans leur raconter d' histoires , parce que , elles , elles prennent des décisions , elles mettent en route des autobus , elles font ceci , cela , on s' occupe de la cantine , du reste , soit dit par parenthèses , la cantine , ça devrait être 100 % bio , le bio , ça ne doit pas être réservé aux riches , tous les gosses ont les mêmes droits , on est d'accord ? Donc , tout bio . et , mais si la collectivité locale , elle lance tout ça , et puis , tout d' un coup , couic , il n' y a plus d' argent , l' Etat n' en donne plus , ou en donne moins , évidemment , qu' est -ce qu' elle fait , la collectivité ? Soit elle vous enlève le service , et vous n' êtes pas contents , soit elle augmente vos impôts , et vous n' êtes pas contents non plus . Donc il faut que les collectivités locales aient une ressource qui soit assurée et garantie . Alors , bon , je ne vais pas vous lancer dans cette histoire là , c' est un petit peu compliqué , mais ça compte . Et puis , la deuxième chose , c' est que oui , il faut des barrières . Moi , je ne suis pas un libre-échangiste , je ne suis pas d'accord , pas d'accord pour que toutes les marchandises rentrent et sortent comme elles veulent , je ne suis pas d'accord . Je pense par exemple qu' une banane dont on est pas garanti qu' elle ne contienne aucun pesticide et qu' elle ait été cultivée par des gens qui sont esclavagisés , comme c' est le cas sur le continent , je ne suis pas d'accord pour qu' elles arrivent dans mon assiette et que j' ai pour motif de me dire : &quot; Ah , ben , je suis content , c' est moins cher . &quot; Je ne suis pas d'accord . Je ne suis pas d'accord pour que toute une série de produits de cette nature soit mis à la disposition de tout le monde au seul motif que ce n' est pas cher . Non . Donc , il faut que nous mettions des barrières , mais des barrières raisonnables , c' est ce que j' appelle le protectionnisme solidaire . Protectionnisme , oui , on protège nos productions . Voilà . Si vous voulez faire des poulets , si on veut qu' il y ait des poulets qui soient produits localement et que vous les mangiez , ben , il faut les protéger des poulets qui arrivent de l' extérieur avec ... Pas les poulets étrangers , hein , je n' ai pas dit ça , hein . Les autres poulets , quoi . Ces poulets , c' est toujours un bon exemple dans les meetings . Une fois , il y a un ami qui m' a dit ça , il m' a dit ... il faisait le registre électoral , oui , ben , ne vous marrez pas parce que ça vous concerne . Alors , c' était dans un pays qui est très exotique , hein . La Bolivie , ce n' est pas chez nous que ça arriverait . Ils avaient plusieurs millions de personnes qui n' avaient aucune inscription sur les listes électorales , ce n' est pas ici que ça se produirait , hein , on est tranquilles , ouais . Alors , donc , ils ont commencé à les organiser . Et alors , évidemment , les réactionnaires ont commencé tout de suite à hurler : &quot; Quoi , c' est les Cubains qui font le registre électoral ? Ils ont emporté les listes à Cuba . &quot; Vous savez , ils lisent les listes à Cuba , ça leur fait du bien . Et ça faisait peur aux gens . Et alors , le gars ... le gars , façon de parler , pardon , je suis irrespectueux , c' était le premier ministre , c' est un ami . Il s' appelle Ramon Quintana , et donc on discute et il me dit : &quot; Ecoutez , vous vous rendez compte , c' est quand même incroyable , ici , on peut manger une aile de poulet - oui , alors , les poulets qui reviennent - et le poulet , on sait d' où il vient , de quelle ferme , qui c' est qui lui a donné à manger , qu' est -ce qu' il a mangé , combien de fois . Parce qu' il y a dix codes-barres qui indiquent tout ça . Et ici , il y a des gens , on ne sait même pas qui c' est , ni d' où ils sortent . Et tout le monde s' en fout . &quot; Et bien , on devrait se poser la question de savoir , si nous , ici , on n' est pas un peu pareils , hein . Pensez -y avant la prochaine élection . Vérifiez un peu autour de vous de savoir si les gens sont bien maîtres de leur destin . Faites attention qu' on leur ait pas trop bourré le crâne à leur dire : &quot; C' est tous les mêmes , ça ne vaut pas la peine , on reste à la maison , on va bouder . &quot; Et c' est les mêmes qui gardent le pouvoir , les mêmes qui continuent à pratiquer les mêmes politiques . Il faut vous inscrire et il faut voter pour les faire dégager , il n' y a pas d' autre moyen , c' est comme ça dans la démocratie . Qu' est -ce que je ne vous ai pas encore dit , qui est sur mon papier ? Non , parce qu' après , vous allez vous fâcher si je continue comme ça . Avec tout ce qui ne me convient pas . Pas la façon avec laquelle nous traitons la situation . Alors voilà . Ou bien ça continue comme avant . Toutes les habitudes sont prises , il n' y a rien qui change . Vous continuerez à râler au même endroit sur les mêmes sujets . Moi , je vous dis juste une chose . Des territoires comme les nôtres , ici , sont fragiles . Vous ne pouvez pas vous figurer que vous allez continuer à diminuer la proportion de terres arables comme vous êtes en train de le faire . Diviser par 4 la surface de terres utilisables . Parce que quand on construit , figurez -vous que ceux qui investissent pour construire , ils ne vont pas à l' endroit où il y a des cailloux , des pentes et le reste . Ils vont à l' endroit où c' est plat . L' endroit où c' est plat , c' est justement l' endroit où on peut planter . Je ne vous donne que ça , comme exemple . Mais vous voyez que vous êtes dans une impasse . Je ne connais pas , moi , la réponse à chaque cas , et pourquoi je la connaîtrais , et pourquoi ce serait à moi de le décider ? C' est bien à vous , quand même , c' est vous qui avez le nez dessus . Mais il faut que vous puissiez intervenir , et pour ça , il faut changer la logique . Si la logique , c' est que le Président de la République vous dit : &quot; Vous allez être le pays pilote pour l' autonomie énergétique , vous allez être le pays pilote pour l' autosuffisance alimentaire . &quot; Et ben , vous allez vous y mettre . Et vous allez tous le faire . Et les collectivités vont s' y mettre , parce qu' elles sauront qu' on va les aider et pas les lâcher en cours de route , une fois qu' elles auront commencé . Vous voyez , c' est à ça que moi , je voudrais appeler . Je ne viens pas vous raconter tout ce qui ne va pas , vous le savez mieux que moi . Comment on s' est moqué de vous , vous le savez mieux que moi . Je viens plutôt vous appeler à ça . A cet élan , on va faire les choses . Nos gamins ont un avenir . Nous-mêmes , on en a , le monsieur qui m' a arrêté dans la rue , qui m' a dit : &quot; M. Mélenchon , qu' est -ce que vous allez dire sur le chômage , parce que moi , je voudrais travailler . &quot; Tout le monde veut vivre dignement en contribuant à l' effort collectif par son travail . Ce n' est pas vrai que le pays soit rempli &quot; d' assistés &quot; , comme ils disent , avec ce vocabulaire qui n' est pas le nôtre . Nous , on parle de solidarité . Ce n' est pas vrai . Il y en a assez qu' on montre du doigt les pauvres en les rendant responsables de leur pauvreté . Les chômeurs en les rendant responsables de leur chômage . Les gens malades en les rendant responsables de maladies qui sont arrivées par l' environnement . Et qu' on entende dire : &quot; Les gens , il faut vous responsabiliser . &quot; Voilà , responsabilisez -vous . Attrapez la grippe en été , il y en a moins qui l' ont . Qu' est -ce que ça veut dire , ces histoires ? Cette manière , toujours , d' inculquer aux braves gens une sorte de dégoût d' eux-mêmes , une méfiance à l' égard d' eux . Leur enlever leur fierté , le goût de ce qu' ils savent faire , de leur envie de contribuer au bien commun , en les humiliant sans cesse , en les montrant du doigt . Vous avez vu cette émission infâme où on montrait des pauvres gens qui étaient culpabilisés parce qu' ils avaient des aides sociales ? Et le peu qu' ils avaient , on le leur reprochait encore . Ça suffit . Ouvrez vos cœurs . Ne supportez plus qu' on vous parle comme ça . Ne laissez plus dire ces choses . Bon , je l' ai fait à votre place , pour le coup , là . Mais enfin , quand même , vous ne pouvez pas être la terre de Césaire et des autres et ne pas entendre ce que je viens de dire , je sais que vous l' entendez . Maintenant , il faut convaincre les autres . Ça ne peut plus durer comme ça . Ça ne peut plus durer , d'abord , ça ne peut plus durer pour la planète . Ça ne peut plus durer pour nos gosses . Ça ne peut plus durer pour la vie qu' ils nous font , l' image qu' on a de nous-mêmes quand on se regarde . Voilà toutes ces choses qu' il faut reprendre . C' est ça , l' enjeu d' une élection . Ce n' est pas juste de savoir si vous allez élire celui -ci ou celui-là , encore que c' est celui -ci qu' il faut élire . Bon . Là , il va falloir finir , alors je vais aller un petit coup dehors , quand même , les pauvres , ils ont été patients . Il reste quelqu'un dehors ou pas ? Qu' est -ce que vous en dites ? Wouaouh . Ça va , là ? Mais vous êtes restés là , tout ce temps ? De qui je pourrais dire du mal , maintenant ? Non , je viens maintenant vous dire ... je viens vous dire au revoir après tout ce que j' ai déjà dit . Alors , les gens , je vais vous parler pays . D'abord , des excuses pour les littéraires qui m' ont entendu , et vous avez entendu tout à l' heure , là , sur la vidéo , citer Aimé Césaire . Mais je l' ai mal fait . Il y a un complot , là . Comment je dois me mettre ? Ça compte marcher un jour , là ? Merci , monsieur . Vous m' entendez encore ? Bon , ça va aller , alors . Je l' ai mal cité . Mais j' espère que je me suis montré à la hauteur , aujourd'hui , de ce que j' ai voulu citer hier . Alors , je vous la relis , telle qu' elle est dans le texte d' Aimé Césaire . Et à vous tous , chacun à sa façon , d' en faire autant . &quot; Ma bouche &quot; , dit Césaire , &quot; ma bouche sera la bouche des malheurs qui n' ont point de bouche . Ma voix , la liberté de celles qui s' affaissent au cachot du désespoir . &quot; La leçon qu' enseigne Césaire , à cet instant , est qu' il n' est pire maladie de l' âme et du cœur que la résignation . Renoncer à changer le monde , c' est renoncer à être soi-même dans un monde meilleur . Ce à quoi je vous invite , quoi qu' il arrive , quelle que soit l' issue de la volonté de notre seul souverain , le peuple , c' est à garder au cœur cette idée , que génération après génération , vous autres , là , moi , qui n' ai plus votre âge , nous tenons le même fil rouge dans le temps , portant la même lumière , la même promesse , pour toujours , de rebond , de renouveau , de recommencement , d' espérance , de nouveauté , de bonheurs possibles . Césaire , encore , discourant contre le colonialisme . Le colonialisme , mes amis , qui est entré comme un pied dans sa chaussure dans le nouvel âge du capitalisme , le colonialisme qui faisait de cette terre , une terre de monoculture pour l' exportation au prix du travail qui coûtait le moins cher possible , celui des esclaves . Le colonialisme a remis son pied dans le capital financier transnationalisé , lui aussi , il vous appelle à continuer à produire pour l' exportation au moins cher possible . C'est-à-dire sur le dos de la souffrance et de l' humiliation du grand nombre , les uns , condamnés au travail toujours plus intense , les autres , à la désespérance du chômage . Le colonialisme , qui pratique le même aveuglement inhumain qu' il pratiquait autrefois , à sa façon aujourd'hui , quand il refuse de voir que 40 % des espèces vertébrées ont déjà disparu de la surface de la planète , que c' est dans les DOM et les TOM que disparaissent le plus grand nombre des êtres vivants , dont la diversité était la magie de vivre , celle des imaginaires , celle des poésies , celle des comparaisons , ce ne sont pas seulement des bêtes qui disparaissent , ce ne sont pas seulement des mondes qui s' engloutissent , c' est une manière d' être un être humain qui disparaît . Voilà le vrai défi auquel vous êtes appelés , et à vous spécialement , jeunes gens , jeunes hommes , jeunes femmes , à vous , qui portez nos espérances , votre devoir est d' infléchir le cours de l' histoire , de ne pas permettre qu' ait lieu cette immense catastrophe qui se dessine sur la civilisation humaine . Portez en vous , pour toujours , la leçon d' Aimé Césaire . &quot; Une civilisation qui s' avère incapable de résoudre les problèmes que suscite son fonctionnement est une civilisation décadente &quot; , dit Césaire . Cette décadence s' interrompra si , à mon appel , puisque c' est moi qui porte cette parole , aujourd'hui , pour vous tous , à mon appel , à notre appel à tous , nous invitons les peuples qui sont ici en Caraïbe , qu' ils soient français , ou qu' ils ne le soient pas , tout notre environnement , géographique et humain , culturel commun , que nous appelons avec cette formule qui est contenue dans le titre de notre programme , à un avenir commun . Nous sommes tous , d'abord , des êtres humains semblables par nos besoins , nos rêves , nos espérances , nos amours . Faites prévaloir ... l' intérêt général humain . N' ayez jamais honte d' être bons , d' être tendres , d' être partageux . N' ayez jamais honte d' être des êtres humains et de mépriser les obstacles que l' on vous présente parce que vous vous sentez capables de les dépasser . Allez ! "
    ]
melenchon_test = [
    "Vous m' entendez , vous m' entendez bien ? Parfait . Bah il faut écouter , elle dit non , elle là-bas ! Bah écoute ! Vous entendez , ce coup-là ? Bon . Je commence avec vous là dehors , parce qu' on a plus assez de place pour que vous soyez tous dedans , et puis ensuite j' irai dedans , et si vous y êtes encore , je viendrai vous saluer avant qu' on se quitte . On m' avait dit : &quot; Il y a tellement de brume ! &quot; Mais moi je vous connais , il n' y a pas de brume assez épaisse pour que vous ne trouviez pas votre chemin pour faire ce que l' on est en train de faire à cet instant . Bien sûr , on est venus écouter quelqu'un qui a des idées à présenter à propos de la condition des salariés . Mais on est d'abord venus pour marquer un moment de protestation contre , d' indignation humaine , contre le sort qui est réservé a tant de milliers d' ouvrières et d' ouvriers , de salariés , de petits rangs qui ne peuvent se défendre parce que toutes leurs vies dépendent de ce pauvre travail . De ces heures qu' on leur accorde , de ces temps de travail qui ne sont jamais complets , et où l' on vit dans l' attente d' heures complémentaires pour pouvoir améliorer la paye à la fin du mois , que les puissants sont prêts à tout moment à leur disputer au centime près ... Quand eux regorgent et se gavent - quelques centimes ici , que l' on supprime aux heures supplémentaires ; quelques heures de travail que l' on va payer moins ; une amplitude horaire que l' on va exagérer , laissant cette femme qui est partie tôt de chez elle le matin , laissant ses enfants , et qui ensuite va rester deux heures , trois heures sans emploi avant de reprendre le travail . C' est ça que nous sommes venus d'abord dire , que vous êtes venus d'abord dire avant que de m' écouter . C' est un acte , notre rassemblement - ce déboulé , comme je l' ai appelé - un acte de solidarité avec cette jeune femme qui a perdu son enfant uniquement du fait du mépris que ceux qui en avaient la responsabilité ont eu à son égard , qui n' ont pas écouté ce qu' elle avait à dire . Mais vous savez , ce n' est pas la première fois , et c' est bien parce que ce n' est pas la première fois que je suis venu . Souvenez -vous : c' était pas si loin d' ici . Souvenez -vous , si vous croyez quelque part qu' il y a une limite à la brutalité de ce monde de la flexibilité , de la compétitivité , bla bla bla ... Souvenez -vous de cette jeune postière - qui est dans la salle sans doute -Émeline , et si vous ne vous souvenez pas , regardez ce film . Il passe là , en principe . Est -ce qu' il passe ? Je vais vous le raconter , parce que visiblement ça ne marche pas . C' est une jeune femme , et elle a un CDD , qu' on renouvelle , comme vous connaissez ça - c' est absolument illégal , hein , mais ça fait rien , ils renouvellent . Et de CDD en CDD , comme elle est postière on la met jamais sur le même parcours . Si bien qu' à chaque fois , tout l' effort particulier du postier ... Car il y a ceux qui n' ont jamais fait aucun travail de cette sorte , qui croient que c' est facile , qu' on passe d' une porte à l' autre avec le courrier . Non ! Il faut connaître sa tournée , et si vous la connaissez bien vous la faites plus vite , donc vous avez une chance de terminer à l' heure qui est prévue - parce que si vous n' avez pas terminé à l' heure qui est prévue , ce qui est prévu c' est que vous ayez fini la tournée . Donc vous allez faire du temps de travail de plus , un temps de travail qui ne vous est pas payé . Eh bien , cette jeune femme , comme elle a la tête un peu dure , les chefs l' ont repérée . J' en profite pour appeler chaque chef à se souvenir de sa responsabilité humaine dans la conduite des équipes . Eh bien , ils la font changer de place , et cette femme a une faiblesse à un moment donné et elle prévient , elle dit : &quot; Mais moi je ne peux pas continuer , je suis mal &quot; &quot; Roh , tu t' écoutes trop , continue ! &quot; Et ainsi de suite , vous connaissez la suite de cette histoire . A la fin elle fait un AVC , et c' est tout juste si on l' a pas traitée de simulatrice . Et comme on n' est pas intervenu assez vite , elle reste avec un handicap . Mesdames et messieurs , mes chèr(e)s ami(e)s , je n' ai pas l' intention de montrer du doigt celui -ci ou celle-là , personnellement responsable de cette situation . Vous voyez , je fais comme vous ! A ma manière j' ai bon cœur , et je suis prêt à oublier la brutalité si celui qui l' a commise s' en repent et prend les mesures pour ne pas la recommencer . Mais que dois -je dire devant vous à cette heure , quand je sais qu' avoir après proposé un obscur arrangement à cette jeune femme , pour terminer , la Poste se déshonore en lui refusant le CDI qu' elle devrait lui donner maintenant après ces souffrances et cette marque qu' elle porte à vie ! De leur indifférence au sort des autres , les puissants veulent toujours avoir le dernier mot ; et c' est avec ça qu' ils impressionnent , qu' ils font peur et que les gens ont peur . Et c' est de cette peur dont nous voulons délivrer le monde du travail , peur du lendemain , peur de perdre son emploi , peur de ne pas avoir sa paye ! Et après ça , quoi ? Peut-être de ne plus avoir son appartement et de se retrouver comme tant de gens , bringuebaler d' un endroit à l' autre ! On m' a dit : &quot; Ce n' est pas la peine de faire du Zola &quot; . Je ne fais pas du Zola ! C' est la vie qui fait du Zola ! C' est vous qui faites du Zola ! C' est vous qui méprisez tous ces gens , qui faites du Zola ! C' est vous qui exagérez ! Mais il y a un principe d' humanité qui est aussi un principe de morale individuelle . A Auchan , c' est la deuxième fois . La première fois , une malheureuse s' était trompée de 85 centimes . Elle a été immédiatement expulsée ; fort heureusement , le syndicat l' a rétablie dans ses droits - pas tous ses droits , mais il a fallu l' action syndicale . Pendant ce temps , évidemment après un crime aussi affreux , on espère qu' elle se repent ! De son côté , madame Lagarde avouait une négligence de 400 millions , pris dans vos poches ! Et ils ne lui en ont tenu d' aucune manière . On lui a fait quelques remontrances , ça ne sera pas dans son casier judiciaire et elle ne fera pas un jour de prison . Et elle ne perdra pas , non plus , un jour de sa paye car le jour où elle était au tribunal elle n' était pas au travail . Eh bien ici , j' ai à dire - mais d'abord je veux que vous entendiez , si cela se peut , de sa bouche l' histoire de Fadila . Et cette fois -ci peut-être que ça va marcher . * Le 22 novembre , Fadila , employée de Auchan City , fait une fausse couche sur son lieu de travail . L' entreprise au courant de sa grossesse et malaises répétés n' a pourtant rien fait pour l' aider . Ici vous avez l' ensemble , ou presque l' ensemble , des bases ( c' est à dire les entreprises où nous sommes présents où nous avons des délégués , ou des militants ) . Déjà pour information , vous voyez le gros paquet qui est là , c' est Auchan City . Tous les gens qui sont derrière ces dossiers-là , c' est des gens qui sont anonymes , qui font pas la Une des médias , et tous ces gens qui sont derrière ces dossiers ils morflent . Parce que quand vous vous êtes présenté aujourd'hui dans une entreprise avec un mandat CGT , avec des revendications , vous avez affaire à une répression qui est absolument terrible et qui reste secrète , que personne ne connaît . Fadila : Aujourd'hui , pourquoi on porte plainte contre Auchan ? Bah c' est justement pour avoir des réponses , c' est la seule manière d' avoir une réponse de la part d' Auchan , parce qu' ils disent n' importe quoi sur l' article qu' ils ont sortis à la presse , c' est tout des mensonges . Encore une fois , ils me font passer , moi , pour la menteuse - oui , la fautive et la menteuse . &quot; Non , on n' a jamais donné de négatif pour aller aux toilettes à Fadila &quot; alors je suis resté dans mon siège , je m' amusais de saigner à ce moment-là &quot; Bah si , tombe mon enfant , meurs ! &quot; Je sais pas , c' est ça qu' ils veulent me dire ? &quot; Bah en fait , elle s' amusait à rester dans son siège et à saigner &quot; Mais c' est n' importe quoi ça ! Et aussi &quot; Non , on est intervenus très vite &quot; . Vous êtes intervenus parce que j' ai fermé ma caisse , si j' aurais pas fermé ma caisse , vous seriez jamais venus me voir . C' est moi qui est intervenue pour moi-même , c' est moi qui a pris la décision de fermer ma caisse et c' est pas vous qui avez appelé les pompiers , c' est moi qui a appelé les pompiers . C' est moi qui a parlé avec les pompiers , il y a un rapport de pompier derrière . Et ici , on reçoit donc énormément de personnes , qui sont dans des situations analogues , et notamment les jeunes précarisés , contrats de professionnalisation , apprentis , CDD , intérim , voilà : le tout-venant . Là on est pas dans la bureaucratie , on est pas dans l' état-major , on construit pas un rapport de force d' entreprise . On le fait aussi , hein , mais au quotidien on est littéralement débordés . On est dans une ville qui connaît , qui a 1/4 de sa population qui vit sous le seuil de pauvreté , avec un taux de chômage considérable et une précarité énorme , et Mulliez comme employeur tout-puissant , avec les méthodes qu' on connait . Voilà . Il y a un nom qui a été cité , eh bien , je vais lui dire ce que j' ai décidé . Quand on est très puissant on a de très grandes responsabilités . Je sais bien , il y a des gens qui sont milliardaires mais l' argent ne donne pas tous les droits . La valeur d' une personne n' est pas décrite par son compte en banque . J' ajoute qu' être milliardaire est immoral . A quoi bon d' avoir 1 million si c' est pour en vouloir un deuxième ? A quoi bon d' avoir 1 milliard si c' est pour en vouloir un deuxième ? Je ne crois pas qu' il y ait de vertu à accumuler de cette manière car cette accumulation , d' une façon ou d' une autre , c' est de la valeur soustraite à la société . C' est de la richesse captée à un endroit qui pourrait être répartie différemment sur tous les autres . Mais qu' on n' aille pas donner de leçons à Fadila , qu' on n' aille pas la traiter de &quot; menteuse &quot; , qu' on n' aille pas répandre des rumeurs sur son compte , qu' on n' aille pas dire qu' elle n' aurait pas respectée son contrat ! Tiens , à propos : son contrat , c' est un contrat de professionnalisation . Ils ont une inventivité extraordinaire pour trouver des mots pour habiller des situations inacceptables . &quot; De professionnalisation &quot; , voici qu' elle apprend le métier de caissière , ce qui permet , parce qu' elle est en contrat de professionnalisation , de ne pas la payer au SMIC . Et ensuite , pour celui qui l' emploie , de bénéficier d' allègements de cotisations sociales , c'est-à-dire qu' il ne paie pas . Qui les paye , ballot ? C' est écrit dans la loi : chaque fois qu' un euro n' est pas versé en cotisations sociales , c' est le budget de l' État qui le prend en charge . Qui c' est l' État ? C' est vous ! C' est avec vos impôts qu' est financée cette situation inacceptable . Cette subvention , cet assistanat qui est fait à qui ? A une chaîne milliardaire ! Pas à des gens qui auraient besoin de solidarité ! Alors , qui sont -ils ? Quels sont leurs droits particuliers , personnels à se comporter de cette façon ? Qui est cet homme ? Où il habite ? En Belgique . Il a le droit , pourquoi pas , mais ses impôts habitent où ? En Belgique aussi ! Evidemment , tout cela doit avoir une fin , et il y aura une fin à cette situation , si vous le décidez . Si je suis élu , l' impôt sera perçu sur la base de la nationalité du cotisant quand il n' est plus en France , c'est-à-dire qu' il sera universel . Et Monsieur Mulliez pourra bien aller en enfer , que je m' y retrouverai pour prendre sa part d' impôt . Les citoyens français devront déclarer partout dans le monde ce qu' ils payent comme impôt à la nation qui les accueille , et il est bien normal qu' ils en payent là où ils se trouvent , car nous pratiquons de même chez nous . Mais ils devront déclarer ce montant au fisc , et celui -ci regardera si - après avoir payé cette somme , en France - ils ne devraient pas encore d' argent . L' impôt sera donc universel . Qu' on ne me dise pas que c' est impossible , puisque ça se pratique aux États-Unis d' Amérique et j' aime mieux vous dire qu' ils le font avec une certaine férocité . Comme vous le savez , des gens comme moi , finissent par tout savoir , et vous savez pourquoi ? Parce que je suis lié à ceux qui font , et ceux qui font viennent me raconter ce qu' ils font . Et je connais les employés de banque qui m' ont expliqué comment on leur a demandé de rechercher les suspicions &quot; d' américanitude &quot; de certains de leurs clients . Vous voyez ? On le fera . J' en prend l' engagement solennel devant vous , et Monsieur Mulliez , comme les autres , paiera ses impôts en France . Nonobstant les autres enquêtes qui courent sur son compte . Mesdames et Messieurs , je ne suis pas venu , seulement , mais ce serait déjà bien , pour parler de ce cas , et faire qu' ensemble nous mettions tout notre poids dans la balance pour que les dirigeants d' Auchan sachent que s' ils ne règlent pas la situation particulière de ce magasin qui s' est spécialisé dans les brutalités , nous allons nous occuper , nous , de faire connaitre la marque pour d' autres raisons que les produits qu' ils vendent . Et que tous autant que nous sommes là , nous saurons quoi faire et que dire autour de nous à propos de cette firme . Mais , je me suis dit &quot; c' est le moment &quot; : puisque c' est une campagne présidentielle , tout le monde écoute . Les gens , c' est bien normal , s' intéressent à des questions que le reste du temps ils laissent de côté , c' est normal , mais maintenant ils s' y intéressent , ils écoutent . Il faut donc faire une campagne électorale qui soit une campagne instructive , et pas seulement une campagne où je viendrais vous raconter mes mérites qui n' ont aucune espèce d' importance dans la situation que nous connaissons . Une campagne où tout d' un coup on repense à tous les problèmes de notre patrie et comment les régler . J' ai commencé une campagne , déjà , en 2012 , dans un endroit , c' était Fralib , peut-être que vous vous souvenez . Les ouvrières , les ouvriers qui étaient là , on fait de leur entreprise qui allait fermer une multinationale déjà , une coopérative ouvrière . Ils ont bataillé , galéré , miséré , mais à la fin ils ont eu le dernier mot . Et à cette occasion , j' étais venu pour commencer dans leur usine et j' avais dit : &quot; Voilà , je vais rendre visible ceux qui ne le sont pas , par ma campagne . &quot; Le mot a eu du succès , je l' ai retrouvé dans plein d' autre bouche . Les invisibles devenant visibles . Mais c' est le moment - dans l' ambiance particulière que je vois déjà se dessiner - où une nouvelle fois revoici les professeurs Cyclotron , les inventeurs permanents d' une même -idée ... Pressez le citron , il en sortira du jus ! Et si on n' a pas assez pressé , pressez encore ! Je viens témoigner , parler pour et au nom de 6 millions d' ouvriers : la classe ouvrière existe ! De 7 millions d' employés qui ont un statut souvent très proche de ce qu' on appelle un ouvrier . 13 millions de personnes ! Il n' y a pas de classe plus puissante en France , ni de classe plus utile car c' est elle qui fait tout ! Tout ! Sans le travail humain il n' y a aucune richesse produite , et personne ne peut ni répartir ni accaparer ! Mais comme on a mal agi , Comme on a culpabilisé ces gens du moment où ils n' étaient plus présents dans d' immenses entreprises où on les voyait , où on les craignait , on s' est dit : &quot; Les voici dispersés ! &quot; Et on les a convaincus eux-mêmes qu' ils n' existaient pas . Les ouvriers , les employés sont les absents de toute représentation . 3 % des gens qui sont interviewés à la TV sont des ouvriers . 61 % sont des cadres - j' ai rien contre les cadres , il y en a plein ici , mais ça c' est pas la vraie vie ! La vraie vie c' est tout le monde , c' est pas seulement certains et les autres pas du tout . Si bien que chacun rentre la tête dans les épaules , et combien ont perdu la fierté que leur enseignaient leur père et leur mère , qui ont sur cette Terre , fait naître tous les droits à partir desquels nous avons fondé la République , et cette forme particulière qu' est la nôtre ? A qui la devons -nous ? Pas à la bienveillance des puissants qui n' ont jamais rien cédés sans qu' on le leur arrache Nous le devons à la détermination , la constance , l' implication au travail et à la lutte des ouvrières , des ouvriers et des salariés : telle est l' histoire de France , et c' est à elle que je rend hommage à cet instant . Les ouvrières , les ouvriers , les salariés , les petits grades n' apparaissent à l' écran que lorsqu' on veut montrer des violences après les manifestations . Les ouvrières , les ouvriers , les petites gens ne sont citées que pour dire qu' ils sont responsables de la montée du Front National , ce qui est un mensonge ! Répété , et à force d' être répété avec l' espoir qu' il soit cru et que ceux-là à qui le message est adressé leur rentre dans la tête . La vérité c' est que 60 % des ouvriers et des petits grades du monde du travail , de l' emploi , ne votent pas : ils s' abstiennent . Je viens leur dire &quot; Sortez de votre trou , mêlez -vous de vos affaires , emparez -vous de cette situation ! &quot; Ne vous auto-éliminez pas de la décision ! N' abandonnez pas vos bulletins de vote ! C' est après eux que sont accrochés votre liberté et votre dignité , car la nouvelle vague des beaux parleurs qui est revenue , est revenue pour vous annoncer de nouveaux sacrifices , de nouvelles brutalités , de nouveaux viols de vos droits , de votre sécurité sociale que l' un veut vous arracher , que l' autre veut vous trafiquer et ainsi de suite . Il le faut ! Il faut que dans cette campagne enfin on accepte de prendre conscience de ce qu' est la situation de cette immense classe . Est -ce que nous sommes pas capables de nous mobiliser d' une manière extraordinaire quand il s' agit de lutter contre le terrorisme ? Et nous avons bien raison ! Mais alors , pourquoi est -ce que nous ne nous mobilisons pas d' une manière extraordinaire quand les conditions dans lesquelles les gens travaillent aujourd'hui fait qu' il y a chaque année 565 morts sur le poste de travail ! Comme ça a failli être le sort de cette malheureuse postière et de combien d' autres de ceux que vous connaissez ! Ensuite , ce sont les accidents de trajet . Non , ce n' est pas les plus importants , c' est sur le poste de travail que ça se passe . 3 morts par jour , 3 morts chaque jour . Chaque jour nous est enlevé quelqu'un dont le savoir-faire , l' éducation , le talent était une contribution à la vie de la société . Chaque jour est enlevé à l' amour de ses proches quelqu'un qui allait gagner leur vie . Voilà la brutalité de ce système . Que fait -on ? Que se passe -t-il ? Rien . Et c' est pas tout . Combien de gens dorénavant soumis à une forme et à une cadence de travail que les puissants ensuite nient se suicident ? Notre présence aujourd'hui accompagne comme une clameur le suicide des infirmières de cet été , de ces 50 postiers paraît -il , qui ont mis un terme à leur vie parce que leurs conditions de travail étaient devenues insupportables . De cette paysanne ou de ce paysan qui tous les 2 jours met fin à sa vie , parce que c' est trop , ce qui leur est demandé au travail . C' est contre tout ça qu' il est temps de se mobiliser , de se rassembler , mais tout commence par le sentiment d' humanité , le sentiment de partager quelque chose avec les autres , qui s' appelle la vie ! Que nous avons en commun , avec nos amours , avec tout ce qui fait le bonheur de vivre . Cessez de regarder les gens comme des pions , comme des machines qu' on peut traiter indifféremment de leur sentiment , de leur état de santé . Regardez comme ils passent leur temps continuellement à instiller le doute . Il n' y a pas une émission où il n' y a pas un Monsieur très savant avec des graphiques à la main qui vous montre quelle bande de tire-aux-flancs vous êtes . Les français travaillent moins que les autres et vous , vous rentrez la tête dans les épaules ! Non ! Ce n' est pas vrai ! Ils travaillent autant que les autres , mieux que les autres ! Parce qu' ils ont une productivité 20 % supérieure à celle de nos principaux concurrents , puisqu'ils vivent tout en terme de concurrence ! On n' en parle jamais de ça . Alors ils vous sortent un graphique et vous , vous êtes au milieu , vous vous dites &quot; Oh là là , on est pas loin du bas &quot; . Vous n' avez rien compris : depuis quand travailler beaucoup est un gage de succès pour une économie ? Vous voulez savoir qui est -ce qui travaille le plus en Europe ? C' est les bulgares ou les roumains . Vous voulez faire bulgares ou roumains ? Ça vous intéresse ? Déjà que j' ai eu un mal de chien à vous convaincre de ne pas vouloir faire les Allemands , jusqu' à ce que vous vous rendiez compte qu' il y a plus de pauvres là-bas qu' il y en a ici ! Enfin , tout ça est absurde ! Tout ça est absurde ! Je viens de vous parler de ces accidents , de ces suicides , de ces maladies et vous avez tous l' air d' être d'accord avec moi que c' est pas une situation acceptable ? Mais vous , vous êtes pas très intelligents , vous n' avez pas fait les grandes écoles et vous êtes même pas chefs ! Et vous êtes pas ministres socialistes . Ni de droite non plus . Alors de quoi vous vous mêlez , hein ? Qu' est -ce qu' ils font pour répondre aux problèmes ? Tout à l' heure j' ai dit rien , non ! Maintenant ça a progressé , ils font moins que rien . Parce que là nous venons de parler de questions de santé . J' ai entendu il y a pas longtemps , Monsieur Macron , tout d' un coup dire que ... Depuis qu' ils n' ont plus de responsabilités , tous ceux-là ils ont tous des idées . Et souvent le contraire de ce qu' ils faisaient et ils nous croient assez ballots pour les croire . Ils se disent que ... Comme on ... Hein ... Alors on va les croire . &quot; Alors Monsieur Macron , il parait que vous dites qu' il faut faire du &quot; préventif &quot; plutôt que de soigner ? &quot; Non mais commençons par dire qu' on va soigner les gens , hein . Non mais avec vous , ça mérite qu' on vérifie ! Mais si vous voulez faire du préventif , comment ça se fait que vous avez fait passer la loi El Khomri qui supprime la visite médicale obligatoire quand on reprend un poste de travail ? Est -ce que le préventif ça commence pas par ça ? Quand ces 2 jeunes femmes ont -elles eut droit à un examen par le médecin du travail ? Notamment Fadila ? Jamais ! Et pourtant la loi s' appliquait jusqu' au 1er janvier . Le 1er janvier maintenant , c' est la loi El Khomri . Et maintenant , il n' y a plus de visite médicale obligatoire . Un infirmier peut vous examiner . Mais un infirmier c' est un boulot spécial , particulier , c' est un métier , vous autres le savez pas , les puissants ? Infirmier , c' est un métier , médecin c' est un autre métier . Brancardier c' est encore un autre métier et ainsi de suite . Vous ne connaissez rien au travail ! Et pourtant vous gesticulez . Le travail , celui -ci est le candidat du travail , l' autre aime le travail et vous faites des discours sur la valeur travail car vous êtes censés être tous des tire-aux-flanc . Vous ne savez pas la valeur du travail , pourtant vous en vivez . Pourtant , vous , vous avez personne à la maison qui vient faire votre travail à votre place . La première valeur du travail ce sont les travailleurs , ce sont les salariés et c' est en leur direction qu' il faut tout organiser . Si vous voulez faire de la prévention , alors rétablissez les droits de la médecine du travail , redonnez à un médecin du travail ce qui était autrefois son droit qui était que lorsqu' un salarié se présentait devant lui , et que d'après son analyse la nature du poste de travail qu' il occupait pouvait nuire à sa santé compte tenu des conditions particulières de telle ou telle personne , qui ne peuvent se mettre dans aucune loi ni aucun texte général , alors il pouvait recommander OU un changement du poste de travail OU des conditions de ce travail OU de l' endroit où pouvait aller la personne en question . Tout ça , c' est fini . Vous arrivez , contrat de professionnalisation parce que vous êtes une femme , que vous êtes jeune et que vous avez pas le choix : &quot; Allez , on va te professionnaliser , tu as le droit à des heures de formation ... DES c' est trop , UNE . On va t' apprendre comment on fait marcher la caisse parce que les autres ils ne savent pas que faire marcher une caisse c' est pas simple et c' est épuisant . Et ils pensent qu' en une heure on apprend , et que ça suffit largement , et le reste du temps &quot; Allez , travaille , tais -toi ! &quot; &quot; Qu' est -ce qu' il y a ? un délégué syndical ? Ça va pas ? Tu veux perdre ton boulot ? &quot; Voilà ! Ce que sont les conditions quotidiennes , la vraie vie , les vraies conditions dans lesquelles ça se passe pour 13 millions de personnes . Je ne dis pas que ça soit comme ça partout , heureusement . Mais il faut bien qu' on parle pour ceux qui en pâtissent le plus , sinon qui le fera ? Et il faut bien qu' en pensant aux plus désarmés , aux plus faibles , on organise la société parce que c' est comme ça qu' on la rend meilleure , depuis toujours . C' est en s' occupant de ceux qui ont le moins de moyens de le faire par eux-mêmes , voilà comment on fait avancer et progresser la société . Donc il faut rétablir cette médecine du travail . Ça revient à dire une chose : il faut abolir la loi El Khomri , et je le ferai . Mais je m' y arrête encore un instant , pour bien me faire comprendre , et je voudrais que vous repartiez avec l' esprit critique dont vous allez avoir besoin pour ne pas vous laisser emballer dans le papier journal dans lequel il y a tous ces beaux articles pour ces belles personnes . Un peu d' esprit critique . Cet homme parle de &quot; préventif &quot; après avoir supprimé la médecine du travail . Quand on fait du préventif , comment le fait -on ? On va pas dire &quot; Dans chaque entreprise , discutez ! &quot; Ah bah non , on sait comment ça va se finir ! Parce que celui qui s' occupe de l' entreprise , son métier c' est pas médecin , c' est entrepreneur . Mais bien sûr , chacun fait d'après ce qu' il comprend et ce qu' il croit urgent . La seule chose qui protège tout le monde c' est la loi . La loi , le règlement , qui prévoit des durées de travail , je vais en parler dans un instant , etc . etc . Par conséquent , si vous voulez que la santé des gens soit protégée et qu' elle ne soit pas soumise aux aléas qu' on peut comprendre , de la nécessité d' une production qui fait qu' on se tourne vers un travailleur et on lui dit &quot; Bon écoute , c' est pas parfait , bon d'accord il manque un clapet pour la sécurité &quot; ou bien &quot; Bon c' est vrai , il y a beaucoup de poussières , enfin , bon , tu vois bien , il faut qu' on rende le produit si on veut que la boîte tourne . &quot; Et le gars ou la fille le fait parce qu' il y a une chose que tous ces gens oublient , c' est que les gens aiment leur métier . Quel qu' il soit , et ils aiment faire bien leur travail quel qu' il soit . Ca c' est une constante de tout le monde , les gens font pour le mieux . Alors , si vous vous soumettez , les décisions concernant la santé , les discussions dans les entreprises , ça va être vite réglé ! Surtout dans les entreprises où c' est les gros durs . Parce qu' en plus ça se fait pas de dire qu' on n' est pas bien , que c' est trop lourd , ça se fait pas . Alors on dit rien et la vie va comme ça . Donc c' est la loi qui vous protège . Où est la loi ? Avant , il y avait la loi et puis la convention collective de branche et puis l' accord dans l' entreprise . C' était impossible d' avoir un accord dans l' entreprise qui soit moins bon que l' accord de branche et l' accord de branche pouvait pas être moins bon que la loi . Ça s' appelle le principe de faveur . On a lutté , tous les syndicats ont lutté pendant des décennies et des générations pour obtenir cet ordre là . Extraordinaire mes ami(e)s , et quand dans une branche on avait une bonne négociation , la loi étendait le bénéfice à tous les travailleurs . Raison pour laquelle , dans notre pays , on est pas obligé d' être syndiqué pour bénéficier d' un accord syndical . Tandis que quand vous allez en Allemagne vous ne bénéficiez pas d' un accord syndical si vous n' êtes pas membre du syndicat . Et si votre patron n' est pas membre du syndicat patronal qui a signé avec le syndicat ouvrier , vous n' avez pas l' avantage . Chez nous , si , tout le monde . C' est donc ça notre démocratie sociale , elle est fragile , elle est délicate . C' est un mouvement entre le bas et le haut et à la fin c' est la loi . Parce que dans toute situation , il y a au moins 3 partenaires : le patron les ouvriers les salariés , les employés , de tous niveaux ; ce que je raconte là c' est pas simplement à la mine , c' est aussi au bureau . Et c' est aussi dans les professions les plus avancés intellectuellement aujourd'hui qui nécessitent un stress psychologique terrible . Bon , ça se passait dans cet ordre là . Eh bien , maintenant ils ont décidé que tout pouvait se négocier dans l' entreprise . Et l' entreprise peut avoir un accord moins bon que la loi , moins bon que l' accord de branche et les mêmes reviennent vous dire qu' il faut faire de la prévention alors qu' ils ont organisé toutes les conditions pour que tout ça se règle par le bas . Et d' un abominable dumping , je vous demande d' y réfléchir , vous savez ce qu' est le dumping , c' est qu' il y en a un qui vend quelque chose moins cher que l' autre et comment il s' est obtenu ce moins cher ? En payant moins cher le salarié . On comprend tous ça , c' est pour ça qu' on est tous contre le statut de &quot; travailleur détaché &quot; . C' est pas la nationalité du travailleur détaché qui compte , ce qui compte c' est qu' on l' ai détaché et qu' on l' ait amené pour faire concurrence sur le marché de l' emploi , et pour obliger à régler par le bas . Qu' est ce qui se passe ? Eh ben si l' un produit à moins cher parce qu' il a abaissé les droits sociaux , qu' il vous a fait travailler plus longtemps , qu' il ne vous paye pas les heures supplémentaires comme avant , alors celui qui est à côté , qui produit la même chose , dans le cadre de la concurrence , qu' est -ce que vous croyez qu' il va faire ? La même chose . Et il viendra vous dire , un par un à votre poste de travail , &quot; Je ne peux pas faire autrement &quot; et vous , vous allez pas dire &quot; Bah on le connait , c' est le patron , c' est une petite boîte ! &quot; et voilà comment tout va et tout se nivelle par le bas . Voilà pourquoi , quand ils ont détruit la hiérarchie des normes - c' est comme ça que ça s' appelle - la loi , puis l' accord de branche , puis l' accord d' entreprise , ils ont détruit tout un ordre social ; et vous allez voir , parce que depuis le 1er janvier seulement s' applique la loi , mais c' est dans toutes vos entreprises , par un accord d' entreprise qu' on va discuter du montant de vos heures supplémentaires - qui de toute façon vont descendre jusqu' à 10 % . Parce que dans l' entreprise où on aura laissé à 15 % , eh bien , celle d' à côté sera à 10 % et alors ils baisseront à 10 % . Et la loi permet cela . Et encore , tenez -vous pour content . Mordez -vous les doigts mais pensez que le suivant vous fera pire . La Madame Le Pen qui soi-disant représente les ouvriers , le Monsieur Fillon qui vous a déjà collé 1 million de chômeurs sur les bras du temps qu' il était premier ministre , ces deux-là disent qu' il faut en finir avec les 35 heures ; mais ils sont pas bien renseignés : il y a longtemps que les 35 heures elles vont mal . Pourquoi ? Parce que 35 heures ça vaut à partir de 35 heures et 1h si on vous paye en heures supplémentaires ; si on vous le paye plus en heures supplémentaires , la durée légale elle sert à quoi ? C' est pourquoi ils ont été au bout , ils ont dit &quot; Ca sert à rien &quot; , donc il y en aura plus ! Et s' il y a plus de durée légale hebdomadaire du travail , ce que Monsieur Fillon vous a annoncé d' avance et quelques autres et la Madame Le Pen qui dit qu' on aura droit de discuter ça dans l' entreprise , vous avez compris que vous allez encore vous faire blouser . Parce que si ça se discute dans l' entreprise , la discussion va avoir vite fait , elle va être vite réglée . Vous le savez comme moi . Eh bien ça veut dire qu' il y aura plus de durée légale hebdomadaire du travail , sinon la limite qui a été proposée par l' Europe : 48h . Et de 35 à 48 , pour ce qui est de la paie &quot; heures supplémentaires &quot; , rien du tout . Voilà ce qu' ils ont prévu , voilà ce qu' il y a dans les cartons . Voilà pourquoi je demande à chacun d' entre nous d' aller au-devant des autres , de se présenter devant l' ami , le camarade , celui ou celle dont on sait que les élections bla bla , c' est tous les mêmes , bla bla ! Il y a alerte mes ami(e)s ! N' écoutez plus les bonimenteurs , regardez de près ce qui va vous arriver si vous ne faites pas le choix qu' on tourne la page de cette période épouvantable où on a cru qu' en compressant les salaires et en poussant sans arrêt les gens dans le dos , en leur tordant les bras , on ferait décoller , magiquement , l' économie et la production . C' est tout le contraire qui s' est produit , ça fait 20 ans que ça dure ! Est -ce que c' est pas le moment d' arrêter ça ? Est -ce que c' est pas le moment de dire que le problème c' est celui de ceux qui se gavent ! Ce n' est pas le problème de l' immigré ! Ce n' est pas le problème de celui qui n' a pas votre religion ! Le problème c' est le banquier , le problème c' est Mulliez ! Voilà le problème : il a un nom , il a une adresse . On peut changer tant et tant de choses , et en particulier ces salariés et ouvriers , ces employés à tous les niveaux au bureau et à l' atelier . Nous allons avoir un besoin immense de mobilisation de tout le monde , parce que nous allons faire changer de haut en bas toute notre manière de produire pour faire la grande transition écologique dont l' humanité a besoin ! Pas seulement vous , sur vos postes de travail pour cesser d' absorber des produits nocifs , pas seulement les Français parce que nous nous occupons de notre pays , mais l' humanité toute entière a besoin que les pays les plus avancés , ceux où le niveau de culture et d' éducation est le plus élevé , ceux-là montrent l' exemple , marchent les premiers , mettent au point les processus de production , les machines , les process d' une façon générale , qu' on pourra ensuite proposer aux autres par transfert de technologies , pour que les pays où l' on est riche et ceux où on est pauvre , TOUS aient les moyens de manger correctement des produits de bonne qualité , d' avoir de l' énergie gratuite et non dangereuse ! Si j' en appelle à la classe ouvrière aujourd'hui - puisque je suis venu ici pour ça - c' est pour lui dire non seulement qu' elle a tort de se laisser maltraiter de cette façon et qu' elle a l' occasion de se défendre avec ses bulletins de vote . Mais aussi pour lui dire : &quot; Nous avons besoin de vous tous ! &quot; Il va falloir se mettre au travail dans des conditions complètement différentes , et ce changement viendra à la fois d' en haut par la loi , par les ingénieurs , les techniciennes et les techniciens , mais il viendra aussi de tous ceux qui sont sur leur poste de travail et qui devront faire , eux , leurs propositions pour les modifier , ces postes de travail . Pour signaler où sont les dangers , parce que vous savez où ils sont les dangers . Ceux qui respiraient de la poussière de charbon savaient qu' ils allaient mourir de la silicose . Combien vous en avez eu ici ? Combien dans cette région ? Et ensuite celui qui était au bout du tapis roulant dont les autres pensaient qu' il avait la bonne place , parce qu' il était pas sur le front de taille - quoique la position était fort dure à l' époque - celui-là respirait toutes les semaines de sa vie , en plus de la poussière de charbon , a respiré de l' amiante parce que les courroies qui faisaient tourner les tapis étaient en amiante , et qu' elles s' usaient à la cadence d' une par semaine . Ce temps peut paraître lointain , mais faites le tour autour de vous et vous savez bien que certaines des productions auxquelles vous participez ne sont pas tenables dans la durée pour votre santé ou pour les gens qui utilisent les produits qui sortent dans ces conditions . Le meilleur agent d' alerte et de transformation écologique , c' est la classe ouvrière ! Il n' y a pas de contradiction entre les deux , quoi que croient certains . Bon , maintenant je rentre . Et je vous retrouve juste après . *musique* Nous sommes voués à un idéal et ils savent bien qu' à cet instant c' est mon tour . J' incarne - pour vous - , je porte AVEC vous . Clairement , vous ne vous attendiez pas à venir à une réunion pour entendre chanter des &quot; Alléluia &quot; à la flexibilité , à la compétitivité et tous les autres mantras des néo-libéraux . On parle de la condition des salariés . J' ai évoqué les plus démunis , les plus désarmés , je vais reprendre un cas plus général . Si on pouvait à peine baisser le son , ce serait parfait pour moi parce que c' est très assourdissant . J' ai traité de questions générales , mais commençons par le commencement pour nous tous . Quand on examine ce qui fait la vie au travail , la vie du travail des salariés ... La première chose c' est d' avoir du travail . Et du travail , il y en a beaucoup qui est nécessaire . Mais tout le travail nécessaire n' est pas aujourd'hui rentable . C' est dommage , il faut donc changer la manière de regarder . Au lieu de partir d' abstractions - produire n' importe quoi , n' importe comment , dans n' importe quelles conditions , du moment que c' est pas cher , et qu' on finira par trouver quelqu'un pour l' acheter - prenons les problèmes par l' autre bout . De quoi a -t-on besoin ? Que faut -il faire ? Quelles sont les tâches nécessaires à ce moment ? Non seulement de l' histoire de notre pays , de l' histoire du monde ? C' est par là que j' ai terminé dehors . Le moment c' est celui de la transition écologique du modèle économique . Ce n' est pas un slogan , c' est une tâche collective , que le pays tout entier doit porter , exactement comme il l' a fait , après avoir été envahi et détruit il a tout reconstruit , et pour tout reconstruire , nos anciens avaient eu le pressentiment qu' on ne pourrait pas reconstruire , qu' on n' aurait pas cet enthousiasme collectif si on ne crée pas les conditions qui permettent à chaque personne de maîtriser sa propre vie et d' être protégé , ou en tout cas accompagné dans les grands défis de la condition humaine - le vieillissement , la maladie - et les bonnes nouvelles : les enfants ... Et c' est par cette solidarité accrue qu' on a redéployé la machine à produire et qu' en quelques années tout à été rattrapé , reconstruit et la société complètement changée . J' avance dans cette campagne avec cet état d' esprit : d'abord , tout ce que nous aurons à faire , je viens de le résumer ; mais je ne vais pas passer à côté d' une question extrêmement importante : c' est celle de la durée du travail . C' est un fait : depuis 1870 , la quantité de travail nécessaire pour produire la richesse que nous produisons a été divisée par 2 . Et la richesse qui a été produite par cette quantité de travail totale divisée par 2 a été multipliée par 30 . Et c' est une très bonne nouvelle ! Les machines , les perfectionnements des processus de production permettent qu' avec une quantité de travail humain moins grande , donc moins de peine , moins de souffrance on produise bien plus que ce dont on a besoin , car notre société est avant toutes choses , une société du gaspillage . La diminution du temps de travail , soit on la laisse se dérouler d' une manière sauvage - c' est à dire &quot; on verra bien &quot; - et elle a besoin de moins , donc on prend moins de monde et les autres on s' en fout . &quot; C' est pas mon sujet &quot; dit l' un , &quot; C' est pas mon sujet &quot; dit l' autre , et vous avez d' un côté des gens qui travaillent beaucoup , qui s' usent au travail et de l' autre d' autres qui n' ont pas de travail et qui souffrent , qui souffrent ... Je veux rester dans mon thème d' aujourd'hui : qui souffrent ! Selon l' INSERM - qui n' est pas une organisation &quot; bolchévique &quot; , mais un institut de recherche sur la santé et les médicaments - 14.000 personnes par an meurent du fait du chômage . Vous m' entendez ? 14.000 ! Par la dépression , par le stress , par l' abandon de soi quand on ne sait plus quoi faire . Le chômage de masse est une souffrance de masse . Je le dis pour rappeler à tous les petits intelligents qui passent leur temps à expliquer qu' il faut stimuler l' envie de retour au travail qu' ils font insulte à des millions de personnes en parlant de cette manière-là . Et que quand un Fillon et quelques autres s' imaginent qu ' en mettant des taux dégressifs d' indemnisation , on va - en vous prenant à la gorge - vous pousser à trouver du travail comme si cela dépendait de vous . Comme si un seul jour vous vous étiez dit &quot; Je n' y vais pas parce que ça me fait plaisir &quot; , ce n' est pas vrai . Chaque fois que quelqu'un vous dit qu' il va faire des taux dégressifs , avec sa belle bouche , son air profond , qui réfléchit aux problèmes auxquels vous n' êtes censés ne rien comprendre , rappelez -vous que celui-là est en train de vous traiter de fainéant , rappelez -vous que celui-là est en train d' insulter celui qui dans votre famille se désespère de ne pas avoir de travail , et allez pas voter pour des marioles pareils ! Ou alors , vous vous faites injure à vous-même . Ensuite , souvenez -vous que la dégressivité , ils ont déjà essayé de la faire une fois , et ça a été fort mal accueilli . Parce que nous autres on sait aussi autre chose , c' est que la moitié des gens qui sont au chômage n' ont aucune indemnité . Aucune ! Alors leur sujet cela n' a pas été de se dire &quot; Comment on pourrait faire pour que les gens , du moins , ne soient pas dans une situation si épouvantable ? &quot; Non , ça a été de se dire : &quot; A ceux qui ont quelque chose , on va voir comment on va pouvoir leur enlever . &quot; Voilà ce qu' est leur manière de voir . Et quand on parle de diminution de temps de travail , pour qu' il y ait une répartition sur l' ensemble des travailleurs , l' ensemble du corps productif , ils disent : &quot; Oh là là , ça va coûter cher , c' est ruineux &quot; ... Puisque j' ai la parole autant que vous sachiez . On a créé 350.000 emplois avec les 35h . Chacun de ces emplois a créé en aide 13.000 euros . C' est un rapport parlementaire , il y a donc la majorité et l' opposition . Le crédit d' impôt compétitivité , on pense que ça a créé ou que ça a aidé à sauvegarder de 50.000 à 100.000 emplois . Nous , 355.000 , eux , 50.000 à 100.000 emplois . Et combien ça nous a coûté ça ? 28 milliards , c' est à dire 280.000 euros d' aide publique par &quot; emploi &quot; créé par le CICE , 13.000 par les 35h . Autrement dit , l' argent qu' on a versé dans ce puits sans fond que sont la marre aux réclamations et aux pleurnicheries du CAC40 , la plus grande organisation d' assistés par ce pays a été ruineux pour rien ! Si Monsieur Gattaz avait tenu parole , s' il avait créé son million d' emplois , et bien même dans ces conditions-là - quoi qu' aucun d' entre nous n' ait cru un seul jour aux clowneries de cet homme , avec son badge 1 million d' emplois - eh bien même dans ces conditions là , compte tenu de ce que ça coûte , chacun de ces emplois en question aurait coûté 3 fois ce que coûte un emploi aidé pour qu' on passe à 35h . Vous n' avez donc aucun complexe à avoir . Mon discours d' aujourd'hui , c' est un discours pour la fierté , pour le fait qu' on relève la tête , qu' on arrête de se soumettre à leur récitation qui vise à nous culpabiliser , à nous humilier , pour nous conduire à faire toujours davantage comme ils veulent . Et davantage moins , toujours moins par rapport à ce que nous voulons faire de nos propres vies . La diminution du temps de travail est une nécessité . Je conviens qu' il faut en discuter , il y a la diminution du temps de travail , dans la vie ( c' est l' âge de la retraite ) , dans l' année ( puisque certains ont un contrat de travail avec une annualisation de leur temps de travail ) , il y en a ici ? Non ? Eh bah je vais vous la raconter , non ? Je vais pas vous la raconter , vous le savez . Mais l' annualisation du temps de travail a permis de généraliser le forfait qui autrefois s' appelait le &quot; forfait jour &quot; , il y avait que les cadres qui avaient ça , parce que comme ils avaient de l' autonomie on a dit : &quot; Bon , calculons gros , déjà tu es bien assez payé ? Tu n' es pas content ? Tu peux t' en aller &quot; &quot; Ah non , je suis content ! &quot; Et on a étendu ça progressivement à tout le monde . Je le sais , parce que quand on a voté la loi sur les 35h , j' étais parlementaire , donc je les ai vus - et j' étais au Sénat hein , c' est le cul-de-basse-fosse de la réaction , donc je peux vous dire que dès qu' il s' agit de lâcher une minute alors là c' était pas possible . Et donc , moi , j' ai déposé un amendement contre le forfait-jour , c' était pour les cadres . J' ai dit : &quot; Je sais comment ça va terminer : non seulement vous allez sur-exploiter les cadres , mais à la fin vous sur-exploiterez tout le monde . &quot; Et c' est bien ce qui s' est produit . Donc la durée du travail dans la semaine , la durée de travail dans l' année , la durée de travail dans la journée , tout ça , ça était une longue bataille mes ami(e)s , ça a pas été une discussion tranquille entre gens bien élevés . Ça a été une bataille , avec un certain moment des morts parce que la troupe tirait sur la foule qui venait réclamer la limitation de la durée du travail . Vous voyez le triangle rouge que je porte là ? C' est celui des déportés socialistes , communistes , syndicalistes par les Nazis ; mais pas que ça , c' est le triangle rouge que portaient les ouvrières et les ouvriers du premier 1er mai en France . Ils avaient mis un petit triangle de cuir rouge , 8h de loisirs , 8h de sommeil , et 8h de travail . La question du temps de travail , elle est essentielle , pourquoi ? Parce qu' elle a avoir avec la façon dont on répartit la richesse . Vous voulez voir le lien ? Je vous l' explique . Le système capitaliste fonctionne sur la gratuité . La gratuité de votre travail . Si vous faites 10h , on va faire rond , ils vous donnent pas la valeur que vous avez produit en 10h . Ils vous donnent 2h ou 3h et tout le reste c' est pour quelqu'un d' autre . Pour payer le matériel , les locaux , les matières premières et les dividendes et l' investissement . Suivant comment vous répartissez cette richesse , ça peut être du temps de libre pour les uns , de l' amélioration des salaires , etc . etc . Nous sommes donc collectivement riches - en oubliant quels sont les rapports de propriété - de 2.000 milliards d' euros par an que produit ce pays . La question qui est posée , c' est exclusivement la question de la répartition et du partage . Comment on partage ça ? Alors la règle actuelle , c' est tout pour les uns , rien pour les autres . Ou le minimum de ce qu' on est obligé de leur lâcher . En 10 ans ... Non , en 30 ans , dans l' ancienne répartition entre le travail et la capital , 10 points sont passés des poches du travail à celles du capital . Du coup , deux choses : nous sommes un des pays d' Europe où il y a le plus de millionnaires . On sait pas pourquoi , il y aussi de plus en plus de pauvres . Plus il y a de millionnaires , plus il y a de pauvres . Il doit y avoir un rapport peut-être ? Un jour quelqu'un m' a dit : &quot; Oh vous faites du misérabilisme , moi ce qui me pose un problème c' est pas qu' il y ait des riches , c' est qu' il y ait des pauvres &quot; . A cette altitude de pensée , je ne peux plus suivre . Mais si vous arrivez pas à comprendre que certains ont beaucoup parce que les autres ont rien , il faut retourner à l' école . Et savoir comment on fait les additions , les multiplications et les divisions . Donc , dans la répartition , une quantité croissante va aux uns , alors on vous dit : &quot; Et alors ? Ils sont plus intelligents , ils ont des idées , ils ont beaucoup d' argent et avec ça ils savent où mettre l' argent , tandis que vous ne savez rien &quot; . &quot; Eux ils savent comment il faut faire et tout &quot; , mais c' est pas vrai . La preuve , c' est que d' année en année , sur cette part qu' ils ne donnent pas au travail , sur cette part qui ne va pas pour les bâtiments , sur cette part qui ne va pas pour la matière première , sur tout ce morceau-là , une fois qu' il est là et qu' il faut le partager entre le futur et le présent ... Le présent : leur jouissance immédiate ; le futur : les machines , les process de production ; eh bien d' année en année , à mesure qu' il y a de plus en plus de gens très riches , la proportion des dividendes s' accroît et ils se payent toujours mieux . Nous sommes le record d' Europe des dividendes , bande de paresseux ! Alors , tout le temps en train de râler ! Eh bien vous voyez , il y a des gens qui réussissent . Et c' est pas tout . Non seulement ils prennent de plus en plus , mais tout ce qu' ils prennent , ils le mettent pas dans l' investissement . Si bien que maintenant , il y a bien plus d' argent qui part en dividendes ( c' est ce qu' on donne a quelqu'un qui a des actions dans une société ) qu' il y a de l' argent qui part dans l' investissement . Autrement dit , petit à petit les mêmes que vous entendez pleurnicher continuellement sur le fait que leur taux de marge n' est pas suffisant et tout un tas de baratin qui sont autant de menteries , les mêmes sont ceux qui n' investissent pas , ne prévoient rien pour l' avenir . Et les grands entreprises du CAC40 -parce que je lis leur paperasse quand même pour savoir - que disent -elles ? &quot; Ah bah nous , nous attendons une indication de tendance du marché &quot; . Mais le marché c' est vous ! Qu' est -ce que vous attendez ? Que ça soit l' épicier du coin ? Le plâtrier du village ? Le maçon du bourg qui crée la tendance ? Non c' est vous qui avez les grandes entreprises et qui pouvez vous lancer dans des grands plans . Eh bien puisque vous ne le faites pas , nous allons le faire nous . C' est ça le sens de la planification écologique . D'abord parce qu' il faut organiser , d'abord prendre conscience du danger . Prendre conscience du danger , vous autres , si vous faites campagne avec moi , passez du temps a faire comprendre que cette fois -ci c' est pas un choix parmi d' autres . Nous devons relever un défi pour l' humanité toute entière ; une fois qu' on a compris ça , eh bien il faut organiser , articuler les uns avec les autres la production , la formation et l' argent qu' on investit . Et si nous le faisons , alors nous allons remettre en marche toute la mécanique , parce qu' eux ne le font pas . Eux prennent pour les dividendes . Eux prennent pour placer dans la sphère financière , parce que dans la sphère financière vous pouvez spéculer et gagner du 10 , du 12 , du 15 % , aucune production jusqu' à présent ne le permettait , sauf à tuer la machine et à tuer les ouvriers . Cet argent qui s' en va dans la sphère financière , naturellement nous allons couper les cordons par lesquels ils passent , parce que c' est toujours de l' argent qui s' accumule d' une manière particulière , c'est-à-dire de l' argent qui pour une part échappe à l' impôt , qui pour une autre part est spéculatif . Et enfin c' est beaucoup d' argent qui circule dans le monde , mes ami(e)s . J' ai pas le temps , c' est pas le sujet aujourd'hui . Beaucoup , beaucoup , beaucoup d' argent . Et qui circule sans aucun rapport avec la production réelle . Sans aucun rapport avec la production réelle ! A l' heure à laquelle je vous parle , le système bancaire lui-même s' en alarme . Comment expliquez -vous que le banquier central européen - qui a décidé de mettre 2.000 milliards , qui en verse 80 tous les mois ... Tous les mois le banquier central donne 80 à 85 milliards - le banquier central c' est celui de la BCE - aux banques pour qu' elles consolident leur bilan et qu' elles placent dans l' activité . Où vous avez vu la queue d' un euro de ces 85 milliards ? Vous savez ce que ça représente ? Chaque mois ? 6 millions de SMIC ! Ça devrait se sentir quand même quelque part ... Même si c' est amorti en cours de route par des gens qui prennent . Rien ! Ça signifie que l' essentiel de cette masse énorme d' argent est directement repartie dans la sphère financière . Vous comprenez pourquoi , quand dit que nous allons lutter et lutter d' une manière implacable contre la financiarisation de l' économie , ce n' est pas un slogan idéologique , seulement ... Bien sûr , c' est un slogan idéologique ; quand je vous dis &quot; être milliardaire c' est immoral &quot; , je reconnais : c' est un choix moral , un choix philosophique , c' est un choix éthique , ce que vous voulez . Mais c' est surtout un choix concret , pratique . On ne peut pas continuer comme ça , et surtout pas nous les français qui sommes un peuple jeune , avec de plus en plus de jeunes . De ce chapitre de mon discours , je veux que vous reteniez une chose : 1 ) il y a besoin de travail , 2 ) il y a l' argent , 3 ) on sait le faire , 4 ) c' est urgent qu' on le fasse . Voilà pourquoi , moi , quand je parle du discours sur le travail pour le condamner , je parle des salariés . Je reviens à une idée toute simple : c' est le progrès social qui entraîne le progrès économique . C' est le progrès écologique qui entraîne le progrès économique . Ce n' est pas le contraire ! Alors ... J' ai parlé du temps de travail , voici ma position et celle du programme &quot; L' avenir en commun &quot; . Très bien , merci ( rire ) . Premièrement , retour aux 35 heures , abolition de la loi El Khomri . La loi El Khomri est inacceptable , inefficace et brutale . C' est bien que vous applaudissiez : allez demander des comptes aux autres , parce que pour ce qui est de bavarder tout le monde est là ... Mais pour faire , c' est une autre paire de manches . La loi El Khomri , c' est comme la 6ème République , la planification écologique : c' est pas à moitié , c' est ou pour ou contre . Voilà , nous on est contre , on l' abrogera . Et les autres doivent dire un par un ce qu' ils comptent faire . Et sûr : après avoir abrogé la loi El Khomri , il y a un retour aux 35h . Je commence par les 35h , les vraies 35h . C' est pas tout d' écrire dans un papier &quot; durée légale , 35h &quot; . Ah oui ? Et la 36ème heure ça vaut combien ? Eh bah ça vaut rien . C' est ce qu' a fait la loi El Khomri . Si vous payer la 36ème heure 10 % de plus , ça ne coûte rien , ça crée pas 1 emploi , bon ça vous rapporte 10 % de plus , mais jusqu' à présent c' était 25 % , non ? Eh bah maintenant c' est 10 % . C' est bon d' avoir un gouvernement de gôche ! D'accord ? Donc il faut renchérir le paiement des heures supplémentaires pour que ça crée de l' emploi et que aussi les gens puissent vivre avec un salaire digne , qui leur permet avec 35h de vivre dignement . Car la vie c' est pas que le travail salarié il y a aussi tout le reste de l' existence qui souvent comporte beaucoup de travail aussi , à la maison , avec les enfants et le reste tout ce qui vous passe par la tête parce que vous n' êtes pas les rouages d' une machine . Vous êtes des êtres humains ! Vous êtes des êtres humains ! Dites bien aux gens , dites -leur bien , vous v'là prévenus . Venez pas après pleurer . Si vous recommencez avec des Macron , des Fillon , des &quot; je sais pas quoi &quot; , de tous ces gens qui ont la même doctrine . Simplement qui vous emballent la marchandise d' une manière différente exactement pour faire avec vous comme fait le marchand d' Auchan quand il vous met des yaourts différents dans les rayons , que c' est pourtant toujours la même chose . C' est ça que font ces gens . Si vous avez Fillon , il n' y a plus de durée légale du travail , vous avez bien compris ? Expliquez -le aux gens . Plus de durée légale du travail , ça veut dire qu' il y a plus d' heures supplémentaires et que quand vous serez annualisé et au forfait-jour vous travaillerez aussi longtemps que celui qui a le carnet de commandes en main aura décidé de vous faire travailler , et vous n' aurez aucun de moyen de vous défendre - sinon d' invoquer le texte européen qui dit qu' on ne peut pas aller au delà de 48h , ce qui nous les français , nous ramènerait au siècle précédent et au début du siècle précédent . Le temps de travail est une question extrêmement sérieuse , ce n' est pas seulement sa durée et son intensité qui est en cause , c' est la façon avec laquelle il roule . Vous savez comme moi que quand vous avez un horaire discontinu , que vous avez d'abord fait 3 ou 4 heures et qu' ensuite votre poste de travail reprend , après une pause de 3 ou 4 heures , la pause en question ne vous est pas payée , et vous tournez en rond . Par conséquent , quand on arrive à faire une journée , il faut que tout le monde puisse la faire dans des conditions où il maîtrise son emploi du temps . La loi El Khomri prévoit maintenant qu' on peut vous prévenir 3 jours avant que votre emploi du temps a changé . Qui peut changer l' organisation de son existence 3 jours avant ? Comment on fait 3 jours avant pour trouver la nounou si il faut ? Changer l' heure de qui va aller chercher le môme à l' école ou à la crèche ? Quelle vie ils ont , quelle vision ils ont de la vie des gens ? Qu' est -ce qui compte d'abord ? Nos gamins ou leurs foutues marchandises ? Bien sûr que ce sont nos gosses et nos vies . Il faut il faut en revenir à la semaine de 7 jours avec 2 jours de congés consécutifs . Et pour tout le monde au même moment . Une fois j' avais une camarade , qui est super sympa , une socialiste , et elle me dit &quot; Rah , voilà comment vous êtes vous , vous voulez tout planifier , organiser , contrôler ! Après tout si les gens veulent faire autrement ? &quot; Mais les gens ils peuvent vouloir ce qu' ils veulent , c' est pas ce qu' on veut nous . Ce qui compte c' est quel genre de vie on veut organiser . Si vous commencez à multiplier le travail le dimanche , bien sûr qu' au début vous avez une personne ou l' autre que ça arrange dans sa vie , je le sais bien . Mais il y a surtout une masse qui n' a pas les moyens de défendre , à qui on va dire &quot; Bah , tu viens dimanche . &quot; &quot; Ah bah c' est pas ce que j' avais prévu ! &quot; &quot; Eh bah si ! Sinon tu peux t' en aller &quot; Et celui-là obligé d' y aller , et celle-là est obligée d' y aller , et plus vous êtes pauvres , plus votre situation est difficile , plus vos horaires de travail sont discontinus , plus vous avez besoin d' heures complémentaires . Les heures complémentaires c' est le rêve de celle ou celui qui n' a pas un temps de travail complet , il passe son temps à espérer qu' il va pouvoir en faire un peu plus . Et en général c' était un peu mieux payé , mais grâce à la loi El Khomri c' est fini . Et les autres se proposent de faire pire avec leur suppression de la durée légale . Vous voyez ? Voilà comment tout ça s' organise . C' est cette brutalité là qu' ils répandent , les horaires de travail et la semaine , elle doit être la même pour tous le monde . Sinon vous transformez votre maison en un hôtel-restaurant . Les uns vont au boulot , les autres sont là , et puis ceux qui sont là s' en vont à l' heure à laquelle les autres arrivent . Et vous savez bien comment ça se passe , quelqu'un devient le domestique de tout le monde dans cette histoire . C' est pas la peine que je dise qui , on connait . C'est-à-dire que tout ça , ça ne fait qu' une chose , d' un bout à l' autre : aggraver la situation des femmes au travail , et tout ça est fait avec le propos et l' intention délibérés de faire que les femmes retournent à la maison , pour que leur poste de travail soit occupé par des hommes et qu' on obtienne de cette manière la baisse du chômage - parce qu' il savent que ça va finir par sauter . Tout ça est fait d' une manière très méthodique , et va toujours dans le même sens . En ayant raccourci les délais pour prévenir d' un changement d' horaire , c' est d'abord les femmes qu' on élimine du travail . Avec l' exemple que j' ai donné tout à l' heure , avec l' imposition du travail du dimanche , c' est d'abord les femmes qu' on élimine du travail . Parce qu' elles savent ce qu' il faudra sacrifier à ce moment-là . Voilà comment , au bout du bout , comme toujours , la violence d' une situation se décharge sur le plus faible , sur celui qui a le moins de droits . C' est pourquoi je m' oppose - et le programme s' oppose - à la continuation du travail du dimanche . Je vais vous parler maintenant des droits des salariés . J' ai parlé du temps de travail , j' ai parlé de la répartition de la richesse , sur les grands objectifs qu' on avait : la transition écologique , mais évidemment ça veut dire aussi la répartition dans les salaires . Clairement , on peut pas continuer comme ça . Vous avez un SMIC qui est à 100 euros de plus que le seuil de pauvreté ! Non mais ils sont toujours en train de venir pleurer sur le fait que certain qui ne travaillent pas , obtiendraient autant que ceux qui travaillent . Ça leur coûte rien de raconter des salades , ce n' est jamais vrai . Mais ça permet qu' on se batte entre nous , ça fait qu' on regarde par-dessus son épaule dans l' assiette de l' autre , au lieu de regarder ce qu' il y a dans celle de celui qui vous donne ce genre de conseils . Parce que comment ça se fait qu' un pays aussi riche , où il y a autant de millionnaires , et où la richesse a autant cru , que les gens qui sont payés au minimum sont payés 100 euros de plus que le seuil de pauvreté ? C' est quand même ça le problème , non ? C' est pas l' autre le problème , c' est ça le vrai problème . Le seuil de pauvreté , ça se calcule de la manière suivante : on prend 60 % du salaire médian , c' est une norme internationale , c' est pas nous qui avons inventé ça , donc tout le monde fait pareil . L' autre jour j' écoutais là , des fois faut écouter , on est pas obligés , j' ai écouté . On me dit &quot; Alors qu' est -ce que vous en pensez , le salaire minimum européen ? &quot; De quoi vous me parlez ? &quot; C' est Monsieur Valls qui a dit ça &quot; Je le connais , c' est un réac , il n' a pas pu dire ça . Donc , je vais chercher le papier , mais c' est comme pour tout , il faut jamais croire ce qu' on vous dit qu' il faut croire . Regardez d'abord , vérifiez . &quot; Ah non , ah non , attendez c' est pas un Smic européen c' était un Smic dans chaque pays ? &quot; Ah , bah attendez , les élections présidentielles c' est en France , pas en Bulgarie . Donc on parle de ce qu' on fait ici , si vous allez proposer un SMIC dans chaque pays européen pourquoi pas après tout , on pourrait faire un critère de convergence . Mais c' est pas ça qu' il dit . &quot; On va proposer un SMIC dans chaque pays européen qui soit à 60 % du revenu médian . &quot; Hop , on fait les calculs et bah vous vous êtes fait avoir , une nouvelle fois , dans le papier Europe , qu' est -ce qu' il y a ? Une baisse du SMIC . Parce que 60 % du revenu médian pour nous ça fait moins que ce que ça fait aujourd'hui , voilà . Moi je ferai le contraire , j' ai pas peur , nous allons , nous , augmenter le SMIC . Parce que notre objectif c' est d' être à 1.700 euros brut pour commencer , net pour terminer . Et on commence tout de suite avec 150 euros de plus . Des ami(e)s m' ont dit &quot; Attention , si tu fais ça on va avoir un problème dans la petite entreprise &quot; , je le sais . C' est la raison pour laquelle de tout ça on discutera bien tranquillement - sachant quelle est ma décision , qui ne changera pas - de la façon de la mettre en oeuvre . Plusieurs ami(e)s économistes ont eu une idée que j' ai trouvée brillante et que pour finir ils se sont accordés entre eux , pour la mettre dans le programme . C' est de faire une caisse commune des cotisations sociales qui permette de faire de cette manière là , de prendre sur l' ensemble du patronat la somme nécessaire pour financer l' ensemble des cotisations . De cette manière , on pourrait le faire reposer différemment sur les uns et sur les autres . Et on serait pas obligés d' écraser , en effet , par une cotisation qui serait trop dure à porter par la petite entreprise , ce que la grande entreprise porte sans difficulté d'autant que les grandes entreprises ont eu cette habileté - soi-disant pour s' occuper de leur cœur de métier - de tout découper en petits morceaux et de faire des petites entreprises qui sont souvent des entreprises qui n' ont qu' un seul client : c' est le cas de 80 % d' entre elles , si bien qu' on pourrait les considérer comme des succursales . Si bien qu' au bout du bout , non seulement la responsabilité du donneur d' ordres ultime ça doit être celui qui donne les ordres , mais c' est aussi lui qui doit payer . Je sais bien qu' il y a pleins d' ami(e)s qui disent : &quot; Ah mais il faut pas y aller trop fort ! &quot; Si vous ne voulez pas y aller trop fort , allez en trouver un autre . Parce que il faut bien regarder cette situation , je vous ai déjà dit 3-4 fois : la richesse qui s' accumule , etc . Je vous donne juste un chiffre , et après je verrai , vers la fin , ce que j' ai à dire . Quelque chose qui va bien , bien vous aider à réfléchir et à vous décomplexer . Au lieu d' avoir tout le temps peur que tout va se casser la figure , qu' on n' a rien le droit de demander , que c' est déjà bien comme ça , etc . Le 3 janvier dernier , ça vous dit quelque chose ? &quot; Non . C' était 2 jours après le 1er janvier . &quot; Peut-être il y en avait qui avait encore mal au crâne , j' en sais rien . Le 3 janvier dernier , à 18h , non plus ? Eh ben vous avez raté quelque chose . Parce que ce jour là , le 3 janvier dernier à 18h précises , les 120 plus grands patrons français ont déjà gagné plus que tous les français gagneront en 2017 . ( sifflets ) Ils leur suffit de 2 jours et 18h pour gagner autant que tous les autres , voilà . En 2016 , ils ont gagné 132 fois plus que le salaire moyen . C' est normal , parce que vous , vous êtes au salaire moyen , vous avez des besoins . Et eux ils ont des besoins 132 fois plus importants que vous . C' est facile à comprendre . Bon , je crois que je vous ai un peu préparé pour entendre le dernier chiffre sans que vous vous effondriez tout de suite en larmes . Le salarié moyen il a a gagné l' année dernière 72 euros par jour . Et le Monsieur ? Et la Madame ? 9 589 en moyenne . Voilà . Vous comprenez ? Alors si vous avez des complexes après pour dire &quot; on va changer la clé de répartition &quot; c' est que , vraiment , c' est mal parti . On peut pas faire fonctionner durablement une société dans un tel déséquilibre . On peut le faire qu' à une condition : c' est que vous soyez morts de peur . Et c' est pour ça qu' on fait cette réunion , qu' on fait ce rassemblement pour dire : &quot; On va faire reculer la peur ! &quot; Mais on va la faire reculer en regardant les possibilités qu' on a de changer notre vie complètement . De revenir à l' idée qu' on peut avoir une bonne vie , que ça vaut la peine de se lever le matin pour aller au travail parce que ça sera pas seulement pour y souffrir mais pour y faire quelque chose . D' important . Et ensuite gagner sa vie et vivre sa vie . En pensant à autre chose que de savoir comment on va boucler le budget pour la fin du mois , en pensant à autre chose que de savoir si on va payer ou le loyer ou l' électricité ou le chauffage . Cette abjecte comptabilité qui pourrit la vie des gens inutilement sans raison , sans raison parce que la richesse accumulée a été immense et qu' elle est là , et qu' on a tant de belles choses à faire . Alors ... Pour ça , il faut aussi , que les droits des salariés changent , qu' on cesse d' avoir une masse confuse avec peu de droits et aucune capacité d' intervention . Le grand Jaurès disait : &quot; La révolution de 1789 a rendu les français rois dans la cité , comme citoyens , mais elle les a laissés serfs dans l' entreprise &quot; . Ces droits ce n' est pas comme le croient les importants une addition de récriminations , ce n' est pas vrai , ils ne comprennent rien à la mentalité d' un salarié . Je l' ai dit tout à l' heure , la mentalité du salarié c' est qu' il aime son travail , le plus souvent . Il connait les moyens de l' améliorer et il le fait des fois à ses dépens . Les salariés doivent avoir des droits plus grands , notamment dans une situation : c' est celle où celui qui possède une entreprise décide de la vendre , ou bien que les tribunaux de commerce - que nous réformerons profondément , pour qu' on puisse parler à ce sujet de tribunaux , avec l' idée d' une justice , et pas seulement d' arrangements de plus en plus obscurs et inexplicables . Comme celui qui a conduit par exemple à préférer qu' on vende toutes les machines de l' entreprise Ecopla , dont vous avez peut-être entendu parler , qui est la dernière boite de ce pays qui fabrique des emballages aluminium . Il n' y en a qu' une ! Elle a été évidemment prise dans le cycle du démantèlement de la grande industrie de l' aluminium qui existait en France , et qui à l' époque s' appelait Pechiney . On découpait tout cela en petits bouts , et de petit bout en petit bout , racheté par l' un , racheté par l' autre , celui-là prenait les machiness l' autre prenait ci , enfin bref . Tout ça a été démantelé , et à la fin il en reste dans des coins des petites entreprises , qui autrefois participaient de cette grande entreprise ; et il y a celle-là : Ecopla . Les clients de cette entreprise disent : &quot; Ah bah nous , nous sommes d'accord pour continuer à vous acheter à vous &quot; - le carnet de commandes est plein . Entre-temps , ils ont été achetés par un fond de pension qui lui-même l' a revendu à un autre fond de pension , et maintenant voilà que le dernier propriétaire du fond de pension - qui a d'ailleurs disparu , il n' est même pas venu - a décidé de fermer la boutique . Les fonds de pensions ils font comme ça , ce sont des parasites . Ils viennent , ils achètent et puis au bout d' un an , une fois qu' ils ont bien regardé , ils achètent en empruntant de l' argent pour acheter , ils vident la trésorerie pour rembourser l' emprunt , ils foutent tout le monde dehors , et une fois qu' ils ont foutu tout le monde dehors ils vendent le terrain et le bâtiment , et ils en tirent au total ... C' est une bonne opération pour eux puisqu'ils ont siphonné la trésorerie , pris pendant un temps de la richesse , des fois ils vendent les brevets que la boite avait accumulés , etc . Donc une entreprise - mot avec lequel tous ces gens font tant de belles phrases - une entreprise c' est d'abord un collectif humain , un savoir-faire et une utilité sociale . Là , dans le cas de Ecopla , il y a tout , et les gens qui sont dans cette entreprise disent : &quot; Nous sommes d'accord , nous voulons nous-mêmes continuer cette entreprise , parce que nous , on sait comment la faire vivre , on sait travailler , on sait faire ça , les clients sont d'accord etc . &quot; &quot; Et on vous demande rien : un différé pour la dette sociale . &quot; Mais dans la dette sociale de l' entreprise , il y a les frais de licenciement des gens , donc si on les licencie pas , ça change déjà un peu la donne . Vous êtes d'accord ? Dans ce cas-là que s' est -il passé ? Le tribunal de commerce a dit : &quot; Pour payer la dette sociale , on vend les machines et on ferme la boîte &quot; . Donc on a payé , ils ont décidé de donner l' entreprise à un Italien qui , lui a annoncé d' avance qu' il resterait pas là et qu' il s' en allait , il prenait le matériel , le carnet de commande , il prenait les machines . Combien valent les machines ? A peu près la dette sociale . 150 millions . Je vous raconte cette histoire , je pourrais vous en raconter 10 comme ça , parce que les gens comprennent plus , c' est des braves gens , c' est pas des enragés , ils comprennent pas , ils disent : &quot; Mais ça c' est pas logique , c' est pas cohérent ! &quot; Et surtout ils sont prêts à faire tourner la boite , ils sont quand même 40 là-dedans . C' est pas rien , 40 familles , vous savez comme moi qu' il n' y a pas autant d' emploi autour de nous . Donc , pour moi , il y a un droit fondamental . Quand une entreprise est mise en vente , il y a un droit de préemption pour les travailleurs si ils veulent en prendre la propriété sous la forme d' une coopérative . Deuxième chose : les travailleurs d' une entreprise ont des idées sur leur entreprise . C' est la raison pour laquelle il faut leur donner 2 possibilités de contribuer à l' intelligence collective , en plus de celle que je viens vous donner . Premièrement , ils doivent avoir un droit de remontrance , c'est-à-dire de voter une motion de défiance à l' égard de la direction d' une entreprise dont le salariés se diraient : &quot; Ce qu' ils sont en train de faire là , ça va mal tourner , ce qu' ils sont en train de faire là c' est suicidaire , donc nous lançons collectivement une alerte &quot; et cette alerte pourrait être un document de départ obligeant tout le monde à discuter et à re-discuter des décisions qui ont été prises , troisièmement . Quatrièmement il est important que dans les entreprises où il y en a , les comités d' entreprise aient un droit de véto sur un plan de licenciement économique , jusqu' à ce que toutes les autres possibilités aient été examinées et vérifiées dans leurs viabilité . Voilà des droits fondamentaux qui introduiraient de la citoyenneté dans l' entreprise , ; voyez -vous , quelle est l' idée ? L' idée c' est que dans un collectif humain , on est plus intelligents tous ensemble que chacun séparément . Et qu' une entreprise c' est d'abord un collectif humain , et c' est d'abord à lui qu' on doit pouvoir faire confiance . Naturellement , c' est le contraire de toutes les brutalités que ces gens-là ont imaginées . J' ai donné quelques pistes . Le 11 prochain me voilà parti à Nantes pour parler de sécurité sociale intégrale . On a beaucoup parlé du travail , depuis tout à l' heure - enfin c' est moi qui parle . Les salaires , la durée , la façon de travailler dans une entreprise . Juste un mot pour terminer . Comme vous le savez , j' ai été ministre de l' enseignement professionnel . Je ne dis pas que je sais tout , loin de là . Mais je crois que je sais un certain nombre de choses sur le travail qualifié . Je n' entends jamais parler de ça . On ne parle jamais de qualification , comme si le travail était un don . D'ailleurs dans certaines catégories sociales , comme on se transmet de père en fils ou de père en fille le même attachement à un métier , on finit par croire que c' est un don familial , quasi-génétique . Celui -ci est médecin , sa fille ou son fils est médecin , ce sont de bons métiers , ou bien architecte ou bien je-ne-sais-quoi . On oublie que tout est métier , et que le niveau des métiers va s' élevant en exigence de qualification . Quand vous entendez quelqu'un dire : &quot; Il faut l' intelligence de la main &quot; , dites -vous que celui qui parle est un con . La main n' a pas d' intelligence ! L' intelligence c' est là , pas là . Et là , il y a tout le reste avec . Les niveaux de qualification pour une production moderne sont de plus en plus élevés . Et vous ne faites pas un grand pays développé uniquement avec des cadres supérieurs - il en faut , vous ne faites pas un pays développé uniquement avec des ingénieurs , il en faut ! Croyez -moi , il en faut . Vous le faites avec toute sorte de cadres intermédiaires et d' ouvriers et d' ouvrières hautement spécialisé(e)s . Allez voir un chaudronnier au travail , un chaudronnier ne fait pas de chaudrons , il fait des carlingues de bateau , d' avion , de train , c' est réglé au millimètre et des fois en dessous du millimètre , c' est un BAC PRO . On ne fait pas un grand pays sans beaucoup de BAC PRO et la question n' est pas sottement de &quot; revaloriser &quot; l' enseignement professionnel , pour la raison qu' il a toujours été valorisé . La preuve , c' est que ceux qui en sortent trouvent du boulot . La question est de savoir quelle place on donne aux qualifications . Si ils ne veulent pas en parler , c' est parce qu' ils ne veulent pas les payer . Je trouve que c' est une honte ce qu' ils ont fait ces dernières années . Entre le gouvernement de droite de Monsieur Fillon qui avait déjà réussi l' exploit de nous donner 1 million de chômeurs , comme je vous l' ai rappelé tout à l' heure , et le gouvernement de gôche ... Ces deux-là ont fermé 226 lycées professionnels . Il y a plus de 60.000 élèves de moins dans cette filière , alors que pendant ce temps , la population scolaire a augmenté . Ils ont donc mis durablement et pour longtemps - si nous n' inversons pas le sens des événements - le pays dans une impasse . On ne peut pas faire la transition écologique dont je vous ai parlé tout à l' heure , on ne peut pas faire l' économie de la mer on ne peut rien faire de tout ça , si on n' a pas les milliers , les dizaines de milliers , les centaines de milliers d' ouvriers et d' ouvrières hautement qualifié(e)s dans tous ces domaines . C' est la raison pour laquelle c' est une tâche d' urgence de reconstituer dans toute son étendue un système de formation professionnelle à la hauteur de l' objectif que l' on se donne . Avec avec une fluidité avec une fluidité qui permet à celui qui a commencé ou à celle qui a commencé un CAP , une fois qu' il sera allé jusqu' au BAC professionnel qui , lui , sera rétabli en 4 ans , parce que c' est très difficile de passer un BAC PRO ... Eh bien il faut ensuite qu' il puisse aller à un Brevet de Technicien Supérieur ( BTS ) parce que nous avons besoin de ces techniciens supérieurs , formés en 2 ans ; à une licence professionnelle - c' est moi qui les ai créées . Il faut que cette fluidité soit garantie , il faut qu' on s' occupe de ces milliers de gens qui sont d'accord pour faire ce travail-là , qui ont compris quel est leur intérêt , celui du pays et celui des choses qu' il y a à faire . Voilà tout ce qu' il faut reconstruire , méfiez vous de ceux qui viennent vous dire qu' il faut de l' apprentissage partout . L' apprentissage ça va pour certains métiers , mais on voit bien que pas un de leurs gosses n' y est . Ils ignorent ce que c' est , ils ignorent pourquoi on va en apprentissage , souvent c' est parce qu' il faut mettre un peu , comme on dit , de beurre dans les épinards . Parce que l' apprenti il a un petit quelque chose , comme vous le savez ; et puis parce qu' on pense souvent avec espoir et confiance que notre jeune qui aura été en apprentissage , ensuite il sera recruté par le patron ... Mais tout ça c' est des histoires . Ça n' existe pas , 25 % des contrats d' apprentissage sont rompus dans les 3 premiers mois . Ce n' est pas une bonne idée , en tout cas ce n' est pas généralisable . Et le gouvernement qui est en place leur a donné 1.000 euros par apprenti qu' ils prennent , eh bien vous savez ce qui se passe ? Il y a de moins en moins d' apprentis . Pourquoi ? Parce que les gens ont compris que ça ne fonctionne pas . Ils veulent pas que leur gosse y aille parce qu' ils se disent : &quot; C' est une impasse &quot; . On peut pas parler de la condition des salariés , on ne peut pas parler du travail sans parler de son contenu , c' est à dire des qualifications qu' il faut . Et une qualification c' est pas comme je vous l' ai dit tout à l' heure , la main : c' est la tête qui compte . Parce que le travail c' est des maths , c' est de la langue , c' est des langues , c' est de la physique , c' est de la chimie , les lois de la nature sont les mêmes pour tous le monde quelle que soit la voie par laquelle on étudie à l' école , que ce soit l' enseignement général ou l' enseignement professionnel , ce n' est pas vrai qu' il y a une école de pauvres et pour les pauvres , ce n' est pas vrai . Plus le contenu intellectuel d' un jeune est développé par le goût qu' on lui insuffle des sciences , de la poésie , de la musique , plus sa compréhension globale du monde va mieux , et plus son aptitude à se qualifier dans les secteurs intellectuels les plus pointus est grande - l' être humain est un tout . Il n' est pas un appendice de machine , ni quand il travaille , ni quand il se prépare à travailler . Bon . Vous m' écouterez mardi hein parce que sinon on passe la nuit , là . C' est de bon coeur Mes ami(e)s J' ai fait pour le mieux , je sais pas ce qui va être retenu de ce que je vous aurai dit . On a un bon travail à faire avec tout le programme que je vous ai présenté . Vous avez compris , hein , ça peut pas marcher que d' en haut , qu' à partir d' en haut . Il faut qu' il y ait une implication populaire , j' essaye de faire ce type de campagne avec vous tous . C' est pas juste une campagne &quot; On va voter Mélenchon &quot; ... Bien sûr qu' il faudra le faire , mais il faut qu' on fasse avancer les idées dans notre pays . Il faut qu' on - même d' un millimètre , même d' un mètre , ce sera ça de pris . Ouvrir les têtes , redonner du courage , redonner de l' envie de vivre , de lutter , se dire que c' est possible de changer profondément l' existence , la nôtre mais celle du monde . Moi je crois à tout ça . Là , en ce moment , on est rediffusés , il y a des milliers d' ami(e)s qui doivent être en train de nous regarder , et puis on va prendre des petits bouts . Nous construisons un mouvement . Un mouvement c' est pas un parti . C' est un mouvement . Là , on le fait pour une tâche : une élection présidentielle . Si nous gagnons - et cette fois -ci , une possibilité se présente qu' on y arrive , à condition d' être opiniâtres , patients , méthodiques , et ainsi de suite . Et si nous y arrivons , le changement qu' il faudra déclencher c' est pas au bout de 2 mois de parlote . C' est séance tenante , dans toutes les directions , tout à la fois . Ça fonctionne si il y a une masse immense de gens conscients . Et le peuple français , vous autres , nous tous , le niveau de conscience s' est beaucoup élevé , nous avons appris beaucoup de choses , et notamment depuis 2005 , ça se voit pas comme ça . Et depuis 2005 , le vote contre le traité constitutionnel européen , des milliers de gens sont entrés dans la conscience politique . Et maintenant , la jeune génération vient aussi aux questions générales et politiques . Elle entre par une porte , elle a compris mieux que les plus anciens , à quel point la défense de l' écosystème est devenue une priorité , parce qu' à leur tour ils pensent à leurs gosses , à leur futur , à ce qu' ils vont faire , et ils voient bien qu' on est en train de transformer les mers en cloaques , que l' air devient irrespirable et ainsi de suite . Donc des dizaines , des centaines de milliers de gens de tous âges , des millions sont disponibles . Pour changer , tout ce qui est là , de fond en comble , une vraie révolution citoyenne - c'est-à-dire où les gens s' emparent chacun du domaine où il est . Voilà ce qu' on va faire , mais on va le construire étape par étape . Une campagne , on alerte , on fait réfléchir , on oblige un peu des fois -faut se bousculer , on est obligés de lire le texte , des fois on comprend rien ... Faites pas les malins , des fois on comprend rien . Moi je comprends rien aussi , alors je me fais expliquer , et voilà comment on se forme . On élève son esprit . Bon , nous faisons un mouvement . J' ai besoin que d' ici à la mi-janvier ... D'abord il me faut des sous , bah voilà quoi , il faut des sous parce que pour les matériels , tout ça là , on a un problème de local donc on lance un appel au don , il y en a pour 70.000 euros à trouver , cette fois -ci . Ensuite , on a besoin que le mouvement sans cesse s' élargisse . Alors il y en a beaucoup , ils sont déjà dans la campagne . Ils font des choses , chacun d' entre vous peut être un comité de campagne , c' est pas que moi qui fait le boulot ni l' équipe qu' il y autour . Je voudrais qu' on arrive à 200.000 signatures d' ici à ... Bon , quelques jours quoi . Ca va , on est à 2 mètres d' y arriver . Si vous en pouvez encore de m' entendre , ce soir on met en ligne une émission pilote . Alors vous verrez ! Vous avez qu' à regarder sur la chaîne YouTube et puis voilà , vous en saurez davantage . Ensuite , chacun d' entre nous peut avoir sa liste , je sais pas si vous avez vu le petit carnet , je ne l' ai pas amené avec moi , mais on travaille sérieusement . Il est où ? Qui est -ce qui en a un ? C' est discipliné là-derrière ... Bon on a inventé ça , mais vous regardez , l' idée c' est que chacun peut ... Ca va il y a 4 mois ! Donc 4 mois pour vérifier que tonton , tata , mon copain , ma copine , le copain de la copine , euh ... Si il est correctement inscrit , après on va pas tordre les bras des gens , mais est -ce qu' au moins ils se sont intéressés à ce qu' on fait , ce qu' on raconte , on a le programme à la main et tout ça . Le petit carnet sert à ça . Je raconte tout ça parce que je pense que la transmission est terminée là . Et ça , c' est le logo de la campagne . Pour que vous le fassiez un peu connaître les ami(e)s . Parce que celui-là c' est moi qui décide qui c' est qui l' a , je vous fais pas un dessin d'accord ? Et puis il faut que les gens puissent nous repérer parce que il y a des gens qui ont du mal quand ils arrivent dans le bureau de vote il y a pleins de piles et puis des fois il y a des gens qui ont du mal à lire il y a 3 millions de personnes dans notre pays qui sont illettrées Attention , ça veut pas dire que c' est des bons à rien , juste il y a une difficulté pour eux . Donc faut penser à tout le monde et nous , on est le camp des petits , des déshérités , des abandonnés - pas que ça hein , on est aussi le camp de très intelligents , très cultivés , des meilleurs joueurs de musique , les meilleurs ingénieurs les meilleurs techniciens , on se débrouille pas trop mal , non , quand même jusqu' à présent ! Donc , il faut qu' on fasse connaitre le logo . Je pense que vous savez tous ce qu' il veut dire . Je vous raconte quand même que moi j' avais trouvé une idée géniale mais les gens qui étaient avec moi ne la trouvaient pas si géniale que ça , donc on a laissé tomber , et on dit : &quot; Qu' est -ce qu' on va faire ? &quot; &quot; C' est pas possible , on peut pas rester comme ça ... &quot; Et je dis &quot; bon , on va prendre les initiales F-I , ça fait FI . Et Alexis Corbière , qui est mon porte parole ... C' est un prof d' histoire , donc comment n' y ai -je pas pensé ? Me dit : &quot; Bah on n' a qu' à prendre phi , c' est la lettre grecque ( φ ) , parce que FI ça fait PHI ( φ ) , et PHI ( φ ) ça fait FI , d'accord ? Voilà la lettre grecque , elle est dessinée pour que vous puissiez le faire avec 1 doigt , comme ça . Pourquoi je vous dis ça , parce qu' il y en a bien qui doivent être facétieux , comme je l' ai été quand j' avais 20 ans . On allait sur les parkings quand il y avait de la neige , eh hop , on faisait des symboles , je ne vous dis pas lesquels ... Sur toutes les bagnoles . On est pas obligés mais enfin , c' est pour dire . Hein ? Fi ( Phi φ ) Fi ( Phi φ ) De la lettre grecque , les créateurs de la philosophie ! Alors , bon , rigolez pas , j' ai une licence de philosophie , moi . Il faut s' en rappeler de temps à autres , et le responsable général du service d' ordre c' est un docteur en philosophie . Et celui -ci qui est tout le temps à côté de moi - regardez , il est mal rasé- aussi , c' est un prof de philo , donc je suis entouré de profs de philo , la philosophie , le mot &quot; philo &quot; , ça veut dire &quot; amour de la sagesse &quot; , c' est un bon symbole pour nous parce que ce que nous proposons , c' est une sagesse du rapport aux autres et enfin , c' est la proportion harmonieuse , c' est comme ça . La proportion harmonieuse dans la nature , 1,61 . Alors les personnes qui mesurent 1m61 , peuvent savoir qu' elles incarnent une forme d' harmonie hein ... Donc c' est comme ça C' est le chiffre de l' harmonie , 1,61 . C' est un bon logo , vous pouvez le porter avec élégance , vous pouvez enlever mon nom si ça pose un problème , on va essayer de faire quelque chose qui ressemble à ça . Notre campagne , alors après le reste je vous le passe , la musique , le reste ... C' est une campagne qui doit être ... C' est pas une campagne de communication , les gens . Comprenez , bon , voilà , de la communication il y en a dans tout , ça aussi c' en est , d' une certaine façon . Mais c' est avant tout une campagne d' implication qui touche la tête et le cœur . On parle pas seulement par des raisonnements , on parle aussi par des affects . Vous êtes dans une société qui se déshumanise , qui se brutalise , c' est pour ça que je suis là aujourd'hui avec vous tous . Donc notre manière de réagir à tout ça , c' est bien sûr des slogans politique , bien sûr de l' explication politique , mais c' est avant tout une contribution par un raisonnement , par de l' affection , par de l' intérêt pour les autres , par la manière de mettre en avant ce qui concerne le caractère le plus humain de notre existence quotidienne . Rien qu' en faisant ça , on commence à changer la société dans laquelle on est . Chacun d' entre vous , quand vous sortez d' ici , demandez -vous quelle genre de révolution citoyenne vous ferez dans votre entreprise . Comment vous allez modifier la manière de travailler - oh vous ne le dites pas aux autres , ce n' est pas la peine , ils vont rigoler . Réfléchissez -y et vous verrez tout d' un coup , qu' en vous sentant responsable du monde , en oubliant les maîtres qui le dominent et leurs raisons absurdes alors vous regardez la vie et tout autour de vous différemment . Voilà tout ce que je vous souhaite , bon succès à nous tous . ",
    "C' est juste ... Je crois qu' une fois de plus c' était pas prévu . Il y en a partout . Alors on imaginait que peut-être il y aurait du monde dans le Hall mais pas dans 2 autres salles , enfin j' ai rien compris à ce qu' on m' a expliqué on m' a dit : &quot; Il y a du monde partout &quot; . Je pense qu' en vous déplaçant vous saviez ce que vous faisiez et que vous faites une démonstration de force et pour tout ce que nous avons à faire ensemble , je vous exprime ma profonde reconnaissance pour la batterie que vous rechargez , là , en ce moment . Je veux vous dire , je parle doucement parce que c' est une cathédrale , si je parle plus fort on n' entend plus rien . C' est difficile pour moi de commencer le discours ici , faut pas m' en vouloir , compte tenu des conditions acoustiques . C' est pas prévu pour ça hein , c' est la faute à personne . Donc vous suivrez , je pense , vous avez déjà entendu le son , il devait être bon pour vous ou pas ? Désolé de vous infliger ça , peut-être que tout à l' heure je repasse faire un p'tit coucou . Merci à vous tous d' être là . Je pense que c' est très impressionnant , d'après ce qu' on m' a dit ça fait 20 ans qu' on a pas rempli cette salle mais c' est sûr qu' on l' a jamais remplie jusqu' à dehors . Ça c' est une première . C' est très encourageant parce que ça a un sens politique , évidemment votre démarche ... Je vois un visage ami devant moi là , Mélenchon : Qu' est -ce que tu fais là , tu es pas d' ici ? Si , tu es de là maintenant ? C' est les Essonniens de la région parisienne , vous savez on est tous des émigrés en région parisienne alors on fini tous par rentrer chez soi . Ça signifie quelque chose que vous soyez si nombreux . J' ai vu des salles qu' on a calculé un peu trop petit parce qu' on s' est trompé puis parce qu' on a pas d' argent , mais là ça va au-delà . Ça signifie un niveau de mobilisation , de sensibilité à cette élection qui est extraordinaire , et c' est la clé du succès . Je vais vous dire pourquoi . Après je ferais le discours là-haut . Eux ils sont bien assis , ils pourront écouter pendant 2 heures . Mais , parce que notre tactique c' est d' agglomérer sans cesse , autour d' un noyau solide . C' est pourquoi nous ne faisons pas de la com ' , nous faisons de l' implication . Nous ne faisons pas des harangues , nous faisons une campagne instructive . Normalement , quand on s' est quitté , quand on se sera quitté , vous aurez mieux compris certaine(s ) chose(s ) , j' espère voilà c' est que vraiment je serai nul , mais vous aurez mieux compris certaine(s ) chose(s ) et vous pourrez vous même les transmettre et c' est comme ça qu' on va avancer . Au mois de février , le tableau se sera décanté et vous allez avoir 10 millions de personnes qui tout d' un coup vont s' intéresser à la campagne , et puis ensuite quand on va arriver vers le mois de mars et d' avril , encore plusieurs millions . A chaque fois ils arrivent et ils découvrent un tableau . Et naturellement , plus tard ils s' y impliquent ça signifie que leur intérêt politique est moyen , après tout ils ont le droit hein , mais ils se raccrochent à ce qui est là , à ceux qui ont déjà pris parti , ils demandent l' avis autour d' eux , ils regardent les réunions et ça qu' on est en train de construire patiemment . J' imagine que si vous êtes là et que si vous supportez ça debout c' est que déjà vous vous sentez assez proche de ce qu' on est en train de faire tous . Soyez patient et discipliner . C' est à dire , discipliner pas par rapport à moi vous ferez bien ce que vous vous voudrez , mais dans la manière de vous organiser pour convaincre . Que chacun d' entre nous se fasse la liste de 10 personnes , s' assurer que les gens sont bien inscrits , qu' ils ont bien compris , que vous même vous ayez bien comprit le pacte qui existe en nous tous , pas seulement avec moi . C' est que on sera jamais d'accord à 100 % , hein , encore heureux , on sera jamais d'accord à 100 % mais le programme &quot; L' avenir en commun &quot; c' est un cadre qui permet de s' y retrouver , je sais pas dans quelle proportion vous vous y retrouvez . C' est ça qui faut proposer aux autres , on leur demande pas de marcher aux pas , de s' embrigader , de prendre une carte , rien de tout ça . Nous sommes un mouvement autour d' un programme . Et c' est très important que cette construction patiente , pédagogique , il doit bien y avoir des profs ici j' espère , c' est très important parce que non seulement ça sert pour gagner mais surtout de toute l' expérience que j' ai de ces situations où nous finissons par gagner alors que tout le monde pense qu' on est pas dans le coup . C' est qu' ensuite il faut gouverner et il faut gouverner avec un mouvement qui part de la population elle-même . On ne peut pas changer une société du dessus , on peut faire beaucoup de choses , il faut avoir la clé d' entrée au dessus , mais on ne peut pas le faire si des millions de gens ne s' y mettent pas car ceux à quoi j' appelle c' est une révolution citoyenne , et je ne veux pas galvauder les mots . C' est pas un euphémisme , une manière de pas oser dire je ne sais quoi , non , ça signifie le retour du pouvoir des individus sur leur propre vie . Et ça c' est vrai en général , parce qu' on fera une constituante , mais c' est vrai en particulier , dans chacune de vos administrations , dans chacune de vos entreprises il faudra révolutionner de manière citoyenne . C' est à dire qu' il faudra changer les process de production , la manière de produire , les matériaux qu' on utilise , la façon de faire . Tout doit être transformer de A jusqu' à Z pour relever le défi qu' il y a devant nous . Et c' est pas seulement un défi social , on va en parler ce soir , mais c' est un défi pour toute l' humanité . Nous sommes confrontés à un événement majeur qui est la transformation de notre écosystème et quand ce soir on va parler de solidarité , de rapports entre les êtres humains , c' est de ça dont on parle . Est -ce que la civilisation humaine est capable de prendre en charge un choc pareil ? Voilà c' est ça le sujet de ce soir . Pardon il faut que j' y aille parce que sinon je fais un discours ici , un autre dedans , ils vont plus pouvoir tenir le coup . A tout à l' heure . *Public : Président ! C' est très impressionnant ! C' est très très impressionnant qu' il y ait tant de monde , on m' avait dit &quot; Bon , on va y arriver , on ouvre et si c' est ... &quot; mais là , vraiment , ouf ! Et on est juste au début du mois de janvier . Vous avez entendu ce que j' ai dit en bas , je vous le redis pas c' est pas la peine . Donc je commence à mon propos . D'abord j' ai une nouvelle à vous donner , que peut-être vous n' avez pas eu , mais qui à mes yeux a une certaine importance et je suis sûr que vous partagerez ce sentiment avec moi . Cet après-midi les sénateurs du groupe communiste ont fait quelque chose de très important et de très utile . Moi je ne suis pas communiste mais je tiens à rendre hommage à ce groupe qui fonctionne sous la présidence de Éliane Assassi . Ils ont déposés une proposition de loi pour abroger la loi El Khomri . Et vous autres , les gens , je sais à vos applaudissements et tout le reste , qu' on pense pareil de ça . Donc , écoutez -moi avec soin parce qu' il va falloir aller l' expliquer à d' autres , il y a beaucoup de naïf , il y a beaucoup d' illusion . Vous savez pourquoi ? Parce que les gens sont braves . Les gens ils veulent croire que tout ça est un malentendu que , ils préféreraient croire que ils ont pas été trahis , et puis ça leur coûte tellement une telle humiliation . Donc ils sont toujours prêt à pardonner . Même à ceux qui les maltraitent . Et donc comme la bataille pour la loi El Khomri c' était il y a quelque mois c' était avant les vacances , donc ils seraient prêts aujourd'hui à dire &quot; Allez , zou , c' est fini , on passe à autre chose &quot; . Ils savent bienque non on va pas passer à autre chose , parce que depuis le 1er janvier cette loi s' applique . Et c' est pas pour rien qu' on leur a dit &quot; Loi El Khomri , vie pourrie &quot; parce que c' est la vie pourrie qui commence . Avec la remise en cause des durées de travail hebdomadaire , du contenu du contrat de travail , l' impossibilité de protester , parce qu' autre fois vous pouviez dire &quot; Bah écoute , tu fais ce que tu veux mais moi j' ai signé un contrat de travail &quot; &quot; Je veux pas de tes nouveaux horaires &quot; , c' est fini tout ça . Et le délai de prévenance qui passe de 7 à 3 jours , vous savez tous ce que ça veut dire . Pour le grand bourgeois , lui , c' est pas un problème , il est souple , dynamique . Comme il fout rien , bon il fait ce qu' il a à faire , mais sinon il téléphone à la nounou &quot; Finalement vous viendrez 1 jour de plus ou vous irez à l ' école &quot; mais vous autres , vous faites pas comme ça , c' est impossible . Vous pouvez pas changer l' organisation de votre vie en 3 jours . Tous les rendez-vous que vous avez pris , supposez , on va parler ce soir de la sécurité sociale , supposez que vous ayez un rendez-vous avec un médecin , un spécialiste . Tiens , va lui dire : &quot; Bah finalement , non ça sera pas jeudi , ça sera vendredi &quot; . Rien du tout , c' est 3 mois après . Bref , c' est ça le contenu pratique , quotidien de la vie des gens qui est dans cette loi , Donc , il y a une loi qui arrive pour abroger la loi El Khomri , c' était cet après-midi . Il y avait un rassemblement avant . Et vous , vous savez pas le résultat mais je vais quand même vous le donner et dans le détail . Au moment du vote , la Droite n' a pas pris part au vote . Pas pris part au vote . Ni pour , ni contre . C' est à dire ni pour abroger la loi , ni pour la maintenir , rien . Exit la droite , elle est majoritaire , sauf un , Serge Dassault et lui il est resté pour soutenir la loi El Khomri , bah il vote avec sa classe , lui . Bah c' est normal , non ? C' est le seul là-dedans qui fait un vote raisonnable si on y réfléchit bien . IL y a eu 25 sénateurs pour abroger la loi El Khomri . Ceux-là aussi , ils votent avec les leur . 20 communistes , Pierre-Yves Collombat , qui est le 1er sénateur qui m' a donné son parrainage , qui était un sénateur socialiste Et 4 sénateurs socialistes , Madame Lienemann , Monsieur Jérôme Durain , Henri Cabanel et Gisèle Jourda . Vous avez bien entendu ? Il y a donc eu , à la fin , 25 sénateurs pour abroger la loi El Khomri , la droite ne s' occupant pas du vote . Et il y a eu 115 sénateurs pour le maintien de la loi El Khomri . 115 , dont 108 du PS . 108 ! Ils sont donc exclusivement responsable de la loi El Khomri , même la droite n' y a pas mis les doigts . Si cette après-midi , ils avaient dit &quot; Tout compte fait , vu le bazar que ça était , on abroge la loi El Khomri &quot; il y avait plus de loi El Khomri aujourd'hui . Je voulais que vous le sachiez . Et à côté de ça vous avez eu 10 abstentions , les 10 écolos , 5 PS et Robert Hue . Voilà , je voulais que vous le sachiez . La loi El Khomri ce n' est pas un petit incident de parcours . C' est une transformation réactionnaire de l' ordre public social . C' est un coup direct porté à ce que nous , nous avons appelé la &quot; République Sociale &quot; , c' est à dire ce mécanisme particulier qui faisait que à mesure que les victoires électorales le permettaient , que la transformation de la société par la loi , le permettait on allait vers une République de plus en plus sociale . Et la colonne vertébrale de cette affaire , excusez -moi si c' est un peu théorique , si c' est de la théorie politique , la colonne vertébrale de cette affaire c' était que la démocratie sociale était en quelque sorte emboîté avec la démocratie politique , celle où on fait les lois . Comment ? Quand vous aviez un accord qui se négocié dans une branche . Eh bien cet accord négocié dans la branche est étendu à tous les travailleurs de la branche , par la loi . Et ensuite , vous aviez un ordre qui se créer . Si l' accord de branche , comme il est , était bon , il s' appliquait à tous le monde , donc on ne pouvait pas voter dans l' entreprise ou décidé dans l' entreprise quelque chose qui soit contre l' accord de branche . Mais l' accord de branche lui-même , il pouvait ou être amélioré ou surplombé par une loi sociale . Vous suivez le mécanisme ? En haut la loi , ensuite des accords avec un principe qu' on appelle &quot; Deux faveurs &quot; d' un étage à l' autre . La loi s' applique , aucun accord de branche , aucun accord d' entreprise n' est supérieur à la loi , il peut pas le détricoter . Vous avez un accord de branche , il est voté pour tous les travailleurs et il s' applique à l' entreprise . Vous voyez comment ça va ? Ça c' était une chose particulière aux français . Quand on vous entend , on vous parler beaucoup , vous savez de la sociale-démocratie , elle a ses mérites , dans le reste de l' Europe . Mais comment ça se passe dans un pays social-démocrate ? Vous avez un accord dans votre entreprise ou dans votre branche mais vous ne bénéficiez de l' accord que si vous êtes membre du syndicat qu' a signé l' accord . Et vous n' avez d'accord qui s' applique dans l' usine à côté qui fait exactement la même chose , que si le patron a signé l' accord de branche . Il y a pas la loi pour tout le monde . Ce sont des accords entre le syndicat et le patronat qui s' applique entre signataires . Résultat : le jour où vous avez un fond de pension qu' arrive là-dedans , il dit : &quot; Mais moi je m' en fous de vos histoires , je ne discute rien du tout , je suis venu placé de l' argent là , je prend tout ce que je peux et je m' en vais &quot; &quot; Donc je négocie rien du tout , je signe rien du tout et c' est pas la peine d' essayer &quot; . Résultat : Au Danemark , qui était un pays phare de la social-démocratie , vous avez 20 % des travailleurs , maintenant qui n' ont plus aucune protection de cette nature parce que le patron il part tout simplement . Donc quand on a fait la loi El Khomri , quand ils ont fait la loi El Khomri , ils ont décidés que tout pouvait se discuter au niveau de l' entreprise Et vous arrêtez pas de les entendre dire , au plus près du terrain , au plus près des réalités , tout ça , c' est des discours fumeux de gens qui enfument les autres parce qu' au plus près du terrain , il y a qu' une chose c' est un rapport de force . C' est comme ça , il faut le comprendre , c' est pas méchanceté , chacun est dans sa logique . Le patron qui essaye , lui , de faire que les coûts de production soient les plus bas , il est dans sa logique . C' est pas sadique . Il est dans la logique qu' il vendra mieux si c' est plus bas et il vendra mieux si c' est plus bas , s' il paye etc . Mais si le patron de cette entreprise qui produit , je dis n' importe quoi , des chaussettes obtient un accord dans son entreprise où les gens travaillent plus , où il y a plus d' heure supplémentaire les gens font le travail mais ne sont pas payés en heure supplémentaire . Bah il a un avantage , le patron de la boite d' à côté , il peut pas tenir le coup , il est obligé d' en faire autant et il va voir ses employé et il leur dit &quot; Ecoute , moi j' y peux rien , mais tu vois bien , ils arrivent à faire moins cher que nous , donc il faut bien qu' on s' aligne &quot; Et comme les gens sont braves , en général , et qu' ils ont tous la peur qu' on comprend , de perdre leur boulot , eh beh ils acceptent . C' est donc une logique de la peur , toute cette société est gouvernée par la peur . Et ils vous présentent ça comme de la modernité , la souplesse , la flexibilité , et blablabla , tout ce baratin que vous entendez sans arrêt , c' est mal traité les gens toujours plus . Et ... moi en général je m' arrête là de ma démonstration mais je leur dis : &quot; Vous êtes tellement bête , vous êtes tellement cupide , vous êtes tellement des technocrates butés , bornés , qui ne comprenaient rien à la société humaine &quot; &quot; Vous ne comprenez pas que des gens qu' on mal-traite travaillent moins bien parce qu' ils en ont ras le bol et puis parce qu' ils ont pas envie quand ils ont déjà mis tout ce qu' ils pouvaient d' énergie d' en mettre encore &quot; &quot; En sachant qu' il y a rien qui revient &quot; Quand ils ont décidé de supprimer les heures supplémentaires , le paiement des heures supplémentaires , qu' ils l' ont passé de 25 % que ça devait être à 10 % , évidemment que ça dégringole dans toutes les boites . 10 % c' est des petites sommes mais qui comptent beaucoup pour les gens , sur un SMIC , on pourrait dire : &quot; Bah qu' est -ce que c' est 2 euros &quot; mais c' est très important quand on vit avec un SMIC 2 euros pour une heure supplémentaire d'ailleurs on fait pas une heure supplémentaire par plaisir , on la fait pour avoir les 2 euros en question . Comment ne le comprennent -ils pas ? Comment comptent -ils que les gens vont travailler toujours davantage sans avoir jamais un retour , une satisfaction , un merci ? Non , ce n' est pas possible et dans cette loi El Khomri , c' est tout le long cet état d' esprit de persécution dans le détail pensant que les gens vont faire plus parce qu' on les brutalise et qu' ils ont peur . C' est pourquoi ... cette loi , il a été impossible pour eux de la passer d' une manière démocratique . C' est une des premières fois que l' on voit dans une grande démocratie , comme est la France , une loi d' une telle importance réorganisant l' ensemble des rapports sociaux et de la négociation entre les travailleurs et leurs employeurs . Une loi d' une telle ampleur qui changeait tant de chose et jusqu' au règle de fonctionnement , ils l' ont fait passée par 49:3 . Autrement dit , personne n' a jamais voté cette loi , à part aujourd'hui où la possibilité était donnée de l' abrogée et où ils ont pu lever la main pour donner leur avis , et quand on leur a demandé de donner leur avis , ils ont votés contre les salariés , voilà ce qu' ils ont fait aujourd'hui ! Alors je vous demande que vous vous en souveniez ! Souvenez -vous en ! Pourquoi ? Parce qu' il y en a des ballots , j' en suis sûr qu' il y en a aussi dans cette salle si vous me le permettez . Il y a des primaires qui vont arriver là , la situation est en train de se décanter . Déjà la dernière fois il a fallu que je me fâche , ça a pas servi à grand chose soit dit par parenthèse , mais au moins je leur ai dis ce que je pensais . Les gens disaient &quot; Ah tu comprends nous on est de Gauche mais on va aller à la primaire de la Droite pour voter pour celui -ci , pour empêcher l' autre de j' sais pas quoi &quot; , qu' est -ce que c' est que cette histoire . Il faut avoir du respect pour soi-même d'abord . Vous dénoncez , vous dénoncez les responsables politiques , vous dites &quot; Ce sont des cyniques , ils n' ont pas de paroles &quot; mais vous avez vu ce que vous êtes en train de faire ? Vous allez à l' élection ou chez quelqu'un qu' est pas de votre bord pour truquer l' élection et faire élire quelqu'un pour qui vous voterez pas , c' est pas honnête . Il faut donc se respecter soi-même , donc vous ferez bien ce que vous voulez les gens , mais venez pas me dire que vous allez voter à la primaire du PS pour faire j' sais pas quoi , parce que moi je vous ris . La chose suivante , voter à cette primaire c' est les amnistier , c' est rien d' autre . Eh ben moi je ne les amnistie pas ! Et d'ailleurs je vais vous dire une chose , après j' en parlerais plus parce que ça me fatigue . D'abord je veux pas perdre mon temps . Réfléchissez -y , vous voulez quoi ? Les aider à avoir beaucoup de voix pour qu' ils disent : &quot; Vous voyez , il y a beaucoup de gens qui nous suivent &quot; , pourquoi ? Pour me rendre le travail plus facile après ? Parce qu' il y a une chose dont je veux que vous soyez bien conscient tous les ami(e)s , tous bien conscient , parce que j' ai lu par ci , par là , que machin , truc et bidule , s' il était élu , le lendemain va me téléphoner , tiens ça va me faire du neuf , moi je répond à tous le monde . Tiens , ça les prend tout d' un coup au bout de 5 ans , de m' appeler au téléphone , et tu pouvais m' appeler avant , par exemple pour parler de l' amnistie sociale . Quand j' ai demandé l' amnistie sociale pour les camarades , les syndicalistes qui ont été condamnés pour avoir défendu et il y avait personne qui me téléphonait , personne . Faites ce que vous voulez , c' est votre droit , je respecte vos procédures mais s' il vous plait , ne nous prenez pas pour des poissons rouges . Ne croyez pas qu' on a tout tout oublié et ne nous racontez pas d' histoire , ce n' est pas la primaire de la Gauche , c' est la primaire du PS . Le PS a le droit d' organiser sa primaire mais alors il faut lui donner le nom qu' elle porte , c' est la primaire du PS , du parti de Gauche , là-dedans il y a le PS et le PRG , c' est tout ! Tout les autres sont dehors . Y compris , y compris ceux qui étaient pour qu' il y ait une primaire comme Monsieur Jadot qui entre temps à changé d' avis , on ne sait pas pourquoi , mais en tout cas il est allé faire une autre primaire . Un peu de respect pour nous tous . Ce que je veux que vous sachiez c' est que quoiqu'il arrive , et d'autant plus maintenant , quoiqu'il arrive , ma candidature et le programme &quot; L' avenir en commun &quot; iront jusqu' au bout de l' élection . Je veux bien qu' on me téléphone tant qu' on veut , je suis prêt à accueillir s' il le faut qui voudrait rejoindre notre combat , nous ne sommes pas des sectaires , nous ne réglons pas des comptes . Nous ne travaillerons pas à la rancune , mais changer la ligne pour laquelle on a eu tant de mal à exister , se battre , mettre au point ce programme , se rassembler comme on est en train de le faire , jamais . Ça , ça sera présent jusqu' au bout ! Alors , après , faut que je finisse avec ça parce que j' ai des sujets plus important , mais bon , il faut ce qu' il faut . Là il y a marqué : &quot; Le tableau se précise &quot; bon voilà . Je viens de vous dire quoi ? Dites franchement , supposer qu' on me téléphone , alors qu' est -ce que je fais ? Mais oui bien sûr alors on va commencer pendant 8 jours , non c' est toi , oui c' est moi , etc . Qui c' est le candidat , on sait pas , bah si je le sais , c' est moi . Déjà il y a un point qui est réglé . Deuxième point , le programme , bien sûr qu' on doit en discuter , et déjà eux , comment ils vont faire ? Comme Hollande est partis , toute la cohésion qui avait là-dedans , elle a disparu . Donc ils ont chacun leur programme , ils ont le droit , je veux pas m' en moquer , mais ils ont chacun le leur . Et après Mesdames , Messieurs , et pour beaucoup d' entre vous , mes chers camarades , qui en connaissaient donc un rayon sur ce genre de chose , il leur reste 92 jours pour se mettre d'accord sur un programme commun entre eux , avant de venir parler avec les autres avant le 1er tour de l' élection présidentielle . Est -ce que vous trouvez que c' est sérieux ? franchement , ils avaient toute la vie pour se préparer , ils ont gouvernés pendant 5 ans , et là , il y a rien de prêt . Non , ce n' est pas sérieux . Hein . Alors avant ils disaient &quot; De toute façon , sérieux ou pas , on s' en fout , c' st nous qui sommes devant , on est le vote utile &quot; . J' ai jamais aimé le vote utile et c' est pas parce que maintenant c' est moi qui suis devant que je vais m' en servir . Mais je sais maintenant au moins qu' il y a un vote inutile , ça me permet de dire : &quot; A quoi bon un candidat du PS ? &quot; Bon , alors après , bon , j' en parle plus parce que pour parler du contenu du programme , il faut d'abord qu' ils en aient arrêter un et qu' on sache lequel . On va pas perdre notre temps à faire le tour , hein . Je viens aux autres alors là , ouf , ça s' aggrave , mais vous mais vous savez il y a un petit air de panique là ... Chez beaucoup . Hein , le casse noix se met bien place , pour qu' il y ait un casse noix il faut que ça sert des 2 bords . Macron ou Mélenchon . Macron ... Mélenchon ... Macron , Mélenchon , et au milieu ça fait de l' huile . Mais ça c' est la bataille à la loyale , vous savez bien , il y a 2 programmes , la solidarité ou la compétition , moi cet homme je le respecte , il défend ses idées , il va au bout , il raconte ses trucs , c' est normal , la démocratie c' est quand même ça , on ne demande pas que tout le monde dise la même chose ou tout le monde dise la même que nous , c' est pas le sujet . Chacun va exposer son programme , les français vont trier , c' est une Constitution absurde qui fait qu' au 1er tour on choisit , au 2ème on élimine , le résultat est qu' on est jamais content . Mais c' est la démocratie , moi j' en accepte toute la règle et à la fin je ferais comme vous tous , je m' inclinerais . Justement , je voudrais bien savoir , vous savez il y a des inclinaisons qui coûtent plus cher que d' autres , et justement , la dame qui avait déjà gagnée l' élection , que c' était même pas la peine de l' organiser n' est -ce pas ? Ou si elle avait pas gagnée il y avait juste à s' occuper du second , notez au passage que le piège pourrait se refermer sur eux , hein . Le second pourrait être quelqu'un qu' ils ont pas prévu , eh oui . Alors , elle , elle commence à paniquer , je pense que c' est un bon signe . Dans cette salle déjà , pour que vous soyez venu à une réunion tout ça , c' est que vous vous intéressez au contenu des échanges , des discussions , de ce que chacun dit . Déjà , vous avez dû vous rendre compte qu' il y a quelque chose qui tournait pas rond . Cette femme là elle nous singe . D' habitude on dit &quot; Singe savant &quot; , là j' ai pas dit &quot; Savant &quot; , j' ai dit &quot; Elle nous singe &quot; . Les discours , on reprend des phrases de nous , entières , des formules , après vous avez toute la ribambelle à micro qui rapplique en disant : &quot; Vous dites comme elle ! &quot; Ah non non , pardon ... Enfin vous avez compris . Ça embrouille tout , personne comprend plus rien et moi , à chaque fois , et ramer et encore ramer . Mais là , apparemment ça va très loin . Vous savez peut-être que j' ai fait une chaîne YouTube , il y en a peut-être d' entre vous qui regardent ça . Alors je suis là , assis sur mon p'tit truc , je raconte dans la semaine comment j' ai vu les choses et il y a pas longtemps on m' a dit &quot; Tiens , si au passage tu pouvais faire un peu de pub , on un mug il y a marqué dessus &quot; L' insoumis(e ) &quot; &quot; . Vous voyez , c' est un p'tit mug . Alors me voilà avec mon p'tit mug , je dis &quot; C' est très bon &quot; tout ça . On vend le truc quoi ! Et là on me dit : &quot; Monsieur Philippot aussi fait la chaîne YouTube &quot; , ouh ! Alors je me dis &quot; Bah il me singe &quot; , quand même c' est moi qui ai commencé , mais le gars alors , eh oh , un peu d' imagination . Il a aussi un peu mug à la main ! Et un mug rouge ! Le mien il est blanc ! Bon ... Je fais la plaisanterie mais je suis comme beaucoup d' entre vous . J' ai de l' expérience , je suis pas un perdreau de l' année , je sors pas de l' œuf . Je repère l' odeur de la panique chez l' ennemi longtemps avant qu' ils aient fini de l' émettre . Déjà avoir trouvé comme idée pour Madame Le Pen comme symbole de prendre une rose , par les temps qui courent , fallait vraiment le trouver ... Au bout d' une dalle mortuaire . Alors il parait que c' est un fleuret , je sais pas qui c' est qu' elle compte menée comme ça au fleuret , mais pas nous . Ce qui est sûr c' est que ça le fait pas quoi . Hein . Ils paniquent , ils descendent , on peut les battre . Et vous savez pourquoi je sais qu' ils paniquent ? Parce que chaque fois qu' ils paniquent ils tournent et retournent à leur fondamentaux , c' est à dire que chaque fois qu' ils ont l' impression qu' ils perdent pied , ils reviennent sur des gros trucs racistes et ainsi de suite et là , elle s' est dépassée parce que comme ils avaient déjà injurié la moitié de la Terre , la quasi-totalité des gens qui n' ont pas sa religion . Alors que faire ? Eh bah elle a trouvé une nouvelle cible , les gosses . Faut le faire hein . Moi je pensais que même les racistes ils respectaient les gosses , ou les xénophobes , j' dis bon , on est tous les mêmes hein . Vous savez , c' est un invariant d' échelle , tout le monde partout dans le monde , on a le même réflexe , c' est à dire si un gosse tombe , qui que vous soyez , quelque soit votre âge , où que vous soyez , vous vous avancez pour l' empêcher de tomber . C' est un geste , un réflexe . C' est pas pareil comme c' est un adulte , si quelqu'un tombe , vous commencez par faire &quot; Ehlala ! &quot; Si c' est un gosse ... Donc c' est pour dire que c' est ce qu' il y a de plus profondément ancré dans le cœur d' un être humain . C' est protéger les plus petits . C' est de ça dont on parle ce soir , protéger celui qui a le plus besoin des autres . Eh bah elle trouve le moyen de dire &quot; Je considère que la solidarité nationale doit s' exprimer à l' égard des Français . &quot; &quot; Je n' ai rien contre les étrangers &quot; , heureusement parce qu' est -ce qu' elle ferait ? &quot; Mais je leur dis : &quot; Si vous venez dans notre pays ne vous attendez pas à ce que vous soyez pris en charge , à être soigné , que vos enfants soient éduqués gratuitement &quot; , maintenant c' est terminé . &quot; Je relis : &quot; Ne vous attendez pas à ce que vous soyez soigné &quot; . Eh bah ça , c' est pas une bonne idée du tout parce que celui qui est malade , il est malade et rend malade les autres si on le soigne pas . Donc , si elle veut que la solidarité nationale doit s' exprimer à l' égard des Français , il faut soigner les étrangers . Il y a pas le choix . Deuxio &quot; Que vos enfants soient éduqués gratuitement &quot; , ah mais ça c' est pas de la solidarité , c' est une organisation du pays . Nous avons fait le choix , nous les Français qui n' avons aucune matière première , d' élever sans cesse le niveau d' éducation de notre peuple pour être parmi ceux qui manient le mieux les techniques , les machines , les sciences et on réussit très bien . &quot; Que vos enfants soient éduqué gratuitement , maintenant c' est terminé &quot; . Qu' est -ce que ça veut dire ? Vous allez laisser les petits pitchounes , on va rien leur apprendre ? Ils vont pas savoir lire , ils vont pas savoir écrire , ils vont pas savoir compter . Leurs pauvres parents , qui regarde ça , alors ils vont parler quelle langue ces petits ? Juste la langue qu' on parle à la maison ? Reste plus qu' à espérer que c' est le Français . Mais les parents étaient trop contents que les gosses s' en sortent , qu' ils aillent à l' école , qu' ils s' améliorent et ils reviennent et c' est comme ça que ça se passe dans une famille d' étranger . C' est comme ça que ça se passe . Et c' est ça que vous voulez arrêter ? Vous allez faire quoi ? Dire &quot; Non , toi on ne veut pas de toi &quot; , alors , le gosse il a son copain dans la cité , ou sa copine , peut-être qu' ils jouent ensemble etc . Mais à l' heure d' aller à l' école , &quot; Toi non ! &quot; &quot; Toi oui ! Rentre mon petit &quot; , &quot; Toi non ! Dehors ! Va-t -en ! &quot; Qu' est -ce que c' est que ce monde ... Comment ils pensent les choses ? Et maintenant , si j' oublie l' aspect humain , parce que dans notre société , quand on parle d' aspect humain , on passe pour un rêveur ou un poète et le poète est méprisé . Alors que moi je considère que c' est la principale vertu de l' esprit , mais bon , si on met ça de côté ... Qu' est -ce que c' est que cette bêtise de croire qu' une société va être meilleur avec des ignorants . Mais il y a que des ignorants pour croire cela ! La société est meilleur plus nous sommes éduqués et être éduqué c' est pas seulement apprendre comme certains croient des choses qui servent à quelque chose comme ils disent , c' est tout ce qui développe la personne humaine . A l' école , on apprend aussi de la musique , on apprend de la littérature , on lit des poèmes , on fait des dessins , et toutes ces choses vous transforment en un être humain meilleur et quand vous êtes meilleur vous , alors toute la société est meilleure . C' est le pari sur lequel repose l' humanisme progressiste que nous incarnons . Le Républicanisme . Bon j' arrête avec elle parce qu' elle va nous rendre malade . Alors maintenant , évidemment , on m' a dit : &quot; Vous vous allez dans la Sarthe pour affronter Monsieur Fillon &quot; , mais renseignez -vous , Monsieur Fillon n' est pas de la Sarthe , il s' est enfui . Les gens de la Sarthe l' ont foutu dehors . Ah oui mais écoutez , faut pas nous en vouloir si vous êtes de la Sarthe , mais on a l' intention de vous le rendre . Il a son château que est par là , bah il a le droit . Tout le mal qui s' est donné pour avoir un château . Bon , mais c' est pas le sujet . Le sujet c' est que cet homme est réputé être notre prochain président de la République , vous savez tous pourquoi , parce que bien sûr il va arriver au deuxième tour , on se demande pourquoi on va voter et justement au deuxième tour il y aura Madame Le Pen , on se demande pourquoi on va voter et quand il sera là , eh ben , vous n' aurez qu' un seul choix , c' est de voter pour lui pour pas voter pour l' autre , on se demande pourquoi on va voter . Ils feraient mieux de le nommer tout de suite . Comme ça on s' épargnerait tout ça . Bien sûr , rien de tout cela ne va se passer comme prévu parce que ça fait déjà une paire d' élection que rien ne se passe comme prévu , mais en attendant , où vous êtes interrogé et invité à réfléchir tout le temps sur cette base stupide . Le vote utile , le vote ceci , le vote cela . Vous n' avez à vous soucier que d' une chose : Quel est le vote nécessaire ? Qu' est -ce qui est bon pour tous ? Qu' est -ce qui est bon pour la patrie ? Qu' est -ce qui est bon pour les autres personnes qui vous entourent ? Et le reste vous ne vous en occupez pas parce que ça s' appelle la théorie du jeu , aller au PMU plutôt que dans un isoloir pour voter , parce que si vous y allez avec une mentalité de PMU , si vous abandonnez vos convictions à la porte du bureau de vote , faut pas vous étonnez pas si vous les retrouver pas en sortant . D'accord ? Donc , Monsieur Fillon étant comme vous le savez par avance , notre prochain président de la République , nous a déjà annoncé son programme et dans son programme il y a le fait qu' il a décidé de supprimer la Sécu . Non non , faut pas dire : &quot; J' ai pas dit ça &quot; , il l' a dit . Alors avec le langage ... J' espère que je vais tomber pile-poil sur ma page ... Oui , j' ai peut-être été un peu vite mais c' est pas grave . Projet santé , avant décembre : &quot; Il faut &quot; ... Je vous le traduirais , parce que c' est de la langue de techno ... &quot; Focaliser l' assurance publique universelle &quot; ça c' est la Sécu . &quot; Focaliser &quot; ça veut dire &quot; elle doit juste s' occuper d' eux &quot; &quot; Sur des affections graves ou de longues durées &quot; Lui , docteur Fillon , et les gens qui l' entourent vont vous dire ce qu' est une affection grave , que les toubibs eux-mêmes ne sont pas capables de vous dire parce que ça dépend l' âge que vous avez , ça dépend la maladie que vous avez , on meurt plus facilement d' une grippe à 60 ans qu ' 25 ans , mais ça ils ne le savent pas eux . &quot; Affection grave ou de longue durée et l' assurance privée sur le reste &quot; Bon je vous le relis pas , tout le monde a compris . Les grandes douleurs pour la Sécurité Sociale , les petits bobos pour l' assurance privée , vous allez vous assurer . Naturellement , tous on a fait , on s' est dit : &quot; ' Mais ils sont fous ! &quot; , de quoi ils parlent ? Qu' est -ce que c' est une petite maladie ? Qu' est -ce que c' est une grande ? C' est pas comme ça qu' il faut prendre le problème . On doit tout rembourser , parce qu' une petite maladie peut devenir une grande et parce que quand une petite maladie est pas soignée , elle devient une grande maladie et à ce moment-là , ça coûte encore plus cher Messieurs les comptables que ça n' aurait coûter si on l' avait régler au début . Ça c' est ce qu' il a dit , comme tout le monde à hurler ... Il revient le 12 décembre et il vous dit ... Le contraire . Mais bien emballé , que vous savez pas ce que ça veut dire . &quot; L' assurance maladie obligatoire universelle , pilier de la solidarité continuera à couvrir les soins comme aujourd'hui et même mieux ... &quot; &quot; Rembourser des soins qui sont largement à la charge des assuré(e)s comme les soins optiques et dentaires &quot; , vous avez vu ? Suffit de gueuler un coup et il baisse le ton . &quot; Il n' est donc pas question de toucher à l' Assurance Maladie et encore moins de la privatiser &quot; Comme nous sommes de vieux guerriers , on le laisse parler et on écoute la phrase suivante . &quot; Il convient par ailleurs ... &quot; C' est là que ça se gâte ... &quot; De responsabiliser chacun &quot; . Ah ... Vous avez compris ? &quot; En simplifiant les nombreux dispositifs de franchise actuel &quot; déjà la phrase elle est un peu longue . La franchise vous savez ce que c' est , c' est un truc tordu et mensonger qui vous fait croire que vous êtes remboursé mais vous l' êtes pas , ça s' appelle une &quot; Franchise &quot; . C' est à dire vous payez . Si vous allez à l' hôpital , 18 euros par jour . Alors il y a plein de franchise . &quot; Quel fouillis , il y a plein de franchise , on comprend rien &quot; bah oui , c' est lui qui les a inventé . Donc il sait qu' on comprend rien , et c' était fait pour , d'ailleurs vous vous souciez pas de comprendre les franchises . Votre souci c' est de savoir si vous êtes à l' hôpital , si vous pouvez payer vos soins , c' est tout . Après , savoir qu' il y a une franchise différente pour autre chose , c' est pas votre sujet . Pour lui , si . Bon , il en était là . Le 12 décembre , il avance bien ce petit . Et nous voici rendu le 11 janvier , là comme on l' a ... bien bombardé , on l' a pas lâché . Voilà ce qu' il dit : &quot; Sur la Sécurité Sociale , on n' a pas été bons &quot; . Ça c' est clair . &quot; Ce que nous avions écrit à été mal compris &quot; , non justement c' est parce qu' on avait compris que vous avez pris peur . &quot; Je l' ai retiré &quot; . Ah bah voilà qu' il commence bien , il est pas encore arrivé qu' il a déjà retiré son projet . &quot; J' ai lancé un débat avec l' ensemble des responsables de la santé blablabla , qui est en train de se dérouler en ce moment , je referais des propositions avant la fin du mois de janvier &quot; eh bah nous , on va l' attendre . Hein ? Mais vous avez compris les gens ce qu' il faut faire ? Il faut y aller . Et dire à tous ceux qui vous entoure : &quot; Vous savez ? Il veut plus payer les petites maladies &quot; , alors les gens vont dire : &quot; C' est quoi la petite maladie parce que moi ça me fait très mal &quot; justement on veut plus vous rembourser , faites comme ça . Et vous allez voir si ça va pas remonter jusqu' à lui et à la fin il oubliera jusqu' à la première lettre . Il n' y a pas de petite victoire pour nous mes ami(e)s . Vous savez s' ils ont le sentiment que vous êtes prêt à vous laissez faire , ils vont jusqu' au bout . Quand j' étais jeune homme , qu' on nous faisait des cours pour nous expliquer comment c' était la lutte sociale , on nous posait la question &quot; Quelle est la limite à l' exploitation ? &quot; Tu connais la réponse ? La résistance à l' exploitation ! C' est tout , il n' y en a pas d' autre . Eh bien là c' est pareil . Dans le débat qui commence vous devez être vigilant , nous devons tous être vigilant à la première heure . Ces gens là sont décidé à passer à l' offensive parce que la masse d' argent qui recouvre toute la protection sociale est un trésor immense qu' ils veulent faire passer dans les mains privées , dans les assurances , dans tout ce qui est privé parce que c' est une masse immense que nous avons précisément réussi à accumuler pour le soustraire aux mains privées . Et c' est parce que nous l' avons soustrait aux mains privées , au fil de nos luttes , que nous avons remporté ce résultat qui est nous tous ici grâce à nos anciens . Nous tous ici nous avons été soignés , nous tous ici nous avons été pris en charge , nous tous ici nous avons mis nos enfants au monde dans des conditions telles que la mortalité de nos compagnes pour les hommes a diminué extraordinairement et la mortalité de nos enfants a diminuée d' une manière radicale . Gloire à la Sécurité Sociale parce que c' est elle qui a permit tout ça . Dans votre ville , vous allez voir le film &quot; La Sociale &quot; , je crois que c' est le 12 que ça passe , hein ou je me trompe de date , voilà jeudi 12 . C' est un bon film . Il faut que vous alliez le voir . C' est Gilles Perret qui a fait ça . Et puis après , vous allez voir , ça va vous redonner le goût , parce que c' est ça qui faut qu' on fasse dans cette campagne , tout le monde a tellement peur . Les gens en ont tellement prit la tête que ils n' arrivent même plus à croire qu' un progrès est possible . Ils pensent que ce qu' on a , si déjà on peut le garder c' est bien , et ils ont bien raison , il faut le défendre , mais bon , ça on a jamais agit comme ça dans le passé , on voyait toujours la masse extraordinaire de richesse et on disait : &quot; Bah il faut partager autrement &quot; c' était ça le progrès , c' est à cette idée là qu' il faut qu' on revienne . Le progrès est possible . On peut améliorer notre vie quotidienne . On peut vivre mieux , il y a rien qui nous condamne à ça parce que le pays détruit ( ? ) à la Sécurité Sociale Et maintenant qu' il est plus riche qu' il ne l' a jamais été de son histoire , on ne pourrait pas faire vivre ça ? Qu' est -ce que c' est que cette histoire . Il y a un mensonge en route . Depuis 1870 , on a multiplié par 3 l' ampleur de la richesse produite par ce pays . Et on a divisé par 2 le temps de travail qu' il fallait pour la produire . Donc ce pays est plus riche que jamais et vous verrez dans ce film &quot; La Sociale &quot; , il y a un vieil homme , il a 93 ans je crois , le malheureux , c' est un bon camarade , il est mort ... après que le film soit sorti , il avait dit : &quot; Bah le film , ça sera mon héritage &quot; . Je sais pas si vous avez vu ce film , mais ces gars à l' époque , ils ont crée la Sécu en rassemblant dans une seule caisse tout ce qui était éparpillé de tous les côtés , c' était des gars de la CGT . Peut-être que vous aimez pas la CGT certains d' entre vous mais c' était la CGT et alors , ils étaient là , vous comprenez , ils étaient la nuit , le jour en train d' organiser leur truc pour que ça marche , pour qu' on puisse rembourser et ça paraissait tellement incroyable aux gens d' être libéré de tout ce qui auparavant avait écrasé leur existence , la peur de la maladie , la peur de l' âge qui vient . C' est tellement vrai que le ministre qui s' occupait de ça , qui est un communiste , vous allez croire que je fais de la pub pour le parti Communiste toute la soirée mais c' est la vérité , il faut rendre à ceux qui l' ont fait ce qu' ils ont fait . C' était Ambroise Croizat . Le pauvre Croizat , il en est mort d' épuisement . Je vous raconte ça , pas pour vous parler d' un parti ou pour vous parler d' une institution , juste pour que , si vous voyez le film , vous voyez , dans les yeux de ce vieil homme , brillé cette flamme extraordinaire qui est la nôtre , celle qu' on a dans le cœur . Ce monde peut nous appartenir , et nous n' avons pas des revendications extraordinaires , nous demandons des choses simples , éduquer nos enfants , que nos anciens puissent se retirer du poste de travail quand le moment est venu , que nous soyons soigné quand il le faut , et pour le reste foutez nous la paix . On saura organisé notre propre bonheur personnellement , on a pas besoin que vous veniez , nous casser la tête , avec vos obligations de ceci , vos obligations de cela , votre injonction à être comme ceci ou comme cela , porter tel habit pour ressembler à ci ou à ça , manger ceci ou cela , toute cette pourriture que vous faites entrer dans le cœur , dans le corps des gens , par la publicité , par la répétition sans cesse d' un seul modèle d' existence humaine . La Sécu ça vient dans la catégorie suivante , d'abord il y a eu l' invention du feu , de l' agriculture , de l' élevage , et la Sécu ! La Sécu elle installe pour des centaines de milliers de gens une règle , les aléas de la condition humaine , tout ce qui nous atteindra à tous à un moment ou un autre , tous , autant que nous sommes ! Nous sommes et nous serons malade . Tous autant que nous sommes . Nous avançons dans la vie et arrive un moment où nous n' avons plus la force d' accomplir ce que nous accomplissions avant , plus facilement . Et puis , compte tenu de la vie que nous avons eu , nous nous présentons à son extrémité d' une manière différente , suivant le parcours que l' on aura eu . Et cela ne tiens pas spécialement à nos gènes , cela tiens au métier que l' on aura effectué . L' espérance de vie en bonne santé est de 59 ans pour un ouvrier et de 63 ans ou de 62 ans pour quelqu'un qui ne l' est pas , je parle d' un homme . 63 ans pour nos compagnes . 62 ans pour les hommes . 59 ans quand on est ouvrier . Eh ben c' est pas parce que les ouvriers ont des gènes moins résistants que ceux des autres . C' est les conditions sociales qui font ça , donc la société toute entière a une responsabilité devant chaque individu et elle l' assume en répondant &quot; Quoiqu'il arrive , qui que tu sois , homme femme , riche ou pauvre , de chacun selon ses moyens , chacun , tous ses besoins , voilà pourquoi , que que tu sois , quelque soit la maladie que tu as , alors nous viendrons tous à la rescousse et nous mettrons les moyens qui le faut pour te guérir sans te demander plus selon qui tu es &quot; . Autrefois , les anciens , avaient repérés , ce qu' on appelait des &quot; risques &quot; , bon le risque &quot; famille &quot; je pense pas que ça soit un risque mais tout le monde comprend ce que ça voulait dire , quand tu as une famille à charge , tu te comportes pas de la même manière que si tu dois juste t' occuper de toi tout seul , tu as la responsabilité du petit groupe que tu as fondé , ta famille . La santé , le temps de travail , etc . il y a 4 caisses . Mais dites les gens , maintenant il y a une nouveauté c' est les risques liés à l' environnement . Donc , ceux qui vous disent : &quot; ' Ah bah on vit mieux , plus longtemps &quot; , il faudrait qu' ils réfléchissent à ce qui est en train de se passer parce que quand vous interrogez les ami(e)s qui travaillent dans la santé , ils vous disent que 80 % des maladies qu' ils ont à traité ont une origine environnementale . C' est une nouveauté , la pollution 48 000 morts par an . Les pesticides , il y en a partout , on est le premier pays d' Europe à en consommer . Pour ma part je me suis opposé à l' usage des glyphosates , l' Europe a encore permis une rallonge . Le premier qui meurt de ces saletés , c' est celui qui les répand par terre , le paysan qui n' a pas le choix parce qu' il est pris comme ça , faut qu' il produise , beaucoup , et comme lui a dit la banque qui lui a prêtée l' argent &quot; Tu dois faire ceci , tu dois faire cela , dans telles et telles conditions et de telles et telles manières et pas autrement . &quot; Les perturbateurs endocriniens . Vous le savez les gens , qu' il y a de plus en plus de mômes qui naissent bizarrement . On voit toute sorte de malformation qu' on voyait pas encore tant que ça , je veux pas gâcher la soirée hein , mais je pense que beaucoup d' entre vous comprennent ce que je suis en train de dire . Et puis vous avez des maladies qui reviennent , qui y étaient plus , mais qui reviennent du fait d' un côté de l' appauvrissement des gens , bah oui ! comme ils sont plus pauvres , ils mangent moins bien , ils habitent dans des maisons où on a froid , où il y a du plomb , où il y a toutes sortes de saletés qui pourrissent la vie des gens . Ça c' est l' appauvrissement . L' appauvrissement de la population il a un coût sanitaire et pas seulement un coût social , qui est déjà énorme . Et puis , à ce coût , s' ajoute , qui y était pas , les maladies qu' existaient pas et qui viennent à cause du réchauffement climatique et vous allez en voir de plus en plus , parce qu' on en voit partout , alors moi ce que j' ai pu voir c' est pas des maladies , mais j' ai vu mes copains vignerons du Jura qui vous racontent que non seulement on vendange un peu plus tôt , mais voilà que maintenant arrive la mouche drosophile chez eux . Et des mouches drosophiles dans le Jura il fallait courir pour en trouver une . Comme il fait plus chaud , elles arrivent là . Donc nous allons être confronté à un nouveau type de maladie , par conséquent , on peut pas dire : &quot; Maintenant , le problème de la santé des êtres humains il est réglé &quot; , parce qu' on a tout réglé , on est au courant de tout , on connait toutes les maladies , on le soigne toutes . Ce n' est pas vrai . Regardez d'ailleurs la limite du système . Voilà la grippe , vous êtes au courant , 1 million de personnes l' ont déjà chopée . Comme nous vivons tous dans des structures de plus en plus denses , on va pas s' en plaindre , mais , c' est le métro , c' est le bus , c' est le train , c' est bien tout ça . Ça nous permet de circuler plus facilement et sans empoisonner l' atmosphère des autres . Mais ça veut dire aussi que ce répand plus facilement les épidémies . Alors là , alors là , 2014-2015 il y a marqué sur mon papier , 18 000 morts de la grippe . Allez expliquer ça à Fillon hein . Au début les gens ils toussent , c' est tout , &quot; c' est une petite maladie on la soigne pas &quot; , pouf , tout le paquet de maison le prend . La preuve , les malheureuses 10 personnes qui sont mortes dans la même résidence de personne âgée ( ? ) pour comprendre comment ça se passe tout ça . 1 million de personnes malades , alors que dit , de quoi sommes -nous malade ? Madame la Ministre de la Santé a appelée ce mercredi , vous allez voir pourquoi c' est pas vous les chefs , c' est des gens intelligents quoi ... Ils pensent mieux que nous ... Madame la Ministre de la Santé a appelée ce mercredi les hôpitaux à reporter les opérations non urgentes . C' est reparti pour un tour , après les maladies pas grave , voici les opérations non urgentes . Sans doute , ils ont du décidé dans son bureau lesquelles . Pour soulager quoi ? Les urgences , qui elles , sont submergées . Et c' est la vérité . Et de quoi sont malades tous ces gens ? De la grippe , oui mais pas seulement . Ils sont malades des plans de santé et de restrictions de moyens partout . Voilà pourquoi on en peut plus aux urgences , voilà pourquoi ça marche plus nulle part . Moi je vais dire , je profite de ce meeting pour dire à Madame Marisol Touraine . &quot; Dans le cas où vous ne sauriez pas , vous avez prévu de fermer encore 16 000 lits d' hôpitaux , supposez que ça soit fait aujourd'hui , qu' est -ce que vous diriez ? Vous diriez quoi aux gens ? Il y a 16 000 lits de moins et en plus , ceux qui s' y trouvent dehors . Parce qu' il y en a d' autres qui arrivent pour les urgences . Après je vais vous en parler un peu plus , mais quand même . Ils veulent supprimer 22 000 emplois et ils sont tous d'accord avec ça . Si vous êtes allé dans des urgences vous le savez . Vous savez la difficulté que vous avez , déjà pour trouver un spécialiste , hein . Vous êtes sans doute déjà allé , l' un ou l' autre ou vous en avez entendu parler de ce que sont les urgences . Vous savez bien que tout ça ne tiens que par le courage et le dévouement absolu des personnels de santé , s' ils étaient pas là le truc se serait effondré , effondré ! Voilà , elle a trouvé la solution , qu' est -ce qu' on fait ? On reporte opérations qui sont pas urgentes . Alors , ça m' amène ça , à entre un peu dans le détail de cette affaire de maladie . Je vous ai montré tout à l' heure comment Monsieur Fillon avait l' intention de procéder . Il a reconnu lui-même qu' il était pas bon , je le répète une deuxième fois . Il a prévu de pas vous rembourser si vous êtes malade , je le répète une deuxième fois . Et il a prévu de vous responsabiliser parce que vous êtes des irresponsables , vous attrapez la grippe tous en même temps au lieu d' étaler , hein . Franchement , et en plus tous à la même période en plein hiver quand il fait froid , pourquoi vous ne l' attrapez pas un peu la grippe en août ? Ou en juillet , pendant vos périodes de congés comme ça , ça compterait pas comme du temps de travail où vous allez vous reposer . Bref , il va vous responsabiliser et , et , il va vous mettre un dispositif de franchise , cette menterie , et un ticket modérateur sur tout . Alors comme il y a plusieurs niveaux vous pouvez faire le pari , il va vous donner le ticket modérateur le plus bas ou le plus haut , le plus haut , c' est connu . Voilà pour lui , mais , du coup ... C' est à moi à dire ce qu' on va faire nous . Moi je vous dis , nous pouvons faire une chose , d'abord commencer par nous dire que toute maladie doit être soignée et qu' il faut y mettre les moyens pour qu' elle soit guérie . Que tout les affaiblissements du corps des êtres humains , comme nous connaissons des moyens d' y pourvoir depuis un bon moment , les lunettes , les prothèses etc . on doit se donner un objectif . L' objectif est le suivant : tout le monde est soigné totalement et tout est 100 % remboursable . Voilà l' objectif du programme . Ah oui mais ... Dit quelqu'un qu' est pas très malin ... &quot; Oui mais Monsieur Macron il dit pareil &quot; Très bien . Tant mieux , non ? Bon , sauf que l' on est rendu méfiant par toute sorte de mésaventure qui nous est arrivé auparavant , on a été voir comment il compte s' y prendre . Moi je vais vous dire comment moi je compte m' y prendre , parce que je vais vous dire pourquoi il y avait un soupçon . J' ai un soupçon ... Je pense qu' il est en train de nous enfumer . Je sais pas pourquoi , j' ai un soupçon . Enfin si , je sais pourquoi , parce que ... J' ai lu dans Médiapart , le 2 novembre 2016 , écoutez vous autres , vous allez reconnaître la langue ... C' est Monsieur macron qui dit : &quot; Je &quot; . C' est déjà ça , il impose pas , il propose ... &quot; Que chacun puisse se responsabiliser &quot; &quot; Et payer en fonction de ses propres risques &quot; Vous avez compris ? L' horreur . Ecoutez -moi bien pour que vous compreniez jusqu' où se machin va , je pense que ce jeune homme sait pas ce qu' il dit , Non mais oui , ils lui ont passé des fiches , il connait rien à tout ça . Il dit oui oui oui , parce que comme il connait que ça la banque , le machin et tout . &quot; Faut se responsabiliser et puis je m' assure contre mes risques , par exemple j' habite au sommet de la Tour Eiffel , je m' assure contre le risque de tomber &quot; Mais là on parle pas de ça mon bonhomme , on parle de la vie à vie , quoi . Qu' est -ce que ça peut bien vouloir dire ? Je vous refais pas le numéro deux fois . Se responsabiliser ... Ecoutez -moi bien les gens , quand vous en entendez un vous dire à la télé ou ailleurs &quot; Il faut se responsabiliser &quot; , il faut que vous compreniez que l' arnaque commence . Rappelez -vous bien de ce mot , &quot; se responsabiliser &quot; , à propos de santé c' est l' arnaque . Personne n' a envie de tomber malade . Personne n' a envie de se gaver de médicament , il peut y arriver qu' on le fasse par erreur , mais c' est pas vrai , c' est pas une envie . On se lève pas le matin en se disant : &quot; Ah , si je pouvais avoir une appendicite du tonnerre de Dieu &quot; Ou bien : &quot; Waw un cancer pour moi &quot; Qu' est -ce que c' est que cette histoire de responsabiliser . Bien . Mais c' est pas tout . Payer en fonction de ses propres risques . C' est 2 horreurs dites en même temps ... C' est quoi votre risque à vous ? Et à vous ? Et à vous ? Je vais vous le dire ... Si vous le savez pas , je vais le savoir à votre place . Vous fumez ? C' est pas bien ... Alors vous fumez , vous vous rendez malade . Et vous venez me demander à moi de vous rembourser . Je veux pas , vous n' avez qu' à pas fumer . Ah , vous faites l' apéro C' est pas sérieux , d'ailleurs vous avez une drôle de tête Quoi ? Vous avez une cirrhose ? Ah zut , mais non non Monsieur , j' ai une cirrhose mais ça n' a rien à voir . M' enfin vous dites ce que vous voulez . Bla bla bla . Mais je l' ai entendu celle-là , je ne l' invente pas ce que je suis en train de vous dire . Ça c' est passé en Angleterre sous Monsieur Tony Blair , qu' ils ont commencé à dire qu' il faut responsabiliser les gens , donc si vous avez un comportement irresponsable , eh bah venez pas vous plaindre et venez pas demander qu' on vous rembourse . Ça c' est pour de l' extérieur à l' intérieur . Puis , après , il y a chacun pour soi . Alors moi je vais dire , &quot; écoutez , j' ai une santé de fer &quot; , ce qui est vrai , j' ai une santé de fer , tout va bien , bon patrimoine génétique , je suis né la bonne année . Eh bah oui Monsieur figurez -vous , eheh 1951 , quand vous achetez une pomme , quand vous achetez une pomme en 1950 , vous mangez une pomme et pour avoir la même quantité de vitamine qu' il y avait dans la pomme que je mangeais quand j' étais petit , vous il faut en manger 10 . Bon appétit . C' est ça la vérité . C' est l' alimentation , c' est comment j' ai été traité , j' ai été soigné , dès tout petit , pour les dents , pour ceci , pour cela , etc . Et voilà comment vous arrivez après , puis j' ai eu la chance de ne pas être dans un métier qui m' a usé le corps comme c' est le cas pour plein de gens . Voilà . Donc si moi je dis : &quot; Bah écoutez , moi , je crois que toute façon , maintenant ça va quoi , j' ai vu ce qui avait à voir , j' espère en voir encore pleins d' autres , mais je m' en plains pas , j' ai pas envie de payer pour vous , là , vous êtes trop jeunes &quot; &quot; Vous allez attraper toutes les maladies , tout ça , pourquoi je paierais ? &quot; C' est ça dire &quot; je vais m' assurer pour mes propres risques &quot; , dire &quot; je m' occupe plus des autres &quot; &quot; Je m' occupe que de moi &quot; . Ça c' est le système de l' assurance . Evidemment l' autre qui lui a des gros risques , &quot; j' ai pas de chance &quot; , d'abord il a fait un travail de chien , il a eu pleins de sortes d' embêtements dans la vie et alors lui , il arrive et son risque ça se voit , rien qu' à le voir Et il va s' assurer , on va dire : &quot; Chacun s' est responsabilisé et paye en fonction de ses risques &quot; , donc si vous avez une tête à risque , vous allez bien payer et vous pourrez pas payer Messieurs , Dames , les gros malins les comptables , vous pourrez pas payer , c' est pas parce que vous inventez ça quand vous avez la quarantaine et que vous crachez le feu , que vous courrez partout , et que vous avez une vie qu' est pas trop épuisante , compte tenu de vos moyens , faut pas que vous croyez que ça va être tout le temps comme ça , parce que tout le monde est malade et tout le monde meurt ! Et 60 % des dépenses de santé , vous les faites dans les derniers temps de votre vie . Alors ça c' est un propos de martien , c' est pas un propos de terrien . Ça ça existe pas ce qu' ils disent . Sauf ... dans une société de violence . La Sécurité Sociale c' est que chacun est responsable de tous les autres comme nous le sommes réellement dans la vie parce que dans la vie nous sommes réellement responsables de tous les autres . Où il est mon papier ... Voilà ... Alors après , vous dites : &quot; Bon d'accord on a compris Monsieur Mélenchon , c' est pas bien et vous , vous payez ça comment ? &quot; Vous voyez comment vous êtes complètement déformé ? Et moi je suis obligé d' apprendre une fiche entière comme ça pour vous répondre ... Mais oui parce que maintenant on connait plus que ça . Vous avez même pas le temps de terminer la phrase qu' on vous enfonce le micro dans la bouche . &quot; Et vous payez ça comment ? &quot; &quot; Comment ? &quot; &quot; Il y en a pour combien ? &quot; Eh ben il en a pour combien , ben je suis content parce que le papé il est vivant , je suis content parce que le gamin on l' a soigné , et combien ça coûte ? Je m' en fous ! Voilà ! Evidemment que je m' en fous pas ... Je suis responsable . Mais qui il leur donne ce droit de nous regarder de haut , en écoutant pour dire après &quot; Oui , comment vous payez ça ? &quot; Parce qu' on y a pas pensé , on est tellement bête . Nous on est le peuple hein , on a pas d' idée , tandis que eux ils sont intelligents , là preuve , il y a qu' à voir comment cette société marche bien depuis qu' ils s' en occupent , c' est à dire depuis toujours , ou presque . Ah ça marche , il y a pas de chômage , tout le monde est content , et ainsi de suite . Alors comment on paye . 100 % , moi je répond Si vous croisez Monsieur macron , demandez -lui , qui est -ce qui paye ? La Sécu ou les complémentaires ? Tout le monde sait ce qu' est une complémentaire ici ? Vous avez la Sécu et pour le reste vous prenez une complémentaire . Ouch ! Ils vous en remboursent encore un petit bout , ah il y en manque encore un petit bout , sur-complémentaire là-dessus Hollande est passé , vous avez vu le geste que j' ai fait , complémentaire , sur-complémentaire , Hollande il est passé : complémentaire , sur-complémentaire , C' est à dire que dans beaucoup de cas , vous êtes moins protégé que vous ne l' étiez avant par la complémentaire que que vous preniez tout seul parce que maintenant vous êtes dans une entreprise , c' est l' entreprise qui négocie le contrat et le contrat qui vous négocie , comme il est mauvais vous avez moins qu' avant . C' est vrai ce que je raconte , vous le savez ? Bon , très bien , mais c' est important qu' on se comprenne parce que ça , c' est des choses tous les jours , et maintenant ... vous avez pris la retraite Monsieur . Qu' est -ce qu' ils vous disent à la complémentaire ? &quot; Ah bah vous êtes en retraite alors vous gagner un peu moins , donc vous payez un peu moins . &quot; C' est ça qu' ils vous disent ? Public : Non . &quot; Vous êtes à la retraite ? Vous payez encore plus ! &quot; On est d'accord , voilà . Alors avant , nous on disait : &quot; C' est nos mutuelles , donc c' est à nous ! &quot; Ça c' était avant . Maintenant grâce à l' Europe qui nous protège , les mutuelles sont devenues des compagnies d' assurance , comme les autres . Ou à peu près , non à peu près parce que des fois c' est moins performant . Non mais c' est la vérité . Moi je connais pas tout le monde ici , je connais même personne , mais je suis sûr j' en prend 3 ou 4 qui étaient à la mutuelle , de telle ou telle branche que je vais pas nommer , et parce qu' avec le syndicat on était d'accord , hein . Et maintenant la mutuelle en question , elle ne fait pas du tout vos affaires , et vous , vous restez fidèle parce que vous pensez que c' est celle du syndicat mais la vérité c' est que tout le système est une arnaque . Donc moi je pense et je vous le dis bien franchement , cet homme Monsieur Macron doit répondre , 100 % remboursable , qui rembourse ? La Sécurité Sociale ou la mutuelle , la complémentaire . Eh ben le résultat sera pas du tout le même parce que s' il vient vous dire : &quot; Je vous rembourse à 100 % et c' est la complémentaire qui va s' en occuper &quot; , qui c' est qui va payer ? C' est vous ! Donc , oui oui vous êtes remboursé à 100 % mais vous crachez plein pot et comme c' est trop cher pour vous , vous vous le payez pas , et comme vous vous le payez pas , parce que la liberté etc . etc . vous êtes pas obligé . Donc c' est un pur mensonge . IL y a qu' une façon de faire du 100 % renouvelable , qui soit sérieux , c' est 100 % par la Sécurité Sociale parce que ça on sait comment ça marche et comment ça rembourse et que nous tous on en a marre de faire des papiers parce que les patrons arrêtent pas de se plaindre qu' on leur fait faire des papiers , et ceci et cela , nous on en a marre de faire des papiers , un coup la Sécu , un coup la complémentaire , la sur-complémentaire , c' est là qu' est déjà , parce que avant je me suis déjà engagé et ça n' a pas changer truc là . Voilà . On connait tous ça ! Donc la Sécu , comment on fait ? Bon là , on rappelle qu' il y a un tiers des français qui renoncent à se soigner , vous avez compris pourquoi . Alors , il y a 2 choses à faire . Premièrement , faut que ça coûte moins cher , à la Sécu . Ça veut dire que , on a une intervention à faire sur le prix du médicament et sur la façon avec lesquels les médicaments sont commercialisés , non seulement comment physiquement ils sont commercialisés que si tu me donnes une boîte pour 1 semaine , c' est pas la peine de me mettre des cachets dedans pour 3 mois . Premier élément mais surtout , deuxième élément : il faut un pôle public du médicament parce que mes ami(e)s , le médicament aujourd'hui c' est la vache à lait de ces gens là , c' est la vache à lait , il y a aucun effort à faire . Tu as trouvé une molécule , un petit coup de ceci , un petit coup de cela , c' est normal que la recherche soit récompensé , c' est normal que ... mais c' est pas normal que ça dure pendant des années et des années et qu' ils continuent à engranger des profits , des bénéfices sur des médicaments dont on peut pas se passer . Et vous savez ceux qui les trouvent les médicaments hein , c' est pas les gros bonnets , pardon de parler vulgairement , un jour je fais une manif avec Sanofi , vous voyez lesquelles c' est , Sanofi c' est une boite formidable , ils ont nommé un PDG américain , flanqué d' un autre qui sont venus tou les deux ils se sont installés un magnifique Q. G à Paris , avenue Foch , avec une belle salle à manger , un chef 4 étoiles , des choses qui importe dans la pharmacie . Et puis , après ils ont commencé à virer les gens , et puis ils ont commencé à étudier ce que les gens faisaient , alors mettons par exemple , tiens j' ai 1 000 euros , je prend un chiffre comme ça , j' ai 1 000 euros , qu' est -ce qu' il fait lui ? Il cherche une molécule pour empêcher vos dents de tomber , il y en a pour combien de temps ? Je sais pas , je cherche . Oui bon ... Et une fois que tu auras trouvé , ça prend combien de temps pour aller sur le marché ? Ah bah les dents ... Ça prend 4 ans . Et toi , qu' est -ce que tu cherches ? &quot; Comment faire que les cheveux blancs devinent pas jaune ? &quot; Combien de temps ça prend ? &quot; Beh écoute , on s' est mis d'accord avec Pepsi-Cola d' un côté , et la savon de Marseille de l' autre , chacun a mit un petit peu , on pense qu' on peut arriver en 2 ans , puis après il nous faut 1 an pour commercialiser ça &quot; Tiens , les cheveux jaunes c' est insupportable , le truc des dents on verra on verra. . D'ailleurs il y a plus d' argent , aller ça va , fais autre chose ... Voilà comment ça se passe . Ce que je vous raconte là c' est les gens de Sanofi qui me l' ont raconté , c' est pas un truc que j' invente . J' étais dans la manif , je marche à côté d' un gars qui ronchonnait et je lui dis : &quot; Alors vous êtes Sanofi , vous ? &quot; Il a un peu les cheveux blancs , je vois bien qu' il est de mon âge , Alors il me dit : &quot; Oui oui , bon maintenant je suis à la retraite &quot; bon , alors &quot; Et qu' est -ce que vous faisiez à Sanofi ? &quot; &quot; Moi j' ai découvert la molécule qui empêche les artères de se boucher &quot; Ce type était là , je me dis : &quot; Mais c' est un Saint cet homme &quot; , faut lui faire une statue , je lui dis &quot; Il y a du monde qui a du être tiré grâce à votre ... &quot; &quot; Oui oui , j' ai &quot; &quot; , il me dit &quot; Je sais pas , peut être 500 000 , 600 000 , peut-être 1 million , j' en sais rien &quot; Ce type avait travaillé et trouvé quelque chose qui a sauvé 1 million de personnes , je lui dis : &quot; Ils vous ont donné quelque chose pour ça ? &quot; , il me regarde et puis , je vois qu' il y a un blanc qui passe comme ça &quot; Je veux dire , vous avez eu ... &quot; , il me dit &quot; Bah oui , j' avais une bonne paye &quot; le gars il a eu une bonne paye , et c' est tout . Et les autres ont fait des millions et des millions et des millions sur sa trouvaille , tant mieux hein , il a sauvé pleins de gens , mais tout ces millions , c' est quoi la justification pour que ça aille dans la poche de Messieurs les actionnaires et pas dans celle des assurés de la Sécurité Sociale dont ce Monsieur qui travaillait et ne pouvait travailler tranquillement que parce qu' on le soignait et quand il était malade il pouvait compter sur la Sécurité Sociale pour être guéri pour retourner chercher , et trouver la molécule qui sauvait les autres . Vous voyez comment tout se tient ? Par conséquent il y a aucune raison pour qu' on admette l' idée que pour trouver et pour chercher il faille qu' il y en a un qui arrive un grand fouet pour vous dire &quot; Allez , il faut être compétitif , flexible , trouver quelque chose qui intéresse les gens qui arrive vite sur le marché et que ça me rapporte des millions &quot; ces millions devraient être à la disposition de tous , donc ... premièrement élément : peser sur le prix du médicament , donc le pôle public du médicament , ça va être long mais qu' est -ce que vous voulez moi j' explique . Qu' est -ce qui vous reste à payer , vous , une fois que vous avez fait la complémentaire , la sur-complémentaire et tout le reste ? Vous en avez pour 16 milliards , il faut donc que je trouve 16 milliards , sans compter la baisse du prix du médicament , pour amener à 100 % payé par la Sécurité Sociale . Comment je m' y prend ? Premièrement , nous supprimons les dépassements d' honoraire chez les médecins . 8 milliards d' euros ! Deuxièmement , faut pas trop qu' ils me cherchent ceux-là , on va dire les choses comme elles sont , ils ont inventé une horreur qui s' appelle le &quot; Numerus Clausus &quot; , résultat il y a pas assez d' étudiant en médecine , que aujourd'hui quand vous voulez faire médecine , vous êtes obligé de faire une école prépa , que c' est vos parents qui vous la payent , alors qu' avant on rentrait directement en 1ère année , maintenant si vous avez pas fait la prépa ... Justement dans laquelle enseigne les types qui ensuite sont dedans qui font les examens , vous ne pouvez pas être médecin . Et cette invention aboutit à ce résultat qui il y a peu de personne capable d' intervenir et notamment dans les spécialités et ils vont tous au même endroit . Voilà pourquoi il y a des déserts dans le pays , et c' est la raison pour laquelle , c' est un petit incident , nous allons faire une médecine publique . Voilà , avec des centres de santé public avec des médecins fonctionnaires . Vous entendez cette horreur ? Fonctionnaire . Des médecins qui sont d'accord et vous avez pleins de jeunes gens qui sont d'accord , vous avez pleins de jeunes gens qui sont d'accord pour avoir cette vie , la preuve , c' est que souvent les jeunes médecins vous en avez pleins qui font des cabinets collectifs , donc ça doit possible , non ? Mais ceux qui veulent pas sont pas obligés , mais veux qui veulent ils pourront . Et à ce moment , on aura à nouveau un maillage du territoire qui fait qu' on pourra prendre les maladies à temps , soigner les gens à temps , avec des toubibs qui font pas une consultations par quart d' heure et où tout le monde passe là dedans ... Il y a beaucoup de toubibs qui font des efforts inouïs , hein , mais c' est absurde tout ça , comment vous êtes mis dans une situation pareille ... Donc les dépassements d' honoraires , non ! Il y en a pour 8 milliards , ensuite , je vous ai parlé des complémentaires tout à l' heure , vous savez que ces gens passent leur temps à nous expliquer que le public ça coûte cher et le privé c' est merveilleux . Résultat : Ça coûte 6 , 5 milliards d' euros de frais de gestion pour les complémentaires . 6,5 milliards , c' est à dire que quand c' est la Sécu ça coûte 8 % en frais de gestion et quand c' est tous les autres c' est à partir de 18 et jusqu' à 25 % de frais de gestion . Autrement dit , ça coûte plus cher que la Sécu les complémentaires et tout ce qu' il y autour . Avec ça ... en ramenant dans la Sécu tout le personnel , mais en ramenant tout le personnel dans la Sécurité Sociale ce qui permettrait de travailler plus tranquillement , ça vous donne 6,5 milliards à rajouter aux 8 que je vous ai dit tout à l' heure nous voici déjà rendu à 14 . Et après , en faisant les comptes , mes camarades découvrent qu' il y a 3,5 milliards d' euros d' exonération de cotisations sociales en faveur des complémentaires santé . Je les reprend . Egal : un peu plus que ce qu' il y a besoin , 17 milliards . Il ne faut pas dire ce n' est pas possible et si quelqu'un conteste mon addition là , je lui propose d' aller visiter un pays lointain et exotique où voir une Sécurité Sociale qui rembourse à 90 % , bien sûr il faut prendre un ou deux avions , 4 ou 5 bateaux , pour finir on y arrive ... La capitale de ce pays étrange , s' appelle Strasbourg , ça se passe en Alsace-Moselle . Localement la Sécurité Sociale du coin rembourse à 90 % et les gens ont 1 point , 1 % de cotisations supplémentaire par rapport à ce que vous avez vous . Eh ben ce % supplémentaire je suis sûr que vous êtes prêt à le payer , si je fais toutes les économies que je viens de vous expliquer . Donc si on peut le faire , je dis à mes ami(e)s d' Alsace-Moselle ... Parce qu' à chaque fois que je parle d' eux je vais abroger le Concordat , il y a toujours un gros malin qui arrive derrière en disant : &quot; Ah oui mais notre droit social il est dedans Mélenchon veut le se supprimer &quot; . Non non ... Mélenchon veut supprimer le Concordat , ça oui , il le supprimera mais le droit local c' est l' inverse , je propose d' étendre à toute la France votre système en l' améliorant un chouia , pour que ça fasse 100 % . Par conséquent , on peut le faire ! Va falloir faire aussi un peu de prévention . Monsieur Macron a dit qu' il était d'accord , fallait faire de la prévention . &quot; Ah non pas vous , ceux qui ont supprimé la médecine du travail , c' est vous , avec la loi El Khomri &quot; , vous le savez ... Qui a supprimé la médecine scolaire ? Mais les gens , si vous ne comprenez pas , vous comprenez ça ? Les petits , si à l' école , vous avez un toubib aujourd'hui pour 11 000 élèves , mais à l' école c' est là qu' on peut regarder comment ça tourne ou pas , on les passe , il faut le temps qu' il faut , il y a toute l' année pour faire la visite et dire attention à ci , à ça , et puis quand un petit jeune à un problème il va pas forcément aller le raconter à ses parents qu' il a un problème , il va voir l' infirmière ou l' infirmier à l' école , on sait tous ça , qu' est -ce qu' on préfère ? Savoir la bêtise ou qu' elle soit réparée ? Bah on préfère qu' elle soit réparée bon sang de bois , on est tous fait du même bois je pense , non ? Donc il faut qu' il y ait cette médecine préventive , il va falloir le faire . Et la politique qu' on propose c' est une politique préventive parce que quand vous dites que vous allez sortir des énergies carbonées grâce à la planification écologique . Eh ben vous faites de la médecine préventive parce qu' il y a moins de gens qui s' empoisonnent avec les produits qui sont liés à la pollution . Quand vous dites que vous faites de l' agriculture bio , vous faites de la médecine préventive parce que vous arrêtez la malbouffe . La malbouffe qui fait qu' aujourd'hui , il y a une telle épidémie d' obésité dans ce pays . C' est pas parce que les gens s' empiffrent , c' est parce qu' ils mangent des choses qui ne sont pas bonnes . L' écologie c' est de la médecine préventive . Et enfin , il y a des défis qu' il faut qu' on relève , vous le savez pas parce que peu de gens vous en parlent mais l' ONU , Qu' est la seule instance internationale que nous pouvons respecter et qu' il faut essayer de suivre , eh bien l' ONU a lancée une alerte , demander à tous les pays de mettre au point une politique de santé de lutte contre la résistance microbienne parce qu' à force de foutre des antibiotiques dans le manger des vaches , des cochons , des poissons , et bien la résistance de l' organisme reste la même tandis que celle des agresseurs viraux , elle augmente . Et donc il va falloir un plan de cette nature , et naturellement si c' est nous qui l' emportons , on le fera . Bon ... Je suis sur la maladie , la prévention , un mot du chômage dont le seul remède est le suivant : le travail . Si vous commencez à partir de l' idée qu' un jour il y aura pas de travail , vous mettez le doigt dans un mauvais engrenage parce que c' est pas vrai . Il y aura toujours une quantité de travail nécessaire . Mon meeting ce soir c' est pas sur le temps de travail , mais il y aura toujours une quantité de travail nécessaire . Si vous voulez que le robot fasse le travail à la place des gens il faut construire le robot et il se construira pas tout seul , ah si , par d' autres robots et à la fin vous croyez réellement que la valeur continuerait à être créer sans travail humain , ça n' existe pas . Mais je met cette question de côté . Moi j' en suis pas à vous dire que je vais faire tout ça et qu' on peut le faire tous ensemble en 5 ans , ce n' est pas vrai . Ce qu' on peut faire c' est relancer l' activité , j' ai déjà dit comment , c' est pas mon sujet ce soir , mais il va nous falloir 3 millions et demi de personnes de plus au travail . Mais je parle du chômage sous un autre angle , sous lequel on en parle pas . Le chômage ça rend malade , et si jamais on le dit ça , le chômage ça rend malade de stress , d' angoisse ... de maladie qui viennent parce que quand vous êtes là à miséré , tourner en rond , vous commencez à plus très bien vous occupez de vous , à plus faire attention à certaine chose , à d' autre . On connait l' addition à la fin , l' addition c' est 18 000 morts à cause du chômage . C' est pas moi qui l' ai inventé , c' est pas mon équipe , c' est l' INSERM qui ont établi ce bilan , 18 000 morts à cause de ça . Le chômage c' est quelque chose qui tue la personne de l' intérieur . Je le dis parce que souvent on oublie ça quand on parle à nos copains qui sont plus au travail . Alors , que vous propose dans cette situation , Monsieur Fillon ? Il a trouvé le remède . Eh bien oui , je vous explique . Vous êtes globalement une bande de tire-au-flanc . C' est bien connu . Par conséquent , Monsieur Fillon propose qu' il y ai une dégressivité de l' indemnisation , c' est à dire plus il y a longtemps que vous êtes au chômage , que je viens de vous dire plus il y a longtemps plus vous êtes malades , plus vous êtes pas bien , justement à ce moment là , on vous enlève des ressources , et pourquoi ? Pour vous obliger à aller au travail où vous vous voulez pas aller . Quelqu'un peut me montrer une personne qui veut pas aller au travail ? C' est ce que je vous ai dit tout à l' heure , ça n' existe pas . C' est des calomnies . C' est pur vilenie de parler comme ça . Peut-être qu' ici il y a des gens ou au travail ou en retraite ou en train d' étudier , mais pensez à celui ou à celle qui n' est pas au travail . Quel effet ça lui fait quand il s' entend dire : &quot; On va faire dégressif parce que vous abusez &quot; , vous abusez ... Eux ils abusent tout le temps , toute la vie , en permanence . Leur seul statut social est un abus . Mais qui abuse ? Ces gens parce qu' ils auraient des indemnités pendant 2 ans , ils veulent les rendre dégressives . Je pense exactement le contraire , non seulement faut pas les rendre dégressives , mais au bout d' un certain temps , tout le monde ait d' une manière ou d' une autre un droit opposable au travail . Alors on me dit : &quot; Comment vous faites une chose pareille , et comment vous le financez ? &quot; A l' heure à laquelle je vous parle tout n' est pas au point dans cette idée , mais ce que je sais , c' est que l' expérimentation a commencé . On dit : &quot; Pas une personne doit être sans au moins comme salaire un SMIC &quot; . Il y a une expérience qui est commencée , grâce à une association qui a fait un travail magnifique , c' est &quot; ATD Quart Monde &quot; peut-être que vous connaissez ce sigle ... Eh bien ils ont montés une activité qui est expérimentale , moi je l' ai vu et j' ai parlé avec des responsable à Châteldon , dans le Puy de Dome , il se trouve que le maire est un de mes ami . Et il y a cette association qui travaille de la manière suivante : elle encaisse ce qu' aurait reçu le chômeur en tant qu' indemnité diverse , et avec ça , ça fait la base qui lui permet d' être , ils appellent ça une association à but d' emploi , c' est elle qui embauche la personne et qui ensuite la place , en recevant cette somme et elle la place en faisant des offres de service , naturellement , il faut discuter avec les petites entreprises du coin pour pas tout fracasser et amener de la main d'oeuvre d' un côté , subventionné , qui tue le travail ordinaire mais on y arrive et la preuve c' est que eux y arrivent . C' est la raison pour laquelle je crois qu' on peut se donner cet objectif , en toute hypothèse . Moi je fais le pari et vous verrez bientôt quand je vous montrerais mes chiffres qui n' est pas encore prêt . Vous verrez que le décollage de l' activité doit nous permettre de pouvoir encaisser ce choc . Mais vous savez que la société a été détruite . Nous devons reconstruire parce que quelqu'un qui est resté longtemps en dehors des circuits , il a du mal a y venir . Un de mes ami m' a raconté , qui est éducateur de rue , il me dit qu' une personne , je prend un cas extrême , qui est resté dans la rue pendant 1 an , il y a beaucoup de monde vous savez dans la rue ... Là déjà depuis le début janvier , il y en a 5 qui sont morts attribués au froid , ça sera + ... A la fin de l' année ça fait 2 000 personnes et il y a des gosses là-dedans , il y a des gosses hein . Tous les ans , qui meurent de froid , d' abandon , dans ce grand pays riche . On le voit moins on m' a dit en province ... Comme on disait la province ... Mais je vous garantie qu' à Paris , il n' y a pas de moyens de ne pas le voir . Les gens sont couchés dans la rue , en bas de chez vous . C' est terrifiant C' est terrible ... Alors , excusez -moi parce que quand je pars là-dedans ça va moins bien après , je passe à la suite allez . Je parlais du chômage qui rend malade et je vous expliquais comment on pouvait faire pour que tout le monde ait son revenu et sa place dans la société . Alors je vous ai dit on va faire un plan de développement de l' économie , ça a tourner , il y a des gens qui vont être embauché , tout ça . Ceux qui seront , qui auront du mal à se remettre en route , on va les accompagner , on va avoir besoin d' eux . Vous savez , j' ai dit 100 fois &quot; faites le prix du malheur &quot; , vous autres les riches , les puissants , vous passez votre temps à pleurer sur les coûts , vous appelez ça des coûts non . Il y a un prix au travail , il y a un prix à la santé , mais faites le prix , faites les coûts du malheur . Combien ça nous coûte ? Cette pauvre personne que vous voyez là , vous êtes certains que vous en aviez pas besoin ? Vous pensez qu' il savait rien faire d' utile pour la société ? Qu' il apportait rien à d' autres ? Qu' il avait pas de famille ? Qu' il avait pas de gens à aimer ? Qu' il était pas aimé lui-même ? Les pauvres nous manque ! Il y en a 9 millions dans ce pays ! 9 millions de personnes qui sont retirées de ce qu' elles pouvaient apporter aux autres . La pauvreté c' est le prix du malheur . C' est des milliers de gens . 9 millions de personnes qui sont mis hors circuit d' une manière ou d' une autre et qui manque à la société . Bon , eh ben on va faire comme on peut mais on va se donner pour objectif que ça cesse ! Nous ne voulons plus vivre notre bonheur personnel au milieu d' un océan de malheur . Nous ne voulons pas ! Alors une autre solution pour qu' il y ait du travail , eh ben , c' est de redonner aux gens le droit de partir à la retraite quand le moment est venu . Ca non plus , il n' y a pas besoin d' être très intelligent pour comprendre qu' en repoussant sans cesse l' âge de la retraite , vous créez quoi ? Des chômeurs de plus , c' est tout ! C' est tout ce que vous faites ! Parce que , je dis vous , ceux qui décidaient ça , comment ils s' appellent déjà ? Celui qui a abrogé la retraite à 60 ans , vous vous en rappelez ? Fillon ! Exactement ! C' est lui qui l' a amenée à 62 ans , et l' autre le revoilà qu' il vient pour vous proposer quoi ? Vous êtes des masochistes ou quoi ? Il vous l' a amené à 62 ans , il vient vous la proposer à 65 ans ... Ce qui veut dire que pour ceux qui ont des carrières incomplètes ou plus exactement qu' ont pas le nombre d' années de cotisations , qui correspondent à cette sauvagerie , ça veut dire que vous attendrez 70 ans pour partir . Tout à l' heure je vous ai donné un âge , je vous ai dit : &quot; l' espérance de vie en bonne santé c' et 63 ans &quot; Ca veut dire que 1 ) vous allez augmenter le nombre de chômeurs , le nombre de malade et de gens que vous allez tuer au travail , parce que c' est ce qui va se passer si vous faites ça . Le travail fera mourir des gens de plus , qui sinon auraient apporté à leur famille , aux siens et à eux-mêmes la part de bonheur qu' ils étaient en droit d' attendre de l' existence et des efforts qu' ils ont accomplis auparavant . Ce sont des brutes et des ignorants qui disent que puisqu'on vit plus longtemps , il faut travailler plus longtemps , mais non on vit plus longtemps parce qu' on travaillait moins longtemps ! Voilà la vérité . Eh ben voilà , ils ont déjà réussi , l' espérance de vie des français a reculé en 2015 pour la première fois depuis 1969 , merci Monsieur Fillon . - 3 mois pour les femmes et - 3 pour les hommes . Pour une fois c' est l' égalité . Voilà , selon l' INSEE , pas l' institut bolchévique . Selon l' INSEE , l' espérance de vie en retraite sans incapacité sévère devrait être stable pour les générations nées entre 1960 et 1990 . Sourire dans le salle ... Sauf ... en cas de poursuite de l' augmentation de la durée de cotisation . Vous êtes dedans donc vous êtes pas dans la bonne cohorte . Nous pouvons faire la retraite à 60 ans et ça ça va aller vite . Nous pouvons faire la retraite à 60 ans avec 40 annuités , vous me croyez pas , je l' entend à la qualité du silence . Vous vous demandez quel lapin je vais sortir de mon chapeau parce qu' évidemment la question que vous allez posé c' est : &quot; Comment on va payer ça ? &quot; Eh bah oui , tout le monde en est là aujourd'hui . Eh bah je vais vous répondre . Vous la connaissez peut-être la réponse et si vous la connaissez pas , vous l' apprenez ce soir et vous la répéterez aux autres . Nous les progressistes , nous pensons la chose suivante : le progrès social fait du progrès économique . C' est ça notre idée fondamentale . On fait pas de la production et de l' économie en partant de produire n' importe quoi , n' importe comment , pourvu que ça soit pas cher . On part de &quot; quoi a -t-on besoin ? &quot; On regarde de quoi on a besoin et on regarde qui c' est qui peut le produire , le public , le privé , etc . etc . Et là c' est pareil . On part de ce qui est nécessaire . Voilà , on a besoin de la retraite à 60 ans . Et on a besoin au bout de 40 annuités , parce que c' est ce qu' on a gagné . me dites pas que le pays a été ruiné par la retraite à 60 ans avec 40 annuités , c' était 39 . Il y avait un problème dans les caisses , oui , il y avait un problème aussi longtemps qu' il y a du chômage . Je me rappelle le nombre de discours que j' ai pu faire au Sénat sur le thème &quot; Mettez 1 million de gens au travail et il y a plus de trous dans la caisse de la Sécu &quot; , eh ben en effet . Eh ben en effet . Ben ça , on l' a vécu , en 2002 , 2001 , 2000 , voilà , c' est comme ça , à cet époque là tous les comptes étaient au vert , comment ? On a eu 2 millions qui sont revenues au travail et parmi ces 2 millions , il y en a , stricto sensu , 355 000 grâce à la réduction du temps de travail . Mais comme ceux-là sont allés bossés , comme ceux-là ont crée de l' activité , ils ont donc crée de l' activité pour les autres . Vous suivez ? Donc au total , ça vous donne 2 millions de personnes . Et comme on a eu 2 millions de personnes au travail et 350 000 postes crée comme ça , eh ben les ami(e)s , c' était la période la plus forte de création d' emploi du siècle ! Et chaque emploi qui était crée là , coûtait en exonération diverse , payé par l' impôt etc . etc . 13 000 euros , c' est à dire 100 fois moins que coûtent ces soit disant emplois que Monsieur Gattaz avait promis et qu' il a jamais fait avec les 40 milliards qu' on lui a donné sans aucune contrepartie . Comment on fait alors ? eh ben je vous répond par une mesure de justice . Ecoutez bien parce que ça permet après à se marrer en société . Je dis ben voilà , on peut payer . Pour ça , faut + de cotisations , tout le monde est d'accord , c' est des maths élémentaires . Alors , où va -t-on trouver les cotisations supplémentaires ? Voilà l' idée , payer les femmes comme les hommes et le surplus de cotisation que ça procurera permet la retraite à 60 ans avec 40 annuités . Alors , ils ont été faire comptes parce qu' ils se sont dit : &quot; Oui Mélenchon il raconte des trucs &quot; non , il est prudent Mélenchon et puis , c' est souvent ils se trompent avec moi , parce que ils croient que comme on est les derniers arrivés , qu' on a pas d' argent , qu' on a pas de grand machins et tout ça , ils pensent que ... mais non ils ont rien compris , je suis entouré d' une foule de gens qui en réalité préparent , travaillent , organisent , et dont je suis le porte parole . Ce sont des bénévoles , mais dites donc les gens , les bénévoles c' est pas des amateurs . C' est ça qu' ils ont pas l' air de savoir , c' est que les bénévoles c' est tout les gens qui font le boulot , la plupart du temps pour les autres parce qu' ils sont payés pour ça , celui qui fait des chiffrages pour le gars de droite ou du PS eh ben des fois dans sa tête , il est avec nous . Eh ben oui , alors qu' est -ce qu' il fait ? Il vient nous le raconter . Il nous explique comment on fait , comment on procède , eh bah oui , et il y en a partout , ahah ! Partout ! Dans tous les métiers ! Les plus improbables ! Dans l' armée , dans la diplomatie , partout , ils se disaient &quot; c' est pas possible &quot; , si , il y en a un , il y en a une , etc . Et donc les chiffres qu' il y a là ils sont en béton armé donc ils ont renoncé à essayer de me prendre en défaut . Alors si , ils ont quand même essayé l' autre matin pour ceux qui suivent la radio , Monsieur Bourdin a dit : &quot; Mais alors vous vous rendez pas compte , ça va être un choc terrible pour l' économie &quot; c' est facile là , je lui dis &quot; Ah bon ? Vous êtes d'accord pour que les femmes soient payés moins que les hommes &quot; &quot; Ah non non je n' ai pas dis ça &quot; Si vous dites ça , vous dites comme moi et voilà , perdu . C' est indéfendable , le contraire . Indéfendable . Alors il y en a un qui l' a dit l' autre soir à la télévision c' est Monsieur Matthieu Croissando parce qu' il avait en face Monsieur Barbier de l' Express , un gars qui assume ses idées et il disait : &quot; Oui mais Mélenchon il parle , il promet des trucs , c' est toujours plus ou moins vrai &quot; il a pas dit en quoi c' était plus ou moins mais voilà , ça sème le doute vous voyez , au fond je suis assez sympathique comme bonhomme , puis bon , à force de répéter tout le temps que je suis méchant , maintenant ça lasse on passe à autre chose . Donc je suis intelligent , cultivé , ils l' ont dit , c' est pas moi hein , mais bon , c' est comment elle a dit Madame , il y a une journaliste s' appelle Madame Ottenheimer elle a dit j' étais Vintage . Alors , ce qu' elle sait pas la Dame , c' est qu' elle est encore plus vintage que moi , parce qu' elle sait pas qu' aujourd'hui vintage c' est vachement à la mode . Alors elle , elle croyait qu' elle allait me démolir mais elle fait l' inverse . Bon enfin bref , et le directeur de l' Obs , c' est quand même le directeur de l' Obs , il dit &quot; Oui m' enfin bon &quot; , il les regarde tous et il dit &quot; bon il y en a un ici qui a envie de dire qu' on paye pas les femmes autant que les hommes ? &quot; Tout le monde ... Terminé . Alors Monsieur Barbier il a du courage parce qu' il a relancé , non mais écoutez quand on a des gens avec qui on peut discuter sur les idées de fond et pas sur mon allure , ma manière de parler et tout ça , ce qui est injuste et blessant Mais sur le fond , alors lui il dit : &quot; C' est pas possible , pour 2 raisons &quot; écoutez ça les filles ça va vous plaire plus que les hommes . &quot; Première raison c' est un choc insupportable pour l' économie &quot; , ah ah , c' est un aveu , si on exploite pas les femmes on peut pas faire tourner l' économie capitaliste correctement ? Premier aveu . Deuxième aveu , vous allez voir , &quot; oui mais si elles sont mieux payées , quand elles vont être à la retraite , elles vont avoir des retraites plus élevées et c' est ruineux &quot; . Voilà , voilà 2 raisons pour lesquelles il est normal de continuer à exploiter les femmes pour qu' elles coûtent pas trop cher à la production et quand elles sont à la retraite , qu' elles nous coûtent pas trop cher en retraire , parce que , parce que c' est comme ça et voilà ! parce qu' on a toujours fait comme ça et on voit pas pourquoi on continuerait pas , on paye moins les femmes , un point c' est tout Monsieur Mélenchon ça suffit . Voilà , là-dessus arrive Madame Le Pen . Alors , j' ai bien réfléchi , on a tout essayé mais je crois que ce qui les embête le plus ça va vous choquer un peu . Ce qui les embête le plus c' est qu' on leur dise qu' ils nous copient . Ça ça les fait rager , parce qu' ils se battent après entre eux , ils sont tous de droite là dedans . Donc quand on leur dit &quot; Vous dites comme Mélenchon &quot; alors ça les met mal et alors à moi ils me disent &quot; Vous dites comme Madame Le Pen , la retraite à 60 ans &quot; normalement si je réagis comme il faudrait je dirais &quot; Moi je n' ai rien à voir avec cette femme , je sais même pas ce qu' elle dit &quot; . Vous voulez la retraite à 60 ans madame Le Pen ? Pourquoi pas , c' est ce qu' on réclame , moi j' ais ça c' était dans le programme commun , il y en a ici qui ont connu le programme commun ? Alors on allait toquer , &quot; toc toc toc &quot; , on allait aux gens expliquait la retraite à 60 ans , alors les plus anciens ils disaient &quot; C' est bien , vous êtes jeune , vous y croyez , ça sera pas pour nous &quot; il faut vous rendre compte que c' était la génération de la Libération et que leur parent à eux sont morts au travail , si je fais l' ordre des générations , mon arrière grand père il est mort à 75 ans , une semaine après avoir arrêté le travail parce qu' on lui a dit &quot; tu peux rester chez toi Barnabé tu n' as plus besoin d' être là &quot; voilà , il est mort après , terminé . Voilà , ça se passait comme ça avant . Donc la génération suivante , elle était bien contente de ce qu' elle avait eu à la libération et quand on leur disait &quot; la retraite à 60 ans &quot; ils disaient comme vous quand je vous raconte 100 % remboursable , vous vous dites &quot; Il est sympa , il y croit , c' est bien , bon ça sera pas pour nous mais bon ça se fera &quot; ben les gens vous me croyez pas , on l' a fait ! D'accord ? Donc c' est pour dire , moi je ne suis dérangé par personne qui dit la retraite à 60 ans avec 40 annuités , je suis d'accord ? Je suis d'accord . Si Madame Le Pen veut faire la retraite à 60 ans avec 40 annuités , elle peut prendre aussi sa carte à la CGT , et alors , ils en veulent pas ! Le problème c' est qu' ils en veulent pas ! Non , enfin , il y a des limites quoi . Ils en veulent pas ... Mais si elle veut , elle a le droit . On va lui donner une adresse , ah non il y en a pas là où elle est , il n' y a pas de CGT . Comment ils l' appellent leur château là , il y a pas de CGT dans le coin . Si ça se trouve les employés qui travaillent chez elle sont à la CGT . Donc ... Moi on me dit : &quot; Vous dites que madame Le Pen et que je me défend plus &quot; sur la retraite à 60 ans et 40 annuités , ils lui ont fait le même numéro à elle , alors là , 3 sauts périlleux arrière parce que ils veulent bien tout ce que l' on veut , mais ça , ceux qui poussent Madame Le Pen dans le dos , ils ont pas inventés tous bastringues pour qu' elle finisse par raconter les mêmes choses que nous . Alors elle a dit : &quot; Je veux être très claire &quot; , en général quand ça commence comme ça , c' est rarement vrai . &quot; la première étape c' est de récréer de l' emploi &quot; En mettant les immigrés dehors et en mettant les autres à la place . &quot; La deuxième étape c' est de stopper et récupérer les dépenses exorbitantes de l' immigration &quot; &quot; Si malgré cela &quot; une fois qu' on a fait les poches au travailleur immigré , une fois que , une fois que , &quot; Il apparaît qu' on ne peut pas maintenir notre système de retraite &quot; , eh , &quot; Maintenir le système de retraite &quot; , &quot; et accorder un départ à 60 ans &quot; &quot; qui a toujours été subordonné dans notre esprit à la condition sine qua none d' avoir 40 ans de cotisations , alors nous nous tournerons vers les français en leur disant la vérité &quot; Tout ça tourner la page , voilà la vérité une fois que rien de tout cela n' aura été fait parce que c' est pas faisable . &quot; Nous avons fait toutes les économies que nous devions faire mais nous avons un système de retraite qui est en danger et fragile , je crois alors que l' État se sera montré exemplaire et que les français &quot; dit Madame Le Pen &quot; accepteront les sacrifices qu' on leur demandera &quot; D'accord ? Vous voilà revenu à la case départ . Elle a commencé par vous dire qu' elle ferait surement la retraite à 60 ans mais qu' à la fin vous vous serrerez la ceinture d' un cran de plus . Point . D'ailleurs , la dame qui s' occupe de ça chez eux , d'abord le communiqué du FN &quot; Dans notre programme il y a pas de retour de la retraite à 60 ans &quot; ça a chauffé entre eux , &quot; et puis quoi à la fin tu vas faire comme Mélenchon , qu' est -ce que c' est que cette histoire ? &quot; &quot; Mais une retraite pleine lorsqu' on aura cotisé 40 ans &quot; dit le Front National , très bien , allez à la CGT . Mais Madame Joëlle Melin , responsable du programme du FN , elle dit &quot; Ah &quot; &quot; Nous aurons du mal &quot; dit -elle , &quot; a rester sur les 40 ans de cotisation &quot; eh ben voilà , c' est qui les ballots ? C' est tous ceux qui les auront cru . &quot; Il faut tenir &quot; vous allez voir l' argument , il est d' une nouveauté , waw , &quot; Nous aurons du mal à rester sur les 40 ans de cotisations car il faut tenir compte &quot; de quoi ? De l' allongement de la vie ! Et voilà ! Le FN et les autres c' est la même chose . Ils sont sur la même ligne politique et quand vous entendrez dire que le FN est pour la retraite à 60 ans , vous rappellerez ce que je viens de vous dire ce soir , je vous avais prévenu ça prend 10 minutes . C' est plus facile de colporter un mensonge que de rétablir une vérité . On le vérifie une fois de plus . Bon , je termine sur un point de tout ça , parce que ça commence être long hein je fatigue , c' est la dépendance , mes ami(e)s , parce que ça ... On va pas pouvoir passer et continuer comme on est là , ça va pas , les gens sont trop maltraités , et il faut que , je vais vous lire la phrase qui est dans notre programme parce que c' est le plus simple , comme ça dure pas 2 heures . Il faut faire face collectivement au vieillissement de la population à la perte de l' autonomie . Il faut construire une prise en charge solidaire de la dépense , et les camarades ont précisé , mes ami(e)s les économistes , financé par une mise à contribution des revenus immobiliers et des successions sur les gros patrimoines . Fussent d'ailleurs des gros patrimoines des personnes en question . Parce que vous savez que des fois , pour garantir des successions , on finit par refuser de payer ce qu' est quand même du à la personne qu' en a besoin . Voilà ce que je dis moi . C' est un gros dossier , Fillon lui dit : &quot; Encourager la généralisation d' une couverture assurantielle dépendance privé sur une base volontaire &quot; autrement dit , vous vous payez votre truc , je vois pas que ça change avec la situation d' aujourd'hui . Quiconque a eu à faire face à ce genre de situation sait que c' est un coût &quot; qui permettront au particulier de déduire de leurs impôts &quot; , tu parles , les malheureux , &quot; une part de leur cotisation à une assurance dépendance &quot; , moi je suis contre ça . Une nouvelle fois on revient sur celui qui peut s' assurer au bon niveau , il a le droit a une prestation , celui qui peut pas ben il reste là où il est dans la situation dans laquelle il est . On peut pas être d'accord avec ça . Mais je vais vous dire une autre chose , parce que ça vous comprenez , c' est pas la peine que j' en parler 2 heures . Je voudrais vous mettre en garde contre une autre idée , parce que j' ai vu ça et ça m' a pas plu . Il y a des gens qui disent &quot; Eh ben voilà , alors , pour aider les personnes qui en ont besoin il faut faire de la traçabilité lumineuse dans les appartement et puis il faut mettre une alarme , et puis il faut mettre une vidéo surveillance pour les aider &quot; Et au bout d' un moment j' écoutais ça , je me disais &quot; qu' est -ce qu' il raconte ? &quot; Quand on est dépendant , on a pas envie de parler avec une caméra ni voir le parterre s' illuminait entre votre lit et les toilettes Ni avoir un machin qui se met en route tout seul . C' est pas ça le sujet . Qui conçoit , on a besoin des autres . Le premier des soins c' est celui qui donne les soins , ou celle qui le donne , et donc , moi je vous met en garde mes ami(e)s d' une façon générale contre l' illusion technocratique . Non , la vie des êtres humains ça sera une vie d' être humain et pendant qu' on y est , je vais vous dire une deuxième chose , figurez -vous que les gens qui aujourd'hui s' occupent des personnes dépendantes , ça peut être pour une dépendance provisoire , c' est pas tout le temps , le personnel qui fait ça , c' est à dire les gens qui viennent à domicile , ils ont souvent des contrats de travail , mais surtout ... c' est un personnel qui est souvent féminin , eh ben figurez -vous que quand on regarde les accidents du travail , il y a une espèce de course avec des moments qui changent , tantôt c' est le bâtiment qui arrive en 1 pour les accidents du travail , tantôt c' est qui ? Ce personnel là , et 80 à 90 % des problèmes physiques que rencontrent les gens qui font ce travail , est lié à la manutention du corps des personnes qui sont dépendantes . Par conséquent , nous avons , non seulement à régler le problème de la façon avec laquelle on prend en charge , au plan financier , mais nous avons aussi à réfléchir et à trouver les moyens et à nous dire qu' on ne fera pas d' économie là-dessus . Ça existe pas faire de l' économie sur l' attention qu' on doit aux autres . Parce que l' attention qu' on aura pas eu pour la personne concernée , on l' aura pas eu pour la personne qui s' en occupe et donc ça nous fera 2 malheureux au lieu d' 1 . Par conséquent , il y a besoin que les conditions de travail de ces personnels soient pris au sérieux , ces gens ne sont pas seulement là , comme se le figure certain , pour manutentionner . Il y a toute une disponibilité qu' ils doivent avoir et pour ça changer leurs conditions de travail . Il faut que ... J' en aurais des choses à dire mais il faut que j' aille à ma conclusion . * Une personne dans le public parle à Jean-Luc JLM : Attends c' est la Sécu ce soir , la Sécu intégrale , le logement ça sera une autre fois , d'accord ... Je fais ce que je peux ... Je vous met en garde contre l' illusion technocratique , je vous l' ai dis il y a un instant . Je vous dois encore quelques mots . Vous savez ce que c' est que la DMS ? Vous savez pas ? La Durée Moyenne de Séjour , c' est de ça dont je vous parle maintenant . Comment un système devient fou , cruel et anti-humain ? La Durée Moyenne de Séjour à l' hôpital . Vous là , fémur cassé ? 20 jours . Le 18ème jour le personnel s' affole , &quot; Mais Monsieur Machin &quot; il a le col du fémur pas en état , il peut pas ... faut qu' on le garde , c' est pas possible ! Durée Moyenne de Séjour 20 jours , 20 jours arrive , dehors . Le personnel de santé qu' est là &quot; Qu' est -ce qu' on fait ? C' est inhumain , on va pas faire ça , donc il faut qu' on le mette dans un centre de repos ou de soin &quot; on téléphone , il n' y a pas de place bien évidemment , puis après on dit quelqu'un dit &quot; bon d'accord , on va faire un effort mais il est autonome ? &quot; Oui oui &quot; c' est pas vrai , et j' ai une amie vous pouvez la regarder , elle fait des petites vidéos qui sont géniales , s' appelle Sabrina Aurora et elle raconte comment le personnel de santé se raconte des histoires les uns aux autres pour que les gens se passent les gens , parce que ce personnel de santé il est de bonne volonté , il veut pas laisser quelqu'un dans le malheur , parce que vous l' emmenez et puis quoi , ça s' appelle le &quot; virage ambulatoire &quot; , ces technocrates ont toujours toutes sortes de mots , le virage ambulatoire c' est &quot; vous sortez de l' hosto , vous restez chez vous &quot; c' est sympa chez moi , sauf si c' est au 6ème étage et que j' ai une patte cassée et qu' il y a pas d' ascenseur . Là c' est plus sympa du tout parce que ça veut dire que je vais être seul et misérable . Et ainsi de suite . Voilà ce que c' est et puis la tarification à l' acte , vous en avez entendu parler ? Le piège total , vous arrivez vous êtes une chose . &quot; Vous avez quoi vous ? Telle maladie , pas telle autre je m' en occupe pas , c' est pas mon sujet &quot; poum poum poum , il y a ça qui est prévu . Si vous en faites moins , tire-au-flanc ! Si vous en faites trop il y a de l' abus , vous en faites moins plouf ! On vous supprime des lits . Il y a donc toute une organisation de la pensée qui est liée au soin , au rapport à la personne , à partir d' une vision technocratique et comptable qui défigure la société et rend malade tout le monde y compris ceux qui s' occupent des malades . Et au fond , c' est ça notre sujet , quand on fait une réunion comme celle -ci , c' est à ça qu' on doit réfléchir . C' est la société qu' on veut . Où c' est sauvé mon dernier papier ? Bon , tant pis , va falloir improviser ... Non pas tout à fait ... On va bien vous saouler dans les temps qui vont venir , vous allez voir les têtes d' œuf qui vont revenir . &quot; Les dépenses obligatoires sont trop élevées &quot; Bon , &quot; nous allons le comparer à la Finlande , l' Islande &quot; enfin ils sont 1/10ème moins nombreux que nous &quot; mais c' est pas grave c' est pour donner une idée &quot; OK . Bref . Tout ces gens vous parlent de prélèvements obligatoires et ils comparent des choses qu' ils n' ont rien à voir les unes avec les autres . Je m' en tiens seulement à la Sécu . Chaque fois que vous entendrez dire que les prélèvements obligatoires sont plus importants chez nous qu' ailleurs , il faut que vous sachiez qu' ailleurs n' a pas la même organisation que nous et que chez nous 40 % de ce qui s' appelle &quot; les prélèvements obligatoires &quot; sont liés à des dépenses sociales de santé et de ce qui va avec . Tandis que dans tous les autres pays , la moyenne c' est 26 % . Autrement dit ils ne comptent pas les dépenses obligatoires assuré par le privé parce que si vous vous cassez une patte , que vous soyez chez nous ou ailleurs , vous vous faites soigné mais si vous vous faites soigné ici , c' est compté dans les dépenses obligatoires tandis que si vous vous faites soigné là-bas , c' est compté dans votre fantaisie et ça rentre pas dans les comptes publics . Résultat , ça vous donne un niveau de prélèvement qui est supérieur ou égal aux États-Unis , à ce que nous avons nous , pour des dépenses de santé qui reste à charge de beaucoup plus considérables , le double ou le triple de ce qui se passe chez nous . Ne vous laissez pas entortiller par les chiffres , ce n' est pas vrai que nous dépensons trop , que la France vit au-dessus de ses moyens . Il y a un sujet sur lequel en effet la France vit au-dessus de ses moyens , c' est que ça classe dominante sociale coûte abominablement cher et que nous pourrions trouver des gens qui nous coûtent beaucoup moins cher pour faire la même chose ! Oui , nous vivons au-dessus de nos moyens avec une oligarchie pareil recors d' Europe de distribution des dividendes alors qu' on a besoin d' investissement massif dans le pays . J' attend qu' on vienne me démontrer le contraire , j' attend qu' au lieu de venir parler de mon style oratoire , de ma cravate , de mon âge , de mes dents ou que sais -je , on vienne me montrer que j' ai tord , que ce n' est pas vrai . Parce que c' est vrai ! Parce que c' est une oligarchie toute puissante qui est un parasite sur ce système et quand je dis qu' il ne doit pas y avoir d' écart de salaire supérieur de 1 à 20 c' est à dire que quand Monsieur Gohsn veut s' enrichir sans rien faire de 6 millions d' euros supplémentaires , eh ben il va falloir qu' il en donne autant à tous ceux qui sont en bas de la hiérarchie parce que c' est ça un écart de 1 à 20 et 1 à 20 c' est beaucoup ! C' est beaucoup beaucoup ! Cette revendication , c' est à nous de la prendre , c' est celle de la Confédération Européenne des Syndicats . La société française , comme toutes les autres , comme toute la civilisation humaine , au delà de tout ce que l' on vient de se raconter , au fond quoi , 2 visions de la vie , pour les uns ... c' est pas des brutes , on est pas non plus dans une ambiance de guerre civile , ceux qui sont pas , qu' on pas le cœur comme nous , qui voit pas les choses comme nous , c' est pas non plus des sauvages , eux aussi ont des familles . Quelle est leur idée ? Leur idée c' est que dans la vie , ce qu' est le moteur de la société c' est la compétition , voilà c' est ça leur idée fondamentale . Alors ça fait pas d' eux des brutes , si pour beaucoup oui , mais bon , pas tous , et ils disent &quot; la compétition , nous on veut bien la solidarité , c' est pas un problème pour nous . On est d'accord pour être solidaire , on est bien élevé &quot; &quot; Mais pas si ça met en cause la compétition &quot; donc la solidarité , oui , jusqu' au point où commence la compétition . Ça c' est la vision des libéraux . Toute la bande là , Macron , Fillon , tout ça . Ils voient la vie comme ça . Et puis nous , on vient de plus loin , de beaucoup plus loin , nous , on se dit &quot; boupf , on est là , on est les enfants desquels ? &quot; si vous pensez à l' histoire profonde de l' humanité . Les enfants desquels ? De celui ou de celle qui s' occupait de son voisin , de celui ou de celle qui s' occupait du plus faible , du blessé qui le soignait , de la femme qui allait mettre l' enfant au monde qu' on protégeait pour pas qu' elle fasse l' objet d' une agression à ce moment là , les petits dont on s' occupe , ceux là ils ont survécus , les autres qu' en avaient rien à foutre , qui s' occupaient de personne , que d' eux mêmes , bah ils ont disparu plus vite que les autres . Et eux , les enfants , il y en a plus . Enfin , on pouvait penser ... Nous , nous sommes les descendants des gentils , des doux , des solidaires , des préoccupés des autres , qui bercent , qui chantent , qui dansent pour mettre une ambiance , pour réconforter , pour rassurer le groupe humain . J' exagère ? Oui , un peu , pas trop . C' est ça le fond qui nous sépare , et maintenant , dans la vie de la vie , ça peut être pas grave , quand tout allait bien , pour tout le monde . Mais maintenant , ça va mal pour un nombre de gens croissant parce que le système est devenu fou , eux-mêmes ne le contrôle plus , plus personne ne le contrôle . L' énorme accumulation du capital qui se fait dans la sphère financière , plus personne ne le contrôle . La folie de la cupidité de la puissance dominante , plus personne ne la contrôle vous avez vu cette histoire avec Trump ? C' est un truc entre eux là , c' est très dangereux , ils sont en train de se battre entre eux , les services de sécurité et le reste , les uns veulent faire la guerre à la Russie , les autres à la Chine , c' est des fous ! Et ils sont en train d' essayer , j' sais pas faire quoi , à leur prochain président , c' est très dangereux tout ça . Ça c' est la guerre , mais c' est pas de ça sur quoi je veux terminer . Nous tous , les êtres humains ... Quelque soit notre religion , notre couleur de peau , notre âge , ce que nous avons prévu , nous sommes confronté au changement climatique . Mes ami(e)s , il est commencé , ne vous demandez pas si on va pouvoir l' arrêter , ce n' est pas possible . Ce n' est plus possible . Les engagements de la COP 21 , déjà qui ne seront pas tenus , prévoient 2 degrés de plus , ils ont mis un degré d' augmentation pour faire plaisir aux îles qui pouvaient pas signer parce que si on mettait pas 1 et demi , ça voulait dire qu' ils signaient qu' 'ils étaient d'accord pour être noyé . On a mit 1 et demi , mais quand on fait le total de ce que chaque État a prévu de faire , ça fait 3 degré et demi d' augmentation de la température . C' est intenable . Le climat va donc changer , les courants froids et chauds dans les mers changent . En décembre dernier , la société de ... géologie , je crois , internationale , a donné un nouveau nom à la période dans laquelle nous vivons , vous vous rappelez peut-être de l' école , le pléistocène , le tertiaire , le quaternaire etc . Bon d'accord ? Ben l' ère dans laquelle nous sommes , le moment dans lequel nous sommes , ils l' appellent &quot; L' anthropocène &quot; , c' est à dire la période où l' être humain change la nature . L' être humain , cette petite créature , sans armes , sans griffes , l' être humain charrie plus de gravats que toutes les forces de la nature . Les fleuves , les rivières , le vent et ainsi de suite . Les traces sont incontestables , on les verra pendant 10 000 ans , 20 000 ans , 50 00 ans , 1 millions d' années il y aura encore la trace de ce que nous avons fait . De choses qui n' existaient pas et qui existent depuis le plastique , des choses qui n' existaient pas et qui existent depuis le combustible nucléaire et les poussières nucléaires qui se sont répandues sur la totalité de la surface de la Terre . Maintenant vous savez que tout le destin de l' humanité est dans les mains de l' humanité , ce à quoi nous devons nous préparer , nous , c' est à affronter cette situation . Comment allons -nous faire ? Chacun pour soi ? Ou tous ensemble ? C' est la vieille leçon , la première leçon que devait tirer les premiers groupes d' êtres humains , pour survivre . Qu' est -ce qui compte ? Moi d'abord ? Ou tous les autres avec moi ? Cette grande question qui partage les tenants de la compétition tout le temps et les tenants de la solidarité revient devant la scène . Vous ne pouvez pas être réellement écologiste et vous l' êtes tous . C' est pas une carte de parti écologiste , c' est une conscience . Nous ne sommes pas , si nous ne tranchons pas cette question là , il n' y a pas de capitalisme vert . Le problème du capitalisme vert c' est la couleur . C' est que le capitalisme ne peut pas être vert . Ils ne peuvent pas faire l' accumulation s' ils refusent l' investissement pour changer les machines , pour changer la production , pour changer le sort des travailleurs qui travaillent derrière ces machines . C' est ce défi qu' on va devoir relever . Alors moi je vous dis , si nous la France nous changeons de cap , si juste nous faisons ça , une constituante et ensuite la planification écologique . Ce que nous allons faire , nous les français , parce que nous sommes une grande puissance technicienne , je suis pas en train de faire du nationalisme , c' est pas ça le sujet , nous sommes une grande puissance technique , et vous savez pourquoi ? Parce que nous avons eu une bonne école républicaine pour tous , parce que nous avons enseigné à tout le monde , y compris les gosses d' étrangers . Parce qu' on a fait ce choix là , parce qu' on a bien traité les gens , mieux qu' ailleurs , on a pu accumulé ce qui compte , la plus grande et meilleure des richesses , le savoir , l' intelligence . C' est une richesse qui , plus vous la partager , plus il y en a , au contraire de toutes les autres richesses . Le savoir , qui va nous aider à répondre et parce que nous sommes cette nation là , nous avons une responsabilité à l' égard du reste de l' humanité . En pensant à ces milliards d' êtres humains qui n' ont pas accès à l' eau potable , qui n' ont pas accès à l' énergie , nous pouvons , nous , parce que nous connaissons les process de production pour le faire , mettre en partage la science et la technique qui permettra à tout ces peuples d' avoir ce dont ils ont besoins , en rompant avec l' ancien modèle d' accumulation , parce que s' ils font tous comme nous , il y a plus de vie possible sur la Terre . Et donc pour que la vie continue , il faudrait que mécaniquement on continue à les exploiter , qu' on continue à les rabaisser , parce que s' ils se mettent au même niveau que nous pour avoir le même nombre de voiture par habitant , le même nombre de produit qui servent à rien , la planète va mourir . Voilà pourquoi il y a un lien entre l' idée de sauver la civilisation humaine , le paradigme écologique , l' idée d' avoir cet objectif et la République qui consiste à dire &quot; nous faisons ce qui est bon pour tous &quot; . Comme nous sommes cette nation , nous pouvons rompre le cercle infernal dans lequel nous sommes , vous avez vu quel président ils ont élus aux États-Unis d' Amérique ? Il croit que le réchauffement climatique est une invention des chinois . Et ainsi de suite , tout ces gens ne s' occupent de rien , les glaciers sont en train de fondre , vous savez de quoi ils s' occupent ? De savoir à qui appartient le sol qu' il y a en dessous . Voilà pourquoi les uns et les autres , les russes , les américains , les danois , tout le monde avancent les navires de guerre , pour dire &quot; ça c' est à moi &quot; Autrement dit toutes les conditions se réunissent , d' une bataille du type du XIXe siècle pour l' accès aux matières premières et aux territoires alors qu' on est au XXIe siècle et devant un péril majeur . C' est pour quoi , nous serons cette puissance de paix et parce que nous ferons ça , je vous dis , parce que nous ferons ça , ce sera un déclic . Un événement , tel qu' on peut dire le siècle comment sera en 2017 , parce qu' en 2017 , une des principales nations du monde aura rompu avec l' ancien système et se sera orienté vers les nouveaux horizons nécessaires à l' humanité toute entière . Exactement ! Voilà à quoi je voudrais passionnément vous servir , je suis votre porte parole , je le vois puisque vous applaudissez ce que je viens de dire . Vous savez comme moi que c' est le plus urgent . C' est à ça que vous voulez qu' on travaille . Vous ne voulez pas perdre votre temps à des querelles secondaires pour éliminer ou choisir entre des individus qui racontent la même chose et qui de toutes façons sont incapables d' imaginer un autre monde . Vous savez les périodes révolutionnaires , moi je plaide pour la révolution citoyenne , elles sont caractérisées par ça , les élites ne croient pas qu' un autre monde soit possible . Ils n' arrivent même pas à l' envisager . Ce ballot de Louis XVI n' arrivait même pas à s' imaginer ce que ça pouvait être un gouvernement du peuple . Ça lui venait pas à l' esprit . Il n' avait jamais connu une chose pareille . Il croyait qu' il y avait des rois , des évêques , et des machins ? tout ça pour maintenir le système en place jusqu' à que ce que tout explose . Ne comptez pas sur leur prise de conscience . Ne croyez pas qu' ils vont comprendre un jour ou l' autre , ils ne comprendront jamais ! Ils ne peuvent pas ! Leurs intérêts , la logique dans laquelle ils sont leur interdit de comprendre qu' il faut passer à autre chose . Que les biens communs sont plus important que les biens particuliers , que l' intérêt général humain est plus important que les intérêts particuliers . Ça il y a que nous qui pouvons le faire , c' est pourquoi je vous dis on commencerait le siècle de cette manière . J' allais vous dire , en pensant à d' autres événements qui se sont passés dans le passé , que quand un engrenage mortel se met en route , rappelez -vous ce qui s' est passé avec la première Guerre Mondiale , seule une intervention populaire rompant avec tout a permis que ça s' arrête parce qu' ils ont eu tellement peur de voir les soldats qui voulaient plus monter et sortir de la tranchée et tirer sur les autres parce qu' ils ont eu tellement peur quand ils ont commencé à voir qu' il fallait les fusiller pour les obliger à obéir parce qu' ils ont eu tellement peur en voyant que les russes étaient soustrait à ce système abominable , qu' ils ont été obligé d' arrêter , quelqu'un a fini par capituler . Eh bien nous , il faut qu' on leur fasse tellement peur en prenant tranquillement , par la démocratie , par les urnes , une décision politique majeure . Nous allons changer le monde et nous allons le faire en changeant notre pays , voilà par quel bout on prend le problème . Et c' est pas trop pour nous , c' est pas exagéré parce que le jour où nous le ferons , vous verrez que des milliards d' êtres humains se tourneront aussi vers nous et regarderont comment nous sommes entrain de faire parce que eux aussi ils veulent pas qu' on dégueulasse toute la planète Eux aussi ils veulent que leurs enfants aient droit à la beauté d' un matin qui se lève , d' une eau qui coule dans la rivière , d' une écrevisse qui s' y trouve , parce que toute vie n' a pas été détruite ! Voilà , bon pour finir , parce que vraiment il faut finir ... Je vous ai parlé de Sécurité Sociale , je vous ai présenté un modèle de la Sécurité Sociale intégrale , vous avez le droit de vous poser des questions , d' en poser , la plateforme est ouverte pour ça , vous serez les bienvenus . Nos ami(e)s travaillent beaucoup , parce que vous savez on exige toujours plus de nous que de n' importe qui d' autre hein , mais le fond va pas le perdre . On parle de Sécu . Et ses petits droits de la vie . Alors , j' ai parlé de solidarité et Jaurès apporte une touche de plus , je le dis avec ses mots , parce que moi j' ai assez dit des choses , des grandes choses auxquelles je croyais et je crois ? écoutez ça : &quot; Entre l' assurance et l' assistance , même littéralement organisé , il y a un abîme . l' assisté , même quand la loi lui donne ce qu' il appelle un &quot; droit &quot; est obligé de plaider pour avoir la réalisation de ce droit &quot; ça se passe en 1910 , j' ai choisi ce discours je sais pourquoi , 1910 c' est les premières retraites ouvrières à 65 ans , alors Jaurès dit &quot; les plus durs dans le mouvement de l' époque disait &quot; ah va pas signer un truc pareil 65 ans , on va cotiser pour les morts &quot; on disait à l' époque et Jaurès dit &quot; non non faut qu' on accepte ça , on l' améliorera &quot; c' était la retraite à 65 ans , vous entendez les gens ? Ce que vous propose Monsieur Fillon c' est de revenir ce qu' on avait à l' époque du Grand Jaurès , bon , 1910 &quot; Il faut que le vieillard de 70 ans démontre qu' il est indigent et son indigence est évaluée , arbitrairement par les pouvoirs et puis il faut que l' individu quémande et dans son attitude d' assisté , il sent encore pesé sur ses épaules , courbés par le travail , le poids de la servitude sociale &quot; &quot; Au contraire , l' assuré a un plein droit . Un droit absolu , un droit inconditionnel , son titre est là aussi certain que l' est pour les bourgeois le titre de rente . A l' âge fixé par la loi , il a le droit d' exiger , il exige , il reçoit un certain chiffre et il ne reçoit pas seulement un recours matériel &quot; Il reçoit un secours de dignité morale . Magnifique idée d' assurance sociale qui crée pour tous les salariés un droit tangible sans humiliation &quot; . Merci ! "
]

The whole corpus consists of 33 speeches, and below it is shown the distribution of them.

In [133]:
print(len(corpus))

33


In [134]:
print(len(melenchon))
print(len(melenchon_test))
print(len(macron))
print(len(macron_test))
print(len(lepen))
print(len(lepen_test))
print(len(jadot))
print(len(hamon))
print(len(fillon))
print(len(fillon_test))

9
2
6
2
4
1
1
1
6
1


# Sentence to wordlist

speech_to_wordlist uses TreeTagger to tag the words, to remove stop words from sentences and to keep those words that are longer than 2 words.

In [135]:
import treetaggerwrapper
fr_tagger = TreeTagger(TAGLANG='fr')
def speech_to_wordlist( speech, remove_stopwords=True ):
    #
    # 1. Remove HTML
    speech_text = BeautifulSoup(speech,"lxml").get_text()
    #
    # 2. Split and tag
    tag_text_uncleaned = fr_tagger.tag_text(speech_text)
    tag_text = treetaggerwrapper.make_tags(tag_text_uncleaned)
    tagged_words = [word[2] if word[2] != '<unknown>' else word[0] for word in tag_text if len(word) ==3]
    tagged_words2 = [word for word in tagged_words if len(word)>2]
    #
    # 3. Optionally remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("french"))
        words = [w for w in tagged_words if not w in stops and len(w)>2]
    else:
        words = tagged_words2
    #
    # 4. Return a list of words
    return(words)

# Speech to sentences

speech_to_sentence converts a speech into a list of sentences where each of those lists is a lists of words where stop words and words shorter than 2 characters have been removed.

In [136]:
def speech_to_sentences( review, tokenizer):
    # Function to split a speech into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call speech_to_wordlist to get a list of words
            sentences.append(speech_to_wordlist( raw_sentence))
        
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

# Corpus to list of sentences

The corpus is transformed into a list of sentences, and then the function speech_to_wordlist is called to transform each sentence into a list of words.

In [145]:
tokenizer = nltk.data.load('tokenizers/punkt/french.pickle')
corpus_tok_sentences_list=[]
for speech in corpus:
    corpus_tok_sentences_list+=speech_to_sentences(speech, tokenizer)

C:\Users\LuisGaldo\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [146]:
print(len(corpus_tok_sentences_list))

9710


# Corpus to frequence representation

The intention of this section is to create a dictionary of the corpus and assign each word a different id (number), in order to do so, they are ordered by frequency and numbers are assigned in order.

In [147]:
#Dictionary with words and their number of appearences
uniqueWords = {}
for sentence in corpus_tok_sentences_list:
    for word in sentence:
        if(word not in uniqueWords):
            uniqueWords[word] = 1
        else:
            uniqueWords[word] += 1
sizeUniqueWords = len(uniqueWords)

In [148]:
#Dictionary with the frequencies of the words
freqWords = {}
for sentence in corpus_tok_sentences_list:
    for word in sentence:
        freqWords[word] = (float)(uniqueWords[word])/sizeUniqueWords

In [149]:
#Dictionary ordered by the frequencies of the words
import operator
sorted_freqWords = sorted(freqWords.items(), key=operator.itemgetter(1),reverse=True)

In [150]:
#Dictionary with words and their ids
vocabulary_inv = {}
vocabulary_inv[0] = "<PAD/>" 
count = 1
for word in sorted_freqWords:
    vocabulary_inv[count] = word[0]
    count += 1
print(len(vocabulary_inv))      

7510


In [151]:
for x in range(len(vocabulary_inv)):
    for y in range(len(corpus_tok_sentences_list)):
        for z in range(len(corpus_tok_sentences_list[y])):
            if(vocabulary_inv[x] == corpus_tok_sentences_list[y][z]):
                corpus_tok_sentences_list[y][z] = x

# Speeches of candidates to list

Now, the same procedure as before is applied. Speeches of candidates are converted into lists of sentences, and these ones are then converted into lists of words. So later these words can be sustituted by their corresponding IDs.

In [160]:
tokenizer = nltk.data.load('tokenizers/punkt/french.pickle')
fillon_tok_sentences_list=[]
for speech in fillon:
    fillon_tok_sentences_list+=speech_to_sentences(speech, tokenizer)

In [161]:
tokenizer = nltk.data.load('tokenizers/punkt/french.pickle')
fillon_test_tok_sentences_list=[]
for speech in fillon_test:
    fillon_test_tok_sentences_list+=speech_to_sentences(speech, tokenizer)

In [162]:
tokenizer = nltk.data.load('tokenizers/punkt/french.pickle')
hamon_tok_sentences_list=[]
for speech in hamon:
    hamon_tok_sentences_list+=speech_to_sentences(speech, tokenizer)

In [163]:
tokenizer = nltk.data.load('tokenizers/punkt/french.pickle')
jadot_tok_sentences_list=[]
for speech in jadot:
    jadot_tok_sentences_list+=speech_to_sentences(speech, tokenizer)

In [164]:
tokenizer = nltk.data.load('tokenizers/punkt/french.pickle')
lepen_tok_sentences_list=[]
for speech in lepen:
    lepen_tok_sentences_list+=speech_to_sentences(speech, tokenizer)

In [165]:
tokenizer = nltk.data.load('tokenizers/punkt/french.pickle')
lepen_test_tok_sentences_list=[]
for speech in lepen_test:
    lepen_test_tok_sentences_list+=speech_to_sentences(speech, tokenizer)

In [166]:
tokenizer = nltk.data.load('tokenizers/punkt/french.pickle')
macron_tok_sentences_list=[]
for speech in macron:
    macron_tok_sentences_list+=speech_to_sentences(speech, tokenizer)

In [167]:
tokenizer = nltk.data.load('tokenizers/punkt/french.pickle')
macron_test_tok_sentences_list=[]
for speech in macron_test:
    macron_test_tok_sentences_list+=speech_to_sentences(speech, tokenizer)

C:\Users\LuisGaldo\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [168]:
tokenizer = nltk.data.load('tokenizers/punkt/french.pickle')
melenchon_tok_sentences_list=[]
for speech in melenchon:
    melenchon_tok_sentences_list+=speech_to_sentences(speech, tokenizer)

C:\Users\LuisGaldo\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [169]:
tokenizer = nltk.data.load('tokenizers/punkt/french.pickle')
melenchon_test_tok_sentences_list=[]
for speech in melenchon_test:
    melenchon_test_tok_sentences_list+=speech_to_sentences(speech, tokenizer)

C:\Users\LuisGaldo\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


# Substituting words per ID of vocabulary_inv

Fillon

In [170]:
for x in range(len(vocabulary_inv)):
    for y in range(len(fillon_tok_sentences_list)):
        for z in range(len(fillon_tok_sentences_list[y])):
            if(vocabulary_inv[x] == fillon_tok_sentences_list[y][z]):
                fillon_tok_sentences_list[y][z] = x 

In [171]:
for x in range(len(vocabulary_inv)):
    for y in range(len(fillon_test_tok_sentences_list)):
        for z in range(len(fillon_test_tok_sentences_list[y])):
            if(vocabulary_inv[x] == fillon_test_tok_sentences_list[y][z]):
                fillon_test_tok_sentences_list[y][z] = x   

Hamon

In [172]:
for x in range(len(vocabulary_inv)):
    for y in range(len(hamon_tok_sentences_list)):
        for z in range(len(hamon_tok_sentences_list[y])):
            if(vocabulary_inv[x] == hamon_tok_sentences_list[y][z]):
                hamon_tok_sentences_list[y][z] = x

Jadot

In [173]:
for x in range(len(vocabulary_inv)):
    for y in range(len(jadot_tok_sentences_list)):
        for z in range(len(jadot_tok_sentences_list[y])):
            if(vocabulary_inv[x] == jadot_tok_sentences_list[y][z]):
                jadot_tok_sentences_list[y][z] = x

Lepen

In [174]:
for x in range(len(vocabulary_inv)):
    for y in range(len(lepen_tok_sentences_list)):
        for z in range(len(lepen_tok_sentences_list[y])):
            if(vocabulary_inv[x] == lepen_tok_sentences_list[y][z]):
                lepen_tok_sentences_list[y][z] = x

In [175]:
for x in range(len(vocabulary_inv)):
    for y in range(len(lepen_test_tok_sentences_list)):
        for z in range(len(lepen_test_tok_sentences_list[y])):
            if(vocabulary_inv[x] == lepen_test_tok_sentences_list[y][z]):
                lepen_test_tok_sentences_list[y][z] = x

Macron

In [176]:
for x in range(len(vocabulary_inv)):
    for y in range(len(macron_tok_sentences_list)):
        for z in range(len(macron_tok_sentences_list[y])):
            if(vocabulary_inv[x] == macron_tok_sentences_list[y][z]):
                macron_tok_sentences_list[y][z] = x

In [177]:
for x in range(len(vocabulary_inv)):
    for y in range(len(macron_test_tok_sentences_list)):
        for z in range(len(macron_test_tok_sentences_list[y])):
            if(vocabulary_inv[x] == macron_test_tok_sentences_list[y][z]):
                macron_test_tok_sentences_list[y][z] = x

Melenchon

In [178]:
for x in range(len(vocabulary_inv)):
    for y in range(len(melenchon_tok_sentences_list)):
        for z in range(len(melenchon_tok_sentences_list[y])):
            if(vocabulary_inv[x] == melenchon_tok_sentences_list[y][z]):
                melenchon_tok_sentences_list[y][z] = x

In [179]:
for x in range(len(vocabulary_inv)):
    for y in range(len(melenchon_test_tok_sentences_list)):
        for z in range(len(melenchon_test_tok_sentences_list[y])):
            if(vocabulary_inv[x] == melenchon_test_tok_sentences_list[y][z]):
                melenchon_test_tok_sentences_list[y][z] = x

# Converting list into numpyarrays

Previous lists are converted into numpyarrays so that they can be passed as an input to the Neural Network

In [180]:
corpus_numpy = np.asarray(corpus_tok_sentences_list)

fillon_numpy = np.asarray(fillon_tok_sentences_list)
fillon_test_numpy = np.asarray(fillon_test_tok_sentences_list)

hamon_numpy = np.asarray(hamon_tok_sentences_list)

jadot_numpy = np.asarray(jadot_tok_sentences_list)

lepen_numpy = np.asarray(lepen_tok_sentences_list)
lepen_test_numpy = np.asarray(lepen_test_tok_sentences_list)

macron_numpy = np.asarray(macron_tok_sentences_list)
macron_test_numpy = np.asarray(macron_test_tok_sentences_list)

melenchon_numpy = np.asarray(melenchon_tok_sentences_list)
melenchon_test_numpy = np.asarray(melenchon_test_tok_sentences_list)

# Padding of the sentences to make them 30 words long

The input to the neural network will be all the sentences used for training. Because of the characteristics of the Neural Network, all the sentences must have the same length. Together with the linguistics team it was decided that the best was 30, so in case a sentence did not have 30 words 0s were added, and in case it was longer, it was cut down to that length.

In [181]:
corpus_numpy = sequence.pad_sequences(corpus_numpy, maxlen=30, padding="post", truncating="post")
fillon_numpy = sequence.pad_sequences(fillon_numpy, maxlen=30, padding="post", truncating="post")
fillon_test_numpy = sequence.pad_sequences(fillon_test_numpy, maxlen=30, padding="post", truncating="post")
hamon_numpy = sequence.pad_sequences(hamon_numpy, maxlen=30, padding="post", truncating="post")
jadot_numpy = sequence.pad_sequences(jadot_numpy, maxlen=30, padding="post", truncating="post")
lepen_numpy = sequence.pad_sequences(lepen_numpy, maxlen=30, padding="post", truncating="post")
lepen_test_numpy = sequence.pad_sequences(lepen_test_numpy, maxlen=30, padding="post", truncating="post")
macron_numpy = sequence.pad_sequences(macron_numpy, maxlen=30, padding="post", truncating="post")
macron_test_numpy = sequence.pad_sequences(macron_test_numpy, maxlen=30, padding="post", truncating="post")
melenchon_numpy = sequence.pad_sequences(melenchon_numpy, maxlen=30, padding="post", truncating="post")
melenchon_test_numpy = sequence.pad_sequences(melenchon_test_numpy, maxlen=30, padding="post", truncating="post")

# Calling word2vec

This function receives as input the corpus. word2Vec represents each wordin a vector of 300 dimensions. In order to build this vector, a window size of 10 is taking into account, this means that the 10 words before and after the given word are taken into account at the time of setting up the values of the 300 dimensions vector. The final shape is the length of the vocabulary (7510 words) times 300 (vector representation of each word).

In [182]:
embedding_weights = train_word2vec(corpus_numpy, vocabulary_inv, num_features=300,
                                       min_word_count=1, context=10)

Training Word2Vec model...
Saving Word2Vec model '300features_1minwords_10context'


In [183]:
print(embedding_weights[0].shape)

(7510, 300)


# Concatenating training and test data

The data from different candidates is concatenated to create a training data set and a testing data set. The training one is composed of 6951 sentences and the testing one of 2759 sentences, all of them 30 words long because of the padding done earlier.
Then, the labels are created in order to pass them to the Neural Network.

In [184]:
train = np.concatenate((fillon_numpy,lepen_numpy,macron_numpy,melenchon_numpy,hamon_numpy, jadot_numpy), axis=0)
test = np.concatenate((fillon_test_numpy, lepen_test_numpy,macron_test_numpy,melenchon_test_numpy), axis=0)

In [185]:
print(train.shape)
print(test.shape)

(6951, 30)
(2759, 30)


In [186]:
w, h = 6, train.shape[0];
labelTrain1 = [[0 for x in range(w)] for y in range(len(fillon_numpy))]
for x in range(len(fillon_numpy)):
    labelTrain1[x][0] = 1
labelTrain2 = [[0 for x in range(w)] for y in range(len(lepen_numpy))]
for x in range(len(lepen_numpy)):
    labelTrain2[x][1] = 2
labelTrain3 = [[0 for x in range(w)] for y in range(len(macron_numpy))]
for x in range(len(macron_numpy)):
    labelTrain3[x][2] = 3
labelTrain4 = [[0 for x in range(w)] for y in range(len(melenchon_numpy))]
for x in range(len(melenchon_numpy)):
    labelTrain4[x][3] = 4
labelTrain5 = [[0 for x in range(w)] for y in range(len(hamon_numpy))]
for x in range(len(hamon_numpy)):
    labelTrain5[x][4] = 5
labelTrain6 = [[0 for x in range(w)] for y in range(len(jadot_numpy))]
for x in range(len(jadot_numpy)):
    labelTrain6[x][5] = 6
labelTrain = np.concatenate((labelTrain1, labelTrain2, labelTrain3,labelTrain4,labelTrain5,labelTrain6), axis=0)

In [187]:
w, h = 6, test.shape[0];
labelTest1 = [[0 for x in range(w)] for y in range(len(fillon_test_numpy))]
for x in range(len(fillon_test_numpy)):
    labelTest1[x][0] = 1
labelTest2 = [[0 for x in range(w)] for y in range(len(lepen_test_numpy))]
for x in range(len(lepen_test_numpy)):
    labelTest2[x][1] = 2
labelTest3 = [[0 for x in range(w)] for y in range(len(macron_test_numpy))]
for x in range(len(macron_test_numpy)):
    labelTest3[x][2] = 3
labelTest4 = [[0 for x in range(w)] for y in range(len(melenchon_test_numpy))]
for x in range(len(melenchon_test_numpy)):
    labelTest4[x][3] = 4
labelTest = np.concatenate((labelTest1, labelTest2, labelTest3,labelTest4), axis=0)

In [188]:
print(train.shape)
print(test.shape)
print(labelTrain.shape)
print(labelTest.shape)

(6951, 30)
(2759, 30)
(6951, 6)
(2759, 6)


# Convolutional Neural Network

Here they are specified the different parameters of the Convolutional Neural Network. They are following the recommendations of the mentioned paper, but there are some modifications in order to adapt the result to another project that was being done parallely.

In [189]:
dropout_prob = (0.5, 0.8)

In [190]:
input_shape = (32,)
model_input = Input(shape=input_shape)

In [191]:
conv_blocks = []
z = Embedding(len(vocabulary_inv), 300, input_length=32, name="embedding")(model_input)
z = Dropout(dropout_prob[0])(z)

In [192]:
print(type(z))
conv = Convolution1D(filters=300,
                     kernel_size=3,
                     padding="valid",
                     activation="relu",
                     strides=1)(z)
conv = MaxPooling1D(pool_size=2)(conv)
conv = Flatten()(conv)
conv_blocks.append(conv)

conv = Convolution1D(filters=300,
                     kernel_size=3,
                     padding="valid",
                     activation="relu",
                     strides=1)(z)
conv = MaxPooling1D(pool_size=2)(conv)
conv = Flatten()(conv)
conv_blocks.append(conv)

z = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]

z = Dropout(dropout_prob[1])(z)
z = Dense(50, activation="relu")(z)
model_output = Dense(6, activation="sigmoid")(z)

model = Model(model_input, model_output)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

<class 'tensorflow.python.framework.ops.Tensor'>


In [193]:
embedding_layer = model.get_layer("embedding")
embedding_layer.set_weights(embedding_weights)

In [194]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 32, 300)      2253000     input_2[0][0]                    
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 32, 300)      0           embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 30, 300)      270300      dropout_3[0][0]                  
__________________________________________________________________________________________________
conv1d_4 (

# Train model

In [197]:
model.fit(newTrain, labelTrain, shuffle=True,batch_size=64, epochs=10, verbose=1)

Epoch 1/10
6951/6951 [==============================] - 17s 2ms/step - loss: -3.3810 - acc: 0.6686
Epoch 2/10
6951/6951 [==============================] - 17s 2ms/step - loss: -3.8149 - acc: 0.6961
Epoch 3/10
6951/6951 [==============================] - 18s 3ms/step - loss: -4.0217 - acc: 0.7083
Epoch 4/10
6951/6951 [==============================] - 17s 2ms/step - loss: -4.1501 - acc: 0.7169
Epoch 5/10
6951/6951 [==============================] - 18s 3ms/step - loss: -4.2391 - acc: 0.7214
Epoch 6/10
6951/6951 [==============================] - 18s 3ms/step - loss: -4.3259 - acc: 0.7239
Epoch 7/10
6951/6951 [==============================] - 18s 3ms/step - loss: -4.3970 - acc: 0.7270
Epoch 8/10
6951/6951 [==============================] - 17s 3ms/step - loss: -4.4599 - acc: 0.7291
Epoch 9/10
6951/6951 [==============================] - 18s 3ms/step - loss: -4.5120 - acc: 0.7329
Epoch 10/10
6951/6951 [==============================] - 18s 3ms/step - loss: -4.5484 - acc: 0.7320


In [198]:
scores = model.evaluate(newTest,labelTest)
print()
print(scores[1]*100)

2759/2759 [==============================] - 2s 638us/step

73.1968107497


# Saving data into files

Data is saved into files so the project can be executed again without taking so long.

In [76]:
import numpy
numpy.savetxt("embedding_weightsFrance.csv", embedding_weights[0], delimiter=",")
numpy.savetxt("newTrainFrance.csv", newTrain, delimiter=",")
numpy.savetxt("labelTrainFrance.csv", labelTrain, delimiter=",")
numpy.savetxt("newTestFrance.csv", newTest, delimiter=",")
numpy.savetxt("labelTestFrance.csv", labelTest, delimiter=",")
numpy.savetxt("trainFrance.csv", train, delimiter=",")
numpy.savetxt("testFrance.csv", test, delimiter=",")

In [77]:
aux_vocabulary_inv = []
for x in range(len(vocabulary_inv)):
    aux_vocabulary_inv.append(vocabulary_inv[x])

In [78]:
import pickle

with open('vocabulary_invFrance', 'wb') as fp:
    pickle.dump(vocabulary_inv, fp)